In [1]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [2]:
import tensorflow as tf
import numpy as np


class BeamSearch():
    def __init__(self, predict, initial_state, prime_labels):
        """Initializes the beam search.

        Args:
            predict:
                A function that takes a `sample` and a `state`. It then performs
                the computation on the last word in `sample`.
            initial_state:
                The initial state of the RNN.
            prime_labels:
                A list of labels corresponding to the priming text. This must
                not be empty.
        """

        if not prime_labels:
            raise ValueError('prime_labels must be a non-empty list.')
        self.predict = predict
        self.initial_state = initial_state
        self.prime_labels = prime_labels

    def predict_samples(self, samples, states):
        probs = []
        next_states = []
        for i in range(len(samples)):
            prob, next_state = self.predict(samples[i], states[i])
            probs.append(prob.squeeze())
            next_states.append(next_state)
        return np.array(probs), next_states

    def search(self, oov, eos, k=1, maxsample=4000, use_unk=False):
        """Return k samples (beams) and their NLL scores.

        Each sample is a sequence of labels, either ending with `eos` or
        truncated to length of `maxsample`. `use_unk` allow usage of `oov`
        (out-of-vocabulary) label in samples
        """

        # A list of probabilities of our samples.
        probs = []

        prime_sample = []
        prime_score = 0
        prime_state = self.initial_state

        # Initialize the live sample with the prime.
        for i, label in enumerate(self.prime_labels):
            prime_sample.append(label)

            # The first word does not contribute to the score as the probs have
            # not yet been determined.
            if i > 0:
                prime_score = prime_score - np.log(probs[0, label])
            probs, prime_state = self.predict(prime_sample, prime_state)

        dead_k = 0  # samples that reached eos
        dead_samples = []
        dead_scores = []
        dead_states = []

        live_k = 1  # samples that did not yet reached eos
        live_samples = [prime_sample]
        live_scores = [prime_score]
        live_states = [prime_state]

        while live_k and dead_k < k:
            # total score for every sample is sum of -log of word prb
            cand_scores = np.array(live_scores)[:, None] - np.log(probs)
            if not use_unk and oov is not None:
                cand_scores[:, oov] = 1e20
            cand_flat = cand_scores.flatten()

            # find the best (lowest) scores we have from all possible samples and new words
            ranks_flat = cand_flat.argsort()[:(k - dead_k)]
            live_scores = cand_flat[ranks_flat]

            # append the new words to their appropriate live sample
            voc_size = probs.shape[1]
            live_samples = [live_samples[r // voc_size] + [r % voc_size] for r in ranks_flat]
            live_states = [live_states[r // voc_size] for r in ranks_flat]

            # live samples that should be dead are...
            zombie = [s[-1] == eos or len(s) >= maxsample for s in live_samples]

            # add zombies to the dead
            dead_samples += [s for s, z in zip(live_samples, zombie) if z]  # remove first label == empty
            dead_scores += [s for s, z in zip(live_scores, zombie) if z]
            dead_states += [s for s, z in zip(live_states, zombie) if z]
            dead_k = len(dead_samples)
            # remove zombies from the living
            live_samples = [s for s, z in zip(live_samples, zombie) if not z]
            live_scores = [s for s, z in zip(live_scores, zombie) if not z]
            live_states = [s for s, z in zip(live_states, zombie) if not z]
            live_k = len(live_samples)

            # Finally, compute the next-step probabilities and states.
            probs, live_states = self.predict_samples(live_samples, live_states)

        return dead_samples + live_samples, dead_scores + live_scores


In [3]:
# -*- coding: utf-8 -*-
import os
import codecs
import collections
from six.moves import cPickle
import numpy as np
import re
import itertools

class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding=None):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        # Let's not read voca and data from file. We many change them.
        if True or not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file, encoding)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def clean_str(self, string):
        """
        Tokenization/string cleaning for all datasets except for SST.
        Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data
        """
        string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)
        string = re.sub(r"\'s", " \'s", string)
        string = re.sub(r"\'ve", " \'ve", string)
        string = re.sub(r"n\'t", " n\'t", string)
        string = re.sub(r"\'re", " \'re", string)
        string = re.sub(r"\'d", " \'d", string)
        string = re.sub(r"\'ll", " \'ll", string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\?", " \? ", string)
        string = re.sub(r"\s{2,}", " ", string)
        return string.strip().lower()

    def build_vocab(self, sentences):
        """
        Builds a vocabulary mapping from word to index based on the sentences.
        Returns vocabulary mapping and inverse vocabulary mapping.
        """
        # Build vocabulary
        word_counts = collections.Counter(sentences)
        # Mapping from index to word
        vocabulary_inv = [x[0] for x in word_counts.most_common()]
        vocabulary_inv = list(sorted(vocabulary_inv))
        # Mapping from word to index
        vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
        return [vocabulary, vocabulary_inv]

    def preprocess(self, input_file, vocab_file, tensor_file, encoding):
        with codecs.open(input_file, "r", encoding=encoding) as f:
            data = f.read()

        # Optional text cleaning or make them lower case, etc.
        #data = self.clean_str(data)
        x_text = data.split()

        self.vocab, self.words = self.build_vocab(x_text)
        self.vocab_size = len(self.words)

        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.words, f)

        #The same operation like this [self.vocab[word] for word in x_text]
        # index of words as our basic data
        self.tensor = np.array(list(map(self.vocab.get, x_text)))
        # Save the data to data.npy
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.words = cPickle.load(f)
        self.vocab_size = len(self.words)
        self.vocab = dict(zip(self.words, range(len(self.words))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)

        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0


<>:44: SyntaxWarning: invalid escape sequence '\('
<>:45: SyntaxWarning: invalid escape sequence '\)'
<>:46: SyntaxWarning: invalid escape sequence '\?'
<>:44: SyntaxWarning: invalid escape sequence '\('
<>:45: SyntaxWarning: invalid escape sequence '\)'
<>:46: SyntaxWarning: invalid escape sequence '\?'
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/3828473892.py:44: SyntaxWarning: invalid escape sequence '\('
  string = re.sub(r"\(", " \( ", string)
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/3828473892.py:45: SyntaxWarning: invalid escape sequence '\)'
  string = re.sub(r"\)", " \) ", string)
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/3828473892.py:46: SyntaxWarning: invalid escape sequence '\?'
  string = re.sub(r"\?", " \? ", string)


In [17]:
import tensorflow as tf
import numpy as np
import argparse
import time
import os
import pickle
from tqdm import tqdm

# Define the Model class
class Model(tf.keras.Model):
    def __init__(self, args):
        super(Model, self).__init__()

        # Define RNN layers using tf.keras layers
        if args.model == 'lstm':
            self.rnn_layer = tf.keras.layers.LSTM(args.rnn_size, return_state=True, return_sequences=True)
        elif args.model == 'gru':
            self.rnn_layer = tf.keras.layers.GRU(args.rnn_size, return_state=True, return_sequences=True)
        else:
            raise ValueError("Unsupported model type")

        # Dense layer for output prediction
        self.dense_layer = tf.keras.layers.Dense(args.vocab_size)

    def call(self, inputs, state=None):
        # Pass the input through the RNN layer
        # output, state_h, state_c = self.rnn_layer(inputs, initial_state=state)
        # logits = self.dense_layer(output)
        # return logits, (state_h, state_c)  # Return the state for LSTM (state_h, state_c)
        inputs = tf.expand_dims(inputs, axis=-1)  
        
        # Pass the input through the RNN layer
        output, state_h, state_c = self.rnn_layer(inputs, initial_state=state)
        logits = self.dense_layer(output)
        return logits, (state_h, state_c) 

    def loss_function(self, y_true, y_pred):
        return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

    def reset_states(self, batch_size):
        # Reset the hidden state of LSTM or GRU cells
        # return [tf.zeros((batch_size, self.rnn_layer.units)) for _ in range(self.rnn_layer._num_constants)]
        return (tf.zeros((batch_size, self.rnn_layer.units)),  # Hidden state (h)
                tf.zeros((batch_size, self.rnn_layer.units)))  # Cell state (c)

# Define the TextLoader class
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding=None):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        # Let's not read voca and data from file. We may change them.
        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file, encoding)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def clean_str(self, string):
        """Tokenization/string cleaning"""
        string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
        string = re.sub(r"\'s", " \'s", string)
        string = re.sub(r"\'ve", " \'ve", string)
        string = re.sub(r"n\'t", " n\'t", string)
        string = re.sub(r"\'re", " \'re", string)
        string = re.sub(r"\'d", " \'d", string)
        string = re.sub(r"\'ll", " \'ll", string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\?", " \? ", string)
        string = re.sub(r"\s{2,}", " ", string)
        return string.strip().lower()

    def build_vocab(self, sentences):
        """Build vocabulary mapping from word to index"""
        word_counts = collections.Counter(sentences)
        vocabulary_inv = [x[0] for x in word_counts.most_common()]
        vocabulary_inv = list(sorted(vocabulary_inv))
        vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
        return [vocabulary, vocabulary_inv]

    def preprocess(self, input_file, vocab_file, tensor_file, encoding):
        with codecs.open(input_file, "r", encoding=encoding) as f:
            data = f.read()

        x_text = data.split()

        self.vocab, self.words = self.build_vocab(x_text)
        self.vocab_size = len(self.words)

        with open(vocab_file, 'wb') as f:
            pickle.dump(self.words, f)

        # Convert words to indexes
        self.tensor = np.array(list(map(self.vocab.get, x_text)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.words = pickle.load(f)
        self.vocab_size = len(self.words)
        self.vocab = dict(zip(self.words, range(len(self.words))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))
        if self.num_batches == 0:
            raise ValueError("Not enough data. Reduce seq_length and batch_size.")

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)

        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]

        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

# # Training function
# def train(args):
#     data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
#     args.vocab_size = data_loader.vocab_size

#     if args.init_from:
#         with open(os.path.join(args.init_from, 'config.pkl'), 'rb') as f:
#             saved_model_args = pickle.load(f)
#         for param in ["model", "rnn_size", "num_layers", "seq_length"]:
#             assert vars(saved_model_args)[param] == vars(args)[param], f"Mismatch in {param}"

#     os.makedirs(args.save_dir, exist_ok=True)

#     with open(os.path.join(args.save_dir, 'config.pkl'), 'wb') as f:
#         pickle.dump(args, f)
#     with open(os.path.join(args.save_dir, 'words_vocab.pkl'), 'wb') as f:
#         pickle.dump((data_loader.words, data_loader.vocab), f)

#     model = Model(args)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)

#     summary_writer = tf.summary.create_file_writer(args.log_dir)

#     for epoch in range(args.num_epochs):
#         data_loader.reset_batch_pointer()

#         # Initialize the state for the first batch
#         state = model.reset_states(args.batch_size)

#         for batch in range(data_loader.num_batches):
#             start = time.time()
#             x_batch, y_batch = data_loader.next_batch()

#             with tf.GradientTape() as tape:
#                 logits, state = model(x_batch, state)  # Pass the current state to the model
#                 loss = model.loss_function(y_batch, logits)

#             gradients = tape.gradient(loss, model.trainable_variables)
#             gradients, _ = tf.clip_by_global_norm(gradients, args.grad_clip)
#             optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#             speed = time.time() - start

#             with summary_writer.as_default():
#                 tf.summary.scalar("Loss", loss, step=epoch * data_loader.num_batches + batch)
#                 tf.summary.scalar("Time per batch", speed, step=epoch * data_loader.num_batches + batch)

#             if batch % args.save_every == 0 or (epoch == args.num_epochs - 1 and batch == data_loader.num_batches - 1):
#                 model.save_weights(os.path.join(args.save_dir, 'model.weights.h5'))
#                 print(f"Model saved at epoch {epoch}, batch {batch}")

# Training function
def train(args):
    data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
    args.vocab_size = data_loader.vocab_size

    # if args.init_from:
    #     with open(os.path.join(args.init_from, 'config.pkl'), 'rb') as f:
    #         saved_model_args = pickle.load(f)
    #     for param in ["model", "rnn_size", "num_layers", "seq_length"]:
    #         assert vars(saved_model_args)[param] == vars(args)[param], f"Mismatch in {param}"

    # os.makedirs(args.save_dir, exist_ok=True)

    # with open(os.path.join(args.save_dir, 'config.pkl'), 'wb') as f:
    #     pickle.dump(args, f)
    # with open(os.path.join(args.save_dir, 'words_vocab.pkl'), 'wb') as f:
    #     pickle.dump((data_loader.words, data_loader.vocab), f)

    # model = Model(args)
    if args.init_from:
        # Check if the model weights exist in the provided directory
        checkpoint_path = os.path.join(args.init_from, 'model.weights.h5')
        
        if os.path.exists(checkpoint_path):
            print(f"Loading model weights from {checkpoint_path}")
            model = Model(args)
            model.load_weights(checkpoint_path)  # Load the model weights
            print("Model weights loaded successfully.")
        else:
            raise ValueError(f"Checkpoint not found: {checkpoint_path}")
    else:
        # If no saved model, initialize a new one
        print("Initializing a new model.")
        model = Model(args)
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)

    summary_writer = tf.summary.create_file_writer(args.log_dir)

    for epoch in range(args.num_epochs):
        data_loader.reset_batch_pointer()

        # Initialize the state for the first batch
        state = model.reset_states(args.batch_size)

        # Add tqdm progress bar for batches
        for batch in tqdm(range(data_loader.num_batches), desc=f'Epoch {epoch+1}/{args.num_epochs}'):
            start = time.time()
            x_batch, y_batch = data_loader.next_batch()

            with tf.GradientTape() as tape:
                logits, state = model(x_batch, state)  # Pass the current state to the model
                loss = model.loss_function(y_batch, logits)

            gradients = tape.gradient(loss, model.trainable_variables)
            gradients, _ = tf.clip_by_global_norm(gradients, args.grad_clip)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            speed = time.time() - start

            with summary_writer.as_default():
                tf.summary.scalar("Loss", loss, step=epoch * data_loader.num_batches + batch)
                tf.summary.scalar("Time per batch", speed, step=epoch * data_loader.num_batches + batch)

            # Print loss and speed per batch
            if batch % 100 == 0:  # Print progress every 100 batches
                print(f"Epoch {epoch+1}, Batch {batch}/{data_loader.num_batches}, Loss: {loss:.4f}, Speed: {speed:.4f}s/batch")

            if batch % args.save_every == 0 or (epoch == args.num_epochs - 1 and batch == data_loader.num_batches - 1):
                model.save_weights(os.path.join(args.save_dir, 'model.weights.h5'))
                print(f"Model saved at epoch {epoch}, batch {batch}")


# # Main function
# def main():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--data_dir', type=str, default='data/corpus', help='data directory containing input.txt')
#     parser.add_argument('--log_dir', type=str, default='logs', help='directory containing tensorboard logs')
#     parser.add_argument('--save_dir', type=str, default='save', help='directory to store checkpointed models')
#     parser.add_argument('--rnn_size', type=int, default=256, help='size of RNN hidden state')
#     parser.add_argument('--num_layers', type=int, default=2, help='number of layers in the RNN')
#     parser.add_argument('--model', type=str, default='lstm', help='rnn, gru, or lstm')
#     parser.add_argument('--batch_size', type=int, default=50, help='minibatch size')
#     parser.add_argument('--seq_length', type=int, default=25, help='RNN sequence length')
#     parser.add_argument('--num_epochs', type=int, default=50, help='number of epochs')
#     parser.add_argument('--save_every', type=int, default=1000, help='save frequency')
#     parser.add_argument('--grad_clip', type=float, default=5.0, help='clip gradients at this value')
#     parser.add_argument('--learning_rate', type=float, default=0.002, help='learning rate')
#     parser.add_argument('--decay_rate', type=float, default=0.97, help='decay rate')
#     parser.add_argument('--init_from', type=str, default='save', help='continue training from a saved model')

#     args, unknown = parser.parse_known_args()

#     train(args)

# if __name__ == '__main__':
#     main()


<>:78: SyntaxWarning: invalid escape sequence '\('
<>:79: SyntaxWarning: invalid escape sequence '\)'
<>:80: SyntaxWarning: invalid escape sequence '\?'
<>:78: SyntaxWarning: invalid escape sequence '\('
<>:79: SyntaxWarning: invalid escape sequence '\)'
<>:80: SyntaxWarning: invalid escape sequence '\?'
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/986553125.py:78: SyntaxWarning: invalid escape sequence '\('
  string = re.sub(r"\(", " \( ", string)
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/986553125.py:79: SyntaxWarning: invalid escape sequence '\)'
  string = re.sub(r"\)", " \) ", string)
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/986553125.py:80: SyntaxWarning: invalid escape sequence '\?'
  string = re.sub(r"\?", " \? ", string)


In [51]:
import tensorflow as tf
import numpy as np
import argparse
import time
import os
import pickle
from tqdm import tqdm

# Define the Model class
class Model(tf.keras.Model):
    def __init__(self, args):
        super(Model, self).__init__()

        # Define RNN layers using tf.keras layers
        if args.model == 'lstm':
            self.rnn_layer = tf.keras.layers.LSTM(args.rnn_size, return_state=True, return_sequences=True)
        elif args.model == 'gru':
            self.rnn_layer = tf.keras.layers.GRU(args.rnn_size, return_state=True, return_sequences=True)
        else:
            raise ValueError("Unsupported model type")

        # Dense layer for output prediction
        self.dense_layer = tf.keras.layers.Dense(args.vocab_size)

    def call(self, inputs, state=None):
        # Pass the input through the RNN layer
        # output, state_h, state_c = self.rnn_layer(inputs, initial_state=state)
        # logits = self.dense_layer(output)
        # return logits, (state_h, state_c)  # Return the state for LSTM (state_h, state_c)
        inputs = tf.expand_dims(inputs, axis=-1)

        # Pass the input through the RNN layer
        output, state_h, state_c = self.rnn_layer(inputs, initial_state=state)
        logits = self.dense_layer(output)
        return logits, (state_h, state_c)

    def loss_function(self, y_true, y_pred):
        return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

    def reset_states(self, batch_size):
        # Reset the hidden state of LSTM or GRU cells
        # return [tf.zeros((batch_size, self.rnn_layer.units)) for _ in range(self.rnn_layer._num_constants)]
        return (tf.zeros((batch_size, self.rnn_layer.units)),  # Hidden state (h)
                tf.zeros((batch_size, self.rnn_layer.units)))  # Cell state (c)

# Define the TextLoader class
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding=None):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        # Let's not read voca and data from file. We may change them.
        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file, encoding)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def clean_str(self, string):
        """Tokenization/string cleaning"""
        string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
        string = re.sub(r"\'s", " \'s", string)
        string = re.sub(r"\'ve", " \'ve", string)
        string = re.sub(r"n\'t", " n\'t", string)
        string = re.sub(r"\'re", " \'re", string)
        string = re.sub(r"\'d", " \'d", string)
        string = re.sub(r"\'ll", " \'ll", string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\?", " \? ", string)
        string = re.sub(r"\s{2,}", " ", string)
        return string.strip().lower()

    def build_vocab(self, sentences):
        """Build vocabulary mapping from word to index"""
        word_counts = collections.Counter(sentences)
        vocabulary_inv = [x[0] for x in word_counts.most_common()]
        vocabulary_inv = list(sorted(vocabulary_inv))
        vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
        return [vocabulary, vocabulary_inv]

    def preprocess(self, input_file, vocab_file, tensor_file, encoding):
        with codecs.open(input_file, "r", encoding=encoding) as f:
            data = f.read()

        x_text = data.split()

        self.vocab, self.words = self.build_vocab(x_text)
        self.vocab_size = len(self.words)

        with open(vocab_file, 'wb') as f:
            pickle.dump(self.words, f)

        # Convert words to indexes
        self.tensor = np.array(list(map(self.vocab.get, x_text)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.words = pickle.load(f)
        self.vocab_size = len(self.words)
        self.vocab = dict(zip(self.words, range(len(self.words))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))
        if self.num_batches == 0:
            raise ValueError("Not enough data. Reduce seq_length and batch_size.")

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)

        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]

        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

# # Training function
# def train(args):
#     data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
#     args.vocab_size = data_loader.vocab_size

#     if args.init_from:
#         with open(os.path.join(args.init_from, 'config.pkl'), 'rb') as f:
#             saved_model_args = pickle.load(f)
#         for param in ["model", "rnn_size", "num_layers", "seq_length"]:
#             assert vars(saved_model_args)[param] == vars(args)[param], f"Mismatch in {param}"

#     os.makedirs(args.save_dir, exist_ok=True)

#     with open(os.path.join(args.save_dir, 'config.pkl'), 'wb') as f:
#         pickle.dump(args, f)
#     with open(os.path.join(args.save_dir, 'words_vocab.pkl'), 'wb') as f:
#         pickle.dump((data_loader.words, data_loader.vocab), f)

#     model = Model(args)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)

#     summary_writer = tf.summary.create_file_writer(args.log_dir)

#     for epoch in range(args.num_epochs):
#         data_loader.reset_batch_pointer()

#         # Initialize the state for the first batch
#         state = model.reset_states(args.batch_size)

#         for batch in range(data_loader.num_batches):
#             start = time.time()
#             x_batch, y_batch = data_loader.next_batch()

#             with tf.GradientTape() as tape:
#                 logits, state = model(x_batch, state)  # Pass the current state to the model
#                 loss = model.loss_function(y_batch, logits)

#             gradients = tape.gradient(loss, model.trainable_variables)
#             gradients, _ = tf.clip_by_global_norm(gradients, args.grad_clip)
#             optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#             speed = time.time() - start

#             with summary_writer.as_default():
#                 tf.summary.scalar("Loss", loss, step=epoch * data_loader.num_batches + batch)
#                 tf.summary.scalar("Time per batch", speed, step=epoch * data_loader.num_batches + batch)

#             if batch % args.save_every == 0 or (epoch == args.num_epochs - 1 and batch == data_loader.num_batches - 1):
#                 model.save_weights(os.path.join(args.save_dir, 'model.weights.h5'))
#                 print(f"Model saved at epoch {epoch}, batch {batch}")

# Training function
def train(args):
    data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
    args.vocab_size = data_loader.vocab_size

    if args.init_from:
        with open(os.path.join(args.init_from, 'config.pkl'), 'rb') as f:
            saved_model_args = pickle.load(f)
        for param in ["model", "rnn_size", "num_layers", "seq_length"]:
            assert vars(saved_model_args)[param] == vars(args)[param], f"Mismatch in {param}"

    os.makedirs(args.save_dir, exist_ok=True)

    with open(os.path.join(args.save_dir, 'config.pkl'), 'wb') as f:
        pickle.dump(args, f)
    with open(os.path.join(args.save_dir, 'words_vocab.pkl'), 'wb') as f:
        pickle.dump((data_loader.words, data_loader.vocab), f)

    model = Model(args)
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)

    summary_writer = tf.summary.create_file_writer(args.log_dir)

    for epoch in range(args.num_epochs):
        data_loader.reset_batch_pointer()

        # Initialize the state for the first batch
        state = model.reset_states(args.batch_size)

        # Add tqdm progress bar for batches
        for batch in tqdm(range(data_loader.num_batches), desc=f'Epoch {epoch+1}/{args.num_epochs}'):
            start = time.time()
            x_batch, y_batch = data_loader.next_batch()

            with tf.GradientTape() as tape:
                logits, state = model(x_batch, state)  # Pass the current state to the model
                loss = model.loss_function(y_batch, logits)

            gradients = tape.gradient(loss, model.trainable_variables)
            gradients, _ = tf.clip_by_global_norm(gradients, args.grad_clip)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            speed = time.time() - start

            with summary_writer.as_default():
                tf.summary.scalar("Loss", loss, step=epoch * data_loader.num_batches + batch)
                tf.summary.scalar("Time per batch", speed, step=epoch * data_loader.num_batches + batch)

            # Print loss and speed per batch
            if batch % 100 == 0:  # Print progress every 100 batches
                print(f"Epoch {epoch+1}, Batch {batch}/{data_loader.num_batches}, Loss: {loss:.4f}, Speed: {speed:.4f}s/batch")

            if batch % args.save_every == 0 or (epoch == args.num_epochs - 1 and batch == data_loader.num_batches - 1):
                model.save_weights(os.path.join(args.save_dir, 'model.weights.h5'))
                print(f"Model saved at epoch {epoch}, batch {batch}")


# Main function
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='data/corpus', help='data directory containing input.txt')
    parser.add_argument('--log_dir', type=str, default='logs', help='directory containing tensorboard logs')
    parser.add_argument('--save_dir', type=str, default='save', help='directory to store checkpointed models')
    parser.add_argument('--rnn_size', type=int, default=256, help='size of RNN hidden state')
    parser.add_argument('--num_layers', type=int, default=2, help='number of layers in the RNN')
    parser.add_argument('--model', type=str, default='lstm', help='rnn, gru, or lstm')
    parser.add_argument('--batch_size', type=int, default=32, help='minibatch size')
    parser.add_argument('--seq_length', type=int, default=50, help='RNN sequence length')
    parser.add_argument('--num_epochs', type=int, default=50, help='number of epochs')
    parser.add_argument('--save_every', type=int, default=1000, help='save frequency')
    parser.add_argument('--grad_clip', type=float, default=2.0, help='clip gradients at this value')
    parser.add_argument('--learning_rate', type=float, default=0.0005, help='learning rate')
    parser.add_argument('--decay_rate', type=float, default=0.99, help='decay rate')
    parser.add_argument('--init_from', type=str, default=None, help='continue training from a saved model')

    args, unknown = parser.parse_known_args()

    train(args)

if __name__ == '__main__':
    main()


<>:78: SyntaxWarning: invalid escape sequence '\('
<>:79: SyntaxWarning: invalid escape sequence '\)'
<>:80: SyntaxWarning: invalid escape sequence '\?'
<>:78: SyntaxWarning: invalid escape sequence '\('
<>:79: SyntaxWarning: invalid escape sequence '\)'
<>:80: SyntaxWarning: invalid escape sequence '\?'
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/3968493471.py:78: SyntaxWarning: invalid escape sequence '\('
  string = re.sub(r"\(", " \( ", string)
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/3968493471.py:79: SyntaxWarning: invalid escape sequence '\)'
  string = re.sub(r"\)", " \) ", string)
/var/folders/yr/w6tkd3sj0hb08ccdhq_y6k_r0000gn/T/ipykernel_91548/3968493471.py:80: SyntaxWarning: invalid escape sequence '\?'
  string = re.sub(r"\?", " \? ", string)


loading preprocessed files


Epoch 1/50:   0%|                                           | 1/662 [00:02<25:42,  2.33s/it]

Epoch 1, Batch 0/662, Loss: 10.6836, Speed: 2.3062s/batch
Model saved at epoch 0, batch 0


Epoch 1/50:   2%|▉                                         | 14/662 [00:28<22:12,  2.06s/it]


KeyboardInterrupt: 

In [8]:
tf.config.list_physical_devices('GPU')


[]

In [31]:
import numpy as np
import tensorflow as tf
import argparse
import os
import pickle
import sys

def main():
    # Ignore the arguments passed by Jupyter
    sys.argv = sys.argv[:1]
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_dir', type=str, default='save', help='Directory to load checkpointed models from')
    parser.add_argument('-n', type=int, default=100, help='Number of words to sample')
    parser.add_argument('--prime', type=str, default=' ', help='Prime text to begin the generation')
    parser.add_argument('--pick', type=int, default=1, help='1 = weighted pick, 2 = beam search pick')
    parser.add_argument('--width', type=int, default=4, help='Width of the beam search')
    parser.add_argument('--sample', type=int, default=1, help='0 to use max at each timestep, 1 to sample at each timestep, 2 to sample on spaces')
    parser.add_argument('--count', '-c', type=int, default=1, help='Number of samples to print')
    parser.add_argument('--quiet', '-q', default=False, action='store_true', help='Suppress printing the prime text (default false)')
    
    args = parser.parse_args()
    sample(args)

def sample(args):  # Don't parse args again, just use them directly
    # Load saved configuration and vocabulary
    with open('save/config.pkl', 'rb') as f:
        saved_args = pickle.load(f)
    
    with open('save/words_vocab.pkl', 'rb') as f:
        words, vocab = pickle.load(f)
    
    # Initialize the model
    model = Model(saved_args)
    
    # Build the model with the correct input shape
    model.build(input_shape=(None, saved_args.seq_length))  # Pass the correct input shape
    
    # Start a TensorFlow session
    with tf.compat.v1.Session() as sess:
        tf.compat.v1.global_variables_initializer().run()
        
        # Load the model weights
        model.load_weights('save/model.weights.h5')  # Load weights directly
        
        # Generate text using the model
        for _ in range(args.count):
            # Initialize the prime sequence
            prime_sequence = [vocab.get(word, vocab.get('<unk>', 0)) for word in args.prime.split()]
            
            # Ensure the sequence is of the correct length (pad if necessary)
            if len(prime_sequence) < saved_args.seq_length:
                pad_token = vocab.get('<pad>', 0)  # Use zero if <pad> doesn't exist
                prime_sequence = [pad_token] * (saved_args.seq_length - len(prime_sequence)) + prime_sequence
            
            generated = prime_sequence
            
            # Generate words iteratively
            for i in range(args.n):
                input_seq = np.array(generated[-saved_args.seq_length:]).reshape(1, -1)
                predictions = model(input_seq)  # Use model call directly, which works in eager execution
                
                # Get the predicted word index (sampling method can be applied here)
                if args.sample == 1:
                    word_idx = np.random.choice(predictions[0].shape[0], p=predictions[0])
                else:
                    word_idx = np.argmax(predictions[0])

                generated.append(word_idx)
                
                # Stop generation if end token is encountered
                if word_idx == vocab.get('<end>', -1):
                    break
            
            # Convert indices back to words and print the generated text
            generated_words = [words[i] for i in generated]
            output = ' '.join(generated_words)
            if not args.quiet:
                print(output)





if __name__ == '__main__':
    main()


/Users/aryanrastogi/miniforge3/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'model_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


TypeError: len is not well defined for a symbolic Tensor (model_14_1/dense_14_1/Add:0). Please call `x.shape` rather than `len(x)` for shape information.

In [32]:
!cat sample.py


from __future__ import print_function
import numpy as np
import tensorflow as tf

import argparse
import time
import os
from six.moves import cPickle

from .utils import TextLoader
from .model import Model

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_dir', type=str, default='save',
                       help='model directory to load stored checkpointed models from')
    parser.add_argument('-n', type=int, default=100,
                       help='number of words to sample')
    parser.add_argument('--prime', type=str, default=' ',
                       help='prime text')
    parser.add_argument('--pick', type=int, default=1,
                       help='1 = weighted pick, 2 = beam search pick')
    parser.add_argument('--width', type=int, default=4,
                       help='width of the beam search')
    parser.add_argument('--sample', type=int, default=1,
                       help='0 to use max at each timestep, 1 to sample at each timest

In [34]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import argparse
import os
import pickle

# from .model import Model  # Import the Model class

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_dir', type=str, default='save',
                        help='Model directory to load stored checkpointed models from')
    parser.add_argument('-n', type=int, default=100,
                        help='Number of words to sample')
    parser.add_argument('--prime', type=str, default=' ',
                        help='Prime text')
    parser.add_argument('--pick', type=int, default=1,
                        help='1 = weighted pick, 2 = beam search pick')
    parser.add_argument('--width', type=int, default=4,
                        help='Width of the beam search')
    parser.add_argument('--sample', type=int, default=1,
                        help='0 to use max at each timestep, 1 to sample at each timestep, 2 to sample on spaces')
    parser.add_argument('--count', '-c', type=int, default=1,
                        help='Number of samples to print')
    parser.add_argument('--quiet', '-q', default=False, action='store_true',
                        help='Suppress printing the prime text (default false)')

    args = parser.parse_args()
    sample(args)


def sample(args):
    # Load saved vocabulary
    with open('save/words_vocab.pkl', 'rb') as f:
        words, vocab = pickle.load(f)

    # Initialize the model (architecture and configuration is defined in your Model class)
    model = Model()

    # Load the model weights directly from the .h5 file
    model.load_weights('save/model.weights.h5')

    # Generate text using the model
    for _ in range(args.count):
        output = model.sample(words, vocab, args.n, args.prime, args.sample, args.pick, args.width, args.quiet)
        print(output)


if __name__ == '__main__':
    main()


TypeError: Model.__init__() missing 1 required positional argument: 'args'

In [48]:
with open('data/corpus/input.txt', 'w') as f:
    f.write("""The Project Gutenberg eBook of The Complete Works of William Shakespeare
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Complete Works of William Shakespeare

Author: William Shakespeare

Release date: January 1, 1994 [eBook #100]
                Most recently updated: October 29, 2024

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***
The Complete Works of William Shakespeare

by William Shakespeare




                    Contents

    THE SONNETS
    ALL’S WELL THAT ENDS WELL
    THE TRAGEDY OF ANTONY AND CLEOPATRA
    AS YOU LIKE IT
    THE COMEDY OF ERRORS
    THE TRAGEDY OF CORIOLANUS
    CYMBELINE
    THE TRAGEDY OF HAMLET, PRINCE OF DENMARK
    THE FIRST PART OF KING HENRY THE FOURTH
    THE SECOND PART OF KING HENRY THE FOURTH
    THE LIFE OF KING HENRY THE FIFTH
    THE FIRST PART OF HENRY THE SIXTH
    THE SECOND PART OF KING HENRY THE SIXTH
    THE THIRD PART OF KING HENRY THE SIXTH
    KING HENRY THE EIGHTH
    THE LIFE AND DEATH OF KING JOHN
    THE TRAGEDY OF JULIUS CAESAR
    THE TRAGEDY OF KING LEAR
    LOVE’S LABOUR’S LOST
    THE TRAGEDY OF MACBETH
    MEASURE FOR MEASURE
    THE MERCHANT OF VENICE
    THE MERRY WIVES OF WINDSOR
    A MIDSUMMER NIGHT’S DREAM
    MUCH ADO ABOUT NOTHING
    THE TRAGEDY OF OTHELLO, THE MOOR OF VENICE
    PERICLES, PRINCE OF TYRE
    KING RICHARD THE SECOND
    KING RICHARD THE THIRD
    THE TRAGEDY OF ROMEO AND JULIET
    THE TAMING OF THE SHREW
    THE TEMPEST
    THE LIFE OF TIMON OF ATHENS
    THE TRAGEDY OF TITUS ANDRONICUS
    TROILUS AND CRESSIDA
    TWELFTH NIGHT; OR, WHAT YOU WILL
    THE TWO GENTLEMEN OF VERONA
    THE TWO NOBLE KINSMEN
    THE WINTER’S TALE
    A LOVER’S COMPLAINT
    THE PASSIONATE PILGRIM
    THE PHOENIX AND THE TURTLE
    THE RAPE OF LUCRECE
    VENUS AND ADONIS




THE SONNETS

                    1

From fairest creatures we desire increase,
That thereby beauty’s rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed’st thy light’s flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel:
Thou that art now the world’s fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And, tender churl, mak’st waste in niggarding:
  Pity the world, or else this glutton be,
  To eat the world’s due, by the grave and thee.


                    2

When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty’s field,
Thy youth’s proud livery so gazed on now,
Will be a tattered weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv’d thy beauty’s use,
If thou couldst answer ‘This fair child of mine
Shall sum my count, and make my old excuse,’
Proving his beauty by succession thine.
  This were to be new made when thou art old,
  And see thy blood warm when thou feel’st it cold.


                    3

Look in thy glass and tell the face thou viewest,
Now is the time that face should form another,
Whose fresh repair if now thou not renewest,
Thou dost beguile the world, unbless some mother.
For where is she so fair whose uneared womb
Disdains the tillage of thy husbandry?
Or who is he so fond will be the tomb
Of his self-love to stop posterity?
Thou art thy mother’s glass and she in thee
Calls back the lovely April of her prime,
So thou through windows of thine age shalt see,
Despite of wrinkles this thy golden time.
  But if thou live remembered not to be,
  Die single and thine image dies with thee.


                    4

Unthrifty loveliness why dost thou spend,
Upon thyself thy beauty’s legacy?
Nature’s bequest gives nothing but doth lend,
And being frank she lends to those are free:
Then beauteous niggard why dost thou abuse,
The bounteous largess given thee to give?
Profitless usurer why dost thou use
So great a sum of sums yet canst not live?
For having traffic with thyself alone,
Thou of thyself thy sweet self dost deceive,
Then how when nature calls thee to be gone,
What acceptable audit canst thou leave?
  Thy unused beauty must be tombed with thee,
  Which used lives th’ executor to be.


                    5

Those hours that with gentle work did frame
The lovely gaze where every eye doth dwell
Will play the tyrants to the very same,
And that unfair which fairly doth excel:
For never-resting time leads summer on
To hideous winter and confounds him there,
Sap checked with frost and lusty leaves quite gone,
Beauty o’er-snowed and bareness every where:
Then were not summer’s distillation left
A liquid prisoner pent in walls of glass,
Beauty’s effect with beauty were bereft,
Nor it nor no remembrance what it was.
  But flowers distilled though they with winter meet,
  Leese but their show, their substance still lives sweet.


                    6

Then let not winter’s ragged hand deface,
In thee thy summer ere thou be distilled:
Make sweet some vial; treasure thou some place,
With beauty’s treasure ere it be self-killed:
That use is not forbidden usury,
Which happies those that pay the willing loan;
That’s for thyself to breed another thee,
Or ten times happier be it ten for one,
Ten times thyself were happier than thou art,
If ten of thine ten times refigured thee:
Then what could death do if thou shouldst depart,
Leaving thee living in posterity?
  Be not self-willed for thou art much too fair,
  To be death’s conquest and make worms thine heir.


                    7

Lo in the orient when the gracious light
Lifts up his burning head, each under eye
Doth homage to his new-appearing sight,
Serving with looks his sacred majesty,
And having climbed the steep-up heavenly hill,
Resembling strong youth in his middle age,
Yet mortal looks adore his beauty still,
Attending on his golden pilgrimage:
But when from highmost pitch with weary car,
Like feeble age he reeleth from the day,
The eyes (fore duteous) now converted are
From his low tract and look another way:
  So thou, thyself out-going in thy noon:
  Unlooked on diest unless thou get a son.


                    8

Music to hear, why hear’st thou music sadly?
Sweets with sweets war not, joy delights in joy:
Why lov’st thou that which thou receiv’st not gladly,
Or else receiv’st with pleasure thine annoy?
If the true concord of well-tuned sounds,
By unions married do offend thine ear,
They do but sweetly chide thee, who confounds
In singleness the parts that thou shouldst bear:
Mark how one string sweet husband to another,
Strikes each in each by mutual ordering;
Resembling sire, and child, and happy mother,
Who all in one, one pleasing note do sing:
  Whose speechless song being many, seeming one,
  Sings this to thee, ‘Thou single wilt prove none’.


                    9

Is it for fear to wet a widow’s eye,
That thou consum’st thyself in single life?
Ah, if thou issueless shalt hap to die,
The world will wail thee like a makeless wife,
The world will be thy widow and still weep,
That thou no form of thee hast left behind,
When every private widow well may keep,
By children’s eyes, her husband’s shape in mind:
Look what an unthrift in the world doth spend
Shifts but his place, for still the world enjoys it;
But beauty’s waste hath in the world an end,
And kept unused the user so destroys it:
  No love toward others in that bosom sits
  That on himself such murd’rous shame commits.


                    10

For shame deny that thou bear’st love to any
Who for thyself art so unprovident.
Grant if thou wilt, thou art beloved of many,
But that thou none lov’st is most evident:
For thou art so possessed with murd’rous hate,
That ’gainst thyself thou stick’st not to conspire,
Seeking that beauteous roof to ruinate
Which to repair should be thy chief desire:
O change thy thought, that I may change my mind,
Shall hate be fairer lodged than gentle love?
Be as thy presence is gracious and kind,
Or to thyself at least kind-hearted prove,
  Make thee another self for love of me,
  That beauty still may live in thine or thee.


                    11

As fast as thou shalt wane so fast thou grow’st,
In one of thine, from that which thou departest,
And that fresh blood which youngly thou bestow’st,
Thou mayst call thine, when thou from youth convertest,
Herein lives wisdom, beauty, and increase,
Without this folly, age, and cold decay,
If all were minded so, the times should cease,
And threescore year would make the world away:
Let those whom nature hath not made for store,
Harsh, featureless, and rude, barrenly perish:
Look whom she best endowed, she gave thee more;
Which bounteous gift thou shouldst in bounty cherish:
  She carved thee for her seal, and meant thereby,
  Thou shouldst print more, not let that copy die.


                    12

When I do count the clock that tells the time,
And see the brave day sunk in hideous night,
When I behold the violet past prime,
And sable curls all silvered o’er with white:
When lofty trees I see barren of leaves,
Which erst from heat did canopy the herd
And summer’s green all girded up in sheaves
Borne on the bier with white and bristly beard:
Then of thy beauty do I question make
That thou among the wastes of time must go,
Since sweets and beauties do themselves forsake,
And die as fast as they see others grow,
  And nothing ’gainst Time’s scythe can make defence
  Save breed to brave him, when he takes thee hence.


                    13

O that you were your self, but love you are
No longer yours, than you yourself here live,
Against this coming end you should prepare,
And your sweet semblance to some other give.
So should that beauty which you hold in lease
Find no determination, then you were
Yourself again after yourself’s decease,
When your sweet issue your sweet form should bear.
Who lets so fair a house fall to decay,
Which husbandry in honour might uphold,
Against the stormy gusts of winter’s day
And barren rage of death’s eternal cold?
  O none but unthrifts, dear my love you know,
  You had a father, let your son say so.


                    14

Not from the stars do I my judgement pluck,
And yet methinks I have astronomy,
But not to tell of good, or evil luck,
Of plagues, of dearths, or seasons’ quality,
Nor can I fortune to brief minutes tell;
Pointing to each his thunder, rain and wind,
Or say with princes if it shall go well
By oft predict that I in heaven find.
But from thine eyes my knowledge I derive,
And constant stars in them I read such art
As truth and beauty shall together thrive
If from thyself, to store thou wouldst convert:
  Or else of thee this I prognosticate,
  Thy end is truth’s and beauty’s doom and date.


                    15

When I consider everything that grows
Holds in perfection but a little moment.
That this huge stage presenteth nought but shows
Whereon the stars in secret influence comment.
When I perceive that men as plants increase,
Cheered and checked even by the self-same sky:
Vaunt in their youthful sap, at height decrease,
And wear their brave state out of memory.
Then the conceit of this inconstant stay,
Sets you most rich in youth before my sight,
Where wasteful Time debateth with Decay
To change your day of youth to sullied night,
  And all in war with Time for love of you,
  As he takes from you, I engraft you new.


                    16

But wherefore do not you a mightier way
Make war upon this bloody tyrant Time?
And fortify yourself in your decay
With means more blessed than my barren rhyme?
Now stand you on the top of happy hours,
And many maiden gardens yet unset,
With virtuous wish would bear you living flowers,
Much liker than your painted counterfeit:
So should the lines of life that life repair
Which this (Time’s pencil) or my pupil pen
Neither in inward worth nor outward fair
Can make you live yourself in eyes of men.
  To give away yourself, keeps yourself still,
  And you must live drawn by your own sweet skill.


                    17

Who will believe my verse in time to come
If it were filled with your most high deserts?
Though yet heaven knows it is but as a tomb
Which hides your life, and shows not half your parts:
If I could write the beauty of your eyes,
And in fresh numbers number all your graces,
The age to come would say this poet lies,
Such heavenly touches ne’er touched earthly faces.
So should my papers (yellowed with their age)
Be scorned, like old men of less truth than tongue,
And your true rights be termed a poet’s rage,
And stretched metre of an antique song.
  But were some child of yours alive that time,
  You should live twice,—in it, and in my rhyme.


                    18

Shall I compare thee to a summer’s day?
Thou art more lovely and more temperate:
Rough winds do shake the darling buds of May,
And summer’s lease hath all too short a date:
Sometime too hot the eye of heaven shines,
And often is his gold complexion dimmed,
And every fair from fair sometime declines,
By chance, or nature’s changing course untrimmed:
But thy eternal summer shall not fade,
Nor lose possession of that fair thou ow’st,
Nor shall death brag thou wand’rest in his shade,
When in eternal lines to time thou grow’st,
  So long as men can breathe or eyes can see,
  So long lives this, and this gives life to thee.


                    19

Devouring Time blunt thou the lion’s paws,
And make the earth devour her own sweet brood,
Pluck the keen teeth from the fierce tiger’s jaws,
And burn the long-lived phoenix, in her blood,
Make glad and sorry seasons as thou fleet’st,
And do whate’er thou wilt swift-footed Time
To the wide world and all her fading sweets:
But I forbid thee one most heinous crime,
O carve not with thy hours my love’s fair brow,
Nor draw no lines there with thine antique pen,
Him in thy course untainted do allow,
For beauty’s pattern to succeeding men.
  Yet do thy worst, old Time; despite thy wrong,
  My love shall in my verse ever live young.


                    20

A woman’s face with nature’s own hand painted,
Hast thou the master mistress of my passion,
A woman’s gentle heart but not acquainted
With shifting change as is false women’s fashion,
An eye more bright than theirs, less false in rolling:
Gilding the object whereupon it gazeth,
A man in hue all hues in his controlling,
Which steals men’s eyes and women’s souls amazeth.
And for a woman wert thou first created,
Till nature as she wrought thee fell a-doting,
And by addition me of thee defeated,
By adding one thing to my purpose nothing.
  But since she pricked thee out for women’s pleasure,
  Mine be thy love and thy love’s use their treasure.


                    21

So is it not with me as with that muse,
Stirred by a painted beauty to his verse,
Who heaven it self for ornament doth use,
And every fair with his fair doth rehearse,
Making a couplement of proud compare
With sun and moon, with earth and sea’s rich gems:
With April’s first-born flowers and all things rare,
That heaven’s air in this huge rondure hems.
O let me true in love but truly write,
And then believe me, my love is as fair,
As any mother’s child, though not so bright
As those gold candles fixed in heaven’s air:
  Let them say more that like of hearsay well,
  I will not praise that purpose not to sell.


                    22

My glass shall not persuade me I am old,
So long as youth and thou are of one date,
But when in thee time’s furrows I behold,
Then look I death my days should expiate.
For all that beauty that doth cover thee,
Is but the seemly raiment of my heart,
Which in thy breast doth live, as thine in me,
How can I then be elder than thou art?
O therefore love be of thyself so wary,
As I not for my self, but for thee will,
Bearing thy heart which I will keep so chary
As tender nurse her babe from faring ill.
  Presume not on thy heart when mine is slain,
  Thou gav’st me thine not to give back again.


                    23

As an unperfect actor on the stage,
Who with his fear is put beside his part,
Or some fierce thing replete with too much rage,
Whose strength’s abundance weakens his own heart;
So I for fear of trust, forget to say,
The perfect ceremony of love’s rite,
And in mine own love’s strength seem to decay,
O’ercharged with burthen of mine own love’s might:
O let my looks be then the eloquence,
And dumb presagers of my speaking breast,
Who plead for love, and look for recompense,
More than that tongue that more hath more expressed.
  O learn to read what silent love hath writ,
  To hear with eyes belongs to love’s fine wit.


                    24

Mine eye hath played the painter and hath stelled,
Thy beauty’s form in table of my heart,
My body is the frame wherein ’tis held,
And perspective it is best painter’s art.
For through the painter must you see his skill,
To find where your true image pictured lies,
Which in my bosom’s shop is hanging still,
That hath his windows glazed with thine eyes:
Now see what good turns eyes for eyes have done,
Mine eyes have drawn thy shape, and thine for me
Are windows to my breast, where-through the sun
Delights to peep, to gaze therein on thee;
  Yet eyes this cunning want to grace their art,
  They draw but what they see, know not the heart.


                    25

Let those who are in favour with their stars,
Of public honour and proud titles boast,
Whilst I whom fortune of such triumph bars
Unlooked for joy in that I honour most;
Great princes’ favourites their fair leaves spread,
But as the marigold at the sun’s eye,
And in themselves their pride lies buried,
For at a frown they in their glory die.
The painful warrior famoused for fight,
After a thousand victories once foiled,
Is from the book of honour razed quite,
And all the rest forgot for which he toiled:
  Then happy I that love and am beloved
  Where I may not remove nor be removed.


                    26

Lord of my love, to whom in vassalage
Thy merit hath my duty strongly knit;
To thee I send this written embassage
To witness duty, not to show my wit.
Duty so great, which wit so poor as mine
May make seem bare, in wanting words to show it;
But that I hope some good conceit of thine
In thy soul’s thought (all naked) will bestow it:
Till whatsoever star that guides my moving,
Points on me graciously with fair aspect,
And puts apparel on my tattered loving,
To show me worthy of thy sweet respect,
  Then may I dare to boast how I do love thee,
  Till then, not show my head where thou mayst prove me.


                    27

Weary with toil, I haste me to my bed,
The dear respose for limbs with travel tired,
But then begins a journey in my head
To work my mind, when body’s work’s expired.
For then my thoughts, from far where I abide,
Intend a zealous pilgrimage to thee,
And keep my drooping eyelids open wide,
Looking on darkness which the blind do see.
Save that my soul’s imaginary sight
Presents thy shadow to my sightless view,
Which like a jewel (hung in ghastly night)
Makes black night beauteous, and her old face new.
  Lo thus by day my limbs, by night my mind,
  For thee, and for my self, no quiet find.


                    28

How can I then return in happy plight
That am debarred the benefit of rest?
When day’s oppression is not eased by night,
But day by night and night by day oppressed.
And each (though enemies to either’s reign)
Do in consent shake hands to torture me,
The one by toil, the other to complain
How far I toil, still farther off from thee.
I tell the day to please him thou art bright,
And dost him grace when clouds do blot the heaven:
So flatter I the swart-complexioned night,
When sparkling stars twire not thou gild’st the even.
  But day doth daily draw my sorrows longer,
  And night doth nightly make grief’s length seem stronger


                    29

When in disgrace with fortune and men’s eyes,
I all alone beweep my outcast state,
And trouble deaf heaven with my bootless cries,
And look upon my self and curse my fate,
Wishing me like to one more rich in hope,
Featured like him, like him with friends possessed,
Desiring this man’s art, and that man’s scope,
With what I most enjoy contented least,
Yet in these thoughts my self almost despising,
Haply I think on thee, and then my state,
(Like to the lark at break of day arising
From sullen earth) sings hymns at heaven’s gate,
  For thy sweet love remembered such wealth brings,
  That then I scorn to change my state with kings.


                    30

When to the sessions of sweet silent thought,
I summon up remembrance of things past,
I sigh the lack of many a thing I sought,
And with old woes new wail my dear time’s waste:
Then can I drown an eye (unused to flow)
For precious friends hid in death’s dateless night,
And weep afresh love’s long since cancelled woe,
And moan th’ expense of many a vanished sight.
Then can I grieve at grievances foregone,
And heavily from woe to woe tell o’er
The sad account of fore-bemoaned moan,
Which I new pay as if not paid before.
  But if the while I think on thee (dear friend)
  All losses are restored, and sorrows end.


                    31

Thy bosom is endeared with all hearts,
Which I by lacking have supposed dead,
And there reigns love and all love’s loving parts,
And all those friends which I thought buried.
How many a holy and obsequious tear
Hath dear religious love stol’n from mine eye,
As interest of the dead, which now appear,
But things removed that hidden in thee lie.
Thou art the grave where buried love doth live,
Hung with the trophies of my lovers gone,
Who all their parts of me to thee did give,
That due of many, now is thine alone.
  Their images I loved, I view in thee,
  And thou, all they, hast all the all of me.


                    32

If thou survive my well-contented day,
When that churl death my bones with dust shall cover
And shalt by fortune once more re-survey
These poor rude lines of thy deceased lover:
Compare them with the bett’ring of the time,
And though they be outstripped by every pen,
Reserve them for my love, not for their rhyme,
Exceeded by the height of happier men.
O then vouchsafe me but this loving thought,
’Had my friend’s Muse grown with this growing age,
A dearer birth than this his love had brought
To march in ranks of better equipage:
  But since he died and poets better prove,
  Theirs for their style I’ll read, his for his love’.


                    33

Full many a glorious morning have I seen,
Flatter the mountain tops with sovereign eye,
Kissing with golden face the meadows green;
Gilding pale streams with heavenly alchemy:
Anon permit the basest clouds to ride,
With ugly rack on his celestial face,
And from the forlorn world his visage hide
Stealing unseen to west with this disgrace:
Even so my sun one early morn did shine,
With all triumphant splendour on my brow,
But out alack, he was but one hour mine,
The region cloud hath masked him from me now.
  Yet him for this, my love no whit disdaineth,
  Suns of the world may stain, when heaven’s sun staineth.


                    34

Why didst thou promise such a beauteous day,
And make me travel forth without my cloak,
To let base clouds o’ertake me in my way,
Hiding thy brav’ry in their rotten smoke?
’Tis not enough that through the cloud thou break,
To dry the rain on my storm-beaten face,
For no man well of such a salve can speak,
That heals the wound, and cures not the disgrace:
Nor can thy shame give physic to my grief,
Though thou repent, yet I have still the loss,
Th’ offender’s sorrow lends but weak relief
To him that bears the strong offence’s cross.
  Ah but those tears are pearl which thy love sheds,
  And they are rich, and ransom all ill deeds.


                    35

No more be grieved at that which thou hast done,
Roses have thorns, and silver fountains mud,
Clouds and eclipses stain both moon and sun,
And loathsome canker lives in sweetest bud.
All men make faults, and even I in this,
Authorizing thy trespass with compare,
My self corrupting salving thy amiss,
Excusing thy sins more than thy sins are:
For to thy sensual fault I bring in sense;
Thy adverse party is thy advocate,
And ’gainst my self a lawful plea commence:
Such civil war is in my love and hate,
  That I an accessary needs must be,
  To that sweet thief which sourly robs from me.


                    36

Let me confess that we two must be twain,
Although our undivided loves are one:
So shall those blots that do with me remain,
Without thy help, by me be borne alone.
In our two loves there is but one respect,
Though in our lives a separable spite,
Which though it alter not love’s sole effect,
Yet doth it steal sweet hours from love’s delight.
I may not evermore acknowledge thee,
Lest my bewailed guilt should do thee shame,
Nor thou with public kindness honour me,
Unless thou take that honour from thy name:
  But do not so, I love thee in such sort,
  As thou being mine, mine is thy good report.


                    37

As a decrepit father takes delight,
To see his active child do deeds of youth,
So I, made lame by Fortune’s dearest spite
Take all my comfort of thy worth and truth.
For whether beauty, birth, or wealth, or wit,
Or any of these all, or all, or more
Entitled in thy parts, do crowned sit,
I make my love engrafted to this store:
So then I am not lame, poor, nor despised,
Whilst that this shadow doth such substance give,
That I in thy abundance am sufficed,
And by a part of all thy glory live:
  Look what is best, that best I wish in thee,
  This wish I have, then ten times happy me.


                    38

How can my Muse want subject to invent
While thou dost breathe that pour’st into my verse,
Thine own sweet argument, too excellent,
For every vulgar paper to rehearse?
O give thyself the thanks if aught in me,
Worthy perusal stand against thy sight,
For who’s so dumb that cannot write to thee,
When thou thyself dost give invention light?
Be thou the tenth Muse, ten times more in worth
Than those old nine which rhymers invocate,
And he that calls on thee, let him bring forth
Eternal numbers to outlive long date.
  If my slight Muse do please these curious days,
  The pain be mine, but thine shall be the praise.


                    39

O how thy worth with manners may I sing,
When thou art all the better part of me?
What can mine own praise to mine own self bring:
And what is’t but mine own when I praise thee?
Even for this, let us divided live,
And our dear love lose name of single one,
That by this separation I may give:
That due to thee which thou deserv’st alone:
O absence what a torment wouldst thou prove,
Were it not thy sour leisure gave sweet leave,
To entertain the time with thoughts of love,
Which time and thoughts so sweetly doth deceive.
  And that thou teachest how to make one twain,
  By praising him here who doth hence remain.


                    40

Take all my loves, my love, yea take them all,
What hast thou then more than thou hadst before?
No love, my love, that thou mayst true love call,
All mine was thine, before thou hadst this more:
Then if for my love, thou my love receivest,
I cannot blame thee, for my love thou usest,
But yet be blamed, if thou thyself deceivest
By wilful taste of what thyself refusest.
I do forgive thy robbery gentle thief
Although thou steal thee all my poverty:
And yet love knows it is a greater grief
To bear greater wrong, than hate’s known injury.
  Lascivious grace, in whom all ill well shows,
  Kill me with spites yet we must not be foes.


                    41

Those pretty wrongs that liberty commits,
When I am sometime absent from thy heart,
Thy beauty, and thy years full well befits,
For still temptation follows where thou art.
Gentle thou art, and therefore to be won,
Beauteous thou art, therefore to be assailed.
And when a woman woos, what woman’s son,
Will sourly leave her till he have prevailed?
Ay me, but yet thou mightst my seat forbear,
And chide thy beauty, and thy straying youth,
Who lead thee in their riot even there
Where thou art forced to break a twofold truth:
  Hers by thy beauty tempting her to thee,
  Thine by thy beauty being false to me.


                    42

That thou hast her it is not all my grief,
And yet it may be said I loved her dearly,
That she hath thee is of my wailing chief,
A loss in love that touches me more nearly.
Loving offenders thus I will excuse ye,
Thou dost love her, because thou know’st I love her,
And for my sake even so doth she abuse me,
Suff’ring my friend for my sake to approve her.
If I lose thee, my loss is my love’s gain,
And losing her, my friend hath found that loss,
Both find each other, and I lose both twain,
And both for my sake lay on me this cross,
  But here’s the joy, my friend and I are one,
  Sweet flattery, then she loves but me alone.


                    43

When most I wink then do mine eyes best see,
For all the day they view things unrespected,
But when I sleep, in dreams they look on thee,
And darkly bright, are bright in dark directed.
Then thou whose shadow shadows doth make bright
How would thy shadow’s form, form happy show,
To the clear day with thy much clearer light,
When to unseeing eyes thy shade shines so!
How would (I say) mine eyes be blessed made,
By looking on thee in the living day,
When in dead night thy fair imperfect shade,
Through heavy sleep on sightless eyes doth stay!
  All days are nights to see till I see thee,
  And nights bright days when dreams do show thee me.


                    44

If the dull substance of my flesh were thought,
Injurious distance should not stop my way,
For then despite of space I would be brought,
From limits far remote, where thou dost stay,
No matter then although my foot did stand
Upon the farthest earth removed from thee,
For nimble thought can jump both sea and land,
As soon as think the place where he would be.
But ah, thought kills me that I am not thought
To leap large lengths of miles when thou art gone,
But that so much of earth and water wrought,
I must attend, time’s leisure with my moan.
  Receiving nought by elements so slow,
  But heavy tears, badges of either’s woe.


                    45

The other two, slight air, and purging fire,
Are both with thee, wherever I abide,
The first my thought, the other my desire,
These present-absent with swift motion slide.
For when these quicker elements are gone
In tender embassy of love to thee,
My life being made of four, with two alone,
Sinks down to death, oppressed with melancholy.
Until life’s composition be recured,
By those swift messengers returned from thee,
Who even but now come back again assured,
Of thy fair health, recounting it to me.
  This told, I joy, but then no longer glad,
  I send them back again and straight grow sad.


                    46

Mine eye and heart are at a mortal war,
How to divide the conquest of thy sight,
Mine eye, my heart thy picture’s sight would bar,
My heart, mine eye the freedom of that right,
My heart doth plead that thou in him dost lie,
A closet never pierced with crystal eyes;
But the defendant doth that plea deny,
And says in him thy fair appearance lies.
To side this title is impanelled
A quest of thoughts, all tenants to the heart,
And by their verdict is determined
The clear eye’s moiety, and the dear heart’s part.
  As thus, mine eye’s due is thy outward part,
  And my heart’s right, thy inward love of heart.


                    47

Betwixt mine eye and heart a league is took,
And each doth good turns now unto the other,
When that mine eye is famished for a look,
Or heart in love with sighs himself doth smother;
With my love’s picture then my eye doth feast,
And to the painted banquet bids my heart:
Another time mine eye is my heart’s guest,
And in his thoughts of love doth share a part.
So either by thy picture or my love,
Thyself away, art present still with me,
For thou not farther than my thoughts canst move,
And I am still with them, and they with thee.
  Or if they sleep, thy picture in my sight
  Awakes my heart, to heart’s and eye’s delight.


                    48

How careful was I when I took my way,
Each trifle under truest bars to thrust,
That to my use it might unused stay
From hands of falsehood, in sure wards of trust!
But thou, to whom my jewels trifles are,
Most worthy comfort, now my greatest grief,
Thou best of dearest, and mine only care,
Art left the prey of every vulgar thief.
Thee have I not locked up in any chest,
Save where thou art not, though I feel thou art,
Within the gentle closure of my breast,
From whence at pleasure thou mayst come and part,
  And even thence thou wilt be stol’n I fear,
  For truth proves thievish for a prize so dear.


                    49

Against that time (if ever that time come)
When I shall see thee frown on my defects,
When as thy love hath cast his utmost sum,
Called to that audit by advised respects,
Against that time when thou shalt strangely pass,
And scarcely greet me with that sun thine eye,
When love converted from the thing it was
Shall reasons find of settled gravity;
Against that time do I ensconce me here
Within the knowledge of mine own desert,
And this my hand, against my self uprear,
To guard the lawful reasons on thy part,
  To leave poor me, thou hast the strength of laws,
  Since why to love, I can allege no cause.


                    50

How heavy do I journey on the way,
When what I seek (my weary travel’s end)
Doth teach that case and that repose to say
’Thus far the miles are measured from thy friend.’
The beast that bears me, tired with my woe,
Plods dully on, to bear that weight in me,
As if by some instinct the wretch did know
His rider loved not speed being made from thee:
The bloody spur cannot provoke him on,
That sometimes anger thrusts into his hide,
Which heavily he answers with a groan,
More sharp to me than spurring to his side,
  For that same groan doth put this in my mind,
  My grief lies onward and my joy behind.


                    51

Thus can my love excuse the slow offence,
Of my dull bearer, when from thee I speed,
From where thou art, why should I haste me thence?
Till I return of posting is no need.
O what excuse will my poor beast then find,
When swift extremity can seem but slow?
Then should I spur though mounted on the wind,
In winged speed no motion shall I know,
Then can no horse with my desire keep pace,
Therefore desire (of perfect’st love being made)
Shall neigh no dull flesh in his fiery race,
But love, for love, thus shall excuse my jade:
  Since from thee going, he went wilful-slow,
  Towards thee I’ll run, and give him leave to go.


                    52

So am I as the rich whose blessed key,
Can bring him to his sweet up-locked treasure,
The which he will not every hour survey,
For blunting the fine point of seldom pleasure.
Therefore are feasts so solemn and so rare,
Since seldom coming in that long year set,
Like stones of worth they thinly placed are,
Or captain jewels in the carcanet.
So is the time that keeps you as my chest
Or as the wardrobe which the robe doth hide,
To make some special instant special-blest,
By new unfolding his imprisoned pride.
  Blessed are you whose worthiness gives scope,
  Being had to triumph, being lacked to hope.


                    53

What is your substance, whereof are you made,
That millions of strange shadows on you tend?
Since every one, hath every one, one shade,
And you but one, can every shadow lend:
Describe Adonis and the counterfeit,
Is poorly imitated after you,
On Helen’s cheek all art of beauty set,
And you in Grecian tires are painted new:
Speak of the spring, and foison of the year,
The one doth shadow of your beauty show,
The other as your bounty doth appear,
And you in every blessed shape we know.
  In all external grace you have some part,
  But you like none, none you for constant heart.


                    54

O how much more doth beauty beauteous seem,
By that sweet ornament which truth doth give!
The rose looks fair, but fairer we it deem
For that sweet odour, which doth in it live:
The canker blooms have full as deep a dye,
As the perfumed tincture of the roses,
Hang on such thorns, and play as wantonly,
When summer’s breath their masked buds discloses:
But for their virtue only is their show,
They live unwooed, and unrespected fade,
Die to themselves. Sweet roses do not so,
Of their sweet deaths, are sweetest odours made:
  And so of you, beauteous and lovely youth,
  When that shall fade, my verse distills your truth.


                    55

Not marble, nor the gilded monuments
Of princes shall outlive this powerful rhyme,
But you shall shine more bright in these contents
Than unswept stone, besmeared with sluttish time.
When wasteful war shall statues overturn,
And broils root out the work of masonry,
Nor Mars his sword, nor war’s quick fire shall burn:
The living record of your memory.
’Gainst death, and all-oblivious enmity
Shall you pace forth, your praise shall still find room,
Even in the eyes of all posterity
That wear this world out to the ending doom.
  So till the judgement that yourself arise,
  You live in this, and dwell in lovers’ eyes.


                    56

Sweet love renew thy force, be it not said
Thy edge should blunter be than appetite,
Which but to-day by feeding is allayed,
To-morrow sharpened in his former might.
So love be thou, although to-day thou fill
Thy hungry eyes, even till they wink with fulness,
To-morrow see again, and do not kill
The spirit of love, with a perpetual dulness:
Let this sad interim like the ocean be
Which parts the shore, where two contracted new,
Come daily to the banks, that when they see:
Return of love, more blest may be the view.
  Or call it winter, which being full of care,
  Makes summer’s welcome, thrice more wished, more rare.


                    57

Being your slave what should I do but tend,
Upon the hours, and times of your desire?
I have no precious time at all to spend;
Nor services to do till you require.
Nor dare I chide the world-without-end hour,
Whilst I (my sovereign) watch the clock for you,
Nor think the bitterness of absence sour,
When you have bid your servant once adieu.
Nor dare I question with my jealous thought,
Where you may be, or your affairs suppose,
But like a sad slave stay and think of nought
Save where you are, how happy you make those.
  So true a fool is love, that in your will,
  (Though you do any thing) he thinks no ill.


                    58

That god forbid, that made me first your slave,
I should in thought control your times of pleasure,
Or at your hand th’ account of hours to crave,
Being your vassal bound to stay your leisure.
O let me suffer (being at your beck)
Th’ imprisoned absence of your liberty,
And patience tame to sufferance bide each check,
Without accusing you of injury.
Be where you list, your charter is so strong,
That you yourself may privilage your time
To what you will, to you it doth belong,
Yourself to pardon of self-doing crime.
  I am to wait, though waiting so be hell,
  Not blame your pleasure be it ill or well.


                    59

If there be nothing new, but that which is,
Hath been before, how are our brains beguiled,
Which labouring for invention bear amiss
The second burthen of a former child!
O that record could with a backward look,
Even of five hundred courses of the sun,
Show me your image in some antique book,
Since mind at first in character was done.
That I might see what the old world could say,
To this composed wonder of your frame,
Whether we are mended, or whether better they,
Or whether revolution be the same.
  O sure I am the wits of former days,
  To subjects worse have given admiring praise.


                    60

Like as the waves make towards the pebbled shore,
So do our minutes hasten to their end,
Each changing place with that which goes before,
In sequent toil all forwards do contend.
Nativity once in the main of light,
Crawls to maturity, wherewith being crowned,
Crooked eclipses ’gainst his glory fight,
And Time that gave, doth now his gift confound.
Time doth transfix the flourish set on youth,
And delves the parallels in beauty’s brow,
Feeds on the rarities of nature’s truth,
And nothing stands but for his scythe to mow.
  And yet to times in hope, my verse shall stand
  Praising thy worth, despite his cruel hand.


                    61

Is it thy will, thy image should keep open
My heavy eyelids to the weary night?
Dost thou desire my slumbers should be broken,
While shadows like to thee do mock my sight?
Is it thy spirit that thou send’st from thee
So far from home into my deeds to pry,
To find out shames and idle hours in me,
The scope and tenure of thy jealousy?
O no, thy love though much, is not so great,
It is my love that keeps mine eye awake,
Mine own true love that doth my rest defeat,
To play the watchman ever for thy sake.
  For thee watch I, whilst thou dost wake elsewhere,
  From me far off, with others all too near.


                    62

Sin of self-love possesseth all mine eye,
And all my soul, and all my every part;
And for this sin there is no remedy,
It is so grounded inward in my heart.
Methinks no face so gracious is as mine,
No shape so true, no truth of such account,
And for my self mine own worth do define,
As I all other in all worths surmount.
But when my glass shows me my self indeed
beated and chopt with tanned antiquity,
Mine own self-love quite contrary I read:
Self, so self-loving were iniquity.
  ’Tis thee, myself, that for myself I praise,
  Painting my age with beauty of thy days.


                    63

Against my love shall be as I am now
With Time’s injurious hand crushed and o’erworn,
When hours have drained his blood and filled his brow
With lines and wrinkles, when his youthful morn
Hath travelled on to age’s steepy night,
And all those beauties whereof now he’s king
Are vanishing, or vanished out of sight,
Stealing away the treasure of his spring:
For such a time do I now fortify
Against confounding age’s cruel knife,
That he shall never cut from memory
My sweet love’s beauty, though my lover’s life.
  His beauty shall in these black lines be seen,
  And they shall live, and he in them still green.


                    64

When I have seen by Time’s fell hand defaced
The rich-proud cost of outworn buried age,
When sometime lofty towers I see down-rased,
And brass eternal slave to mortal rage.
When I have seen the hungry ocean gain
Advantage on the kingdom of the shore,
And the firm soil win of the watery main,
Increasing store with loss, and loss with store.
When I have seen such interchange of State,
Or state it self confounded, to decay,
Ruin hath taught me thus to ruminate:
That Time will come and take my love away.
  This thought is as a death which cannot choose
  But weep to have, that which it fears to lose.


                    65

Since brass, nor stone, nor earth, nor boundless sea,
But sad mortality o’ersways their power,
How with this rage shall beauty hold a plea,
Whose action is no stronger than a flower?
O how shall summer’s honey breath hold out,
Against the wrackful siege of batt’ring days,
When rocks impregnable are not so stout,
Nor gates of steel so strong but time decays?
O fearful meditation, where alack,
Shall Time’s best jewel from Time’s chest lie hid?
Or what strong hand can hold his swift foot back,
Or who his spoil of beauty can forbid?
  O none, unless this miracle have might,
  That in black ink my love may still shine bright.


                    66

Tired with all these, for restful death I cry:
As to behold desert a beggar born,
And needy nothing trimmed in jollity,
And purest faith unhappily forsworn,
And gilded honour shamefully misplaced,
And maiden virtue rudely strumpeted,
And right perfection wrongfully disgraced,
And strength by limping sway disabled
And art made tongue-tied by authority,
And folly, doctor-like, controlling skill,
And simple truth miscalled simplicity,
And captive good attending captain ill.
  Tired with all these, from these would I be gone,
  Save that to die, I leave my love alone.


                    67

Ah wherefore with infection should he live,
And with his presence grace impiety,
That sin by him advantage should achieve,
And lace it self with his society?
Why should false painting imitate his cheek,
And steal dead seeming of his living hue?
Why should poor beauty indirectly seek,
Roses of shadow, since his rose is true?
Why should he live, now nature bankrupt is,
Beggared of blood to blush through lively veins,
For she hath no exchequer now but his,
And proud of many, lives upon his gains?
  O him she stores, to show what wealth she had,
  In days long since, before these last so bad.


                    68

Thus is his cheek the map of days outworn,
When beauty lived and died as flowers do now,
Before these bastard signs of fair were born,
Or durst inhabit on a living brow:
Before the golden tresses of the dead,
The right of sepulchres, were shorn away,
To live a second life on second head,
Ere beauty’s dead fleece made another gay:
In him those holy antique hours are seen,
Without all ornament, it self and true,
Making no summer of another’s green,
Robbing no old to dress his beauty new,
  And him as for a map doth Nature store,
  To show false Art what beauty was of yore.


                    69

Those parts of thee that the world’s eye doth view,
Want nothing that the thought of hearts can mend:
All tongues, the voice of souls, give thee that due,
Uttering bare truth, even so as foes commend.
Thy outward thus with outward praise is crowned,
But those same tongues that give thee so thine own,
In other accents do this praise confound
By seeing farther than the eye hath shown.
They look into the beauty of thy mind,
And that in guess they measure by thy deeds,
Then churls their thoughts (although their eyes were kind)
To thy fair flower add the rank smell of weeds:
  But why thy odour matcheth not thy show,
  The soil is this, that thou dost common grow.


                    70

That thou art blamed shall not be thy defect,
For slander’s mark was ever yet the fair,
The ornament of beauty is suspect,
A crow that flies in heaven’s sweetest air.
So thou be good, slander doth but approve,
Thy worth the greater being wooed of time,
For canker vice the sweetest buds doth love,
And thou present’st a pure unstained prime.
Thou hast passed by the ambush of young days,
Either not assailed, or victor being charged,
Yet this thy praise cannot be so thy praise,
To tie up envy, evermore enlarged,
  If some suspect of ill masked not thy show,
  Then thou alone kingdoms of hearts shouldst owe.


                    71

No longer mourn for me when I am dead,
Than you shall hear the surly sullen bell
Give warning to the world that I am fled
From this vile world with vilest worms to dwell:
Nay if you read this line, remember not,
The hand that writ it, for I love you so,
That I in your sweet thoughts would be forgot,
If thinking on me then should make you woe.
O if, I say, you look upon this verse,
When I (perhaps) compounded am with clay,
Do not so much as my poor name rehearse;
But let your love even with my life decay.
  Lest the wise world should look into your moan,
  And mock you with me after I am gone.


                    72

O lest the world should task you to recite,
What merit lived in me that you should love
After my death, dear love, forget me quite,
For you in me can nothing worthy prove.
Unless you would devise some virtuous lie,
To do more for me than mine own desert,
And hang more praise upon deceased I,
Than niggard truth would willingly impart:
O lest your true love may seem false in this,
That you for love speak well of me untrue,
My name be buried where my body is,
And live no more to shame nor me, nor you.
  For I am shamed by that which I bring forth,
  And so should you, to love things nothing worth.


                    73

That time of year thou mayst in me behold,
When yellow leaves, or none, or few do hang
Upon those boughs which shake against the cold,
Bare ruined choirs, where late the sweet birds sang.
In me thou seest the twilight of such day,
As after sunset fadeth in the west,
Which by and by black night doth take away,
Death’s second self that seals up all in rest.
In me thou seest the glowing of such fire,
That on the ashes of his youth doth lie,
As the death-bed, whereon it must expire,
Consumed with that which it was nourished by.
  This thou perceiv’st, which makes thy love more strong,
  To love that well, which thou must leave ere long.


                    74

But be contented when that fell arrest,
Without all bail shall carry me away,
My life hath in this line some interest,
Which for memorial still with thee shall stay.
When thou reviewest this, thou dost review,
The very part was consecrate to thee,
The earth can have but earth, which is his due,
My spirit is thine the better part of me,
So then thou hast but lost the dregs of life,
The prey of worms, my body being dead,
The coward conquest of a wretch’s knife,
Too base of thee to be remembered,
  The worth of that, is that which it contains,
  And that is this, and this with thee remains.


                    75

So are you to my thoughts as food to life,
Or as sweet-seasoned showers are to the ground;
And for the peace of you I hold such strife
As ’twixt a miser and his wealth is found.
Now proud as an enjoyer, and anon
Doubting the filching age will steal his treasure,
Now counting best to be with you alone,
Then bettered that the world may see my pleasure,
Sometime all full with feasting on your sight,
And by and by clean starved for a look,
Possessing or pursuing no delight
Save what is had, or must from you be took.
  Thus do I pine and surfeit day by day,
  Or gluttoning on all, or all away.


                    76

Why is my verse so barren of new pride?
So far from variation or quick change?
Why with the time do I not glance aside
To new-found methods, and to compounds strange?
Why write I still all one, ever the same,
And keep invention in a noted weed,
That every word doth almost tell my name,
Showing their birth, and where they did proceed?
O know sweet love I always write of you,
And you and love are still my argument:
So all my best is dressing old words new,
Spending again what is already spent:
  For as the sun is daily new and old,
  So is my love still telling what is told.


                    77

Thy glass will show thee how thy beauties wear,
Thy dial how thy precious minutes waste,
These vacant leaves thy mind’s imprint will bear,
And of this book, this learning mayst thou taste.
The wrinkles which thy glass will truly show,
Of mouthed graves will give thee memory,
Thou by thy dial’s shady stealth mayst know,
Time’s thievish progress to eternity.
Look what thy memory cannot contain,
Commit to these waste blanks, and thou shalt find
Those children nursed, delivered from thy brain,
To take a new acquaintance of thy mind.
  These offices, so oft as thou wilt look,
  Shall profit thee, and much enrich thy book.


                    78

So oft have I invoked thee for my muse,
And found such fair assistance in my verse,
As every alien pen hath got my use,
And under thee their poesy disperse.
Thine eyes, that taught the dumb on high to sing,
And heavy ignorance aloft to fly,
Have added feathers to the learned’s wing,
And given grace a double majesty.
Yet be most proud of that which I compile,
Whose influence is thine, and born of thee,
In others’ works thou dost but mend the style,
And arts with thy sweet graces graced be.
  But thou art all my art, and dost advance
  As high as learning, my rude ignorance.


                    79

Whilst I alone did call upon thy aid,
My verse alone had all thy gentle grace,
But now my gracious numbers are decayed,
And my sick muse doth give an other place.
I grant (sweet love) thy lovely argument
Deserves the travail of a worthier pen,
Yet what of thee thy poet doth invent,
He robs thee of, and pays it thee again,
He lends thee virtue, and he stole that word,
From thy behaviour, beauty doth he give
And found it in thy cheek: he can afford
No praise to thee, but what in thee doth live.
  Then thank him not for that which he doth say,
  Since what he owes thee, thou thyself dost pay.


                    80

O how I faint when I of you do write,
Knowing a better spirit doth use your name,
And in the praise thereof spends all his might,
To make me tongue-tied speaking of your fame.
But since your worth, wide as the ocean is,
The humble as the proudest sail doth bear,
My saucy bark (inferior far to his)
On your broad main doth wilfully appear.
Your shallowest help will hold me up afloat,
Whilst he upon your soundless deep doth ride,
Or (being wrecked) I am a worthless boat,
He of tall building, and of goodly pride.
  Then if he thrive and I be cast away,
  The worst was this: my love was my decay.


                    81

Or I shall live your epitaph to make,
Or you survive when I in earth am rotten,
From hence your memory death cannot take,
Although in me each part will be forgotten.
Your name from hence immortal life shall have,
Though I (once gone) to all the world must die,
The earth can yield me but a common grave,
When you entombed in men’s eyes shall lie,
Your monument shall be my gentle verse,
Which eyes not yet created shall o’er-read,
And tongues to be, your being shall rehearse,
When all the breathers of this world are dead,
  You still shall live, such virtue hath my pen,
  Where breath most breathes, even in the mouths of men.


                    82

I grant thou wert not married to my muse,
And therefore mayst without attaint o’erlook
The dedicated words which writers use
Of their fair subject, blessing every book.
Thou art as fair in knowledge as in hue,
Finding thy worth a limit past my praise,
And therefore art enforced to seek anew,
Some fresher stamp of the time-bettering days.
And do so love, yet when they have devised,
What strained touches rhetoric can lend,
Thou truly fair, wert truly sympathized,
In true plain words, by thy true-telling friend.
  And their gross painting might be better used,
  Where cheeks need blood, in thee it is abused.


                    83

I never saw that you did painting need,
And therefore to your fair no painting set,
I found (or thought I found) you did exceed,
That barren tender of a poet’s debt:
And therefore have I slept in your report,
That you yourself being extant well might show,
How far a modern quill doth come too short,
Speaking of worth, what worth in you doth grow.
This silence for my sin you did impute,
Which shall be most my glory being dumb,
For I impair not beauty being mute,
When others would give life, and bring a tomb.
  There lives more life in one of your fair eyes,
  Than both your poets can in praise devise.


                    84

Who is it that says most, which can say more,
Than this rich praise: that you alone are you,
In whose confine immured is the store,
Which should example where your equal grew.
Lean penury within that pen doth dwell,
That to his subject lends not some small glory,
But he that writes of you, if he can tell,
That you are you, so dignifies his story.
Let him but copy what in you is writ,
Not making worse what nature made so clear,
And such a counterpart shall fame his wit,
Making his style admired every where.
  You to your beauteous blessings add a curse,
  Being fond on praise, which makes your praises worse.


                    85

My tongue-tied muse in manners holds her still,
While comments of your praise richly compiled,
Reserve their character with golden quill,
And precious phrase by all the Muses filed.
I think good thoughts, whilst other write good words,
And like unlettered clerk still cry Amen,
To every hymn that able spirit affords,
In polished form of well refined pen.
Hearing you praised, I say ’tis so, ’tis true,
And to the most of praise add something more,
But that is in my thought, whose love to you
(Though words come hindmost) holds his rank before,
  Then others, for the breath of words respect,
  Me for my dumb thoughts, speaking in effect.


                    86

Was it the proud full sail of his great verse,
Bound for the prize of (all too precious) you,
That did my ripe thoughts in my brain inhearse,
Making their tomb the womb wherein they grew?
Was it his spirit, by spirits taught to write,
Above a mortal pitch, that struck me dead?
No, neither he, nor his compeers by night
Giving him aid, my verse astonished.
He nor that affable familiar ghost
Which nightly gulls him with intelligence,
As victors of my silence cannot boast,
I was not sick of any fear from thence.
  But when your countenance filled up his line,
  Then lacked I matter, that enfeebled mine.


                    87

Farewell! thou art too dear for my possessing,
And like enough thou know’st thy estimate,
The charter of thy worth gives thee releasing:
My bonds in thee are all determinate.
For how do I hold thee but by thy granting,
And for that riches where is my deserving?
The cause of this fair gift in me is wanting,
And so my patent back again is swerving.
Thyself thou gav’st, thy own worth then not knowing,
Or me to whom thou gav’st it, else mistaking,
So thy great gift upon misprision growing,
Comes home again, on better judgement making.
  Thus have I had thee as a dream doth flatter,
  In sleep a king, but waking no such matter.


                    88

When thou shalt be disposed to set me light,
And place my merit in the eye of scorn,
Upon thy side, against my self I’ll fight,
And prove thee virtuous, though thou art forsworn:
With mine own weakness being best acquainted,
Upon thy part I can set down a story
Of faults concealed, wherein I am attainted:
That thou in losing me, shalt win much glory:
And I by this will be a gainer too,
For bending all my loving thoughts on thee,
The injuries that to my self I do,
Doing thee vantage, double-vantage me.
  Such is my love, to thee I so belong,
  That for thy right, my self will bear all wrong.


                    89

Say that thou didst forsake me for some fault,
And I will comment upon that offence,
Speak of my lameness, and I straight will halt:
Against thy reasons making no defence.
Thou canst not (love) disgrace me half so ill,
To set a form upon desired change,
As I’ll my self disgrace, knowing thy will,
I will acquaintance strangle and look strange:
Be absent from thy walks and in my tongue,
Thy sweet beloved name no more shall dwell,
Lest I, too much profane, should do it wrong,
And haply of our old acquaintance tell.
  For thee, against my self I’ll vow debate,
  For I must ne’er love him whom thou dost hate.


                    90

Then hate me when thou wilt, if ever, now,
Now while the world is bent my deeds to cross,
join with the spite of fortune, make me bow,
And do not drop in for an after-loss:
Ah do not, when my heart hath ’scaped this sorrow,
Come in the rearward of a conquered woe,
Give not a windy night a rainy morrow,
To linger out a purposed overthrow.
If thou wilt leave me, do not leave me last,
When other petty griefs have done their spite,
But in the onset come, so shall I taste
At first the very worst of fortune’s might.
  And other strains of woe, which now seem woe,
  Compared with loss of thee, will not seem so.


                    91

Some glory in their birth, some in their skill,
Some in their wealth, some in their body’s force,
Some in their garments though new-fangled ill:
Some in their hawks and hounds, some in their horse.
And every humour hath his adjunct pleasure,
Wherein it finds a joy above the rest,
But these particulars are not my measure,
All these I better in one general best.
Thy love is better than high birth to me,
Richer than wealth, prouder than garments’ costs,
Of more delight than hawks and horses be:
And having thee, of all men’s pride I boast.
  Wretched in this alone, that thou mayst take,
  All this away, and me most wretched make.


                    92

But do thy worst to steal thyself away,
For term of life thou art assured mine,
And life no longer than thy love will stay,
For it depends upon that love of thine.
Then need I not to fear the worst of wrongs,
When in the least of them my life hath end,
I see, a better state to me belongs
Than that, which on thy humour doth depend.
Thou canst not vex me with inconstant mind,
Since that my life on thy revolt doth lie,
O what a happy title do I find,
Happy to have thy love, happy to die!
  But what’s so blessed-fair that fears no blot?
  Thou mayst be false, and yet I know it not.


                    93

So shall I live, supposing thou art true,
Like a deceived husband, so love’s face,
May still seem love to me, though altered new:
Thy looks with me, thy heart in other place.
For there can live no hatred in thine eye,
Therefore in that I cannot know thy change,
In many’s looks, the false heart’s history
Is writ in moods and frowns and wrinkles strange.
But heaven in thy creation did decree,
That in thy face sweet love should ever dwell,
Whate’er thy thoughts, or thy heart’s workings be,
Thy looks should nothing thence, but sweetness tell.
  How like Eve’s apple doth thy beauty grow,
  If thy sweet virtue answer not thy show.


                    94

They that have power to hurt, and will do none,
That do not do the thing, they most do show,
Who moving others, are themselves as stone,
Unmoved, cold, and to temptation slow:
They rightly do inherit heaven’s graces,
And husband nature’s riches from expense,
They are the lords and owners of their faces,
Others, but stewards of their excellence:
The summer’s flower is to the summer sweet,
Though to it self, it only live and die,
But if that flower with base infection meet,
The basest weed outbraves his dignity:
  For sweetest things turn sourest by their deeds,
  Lilies that fester, smell far worse than weeds.


                    95

How sweet and lovely dost thou make the shame,
Which like a canker in the fragrant rose,
Doth spot the beauty of thy budding name!
O in what sweets dost thou thy sins enclose!
That tongue that tells the story of thy days,
(Making lascivious comments on thy sport)
Cannot dispraise, but in a kind of praise,
Naming thy name, blesses an ill report.
O what a mansion have those vices got,
Which for their habitation chose out thee,
Where beauty’s veil doth cover every blot,
And all things turns to fair, that eyes can see!
  Take heed (dear heart) of this large privilege,
  The hardest knife ill-used doth lose his edge.


                    96

Some say thy fault is youth, some wantonness,
Some say thy grace is youth and gentle sport,
Both grace and faults are loved of more and less:
Thou mak’st faults graces, that to thee resort:
As on the finger of a throned queen,
The basest jewel will be well esteemed:
So are those errors that in thee are seen,
To truths translated, and for true things deemed.
How many lambs might the stern wolf betray,
If like a lamb he could his looks translate!
How many gazers mightst thou lead away,
If thou wouldst use the strength of all thy state!
  But do not so, I love thee in such sort,
  As thou being mine, mine is thy good report.


                    97

How like a winter hath my absence been
From thee, the pleasure of the fleeting year!
What freezings have I felt, what dark days seen!
What old December’s bareness everywhere!
And yet this time removed was summer’s time,
The teeming autumn big with rich increase,
Bearing the wanton burden of the prime,
Like widowed wombs after their lords’ decease:
Yet this abundant issue seemed to me
But hope of orphans, and unfathered fruit,
For summer and his pleasures wait on thee,
And thou away, the very birds are mute.
  Or if they sing, ’tis with so dull a cheer,
  That leaves look pale, dreading the winter’s near.


                    98

From you have I been absent in the spring,
When proud-pied April (dressed in all his trim)
Hath put a spirit of youth in every thing:
That heavy Saturn laughed and leaped with him.
Yet nor the lays of birds, nor the sweet smell
Of different flowers in odour and in hue,
Could make me any summer’s story tell:
Or from their proud lap pluck them where they grew:
Nor did I wonder at the lily’s white,
Nor praise the deep vermilion in the rose,
They were but sweet, but figures of delight:
Drawn after you, you pattern of all those.
  Yet seemed it winter still, and you away,
  As with your shadow I with these did play.


                    99

The forward violet thus did I chide,
Sweet thief, whence didst thou steal thy sweet that smells,
If not from my love’s breath? The purple pride
Which on thy soft cheek for complexion dwells,
In my love’s veins thou hast too grossly dyed.
The lily I condemned for thy hand,
And buds of marjoram had stol’n thy hair,
The roses fearfully on thorns did stand,
One blushing shame, another white despair:
A third nor red, nor white, had stol’n of both,
And to his robbery had annexed thy breath,
But for his theft in pride of all his growth
A vengeful canker eat him up to death.
  More flowers I noted, yet I none could see,
  But sweet, or colour it had stol’n from thee.


                    100

Where art thou Muse that thou forget’st so long,
To speak of that which gives thee all thy might?
Spend’st thou thy fury on some worthless song,
Darkening thy power to lend base subjects light?
Return forgetful Muse, and straight redeem,
In gentle numbers time so idly spent,
Sing to the ear that doth thy lays esteem,
And gives thy pen both skill and argument.
Rise resty Muse, my love’s sweet face survey,
If time have any wrinkle graven there,
If any, be a satire to decay,
And make time’s spoils despised everywhere.
  Give my love fame faster than Time wastes life,
  So thou prevent’st his scythe, and crooked knife.


                    101

O truant Muse what shall be thy amends,
For thy neglect of truth in beauty dyed?
Both truth and beauty on my love depends:
So dost thou too, and therein dignified:
Make answer Muse, wilt thou not haply say,
’Truth needs no colour with his colour fixed,
Beauty no pencil, beauty’s truth to lay:
But best is best, if never intermixed’?
Because he needs no praise, wilt thou be dumb?
Excuse not silence so, for’t lies in thee,
To make him much outlive a gilded tomb:
And to be praised of ages yet to be.
  Then do thy office Muse, I teach thee how,
  To make him seem long hence, as he shows now.


                    102

My love is strengthened though more weak in seeming,
I love not less, though less the show appear,
That love is merchandized, whose rich esteeming,
The owner’s tongue doth publish every where.
Our love was new, and then but in the spring,
When I was wont to greet it with my lays,
As Philomel in summer’s front doth sing,
And stops her pipe in growth of riper days:
Not that the summer is less pleasant now
Than when her mournful hymns did hush the night,
But that wild music burthens every bough,
And sweets grown common lose their dear delight.
  Therefore like her, I sometime hold my tongue:
  Because I would not dull you with my song.


                    103

Alack what poverty my muse brings forth,
That having such a scope to show her pride,
The argument all bare is of more worth
Than when it hath my added praise beside.
O blame me not if I no more can write!
Look in your glass and there appears a face,
That over-goes my blunt invention quite,
Dulling my lines, and doing me disgrace.
Were it not sinful then striving to mend,
To mar the subject that before was well?
For to no other pass my verses tend,
Than of your graces and your gifts to tell.
  And more, much more than in my verse can sit,
  Your own glass shows you, when you look in it.


                    104

To me fair friend you never can be old,
For as you were when first your eye I eyed,
Such seems your beauty still: three winters cold,
Have from the forests shook three summers’ pride,
Three beauteous springs to yellow autumn turned,
In process of the seasons have I seen,
Three April perfumes in three hot Junes burned,
Since first I saw you fresh which yet are green.
Ah yet doth beauty like a dial hand,
Steal from his figure, and no pace perceived,
So your sweet hue, which methinks still doth stand
Hath motion, and mine eye may be deceived.
  For fear of which, hear this thou age unbred,
  Ere you were born was beauty’s summer dead.


                    105

Let not my love be called idolatry,
Nor my beloved as an idol show,
Since all alike my songs and praises be
To one, of one, still such, and ever so.
Kind is my love to-day, to-morrow kind,
Still constant in a wondrous excellence,
Therefore my verse to constancy confined,
One thing expressing, leaves out difference.
Fair, kind, and true, is all my argument,
Fair, kind, and true, varying to other words,
And in this change is my invention spent,
Three themes in one, which wondrous scope affords.
  Fair, kind, and true, have often lived alone.
  Which three till now, never kept seat in one.


                    106

When in the chronicle of wasted time,
I see descriptions of the fairest wights,
And beauty making beautiful old rhyme,
In praise of ladies dead, and lovely knights,
Then in the blazon of sweet beauty’s best,
Of hand, of foot, of lip, of eye, of brow,
I see their antique pen would have expressed,
Even such a beauty as you master now.
So all their praises are but prophecies
Of this our time, all you prefiguring,
And for they looked but with divining eyes,
They had not skill enough your worth to sing:
  For we which now behold these present days,
  Have eyes to wonder, but lack tongues to praise.


                    107

Not mine own fears, nor the prophetic soul,
Of the wide world, dreaming on things to come,
Can yet the lease of my true love control,
Supposed as forfeit to a confined doom.
The mortal moon hath her eclipse endured,
And the sad augurs mock their own presage,
Incertainties now crown themselves assured,
And peace proclaims olives of endless age.
Now with the drops of this most balmy time,
My love looks fresh, and death to me subscribes,
Since spite of him I’ll live in this poor rhyme,
While he insults o’er dull and speechless tribes.
  And thou in this shalt find thy monument,
  When tyrants’ crests and tombs of brass are spent.


                    108

What’s in the brain that ink may character,
Which hath not figured to thee my true spirit,
What’s new to speak, what now to register,
That may express my love, or thy dear merit?
Nothing sweet boy, but yet like prayers divine,
I must each day say o’er the very same,
Counting no old thing old, thou mine, I thine,
Even as when first I hallowed thy fair name.
So that eternal love in love’s fresh case,
Weighs not the dust and injury of age,
Nor gives to necessary wrinkles place,
But makes antiquity for aye his page,
  Finding the first conceit of love there bred,
  Where time and outward form would show it dead.


                    109

O never say that I was false of heart,
Though absence seemed my flame to qualify,
As easy might I from my self depart,
As from my soul which in thy breast doth lie:
That is my home of love, if I have ranged,
Like him that travels I return again,
Just to the time, not with the time exchanged,
So that my self bring water for my stain,
Never believe though in my nature reigned,
All frailties that besiege all kinds of blood,
That it could so preposterously be stained,
To leave for nothing all thy sum of good:
  For nothing this wide universe I call,
  Save thou my rose, in it thou art my all.


                    110

Alas ’tis true, I have gone here and there,
And made my self a motley to the view,
Gored mine own thoughts, sold cheap what is most dear,
Made old offences of affections new.
Most true it is, that I have looked on truth
Askance and strangely: but by all above,
These blenches gave my heart another youth,
And worse essays proved thee my best of love.
Now all is done, have what shall have no end,
Mine appetite I never more will grind
On newer proof, to try an older friend,
A god in love, to whom I am confined.
  Then give me welcome, next my heaven the best,
  Even to thy pure and most most loving breast.


                    111

O for my sake do you with Fortune chide,
The guilty goddess of my harmful deeds,
That did not better for my life provide,
Than public means which public manners breeds.
Thence comes it that my name receives a brand,
And almost thence my nature is subdued
To what it works in, like the dyer’s hand:
Pity me then, and wish I were renewed,
Whilst like a willing patient I will drink,
Potions of eisel ’gainst my strong infection,
No bitterness that I will bitter think,
Nor double penance to correct correction.
  Pity me then dear friend, and I assure ye,
  Even that your pity is enough to cure me.


                    112

Your love and pity doth th’ impression fill,
Which vulgar scandal stamped upon my brow,
For what care I who calls me well or ill,
So you o’er-green my bad, my good allow?
You are my all the world, and I must strive,
To know my shames and praises from your tongue,
None else to me, nor I to none alive,
That my steeled sense or changes right or wrong.
In so profound abysm I throw all care
Of others’ voices, that my adder’s sense,
To critic and to flatterer stopped are:
Mark how with my neglect I do dispense.
  You are so strongly in my purpose bred,
  That all the world besides methinks are dead.


                    113

Since I left you, mine eye is in my mind,
And that which governs me to go about,
Doth part his function, and is partly blind,
Seems seeing, but effectually is out:
For it no form delivers to the heart
Of bird, of flower, or shape which it doth latch,
Of his quick objects hath the mind no part,
Nor his own vision holds what it doth catch:
For if it see the rud’st or gentlest sight,
The most sweet favour or deformed’st creature,
The mountain, or the sea, the day, or night:
The crow, or dove, it shapes them to your feature.
  Incapable of more, replete with you,
  My most true mind thus maketh mine untrue.


                    114

Or whether doth my mind being crowned with you
Drink up the monarch’s plague this flattery?
Or whether shall I say mine eye saith true,
And that your love taught it this alchemy?
To make of monsters, and things indigest,
Such cherubins as your sweet self resemble,
Creating every bad a perfect best
As fast as objects to his beams assemble:
O ’tis the first, ’tis flattery in my seeing,
And my great mind most kingly drinks it up,
Mine eye well knows what with his gust is ’greeing,
And to his palate doth prepare the cup.
  If it be poisoned, ’tis the lesser sin,
  That mine eye loves it and doth first begin.


                    115

Those lines that I before have writ do lie,
Even those that said I could not love you dearer,
Yet then my judgement knew no reason why,
My most full flame should afterwards burn clearer,
But reckoning time, whose millioned accidents
Creep in ’twixt vows, and change decrees of kings,
Tan sacred beauty, blunt the sharp’st intents,
Divert strong minds to the course of alt’ring things:
Alas why fearing of time’s tyranny,
Might I not then say ‘Now I love you best,’
When I was certain o’er incertainty,
Crowning the present, doubting of the rest?
  Love is a babe, then might I not say so
  To give full growth to that which still doth grow.


                    116

Let me not to the marriage of true minds
Admit impediments, love is not love
Which alters when it alteration finds,
Or bends with the remover to remove.
O no, it is an ever-fixed mark
That looks on tempests and is never shaken;
It is the star to every wand’ring bark,
Whose worth’s unknown, although his height be taken.
Love’s not Time’s fool, though rosy lips and cheeks
Within his bending sickle’s compass come,
Love alters not with his brief hours and weeks,
But bears it out even to the edge of doom:
  If this be error and upon me proved,
  I never writ, nor no man ever loved.


                    117

Accuse me thus, that I have scanted all,
Wherein I should your great deserts repay,
Forgot upon your dearest love to call,
Whereto all bonds do tie me day by day,
That I have frequent been with unknown minds,
And given to time your own dear-purchased right,
That I have hoisted sail to all the winds
Which should transport me farthest from your sight.
Book both my wilfulness and errors down,
And on just proof surmise, accumulate,
Bring me within the level of your frown,
But shoot not at me in your wakened hate:
  Since my appeal says I did strive to prove
  The constancy and virtue of your love.


                    118

Like as to make our appetite more keen
With eager compounds we our palate urge,
As to prevent our maladies unseen,
We sicken to shun sickness when we purge.
Even so being full of your ne’er-cloying sweetness,
To bitter sauces did I frame my feeding;
And sick of welfare found a kind of meetness,
To be diseased ere that there was true needing.
Thus policy in love t’ anticipate
The ills that were not, grew to faults assured,
And brought to medicine a healthful state
Which rank of goodness would by ill be cured.
  But thence I learn and find the lesson true,
  Drugs poison him that so fell sick of you.


                    119

What potions have I drunk of Siren tears
Distilled from limbecks foul as hell within,
Applying fears to hopes, and hopes to fears,
Still losing when I saw my self to win!
What wretched errors hath my heart committed,
Whilst it hath thought it self so blessed never!
How have mine eyes out of their spheres been fitted
In the distraction of this madding fever!
O benefit of ill, now I find true
That better is, by evil still made better.
And ruined love when it is built anew
Grows fairer than at first, more strong, far greater.
  So I return rebuked to my content,
  And gain by ills thrice more than I have spent.


                    120

That you were once unkind befriends me now,
And for that sorrow, which I then did feel,
Needs must I under my transgression bow,
Unless my nerves were brass or hammered steel.
For if you were by my unkindness shaken
As I by yours, y’have passed a hell of time,
And I a tyrant have no leisure taken
To weigh how once I suffered in your crime.
O that our night of woe might have remembered
My deepest sense, how hard true sorrow hits,
And soon to you, as you to me then tendered
The humble salve, which wounded bosoms fits!
  But that your trespass now becomes a fee,
  Mine ransoms yours, and yours must ransom me.


                    121

’Tis better to be vile than vile esteemed,
When not to be, receives reproach of being,
And the just pleasure lost, which is so deemed,
Not by our feeling, but by others’ seeing.
For why should others’ false adulterate eyes
Give salutation to my sportive blood?
Or on my frailties why are frailer spies,
Which in their wills count bad what I think good?
No, I am that I am, and they that level
At my abuses, reckon up their own,
I may be straight though they themselves be bevel;
By their rank thoughts, my deeds must not be shown
  Unless this general evil they maintain,
  All men are bad and in their badness reign.


                    122

Thy gift, thy tables, are within my brain
Full charactered with lasting memory,
Which shall above that idle rank remain
Beyond all date even to eternity.
Or at the least, so long as brain and heart
Have faculty by nature to subsist,
Till each to razed oblivion yield his part
Of thee, thy record never can be missed:
That poor retention could not so much hold,
Nor need I tallies thy dear love to score,
Therefore to give them from me was I bold,
To trust those tables that receive thee more:
  To keep an adjunct to remember thee
  Were to import forgetfulness in me.


                    123

No! Time, thou shalt not boast that I do change,
Thy pyramids built up with newer might
To me are nothing novel, nothing strange,
They are but dressings of a former sight:
Our dates are brief, and therefore we admire,
What thou dost foist upon us that is old,
And rather make them born to our desire,
Than think that we before have heard them told:
Thy registers and thee I both defy,
Not wond’ring at the present, nor the past,
For thy records, and what we see doth lie,
Made more or less by thy continual haste:
  This I do vow and this shall ever be,
  I will be true despite thy scythe and thee.


                    124

If my dear love were but the child of state,
It might for Fortune’s bastard be unfathered,
As subject to time’s love or to time’s hate,
Weeds among weeds, or flowers with flowers gathered.
No it was builded far from accident,
It suffers not in smiling pomp, nor falls
Under the blow of thralled discontent,
Whereto th’ inviting time our fashion calls:
It fears not policy that heretic,
Which works on leases of short-numbered hours,
But all alone stands hugely politic,
That it nor grows with heat, nor drowns with showers.
  To this I witness call the fools of time,
  Which die for goodness, who have lived for crime.


                    125

Were’t aught to me I bore the canopy,
With my extern the outward honouring,
Or laid great bases for eternity,
Which proves more short than waste or ruining?
Have I not seen dwellers on form and favour
Lose all, and more by paying too much rent
For compound sweet; forgoing simple savour,
Pitiful thrivers in their gazing spent?
No, let me be obsequious in thy heart,
And take thou my oblation, poor but free,
Which is not mixed with seconds, knows no art,
But mutual render, only me for thee.
  Hence, thou suborned informer, a true soul
  When most impeached, stands least in thy control.


                    126

O thou my lovely boy who in thy power,
Dost hold Time’s fickle glass his fickle hour:
Who hast by waning grown, and therein show’st,
Thy lovers withering, as thy sweet self grow’st.
If Nature (sovereign mistress over wrack)
As thou goest onwards still will pluck thee back,
She keeps thee to this purpose, that her skill
May time disgrace, and wretched minutes kill.
Yet fear her O thou minion of her pleasure,
She may detain, but not still keep her treasure!
  Her audit (though delayed) answered must be,
  And her quietus is to render thee.


                    127

In the old age black was not counted fair,
Or if it were it bore not beauty’s name:
But now is black beauty’s successive heir,
And beauty slandered with a bastard shame,
For since each hand hath put on nature’s power,
Fairing the foul with art’s false borrowed face,
Sweet beauty hath no name, no holy bower,
But is profaned, if not lives in disgrace.
Therefore my mistress’ eyes are raven black,
Her eyes so suited, and they mourners seem
At such who not born fair no beauty lack,
Slandering creation with a false esteem,
  Yet so they mourn becoming of their woe,
  That every tongue says beauty should look so.


                    128

How oft when thou, my music, music play’st,
Upon that blessed wood whose motion sounds
With thy sweet fingers when thou gently sway’st
The wiry concord that mine ear confounds,
Do I envy those jacks that nimble leap,
To kiss the tender inward of thy hand,
Whilst my poor lips which should that harvest reap,
At the wood’s boldness by thee blushing stand.
To be so tickled they would change their state
And situation with those dancing chips,
O’er whom thy fingers walk with gentle gait,
Making dead wood more blest than living lips,
  Since saucy jacks so happy are in this,
  Give them thy fingers, me thy lips to kiss.


                    129

Th’ expense of spirit in a waste of shame
Is lust in action, and till action, lust
Is perjured, murd’rous, bloody full of blame,
Savage, extreme, rude, cruel, not to trust,
Enjoyed no sooner but despised straight,
Past reason hunted, and no sooner had
Past reason hated as a swallowed bait,
On purpose laid to make the taker mad.
Mad in pursuit and in possession so,
Had, having, and in quest, to have extreme,
A bliss in proof, and proved, a very woe;
Before a joy proposed behind a dream.
  All this the world well knows yet none knows well,
  To shun the heaven that leads men to this hell.


                    130

My mistress’ eyes are nothing like the sun,
Coral is far more red, than her lips red,
If snow be white, why then her breasts are dun:
If hairs be wires, black wires grow on her head:
I have seen roses damasked, red and white,
But no such roses see I in her cheeks,
And in some perfumes is there more delight,
Than in the breath that from my mistress reeks.
I love to hear her speak, yet well I know,
That music hath a far more pleasing sound:
I grant I never saw a goddess go;
My mistress when she walks treads on the ground.
  And yet by heaven I think my love as rare,
  As any she belied with false compare.


                    131

Thou art as tyrannous, so as thou art,
As those whose beauties proudly make them cruel;
For well thou know’st to my dear doting heart
Thou art the fairest and most precious jewel.
Yet in good faith some say that thee behold,
Thy face hath not the power to make love groan;
To say they err, I dare not be so bold,
Although I swear it to my self alone.
And to be sure that is not false I swear,
A thousand groans but thinking on thy face,
One on another’s neck do witness bear
Thy black is fairest in my judgement’s place.
  In nothing art thou black save in thy deeds,
  And thence this slander as I think proceeds.


                    132

Thine eyes I love, and they as pitying me,
Knowing thy heart torment me with disdain,
Have put on black, and loving mourners be,
Looking with pretty ruth upon my pain.
And truly not the morning sun of heaven
Better becomes the grey cheeks of the east,
Nor that full star that ushers in the even
Doth half that glory to the sober west
As those two mourning eyes become thy face:
O let it then as well beseem thy heart
To mourn for me since mourning doth thee grace,
And suit thy pity like in every part.
  Then will I swear beauty herself is black,
  And all they foul that thy complexion lack.


                    133

Beshrew that heart that makes my heart to groan
For that deep wound it gives my friend and me;
Is’t not enough to torture me alone,
But slave to slavery my sweet’st friend must be?
Me from my self thy cruel eye hath taken,
And my next self thou harder hast engrossed,
Of him, my self, and thee I am forsaken,
A torment thrice three-fold thus to be crossed:
Prison my heart in thy steel bosom’s ward,
But then my friend’s heart let my poor heart bail,
Whoe’er keeps me, let my heart be his guard,
Thou canst not then use rigour in my gaol.
  And yet thou wilt, for I being pent in thee,
  Perforce am thine and all that is in me.


                    134

So now I have confessed that he is thine,
And I my self am mortgaged to thy will,
My self I’ll forfeit, so that other mine,
Thou wilt restore to be my comfort still:
But thou wilt not, nor he will not be free,
For thou art covetous, and he is kind,
He learned but surety-like to write for me,
Under that bond that him as fist doth bind.
The statute of thy beauty thou wilt take,
Thou usurer that put’st forth all to use,
And sue a friend, came debtor for my sake,
So him I lose through my unkind abuse.
  Him have I lost, thou hast both him and me,
  He pays the whole, and yet am I not free.


                    135

Whoever hath her wish, thou hast thy will,
And Will to boot, and Will in overplus,
More than enough am I that vex thee still,
To thy sweet will making addition thus.
Wilt thou whose will is large and spacious,
Not once vouchsafe to hide my will in thine?
Shall will in others seem right gracious,
And in my will no fair acceptance shine?
The sea all water, yet receives rain still,
And in abundance addeth to his store,
So thou being rich in will add to thy will
One will of mine to make thy large will more.
  Let no unkind, no fair beseechers kill,
  Think all but one, and me in that one Will.


                    136

If thy soul check thee that I come so near,
Swear to thy blind soul that I was thy Will,
And will thy soul knows is admitted there,
Thus far for love, my love-suit sweet fulfil.
Will will fulfil the treasure of thy love,
Ay, fill it full with wills, and my will one,
In things of great receipt with case we prove,
Among a number one is reckoned none.
Then in the number let me pass untold,
Though in thy store’s account I one must be,
For nothing hold me, so it please thee hold,
That nothing me, a something sweet to thee.
  Make but my name thy love, and love that still,
  And then thou lov’st me for my name is Will.


                    137

Thou blind fool Love, what dost thou to mine eyes,
That they behold and see not what they see?
They know what beauty is, see where it lies,
Yet what the best is, take the worst to be.
If eyes corrupt by over-partial looks,
Be anchored in the bay where all men ride,
Why of eyes’ falsehood hast thou forged hooks,
Whereto the judgement of my heart is tied?
Why should my heart think that a several plot,
Which my heart knows the wide world’s common place?
Or mine eyes seeing this, say this is not
To put fair truth upon so foul a face?
  In things right true my heart and eyes have erred,
  And to this false plague are they now transferred.


                    138

When my love swears that she is made of truth,
I do believe, her though I know she lies,
That she might think me some untutored youth,
Unlearned in the world’s false subtleties.
Thus vainly thinking that she thinks me young,
Although she knows my days are past the best,
Simply I credit her false-speaking tongue;
On both sides thus is simple truth suppressed.
But wherefore says she not she is unjust?
And wherefore say not I that I am old?
O love’s best habit is in seeming trust,
And age in love loves not to have years told.
  Therefore I lie with her, and she with me,
  And in our faults by lies we flattered be.


                    139

O call not me to justify the wrong,
That thy unkindness lays upon my heart,
Wound me not with thine eye but with thy tongue,
Use power with power, and slay me not by art,
Tell me thou lov’st elsewhere; but in my sight,
Dear heart forbear to glance thine eye aside,
What need’st thou wound with cunning when thy might
Is more than my o’erpressed defence can bide?
Let me excuse thee, ah my love well knows,
Her pretty looks have been mine enemies,
And therefore from my face she turns my foes,
That they elsewhere might dart their injuries:
  Yet do not so, but since I am near slain,
  Kill me outright with looks, and rid my pain.


                    140

Be wise as thou art cruel, do not press
My tongue-tied patience with too much disdain:
Lest sorrow lend me words and words express,
The manner of my pity-wanting pain.
If I might teach thee wit better it were,
Though not to love, yet love to tell me so,
As testy sick men when their deaths be near,
No news but health from their physicians know.
For if I should despair I should grow mad,
And in my madness might speak ill of thee,
Now this ill-wresting world is grown so bad,
Mad slanderers by mad ears believed be.
  That I may not be so, nor thou belied,
  Bear thine eyes straight, though thy proud heart go wide.


                    141

In faith I do not love thee with mine eyes,
For they in thee a thousand errors note,
But ’tis my heart that loves what they despise,
Who in despite of view is pleased to dote.
Nor are mine ears with thy tongue’s tune delighted,
Nor tender feeling to base touches prone,
Nor taste, nor smell, desire to be invited
To any sensual feast with thee alone:
But my five wits, nor my five senses can
Dissuade one foolish heart from serving thee,
Who leaves unswayed the likeness of a man,
Thy proud heart’s slave and vassal wretch to be:
  Only my plague thus far I count my gain,
  That she that makes me sin, awards me pain.


                    142

Love is my sin, and thy dear virtue hate,
Hate of my sin, grounded on sinful loving,
O but with mine, compare thou thine own state,
And thou shalt find it merits not reproving,
Or if it do, not from those lips of thine,
That have profaned their scarlet ornaments,
And sealed false bonds of love as oft as mine,
Robbed others’ beds’ revenues of their rents.
Be it lawful I love thee as thou lov’st those,
Whom thine eyes woo as mine importune thee,
Root pity in thy heart that when it grows,
Thy pity may deserve to pitied be.
  If thou dost seek to have what thou dost hide,
  By self-example mayst thou be denied.


                    143

Lo as a careful huswife runs to catch,
One of her feathered creatures broke away,
Sets down her babe and makes all swift dispatch
In pursuit of the thing she would have stay:
Whilst her neglected child holds her in chase,
Cries to catch her whose busy care is bent,
To follow that which flies before her face:
Not prizing her poor infant’s discontent;
So run’st thou after that which flies from thee,
Whilst I thy babe chase thee afar behind,
But if thou catch thy hope turn back to me:
And play the mother’s part, kiss me, be kind.
  So will I pray that thou mayst have thy Will,
  If thou turn back and my loud crying still.


                    144

Two loves I have, of comfort and despair,
Which, like two spirits, do suggest me still:
The better angel is a man right fair,
The worser spirit a woman coloured ill.
To win me soon to hell my female evil
Tempteth my better angel from my side,
And would corrupt my saint to be a devil,
Wooing his purity with her foul pride.
And whether that my angel be turned fiend
Suspect I may, yet not directly tell;
But being both from me both to each friend,
I guess one angel in another’s hell.
  Yet this shall I ne’er know, but live in doubt,
  Till my bad angel fire my good one out.


                    145

Those lips that Love’s own hand did make,
Breathed forth the sound that said ‘I hate’,
To me that languished for her sake:
But when she saw my woeful state,
Straight in her heart did mercy come,
Chiding that tongue that ever sweet,
Was used in giving gentle doom:
And taught it thus anew to greet:
‘I hate’ she altered with an end,
That followed it as gentle day,
Doth follow night who like a fiend
From heaven to hell is flown away.
  ‘I hate’, from hate away she threw,
  And saved my life saying ‘not you’.


                    146

Poor soul the centre of my sinful earth,
My sinful earth these rebel powers array,
Why dost thou pine within and suffer dearth
Painting thy outward walls so costly gay?
Why so large cost having so short a lease,
Dost thou upon thy fading mansion spend?
Shall worms inheritors of this excess
Eat up thy charge? is this thy body’s end?
Then soul live thou upon thy servant’s loss,
And let that pine to aggravate thy store;
Buy terms divine in selling hours of dross;
Within be fed, without be rich no more,
  So shall thou feed on death, that feeds on men,
  And death once dead, there’s no more dying then.


                    147

My love is as a fever longing still,
For that which longer nurseth the disease,
Feeding on that which doth preserve the ill,
Th’ uncertain sickly appetite to please:
My reason the physician to my love,
Angry that his prescriptions are not kept
Hath left me, and I desperate now approve,
Desire is death, which physic did except.
Past cure I am, now reason is past care,
And frantic-mad with evermore unrest,
My thoughts and my discourse as mad men’s are,
At random from the truth vainly expressed.
  For I have sworn thee fair, and thought thee bright,
  Who art as black as hell, as dark as night.


                    148

O me! what eyes hath love put in my head,
Which have no correspondence with true sight,
Or if they have, where is my judgement fled,
That censures falsely what they see aright?
If that be fair whereon my false eyes dote,
What means the world to say it is not so?
If it be not, then love doth well denote,
Love’s eye is not so true as all men’s: no,
How can it? O how can love’s eye be true,
That is so vexed with watching and with tears?
No marvel then though I mistake my view,
The sun it self sees not, till heaven clears.
  O cunning love, with tears thou keep’st me blind,
  Lest eyes well-seeing thy foul faults should find.


                    149

Canst thou O cruel, say I love thee not,
When I against my self with thee partake?
Do I not think on thee when I forgot
Am of my self, all-tyrant, for thy sake?
Who hateth thee that I do call my friend,
On whom frown’st thou that I do fawn upon,
Nay if thou lour’st on me do I not spend
Revenge upon my self with present moan?
What merit do I in my self respect,
That is so proud thy service to despise,
When all my best doth worship thy defect,
Commanded by the motion of thine eyes?
  But love hate on for now I know thy mind,
  Those that can see thou lov’st, and I am blind.


                    150

O from what power hast thou this powerful might,
With insufficiency my heart to sway,
To make me give the lie to my true sight,
And swear that brightness doth not grace the day?
Whence hast thou this becoming of things ill,
That in the very refuse of thy deeds,
There is such strength and warrantise of skill,
That in my mind thy worst all best exceeds?
Who taught thee how to make me love thee more,
The more I hear and see just cause of hate?
O though I love what others do abhor,
With others thou shouldst not abhor my state.
  If thy unworthiness raised love in me,
  More worthy I to be beloved of thee.


                    151

Love is too young to know what conscience is,
Yet who knows not conscience is born of love?
Then gentle cheater urge not my amiss,
Lest guilty of my faults thy sweet self prove.
For thou betraying me, I do betray
My nobler part to my gross body’s treason,
My soul doth tell my body that he may,
Triumph in love, flesh stays no farther reason,
But rising at thy name doth point out thee,
As his triumphant prize, proud of this pride,
He is contented thy poor drudge to be,
To stand in thy affairs, fall by thy side.
  No want of conscience hold it that I call,
  Her love, for whose dear love I rise and fall.


                    152

In loving thee thou know’st I am forsworn,
But thou art twice forsworn to me love swearing,
In act thy bed-vow broke and new faith torn,
In vowing new hate after new love bearing:
But why of two oaths’ breach do I accuse thee,
When I break twenty? I am perjured most,
For all my vows are oaths but to misuse thee:
And all my honest faith in thee is lost.
For I have sworn deep oaths of thy deep kindness:
Oaths of thy love, thy truth, thy constancy,
And to enlighten thee gave eyes to blindness,
Or made them swear against the thing they see.
  For I have sworn thee fair: more perjured I,
  To swear against the truth so foul a lie.


                    153

Cupid laid by his brand and fell asleep,
A maid of Dian’s this advantage found,
And his love-kindling fire did quickly steep
In a cold valley-fountain of that ground:
Which borrowed from this holy fire of Love,
A dateless lively heat still to endure,
And grew a seething bath which yet men prove,
Against strange maladies a sovereign cure:
But at my mistress’ eye Love’s brand new-fired,
The boy for trial needs would touch my breast,
I sick withal the help of bath desired,
And thither hied a sad distempered guest.
  But found no cure, the bath for my help lies,
  Where Cupid got new fire; my mistress’ eyes.


                    154

The little Love-god lying once asleep,
Laid by his side his heart-inflaming brand,
Whilst many nymphs that vowed chaste life to keep,
Came tripping by, but in her maiden hand,
The fairest votary took up that fire,
Which many legions of true hearts had warmed,
And so the general of hot desire,
Was sleeping by a virgin hand disarmed.
This brand she quenched in a cool well by,
Which from Love’s fire took heat perpetual,
Growing a bath and healthful remedy,
For men diseased; but I, my mistress’ thrall,
  Came there for cure and this by that I prove,
  Love’s fire heats water, water cools not love.


THE END



ALL’S WELL THAT ENDS WELL



Contents

ACT I
Scene I. Rossillon. A room in the Countess’s palace.
Scene II. Paris. A room in the King’s palace.
Scene III. Rossillon. A Room in the Palace.


ACT II
Scene I. Paris. A room in the King’s palace.
Scene II. Rossillon. A room in the Countess’s palace.
Scene III. Paris. The King’s palace.
Scene IV. Paris. The King’s palace.
Scene V. Another room in the same.


ACT III
Scene I. Florence. A room in the Duke’s palace.
Scene II. Rossillon. A room in the Countess’s palace.
Scene III. Florence. Before the Duke’s palace.
Scene IV. Rossillon. A room in the Countess’s palace.
Scene V. Without the walls of Florence.
Scene VI. Camp before Florence.
Scene VII. Florence. A room in the Widow’s house.


ACT IV
Scene I. Without the Florentine camp.
Scene II. Florence. A room in the Widow’s house.
Scene III. The Florentine camp.
Scene IV. Florence. A room in the Widow’s house.
Scene V. Rossillon. A room in the Countess’s palace.


ACT V
Scene I. Marseilles. A street.
Scene II. Rossillon. The inner court of the Countess’s palace.
Scene III. The same. A room in the Countess’s palace.


 Dramatis Personæ

KING OF FRANCE.
THE DUKE OF FLORENCE.
BERTRAM, Count of Rossillon.
LAFEW, an old Lord.
PAROLLES, a follower of Bertram.
Several young French Lords, that serve with Bertram in the Florentine
War.
RYNALDO, servant to the Countess of Rossillon.
Clown, servant to the Countess of Rossillon.
A Page, servant to the Countess of Rossillon.
COUNTESS OF ROSSILLON, mother to Bertram.
HELENA, a Gentlewoman protected by the Countess.
An old WIDOW of Florence.
DIANA, daughter to the Widow.
VIOLENTA, neighbour and friend to the Widow.
MARIANA, neighbour and friend to the Widow.

Lords attending on the KING; Officers; Soldiers, &c., French and
Florentine.

SCENE: Partly in France, and partly in Tuscany.


ACT I

SCENE I. Rossillon. A room in the Countess’s palace.

 Enter Bertram, the Countess of Rossillon, Helena, and Lafew, all in
 black.

COUNTESS.
In delivering my son from me, I bury a second husband.

BERTRAM.
And I in going, madam, weep o’er my father’s death anew; but I must
attend his majesty’s command, to whom I am now in ward, evermore in
subjection.

LAFEW.
You shall find of the king a husband, madam; you, sir, a father. He
that so generally is at all times good, must of necessity hold his
virtue to you, whose worthiness would stir it up where it wanted,
rather than lack it where there is such abundance.

COUNTESS.
What hope is there of his majesty’s amendment?

LAFEW.
He hath abandon’d his physicians, madam; under whose practices he hath
persecuted time with hope, and finds no other advantage in the process
but only the losing of hope by time.

COUNTESS.
This young gentlewoman had a father—O that “had!”, how sad a passage
’tis!—whose skill was almost as great as his honesty; had it stretch’d
so far, would have made nature immortal, and death should have play for
lack of work. Would for the king’s sake he were living! I think it
would be the death of the king’s disease.

LAFEW.
How called you the man you speak of, madam?

COUNTESS.
He was famous, sir, in his profession, and it was his great right to be
so: Gerard de Narbon.

LAFEW.
He was excellent indeed, madam; the king very lately spoke of him
admiringly, and mourningly; he was skilful enough to have liv’d still,
if knowledge could be set up against mortality.

BERTRAM.
What is it, my good lord, the king languishes of?

LAFEW.
A fistula, my lord.

BERTRAM.
I heard not of it before.

LAFEW.
I would it were not notorious. Was this gentlewoman the daughter of
Gerard de Narbon?

COUNTESS.
His sole child, my lord, and bequeathed to my overlooking. I have those
hopes of her good that her education promises her dispositions she
inherits, which makes fair gifts fairer; for where an unclean mind
carries virtuous qualities, there commendations go with pity, they are
virtues and traitors too. In her they are the better for their
simpleness; she derives her honesty, and achieves her goodness.

LAFEW.
Your commendations, madam, get from her tears.

COUNTESS.
’Tis the best brine a maiden can season her praise in. The remembrance
of her father never approaches her heart but the tyranny of her sorrows
takes all livelihood from her cheek. No more of this, Helena; go to, no
more, lest it be rather thought you affect a sorrow than to have.

HELENA.
I do affect a sorrow indeed, but I have it too.

LAFEW.
Moderate lamentation is the right of the dead; excessive grief the
enemy to the living.

COUNTESS.
If the living be enemy to the grief, the excess makes it soon mortal.

BERTRAM.
Madam, I desire your holy wishes.

LAFEW.
How understand we that?

COUNTESS.
Be thou blest, Bertram, and succeed thy father
In manners, as in shape! Thy blood and virtue
Contend for empire in thee, and thy goodness
Share with thy birthright! Love all, trust a few,
Do wrong to none. Be able for thine enemy
Rather in power than use; and keep thy friend
Under thy own life’s key. Be check’d for silence,
But never tax’d for speech. What heaven more will,
That thee may furnish and my prayers pluck down,
Fall on thy head! Farewell. My lord,
’Tis an unseason’d courtier; good my lord,
Advise him.

LAFEW.
He cannot want the best
That shall attend his love.

COUNTESS.
Heaven bless him! Farewell, Bertram.

 [_Exit Countess._]

BERTRAM.
The best wishes that can be forg’d in your thoughts be servants to you!
[_To Helena._] Be comfortable to my mother, your mistress, and make
much of her.

LAFEW.
Farewell, pretty lady, you must hold the credit of your father.

 [_Exeunt Bertram and Lafew._]

HELENA.
O, were that all! I think not on my father,
And these great tears grace his remembrance more
Than those I shed for him. What was he like?
I have forgot him; my imagination
Carries no favour in’t but Bertram’s.
I am undone: there is no living, none,
If Bertram be away. ’Twere all one
That I should love a bright particular star,
And think to wed it, he is so above me.
In his bright radiance and collateral light
Must I be comforted, not in his sphere.
Th’ambition in my love thus plagues itself:
The hind that would be mated by the lion
Must die for love. ’Twas pretty, though a plague,
To see him every hour; to sit and draw
His arched brows, his hawking eye, his curls,
In our heart’s table,—heart too capable
Of every line and trick of his sweet favour.
But now he’s gone, and my idolatrous fancy
Must sanctify his relics. Who comes here?

 Enter Parolles.

One that goes with him: I love him for his sake,
And yet I know him a notorious liar,
Think him a great way fool, solely a coward;
Yet these fix’d evils sit so fit in him
That they take place when virtue’s steely bones
Looks bleak i’ th’ cold wind: withal, full oft we see
Cold wisdom waiting on superfluous folly.

PAROLLES.
Save you, fair queen!

HELENA.
And you, monarch!

PAROLLES.
No.

HELENA.
And no.

PAROLLES.
Are you meditating on virginity?

HELENA.
Ay. You have some stain of soldier in you; let me ask you a question.
Man is enemy to virginity; how may we barricado it against him?

PAROLLES.
Keep him out.

HELENA.
But he assails; and our virginity, though valiant, in the defence, yet
is weak. Unfold to us some warlike resistance.

PAROLLES.
There is none. Man setting down before you will undermine you and blow
you up.

HELENA.
Bless our poor virginity from underminers and blowers-up! Is there no
military policy how virgins might blow up men?

PAROLLES.
Virginity being blown down, man will quicklier be blown up; marry, in
blowing him down again, with the breach yourselves made, you lose your
city. It is not politic in the commonwealth of nature to preserve
virginity. Loss of virginity is rational increase, and there was never
virgin got till virginity was first lost. That you were made of is
metal to make virgins. Virginity, by being once lost, may be ten times
found; by being ever kept, it is ever lost. ’Tis too cold a companion.
Away with it!

HELENA.
I will stand for’t a little, though therefore I die a virgin.

PAROLLES.
There’s little can be said in’t; ’tis against the rule of nature. To
speak on the part of virginity is to accuse your mothers; which is most
infallible disobedience. He that hangs himself is a virgin: virginity
murders itself, and should be buried in highways out of all sanctified
limit, as a desperate offendress against nature. Virginity breeds
mites, much like a cheese; consumes itself to the very paring, and so
dies with feeding his own stomach. Besides, virginity is peevish,
proud, idle, made of self-love, which is the most inhibited sin in the
canon. Keep it not; you cannot choose but lose by’t. Out with’t! Within
the year it will make itself two, which is a goodly increase, and the
principal itself not much the worse. Away with it!

HELENA.
How might one do, sir, to lose it to her own liking?

PAROLLES.
Let me see. Marry, ill, to like him that ne’er it likes. ’Tis a
commodity will lose the gloss with lying; the longer kept, the less
worth. Off with’t while ’tis vendible; answer the time of request.
Virginity, like an old courtier, wears her cap out of fashion, richly
suited, but unsuitable, just like the brooch and the toothpick, which
wear not now. Your date is better in your pie and your porridge than in
your cheek. And your virginity, your old virginity, is like one of our
French wither’d pears; it looks ill, it eats drily; marry, ’tis a
wither’d pear; it was formerly better; marry, yet ’tis a wither’d pear.
Will you anything with it?

HELENA.
Not my virginity yet.
There shall your master have a thousand loves,
A mother, and a mistress, and a friend,
A phoenix, captain, and an enemy,
A guide, a goddess, and a sovereign,
A counsellor, a traitress, and a dear:
His humble ambition, proud humility,
His jarring concord, and his discord dulcet,
His faith, his sweet disaster; with a world
Of pretty, fond, adoptious christendoms
That blinking Cupid gossips. Now shall he—
I know not what he shall. God send him well!
The court’s a learning-place; and he is one.

PAROLLES.
What one, i’ faith?

HELENA.
That I wish well. ’Tis pity—

PAROLLES.
What’s pity?

HELENA.
That wishing well had not a body in’t
Which might be felt, that we, the poorer born,
Whose baser stars do shut us up in wishes,
Might with effects of them follow our friends,
And show what we alone must think, which never
Returns us thanks.

 Enter a Page.

PAGE.
Monsieur Parolles, my lord calls for you.

 [_Exit Page._]

PAROLLES.
Little Helen, farewell. If I can remember thee, I will think of thee at
court.

HELENA.
Monsieur Parolles, you were born under a charitable star.

PAROLLES.
Under Mars, I.

HELENA.
I especially think, under Mars.

PAROLLES.
Why under Mars?

HELENA.
The wars hath so kept you under, that you must needs be born under
Mars.

PAROLLES.
When he was predominant.

HELENA.
When he was retrograde, I think rather.

PAROLLES.
Why think you so?

HELENA.
You go so much backward when you fight.

PAROLLES.
That’s for advantage.

HELENA.
So is running away, when fear proposes the safety: but the composition
that your valour and fear makes in you is a virtue of a good wing, and
I like the wear well.

PAROLLES.
I am so full of business I cannot answer thee acutely. I will return
perfect courtier; in the which my instruction shall serve to naturalize
thee, so thou wilt be capable of a courtier’s counsel, and understand
what advice shall thrust upon thee; else thou diest in thine
unthankfulness, and thine ignorance makes thee away. Farewell. When
thou hast leisure, say thy prayers; when thou hast none, remember thy
friends. Get thee a good husband, and use him as he uses thee. So,
farewell.

 [_Exit._]

HELENA.
Our remedies oft in ourselves do lie,
Which we ascribe to heaven: the fated sky
Gives us free scope; only doth backward pull
Our slow designs when we ourselves are dull.
What power is it which mounts my love so high,
That makes me see, and cannot feed mine eye?
The mightiest space in fortune nature brings
To join like likes, and kiss like native things.
Impossible be strange attempts to those
That weigh their pains in sense, and do suppose
What hath been cannot be. Who ever strove
To show her merit that did miss her love?
The king’s disease,—my project may deceive me,
But my intents are fix’d, and will not leave me.

 [_Exit._]

SCENE II. Paris. A room in the King’s palace.

 Flourish of cornets. Enter the King of France, with letters; Lords and
 others attending.

KING.
The Florentines and Senoys are by th’ ears;
Have fought with equal fortune, and continue
A braving war.

FIRST LORD.
So ’tis reported, sir.

KING.
Nay, ’tis most credible, we here receive it,
A certainty, vouch’d from our cousin Austria,
With caution, that the Florentine will move us
For speedy aid; wherein our dearest friend
Prejudicates the business, and would seem
To have us make denial.

FIRST LORD.
His love and wisdom,
Approv’d so to your majesty, may plead
For amplest credence.

KING.
He hath arm’d our answer,
And Florence is denied before he comes:
Yet, for our gentlemen that mean to see
The Tuscan service, freely have they leave
To stand on either part.

SECOND LORD.
It well may serve
A nursery to our gentry, who are sick
For breathing and exploit.

KING.
What’s he comes here?

 Enter Bertram, Lafew and Parolles.

FIRST LORD.
It is the Count Rossillon, my good lord,
Young Bertram.

KING.
Youth, thou bear’st thy father’s face;
Frank nature, rather curious than in haste,
Hath well compos’d thee. Thy father’s moral parts
Mayst thou inherit too! Welcome to Paris.

BERTRAM.
My thanks and duty are your majesty’s.

KING.
I would I had that corporal soundness now,
As when thy father and myself in friendship
First tried our soldiership. He did look far
Into the service of the time, and was
Discipled of the bravest. He lasted long,
But on us both did haggish age steal on,
And wore us out of act. It much repairs me
To talk of your good father; in his youth
He had the wit which I can well observe
Today in our young lords; but they may jest
Till their own scorn return to them unnoted
Ere they can hide their levity in honour
So like a courtier, contempt nor bitterness
Were in his pride or sharpness; if they were,
His equal had awak’d them, and his honour,
Clock to itself, knew the true minute when
Exception bid him speak, and at this time
His tongue obey’d his hand. Who were below him
He us’d as creatures of another place,
And bow’d his eminent top to their low ranks,
Making them proud of his humility,
In their poor praise he humbled. Such a man
Might be a copy to these younger times;
Which, followed well, would demonstrate them now
But goers backward.

BERTRAM.
His good remembrance, sir,
Lies richer in your thoughts than on his tomb;
So in approof lives not his epitaph
As in your royal speech.

KING.
Would I were with him! He would always say,—
Methinks I hear him now; his plausive words
He scatter’d not in ears, but grafted them
To grow there and to bear,—“Let me not live,”
This his good melancholy oft began
On the catastrophe and heel of pastime,
When it was out,—“Let me not live” quoth he,
“After my flame lacks oil, to be the snuff
Of younger spirits, whose apprehensive senses
All but new things disdain; whose judgments are
Mere fathers of their garments; whose constancies
Expire before their fashions.” This he wish’d.
I, after him, do after him wish too,
Since I nor wax nor honey can bring home,
I quickly were dissolved from my hive
To give some labourers room.

SECOND LORD.
You’re lov’d, sir;
They that least lend it you shall lack you first.

KING.
I fill a place, I know’t. How long is’t, Count,
Since the physician at your father’s died?
He was much fam’d.

BERTRAM.
Some six months since, my lord.

KING.
If he were living, I would try him yet;—
Lend me an arm;—the rest have worn me out
With several applications; nature and sickness
Debate it at their leisure. Welcome, Count;
My son’s no dearer.

BERTRAM.
Thank your majesty.

 [_Exeunt. Flourish._]

SCENE III. Rossillon. A Room in the Palace.

 Enter Countess, Steward and Clown.

COUNTESS.
I will now hear. What say you of this gentlewoman?

STEWARD.
Madam, the care I have had to even your content, I wish might be found
in the calendar of my past endeavours; for then we wound our modesty,
and make foul the clearness of our deservings, when of ourselves we
publish them.

COUNTESS.
What does this knave here? Get you gone, sirrah. The complaints I have
heard of you I do not all believe; ’tis my slowness that I do not; for
I know you lack not folly to commit them, and have ability enough to
make such knaveries yours.

CLOWN.
’Tis not unknown to you, madam, I am a poor fellow.

COUNTESS.
Well, sir.

CLOWN.
No, madam, ’tis not so well that I am poor, though many of the rich are
damned; but if I may have your ladyship’s good will to go to the world,
Isbel the woman and I will do as we may.

COUNTESS.
Wilt thou needs be a beggar?

CLOWN.
I do beg your good will in this case.

COUNTESS.
In what case?

CLOWN.
In Isbel’s case and mine own. Service is no heritage, and I think I
shall never have the blessing of God till I have issue of my body; for
they say barnes are blessings.

COUNTESS.
Tell me thy reason why thou wilt marry.

CLOWN.
My poor body, madam, requires it; I am driven on by the flesh, and he
must needs go that the devil drives.

COUNTESS.
Is this all your worship’s reason?

CLOWN.
Faith, madam, I have other holy reasons, such as they are.

COUNTESS.
May the world know them?

CLOWN.
I have been, madam, a wicked creature, as you and all flesh and blood
are; and indeed I do marry that I may repent.

COUNTESS.
Thy marriage, sooner than thy wickedness.

CLOWN.
I am out of friends, madam, and I hope to have friends for my wife’s
sake.

COUNTESS.
Such friends are thine enemies, knave.

CLOWN.
Y’are shallow, madam, in great friends; for the knaves come to do that
for me which I am a-weary of. He that ears my land spares my team, and
gives me leave to in the crop: if I be his cuckold, he’s my drudge. He
that comforts my wife is the cherisher of my flesh and blood; he that
cherishes my flesh and blood loves my flesh and blood; he that loves my
flesh and blood is my friend; ergo, he that kisses my wife is my
friend. If men could be contented to be what they are, there were no
fear in marriage; for young Charbon the puritan and old Poysam the
papist, howsome’er their hearts are sever’d in religion, their heads
are both one; they may jowl horns together like any deer i’ the herd.

COUNTESS.
Wilt thou ever be a foul-mouth’d and calumnious knave?

CLOWN.
A prophet I, madam; and I speak the truth the next way:
   _For I the ballad will repeat,
   Which men full true shall find;
   Your marriage comes by destiny,
   Your cuckoo sings by kind._

COUNTESS.
Get you gone, sir; I’ll talk with you more anon.

STEWARD.
May it please you, madam, that he bid Helen come to you; of her I am to
speak.

COUNTESS.
Sirrah, tell my gentlewoman I would speak with her; Helen I mean.

CLOWN.
[_Sings._]
_   Was this fair face the cause, quoth she,
     Why the Grecians sacked Troy?
   Fond done, done fond,
      Was this King Priam’s joy?
   With that she sighed as she stood,
   With that she sighed as she stood,
      And gave this sentence then:
   Among nine bad if one be good,
   Among nine bad if one be good,
     There’s yet one good in ten._

COUNTESS.
What, one good in ten? You corrupt the song, sirrah.

CLOWN.
One good woman in ten, madam, which is a purifying o’ the song. Would
God would serve the world so all the year! We’d find no fault with the
tithe-woman, if I were the parson. One in ten, quoth ’a! And we might
have a good woman born but or every blazing star, or at an earthquake,
’twould mend the lottery well; a man may draw his heart out ere he
pluck one.

COUNTESS.
You’ll be gone, sir knave, and do as I command you!

CLOWN.
That man should be at woman’s command, and yet no hurt done! Though
honesty be no puritan, yet it will do no hurt; it will wear the
surplice of humility over the black gown of a big heart. I am going,
forsooth; the business is for Helen to come hither.

 [_Exit._]

COUNTESS.
Well, now.

STEWARD.
I know, madam, you love your gentlewoman entirely.

COUNTESS.
Faith I do. Her father bequeath’d her to me, and she herself, without
other advantage, may lawfully make title to as much love as she finds;
there is more owing her than is paid, and more shall be paid her than
she’ll demand.

STEWARD.
Madam, I was very late more near her than I think she wish’d me; alone
she was, and did communicate to herself her own words to her own ears;
she thought, I dare vow for her, they touch’d not any stranger sense.
Her matter was, she loved your son. Fortune, she said, was no goddess,
that had put such difference betwixt their two estates; Love no god,
that would not extend his might only where qualities were level; Diana
no queen of virgins, that would suffer her poor knight surpris’d,
without rescue in the first assault or ransom afterward. This she
deliver’d in the most bitter touch of sorrow that e’er I heard virgin
exclaim in, which I held my duty speedily to acquaint you withal;
sithence, in the loss that may happen, it concerns you something to
know it.

COUNTESS.
You have discharg’d this honestly; keep it to yourself; many
likelihoods inform’d me of this before, which hung so tottering in the
balance that I could neither believe nor misdoubt. Pray you leave me;
stall this in your bosom; and I thank you for your honest care. I will
speak with you further anon.

 [_Exit Steward._]

 Enter Helena.

Even so it was with me when I was young;
If ever we are nature’s, these are ours; this thorn
Doth to our rose of youth rightly belong;
Our blood to us, this to our blood is born;
It is the show and seal of nature’s truth,
Where love’s strong passion is impress’d in youth.
By our remembrances of days foregone,
Such were our faults, or then we thought them none.
Her eye is sick on’t; I observe her now.

HELENA.
What is your pleasure, madam?

COUNTESS.
You know, Helen,
I am a mother to you.

HELENA.
Mine honourable mistress.

COUNTESS.
Nay, a mother.
Why not a mother? When I said a mother,
Methought you saw a serpent. What’s in mother,
That you start at it? I say I am your mother,
And put you in the catalogue of those
That were enwombed mine. ’Tis often seen
Adoption strives with nature, and choice breeds
A native slip to us from foreign seeds.
You ne’er oppress’d me with a mother’s groan,
Yet I express to you a mother’s care.
God’s mercy, maiden! does it curd thy blood
To say I am thy mother? What’s the matter,
That this distempered messenger of wet,
The many-colour’d Iris, rounds thine eye?
—Why, that you are my daughter?

HELENA.
That I am not.

COUNTESS.
I say, I am your mother.

HELENA.
Pardon, madam;
The Count Rossillon cannot be my brother.
I am from humble, he from honoured name;
No note upon my parents, his all noble,
My master, my dear lord he is; and I
His servant live, and will his vassal die.
He must not be my brother.

COUNTESS.
Nor I your mother?

HELENA.
You are my mother, madam; would you were—
So that my lord your son were not my brother,—
Indeed my mother! or were you both our mothers,
I care no more for than I do for heaven,
So I were not his sister. Can’t no other,
But, I your daughter, he must be my brother?

COUNTESS.
Yes, Helen, you might be my daughter-in-law.
God shield you mean it not! daughter and mother
So strive upon your pulse. What! pale again?
My fear hath catch’d your fondness; now I see
The mystery of your loneliness, and find
Your salt tears’ head. Now to all sense ’tis gross
You love my son; invention is asham’d,
Against the proclamation of thy passion
To say thou dost not. Therefore tell me true;
But tell me then, ’tis so; for, look, thy cheeks
Confess it, t’one to th’other; and thine eyes
See it so grossly shown in thy behaviours,
That in their kind they speak it; only sin
And hellish obstinacy tie thy tongue,
That truth should be suspected. Speak, is’t so?
If it be so, you have wound a goodly clew;
If it be not, forswear’t: howe’er, I charge thee,
As heaven shall work in me for thine avail,
To tell me truly.

HELENA.
Good madam, pardon me.

COUNTESS.
Do you love my son?

HELENA.
Your pardon, noble mistress.

COUNTESS.
Love you my son?

HELENA.
Do not you love him, madam?

COUNTESS.
Go not about; my love hath in’t a bond
Whereof the world takes note. Come, come, disclose
The state of your affection, for your passions
Have to the full appeach’d.

HELENA.
Then I confess,
Here on my knee, before high heaven and you,
That before you, and next unto high heaven,
I love your son.
My friends were poor, but honest; so’s my love.
Be not offended; for it hurts not him
That he is lov’d of me; I follow him not
By any token of presumptuous suit,
Nor would I have him till I do deserve him;
Yet never know how that desert should be.
I know I love in vain, strive against hope;
Yet in this captious and inteemable sieve
I still pour in the waters of my love
And lack not to lose still. Thus, Indian-like,
Religious in mine error, I adore
The sun that looks upon his worshipper,
But knows of him no more. My dearest madam,
Let not your hate encounter with my love,
For loving where you do; but if yourself,
Whose aged honour cites a virtuous youth,
Did ever, in so true a flame of liking,
Wish chastely, and love dearly, that your Dian
Was both herself and love; O then, give pity
To her whose state is such that cannot choose
But lend and give where she is sure to lose;
That seeks not to find that her search implies,
But riddle-like, lives sweetly where she dies!

COUNTESS.
Had you not lately an intent,—speak truly,—
To go to Paris?

HELENA.
Madam, I had.

COUNTESS.
Wherefore? tell true.

HELENA.
I will tell truth; by grace itself I swear.
You know my father left me some prescriptions
Of rare and prov’d effects, such as his reading
And manifest experience had collected
For general sovereignty; and that he will’d me
In heedfull’st reservation to bestow them,
As notes whose faculties inclusive were
More than they were in note. Amongst the rest
There is a remedy, approv’d, set down,
To cure the desperate languishings whereof
The king is render’d lost.

COUNTESS.
This was your motive
For Paris, was it? Speak.

HELENA.
My lord your son made me to think of this;
Else Paris, and the medicine, and the king,
Had from the conversation of my thoughts
Haply been absent then.

COUNTESS.
But think you, Helen,
If you should tender your supposed aid,
He would receive it? He and his physicians
Are of a mind; he, that they cannot help him;
They, that they cannot help. How shall they credit
A poor unlearned virgin, when the schools,
Embowell’d of their doctrine, have let off
The danger to itself?

HELENA.
There’s something in’t
More than my father’s skill, which was the great’st
Of his profession, that his good receipt
Shall for my legacy be sanctified
By th’ luckiest stars in heaven; and would your honour
But give me leave to try success, I’d venture
The well-lost life of mine on his grace’s cure.
By such a day, an hour.

COUNTESS.
Dost thou believe’t?

HELENA.
Ay, madam, knowingly.

COUNTESS.
Why, Helen, thou shalt have my leave and love,
Means and attendants, and my loving greetings
To those of mine in court. I’ll stay at home,
And pray God’s blessing into thy attempt.
Be gone tomorrow; and be sure of this,
What I can help thee to, thou shalt not miss.

 [_Exeunt._]


ACT II.

SCENE I. Paris. A room in the King’s palace.

 Flourish. Enter the King with young Lords taking leave for the
 Florentine war; Bertram, Parolles and Attendants.

KING.
Farewell, young lords; these warlike principles
Do not throw from you; and you, my lords, farewell;
Share the advice betwixt you; if both gain all,
The gift doth stretch itself as ’tis receiv’d,
And is enough for both.

FIRST LORD.
’Tis our hope, sir,
After well-ent’red soldiers, to return
And find your grace in health.

KING.
No, no, it cannot be; and yet my heart
Will not confess he owes the malady
That doth my life besiege. Farewell, young lords.
Whether I live or die, be you the sons
Of worthy Frenchmen; let higher Italy,—
Those bated that inherit but the fall
Of the last monarchy—see that you come
Not to woo honour, but to wed it, when
The bravest questant shrinks: find what you seek,
That fame may cry you loud. I say farewell.

SECOND LORD.
Health, at your bidding serve your majesty!

KING.
Those girls of Italy, take heed of them;
They say our French lack language to deny
If they demand; beware of being captives
Before you serve.

BOTH.
Our hearts receive your warnings.

KING.
Farewell.—Come hither to me.

 [_The King retires to a couch._]

FIRST LORD.
O my sweet lord, that you will stay behind us!

PAROLLES.
’Tis not his fault; the spark.

SECOND LORD.
O, ’tis brave wars!

PAROLLES.
Most admirable! I have seen those wars.

BERTRAM.
I am commanded here, and kept a coil with,
“Too young”, and “the next year” and “’tis too early”.

PAROLLES.
An thy mind stand to’t, boy, steal away bravely.

BERTRAM.
I shall stay here the forehorse to a smock,
Creaking my shoes on the plain masonry,
Till honour be bought up, and no sword worn
But one to dance with. By heaven, I’ll steal away.

FIRST LORD.
There’s honour in the theft.

PAROLLES.
Commit it, count.

SECOND LORD.
I am your accessary; and so farewell.

BERTRAM.
I grow to you, and our parting is a tortur’d body.

FIRST LORD.
Farewell, captain.

SECOND LORD.
Sweet Monsieur Parolles!

PAROLLES.
Noble heroes, my sword and yours are kin. Good sparks and lustrous, a
word, good metals. You shall find in the regiment of the Spinii one
Captain Spurio, with his cicatrice, an emblem of war, here on his
sinister cheek; it was this very sword entrench’d it. Say to him I
live; and observe his reports for me.

FIRST LORD.
We shall, noble captain.

PAROLLES.
Mars dote on you for his novices!

 [_Exeunt Lords._]

What will ye do?

BERTRAM.
Stay the king.

PAROLLES.
Use a more spacious ceremony to the noble lords; you have restrain’d
yourself within the list of too cold an adieu. Be more expressive to
them; for they wear themselves in the cap of the time; there do muster
true gait; eat, speak, and move, under the influence of the most
receiv’d star; and though the devil lead the measure, such are to be
followed. After them, and take a more dilated farewell.

BERTRAM.
And I will do so.

PAROLLES.
Worthy fellows, and like to prove most sinewy sword-men.

 [_Exeunt Bertram and Parolles._]

 Enter Lafew.

LAFEW.
Pardon, my lord [_kneeling_], for me and for my tidings.

KING.
I’ll fee thee to stand up.

LAFEW.
Then here’s a man stands that has brought his pardon.
I would you had kneel’d, my lord, to ask me mercy,
And that at my bidding you could so stand up.

KING.
I would I had; so I had broke thy pate,
And ask’d thee mercy for’t.

LAFEW.
Good faith, across;
But, my good lord, ’tis thus: will you be cur’d
Of your infirmity?

KING.
No.

LAFEW.
O, will you eat
No grapes, my royal fox? Yes, but you will
My noble grapes, and if my royal fox
Could reach them. I have seen a medicine
That’s able to breathe life into a stone,
Quicken a rock, and make you dance canary
With sprightly fire and motion; whose simple touch
Is powerful to araise King Pippen, nay,
To give great Charlemain a pen in’s hand
And write to her a love-line.

KING.
What ‘her’ is this?

LAFEW.
Why, doctor ‘she’! My lord, there’s one arriv’d,
If you will see her. Now, by my faith and honour,
If seriously I may convey my thoughts
In this my light deliverance, I have spoke
With one that in her sex, her years, profession,
Wisdom, and constancy, hath amaz’d me more
Than I dare blame my weakness. Will you see her,
For that is her demand, and know her business?
That done, laugh well at me.

KING.
Now, good Lafew,
Bring in the admiration; that we with thee
May spend our wonder too, or take off thine
By wond’ring how thou took’st it.

LAFEW.
Nay, I’ll fit you,
And not be all day neither.

 [_Exit Lafew._]

KING.
Thus he his special nothing ever prologues.

 Enter Lafew with Helena.

LAFEW.
Nay, come your ways.

KING.
This haste hath wings indeed.

LAFEW.
Nay, come your ways.
This is his majesty, say your mind to him.
A traitor you do look like, but such traitors
His majesty seldom fears; I am Cressid’s uncle,
That dare leave two together. Fare you well.

 [_Exit._]

KING.
Now, fair one, does your business follow us?

HELENA.
Ay, my good lord.
Gerard de Narbon was my father,
In what he did profess, well found.

KING.
I knew him.

HELENA.
The rather will I spare my praises towards him.
Knowing him is enough. On his bed of death
Many receipts he gave me; chiefly one,
Which, as the dearest issue of his practice,
And of his old experience the only darling,
He bade me store up as a triple eye,
Safer than mine own two; more dear I have so,
And hearing your high majesty is touch’d
With that malignant cause, wherein the honour
Of my dear father’s gift stands chief in power,
I come to tender it, and my appliance,
With all bound humbleness.

KING.
We thank you, maiden,
But may not be so credulous of cure,
When our most learned doctors leave us, and
The congregated college have concluded
That labouring art can never ransom nature
From her inaidable estate. I say we must not
So stain our judgment, or corrupt our hope,
To prostitute our past-cure malady
To empirics, or to dissever so
Our great self and our credit, to esteem
A senseless help, when help past sense we deem.

HELENA.
My duty then shall pay me for my pains.
I will no more enforce mine office on you,
Humbly entreating from your royal thoughts
A modest one to bear me back again.

KING.
I cannot give thee less, to be call’d grateful.
Thou thought’st to help me; and such thanks I give
As one near death to those that wish him live.
But what at full I know, thou know’st no part;
I knowing all my peril, thou no art.

HELENA.
What I can do can do no hurt to try,
Since you set up your rest ’gainst remedy.
He that of greatest works is finisher
Oft does them by the weakest minister.
So holy writ in babes hath judgment shown,
When judges have been babes. Great floods have flown
From simple sources, and great seas have dried
When miracles have by the great’st been denied.
Oft expectation fails, and most oft there
Where most it promises; and oft it hits
Where hope is coldest, and despair most fits.

KING.
I must not hear thee. Fare thee well, kind maid.
Thy pains, not us’d, must by thyself be paid;
Proffers, not took, reap thanks for their reward.

HELENA.
Inspired merit so by breath is barr’d.
It is not so with Him that all things knows
As ’tis with us that square our guess by shows;
But most it is presumption in us when
The help of heaven we count the act of men.
Dear sir, to my endeavours give consent;
Of heaven, not me, make an experiment.
I am not an impostor, that proclaim
Myself against the level of mine aim,
But know I think, and think I know most sure,
My art is not past power nor you past cure.

KING.
Art thou so confident? Within what space
Hop’st thou my cure?

HELENA.
The greatest grace lending grace.
Ere twice the horses of the sun shall bring
Their fiery torcher his diurnal ring,
Ere twice in murk and occidental damp
Moist Hesperus hath quench’d her sleepy lamp;
Or four and twenty times the pilot’s glass
Hath told the thievish minutes how they pass;
What is infirm from your sound parts shall fly,
Health shall live free, and sickness freely die.

KING.
Upon thy certainty and confidence
What dar’st thou venture?

HELENA.
Tax of impudence,
A strumpet’s boldness, a divulged shame,
Traduc’d by odious ballads; my maiden’s name
Sear’d otherwise; nay worse of worst extended
With vilest torture, let my life be ended.

KING.
Methinks in thee some blessed spirit doth speak
His powerful sound within an organ weak;
And what impossibility would slay
In common sense, sense saves another way.
Thy life is dear, for all that life can rate
Worth name of life in thee hath estimate:
Youth, beauty, wisdom, courage, all
That happiness and prime can happy call.
Thou this to hazard needs must intimate
Skill infinite, or monstrous desperate.
Sweet practiser, thy physic I will try,
That ministers thine own death if I die.

HELENA.
If I break time, or flinch in property
Of what I spoke, unpitied let me die,
And well deserv’d. Not helping, death’s my fee;
But if I help, what do you promise me?

KING.
Make thy demand.

HELENA.
But will you make it even?

KING.
Ay, by my sceptre and my hopes of heaven.

HELENA.
Then shalt thou give me, with thy kingly hand
What husband in thy power I will command:
Exempted be from me the arrogance
To choose from forth the royal blood of France,
My low and humble name to propagate
With any branch or image of thy state;
But such a one, thy vassal, whom I know
Is free for me to ask, thee to bestow.

KING.
Here is my hand; the premises observ’d,
Thy will by my performance shall be serv’d;
So make the choice of thy own time, for I,
Thy resolv’d patient, on thee still rely.
More should I question thee, and more I must,
Though more to know could not be more to trust:
From whence thou cam’st, how tended on; but rest
Unquestion’d welcome, and undoubted bless’d.
Give me some help here, ho! If thou proceed
As high as word, my deed shall match thy deed.

 [_Flourish. Exeunt._]

SCENE II. Rossillon. A room in the Countess’s palace.

 Enter Countess and Clown.

COUNTESS.
Come on, sir; I shall now put you to the height of your breeding.

CLOWN.
I will show myself highly fed and lowly taught. I know my business is
but to the court.

COUNTESS.
To the court! Why, what place make you special, when you put off that
with such contempt? But to the court!

CLOWN.
Truly, madam, if God have lent a man any manners, he may easily put it
off at court: he that cannot make a leg, put off’s cap, kiss his hand,
and say nothing, has neither leg, hands, lip, nor cap; and indeed such
a fellow, to say precisely, were not for the court; but for me, I have
an answer will serve all men.

COUNTESS.
Marry, that’s a bountiful answer that fits all questions.

CLOWN.
It is like a barber’s chair, that fits all buttocks—the pin-buttock,
the quatch-buttock, the brawn-buttock, or any buttock.

COUNTESS.
Will your answer serve fit to all questions?

CLOWN.
As fit as ten groats is for the hand of an attorney, as your French
crown for your taffety punk, as Tib’s rush for Tom’s forefinger, as a
pancake for Shrove-Tuesday, a morris for May-day, as the nail to his
hole, the cuckold to his horn, as a scolding quean to a wrangling
knave, as the nun’s lip to the friar’s mouth; nay, as the pudding to
his skin.

COUNTESS.
Have you, I say, an answer of such fitness for all questions?

CLOWN.
From below your duke to beneath your constable, it will fit any
question.

COUNTESS.
It must be an answer of most monstrous size that must fit all demands.

CLOWN.
But a trifle neither, in good faith, if the learned should speak truth
of it. Here it is, and all that belongs to’t. Ask me if I am a
courtier; it shall do you no harm to learn.

COUNTESS.
To be young again, if we could: I will be a fool in question, hoping to
be the wiser by your answer. I pray you, sir, are you a courtier?

CLOWN.
O Lord, sir! There’s a simple putting off. More, more, a hundred of
them.

COUNTESS.
Sir, I am a poor friend of yours, that loves you.

CLOWN.
O Lord, sir! Thick, thick; spare not me.

COUNTESS.
I think, sir, you can eat none of this homely meat.

CLOWN.
O Lord, sir! Nay, put me to’t, I warrant you.

COUNTESS.
You were lately whipp’d, sir, as I think.

CLOWN.
O Lord, sir! Spare not me.

COUNTESS.
Do you cry ‘O Lord, sir!’ at your whipping, and ‘spare not me’? Indeed
your ‘O Lord, sir!’ is very sequent to your whipping. You would answer
very well to a whipping, if you were but bound to’t.

CLOWN.
I ne’er had worse luck in my life in my ‘O Lord, sir!’ I see things may
serve long, but not serve ever.

COUNTESS.
I play the noble housewife with the time, to entertain it so merrily
with a fool.

CLOWN.
O Lord, sir! Why, there’t serves well again.

COUNTESS.
An end, sir! To your business. Give Helen this,
And urge her to a present answer back.
Commend me to my kinsmen and my son.
This is not much.

CLOWN.
Not much commendation to them?

COUNTESS.
Not much employment for you. You understand me?

CLOWN.
Most fruitfully. I am there before my legs.

COUNTESS.
Haste you again.

 [_Exeunt severally._]

SCENE III. Paris. The King’s palace.

 Enter Bertram, Lafew and Parolles.

LAFEW.
They say miracles are past; and we have our philosophical persons to
make modern and familiar things supernatural and causeless. Hence is it
that we make trifles of terrors, ensconcing ourselves into seeming
knowledge when we should submit ourselves to an unknown fear.

PAROLLES.
Why, ’tis the rarest argument of wonder that hath shot out in our
latter times.

BERTRAM.
And so ’tis.

LAFEW.
To be relinquish’d of the artists,—

PAROLLES.
So I say; both of Galen and Paracelsus.

LAFEW.
Of all the learned and authentic fellows,—

PAROLLES.
Right; so I say.

LAFEW.
That gave him out incurable,—

PAROLLES.
Why, there ’tis; so say I too.

LAFEW.
Not to be helped.

PAROLLES.
Right; as ’twere a man assur’d of a—

LAFEW.
Uncertain life and sure death.

PAROLLES.
Just; you say well. So would I have said.

LAFEW.
I may truly say, it is a novelty to the world.

PAROLLES.
It is indeed; if you will have it in showing, you shall read it in what
do you call there?

LAFEW.
A showing of a heavenly effect in an earthly actor.

PAROLLES.
That’s it; I would have said the very same.

LAFEW.
Why, your dolphin is not lustier; fore me, I speak in respect—

PAROLLES.
Nay, ’tis strange, ’tis very strange; that is the brief and the tedious
of it; and he’s of a most facinerious spirit that will not acknowledge
it to be the—

LAFEW.
Very hand of heaven.

PAROLLES.
Ay, so I say.

LAFEW.
In a most weak—

PAROLLES.
And debile minister, great power, great transcendence, which should
indeed give us a further use to be made than alone the recov’ry of the
king, as to be—

LAFEW.
Generally thankful.

PAROLLES.
I would have said it; you say well. Here comes the king.

 Enter King, Helena and Attendants.

LAFEW.
Lustique, as the Dutchman says. I’ll like a maid the better, whilst I
have a tooth in my head. Why, he’s able to lead her a coranto.

PAROLLES.
_Mor du vinager!_ is not this Helen?

LAFEW.
Fore God, I think so.

KING.
Go, call before me all the lords in court.

 [_Exit an Attendant._]

Sit, my preserver, by thy patient’s side,
And with this healthful hand, whose banish’d sense
Thou has repeal’d, a second time receive
The confirmation of my promis’d gift,
Which but attends thy naming.

 Enter several Lords.

Fair maid, send forth thine eye. This youthful parcel
Of noble bachelors stand at my bestowing,
O’er whom both sovereign power and father’s voice
I have to use. Thy frank election make;
Thou hast power to choose, and they none to forsake.

HELENA.
To each of you one fair and virtuous mistress
Fall, when love please! Marry, to each but one!

LAFEW.
I’d give bay curtal and his furniture
My mouth no more were broken than these boys’,
And writ as little beard.

KING.
Peruse them well.
Not one of those but had a noble father.

 She addresses her to a Lord.

HELENA.
Gentlemen,
Heaven hath through me restor’d the king to health.

ALL.
We understand it, and thank heaven for you.

HELENA.
I am a simple maid, and therein wealthiest
That I protest I simply am a maid.
Please it, your majesty, I have done already.
The blushes in my cheeks thus whisper me:
“We blush that thou shouldst choose; but, be refused,
Let the white death sit on thy cheek for ever,
We’ll ne’er come there again.”

KING.
Make choice; and, see,
Who shuns thy love shuns all his love in me.

HELENA.
Now, Dian, from thy altar do I fly,
And to imperial Love, that god most high,
Do my sighs stream. [_To first Lord._] Sir, will you hear my suit?

FIRST LORD.
And grant it.

HELENA.
Thanks, sir; all the rest is mute.

LAFEW.
I had rather be in this choice than throw ames-ace for my life.

HELENA.
[_To second Lord._] The honour, sir, that flames in your fair eyes,
Before I speak, too threat’ningly replies.
Love make your fortunes twenty times above
Her that so wishes, and her humble love!

SECOND LORD.
No better, if you please.

HELENA.
My wish receive,
Which great Love grant; and so I take my leave.

LAFEW.
Do all they deny her? An they were sons of mine I’d have them whipp’d;
or I would send them to th’ Turk to make eunuchs of.

HELENA.
[_To third Lord._] Be not afraid that I your hand should take;
I’ll never do you wrong for your own sake.
Blessing upon your vows, and in your bed
Find fairer fortune, if you ever wed!

LAFEW.
These boys are boys of ice, they’ll none have her. Sure, they are
bastards to the English; the French ne’er got ’em.

HELENA.
[_To fourth Lord._] You are too young, too happy, and too good,
To make yourself a son out of my blood.

FOURTH LORD.
Fair one, I think not so.

LAFEW.
There’s one grape yet. I am sure thy father drank wine. But if thou
beest not an ass, I am a youth of fourteen; I have known thee already.

HELENA.
[_To Bertram._] I dare not say I take you, but I give
Me and my service, ever whilst I live,
Into your guiding power. This is the man.

KING.
Why, then, young Bertram, take her; she’s thy wife.

BERTRAM.
My wife, my liege! I shall beseech your highness,
In such a business give me leave to use
The help of mine own eyes.

KING.
Know’st thou not, Bertram,
What she has done for me?

BERTRAM.
Yes, my good lord,
But never hope to know why I should marry her.

KING.
Thou know’st she has rais’d me from my sickly bed.

BERTRAM.
But follows it, my lord, to bring me down
Must answer for your raising? I know her well;
She had her breeding at my father’s charge:
A poor physician’s daughter my wife! Disdain
Rather corrupt me ever!

KING.
’Tis only title thou disdain’st in her, the which
I can build up. Strange is it that our bloods,
Of colour, weight, and heat, pour’d all together,
Would quite confound distinction, yet stands off
In differences so mighty. If she be
All that is virtuous, save what thou dislik’st,
A poor physician’s daughter,—thou dislik’st—
Of virtue for the name. But do not so.
From lowest place when virtuous things proceed,
The place is dignified by the doer’s deed.
Where great additions swell’s, and virtue none,
It is a dropsied honour. Good alone
Is good without a name; vileness is so:
The property by what it is should go,
Not by the title. She is young, wise, fair;
In these to nature she’s immediate heir;
And these breed honour: that is honour’s scorn
Which challenges itself as honour’s born,
And is not like the sire. Honours thrive
When rather from our acts we them derive
Than our fore-goers. The mere word’s a slave,
Debauch’d on every tomb, on every grave
A lying trophy, and as oft is dumb
Where dust and damn’d oblivion is the tomb
Of honour’d bones indeed. What should be said?
If thou canst like this creature as a maid,
I can create the rest. Virtue and she
Is her own dower; honour and wealth from me.

BERTRAM.
I cannot love her, nor will strive to do ’t.

KING.
Thou wrong’st thyself, if thou shouldst strive to choose.

HELENA.
That you are well restor’d, my lord, I am glad.
Let the rest go.

KING.
My honour’s at the stake, which to defeat,
I must produce my power. Here, take her hand,
Proud scornful boy, unworthy this good gift,
That dost in vile misprision shackle up
My love and her desert; that canst not dream
We, poising us in her defective scale,
Shall weigh thee to the beam; that wilt not know
It is in us to plant thine honour where
We please to have it grow. Check thy contempt;
Obey our will, which travails in thy good;
Believe not thy disdain, but presently
Do thine own fortunes that obedient right
Which both thy duty owes and our power claims;
Or I will throw thee from my care for ever
Into the staggers and the careless lapse
Of youth and ignorance; both my revenge and hate
Loosing upon thee in the name of justice,
Without all terms of pity. Speak! Thine answer!

BERTRAM.
Pardon, my gracious lord; for I submit
My fancy to your eyes. When I consider
What great creation, and what dole of honour
Flies where you bid it, I find that she, which late
Was in my nobler thoughts most base, is now
The praised of the king; who, so ennobled,
Is as ’twere born so.

KING.
Take her by the hand,
And tell her she is thine; to whom I promise
A counterpoise; if not to thy estate,
A balance more replete.

BERTRAM.
I take her hand.

KING.
Good fortune and the favour of the king
Smile upon this contract; whose ceremony
Shall seem expedient on the now-born brief,
And be perform’d tonight. The solemn feast
Shall more attend upon the coming space,
Expecting absent friends. As thou lov’st her,
Thy love’s to me religious; else, does err.

 [_Exeunt King, Bertram, Helena, Lords, and Attendants._]

LAFEW.
Do you hear, monsieur? A word with you.

PAROLLES.
Your pleasure, sir.

LAFEW.
Your lord and master did well to make his recantation.

PAROLLES.
Recantation! My lord! My master!

LAFEW.
Ay. Is it not a language I speak?

PAROLLES.
A most harsh one, and not to be understood without bloody succeeding.
My master!

LAFEW.
Are you companion to the Count Rossillon?

PAROLLES.
To any count; to all counts; to what is man.

LAFEW.
To what is count’s man: count’s master is of another style.

PAROLLES.
You are too old, sir; let it satisfy you, you are too old.

LAFEW.
I must tell thee, sirrah, I write man; to which title age cannot bring
thee.

PAROLLES.
What I dare too well do, I dare not do.

LAFEW.
I did think thee, for two ordinaries, to be a pretty wise fellow; thou
didst make tolerable vent of thy travel; it might pass. Yet the scarfs
and the bannerets about thee did manifoldly dissuade me from believing
thee a vessel of too great a burden. I have now found thee; when I lose
thee again I care not. Yet art thou good for nothing but taking up, and
that thou art scarce worth.

PAROLLES.
Hadst thou not the privilege of antiquity upon thee—

LAFEW.
Do not plunge thyself too far in anger, lest thou hasten thy trial;
which if—Lord have mercy on thee for a hen! So, my good window of
lattice, fare thee well; thy casement I need not open, for I look
through thee. Give me thy hand.

PAROLLES.
My lord, you give me most egregious indignity.

LAFEW.
Ay, with all my heart; and thou art worthy of it.

PAROLLES.
I have not, my lord, deserv’d it.

LAFEW.
Yes, good faith, every dram of it; and I will not bate thee a scruple.

PAROLLES.
Well, I shall be wiser.

LAFEW.
Ev’n as soon as thou canst, for thou hast to pull at a smack o’ th’
contrary. If ever thou beest bound in thy scarf and beaten, thou shalt
find what it is to be proud of thy bondage. I have a desire to hold my
acquaintance with thee, or rather my knowledge, that I may say in the
default, “He is a man I know.”

PAROLLES.
My lord, you do me most insupportable vexation.

LAFEW.
I would it were hell-pains for thy sake, and my poor doing eternal; for
doing I am past, as I will by thee, in what motion age will give me
leave.

 [_Exit._]

PAROLLES.
Well, thou hast a son shall take this disgrace off me; scurvy, old,
filthy, scurvy lord! Well, I must be patient; there is no fettering of
authority. I’ll beat him, by my life, if I can meet him with any
convenience, an he were double and double a lord. I’ll have no more
pity of his age than I would have of—I’ll beat him, and if I could but
meet him again.

 Enter Lafew.

LAFEW.
Sirrah, your lord and master’s married; there’s news for you; you have
a new mistress.

PAROLLES.
I most unfeignedly beseech your lordship to make some reservation of
your wrongs. He is my good lord; whom I serve above is my master.

LAFEW.
Who? God?

PAROLLES.
Ay, sir.

LAFEW.
The devil it is that’s thy master. Why dost thou garter up thy arms o’
this fashion? Dost make hose of thy sleeves? Do other servants so? Thou
wert best set thy lower part where thy nose stands. By mine honour, if
I were but two hours younger, I’d beat thee. Methink’st thou art a
general offence, and every man should beat thee. I think thou wast
created for men to breathe themselves upon thee.

PAROLLES.
This is hard and undeserved measure, my lord.

LAFEW.
Go to, sir; you were beaten in Italy for picking a kernel out of a
pomegranate; you are a vagabond, and no true traveller. You are more
saucy with lords and honourable personages than the commission of your
birth and virtue gives you heraldry. You are not worth another word,
else I’d call you knave. I leave you.

 [_Exit._]

 Enter Bertram.

PAROLLES.
Good, very good, it is so then. Good, very good; let it be conceal’d
awhile.

BERTRAM.
Undone, and forfeited to cares for ever!

PAROLLES.
What’s the matter, sweetheart?

BERTRAM.
Although before the solemn priest I have sworn,
I will not bed her.

PAROLLES.
What, what, sweetheart?

BERTRAM.
O my Parolles, they have married me!
I’ll to the Tuscan wars, and never bed her.

PAROLLES.
France is a dog-hole, and it no more merits
The tread of a man’s foot: to the wars!

BERTRAM.
There’s letters from my mother; what th’ import is
I know not yet.

PAROLLES.
Ay, that would be known. To th’ wars, my boy, to th’ wars!
He wears his honour in a box unseen
That hugs his kicky-wicky here at home,
Spending his manly marrow in her arms,
Which should sustain the bound and high curvet
Of Mars’s fiery steed. To other regions!
France is a stable; we that dwell in’t, jades,
Therefore, to th’ war!

BERTRAM.
It shall be so; I’ll send her to my house,
Acquaint my mother with my hate to her,
And wherefore I am fled; write to the king
That which I durst not speak. His present gift
Shall furnish me to those Italian fields
Where noble fellows strike. War is no strife
To the dark house and the detested wife.

PAROLLES.
Will this caprichio hold in thee, art sure?

BERTRAM.
Go with me to my chamber and advise me.
I’ll send her straight away. Tomorrow
I’ll to the wars, she to her single sorrow.

PAROLLES.
Why, these balls bound; there’s noise in it. ’Tis hard:
A young man married is a man that’s marr’d.
Therefore away, and leave her bravely; go.
The king has done you wrong; but hush ’tis so.

 [_Exeunt._]

SCENE IV. Paris. The King’s palace.

 Enter Helena and Clown.

HELENA.
My mother greets me kindly: is she well?

CLOWN.
She is not well, but yet she has her health; she’s very merry, but yet
she is not well. But thanks be given, she’s very well, and wants
nothing i’ the world; but yet she is not well.

HELENA.
If she be very well, what does she ail that she’s not very well?

CLOWN.
Truly, she’s very well indeed, but for two things.

HELENA.
What two things?

CLOWN.
One, that she’s not in heaven, whither God send her quickly! The other,
that she’s in earth, from whence God send her quickly!

 Enter Parolles.

PAROLLES.
Bless you, my fortunate lady!

HELENA.
I hope, sir, I have your good will to have mine own good fortune.

PAROLLES.
You had my prayers to lead them on; and to keep them on, have them
still. O, my knave how does my old lady?

CLOWN.
So that you had her wrinkles and I her money, I would she did as you
say.

PAROLLES.
Why, I say nothing.

CLOWN.
Marry, you are the wiser man; for many a man’s tongue shakes out his
master’s undoing. To say nothing, to do nothing, to know nothing, and
to have nothing, is to be a great part of your title; which is within a
very little of nothing.

PAROLLES.
Away! Thou art a knave.

CLOWN.
You should have said, sir, before a knave thou art a knave; that is
before me thou art a knave. This had been truth, sir.

PAROLLES.
Go to, thou art a witty fool; I have found thee.

CLOWN.
Did you find me in yourself, sir? or were you taught to find me? The
search, sir, was profitable; and much fool may you find in you, even to
the world’s pleasure and the increase of laughter.

PAROLLES.
A good knave, i’ faith, and well fed.
Madam, my lord will go away tonight;
A very serious business calls on him.
The great prerogative and right of love,
Which, as your due, time claims, he does acknowledge;
But puts it off to a compell’d restraint;
Whose want, and whose delay, is strew’d with sweets;
Which they distil now in the curbed time,
To make the coming hour o’erflow with joy
And pleasure drown the brim.

HELENA.
What’s his will else?

PAROLLES.
That you will take your instant leave o’ the king,
And make this haste as your own good proceeding,
Strengthen’d with what apology you think
May make it probable need.

HELENA.
What more commands he?

PAROLLES.
That, having this obtain’d, you presently
Attend his further pleasure.

HELENA.
In everything I wait upon his will.

PAROLLES.
I shall report it so.

HELENA.
I pray you. Come, sirrah.

 [_Exeunt._]

SCENE V. Another room in the same.

 Enter Lafew and Bertram.

LAFEW.
But I hope your lordship thinks not him a soldier.

BERTRAM.
Yes, my lord, and of very valiant approof.

LAFEW.
You have it from his own deliverance.

BERTRAM.
And by other warranted testimony.

LAFEW.
Then my dial goes not true; I took this lark for a bunting.

BERTRAM.
I do assure you, my lord, he is very great in knowledge, and
accordingly valiant.

LAFEW.
I have, then, sinned against his experience and transgressed against
his valour; and my state that way is dangerous, since I cannot yet find
in my heart to repent. Here he comes; I pray you make us friends; I
will pursue the amity.

 Enter Parolles.

PAROLLES.
[_To Bertram._] These things shall be done, sir.

LAFEW.
Pray you, sir, who’s his tailor?

PAROLLES.
Sir!

LAFEW.
O, I know him well, I, sir; he, sir, is a good workman, a very good
tailor.

BERTRAM.
[_Aside to Parolles._] Is she gone to the king?

PAROLLES.
She is.

BERTRAM.
Will she away tonight?

PAROLLES.
As you’ll have her.

BERTRAM.
I have writ my letters, casketed my treasure,
Given order for our horses; and tonight,
When I should take possession of the bride,
End ere I do begin.

LAFEW.
A good traveller is something at the latter end of a dinner; but one
that lies three-thirds and uses a known truth to pass a thousand
nothings with, should be once heard and thrice beaten.— God save you,
Captain.

BERTRAM.
Is there any unkindness between my lord and you, monsieur?

PAROLLES.
I know not how I have deserved to run into my lord’s displeasure.

LAFEW.
You have made shift to run into ’t, boots and spurs and all, like him
that leapt into the custard; and out of it you’ll run again, rather
than suffer question for your residence.

BERTRAM.
It may be you have mistaken him, my lord.

LAFEW.
And shall do so ever, though I took him at his prayers. Fare you well,
my lord; and believe this of me, there can be no kernal in this light
nut; the soul of this man is his clothes; trust him not in matter of
heavy consequence; I have kept of them tame, and know their natures.
Farewell, monsieur; I have spoken better of you than you have or will
to deserve at my hand; but we must do good against evil.

 [_Exit._]

PAROLLES.
An idle lord, I swear.

BERTRAM.
I think so.

PAROLLES.
Why, do you not know him?

BERTRAM.
Yes, I do know him well; and common speech
Gives him a worthy pass. Here comes my clog.

 Enter Helena.

HELENA.
I have, sir, as I was commanded from you,
Spoke with the king, and have procur’d his leave
For present parting; only he desires
Some private speech with you.

BERTRAM.
I shall obey his will.
You must not marvel, Helen, at my course,
Which holds not colour with the time, nor does
The ministration and required office
On my particular. Prepared I was not
For such a business; therefore am I found
So much unsettled: this drives me to entreat you;
That presently you take your way for home,
And rather muse than ask why I entreat you:
For my respects are better than they seem;
And my appointments have in them a need
Greater than shows itself at the first view
To you that know them not. This to my mother.

 [_Giving a letter._]

’Twill be two days ere I shall see you; so
I leave you to your wisdom.

HELENA.
Sir, I can nothing say
But that I am your most obedient servant.

BERTRAM.
Come, come, no more of that.

HELENA.
And ever shall
With true observance seek to eke out that
Wherein toward me my homely stars have fail’d
To equal my great fortune.

BERTRAM.
Let that go.
My haste is very great. Farewell; hie home.

HELENA.
Pray, sir, your pardon.

BERTRAM.
Well, what would you say?

HELENA.
I am not worthy of the wealth I owe;
Nor dare I say ’tis mine, and yet it is;
But, like a timorous thief, most fain would steal
What law does vouch mine own.

BERTRAM.
What would you have?

HELENA.
Something; and scarce so much; nothing indeed.
I would not tell you what I would, my lord. Faith, yes,
Strangers and foes do sunder and not kiss.

BERTRAM.
I pray you, stay not, but in haste to horse.

HELENA.
I shall not break your bidding, good my lord.
Where are my other men, monsieur?
Farewell,

 [_Exit Helena._]

BERTRAM.
Go thou toward home, where I will never come
Whilst I can shake my sword or hear the drum.
Away, and for our flight.

PAROLLES.
Bravely, coragio!

 [_Exeunt._]


ACT III.

SCENE I. Florence. A room in the Duke’s palace.

 Flourish. Enter the Duke of Florence attended; two French Lords, and
 Soldiers.

DUKE.
So that, from point to point, now have you heard
The fundamental reasons of this war,
Whose great decision hath much blood let forth,
And more thirsts after.

FIRST LORD.
Holy seems the quarrel
Upon your Grace’s part; black and fearful
On the opposer.

DUKE.
Therefore we marvel much our cousin France
Would, in so just a business, shut his bosom
Against our borrowing prayers.

SECOND LORD.
Good my lord,
The reasons of our state I cannot yield,
But like a common and an outward man
That the great figure of a council frames
By self-unable motion; therefore dare not
Say what I think of it, since I have found
Myself in my incertain grounds to fail
As often as I guess’d.

DUKE.
Be it his pleasure.

FIRST LORD.
But I am sure the younger of our nature,
That surfeit on their ease, will day by day
Come here for physic.

DUKE.
Welcome shall they be;
And all the honours that can fly from us
Shall on them settle. You know your places well;
When better fall, for your avails they fell.
Tomorrow to the field.

 [_Flourish. Exeunt._]

SCENE II. Rossillon. A room in the Countess’s palace.

 Enter Countess and Clown.

COUNTESS.
It hath happen’d all as I would have had it, save that he comes not
along with her.

CLOWN.
By my troth, I take my young lord to be a very melancholy man.

COUNTESS.
By what observance, I pray you?

CLOWN.
Why, he will look upon his boot and sing; mend the ruff and sing; ask
questions and sing; pick his teeth and sing. I know a man that had this
trick of melancholy sold a goodly manor for a song.

COUNTESS.
Let me see what he writes, and when he means to come.

 [_Opening a letter._]

CLOWN.
I have no mind to Isbel since I was at court. Our old lings and our
Isbels o’ th’ country are nothing like your old ling and your Isbels o’
th’ court. The brains of my Cupid’s knock’d out, and I begin to love,
as an old man loves money, with no stomach.

COUNTESS.
What have we here?

CLOWN.
E’en that you have there.

 [_Exit._]

COUNTESS.
[_Reads._] _I have sent you a daughter-in-law; she hath recovered the
king and undone me. I have wedded her, not bedded her, and sworn to
make the “not” eternal. You shall hear I am run away; know it before
the report come. If there be breadth enough in the world, I will hold a
long distance. My duty to you.
              Your unfortunate son,_
                            BERTRAM.

This is not well, rash and unbridled boy,
To fly the favours of so good a king,
To pluck his indignation on thy head
By the misprizing of a maid too virtuous
For the contempt of empire.

 Enter Clown.

CLOWN.
O madam, yonder is heavy news within between two soldiers and my young
lady.

COUNTESS.
What is the matter?

CLOWN.
Nay, there is some comfort in the news, some comfort; your son will not
be kill’d so soon as I thought he would.

COUNTESS.
Why should he be kill’d?

CLOWN.
So say I, madam, if he run away, as I hear he does; the danger is in
standing to’t; that’s the loss of men, though it be the getting of
children. Here they come will tell you more. For my part, I only hear
your son was run away.

 [_Exit._]

 Enter Helena and the two Gentlemen.

FIRST GENTLEMAN.
Save you, good madam.

HELENA.
Madam, my lord is gone, for ever gone.

SECOND GENTLEMAN.
Do not say so.

COUNTESS.
Think upon patience. Pray you, gentlemen,—
I have felt so many quirks of joy and grief
That the first face of neither on the start
Can woman me unto ’t. Where is my son, I pray you?

SECOND GENTLEMAN.
Madam, he’s gone to serve the Duke of Florence;
We met him thitherward, for thence we came,
And, after some despatch in hand at court,
Thither we bend again.

HELENA.
Look on this letter, madam; here’s my passport.

[_Reads._] _When thou canst get the ring upon my finger, which never
shall come off, and show me a child begotten of thy body that I am
father to, then call me husband; but in such a “then” I write a
“never”._
This is a dreadful sentence.

COUNTESS.
Brought you this letter, gentlemen?

FIRST GENTLEMAN.
Ay, madam; And for the contents’ sake, are sorry for our pains.

COUNTESS.
I pr’ythee, lady, have a better cheer;
If thou engrossest all the griefs are thine,
Thou robb’st me of a moiety. He was my son,
But I do wash his name out of my blood,
And thou art all my child. Towards Florence is he?

SECOND GENTLEMAN.
Ay, madam.

COUNTESS.
And to be a soldier?

SECOND GENTLEMAN.
Such is his noble purpose, and, believe’t,
The duke will lay upon him all the honour
That good convenience claims.

COUNTESS.
Return you thither?

FIRST GENTLEMAN.
Ay, madam, with the swiftest wing of speed.

HELENA.
[_Reads._] _Till I have no wife, I have nothing in France._
’Tis bitter.

COUNTESS.
Find you that there?

HELENA.
Ay, madam.

FIRST GENTLEMAN.
’Tis but the boldness of his hand haply, which his heart was not
consenting to.

COUNTESS.
Nothing in France until he have no wife!
There’s nothing here that is too good for him
But only she, and she deserves a lord
That twenty such rude boys might tend upon,
And call her hourly mistress. Who was with him?

FIRST GENTLEMAN.
A servant only, and a gentleman which I have sometime known.

COUNTESS.
Parolles, was it not?

FIRST GENTLEMAN.
Ay, my good lady, he.

COUNTESS.
A very tainted fellow, and full of wickedness.
My son corrupts a well-derived nature
With his inducement.

FIRST GENTLEMAN.
Indeed, good lady,
The fellow has a deal of that too much,
Which holds him much to have.

COUNTESS.
Y’are welcome, gentlemen.
I will entreat you, when you see my son,
To tell him that his sword can never win
The honour that he loses: more I’ll entreat you
Written to bear along.

SECOND GENTLEMAN.
We serve you, madam,
In that and all your worthiest affairs.

COUNTESS.
Not so, but as we change our courtesies.
Will you draw near?

 [_Exeunt Countess and Gentlemen._]

HELENA.
“Till I have no wife, I have nothing in France.”
Nothing in France until he has no wife!
Thou shalt have none, Rossillon, none in France;
Then hast thou all again. Poor lord, is’t I
That chase thee from thy country, and expose
Those tender limbs of thine to the event
Of the none-sparing war? And is it I
That drive thee from the sportive court, where thou
Wast shot at with fair eyes, to be the mark
Of smoky muskets? O you leaden messengers,
That ride upon the violent speed of fire,
Fly with false aim; move the still-peering air,
That sings with piercing; do not touch my lord.
Whoever shoots at him, I set him there;
Whoever charges on his forward breast,
I am the caitiff that do hold him to’t;
And though I kill him not, I am the cause
His death was so effected. Better ’twere
I met the ravin lion when he roar’d
With sharp constraint of hunger; better ’twere
That all the miseries which nature owes
Were mine at once. No; come thou home, Rossillon,
Whence honour but of danger wins a scar,
As oft it loses all. I will be gone;
My being here it is that holds thee hence.
Shall I stay here to do’t? No, no, although
The air of paradise did fan the house,
And angels offic’d all. I will be gone,
That pitiful rumour may report my flight
To consolate thine ear. Come, night; end, day;
For with the dark, poor thief, I’ll steal away.

 [_Exit._]

SCENE III. Florence. Before the Duke’s palace.

 Flourish. Enter the Duke of Florence, Bertram, drum and trumpets,
 Soldiers, Parolles.

DUKE.
The general of our horse thou art, and we,
Great in our hope, lay our best love and credence
Upon thy promising fortune.

BERTRAM.
Sir, it is
A charge too heavy for my strength; but yet
We’ll strive to bear it for your worthy sake
To th’extreme edge of hazard.

DUKE.
Then go thou forth;
And fortune play upon thy prosperous helm,
As thy auspicious mistress!

BERTRAM.
This very day,
Great Mars, I put myself into thy file;
Make me but like my thoughts, and I shall prove
A lover of thy drum, hater of love.

 [_Exeunt._]

SCENE IV. Rossillon. A room in the Countess’s palace.

 Enter Countess and Steward.

COUNTESS.
Alas! and would you take the letter of her?
Might you not know she would do as she has done,
By sending me a letter? Read it again.

STEWARD.
[_Reads._] _I am Saint Jaques’ pilgrim, thither gone.
Ambitious love hath so in me offended
That barefoot plod I the cold ground upon,
With sainted vow my faults to have amended.
Write, write, that from the bloody course of war
My dearest master, your dear son, may hie.
Bless him at home in peace, whilst I from far
His name with zealous fervour sanctify.
His taken labours bid him me forgive;
I, his despiteful Juno, sent him forth
From courtly friends, with camping foes to live,
Where death and danger dog the heels of worth.
He is too good and fair for death and me;
Whom I myself embrace to set him free._

COUNTESS.
Ah, what sharp stings are in her mildest words!
Rynaldo, you did never lack advice so much
As letting her pass so; had I spoke with her,
I could have well diverted her intents,
Which thus she hath prevented.

STEWARD.
Pardon me, madam;
If I had given you this at over-night,
She might have been o’erta’en; and yet she writes
Pursuit would be but vain.

COUNTESS.
What angel shall
Bless this unworthy husband? He cannot thrive,
Unless her prayers, whom heaven delights to hear
And loves to grant, reprieve him from the wrath
Of greatest justice. Write, write, Rynaldo,
To this unworthy husband of his wife;
Let every word weigh heavy of her worth,
That he does weigh too light; my greatest grief,
Though little he do feel it, set down sharply.
Dispatch the most convenient messenger.
When haply he shall hear that she is gone
He will return; and hope I may that she,
Hearing so much, will speed her foot again,
Led hither by pure love. Which of them both
Is dearest to me I have no skill in sense
To make distinction. Provide this messenger.
My heart is heavy, and mine age is weak;
Grief would have tears, and sorrow bids me speak.

 [_Exeunt._]

SCENE V. Without the walls of Florence.

 Enter an old Widow of Florence, Diana, Violenta, Mariana and other
 Citizens.

WIDOW.
Nay, come; for if they do approach the city, we shall lose all the
sight.

DIANA.
They say the French count has done most honourable service.

WIDOW.
It is reported that he has taken their great’st commander, and that
with his own hand he slew the duke’s brother.

 [_A tucket afar off._]

We have lost our labour; they are gone a contrary way. Hark! you may
know by their trumpets.

MARIANA.
Come, let’s return again, and suffice ourselves with the report of it.
Well, Diana, take heed of this French earl; the honour of a maid is her
name; and no legacy is so rich as honesty.

WIDOW.
I have told my neighbour how you have been solicited by a gentleman his
companion.

MARIANA.
I know that knave; hang him! one Parolles; a filthy officer he is in
those suggestions for the young earl. Beware of them, Diana; their
promises, enticements, oaths, tokens, and all these engines of lust,
are not the things they go under; many a maid hath been seduced by
them; and the misery is, example, that so terrible shows in the wreck
of maidenhood, cannot for all that dissuade succession, but that they
are limed with the twigs that threaten them. I hope I need not to
advise you further; but I hope your own grace will keep you where you
are, though there were no further danger known but the modesty which is
so lost.

DIANA.
You shall not need to fear me.

 Enter Helena in the dress of a pilgrim.

WIDOW.
I hope so. Look, here comes a pilgrim. I know she will lie at my house;
thither they send one another; I’ll question her. God save you,
pilgrim! Whither are bound?

HELENA.
To Saint Jaques le Grand.
Where do the palmers lodge, I do beseech you?

WIDOW.
At the Saint Francis here, beside the port.

HELENA.
Is this the way?

 [_A march afar._]

WIDOW.
Ay, marry, is’t. Hark you, they come this way.
If you will tarry, holy pilgrim,
But till the troops come by,
I will conduct you where you shall be lodg’d;
The rather for I think I know your hostess
As ample as myself.

HELENA.
Is it yourself?

WIDOW.
If you shall please so, pilgrim.

HELENA.
I thank you, and will stay upon your leisure.

WIDOW.
You came, I think, from France?

HELENA.
I did so.

WIDOW.
Here you shall see a countryman of yours
That has done worthy service.

HELENA.
His name, I pray you.

DIANA.
The Count Rossillon. Know you such a one?

HELENA.
But by the ear, that hears most nobly of him;
His face I know not.

DIANA.
Whatsome’er he is,
He’s bravely taken here. He stole from France,
As ’tis reported, for the king had married him
Against his liking. Think you it is so?

HELENA.
Ay, surely, mere the truth; I know his lady.

DIANA.
There is a gentleman that serves the count
Reports but coarsely of her.

HELENA.
What’s his name?

DIANA.
Monsieur Parolles.

HELENA.
O, I believe with him,
In argument of praise, or to the worth
Of the great count himself, she is too mean
To have her name repeated; all her deserving
Is a reserved honesty, and that
I have not heard examin’d.

DIANA.
Alas, poor lady!
’Tis a hard bondage to become the wife
Of a detesting lord.

WIDOW.
Ay, right; good creature, wheresoe’er she is,
Her heart weighs sadly. This young maid might do her
A shrewd turn, if she pleas’d.

HELENA.
How do you mean?
Maybe the amorous count solicits her
In the unlawful purpose.

WIDOW.
He does indeed,
And brokes with all that can in such a suit
Corrupt the tender honour of a maid;
But she is arm’d for him, and keeps her guard
In honestest defence.

 Enter, with a drum and colours, a party of the Florentine army,
 Bertram and Parolles.

MARIANA.
The gods forbid else!

WIDOW.
So, now they come.
That is Antonio, the Duke’s eldest son;
That Escalus.

HELENA.
Which is the Frenchman?

DIANA.
He;
That with the plume; ’tis a most gallant fellow.
I would he lov’d his wife; if he were honester
He were much goodlier. Is’t not a handsome gentleman?

HELENA.
I like him well.

DIANA.
’Tis pity he is not honest. Yond’s that same knave
That leads him to these places. Were I his lady
I would poison that vile rascal.

HELENA.
Which is he?

DIANA.
That jack-an-apes with scarfs. Why is he melancholy?

HELENA.
Perchance he’s hurt i’ the battle.

PAROLLES.
Lose our drum! Well.

MARIANA.
He’s shrewdly vex’d at something. Look, he has spied us.

WIDOW.
Marry, hang you!

MARIANA.
And your courtesy, for a ring-carrier!

 [_Exeunt Bertram, Parolles, Officers and Soldiers._]

WIDOW.
The troop is past. Come, pilgrim, I will bring you
Where you shall host; of enjoin’d penitents
There’s four or five, to great Saint Jaques bound,
Already at my house.

HELENA.
I humbly thank you.
Please it this matron and this gentle maid
To eat with us tonight; the charge and thanking
Shall be for me; and, to requite you further,
I will bestow some precepts of this virgin,
Worthy the note.

BOTH.
We’ll take your offer kindly.

 [_Exeunt._]

SCENE VI. Camp before Florence.

 Enter Bertram and the two French Lords.

FIRST LORD.
Nay, good my lord, put him to’t; let him have his way.

SECOND LORD.
If your lordship find him not a hilding, hold me no more in your
respect.

FIRST LORD.
On my life, my lord, a bubble.

BERTRAM.
Do you think I am so far deceived in him?

FIRST LORD.
Believe it, my lord, in mine own direct knowledge, without any malice,
but to speak of him as my kinsman, he’s a most notable coward, an
infinite and endless liar, an hourly promise-breaker, the owner of no
one good quality worthy your lordship’s entertainment.

SECOND LORD.
It were fit you knew him; lest, reposing too far in his virtue, which
he hath not, he might at some great and trusty business, in a main
danger fail you.

BERTRAM.
I would I knew in what particular action to try him.

SECOND LORD.
None better than to let him fetch off his drum, which you hear him so
confidently undertake to do.

FIRST LORD.
I with a troop of Florentines will suddenly surprise him; such I will
have whom I am sure he knows not from the enemy; we will bind and
hoodwink him so that he shall suppose no other but that he is carried
into the leaguer of the adversaries when we bring him to our own tents.
Be but your lordship present at his examination; if he do not for the
promise of his life, and in the highest compulsion of base fear, offer
to betray you, and deliver all the intelligence in his power against
you, and that with the divine forfeit of his soul upon oath, never
trust my judgment in anything.

SECOND LORD.
O, for the love of laughter, let him fetch his drum; he says he has a
stratagem for’t. When your lordship sees the bottom of his success
in’t, and to what metal this counterfeit lump of ore will be melted, if
you give him not John Drum’s entertainment, your inclining cannot be
removed. Here he comes.

 Enter Parolles.

FIRST LORD.
O, for the love of laughter, hinder not the honour of his design: let
him fetch off his drum in any hand.

BERTRAM.
How now, monsieur! This drum sticks sorely in your disposition.

SECOND LORD.
A pox on ’t; let it go; ’tis but a drum.

PAROLLES.
But a drum! Is’t but a drum? A drum so lost! There was excellent
command, to charge in with our horse upon our own wings, and to rend
our own soldiers.

SECOND LORD.
That was not to be blam’d in the command of the service; it was a
disaster of war that Caesar himself could not have prevented, if he had
been there to command.

BERTRAM.
Well, we cannot greatly condemn our success: some dishonour we had in
the loss of that drum, but it is not to be recovered.

PAROLLES.
It might have been recovered.

BERTRAM.
It might, but it is not now.

PAROLLES.
It is to be recovered. But that the merit of service is seldom
attributed to the true and exact performer, I would have that drum or
another, or _hic jacet_.

BERTRAM.
Why, if you have a stomach, to’t, monsieur, if you think your mystery
in stratagem can bring this instrument of honour again into his native
quarter, be magnanimous in the enterprise, and go on; I will grace the
attempt for a worthy exploit; if you speed well in it, the duke shall
both speak of it and extend to you what further becomes his greatness,
even to the utmost syllable of your worthiness.

PAROLLES.
By the hand of a soldier, I will undertake it.

BERTRAM.
But you must not now slumber in it.

PAROLLES.
I’ll about it this evening; and I will presently pen down my dilemmas,
encourage myself in my certainty, put myself into my mortal
preparation; and by midnight look to hear further from me.

BERTRAM.
May I be bold to acquaint his grace you are gone about it?

PAROLLES.
I know not what the success will be, my lord, but the attempt I vow.

BERTRAM.
I know th’art valiant; and to the possibility of thy soldiership, will
subscribe for thee. Farewell.

PAROLLES.
I love not many words.

 [_Exit._]

FIRST LORD.
No more than a fish loves water. Is not this a strange fellow, my lord,
that so confidently seems to undertake this business, which he knows is
not to be done; damns himself to do, and dares better be damn’d than to
do’t.

SECOND LORD.
You do not know him, my lord, as we do; certain it is that he will
steal himself into a man’s favour, and for a week escape a great deal
of discoveries, but when you find him out, you have him ever after.

BERTRAM.
Why, do you think he will make no deed at all of this, that so
seriously he does address himself unto?

FIRST LORD.
None in the world; but return with an invention, and clap upon you two
or three probable lies; but we have almost embossed him; you shall see
his fall tonight; for indeed he is not for your lordship’s respect.

SECOND LORD.
We’ll make you some sport with the fox ere we case him. He was first
smok’d by the old Lord Lafew; when his disguise and he is parted, tell
me what a sprat you shall find him; which you shall see this very
night.

FIRST LORD.
I must go look my twigs. He shall be caught.

BERTRAM.
Your brother, he shall go along with me.

FIRST LORD.
As’t please your lordship. I’ll leave you.

 [_Exit._]

BERTRAM.
Now will I lead you to the house, and show you
The lass I spoke of.

SECOND LORD.
But you say she’s honest.

BERTRAM.
That’s all the fault. I spoke with her but once,
And found her wondrous cold, but I sent to her
By this same coxcomb that we have i’ the wind
Tokens and letters which she did re-send,
And this is all I have done. She’s a fair creature;
Will you go see her?

SECOND LORD.
With all my heart, my lord.

 [_Exeunt._]

SCENE VII. Florence. A room in the Widow’s house.

 Enter Helena and Widow.

HELENA.
If you misdoubt me that I am not she,
I know not how I shall assure you further,
But I shall lose the grounds I work upon.

WIDOW.
Though my estate be fall’n, I was well born,
Nothing acquainted with these businesses,
And would not put my reputation now
In any staining act.

HELENA.
Nor would I wish you.
First give me trust, the count he is my husband,
And what to your sworn counsel I have spoken
Is so from word to word; and then you cannot,
By the good aid that I of you shall borrow,
Err in bestowing it.

WIDOW.
I should believe you,
For you have show’d me that which well approves
Y’are great in fortune.

HELENA.
Take this purse of gold,
And let me buy your friendly help thus far,
Which I will over-pay, and pay again
When I have found it. The count he woos your daughter
Lays down his wanton siege before her beauty,
Resolv’d to carry her; let her in fine consent,
As we’ll direct her how ’tis best to bear it.
Now his important blood will naught deny
That she’ll demand; a ring the county wears,
That downward hath succeeded in his house
From son to son, some four or five descents
Since the first father wore it. This ring he holds
In most rich choice; yet, in his idle fire,
To buy his will, it would not seem too dear,
Howe’er repented after.

WIDOW.
Now I see
The bottom of your purpose.

HELENA.
You see it lawful then; it is no more
But that your daughter, ere she seems as won,
Desires this ring; appoints him an encounter;
In fine, delivers me to fill the time,
Herself most chastely absent. After,
To marry her, I’ll add three thousand crowns
To what is pass’d already.

WIDOW.
I have yielded.
Instruct my daughter how she shall persever,
That time and place with this deceit so lawful
May prove coherent. Every night he comes
With musics of all sorts, and songs compos’d
To her unworthiness: it nothing steads us
To chide him from our eaves; for he persists
As if his life lay on ’t.

HELENA.
Why then tonight
Let us assay our plot; which, if it speed,
Is wicked meaning in a lawful deed,
And lawful meaning in a lawful act,
Where both not sin, and yet a sinful fact.
But let’s about it.

 [_Exeunt._]


ACT IV.

SCENE I. Without the Florentine camp.

 Enter first Lord with five or six Soldiers in ambush.

FIRST LORD.
He can come no other way but by this hedge-corner. When you sally upon
him, speak what terrible language you will; though you understand it
not yourselves, no matter; for we must not seem to understand him,
unless someone among us, whom we must produce for an interpreter.

FIRST SOLDIER.
Good captain, let me be th’ interpreter.

FIRST LORD.
Art not acquainted with him? Knows he not thy voice?

FIRST SOLDIER.
No sir, I warrant you.

FIRST LORD.
But what linsey-woolsey has thou to speak to us again?

FIRST SOLDIER.
E’en such as you speak to me.

FIRST LORD.
He must think us some band of strangers i’ the adversary’s
entertainment. Now he hath a smack of all neighbouring languages,
therefore we must every one be a man of his own fancy; not to know what
we speak one to another, so we seem to know, is to know straight our
purpose: choughs’ language, gabble enough, and good enough. As for you,
interpreter, you must seem very politic. But couch, ho! Here he comes;
to beguile two hours in a sleep, and then to return and swear the lies
he forges.

 Enter Parolles.

PAROLLES.
Ten o’clock. Within these three hours ’twill be time enough to go home.
What shall I say I have done? It must be a very plausive invention that
carries it. They begin to smoke me, and disgraces have of late knock’d
too often at my door. I find my tongue is too foolhardy, but my heart
hath the fear of Mars before it, and of his creatures, not daring the
reports of my tongue.

FIRST LORD.
[_Aside._] This is the first truth that e’er thine own tongue was
guilty of.

PAROLLES.
What the devil should move me to undertake the recovery of this drum,
being not ignorant of the impossibility, and knowing I had no such
purpose? I must give myself some hurts, and say I got them in exploit;
yet slight ones will not carry it. They will say “Came you off with so
little?” and great ones I dare not give. Wherefore, what’s the
instance? Tongue, I must put you into a butter-woman’s mouth, and buy
myself another of Bajazet’s mule, if you prattle me into these perils.

FIRST LORD.
[_Aside._] Is it possible he should know what he is, and be that he is?

PAROLLES.
I would the cutting of my garments would serve the turn, or the
breaking of my Spanish sword.

FIRST LORD.
[_Aside._] We cannot afford you so.

PAROLLES.
Or the baring of my beard, and to say it was in stratagem.

FIRST LORD.
[_Aside._] ’Twould not do.

PAROLLES.
Or to drown my clothes, and say I was stripped.

FIRST LORD.
[_Aside._] Hardly serve.

PAROLLES.
Though I swore I leap’d from the window of the citadel,—

FIRST LORD.
[_Aside._] How deep?

PAROLLES.
Thirty fathom.

FIRST LORD.
[_Aside._] Three great oaths would scarce make that be believed.

PAROLLES.
I would I had any drum of the enemy’s; I would swear I recover’d it.

FIRST LORD.
[_Aside._] You shall hear one anon.

PAROLLES.
A drum now of the enemy’s!

 [_Alarum within._]

FIRST LORD.
_Throca movousus, cargo, cargo, cargo._

ALL.
_Cargo, cargo, cargo, villianda par corbo, cargo._

 [_They seize and blindfold him._]

PAROLLES.
O, ransom, ransom! Do not hide mine eyes.

FIRST SOLDIER.
_Boskos thromuldo boskos._

PAROLLES.
I know you are the Muskos’ regiment,
And I shall lose my life for want of language.
If there be here German, or Dane, Low Dutch,
Italian, or French, let him speak to me,
I’ll discover that which shall undo the Florentine.

FIRST SOLDIER.
_Boskos vauvado._ I understand thee, and can speak thy tongue.
_Kerelybonto._ Sir, Betake thee to thy faith, for seventeen poniards
are at thy bosom.

PAROLLES.
O!

FIRST SOLDIER.
O, pray, pray, pray!
_Manka revania dulche._

FIRST LORD.
_Oscorbidulchos volivorco._

FIRST SOLDIER.
The General is content to spare thee yet;
And, hoodwink’d as thou art, will lead thee on
To gather from thee. Haply thou mayst inform
Something to save thy life.

PAROLLES.
O, let me live,
And all the secrets of our camp I’ll show,
Their force, their purposes; nay, I’ll speak that
Which you will wonder at.

FIRST SOLDIER.
But wilt thou faithfully?

PAROLLES.
If I do not, damn me.

FIRST SOLDIER.
_Acordo linta._
Come on; thou art granted space.

 [_Exit, with Parolles guarded._]

 A short alarum within.

FIRST LORD.
Go tell the Count Rossillon and my brother
We have caught the woodcock, and will keep him muffled
Till we do hear from them.

SECOND SOLDIER.
Captain, I will.

FIRST LORD.
’A will betray us all unto ourselves;
Inform on that.

SECOND SOLDIER.
So I will, sir.

FIRST LORD.
Till then I’ll keep him dark, and safely lock’d.

 [_Exeunt._]

SCENE II. Florence. A room in the Widow’s house.

 Enter Bertram and Diana.

BERTRAM.
They told me that your name was Fontybell.

DIANA.
No, my good lord, Diana.

BERTRAM.
Titled goddess;
And worth it, with addition! But, fair soul,
In your fine frame hath love no quality?
If the quick fire of youth light not your mind,
You are no maiden but a monument;
When you are dead, you should be such a one
As you are now; for you are cold and stern,
And now you should be as your mother was
When your sweet self was got.

DIANA.
She then was honest.

BERTRAM.
So should you be.

DIANA.
No.
My mother did but duty; such, my lord,
As you owe to your wife.

BERTRAM.
No more a’ that!
I pr’ythee do not strive against my vows;
I was compell’d to her; but I love thee
By love’s own sweet constraint, and will for ever
Do thee all rights of service.

DIANA.
Ay, so you serve us
Till we serve you; but when you have our roses,
You barely leave our thorns to prick ourselves,
And mock us with our bareness.

BERTRAM.
How have I sworn?

DIANA.
’Tis not the many oaths that makes the truth,
But the plain single vow that is vow’d true.
What is not holy, that we swear not by,
But take the highest to witness: then, pray you, tell me,
If I should swear by Jove’s great attributes
I lov’d you dearly, would you believe my oaths
When I did love you ill? This has no holding,
To swear by him whom I protest to love
That I will work against him. Therefore your oaths
Are words and poor conditions; but unseal’d,—
At least in my opinion.

BERTRAM.
Change it, change it.
Be not so holy-cruel. Love is holy;
And my integrity ne’er knew the crafts
That you do charge men with. Stand no more off,
But give thyself unto my sick desires,
Who then recovers. Say thou art mine, and ever
My love as it begins shall so persever.

DIANA.
I see that men make hopes in such a case,
That we’ll forsake ourselves. Give me that ring.

BERTRAM.
I’ll lend it thee, my dear, but have no power
To give it from me.

DIANA.
Will you not, my lord?

BERTRAM.
It is an honour ’longing to our house,
Bequeathed down from many ancestors,
Which were the greatest obloquy i’ the world
In me to lose.

DIANA.
Mine honour’s such a ring;
My chastity’s the jewel of our house,
Bequeathed down from many ancestors,
Which were the greatest obloquy i’ the world
In me to lose. Thus your own proper wisdom
Brings in the champion honour on my part
Against your vain assault.

BERTRAM.
Here, take my ring;
My house, mine honour, yea, my life be thine,
And I’ll be bid by thee.

DIANA.
When midnight comes, knock at my chamber window;
I’ll order take my mother shall not hear.
Now will I charge you in the band of truth,
When you have conquer’d my yet maiden-bed,
Remain there but an hour, nor speak to me.
My reasons are most strong; and you shall know them
When back again this ring shall be deliver’d;
And on your finger in the night, I’ll put
Another ring, that what in time proceeds
May token to the future our past deeds.
Adieu till then; then fail not. You have won
A wife of me, though there my hope be done.

BERTRAM.
A heaven on earth I have won by wooing thee.

 [_Exit._]

DIANA.
For which live long to thank both heaven and me!
You may so in the end.
My mother told me just how he would woo,
As if she sat in’s heart. She says all men
Have the like oaths. He had sworn to marry me
When his wife’s dead; therefore I’ll lie with him
When I am buried. Since Frenchmen are so braid,
Marry that will, I live and die a maid.
Only, in this disguise, I think’t no sin
To cozen him that would unjustly win.

 [_Exit._]

SCENE III. The Florentine camp.

 Enter the two French Lords and two or three Soldiers.

FIRST LORD.
You have not given him his mother’s letter?

SECOND LORD.
I have deliv’red it an hour since; there is something in’t that stings
his nature; for on the reading it, he chang’d almost into another man.

FIRST LORD.
He has much worthy blame laid upon him for shaking off so good a wife
and so sweet a lady.

SECOND LORD.
Especially he hath incurred the everlasting displeasure of the king,
who had even tun’d his bounty to sing happiness to him. I will tell you
a thing, but you shall let it dwell darkly with you.

FIRST LORD.
When you have spoken it, ’tis dead, and I am the grave of it.

SECOND LORD.
He hath perverted a young gentlewoman here in Florence, of a most
chaste renown, and this night he fleshes his will in the spoil of her
honour; he hath given her his monumental ring, and thinks himself made
in the unchaste composition.

FIRST LORD.
Now, God delay our rebellion! As we are ourselves, what things are we!

SECOND LORD.
Merely our own traitors. And as in the common course of all treasons,
we still see them reveal themselves till they attain to their abhorr’d
ends; so he that in this action contrives against his own nobility, in
his proper stream, o’erflows himself.

FIRST LORD.
Is it not meant damnable in us to be trumpeters of our unlawful
intents? We shall not then have his company tonight?

SECOND LORD.
Not till after midnight; for he is dieted to his hour.

FIRST LORD.
That approaches apace. I would gladly have him see his company
anatomized, that he might take a measure of his own judgments, wherein
so curiously he had set this counterfeit.

SECOND LORD.
We will not meddle with him till he come; for his presence must be the
whip of the other.

FIRST LORD.
In the meantime, what hear you of these wars?

SECOND LORD.
I hear there is an overture of peace.

FIRST LORD.
Nay, I assure you, a peace concluded.

SECOND LORD.
What will Count Rossillon do then? Will he travel higher, or return
again into France?

FIRST LORD.
I perceive by this demand, you are not altogether of his council.

SECOND LORD.
Let it be forbid, sir! So should I be a great deal of his act.

FIRST LORD.
Sir, his wife some two months since fled from his house. Her pretence
is a pilgrimage to Saint Jaques le Grand; which holy undertaking with
most austere sanctimony she accomplished; and there residing, the
tenderness of her nature became as a prey to her grief; in fine, made a
groan of her last breath, and now she sings in heaven.

SECOND LORD.
How is this justified?

FIRST LORD.
The stronger part of it by her own letters, which makes her story true,
even to the point of her death. Her death itself, which could not be
her office to say is come, was faithfully confirm’d by the rector of
the place.

SECOND LORD.
Hath the count all this intelligence?

FIRST LORD.
Ay, and the particular confirmations, point from point, to the full
arming of the verity.

SECOND LORD.
I am heartily sorry that he’ll be glad of this.

FIRST LORD.
How mightily sometimes we make us comforts of our losses!

SECOND LORD.
And how mightily some other times we drown our gain in tears! The great
dignity that his valour hath here acquir’d for him shall at home be
encountered with a shame as ample.

FIRST LORD.
The web of our life is of a mingled yarn, good and ill together; our
virtues would be proud if our faults whipped them not; and our crimes
would despair if they were not cherish’d by our virtues.

 Enter a Messenger.

How now? Where’s your master?

MESSENGER.
He met the duke in the street, sir; of whom he hath taken a solemn
leave: his lordship will next morning for France. The duke hath offered
him letters of commendations to the king.

SECOND LORD.
They shall be no more than needful there, if they were more than they
can commend.

 Enter Bertram.

FIRST LORD.
They cannot be too sweet for the king’s tartness. Here’s his lordship
now. How now, my lord, is’t not after midnight?

BERTRAM.
I have tonight despatch’d sixteen businesses, a month’s length apiece;
by an abstract of success: I have congied with the duke, done my adieu
with his nearest; buried a wife, mourn’d for her, writ to my lady
mother I am returning, entertained my convoy, and between these main
parcels of despatch effected many nicer needs: the last was the
greatest, but that I have not ended yet.

SECOND LORD.
If the business be of any difficulty and this morning your departure
hence, it requires haste of your lordship.

BERTRAM.
I mean the business is not ended, as fearing to hear of it hereafter.
But shall we have this dialogue between the Fool and the Soldier? Come,
bring forth this counterfeit module has deceiv’d me like a
double-meaning prophesier.

SECOND LORD.
Bring him forth.

 [_Exeunt Soldiers._]

Has sat i’ the stocks all night, poor gallant knave.

BERTRAM.
No matter; his heels have deserv’d it, in usurping his spurs so long.
How does he carry himself?

FIRST LORD.
I have told your lordship already; the stocks carry him. But to answer
you as you would be understood: he weeps like a wench that had shed her
milk; he hath confessed himself to Morgan, whom he supposes to be a
friar, from the time of his remembrance to this very instant disaster
of his setting i’ the stocks. And what think you he hath confessed?

BERTRAM.
Nothing of me, has he?

SECOND LORD.
His confession is taken, and it shall be read to his face; if your
lordship be in’t, as I believe you are, you must have the patience to
hear it.

 Enter Soldiers with Parolles.

BERTRAM.
A plague upon him! muffled! he can say nothing of me; hush, hush!

FIRST LORD.
Hoodman comes! _Portotartarossa._

FIRST SOLDIER.
He calls for the tortures. What will you say without ’em?

PAROLLES.
I will confess what I know without constraint. If ye pinch me like a
pasty I can say no more.

FIRST SOLDIER.
_Bosko chimurcho._

FIRST LORD.
_Boblibindo chicurmurco._

FIRST SOLDIER.
You are a merciful general. Our general bids you answer to what I shall
ask you out of a note.

PAROLLES.
And truly, as I hope to live.

FIRST SOLDIER.
‘First demand of him how many horse the duke is strong.’ What say you
to that?

PAROLLES.
Five or six thousand; but very weak and unserviceable: the troops are
all scattered, and the commanders very poor rogues, upon my reputation
and credit, and as I hope to live.

FIRST SOLDIER.
Shall I set down your answer so?

PAROLLES.
Do. I’ll take the sacrament on ’t, how and which way you will.

BERTRAM.
All’s one to him. What a past-saving slave is this!

FIRST LORD.
You are deceived, my lord; this is Monsieur Parolles, the gallant
militarist (that was his own phrase), that had the whole theoric of war
in the knot of his scarf, and the practice in the chape of his dagger.

SECOND LORD.
I will never trust a man again for keeping his sword clean, nor believe
he can have everything in him by wearing his apparel neatly.

FIRST SOLDIER.
Well, that’s set down.

PAROLLES.
‘Five or six thousand horse’ I said—I will say true—or thereabouts, set
down,—for I’ll speak truth.

FIRST LORD.
He’s very near the truth in this.

BERTRAM.
But I con him no thanks for’t in the nature he delivers it.

PAROLLES.
Poor rogues, I pray you say.

FIRST SOLDIER.
Well, that’s set down.

PAROLLES.
I humbly thank you, sir; a truth’s a truth, the rogues are marvellous
poor.

FIRST SOLDIER.
‘Demand of him of what strength they are a-foot.’ What say you to that?

PAROLLES.
By my troth, sir, if I were to live this present hour, I will tell
true. Let me see: Spurio, a hundred and fifty, Sebastian, so many;
Corambus, so many; Jaques, so many; Guiltian, Cosmo, Lodowick, and
Gratii, two hundred fifty each; mine own company, Chitopher, Vaumond,
Bentii, two hundred fifty each: so that the muster-file, rotten and
sound, upon my life, amounts not to fifteen thousand poll; half of the
which dare not shake the snow from off their cassocks lest they shake
themselves to pieces.

BERTRAM.
What shall be done to him?

FIRST LORD.
Nothing, but let him have thanks. Demand of him my condition, and what
credit I have with the duke.

FIRST SOLDIER.
Well, that’s set down. ‘You shall demand of him whether one Captain
Dumaine be i’ the camp, a Frenchman; what his reputation is with the
duke, what his valour, honesty and expertness in wars; or whether he
thinks it were not possible with well-weighing sums of gold to corrupt
him to a revolt.’ What say you to this? What do you know of it?

PAROLLES.
I beseech you, let me answer to the particular of the inter’gatories.
Demand them singly.

FIRST SOLDIER.
Do you know this Captain Dumaine?

PAROLLES.
I know him: he was a botcher’s ’prentice in Paris, from whence he was
whipped for getting the shrieve’s fool with child, a dumb innocent that
could not say him nay.

 [_First Lord lifts up his hand in anger._]

BERTRAM.
Nay, by your leave, hold your hands; though I know his brains are
forfeit to the next tile that falls.

FIRST SOLDIER.
Well, is this captain in the Duke of Florence’s camp?

PAROLLES.
Upon my knowledge, he is, and lousy.

FIRST LORD.
Nay, look not so upon me; we shall hear of your lordship anon.

FIRST SOLDIER.
What is his reputation with the duke?

PAROLLES.
The duke knows him for no other but a poor officer of mine, and writ to
me this other day to turn him out o’ the band. I think I have his
letter in my pocket.

FIRST SOLDIER.
Marry, we’ll search.

PAROLLES.
In good sadness, I do not know; either it is there or it is upon a
file, with the duke’s other letters, in my tent.

FIRST SOLDIER.
Here ’tis; here’s a paper; shall I read it to you?

PAROLLES.
I do not know if it be it or no.

BERTRAM.
Our interpreter does it well.

FIRST LORD.
Excellently.

FIRST SOLDIER.
[_Reads._] _Dian, the Count’s a fool, and full of gold._

PAROLLES.
That is not the duke’s letter, sir; that is an advertisement to a
proper maid in Florence, one Diana, to take heed of the allurement of
one Count Rossillon, a foolish idle boy, but for all that very ruttish.
I pray you, sir, put it up again.

FIRST SOLDIER.
Nay, I’ll read it first by your favour.

PAROLLES.
My meaning in’t, I protest, was very honest in the behalf of the maid;
for I knew the young count to be a dangerous and lascivious boy, who is
a whale to virginity, and devours up all the fry it finds.

BERTRAM.
Damnable both sides rogue!

FIRST SOLDIER.
[_Reads._]
_When he swears oaths, bid him drop gold, and take it;
After he scores, he never pays the score.
Half won is match well made; match, and well make it;
He ne’er pays after-debts, take it before.
And say a soldier, ‘Dian,’ told thee this:
Men are to mell with, boys are not to kiss;
For count of this, the count’s a fool, I know it,
Who pays before, but not when he does owe it.
Thine, as he vow’d to thee in thine ear,_
                                     PAROLLES.

BERTRAM.
He shall be whipped through the army with this rhyme in’s forehead.

SECOND LORD.
This is your devoted friend, sir, the manifold linguist, and the
armipotent soldier.

BERTRAM.
I could endure anything before but a cat, and now he’s a cat to me.

FIRST SOLDIER.
I perceive, sir, by our general’s looks we shall be fain to hang you.

PAROLLES.
My life, sir, in any case. Not that I am afraid to die, but that, my
offences being many, I would repent out the remainder of nature. Let me
live, sir, in a dungeon, i’ the stocks, or anywhere, so I may live.

FIRST SOLDIER.
We’ll see what may be done, so you confess freely. Therefore, once more
to this Captain Dumaine: you have answer’d to his reputation with the
duke, and to his valour. What is his honesty?

PAROLLES.
He will steal, sir, an egg out of a cloister: for rapes and ravishments
he parallels Nessus. He professes not keeping of oaths; in breaking
them he is stronger than Hercules. He will lie, sir, with such
volubility that you would think truth were a fool: drunkenness is his
best virtue, for he will be swine-drunk, and in his sleep he does
little harm, save to his bedclothes about him; but they know his
conditions and lay him in straw. I have but little more to say, sir, of
his honesty; he has everything that an honest man should not have; what
an honest man should have, he has nothing.

FIRST LORD.
I begin to love him for this.

BERTRAM.
For this description of thine honesty? A pox upon him for me, he’s more
and more a cat.

FIRST SOLDIER.
What say you to his expertness in war?

PAROLLES.
Faith, sir, has led the drum before the English tragedians,—to belie
him I will not,—and more of his soldiership I know not, except in that
country he had the honour to be the officer at a place there called
Mile-end, to instruct for the doubling of files. I would do the man
what honour I can, but of this I am not certain.

FIRST LORD.
He hath out-villain’d villainy so far that the rarity redeems him.

BERTRAM.
A pox on him! He’s a cat still.

FIRST SOLDIER.
His qualities being at this poor price, I need not to ask you if gold
will corrupt him to revolt.

PAROLLES.
Sir, for a quart d’ecu he will sell the fee-simple of his salvation,
the inheritance of it, and cut the entail from all remainders, and a
perpetual succession for it perpetually.

FIRST SOLDIER.
What’s his brother, the other Captain Dumaine?

SECOND LORD.
Why does he ask him of me?

FIRST SOLDIER.
What’s he?

PAROLLES.
E’en a crow o’ the same nest; not altogether so great as the first in
goodness, but greater a great deal in evil. He excels his brother for a
coward, yet his brother is reputed one of the best that is. In a
retreat he outruns any lackey; marry, in coming on he has the cramp.

FIRST SOLDIER.
If your life be saved, will you undertake to betray the Florentine?

PAROLLES.
Ay, and the captain of his horse, Count Rossillon.

FIRST SOLDIER.
I’ll whisper with the general, and know his pleasure.

PAROLLES.
[_Aside._] I’ll no more drumming; a plague of all drums! Only to seem
to deserve well, and to beguile the supposition of that lascivious
young boy the count, have I run into this danger: yet who would have
suspected an ambush where I was taken?

FIRST SOLDIER.
There is no remedy, sir, but you must die. The general says you that
have so traitorously discovered the secrets of your army, and made such
pestiferous reports of men very nobly held, can serve the world for no
honest use; therefore you must die. Come, headsman, off with his head.

PAROLLES.
O Lord! sir, let me live, or let me see my death.

FIRST SOLDIER.
That shall you, and take your leave of all your friends.

 [_Unmuffling him._]

So, look about you; know you any here?

BERTRAM.
Good morrow, noble captain.

SECOND LORD.
God bless you, Captain Parolles.

FIRST LORD.
God save you, noble captain.

SECOND LORD.
Captain, what greeting will you to my Lord Lafew? I am for France.

FIRST LORD.
Good Captain, will you give me a copy of the sonnet you writ to Diana
in behalf of the Count Rossillon? And I were not a very coward I’d
compel it of you; but fare you well.

 [_Exeunt Bertram, Lords &c._]

FIRST SOLDIER.
You are undone, captain: all but your scarf; that has a knot on’t yet.

PAROLLES.
Who cannot be crushed with a plot?

FIRST SOLDIER.
If you could find out a country where but women were that had received
so much shame, you might begin an impudent nation. Fare ye well, sir. I
am for France too; we shall speak of you there.

 [_Exeunt._]

PAROLLES.
Yet am I thankful. If my heart were great
’Twould burst at this. Captain I’ll be no more,
But I will eat, and drink, and sleep as soft
As captain shall. Simply the thing I am
Shall make me live. Who knows himself a braggart,
Let him fear this; for it will come to pass
That every braggart shall be found an ass.
Rust, sword; cool, blushes; and, Parolles live
Safest in shame; being fool’d, by foolery thrive.
There’s place and means for every man alive.
I’ll after them.

 [_Exit._]

SCENE IV. Florence. A room in the Widow’s house.

 Enter Helena, Widow and Diana.

HELENA.
That you may well perceive I have not wrong’d you
One of the greatest in the Christian world
Shall be my surety; fore whose throne ’tis needful,
Ere I can perfect mine intents, to kneel.
Time was I did him a desired office,
Dear almost as his life; which gratitude
Through flinty Tartar’s bosom would peep forth,
And answer thanks. I duly am inform’d
His grace is at Marseilles; to which place
We have convenient convoy. You must know
I am supposed dead. The army breaking,
My husband hies him home, where, heaven aiding,
And by the leave of my good lord the king,
We’ll be before our welcome.

WIDOW.
Gentle madam,
You never had a servant to whose trust
Your business was more welcome.

HELENA.
Nor you, mistress,
Ever a friend whose thoughts more truly labour
To recompense your love. Doubt not but heaven
Hath brought me up to be your daughter’s dower,
As it hath fated her to be my motive
And helper to a husband. But, O strange men!
That can such sweet use make of what they hate,
When saucy trusting of the cozen’d thoughts
Defiles the pitchy night; so lust doth play
With what it loathes, for that which is away.
But more of this hereafter. You, Diana,
Under my poor instructions yet must suffer
Something in my behalf.

DIANA.
Let death and honesty
Go with your impositions, I am yours
Upon your will to suffer.

HELENA.
Yet, I pray you;
But with the word the time will bring on summer,
When briers shall have leaves as well as thorns,
And be as sweet as sharp. We must away;
Our waggon is prepar’d, and time revives us.
All’s well that ends well; still the fine’s the crown.
Whate’er the course, the end is the renown.

 [_Exeunt._]

SCENE V. Rossillon. A room in the Countess’s palace.

 Enter Clown, Countess and Lafew.

LAFEW.
No, no, no, your son was misled with a snipt-taffeta fellow there,
whose villanous saffron would have made all the unbak’d and doughy
youth of a nation in his colour. Your daughter-in-law had been alive at
this hour, and your son here at home, more advanc’d by the king than by
that red-tail’d humble-bee I speak of.

COUNTESS.
I would I had not known him; it was the death of the most virtuous
gentlewoman that ever nature had praise for creating. If she had
partaken of my flesh and cost me the dearest groans of a mother, I
could not have owed her a more rooted love.

LAFEW.
’Twas a good lady, ’twas a good lady. We may pick a thousand salads ere
we light on such another herb.

CLOWN.
Indeed, sir, she was the sweet marjoram of the salad, or, rather, the
herb of grace.

LAFEW.
They are not herbs, you knave; they are nose-herbs.

CLOWN.
I am no great Nebuchadnezzar, sir; I have not much skill in grass.

LAFEW.
Whether dost thou profess thyself,—a knave or a fool?

CLOWN.
A fool, sir, at a woman’s service, and a knave at a man’s.

LAFEW.
Your distinction?

CLOWN.
I would cozen the man of his wife, and do his service.

LAFEW.
So you were a knave at his service indeed.

CLOWN.
And I would give his wife my bauble, sir, to do her service.

LAFEW.
I will subscribe for thee; thou art both knave and fool.

CLOWN.
At your service.

LAFEW.
No, no, no.

CLOWN.
Why, sir, if I cannot serve you, I can serve as great a prince as you
are.

LAFEW.
Who’s that? a Frenchman?

CLOWN.
Faith, sir, ’a has an English name; but his phisnomy is more hotter in
France than there.

LAFEW.
What prince is that?

CLOWN.
The black prince, sir; alias the prince of darkness; alias the devil.

LAFEW.
Hold thee, there’s my purse. I give thee not this to suggest thee from
thy master thou talk’st of; serve him still.

CLOWN.
I am a woodland fellow, sir, that always loved a great fire, and the
master I speak of ever keeps a good fire. But sure he is the prince of
the world; let his nobility remain in’s court. I am for the house with
the narrow gate, which I take to be too little for pomp to enter: some
that humble themselves may, but the many will be too chill and tender,
and they’ll be for the flow’ry way that leads to the broad gate and the
great fire.

LAFEW.
Go thy ways, I begin to be a-weary of thee; and I tell thee so before,
because I would not fall out with thee. Go thy ways; let my horses be
well look’d to, without any tricks.

CLOWN.
If I put any tricks upon ’em, sir, they shall be jades’ tricks, which
are their own right by the law of nature.

 [_Exit._]

LAFEW.
A shrewd knave, and an unhappy.

COUNTESS.
So he is. My lord that’s gone made himself much sport out of him; by
his authority he remains here, which he thinks is a patent for his
sauciness; and indeed he has no pace, but runs where he will.

LAFEW.
I like him well; ’tis not amiss. And I was about to tell you, since I
heard of the good lady’s death, and that my lord your son was upon his
return home, I moved the king my master to speak in the behalf of my
daughter; which, in the minority of them both, his majesty out of a
self-gracious remembrance did first propose. His highness hath promis’d
me to do it; and, to stop up the displeasure he hath conceived against
your son, there is no fitter matter. How does your ladyship like it?

COUNTESS.
With very much content, my lord, and I wish it happily effected.

LAFEW.
His highness comes post from Marseilles, of as able body as when he
number’d thirty; he will be here tomorrow, or I am deceived by him that
in such intelligence hath seldom fail’d.

COUNTESS.
It rejoices me that I hope I shall see him ere I die. I have letters
that my son will be here tonight. I shall beseech your lordship to
remain with me till they meet together.

LAFEW.
Madam, I was thinking with what manners I might safely be admitted.

COUNTESS.
You need but plead your honourable privilege.

LAFEW.
Lady, of that I have made a bold charter; but, I thank my God, it holds
yet.

 Enter Clown.

CLOWN.
O madam, yonder’s my lord your son with a patch of velvet on’s face;
whether there be a scar under’t or no, the velvet knows; but ’tis a
goodly patch of velvet. His left cheek is a cheek of two pile and a
half, but his right cheek is worn bare.

LAFEW.
A scar nobly got, or a noble scar, is a good livery of honour; so
belike is that.

CLOWN.
But it is your carbonado’d face.

LAFEW.
Let us go see your son, I pray you. I long to talk with the young noble
soldier.

CLOWN.
Faith, there’s a dozen of ’em, with delicate fine hats, and most
courteous feathers, which bow the head and nod at every man.

 [_Exeunt._]


ACT V.

SCENE I. Marseilles. A street.

 Enter Helena, Widow and Diana with two Attendants.

HELENA.
But this exceeding posting day and night
Must wear your spirits low. We cannot help it.
But since you have made the days and nights as one,
To wear your gentle limbs in my affairs,
Be bold you do so grow in my requital
As nothing can unroot you. In happy time;—

 Enter a Gentleman.

This man may help me to his majesty’s ear,
If he would spend his power. God save you, sir.

GENTLEMAN.
And you.

HELENA.
Sir, I have seen you in the court of France.

GENTLEMAN.
I have been sometimes there.

HELENA.
I do presume, sir, that you are not fallen
From the report that goes upon your goodness;
And therefore, goaded with most sharp occasions,
Which lay nice manners by, I put you to
The use of your own virtues, for the which
I shall continue thankful.

GENTLEMAN.
What’s your will?

HELENA.
That it will please you
To give this poor petition to the king,
And aid me with that store of power you have
To come into his presence.

GENTLEMAN.
The king’s not here.

HELENA.
Not here, sir?

GENTLEMAN.
Not indeed.
He hence remov’d last night, and with more haste
Than is his use.

WIDOW.
Lord, how we lose our pains!

HELENA.
All’s well that ends well yet,
Though time seem so adverse and means unfit.
I do beseech you, whither is he gone?

GENTLEMAN.
Marry, as I take it, to Rossillon;
Whither I am going.

HELENA.
I do beseech you, sir,
Since you are like to see the king before me,
Commend the paper to his gracious hand,
Which I presume shall render you no blame,
But rather make you thank your pains for it.
I will come after you with what good speed
Our means will make us means.

GENTLEMAN.
This I’ll do for you.

HELENA.
And you shall find yourself to be well thank’d,
Whate’er falls more. We must to horse again.
Go, go, provide.

 [_Exeunt._]

SCENE II. Rossillon. The inner court of the Countess’s palace.

 Enter Clown and Parolles.

PAROLLES.
Good Monsieur Lavache, give my Lord Lafew this letter; I have ere now,
sir, been better known to you, when I have held familiarity with
fresher clothes; but I am now, sir, muddied in Fortune’s mood, and
smell somewhat strong of her strong displeasure.

CLOWN.
Truly, Fortune’s displeasure is but sluttish, if it smell so strongly
as thou speak’st of. I will henceforth eat no fish of Fortune’s
buttering. Pr’ythee, allow the wind.

PAROLLES.
Nay, you need not to stop your nose, sir. I spake but by a metaphor.

CLOWN.
Indeed, sir, if your metaphor stink, I will stop my nose, or against
any man’s metaphor. Pr’ythee, get thee further.

PAROLLES.
Pray you, sir, deliver me this paper.

CLOWN.
Foh, pr’ythee stand away. A paper from Fortune’s close-stool to give to
a nobleman! Look here he comes himself.

 Enter Lafew.

Here is a pur of Fortune’s, sir, or of Fortune’s cat, but not a
musk-cat, that has fallen into the unclean fishpond of her displeasure,
and as he says, is muddied withal. Pray you, sir, use the carp as you
may, for he looks like a poor, decayed, ingenious, foolish, rascally
knave. I do pity his distress in my similes of comfort, and leave him
to your lordship.

 [_Exit._]

PAROLLES.
My lord, I am a man whom Fortune hath cruelly scratch’d.

LAFEW.
And what would you have me to do? ’Tis too late to pare her nails now.
Wherein have you played the knave with Fortune that she should scratch
you, who of herself is a good lady, and would not have knaves thrive
long under her? There’s a quart d’ecu for you. Let the justices make
you and Fortune friends; I am for other business.

PAROLLES.
I beseech your honour to hear me one single word.

LAFEW.
You beg a single penny more. Come, you shall ha’t; save your word.

PAROLLES.
My name, my good lord, is Parolles.

LAFEW.
You beg more than word then. Cox my passion! Give me your hand. How
does your drum?

PAROLLES.
O my good lord, you were the first that found me.

LAFEW.
Was I, in sooth? And I was the first that lost thee.

PAROLLES.
It lies in you, my lord, to bring me in some grace, for you did bring
me out.

LAFEW.
Out upon thee, knave! dost thou put upon me at once both the office of
God and the devil? One brings thee in grace, and the other brings thee
out.

 [_Trumpets sound._]

The king’s coming; I know by his trumpets. Sirrah, inquire further
after me. I had talk of you last night; though you are a fool and a
knave, you shall eat. Go to; follow.

PAROLLES.
I praise God for you.

 [_Exeunt._]

SCENE III. The same. A room in the Countess’s palace.

 Flourish. Enter King, Countess, Lafew, Lords, Gentlemen, Guards &c.

KING.
We lost a jewel of her, and our esteem
Was made much poorer by it; but your son,
As mad in folly, lack’d the sense to know
Her estimation home.

COUNTESS.
’Tis past, my liege,
And I beseech your majesty to make it
Natural rebellion, done i’ the blaze of youth,
When oil and fire, too strong for reason’s force,
O’erbears it and burns on.

KING.
My honour’d lady,
I have forgiven and forgotten all,
Though my revenges were high bent upon him,
And watch’d the time to shoot.

LAFEW.
This I must say,—
But first, I beg my pardon,—the young lord
Did to his majesty, his mother, and his lady,
Offence of mighty note; but to himself
The greatest wrong of all. He lost a wife
Whose beauty did astonish the survey
Of richest eyes; whose words all ears took captive;
Whose dear perfection hearts that scorn’d to serve
Humbly call’d mistress.

KING.
Praising what is lost
Makes the remembrance dear. Well, call him hither;
We are reconcil’d, and the first view shall kill
All repetition. Let him not ask our pardon;
The nature of his great offence is dead,
And deeper than oblivion do we bury
Th’ incensing relics of it. Let him approach
A stranger, no offender; and inform him
So ’tis our will he should.

GENTLEMAN.
I shall, my liege.

 [_Exit Gentleman._]

KING.
What says he to your daughter? Have you spoke?

LAFEW.
All that he is hath reference to your highness.

KING.
Then shall we have a match. I have letters sent me
That sets him high in fame.

 Enter Bertram.

LAFEW.
He looks well on ’t.

KING.
I am not a day of season,
For thou mayst see a sunshine and a hail
In me at once. But to the brightest beams
Distracted clouds give way; so stand thou forth;
The time is fair again.

BERTRAM.
My high-repented blames
Dear sovereign, pardon to me.

KING.
All is whole.
Not one word more of the consumed time.
Let’s take the instant by the forward top;
For we are old, and on our quick’st decrees
Th’inaudible and noiseless foot of time
Steals ere we can effect them. You remember
The daughter of this lord?

BERTRAM.
Admiringly, my liege. At first
I stuck my choice upon her, ere my heart
Durst make too bold herald of my tongue:
Where the impression of mine eye infixing,
Contempt his scornful perspective did lend me,
Which warp’d the line of every other favour,
Scorn’d a fair colour, or express’d it stolen,
Extended or contracted all proportions
To a most hideous object. Thence it came
That she whom all men prais’d, and whom myself,
Since I have lost, have lov’d, was in mine eye
The dust that did offend it.

KING.
Well excus’d:
That thou didst love her, strikes some scores away
From the great compt: but love that comes too late,
Like a remorseful pardon slowly carried,
To the great sender turns a sour offence,
Crying, That’s good that’s gone. Our rash faults
Make trivial price of serious things we have,
Not knowing them until we know their grave.
Oft our displeasures, to ourselves unjust,
Destroy our friends, and after weep their dust:
Our own love waking cries to see what’s done,
While shameful hate sleeps out the afternoon.
Be this sweet Helen’s knell, and now forget her.
Send forth your amorous token for fair Maudlin.
The main consents are had, and here we’ll stay
To see our widower’s second marriage-day.

COUNTESS.
Which better than the first, O dear heaven, bless!
Or, ere they meet, in me, O nature, cesse!

LAFEW.
Come on, my son, in whom my house’s name
Must be digested; give a favour from you,
To sparkle in the spirits of my daughter,
That she may quickly come.

 [_Bertram gives a ring to Lafew._]

By my old beard,
And ev’ry hair that’s on ’t, Helen that’s dead
Was a sweet creature: such a ring as this,
The last that e’er I took her leave at court,
I saw upon her finger.

BERTRAM.
Hers it was not.

KING.
Now, pray you, let me see it; for mine eye,
While I was speaking, oft was fasten’d to it.
This ring was mine; and when I gave it Helen
I bade her, if her fortunes ever stood
Necessitied to help, that by this token
I would relieve her. Had you that craft to ’reave her
Of what should stead her most?

BERTRAM.
My gracious sovereign,
Howe’er it pleases you to take it so,
The ring was never hers.

COUNTESS.
Son, on my life,
I have seen her wear it; and she reckon’d it
At her life’s rate.

LAFEW.
I am sure I saw her wear it.

BERTRAM.
You are deceiv’d, my lord; she never saw it.
In Florence was it from a casement thrown me,
Wrapp’d in a paper, which contain’d the name
Of her that threw it. Noble she was, and thought
I stood engag’d; but when I had subscrib’d
To mine own fortune, and inform’d her fully
I could not answer in that course of honour
As she had made the overture, she ceas’d,
In heavy satisfaction, and would never
Receive the ring again.

KING.
Plutus himself,
That knows the tinct and multiplying medicine,
Hath not in nature’s mystery more science
Than I have in this ring. ’Twas mine, ’twas Helen’s,
Whoever gave it you. Then if you know
That you are well acquainted with yourself,
Confess ’twas hers, and by what rough enforcement
You got it from her. She call’d the saints to surety
That she would never put it from her finger
Unless she gave it to yourself in bed,
Where you have never come, or sent it us
Upon her great disaster.

BERTRAM.
She never saw it.

KING.
Thou speak’st it falsely, as I love mine honour,
And mak’st conjectural fears to come into me
Which I would fain shut out. If it should prove
That thou art so inhuman,—’twill not prove so:
And yet I know not, thou didst hate her deadly.
And she is dead; which nothing but to close
Her eyes myself, could win me to believe
More than to see this ring. Take him away.

 [_Guards seize Bertram._]

My fore-past proofs, howe’er the matter fall,
Shall tax my fears of little vanity,
Having vainly fear’d too little. Away with him.
We’ll sift this matter further.

BERTRAM.
If you shall prove
This ring was ever hers, you shall as easy
Prove that I husbanded her bed in Florence,
Where she yet never was.

 [_Exit, guarded._]

KING.
I am wrapp’d in dismal thinkings.

 Enter a Gentleman.

GENTLEMAN.
Gracious sovereign,
Whether I have been to blame or no, I know not:
Here’s a petition from a Florentine,
Who hath for four or five removes come short
To tender it herself. I undertook it,
Vanquish’d thereto by the fair grace and speech
Of the poor suppliant, who by this, I know,
Is here attending: her business looks in her
With an importing visage, and she told me
In a sweet verbal brief, it did concern
Your highness with herself.

KING.
[_Reads._] _Upon his many protestations to marry me when his wife was
dead, I blush to say it, he won me. Now is the Count Rossillon a
widower; his vows are forfeited to me, and my honour’s paid to him. He
stole from Florence, taking no leave, and I follow him to his country
for justice. Grant it me, O king, in you it best lies; otherwise a
seducer flourishes, and a poor maid is undone._
                                    DIANA CAPILET.

LAFEW.
I will buy me a son-in-law in a fair, and toll for this. I’ll none of
him.

KING.
The heavens have thought well on thee, Lafew,
To bring forth this discovery. Seek these suitors.
Go speedily, and bring again the count.

 [_Exeunt Gentleman and some Attendants._]

I am afeard the life of Helen, lady,
Was foully snatch’d.

COUNTESS.
Now, justice on the doers!

 Enter Bertram, guarded.

KING.
I wonder, sir, since wives are monsters to you,
And that you fly them as you swear them lordship,
Yet you desire to marry. What woman’s that?

 Enter Widow and Diana.

DIANA.
I am, my lord, a wretched Florentine,
Derived from the ancient Capilet;
My suit, as I do understand, you know,
And therefore know how far I may be pitied.

WIDOW.
I am her mother, sir, whose age and honour
Both suffer under this complaint we bring,
And both shall cease, without your remedy.

KING.
Come hither, count; do you know these women?

BERTRAM.
My lord, I neither can nor will deny
But that I know them. Do they charge me further?

DIANA.
Why do you look so strange upon your wife?

BERTRAM.
She’s none of mine, my lord.

DIANA.
If you shall marry,
You give away this hand, and that is mine,
You give away heaven’s vows, and those are mine,
You give away myself, which is known mine;
For I by vow am so embodied yours
That she which marries you must marry me,
Either both or none.

LAFEW.
[_To Bertram_] Your reputation comes too short for my daughter; you are
no husband for her.

BERTRAM.
My lord, this is a fond and desperate creature
Whom sometime I have laugh’d with. Let your highness
Lay a more noble thought upon mine honour
Than for to think that I would sink it here.

KING.
Sir, for my thoughts, you have them ill to friend
Till your deeds gain them; fairer prove your honour
Than in my thought it lies!

DIANA.
Good my lord,
Ask him upon his oath, if he does think
He had not my virginity.

KING.
What say’st thou to her?

BERTRAM.
She’s impudent, my lord,
And was a common gamester to the camp.

DIANA.
He does me wrong, my lord; if I were so
He might have bought me at a common price.
Do not believe him. O, behold this ring,
Whose high respect and rich validity
Did lack a parallel; yet for all that
He gave it to a commoner o’ the camp,
If I be one.

COUNTESS.
He blushes, and ’tis it.
Of six preceding ancestors, that gem
Conferr’d by testament to th’ sequent issue,
Hath it been owed and worn. This is his wife;
That ring’s a thousand proofs.

KING.
Methought you said
You saw one here in court could witness it.

DIANA.
I did, my lord, but loath am to produce
So bad an instrument; his name’s Parolles.

LAFEW.
I saw the man today, if man he be.

KING.
Find him, and bring him hither.

 [_Exit an Attendant._]

BERTRAM.
What of him?
He’s quoted for a most perfidious slave,
With all the spots o’ the world tax’d and debauch’d:
Whose nature sickens but to speak a truth.
Am I or that or this for what he’ll utter,
That will speak anything?

KING.
She hath that ring of yours.

BERTRAM.
I think she has. Certain it is I lik’d her
And boarded her i’ the wanton way of youth.
She knew her distance, and did angle for me,
Madding my eagerness with her restraint,
As all impediments in fancy’s course
Are motives of more fancy; and in fine,
Her infinite cunning with her modern grace,
Subdu’d me to her rate; she got the ring,
And I had that which any inferior might
At market-price have bought.

DIANA.
I must be patient.
You that have turn’d off a first so noble wife
May justly diet me. I pray you yet,—
Since you lack virtue, I will lose a husband—
Send for your ring, I will return it home,
And give me mine again.

BERTRAM.
I have it not.

KING.
What ring was yours, I pray you?

DIANA.
Sir, much like
The same upon your finger.

KING.
Know you this ring? This ring was his of late.

DIANA.
And this was it I gave him, being abed.

KING.
The story then goes false you threw it him
Out of a casement.

DIANA.
I have spoke the truth.

 Enter Attendant with Parolles.

BERTRAM.
My lord, I do confess the ring was hers.

KING.
You boggle shrewdly; every feather starts you.
Is this the man you speak of?

DIANA.
Ay, my lord.

KING.
Tell me, sirrah, but tell me true I charge you,
Not fearing the displeasure of your master,
Which on your just proceeding, I’ll keep off,—
By him and by this woman here what know you?

PAROLLES.
So please your majesty, my master hath been an honourable gentleman.
Tricks he hath had in him, which gentlemen have.

KING.
Come, come, to the purpose. Did he love this woman?

PAROLLES.
Faith, sir, he did love her; but how?

KING.
How, I pray you?

PAROLLES.
He did love her, sir, as a gentleman loves a woman.

KING.
How is that?

PAROLLES.
He lov’d her, sir, and lov’d her not.

KING.
As thou art a knave and no knave.
What an equivocal companion is this!

PAROLLES.
I am a poor man, and at your majesty’s command.

LAFEW.
He’s a good drum, my lord, but a naughty orator.

DIANA.
Do you know he promised me marriage?

PAROLLES.
Faith, I know more than I’ll speak.

KING.
But wilt thou not speak all thou know’st?

PAROLLES.
Yes, so please your majesty. I did go between them as I said; but more
than that, he loved her, for indeed he was mad for her, and talked of
Satan, and of Limbo, and of furies, and I know not what: yet I was in
that credit with them at that time that I knew of their going to bed;
and of other motions, as promising her marriage, and things which would
derive me ill will to speak of; therefore I will not speak what I know.

KING.
Thou hast spoken all already, unless thou canst say they are married;
but thou art too fine in thy evidence; therefore stand aside. This
ring, you say, was yours?

DIANA.
Ay, my good lord.

KING.
Where did you buy it? Or who gave it you?

DIANA.
It was not given me, nor I did not buy it.

KING.
Who lent it you?

DIANA.
It was not lent me neither.

KING.
Where did you find it then?

DIANA.
I found it not.

KING.
If it were yours by none of all these ways,
How could you give it him?

DIANA.
I never gave it him.

LAFEW.
This woman’s an easy glove, my lord; she goes off and on at pleasure.

KING.
This ring was mine, I gave it his first wife.

DIANA.
It might be yours or hers for ought I know.

KING.
Take her away, I do not like her now.
To prison with her. And away with him.
Unless thou tell’st me where thou hadst this ring,
Thou diest within this hour.

DIANA.
I’ll never tell you.

KING.
Take her away.

DIANA.
I’ll put in bail, my liege.

KING.
I think thee now some common customer.

DIANA.
By Jove, if ever I knew man, ’twas you.

KING.
Wherefore hast thou accus’d him all this while?

DIANA.
Because he’s guilty, and he is not guilty.
He knows I am no maid, and he’ll swear to’t:
I’ll swear I am a maid, and he knows not.
Great King, I am no strumpet, by my life;
I am either maid, or else this old man’s wife.

 [_Pointing to Lafew._]

KING.
She does abuse our ears; to prison with her.

DIANA.
Good mother, fetch my bail. Stay, royal sir;

 [_Exit Widow._]

The jeweller that owes the ring is sent for,
And he shall surety me. But for this lord
Who hath abus’d me as he knows himself,
Though yet he never harm’d me, here I quit him.
He knows himself my bed he hath defil’d;
And at that time he got his wife with child.
Dead though she be, she feels her young one kick;
So there’s my riddle: one that’s dead is quick,
And now behold the meaning.

 Enter Widow with Helena.

KING.
Is there no exorcist
Beguiles the truer office of mine eyes?
Is’t real that I see?

HELENA.
No, my good lord;
’Tis but the shadow of a wife you see,
The name, and not the thing.

BERTRAM.
Both, both. O, pardon!

HELENA.
O, my good lord, when I was like this maid;
I found you wondrous kind. There is your ring,
And, look you, here’s your letter. This it says,
‘When from my finger you can get this ring,
And is by me with child, &c.’ This is done;
Will you be mine now you are doubly won?

BERTRAM.
If she, my liege, can make me know this clearly,
I’ll love her dearly, ever, ever dearly.

HELENA.
If it appear not plain, and prove untrue,
Deadly divorce step between me and you!
O my dear mother, do I see you living?

LAFEW.
Mine eyes smell onions; I shall weep anon.
[_to Parolles_] Good Tom Drum, lend me a handkercher.
So, I thank thee. Wait on me home, I’ll make sport with thee.
Let thy courtesies alone, they are scurvy ones.

KING.
Let us from point to point this story know,
To make the even truth in pleasure flow.
[_To Diana._] If thou beest yet a fresh uncropped flower,
Choose thou thy husband, and I’ll pay thy dower;
For I can guess that by thy honest aid,
Thou kept’st a wife herself, thyself a maid.
Of that and all the progress more and less,
Resolvedly more leisure shall express.
All yet seems well, and if it end so meet,
The bitter past, more welcome is the sweet.

 [_Flourish._]

[EPILOGUE]

_The king’s a beggar, now the play is done;
All is well ended if this suit be won,
That you express content; which we will pay
With strife to please you, day exceeding day.
Ours be your patience then, and yours our parts;
Your gentle hands lend us, and take our hearts._

 [_Exeunt omnes._]




THE TRAGEDY OF ANTONY AND CLEOPATRA


Contents

ACT I
Scene I.
Alexandria. A Room in Cleopatra’s palace.
Scene II.
Alexandria. Another Room in Cleopatra’s palace.
Scene III.
Alexandria. A Room in Cleopatra’s palace.
Scene IV.
Rome. An Apartment in Caesar’s House
Scene V.
Alexandria. A Room in the Palace.

ACT II
Scene I.
Messina. A Room in Pompey’s house.
Scene II.
Rome. A Room in the House of Lepidus.
Scene III.
Rome. A Room in Caesar’s House.
Scene IV.
Rome. A street.
Scene V.
Alexandria. A Room in the Palace.
Scene VI.
Near Misenum.
Scene VII.
On board Pompey’s Galley, lying near Misenum.

ACT III
Scene I.
A plain in Syria.
Scene II.
Rome. An Ante-chamber in Caesar’s house.
Scene III.
Alexandria. A Room in the Palace.
Scene IV.
Athens. A Room in Antony’s House.
Scene V.
Athens. Another Room in Antony’s House.
Scene VI.
Rome. A Room in Caesar’s House.
Scene VII.
Antony’s Camp near the Promontory of Actium.
Scene VIII.
A plain near Actium.
Scene IX.
Another part of the Plain.
Scene X.
Another part of the Plain.
Scene XI.
Alexandria. A Room in the Palace.
Scene XII.
Caesar’s camp in Egypt.
Scene XIII.
Alexandria. A Room in the Palace.

ACT IV
Scene I.
Caesar’s Camp at Alexandria.
Scene II.
Alexandria. A Room in the Palace.
Scene III.
Alexandria. Before the Palace.
Scene IV.
Alexandria. A Room in the Palace.
Scene V.
Antony’s camp near Alexandria.
Scene VI.
Alexandria. Caesar’s camp.
Scene VII.
Field of battle between the Camps.
Scene VIII.
Under the Walls of Alexandria.
Scene IX.
Caesar’s camp.
Scene X.
Ground between the two Camps.
Scene XI.
Another part of the Ground.
Scene XII.
Another part of the Ground.
Scene XIII.
Alexandria. A Room in the Palace.
Scene XIV.
Alexandria. Another Room.
Scene XV.
Alexandria. A monument.

ACT V
Scene I.
Caesar’s Camp before Alexandria.
Scene II.
Alexandria. A Room in the Monument.


Dramatis Personæ

MARK ANTONY, Triumvir
OCTAVIUS CAESAR, Triumvir
LEPIDUS, Triumvir
SEXTUS POMPEIUS,
DOMITIUS ENOBARBUS, friend to Antony
VENTIDIUS, friend to Antony
EROS, friend to Antony
SCARUS, friend to Antony
DERCETUS, friend to Antony
DEMETRIUS, friend to Antony
PHILO, friend to Antony
MAECENAS, friend to Caesar
AGRIPPA, friend to Caesar
DOLABELLA, friend to Caesar
PROCULEIUS, friend to Caesar
THIDIAS, friend to Caesar
GALLUS, friend to Caesar
MENAS, friend to Pompey
MENECRATES, friend to Pompey
VARRIUS, friend to Pompey
TAURUS, Lieutenant-General to Caesar
CANIDIUS, Lieutenant-General to Antony
SILIUS, an Officer in Ventidius’s army
EUPHRONIUS, an Ambassador from Antony to Caesar
ALEXAS, attendant on Cleopatra
MARDIAN, attendant on Cleopatra
SELEUCUS, attendant on Cleopatra
DIOMEDES, attendant on Cleopatra
A SOOTHSAYER
A CLOWN

CLEOPATRA, Queen of Egypt
OCTAVIA, sister to Caesar and wife to Antony
CHARMIAN, Attendant on Cleopatra
IRAS, Attendant on Cleopatra

Officers, Soldiers, Messengers, and other Attendants

SCENE: Dispersed, in several parts of the Roman Empire.




ACT I


SCENE I. Alexandria. A Room in Cleopatra’s palace.

 Enter Demetrius and Philo.

PHILO.
Nay, but this dotage of our general’s
O’erflows the measure. Those his goodly eyes,
That o’er the files and musters of the war
Have glowed like plated Mars, now bend, now turn
The office and devotion of their view
Upon a tawny front. His captain’s heart,
Which in the scuffles of great fights hath burst
The buckles on his breast, reneges all temper
And is become the bellows and the fan
To cool a gipsy’s lust.

 Flourish. Enter Antony and Cleopatra, her Ladies, the Train, with
 Eunuchs fanning her.

Look where they come:
Take but good note, and you shall see in him
The triple pillar of the world transform’d
Into a strumpet’s fool. Behold and see.

CLEOPATRA.
If it be love indeed, tell me how much.

ANTONY.
There’s beggary in the love that can be reckoned.

CLEOPATRA.
I’ll set a bourn how far to be beloved.

ANTONY.
Then must thou needs find out new heaven, new earth.

 Enter a Messenger.

MESSENGER.
News, my good lord, from Rome.

ANTONY.
Grates me, the sum.

CLEOPATRA.
Nay, hear them, Antony.
Fulvia perchance is angry; or who knows
If the scarce-bearded Caesar have not sent
His powerful mandate to you: “Do this or this;
Take in that kingdom and enfranchise that.
Perform’t, or else we damn thee.”

ANTONY.
How, my love?

CLEOPATRA.
Perchance! Nay, and most like.
You must not stay here longer; your dismission
Is come from Caesar; therefore hear it, Antony.
Where’s Fulvia’s process?—Caesar’s I would say? Both?
Call in the messengers. As I am Egypt’s queen,
Thou blushest, Antony, and that blood of thine
Is Caesar’s homager; else so thy cheek pays shame
When shrill-tongued Fulvia scolds. The messengers!

ANTONY.
Let Rome in Tiber melt, and the wide arch
Of the ranged empire fall! Here is my space.
Kingdoms are clay. Our dungy earth alike
Feeds beast as man. The nobleness of life
Is to do thus [_Embracing_]; when such a mutual pair
And such a twain can do’t, in which I bind,
On pain of punishment, the world to weet
We stand up peerless.

CLEOPATRA.
Excellent falsehood!
Why did he marry Fulvia, and not love her?
I’ll seem the fool I am not. Antony
Will be himself.

ANTONY.
But stirred by Cleopatra.
Now, for the love of Love and her soft hours,
Let’s not confound the time with conference harsh.
There’s not a minute of our lives should stretch
Without some pleasure now. What sport tonight?

CLEOPATRA.
Hear the ambassadors.

ANTONY.
Fie, wrangling queen!
Whom everything becomes—to chide, to laugh,
To weep; whose every passion fully strives
To make itself, in thee fair and admired!
No messenger but thine, and all alone
Tonight we’ll wander through the streets and note
The qualities of people. Come, my queen,
Last night you did desire it. Speak not to us.

 [_Exeunt Antony and Cleopatra with the Train._]

DEMETRIUS.
Is Caesar with Antonius prized so slight?

PHILO.
Sir, sometimes when he is not Antony,
He comes too short of that great property
Which still should go with Antony.

DEMETRIUS.
I am full sorry
That he approves the common liar who
Thus speaks of him at Rome, but I will hope
Of better deeds tomorrow. Rest you happy!

 [_Exeunt._]

SCENE II. Alexandria. Another Room in Cleopatra’s palace.

 Enter Enobarbus, a Soothsayer, Charmian, Iras, Mardian and Alexas.

CHARMIAN.
Lord Alexas, sweet Alexas, most anything Alexas, almost most absolute
Alexas, where’s the soothsayer that you praised so to th’ queen? O,
that I knew this husband which you say must charge his horns with
garlands!

ALEXAS.
Soothsayer!

SOOTHSAYER.
Your will?

CHARMIAN.
Is this the man? Is’t you, sir, that know things?

SOOTHSAYER.
In nature’s infinite book of secrecy
A little I can read.

ALEXAS.
Show him your hand.

ENOBARBUS.
Bring in the banquet quickly; wine enough
Cleopatra’s health to drink.

CHARMIAN.
Good, sir, give me good fortune.

SOOTHSAYER.
I make not, but foresee.

CHARMIAN.
Pray, then, foresee me one.

SOOTHSAYER.
You shall be yet far fairer than you are.

CHARMIAN.
He means in flesh.

IRAS.
No, you shall paint when you are old.

CHARMIAN.
Wrinkles forbid!

ALEXAS.
Vex not his prescience. Be attentive.

CHARMIAN.
Hush!

SOOTHSAYER.
You shall be more beloving than beloved.

CHARMIAN.
I had rather heat my liver with drinking.

ALEXAS.
Nay, hear him.

CHARMIAN.
Good now, some excellent fortune! Let me be married to three kings in a
forenoon and widow them all. Let me have a child at fifty, to whom
Herod of Jewry may do homage. Find me to marry me with Octavius Caesar,
and companion me with my mistress.

SOOTHSAYER.
You shall outlive the lady whom you serve.

CHARMIAN.
O, excellent! I love long life better than figs.

SOOTHSAYER.
You have seen and proved a fairer former fortune
Than that which is to approach.

CHARMIAN.
Then belike my children shall have no names. Prithee, how many boys and
wenches must I have?

SOOTHSAYER.
If every of your wishes had a womb,
And fertile every wish, a million.

CHARMIAN.
Out, fool! I forgive thee for a witch.

ALEXAS.
You think none but your sheets are privy to your wishes.

CHARMIAN.
Nay, come, tell Iras hers.

ALEXAS.
We’ll know all our fortunes.

ENOBARBUS.
Mine, and most of our fortunes tonight, shall be drunk to bed.

IRAS.
There’s a palm presages chastity, if nothing else.

CHARMIAN.
E’en as the o’erflowing Nilus presageth famine.

IRAS.
Go, you wild bedfellow, you cannot soothsay.

CHARMIAN.
Nay, if an oily palm be not a fruitful prognostication, I cannot
scratch mine ear. Prithee, tell her but workaday fortune.

SOOTHSAYER.
Your fortunes are alike.

IRAS.
But how, but how? give me particulars.

SOOTHSAYER.
I have said.

IRAS.
Am I not an inch of fortune better than she?

CHARMIAN.
Well, if you were but an inch of fortune better than I, where would you
choose it?

IRAS.
Not in my husband’s nose.

CHARMIAN.
Our worser thoughts heavens mend! Alexas—come, his fortune! his
fortune! O, let him marry a woman that cannot go, sweet Isis, I beseech
thee, and let her die too, and give him a worse, and let worse follow
worse, till the worst of all follow him laughing to his grave,
fiftyfold a cuckold! Good Isis, hear me this prayer, though thou deny
me a matter of more weight; good Isis, I beseech thee!

IRAS.
Amen. Dear goddess, hear that prayer of the people! For, as it is a
heartbreaking to see a handsome man loose-wived, so it is a deadly
sorrow to behold a foul knave uncuckolded. Therefore, dear Isis, keep
decorum and fortune him accordingly!

CHARMIAN.
Amen.

ALEXAS.
Lo now, if it lay in their hands to make me a cuckold, they would make
themselves whores but they’d do’t!

 Enter Cleopatra.

ENOBARBUS.
Hush, Here comes Antony.

CHARMIAN.
Not he, the queen.

CLEOPATRA.
Saw you my lord?

ENOBARBUS.
No, lady.

CLEOPATRA.
Was he not here?

CHARMIAN.
No, madam.

CLEOPATRA.
He was disposed to mirth; but on the sudden
A Roman thought hath struck him. Enobarbus!

ENOBARBUS.
Madam?

CLEOPATRA.
Seek him and bring him hither. Where’s Alexas?

ALEXAS.
Here, at your service. My lord approaches.

 Enter Antony with a Messenger.

CLEOPATRA.
We will not look upon him. Go with us.

 [_Exeunt Cleopatra, Enobarbus, Charmian, Iras, Alexas and
 Soothsayer._]

MESSENGER.
Fulvia thy wife first came into the field.

ANTONY.
Against my brother Lucius.

MESSENGER.
Ay.
But soon that war had end, and the time’s state
Made friends of them, jointing their force ’gainst Caesar,
Whose better issue in the war from Italy
Upon the first encounter drave them.

ANTONY.
Well, what worst?

MESSENGER.
The nature of bad news infects the teller.

ANTONY.
When it concerns the fool or coward. On.
Things that are past are done with me. ’Tis thus:
Who tells me true, though in his tale lie death,
I hear him as he flattered.

MESSENGER.
Labienus—
This is stiff news—hath with his Parthian force
Extended Asia from Euphrates
His conquering banner shook from Syria
To Lydia and to Ionia,
Whilst—

ANTONY.
“Antony”, thou wouldst say—

MESSENGER.
O, my lord!

ANTONY.
Speak to me home; mince not the general tongue.
Name Cleopatra as she is called in Rome;
Rail thou in Fulvia’s phrase, and taunt my faults
With such full licence as both truth and malice
Have power to utter. O, then we bring forth weeds
When our quick minds lie still, and our ills told us
Is as our earing. Fare thee well awhile.

MESSENGER.
At your noble pleasure.

 [_Exit Messenger._]

 Enter another Messenger.

ANTONY.
From Sicyon, ho, the news? Speak there!

SECOND MESSENGER.
The man from Sicyon—

ANTONY.
Is there such a one?

SECOND MESSENGER.
He stays upon your will.

ANTONY.
Let him appear.

 [_Exit second Messenger._]

These strong Egyptian fetters I must break,
Or lose myself in dotage.

 Enter another Messenger with a letter.

What are you?

THIRD MESSENGER.
Fulvia thy wife is dead.

ANTONY.
Where died she?

THIRD MESSENGER.
In Sicyon:
Her length of sickness, with what else more serious
Importeth thee to know, this bears.

 [_Gives a letter._]

ANTONY.
Forbear me.

 [_Exit third Messenger._]

There’s a great spirit gone! Thus did I desire it.
What our contempts doth often hurl from us,
We wish it ours again. The present pleasure,
By revolution lowering, does become
The opposite of itself. She’s good, being gone.
The hand could pluck her back that shoved her on.
I must from this enchanting queen break off.
Ten thousand harms, more than the ills I know,
My idleness doth hatch. How now, Enobarbus!

 Enter Enobarbus.

ENOBARBUS.
What’s your pleasure, sir?

ANTONY.
I must with haste from hence.

ENOBARBUS.
Why then we kill all our women. We see how mortal an unkindness is to
them. If they suffer our departure, death’s the word.

ANTONY.
I must be gone.

ENOBARBUS.
Under a compelling occasion, let women die. It were pity to cast them
away for nothing, though, between them and a great cause they should be
esteemed nothing. Cleopatra, catching but the least noise of this, dies
instantly. I have seen her die twenty times upon far poorer moment. I
do think there is mettle in death which commits some loving act upon
her, she hath such a celerity in dying.

ANTONY.
She is cunning past man’s thought.

ENOBARBUS.
Alack, sir, no; her passions are made of nothing but the finest part of
pure love. We cannot call her winds and waters sighs and tears; they
are greater storms and tempests than almanacs can report. This cannot
be cunning in her; if it be, she makes a shower of rain as well as
Jove.

ANTONY.
Would I had never seen her!

ENOBARBUS.
O, sir, you had then left unseen a wonderful piece of work, which not
to have been blest withal would have discredited your travel.

ANTONY.
Fulvia is dead.

ENOBARBUS.
Sir?

ANTONY.
Fulvia is dead.

ENOBARBUS.
Fulvia?

ANTONY.
Dead.

ENOBARBUS.
Why, sir, give the gods a thankful sacrifice. When it pleaseth their
deities to take the wife of a man from him, it shows to man the tailors
of the earth; comforting therein that when old robes are worn out,
there are members to make new. If there were no more women but Fulvia,
then had you indeed a cut, and the case to be lamented. This grief is
crowned with consolation; your old smock brings forth a new petticoat:
and indeed the tears live in an onion that should water this sorrow.

ANTONY.
The business she hath broached in the state
Cannot endure my absence.

ENOBARBUS.
And the business you have broached here cannot be without you,
especially that of Cleopatra’s, which wholly depends on your abode.

ANTONY.
No more light answers. Let our officers
Have notice what we purpose. I shall break
The cause of our expedience to the Queen,
And get her leave to part. For not alone
The death of Fulvia, with more urgent touches,
Do strongly speak to us, but the letters too
Of many our contriving friends in Rome
Petition us at home. Sextus Pompeius
Hath given the dare to Caesar, and commands
The empire of the sea. Our slippery people,
Whose love is never linked to the deserver
Till his deserts are past, begin to throw
Pompey the Great and all his dignities
Upon his son, who, high in name and power,
Higher than both in blood and life, stands up
For the main soldier; whose quality, going on,
The sides o’ th’ world may danger. Much is breeding
Which, like the courser’s hair, hath yet but life
And not a serpent’s poison. Say our pleasure
To such whose place is under us, requires
Our quick remove from hence.

ENOBARBUS.
I shall do’t.

 [_Exeunt._]

SCENE III. Alexandria. A Room in Cleopatra’s palace.

 Enter Cleopatra, Charmian, Alexas and Iras.

CLEOPATRA.
Where is he?

CHARMIAN.
I did not see him since.

CLEOPATRA.
See where he is, who’s with him, what he does.
I did not send you. If you find him sad,
Say I am dancing; if in mirth, report
That I am sudden sick. Quick, and return.

 [_Exit Alexas._]

CHARMIAN.
Madam, methinks, if you did love him dearly,
You do not hold the method to enforce
The like from him.

CLEOPATRA.
What should I do I do not?

CHARMIAN.
In each thing give him way; cross him in nothing.

CLEOPATRA.
Thou teachest like a fool: the way to lose him.

CHARMIAN.
Tempt him not so too far; I wish, forbear.
In time we hate that which we often fear.
But here comes Antony.

 Enter Antony.

CLEOPATRA.
I am sick and sullen.

ANTONY.
I am sorry to give breathing to my purpose—

CLEOPATRA.
Help me away, dear Charmian! I shall fall.
It cannot be thus long; the sides of nature
Will not sustain it.

ANTONY.
Now, my dearest queen—

CLEOPATRA.
Pray you, stand farther from me.

ANTONY.
What’s the matter?

CLEOPATRA.
I know by that same eye there’s some good news.
What, says the married woman you may go?
Would she had never given you leave to come!
Let her not say ’tis I that keep you here.
I have no power upon you; hers you are.

ANTONY.
The gods best know—

CLEOPATRA.
O, never was there queen
So mightily betrayed! Yet at the first
I saw the treasons planted.

ANTONY.
Cleopatra—

CLEOPATRA.
Why should I think you can be mine and true,
Though you in swearing shake the throned gods,
Who have been false to Fulvia? Riotous madness,
To be entangled with those mouth-made vows
Which break themselves in swearing!

ANTONY.
Most sweet queen—

CLEOPATRA.
Nay, pray you seek no colour for your going,
But bid farewell and go. When you sued staying,
Then was the time for words. No going then,
Eternity was in our lips and eyes,
Bliss in our brows’ bent; none our parts so poor
But was a race of heaven. They are so still,
Or thou, the greatest soldier of the world,
Art turned the greatest liar.

ANTONY.
How now, lady!

CLEOPATRA.
I would I had thy inches, thou shouldst know
There were a heart in Egypt.

ANTONY.
Hear me, queen:
The strong necessity of time commands
Our services awhile, but my full heart
Remains in use with you. Our Italy
Shines o’er with civil swords; Sextus Pompeius
Makes his approaches to the port of Rome;
Equality of two domestic powers
Breed scrupulous faction; the hated, grown to strength,
Are newly grown to love; the condemned Pompey,
Rich in his father’s honour, creeps apace
Into the hearts of such as have not thrived
Upon the present state, whose numbers threaten;
And quietness, grown sick of rest, would purge
By any desperate change. My more particular,
And that which most with you should safe my going,
Is Fulvia’s death.

CLEOPATRA.
Though age from folly could not give me freedom,
It does from childishness. Can Fulvia die?

ANTONY.
She’s dead, my queen.
Look here, and at thy sovereign leisure read
The garboils she awaked; at the last, best,
See when and where she died.

CLEOPATRA.
O most false love!
Where be the sacred vials thou shouldst fill
With sorrowful water? Now I see, I see,
In Fulvia’s death how mine received shall be.

ANTONY.
Quarrel no more, but be prepared to know
The purposes I bear; which are, or cease,
As you shall give th’ advice. By the fire
That quickens Nilus’ slime, I go from hence
Thy soldier, servant, making peace or war
As thou affects.

CLEOPATRA.
Cut my lace, Charmian, come!
But let it be; I am quickly ill and well,
So Antony loves.

ANTONY.
My precious queen, forbear,
And give true evidence to his love, which stands
An honourable trial.

CLEOPATRA.
So Fulvia told me.
I prithee, turn aside and weep for her,
Then bid adieu to me, and say the tears
Belong to Egypt. Good now, play one scene
Of excellent dissembling, and let it look
Like perfect honour.

ANTONY.
You’ll heat my blood. No more.

CLEOPATRA.
You can do better yet, but this is meetly.

ANTONY.
Now, by my sword—

CLEOPATRA.
And target. Still he mends.
But this is not the best. Look, prithee, Charmian,
How this Herculean Roman does become
The carriage of his chafe.

ANTONY.
I’ll leave you, lady.

CLEOPATRA.
Courteous lord, one word.
Sir, you and I must part, but that’s not it;
Sir, you and I have loved, but there’s not it;
That you know well. Something it is I would—
O, my oblivion is a very Antony,
And I am all forgotten.

ANTONY.
But that your royalty
Holds idleness your subject, I should take you
For idleness itself.

CLEOPATRA.
’Tis sweating labour
To bear such idleness so near the heart
As Cleopatra this. But, sir, forgive me,
Since my becomings kill me when they do not
Eye well to you. Your honour calls you hence;
Therefore be deaf to my unpitied folly,
And all the gods go with you! Upon your sword
Sit laurel victory, and smooth success
Be strewed before your feet!

ANTONY.
Let us go. Come.
Our separation so abides and flies
That thou, residing here, goes yet with me,
And I, hence fleeting, here remain with thee.
Away!

 [_Exeunt._]

SCENE IV. Rome. An Apartment in Caesar’s House.

 Enter Octavius [Caesar], Lepidus and their train.

CAESAR.
You may see, Lepidus, and henceforth know,
It is not Caesar’s natural vice to hate
Our great competitor. From Alexandria
This is the news: he fishes, drinks, and wastes
The lamps of night in revel: is not more manlike
Than Cleopatra, nor the queen of Ptolemy
More womanly than he; hardly gave audience, or
Vouchsafed to think he had partners. You shall find there
A man who is the abstract of all faults
That all men follow.

LEPIDUS.
I must not think there are
Evils enough to darken all his goodness.
His faults in him seem as the spots of heaven,
More fiery by night’s blackness; hereditary
Rather than purchased; what he cannot change
Than what he chooses.

CAESAR.
You are too indulgent. Let’s grant it is not
Amiss to tumble on the bed of Ptolemy,
To give a kingdom for a mirth, to sit
And keep the turn of tippling with a slave,
To reel the streets at noon, and stand the buffet
With knaves that smell of sweat. Say this becomes him—
As his composure must be rare indeed
Whom these things cannot blemish—yet must Antony
No way excuse his foils when we do bear
So great weight in his lightness. If he filled
His vacancy with his voluptuousness,
Full surfeits and the dryness of his bones
Call on him for’t. But to confound such time
That drums him from his sport, and speaks as loud
As his own state and ours, ’tis to be chid
As we rate boys who, being mature in knowledge,
Pawn their experience to their present pleasure
And so rebel to judgment.

 Enter a Messenger.

LEPIDUS.
Here’s more news.

MESSENGER.
Thy biddings have been done, and every hour,
Most noble Caesar, shalt thou have report
How ’tis abroad. Pompey is strong at sea,
And it appears he is beloved of those
That only have feared Caesar. To the ports
The discontents repair, and men’s reports
Give him much wronged.

CAESAR.
I should have known no less.
It hath been taught us from the primal state
That he which is was wished until he were,
And the ebbed man, ne’er loved till ne’er worth love,
Comes deared by being lacked. This common body,
Like to a vagabond flag upon the stream,
Goes to and back, lackeying the varying tide,
To rot itself with motion.

 Enter a second Messenger.

SECOND MESSENGER.
Caesar, I bring thee word
Menecrates and Menas, famous pirates,
Make the sea serve them, which they ear and wound
With keels of every kind. Many hot inroads
They make in Italy—the borders maritime
Lack blood to think on’t—and flush youth revolt.
No vessel can peep forth but ’tis as soon
Taken as seen; for Pompey’s name strikes more
Than could his war resisted.

CAESAR.
Antony,
Leave thy lascivious wassails. When thou once
Was beaten from Modena, where thou slew’st
Hirtius and Pansa, consuls, at thy heel
Did famine follow, whom thou fought’st against,
Though daintily brought up, with patience more
Than savages could suffer. Thou didst drink
The stale of horses and the gilded puddle
Which beasts would cough at. Thy palate then did deign
The roughest berry on the rudest hedge.
Yea, like the stag when snow the pasture sheets,
The barks of trees thou browsed. On the Alps
It is reported thou didst eat strange flesh
Which some did die to look on. And all this—
It wounds thine honour that I speak it now—
Was borne so like a soldier that thy cheek
So much as lanked not.

LEPIDUS.
’Tis pity of him.

CAESAR.
Let his shames quickly
Drive him to Rome. ’Tis time we twain
Did show ourselves i’ th’ field, and to that end
Assemble we immediate council. Pompey
Thrives in our idleness.

LEPIDUS.
Tomorrow, Caesar,
I shall be furnished to inform you rightly
Both what by sea and land I can be able
To front this present time.

CAESAR.
Till which encounter
It is my business too. Farewell.

LEPIDUS.
Farewell, my lord. What you shall know meantime
Of stirs abroad, I shall beseech you, sir,
To let me be partaker.

CAESAR.
Doubt not, sir.
I knew it for my bond.

 [_Exeunt._]

SCENE V. Alexandria. A Room in the Palace.

 Enter Cleopatra, Charmian, Iras and Mardian.

CLEOPATRA.
Charmian!

CHARMIAN.
Madam?

CLEOPATRA.
Ha, ha!
Give me to drink mandragora.

CHARMIAN.
Why, madam?

CLEOPATRA.
That I might sleep out this great gap of time
My Antony is away.

CHARMIAN.
You think of him too much.

CLEOPATRA.
O, ’tis treason!

CHARMIAN.
Madam, I trust not so.

CLEOPATRA.
Thou, eunuch Mardian!

MARDIAN.
What’s your highness’ pleasure?

CLEOPATRA.
Not now to hear thee sing. I take no pleasure
In aught an eunuch has. ’Tis well for thee
That, being unseminared, thy freer thoughts
May not fly forth of Egypt. Hast thou affections?

MARDIAN.
Yes, gracious madam.

CLEOPATRA.
Indeed?

MARDIAN.
Not in deed, madam, for I can do nothing
But what indeed is honest to be done.
Yet have I fierce affections, and think
What Venus did with Mars.

CLEOPATRA.
O, Charmian,
Where think’st thou he is now? Stands he, or sits he?
Or does he walk? Or is he on his horse?
O happy horse, to bear the weight of Antony!
Do bravely, horse, for wot’st thou whom thou mov’st?
The demi-Atlas of this earth, the arm
And burgonet of men. He’s speaking now,
Or murmuring “Where’s my serpent of old Nile?”
For so he calls me. Now I feed myself
With most delicious poison. Think on me
That am with Phœbus’ amorous pinches black,
And wrinkled deep in time? Broad-fronted Caesar,
When thou wast here above the ground, I was
A morsel for a monarch. And great Pompey
Would stand and make his eyes grow in my brow;
There would he anchor his aspect, and die
With looking on his life.

 Enter Alexas.

ALEXAS.
Sovereign of Egypt, hail!

CLEOPATRA.
How much unlike art thou Mark Antony!
Yet, coming from him, that great medicine hath
With his tinct gilded thee.
How goes it with my brave Mark Antony?

ALEXAS.
Last thing he did, dear queen,
He kissed—the last of many doubled kisses—
This orient pearl. His speech sticks in my heart.

CLEOPATRA.
Mine ear must pluck it thence.

ALEXAS.
“Good friend,” quoth he,
“Say, the firm Roman to great Egypt sends
This treasure of an oyster; at whose foot,
To mend the petty present, I will piece
Her opulent throne with kingdoms. All the east,
Say thou, shall call her mistress.” So he nodded
And soberly did mount an arm-gaunt steed,
Who neighed so high that what I would have spoke
Was beastly dumbed by him.

CLEOPATRA.
What, was he sad or merry?

ALEXAS.
Like to the time o’ th’ year between the extremes
Of hot and cold, he was nor sad nor merry.

CLEOPATRA.
O well-divided disposition!—Note him,
Note him, good Charmian, ’tis the man; but note him:
He was not sad, for he would shine on those
That make their looks by his; he was not merry,
Which seemed to tell them his remembrance lay
In Egypt with his joy; but between both.
O heavenly mingle!—Be’st thou sad or merry,
The violence of either thee becomes,
So does it no man else.—Met’st thou my posts?

ALEXAS.
Ay, madam, twenty several messengers.
Why do you send so thick?

CLEOPATRA.
Who’s born that day
When I forget to send to Antony
Shall die a beggar.—Ink and paper, Charmian.—
Welcome, my good Alexas.—Did I, Charmian,
Ever love Caesar so?

CHARMIAN.
O that brave Caesar!

CLEOPATRA.
Be choked with such another emphasis!
Say “the brave Antony.”

CHARMIAN.
The valiant Caesar!

CLEOPATRA.
By Isis, I will give thee bloody teeth
If thou with Caesar paragon again
My man of men.

CHARMIAN.
By your most gracious pardon,
I sing but after you.

CLEOPATRA.
My salad days,
When I was green in judgment, cold in blood,
To say as I said then. But come, away,
Get me ink and paper.
He shall have every day a several greeting,
Or I’ll unpeople Egypt.

 [_Exeunt._]




ACT II


SCENE I. Messina. A Room in Pompey’s house.

 Enter Pompey, Menecrates and Menas in warlike manner.

POMPEY.
If the great gods be just, they shall assist
The deeds of justest men.

MENECRATES.
Know, worthy Pompey,
That what they do delay they not deny.

POMPEY.
Whiles we are suitors to their throne, decays
The thing we sue for.

MENECRATES.
We, ignorant of ourselves,
Beg often our own harms, which the wise powers
Deny us for our good; so find we profit
By losing of our prayers.

POMPEY.
I shall do well.
The people love me, and the sea is mine;
My powers are crescent, and my auguring hope
Says it will come to th’ full. Mark Antony
In Egypt sits at dinner, and will make
No wars without doors. Caesar gets money where
He loses hearts. Lepidus flatters both,
Of both is flattered; but he neither loves
Nor either cares for him.

MENAS.
Caesar and Lepidus
Are in the field. A mighty strength they carry.

POMPEY.
Where have you this? ’Tis false.

MENAS.
From Silvius, sir.

POMPEY.
He dreams. I know they are in Rome together,
Looking for Antony. But all the charms of love,
Salt Cleopatra, soften thy waned lip!
Let witchcraft join with beauty, lust with both;
Tie up the libertine in a field of feasts;
Keep his brain fuming. Epicurean cooks
Sharpen with cloyless sauce his appetite,
That sleep and feeding may prorogue his honour
Even till a Lethe’d dullness—

 Enter Varrius.

How now, Varrius!

VARRIUS.
This is most certain that I shall deliver:
Mark Antony is every hour in Rome
Expected. Since he went from Egypt ’tis
A space for farther travel.

POMPEY.
I could have given less matter
A better ear.—Menas, I did not think
This amorous surfeiter would have donned his helm
For such a petty war. His soldiership
Is twice the other twain. But let us rear
The higher our opinion, that our stirring
Can from the lap of Egypt’s widow pluck
The ne’er lust-wearied Antony.

MENAS.
I cannot hope
Caesar and Antony shall well greet together.
His wife that’s dead did trespasses to Caesar;
His brother warred upon him, although I think,
Not moved by Antony.

POMPEY.
I know not, Menas,
How lesser enmities may give way to greater.
Were’t not that we stand up against them all,
’Twere pregnant they should square between themselves,
For they have entertained cause enough
To draw their swords. But how the fear of us
May cement their divisions, and bind up
The petty difference, we yet not know.
Be’t as our gods will have’t! It only stands
Our lives upon to use our strongest hands.
Come, Menas.

 [_Exeunt._]

SCENE II. Rome. A Room in the House of Lepidus.

 Enter Enobarbus and Lepidus.

LEPIDUS.
Good Enobarbus, ’tis a worthy deed,
And shall become you well, to entreat your captain
To soft and gentle speech.

ENOBARBUS.
I shall entreat him
To answer like himself. If Caesar move him,
Let Antony look over Caesar’s head
And speak as loud as Mars. By Jupiter,
Were I the wearer of Antonius’ beard,
I would not shave’t today.

LEPIDUS.
’Tis not a time
For private stomaching.

ENOBARBUS.
Every time
Serves for the matter that is then born in’t.

LEPIDUS.
But small to greater matters must give way.

ENOBARBUS.
Not if the small come first.

LEPIDUS.
Your speech is passion;
But pray you stir no embers up. Here comes
The noble Antony.

 Enter Antony and Ventidius.

ENOBARBUS.
And yonder Caesar.

 Enter Caesar, Maecenas and Agrippa.

ANTONY.
If we compose well here, to Parthia.
Hark, Ventidius.

CAESAR.
I do not know, Maecenas. Ask Agrippa.

LEPIDUS.
Noble friends,
That which combined us was most great, and let not
A leaner action rend us. What’s amiss,
May it be gently heard. When we debate
Our trivial difference loud, we do commit
Murder in healing wounds. Then, noble partners,
The rather for I earnestly beseech,
Touch you the sourest points with sweetest terms,
Nor curstness grow to th’ matter.

ANTONY.
’Tis spoken well.
Were we before our armies, and to fight,
I should do thus.

CAESAR.
Welcome to Rome.

ANTONY.
Thank you.

CAESAR.
Sit.

ANTONY.
Sit, sir.

CAESAR.
Nay, then.

ANTONY.
I learn you take things ill which are not so,
Or being, concern you not.

CAESAR.
I must be laughed at
If, or for nothing or a little, I
Should say myself offended, and with you
Chiefly i’ th’ world; more laughed at that I should
Once name you derogately when to sound your name
It not concerned me.

ANTONY.
My being in Egypt, Caesar,
What was’t to you?

CAESAR.
No more than my residing here at Rome
Might be to you in Egypt. Yet if you there
Did practise on my state, your being in Egypt
Might be my question.

ANTONY.
How intend you, practised?

CAESAR.
You may be pleased to catch at mine intent
By what did here befall me. Your wife and brother
Made wars upon me, and their contestation
Was theme for you; you were the word of war.

ANTONY.
You do mistake your business. My brother never
Did urge me in his act. I did inquire it,
And have my learning from some true reports
That drew their swords with you. Did he not rather
Discredit my authority with yours,
And make the wars alike against my stomach,
Having alike your cause? Of this my letters
Before did satisfy you. If you’ll patch a quarrel,
As matter whole you have not to make it with,
It must not be with this.

CAESAR.
You praise yourself
By laying defects of judgment to me; but
You patched up your excuses.

ANTONY.
Not so, not so.
I know you could not lack—I am certain on’t—
Very necessity of this thought, that I,
Your partner in the cause ’gainst which he fought,
Could not with graceful eyes attend those wars
Which fronted mine own peace. As for my wife,
I would you had her spirit in such another.
The third o’ th’ world is yours, which with a snaffle
You may pace easy, but not such a wife.

ENOBARBUS.
Would we had all such wives, that the men
Might go to wars with the women.

ANTONY.
So much uncurbable, her garboils, Caesar,
Made out of her impatience—which not wanted
Shrewdness of policy too—I grieving grant
Did you too much disquiet. For that you must
But say I could not help it.

CAESAR.
I wrote to you
When rioting in Alexandria; you
Did pocket up my letters, and with taunts
Did gibe my missive out of audience.

ANTONY.
Sir,
He fell upon me ere admitted, then.
Three kings I had newly feasted, and did want
Of what I was i’ th’ morning. But next day
I told him of myself, which was as much
As to have asked him pardon. Let this fellow
Be nothing of our strife; if we contend,
Out of our question wipe him.

CAESAR.
You have broken
The article of your oath, which you shall never
Have tongue to charge me with.

LEPIDUS.
Soft, Caesar!

ANTONY.
No, Lepidus, let him speak.
The honour is sacred which he talks on now,
Supposing that I lacked it. But on, Caesar:
The article of my oath?

CAESAR.
To lend me arms and aid when I required them,
The which you both denied.

ANTONY.
Neglected, rather;
And then when poisoned hours had bound me up
From mine own knowledge. As nearly as I may
I’ll play the penitent to you. But mine honesty
Shall not make poor my greatness, nor my power
Work without it. Truth is that Fulvia,
To have me out of Egypt, made wars here,
For which myself, the ignorant motive, do
So far ask pardon as befits mine honour
To stoop in such a case.

LEPIDUS.
’Tis noble spoken.

MAECENAS.
If it might please you to enforce no further
The griefs between ye; to forget them quite
Were to remember that the present need
Speaks to atone you.

LEPIDUS.
Worthily spoken, Maecenas.

ENOBARBUS.
Or, if you borrow one another’s love for the instant, you may, when you
hear no more words of Pompey, return it again. You shall have time to
wrangle in when you have nothing else to do.

ANTONY.
Thou art a soldier only. Speak no more.

ENOBARBUS.
That truth should be silent I had almost forgot.

ANTONY.
You wrong this presence; therefore speak no more.

ENOBARBUS.
Go to, then. Your considerate stone!

CAESAR.
I do not much dislike the matter, but
The manner of his speech; for’t cannot be
We shall remain in friendship, our conditions
So differing in their acts. Yet if I knew
What hoop should hold us staunch, from edge to edge
O’ th’ world I would pursue it.

AGRIPPA.
Give me leave, Caesar.

CAESAR.
Speak, Agrippa.

AGRIPPA.
Thou hast a sister by the mother’s side,
Admired Octavia. Great Mark Antony
Is now a widower.

CAESAR.
Say not so, Agrippa.
If Cleopatra heard you, your reproof
Were well deserved of rashness.

ANTONY.
I am not married, Caesar. Let me hear
Agrippa further speak.

AGRIPPA.
To hold you in perpetual amity,
To make you brothers, and to knit your hearts
With an unslipping knot, take Antony
Octavia to his wife; whose beauty claims
No worse a husband than the best of men;
Whose virtue and whose general graces speak
That which none else can utter. By this marriage
All little jealousies, which now seem great,
And all great fears, which now import their dangers,
Would then be nothing. Truths would be tales,
Where now half-tales be truths. Her love to both
Would each to other, and all loves to both,
Draw after her. Pardon what I have spoke,
For ’tis a studied, not a present thought,
By duty ruminated.

ANTONY.
Will Caesar speak?

CAESAR.
Not till he hears how Antony is touched
With what is spoke already.

ANTONY.
What power is in Agrippa,
If I would say “Agrippa, be it so,”
To make this good?

CAESAR.
The power of Caesar, and
His power unto Octavia.

ANTONY.
May I never
To this good purpose, that so fairly shows,
Dream of impediment! Let me have thy hand.
Further this act of grace; and from this hour
The heart of brothers govern in our loves
And sway our great designs!

CAESAR.
There’s my hand.
A sister I bequeath you, whom no brother
Did ever love so dearly. Let her live
To join our kingdoms and our hearts; and never
Fly off our loves again!

LEPIDUS.
Happily, amen!

ANTONY.
I did not think to draw my sword ’gainst Pompey,
For he hath laid strange courtesies and great
Of late upon me. I must thank him only,
Lest my remembrance suffer ill report;
At heel of that, defy him.

LEPIDUS.
Time calls upon ’s.
Of us must Pompey presently be sought,
Or else he seeks out us.

ANTONY.
Where lies he?

CAESAR.
About the Mount Misena.

ANTONY.
What is his strength by land?

CAESAR.
Great and increasing; but by sea
He is an absolute master.

ANTONY.
So is the fame.
Would we had spoke together! Haste we for it.
Yet, ere we put ourselves in arms, dispatch we
The business we have talked of.

CAESAR.
With most gladness,
And do invite you to my sister’s view,
Whither straight I’ll lead you.

ANTONY.
Let us, Lepidus, not lack your company.

LEPIDUS.
Noble Antony, not sickness should detain me.

 [_Flourish. Exeunt all except Enobarbus, Agrippa and Maecenas._]

MAECENAS.
Welcome from Egypt, sir.

ENOBARBUS.
Half the heart of Caesar, worthy Maecenas! My honourable friend,
Agrippa!

AGRIPPA.
Good Enobarbus!

MAECENAS.
We have cause to be glad that matters are so well digested. You stayed
well by ’t in Egypt.

ENOBARBUS.
Ay, sir, we did sleep day out of countenance and made the night light
with drinking.

MAECENAS.
Eight wild boars roasted whole at a breakfast, and but twelve persons
there. Is this true?

ENOBARBUS.
This was but as a fly by an eagle. We had much more monstrous matter of
feast, which worthily deserved noting.

MAECENAS.
She’s a most triumphant lady, if report be square to her.

ENOBARBUS.
When she first met Mark Antony, she pursed up his heart upon the river
of Cydnus.

AGRIPPA.
There she appeared indeed, or my reporter devised well for her.

ENOBARBUS.
I will tell you.
The barge she sat in, like a burnished throne,
Burned on the water. The poop was beaten gold;
Purple the sails, and so perfumed that
The winds were love-sick with them; the oars were silver,
Which to the tune of flutes kept stroke, and made
The water which they beat to follow faster,
As amorous of their strokes. For her own person,
It beggared all description: she did lie
In her pavilion, cloth-of-gold of tissue,
O’erpicturing that Venus where we see
The fancy outwork nature. On each side her
Stood pretty dimpled boys, like smiling Cupids,
With divers-coloured fans, whose wind did seem
To glow the delicate cheeks which they did cool,
And what they undid did.

AGRIPPA.
O, rare for Antony!

ENOBARBUS.
Her gentlewomen, like the Nereides,
So many mermaids, tended her i’ th’ eyes,
And made their bends adornings. At the helm
A seeming mermaid steers. The silken tackle
Swell with the touches of those flower-soft hands
That yarely frame the office. From the barge
A strange invisible perfume hits the sense
Of the adjacent wharfs. The city cast
Her people out upon her, and Antony,
Enthroned i’ th’ market-place, did sit alone,
Whistling to th’ air, which, but for vacancy,
Had gone to gaze on Cleopatra too,
And made a gap in nature.

AGRIPPA.
Rare Egyptian!

ENOBARBUS.
Upon her landing, Antony sent to her,
Invited her to supper. She replied
It should be better he became her guest,
Which she entreated. Our courteous Antony,
Whom ne’er the word of “No” woman heard speak,
Being barbered ten times o’er, goes to the feast,
And, for his ordinary, pays his heart
For what his eyes eat only.

AGRIPPA.
Royal wench!
She made great Caesar lay his sword to bed.
He ploughed her, and she cropped.

ENOBARBUS.
I saw her once
Hop forty paces through the public street
And, having lost her breath, she spoke and panted,
That she did make defect perfection,
And, breathless, pour breath forth.

MAECENAS.
Now Antony must leave her utterly.

ENOBARBUS.
Never. He will not.
Age cannot wither her, nor custom stale
Her infinite variety. Other women cloy
The appetites they feed, but she makes hungry
Where most she satisfies. For vilest things
Become themselves in her, that the holy priests
Bless her when she is riggish.

MAECENAS.
If beauty, wisdom, modesty can settle
The heart of Antony, Octavia is
A blessed lottery to him.

AGRIPPA.
Let us go.
Good Enobarbus, make yourself my guest
Whilst you abide here.

ENOBARBUS.
Humbly, sir, I thank you.

 [_Exeunt._]

SCENE III. Rome. A Room in Caesar’s House.

 Enter Antony, Caesar, Octavia between them.

ANTONY.
The world and my great office will sometimes
Divide me from your bosom.

OCTAVIA.
All which time
Before the gods my knee shall bow my prayers
To them for you.

ANTONY.
Good night, sir.—My Octavia,
Read not my blemishes in the world’s report.
I have not kept my square, but that to come
Shall all be done by th’ rule. Good night, dear lady.

OCTAVIA.
Good night, sir.

CAESAR.
Good night.

 [_Exeunt Caesar and Octavia._]

 Enter Soothsayer.

ANTONY.
Now, sirrah, you do wish yourself in Egypt?

SOOTHSAYER.
Would I had never come from thence, nor you thither!

ANTONY.
If you can, your reason.

SOOTHSAYER.
I see it in my motion, have it not in my tongue.
But yet hie you to Egypt again.

ANTONY.
Say to me,
Whose fortunes shall rise higher, Caesar’s or mine?

SOOTHSAYER.
Caesar’s.
Therefore, O Antony, stay not by his side.
Thy dæmon—that thy spirit which keeps thee—is
Noble, courageous, high, unmatchable,
Where Caesar’s is not. But near him, thy angel
Becomes afeard, as being o’erpowered. Therefore
Make space enough between you.

ANTONY.
Speak this no more.

SOOTHSAYER.
To none but thee; no more but when to thee.
If thou dost play with him at any game,
Thou art sure to lose; and of that natural luck
He beats thee ’gainst the odds. Thy lustre thickens
When he shines by. I say again, thy spirit
Is all afraid to govern thee near him;
But, he away, ’tis noble.

ANTONY.
Get thee gone.
Say to Ventidius I would speak with him.

 [_Exit Soothsayer._]

He shall to Parthia. Be it art or hap,
He hath spoken true. The very dice obey him,
And in our sports my better cunning faints
Under his chance. If we draw lots, he speeds;
His cocks do win the battle still of mine
When it is all to naught, and his quails ever
Beat mine, inhooped, at odds. I will to Egypt:
And though I make this marriage for my peace,
I’ th’ East my pleasure lies.

 Enter Ventidius.

O, come, Ventidius,
You must to Parthia. Your commission’s ready.
Follow me and receive ’t.

 [_Exeunt._]

SCENE IV. Rome. A street.

 Enter Lepidus, Maecenas and Agrippa.

LEPIDUS.
Trouble yourselves no further. Pray you hasten
Your generals after.

AGRIPPA.
Sir, Mark Antony
Will e’en but kiss Octavia, and we’ll follow.

LEPIDUS.
Till I shall see you in your soldier’s dress,
Which will become you both, farewell.

MAECENAS.
We shall,
As I conceive the journey, be at the Mount
Before you, Lepidus.

LEPIDUS.
Your way is shorter;
My purposes do draw me much about.
You’ll win two days upon me.

BOTH.
Sir, good success!

LEPIDUS.
Farewell.

 [_Exeunt._]

SCENE V. Alexandria. A Room in the Palace.

 Enter Cleopatra, Charmian, Iras, Alexas.

CLEOPATRA.
Give me some music—music, moody food
Of us that trade in love.

ALL.
The music, ho!

 Enter Mardian, the eunuch.

CLEOPATRA.
Let it alone. Let’s to billiards. Come, Charmian.

CHARMIAN.
My arm is sore. Best play with Mardian.

CLEOPATRA.
As well a woman with an eunuch played
As with a woman. Come, you’ll play with me, sir?

MARDIAN.
As well as I can, madam.

CLEOPATRA.
And when good will is showed, though’t come too short,
The actor may plead pardon. I’ll none now.
Give me mine angle; we’ll to the river. There,
My music playing far off, I will betray
Tawny-finned fishes. My bended hook shall pierce
Their slimy jaws, and as I draw them up
I’ll think them every one an Antony,
And say “Ah, ha! You’re caught.”

CHARMIAN.
’Twas merry when
You wagered on your angling; when your diver
Did hang a salt fish on his hook, which he
With fervency drew up.

CLEOPATRA.
That time?—O times!—
I laughed him out of patience; and that night
I laughed him into patience, and next morn,
Ere the ninth hour, I drunk him to his bed,
Then put my tires and mantles on him, whilst
I wore his sword Philippan.

 Enter Messenger.

O! from Italy!
Ram thou thy fruitful tidings in mine ears,
That long time have been barren.

MESSENGER.
Madam, madam—

CLEOPATRA.
Antony’s dead! If thou say so, villain,
Thou kill’st thy mistress. But well and free,
If thou so yield him, there is gold, and here
My bluest veins to kiss, a hand that kings
Have lipped, and trembled kissing.

MESSENGER.
First, madam, he’s well.

CLEOPATRA.
Why, there’s more gold.
But sirrah, mark, we use
To say the dead are well. Bring it to that,
The gold I give thee will I melt and pour
Down thy ill-uttering throat.

MESSENGER.
Good madam, hear me.

CLEOPATRA.
Well, go to, I will.
But there’s no goodness in thy face if Antony
Be free and healthful. So tart a favour
To trumpet such good tidings! If not well,
Thou shouldst come like a Fury crowned with snakes,
Not like a formal man.

MESSENGER.
Will’t please you hear me?

CLEOPATRA.
I have a mind to strike thee ere thou speak’st.
Yet if thou say Antony lives, is well,
Or friends with Caesar, or not captive to him,
I’ll set thee in a shower of gold and hail
Rich pearls upon thee.

MESSENGER.
Madam, he’s well.

CLEOPATRA.
Well said.

MESSENGER.
And friends with Caesar.

CLEOPATRA.
Th’ art an honest man.

MESSENGER.
Caesar and he are greater friends than ever.

CLEOPATRA.
Make thee a fortune from me.

MESSENGER.
But yet, madam—

CLEOPATRA.
I do not like “But yet”, it does allay
The good precedence. Fie upon “But yet”!
“But yet” is as a gaoler to bring forth
Some monstrous malefactor. Prithee, friend,
Pour out the pack of matter to mine ear,
The good and bad together: he’s friends with Caesar,
In state of health, thou say’st; and, thou say’st, free.

MESSENGER.
Free, madam? No. I made no such report.
He’s bound unto Octavia.

CLEOPATRA.
For what good turn?

MESSENGER.
For the best turn i’ th’ bed.

CLEOPATRA.
I am pale, Charmian.

MESSENGER.
Madam, he’s married to Octavia.

CLEOPATRA.
The most infectious pestilence upon thee!

 [_Strikes him down._]

MESSENGER.
Good madam, patience.

CLEOPATRA.
What say you?

 [_Strikes him again._]

Hence, horrible villain, or I’ll spurn thine eyes
Like balls before me! I’ll unhair thy head!

 [_She hales him up and down._]

Thou shalt be whipped with wire and stewed in brine,
Smarting in ling’ring pickle.

MESSENGER.
Gracious madam,
I that do bring the news made not the match.

CLEOPATRA.
Say ’tis not so, a province I will give thee,
And make thy fortunes proud. The blow thou hadst
Shall make thy peace for moving me to rage,
And I will boot thee with what gift beside
Thy modesty can beg.

MESSENGER.
He’s married, madam.

CLEOPATRA.
Rogue, thou hast lived too long.

 [_Draws a knife._]

MESSENGER.
Nay then I’ll run.
What mean you, madam? I have made no fault.

 [_Exit._]

CHARMIAN.
Good madam, keep yourself within yourself.
The man is innocent.

CLEOPATRA.
Some innocents ’scape not the thunderbolt.
Melt Egypt into Nile, and kindly creatures
Turn all to serpents! Call the slave again.
Though I am mad, I will not bite him. Call!

CHARMIAN.
He is afeard to come.

CLEOPATRA.
I will not hurt him.

 [_Exit Charmian._]

These hands do lack nobility that they strike
A meaner than myself, since I myself
Have given myself the cause.

 Enter the Messenger again with Charmian.

Come hither, sir.
Though it be honest, it is never good
To bring bad news. Give to a gracious message
An host of tongues, but let ill tidings tell
Themselves when they be felt.

MESSENGER.
I have done my duty.

CLEOPATRA.
Is he married?
I cannot hate thee worser than I do
If thou again say “Yes.”

MESSENGER.
He’s married, madam.

CLEOPATRA.
The gods confound thee! Dost thou hold there still!

MESSENGER.
Should I lie, madam?

CLEOPATRA.
O, I would thou didst,
So half my Egypt were submerged and made
A cistern for scaled snakes! Go, get thee hence.
Hadst thou Narcissus in thy face, to me
Thou wouldst appear most ugly. He is married?

MESSENGER.
I crave your highness’ pardon.

CLEOPATRA.
He is married?

MESSENGER.
Take no offence that I would not offend you.
To punish me for what you make me do
Seems much unequal. He’s married to Octavia.

CLEOPATRA.
O, that his fault should make a knave of thee
That art not what thou’rt sure of! Get thee hence!
The merchandise which thou hast brought from Rome
Are all too dear for me. Lie they upon thy hand,
And be undone by ’em!

 [_Exit Messenger._]

CHARMIAN.
Good your highness, patience.

CLEOPATRA.
In praising Antony I have dispraised Caesar.

CHARMIAN.
Many times, madam.

CLEOPATRA.
I am paid for’t now.
Lead me from hence;
I faint. O Iras, Charmian! ’Tis no matter.
Go to the fellow, good Alexas, bid him
Report the feature of Octavia, her years,
Her inclination; let him not leave out
The colour of her hair. Bring me word quickly.

 [_Exit Alexas._]

Let him for ever go—let him not, Charmian.
Though he be painted one way like a Gorgon,
The other way ’s a Mars. [_To Mardian_] Bid you Alexas
Bring me word how tall she is. Pity me, Charmian,
But do not speak to me. Lead me to my chamber.

 [_Exeunt._]

SCENE VI. Near Misenum.

 Flourish. Enter Pompey and Menas at one door, with drum and trumpet;
 at another, Caesar, Lepidus, Antony, Enobarbus, Maecenas, Agrippa,
 with Soldiers marching.

POMPEY.
Your hostages I have, so have you mine,
And we shall talk before we fight.

CAESAR.
Most meet
That first we come to words, and therefore have we
Our written purposes before us sent,
Which if thou hast considered, let us know
If ’twill tie up thy discontented sword
And carry back to Sicily much tall youth
That else must perish here.

POMPEY.
To you all three,
The senators alone of this great world,
Chief factors for the gods: I do not know
Wherefore my father should revengers want,
Having a son and friends, since Julius Caesar,
Who at Philippi the good Brutus ghosted,
There saw you labouring for him. What was’t
That moved pale Cassius to conspire? And what
Made the all-honoured, honest Roman, Brutus,
With the armed rest, courtiers of beauteous freedom,
To drench the Capitol, but that they would
Have one man but a man? And that is it
Hath made me rig my navy, at whose burden
The angered ocean foams, with which I meant
To scourge th’ ingratitude that despiteful Rome
Cast on my noble father.

CAESAR.
Take your time.

ANTONY.
Thou canst not fear us, Pompey, with thy sails.
We’ll speak with thee at sea. At land thou know’st
How much we do o’ercount thee.

POMPEY.
At land indeed
Thou dost o’ercount me of my father’s house;
But since the cuckoo builds not for himself,
Remain in’t as thou mayst.

LEPIDUS.
Be pleased to tell us—
For this is from the present—how you take
The offers we have sent you.

CAESAR.
There’s the point.

ANTONY.
Which do not be entreated to, but weigh
What it is worth embraced.

CAESAR.
And what may follow
To try a larger fortune.

POMPEY.
You have made me offer
Of Sicily, Sardinia; and I must
Rid all the sea of pirates; then to send
Measures of wheat to Rome. This ’greed upon,
To part with unhacked edges and bear back
Our targes undinted.

CAESAR, ANTONY, and LEPIDUS.
That’s our offer.

POMPEY.
Know, then,
I came before you here a man prepared
To take this offer. But Mark Antony
Put me to some impatience. Though I lose
The praise of it by telling, you must know
When Caesar and your brother were at blows,
Your mother came to Sicily and did find
Her welcome friendly.

ANTONY.
I have heard it, Pompey,
And am well studied for a liberal thanks
Which I do owe you.

POMPEY.
Let me have your hand.
I did not think, sir, to have met you here.

ANTONY.
The beds i’ th’ East are soft; and thanks to you,
That called me timelier than my purpose hither,
For I have gained by ’t.

CAESAR.
Since I saw you last,
There is a change upon you.

POMPEY.
Well, I know not
What counts harsh Fortune casts upon my face,
But in my bosom shall she never come
To make my heart her vassal.

LEPIDUS.
Well met here.

POMPEY.
I hope so, Lepidus. Thus we are agreed.
I crave our composition may be written
And sealed between us.

CAESAR.
That’s the next to do.

POMPEY.
We’ll feast each other ere we part, and let’s
Draw lots who shall begin.

ANTONY.
That will I, Pompey.

POMPEY.
No, Antony, take the lot.
But, first or last, your fine Egyptian cookery
Shall have the fame. I have heard that Julius Caesar
Grew fat with feasting there.

ANTONY.
You have heard much.

POMPEY.
I have fair meanings, sir.

ANTONY.
And fair words to them.

POMPEY.
Then so much have I heard.
And I have heard Apollodorus carried—

ENOBARBUS.
No more of that. He did so.

POMPEY.
What, I pray you?

ENOBARBUS.
A certain queen to Caesar in a mattress.

POMPEY.
I know thee now. How far’st thou, soldier?

ENOBARBUS.
Well;
And well am like to do, for I perceive
Four feasts are toward.

POMPEY.
Let me shake thy hand.
I never hated thee. I have seen thee fight
When I have envied thy behaviour.

ENOBARBUS.
Sir,
I never loved you much, but I ha’ praised ye
When you have well deserved ten times as much
As I have said you did.

POMPEY.
Enjoy thy plainness;
It nothing ill becomes thee.
Aboard my galley I invite you all.
Will you lead, lords?

CAESAR, ANTONY, and LEPIDUS.
Show’s the way, sir.

POMPEY.
Come.

 [_Exeunt all but Enobarbus and Menas._]

MENAS.
[_Aside_.] Thy father, Pompey, would ne’er have made this treaty.—
You and I have known, sir.

ENOBARBUS.
At sea, I think.

MENAS.
We have, sir.

ENOBARBUS.
You have done well by water.

MENAS.
And you by land.

ENOBARBUS.
I will praise any man that will praise me, though it cannot be denied
what I have done by land.

MENAS.
Nor what I have done by water.

ENOBARBUS.
Yes, something you can deny for your own safety: you have been a great
thief by sea.

MENAS.
And you by land.

ENOBARBUS.
There I deny my land service. But give me your hand, Menas. If our eyes
had authority, here they might take two thieves kissing.

MENAS.
All men’s faces are true, whatsome’er their hands are.

ENOBARBUS.
But there is never a fair woman has a true face.

MENAS.
No slander. They steal hearts.

ENOBARBUS.
We came hither to fight with you.

MENAS.
For my part, I am sorry it is turned to a drinking. Pompey doth this
day laugh away his fortune.

ENOBARBUS.
If he do, sure he cannot weep ’t back again.

MENAS.
You have said, sir. We looked not for Mark Antony here. Pray you, is he
married to Cleopatra?

ENOBARBUS.
Caesar’s sister is called Octavia.

MENAS.
True, sir. She was the wife of Caius Marcellus.

ENOBARBUS.
But she is now the wife of Marcus Antonius.

MENAS.
Pray you, sir?

ENOBARBUS.
’Tis true.

MENAS.
Then is Caesar and he for ever knit together.

ENOBARBUS.
If I were bound to divine of this unity, I would not prophesy so.

MENAS.
I think the policy of that purpose made more in the marriage than the
love of the parties.

ENOBARBUS.
I think so too. But you shall find the band that seems to tie their
friendship together will be the very strangler of their amity. Octavia
is of a holy, cold, and still conversation.

MENAS.
Who would not have his wife so?

ENOBARBUS.
Not he that himself is not so; which is Mark Antony. He will to his
Egyptian dish again. Then shall the sighs of Octavia blow the fire up
in Caesar, and, as I said before, that which is the strength of their
amity shall prove the immediate author of their variance. Antony will
use his affection where it is. He married but his occasion here.

MENAS.
And thus it may be. Come, sir, will you aboard? I have a health for
you.

ENOBARBUS.
I shall take it, sir. We have used our throats in Egypt.

MENAS.
Come, let’s away.

 [_Exeunt._]

SCENE VII. On board Pompey’s Galley, lying near Misenum.

 Music. Enter two or three Servants with a banquet.

FIRST SERVANT.
Here they’ll be, man. Some o’ their plants are ill-rooted already; the
least wind i’ th’ world will blow them down.

SECOND SERVANT.
Lepidus is high-coloured.

FIRST SERVANT.
They have made him drink alms-drink.

SECOND SERVANT.
As they pinch one another by the disposition, he cries out “no more”,
reconciles them to his entreaty and himself to th’ drink.

FIRST SERVANT.
But it raises the greater war between him and his discretion.

SECOND SERVANT.
Why, this it is to have a name in great men’s fellowship. I had as lief
have a reed that will do me no service as a partisan I could not heave.

FIRST SERVANT.
To be called into a huge sphere, and not to be seen to move in ’t, are
the holes where eyes should be, which pitifully disaster the cheeks.

 A sennet sounded. Enter Caesar, Antony, Pompey, Lepidus, Agrippa,
 Maecenas, Enobarbus, Menas with other Captains.

ANTONY.
[_To Caesar_.] Thus do they, sir: they take the flow o’ th’ Nile
By certain scales i’ th’ pyramid; they know
By th’ height, the lowness, or the mean, if dearth
Or foison follow. The higher Nilus swells,
The more it promises. As it ebbs, the seedsman
Upon the slime and ooze scatters his grain,
And shortly comes to harvest.

LEPIDUS.
You’ve strange serpents there?

ANTONY.
Ay, Lepidus.

LEPIDUS.
Your serpent of Egypt is bred now of your mud by the operation of your
sun; so is your crocodile.

ANTONY.
They are so.

POMPEY.
Sit, and some wine! A health to Lepidus!

LEPIDUS.
I am not so well as I should be, but I’ll ne’er out.

ENOBARBUS.
Not till you have slept. I fear me you’ll be in till then.

LEPIDUS.
Nay, certainly, I have heard the Ptolemies’ pyramises are very goodly
things. Without contradiction I have heard that.

MENAS.
[_Aside to Pompey_.] Pompey, a word.

POMPEY.
[_Aside to Menas_.] Say in mine ear what is ’t?

MENAS.
[_Whispers in ’s ear._] Forsake thy seat, I do beseech thee, captain,
And hear me speak a word.

POMPEY.
[_Aside to Menas._] Forbear me till anon.—
This wine for Lepidus!

LEPIDUS.
What manner o’ thing is your crocodile?

ANTONY.
It is shaped, sir, like itself, and it is as broad as it hath breadth.
It is just so high as it is, and moves with it own organs. It lives by
that which nourisheth it, and the elements once out of it, it
transmigrates.

LEPIDUS.
What colour is it of?

ANTONY.
Of its own colour too.

LEPIDUS.
’Tis a strange serpent.

ANTONY.
’Tis so, and the tears of it are wet.

CAESAR.
Will this description satisfy him?

ANTONY.
With the health that Pompey gives him, else he is a very epicure.

POMPEY.
[_Aside to Menas._] Go hang, sir, hang! Tell me of that? Away!
Do as I bid you.—Where’s this cup I called for?

MENAS.
[_Aside to Pompey_.] If for the sake of merit thou wilt hear me,
Rise from thy stool.

POMPEY.
[_Aside to Menas_.] I think thou’rt mad.

 [_Rises and walks aside._]

The matter?

MENAS.
I have ever held my cap off to thy fortunes.

POMPEY.
Thou hast served me with much faith. What’s else to say?—
Be jolly, lords.

ANTONY.
These quicksands, Lepidus,
Keep off them, for you sink.

MENAS.
Wilt thou be lord of all the world?

POMPEY.
What sayst thou?

MENAS.
Wilt thou be lord of the whole world?
That’s twice.

POMPEY.
How should that be?

MENAS.
But entertain it,
And though you think me poor, I am the man
Will give thee all the world.

POMPEY.
Hast thou drunk well?

MENAS.
No, Pompey, I have kept me from the cup.
Thou art, if thou dar’st be, the earthly Jove.
Whate’er the ocean pales or sky inclips
Is thine, if thou wilt have’t.

POMPEY.
Show me which way.

MENAS.
These three world-sharers, these competitors,
Are in thy vessel. Let me cut the cable,
And when we are put off, fall to their throats.
All then is thine.

POMPEY.
Ah, this thou shouldst have done
And not have spoke on ’t! In me ’tis villainy;
In thee ’t had been good service. Thou must know
’Tis not my profit that does lead mine honour;
Mine honour it. Repent that e’er thy tongue
Hath so betray’d thine act. Being done unknown,
I should have found it afterwards well done,
But must condemn it now. Desist, and drink.

MENAS.
[_Aside_.] For this,
I’ll never follow thy palled fortunes more.
Who seeks, and will not take when once ’tis offered,
Shall never find it more.

POMPEY.
This health to Lepidus!

ANTONY.
Bear him ashore. I’ll pledge it for him, Pompey.

ENOBARBUS.
Here’s to thee, Menas!

MENAS.
Enobarbus, welcome!

POMPEY.
Fill till the cup be hid.

ENOBARBUS.
There’s a strong fellow, Menas.

 [_Pointing to the servant who carries off Lepidus._]

MENAS.
Why?

ENOBARBUS.
’A bears the third part of the world, man. Seest not?

MENAS.
The third part, then, is drunk. Would it were all,
That it might go on wheels!

ENOBARBUS.
Drink thou. Increase the reels.

MENAS.
Come.

POMPEY.
This is not yet an Alexandrian feast.

ANTONY.
It ripens towards it. Strike the vessels, ho!
Here is to Caesar!

CAESAR.
I could well forbear’t.
It’s monstrous labour when I wash my brain
And it grows fouler.

ANTONY.
Be a child o’ the time.

CAESAR.
Possess it, I’ll make answer.
But I had rather fast from all, four days,
Than drink so much in one.

ENOBARBUS.
[_To Antony_.] Ha, my brave emperor,
Shall we dance now the Egyptian Bacchanals
And celebrate our drink?

POMPEY.
Let’s ha’t, good soldier.

ANTONY.
Come, let’s all take hands
Till that the conquering wine hath steeped our sense
In soft and delicate Lethe.

ENOBARBUS.
All take hands.
Make battery to our ears with the loud music,
The while I’ll place you; then the boy shall sing.
The holding every man shall beat as loud
As his strong sides can volley.

 Music plays. Enobarbus places them hand in hand.

       THE SONG.
  Come, thou monarch of the vine,
  Plumpy Bacchus with pink eyne!
  In thy vats our cares be drowned,
  With thy grapes our hairs be crowned.
  Cup us till the world go round,
  Cup us till the world go round!

CAESAR.
What would you more? Pompey, good night. Good brother,
Let me request you off. Our graver business
Frowns at this levity.—Gentle lords, let’s part.
You see we have burnt our cheeks. Strong Enobarb
Is weaker than the wine, and mine own tongue
Splits what it speaks. The wild disguise hath almost
Anticked us all. What needs more words. Good night.
Good Antony, your hand.

POMPEY.
I’ll try you on the shore.

ANTONY.
And shall, sir. Give’s your hand.

POMPEY.
O Antony,
You have my father’s house.
But, what? We are friends. Come, down into the boat.

ENOBARBUS.
Take heed you fall not.

 [_Exeunt Pompey, Caesar, Antony and Attendants._]

Menas, I’ll not on shore.

MENAS.
No, to my cabin. These drums, these trumpets, flutes! What!
Let Neptune hear we bid a loud farewell
To these great fellows. Sound and be hanged, sound out!

 [_Sound a flourish with drums._]

ENOBARBUS.
Hoo, says ’a! There’s my cap!

MENAS.
Hoo! Noble captain, come.

 [_Exeunt._]




ACT III


SCENE I. A plain in Syria.

 Enter Ventidius as it were in triumph, with Silius and other Romans,
 Officers and Soldiers; the dead body of Pacorus borne before him.

VENTIDIUS.
Now, darting Parthia, art thou struck, and now
Pleased Fortune does of Marcus Crassus’ death
Make me revenger. Bear the king’s son’s body
Before our army. Thy Pacorus, Orodes,
Pays this for Marcus Crassus.

SILIUS.
Noble Ventidius,
Whilst yet with Parthian blood thy sword is warm,
The fugitive Parthians follow. Spur through Media,
Mesopotamia, and the shelters whither
The routed fly. So thy grand captain Antony
Shall set thee on triumphant chariots, and
Put garlands on thy head.

VENTIDIUS.
O Silius, Silius,
I have done enough. A lower place, note well,
May make too great an act. For learn this, Silius:
Better to leave undone than by our deed
Acquire too high a fame when him we serve’s away.
Caesar and Antony have ever won
More in their officer, than person. Sossius,
One of my place in Syria, his lieutenant,
For quick accumulation of renown,
Which he achieved by th’ minute, lost his favour.
Who does i’ th’ wars more than his captain can
Becomes his captain’s captain; and ambition,
The soldier’s virtue, rather makes choice of loss
Than gain which darkens him.
I could do more to do Antonius good,
But ’twould offend him, and in his offence
Should my performance perish.

SILIUS.
Thou hast, Ventidius, that
Without the which a soldier and his sword
Grants scarce distinction. Thou wilt write to Antony?

VENTIDIUS.
I’ll humbly signify what in his name,
That magical word of war, we have effected;
How, with his banners, and his well-paid ranks,
The ne’er-yet-beaten horse of Parthia
We have jaded out o’ th’ field.

SILIUS.
Where is he now?

VENTIDIUS.
He purposeth to Athens, whither, with what haste
The weight we must convey with ’s will permit,
We shall appear before him.—On there, pass along!

 [_Exeunt._]

SCENE II. Rome. An Ante-chamber in Caesar’s house.

 Enter Agrippa at one door, Enobarbus at another.

AGRIPPA.
What, are the brothers parted?

ENOBARBUS.
They have dispatched with Pompey; he is gone.
The other three are sealing. Octavia weeps
To part from Rome. Caesar is sad, and Lepidus,
Since Pompey’s feast, as Menas says, is troubled
With the greensickness.

AGRIPPA.
’Tis a noble Lepidus.

ENOBARBUS.
A very fine one. O, how he loves Caesar!

AGRIPPA.
Nay, but how dearly he adores Mark Antony!

ENOBARBUS.
Caesar? Why he’s the Jupiter of men.

AGRIPPA.
What’s Antony? The god of Jupiter.

ENOBARBUS.
Spake you of Caesar? How, the nonpareil!

AGRIPPA.
O, Antony! O thou Arabian bird!

ENOBARBUS.
Would you praise Caesar, say “Caesar”. Go no further.

AGRIPPA.
Indeed, he plied them both with excellent praises.

ENOBARBUS.
But he loves Caesar best, yet he loves Antony.
Hoo! Hearts, tongues, figures, scribes, bards, poets, cannot
Think, speak, cast, write, sing, number—hoo!—
His love to Antony. But as for Caesar,
Kneel down, kneel down, and wonder.

AGRIPPA.
Both he loves.

ENOBARBUS.
They are his shards, and he their beetle.

 [_Trumpets within._]

So,
This is to horse. Adieu, noble Agrippa.

AGRIPPA.
Good fortune, worthy soldier, and farewell.

 Enter Caesar, Antony, Lepidus and Octavia.

ANTONY.
No further, sir.

CAESAR.
You take from me a great part of myself.
Use me well in’t. Sister, prove such a wife
As my thoughts make thee, and as my farthest bond
Shall pass on thy approof. Most noble Antony,
Let not the piece of virtue which is set
Betwixt us, as the cement of our love
To keep it builded, be the ram to batter
The fortress of it. For better might we
Have loved without this mean, if on both parts
This be not cherished.

ANTONY.
Make me not offended
In your distrust.

CAESAR.
I have said.

ANTONY.
You shall not find,
Though you be therein curious, the least cause
For what you seem to fear. So the gods keep you,
And make the hearts of Romans serve your ends.
We will here part.

CAESAR.
Farewell, my dearest sister, fare thee well.
The elements be kind to thee, and make
Thy spirits all of comfort! Fare thee well.

OCTAVIA.
My noble brother!

ANTONY.
The April’s in her eyes. It is love’s spring,
And these the showers to bring it on.—Be cheerful.

OCTAVIA.
Sir, look well to my husband’s house, and—

CAESAR.
What, Octavia?

OCTAVIA.
I’ll tell you in your ear.

ANTONY.
Her tongue will not obey her heart, nor can
Her heart inform her tongue—the swan’s-down feather,
That stands upon the swell at the full of tide,
And neither way inclines.

ENOBARBUS.
[_Aside to Agrippa_.] Will Caesar weep?

AGRIPPA.
[_Aside to Enobarbus_.] He has a cloud in ’s face.

ENOBARBUS.
[_Aside to Agrippa_.] He were the worse for that were he a horse;
So is he, being a man.

AGRIPPA.
[_Aside to Enobarbus_.] Why, Enobarbus,
When Antony found Julius Caesar dead,
He cried almost to roaring, and he wept
When at Philippi he found Brutus slain.

ENOBARBUS.
[_Aside to Agrippa_.] That year, indeed, he was troubled with a rheum;
What willingly he did confound he wailed,
Believe ’t, till I weep too.

CAESAR.
No, sweet Octavia,
You shall hear from me still. The time shall not
Outgo my thinking on you.

ANTONY.
Come, sir, come,
I’ll wrestle with you in my strength of love.
Look, here I have you, thus I let you go,
And give you to the gods.

CAESAR.
Adieu, be happy!

LEPIDUS.
Let all the number of the stars give light
To thy fair way!

CAESAR.
Farewell, farewell!

 [_Kisses Octavia._]

ANTONY.
Farewell!

 [_Trumpets sound. Exeunt._]

SCENE III. Alexandria. A Room in the Palace.

 Enter Cleopatra, Charmian, Iras and Alexas.

CLEOPATRA.
Where is the fellow?

ALEXAS.
Half afeared to come.

CLEOPATRA.
Go to, go to.

 Enter a Messenger as before.

Come hither, sir.

ALEXAS.
Good majesty,
Herod of Jewry dare not look upon you
But when you are well pleased.

CLEOPATRA.
That Herod’s head
I’ll have! But how, when Antony is gone,
Through whom I might command it?—Come thou near.

MESSENGER.
Most gracious majesty!

CLEOPATRA.
Didst thou behold Octavia?

MESSENGER.
Ay, dread queen.

CLEOPATRA.
Where?

MESSENGER.
Madam, in Rome
I looked her in the face, and saw her led
Between her brother and Mark Antony.

CLEOPATRA.
Is she as tall as me?

MESSENGER.
She is not, madam.

CLEOPATRA.
Didst hear her speak? Is she shrill-tongued or low?

MESSENGER.
Madam, I heard her speak. She is low-voiced.

CLEOPATRA.
That’s not so good. He cannot like her long.

CHARMIAN.
Like her? O Isis! ’Tis impossible.

CLEOPATRA.
I think so, Charmian: dull of tongue and dwarfish!
What majesty is in her gait? Remember,
If e’er thou look’dst on majesty.

MESSENGER.
She creeps.
Her motion and her station are as one.
She shows a body rather than a life,
A statue than a breather.

CLEOPATRA.
Is this certain?

MESSENGER.
Or I have no observance.

CHARMIAN.
Three in Egypt
Cannot make better note.

CLEOPATRA.
He’s very knowing;
I do perceive’t. There’s nothing in her yet.
The fellow has good judgment.

CHARMIAN.
Excellent.

CLEOPATRA.
Guess at her years, I prithee.

MESSENGER.
Madam,
She was a widow.

CLEOPATRA.
Widow! Charmian, hark!

MESSENGER.
And I do think she’s thirty.

CLEOPATRA.
Bear’st thou her face in mind? Is’t long or round?

MESSENGER.
Round even to faultiness.

CLEOPATRA.
For the most part, too, they are foolish that are so.
Her hair, what colour?

MESSENGER.
Brown, madam, and her forehead
As low as she would wish it.

CLEOPATRA.
There’s gold for thee.
Thou must not take my former sharpness ill.
I will employ thee back again; I find thee
Most fit for business. Go make thee ready;
Our letters are prepared.

 [_Exit Messenger._]

CHARMIAN.
A proper man.

CLEOPATRA.
Indeed, he is so. I repent me much
That so I harried him. Why, methinks, by him,
This creature’s no such thing.

CHARMIAN.
Nothing, madam.

CLEOPATRA.
The man hath seen some majesty, and should know.

CHARMIAN.
Hath he seen majesty? Isis else defend,
And serving you so long!

CLEOPATRA.
I have one thing more to ask him yet, good Charmian.
But ’tis no matter; thou shalt bring him to me
Where I will write. All may be well enough.

CHARMIAN.
I warrant you, madam.

 [_Exeunt._]

SCENE IV. Athens. A Room in Antony’s House.

 Enter Antony and Octavia.

ANTONY.
Nay, nay, Octavia, not only that—
That were excusable, that and thousands more
Of semblable import—but he hath waged
New wars ’gainst Pompey; made his will, and read it
To public ear;
Spoke scantly of me; when perforce he could not
But pay me terms of honour, cold and sickly
He vented them; most narrow measure lent me;
When the best hint was given him, he not took ’t,
Or did it from his teeth.

OCTAVIA.
O, my good lord,
Believe not all, or if you must believe,
Stomach not all. A more unhappy lady,
If this division chance, ne’er stood between,
Praying for both parts.
The good gods will mock me presently
When I shall pray “O, bless my lord and husband!”
Undo that prayer by crying out as loud
“O, bless my brother!” Husband win, win brother,
Prays and destroys the prayer; no midway
’Twixt these extremes at all.

ANTONY.
Gentle Octavia,
Let your best love draw to that point which seeks
Best to preserve it. If I lose mine honour,
I lose myself; better I were not yours
Than yours so branchless. But, as you requested,
Yourself shall go between’s. The meantime, lady,
I’ll raise the preparation of a war
Shall stain your brother. Make your soonest haste,
So your desires are yours.

OCTAVIA.
Thanks to my lord.
The Jove of power make me, most weak, most weak,
Your reconciler! Wars ’twixt you twain would be
As if the world should cleave, and that slain men
Should solder up the rift.

ANTONY.
When it appears to you where this begins,
Turn your displeasure that way, for our faults
Can never be so equal that your love
Can equally move with them. Provide your going;
Choose your own company, and command what cost
Your heart has mind to.

 [_Exeunt._]

SCENE V. Athens. Another Room in Antony’s House.

 Enter Enobarbus and Eros meeting.

ENOBARBUS.
How now, friend Eros?

EROS.
There’s strange news come, sir.

ENOBARBUS.
What, man?

EROS.
Caesar and Lepidus have made wars upon Pompey.

ENOBARBUS.
This is old. What is the success?

EROS.
Caesar, having made use of him in the wars ’gainst Pompey, presently
denied him rivality; would not let him partake in the glory of the
action, and, not resting here, accuses him of letters he had formerly
wrote to Pompey; upon his own appeal, seizes him. So the poor third is
up, till death enlarge his confine.

ENOBARBUS.
Then, world, thou hast a pair of chaps, no more,
And throw between them all the food thou hast,
They’ll grind the one the other. Where’s Antony?

EROS.
He’s walking in the garden, thus, and spurns
The rush that lies before him; cries “Fool Lepidus!”
And threats the throat of that his officer
That murdered Pompey.

ENOBARBUS.
Our great navy’s rigged.

EROS.
For Italy and Caesar. More, Domitius:
My lord desires you presently. My news
I might have told hereafter.

ENOBARBUS.
’Twill be naught,
But let it be. Bring me to Antony.

EROS.
Come, sir.

 [_Exeunt._]

SCENE VI. Rome. A Room in Caesar’s House.

 Enter Agrippa, Maecenas and Caesar.

CAESAR.
Contemning Rome, he has done all this, and more
In Alexandria. Here’s the manner of ’t:
I’ th’ market-place, on a tribunal silvered,
Cleopatra and himself in chairs of gold
Were publicly enthroned. At the feet sat
Caesarion, whom they call my father’s son,
And all the unlawful issue that their lust
Since then hath made between them. Unto her
He gave the stablishment of Egypt; made her
Of lower Syria, Cyprus, Lydia,
Absolute queen.

MAECENAS.
This in the public eye?

CAESAR.
I’ th’ common showplace where they exercise.
His sons he there proclaimed the kings of kings:
Great Media, Parthia, and Armenia
He gave to Alexander; to Ptolemy he assigned
Syria, Cilicia, and Phoenicia. She
In th’ habiliments of the goddess Isis
That day appeared, and oft before gave audience,
As ’tis reported, so.

MAECENAS.
Let Rome be thus informed.

AGRIPPA.
Who, queasy with his insolence already,
Will their good thoughts call from him.

CAESAR.
The people knows it and have now received
His accusations.

AGRIPPA.
Who does he accuse?

CAESAR.
Caesar, and that, having in Sicily
Sextus Pompeius spoiled, we had not rated him
His part o’ th’ isle. Then does he say he lent me
Some shipping, unrestored. Lastly, he frets
That Lepidus of the triumvirate
Should be deposed and, being, that we detain
All his revenue.

AGRIPPA.
Sir, this should be answered.

CAESAR.
’Tis done already, and messenger gone.
I have told him Lepidus was grown too cruel,
That he his high authority abused,
And did deserve his change. For what I have conquered
I grant him part; but then in his Armenia
And other of his conquered kingdoms, I
Demand the like.

MAECENAS.
He’ll never yield to that.

CAESAR.
Nor must not then be yielded to in this.

 Enter Octavia with her train.

OCTAVIA.
Hail, Caesar, and my lord! Hail, most dear Caesar!

CAESAR.
That ever I should call thee castaway!

OCTAVIA.
You have not called me so, nor have you cause.

CAESAR.
Why have you stolen upon us thus? You come not
Like Caesar’s sister. The wife of Antony
Should have an army for an usher, and
The neighs of horse to tell of her approach
Long ere she did appear. The trees by th’ way
Should have borne men, and expectation fainted,
Longing for what it had not. Nay, the dust
Should have ascended to the roof of heaven,
Raised by your populous troops. But you are come
A market-maid to Rome, and have prevented
The ostentation of our love, which, left unshown,
Is often left unloved. We should have met you
By sea and land, supplying every stage
With an augmented greeting.

OCTAVIA.
Good my lord,
To come thus was I not constrained, but did it
On my free will. My lord, Mark Antony,
Hearing that you prepared for war, acquainted
My grieved ear withal, whereon I begged
His pardon for return.

CAESAR.
Which soon he granted,
Being an abstract ’tween his lust and him.

OCTAVIA.
Do not say so, my lord.

CAESAR.
I have eyes upon him,
And his affairs come to me on the wind.
Where is he now?

OCTAVIA.
My lord, in Athens.

CAESAR.
No, my most wronged sister. Cleopatra
Hath nodded him to her. He hath given his empire
Up to a whore, who now are levying
The kings o’ th’ earth for war. He hath assembled
Bocchus, the king of Libya; Archelaus
Of Cappadocia; Philadelphos, king
Of Paphlagonia; the Thracian king, Adallas;
King Manchus of Arabia; King of Pont;
Herod of Jewry; Mithridates, king
Of Comagene; Polemon and Amyntas,
The kings of Mede and Lycaonia,
With a more larger list of sceptres.

OCTAVIA.
Ay me, most wretched,
That have my heart parted betwixt two friends
That does afflict each other!

CAESAR.
Welcome hither.
Your letters did withhold our breaking forth
Till we perceived both how you were wrong led
And we in negligent danger. Cheer your heart.
Be you not troubled with the time, which drives
O’er your content these strong necessities,
But let determined things to destiny
Hold unbewailed their way. Welcome to Rome,
Nothing more dear to me. You are abused
Beyond the mark of thought, and the high gods,
To do you justice, make their ministers
Of us and those that love you. Best of comfort,
And ever welcome to us.

AGRIPPA.
Welcome, lady.

MAECENAS.
Welcome, dear madam.
Each heart in Rome does love and pity you.
Only th’ adulterous Antony, most large
In his abominations, turns you off
And gives his potent regiment to a trull
That noises it against us.

OCTAVIA.
Is it so, sir?

CAESAR.
Most certain. Sister, welcome. Pray you
Be ever known to patience. My dear’st sister!

 [_Exeunt._]

SCENE VII. Antony’s Camp near the Promontory of Actium.

 Enter Cleopatra and Enobarbus.

CLEOPATRA.
I will be even with thee, doubt it not.

ENOBARBUS.
But why, why, why?

CLEOPATRA.
Thou hast forspoke my being in these wars
And say’st it is not fit.

ENOBARBUS.
Well, is it, is it?

CLEOPATRA.
Is ’t not denounced against us? Why should not we
Be there in person?

ENOBARBUS.
Well, I could reply:
If we should serve with horse and mares together,
The horse were merely lost. The mares would bear
A soldier and his horse.

CLEOPATRA.
What is’t you say?

ENOBARBUS.
Your presence needs must puzzle Antony,
Take from his heart, take from his brain, from ’s time,
What should not then be spared. He is already
Traduced for levity, and ’tis said in Rome
That Photinus, an eunuch, and your maids
Manage this war.

CLEOPATRA.
Sink Rome, and their tongues rot
That speak against us! A charge we bear i’ th’ war,
And, as the president of my kingdom, will
Appear there for a man. Speak not against it.
I will not stay behind.

 Enter Antony and Canidius.

ENOBARBUS.
Nay, I have done.
Here comes the Emperor.

ANTONY.
Is it not strange, Canidius,
That from Tarentum and Brundusium
He could so quickly cut the Ionian sea
And take in Toryne?—You have heard on ’t, sweet?

CLEOPATRA.
Celerity is never more admired
Than by the negligent.

ANTONY.
A good rebuke,
Which might have well becomed the best of men
To taunt at slackness.—Canidius, we
Will fight with him by sea.

CLEOPATRA.
By sea, what else?

CANIDIUS.
Why will my lord do so?

ANTONY.
For that he dares us to ’t.

ENOBARBUS.
So hath my lord dared him to single fight.

CANIDIUS.
Ay, and to wage this battle at Pharsalia,
Where Caesar fought with Pompey. But these offers,
Which serve not for his vantage, he shakes off,
And so should you.

ENOBARBUS.
Your ships are not well manned,
Your mariners are muleteers, reapers, people
Engrossed by swift impress. In Caesar’s fleet
Are those that often have ’gainst Pompey fought.
Their ships are yare, yours heavy. No disgrace
Shall fall you for refusing him at sea,
Being prepared for land.

ANTONY.
By sea, by sea.

ENOBARBUS.
Most worthy sir, you therein throw away
The absolute soldiership you have by land;
Distract your army, which doth most consist
Of war-marked footmen; leave unexecuted
Your own renowned knowledge; quite forgo
The way which promises assurance; and
Give up yourself merely to chance and hazard
From firm security.

ANTONY.
I’ll fight at sea.

CLEOPATRA.
I have sixty sails, Caesar none better.

ANTONY.
Our overplus of shipping will we burn,
And with the rest full-manned, from th’ head of Actium
Beat th’ approaching Caesar. But if we fail,
We then can do ’t at land.

 Enter a Messenger.

Thy business?

MESSENGER.
The news is true, my lord; he is descried.
Caesar has taken Toryne.

ANTONY.
Can he be there in person? ’Tis impossible;
Strange that his power should be. Canidius,
Our nineteen legions thou shalt hold by land,
And our twelve thousand horse. We’ll to our ship.
Away, my Thetis!

 Enter a Soldier.

How now, worthy soldier?

SOLDIER.
O noble emperor, do not fight by sea.
Trust not to rotten planks. Do you misdoubt
This sword and these my wounds? Let th’ Egyptians
And the Phoenicians go a-ducking. We
Have used to conquer standing on the earth
And fighting foot to foot.

ANTONY.
Well, well, away.

 [_Exeunt Antony, Cleopatra and Enobarbus._]

SOLDIER.
By Hercules, I think I am i’ th’ right.

CANIDIUS.
Soldier, thou art. But his whole action grows
Not in the power on ’t. So our leader’s led,
And we are women’s men.

SOLDIER.
You keep by land
The legions and the horse whole, do you not?

CANIDIUS.
Marcus Octavius, Marcus Justeius,
Publicola, and Caelius are for sea,
But we keep whole by land. This speed of Caesar’s
Carries beyond belief.

SOLDIER.
While he was yet in Rome,
His power went out in such distractions as
Beguiled all spies.

CANIDIUS.
Who’s his lieutenant, hear you?

SOLDIER.
They say one Taurus.

CANIDIUS.
Well I know the man.

 Enter a Messenger.

MESSENGER.
The Emperor calls Canidius.

CANIDIUS.
With news the time’s with labour, and throes forth
Each minute some.

 [_Exeunt._]

SCENE VIII. A plain near Actium.

 Enter Caesar with his army and Taurus marching.

CAESAR.
Taurus!

TAURUS.
My lord?

CAESAR.
Strike not by land; keep whole; provoke not battle
Till we have done at sea. Do not exceed
The prescript of this scroll. Our fortune lies
Upon this jump.

 [_Exeunt._]

SCENE IX. Another part of the Plain.

 Enter Antony and Enobarbus.

ANTONY.
Set we our squadrons on yon side o’ th’ hill
In eye of Caesar’s battle, from which place
We may the number of the ships behold
And so proceed accordingly.

 [_Exeunt._]

SCENE X. Another part of the Plain.

Canidius marching with his land army one way over the stage, and
Taurus, the Lieutenant of Caesar, with his Army, the other way. After
their going in, is heard the noise of a sea fight.

 Alarum. Enter Enobarbus.

ENOBARBUS.
Naught, naught, all naught! I can behold no longer.
Th’ Antoniad, the Egyptian admiral,
With all their sixty, fly and turn the rudder.
To see ’t mine eyes are blasted.

 Enter Scarus.

SCARUS.
Gods and goddesses,
All the whole synod of them!

ENOBARBUS.
What’s thy passion?

SCARUS.
The greater cantle of the world is lost
With very ignorance. We have kissed away
Kingdoms and provinces.

ENOBARBUS.
How appears the fight?

SCARUS.
On our side, like the tokened pestilence,
Where death is sure. Yon ribaudred nag of Egypt,
Whom leprosy o’ertake, i’ th’ midst o’ th’ fight,
When vantage like a pair of twins appeared,
Both as the same—or, rather, ours the elder—
The breeze upon her, like a cow in June,
Hoists sails and flies.

ENOBARBUS.
That I beheld.
Mine eyes did sicken at the sight and could not
Endure a further view.

SCARUS.
She once being loofed,
The noble ruin of her magic, Antony,
Claps on his sea-wing and, like a doting mallard,
Leaving the fight in height, flies after her.
I never saw an action of such shame.
Experience, manhood, honour, ne’er before
Did violate so itself.

ENOBARBUS.
Alack, alack!

 Enter Canidius.

CANIDIUS.
Our fortune on the sea is out of breath
And sinks most lamentably. Had our general
Been what he knew himself, it had gone well.
O, he has given example for our flight
Most grossly by his own!

ENOBARBUS.
Ay, are you thereabouts?
Why, then, good night indeed.

CANIDIUS.
Toward Peloponnesus are they fled.

SCARUS.
’Tis easy to’t, and there I will attend
What further comes.

CANIDIUS.
To Caesar will I render
My legions and my horse. Six kings already
Show me the way of yielding.

ENOBARBUS.
I’ll yet follow
The wounded chance of Antony, though my reason
Sits in the wind against me.

 [_Exeunt._]

SCENE XI. Alexandria. A Room in the Palace.

 Enter Antony with attendants.

ANTONY.
Hark, the land bids me tread no more upon’t.
It is ashamed to bear me. Friends, come hither.
I am so lated in the world that I
Have lost my way for ever. I have a ship
Laden with gold. Take that, divide it. Fly,
And make your peace with Caesar.

ALL.
Fly? Not we.

ANTONY.
I have fled myself, and have instructed cowards
To run and show their shoulders. Friends, be gone.
I have myself resolved upon a course
Which has no need of you. Be gone.
My treasure’s in the harbour. Take it. O,
I followed that I blush to look upon.
My very hairs do mutiny, for the white
Reprove the brown for rashness, and they them
For fear and doting. Friends, be gone. You shall
Have letters from me to some friends that will
Sweep your way for you. Pray you, look not sad,
Nor make replies of loathness. Take the hint
Which my despair proclaims. Let that be left
Which leaves itself. To the sea-side straightway.
I will possess you of that ship and treasure.
Leave me, I pray, a little—pray you, now,
Nay, do so; for indeed I have lost command.
Therefore I pray you. I’ll see you by and by.

 [_Sits down._]

 Enter Cleopatra led by Charmian, Iras and Eros.

EROS.
Nay, gentle madam, to him! Comfort him.

IRAS.
Do, most dear queen.

CHARMIAN.
Do! Why, what else?

CLEOPATRA.
Let me sit down. O Juno!

ANTONY.
No, no, no, no, no.

EROS.
See you here, sir?

ANTONY.
O, fie, fie, fie!

CHARMIAN.
Madam.

IRAS.
Madam, O good empress!

EROS.
Sir, sir!

ANTONY.
Yes, my lord, yes. He at Philippi kept
His sword e’en like a dancer, while I struck
The lean and wrinkled Cassius, and ’twas I
That the mad Brutus ended. He alone
Dealt on lieutenantry, and no practice had
In the brave squares of war. Yet now—no matter.

CLEOPATRA.
Ah, stand by.

EROS.
The Queen, my lord, the Queen!

IRAS.
Go to him, madam; speak to him.
He is unqualitied with very shame.

CLEOPATRA.
Well then, sustain me. O!

EROS.
Most noble sir, arise. The Queen approaches.
Her head’s declined, and death will seize her but
Your comfort makes the rescue.

ANTONY.
I have offended reputation,
A most unnoble swerving.

EROS.
Sir, the Queen.

ANTONY.
O, whither hast thou led me, Egypt? See
How I convey my shame out of thine eyes
By looking back what I have left behind
’Stroyed in dishonour.

CLEOPATRA.
O my lord, my lord,
Forgive my fearful sails! I little thought
You would have followed.

ANTONY.
Egypt, thou knew’st too well
My heart was to thy rudder tied by th’ strings,
And thou shouldst tow me after. O’er my spirit
Thy full supremacy thou knew’st, and that
Thy beck might from the bidding of the gods
Command me.

CLEOPATRA.
O, my pardon!

ANTONY.
Now I must
To the young man send humble treaties, dodge
And palter in the shifts of lowness, who
With half the bulk o’ th’ world played as I pleased,
Making and marring fortunes. You did know
How much you were my conqueror, and that
My sword, made weak by my affection, would
Obey it on all cause.

CLEOPATRA.
Pardon, pardon!

ANTONY.
Fall not a tear, I say; one of them rates
All that is won and lost. Give me a kiss.
Even this repays me.
We sent our schoolmaster. Is he come back?
Love, I am full of lead. Some wine
Within there, and our viands! Fortune knows
We scorn her most when most she offers blows.

 [_Exeunt._]

SCENE XII. Caesar’s camp in Egypt.

 Enter Caesar, Agrippa, Dolabella with others.

CAESAR.
Let him appear that’s come from Antony.
Know you him?

DOLABELLA.
Caesar, ’tis his schoolmaster—
An argument that he is plucked, when hither
He sends so poor a pinion of his wing,
Which had superfluous kings for messengers
Not many moons gone by.

 Enter Ambassador from Anthony.

CAESAR.
Approach, and speak.

AMBASSADOR.
Such as I am, I come from Antony.
I was of late as petty to his ends
As is the morn-dew on the myrtle leaf
To his grand sea.

CAESAR.
Be’t so. Declare thine office.

AMBASSADOR.
Lord of his fortunes he salutes thee, and
Requires to live in Egypt, which not granted,
He lessens his requests, and to thee sues
To let him breathe between the heavens and earth,
A private man in Athens. This for him.
Next, Cleopatra does confess thy greatness,
Submits her to thy might, and of thee craves
The circle of the Ptolemies for her heirs,
Now hazarded to thy grace.

CAESAR.
For Antony,
I have no ears to his request. The queen
Of audience nor desire shall fail, so she
From Egypt drive her all-disgraced friend,
Or take his life there. This if she perform,
She shall not sue unheard. So to them both.

AMBASSADOR.
Fortune pursue thee!

CAESAR.
Bring him through the bands.

 [_Exit Ambassador, attended._]

[_To Thidias_.] To try thy eloquence now ’tis time. Dispatch.
From Antony win Cleopatra. Promise,
And in our name, what she requires; add more,
From thine invention, offers. Women are not
In their best fortunes strong, but want will perjure
The ne’er-touch’d vestal. Try thy cunning, Thidias;
Make thine own edict for thy pains, which we
Will answer as a law.

THIDIAS.
Caesar, I go.

CAESAR.
Observe how Antony becomes his flaw,
And what thou think’st his very action speaks
In every power that moves.

THIDIAS.
Caesar, I shall.

 [_Exeunt._]

SCENE XIII. Alexandria. A Room in the Palace.

 Enter Cleopatra, Enobarbus, Charmian and Iras.

CLEOPATRA.
What shall we do, Enobarbus?

ENOBARBUS.
Think, and die.

CLEOPATRA.
Is Antony or we in fault for this?

ENOBARBUS.
Antony only, that would make his will
Lord of his reason. What though you fled
From that great face of war, whose several ranges
Frighted each other? Why should he follow?
The itch of his affection should not then
Have nicked his captainship, at such a point,
When half to half the world opposed, he being
The mered question. ’Twas a shame no less
Than was his loss, to course your flying flags
And leave his navy gazing.

CLEOPATRA.
Prithee, peace.

 Enter the Ambassador with Antony.

ANTONY.
Is that his answer?

AMBASSADOR.
Ay, my lord.

ANTONY.
The Queen shall then have courtesy, so she
Will yield us up.

AMBASSADOR.
He says so.

ANTONY.
Let her know’t.—
To the boy Caesar send this grizzled head,
And he will fill thy wishes to the brim
With principalities.

CLEOPATRA.
That head, my lord?

ANTONY.
To him again. Tell him he wears the rose
Of youth upon him, from which the world should note
Something particular: his coin, ships, legions,
May be a coward’s; whose ministers would prevail
Under the service of a child as soon
As i’ th’ command of Caesar. I dare him therefore
To lay his gay comparisons apart,
And answer me declined, sword against sword,
Ourselves alone. I’ll write it. Follow me.

 [_Exeunt Antony and Ambassador._]

ENOBARBUS.
Yes, like enough high-battled Caesar will
Unstate his happiness, and be staged to th’ show
Against a sworder! I see men’s judgments are
A parcel of their fortunes, and things outward
Do draw the inward quality after them
To suffer all alike. That he should dream,
Knowing all measures, the full Caesar will
Answer his emptiness! Caesar, thou hast subdued
His judgment too.

 Enter a Servant.

SERVANT.
A messenger from Caesar.

CLEOPATRA.
What, no more ceremony? See, my women,
Against the blown rose may they stop their nose
That kneeled unto the buds. Admit him, sir.

 [_Exit Servant._]

ENOBARBUS.
[_Aside_.] Mine honesty and I begin to square.
The loyalty well held to fools does make
Our faith mere folly. Yet he that can endure
To follow with allegiance a fallen lord
Does conquer him that did his master conquer,
And earns a place i’ th’ story.

 Enter Thidias.

CLEOPATRA.
Caesar’s will?

THIDIAS.
Hear it apart.

CLEOPATRA.
None but friends. Say boldly.

THIDIAS.
So haply are they friends to Antony.

ENOBARBUS.
He needs as many, sir, as Caesar has,
Or needs not us. If Caesar please, our master
Will leap to be his friend. For us, you know
Whose he is we are, and that is Caesar’s.

THIDIAS.
So.—
Thus then, thou most renowned: Caesar entreats
Not to consider in what case thou stand’st
Further than he is Caesar.

CLEOPATRA.
Go on; right royal.

THIDIAS.
He knows that you embrace not Antony
As you did love, but as you feared him.

CLEOPATRA.
O!

THIDIAS.
The scars upon your honour, therefore, he
Does pity as constrained blemishes,
Not as deserved.

CLEOPATRA.
He is a god and knows
What is most right. Mine honour was not yielded,
But conquered merely.

ENOBARBUS.
[_Aside_.] To be sure of that,
I will ask Antony. Sir, sir, thou art so leaky
That we must leave thee to thy sinking, for
Thy dearest quit thee.

 [_Exit Enobarbus._]

THIDIAS.
Shall I say to Caesar
What you require of him? For he partly begs
To be desired to give. It much would please him
That of his fortunes you should make a staff
To lean upon. But it would warm his spirits
To hear from me you had left Antony,
And put yourself under his shroud,
The universal landlord.

CLEOPATRA.
What’s your name?

THIDIAS.
My name is Thidias.

CLEOPATRA.
Most kind messenger,
Say to great Caesar this in deputation:
I kiss his conqu’ring hand. Tell him I am prompt
To lay my crown at’s feet, and there to kneel.
Tell him, from his all-obeying breath I hear
The doom of Egypt.

THIDIAS.
’Tis your noblest course.
Wisdom and fortune combating together,
If that the former dare but what it can,
No chance may shake it. Give me grace to lay
My duty on your hand.

CLEOPATRA.
Your Caesar’s father oft,
When he hath mused of taking kingdoms in,
Bestowed his lips on that unworthy place
As it rained kisses.

 Enter Antony and Enobarbus.

ANTONY.
Favours, by Jove that thunders!
What art thou, fellow?

THIDIAS.
One that but performs
The bidding of the fullest man and worthiest
To have command obeyed.

ENOBARBUS.
[_Aside_.] You will be whipped.

ANTONY.
Approach there.—Ah, you kite!—Now, gods and devils,
Authority melts from me. Of late when I cried “Ho!”
Like boys unto a muss, kings would start forth
And cry “Your will?” Have you no ears? I am
Antony yet.

 Enter Servants.

Take hence this jack and whip him.

ENOBARBUS.
’Tis better playing with a lion’s whelp
Than with an old one dying.

ANTONY.
Moon and stars!
Whip him. Were’t twenty of the greatest tributaries
That do acknowledge Caesar, should I find them
So saucy with the hand of she here—what’s her name
Since she was Cleopatra? Whip him, fellows,
Till like a boy you see him cringe his face
And whine aloud for mercy. Take him hence.

THIDIAS.
Mark Antony—

ANTONY.
Tug him away. Being whipp’d,
Bring him again. This jack of Caesar’s shall
Bear us an errand to him.

 [_Exeunt Servants with Thidias._]

You were half blasted ere I knew you. Ha!
Have I my pillow left unpressed in Rome,
Forborne the getting of a lawful race,
And by a gem of women, to be abused
By one that looks on feeders?

CLEOPATRA.
Good my lord—

ANTONY.
You have been a boggler ever.
But when we in our viciousness grow hard—
O misery on’t!—the wise gods seal our eyes,
In our own filth drop our clear judgments, make us
Adore our errors, laugh at’s while we strut
To our confusion.

CLEOPATRA.
O, is’t come to this?

ANTONY.
I found you as a morsel cold upon
Dead Caesar’s trencher; nay, you were a fragment
Of Gneius Pompey’s, besides what hotter hours,
Unregistered in vulgar fame, you have
Luxuriously pick’d out. For I am sure,
Though you can guess what temperance should be,
You know not what it is.

CLEOPATRA.
Wherefore is this?

ANTONY.
To let a fellow that will take rewards
And say “God quit you!” be familiar with
My playfellow, your hand, this kingly seal
And plighter of high hearts! O that I were
Upon the hill of Basan, to outroar
The horned herd! For I have savage cause,
And to proclaim it civilly were like
A haltered neck which does the hangman thank
For being yare about him.

 Enter a Servant with Thidias.

Is he whipped?

SERVANT.
Soundly, my lord.

ANTONY.
Cried he? And begged he pardon?

SERVANT.
He did ask favour.

ANTONY.
If that thy father live, let him repent
Thou wast not made his daughter; and be thou sorry
To follow Caesar in his triumph, since
Thou hast been whipped for following him. Henceforth
The white hand of a lady fever thee;
Shake thou to look on’t. Get thee back to Caesar;
Tell him thy entertainment. Look thou say
He makes me angry with him; for he seems
Proud and disdainful, harping on what I am,
Not what he knew I was. He makes me angry,
And at this time most easy ’tis to do’t,
When my good stars that were my former guides
Have empty left their orbs and shot their fires
Into th’ abysm of hell. If he mislike
My speech and what is done, tell him he has
Hipparchus, my enfranched bondman, whom
He may at pleasure whip, or hang, or torture,
As he shall like, to quit me. Urge it thou.
Hence with thy stripes, be gone.

 [_Exit Thidias._]

CLEOPATRA.
Have you done yet?

ANTONY.
Alack, our terrene moon is now eclipsed,
And it portends alone the fall of Antony.

CLEOPATRA.
I must stay his time.

ANTONY.
To flatter Caesar, would you mingle eyes
With one that ties his points?

CLEOPATRA.
Not know me yet?

ANTONY.
Cold-hearted toward me?

CLEOPATRA.
Ah, dear, if I be so,
From my cold heart let heaven engender hail
And poison it in the source, and the first stone
Drop in my neck; as it determines, so
Dissolve my life! The next Caesarion smite,
Till, by degrees the memory of my womb,
Together with my brave Egyptians all,
By the discandying of this pelleted storm,
Lie graveless, till the flies and gnats of Nile
Have buried them for prey!

ANTONY.
I am satisfied.
Caesar sits down in Alexandria, where
I will oppose his fate. Our force by land
Hath nobly held; our severed navy too
Have knit again, and fleet, threat’ning most sea-like.
Where hast thou been, my heart? Dost thou hear, lady?
If from the field I shall return once more
To kiss these lips, I will appear in blood.
I and my sword will earn our chronicle.
There’s hope in’t yet.

CLEOPATRA.
That’s my brave lord!

ANTONY.
I will be treble-sinewed, hearted, breathed,
And fight maliciously. For when mine hours
Were nice and lucky, men did ransom lives
Of me for jests. But now I’ll set my teeth
And send to darkness all that stop me. Come,
Let’s have one other gaudy night. Call to me
All my sad captains. Fill our bowls once more
Let’s mock the midnight bell.

CLEOPATRA.
It is my birthday.
I had thought t’have held it poor, but since my lord
Is Antony again, I will be Cleopatra.

ANTONY.
We will yet do well.

CLEOPATRA.
Call all his noble captains to my lord.

ANTONY.
Do so; we’ll speak to them; and tonight I’ll force
The wine peep through their scars. Come on, my queen,
There’s sap in’t yet. The next time I do fight
I’ll make Death love me, for I will contend
Even with his pestilent scythe.

 [_Exeunt all but Enobarbus._]

ENOBARBUS.
Now he’ll outstare the lightning. To be furious
Is to be frighted out of fear, and in that mood
The dove will peck the estridge; and I see still
A diminution in our captain’s brain
Restores his heart. When valour preys on reason,
It eats the sword it fights with. I will seek
Some way to leave him.

 [_Exit._]




ACT IV


SCENE I. Caesar’s Camp at Alexandria.

 Enter Caesar, Agrippa, and Maecenas, with his army.
Caesar reading a letter.

CAESAR.
He calls me boy, and chides as he had power
To beat me out of Egypt. My messenger
He hath whipped with rods; dares me to personal combat,
Caesar to Antony. Let the old ruffian know
I have many other ways to die; meantime
Laugh at his challenge.

MAECENAS.
Caesar must think,
When one so great begins to rage, he’s hunted
Even to falling. Give him no breath, but now
Make boot of his distraction. Never anger
Made good guard for itself.

CAESAR.
Let our best heads
Know that tomorrow the last of many battles
We mean to fight. Within our files there are,
Of those that served Mark Antony but late,
Enough to fetch him in. See it done,
And feast the army; we have store to do’t,
And they have earned the waste. Poor Antony!

 [_Exeunt._]

SCENE II. Alexandria. A Room in the Palace.

 Enter Antony, Cleopatra, Enobarbus, Charmian, Iras, Alexas with
 others.

ANTONY.
He will not fight with me, Domitius?

ENOBARBUS.
No.

ANTONY.
Why should he not?

ENOBARBUS.
He thinks, being twenty times of better fortune,
He is twenty men to one.

ANTONY.
Tomorrow, soldier,
By sea and land I’ll fight. Or I will live,
Or bathe my dying honour in the blood
Shall make it live again. Woo’t thou fight well?

ENOBARBUS.
I’ll strike, and cry “Take all.”

ANTONY.
Well said. Come on.
Call forth my household servants. Let’s tonight
Be bounteous at our meal.—

 Enter Servants.

Give me thy hand.
Thou has been rightly honest; so hast thou,
Thou, and thou, and thou. You have served me well,
And kings have been your fellows.

CLEOPATRA.
[_Aside to Enobarbus_.] What means this?

ENOBARBUS.
[_Aside to Cleopatra_.] ’Tis one of those odd tricks which sorrow
shoots
Out of the mind.

ANTONY.
And thou art honest too.
I wish I could be made so many men,
And all of you clapped up together in
An Antony, that I might do you service
So good as you have done.

ALL THE SERVANTS.
The gods forbid!

ANTONY.
Well, my good fellows, wait on me tonight.
Scant not my cups, and make as much of me
As when mine empire was your fellow too
And suffered my command.

CLEOPATRA.
[_Aside to Enobarbus_.] What does he mean?

ENOBARBUS.
[_Aside to Cleopatra_.] To make his followers weep.

ANTONY.
Tend me tonight;
May be it is the period of your duty.
Haply you shall not see me more, or if,
A mangled shadow. Perchance tomorrow
You’ll serve another master. I look on you
As one that takes his leave. Mine honest friends,
I turn you not away, but, like a master
Married to your good service, stay till death.
Tend me tonight two hours, I ask no more,
And the gods yield you for’t!

ENOBARBUS.
What mean you, sir,
To give them this discomfort? Look, they weep,
And I, an ass, am onion-eyed. For shame,
Transform us not to women.

ANTONY.
Ho, ho, ho!
Now the witch take me if I meant it thus!
Grace grow where those drops fall! My hearty friends,
You take me in too dolorous a sense,
For I spake to you for your comfort, did desire you
To burn this night with torches. Know, my hearts,
I hope well of tomorrow, and will lead you
Where rather I’ll expect victorious life
Than death and honour. Let’s to supper, come,
And drown consideration.

 [_Exeunt._]

SCENE III. Alexandria. Before the Palace.

 Enter a Company of Soldiers.

FIRST SOLDIER.
Brother, good night. Tomorrow is the day.

SECOND SOLDIER.
It will determine one way. Fare you well.
Heard you of nothing strange about the streets?

FIRST SOLDIER.
Nothing. What news?

SECOND SOLDIER.
Belike ’tis but a rumour. Good night to you.

FIRST SOLDIER.
Well, sir, good night.

 Enter two other Soldiers.

SECOND SOLDIER.
Soldiers, have careful watch.

THIRD SOLDIER.
And you. Good night, good night.

 [_They place themselves in every corner of the stage._]

SECOND SOLDIER.
Here we. And if tomorrow
Our navy thrive, I have an absolute hope
Our landmen will stand up.

FIRST SOLDIER.
’Tis a brave army, and full of purpose.

 [_Music of the hautboys under the stage._]

SECOND SOLDIER.
Peace, what noise?

FIRST SOLDIER.
List, list!

SECOND SOLDIER.
Hark!

FIRST SOLDIER.
Music i’ th’ air.

THIRD SOLDIER.
Under the earth.

FOURTH SOLDIER.
It signs well, does it not?

THIRD SOLDIER.
No.

FIRST SOLDIER.
Peace, I say! What should this mean?

SECOND SOLDIER.
’Tis the god Hercules, whom Antony loved,
Now leaves him.

FIRST SOLDIER.
Walk. Let’s see if other watchmen
Do hear what we do.

 [_They advance to another post._]

SECOND SOLDIER.
How now, masters!

ALL.
How now! How now! Do you hear this?

FIRST SOLDIER.
Ay. Is’t not strange?

THIRD SOLDIER.
Do you hear, masters? Do you hear?

FIRST SOLDIER.
Follow the noise so far as we have quarter.
Let’s see how it will give off.

ALL.
Content. ’Tis strange.

 [_Exeunt._]

SCENE IV. Alexandria. A Room in the Palace.

 Enter Antony and Cleopatra with others.

ANTONY.
Eros! Mine armour, Eros!

CLEOPATRA.
Sleep a little.

ANTONY.
No, my chuck.—Eros! Come, mine armour, Eros!

 Enter Eros with armour.

Come, good fellow, put thine iron on.
If fortune be not ours today, it is
Because we brave her. Come.

CLEOPATRA.
Nay, I’ll help too.
What’s this for?

ANTONY.
Ah, let be, let be! Thou art
The armourer of my heart. False, false. This, this!

CLEOPATRA.
Sooth, la, I’ll help. Thus it must be.

ANTONY.
Well, well,
We shall thrive now. Seest thou, my good fellow?
Go put on thy defences.

EROS.
Briefly, sir.

CLEOPATRA.
Is not this buckled well?

ANTONY.
Rarely, rarely.
He that unbuckles this, till we do please
To daff’t for our repose, shall hear a storm.
Thou fumblest, Eros, and my queen’s a squire
More tight at this than thou. Dispatch. O love,
That thou couldst see my wars today, and knew’st
The royal occupation, thou shouldst see
A workman in’t.

 Enter an Officer, armed.

Good morrow to thee. Welcome.
Thou look’st like him that knows a warlike charge.
To business that we love we rise betime
And go to’t with delight.

OFFICER.
A thousand, sir,
Early though’t be, have on their riveted trim
And at the port expect you.

 [_Shout. Trumpets flourish._]

 Enter other Captains and Soldiers.

CAPTAIN.
The morn is fair. Good morrow, general.

ALL.
Good morrow, general.

ANTONY.
’Tis well blown, lads.
This morning, like the spirit of a youth
That means to be of note, begins betimes.
So, so. Come, give me that. This way. Well said.
Fare thee well, dame.
Whate’er becomes of me,
This is a soldier’s kiss. [_Kisses her._] Rebukeable
And worthy shameful check it were, to stand
On more mechanic compliment. I’ll leave thee
Now like a man of steel.—You that will fight,
Follow me close, I’ll bring you to’t. Adieu.

 [_Exeunt Antony, Eros, Captains and Soldiers._]

CHARMIAN.
Please you, retire to your chamber.

CLEOPATRA.
Lead me.
He goes forth gallantly. That he and Caesar might
Determine this great war in single fight!
Then Antony—but now—. Well, on.

 [_Exeunt._]

SCENE V. Antony’s camp near Alexandria.

 Trumpets sound. Enter Antony and Eros, a Soldier meeting them.

SOLDIER.
The gods make this a happy day to Antony!

ANTONY.
Would thou and those thy scars had once prevailed
To make me fight at land!

SOLDIER.
Hadst thou done so,
The kings that have revolted and the soldier
That has this morning left thee would have still
Followed thy heels.

ANTONY.
Who’s gone this morning?

SOLDIER.
Who?
One ever near thee. Call for Enobarbus,
He shall not hear thee, or from Caesar’s camp
Say “I am none of thine.”

ANTONY.
What sayest thou?

SOLDIER.
Sir,
He is with Caesar.

EROS.
Sir, his chests and treasure
He has not with him.

ANTONY.
Is he gone?

SOLDIER.
Most certain.

ANTONY.
Go, Eros, send his treasure after. Do it.
Detain no jot, I charge thee. Write to him—
I will subscribe—gentle adieus and greetings.
Say that I wish he never find more cause
To change a master. O, my fortunes have
Corrupted honest men! Dispatch.—Enobarbus!

 [_Exeunt._]

SCENE VI. Alexandria. Caesar’s camp.

 Flourish. Enter Agrippa, Caesar with Enobarbus and Dolabella.

CAESAR.
Go forth, Agrippa, and begin the fight.
Our will is Antony be took alive;
Make it so known.

AGRIPPA.
Caesar, I shall.

 [_Exit._]

CAESAR.
The time of universal peace is near.
Prove this a prosp’rous day, the three-nooked world
Shall bear the olive freely.

 Enter a Messenger.

MESSENGER.
Antony
Is come into the field.

CAESAR.
Go charge Agrippa
Plant those that have revolted in the van
That Antony may seem to spend his fury
Upon himself.

 [_Exeunt Caesar and his Train._]

ENOBARBUS.
Alexas did revolt and went to Jewry on
Affairs of Antony; there did dissuade
Great Herod to incline himself to Caesar
And leave his master Antony. For this pains
Caesar hath hanged him. Canidius and the rest
That fell away have entertainment but
No honourable trust. I have done ill,
Of which I do accuse myself so sorely
That I will joy no more.

 Enter a Soldier of Caesar’s.

SOLDIER.
Enobarbus, Antony
Hath after thee sent all thy treasure, with
His bounty overplus. The messenger
Came on my guard, and at thy tent is now
Unloading of his mules.

ENOBARBUS.
I give it you.

SOLDIER.
Mock not, Enobarbus.
I tell you true. Best you safed the bringer
Out of the host. I must attend mine office,
Or would have done’t myself. Your emperor
Continues still a Jove.

 [_Exit._]

ENOBARBUS.
I am alone the villain of the earth,
And feel I am so most. O Antony,
Thou mine of bounty, how wouldst thou have paid
My better service, when my turpitude
Thou dost so crown with gold! This blows my heart.
If swift thought break it not, a swifter mean
Shall outstrike thought, but thought will do’t, I feel.
I fight against thee! No, I will go seek
Some ditch wherein to die; the foul’st best fits
My latter part of life.

 [_Exit._]

SCENE VII. Field of battle between the Camps.

 Alarum. Drums and Trumpets. Enter Agrippa and others.

AGRIPPA.
Retire! We have engaged ourselves too far.
Caesar himself has work, and our oppression
Exceeds what we expected.

 [_Exeunt._]

 Alarums. Enter Antony and Scarus wounded.

SCARUS.
O my brave emperor, this is fought indeed!
Had we done so at first, we had droven them home
With clouts about their heads.

ANTONY.
Thou bleed’st apace.

SCARUS.
I had a wound here that was like a T,
But now ’tis made an H.

 _Sounds retreat far off._
ANTONY.
They do retire.

SCARUS.
We’ll beat ’em into bench-holes. I have yet
Room for six scotches more.

 Enter Eros.

EROS.
They are beaten, sir, and our advantage serves
For a fair victory.

SCARUS.
Let us score their backs
And snatch ’em up as we take hares, behind.
’Tis sport to maul a runner.

ANTONY.
I will reward thee
Once for thy sprightly comfort, and tenfold
For thy good valour. Come thee on.

SCARUS.
I’ll halt after.

 [_Exeunt._]

SCENE VIII. Under the Walls of Alexandria.

 Alarum. Enter Antony again in a march; Scarus with others.

ANTONY.
We have beat him to his camp. Run one before
And let the Queen know of our gests.
Tomorrow,
Before the sun shall see’s, we’ll spill the blood
That has today escaped. I thank you all,
For doughty-handed are you, and have fought
Not as you served the cause, but as’t had been
Each man’s like mine. You have shown all Hectors.
Enter the city, clip your wives, your friends,
Tell them your feats; whilst they with joyful tears
Wash the congealment from your wounds and kiss
The honoured gashes whole.

 Enter Cleopatra.

[_To Scarus_.] Give me thy hand.
To this great fairy I’ll commend thy acts,
Make her thanks bless thee. O thou day o’ th’ world,
Chain mine armed neck. Leap thou, attire and all,
Through proof of harness to my heart, and there
Ride on the pants triumphing.

CLEOPATRA.
Lord of lords!
O infinite virtue, com’st thou smiling from
The world’s great snare uncaught?

ANTONY.
Mine nightingale,
We have beat them to their beds. What, girl! Though grey
Do something mingle with our younger brown, yet ha’ we
A brain that nourishes our nerves and can
Get goal for goal of youth. Behold this man.
Commend unto his lips thy favouring hand.—
Kiss it, my warrior. He hath fought today
As if a god, in hate of mankind, had
Destroyed in such a shape.

CLEOPATRA.
I’ll give thee, friend,
An armour all of gold. It was a king’s.

ANTONY.
He has deserved it, were it carbuncled
Like holy Phœbus’ car. Give me thy hand.
Through Alexandria make a jolly march;
Bear our hacked targets like the men that owe them.
Had our great palace the capacity
To camp this host, we all would sup together
And drink carouses to the next day’s fate,
Which promises royal peril.—Trumpeters,
With brazen din blast you the city’s ear;
Make mingle with our rattling tabourines,
That heaven and earth may strike their sounds together,
Applauding our approach.

 [_Exeunt._]

SCENE IX. Caesar’s camp.

 Enter a Sentry and his company. Enobarbus follows.

SENTRY.
If we be not relieved within this hour,
We must return to th’ court of guard. The night
Is shiny, and they say we shall embattle
By th’ second hour i’ th’ morn.

FIRST WATCH.
This last day was a shrewd one to’s.

ENOBARBUS.
O, bear me witness, night.—

SECOND WATCH.
What man is this?

FIRST WATCH.
Stand close and list him.

ENOBARBUS.
Be witness to me, O thou blessed moon,
When men revolted shall upon record
Bear hateful memory, poor Enobarbus did
Before thy face repent.

SENTRY.
Enobarbus?

SECOND WATCH.
Peace! Hark further.

ENOBARBUS.
O sovereign mistress of true melancholy,
The poisonous damp of night disponge upon me,
That life, a very rebel to my will,
May hang no longer on me. Throw my heart
Against the flint and hardness of my fault,
Which, being dried with grief, will break to powder
And finish all foul thoughts. O Antony,
Nobler than my revolt is infamous,
Forgive me in thine own particular,
But let the world rank me in register
A master-leaver and a fugitive.
O Antony! O Antony!

 [_Dies._]

FIRST WATCH.
Let’s speak to him.

SENTRY.
Let’s hear him, for the things he speaks may concern Caesar.

SECOND WATCH.
Let’s do so. But he sleeps.

SENTRY.
Swoons rather, for so bad a prayer as his
Was never yet for sleep.

FIRST WATCH.
Go we to him.

SECOND WATCH.
Awake, sir, awake! Speak to us.

FIRST WATCH.
Hear you, sir?

SENTRY.
The hand of death hath raught him.

 [_Drums afar off._]

Hark! The drums
Demurely wake the sleepers. Let us bear him
To th’ court of guard; he is of note. Our hour
Is fully out.

SECOND WATCH.
Come on, then. He may recover yet.

 [_Exeunt with the body._]

SCENE X. Ground between the two Camps.

 Enter Antony and Scarus with their army.

ANTONY.
Their preparation is today by sea;
We please them not by land.

SCARUS.
For both, my lord.

ANTONY.
I would they’d fight i’ th’ fire or i’ th’ air;
We’d fight there too. But this it is: our foot
Upon the hills adjoining to the city
Shall stay with us—order for sea is given;
They have put forth the haven—
Where their appointment we may best discover
And look on their endeavour.

 [_Exeunt._]

SCENE XI. Another part of the Ground.

 Enter Caesar and his army.

CAESAR.
But being charged, we will be still by land,
Which, as I take’t, we shall, for his best force
Is forth to man his galleys. To the vales,
And hold our best advantage.

 [_Exeunt._]

SCENE XII. Another part of the Ground.

 Alarum afar off, as at a sea fight. Enter Antony and Scarus.

ANTONY.
Yet they are not joined. Where yond pine does stand
I shall discover all. I’ll bring thee word
Straight how ’tis like to go.

 [_Exit._]

SCARUS.
Swallows have built
In Cleopatra’s sails their nests. The augurs
Say they know not, they cannot tell; look grimly,
And dare not speak their knowledge. Antony
Is valiant and dejected, and by starts
His fretted fortunes give him hope and fear
Of what he has and has not.

 Enter Antony.

ANTONY.
All is lost!
This foul Egyptian hath betrayed me.
My fleet hath yielded to the foe, and yonder
They cast their caps up and carouse together
Like friends long lost. Triple-turned whore! ’Tis thou
Hast sold me to this novice, and my heart
Makes only wars on thee. Bid them all fly;
For when I am revenged upon my charm,
I have done all. Bid them all fly! Be gone!

 [_Exit Scarus._]

O sun, thy uprise shall I see no more.
Fortune and Antony part here; even here
Do we shake hands. All come to this! The hearts
That spanieled me at heels, to whom I gave
Their wishes, do discandy, melt their sweets
On blossoming Caesar, and this pine is barked
That overtopped them all. Betray’d I am:
O this false soul of Egypt! This grave charm,
Whose eye becked forth my wars and called them home,
Whose bosom was my crownet, my chief end,
Like a right gypsy hath at fast and loose
Beguiled me to the very heart of loss.
What, Eros, Eros!

 Enter Cleopatra.

Ah, thou spell! Avaunt!

CLEOPATRA.
Why is my lord enraged against his love?

ANTONY.
Vanish, or I shall give thee thy deserving
And blemish Caesar’s triumph. Let him take thee
And hoist thee up to the shouting plebeians!
Follow his chariot, like the greatest spot
Of all thy sex; most monster-like be shown
For poor’st diminutives, for dolts, and let
Patient Octavia plough thy visage up
With her prepared nails.

 [_Exit Cleopatra._]

’Tis well thou’rt gone,
If it be well to live; but better ’twere
Thou fell’st into my fury, for one death
Might have prevented many.—Eros, ho!—
The shirt of Nessus is upon me. Teach me,
Alcides, thou mine ancestor, thy rage.
Let me lodge Lichas on the horns o’ th’ moon,
And with those hands that grasped the heaviest club
Subdue my worthiest self. The witch shall die.
To the young Roman boy she hath sold me, and I fall
Under this plot. She dies for’t.—Eros, ho!

 [_Exit._]

SCENE XIII. Alexandria. A Room in the Palace.

 Enter Cleopatra, Charmian, Iras and Mardian.

CLEOPATRA.
Help me, my women! O, he is more mad
Than Telamon for his shield; the boar of Thessaly
Was never so embossed.

CHARMIAN.
To th’ monument!
There lock yourself and send him word you are dead.
The soul and body rive not more in parting
Than greatness going off.

CLEOPATRA.
To th’ monument!
Mardian, go tell him I have slain myself.
Say that the last I spoke was “Antony”,
And word it, prithee, piteously. Hence, Mardian,
And bring me how he takes my death.—To th’ monument!

 [_Exeunt._]

SCENE XIV. Alexandria. Another Room.

 Enter Antony and Eros.

ANTONY.
Eros, thou yet behold’st me?

EROS.
Ay, noble lord.

ANTONY.
Sometime we see a cloud that’s dragonish,
A vapour sometime like a bear or lion,
A towered citadel, a pendant rock,
A forked mountain, or blue promontory
With trees upon’t, that nod unto the world
And mock our eyes with air. Thou hast seen these signs.
They are black vesper’s pageants.

EROS.
Ay, my lord.

ANTONY.
That which is now a horse, even with a thought
The rack dislimns and makes it indistinct
As water is in water.

EROS.
It does, my lord.

ANTONY.
My good knave Eros, now thy captain is
Even such a body. Here I am Antony,
Yet cannot hold this visible shape, my knave.
I made these wars for Egypt, and the Queen,
Whose heart I thought I had, for she had mine,
Which, whilst it was mine, had annexed unto’t
A million more, now lost—she, Eros, has
Packed cards with Caesar, and false-played my glory
Unto an enemy’s triumph.
Nay, weep not, gentle Eros. There is left us
Ourselves to end ourselves.

 Enter Mardian.

O, thy vile lady!
She has robbed me of my sword.

MARDIAN.
No, Antony,
My mistress loved thee and her fortunes mingled
With thine entirely.

ANTONY.
Hence, saucy eunuch! Peace!
She hath betrayed me and shall die the death.

MARDIAN.
Death of one person can be paid but once,
And that she has discharged. What thou wouldst do
Is done unto thy hand. The last she spake
Was “Antony! Most noble Antony!”
Then in the midst a tearing groan did break
The name of Antony; it was divided
Between her heart and lips. She rendered life,
Thy name so buried in her.

ANTONY.
Dead, then?

MARDIAN.
Dead.

ANTONY.
Unarm, Eros. The long day’s task is done,
And we must sleep.—That thou depart’st hence safe
Does pay thy labour richly. Go.

 [_Exit Mardian._]

Off, pluck off!
The sevenfold shield of Ajax cannot keep
The battery from my heart. O, cleave, my sides!
Heart, once be stronger than thy continent;
Crack thy frail case! Apace, Eros, apace!
No more a soldier. Bruised pieces, go;
You have been nobly borne. From me awhile.

 [_Exit Eros._]

I will o’ertake thee, Cleopatra, and
Weep for my pardon. So it must be, for now
All length is torture. Since the torch is out,
Lie down, and stray no farther. Now all labour
Mars what it does; yea, very force entangles
Itself with strength. Seal then, and all is done.
Eros!—I come, my queen.—Eros!—Stay for me.
Where souls do couch on flowers, we’ll hand in hand,
And with our sprightly port make the ghosts gaze.
Dido and her Aeneas shall want troops,
And all the haunt be ours.—Come, Eros, Eros!

 Enter Eros.

EROS.
What would my lord?

ANTONY.
Since Cleopatra died,
I have lived in such dishonour that the gods
Detest my baseness. I, that with my sword
Quartered the world, and o’er green Neptune’s back
With ships made cities, condemn myself to lack
The courage of a woman; less noble mind
Than she which, by her death, our Caesar tells
“I am conqueror of myself.” Thou art sworn, Eros,
That when the exigent should come, which now
Is come indeed, when I should see behind me
Th’ inevitable prosecution of
Disgrace and horror, that on my command
Thou then wouldst kill me. Do’t. The time is come.
Thou strik’st not me; ’tis Caesar thou defeat’st.
Put colour in thy cheek.

EROS.
The gods withhold me!
Shall I do that which all the Parthian darts,
Though enemy, lost aim and could not?

ANTONY.
Eros,
Wouldst thou be windowed in great Rome and see
Thy master thus with pleached arms, bending down
His corrigible neck, his face subdued
To penetrative shame, whilst the wheeled seat
Of fortunate Caesar, drawn before him, branded
His baseness that ensued?

EROS.
I would not see’t.

ANTONY.
Come, then, for with a wound I must be cured.
Draw that thy honest sword, which thou hast worn
Most useful for thy country.

EROS.
O, sir, pardon me!

ANTONY.
When I did make thee free, swor’st thou not then
To do this when I bade thee? Do it at once,
Or thy precedent services are all
But accidents unpurposed. Draw, and come.

EROS.
Turn from me then that noble countenance
Wherein the worship of the whole world lies.

ANTONY.
Lo thee!

 [_Turning from him._]

EROS.
My sword is drawn.

ANTONY.
Then let it do at once
The thing why thou hast drawn it.

EROS.
My dear master,
My captain and my emperor, let me say,
Before I strike this bloody stroke, farewell.

ANTONY.
’Tis said, man, and farewell.

EROS.
Farewell, great chief. Shall I strike now?

ANTONY.
Now, Eros.

EROS.
Why, there then! [_kills himself_]
Thus do I escape the sorrow
Of Antony’s death.

 [_Dies._]

ANTONY.
Thrice nobler than myself!
Thou teachest me, O valiant Eros, what
I should and thou couldst not. My queen and Eros
Have by their brave instruction got upon me
A nobleness in record. But I will be
A bridegroom in my death and run into’t
As to a lover’s bed. Come then, and, Eros,
Thy master dies thy scholar. To do thus

 [_Falling on his sword._]

I learned of thee. How? Not dead? Not dead?
The guard, ho! O, dispatch me!

 Enter a company of the Guard, one of them Dercetus.

FIRST GUARD.
What’s the noise?

ANTONY.
I have done my work ill, friends. O, make an end
Of what I have begun.

SECOND GUARD.
The star is fallen.

FIRST GUARD.
And time is at his period.

ALL.
Alas, and woe!

ANTONY.
Let him that loves me strike me dead.

FIRST GUARD.
Not I.

SECOND GUARD.
Nor I.

THIRD GUARD.
Nor anyone.

 [_Exeunt guard._]

DERCETUS.
Thy death and fortunes bid thy followers fly.
This sword but shown to Caesar with this tidings,
Shall enter me with him.

 Enter Diomedes.

DIOMEDES.
Where’s Antony?

DERCETUS.
There, Diomed, there.

DIOMEDES.
Lives he? Wilt thou not answer, man?

 [_Exit Dercetus._]

ANTONY.
Art thou there, Diomed? Draw thy sword and give me
Sufficing strokes for death.

DIOMEDES.
Most absolute lord,
My mistress Cleopatra sent me to thee.

ANTONY.
When did she send thee?

DIOMEDES.
Now, my lord.

ANTONY.
Where is she?

DIOMEDES.
Locked in her monument. She had a prophesying fear
Of what hath come to pass. For when she saw—
Which never shall be found—you did suspect
She had disposed with Caesar, and that your rage
Would not be purged, she sent you word she was dead;
But fearing since how it might work, hath sent
Me to proclaim the truth, and I am come,
I dread, too late.

ANTONY.
Too late, good Diomed. Call my guard, I prithee.

DIOMEDES.
What, ho! The emperor’s guard! The guard, what ho!
Come, your lord calls!

 Enter four or five of the Guard of Antony.

ANTONY.
Bear me, good friends, where Cleopatra bides.
’Tis the last service that I shall command you.

FIRST GUARD.
Woe, woe are we, sir, you may not live to wear
All your true followers out.

ALL.
Most heavy day!

ANTONY.
Nay, good my fellows, do not please sharp fate
To grace it with your sorrows. Bid that welcome
Which comes to punish us, and we punish it,
Seeming to bear it lightly. Take me up.
I have led you oft; carry me now, good friends,
And have my thanks for all.

 [_Exeunt, bearing Antony._]

SCENE XV. Alexandria. A monument.

 Enter Cleopatra and her maids aloft, with Charmian and Iras.

CLEOPATRA.
O Charmian, I will never go from hence.

CHARMIAN.
Be comforted, dear madam.

CLEOPATRA.
No, I will not.
All strange and terrible events are welcome,
But comforts we despise. Our size of sorrow,
Proportioned to our cause, must be as great
As that which makes it.

 Enter, below Diomedes.

How now! Is he dead?

DIOMEDES.
His death’s upon him, but not dead.
Look out o’ th’ other side your monument;
His guard have brought him thither.

 Enter, below Antony borne by the Guard.

CLEOPATRA.
O sun,
Burn the great sphere thou mov’st in! Darkling stand
The varying shore o’ th’ world. O Antony,
Antony, Antony! Help, Charmian! Help, Iras, help!
Help, friends below! Let’s draw him hither.

ANTONY.
Peace!
Not Caesar’s valour hath o’erthrown Antony,
But Antony’s hath triumphed on itself.

CLEOPATRA.
So it should be, that none but Antony
Should conquer Antony, but woe ’tis so!

ANTONY.
I am dying, Egypt, dying. Only
I here importune death awhile until
Of many thousand kisses the poor last
I lay upon thy lips.

CLEOPATRA.
I dare not, dear
Dear my lord, pardon. I dare not,
Lest I be taken. Not th’ imperious show
Of the full-fortuned Caesar ever shall
Be brooched with me; if knife, drugs, serpents, have
Edge, sting, or operation, I am safe.
Your wife Octavia, with her modest eyes
And still conclusion, shall acquire no honour
Demuring upon me. But come, come, Antony—
Help me, my women—we must draw thee up.
Assist, good friends.

ANTONY.
O, quick, or I am gone.

CLEOPATRA.
Here’s sport indeed! How heavy weighs my lord!
Our strength is all gone into heaviness;
That makes the weight. Had I great Juno’s power,
The strong-winged Mercury should fetch thee up
And set thee by Jove’s side. Yet come a little;
Wishers were ever fools. O come, come come,

 [_They heave Antony aloft to Cleopatra._]

And welcome, welcome! Die where thou hast lived;
Quicken with kissing. Had my lips that power,
Thus would I wear them out.

ALL.
A heavy sight!

ANTONY.
I am dying, Egypt, dying.
Give me some wine, and let me speak a little.

CLEOPATRA.
No, let me speak, and let me rail so high
That the false huswife Fortune break her wheel,
Provoked by my offence.

ANTONY.
One word, sweet queen:
Of Caesar seek your honour, with your safety. O!

CLEOPATRA.
They do not go together.

ANTONY.
Gentle, hear me.
None about Caesar trust but Proculeius.

CLEOPATRA.
My resolution and my hands I’ll trust;
None about Caesar.

ANTONY.
The miserable change now at my end
Lament nor sorrow at, but please your thoughts
In feeding them with those my former fortunes
Wherein I lived the greatest prince o’ th’ world,
The noblest; and do now not basely die,
Not cowardly put off my helmet to
My countryman; a Roman by a Roman
Valiantly vanquished. Now my spirit is going;
I can no more.

CLEOPATRA.
Noblest of men, woo’t die?
Hast thou no care of me? Shall I abide
In this dull world, which in thy absence is
No better than a sty? O, see, my women,

 [_ Antony dies._]

The crown o’ th’ earth doth melt.—My lord!
O, withered is the garland of the war,
The soldier’s pole is fallen; young boys and girls
Are level now with men. The odds is gone,
And there is nothing left remarkable
Beneath the visiting moon.

 [_Faints._]

CHARMIAN.
O, quietness, lady!

IRAS.
She is dead too, our sovereign.

CHARMIAN.
Lady!

IRAS.
Madam!

CHARMIAN.
O madam, madam, madam!

IRAS.
Royal Egypt, Empress!

CHARMIAN.
Peace, peace, Iras!

CLEOPATRA.
No more but e’en a woman, and commanded
By such poor passion as the maid that milks
And does the meanest chares. It were for me
To throw my sceptre at the injurious gods,
To tell them that this world did equal theirs
Till they had stolen our jewel. All’s but naught;
Patience is sottish, and impatience does
Become a dog that’s mad. Then is it sin
To rush into the secret house of death
Ere death dare come to us? How do you, women?
What, what! good cheer! Why, how now, Charmian?
My noble girls! Ah, women, women! Look,
Our lamp is spent, it’s out! Good sirs, take heart.
We’ll bury him; and then, what’s brave, what’s noble,
Let’s do it after the high Roman fashion
And make death proud to take us. Come, away.
This case of that huge spirit now is cold.
Ah, women, women! Come, we have no friend
But resolution and the briefest end.

 [_Exeunt, bearing off Antony’s body._]




ACT V


SCENE I. Caesar’s Camp before Alexandria.

 Enter Caesar, Agrippa, Dolabella, Maecenas, Gallus, Proculeius with
 his council of war.

CAESAR.
Go to him, Dolabella, bid him yield.
Being so frustrate, tell him, he mocks
The pauses that he makes.

DOLABELLA.
Caesar, I shall.

 [_Exit._]

 Enter Dercetus with the sword of Antony.

CAESAR.
Wherefore is that? And what art thou that dar’st
Appear thus to us?

DERCETUS.
I am called Dercetus.
Mark Antony I served, who best was worthy
Best to be served. Whilst he stood up and spoke,
He was my master, and I wore my life
To spend upon his haters. If thou please
To take me to thee, as I was to him
I’ll be to Caesar; if thou pleasest not,
I yield thee up my life.

CAESAR.
What is’t thou say’st?

DERCETUS.
I say, O Caesar, Antony is dead.

CAESAR.
The breaking of so great a thing should make
A greater crack. The round world
Should have shook lions into civil streets,
And citizens to their dens. The death of Antony
Is not a single doom; in the name lay
A moiety of the world.

DERCETUS.
He is dead, Caesar,
Not by a public minister of justice,
Nor by a hired knife, but that self hand
Which writ his honour in the acts it did
Hath, with the courage which the heart did lend it,
Splitted the heart. This is his sword.
I robbed his wound of it. Behold it stained
With his most noble blood.

CAESAR.
Look you sad, friends?
The gods rebuke me, but it is tidings
To wash the eyes of kings.

AGRIPPA.
And strange it is
That nature must compel us to lament
Our most persisted deeds.

MAECENAS.
His taints and honours
Waged equal with him.

AGRIPPA.
A rarer spirit never
Did steer humanity, but you gods will give us
Some faults to make us men. Caesar is touched.

MAECENAS.
When such a spacious mirror’s set before him,
He needs must see himself.

CAESAR.
O Antony,
I have followed thee to this, but we do lance
Diseases in our bodies. I must perforce
Have shown to thee such a declining day
Or look on thine. We could not stall together
In the whole world. But yet let me lament
With tears as sovereign as the blood of hearts,
That thou, my brother, my competitor
In top of all design, my mate in empire,
Friend and companion in the front of war,
The arm of mine own body, and the heart
Where mine his thoughts did kindle, that our stars,
Unreconciliable, should divide
Our equalness to this. Hear me, good friends—

 Enter an Egyptian.

But I will tell you at some meeter season.
The business of this man looks out of him;
We’ll hear him what he says. Whence are you?

EGYPTIAN.
A poor Egyptian yet. The queen, my mistress,
Confined in all she has, her monument,
Of thy intents desires instruction,
That she preparedly may frame herself
To the way she’s forced to.

CAESAR.
Bid her have good heart.
She soon shall know of us, by some of ours,
How honourable and how kindly we
Determine for her. For Caesar cannot lean
To be ungentle.

EGYPTIAN.
So the gods preserve thee!

 [_Exit._]

CAESAR.
Come hither, Proculeius. Go and say
We purpose her no shame. Give her what comforts
The quality of her passion shall require,
Lest, in her greatness, by some mortal stroke
She do defeat us, for her life in Rome
Would be eternal in our triumph. Go,
And with your speediest bring us what she says
And how you find of her.

PROCULEIUS.
Caesar, I shall.

 [_Exit Proculeius._]

CAESAR.
Gallus, go you along.

 [_Exit Gallus._]

Where’s Dolabella, to second Proculeius?

ALL.
Dolabella!

CAESAR.
Let him alone, for I remember now
How he’s employed. He shall in time be ready.
Go with me to my tent, where you shall see
How hardly I was drawn into this war,
How calm and gentle I proceeded still
In all my writings. Go with me and see
What I can show in this.

 [_Exeunt._]

SCENE II. Alexandria. A Room in the Monument.

 Enter Cleopatra, Charmian and Iras.

CLEOPATRA.
My desolation does begin to make
A better life. ’Tis paltry to be Caesar;
Not being Fortune, he’s but Fortune’s knave,
A minister of her will. And it is great
To do that thing that ends all other deeds,
Which shackles accidents and bolts up change,
Which sleeps and never palates more the dung,
The beggar’s nurse and Caesar’s.

 Enter Proculeius.

PROCULEIUS.
Caesar sends greetings to the queen of Egypt,
And bids thee study on what fair demands
Thou mean’st to have him grant thee.

CLEOPATRA.
What’s thy name?

PROCULEIUS.
My name is Proculeius.

CLEOPATRA.
Antony
Did tell me of you, bade me trust you, but
I do not greatly care to be deceived
That have no use for trusting. If your master
Would have a queen his beggar, you must tell him
That majesty, to keep decorum, must
No less beg than a kingdom. If he please
To give me conquered Egypt for my son,
He gives me so much of mine own as I
Will kneel to him with thanks.

PROCULEIUS.
Be of good cheer.
You are fallen into a princely hand; fear nothing.
Make your full reference freely to my lord,
Who is so full of grace that it flows over
On all that need. Let me report to him
Your sweet dependency, and you shall find
A conqueror that will pray in aid for kindness
Where he for grace is kneeled to.

CLEOPATRA.
Pray you tell him
I am his fortune’s vassal and I send him
The greatness he has got. I hourly learn
A doctrine of obedience, and would gladly
Look him i’ th’ face.

PROCULEIUS.
This I’ll report, dear lady.
Have comfort, for I know your plight is pitied
Of him that caused it.

 Enter Gallus and Roman Soldiers.

You see how easily she may be surprised.
Guard her till Caesar come.

IRAS.
Royal queen!

CHARMIAN.
O Cleopatra, thou art taken, queen!

CLEOPATRA.
Quick, quick, good hands.

 [_Drawing a dagger._]

PROCULEIUS.
Hold, worthy lady, hold!

 [_Seizes and disarms her._]

Do not yourself such wrong, who are in this
Relieved, but not betrayed.

CLEOPATRA.
What, of death too,
That rids our dogs of languish?

PROCULEIUS.
Cleopatra,
Do not abuse my master’s bounty by
Th’ undoing of yourself. Let the world see
His nobleness well acted, which your death
Will never let come forth.

CLEOPATRA.
Where art thou, Death?
Come hither, come! Come, come, and take a queen
Worth many babes and beggars!

PROCULEIUS.
O, temperance, lady!

CLEOPATRA.
Sir, I will eat no meat; I’ll not drink, sir;
If idle talk will once be necessary,
I’ll not sleep neither. This mortal house I’ll ruin,
Do Caesar what he can. Know, sir, that I
Will not wait pinioned at your master’s court,
Nor once be chastised with the sober eye
Of dull Octavia. Shall they hoist me up
And show me to the shouting varletry
Of censuring Rome? Rather a ditch in Egypt
Be gentle grave unto me! Rather on Nilus’ mud
Lay me stark-naked, and let the water-flies
Blow me into abhorring! Rather make
My country’s high pyramides my gibbet
And hang me up in chains!

PROCULEIUS.
You do extend
These thoughts of horror further than you shall
Find cause in Caesar.

 Enter Dolabella.

DOLABELLA.
Proculeius,
What thou hast done thy master Caesar knows,
And he hath sent for thee. For the queen,
I’ll take her to my guard.

PROCULEIUS.
So, Dolabella,
It shall content me best. Be gentle to her.
[_To Cleopatra._] To Caesar I will speak what you shall please,
If you’ll employ me to him.

CLEOPATRA.
Say I would die.

 [_Exeunt Proculeius and Soldiers._]

DOLABELLA.
Most noble empress, you have heard of me?

CLEOPATRA.
I cannot tell.

DOLABELLA.
Assuredly you know me.

CLEOPATRA.
No matter, sir, what I have heard or known.
You laugh when boys or women tell their dreams;
Is’t not your trick?

DOLABELLA.
I understand not, madam.

CLEOPATRA.
I dreamt there was an Emperor Antony.
O, such another sleep, that I might see
But such another man!

DOLABELLA.
If it might please you—

CLEOPATRA.
His face was as the heavens, and therein stuck
A sun and moon, which kept their course, and lighted
The little O, the earth.

DOLABELLA.
Most sovereign creature—

CLEOPATRA.
His legs bestrid the ocean; his reared arm
Crested the world; his voice was propertied
As all the tuned spheres, and that to friends;
But when he meant to quail and shake the orb,
He was as rattling thunder. For his bounty,
There was no winter in’t; an autumn ’twas
That grew the more by reaping. His delights
Were dolphin-like; they showed his back above
The element they lived in. In his livery
Walked crowns and crownets; realms and islands were
As plates dropped from his pocket.

DOLABELLA.
Cleopatra—

CLEOPATRA.
Think you there was or might be such a man
As this I dreamt of?

DOLABELLA.
Gentle madam, no.

CLEOPATRA.
You lie up to the hearing of the gods!
But if there be nor ever were one such,
It’s past the size of dreaming. Nature wants stuff
To vie strange forms with fancy; yet t’ imagine
An Antony were nature’s piece ’gainst fancy,
Condemning shadows quite.

DOLABELLA.
Hear me, good madam.
Your loss is, as yourself, great; and you bear it
As answering to the weight. Would I might never
O’ertake pursued success, but I do feel,
By the rebound of yours, a grief that smites
My very heart at root.

CLEOPATRA.
I thank you, sir.
Know you what Caesar means to do with me?

DOLABELLA.
I am loath to tell you what I would you knew.

CLEOPATRA.
Nay, pray you, sir.

DOLABELLA.
Though he be honourable—

CLEOPATRA.
He’ll lead me, then, in triumph.

DOLABELLA.
Madam, he will. I know it.

 Flourish. Enter Caesar, Proculeius, Gallus, Maecenas and others of his
 train.

ALL.
Make way there! Caesar!

CAESAR.
Which is the Queen of Egypt?

DOLABELLA.
It is the Emperor, madam.

 [_Cleopatra kneels._]

CAESAR.
Arise, you shall not kneel.
I pray you, rise. Rise, Egypt.

CLEOPATRA.
Sir, the gods
Will have it thus. My master and my lord
I must obey.

CAESAR.
Take to you no hard thoughts.
The record of what injuries you did us,
Though written in our flesh, we shall remember
As things but done by chance.

CLEOPATRA.
Sole sir o’ th’ world,
I cannot project mine own cause so well
To make it clear, but do confess I have
Been laden with like frailties which before
Have often shamed our sex.

CAESAR.
Cleopatra, know
We will extenuate rather than enforce.
If you apply yourself to our intents,
Which towards you are most gentle, you shall find
A benefit in this change; but if you seek
To lay on me a cruelty by taking
Antony’s course, you shall bereave yourself
Of my good purposes, and put your children
To that destruction which I’ll guard them from
If thereon you rely. I’ll take my leave.

CLEOPATRA.
And may, through all the world. ’Tis yours, and we,
Your scutcheons and your signs of conquest, shall
Hang in what place you please. Here, my good lord.

CAESAR.
You shall advise me in all for Cleopatra.

CLEOPATRA.
This is the brief of money, plate, and jewels
I am possessed of. ’Tis exactly valued,
Not petty things admitted. Where’s Seleucus?

 Enter Seleucus.

SELEUCUS.
Here, madam.

CLEOPATRA.
This is my treasurer. Let him speak, my lord,
Upon his peril, that I have reserved
To myself nothing. Speak the truth, Seleucus.

SELEUCUS.
Madam, I had rather seal my lips
Than to my peril speak that which is not.

CLEOPATRA.
What have I kept back?

SELEUCUS.
Enough to purchase what you have made known.

CAESAR.
Nay, blush not, Cleopatra. I approve
Your wisdom in the deed.

CLEOPATRA.
See, Caesar! O, behold,
How pomp is followed! Mine will now be yours
And should we shift estates, yours would be mine.
The ingratitude of this Seleucus does
Even make me wild. O slave, of no more trust
Than love that’s hired! What, goest thou back? Thou shalt
Go back, I warrant thee! But I’ll catch thine eyes
Though they had wings. Slave, soulless villain, dog!
O rarely base!

CAESAR.
Good queen, let us entreat you.

CLEOPATRA.
O Caesar, what a wounding shame is this,
That thou vouchsafing here to visit me,
Doing the honour of thy lordliness
To one so meek, that mine own servant should
Parcel the sum of my disgraces by
Addition of his envy! Say, good Caesar,
That I some lady trifles have reserved,
Immoment toys, things of such dignity
As we greet modern friends withal; and say
Some nobler token I have kept apart
For Livia and Octavia, to induce
Their mediation, must I be unfolded
With one that I have bred? The gods! It smites me
Beneath the fall I have.
[_To Seleucus_.] Prithee go hence,
Or I shall show the cinders of my spirits
Through th’ ashes of my chance. Wert thou a man,
Thou wouldst have mercy on me.

CAESAR.
Forbear, Seleucus.

 [_Exit Seleucus._]

CLEOPATRA.
Be it known that we, the greatest, are misthought
For things that others do; and when we fall,
We answer others’ merits in our name,
Are therefore to be pitied.

CAESAR.
Cleopatra,
Not what you have reserved nor what acknowledged
Put we i’ th’ roll of conquest. Still be’t yours;
Bestow it at your pleasure, and believe
Caesar’s no merchant to make prize with you
Of things that merchants sold. Therefore be cheered;
Make not your thoughts your prisons. No, dear queen;
For we intend so to dispose you as
Yourself shall give us counsel. Feed and sleep.
Our care and pity is so much upon you
That we remain your friend; and so, adieu.

CLEOPATRA.
My master and my lord!

CAESAR.
Not so. Adieu.

 [_Flourish. Exeunt Caesar and his train._]

CLEOPATRA.
He words me, girls, he words me, that I should not
Be noble to myself. But hark thee, Charmian!

 [_Whispers to Charmian._]

IRAS.
Finish, good lady. The bright day is done,
And we are for the dark.

CLEOPATRA.
Hie thee again.
I have spoke already, and it is provided.
Go put it to the haste.

CHARMIAN.
Madam, I will.

 Enter Dolabella.

DOLABELLA.
Where’s the Queen?

CHARMIAN.
Behold, sir.

 [_Exit._]

CLEOPATRA.
Dolabella!

DOLABELLA.
Madam, as thereto sworn by your command,
Which my love makes religion to obey,
I tell you this: Caesar through Syria
Intends his journey, and within three days
You with your children will he send before.
Make your best use of this. I have performed
Your pleasure and my promise.

CLEOPATRA.
Dolabella,
I shall remain your debtor.

DOLABELLA.
I your servant.
Adieu, good queen. I must attend on Caesar.

CLEOPATRA.
Farewell, and thanks.

 [_Exit Dolabella._]

Now, Iras, what think’st thou?
Thou an Egyptian puppet shall be shown
In Rome as well as I. Mechanic slaves
With greasy aprons, rules, and hammers shall
Uplift us to the view. In their thick breaths,
Rank of gross diet, shall we be enclouded,
And forced to drink their vapour.

IRAS.
The gods forbid!

CLEOPATRA.
Nay, ’tis most certain, Iras. Saucy lictors
Will catch at us like strumpets, and scald rhymers
Ballad us out o’ tune. The quick comedians
Extemporally will stage us and present
Our Alexandrian revels; Antony
Shall be brought drunken forth, and I shall see
Some squeaking Cleopatra boy my greatness
I’ th’ posture of a whore.

IRAS.
O the good gods!

CLEOPATRA.
Nay, that’s certain.

IRAS.
I’ll never see’t, for I am sure mine nails
Are stronger than mine eyes.

CLEOPATRA.
Why, that’s the way
To fool their preparation and to conquer
Their most absurd intents.

 Enter Charmian.

Now, Charmian!
Show me, my women, like a queen. Go fetch
My best attires. I am again for Cydnus
To meet Mark Antony. Sirrah, Iras, go.
Now, noble Charmian, we’ll dispatch indeed,
And when thou hast done this chare, I’ll give thee leave
To play till doomsday. Bring our crown and all.

 [_Exit Iras. A noise within._]

Wherefore’s this noise?

 Enter a Guardsman.

GUARDSMAN.
Here is a rural fellow
That will not be denied your highness’ presence.
He brings you figs.

CLEOPATRA.
Let him come in.

 [_Exit Guardsman._]

What poor an instrument
May do a noble deed! He brings me liberty.
My resolution’s placed, and I have nothing
Of woman in me. Now from head to foot
I am marble-constant. Now the fleeting moon
No planet is of mine.

 Enter Guardsman and Clown with a basket.

GUARDSMAN.
This is the man.

CLEOPATRA.
Avoid, and leave him.

 [_Exit Guardsman._]

Hast thou the pretty worm of Nilus there
That kills and pains not?

CLOWN.
Truly, I have him, but I would not be the party that should desire you
to touch him, for his biting is immortal. Those that do die of it do
seldom or never recover.

CLEOPATRA.
Remember’st thou any that have died on’t?

CLOWN.
Very many, men and women too. I heard of one of them no longer than
yesterday—a very honest woman, but something given to lie; as a woman
should not do but in the way of honesty—how she died of the biting of
it, what pain she felt. Truly she makes a very good report o’ th’ worm;
but he that will believe all that they say shall never be saved by half
that they do. But this is most falliable, the worm’s an odd worm.

CLEOPATRA.
Get thee hence. Farewell.

CLOWN.
I wish you all joy of the worm.

 [_Sets down the basket._]

CLEOPATRA.
Farewell.

CLOWN.
You must think this, look you, that the worm will do his kind.

CLEOPATRA.
Ay, ay, farewell.

CLOWN.
Look you, the worm is not to be trusted but in the keeping of wise
people; for indeed there is no goodness in the worm.

CLEOPATRA.
Take thou no care; it shall be heeded.

CLOWN.
Very good. Give it nothing, I pray you, for it is not worth the
feeding.

CLEOPATRA.
Will it eat me?

CLOWN.
You must not think I am so simple but I know the devil himself will not
eat a woman. I know that a woman is a dish for the gods if the devil
dress her not. But truly, these same whoreson devils do the gods great
harm in their women, for in every ten that they make, the devils mar
five.

CLEOPATRA.
Well, get thee gone. Farewell.

CLOWN.
Yes, forsooth. I wish you joy o’ th’ worm.

 [_Exit._]

 Enter Iras with a robe, crown, &c.

CLEOPATRA.
Give me my robe. Put on my crown. I have
Immortal longings in me. Now no more
The juice of Egypt’s grape shall moist this lip.
Yare, yare, good Iras; quick. Methinks I hear
Antony call. I see him rouse himself
To praise my noble act. I hear him mock
The luck of Caesar, which the gods give men
To excuse their after wrath. Husband, I come!
Now to that name my courage prove my title!
I am fire and air; my other elements
I give to baser life.—So, have you done?
Come then, and take the last warmth of my lips.
Farewell, kind Charmian. Iras, long farewell.

 [_Kisses them. Iras falls and dies._]

Have I the aspic in my lips? Dost fall?
If thou and nature can so gently part,
The stroke of death is as a lover’s pinch,
Which hurts and is desired. Dost thou lie still?
If thus thou vanishest, thou tell’st the world
It is not worth leave-taking.

CHARMIAN.
Dissolve, thick cloud, and rain, that I may say
The gods themselves do weep!

CLEOPATRA.
This proves me base.
If she first meet the curled Antony,
He’ll make demand of her, and spend that kiss
Which is my heaven to have.—Come, thou mortal wretch,

 [_To an asp, which she applies to her breast._]

With thy sharp teeth this knot intrinsicate
Of life at once untie. Poor venomous fool,
Be angry and dispatch. O couldst thou speak,
That I might hear thee call great Caesar ass
Unpolicied!

CHARMIAN.
O eastern star!

CLEOPATRA.
Peace, peace!
Dost thou not see my baby at my breast
That sucks the nurse asleep?

CHARMIAN.
O, break! O, break!

CLEOPATRA.
As sweet as balm, as soft as air, as gentle—
O Antony!—Nay, I will take thee too.

 [_Applying another asp to her arm._]

What should I stay—

 [_Dies._]

CHARMIAN.
In this vile world? So, fare thee well.
Now boast thee, Death, in thy possession lies
A lass unparalleled. Downy windows, close,
And golden Phœbus never be beheld
Of eyes again so royal! Your crown’s awry;
I’ll mend it and then play.

 Enter the Guard rustling in.

FIRST GUARD.
Where’s the queen?

CHARMIAN.
Speak softly. Wake her not.

FIRST GUARD.
Caesar hath sent—

CHARMIAN.
Too slow a messenger.

 [_Applies an asp._]

O, come apace, dispatch! I partly feel thee.

FIRST GUARD.
Approach, ho! All’s not well. Caesar’s beguiled.

SECOND GUARD.
There’s Dolabella sent from Caesar. Call him.

FIRST GUARD.
What work is here, Charmian? Is this well done?

CHARMIAN.
It is well done, and fitting for a princess
Descended of so many royal kings.
Ah, soldier!

 [_Charmian dies._]

 Enter Dolabella.

DOLABELLA.
How goes it here?

SECOND GUARD.
All dead.

DOLABELLA.
Caesar, thy thoughts
Touch their effects in this. Thyself art coming
To see performed the dreaded act which thou
So sought’st to hinder.

 Enter Caesar and all his train, marching.

ALL.
A way there, a way for Caesar!

DOLABELLA.
O sir, you are too sure an augurer:
That you did fear is done.

CAESAR.
Bravest at the last,
She levelled at our purposes and, being royal,
Took her own way. The manner of their deaths?
I do not see them bleed.

DOLABELLA.
Who was last with them?

FIRST GUARD.
A simple countryman that brought her figs.
This was his basket.

CAESAR.
Poisoned then.

FIRST GUARD.
O Caesar,
This Charmian lived but now; she stood and spake.
I found her trimming up the diadem
On her dead mistress; tremblingly she stood,
And on the sudden dropped.

CAESAR.
O noble weakness!
If they had swallowed poison ’twould appear
By external swelling; but she looks like sleep,
As she would catch another Antony
In her strong toil of grace.

DOLABELLA.
Here on her breast
There is a vent of blood, and something blown.
The like is on her arm.

FIRST GUARD.
This is an aspic’s trail, and these fig leaves
Have slime upon them, such as th’ aspic leaves
Upon the caves of Nile.

CAESAR.
Most probable
That so she died, for her physician tells me
She hath pursued conclusions infinite
Of easy ways to die. Take up her bed,
And bear her women from the monument.
She shall be buried by her Antony.
No grave upon the earth shall clip in it
A pair so famous. High events as these
Strike those that make them; and their story is
No less in pity than his glory which
Brought them to be lamented. Our army shall
In solemn show attend this funeral,
And then to Rome. Come, Dolabella, see
High order in this great solemnity.

 [_Exeunt omnes._]




AS YOU LIKE IT




Contents

 ACT I
 Scene I. An Orchard near Oliver’s house
 Scene II. A Lawn before the Duke’s Palace
 Scene III. A Room in the Palace

 ACT II
 Scene I. The Forest of Arden
 Scene II. A Room in the Palace
 Scene III. Before Oliver’s House
 Scene IV. The Forest of Arden
 Scene V. Another part of the Forest
 Scene VI. Another part of the Forest
 Scene VII. Another part of the Forest

 ACT III
 Scene I. A Room in the Palace
 Scene II. The Forest of Arden
 Scene III. Another part of the Forest
 Scene IV. Another part of the Forest. Before a Cottage
 Scene V. Another part of the Forest

 ACT IV
 Scene I. The Forest of Arden
 Scene II. Another part of the Forest
 Scene III. Another part of the Forest

 ACT V
 Scene I. The Forest of Arden
 Scene II. Another part of the Forest
 Scene III. Another part of the Forest
 Scene IV. Another part of the Forest
 Epilogue


Dramatis Personæ

ORLANDO, youngest son of Sir Rowland de Boys
OLIVER, eldest son of Sir Rowland de Boys
JAQUES DE BOYS, second son of Sir Rowland de Boys
ADAM, Servant to Oliver
DENNIS, Servant to Oliver

ROSALIND, Daughter of Duke Senior
CELIA, Daughter of Duke Frederick
TOUCHSTONE, a Clown

DUKE SENIOR (Ferdinand), living in exile

JAQUES, Lord attending on the Duke Senior
AMIENS, Lord attending on the Duke Senior

DUKE FREDERICK, Brother to the Duke, and Usurper of his Dominions
CHARLES, his Wrestler
LE BEAU, a Courtier attending upon Frederick

CORIN, Shepherd
SILVIUS, Shepherd
PHOEBE, a Shepherdess
AUDREY, a Country Wench
WILLIAM, a Country Fellow, in love with Audrey
SIR OLIVER MARTEXT, a Vicar

A person representing HYMEN

Lords belonging to the two Dukes; Pages, Foresters, and other
Attendants.

The scene lies first near Oliver’s house; afterwards partly in the
Usurper’s court and partly in the Forest of Arden.




ACT I

SCENE I. An Orchard near Oliver’s house


Enter Orlando and Adam.

ORLANDO.
As I remember, Adam, it was upon this fashion bequeathed me by will but
poor a thousand crowns, and, as thou sayst, charged my brother, on his
blessing, to breed me well; and there begins my sadness. My brother
Jaques he keeps at school, and report speaks goldenly of his profit.
For my part, he keeps me rustically at home, or, to speak more
properly, stays me here at home unkept; for call you that keeping, for
a gentleman of my birth, that differs not from the stalling of an ox?
His horses are bred better, for, besides that they are fair with their
feeding, they are taught their manage and to that end riders dearly
hired; but I, his brother, gain nothing under him but growth, for the
which his animals on his dunghills are as much bound to him as I.
Besides this nothing that he so plentifully gives me, the something
that nature gave me his countenance seems to take from me. He lets me
feed with his hinds, bars me the place of a brother, and as much as in
him lies, mines my gentility with my education. This is it, Adam, that
grieves me, and the spirit of my father, which I think is within me,
begins to mutiny against this servitude. I will no longer endure it,
though yet I know no wise remedy how to avoid it.

Enter Oliver.

ADAM.
Yonder comes my master, your brother.

ORLANDO.
Go apart, Adam, and thou shalt hear how he will shake me up.

[_Adam retires._]

OLIVER.
Now, sir, what make you here?

ORLANDO.
Nothing. I am not taught to make anything.

OLIVER.
What mar you then, sir?

ORLANDO.
Marry, sir, I am helping you to mar that which God made, a poor
unworthy brother of yours, with idleness.

OLIVER.
Marry, sir, be better employed, and be naught awhile.

ORLANDO.
Shall I keep your hogs, and eat husks with them? What prodigal portion
have I spent that I should come to such penury?

OLIVER.
Know you where you are, sir?

ORLANDO.
O, sir, very well: here in your orchard.

OLIVER.
Know you before whom, sir?

ORLANDO.
Ay, better than him I am before knows me. I know you are my eldest
brother, and in the gentle condition of blood you should so know me.
The courtesy of nations allows you my better in that you are the
first-born, but the same tradition takes not away my blood, were there
twenty brothers betwixt us. I have as much of my father in me as you,
albeit I confess your coming before me is nearer to his reverence.

OLIVER.
What, boy!

ORLANDO.
Come, come, elder brother, you are too young in this.

OLIVER.
Wilt thou lay hands on me, villain?

ORLANDO.
I am no villain. I am the youngest son of Sir Rowland de Boys; he was
my father, and he is thrice a villain that says such a father begot
villains. Wert thou not my brother, I would not take this hand from thy
throat till this other had pulled out thy tongue for saying so. Thou
has railed on thyself.

ADAM.
[_Coming forward_.] Sweet masters, be patient. For your father’s
remembrance, be at accord.

OLIVER.
Let me go, I say.

ORLANDO.
I will not till I please. You shall hear me. My father charged you in
his will to give me good education. You have trained me like a peasant,
obscuring and hiding from me all gentleman-like qualities. The spirit
of my father grows strong in me, and I will no longer endure it.
Therefore allow me such exercises as may become a gentleman, or give me
the poor allottery my father left me by testament; with that I will go
buy my fortunes.

OLIVER.
And what wilt thou do? Beg when that is spent? Well, sir, get you in. I
will not long be troubled with you. You shall have some part of your
will. I pray you leave me.

ORLANDO.
I no further offend you than becomes me for my good.

OLIVER.
Get you with him, you old dog.

ADAM.
Is “old dog” my reward? Most true, I have lost my teeth in your
service. God be with my old master. He would not have spoke such a
word.

[_Exeunt Orlando and Adam._]

OLIVER.
Is it even so? Begin you to grow upon me? I will physic your rankness,
and yet give no thousand crowns neither. Holla, Dennis!

Enter Dennis.

DENNIS
Calls your worship?

OLIVER.
Was not Charles, the Duke’s wrestler, here to speak with me?

DENNIS
So please you, he is here at the door and importunes access to you.

OLIVER.
Call him in.

[_Exit Dennis._]

’Twill be a good way, and tomorrow the wrestling is.

Enter Charles.

CHARLES.
Good morrow to your worship.

OLIVER.
Good Monsieur Charles. What’s the new news at the new court?

CHARLES.
There’s no news at the court, sir, but the old news. That is, the old
Duke is banished by his younger brother the new Duke, and three or four
loving lords have put themselves into voluntary exile with him, whose
lands and revenues enrich the new Duke; therefore he gives them good
leave to wander.

OLIVER.
Can you tell if Rosalind, the Duke’s daughter, be banished with her
father?

CHARLES.
O, no; for the Duke’s daughter, her cousin, so loves her, being ever
from their cradles bred together, that she would have followed her
exile or have died to stay behind her. She is at the court and no less
beloved of her uncle than his own daughter, and never two ladies loved
as they do.

OLIVER.
Where will the old Duke live?

CHARLES.
They say he is already in the Forest of Arden, and a many merry men
with him; and there they live like the old Robin Hood of England. They
say many young gentlemen flock to him every day and fleet the time
carelessly, as they did in the golden world.

OLIVER.
What, you wrestle tomorrow before the new Duke?

CHARLES.
Marry, do I, sir, and I came to acquaint you with a matter. I am given,
sir, secretly to understand that your younger brother Orlando hath a
disposition to come in disguised against me to try a fall. Tomorrow,
sir, I wrestle for my credit, and he that escapes me without some
broken limb shall acquit him well. Your brother is but young and
tender, and for your love I would be loath to foil him, as I must for
my own honour if he come in. Therefore, out of my love to you, I came
hither to acquaint you withal, that either you might stay him from his
intendment, or brook such disgrace well as he shall run into, in that
it is a thing of his own search and altogether against my will.

OLIVER.
Charles, I thank thee for thy love to me, which thou shalt find I will
most kindly requite. I had myself notice of my brother’s purpose
herein, and have by underhand means laboured to dissuade him from it;
but he is resolute. I’ll tell thee, Charles, it is the stubbornest
young fellow of France, full of ambition, an envious emulator of every
man’s good parts, a secret and villainous contriver against me his
natural brother. Therefore use thy discretion. I had as lief thou didst
break his neck as his finger. And thou wert best look to’t; for if thou
dost him any slight disgrace, or if he do not mightily grace himself on
thee, he will practise against thee by poison, entrap thee by some
treacherous device, and never leave thee till he hath ta’en thy life by
some indirect means or other. For I assure thee (and almost with tears
I speak it) there is not one so young and so villainous this day
living. I speak but brotherly of him, but should I anatomize him to
thee as he is, I must blush and weep, and thou must look pale and
wonder.

CHARLES.
I am heartily glad I came hither to you. If he come tomorrow I’ll give
him his payment. If ever he go alone again I’ll never wrestle for prize
more. And so, God keep your worship.

[_Exit._]

OLIVER.
Farewell, good Charles. Now will I stir this gamester. I hope I shall
see an end of him; for my soul—yet I know not why—hates nothing more
than he. Yet he’s gentle, never schooled and yet learned, full of noble
device, of all sorts enchantingly beloved, and indeed so much in the
heart of the world, and especially of my own people, who best know him,
that I am altogether misprized. But it shall not be so long; this
wrestler shall clear all. Nothing remains but that I kindle the boy
thither, which now I’ll go about.

[_Exit._]

SCENE II. A Lawn before the Duke’s Palace

Enter Rosalind and Celia.

CELIA.
I pray thee, Rosalind, sweet my coz, be merry.

ROSALIND.
Dear Celia, I show more mirth than I am mistress of, and would you yet
I were merrier? Unless you could teach me to forget a banished father,
you must not learn me how to remember any extraordinary pleasure.

CELIA.
Herein I see thou lov’st me not with the full weight that I love thee.
If my uncle, thy banished father, had banished thy uncle, the Duke my
father, so thou hadst been still with me, I could have taught my love
to take thy father for mine. So wouldst thou, if the truth of thy love
to me were so righteously tempered as mine is to thee.

ROSALIND.
Well, I will forget the condition of my estate to rejoice in yours.

CELIA.
You know my father hath no child but I, nor none is like to have; and
truly, when he dies thou shalt be his heir, for what he hath taken away
from thy father perforce, I will render thee again in affection. By
mine honour I will! And when I break that oath, let me turn monster.
Therefore, my sweet Rose, my dear Rose, be merry.

ROSALIND.
From henceforth I will, coz, and devise sports. Let me see—what think
you of falling in love?

CELIA.
Marry, I prithee do, to make sport withal; but love no man in good
earnest, nor no further in sport neither than with safety of a pure
blush thou mayst in honour come off again.

ROSALIND.
What shall be our sport, then?

CELIA.
Let us sit and mock the good housewife Fortune from her wheel, that her
gifts may henceforth be bestowed equally.

ROSALIND.
I would we could do so, for her benefits are mightily misplaced, and
the bountiful blind woman doth most mistake in her gifts to women.

CELIA.
’Tis true, for those that she makes fair she scarce makes honest, and
those that she makes honest she makes very ill-favouredly.

ROSALIND.
Nay, now thou goest from Fortune’s office to Nature’s. Fortune reigns
in gifts of the world, not in the lineaments of Nature.

Enter Touchstone.

CELIA.
No? When Nature hath made a fair creature, may she not by Fortune fall
into the fire? Though Nature hath given us wit to flout at Fortune,
hath not Fortune sent in this fool to cut off the argument?

ROSALIND.
Indeed, there is Fortune too hard for Nature, when Fortune makes
Nature’s natural the cutter-off of Nature’s wit.

CELIA.
Peradventure this is not Fortune’s work neither, but Nature’s, who
perceiveth our natural wits too dull to reason of such goddesses, and
hath sent this natural for our whetstone; for always the dullness of
the fool is the whetstone of the wits.—How now, wit, whither wander
you?

TOUCHSTONE.
Mistress, you must come away to your father.

CELIA.
Were you made the messenger?

TOUCHSTONE.
No, by mine honour, but I was bid to come for you.

ROSALIND.
Where learned you that oath, fool?

TOUCHSTONE.
Of a certain knight that swore by his honour they were good pancakes,
and swore by his honour the mustard was naught. Now, I’ll stand to it,
the pancakes were naught and the mustard was good, and yet was not the
knight forsworn.

CELIA.
How prove you that in the great heap of your knowledge?

ROSALIND.
Ay, marry, now unmuzzle your wisdom.

TOUCHSTONE.
Stand you both forth now: stroke your chins, and swear by your beards
that I am a knave.

CELIA.
By our beards, if we had them, thou art.

TOUCHSTONE.
By my knavery, if I had it, then I were. But if you swear by that that
is not, you are not forsworn. No more was this knight swearing by his
honour, for he never had any; or if he had, he had sworn it away before
ever he saw those pancackes or that mustard.

CELIA.
Prithee, who is’t that thou mean’st?

TOUCHSTONE.
One that old Frederick, your father, loves.

CELIA.
My father’s love is enough to honour him. Enough! Speak no more of him.
You’ll be whipped for taxation one of these days.

TOUCHSTONE.
The more pity that fools may not speak wisely what wise men do
foolishly.

CELIA.
By my troth, thou sayest true. For since the little wit that fools have
was silenced, the little foolery that wise men have makes a great show.
Here comes Monsieur Le Beau.

Enter Le Beau.

ROSALIND.
With his mouth full of news.

CELIA.
Which he will put on us as pigeons feed their young.

ROSALIND.
Then shall we be news-crammed.

CELIA.
All the better; we shall be the more marketable.
_Bonjour_, Monsieur Le Beau. What’s the news?

LE BEAU.
Fair princess, you have lost much good sport.

CELIA.
Sport! Of what colour?

LE BEAU.
What colour, madam? How shall I answer you?

ROSALIND.
As wit and fortune will.

TOUCHSTONE.
Or as the destinies decrees.

CELIA.
Well said. That was laid on with a trowel.

TOUCHSTONE.
Nay, if I keep not my rank—

ROSALIND.
Thou losest thy old smell.

LE BEAU.
You amaze me, ladies. I would have told you of good wrestling, which
you have lost the sight of.

ROSALIND.
Yet tell us the manner of the wrestling.

LE BEAU.
I will tell you the beginning and, if it please your ladyships, you may
see the end, for the best is yet to do; and here, where you are, they
are coming to perform it.

CELIA.
Well, the beginning that is dead and buried.

LE BEAU.
There comes an old man and his three sons—

CELIA.
I could match this beginning with an old tale.

LE BEAU.
Three proper young men of excellent growth and presence.

ROSALIND.
With bills on their necks: “Be it known unto all men by these
presents.”

LE BEAU.
The eldest of the three wrestled with Charles, the Duke’s wrestler,
which Charles in a moment threw him and broke three of his ribs, that
there is little hope of life in him. So he served the second, and so
the third. Yonder they lie, the poor old man their father making such
pitiful dole over them that all the beholders take his part with
weeping.

ROSALIND.
Alas!

TOUCHSTONE.
But what is the sport, monsieur, that the ladies have lost?

LE BEAU.
Why, this that I speak of.

TOUCHSTONE.
Thus men may grow wiser every day. It is the first time that ever I
heard breaking of ribs was sport for ladies.

CELIA.
Or I, I promise thee.

ROSALIND.
But is there any else longs to see this broken music in his sides? Is
there yet another dotes upon rib-breaking? Shall we see this wrestling,
cousin?

LE BEAU.
You must if you stay here, for here is the place appointed for the
wrestling, and they are ready to perform it.

CELIA.
Yonder, sure, they are coming. Let us now stay and see it.

Flourish. Enter Duke Frederick, Lords, Orlando, Charles and Attendants.

DUKE FREDERICK.
Come on. Since the youth will not be entreated, his own peril on his
forwardness.

ROSALIND.
Is yonder the man?

LE BEAU.
Even he, madam.

CELIA.
Alas, he is too young. Yet he looks successfully.

DUKE FREDERICK.
How now, daughter and cousin? Are you crept hither to see the
wrestling?

ROSALIND.
Ay, my liege, so please you give us leave.

DUKE FREDERICK.
You will take little delight in it, I can tell you, there is such odds
in the man. In pity of the challenger’s youth I would fain dissuade
him, but he will not be entreated. Speak to him, ladies; see if you can
move him.

CELIA.
Call him hither, good Monsieur Le Beau.

DUKE FREDERICK.
Do so; I’ll not be by.

[_Duke Frederick steps aside._]

LE BEAU.
Monsieur the challenger, the Princess calls for you.

ORLANDO.
I attend them with all respect and duty.

ROSALIND.
Young man, have you challenged Charles the wrestler?

ORLANDO.
No, fair princess. He is the general challenger. I come but in as
others do, to try with him the strength of my youth.

CELIA.
Young gentleman, your spirits are too bold for your years. You have
seen cruel proof of this man’s strength. If you saw yourself with your
eyes or knew yourself with your judgement, the fear of your adventure
would counsel you to a more equal enterprise. We pray you for your own
sake to embrace your own safety and give over this attempt.

ROSALIND.
Do, young sir. Your reputation shall not therefore be misprized. We
will make it our suit to the Duke that the wrestling might not go
forward.

ORLANDO.
I beseech you, punish me not with your hard thoughts, wherein I confess
me much guilty to deny so fair and excellent ladies anything. But let
your fair eyes and gentle wishes go with me to my trial, wherein if I
be foiled there is but one shamed that was never gracious; if killed,
but one dead that is willing to be so. I shall do my friends no wrong,
for I have none to lament me; the world no injury, for in it I have
nothing. Only in the world I fill up a place, which may be better
supplied when I have made it empty.

ROSALIND.
The little strength that I have, I would it were with you.

CELIA.
And mine to eke out hers.

ROSALIND.
Fare you well. Pray heaven I be deceived in you.

CELIA.
Your heart’s desires be with you.

CHARLES.
Come, where is this young gallant that is so desirous to lie with his
mother earth?

ORLANDO.
Ready, sir; but his will hath in it a more modest working.

DUKE FREDERICK.
You shall try but one fall.

CHARLES.
No, I warrant your grace you shall not entreat him to a second, that
have so mightily persuaded him from a first.

ORLANDO.
You mean to mock me after; you should not have mocked me before. But
come your ways.

ROSALIND.
Now, Hercules be thy speed, young man!

CELIA.
I would I were invisible, to catch the strong fellow by the leg.

[_Orlando and Charles wrestle._]

ROSALIND.
O excellent young man!

CELIA.
If I had a thunderbolt in mine eye, I can tell who should down.

[_Charles is thrown. Shout._]

DUKE FREDERICK.
No more, no more.

ORLANDO.
Yes, I beseech your grace. I am not yet well breathed.

DUKE FREDERICK.
How dost thou, Charles?

LE BEAU.
He cannot speak, my lord.

DUKE FREDERICK.
Bear him away.

[_Charles is carried off by Attendants._]

What is thy name, young man?

ORLANDO.
Orlando, my liege, the youngest son of Sir Rowland de Boys.

DUKE FREDERICK.
I would thou hadst been son to some man else.
The world esteemed thy father honourable,
But I did find him still mine enemy.
Thou shouldst have better pleased me with this deed
Hadst thou descended from another house.
But fare thee well, thou art a gallant youth.
I would thou hadst told me of another father.

[_Exeunt Duke Frederick, Le Beau and Lords._]

CELIA.
Were I my father, coz, would I do this?

ORLANDO.
I am more proud to be Sir Rowland’s son,
His youngest son, and would not change that calling
To be adopted heir to Frederick.

ROSALIND.
My father loved Sir Rowland as his soul,
And all the world was of my father’s mind.
Had I before known this young man his son,
I should have given him tears unto entreaties
Ere he should thus have ventured.

CELIA.
Gentle cousin,
Let us go thank him and encourage him.
My father’s rough and envious disposition
Sticks me at heart.—Sir, you have well deserved.
If you do keep your promises in love
But justly, as you have exceeded promise,
Your mistress shall be happy.

ROSALIND.
Gentleman,

[_Giving him a chain from her neck_.]

Wear this for me—one out of suits with Fortune,
That could give more but that her hand lacks means.—
Shall we go, coz?

CELIA.
Ay.—Fare you well, fair gentleman.

ORLANDO.
Can I not say, I thank you? My better parts
Are all thrown down, and that which here stands up
Is but a quintain, a mere lifeless block.

ROSALIND.
He calls us back. My pride fell with my fortunes.
I’ll ask him what he would.—Did you call, sir?—
Sir, you have wrestled well and overthrown
More than your enemies.

CELIA.
Will you go, coz?

ROSALIND.
Have with you.—Fare you well.

[_Exeunt Rosalind and Celia._]

ORLANDO.
What passion hangs these weights upon my tongue?
I cannot speak to her, yet she urged conference.
O poor Orlando, thou art overthrown.
Or Charles or something weaker masters thee.

Enter Le Beau.

LE BEAU.
Good sir, I do in friendship counsel you
To leave this place. Albeit you have deserved
High commendation, true applause, and love,
Yet such is now the Duke’s condition
That he misconsters all that you have done.
The Duke is humorous; what he is indeed
More suits you to conceive than I to speak of.

ORLANDO.
I thank you, sir; and pray you tell me this:
Which of the two was daughter of the Duke
That here was at the wrestling?

LE BEAU.
Neither his daughter, if we judge by manners,
But yet indeed the smaller is his daughter.
The other is daughter to the banished Duke,
And here detained by her usurping uncle
To keep his daughter company, whose loves
Are dearer than the natural bond of sisters.
But I can tell you that of late this Duke
Hath ta’en displeasure ’gainst his gentle niece,
Grounded upon no other argument
But that the people praise her for her virtues
And pity her for her good father’s sake;
And, on my life, his malice ’gainst the lady
Will suddenly break forth. Sir, fare you well.
Hereafter, in a better world than this,
I shall desire more love and knowledge of you.

ORLANDO.
I rest much bounden to you; fare you well!

[_Exit Le Beau._]

Thus must I from the smoke into the smother,
From tyrant Duke unto a tyrant brother.
But heavenly Rosalind!

[_Exit._]

SCENE III. A Room in the Palace

Enter Celia and Rosalind.

CELIA.
Why, cousin, why, Rosalind! Cupid have mercy! Not a word?

ROSALIND.
Not one to throw at a dog.

CELIA.
No, thy words are too precious to be cast away upon curs. Throw some of
them at me. Come, lame me with reasons.

ROSALIND.
Then there were two cousins laid up, when the one should be lamed with
reasons and the other mad without any.

CELIA.
But is all this for your father?

ROSALIND.
No, some of it is for my child’s father. O, how full of briers is this
working-day world!

CELIA.
They are but burs, cousin, thrown upon thee in holiday foolery. If we
walk not in the trodden paths, our very petticoats will catch them.

ROSALIND.
I could shake them off my coat; these burs are in my heart.

CELIA.
Hem them away.

ROSALIND.
I would try, if I could cry “hem” and have him.

CELIA.
Come, come, wrestle with thy affections.

ROSALIND.
O, they take the part of a better wrestler than myself.

CELIA.
O, a good wish upon you! You will try in time, in despite of a fall.
But turning these jests out of service, let us talk in good earnest. Is
it possible on such a sudden you should fall into so strong a liking
with old Sir Rowland’s youngest son?

ROSALIND.
The Duke my father loved his father dearly.

CELIA.
Doth it therefore ensue that you should love his son dearly? By this
kind of chase I should hate him, for my father hated his father dearly;
yet I hate not Orlando.

ROSALIND.
No, faith, hate him not, for my sake.

CELIA.
Why should I not? Doth he not deserve well?

Enter Duke Frederick with Lords.

ROSALIND.
Let me love him for that, and do you love him because I do.—Look, here
comes the Duke.

CELIA.
With his eyes full of anger.

DUKE FREDERICK.
Mistress, dispatch you with your safest haste,
And get you from our court.

ROSALIND.
Me, uncle?

DUKE FREDERICK.
You, cousin.
Within these ten days if that thou be’st found
So near our public court as twenty miles,
Thou diest for it.

ROSALIND.
I do beseech your Grace,
Let me the knowledge of my fault bear with me.
If with myself I hold intelligence,
Or have acquaintance with mine own desires,
If that I do not dream, or be not frantic—
As I do trust I am not—then, dear uncle,
Never so much as in a thought unborn
Did I offend your Highness.

DUKE FREDERICK.
Thus do all traitors.
If their purgation did consist in words,
They are as innocent as grace itself.
Let it suffice thee that I trust thee not.

ROSALIND.
Yet your mistrust cannot make me a traitor.
Tell me whereon the likelihood depends.

DUKE FREDERICK.
Thou art thy father’s daughter, there’s enough.

ROSALIND.
So was I when your highness took his dukedom;
So was I when your highness banished him.
Treason is not inherited, my lord,
Or, if we did derive it from our friends,
What’s that to me? My father was no traitor.
Then, good my liege, mistake me not so much
To think my poverty is treacherous.

CELIA.
Dear sovereign, hear me speak.

DUKE FREDERICK.
Ay, Celia, we stayed her for your sake,
Else had she with her father ranged along.

CELIA.
I did not then entreat to have her stay;
It was your pleasure and your own remorse.
I was too young that time to value her,
But now I know her. If she be a traitor,
Why, so am I. We still have slept together,
Rose at an instant, learned, played, ate together,
And wheresoe’er we went, like Juno’s swans,
Still we went coupled and inseparable.

DUKE FREDERICK.
She is too subtle for thee, and her smoothness,
Her very silence, and her patience
Speak to the people, and they pity her.
Thou art a fool. She robs thee of thy name,
And thou wilt show more bright and seem more virtuous
When she is gone. Then open not thy lips.
Firm and irrevocable is my doom
Which I have passed upon her. She is banished.

CELIA.
Pronounce that sentence then on me, my liege.
I cannot live out of her company.

DUKE FREDERICK.
You are a fool. You, niece, provide yourself.
If you outstay the time, upon mine honour
And in the greatness of my word, you die.

[_Exeunt Duke Frederick and Lords._]

CELIA.
O my poor Rosalind, whither wilt thou go?
Wilt thou change fathers? I will give thee mine.
I charge thee, be not thou more grieved than I am.

ROSALIND.
I have more cause.

CELIA.
Thou hast not, cousin.
Prithee be cheerful. Know’st thou not the Duke
Hath banished me, his daughter?

ROSALIND.
That he hath not.

CELIA.
No, hath not? Rosalind lacks then the love
Which teacheth thee that thou and I am one.
Shall we be sundered? Shall we part, sweet girl?
No, let my father seek another heir.
Therefore devise with me how we may fly,
Whither to go, and what to bear with us,
And do not seek to take your change upon you,
To bear your griefs yourself and leave me out.
For, by this heaven, now at our sorrows pale,
Say what thou canst, I’ll go along with thee.

ROSALIND.
Why, whither shall we go?

CELIA.
To seek my uncle in the Forest of Arden.

ROSALIND.
Alas, what danger will it be to us,
Maids as we are, to travel forth so far?
Beauty provoketh thieves sooner than gold.

CELIA.
I’ll put myself in poor and mean attire,
And with a kind of umber smirch my face.
The like do you; so shall we pass along
And never stir assailants.

ROSALIND.
Were it not better,
Because that I am more than common tall,
That I did suit me all points like a man?
A gallant curtal-axe upon my thigh,
A boar-spear in my hand, and in my heart
Lie there what hidden woman’s fear there will,
We’ll have a swashing and a martial outside,
As many other mannish cowards have
That do outface it with their semblances.

CELIA.
What shall I call thee when thou art a man?

ROSALIND.
I’ll have no worse a name than Jove’s own page,
And therefore look you call me Ganymede.
But what will you be called?

CELIA.
Something that hath a reference to my state:
No longer Celia, but Aliena.

ROSALIND.
But, cousin, what if we assayed to steal
The clownish fool out of your father’s court?
Would he not be a comfort to our travel?

CELIA.
He’ll go along o’er the wide world with me.
Leave me alone to woo him. Let’s away,
And get our jewels and our wealth together,
Devise the fittest time and safest way
To hide us from pursuit that will be made
After my flight. Now go we in content
To liberty, and not to banishment.

[_Exeunt._]




ACT II

SCENE I. The Forest of Arden


Enter Duke Senior, Amiens and two or three Lords, dressed as foresters.

DUKE SENIOR.
Now, my co-mates and brothers in exile,
Hath not old custom made this life more sweet
Than that of painted pomp? Are not these woods
More free from peril than the envious court?
Here feel we not the penalty of Adam,
The seasons’ difference, as the icy fang
And churlish chiding of the winter’s wind,
Which when it bites and blows upon my body
Even till I shrink with cold, I smile and say:
“This is no flattery. These are counsellors
That feelingly persuade me what I am.”
Sweet are the uses of adversity,
Which, like the toad, ugly and venomous,
Wears yet a precious jewel in his head;
And this our life, exempt from public haunt,
Finds tongues in trees, books in the running brooks,
Sermons in stones, and good in everything.

AMIENS.
I would not change it. Happy is your grace,
That can translate the stubbornness of fortune
Into so quiet and so sweet a style.

DUKE SENIOR.
Come, shall we go and kill us venison?
And yet it irks me the poor dappled fools,
Being native burghers of this desert city,
Should in their own confines with forked heads
Have their round haunches gored.

FIRST LORD.
Indeed, my lord,
The melancholy Jaques grieves at that,
And in that kind swears you do more usurp
Than doth your brother that hath banished you.
Today my lord of Amiens and myself
Did steal behind him as he lay along
Under an oak, whose antique root peeps out
Upon the brook that brawls along this wood;
To the which place a poor sequestered stag,
That from the hunter’s aim had ta’en a hurt,
Did come to languish; and indeed, my lord,
The wretched animal heaved forth such groans
That their discharge did stretch his leathern coat
Almost to bursting, and the big round tears
Coursed one another down his innocent nose
In piteous chase. And thus the hairy fool,
Much marked of the melancholy Jaques,
Stood on th’ extremest verge of the swift brook,
Augmenting it with tears.

DUKE SENIOR.
But what said Jaques?
Did he not moralize this spectacle?

FIRST LORD.
O yes, into a thousand similes.
First, for his weeping into the needless stream:
“Poor deer,” quoth he “thou mak’st a testament
As worldlings do, giving thy sum of more
To that which had too much.” Then, being there alone,
Left and abandoned of his velvet friends:
“’Tis right”; quoth he, “thus misery doth part
The flux of company.” Anon a careless herd,
Full of the pasture, jumps along by him
And never stays to greet him. “Ay,” quoth Jaques,
“Sweep on, you fat and greasy citizens!
’Tis just the fashion. Wherefore do you look
Upon that poor and broken bankrupt there?”
Thus most invectively he pierceth through
The body of the country, city, court,
Yea, and of this our life, swearing that we
Are mere usurpers, tyrants, and what’s worse,
To fright the animals and to kill them up
In their assigned and native dwelling-place.

DUKE SENIOR.
And did you leave him in this contemplation?

SECOND LORD.
We did, my lord, weeping and commenting
Upon the sobbing deer.

DUKE SENIOR.
Show me the place.
I love to cope him in these sullen fits,
For then he’s full of matter.

FIRST LORD.
I’ll bring you to him straight.

[_Exeunt._]

SCENE II. A Room in the Palace

Enter Duke Frederick with Lords.

DUKE FREDERICK.
Can it be possible that no man saw them?
It cannot be! Some villains of my court
Are of consent and sufferance in this.

FIRST LORD.
I cannot hear of any that did see her.
The ladies, her attendants of her chamber,
Saw her abed, and in the morning early
They found the bed untreasured of their mistress.

SECOND LORD.
My lord, the roynish clown, at whom so oft
Your grace was wont to laugh, is also missing.
Hesperia, the princess’ gentlewoman,
Confesses that she secretly o’erheard
Your daughter and her cousin much commend
The parts and graces of the wrestler
That did but lately foil the sinewy Charles;
And she believes wherever they are gone
That youth is surely in their company.

DUKE FREDERICK.
Send to his brother; fetch that gallant hither.
If he be absent, bring his brother to me.
I’ll make him find him. Do this suddenly!
And let not search and inquisition quail
To bring again these foolish runaways.

[_Exeunt._]

SCENE III. Before Oliver’s House

Enter Orlando and Adam, meeting.

ORLANDO.
Who’s there?

ADAM.
What, my young master? O my gentle master,
O my sweet master, O you memory
Of old Sir Rowland! Why, what make you here?
Why are you virtuous? Why do people love you?
And wherefore are you gentle, strong, and valiant?
Why would you be so fond to overcome
The bonny prizer of the humorous Duke?
Your praise is come too swiftly home before you.
Know you not, master, to some kind of men
Their graces serve them but as enemies?
No more do yours. Your virtues, gentle master,
Are sanctified and holy traitors to you.
O, what a world is this, when what is comely
Envenoms him that bears it!

ORLANDO.
Why, what’s the matter?

ADAM.
O unhappy youth,
Come not within these doors! Within this roof
The enemy of all your graces lives.
Your brother—no, no brother, yet the son—
Yet not the son; I will not call him son—
Of him I was about to call his father,
Hath heard your praises, and this night he means
To burn the lodging where you use to lie,
And you within it. If he fail of that,
He will have other means to cut you off;
I overheard him and his practices.
This is no place; this house is but a butchery.
Abhor it, fear it, do not enter it.

ORLANDO.
Why, whither, Adam, wouldst thou have me go?

ADAM.
No matter whither, so you come not here.

ORLANDO.
What, wouldst thou have me go and beg my food,
Or with a base and boisterous sword enforce
A thievish living on the common road?
This I must do, or know not what to do.
Yet this I will not do, do how I can.
I rather will subject me to the malice
Of a diverted blood and bloody brother.

ADAM.
But do not so. I have five hundred crowns,
The thrifty hire I saved under your father,
Which I did store to be my foster-nurse,
When service should in my old limbs lie lame,
And unregarded age in corners thrown.
Take that, and He that doth the ravens feed,
Yea, providently caters for the sparrow,
Be comfort to my age. Here is the gold.
All this I give you. Let me be your servant.
Though I look old, yet I am strong and lusty,
For in my youth I never did apply
Hot and rebellious liquors in my blood,
Nor did not with unbashful forehead woo
The means of weakness and debility.
Therefore my age is as a lusty winter,
Frosty but kindly. Let me go with you.
I’ll do the service of a younger man
In all your business and necessities.

ORLANDO.
O good old man, how well in thee appears
The constant service of the antique world,
When service sweat for duty, not for meed.
Thou art not for the fashion of these times,
Where none will sweat but for promotion,
And having that do choke their service up
Even with the having. It is not so with thee.
But, poor old man, thou prun’st a rotten tree,
That cannot so much as a blossom yield
In lieu of all thy pains and husbandry.
But come thy ways, we’ll go along together,
And ere we have thy youthful wages spent
We’ll light upon some settled low content.

ADAM.
Master, go on and I will follow thee
To the last gasp with truth and loyalty.
From seventeen years till now almost fourscore
Here lived I, but now live here no more.
At seventeen years many their fortunes seek,
But at fourscore it is too late a week.
Yet fortune cannot recompense me better
Than to die well and not my master’s debtor.

[_Exeunt._]

SCENE IV. The Forest of Arden

Enter Rosalind as Ganymede, Celia as Aliena, and Touchstone.

ROSALIND.
O Jupiter, how weary are my spirits!

TOUCHSTONE.
I care not for my spirits, if my legs were not weary.

ROSALIND.
I could find in my heart to disgrace my man’s apparel, and to cry like
a woman, but I must comfort the weaker vessel, as doublet and hose
ought to show itself courageous to petticoat. Therefore, courage, good
Aliena.

CELIA.
I pray you bear with me, I cannot go no further.

TOUCHSTONE.
For my part, I had rather bear with you than bear you. Yet I should
bear no cross if I did bear you, for I think you have no money in your
purse.

ROSALIND.
Well, this is the forest of Arden.

TOUCHSTONE.
Ay, now am I in Arden, the more fool I! When I was at home I was in a
better place, but travellers must be content.

Enter Corin and Silvius.

ROSALIND.
Ay, be so, good Touchstone. Look you, who comes here? A young man and
an old in solemn talk.

CORIN.
That is the way to make her scorn you still.

SILVIUS.
O Corin, that thou knew’st how I do love her!

CORIN.
I partly guess, for I have loved ere now.

SILVIUS.
No, Corin, being old, thou canst not guess,
Though in thy youth thou wast as true a lover
As ever sighed upon a midnight pillow.
But if thy love were ever like to mine—
As sure I think did never man love so—
How many actions most ridiculous
Hast thou been drawn to by thy fantasy?

CORIN.
Into a thousand that I have forgotten.

SILVIUS.
O, thou didst then never love so heartily!
If thou rememb’rest not the slightest folly
That ever love did make thee run into,
Thou hast not loved.
Or if thou hast not sat as I do now,
Wearing thy hearer in thy mistress’ praise,
Thou hast not loved.
Or if thou hast not broke from company
Abruptly, as my passion now makes me,
Thou hast not loved.
O Phoebe, Phoebe, Phoebe!

[_Exit Silvius._]

ROSALIND.
Alas, poor shepherd, searching of thy wound,
I have by hard adventure found mine own.

TOUCHSTONE.
And I mine. I remember when I was in love I broke my sword upon a stone
and bid him take that for coming a-night to Jane Smile; and I remember
the kissing of her batlet, and the cow’s dugs that her pretty chopped
hands had milked; and I remember the wooing of a peascod instead of
her, from whom I took two cods, and, giving her them again, said with
weeping tears, “Wear these for my sake.” We that are true lovers run
into strange capers. But as all is mortal in nature, so is all nature
in love mortal in folly.

ROSALIND.
Thou speak’st wiser than thou art ware of.

TOUCHSTONE.
Nay, I shall ne’er be ware of mine own wit till I break my shins
against it.

ROSALIND.
Jove, Jove, this shepherd’s passion
Is much upon my fashion.

TOUCHSTONE.
And mine, but it grows something stale with me.

CELIA.
I pray you, one of you question yond man
If he for gold will give us any food.
I faint almost to death.

TOUCHSTONE.
Holla, you clown!

ROSALIND.
Peace, fool, he’s not thy kinsman.

CORIN.
Who calls?

TOUCHSTONE.
Your betters, sir.

CORIN.
Else are they very wretched.

ROSALIND.
Peace, I say.—Good even to you, friend.

CORIN.
And to you, gentle sir, and to you all.

ROSALIND.
I prithee, shepherd, if that love or gold
Can in this desert place buy entertainment,
Bring us where we may rest ourselves and feed.
Here’s a young maid with travel much oppressed,
And faints for succour.

CORIN.
Fair sir, I pity her
And wish, for her sake more than for mine own,
My fortunes were more able to relieve her.
But I am shepherd to another man
And do not shear the fleeces that I graze.
My master is of churlish disposition
And little recks to find the way to heaven
By doing deeds of hospitality.
Besides, his cote, his flocks, and bounds of feed
Are now on sale, and at our sheepcote now,
By reason of his absence, there is nothing
That you will feed on. But what is, come see,
And in my voice most welcome shall you be.

ROSALIND.
What is he that shall buy his flock and pasture?

CORIN.
That young swain that you saw here but erewhile,
That little cares for buying anything.

ROSALIND.
I pray thee, if it stand with honesty,
Buy thou the cottage, pasture, and the flock,
And thou shalt have to pay for it of us.

CELIA.
And we will mend thy wages. I like this place,
And willingly could waste my time in it.

CORIN.
Assuredly the thing is to be sold.
Go with me. If you like upon report
The soil, the profit, and this kind of life,
I will your very faithful feeder be,
And buy it with your gold right suddenly.

[_Exeunt._]

SCENE V. Another part of the Forest

Enter Amiens, Jaques and others.

AMIENS.
[_Sings_.]

        Under the greenwood tree,
        Who loves to lie with me
        And turn his merry note
        Unto the sweet bird’s throat,
    Come hither, come hither, come hither!
        Here shall he see
        No enemy
    But winter and rough weather.

JAQUES.
More, more, I prithee, more.

AMIENS.
It will make you melancholy, Monsieur Jaques.

JAQUES.
I thank it. More, I prithee, more. I can suck melancholy out of a song
as a weasel sucks eggs. More, I prithee, more.

AMIENS.
My voice is ragged. I know I cannot please you.

JAQUES.
I do not desire you to please me; I do desire you to sing. Come, more,
another _stanzo_. Call you ’em _stanzos?_

AMIENS.
What you will, Monsieur Jaques.

JAQUES.
Nay, I care not for their names. They owe me nothing. Will you sing?

AMIENS.
More at your request than to please myself.

JAQUES.
Well then, if ever I thank any man, I’ll thank you; but that they call
compliment is like th’ encounter of two dog-apes. And when a man thanks
me heartily, methinks I have given him a penny and he renders me the
beggarly thanks. Come, sing; and you that will not, hold your tongues.

AMIENS.
Well, I’ll end the song.—Sirs, cover the while. The Duke will drink
under this tree; he hath been all this day to look you.

JAQUES.
And I have been all this day to avoid him. He is too disputable for my
company. I think of as many matters as he, but I give heaven thanks and
make no boast of them. Come, warble, come.

AMIENS.
[_Sings_.]

        Who doth ambition shun
        And loves to live i’ th’ sun,
        Seeking the food he eats
        And pleased with what he gets,
    Come hither, come hither, come hither.
        Here shall he see
        No enemy
    But winter and rough weather.

JAQUES.
I’ll give you a verse to this note that I made yesterday in despite of
my invention.

AMIENS.
And I’ll sing it.

JAQUES.
Thus it goes:

        If it do come to pass
        That any man turn ass,
        Leaving his wealth and ease
        A stubborn will to please,
    Ducdame, ducdame, ducdame;
        Here shall he see
        Gross fools as he,
    An if he will come to me.

AMIENS.
What’s that “ducdame?”

JAQUES.
’Tis a Greek invocation to call fools into a circle. I’ll go sleep if I
can; if I cannot, I’ll rail against all the first-born of Egypt.

AMIENS.
And I’ll go seek the Duke; his banquet is prepared.

[_Exeunt severally._]

SCENE VI. Another part of the Forest

Enter Orlando and Adam.

ADAM.
Dear master, I can go no further. O, I die for food! Here lie I down
and measure out my grave. Farewell, kind master.

ORLANDO.
Why, how now, Adam? No greater heart in thee? Live a little, comfort a
little, cheer thyself a little. If this uncouth forest yield anything
savage, I will either be food for it or bring it for food to thee. Thy
conceit is nearer death than thy powers. For my sake, be comfortable.
Hold death awhile at the arm’s end. I will here be with thee presently,
and if I bring thee not something to eat, I’ll give thee leave to die.
But if thou diest before I come, thou art a mocker of my labour. Well
said, thou look’st cheerly, and I’ll be with thee quickly. Yet thou
liest in the bleak air. Come, I will bear thee to some shelter and thou
shalt not die for lack of a dinner if there live anything in this
desert. Cheerly, good Adam!

[_Exeunt._]

SCENE VII. Another part of the Forest

Enter Duke Senior, Amiens and Lords as outlaws.

DUKE SENIOR.
I think he be transformed into a beast,
For I can nowhere find him like a man.

FIRST LORD.
My lord, he is but even now gone hence;
Here was he merry, hearing of a song.

DUKE SENIOR.
If he, compact of jars, grow musical,
We shall have shortly discord in the spheres.
Go seek him, tell him I would speak with him.

Enter Jaques.

FIRST LORD.
He saves my labour by his own approach.

DUKE SENIOR.
Why, how now, monsieur? What a life is this
That your poor friends must woo your company?
What, you look merrily.

JAQUES.
A fool, a fool! I met a fool i’ th’ forest,
A motley fool. A miserable world!
As I do live by food, I met a fool,
Who laid him down and basked him in the sun,
And railed on Lady Fortune in good terms,
In good set terms, and yet a motley fool.
“Good morrow, fool,” quoth I. “No, sir,” quoth he,
“Call me not fool till heaven hath sent me fortune.”
And then he drew a dial from his poke,
And, looking on it with lack-lustre eye,
Says very wisely, “It is ten o’clock.
Thus we may see,” quoth he, “how the world wags.
’Tis but an hour ago since it was nine,
And after one hour more ’twill be eleven.
And so from hour to hour we ripe and ripe,
And then from hour to hour we rot and rot,
And thereby hangs a tale.” When I did hear
The motley fool thus moral on the time,
My lungs began to crow like chanticleer,
That fools should be so deep-contemplative,
And I did laugh sans intermission
An hour by his dial. O noble fool!
A worthy fool! Motley’s the only wear.

DUKE SENIOR.
What fool is this?

JAQUES.
O worthy fool!—One that hath been a courtier,
And says if ladies be but young and fair,
They have the gift to know it. And in his brain,
Which is as dry as the remainder biscuit
After a voyage, he hath strange places crammed
With observation, the which he vents
In mangled forms. O that I were a fool!
I am ambitious for a motley coat.

DUKE SENIOR.
Thou shalt have one.

JAQUES.
It is my only suit,
Provided that you weed your better judgements
Of all opinion that grows rank in them
That I am wise. I must have liberty
Withal, as large a charter as the wind,
To blow on whom I please, for so fools have.
And they that are most galled with my folly,
They most must laugh. And why, sir, must they so?
The “why” is plain as way to parish church.
He that a fool doth very wisely hit
Doth very foolishly, although he smart,
Not to seem senseless of the bob. If not,
The wise man’s folly is anatomized
Even by the squandering glances of the fool.
Invest me in my motley. Give me leave
To speak my mind, and I will through and through
Cleanse the foul body of th’ infected world,
If they will patiently receive my medicine.

DUKE SENIOR.
Fie on thee! I can tell what thou wouldst do.

JAQUES.
What, for a counter, would I do but good?

DUKE SENIOR.
Most mischievous foul sin, in chiding sin;
For thou thyself hast been a libertine,
As sensual as the brutish sting itself,
And all th’ embossed sores and headed evils
That thou with license of free foot hast caught
Wouldst thou disgorge into the general world.

JAQUES.
Why, who cries out on pride
That can therein tax any private party?
Doth it not flow as hugely as the sea
Till that the weary very means do ebb?
What woman in the city do I name
When that I say the city-woman bears
The cost of princes on unworthy shoulders?
Who can come in and say that I mean her,
When such a one as she, such is her neighbour?
Or what is he of basest function
That says his bravery is not on my cost,
Thinking that I mean him, but therein suits
His folly to the mettle of my speech?
There then. How then, what then? Let me see wherein
My tongue hath wronged him. If it do him right,
Then he hath wronged himself. If he be free,
Why then my taxing like a wild-goose flies
Unclaimed of any man. But who comes here?

Enter Orlando with sword drawn.

ORLANDO.
Forbear, and eat no more.

JAQUES.
Why, I have eat none yet.

ORLANDO.
Nor shalt not till necessity be served.

JAQUES.
Of what kind should this cock come of?

DUKE SENIOR.
Art thou thus boldened, man, by thy distress?
Or else a rude despiser of good manners,
That in civility thou seem’st so empty?

ORLANDO.
You touched my vein at first. The thorny point
Of bare distress hath ta’en from me the show
Of smooth civility; yet am I inland bred
And know some nurture. But forbear, I say!
He dies that touches any of this fruit
Till I and my affairs are answered.

JAQUES.
An you will not be answered with reason, I must die.

DUKE SENIOR.
What would you have? Your gentleness shall force
More than your force move us to gentleness.

ORLANDO.
I almost die for food, and let me have it.

DUKE SENIOR.
Sit down and feed, and welcome to our table.

ORLANDO.
Speak you so gently? Pardon me, I pray you.
I thought that all things had been savage here
And therefore put I on the countenance
Of stern commandment. But whate’er you are
That in this desert inaccessible,
Under the shade of melancholy boughs,
Lose and neglect the creeping hours of time,
If ever you have looked on better days,
If ever been where bells have knolled to church,
If ever sat at any good man’s feast,
If ever from your eyelids wiped a tear,
And know what ’tis to pity and be pitied,
Let gentleness my strong enforcement be,
In the which hope I blush and hide my sword.

DUKE SENIOR.
True is it that we have seen better days,
And have with holy bell been knolled to church,
And sat at good men’s feasts, and wiped our eyes
Of drops that sacred pity hath engendered.
And therefore sit you down in gentleness,
And take upon command what help we have
That to your wanting may be ministered.

ORLANDO.
Then but forbear your food a little while,
Whiles, like a doe, I go to find my fawn,
And give it food. There is an old poor man
Who after me hath many a weary step
Limped in pure love. Till he be first sufficed,
Oppressed with two weak evils, age and hunger,
I will not touch a bit.

DUKE SENIOR.
Go find him out,
And we will nothing waste till you return.

ORLANDO.
I thank ye, and be blest for your good comfort.

[_Exit._]

DUKE SENIOR.
Thou seest we are not all alone unhappy.
This wide and universal theatre
Presents more woeful pageants than the scene
Wherein we play in.

JAQUES.
All the world’s a stage,
And all the men and women merely players;
They have their exits and their entrances,
And one man in his time plays many parts,
His acts being seven ages. At first the infant,
Mewling and puking in the nurse’s arms;
Then the whining schoolboy, with his satchel
And shining morning face, creeping like snail
Unwillingly to school. And then the lover,
Sighing like furnace, with a woeful ballad
Made to his mistress’ eyebrow. Then a soldier,
Full of strange oaths and bearded like the pard,
Jealous in honour, sudden and quick in quarrel,
Seeking the bubble reputation
Even in the cannon’s mouth. And then the justice,
In fair round belly with good capon lined,
With eyes severe and beard of formal cut,
Full of wise saws and modern instances;
And so he plays his part. The sixth age shifts
Into the lean and slippered pantaloon,
With spectacles on nose and pouch on side,
His youthful hose, well saved, a world too wide
For his shrunk shank, and his big manly voice,
Turning again toward childish treble, pipes
And whistles in his sound. Last scene of all,
That ends this strange eventful history,
Is second childishness and mere oblivion,
Sans teeth, sans eyes, sans taste, sans everything.

Enter Orlando bearing Adam.

DUKE SENIOR.
Welcome. Set down your venerable burden,
And let him feed.

ORLANDO.
I thank you most for him.

ADAM.
So had you need;
I scarce can speak to thank you for myself.

DUKE SENIOR.
Welcome, fall to. I will not trouble you
As yet to question you about your fortunes.
Give us some music, and good cousin, sing.

SONG.


AMIENS. (_Sings_.)
        Blow, blow, thou winter wind,
        Thou art not so unkind
            As man’s ingratitude.
        Thy tooth is not so keen,
        Because thou art not seen,
            Although thy breath be rude.
Heigh-ho, sing heigh-ho, unto the green holly.
Most friendship is feigning, most loving mere folly.
        Then, heigh-ho, the holly!
            This life is most jolly.

         Freeze, freeze, thou bitter sky,
        That dost not bite so nigh
            As benefits forgot.
        Though thou the waters warp,
        Thy sting is not so sharp
            As friend remembered not.
Heigh-ho, sing heigh-ho, unto the green holly.
Most friendship is feigning, most loving mere folly.
        Then, heigh-ho, the holly!
            This life is most jolly.

DUKE SENIOR.
If that you were the good Sir Rowland’s son,
As you have whispered faithfully you were,
And as mine eye doth his effigies witness
Most truly limned and living in your face,
Be truly welcome hither. I am the Duke
That loved your father. The residue of your fortune
Go to my cave and tell me.—Good old man,
Thou art right welcome as thy master is.
Support him by the arm. [_To Orlando_.] Give me your hand,
And let me all your fortunes understand.

[_Exeunt._]




ACT III

SCENE I. A Room in the Palace


Enter Duke Frederick, Lords and Oliver.

DUKE FREDERICK.
Not see him since? Sir, sir, that cannot be.
But were I not the better part made mercy,
I should not seek an absent argument
Of my revenge, thou present. But look to it:
Find out thy brother wheresoe’er he is.
Seek him with candle. Bring him dead or living
Within this twelvemonth, or turn thou no more
To seek a living in our territory.
Thy lands, and all things that thou dost call thine
Worth seizure, do we seize into our hands,
Till thou canst quit thee by thy brother’s mouth
Of what we think against thee.

OLIVER.
O that your highness knew my heart in this:
I never loved my brother in my life.

DUKE FREDERICK.
More villain thou. Well, push him out of doors,
And let my officers of such a nature
Make an extent upon his house and lands.
Do this expediently, and turn him going.

[_Exeunt._]

SCENE II. The Forest of Arden

Enter Orlando with a paper.

ORLANDO.
Hang there, my verse, in witness of my love.
     And thou, thrice-crowned queen of night, survey
With thy chaste eye, from thy pale sphere above,
     Thy huntress’ name that my full life doth sway.
O Rosalind, these trees shall be my books,
     And in their barks my thoughts I’ll character,
That every eye which in this forest looks
     Shall see thy virtue witnessed everywhere.
Run, run, Orlando, carve on every tree
The fair, the chaste, and unexpressive she.

[_Exit._]

Enter Corin and Touchstone.

CORIN.
And how like you this shepherd’s life, Master Touchstone?

TOUCHSTONE.
Truly, shepherd, in respect of itself, it is a good life; but in
respect that it is a shepherd’s life, it is naught. In respect that it
is solitary, I like it very well; but in respect that it is private, it
is a very vile life. Now in respect it is in the fields, it pleaseth me
well; but in respect it is not in the court, it is tedious. As it is a
spare life, look you, it fits my humour well; but as there is no more
plenty in it, it goes much against my stomach. Hast any philosophy in
thee, shepherd?

CORIN.
No more but that I know the more one sickens, the worse at ease he is;
and that he that wants money, means, and content is without three good
friends; that the property of rain is to wet, and fire to burn; that
good pasture makes fat sheep; and that a great cause of the night is
lack of the sun; that he that hath learned no wit by nature nor art may
complain of good breeding or comes of a very dull kindred.

TOUCHSTONE.
Such a one is a natural philosopher. Wast ever in court, shepherd?

CORIN.
No, truly.

TOUCHSTONE.
Then thou art damned.

CORIN.
Nay, I hope.

TOUCHSTONE.
Truly, thou art damned, like an ill-roasted egg, all on one side.

CORIN.
For not being at court? Your reason.

TOUCHSTONE.
Why, if thou never wast at court, thou never saw’st good manners; if
thou never saw’st good manners, then thy manners must be wicked, and
wickedness is sin, and sin is damnation. Thou art in a parlous state,
shepherd.

CORIN.
Not a whit, Touchstone. Those that are good manners at the court are as
ridiculous in the country as the behaviour of the country is most
mockable at the court. You told me you salute not at the court but you
kiss your hands. That courtesy would be uncleanly if courtiers were
shepherds.

TOUCHSTONE.
Instance, briefly. Come, instance.

CORIN.
Why, we are still handling our ewes, and their fells, you know, are
greasy.

TOUCHSTONE.
Why, do not your courtier’s hands sweat? And is not the grease of a
mutton as wholesome as the sweat of a man? Shallow, shallow. A better
instance, I say. Come.

CORIN.
Besides, our hands are hard.

TOUCHSTONE.
Your lips will feel them the sooner. Shallow again. A more sounder
instance, come.

CORIN.
And they are often tarred over with the surgery of our sheep; and would
you have us kiss tar? The courtier’s hands are perfumed with civet.

TOUCHSTONE.
Most shallow man! Thou worm’s meat in respect of a good piece of flesh
indeed! Learn of the wise and perpend. Civet is of a baser birth than
tar, the very uncleanly flux of a cat. Mend the instance, shepherd.

CORIN.
You have too courtly a wit for me. I’ll rest.

TOUCHSTONE.
Wilt thou rest damned? God help thee, shallow man! God make incision in
thee, thou art raw.

CORIN.
Sir, I am a true labourer. I earn that I eat, get that I wear, owe no
man hate, envy no man’s happiness, glad of other men’s good, content
with my harm; and the greatest of my pride is to see my ewes graze and
my lambs suck.

TOUCHSTONE.
That is another simple sin in you, to bring the ewes and the rams
together and to offer to get your living by the copulation of cattle;
to be bawd to a bell-wether and to betray a she-lamb of a twelvemonth
to crooked-pated, old, cuckoldly ram, out of all reasonable match. If
thou be’st not damned for this, the devil himself will have no
shepherds. I cannot see else how thou shouldst ’scape.

Enter Rosalind as Ganymede.

CORIN.
Here comes young Master Ganymede, my new mistress’s brother.

ROSALIND.
[_Reads_.]
         _From the east to western Inde
         No jewel is like Rosalind.
         Her worth being mounted on the wind,
         Through all the world bears Rosalind.
         All the pictures fairest lined
         Are but black to Rosalind.
         Let no face be kept in mind
         But the fair of Rosalind._

TOUCHSTONE.
I’ll rhyme you so eight years together, dinners and suppers and
sleeping hours excepted. It is the right butter-women’s rank to market.

ROSALIND.
Out, fool!

TOUCHSTONE.
         For a taste:
         If a hart do lack a hind,
         Let him seek out Rosalind.
         If the cat will after kind,
         So be sure will Rosalind.
         Winter garments must be lined,
         So must slender Rosalind.
         They that reap must sheaf and bind,
         Then to cart with Rosalind.
         Sweetest nut hath sourest rind,
         Such a nut is Rosalind.
         He that sweetest rose will find
         Must find love’s prick, and Rosalind.
This is the very false gallop of verses. Why do you infect yourself
with them?

ROSALIND.
Peace, you dull fool, I found them on a tree.

TOUCHSTONE.
Truly, the tree yields bad fruit.

ROSALIND.
I’ll graft it with you, and then I shall graft it with a medlar. Then
it will be the earliest fruit i’ th’ country, for you’ll be rotten ere
you be half ripe, and that’s the right virtue of the medlar.

TOUCHSTONE.
You have said, but whether wisely or no, let the forest judge.

Enter Celia as Aliena, reading a paper.

ROSALIND.
Peace, here comes my sister, reading. Stand aside.

CELIA.
[_Reads_.]
            _Why should this a desert be?
                For it is unpeopled? No!
            Tongues I’ll hang on every tree
                That shall civil sayings show.
            Some, how brief the life of man
                Runs his erring pilgrimage,
            That the streching of a span
                Buckles in his sum of age;
            Some, of violated vows
                ’Twixt the souls of friend and friend.
            But upon the fairest boughs,
                Or at every sentence’ end,
            Will I “Rosalinda” write,
                Teaching all that read to know
            The quintessence of every sprite
                Heaven would in little show.
            Therefore heaven nature charged
                That one body should be filled
            With all graces wide-enlarged.
                Nature presently distilled
            Helen’s cheek, but not her heart,
                Cleopatra’s majesty;
            Atalanta’s better part,
                Sad Lucretia’s modesty.
            Thus Rosalind of many parts
                By heavenly synod was devised,
            Of many faces, eyes, and hearts
                To have the touches dearest prized.
            Heaven would that she these gifts should have,
                And I to live and die her slave._

ROSALIND.
O most gentle Jupiter, what tedious homily of love have you wearied
your parishioners withal, and never cried “Have patience, good people!”

CELIA.
How now! Back, friends. Shepherd, go off a little. Go with him, sirrah.

TOUCHSTONE.
Come, shepherd, let us make an honourable retreat, though not with bag
and baggage, yet with scrip and scrippage.

[_Exeunt Corin and Touchstone._]

CELIA.
Didst thou hear these verses?

ROSALIND.
O yes, I heard them all, and more too, for some of them had in them
more feet than the verses would bear.

CELIA.
That’s no matter. The feet might bear the verses.

ROSALIND.
Ay, but the feet were lame and could not bear themselves without the
verse, and therefore stood lamely in the verse.

CELIA.
But didst thou hear without wondering how thy name should be hanged and
carved upon these trees?

ROSALIND.
I was seven of the nine days out of the wonder before you came; for
look here what I found on a palm-tree. I was never so berhymed since
Pythagoras’ time that I was an Irish rat, which I can hardly remember.

CELIA.
Trow you who hath done this?

ROSALIND.
Is it a man?

CELIA.
And a chain, that you once wore, about his neck. Change you colour?

ROSALIND.
I prithee, who?

CELIA.
O Lord, Lord, it is a hard matter for friends to meet; but mountains
may be removed with earthquakes and so encounter.

ROSALIND.
Nay, but who is it?

CELIA.
Is it possible?

ROSALIND.
Nay, I prithee now, with most petitionary vehemence, tell me who it is.

CELIA.
O wonderful, wonderful, most wonderful wonderful, and yet again
wonderful, and after that, out of all whooping!

ROSALIND.
Good my complexion! Dost thou think, though I am caparisoned like a
man, I have a doublet and hose in my disposition? One inch of delay
more is a South Sea of discovery. I prithee tell me who is it quickly,
and speak apace. I would thou couldst stammer, that thou mightst pour
this concealed man out of thy mouth, as wine comes out of
narrow-mouthed bottle—either too much at once or none at all. I prithee
take the cork out of thy mouth that I may drink thy tidings.

CELIA.
So you may put a man in your belly.

ROSALIND.
Is he of God’s making? What manner of man? Is his head worth a hat, or
his chin worth a beard?

CELIA.
Nay, he hath but a little beard.

ROSALIND.
Why, God will send more if the man will be thankful. Let me stay the
growth of his beard, if thou delay me not the knowledge of his chin.

CELIA.
It is young Orlando, that tripped up the wrestler’s heels and your
heart both in an instant.

ROSALIND.
Nay, but the devil take mocking! Speak sad brow and true maid.

CELIA.
I’ faith, coz, ’tis he.

ROSALIND.
Orlando?

CELIA.
Orlando.

ROSALIND.
Alas the day, what shall I do with my doublet and hose? What did he
when thou saw’st him? What said he? How looked he? Wherein went he?
What makes he here? Did he ask for me? Where remains he? How parted he
with thee? And when shalt thou see him again? Answer me in one word.

CELIA.
You must borrow me Gargantua’s mouth first. ’Tis a word too great for
any mouth of this age’s size. To say ay and no to these particulars is
more than to answer in a catechism.

ROSALIND.
But doth he know that I am in this forest and in man’s apparel? Looks
he as freshly as he did the day he wrestled?

CELIA.
It is as easy to count atomies as to resolve the propositions of a
lover. But take a taste of my finding him, and relish it with good
observance. I found him under a tree, like a dropped acorn.

ROSALIND.
It may well be called Jove’s tree when it drops forth such fruit.

CELIA.
Give me audience, good madam.

ROSALIND.
Proceed.

CELIA.
There lay he, stretched along like a wounded knight.

ROSALIND.
Though it be pity to see such a sight, it well becomes the ground.

CELIA.
Cry “holla!” to thy tongue, I prithee. It curvets unseasonably. He was
furnished like a hunter.

ROSALIND.
O, ominous! He comes to kill my heart.

CELIA.
I would sing my song without a burden. Thou bring’st me out of tune.

ROSALIND.
Do you not know I am a woman? When I think, I must speak. Sweet, say
on.

Enter Orlando and Jaques.

CELIA.
You bring me out. Soft, comes he not here?

ROSALIND.
’Tis he! Slink by, and note him.

[_Rosalind and Celia step aside._]

JAQUES.
I thank you for your company but, good faith, I had as lief have been
myself alone.

ORLANDO.
And so had I, but yet, for fashion sake, I thank you too for your
society.

JAQUES.
God be wi’ you, let’s meet as little as we can.

ORLANDO.
I do desire we may be better strangers.

JAQUES.
I pray you, mar no more trees with writing love songs in their barks.

ORLANDO.
I pray you, mar no more of my verses with reading them ill-favouredly.

JAQUES.
Rosalind is your love’s name?

ORLANDO.
Yes, just.

JAQUES.
I do not like her name.

ORLANDO.
There was no thought of pleasing you when she was christened.

JAQUES.
What stature is she of?

ORLANDO.
Just as high as my heart.

JAQUES.
You are full of pretty answers. Have you not been acquainted with
goldsmiths’ wives, and conned them out of rings?

ORLANDO.
Not so; but I answer you right painted cloth, from whence you have
studied your questions.

JAQUES.
You have a nimble wit. I think ’twas made of Atalanta’s heels. Will you
sit down with me? And we two will rail against our mistress the world
and all our misery.

ORLANDO.
I will chide no breather in the world but myself, against whom I know
most faults.

JAQUES.
The worst fault you have is to be in love.

ORLANDO.
’Tis a fault I will not change for your best virtue. I am weary of you.

JAQUES.
By my troth, I was seeking for a fool when I found you.

ORLANDO.
He is drowned in the brook. Look but in, and you shall see him.

JAQUES.
There I shall see mine own figure.

ORLANDO.
Which I take to be either a fool or a cipher.

JAQUES.
I’ll tarry no longer with you. Farewell, good Signior Love.

ORLANDO.
I am glad of your departure. Adieu, good Monsieur Melancholy.

[_Exit Jaques.—Celia and Rosalind come forward._]

ROSALIND.
I will speak to him like a saucy lackey, and under that habit play the
knave with him.
Do you hear, forester?

ORLANDO.
Very well. What would you?

ROSALIND.
I pray you, what is’t o’clock?

ORLANDO.
You should ask me what time o’ day. There’s no clock in the forest.

ROSALIND.
Then there is no true lover in the forest, else sighing every minute
and groaning every hour would detect the lazy foot of time as well as a
clock.

ORLANDO.
And why not the swift foot of time? Had not that been as proper?

ROSALIND.
By no means, sir. Time travels in divers paces with divers persons.
I’ll tell you who time ambles withal, who time trots withal, who time
gallops withal, and who he stands still withal.

ORLANDO.
I prithee, who doth he trot withal?

ROSALIND.
Marry, he trots hard with a young maid between the contract of her
marriage and the day it is solemnized. If the interim be but a
se’nnight, time’s pace is so hard that it seems the length of seven
year.

ORLANDO.
Who ambles time withal?

ROSALIND.
With a priest that lacks Latin and a rich man that hath not the gout;
for the one sleeps easily because he cannot study, and the other lives
merrily because he feels no pain; the one lacking the burden of lean
and wasteful learning, the other knowing no burden of heavy tedious
penury. These time ambles withal.

ORLANDO.
Who doth he gallop withal?

ROSALIND.
With a thief to the gallows; for though he go as softly as foot can
fall, he thinks himself too soon there.

ORLANDO.
Who stays it still withal?

ROSALIND.
With lawyers in the vacation; for they sleep between term and term, and
then they perceive not how time moves.

ORLANDO.
Where dwell you, pretty youth?

ROSALIND.
With this shepherdess, my sister, here in the skirts of the forest,
like fringe upon a petticoat.

ORLANDO.
Are you native of this place?

ROSALIND.
As the coney that you see dwell where she is kindled.

ORLANDO.
Your accent is something finer than you could purchase in so removed a
dwelling.

ROSALIND.
I have been told so of many. But indeed an old religious uncle of mine
taught me to speak, who was in his youth an inland man, one that knew
courtship too well, for there he fell in love. I have heard him read
many lectures against it, and I thank God I am not a woman, to be
touched with so many giddy offences as he hath generally taxed their
whole sex withal.

ORLANDO.
Can you remember any of the principal evils that he laid to the charge
of women?

ROSALIND.
There were none principal. They were all like one another as halfpence
are, every one fault seeming monstrous till his fellow fault came to
match it.

ORLANDO.
I prithee recount some of them.

ROSALIND.
No. I will not cast away my physic but on those that are sick. There is
a man haunts the forest that abuses our young plants with carving
“Rosalind” on their barks; hangs odes upon hawthorns and elegies on
brambles; all, forsooth, deifying the name of Rosalind. If I could meet
that fancy-monger, I would give him some good counsel, for he seems to
have the quotidian of love upon him.

ORLANDO.
I am he that is so love-shaked. I pray you tell me your remedy.

ROSALIND.
There is none of my uncle’s marks upon you. He taught me how to know a
man in love, in which cage of rushes I am sure you are not prisoner.

ORLANDO.
What were his marks?

ROSALIND.
A lean cheek, which you have not; a blue eye and sunken, which you have
not; an unquestionable spirit, which you have not; a beard neglected,
which you have not—but I pardon you for that, for simply your having in
beard is a younger brother’s revenue. Then your hose should be
ungartered, your bonnet unbanded, your sleeve unbuttoned, your shoe
untied, and everything about you demonstrating a careless desolation.
But you are no such man. You are rather point-device in your
accoutrements, as loving yourself than seeming the lover of any other.

ORLANDO.
Fair youth, I would I could make thee believe I love.

ROSALIND.
Me believe it? You may as soon make her that you love believe it, which
I warrant she is apter to do than to confess she does. That is one of
the points in the which women still give the lie to their consciences.
But, in good sooth, are you he that hangs the verses on the trees,
wherein Rosalind is so admired?

ORLANDO.
I swear to thee, youth, by the white hand of Rosalind, I am that he,
that unfortunate he.

ROSALIND.
But are you so much in love as your rhymes speak?

ORLANDO.
Neither rhyme nor reason can express how much.

ROSALIND.
Love is merely a madness, and, I tell you, deserves as well a dark
house and a whip as madmen do; and the reason why they are not so
punished and cured is that the lunacy is so ordinary that the whippers
are in love too. Yet I profess curing it by counsel.

ORLANDO.
Did you ever cure any so?

ROSALIND.
Yes, one, and in this manner. He was to imagine me his love, his
mistress, and I set him every day to woo me; at which time would I,
being but a moonish youth, grieve, be effeminate, changeable, longing
and liking, proud, fantastical, apish, shallow, inconstant, full of
tears, full of smiles; for every passion something and for no passion
truly anything, as boys and women are for the most part cattle of this
colour; would now like him, now loathe him; then entertain him, then
forswear him; now weep for him, then spit at him; that I drave my
suitor from his mad humour of love to a living humour of madness, which
was to forswear the full stream of the world and to live in a nook
merely monastic. And thus I cured him, and this way will I take upon me
to wash your liver as clean as a sound sheep’s heart, that there shall
not be one spot of love in ’t.

ORLANDO.
I would not be cured, youth.

ROSALIND.
I would cure you, if you would but call me Rosalind and come every day
to my cote and woo me.

ORLANDO.
Now, by the faith of my love, I will. Tell me where it is.

ROSALIND.
Go with me to it, and I’ll show it you; and by the way you shall tell
me where in the forest you live. Will you go?

ORLANDO.
With all my heart, good youth.

ROSALIND.
Nay, you must call me Rosalind. Come, sister, will you go?

[_Exeunt._]

SCENE III. Another part of the Forest

Enter Touchstone and Audrey; Jaques at a distance observing them.

TOUCHSTONE.
Come apace, good Audrey. I will fetch up your goats, Audrey. And how,
Audrey? Am I the man yet? Doth my simple feature content you?

AUDREY.
Your features, Lord warrant us! What features?

TOUCHSTONE.
I am here with thee and thy goats, as the most capricious poet, honest
Ovid, was among the Goths.

JAQUES.
[_Aside_.] O knowledge ill-inhabited, worse than Jove in a thatched
house!

TOUCHSTONE.
When a man’s verses cannot be understood, nor a man’s good wit seconded
with the forward child, understanding, it strikes a man more dead than
a great reckoning in a little room. Truly, I would the gods had made
thee poetical.

AUDREY.
I do not know what “poetical” is. Is it honest in deed and word? Is it
a true thing?

TOUCHSTONE.
No, truly; for the truest poetry is the most feigning, and lovers are
given to poetry, and what they swear in poetry may be said, as lovers,
they do feign.

AUDREY.
Do you wish, then, that the gods had made me poetical?

TOUCHSTONE.
I do, truly, for thou swear’st to me thou art honest. Now if thou wert
a poet, I might have some hope thou didst feign.

AUDREY.
Would you not have me honest?

TOUCHSTONE.
No, truly, unless thou wert hard-favoured; for honesty coupled to
beauty is to have honey a sauce to sugar.

JAQUES.
[_Aside_.] A material fool!

AUDREY.
Well, I am not fair, and therefore I pray the gods make me honest.

TOUCHSTONE.
Truly, and to cast away honesty upon a foul slut were to put good meat
into an unclean dish.

AUDREY.
I am not a slut, though I thank the gods I am foul.

TOUCHSTONE.
Well, praised be the gods for thy foulness; sluttishness may come
hereafter. But be it as it may be, I will marry thee. And to that end I
have been with Sir Oliver Martext, the vicar of the next village, who
hath promised to meet me in this place of the forest and to couple us.

JAQUES.
[_Aside_.] I would fain see this meeting.

AUDREY.
Well, the gods give us joy!

TOUCHSTONE.
Amen. A man may, if he were of a fearful heart, stagger in this
attempt, for here we have no temple but the wood, no assembly but
horn-beasts. But what though? Courage! As horns are odious, they are
necessary. It is said, “Many a man knows no end of his goods.” Right.
Many a man has good horns and knows no end of them. Well, that is the
dowry of his wife; ’tis none of his own getting. Horns? Even so. Poor
men alone? No, no, the noblest deer hath them as huge as the rascal. Is
the single man therefore blessed? No. As a walled town is more worthier
than a village, so is the forehead of a married man more honourable
than the bare brow of a bachelor. And by how much defence is better
than no skill, by so much is horn more precious than to want.

Enter Sir Oliver Martext.

Here comes Sir Oliver. Sir Oliver Martext, you are well met. Will you
dispatch us here under this tree, or shall we go with you to your
chapel?

MARTEXT.
Is there none here to give the woman?

TOUCHSTONE.
I will not take her on gift of any man.

MARTEXT.
Truly, she must be given, or the marriage is not lawful.

JAQUES.
[_Coming forward_.] Proceed, proceed. I’ll give her.

TOUCHSTONE.
Good even, good Master What-ye-call’t, how do you, sir? You are very
well met. God ’ild you for your last company. I am very glad to see
you. Even a toy in hand here, sir. Nay, pray be covered.

JAQUES.
Will you be married, motley?

TOUCHSTONE.
As the ox hath his bow, sir, the horse his curb, and the falcon her
bells, so man hath his desires; and as pigeons bill, so wedlock would
be nibbling.

JAQUES.
And will you, being a man of your breeding, be married under a bush
like a beggar? Get you to church, and have a good priest that can tell
you what marriage is. This fellow will but join you together as they
join wainscot; then one of you will prove a shrunk panel, and like
green timber, warp, warp.

TOUCHSTONE.
[_Aside_.] I am not in the mind but I were better to be married of him
than of another, for he is not like to marry me well, and not being
well married, it will be a good excuse for me hereafter to leave my
wife.

JAQUES.
Go thou with me, and let me counsel thee.

TOUCHSTONE.
Come, sweet Audrey. We must be married, or we must live in bawdry.
Farewell, good Master Oliver. Not
             _O sweet Oliver,
             O brave Oliver,
         Leave me not behind thee._
But
             _Wind away,—
             Begone, I say,
         I will not to wedding with thee._

[_Exeunt Touchstone, Audrey and Jaques._]

MARTEXT.
’Tis no matter. Ne’er a fantastical knave of them all shall flout me
out of my calling.

[_Exit._]

SCENE IV. Another part of the Forest. Before a Cottage

Enter Rosalind and Celia.

ROSALIND.
Never talk to me, I will weep.

CELIA.
Do, I prithee, but yet have the grace to consider that tears do not
become a man.

ROSALIND.
But have I not cause to weep?

CELIA.
As good cause as one would desire; therefore weep.

ROSALIND.
His very hair is of the dissembling colour.

CELIA.
Something browner than Judas’s. Marry, his kisses are Judas’s own
children.

ROSALIND.
I’ faith, his hair is of a good colour.

CELIA.
An excellent colour. Your chestnut was ever the only colour.

ROSALIND.
And his kissing is as full of sanctity as the touch of holy bread.

CELIA.
He hath bought a pair of cast lips of Diana. A nun of winter’s
sisterhood kisses not more religiously; the very ice of chastity is in
them.

ROSALIND.
But why did he swear he would come this morning, and comes not?

CELIA.
Nay, certainly, there is no truth in him.

ROSALIND.
Do you think so?

CELIA.
Yes. I think he is not a pick-purse nor a horse-stealer, but for his
verity in love, I do think him as concave as a covered goblet or a
worm-eaten nut.

ROSALIND.
Not true in love?

CELIA.
Yes, when he is in, but I think he is not in.

ROSALIND.
You have heard him swear downright he was.

CELIA.
“Was” is not “is”. Besides, the oath of a lover is no stronger than the
word of a tapster. They are both the confirmer of false reckonings. He
attends here in the forest on the Duke your father.

ROSALIND.
I met the Duke yesterday, and had much question with him. He asked me
of what parentage I was. I told him, of as good as he, so he laughed
and let me go. But what talk we of fathers when there is such a man as
Orlando?

CELIA.
O, that’s a brave man! He writes brave verses, speaks brave words,
swears brave oaths, and breaks them bravely, quite traverse, athwart
the heart of his lover, as a puny tilter, that spurs his horse but on
one side, breaks his staff like a noble goose. But all’s brave that
youth mounts and folly guides. Who comes here?

Enter Corin.

CORIN.
Mistress and master, you have oft enquired
After the shepherd that complained of love,
Who you saw sitting by me on the turf,
Praising the proud disdainful shepherdess
That was his mistress.

CELIA.
Well, and what of him?

CORIN.
If you will see a pageant truly played
Between the pale complexion of true love
And the red glow of scorn and proud disdain,
Go hence a little, and I shall conduct you,
If you will mark it.

ROSALIND.
O, come, let us remove.
The sight of lovers feedeth those in love.
Bring us to this sight, and you shall say
I’ll prove a busy actor in their play.

[_Exeunt._]

SCENE V. Another part of the Forest

Enter Silvius and Phoebe.

SILVIUS.
Sweet Phoebe, do not scorn me, do not, Phoebe.
Say that you love me not, but say not so
In bitterness. The common executioner,
Whose heart th’ accustomed sight of death makes hard,
Falls not the axe upon the humbled neck
But first begs pardon. Will you sterner be
Than he that dies and lives by bloody drops?

Enter Rosalind, Celia and Corin, at a distance.

PHOEBE.
I would not be thy executioner;
I fly thee, for I would not injure thee.
Thou tell’st me there is murder in mine eye.
’Tis pretty, sure, and very probable
That eyes, that are the frail’st and softest things,
Who shut their coward gates on atomies,
Should be called tyrants, butchers, murderers.
Now I do frown on thee with all my heart,
And if mine eyes can wound, now let them kill thee.
Now counterfeit to swoon; why, now fall down;
Or if thou canst not, O, for shame, for shame,
Lie not, to say mine eyes are murderers.
Now show the wound mine eye hath made in thee.
Scratch thee but with a pin, and there remains
Some scar of it; lean upon a rush,
The cicatrice and capable impressure
Thy palm some moment keeps. But now mine eyes,
Which I have darted at thee, hurt thee not;
Nor I am sure there is not force in eyes
That can do hurt.

SILVIUS.
O dear Phoebe,
If ever—as that ever may be near—
You meet in some fresh cheek the power of fancy,
Then shall you know the wounds invisible
That love’s keen arrows make.

PHOEBE.
But till that time
Come not thou near me. And when that time comes,
Afflict me with thy mocks, pity me not,
As till that time I shall not pity thee.

ROSALIND.
[_Advancing_.] And why, I pray you? Who might be your mother,
That you insult, exult, and all at once,
Over the wretched? What though you have no beauty—
As, by my faith, I see no more in you
Than without candle may go dark to bed—
Must you be therefore proud and pitiless?
Why, what means this? Why do you look on me?
I see no more in you than in the ordinary
Of nature’s sale-work. ’Od’s my little life,
I think she means to tangle my eyes too!
No, faith, proud mistress, hope not after it.
’Tis not your inky brows, your black silk hair,
Your bugle eyeballs, nor your cheek of cream,
That can entame my spirits to your worship.
You foolish shepherd, wherefore do you follow her,
Like foggy south, puffing with wind and rain?
You are a thousand times a properer man
Than she a woman. ’Tis such fools as you
That makes the world full of ill-favoured children.
’Tis not her glass but you that flatters her,
And out of you she sees herself more proper
Than any of her lineaments can show her.
But, mistress, know yourself; down on your knees,
And thank heaven, fasting, for a good man’s love.
For I must tell you friendly in your ear,
Sell when you can; you are not for all markets.
Cry the man mercy, love him, take his offer;
Foul is most foul, being foul to be a scoffer.
So take her to thee, shepherd. Fare you well.

PHOEBE.
Sweet youth, I pray you chide a year together!
I had rather hear you chide than this man woo.

ROSALIND.
He’s fall’n in love with your foulness, and she’ll fall in love with my
anger. If it be so, as fast as she answers thee with frowning looks,
I’ll sauce her with bitter words. Why look you so upon me?

PHOEBE.
For no ill will I bear you.

ROSALIND.
I pray you do not fall in love with me,
For I am falser than vows made in wine.
Besides, I like you not. If you will know my house,
’Tis at the tuft of olives here hard by.
Will you go, sister? Shepherd, ply her hard.
Come, sister. Shepherdess, look on him better,
And be not proud. Though all the world could see,
None could be so abused in sight as he.
Come, to our flock.

[_Exeunt Rosalind, Celia and Corin._]

PHOEBE.
Dead shepherd, now I find thy saw of might:
“Who ever loved that loved not at first sight?”

SILVIUS.
Sweet Phoebe—

PHOEBE.
Ha, what sayst thou, Silvius?

SILVIUS.
Sweet Phoebe, pity me.

PHOEBE.
Why, I am sorry for thee, gentle Silvius.

SILVIUS.
Wherever sorrow is, relief would be.
If you do sorrow at my grief in love,
By giving love your sorrow and my grief
Were both extermined.

PHOEBE.
Thou hast my love. Is not that neighbourly?

SILVIUS.
I would have you.

PHOEBE.
Why, that were covetousness.
Silvius, the time was that I hated thee;
And yet it is not that I bear thee love;
But since that thou canst talk of love so well,
Thy company, which erst was irksome to me,
I will endure, and I’ll employ thee too.
But do not look for further recompense
Than thine own gladness that thou art employed.

SILVIUS.
So holy and so perfect is my love,
And I in such a poverty of grace,
That I shall think it a most plenteous crop
To glean the broken ears after the man
That the main harvest reaps. Loose now and then
A scattered smile, and that I’ll live upon.

PHOEBE.
Know’st thou the youth that spoke to me erewhile?

SILVIUS.
Not very well, but I have met him oft,
And he hath bought the cottage and the bounds
That the old carlot once was master of.

PHOEBE.
Think not I love him, though I ask for him.
’Tis but a peevish boy—yet he talks well.
But what care I for words? Yet words do well
When he that speaks them pleases those that hear.
It is a pretty youth—not very pretty—
But sure he’s proud, and yet his pride becomes him.
He’ll make a proper man. The best thing in him
Is his complexion; and faster than his tongue
Did make offence, his eye did heal it up.
He is not very tall, yet for his years he’s tall;
His leg is but so-so, and yet ’tis well.
There was a pretty redness in his lip,
A little riper and more lusty red
Than that mixed in his cheek. ’Twas just the difference
Betwixt the constant red and mingled damask.
There be some women, Silvius, had they marked him
In parcels as I did, would have gone near
To fall in love with him; but for my part
I love him not nor hate him not; and yet
I have more cause to hate him than to love him.
For what had he to do to chide at me?
He said mine eyes were black and my hair black,
And now I am remembered, scorned at me.
I marvel why I answered not again.
But that’s all one: omittance is no quittance.
I’ll write to him a very taunting letter,
And thou shalt bear it. Wilt thou, Silvius?

SILVIUS.
Phoebe, with all my heart.

PHOEBE.
I’ll write it straight,
The matter’s in my head and in my heart.
I will be bitter with him and passing short.
Go with me, Silvius.

[_Exeunt._]




ACT IV

SCENE I. The Forest of Arden


Enter Rosalind, Celia and Jaques.

JAQUES.
I prithee, pretty youth, let me be better acquainted with thee.

ROSALIND.
They say you are a melancholy fellow.

JAQUES.
I am so; I do love it better than laughing.

ROSALIND.
Those that are in extremity of either are abominable fellows, and
betray themselves to every modern censure worse than drunkards.

JAQUES.
Why, ’tis good to be sad and say nothing.

ROSALIND.
Why then, ’tis good to be a post.

JAQUES.
I have neither the scholar’s melancholy, which is emulation; nor the
musician’s, which is fantastical; nor the courtier’s, which is proud;
nor the soldier’s, which is ambitious; nor the lawyer’s, which is
politic; nor the lady’s, which is nice; nor the lover’s, which is all
these; but it is a melancholy of mine own, compounded of many simples,
extracted from many objects, and indeed the sundry contemplation of my
travels, in which my often rumination wraps me in a most humorous
sadness.

ROSALIND.
A traveller! By my faith, you have great reason to be sad. I fear you
have sold your own lands to see other men’s. Then to have seen much and
to have nothing is to have rich eyes and poor hands.

JAQUES.
Yes, I have gained my experience.

ROSALIND.
And your experience makes you sad. I had rather have a fool to make me
merry than experience to make me sad—and to travel for it too.

Enter Orlando.

ORLANDO.
Good day and happiness, dear Rosalind!

JAQUES.
Nay, then, God be wi’ you, an you talk in blank verse.

ROSALIND.
Farewell, Monsieur Traveller. Look you lisp and wear strange suits;
disable all the benefits of your own country; be out of love with your
nativity, and almost chide God for making you that countenance you are,
or I will scarce think you have swam in a gondola.

[_Exit Jaques._]

Why, how now, Orlando, where have you been all this while? You a lover!
An you serve me such another trick, never come in my sight more.

ORLANDO.
My fair Rosalind, I come within an hour of my promise.

ROSALIND.
Break an hour’s promise in love? He that will divide a minute into a
thousand parts, and break but a part of the thousand part of a minute
in the affairs of love, it may be said of him that Cupid hath clapped
him o’ the shoulder, but I’ll warrant him heart-whole.

ORLANDO.
Pardon me, dear Rosalind.

ROSALIND.
Nay, an you be so tardy, come no more in my sight. I had as lief be
wooed of a snail.

ORLANDO.
Of a snail?

ROSALIND.
Ay, of a snail, for though he comes slowly, he carries his house on his
head—a better jointure, I think, than you make a woman. Besides, he
brings his destiny with him.

ORLANDO.
What’s that?

ROSALIND.
Why, horns, which such as you are fain to be beholding to your wives
for. But he comes armed in his fortune and prevents the slander of his
wife.

ORLANDO.
Virtue is no horn-maker and my Rosalind is virtuous.

ROSALIND.
And I am your Rosalind.

CELIA.
It pleases him to call you so, but he hath a Rosalind of a better leer
than you.

ROSALIND.
Come, woo me, woo me, for now I am in a holiday humour, and like enough
to consent. What would you say to me now, an I were your very, very
Rosalind?

ORLANDO.
I would kiss before I spoke.

ROSALIND.
Nay, you were better speak first, and when you were gravelled for lack
of matter, you might take occasion to kiss. Very good orators, when
they are out, they will spit; and for lovers lacking—God warn
us—matter, the cleanliest shift is to kiss.

ORLANDO.
How if the kiss be denied?

ROSALIND.
Then she puts you to entreaty, and there begins new matter.

ORLANDO.
Who could be out, being before his beloved mistress?

ROSALIND.
Marry, that should you, if I were your mistress, or I should think my
honesty ranker than my wit.

ORLANDO.
What, of my suit?

ROSALIND.
Not out of your apparel, and yet out of your suit. Am not I your
Rosalind?

ORLANDO.
I take some joy to say you are because I would be talking of her.

ROSALIND.
Well, in her person, I say I will not have you.

ORLANDO.
Then, in mine own person, I die.

ROSALIND.
No, faith, die by attorney. The poor world is almost six thousand years
old, and in all this time there was not any man died in his own person,
_videlicet_, in a love-cause. Troilus had his brains dashed out with a
Grecian club, yet he did what he could to die before, and he is one of
the patterns of love. Leander, he would have lived many a fair year
though Hero had turned nun, if it had not been for a hot midsummer
night; for, good youth, he went but forth to wash him in the Hellespont
and, being taken with the cramp, was drowned; and the foolish
chroniclers of that age found it was Hero of Sestos. But these are all
lies. Men have died from time to time and worms have eaten them, but
not for love.

ORLANDO.
I would not have my right Rosalind of this mind, for I protest her
frown might kill me.

ROSALIND.
By this hand, it will not kill a fly. But come, now I will be your
Rosalind in a more coming-on disposition, and ask me what you will, I
will grant it.

ORLANDO.
Then love me, Rosalind.

ROSALIND.
Yes, faith, will I, Fridays and Saturdays and all.

ORLANDO.
And wilt thou have me?

ROSALIND.
Ay, and twenty such.

ORLANDO.
What sayest thou?

ROSALIND.
Are you not good?

ORLANDO.
I hope so.

ROSALIND.
Why then, can one desire too much of a good thing?—Come, sister, you
shall be the priest and marry us.—Give me your hand, Orlando.—What do
you say, sister?

ORLANDO.
Pray thee, marry us.

CELIA.
I cannot say the words.

ROSALIND.
You must begin “Will you, Orlando—”


CELIA.
Go to.—Will you, Orlando, have to wife this Rosalind?

ORLANDO.
I will.

ROSALIND.
Ay, but when?

ORLANDO.
Why now, as fast as she can marry us.

ROSALIND.
Then you must say “I take thee, Rosalind, for wife.”

ORLANDO.
I take thee, Rosalind, for wife.

ROSALIND.
I might ask you for your commission. But I do take thee, Orlando, for
my husband. There’s a girl goes before the priest, and certainly a
woman’s thought runs before her actions.

ORLANDO.
So do all thoughts. They are winged.

ROSALIND.
Now tell me how long you would have her after you have possessed her.

ORLANDO.
For ever and a day.

ROSALIND.
Say “a day” without the “ever.” No, no, Orlando, men are April when
they woo, December when they wed. Maids are May when they are maids,
but the sky changes when they are wives. I will be more jealous of thee
than a Barbary cock-pigeon over his hen, more clamorous than a parrot
against rain, more new-fangled than an ape, more giddy in my desires
than a monkey. I will weep for nothing, like Diana in the fountain, and
I will do that when you are disposed to be merry. I will laugh like a
hyena, and that when thou are inclined to sleep.

ORLANDO.
But will my Rosalind do so?

ROSALIND.
By my life, she will do as I do.

ORLANDO.
O, but she is wise.

ROSALIND.
Or else she could not have the wit to do this. The wiser, the
waywarder. Make the doors upon a woman’s wit, and it will out at the
casement. Shut that, and ’twill out at the keyhole. Stop that, ’twill
fly with the smoke out at the chimney.

ORLANDO.
A man that had a wife with such a wit, he might say, “Wit, whither
wilt?”

ROSALIND.
Nay, you might keep that check for it till you met your wife’s wit
going to your neighbour’s bed.

ORLANDO.
And what wit could wit have to excuse that?

ROSALIND.
Marry, to say she came to seek you there. You shall never take her
without her answer unless you take her without her tongue. O, that
woman that cannot make her fault her husband’s occasion, let her never
nurse her child herself, for she will breed it like a fool.

ORLANDO.
For these two hours, Rosalind, I will leave thee.

ROSALIND.
Alas, dear love, I cannot lack thee two hours.

ORLANDO.
I must attend the Duke at dinner. By two o’clock I will be with thee
again.

ROSALIND.
Ay, go your ways, go your ways. I knew what you would prove. My friends
told me as much, and I thought no less. That flattering tongue of yours
won me. ’Tis but one cast away, and so, come death! Two o’clock is your
hour?

ORLANDO.
Ay, sweet Rosalind.

ROSALIND.
By my troth, and in good earnest, and so God mend me, and by all pretty
oaths that are not dangerous, if you break one jot of your promise or
come one minute behind your hour, I will think you the most pathetical
break-promise, and the most hollow lover, and the most unworthy of her
you call Rosalind that may be chosen out of the gross band of the
unfaithful. Therefore beware my censure, and keep your promise.

ORLANDO.
With no less religion than if thou wert indeed my Rosalind. So, adieu.

ROSALIND.
Well, Time is the old justice that examines all such offenders, and let
time try. Adieu.

[_Exit Orlando._]

CELIA.
You have simply misused our sex in your love-prate! We must have your
doublet and hose plucked over your head and show the world what the
bird hath done to her own nest.

ROSALIND.
O coz, coz, coz, my pretty little coz, that thou didst know how many
fathom deep I am in love! But it cannot be sounded; my affection hath
an unknown bottom, like the Bay of Portugal.

CELIA.
Or rather, bottomless, that as fast as you pour affection in, it runs
out.

ROSALIND.
No, that same wicked bastard of Venus, that was begot of thought,
conceived of spleen, and born of madness, that blind rascally boy that
abuses everyone’s eyes because his own are out, let him be judge how
deep I am in love. I’ll tell thee, Aliena, I cannot be out of the sight
of Orlando. I’ll go find a shadow and sigh till he come.

CELIA.
And I’ll sleep.

[_Exeunt._]

SCENE II. Another part of the Forest

Enter Jaques and Lords, like foresters.

JAQUES.
Which is he that killed the deer?

FIRST LORD.
Sir, it was I.

JAQUES.
Let’s present him to the Duke, like a Roman conqueror, and it would do
well to set the deer’s horns upon his head for a branch of victory.
Have you no song, forester, for this purpose?

SECOND LORD.
Yes, sir.

JAQUES.
Sing it. ’Tis no matter how it be in tune, so it make noise enough.

SONG

SECOND LORD.
[_Sings_.]
  What shall he have that killed the deer?
        His leather skin and horns to wear.
                  Then sing him home:
  [_The rest shall bear this burden_.]
            Take thou no scorn to wear the horn.
            It was a crest ere thou wast born.
                  Thy father’s father wore it
                  And thy father bore it.
            The horn, the horn, the lusty horn
            Is not a thing to laugh to scorn.

[_Exeunt._]

SCENE III. Another part of the Forest

Enter Rosalind and Celia.

ROSALIND.
How say you now? Is it not past two o’clock? And here much Orlando.

CELIA.
I warrant you, with pure love and troubled brain he hath ta’en his bow
and arrows and is gone forth to sleep.

Enter Silvius.

Look who comes here.

SILVIUS.
My errand is to you, fair youth.
My gentle Phoebe did bid me give you this.

[_Giving a letter._]

I know not the contents, but, as I guess
By the stern brow and waspish action
Which she did use as she was writing of it,
It bears an angry tenor. Pardon me,
I am but as a guiltless messenger.

ROSALIND.
Patience herself would startle at this letter
And play the swaggerer. Bear this, bear all!
She says I am not fair, that I lack manners;
She calls me proud, and that she could not love me,
Were man as rare as phoenix. ’Od’s my will,
Her love is not the hare that I do hunt.
Why writes she so to me? Well, shepherd, well,
This is a letter of your own device.

SILVIUS.
No, I protest, I know not the contents.
Phoebe did write it.

ROSALIND.
Come, come, you are a fool,
And turned into the extremity of love.
I saw her hand. She has a leathern hand,
A freestone-coloured hand. I verily did think
That her old gloves were on, but ’twas her hands.
She has a huswife’s hand—but that’s no matter.
I say she never did invent this letter;
This is a man’s invention, and his hand.

SILVIUS.
Sure, it is hers.

ROSALIND.
Why, ’tis a boisterous and a cruel style,
A style for challengers. Why, she defies me,
Like Turk to Christian. Women’s gentle brain
Could not drop forth such giant-rude invention,
Such Ethiop words, blacker in their effect
Than in their countenance. Will you hear the letter?

SILVIUS.
So please you, for I never heard it yet,
Yet heard too much of Phoebe’s cruelty.

ROSALIND.
She Phoebes me. Mark how the tyrant writes.

[_Reads._]

            _Art thou god to shepherd turned,
            That a maiden’s heart hath burned?_
Can a woman rail thus?

SILVIUS.
Call you this railing?

ROSALIND.
            _Why, thy godhead laid apart,
            Warr’st thou with a woman’s heart?_
Did you ever hear such railing?
            _Whiles the eye of man did woo me,
            That could do no vengeance to me._
Meaning me a beast.
            _If the scorn of your bright eyne
            Have power to raise such love in mine,
            Alack, in me what strange effect
            Would they work in mild aspect?
            Whiles you chid me, I did love,
            How then might your prayers move?
            He that brings this love to thee
            Little knows this love in me;
            And by him seal up thy mind,
            Whether that thy youth and kind
            Will the faithful offer take
            Of me, and all that I can make,
            Or else by him my love deny,
            And then I’ll study how to die._

SILVIUS.
Call you this chiding?

CELIA.
Alas, poor shepherd.

ROSALIND.
Do you pity him? No, he deserves no pity.—Wilt thou love such a woman?
What, to make thee an instrument and play false strains upon thee? Not
to be endured! Well, go your way to her, for I see love hath made thee
a tame snake, and say this to her: that if she love me, I charge her to
love thee; if she will not, I will never have her unless thou entreat
for her. If you be a true lover, hence, and not a word, for here comes
more company.

[_Exit Silvius._]

Enter Oliver.

OLIVER.
Good morrow, fair ones. Pray you, if you know,
Where in the purlieus of this forest stands
A sheepcote fenced about with olive trees?

CELIA.
West of this place, down in the neighbour bottom;
The rank of osiers, by the murmuring stream,
Left on your right hand, brings you to the place.
But at this hour the house doth keep itself.
There’s none within.

OLIVER.
If that an eye may profit by a tongue,
Then should I know you by description,
Such garments, and such years. “The boy is fair,
Of female favour, and bestows himself
Like a ripe sister; the woman low,
And browner than her brother.” Are not you
The owner of the house I did inquire for?

CELIA.
It is no boast, being asked, to say we are.

OLIVER.
Orlando doth commend him to you both,
And to that youth he calls his Rosalind
He sends this bloody napkin. Are you he?

ROSALIND.
I am. What must we understand by this?

OLIVER.
Some of my shame, if you will know of me
What man I am, and how, and why, and where
This handkerchief was stained.

CELIA.
I pray you tell it.

OLIVER.
When last the young Orlando parted from you,
He left a promise to return again
Within an hour, and pacing through the forest,
Chewing the food of sweet and bitter fancy,
Lo, what befell. He threw his eye aside,
And mark what object did present itself.
Under an oak, whose boughs were mossed with age
And high top bald with dry antiquity,
A wretched ragged man, o’ergrown with hair,
Lay sleeping on his back; about his neck
A green and gilded snake had wreathed itself,
Who with her head, nimble in threats, approached
The opening of his mouth. But suddenly,
Seeing Orlando, it unlinked itself
And with indented glides did slip away
Into a bush; under which bush’s shade
A lioness, with udders all drawn dry,
Lay couching, head on ground, with catlike watch
When that the sleeping man should stir. For ’tis
The royal disposition of that beast
To prey on nothing that doth seem as dead.
This seen, Orlando did approach the man
And found it was his brother, his elder brother.

CELIA.
O, I have heard him speak of that same brother,
And he did render him the most unnatural
That lived amongst men.

OLIVER.
And well he might so do,
For well I know he was unnatural.

ROSALIND.
But, to Orlando: did he leave him there,
Food to the sucked and hungry lioness?

OLIVER.
Twice did he turn his back and purposed so;
But kindness, nobler ever than revenge,
And nature, stronger than his just occasion,
Made him give battle to the lioness,
Who quickly fell before him; in which hurtling
From miserable slumber I awaked.

CELIA.
Are you his brother?

ROSALIND.
Was it you he rescued?

CELIA.
Was’t you that did so oft contrive to kill him?

OLIVER.
’Twas I; but ’tis not I. I do not shame
To tell you what I was, since my conversion
So sweetly tastes, being the thing I am.

ROSALIND.
But, for the bloody napkin?

OLIVER.
By and by.
When from the first to last betwixt us two
Tears our recountments had most kindly bathed—
As how I came into that desert place—
In brief, he led me to the gentle Duke,
Who gave me fresh array and entertainment,
Committing me unto my brother’s love,
Who led me instantly unto his cave,
There stripped himself, and here upon his arm
The lioness had torn some flesh away,
Which all this while had bled; and now he fainted,
And cried in fainting upon Rosalind.
Brief, I recovered him, bound up his wound,
And after some small space, being strong at heart,
He sent me hither, stranger as I am,
To tell this story, that you might excuse
His broken promise, and to give this napkin,
Dyed in his blood, unto the shepherd youth
That he in sport doth call his Rosalind.

[_Rosalind faints._]

CELIA.
Why, how now, Ganymede, sweet Ganymede!

OLIVER.
Many will swoon when they do look on blood.

CELIA.
There is more in it. Cousin—Ganymede!

OLIVER.
Look, he recovers.

ROSALIND.
I would I were at home.

CELIA.
We’ll lead you thither.
I pray you, will you take him by the arm?

OLIVER.
Be of good cheer, youth. You a man? You lack a man’s heart.

ROSALIND.
I do so, I confess it. Ah, sirrah, a body would think this was well
counterfeited. I pray you tell your brother how well I counterfeited.
Heigh-ho.

OLIVER.
This was not counterfeit. There is too great testimony in your
complexion that it was a passion of earnest.

ROSALIND.
Counterfeit, I assure you.

OLIVER.
Well then, take a good heart, and counterfeit to be a man.

ROSALIND.
So I do. But, i’ faith, I should have been a woman by right.

CELIA.
Come, you look paler and paler. Pray you draw homewards. Good sir, go
with us.

OLIVER.
That will I, for I must bear answer back
How you excuse my brother, Rosalind.

ROSALIND.
I shall devise something. But I pray you commend my counterfeiting to
him. Will you go?

[_Exeunt._]




ACT V

SCENE I. The Forest of Arden


Enter Touchstone and Audrey.

TOUCHSTONE.
We shall find a time, Audrey; patience, gentle Audrey.

AUDREY.
Faith, the priest was good enough, for all the old gentleman’s saying.

TOUCHSTONE.
A most wicked Sir Oliver, Audrey, a most vile Martext. But Audrey,
there is a youth here in the forest lays claim to you.

AUDREY.
Ay, I know who ’tis. He hath no interest in me in the world.

Enter William.

Here comes the man you mean.

TOUCHSTONE.
It is meat and drink to me to see a clown. By my troth, we that have
good wits have much to answer for. We shall be flouting; we cannot
hold.

WILLIAM.
Good ev’n, Audrey.

AUDREY.
God ye good ev’n, William.

WILLIAM.
And good ev’n to you, sir.

TOUCHSTONE.
Good ev’n, gentle friend. Cover thy head, cover thy head. Nay, prithee,
be covered. How old are you, friend?

WILLIAM.
Five-and-twenty, sir.

TOUCHSTONE.
A ripe age. Is thy name William?

WILLIAM.
William, sir.

TOUCHSTONE.
A fair name. Wast born i’ th’ forest here?

WILLIAM.
Ay, sir, I thank God.

TOUCHSTONE.
“Thank God.” A good answer. Art rich?

WILLIAM.
Faith, sir, so-so.

TOUCHSTONE.
“So-so” is good, very good, very excellent good. And yet it is not, it
is but so-so. Art thou wise?

WILLIAM.
Ay, sir, I have a pretty wit.

TOUCHSTONE.
Why, thou sayst well. I do now remember a saying: “The fool doth think
he is wise, but the wise man knows himself to be a fool.” The heathen
philosopher, when he had a desire to eat a grape, would open his lips
when he put it into his mouth, meaning thereby that grapes were made to
eat and lips to open. You do love this maid?

WILLIAM.
I do, sir.

TOUCHSTONE.
Give me your hand. Art thou learned?

WILLIAM.
No, sir.

TOUCHSTONE.
Then learn this of me: to have is to have. For it is a figure in
rhetoric that drink, being poured out of cup into a glass, by filling
the one doth empty the other. For all your writers do consent that
_ipse_ is “he.” Now, you are not _ipse_, for I am he.

WILLIAM.
Which he, sir?

TOUCHSTONE.
He, sir, that must marry this woman. Therefore, you clown,
abandon—which is in the vulgar, “leave”—the society—which in the
boorish is “company”—of this female—which in the common is “woman”;
which together is, abandon the society of this female, or, clown, thou
perishest; or, to thy better understanding, diest; or, to wit, I kill
thee, make thee away, translate thy life into death, thy liberty into
bondage. I will deal in poison with thee, or in bastinado, or in steel.
I will bandy with thee in faction; will o’errun thee with policy. I
will kill thee a hundred and fifty ways! Therefore tremble and depart.

AUDREY.
Do, good William.

WILLIAM.
God rest you merry, sir.

[_Exit._]

Enter Corin.

CORIN.
Our master and mistress seek you. Come away, away.

TOUCHSTONE.
Trip, Audrey, trip, Audrey! I attend, I attend.

[_Exeunt._]

SCENE II. Another part of the Forest

Enter Orlando and Oliver.

ORLANDO.
Is’t possible that on so little acquaintance you should like her? That
but seeing, you should love her? And loving woo? And wooing, she should
grant? And will you persever to enjoy her?

OLIVER.
Neither call the giddiness of it in question, the poverty of her, the
small acquaintance, my sudden wooing, nor her sudden consenting. But
say with me, I love Aliena; say with her that she loves me; consent
with both that we may enjoy each other. It shall be to your good, for
my father’s house and all the revenue that was old Sir Rowland’s will I
estate upon you, and here live and die a shepherd.

Enter Rosalind.

ORLANDO.
You have my consent. Let your wedding be tomorrow. Thither will I
invite the Duke and all’s contented followers. Go you and prepare
Aliena; for, look you, here comes my Rosalind.

ROSALIND.
God save you, brother.

OLIVER.
And you, fair sister.

[_Exit._]

ROSALIND.
O my dear Orlando, how it grieves me to see thee wear thy heart in a
scarf!

ORLANDO.
It is my arm.

ROSALIND.
I thought thy heart had been wounded with the claws of a lion.

ORLANDO.
Wounded it is, but with the eyes of a lady.

ROSALIND.
Did your brother tell you how I counterfeited to swoon when he showed
me your handkercher?

ORLANDO.
Ay, and greater wonders than that.

ROSALIND.
O, I know where you are. Nay, ’tis true. There was never anything so
sudden but the fight of two rams, and Caesar’s thrasonical brag of “I
came, saw and overcame.” For your brother and my sister no sooner met
but they looked; no sooner looked but they loved; no sooner loved but
they sighed; no sooner sighed but they asked one another the reason; no
sooner knew the reason but they sought the remedy; and in these degrees
have they made pair of stairs to marriage, which they will climb
incontinent, or else be incontinent before marriage. They are in the
very wrath of love, and they will together. Clubs cannot part them.

ORLANDO.
They shall be married tomorrow, and I will bid the Duke to the nuptial.
But O, how bitter a thing it is to look into happiness through another
man’s eyes! By so much the more shall I tomorrow be at the height of
heart-heaviness, by how much I shall think my brother happy in having
what he wishes for.

ROSALIND.
Why, then, tomorrow I cannot serve your turn for Rosalind?

ORLANDO.
I can live no longer by thinking.

ROSALIND.
I will weary you then no longer with idle talking. Know of me then—for
now I speak to some purpose—that I know you are a gentleman of good
conceit. I speak not this that you should bear a good opinion of my
knowledge, insomuch I say I know you are. Neither do I labour for a
greater esteem than may in some little measure draw a belief from you,
to do yourself good, and not to grace me. Believe then, if you please,
that I can do strange things. I have, since I was three year old,
conversed with a magician, most profound in his art and yet not
damnable. If you do love Rosalind so near the heart as your gesture
cries it out, when your brother marries Aliena shall you marry her. I
know into what straits of fortune she is driven and it is not
impossible to me, if it appear not inconvenient to you, to set her
before your eyes tomorrow, human as she is, and without any danger.

ORLANDO.
Speak’st thou in sober meanings?

ROSALIND.
By my life, I do, which I tender dearly, though I say I am a magician.
Therefore put you in your best array, bid your friends; for if you will
be married tomorrow, you shall, and to Rosalind, if you will.

Enter Silvius and Phoebe.

Look, here comes a lover of mine and a lover of hers.

PHOEBE.
Youth, you have done me much ungentleness
To show the letter that I writ to you.

ROSALIND.
I care not if I have; it is my study
To seem despiteful and ungentle to you.
You are there followed by a faithful shepherd.
Look upon him, love him; he worships you.

PHOEBE.
Good shepherd, tell this youth what ’tis to love.

SILVIUS.
It is to be all made of sighs and tears,
And so am I for Phoebe.

PHOEBE.
And I for Ganymede.

ORLANDO.
And I for Rosalind.

ROSALIND.
And I for no woman.

SILVIUS.
It is to be all made of faith and service,
And so am I for Phoebe.

PHOEBE.
And I for Ganymede.

ORLANDO.
And I for Rosalind.

ROSALIND.
And I for no woman.

SILVIUS.
It is to be all made of fantasy,
All made of passion, and all made of wishes,
All adoration, duty, and observance,
All humbleness, all patience, and impatience,
All purity, all trial, all observance,
And so am I for Phoebe.

PHOEBE.
And so am I for Ganymede.

ORLANDO.
And so am I for Rosalind.

ROSALIND.
And so am I for no woman.

PHOEBE.
[_To Rosalind_.] If this be so, why blame you me to love you?

SILVIUS.
[_To Phoebe_.] If this be so, why blame you me to love you?

ORLANDO.
If this be so, why blame you me to love you?

ROSALIND.
Why do you speak too, “Why blame you me to love you?”

ORLANDO.
To her that is not here, nor doth not hear.

ROSALIND.
Pray you, no more of this, ’tis like the howling of Irish wolves
against the moon.
[_to Silvius_.] I will help you if I can.
[_to Phoebe_.] I would love you if I could.—Tomorrow meet me all
together.
[_to Phoebe_.] I will marry you, if ever I marry woman, and I’ll be
married tomorrow.
[_to Orlando_.] I will satisfy you if ever I satisfied man, and you
shall be married tomorrow.
[_to Silvius_.] I will content you, if what pleases you contents you,
and you shall be married tomorrow.
[_to Orlando_.] As you love Rosalind, meet.
[_to Silvius_.] As you love Phoebe, meet.—And as I love no woman, I’ll
meet. So fare you well. I have left you commands.

SILVIUS.
I’ll not fail, if I live.

PHOEBE.
Nor I.

ORLANDO.
Nor I.

[_Exeunt._]

SCENE III. Another part of the Forest

Enter Touchstone and Audrey.

TOUCHSTONE.
Tomorrow is the joyful day, Audrey, tomorrow will we be married.

AUDREY.
I do desire it with all my heart; and I hope it is no dishonest desire
to desire to be a woman of the world.

Enter two Pages.

Here come two of the banished Duke’s pages.

FIRST PAGE.
Well met, honest gentleman.

TOUCHSTONE.
By my troth, well met. Come sit, sit, and a song.

SECOND PAGE.
We are for you, sit i’ th’ middle.

FIRST PAGE.
Shall we clap into’t roundly, without hawking or spitting or saying we
are hoarse, which are the only prologues to a bad voice?

SECOND PAGE.
I’faith, i’faith, and both in a tune like two gipsies on a horse.

                        SONG

PAGES.
[_Sing_.]
    It was a lover and his lass,
        With a hey, and a ho, and a hey nonino,
    That o’er the green cornfield did pass
        In the spring-time, the only pretty ring time,
        When birds do sing, hey ding a ding, ding.
    Sweet lovers love the spring.

    Between the acres of the rye,
        With a hey, and a ho, and a hey nonino,
    These pretty country folks would lie,
        In the spring-time, the only pretty ring time,
        When birds do sing, hey ding a ding, ding.
    Sweet lovers love the spring.

    This carol they began that hour,
        With a hey, and a ho, and a hey nonino,
    How that a life was but a flower,
        In the spring-time, the only pretty ring time,
        When birds do sing, hey ding a ding, ding.
    Sweet lovers love the spring.

    And therefore take the present time,
        With a hey, and a ho, and a hey nonino,
    For love is crowned with the prime,
        In the spring-time, the only pretty ring time,
        When birds do sing, hey ding a ding, ding.
    Sweet lovers love the spring.

TOUCHSTONE
Truly, young gentlemen, though there was no great matter in the ditty,
yet the note was very untuneable.

FIRST PAGE.
You are deceived, sir, we kept time, we lost not our time.

TOUCHSTONE.
By my troth, yes. I count it but time lost to hear such a foolish song.
God be wi’ you, and God mend your voices. Come, Audrey.

[_Exeunt._]

SCENE IV. Another part of the Forest

Enter Duke Senior, Amiens, Jaques, Orlando, Oliver and Celia.

DUKE SENIOR.
Dost thou believe, Orlando, that the boy
Can do all this that he hath promised?

ORLANDO.
I sometimes do believe and sometimes do not,
As those that fear they hope, and know they fear.

Enter Rosalind, Silvius and Phoebe.

ROSALIND.
Patience once more whiles our compact is urged.
[_To the Duke._] You say, if I bring in your Rosalind,
You will bestow her on Orlando here?

DUKE SENIOR.
That would I, had I kingdoms to give with her.

ROSALIND.
[_To Orlando_.] And you say you will have her when I bring her?

ORLANDO.
That would I, were I of all kingdoms king.

ROSALIND.
[_To Phoebe_.] You say you’ll marry me if I be willing?

PHOEBE.
That will I, should I die the hour after.

ROSALIND.
But if you do refuse to marry me,
You’ll give yourself to this most faithful shepherd?

PHOEBE.
So is the bargain.

ROSALIND.
[_To Silvius_.] You say that you’ll have Phoebe if she will?

SILVIUS.
Though to have her and death were both one thing.

ROSALIND.
I have promised to make all this matter even.
Keep you your word, O Duke, to give your daughter,
You yours, Orlando, to receive his daughter.
Keep your word, Phoebe, that you’ll marry me,
Or else, refusing me, to wed this shepherd.
Keep your word, Silvius, that you’ll marry her
If she refuse me. And from hence I go
To make these doubts all even.

[_Exeunt Rosalind and Celia._]

DUKE SENIOR.
I do remember in this shepherd boy
Some lively touches of my daughter’s favour.

ORLANDO.
My lord, the first time that I ever saw him
Methought he was a brother to your daughter.
But, my good lord, this boy is forest-born
And hath been tutored in the rudiments
Of many desperate studies by his uncle,
Whom he reports to be a great magician,
Obscured in the circle of this forest.

Enter Touchstone and Audrey.

JAQUES.
There is sure another flood toward, and these couples are coming to the
ark. Here comes a pair of very strange beasts, which in all tongues are
called fools.

TOUCHSTONE.
Salutation and greeting to you all.

JAQUES.
Good my lord, bid him welcome. This is the motley-minded gentleman that
I have so often met in the forest. He hath been a courtier, he swears.

TOUCHSTONE.
If any man doubt that, let him put me to my purgation. I have trod a
measure; I have flattered a lady; I have been politic with my friend,
smooth with mine enemy; I have undone three tailors; I have had four
quarrels, and like to have fought one.

JAQUES.
And how was that ta’en up?

TOUCHSTONE.
Faith, we met, and found the quarrel was upon the seventh cause.

JAQUES.
How seventh cause?—Good my lord, like this fellow?

DUKE SENIOR.
I like him very well.

TOUCHSTONE.
God ’ild you, sir, I desire you of the like. I press in here, sir,
amongst the rest of the country copulatives, to swear and to forswear
according as marriage binds and blood breaks. A poor virgin, sir, an
ill-favoured thing, sir, but mine own; a poor humour of mine, sir, to
take that that no man else will. Rich honesty dwells like a miser, sir,
in a poor house, as your pearl in your foul oyster.

DUKE SENIOR.
By my faith, he is very swift and sententious.

TOUCHSTONE.
According to the fool’s bolt, sir, and such dulcet diseases.

JAQUES.
But, for the seventh cause. How did you find the quarrel on the seventh
cause?

TOUCHSTONE.
Upon a lie seven times removed—bear your body more seeming, Audrey—as
thus, sir. I did dislike the cut of a certain courtier’s beard. He sent
me word if I said his beard was not cut well, he was in the mind it
was. This is called the “retort courteous”. If I sent him word again it
was not well cut, he would send me word he cut it to please himself.
This is called the “quip modest”. If again it was not well cut, he
disabled my judgement. This is called the “reply churlish”. If again it
was not well cut, he would answer I spake not true. This is called the
“reproof valiant”. If again it was not well cut, he would say I lie.
This is called the “countercheck quarrelsome”, and so, to the “lie
circumstantial”, and the “lie direct”.

JAQUES.
And how oft did you say his beard was not well cut?

TOUCHSTONE.
I durst go no further than the lie circumstantial, nor he durst not
give me the lie direct; and so we measured swords and parted.

JAQUES.
Can you nominate in order now the degrees of the lie?

TOUCHSTONE.
O sir, we quarrel in print, by the book, as you have books for good
manners. I will name you the degrees: the first, the retort courteous;
the second, the quip modest; the third, the reply churlish; the fourth,
the reproof valiant; the fifth, the countercheck quarrelsome; the
sixth, the lie with circumstance; the seventh, the lie direct. All
these you may avoid but the lie direct and you may avoid that too with
an “if”. I knew when seven justices could not take up a quarrel, but
when the parties were met themselves, one of them thought but of an
“if”, as, “if you said so, then I said so;” and they shook hands, and
swore brothers. Your “if” is the only peacemaker; much virtue in “if.”

JAQUES.
Is not this a rare fellow, my lord? He’s as good at anything, and yet a
fool.

DUKE SENIOR.
He uses his folly like a stalking-horse, and under the presentation of
that he shoots his wit.

Enter Hymen, Rosalind in woman’s clothes, and Celia. Still music.

HYMEN.
    Then is there mirth in heaven
    When earthly things made even
        Atone together.
    Good Duke, receive thy daughter.
    Hymen from heaven brought her,
        Yea, brought her hither,
    That thou mightst join her hand with his,
    Whose heart within his bosom is.

ROSALIND.
[_To Duke Senior_.] To you I give myself, for I am yours.
[_To Orlando_.] To you I give myself, for I am yours.

DUKE SENIOR.
If there be truth in sight, you are my daughter.

ORLANDO.
If there be truth in sight, you are my Rosalind.

PHOEBE.
If sight and shape be true,
Why then, my love adieu.

ROSALIND.
[_To Duke Senior_.] I’ll have no father, if you be not he.
[_To Orlando_.] I’ll have no husband, if you be not he.
[_To Phoebe_.] Nor ne’er wed woman, if you be not she.

HYMEN.
    Peace, ho! I bar confusion.
    ’Tis I must make conclusion
        Of these most strange events.
    Here’s eight that must take hands
    To join in Hymen’s bands,
        If truth holds true contents.
[_To Orlando and Rosalind_.] You and you no cross shall part.
[_To Celia and Oliver_.] You and you are heart in heart.
[_To Phoebe_.] You to his love must accord
Or have a woman to your lord.
[_To Audrey and Touchstone_.] You and you are sure together
As the winter to foul weather.
Whiles a wedlock hymn we sing,
Feed yourselves with questioning,
That reason wonder may diminish
How thus we met, and these things finish.

                         SONG
      Wedding is great Juno’s crown,
            O blessed bond of board and bed.
      ’Tis Hymen peoples every town,
            High wedlock then be honoured.
      Honour, high honour, and renown
      To Hymen, god of every town.

DUKE SENIOR.
O my dear niece, welcome thou art to me
Even daughter, welcome in no less degree.

PHOEBE.
[_To Silvius_.] I will not eat my word, now thou art mine,
Thy faith my fancy to thee doth combine.

Enter Jaques de Boys.

JAQUES DE BOYS.
Let me have audience for a word or two.
I am the second son of old Sir Rowland,
That bring these tidings to this fair assembly.
Duke Frederick, hearing how that every day
Men of great worth resorted to this forest,
Addressed a mighty power, which were on foot
In his own conduct, purposely to take
His brother here and put him to the sword;
And to the skirts of this wild wood he came,
Where, meeting with an old religious man,
After some question with him, was converted
Both from his enterprise and from the world,
His crown bequeathing to his banished brother,
And all their lands restored to them again
That were with him exiled. This to be true
I do engage my life.

DUKE SENIOR.
Welcome, young man.
Thou offer’st fairly to thy brother’s wedding:
To one his lands withheld, and to the other
A land itself at large, a potent dukedom.
First, in this forest let us do those ends
That here were well begun and well begot;
And after, every of this happy number
That have endured shrewd days and nights with us
Shall share the good of our returned fortune,
According to the measure of their states.
Meantime, forget this new-fall’n dignity,
And fall into our rustic revelry.
Play, music! And you brides and bridegrooms all,
With measure heaped in joy to th’ measures fall.

JAQUES.
Sir, by your patience. If I heard you rightly,
The Duke hath put on a religious life
And thrown into neglect the pompous court.

JAQUES DE BOYS.
He hath.

JAQUES.
To him will I. Out of these convertites
There is much matter to be heard and learned.
[_To Duke Senior_.] You to your former honour I bequeath;
Your patience and your virtue well deserves it.
[_To Orlando_.] You to a love that your true faith doth merit.
[_To Oliver_.] You to your land, and love, and great allies.
[_To Silvius_.] You to a long and well-deserved bed.
[_To Touchstone_.] And you to wrangling, for thy loving voyage
Is but for two months victualled.—So to your pleasures,
I am for other than for dancing measures.

DUKE SENIOR.
Stay, Jaques, stay.

JAQUES.
To see no pastime, I. What you would have
I’ll stay to know at your abandoned cave.

[_Exit._]

DUKE SENIOR.
Proceed, proceed! We will begin these rites,
As we do trust they’ll end, in true delights.

[_Dance. Exeunt all but Rosalind._]

EPILOGUE

ROSALIND.
It is not the fashion to see the lady the epilogue, but it is no more
unhandsome than to see the lord the prologue. If it be true that good
wine needs no bush, ’tis true that a good play needs no epilogue. Yet
to good wine they do use good bushes, and good plays prove the better
by the help of good epilogues. What a case am I in then, that am
neither a good epilogue nor cannot insinuate with you in the behalf of
a good play! I am not furnished like a beggar; therefore to beg will
not become me. My way is to conjure you, and I’ll begin with the women.
I charge you, O women, for the love you bear to men, to like as much of
this play as please you. And I charge you, O men, for the love you bear
to women—as I perceive by your simpering, none of you hates them—that
between you and the women the play may please. If I were a woman, I
would kiss as many of you as had beards that pleased me, complexions
that liked me, and breaths that I defied not. And I am sure as many as
have good beards, or good faces, or sweet breaths will for my kind
offer, when I make curtsy, bid me farewell.

[_Exit._]




THE COMEDY OF ERRORS




Contents

ACT I
Scene I. A hall in the Duke’s palace
Scene II. A public place

ACT II
Scene I. A public place
Scene II. The same

ACT III
Scene I. The same
Scene II. The same

ACT IV
Scene I. The same
Scene II. The same
Scene III. The same
Scene IV. The same

ACT V
Scene I. The same




Dramatis Personæ

SOLINUS, Duke of Ephesus.
EGEON, a Merchant of Syracuse.

ANTIPHOLUS OF EPHESUS, Twin brothers and sons to Egeon and
ANTIPHOLUS OF SYRACUSE, Emilia, but unknown to each other.

DROMIO OF EPHESUS, Twin brothers, and attendants on
DROMIO OF SYRACUSE, the two Antipholuses.

BALTHASAR, a Merchant.
ANGELO, a Goldsmith.
A MERCHANT, friend to Antipholus of Syracuse.
PINCH, a Schoolmaster and a Conjurer.
EMILIA, Wife to Egeon, an Abbess at Ephesus.
ADRIANA, Wife to Antipholus of Ephesus.
LUCIANA, her Sister.
LUCE, her Servant.
A COURTESAN
Messenger, Jailer, Officers, Attendants

SCENE: Ephesus


ACT I

SCENE I. A hall in the Duke’s palace

 Enter Duke, Egeon, Jailer, Officers and other Attendants.

EGEON.
Proceed, Solinus, to procure my fall,
And by the doom of death end woes and all.

DUKE.
Merchant of Syracusa, plead no more.
I am not partial to infringe our laws.
The enmity and discord which of late
Sprung from the rancorous outrage of your Duke
To merchants, our well-dealing countrymen,
Who, wanting guilders to redeem their lives,
Have seal’d his rigorous statutes with their bloods,
Excludes all pity from our threat’ning looks.
For since the mortal and intestine jars
’Twixt thy seditious countrymen and us,
It hath in solemn synods been decreed,
Both by the Syracusians and ourselves,
To admit no traffic to our adverse towns;
Nay more, if any born at Ephesus
Be seen at Syracusian marts and fairs;
Again, if any Syracusian born
Come to the bay of Ephesus, he dies,
His goods confiscate to the Duke’s dispose,
Unless a thousand marks be levied
To quit the penalty and to ransom him.
Thy substance, valued at the highest rate,
Cannot amount unto a hundred marks;
Therefore by law thou art condemn’d to die.

EGEON.
Yet this my comfort; when your words are done,
My woes end likewise with the evening sun.

DUKE.
Well, Syracusian, say in brief the cause
Why thou departedst from thy native home,
And for what cause thou cam’st to Ephesus.

EGEON.
A heavier task could not have been impos’d
Than I to speak my griefs unspeakable;
Yet, that the world may witness that my end
Was wrought by nature, not by vile offence,
I’ll utter what my sorrow gives me leave.
In Syracusa was I born, and wed
Unto a woman happy but for me,
And by me, had not our hap been bad.
With her I liv’d in joy; our wealth increas’d
By prosperous voyages I often made
To Epidamnum, till my factor’s death,
And the great care of goods at random left,
Drew me from kind embracements of my spouse;
From whom my absence was not six months old
Before herself (almost at fainting under
The pleasing punishment that women bear)
Had made provision for her following me,
And soon and safe arrived where I was.
There had she not been long but she became
A joyful mother of two goodly sons,
And, which was strange, the one so like the other
As could not be distinguish’d but by names.
That very hour, and in the self-same inn,
A mean woman was delivered
Of such a burden, male twins, both alike.
Those, for their parents were exceeding poor,
I bought, and brought up to attend my sons.
My wife, not meanly proud of two such boys,
Made daily motions for our home return.
Unwilling I agreed; alas, too soon
We came aboard.
A league from Epidamnum had we sail’d
Before the always-wind-obeying deep
Gave any tragic instance of our harm;
But longer did we not retain much hope;
For what obscured light the heavens did grant
Did but convey unto our fearful minds
A doubtful warrant of immediate death,
Which though myself would gladly have embrac’d,
Yet the incessant weepings of my wife,
Weeping before for what she saw must come,
And piteous plainings of the pretty babes,
That mourn’d for fashion, ignorant what to fear,
Forc’d me to seek delays for them and me.
And this it was (for other means was none).
The sailors sought for safety by our boat,
And left the ship, then sinking-ripe, to us.
My wife, more careful for the latter-born,
Had fast’ned him unto a small spare mast,
Such as sea-faring men provide for storms.
To him one of the other twins was bound,
Whilst I had been like heedful of the other.
The children thus dispos’d, my wife and I,
Fixing our eyes on whom our care was fix’d,
Fast’ned ourselves at either end the mast,
And, floating straight, obedient to the stream,
Was carried towards Corinth, as we thought.
At length the sun, gazing upon the earth,
Dispers’d those vapours that offended us,
And by the benefit of his wished light
The seas wax’d calm, and we discovered
Two ships from far, making amain to us,
Of Corinth that, of Epidaurus this.
But ere they came—O, let me say no more!
Gather the sequel by that went before.

DUKE.
Nay, forward, old man, do not break off so,
For we may pity, though not pardon thee.

EGEON.
O, had the gods done so, I had not now
Worthily term’d them merciless to us.
For, ere the ships could meet by twice five leagues,
We were encountered by a mighty rock,
Which being violently borne upon,
Our helpful ship was splitted in the midst;
So that, in this unjust divorce of us,
Fortune had left to both of us alike
What to delight in, what to sorrow for.
Her part, poor soul, seeming as burdened
With lesser weight, but not with lesser woe,
Was carried with more speed before the wind,
And in our sight they three were taken up
By fishermen of Corinth, as we thought.
At length another ship had seiz’d on us;
And, knowing whom it was their hap to save,
Gave healthful welcome to their ship-wrack’d guests,
And would have reft the fishers of their prey,
Had not their bark been very slow of sail;
And therefore homeward did they bend their course.
Thus have you heard me sever’d from my bliss,
That by misfortunes was my life prolong’d
To tell sad stories of my own mishaps.

DUKE.
And for the sake of them thou sorrowest for,
Do me the favour to dilate at full
What have befall’n of them and thee till now.

EGEON.
My youngest boy, and yet my eldest care,
At eighteen years became inquisitive
After his brother, and importun’d me
That his attendant, so his case was like,
Reft of his brother, but retain’d his name,
Might bear him company in the quest of him;
Whom whilst I laboured of a love to see,
I hazarded the loss of whom I lov’d.
Five summers have I spent in farthest Greece,
Roaming clean through the bounds of Asia,
And, coasting homeward, came to Ephesus,
Hopeless to find, yet loath to leave unsought
Or that or any place that harbours men.
But here must end the story of my life;
And happy were I in my timely death,
Could all my travels warrant me they live.

DUKE.
Hapless Egeon, whom the fates have mark’d
To bear the extremity of dire mishap;
Now, trust me, were it not against our laws,
Against my crown, my oath, my dignity,
Which princes, would they, may not disannul,
My soul should sue as advocate for thee.
But though thou art adjudged to the death,
And passed sentence may not be recall’d
But to our honour’s great disparagement,
Yet will I favour thee in what I can.
Therefore, merchant, I’ll limit thee this day
To seek thy health by beneficial help.
Try all the friends thou hast in Ephesus;
Beg thou, or borrow, to make up the sum,
And live; if no, then thou art doom’d to die.
Jailer, take him to thy custody.

JAILER.
I will, my lord.

EGEON.
Hopeless and helpless doth Egeon wend,
But to procrastinate his lifeless end.

 [_Exeunt._]

SCENE II. A public place

 Enter Antipholus and Dromio of Syracuse and a Merchant.

MERCHANT.
Therefore give out you are of Epidamnum,
Lest that your goods too soon be confiscate.
This very day a Syracusian merchant
Is apprehended for arrival here,
And, not being able to buy out his life,
According to the statute of the town
Dies ere the weary sun set in the west.
There is your money that I had to keep.

ANTIPHOLUS OF SYRACUSE.
Go bear it to the Centaur, where we host,
And stay there, Dromio, till I come to thee.
Within this hour it will be dinnertime;
Till that, I’ll view the manners of the town,
Peruse the traders, gaze upon the buildings,
And then return and sleep within mine inn,
For with long travel I am stiff and weary.
Get thee away.

DROMIO OF SYRACUSE.
Many a man would take you at your word,
And go indeed, having so good a mean.

 [_Exit Dromio._]

ANTIPHOLUS OF SYRACUSE.
A trusty villain, sir, that very oft,
When I am dull with care and melancholy,
Lightens my humour with his merry jests.
What, will you walk with me about the town,
And then go to my inn and dine with me?

MERCHANT.
I am invited, sir, to certain merchants,
Of whom I hope to make much benefit.
I crave your pardon. Soon, at five o’clock,
Please you, I’ll meet with you upon the mart,
And afterward consort you till bedtime.
My present business calls me from you now.

ANTIPHOLUS OF SYRACUSE.
Farewell till then: I will go lose myself,
And wander up and down to view the city.

MERCHANT.
Sir, I commend you to your own content.

 [_Exit Merchant._]

ANTIPHOLUS OF SYRACUSE.
He that commends me to mine own content
Commends me to the thing I cannot get.
I to the world am like a drop of water
That in the ocean seeks another drop,
Who, failing there to find his fellow forth,
Unseen, inquisitive, confounds himself.
So I, to find a mother and a brother,
In quest of them, unhappy, lose myself.

 Enter Dromio of Ephesus.

Here comes the almanac of my true date.
What now? How chance thou art return’d so soon?

DROMIO OF EPHESUS.
Return’d so soon? rather approach’d too late.
The capon burns, the pig falls from the spit;
The clock hath strucken twelve upon the bell;
My mistress made it one upon my cheek.
She is so hot because the meat is cold;
The meat is cold because you come not home;
You come not home because you have no stomach;
You have no stomach, having broke your fast;
But we that know what ’tis to fast and pray,
Are penitent for your default today.

ANTIPHOLUS OF SYRACUSE.
Stop in your wind, sir, tell me this, I pray:
Where have you left the money that I gave you?

DROMIO OF EPHESUS.
O, sixpence that I had o’ Wednesday last
To pay the saddler for my mistress’ crupper:
The saddler had it, sir, I kept it not.

ANTIPHOLUS OF SYRACUSE.
I am not in a sportive humour now.
Tell me, and dally not, where is the money?
We being strangers here, how dar’st thou trust
So great a charge from thine own custody?

DROMIO OF EPHESUS.
I pray you jest, sir, as you sit at dinner:
I from my mistress come to you in post;
If I return, I shall be post indeed,
For she will score your fault upon my pate.
Methinks your maw, like mine, should be your clock,
And strike you home without a messenger.

ANTIPHOLUS OF SYRACUSE.
Come, Dromio, come, these jests are out of season,
Reserve them till a merrier hour than this.
Where is the gold I gave in charge to thee?

DROMIO OF EPHESUS.
To me, sir? why, you gave no gold to me!

ANTIPHOLUS OF SYRACUSE.
Come on, sir knave, have done your foolishness,
And tell me how thou hast dispos’d thy charge.

DROMIO OF EPHESUS.
My charge was but to fetch you from the mart
Home to your house, the Phoenix, sir, to dinner.
My mistress and her sister stay for you.

ANTIPHOLUS OF SYRACUSE.
Now, as I am a Christian, answer me
In what safe place you have bestow’d my money,
Or I shall break that merry sconce of yours
That stands on tricks when I am undispos’d;
Where is the thousand marks thou hadst of me?

DROMIO OF EPHESUS.
I have some marks of yours upon my pate,
Some of my mistress’ marks upon my shoulders,
But not a thousand marks between you both.
If I should pay your worship those again,
Perchance you will not bear them patiently.

ANTIPHOLUS OF SYRACUSE.
Thy mistress’ marks? what mistress, slave, hast thou?

DROMIO OF EPHESUS.
Your worship’s wife, my mistress at the Phoenix;
She that doth fast till you come home to dinner,
And prays that you will hie you home to dinner.

ANTIPHOLUS OF SYRACUSE.
What, wilt thou flout me thus unto my face,
Being forbid? There, take you that, sir knave.

DROMIO OF EPHESUS.
What mean you, sir? for God’s sake hold your hands.
Nay, an you will not, sir, I’ll take my heels.

 [_Exit Dromio._]

ANTIPHOLUS OF SYRACUSE.
Upon my life, by some device or other
The villain is o’er-raught of all my money.
They say this town is full of cozenage,
As nimble jugglers that deceive the eye,
Dark-working sorcerers that change the mind,
Soul-killing witches that deform the body,
Disguised cheaters, prating mountebanks,
And many such-like liberties of sin:
If it prove so, I will be gone the sooner.
I’ll to the Centaur to go seek this slave.
I greatly fear my money is not safe.

 [_Exit._]



ACT II

SCENE I. A public place

 Enter Adriana, wife to Antipholus (of Ephesus) with Luciana her
 sister.

ADRIANA.
Neither my husband nor the slave return’d
That in such haste I sent to seek his master?
Sure, Luciana, it is two o’clock.

LUCIANA.
Perhaps some merchant hath invited him,
And from the mart he’s somewhere gone to dinner.
Good sister, let us dine, and never fret;
A man is master of his liberty;
Time is their master, and when they see time,
They’ll go or come. If so, be patient, sister.

ADRIANA.
Why should their liberty than ours be more?

LUCIANA.
Because their business still lies out o’ door.

ADRIANA.
Look when I serve him so, he takes it ill.

LUCIANA.
O, know he is the bridle of your will.

ADRIANA.
There’s none but asses will be bridled so.

LUCIANA.
Why, headstrong liberty is lash’d with woe.
There’s nothing situate under heaven’s eye
But hath his bound in earth, in sea, in sky.
The beasts, the fishes, and the winged fowls
Are their males’ subjects, and at their controls.
Man, more divine, the masters of all these,
Lord of the wide world and wild wat’ry seas,
Indued with intellectual sense and souls,
Of more pre-eminence than fish and fowls,
Are masters to their females, and their lords:
Then let your will attend on their accords.

ADRIANA.
This servitude makes you to keep unwed.

LUCIANA.
Not this, but troubles of the marriage-bed.

ADRIANA.
But, were you wedded, you would bear some sway.

LUCIANA.
Ere I learn love, I’ll practise to obey.

ADRIANA.
How if your husband start some other where?

LUCIANA.
Till he come home again, I would forbear.

ADRIANA.
Patience unmov’d! No marvel though she pause;
They can be meek that have no other cause.
A wretched soul bruis’d with adversity,
We bid be quiet when we hear it cry;
But were we burd’ned with like weight of pain,
As much, or more, we should ourselves complain:
So thou, that hast no unkind mate to grieve thee,
With urging helpless patience would relieve me:
But if thou live to see like right bereft,
This fool-begg’d patience in thee will be left.

LUCIANA.
Well, I will marry one day, but to try.
Here comes your man, now is your husband nigh.

 Enter Dromio of Ephesus.

ADRIANA.
Say, is your tardy master now at hand?

DROMIO OF EPHESUS.
Nay, he’s at two hands with me, and that my two ears can witness.

ADRIANA.
Say, didst thou speak with him? know’st thou his mind?

DROMIO OF EPHESUS.
Ay, ay, he told his mind upon mine ear.
Beshrew his hand, I scarce could understand it.

LUCIANA.
Spake he so doubtfully thou couldst not feel his meaning?

DROMIO OF EPHESUS.
Nay, he struck so plainly I could too well feel his blows; and withal
so doubtfully that I could scarce understand them.

ADRIANA.
But say, I prithee, is he coming home?
It seems he hath great care to please his wife.

DROMIO OF EPHESUS.
Why, mistress, sure my master is horn-mad.

ADRIANA.
Horn-mad, thou villain?

DROMIO OF EPHESUS.
I mean not cuckold-mad,
But sure he’s stark mad.
When I desir’d him to come home to dinner,
He ask’d me for a thousand marks in gold.
“’Tis dinner time,” quoth I. “My gold,” quoth he.
“Your meat doth burn” quoth I. “My gold,” quoth he.
“Will you come home?” quoth I. “My gold,” quoth he.
“Where is the thousand marks I gave thee, villain?”
“The pig” quoth I “is burn’d”. “My gold,” quoth he.
“My mistress, sir,” quoth I. “Hang up thy mistress;
I know not thy mistress; out on thy mistress!”

LUCIANA.
Quoth who?

DROMIO OF EPHESUS.
Quoth my master.
“I know,” quoth he, “no house, no wife, no mistress.”
So that my errand, due unto my tongue,
I thank him, I bare home upon my shoulders;
For, in conclusion, he did beat me there.

ADRIANA.
Go back again, thou slave, and fetch him home.

DROMIO OF EPHESUS.
Go back again, and be new beaten home?
For God’s sake, send some other messenger.

ADRIANA.
Back slave, or I will break thy pate across.

DROMIO OF EPHESUS.
And he will bless that cross with other beating.
Between you I shall have a holy head.

ADRIANA.
Hence, prating peasant. Fetch thy master home.

DROMIO OF EPHESUS.
Am I so round with you, as you with me,
That like a football you do spurn me thus?
You spurn me hence, and he will spurn me hither.
If I last in this service, you must case me in leather.

 [_Exit._]

LUCIANA.
Fie, how impatience loureth in your face.

ADRIANA.
His company must do his minions grace,
Whilst I at home starve for a merry look.
Hath homely age th’ alluring beauty took
From my poor cheek? then he hath wasted it.
Are my discourses dull? barren my wit?
If voluble and sharp discourse be marr’d,
Unkindness blunts it more than marble hard.
Do their gay vestments his affections bait?
That’s not my fault; he’s master of my state.
What ruins are in me that can be found
By him not ruin’d? Then is he the ground
Of my defeatures. My decayed fair
A sunny look of his would soon repair;
But, too unruly deer, he breaks the pale
And feeds from home; poor I am but his stale.

LUCIANA.
Self-harming jealousy! fie, beat it hence.

ADRIANA.
Unfeeling fools can with such wrongs dispense.
I know his eye doth homage otherwhere,
Or else what lets it but he would be here?
Sister, you know he promis’d me a chain;
Would that alone, a love he would detain,
So he would keep fair quarter with his bed.
I see the jewel best enamelled
Will lose his beauty; yet the gold bides still
That others touch, yet often touching will
Wear gold; and no man that hath a name
By falsehood and corruption doth it shame.
Since that my beauty cannot please his eye,
I’ll weep what’s left away, and weeping die.

LUCIANA.
How many fond fools serve mad jealousy!

 [_Exeunt._]

SCENE II. The same

 Enter Antipholus of Syracuse.

ANTIPHOLUS OF SYRACUSE.
The gold I gave to Dromio is laid up
Safe at the Centaur, and the heedful slave
Is wander’d forth in care to seek me out.
By computation and mine host’s report.
I could not speak with Dromio since at first
I sent him from the mart. See, here he comes.

 Enter Dromio of Syracuse.

How now, sir! is your merry humour alter’d?
As you love strokes, so jest with me again.
You know no Centaur? you receiv’d no gold?
Your mistress sent to have me home to dinner?
My house was at the Phoenix? Wast thou mad,
That thus so madly thou didst answer me?

DROMIO OF SYRACUSE.
What answer, sir? when spake I such a word?

ANTIPHOLUS OF SYRACUSE.
Even now, even here, not half an hour since.

DROMIO OF SYRACUSE.
I did not see you since you sent me hence,
Home to the Centaur with the gold you gave me.

ANTIPHOLUS OF SYRACUSE.
Villain, thou didst deny the gold’s receipt,
And told’st me of a mistress and a dinner,
For which I hope thou felt’st I was displeas’d.

DROMIO OF SYRACUSE.
I am glad to see you in this merry vein.
What means this jest, I pray you, master, tell me?

ANTIPHOLUS OF SYRACUSE.
Yea, dost thou jeer and flout me in the teeth?
Think’st thou I jest? Hold, take thou that, and that.

 [_Beats Dromio._]

DROMIO OF SYRACUSE.
Hold, sir, for God’s sake, now your jest is earnest.
Upon what bargain do you give it me?

ANTIPHOLUS OF SYRACUSE.
Because that I familiarly sometimes
Do use you for my fool, and chat with you,
Your sauciness will jest upon my love,
And make a common of my serious hours.
When the sun shines let foolish gnats make sport,
But creep in crannies when he hides his beams.
If you will jest with me, know my aspect,
And fashion your demeanour to my looks,
Or I will beat this method in your sconce.

DROMIO OF SYRACUSE.
Sconce, call you it? so you would leave battering, I had rather have it
a head. And you use these blows long, I must get a sconce for my head,
and ensconce it too, or else I shall seek my wit in my shoulders. But I
pray, sir, why am I beaten?

ANTIPHOLUS OF SYRACUSE.
Dost thou not know?

DROMIO OF SYRACUSE.
Nothing, sir, but that I am beaten.

ANTIPHOLUS OF SYRACUSE.
Shall I tell you why?

DROMIO OF SYRACUSE.
Ay, sir, and wherefore; for they say, every why hath a wherefore.

ANTIPHOLUS OF SYRACUSE.
Why, first, for flouting me; and then wherefore,
For urging it the second time to me.

DROMIO OF SYRACUSE.
Was there ever any man thus beaten out of season,
When in the why and the wherefore is neither rhyme nor reason?
Well, sir, I thank you.

ANTIPHOLUS OF SYRACUSE.
Thank me, sir, for what?

DROMIO OF SYRACUSE.
Marry, sir, for this something that you gave me for nothing.

ANTIPHOLUS OF SYRACUSE.
I’ll make you amends next, to give you nothing for something.
But say, sir, is it dinner-time?

DROMIO OF SYRACUSE.
No, sir; I think the meat wants that I have.

ANTIPHOLUS OF SYRACUSE.
In good time, sir, what’s that?

DROMIO OF SYRACUSE.
Basting.

ANTIPHOLUS OF SYRACUSE.
Well, sir, then ’twill be dry.

DROMIO OF SYRACUSE.
If it be, sir, I pray you eat none of it.

ANTIPHOLUS OF SYRACUSE.
Your reason?

DROMIO OF SYRACUSE.
Lest it make you choleric, and purchase me another dry basting.

ANTIPHOLUS OF SYRACUSE.
Well, sir, learn to jest in good time.
There’s a time for all things.

DROMIO OF SYRACUSE.
I durst have denied that before you were so choleric.

ANTIPHOLUS OF SYRACUSE.
By what rule, sir?

DROMIO OF SYRACUSE.
Marry, sir, by a rule as plain as the plain bald pate of Father Time
himself.

ANTIPHOLUS OF SYRACUSE.
Let’s hear it.

DROMIO OF SYRACUSE.
There’s no time for a man to recover his hair that grows bald by
nature.

ANTIPHOLUS OF SYRACUSE.
May he not do it by fine and recovery?

DROMIO OF SYRACUSE.
Yes, to pay a fine for a periwig, and recover the lost hair of another
man.

ANTIPHOLUS OF SYRACUSE.
Why is Time such a niggard of hair, being, as it is, so plentiful an
excrement?

DROMIO OF SYRACUSE.
Because it is a blessing that he bestows on beasts, and what he hath
scanted men in hair he hath given them in wit.

ANTIPHOLUS OF SYRACUSE.
Why, but there’s many a man hath more hair than wit.

DROMIO OF SYRACUSE.
Not a man of those but he hath the wit to lose his hair.

ANTIPHOLUS OF SYRACUSE.
Why, thou didst conclude hairy men plain dealers without wit.

DROMIO OF SYRACUSE.
The plainer dealer, the sooner lost. Yet he loseth it in a kind of
jollity.

ANTIPHOLUS OF SYRACUSE.
For what reason?

DROMIO OF SYRACUSE.
For two, and sound ones too.

ANTIPHOLUS OF SYRACUSE.
Nay, not sound, I pray you.

DROMIO OF SYRACUSE.
Sure ones, then.

ANTIPHOLUS OF SYRACUSE.
Nay, not sure, in a thing falsing.

DROMIO OF SYRACUSE.
Certain ones, then.

ANTIPHOLUS OF SYRACUSE.
Name them.

DROMIO OF SYRACUSE.
The one, to save the money that he spends in tiring; the other, that at
dinner they should not drop in his porridge.

ANTIPHOLUS OF SYRACUSE.
You would all this time have proved there is no time for all things.

DROMIO OF SYRACUSE.
Marry, and did, sir; namely, e’en no time to recover hair lost by
nature.

ANTIPHOLUS OF SYRACUSE.
But your reason was not substantial why there is no time to recover.

DROMIO OF SYRACUSE.
Thus I mend it: Time himself is bald, and therefore, to the world’s end
will have bald followers.

ANTIPHOLUS OF SYRACUSE.
I knew ’twould be a bald conclusion.
But soft! who wafts us yonder?

 Enter Adriana and Luciana.

ADRIANA.
Ay, ay, Antipholus, look strange and frown,
Some other mistress hath thy sweet aspects.
I am not Adriana, nor thy wife.
The time was once when thou unurg’d wouldst vow
That never words were music to thine ear,
That never object pleasing in thine eye,
That never touch well welcome to thy hand,
That never meat sweet-savour’d in thy taste,
Unless I spake, or look’d, or touch’d, or carv’d to thee.
How comes it now, my husband, O, how comes it,
That thou art then estranged from thyself?
Thyself I call it, being strange to me,
That, undividable, incorporate,
Am better than thy dear self’s better part.
Ah, do not tear away thyself from me;
For know, my love, as easy mayst thou fall
A drop of water in the breaking gulf,
And take unmingled thence that drop again
Without addition or diminishing,
As take from me thyself, and not me too.
How dearly would it touch thee to the quick,
Should’st thou but hear I were licentious?
And that this body, consecrate to thee,
By ruffian lust should be contaminate?
Wouldst thou not spit at me, and spurn at me,
And hurl the name of husband in my face,
And tear the stain’d skin off my harlot brow,
And from my false hand cut the wedding-ring,
And break it with a deep-divorcing vow?
I know thou canst; and therefore, see thou do it.
I am possess’d with an adulterate blot;
My blood is mingled with the crime of lust;
For if we two be one, and thou play false,
I do digest the poison of thy flesh,
Being strumpeted by thy contagion.
Keep then fair league and truce with thy true bed,
I live distain’d, thou undishonoured.

ANTIPHOLUS OF SYRACUSE.
Plead you to me, fair dame? I know you not.
In Ephesus I am but two hours old,
As strange unto your town as to your talk,
Who, every word by all my wit being scann’d,
Wants wit in all one word to understand.

LUCIANA.
Fie, brother, how the world is chang’d with you.
When were you wont to use my sister thus?
She sent for you by Dromio home to dinner.

ANTIPHOLUS OF SYRACUSE.
By Dromio?

DROMIO OF SYRACUSE.
By me?

ADRIANA.
By thee; and this thou didst return from him,
That he did buffet thee, and in his blows
Denied my house for his, me for his wife.

ANTIPHOLUS OF SYRACUSE.
Did you converse, sir, with this gentlewoman?
What is the course and drift of your compact?

DROMIO OF SYRACUSE.
I, sir? I never saw her till this time.

ANTIPHOLUS OF SYRACUSE.
Villain, thou liest, for even her very words
Didst thou deliver to me on the mart.

DROMIO OF SYRACUSE.
I never spake with her in all my life.

ANTIPHOLUS OF SYRACUSE.
How can she thus, then, call us by our names?
Unless it be by inspiration.

ADRIANA.
How ill agrees it with your gravity
To counterfeit thus grossly with your slave,
Abetting him to thwart me in my mood;
Be it my wrong, you are from me exempt,
But wrong not that wrong with a more contempt.
Come, I will fasten on this sleeve of thine.
Thou art an elm, my husband, I a vine,
Whose weakness, married to thy stronger state,
Makes me with thy strength to communicate:
If aught possess thee from me, it is dross,
Usurping ivy, brier, or idle moss,
Who all, for want of pruning, with intrusion
Infect thy sap, and live on thy confusion.

ANTIPHOLUS OF SYRACUSE.
To me she speaks; she moves me for her theme.
What, was I married to her in my dream?
Or sleep I now, and think I hear all this?
What error drives our eyes and ears amiss?
Until I know this sure uncertainty
I’ll entertain the offer’d fallacy.

LUCIANA.
Dromio, go bid the servants spread for dinner.

DROMIO OF SYRACUSE.
O, for my beads! I cross me for a sinner.
This is the fairy land; O spite of spites!
We talk with goblins, owls, and sprites;
If we obey them not, this will ensue:
They’ll suck our breath, or pinch us black and blue.

LUCIANA.
Why prat’st thou to thyself, and answer’st not?
Dromio, thou drone, thou snail, thou slug, thou sot.

DROMIO OF SYRACUSE.
I am transformed, master, am I not?

ANTIPHOLUS OF SYRACUSE.
I think thou art in mind, and so am I.

DROMIO OF SYRACUSE.
Nay, master, both in mind and in my shape.

ANTIPHOLUS OF SYRACUSE.
Thou hast thine own form.

DROMIO OF SYRACUSE.
No, I am an ape.

LUCIANA.
If thou art chang’d to aught, ’tis to an ass.

DROMIO OF SYRACUSE.
’Tis true; she rides me, and I long for grass.
’Tis so, I am an ass; else it could never be
But I should know her as well as she knows me.

ADRIANA.
Come, come, no longer will I be a fool,
To put the finger in the eye and weep
Whilst man and master laughs my woes to scorn.
Come, sir, to dinner; Dromio, keep the gate.
Husband, I’ll dine above with you today,
And shrive you of a thousand idle pranks.
Sirrah, if any ask you for your master,
Say he dines forth, and let no creature enter.
Come, sister; Dromio, play the porter well.

ANTIPHOLUS OF SYRACUSE.
Am I in earth, in heaven, or in hell?
Sleeping or waking, mad, or well-advis’d?
Known unto these, and to myself disguis’d!
I’ll say as they say, and persever so,
And in this mist at all adventures go.

DROMIO OF SYRACUSE.
Master, shall I be porter at the gate?

ADRIANA.
Ay; and let none enter, lest I break your pate.

LUCIANA.
Come, come, Antipholus, we dine too late.

 [_Exeunt._]



ACT III

SCENE I. The same

 Enter Antipholus of Ephesus, his man Dromio of Ephesus, Angelo the
 goldsmith and Balthasar the merchant.

ANTIPHOLUS OF EPHESUS.
Good Signior Angelo, you must excuse us all,
My wife is shrewish when I keep not hours.
Say that I linger’d with you at your shop
To see the making of her carcanet,
And that tomorrow you will bring it home.
But here’s a villain that would face me down.
He met me on the mart, and that I beat him,
And charg’d him with a thousand marks in gold,
And that I did deny my wife and house.
Thou drunkard, thou, what didst thou mean by this?

DROMIO OF EPHESUS.
Say what you will, sir, but I know what I know.
That you beat me at the mart I have your hand to show;
If the skin were parchment, and the blows you gave were ink,
Your own handwriting would tell you what I think.

ANTIPHOLUS OF EPHESUS.
I think thou art an ass.

DROMIO OF EPHESUS.
Marry, so it doth appear
By the wrongs I suffer and the blows I bear.
I should kick, being kick’d; and being at that pass,
You would keep from my heels, and beware of an ass.

ANTIPHOLUS OF EPHESUS.
You’re sad, Signior Balthasar; pray God our cheer
May answer my good will and your good welcome here.

BALTHASAR.
I hold your dainties cheap, sir, and your welcome dear.

ANTIPHOLUS OF EPHESUS.
O, Signior Balthasar, either at flesh or fish
A table full of welcome makes scarce one dainty dish.

BALTHASAR.
Good meat, sir, is common; that every churl affords.

ANTIPHOLUS OF EPHESUS.
And welcome more common, for that’s nothing but words.

BALTHASAR
Small cheer and great welcome makes a merry feast.

ANTIPHOLUS OF EPHESUS.
Ay, to a niggardly host and more sparing guest.
But though my cates be mean, take them in good part;
Better cheer may you have, but not with better heart.
But soft; my door is lock’d. Go bid them let us in.

DROMIO OF EPHESUS.
Maud, Bridget, Marian, Cicely, Gillian, Ginn!

DROMIO OF SYRACUSE.
[_Within._] Mome, malt-horse, capon, coxcomb, idiot, patch!
Either get thee from the door or sit down at the hatch:
Dost thou conjure for wenches, that thou call’st for such store
When one is one too many? Go, get thee from the door.

DROMIO OF EPHESUS.
What patch is made our porter? My master stays in the street.

DROMIO OF SYRACUSE.
Let him walk from whence he came, lest he catch cold on’s feet.

ANTIPHOLUS OF EPHESUS.
Who talks within there? Ho, open the door.

DROMIO OF SYRACUSE.
Right, sir, I’ll tell you when an you’ll tell me wherefore.

ANTIPHOLUS OF EPHESUS.
Wherefore? For my dinner. I have not dined today.

DROMIO OF SYRACUSE.
Nor today here you must not; come again when you may.

ANTIPHOLUS OF EPHESUS.
What art thou that keep’st me out from the house I owe?

DROMIO OF SYRACUSE.
The porter for this time, sir, and my name is Dromio.

DROMIO OF EPHESUS.
O villain, thou hast stolen both mine office and my name;
The one ne’er got me credit, the other mickle blame.
If thou hadst been Dromio today in my place,
Thou wouldst have chang’d thy face for a name, or thy name for an ass.

 Enter Luce concealed from Antipholus of Ephesus and his companions.

LUCE.
[_Within._] What a coil is there, Dromio, who are those at the gate?

DROMIO OF EPHESUS.
Let my master in, Luce.

LUCE.
Faith, no, he comes too late,
And so tell your master.

DROMIO OF EPHESUS.
O Lord, I must laugh;
Have at you with a proverb:—Shall I set in my staff?

LUCE.
Have at you with another: that’s—When? can you tell?

DROMIO OF SYRACUSE.
If thy name be called Luce,—Luce, thou hast answer’d him well.

ANTIPHOLUS OF EPHESUS.
Do you hear, you minion? you’ll let us in, I hope?

LUCE.
I thought to have ask’d you.

DROMIO OF SYRACUSE.
And you said no.

DROMIO OF EPHESUS.
So, come, help. Well struck, there was blow for blow.

ANTIPHOLUS OF EPHESUS.
Thou baggage, let me in.

LUCE.
Can you tell for whose sake?

DROMIO OF EPHESUS.
Master, knock the door hard.

LUCE.
Let him knock till it ache.

ANTIPHOLUS OF EPHESUS.
You’ll cry for this, minion, if I beat the door down.

LUCE.
What needs all that, and a pair of stocks in the town?

 Enter Adriana concealed from Antipholus of Ephesus and his companions.

ADRIANA.
[_Within._] Who is that at the door that keeps all this noise?

DROMIO OF SYRACUSE.
By my troth, your town is troubled with unruly boys.

ANTIPHOLUS OF EPHESUS.
Are you there, wife? you might have come before.

ADRIANA.
Your wife, sir knave? go, get you from the door.

DROMIO OF EPHESUS.
If you went in pain, master, this knave would go sore.

ANGELO.
Here is neither cheer, sir, nor welcome. We would fain have either.

BALTHASAR.
In debating which was best, we shall part with neither.

DROMIO OF EPHESUS.
They stand at the door, master; bid them welcome hither.

ANTIPHOLUS OF EPHESUS.
There is something in the wind, that we cannot get in.

DROMIO OF EPHESUS.
You would say so, master, if your garments were thin.
Your cake here is warm within; you stand here in the cold.
It would make a man mad as a buck to be so bought and sold.

ANTIPHOLUS OF EPHESUS.
Go, fetch me something, I’ll break ope the gate.

DROMIO OF SYRACUSE.
Break any breaking here, and I’ll break your knave’s pate.

DROMIO OF EPHESUS.
A man may break a word with you, sir, and words are but wind;
Ay, and break it in your face, so he break it not behind.

DROMIO OF SYRACUSE.
It seems thou want’st breaking; out upon thee, hind!

DROMIO OF EPHESUS.
Here’s too much “out upon thee”; I pray thee, let me in.

DROMIO OF SYRACUSE.
Ay, when fowls have no feathers and fish have no fin.

ANTIPHOLUS OF EPHESUS.
Well, I’ll break in; go, borrow me a crow.

DROMIO OF EPHESUS.
A crow without feather; master, mean you so?
For a fish without a fin, there’s a fowl without a feather.
If a crow help us in, sirrah, we’ll pluck a crow together.

ANTIPHOLUS OF EPHESUS.
Go, get thee gone; fetch me an iron crow.

BALTHASAR.
Have patience, sir. O, let it not be so:
Herein you war against your reputation,
And draw within the compass of suspect
The unviolated honour of your wife.
Once this,—your long experience of her wisdom,
Her sober virtue, years, and modesty,
Plead on her part some cause to you unknown;
And doubt not, sir, but she will well excuse
Why at this time the doors are made against you.
Be rul’d by me; depart in patience,
And let us to the Tiger all to dinner,
And about evening, come yourself alone
To know the reason of this strange restraint.
If by strong hand you offer to break in
Now in the stirring passage of the day,
A vulgar comment will be made of it;
And that supposed by the common rout
Against your yet ungalled estimation
That may with foul intrusion enter in,
And dwell upon your grave when you are dead;
For slander lives upon succession,
For ever hous’d where it gets possession.

ANTIPHOLUS OF EPHESUS.
You have prevail’d. I will depart in quiet,
And, in despite of mirth, mean to be merry.
I know a wench of excellent discourse,
Pretty and witty; wild, and yet, too, gentle;
There will we dine. This woman that I mean,
My wife (but, I protest, without desert)
Hath oftentimes upbraided me withal;
To her will we to dinner.—Get you home
And fetch the chain, by this I know ’tis made.
Bring it, I pray you, to the Porpentine,
For there’s the house. That chain will I bestow
(Be it for nothing but to spite my wife)
Upon mine hostess there. Good sir, make haste.
Since mine own doors refuse to entertain me,
I’ll knock elsewhere, to see if they’ll disdain me.

ANGELO.
I’ll meet you at that place some hour hence.

ANTIPHOLUS OF EPHESUS.
Do so; this jest shall cost me some expense.

 [_Exeunt._]

SCENE II. The same

 Enter Luciana with Antipholus of Syracuse.

LUCIANA.
And may it be that you have quite forgot
A husband’s office? Shall, Antipholus,
Even in the spring of love, thy love-springs rot?
Shall love, in building, grow so ruinous?
If you did wed my sister for her wealth,
Then for her wealth’s sake use her with more kindness;
Or if you like elsewhere, do it by stealth,
Muffle your false love with some show of blindness.
Let not my sister read it in your eye;
Be not thy tongue thy own shame’s orator;
Look sweet, speak fair, become disloyalty;
Apparel vice like virtue’s harbinger;
Bear a fair presence though your heart be tainted;
Teach sin the carriage of a holy saint,
Be secret-false. What need she be acquainted?
What simple thief brags of his own attaint?
’Tis double wrong to truant with your bed
And let her read it in thy looks at board.
Shame hath a bastard fame, well managed;
Ill deeds is doubled with an evil word.
Alas, poor women, make us but believe,
Being compact of credit, that you love us.
Though others have the arm, show us the sleeve;
We in your motion turn, and you may move us.
Then, gentle brother, get you in again;
Comfort my sister, cheer her, call her wife.
’Tis holy sport to be a little vain
When the sweet breath of flattery conquers strife.

ANTIPHOLUS OF SYRACUSE.
Sweet mistress, what your name is else, I know not,
Nor by what wonder you do hit on mine;
Less in your knowledge and your grace you show not
Than our earth’s wonder, more than earth divine.
Teach me, dear creature, how to think and speak;
Lay open to my earthy gross conceit,
Smother’d in errors, feeble, shallow, weak,
The folded meaning of your words’ deceit.
Against my soul’s pure truth why labour you
To make it wander in an unknown field?
Are you a god? would you create me new?
Transform me, then, and to your power I’ll yield.
But if that I am I, then well I know
Your weeping sister is no wife of mine,
Nor to her bed no homage do I owe.
Far more, far more, to you do I decline.
O, train me not, sweet mermaid, with thy note
To drown me in thy sister’s flood of tears.
Sing, siren, for thyself, and I will dote;
Spread o’er the silver waves thy golden hairs,
And as a bed I’ll take thee, and there lie,
And, in that glorious supposition think
He gains by death that hath such means to die.
Let love, being light, be drowned if she sink!

LUCIANA.
What, are you mad, that you do reason so?

ANTIPHOLUS OF SYRACUSE.
Not mad, but mated; how, I do not know.

LUCIANA.
It is a fault that springeth from your eye.

ANTIPHOLUS OF SYRACUSE.
For gazing on your beams, fair sun, being by.

LUCIANA.
Gaze where you should, and that will clear your sight.

ANTIPHOLUS OF SYRACUSE.
As good to wink, sweet love, as look on night.

LUCIANA.
Why call you me love? Call my sister so.

ANTIPHOLUS OF SYRACUSE.
Thy sister’s sister.

LUCIANA.
That’s my sister.

ANTIPHOLUS OF SYRACUSE.
No,
It is thyself, mine own self’s better part,
Mine eye’s clear eye, my dear heart’s dearer heart,
My food, my fortune, and my sweet hope’s aim,
My sole earth’s heaven, and my heaven’s claim.

LUCIANA.
All this my sister is, or else should be.

ANTIPHOLUS OF SYRACUSE.
Call thyself sister, sweet, for I aim thee;
Thee will I love, and with thee lead my life;
Thou hast no husband yet, nor I no wife.
Give me thy hand.

LUCIANA.
O, soft, sir, hold you still;
I’ll fetch my sister to get her goodwill.

 [_Exit Luciana._]

 Enter Dromio of Syracuse.

ANTIPHOLUS OF SYRACUSE.
Why, how now, Dromio? where runn’st thou so fast?

DROMIO OF SYRACUSE.
Do you know me, sir? Am I Dromio? Am I your man? Am I myself?

ANTIPHOLUS OF SYRACUSE.
Thou art Dromio, thou art my man, thou art thyself.

DROMIO OF SYRACUSE.
I am an ass, I am a woman’s man, and besides myself.

ANTIPHOLUS OF SYRACUSE.
What woman’s man? and how besides thyself?

DROMIO OF SYRACUSE.
Marry, sir, besides myself, I am due to a woman, one that claims me,
one that haunts me, one that will have me.

ANTIPHOLUS OF SYRACUSE.
What claim lays she to thee?

DROMIO OF SYRACUSE.
Marry, sir, such claim as you would lay to your horse, and she would
have me as a beast; not that I being a beast she would have me, but
that she being a very beastly creature lays claim to me.

ANTIPHOLUS OF SYRACUSE.
What is she?

DROMIO OF SYRACUSE.
A very reverent body; ay, such a one as a man may not speak of without
he say “sir-reverence”. I have but lean luck in the match, and yet is
she a wondrous fat marriage.

ANTIPHOLUS OF SYRACUSE.
How dost thou mean a “fat marriage”?

DROMIO OF SYRACUSE.
Marry, sir, she’s the kitchen wench, and all grease, and I know not
what use to put her to but to make a lamp of her and run from her by
her own light. I warrant her rags and the tallow in them will burn a
Poland winter. If she lives till doomsday, she’ll burn a week longer
than the whole world.

ANTIPHOLUS OF SYRACUSE.
What complexion is she of?

DROMIO OF SYRACUSE.
Swart like my shoe, but her face nothing like so clean kept. For why?
she sweats, a man may go overshoes in the grime of it.

ANTIPHOLUS OF SYRACUSE.
That’s a fault that water will mend.

DROMIO OF SYRACUSE.
No, sir, ’tis in grain; Noah’s flood could not do it.

ANTIPHOLUS OF SYRACUSE.
What’s her name?

DROMIO OF SYRACUSE.
Nell, sir; but her name and three quarters, that’s an ell and three
quarters, will not measure her from hip to hip.

ANTIPHOLUS OF SYRACUSE.
Then she bears some breadth?

DROMIO OF SYRACUSE.
No longer from head to foot than from hip to hip. She is spherical,
like a globe. I could find out countries in her.

ANTIPHOLUS OF SYRACUSE.
In what part of her body stands Ireland?

DROMIO OF SYRACUSE.
Marry, sir, in her buttocks; I found it out by the bogs.

ANTIPHOLUS OF SYRACUSE.
Where Scotland?

DROMIO OF SYRACUSE.
I found it by the barrenness, hard in the palm of the hand.

ANTIPHOLUS OF SYRACUSE.
Where France?

DROMIO OF SYRACUSE.
In her forehead; armed and reverted, making war against her hair.

ANTIPHOLUS OF SYRACUSE.
Where England?

DROMIO OF SYRACUSE.
I looked for the chalky cliffs, but I could find no whiteness in them.
But I guess it stood in her chin, by the salt rheum that ran between
France and it.

ANTIPHOLUS OF SYRACUSE.
Where Spain?

DROMIO OF SYRACUSE.
Faith, I saw it not; but I felt it hot in her breath.

ANTIPHOLUS OF SYRACUSE.
Where America, the Indies?

DROMIO OF SYRACUSE.
O, sir, upon her nose, all o’er-embellished with rubies, carbuncles,
sapphires, declining their rich aspect to the hot breath of Spain, who
sent whole armadoes of carracks to be ballast at her nose.

ANTIPHOLUS OF SYRACUSE.
Where stood Belgia, the Netherlands?

DROMIO OF SYRACUSE.
O, sir, I did not look so low. To conclude: this drudge or diviner laid
claim to me, called me Dromio, swore I was assured to her, told me what
privy marks I had about me, as the mark of my shoulder, the mole in my
neck, the great wart on my left arm, that I, amazed, ran from her as a
witch. And, I think, if my breast had not been made of faith, and my
heart of steel, she had transformed me to a curtal dog, and made me
turn i’ the wheel.

ANTIPHOLUS OF SYRACUSE.
Go, hie thee presently, post to the road;
And if the wind blow any way from shore,
I will not harbour in this town tonight.
If any bark put forth, come to the mart,
Where I will walk till thou return to me.
If everyone knows us, and we know none,
’Tis time, I think, to trudge, pack and be gone.

DROMIO OF SYRACUSE.
As from a bear a man would run for life,
So fly I from her that would be my wife.

 [_Exit._]

ANTIPHOLUS OF SYRACUSE.
There’s none but witches do inhabit here,
And therefore ’tis high time that I were hence.
She that doth call me husband, even my soul
Doth for a wife abhor. But her fair sister,
Possess’d with such a gentle sovereign grace,
Of such enchanting presence and discourse,
Hath almost made me traitor to myself.
But lest myself be guilty to self-wrong,
I’ll stop mine ears against the mermaid’s song.

 Enter Angelo with the chain.

ANGELO.
Master Antipholus.

ANTIPHOLUS OF SYRACUSE.
Ay, that’s my name.

ANGELO.
I know it well, sir. Lo, here is the chain;
I thought to have ta’en you at the Porpentine,
The chain unfinish’d made me stay thus long.

ANTIPHOLUS OF SYRACUSE.
What is your will that I shall do with this?

ANGELO.
What please yourself, sir; I have made it for you.

ANTIPHOLUS OF SYRACUSE.
Made it for me, sir! I bespoke it not.

ANGELO.
Not once, nor twice, but twenty times you have.
Go home with it, and please your wife withal,
And soon at supper-time I’ll visit you,
And then receive my money for the chain.

ANTIPHOLUS OF SYRACUSE.
I pray you, sir, receive the money now,
For fear you ne’er see chain nor money more.

ANGELO.
You are a merry man, sir; fare you well.

 [_Exit._]

ANTIPHOLUS OF SYRACUSE.
What I should think of this I cannot tell,
But this I think, there’s no man is so vain
That would refuse so fair an offer’d chain.
I see a man here needs not live by shifts,
When in the streets he meets such golden gifts.
I’ll to the mart, and there for Dromio stay;
If any ship put out, then straight away.

 [_Exit._]



ACT IV

SCENE I. The same

 Enter Merchant, Angelo and an Officer.

MERCHANT.
You know since Pentecost the sum is due,
And since I have not much importun’d you,
Nor now I had not, but that I am bound
To Persia, and want guilders for my voyage;
Therefore make present satisfaction,
Or I’ll attach you by this officer.

ANGELO.
Even just the sum that I do owe to you
Is growing to me by Antipholus,
And in the instant that I met with you
He had of me a chain; at five o’clock
I shall receive the money for the same.
Pleaseth you walk with me down to his house,
I will discharge my bond, and thank you too.

 Enter Antipholus of Ephesus and Dromio of Ephesus from the
 Courtesan’s.

OFFICER.
That labour may you save. See where he comes.

ANTIPHOLUS OF EPHESUS.
While I go to the goldsmith’s house, go thou
And buy a rope’s end; that will I bestow
Among my wife and her confederates
For locking me out of my doors by day.
But soft, I see the goldsmith; get thee gone;
Buy thou a rope, and bring it home to me.

DROMIO OF EPHESUS.
I buy a thousand pound a year! I buy a rope!

 [_Exit Dromio._]

ANTIPHOLUS OF EPHESUS.
A man is well holp up that trusts to you,
I promised your presence and the chain,
But neither chain nor goldsmith came to me.
Belike you thought our love would last too long
If it were chain’d together, and therefore came not.

ANGELO.
Saving your merry humour, here’s the note
How much your chain weighs to the utmost carat,
The fineness of the gold, and chargeful fashion,
Which doth amount to three odd ducats more
Than I stand debted to this gentleman.
I pray you, see him presently discharg’d,
For he is bound to sea, and stays but for it.

ANTIPHOLUS OF EPHESUS.
I am not furnished with the present money;
Besides, I have some business in the town.
Good signior, take the stranger to my house,
And with you take the chain, and bid my wife
Disburse the sum on the receipt thereof;
Perchance I will be there as soon as you.

ANGELO.
Then you will bring the chain to her yourself.

ANTIPHOLUS OF EPHESUS.
No, bear it with you, lest I come not time enough.

ANGELO.
Well, sir, I will. Have you the chain about you?

ANTIPHOLUS OF EPHESUS.
And if I have not, sir, I hope you have,
Or else you may return without your money.

ANGELO.
Nay, come, I pray you, sir, give me the chain;
Both wind and tide stays for this gentleman,
And I, to blame, have held him here too long.

ANTIPHOLUS OF EPHESUS.
Good Lord, you use this dalliance to excuse
Your breach of promise to the Porpentine.
I should have chid you for not bringing it,
But, like a shrew, you first begin to brawl.

MERCHANT.
The hour steals on; I pray you, sir, dispatch.

ANGELO.
You hear how he importunes me. The chain!

ANTIPHOLUS OF EPHESUS.
Why, give it to my wife, and fetch your money.

ANGELO.
Come, come, you know I gave it you even now.
Either send the chain or send by me some token.

ANTIPHOLUS OF EPHESUS.
Fie, now you run this humour out of breath.
Come, where’s the chain? I pray you, let me see it.

MERCHANT.
My business cannot brook this dalliance.
Good sir, say whe’er you’ll answer me or no;
If not, I’ll leave him to the officer.

ANTIPHOLUS OF EPHESUS.
I answer you? What should I answer you?

ANGELO.
The money that you owe me for the chain.

ANTIPHOLUS OF EPHESUS.
I owe you none till I receive the chain.

ANGELO.
You know I gave it you half an hour since.

ANTIPHOLUS OF EPHESUS.
You gave me none. You wrong me much to say so.

ANGELO.
You wrong me more, sir, in denying it.
Consider how it stands upon my credit.

MERCHANT.
Well, officer, arrest him at my suit.

OFFICER.
I do, and charge you in the duke’s name to obey me.

ANGELO.
This touches me in reputation.
Either consent to pay this sum for me,
Or I attach you by this officer.

ANTIPHOLUS OF EPHESUS.
Consent to pay thee that I never had?
Arrest me, foolish fellow, if thou dar’st.

ANGELO.
Here is thy fee; arrest him, officer.
I would not spare my brother in this case
If he should scorn me so apparently.

OFFICER.
I do arrest you, sir. You hear the suit.

ANTIPHOLUS OF EPHESUS.
I do obey thee till I give thee bail.
But, sirrah, you shall buy this sport as dear
As all the metal in your shop will answer.

ANGELO.
Sir, sir, I shall have law in Ephesus,
To your notorious shame, I doubt it not.

 Enter Dromio of Syracuse from the bay.

DROMIO OF SYRACUSE.
Master, there’s a bark of Epidamnum
That stays but till her owner comes aboard,
And then, sir, bears away. Our fraughtage, sir,
I have convey’d aboard, and I have bought
The oil, the balsamum, and aqua-vitae.
The ship is in her trim; the merry wind
Blows fair from land; they stay for nought at all
But for their owner, master, and yourself.

ANTIPHOLUS OF EPHESUS.
How now? a madman? Why, thou peevish sheep,
What ship of Epidamnum stays for me?

DROMIO OF SYRACUSE.
A ship you sent me to, to hire waftage.

ANTIPHOLUS OF EPHESUS.
Thou drunken slave, I sent thee for a rope,
And told thee to what purpose and what end.

DROMIO OF SYRACUSE.
You sent me for a rope’s end as soon.
You sent me to the bay, sir, for a bark.

ANTIPHOLUS OF EPHESUS.
I will debate this matter at more leisure,
And teach your ears to list me with more heed.
To Adriana, villain, hie thee straight:
Give her this key, and tell her in the desk
That’s cover’d o’er with Turkish tapestry
There is a purse of ducats; let her send it.
Tell her I am arrested in the street,
And that shall bail me. Hie thee, slave; be gone.
On, officer, to prison till it come.

 [_Exeunt Merchant, Angelo, Officer and Antipholus of Ephesus._]

DROMIO OF SYRACUSE.
To Adriana, that is where we din’d,
Where Dowsabel did claim me for her husband.
She is too big, I hope, for me to compass.
Thither I must, although against my will,
For servants must their masters’ minds fulfil.

 [_Exit._]

SCENE II. The same

 Enter Adriana and Luciana.

ADRIANA.
Ah, Luciana, did he tempt thee so?
Might’st thou perceive austerely in his eye
That he did plead in earnest, yea or no?
Look’d he or red or pale, or sad or merrily?
What observation mad’st thou in this case
Of his heart’s meteors tilting in his face?

LUCIANA.
First he denied you had in him no right.

ADRIANA.
He meant he did me none; the more my spite.

LUCIANA.
Then swore he that he was a stranger here.

ADRIANA.
And true he swore, though yet forsworn he were.

LUCIANA.
Then pleaded I for you.

ADRIANA.
And what said he?

LUCIANA.
That love I begg’d for you he begg’d of me.

ADRIANA.
With what persuasion did he tempt thy love?

LUCIANA.
With words that in an honest suit might move.
First he did praise my beauty, then my speech.

ADRIANA.
Did’st speak him fair?

LUCIANA.
Have patience, I beseech.

ADRIANA.
I cannot, nor I will not hold me still.
My tongue, though not my heart, shall have his will.
He is deformed, crooked, old, and sere,
Ill-fac’d, worse bodied, shapeless everywhere;
Vicious, ungentle, foolish, blunt, unkind,
Stigmatical in making, worse in mind.

LUCIANA.
Who would be jealous then of such a one?
No evil lost is wail’d when it is gone.

ADRIANA.
Ah, but I think him better than I say,
And yet would herein others’ eyes were worse:
Far from her nest the lapwing cries away;
My heart prays for him, though my tongue do curse.

 Enter Dromio of Syracuse.

DROMIO OF SYRACUSE.
Here, go; the desk, the purse, sweet now, make haste.

LUCIANA.
How hast thou lost thy breath?

DROMIO OF SYRACUSE.
By running fast.

ADRIANA.
Where is thy master, Dromio? is he well?

DROMIO OF SYRACUSE.
No, he’s in Tartar limbo, worse than hell.
A devil in an everlasting garment hath him,
One whose hard heart is button’d up with steel;
A fiend, a fairy, pitiless and rough;
A wolf, nay worse, a fellow all in buff;
A back-friend, a shoulder-clapper, one that countermands
The passages of alleys, creeks, and narrow lands;
A hound that runs counter, and yet draws dryfoot well,
One that, before the judgement, carries poor souls to hell.

ADRIANA.
Why, man, what is the matter?

DROMIO OF SYRACUSE.
I do not know the matter. He is ’rested on the case.

ADRIANA.
What, is he arrested? Tell me at whose suit?

DROMIO OF SYRACUSE.
I know not at whose suit he is arrested, well;
But he’s in a suit of buff which ’rested him, that can I tell.
Will you send him, mistress, redemption, the money in his desk?

ADRIANA.
Go fetch it, sister. This I wonder at,

 [_Exit Luciana._]

Thus he unknown to me should be in debt.
Tell me, was he arrested on a band?

DROMIO OF SYRACUSE.
Not on a band, but on a stronger thing;
A chain, a chain. Do you not hear it ring?

ADRIANA.
What, the chain?

DROMIO OF SYRACUSE.
No, no, the bell, ’tis time that I were gone.
It was two ere I left him, and now the clock strikes one.

ADRIANA.
The hours come back! That did I never hear.

DROMIO OF SYRACUSE.
O yes, if any hour meet a sergeant, ’a turns back for very fear.

ADRIANA.
As if time were in debt. How fondly dost thou reason!

DROMIO OF SYRACUSE.
Time is a very bankrupt, and owes more than he’s worth to season.
Nay, he’s a thief too. Have you not heard men say
That time comes stealing on by night and day?
If he be in debt and theft, and a sergeant in the way,
Hath he not reason to turn back an hour in a day?

 Enter Luciana.

ADRIANA.
Go, Dromio, there’s the money, bear it straight,
And bring thy master home immediately.
Come, sister, I am press’d down with conceit;
Conceit, my comfort and my injury.

 [_Exeunt._]

SCENE III. The same

 Enter Antipholus of Syracuse.

ANTIPHOLUS OF SYRACUSE.
There’s not a man I meet but doth salute me
As if I were their well-acquainted friend,
And everyone doth call me by my name.
Some tender money to me, some invite me;
Some other give me thanks for kindnesses;
Some offer me commodities to buy.
Even now a tailor call’d me in his shop,
And show’d me silks that he had bought for me,
And therewithal took measure of my body.
Sure, these are but imaginary wiles,
And Lapland sorcerers inhabit here.

 Enter Dromio of Syracuse.

DROMIO OF SYRACUSE.
Master, here’s the gold you sent me for.
What, have you got the picture of old Adam new apparelled?

ANTIPHOLUS OF SYRACUSE.
What gold is this? What Adam dost thou mean?

DROMIO OF SYRACUSE.
Not that Adam that kept the paradise, but that Adam that keeps the
prison; he that goes in the calf’s skin that was killed for the
Prodigal; he that came behind you, sir, like an evil angel, and bid you
forsake your liberty.

ANTIPHOLUS OF SYRACUSE.
I understand thee not.

DROMIO OF SYRACUSE.
No? Why, ’tis a plain case: he that went like a bass-viol in a case of
leather; the man, sir, that, when gentlemen are tired, gives them a
sob, and ’rests them; he, sir, that takes pity on decayed men and gives
them suits of durance; he that sets up his rest to do more exploits
with his mace than a morris-pike.

ANTIPHOLUS OF SYRACUSE.
What! thou mean’st an officer?

DROMIO OF SYRACUSE.
Ay, sir, the sergeant of the band; he that brings any man to answer it
that breaks his band; one that thinks a man always going to bed, and
says, “God give you good rest.”

ANTIPHOLUS OF SYRACUSE.
Well, sir, there rest in your foolery. Is there any ship puts forth
tonight? may we be gone?

DROMIO OF SYRACUSE.
Why, sir, I brought you word an hour since that the bark _Expedition_
put forth tonight, and then were you hindered by the sergeant to tarry
for the hoy _Delay_. Here are the angels that you sent for to deliver
you.

ANTIPHOLUS OF SYRACUSE.
The fellow is distract, and so am I,
And here we wander in illusions.
Some blessed power deliver us from hence!

 Enter a Courtesan.

COURTESAN.
Well met, well met, Master Antipholus.
I see, sir, you have found the goldsmith now.
Is that the chain you promis’d me today?

ANTIPHOLUS OF SYRACUSE.
Satan, avoid! I charge thee, tempt me not.

DROMIO OF SYRACUSE.
Master, is this Mistress Satan?

ANTIPHOLUS OF SYRACUSE.
It is the devil.

DROMIO OF SYRACUSE.
Nay, she is worse, she is the devil’s dam; and here she comes in the
habit of a light wench, and thereof comes that the wenches say “God
damn me”, that’s as much to say, “God make me a light wench.” It is
written they appear to men like angels of light. Light is an effect of
fire, and fire will burn; ergo, light wenches will burn. Come not near
her.

COURTESAN.
Your man and you are marvellous merry, sir.
Will you go with me? We’ll mend our dinner here.

DROMIO OF SYRACUSE.
Master, if you do, expect spoon-meat, or bespeak a long spoon.

ANTIPHOLUS OF SYRACUSE.
Why, Dromio?

DROMIO OF SYRACUSE.
Marry, he must have a long spoon that must eat with the devil.

ANTIPHOLUS OF SYRACUSE.
Avoid then, fiend! What tell’st thou me of supping?
Thou art, as you are all, a sorceress.
I conjure thee to leave me and be gone.

COURTESAN.
Give me the ring of mine you had at dinner,
Or, for my diamond, the chain you promis’d,
And I’ll be gone, sir, and not trouble you.

DROMIO OF SYRACUSE.
Some devils ask but the paring of one’s nail,
A rush, a hair, a drop of blood, a pin,
A nut, a cherry-stone; but she, more covetous,
Would have a chain.
Master, be wise; and if you give it her,
The devil will shake her chain and fright us with it.

COURTESAN.
I pray you, sir, my ring, or else the chain;
I hope you do not mean to cheat me so.

ANTIPHOLUS OF SYRACUSE.
Avaunt, thou witch! Come, Dromio, let us go.

DROMIO OF SYRACUSE.
Fly pride, says the peacock. Mistress, that you know.

 [_Exeunt Antipholus of Syracuse and Dromio of Syracuse._]

COURTESAN.
Now, out of doubt Antipholus is mad,
Else would he never so demean himself.
A ring he hath of mine worth forty ducats,
And for the same he promis’d me a chain;
Both one and other he denies me now.
The reason that I gather he is mad,
Besides this present instance of his rage,
Is a mad tale he told today at dinner
Of his own doors being shut against his entrance.
Belike his wife, acquainted with his fits,
On purpose shut the doors against his way.
My way is now to hie home to his house,
And tell his wife that, being lunatic,
He rush’d into my house and took perforce
My ring away. This course I fittest choose,
For forty ducats is too much to lose.

 [_Exit._]

SCENE IV. The same

 Enter Antipholus of Ephesus with an Officer.

ANTIPHOLUS OF EPHESUS.
Fear me not, man, I will not break away:
I’ll give thee ere I leave thee so much money,
To warrant thee, as I am ’rested for.
My wife is in a wayward mood today,
And will not lightly trust the messenger
That I should be attach’d in Ephesus;
I tell you ’twill sound harshly in her ears.

 Enter Dromio of Ephesus with a rope’s end.

Here comes my man. I think he brings the money.
How now, sir! have you that I sent you for?

DROMIO OF EPHESUS.
Here’s that, I warrant you, will pay them all.

ANTIPHOLUS OF EPHESUS.
But where’s the money?

DROMIO OF EPHESUS.
Why, sir, I gave the money for the rope.

ANTIPHOLUS OF EPHESUS.
Five hundred ducats, villain, for a rope?

DROMIO OF EPHESUS.
I’ll serve you, sir, five hundred at the rate.

ANTIPHOLUS OF EPHESUS.
To what end did I bid thee hie thee home?

DROMIO OF EPHESUS.
To a rope’s end, sir; and to that end am I return’d.

ANTIPHOLUS OF EPHESUS.
And to that end, sir, I will welcome you.

 [_Beating him._]

OFFICER.
Good sir, be patient.

DROMIO OF EPHESUS.
Nay, ’tis for me to be patient. I am in adversity.

OFFICER.
Good now, hold thy tongue.

DROMIO OF EPHESUS.
Nay, rather persuade him to hold his hands.

ANTIPHOLUS OF EPHESUS.
Thou whoreson, senseless villain.

DROMIO OF EPHESUS.
I would I were senseless, sir, that I might not feel your blows.

ANTIPHOLUS OF EPHESUS.
Thou art sensible in nothing but blows, and so is an ass.

DROMIO OF EPHESUS.
I am an ass indeed; you may prove it by my long ears. I have served him
from the hour of my nativity to this instant, and have nothing at his
hands for my service but blows. When I am cold, he heats me with
beating; when I am warm he cools me with beating. I am waked with it
when I sleep, raised with it when I sit, driven out of doors with it
when I go from home, welcomed home with it when I return. Nay, I bear
it on my shoulders as a beggar wont her brat; and I think when he hath
lamed me, I shall beg with it from door to door.

 Enter Adriana, Luciana, Courtesan and a Schoolmaster called Pinch.

ANTIPHOLUS OF EPHESUS.
Come, go along, my wife is coming yonder.

DROMIO OF EPHESUS.
Mistress, _respice finem_, respect your end, or rather, the prophesy
like the parrot, “Beware the rope’s end.”

ANTIPHOLUS OF EPHESUS.
Wilt thou still talk?

 [_Beats him._]

COURTESAN.
How say you now? Is not your husband mad?

ADRIANA.
His incivility confirms no less.
Good Doctor Pinch, you are a conjurer;
Establish him in his true sense again,
And I will please you what you will demand.

LUCIANA.
Alas, how fiery and how sharp he looks!

COURTESAN.
Mark how he trembles in his ecstasy.

PINCH.
Give me your hand, and let me feel your pulse.

ANTIPHOLUS OF EPHESUS.
There is my hand, and let it feel your ear.

PINCH.
I charge thee, Satan, hous’d within this man,
To yield possession to my holy prayers,
And to thy state of darkness hie thee straight.
I conjure thee by all the saints in heaven.

ANTIPHOLUS OF EPHESUS.
Peace, doting wizard, peace; I am not mad.

ADRIANA.
O, that thou wert not, poor distressed soul!

ANTIPHOLUS OF EPHESUS.
You minion, you, are these your customers?
Did this companion with the saffron face
Revel and feast it at my house today,
Whilst upon me the guilty doors were shut,
And I denied to enter in my house?

ADRIANA.
O husband, God doth know you din’d at home,
Where would you had remain’d until this time,
Free from these slanders and this open shame.

ANTIPHOLUS OF EPHESUS.
Din’d at home? Thou villain, what sayest thou?

DROMIO OF EPHESUS.
Sir, sooth to say, you did not dine at home.

ANTIPHOLUS OF EPHESUS.
Were not my doors lock’d up and I shut out?

DROMIO OF EPHESUS.
Perdy, your doors were lock’d, and you shut out.

ANTIPHOLUS OF EPHESUS.
And did not she herself revile me there?

DROMIO OF EPHESUS.
Sans fable, she herself revil’d you there.

ANTIPHOLUS OF EPHESUS.
Did not her kitchen-maid rail, taunt, and scorn me?

DROMIO OF EPHESUS.
Certes, she did, the kitchen-vestal scorn’d you.

ANTIPHOLUS OF EPHESUS.
And did not I in rage depart from thence?

DROMIO OF EPHESUS.
In verity, you did; my bones bear witness,
That since have felt the vigour of his rage.

ADRIANA.
Is’t good to soothe him in these contraries?

PINCH.
It is no shame; the fellow finds his vein,
And yielding to him, humours well his frenzy.

ANTIPHOLUS OF EPHESUS.
Thou hast suborn’d the goldsmith to arrest me.

ADRIANA.
Alas! I sent you money to redeem you
By Dromio here, who came in haste for it.

DROMIO OF EPHESUS.
Money by me? Heart and goodwill you might,
But surely, master, not a rag of money.

ANTIPHOLUS OF EPHESUS.
Went’st not thou to her for a purse of ducats?

ADRIANA.
He came to me, and I deliver’d it.

LUCIANA.
And I am witness with her that she did.

DROMIO OF EPHESUS.
God and the rope-maker bear me witness
That I was sent for nothing but a rope.

PINCH.
Mistress, both man and master is possess’d,
I know it by their pale and deadly looks.
They must be bound and laid in some dark room.

ANTIPHOLUS OF EPHESUS.
Say, wherefore didst thou lock me forth today,
And why dost thou deny the bag of gold?

ADRIANA.
I did not, gentle husband, lock thee forth.

DROMIO OF EPHESUS.
And gentle master, I receiv’d no gold;
But I confess, sir, that we were lock’d out.

ADRIANA.
Dissembling villain, thou speak’st false in both.

ANTIPHOLUS OF EPHESUS.
Dissembling harlot, thou art false in all,
And art confederate with a damned pack
To make a loathsome abject scorn of me.
But with these nails I’ll pluck out these false eyes
That would behold in me this shameful sport.

 [_Enter three or four, and offer to bind him. He strives. _]

ADRIANA.
O, bind him, bind him; let him not come near me.

PINCH.
More company; the fiend is strong within him.

LUCIANA.
Ay me, poor man, how pale and wan he looks!

ANTIPHOLUS OF EPHESUS.
What, will you murder me? Thou jailer, thou,
I am thy prisoner. Wilt thou suffer them
To make a rescue?

OFFICER.
Masters, let him go.
He is my prisoner, and you shall not have him.

PINCH.
Go, bind this man, for he is frantic too.

ADRIANA.
What wilt thou do, thou peevish officer?
Hast thou delight to see a wretched man
Do outrage and displeasure to himself?

OFFICER.
He is my prisoner. If I let him go,
The debt he owes will be requir’d of me.

ADRIANA.
I will discharge thee ere I go from thee;
Bear me forthwith unto his creditor,
And knowing how the debt grows, I will pay it.
Good master doctor, see him safe convey’d
Home to my house. O most unhappy day!

ANTIPHOLUS OF EPHESUS.
O most unhappy strumpet!

DROMIO OF EPHESUS.
Master, I am here enter’d in bond for you.

ANTIPHOLUS OF EPHESUS.
Out on thee, villain! wherefore dost thou mad me?

DROMIO OF EPHESUS.
Will you be bound for nothing? Be mad, good master; cry, “the devil”.

LUCIANA.
God help, poor souls, how idly do they talk!

ADRIANA.
Go bear him hence. Sister, go you with me.

 [_Exeunt Pinch and Assistants, with Antipholus of Ephesus and Dromio
 of Ephesus._]

Say now, whose suit is he arrested at?

OFFICER.
One Angelo, a goldsmith; do you know him?

ADRIANA.
I know the man. What is the sum he owes?

OFFICER.
Two hundred ducats.

ADRIANA.
Say, how grows it due?

OFFICER.
Due for a chain your husband had of him.

ADRIANA.
He did bespeak a chain for me, but had it not.

COURTESAN.
When as your husband, all in rage, today
Came to my house and took away my ring,
The ring I saw upon his finger now,
Straight after did I meet him with a chain.

ADRIANA.
It may be so, but I did never see it.
Come, jailer, bring me where the goldsmith is,
I long to know the truth hereof at large.

 Enter Antipholus of Syracuse with his rapier drawn, and Dromio of
 Syracuse.

LUCIANA.
God, for thy mercy, they are loose again!

ADRIANA.
And come with naked swords. Let’s call more help
To have them bound again.

OFFICER.
Away, they’ll kill us.

 [_Exeunt, as fast as may be, frighted._]

ANTIPHOLUS OF SYRACUSE.
I see these witches are afraid of swords.

DROMIO OF SYRACUSE.
She that would be your wife now ran from you.

ANTIPHOLUS OF SYRACUSE.
Come to the Centaur, fetch our stuff from thence.
I long that we were safe and sound aboard.

DROMIO OF SYRACUSE.
Faith, stay here this night, they will surely do us no harm; you saw
they speak us fair, give us gold. Methinks they are such a gentle
nation that, but for the mountain of mad flesh that claims marriage of
me, I could find in my heart to stay here still and turn witch.

ANTIPHOLUS OF SYRACUSE.
I will not stay tonight for all the town;
Therefore away, to get our stuff aboard.

 [_Exeunt._]



ACT V

SCENE I. The same

 Enter Merchant and Angelo.

ANGELO.
I am sorry, sir, that I have hinder’d you,
But I protest he had the chain of me,
Though most dishonestly he doth deny it.

MERCHANT.
How is the man esteem’d here in the city?

ANGELO.
Of very reverend reputation, sir,
Of credit infinite, highly belov’d,
Second to none that lives here in the city.
His word might bear my wealth at any time.

MERCHANT.
Speak softly. Yonder, as I think, he walks.

 Enter Antipholus of Syracuse and Dromio of Syracuse.

ANGELO.
’Tis so; and that self chain about his neck
Which he forswore most monstrously to have.
Good sir, draw near to me, I’ll speak to him.
Signior Antipholus, I wonder much
That you would put me to this shame and trouble,
And not without some scandal to yourself,
With circumstance and oaths so to deny
This chain, which now you wear so openly.
Beside the charge, the shame, imprisonment,
You have done wrong to this my honest friend,
Who, but for staying on our controversy,
Had hoisted sail and put to sea today.
This chain you had of me, can you deny it?

ANTIPHOLUS OF SYRACUSE.
I think I had: I never did deny it.

MERCHANT.
Yes, that you did, sir, and forswore it too.

ANTIPHOLUS OF SYRACUSE.
Who heard me to deny it or forswear it?

MERCHANT.
These ears of mine, thou know’st, did hear thee.
Fie on thee, wretch. ’Tis pity that thou liv’st
To walk where any honest men resort.

ANTIPHOLUS OF SYRACUSE.
Thou art a villain to impeach me thus;
I’ll prove mine honour and mine honesty
Against thee presently, if thou dar’st stand.

MERCHANT.
I dare, and do defy thee for a villain.

 [_They draw._]

 Enter Adriana, Luciana, Courtesan and others.

ADRIANA.
Hold, hurt him not, for God’s sake, he is mad.
Some get within him, take his sword away.
Bind Dromio too, and bear them to my house.

DROMIO OF SYRACUSE.
Run, master, run, for God’s sake, take a house.
This is some priory; in, or we are spoil’d.

 [_Exeunt Antipholus of Syracuse and Dromio of Syracuse to the
 priory._]

 Enter Lady Abbess.

ABBESS.
Be quiet, people. Wherefore throng you hither?

ADRIANA.
To fetch my poor distracted husband hence.
Let us come in, that we may bind him fast
And bear him home for his recovery.

ANGELO.
I knew he was not in his perfect wits.

MERCHANT.
I am sorry now that I did draw on him.

ABBESS.
How long hath this possession held the man?

ADRIANA.
This week he hath been heavy, sour, sad,
And much different from the man he was.
But till this afternoon his passion
Ne’er brake into extremity of rage.

ABBESS.
Hath he not lost much wealth by wreck of sea?
Buried some dear friend? Hath not else his eye
Stray’d his affection in unlawful love?
A sin prevailing much in youthful men
Who give their eyes the liberty of gazing?
Which of these sorrows is he subject to?

ADRIANA.
To none of these, except it be the last,
Namely, some love that drew him oft from home.

ABBESS.
You should for that have reprehended him.

ADRIANA.
Why, so I did.

ABBESS.
Ay, but not rough enough.

ADRIANA.
As roughly as my modesty would let me.

ABBESS.
Haply in private.

ADRIANA.
And in assemblies too.

ABBESS.
Ay, but not enough.

ADRIANA.
It was the copy of our conference.
In bed he slept not for my urging it;
At board he fed not for my urging it;
Alone, it was the subject of my theme;
In company I often glanced it;
Still did I tell him it was vile and bad.

ABBESS.
And thereof came it that the man was mad.
The venom clamours of a jealous woman
Poisons more deadly than a mad dog’s tooth.
It seems his sleeps were hindered by thy railing,
And thereof comes it that his head is light.
Thou say’st his meat was sauc’d with thy upbraidings.
Unquiet meals make ill digestions;
Thereof the raging fire of fever bred,
And what’s a fever but a fit of madness?
Thou say’st his sports were hinder’d by thy brawls.
Sweet recreation barr’d, what doth ensue
But moody and dull melancholy,
Kinsman to grim and comfortless despair,
And at her heels a huge infectious troop
Of pale distemperatures and foes to life?
In food, in sport, and life-preserving rest
To be disturb’d would mad or man or beast.
The consequence is, then, thy jealous fits
Hath scar’d thy husband from the use of’s wits.

LUCIANA.
She never reprehended him but mildly,
When he demean’d himself rough, rude, and wildly.
Why bear you these rebukes and answer not?

ADRIANA.
She did betray me to my own reproof.
Good people, enter and lay hold on him.

ABBESS.
No, not a creature enters in my house.

ADRIANA.
Then let your servants bring my husband forth.

ABBESS.
Neither. He took this place for sanctuary,
And it shall privilege him from your hands
Till I have brought him to his wits again,
Or lose my labour in assaying it.

ADRIANA.
I will attend my husband, be his nurse,
Diet his sickness, for it is my office,
And will have no attorney but myself;
And therefore let me have him home with me.

ABBESS.
Be patient, for I will not let him stir
Till I have used the approved means I have,
With wholesome syrups, drugs, and holy prayers,
To make of him a formal man again.
It is a branch and parcel of mine oath,
A charitable duty of my order;
Therefore depart, and leave him here with me.

ADRIANA.
I will not hence and leave my husband here;
And ill it doth beseem your holiness
To separate the husband and the wife.

ABBESS.
Be quiet and depart. Thou shalt not have him.

 [_Exit Abbess._]

LUCIANA.
Complain unto the duke of this indignity.

ADRIANA.
Come, go. I will fall prostrate at his feet,
And never rise until my tears and prayers
Have won his grace to come in person hither
And take perforce my husband from the abbess.

MERCHANT.
By this, I think, the dial points at five.
Anon, I’m sure, the Duke himself in person
Comes this way to the melancholy vale,
The place of death and sorry execution
Behind the ditches of the abbey here.

ANGELO.
Upon what cause?

MERCHANT.
To see a reverend Syracusian merchant,
Who put unluckily into this bay
Against the laws and statutes of this town,
Beheaded publicly for his offence.

ANGELO.
See where they come. We will behold his death.

LUCIANA.
Kneel to the Duke before he pass the abbey.

 Enter the Duke, attended; Egeon, bareheaded; with the Headsman and
 other Officers.

DUKE.
Yet once again proclaim it publicly,
If any friend will pay the sum for him,
He shall not die; so much we tender him.

ADRIANA.
Justice, most sacred duke, against the abbess!

DUKE.
She is a virtuous and a reverend lady,
It cannot be that she hath done thee wrong.

ADRIANA.
May it please your grace, Antipholus, my husband,
Who I made lord of me and all I had
At your important letters, this ill day
A most outrageous fit of madness took him;
That desp’rately he hurried through the street,
With him his bondman all as mad as he,
Doing displeasure to the citizens
By rushing in their houses, bearing thence
Rings, jewels, anything his rage did like.
Once did I get him bound and sent him home,
Whilst to take order for the wrongs I went,
That here and there his fury had committed.
Anon, I wot not by what strong escape,
He broke from those that had the guard of him,
And with his mad attendant and himself,
Each one with ireful passion, with drawn swords,
Met us again, and, madly bent on us,
Chased us away; till raising of more aid,
We came again to bind them. Then they fled
Into this abbey, whither we pursued them.
And here the abbess shuts the gates on us,
And will not suffer us to fetch him out,
Nor send him forth that we may bear him hence.
Therefore, most gracious duke, with thy command
Let him be brought forth and borne hence for help.

DUKE.
Long since thy husband serv’d me in my wars,
And I to thee engag’d a prince’s word,
When thou didst make him master of thy bed,
To do him all the grace and good I could.
Go, some of you, knock at the abbey gate,
And bid the lady abbess come to me.
I will determine this before I stir.

 Enter a Messenger.

MESSENGER.
O mistress, mistress, shift and save yourself.
My master and his man are both broke loose,
Beaten the maids a-row, and bound the doctor,
Whose beard they have singed off with brands of fire,
And ever as it blazed they threw on him
Great pails of puddled mire to quench the hair.
My master preaches patience to him, and the while
His man with scissors nicks him like a fool;
And sure (unless you send some present help)
Between them they will kill the conjurer.

ADRIANA.
Peace, fool, thy master and his man are here,
And that is false thou dost report to us.

MESSENGER.
Mistress, upon my life, I tell you true.
I have not breath’d almost since I did see it.
He cries for you, and vows, if he can take you,
To scorch your face and to disfigure you.

 [_Cry within._]

Hark, hark, I hear him, mistress. Fly, be gone!

DUKE.
Come, stand by me, fear nothing. Guard with halberds.

ADRIANA.
Ay me, it is my husband. Witness you
That he is borne about invisible.
Even now we hous’d him in the abbey here,
And now he’s there, past thought of human reason.

 Enter Antipholus and Dromio of Ephesus.

ANTIPHOLUS OF EPHESUS.
Justice, most gracious duke; O, grant me justice!
Even for the service that long since I did thee
When I bestrid thee in the wars, and took
Deep scars to save thy life; even for the blood
That then I lost for thee, now grant me justice.

EGEON.
Unless the fear of death doth make me dote,
I see my son Antipholus and Dromio.

ANTIPHOLUS OF EPHESUS.
Justice, sweet prince, against that woman there.
She whom thou gav’st to me to be my wife;
That hath abused and dishonour’d me
Even in the strength and height of injury.
Beyond imagination is the wrong
That she this day hath shameless thrown on me.

DUKE.
Discover how, and thou shalt find me just.

ANTIPHOLUS OF EPHESUS.
This day, great duke, she shut the doors upon me
While she with harlots feasted in my house.

DUKE.
A grievous fault. Say, woman, didst thou so?

ADRIANA.
No, my good lord. Myself, he, and my sister
Today did dine together. So befall my soul
As this is false he burdens me withal.

LUCIANA.
Ne’er may I look on day nor sleep on night
But she tells to your highness simple truth.

ANGELO.
O perjur’d woman! They are both forsworn.
In this the madman justly chargeth them.

ANTIPHOLUS OF EPHESUS.
My liege, I am advised what I say,
Neither disturb’d with the effect of wine,
Nor heady-rash, provok’d with raging ire,
Albeit my wrongs might make one wiser mad.
This woman lock’d me out this day from dinner.
That goldsmith there, were he not pack’d with her,
Could witness it, for he was with me then,
Who parted with me to go fetch a chain,
Promising to bring it to the Porpentine,
Where Balthasar and I did dine together.
Our dinner done, and he not coming thither,
I went to seek him. In the street I met him,
And in his company that gentleman.
There did this perjur’d goldsmith swear me down
That I this day of him receiv’d the chain,
Which, God he knows, I saw not. For the which
He did arrest me with an officer.
I did obey, and sent my peasant home
For certain ducats. He with none return’d.
Then fairly I bespoke the officer
To go in person with me to my house.
By th’ way we met
My wife, her sister, and a rabble more
Of vile confederates. Along with them
They brought one Pinch, a hungry lean-faced villain,
A mere anatomy, a mountebank,
A threadbare juggler, and a fortune-teller;
A needy, hollow-ey’d, sharp-looking wretch;
A living dead man. This pernicious slave,
Forsooth, took on him as a conjurer,
And gazing in mine eyes, feeling my pulse,
And with no face (as ’twere) outfacing me,
Cries out, I was possess’d. Then altogether
They fell upon me, bound me, bore me thence,
And in a dark and dankish vault at home
There left me and my man, both bound together,
Till gnawing with my teeth my bonds in sunder,
I gain’d my freedom and immediately
Ran hither to your Grace, whom I beseech
To give me ample satisfaction
For these deep shames and great indignities.

ANGELO.
My lord, in truth, thus far I witness with him,
That he din’d not at home, but was lock’d out.

DUKE.
But had he such a chain of thee, or no?

ANGELO.
He had, my lord, and when he ran in here
These people saw the chain about his neck.

MERCHANT.
Besides, I will be sworn these ears of mine
Heard you confess you had the chain of him,
After you first forswore it on the mart,
And thereupon I drew my sword on you;
And then you fled into this abbey here,
From whence I think you are come by miracle.

ANTIPHOLUS OF EPHESUS.
I never came within these abbey walls,
Nor ever didst thou draw thy sword on me.
I never saw the chain, so help me heaven;
And this is false you burden me withal.

DUKE.
Why, what an intricate impeach is this!
I think you all have drunk of Circe’s cup.
If here you hous’d him, here he would have been.
If he were mad, he would not plead so coldly.
You say he din’d at home, the goldsmith here
Denies that saying. Sirrah, what say you?

DROMIO OF EPHESUS.
Sir, he dined with her there, at the Porpentine.

COURTESAN.
He did, and from my finger snatch’d that ring.

ANTIPHOLUS OF EPHESUS.
’Tis true, my liege, this ring I had of her.

DUKE.
Saw’st thou him enter at the abbey here?

COURTESAN.
As sure, my liege, as I do see your grace.

DUKE.
Why, this is strange. Go call the abbess hither.
I think you are all mated, or stark mad.

 [_Exit one to the Abbess._]

EGEON.
Most mighty Duke, vouchsafe me speak a word;
Haply I see a friend will save my life
And pay the sum that may deliver me.

DUKE.
Speak freely, Syracusian, what thou wilt.

EGEON.
Is not your name, sir, call’d Antipholus?
And is not that your bondman Dromio?

DROMIO OF EPHESUS.
Within this hour I was his bondman, sir,
But he, I thank him, gnaw’d in two my cords.
Now am I Dromio, and his man, unbound.

EGEON.
I am sure you both of you remember me.

DROMIO OF EPHESUS.
Ourselves we do remember, sir, by you.
For lately we were bound as you are now.
You are not Pinch’s patient, are you, sir?

EGEON.
Why look you strange on me? you know me well.

ANTIPHOLUS OF EPHESUS.
I never saw you in my life till now.

EGEON.
O! grief hath chang’d me since you saw me last,
And careful hours with time’s deformed hand,
Have written strange defeatures in my face.
But tell me yet, dost thou not know my voice?

ANTIPHOLUS OF EPHESUS.
Neither.

EGEON.
Dromio, nor thou?

DROMIO OF EPHESUS.
No, trust me, sir, nor I.

EGEON.
I am sure thou dost.

DROMIO OF EPHESUS.
Ay, sir, but I am sure I do not, and whatsoever a man denies, you are
now bound to believe him.

EGEON.
Not know my voice! O time’s extremity,
Hast thou so crack’d and splitted my poor tongue
In seven short years that here my only son
Knows not my feeble key of untun’d cares?
Though now this grained face of mine be hid
In sap-consuming winter’s drizzled snow,
And all the conduits of my blood froze up,
Yet hath my night of life some memory,
My wasting lamps some fading glimmer left,
My dull deaf ears a little use to hear.
All these old witnesses, I cannot err,
Tell me thou art my son Antipholus.

ANTIPHOLUS OF EPHESUS.
I never saw my father in my life.

EGEON.
But seven years since, in Syracusa, boy,
Thou know’st we parted; but perhaps, my son,
Thou sham’st to acknowledge me in misery.

ANTIPHOLUS OF EPHESUS.
The duke and all that know me in the city,
Can witness with me that it is not so.
I ne’er saw Syracusa in my life.

DUKE.
I tell thee, Syracusian, twenty years
Have I been patron to Antipholus,
During which time he ne’er saw Syracusa.
I see thy age and dangers make thee dote.

 Enter the Abbess with Antipholus of Syracuse and Dromio of Syracuse.

ABBESS.
Most mighty duke, behold a man much wrong’d.

 [_All gather to see them._]

ADRIANA.
I see two husbands, or mine eyes deceive me.

DUKE.
One of these men is _genius_ to the other;
And so of these, which is the natural man,
And which the spirit? Who deciphers them?

DROMIO OF SYRACUSE.
I, sir, am Dromio, command him away.

DROMIO OF EPHESUS.
I, sir, am Dromio, pray let me stay.

ANTIPHOLUS OF SYRACUSE.
Egeon, art thou not? or else his ghost?

DROMIO OF SYRACUSE.
O, my old master, who hath bound him here?

ABBESS.
Whoever bound him, I will loose his bonds,
And gain a husband by his liberty.
Speak, old Egeon, if thou be’st the man
That hadst a wife once called Emilia,
That bore thee at a burden two fair sons.
O, if thou be’st the same Egeon, speak,
And speak unto the same Emilia!

DUKE.
Why, here begins his morning story right:
These two Antipholus’, these two so like,
And these two Dromios, one in semblance,
Besides her urging of her wreck at sea.
These are the parents to these children,
Which accidentally are met together.

EGEON.
If I dream not, thou art Emilia.
If thou art she, tell me where is that son
That floated with thee on the fatal raft?

ABBESS.
By men of Epidamnum, he and I
And the twin Dromio, all were taken up;
But, by and by, rude fishermen of Corinth
By force took Dromio and my son from them,
And me they left with those of Epidamnum.
What then became of them I cannot tell;
I to this fortune that you see me in.

DUKE.
Antipholus, thou cam’st from Corinth first?

ANTIPHOLUS OF SYRACUSE.
No, sir, not I, I came from Syracuse.

DUKE.
Stay, stand apart, I know not which is which.

ANTIPHOLUS OF EPHESUS.
I came from Corinth, my most gracious lord.

DROMIO OF EPHESUS.
And I with him.

ANTIPHOLUS OF EPHESUS.
Brought to this town by that most famous warrior,
Duke Menaphon, your most renowned uncle.

ADRIANA.
Which of you two did dine with me today?

ANTIPHOLUS OF SYRACUSE.
I, gentle mistress.

ADRIANA.
And are not you my husband?

ANTIPHOLUS OF EPHESUS.
No, I say nay to that.

ANTIPHOLUS OF SYRACUSE.
And so do I, yet did she call me so;
And this fair gentlewoman, her sister here,
Did call me brother. What I told you then,
I hope I shall have leisure to make good,
If this be not a dream I see and hear.

ANGELO.
That is the chain, sir, which you had of me.

ANTIPHOLUS OF SYRACUSE.
I think it be, sir. I deny it not.

ANTIPHOLUS OF EPHESUS.
And you, sir, for this chain arrested me.

ANGELO.
I think I did, sir. I deny it not.

ADRIANA.
I sent you money, sir, to be your bail
By Dromio, but I think he brought it not.

DROMIO OF EPHESUS.
No, none by me.

ANTIPHOLUS OF SYRACUSE.
This purse of ducats I receiv’d from you,
And Dromio my man did bring them me.
I see we still did meet each other’s man,
And I was ta’en for him, and he for me,
And thereupon these errors are arose.

ANTIPHOLUS OF EPHESUS.
These ducats pawn I for my father here.

DUKE.
It shall not need, thy father hath his life.

COURTESAN.
Sir, I must have that diamond from you.

ANTIPHOLUS OF EPHESUS.
There, take it, and much thanks for my good cheer.

ABBESS.
Renowned duke, vouchsafe to take the pains
To go with us into the abbey here,
And hear at large discoursed all our fortunes;
And all that are assembled in this place,
That by this sympathised one day’s error
Have suffer’d wrong, go, keep us company,
And we shall make full satisfaction.
Thirty-three years have I but gone in travail
Of you, my sons, and till this present hour
My heavy burden ne’er delivered.
The duke, my husband, and my children both,
And you, the calendars of their nativity,
Go to a gossips’ feast, and go with me.
After so long grief, such nativity.

DUKE.
With all my heart, I’ll gossip at this feast.

 [_Exeunt except the two Dromios and two Brothers._]

DROMIO OF SYRACUSE.
Master, shall I fetch your stuff from shipboard?

ANTIPHOLUS OF EPHESUS.
Dromio, what stuff of mine hast thou embark’d?

DROMIO OF SYRACUSE.
Your goods that lay at host, sir, in the Centaur.

ANTIPHOLUS OF SYRACUSE.
He speaks to me; I am your master, Dromio.
Come, go with us. We’ll look to that anon.
Embrace thy brother there, rejoice with him.

 [_Exeunt Antipholus of Syracuse and Antipholus of Ephesus._]

DROMIO OF SYRACUSE.
There is a fat friend at your master’s house,
That kitchen’d me for you today at dinner.
She now shall be my sister, not my wife.

DROMIO OF EPHESUS.
Methinks you are my glass, and not my brother.
I see by you I am a sweet-faced youth.
Will you walk in to see their gossiping?

DROMIO OF SYRACUSE.
Not I, sir, you are my elder.

DROMIO OF EPHESUS.
That’s a question, how shall we try it?

DROMIO OF SYRACUSE.
We’ll draw cuts for the senior. Till then, lead thou first.

DROMIO OF EPHESUS.
Nay, then, thus:
We came into the world like brother and brother,
And now let’s go hand in hand, not one before another.

 [_Exeunt._]




THE TRAGEDY OF CORIOLANUS




Contents

 ACT I
 Scene I. Rome. A street
 Scene II. Corioles. The Senate House
 Scene III. Rome. An apartment in Martius’ house
 Scene IV. Before Corioles
 Scene V. Within Corioles. A street
 Scene VI. Near the camp of Cominius
 Scene VII. The gates of Corioles
 Scene VIII. A field of battle between the Roman and the Volscian camps
 Scene IX. The Roman camp
 Scene X. The camp of the Volsces

 ACT II
 Scene I. Rome. A public place
 Scene II. Rome. The Capitol
 Scene III. Rome. The Forum

 ACT III
 Scene I. Rome. A street
 Scene II. Rome. A room in Coriolanus’s house
 Scene III. Rome. The Forum

 ACT IV
 Scene I. Rome. Before a gate of the city
 Scene II. Rome. A street near the gate
 Scene III. A highway between Rome and Antium
 Scene IV. Antium. Before Aufidius’s house
 Scene V. Antium. A hall in Aufidius’s house
 Scene VI. Rome. A public place
 Scene VII. A camp at a short distance from Rome

 ACT V
 Scene I. Rome. A public place
 Scene II. An Advanced post of the Volscian camp before Rome.
 Scene III. The tent of Coriolanus
 Scene IV. Rome. A public place
 Scene V. Rome. A street near the gate
 Scene VI. Antium. A public place




Dramatis Personæ

CAIUS MARTIUS CORIOLANUS, a noble Roman
VOLUMNIA, his mother
VIRGILIA, his wife
YOUNG MARTIUS, their son
VALERIA, friend to Volumnia and Virgilia
A GENTLEWOMAN, Volumnia’s attendant

MENENIUS AGRIPPA, Friend to Coriolanus
COMINIUS, General against the Volscians
TITUS LARTIUS, General against the Volscians
SICINIUS VELUTUS, Tribune of the People
JUNIUS BRUTUS, Tribune of the People
A ROMAN HERALD

TULLUS AUFIDIUS, General of the Volscians
LIEUTENANT, to Aufidius
Conspirators with Aufidius
A CITIZEN of Antium
TWO VOLSCIAN GUARDS

Roman and Volscian Senators, Patricians, Aediles, Lictors, Soldiers,
Citizens, Messengers, Servants to Aufidius, and other Attendants

SCENE: Partly in Rome, and partly in the territories of the Volscians
and Antiates.




ACT I

SCENE I. Rome. A street


Enter a company of mutinous Citizens, with staves, clubs, and other
weapons.

FIRST CITIZEN.
Before we proceed any further, hear me speak.

ALL.
Speak, speak!

FIRST CITIZEN.
You are all resolved rather to die than to famish?

ALL.
Resolved, resolved!

FIRST CITIZEN.
First, you know Caius Martius is chief enemy to the people.

ALL.
We know’t, we know’t!

FIRST CITIZEN.
Let us kill him, and we’ll have corn at our own price. Is’t a verdict?

ALL.
No more talking on’t; let it be done. Away, away!

SECOND CITIZEN.
One word, good citizens.

FIRST CITIZEN.
We are accounted poor citizens, the patricians good. What authority
surfeits on would relieve us. If they would yield us but the
superfluity while it were wholesome, we might guess they relieved us
humanely. But they think we are too dear. The leanness that afflicts
us, the object of our misery, is as an inventory to particularize their
abundance; our sufferance is a gain to them. Let us revenge this with
our pikes ere we become rakes; for the gods know I speak this in hunger
for bread, not in thirst for revenge.

SECOND CITIZEN.
Would you proceed especially against Caius Martius?

FIRST CITIZEN.
Against him first. He’s a very dog to the commonalty.

SECOND CITIZEN.
Consider you what services he has done for his country?

FIRST CITIZEN.
Very well, and could be content to give him good report for’t, but that
he pays himself with being proud.

SECOND CITIZEN.
Nay, but speak not maliciously.

FIRST CITIZEN.
I say unto you, what he hath done famously he did it to that end.
Though soft-conscienced men can be content to say it was for his
country, he did it to please his mother and to be partly proud, which
he is, even to the altitude of his virtue.

SECOND CITIZEN.
What he cannot help in his nature you account a vice in him. You must
in no way say he is covetous.

FIRST CITIZEN.
If I must not, I need not be barren of accusations. He hath faults,
with surplus, to tire in repetition. [_Shouts within_.] What shouts are
these? The other side o’ th’ city is risen. Why stay we prating here?
To th’ Capitol!

ALL.
Come, come!

Enter Menenius Agrippa.

FIRST CITIZEN.
Soft, who comes here?

SECOND CITIZEN.
Worthy Menenius Agrippa, one that hath always loved the people.

FIRST CITIZEN.
He’s one honest enough. Would all the rest were so!

MENENIUS.
What work’s, my countrymen, in hand? Where go you
With bats and clubs? The matter? Speak, I pray you.

FIRST CITIZEN.
Our business is not unknown to th’ Senate. They have had inkling this
fortnight what we intend to do, which now we’ll show ’em in deeds. They
say poor suitors have strong breaths; they shall know we have strong
arms too.

MENENIUS.
Why, masters, my good friends, mine honest neighbours,
Will you undo yourselves?

FIRST CITIZEN.
We cannot, sir; we are undone already.

MENENIUS.
I tell you, friends, most charitable care
Have the patricians of you. For your wants,
Your suffering in this dearth, you may as well
Strike at the heaven with your staves as lift them
Against the Roman state, whose course will on
The way it takes, cracking ten thousand curbs
Of more strong link asunder than can ever
Appear in your impediment. For the dearth,
The gods, not the patricians, make it, and
Your knees to them, not arms, must help. Alack,
You are transported by calamity
Thither where more attends you, and you slander
The helms o’ th’ state, who care for you like fathers,
When you curse them as enemies.

FIRST CITIZEN.
Care for us? True, indeed! They ne’er cared for us yet. Suffer us to
famish, and their storehouses crammed with grain; make edicts for usury
to support usurers; repeal daily any wholesome act established against
the rich, and provide more piercing statutes daily to chain up and
restrain the poor. If the wars eat us not up, they will; and there’s
all the love they bear us.

MENENIUS.
Either you must confess yourselves wondrous malicious
Or be accused of folly. I shall tell you
A pretty tale. It may be you have heard it,
But since it serves my purpose, I will venture
To stale’t a little more.

FIRST CITIZEN.
Well, I’ll hear it, sir; yet you must not think to fob off our disgrace
with a tale. But, an’t please you, deliver.

MENENIUS.
There was a time when all the body’s members
Rebelled against the belly, thus accused it:
That only like a gulf it did remain
I’ th’ midst o’ th’ body, idle and unactive,
Still cupboarding the viand, never bearing
Like labour with the rest, where th’ other instruments
Did see and hear, devise, instruct, walk, feel,
And, mutually participate, did minister
Unto the appetite and affection common
Of the whole body. The belly answered—

FIRST CITIZEN.
Well, sir, what answer made the belly?

MENENIUS.
Sir, I shall tell you. With a kind of smile,
Which ne’er came from the lungs, but even thus—
For, look you, I may make the belly smile
As well as speak—it tauntingly replied
To th’ discontented members, the mutinous parts
That envied his receipt; even so most fitly
As you malign our senators for that
They are not such as you.

FIRST CITIZEN.
Your belly’s answer—what?
The kingly crowned head, the vigilant eye,
The counsellor heart, the arm our soldier,
Our steed the leg, the tongue our trumpeter,
With other muniments and petty helps
Is this our fabric, if that they—

MENENIUS.
What then?
’Fore me, this fellow speaks. What then? What then?

FIRST CITIZEN.
Should by the cormorant belly be restrained,
Who is the sink o’ th’ body—

MENENIUS.
Well, what then?

FIRST CITIZEN.
The former agents, if they did complain,
What could the belly answer?

MENENIUS.
I will tell you,
If you’ll bestow a small—of what you have little—
Patience awhile, you’st hear the belly’s answer.

FIRST CITIZEN.
You are long about it.

MENENIUS.
Note me this, good friend;
Your most grave belly was deliberate,
Not rash like his accusers, and thus answered:
“True is it, my incorporate friends,” quoth he,
“That I receive the general food at first
Which you do live upon; and fit it is,
Because I am the storehouse and the shop
Of the whole body. But, if you do remember,
I send it through the rivers of your blood
Even to the court, the heart, to th’ seat o’ th’ brain;
And, through the cranks and offices of man,
The strongest nerves and small inferior veins
From me receive that natural competency
Whereby they live. And though that all at once,
You, my good friends”—this says the belly, mark me—

FIRST CITIZEN.
Ay, sir, well, well.

MENENIUS.
“Though all at once cannot
See what I do deliver out to each,
Yet I can make my audit up, that all
From me do back receive the flour of all,
And leave me but the bran.” What say you to’t?

FIRST CITIZEN.
It was an answer. How apply you this?

MENENIUS.
The senators of Rome are this good belly,
And you the mutinous members. For examine
Their counsels and their cares, digest things rightly
Touching the weal o’ th’ common, you shall find
No public benefit which you receive
But it proceeds or comes from them to you
And no way from yourselves. What do you think,
You, the great toe of this assembly?

FIRST CITIZEN.
I the great toe? Why the great toe?

MENENIUS.
For that, being one o’ th’ lowest, basest, poorest,
Of this most wise rebellion, thou goest foremost.
Thou rascal, that art worst in blood to run,
Lead’st first to win some vantage.
But make you ready your stiff bats and clubs.
Rome and her rats are at the point of battle;
The one side must have bale.

Enter Caius Martius.

Hail, noble Martius.

MARTIUS.
Thanks.—What’s the matter, you dissentious rogues,
That, rubbing the poor itch of your opinion,
Make yourselves scabs?

FIRST CITIZEN.
We have ever your good word.

MARTIUS.
He that will give good words to thee will flatter
Beneath abhorring. What would you have, you curs,
That like nor peace nor war? The one affrights you;
The other makes you proud. He that trusts to you,
Where he should find you lions, finds you hares;
Where foxes, geese. You are no surer, no,
Than is the coal of fire upon the ice
Or hailstone in the sun. Your virtue is
To make him worthy whose offence subdues him,
And curse that justice did it. Who deserves greatness
Deserves your hate; and your affections are
A sick man’s appetite, who desires most that
Which would increase his evil. He that depends
Upon your favours swims with fins of lead,
And hews down oaks with rushes. Hang ye! Trust ye?
With every minute you do change a mind
And call him noble that was now your hate,
Him vile that was your garland. What’s the matter,
That in these several places of the city
You cry against the noble senate, who,
Under the gods, keep you in awe, which else
Would feed on one another?—What’s their seeking?

MENENIUS.
For corn at their own rates, whereof they say
The city is well stored.

MARTIUS.
Hang ’em! They say?
They’ll sit by th’ fire and presume to know
What’s done i’ th’ Capitol, who’s like to rise,
Who thrives and who declines; side factions and give out
Conjectural marriages, making parties strong
And feebling such as stand not in their liking
Below their cobbled shoes. They say there’s grain enough?
Would the nobility lay aside their ruth
And let me use my sword, I’d make a quarry
With thousands of these quartered slaves as high
As I could pick my lance.

MENENIUS.
Nay, these are almost thoroughly persuaded;
For though abundantly they lack discretion,
Yet are they passing cowardly. But I beseech you,
What says the other troop?

MARTIUS.
They are dissolved. Hang ’em!
They said they were an-hungry, sighed forth proverbs
That hunger broke stone walls, that dogs must eat,
That meat was made for mouths, that the gods sent not
Corn for the rich men only. With these shreds
They vented their complainings, which being answered
And a petition granted them—a strange one,
To break the heart of generosity
And make bold power look pale—they threw their caps
As they would hang them on the horns o’ th’ moon,
Shouting their emulation.

MENENIUS.
What is granted them?

MARTIUS.
Five tribunes to defend their vulgar wisdoms,
Of their own choice. One’s Junius Brutus,
Sicinius Velutus, and I know not. ’Sdeath!
The rabble should have first unroofed the city
Ere so prevailed with me. It will in time
Win upon power and throw forth greater themes
For insurrection’s arguing.

MENENIUS.
This is strange.

MARTIUS.
Go get you home, you fragments.

Enter a Messenger hastily.

MESSENGER.
Where’s Caius Martius?

MARTIUS.
Here. What’s the matter?

MESSENGER.
The news is, sir, the Volsces are in arms.

MARTIUS.
I am glad on’t. Then we shall ha’ means to vent
Our musty superfluity.

Enter Sicinius Velutus, Junius Brutus, two Tribunes; Cominius, Titus
Lartius with other Senators.

See, our best elders.

FIRST SENATOR.
Martius, ’tis true that you have lately told us:
The Volsces are in arms.

MARTIUS.
They have a leader,
Tullus Aufidius, that will put you to’t.
I sin in envying his nobility,
And, were I anything but what I am,
I would wish me only he.

COMINIUS.
You have fought together.

MARTIUS.
Were half to half the world by th’ ears and he
Upon my party, I’d revolt, to make
Only my wars with him. He is a lion
That I am proud to hunt.

FIRST SENATOR.
Then, worthy Martius,
Attend upon Cominius to these wars.

COMINIUS.
It is your former promise.

MARTIUS.
Sir, it is,
And I am constant.—Titus Lartius, thou
Shalt see me once more strike at Tullus’ face.
What, art thou stiff? Stand’st out?

TITUS LARTIUS.
No, Caius Martius,
I’ll lean upon one crutch and fight with th’ other
Ere stay behind this business.

MENENIUS.
O, true bred!

FIRST SENATOR.
Your company to th’ Capitol, where I know
Our greatest friends attend us.

TITUS LARTIUS.
Lead you on.
Follow Cominius. We must follow you;
Right worthy your priority.

COMINIUS.
Noble Martius.

FIRST SENATOR.
[_To the Citizens_.]
Hence to your homes, begone.

MARTIUS.
Nay, let them follow.
The Volsces have much corn; take these rats thither
To gnaw their garners. Worshipful mutineers,
Your valour puts well forth. Pray follow.

[_Exeunt. Sicinius and Brutus remain_.]

SICINIUS.
Was ever man so proud as is this Martius?

BRUTUS.
He has no equal.

SICINIUS.
When we were chosen tribunes for the people—

BRUTUS.
Marked you his lip and eyes?

SICINIUS.
Nay, but his taunts.

BRUTUS.
Being moved, he will not spare to gird the gods.

SICINIUS.
Bemock the modest moon.

BRUTUS.
The present wars devour him! He is grown
Too proud to be so valiant.

SICINIUS.
Such a nature,
Tickled with good success, disdains the shadow
Which he treads on at noon. But I do wonder
His insolence can brook to be commanded
Under Cominius.

BRUTUS.
Fame, at the which he aims,
In whom already he’s well graced, cannot
Better be held nor more attained than by
A place below the first; for what miscarries
Shall be the General’s fault, though he perform
To th’ utmost of a man, and giddy censure
Will then cry out of Martius “O, if he
Had borne the business!”

SICINIUS.
Besides, if things go well,
Opinion that so sticks on Martius shall
Of his demerits rob Cominius.

BRUTUS.
Come.
Half all Cominius’ honours are to Martius,
Though Martius earned them not, and all his faults
To Martius shall be honours, though indeed
In aught he merit not.

SICINIUS.
Let’s hence and hear
How the dispatch is made, and in what fashion,
More than in singularity, he goes
Upon this present action.

BRUTUS.
Let’s along.

[_Exeunt._]

SCENE II. Corioles. The Senate House

Enter Tullus Aufidius with Senators of Corioles.

FIRST SENATOR.
So, your opinion is, Aufidius,
That they of Rome are entered in our counsels
And know how we proceed.

AUFIDIUS.
Is it not yours?
What ever have been thought on in this state
That could be brought to bodily act ere Rome
Had circumvention? ’Tis not four days gone
Since I heard thence. These are the words—I think
I have the letter here. Yes, here it is.
[_Reads_.] _They have pressed a power, but it is not known
Whether for east or west. The dearth is great.
The people mutinous; and, it is rumoured,
Cominius, Martius your old enemy,
Who is of Rome worse hated than of you,—
And Titus Lartius, a most valiant Roman,
These three lead on this preparation
Whither ’tis bent. Most likely ’tis for you.
Consider of it._

FIRST SENATOR.
Our army’s in the field.
We never yet made doubt but Rome was ready
To answer us.

AUFIDIUS.
Nor did you think it folly
To keep your great pretences veiled till when
They needs must show themselves, which, in the hatching,
It seemed, appeared to Rome. By the discovery
We shall be shortened in our aim, which was
To take in many towns ere almost Rome
Should know we were afoot.

SECOND SENATOR.
Noble Aufidius,
Take your commission; hie you to your bands.
Let us alone to guard Corioles.
If they set down before’s, for the remove
Bring up your army. But I think you’ll find
They’ve not prepared for us.

AUFIDIUS.
O, doubt not that;
I speak from certainties. Nay, more,
Some parcels of their power are forth already,
And only hitherward. I leave your Honours.
If we and Caius Martius chance to meet,
’Tis sworn between us we shall ever strike
Till one can do no more.

ALL.
The gods assist you!

AUFIDIUS.
And keep your Honours safe!

FIRST SENATOR.
Farewell.

SECOND SENATOR.
Farewell.

ALL.
Farewell.

[_Exeunt._]

SCENE III. Rome. An apartment in Martius’ house

Enter Volumnia and Virgilia, mother and wife to Martius. They set them
down on two low stools and sew.

VOLUMNIA.
I pray you, daughter, sing, or express yourself in a more comfortable
sort. If my son were my husband, I should freelier rejoice in that
absence wherein he won honour than in the embracements of his bed where
he would show most love. When yet he was but tender-bodied and the only
son of my womb, when youth with comeliness plucked all gaze his way,
when for a day of kings’ entreaties a mother should not sell him an
hour from her beholding, I, considering how honour would become such a
person—that it was no better than picture-like to hang by th’ wall, if
renown made it not stir—was pleased to let him seek danger where he was
like to find fame. To a cruel war I sent him, from whence he returned,
his brows bound with oak. I tell thee, daughter, I sprang not more in
joy at first hearing he was a man-child than now in first seeing he had
proved himself a man.

VIRGILIA.
But had he died in the business, madam, how then?

VOLUMNIA.
Then his good report should have been my son; I therein would have
found issue. Hear me profess sincerely: had I a dozen sons, each in my
love alike and none less dear than thine and my good Martius, I had
rather had eleven die nobly for their country than one voluptuously
surfeit out of action.

Enter a Gentlewoman.

GENTLEWOMAN.
Madam, the Lady Valeria is come to visit you.

VIRGILIA.
Beseech you, give me leave to retire myself.

VOLUMNIA.
Indeed you shall not.
Methinks I hear hither your husband’s drum,
See him pluck Aufidius down by th’ hair;
As children from a bear, the Volsces shunning him.
Methinks I see him stamp thus and call thus:
“Come on, you cowards! You were got in fear,
Though you were born in Rome.” His bloody brow
With his mailed hand then wiping, forth he goes
Like to a harvestman that’s tasked to mow
Or all or lose his hire.

VIRGILIA.
His bloody brow? O Jupiter, no blood!

VOLUMNIA.
Away, you fool! It more becomes a man
Than gilt his trophy. The breasts of Hecuba,
When she did suckle Hector, looked not lovelier
Than Hector’s forehead when it spit forth blood
At Grecian sword, contemning.—Tell Valeria
We are fit to bid her welcome.

[_Exit Gentlewoman._]

VIRGILIA.
Heavens bless my lord from fell Aufidius!

VOLUMNIA.
He’ll beat Aufidius’ head below his knee
And tread upon his neck.

Enter Valeria with an Usher and a Gentlewoman.

VALERIA.
My ladies both, good day to you.

VOLUMNIA.
Sweet madam.

VIRGILIA.
I am glad to see your Ladyship.

VALERIA.
How do you both? You are manifest housekeepers. What are you sewing
here? A fine spot, in good faith. How does your little son?

VIRGILIA.
I thank your Ladyship; well, good madam.

VOLUMNIA.
He had rather see the swords and hear a drum than look upon his
schoolmaster.

VALERIA.
O’ my word, the father’s son! I’ll swear ’tis a very pretty boy. O’ my
troth, I looked upon him o’ Wednesday half an hour together. H’as such
a confirmed countenance. I saw him run after a gilded butterfly, and
when he caught it, he let it go again, and after it again, and over and
over he comes, and up again, catched it again. Or whether his fall
enraged him or how ’twas, he did so set his teeth and tear it. O, I
warrant how he mammocked it!

VOLUMNIA.
One on’s father’s moods.

VALERIA.
Indeed, la, ’tis a noble child.

VIRGILIA.
A crack, madam.

VALERIA.
Come, lay aside your stitchery. I must have you play the idle huswife
with me this afternoon.

VIRGILIA.
No, good madam, I will not out of doors.

VALERIA.
Not out of doors?

VOLUMNIA.
She shall, she shall.

VIRGILIA.
Indeed, no, by your patience. I’ll not over the threshold till my lord
return from the wars.

VALERIA.
Fie, you confine yourself most unreasonably. Come, you must go visit
the good lady that lies in.

VIRGILIA.
I will wish her speedy strength and visit her with my prayers, but I
cannot go thither.

VOLUMNIA.
Why, I pray you?

VIRGILIA.
’Tis not to save labour, nor that I want love.

VALERIA.
You would be another Penelope. Yet they say all the yarn she spun in
Ulysses’ absence did but fill Ithaca full of moths. Come, I would your
cambric were sensible as your finger, that you might leave pricking it
for pity. Come, you shall go with us.

VIRGILIA.
No, good madam, pardon me; indeed, I will not forth.

VALERIA.
In truth, la, go with me, and I’ll tell you excellent news of your
husband.

VIRGILIA.
O, good madam, there can be none yet.

VALERIA.
Verily, I do not jest with you. There came news from him last night.

VIRGILIA.
Indeed, madam!

VALERIA.
In earnest, it’s true. I heard a senator speak it. Thus it is: the
Volsces have an army forth, against whom Cominius the General is gone
with one part of our Roman power. Your lord and Titus Lartius are set
down before their city Corioles. They nothing doubt prevailing, and to
make it brief wars. This is true, on mine honour, and so, I pray, go
with us.

VIRGILIA.
Give me excuse, good madam. I will obey you in everything hereafter.

VOLUMNIA.
Let her alone, lady. As she is now, she will but disease our better
mirth.

VALERIA.
In troth, I think she would.—Fare you well, then.—Come, good sweet
lady.—Prithee, Virgilia, turn thy solemness out o’ door, and go along
with us.

VIRGILIA.
No, at a word, madam. Indeed I must not. I wish you much mirth.

VALERIA.
Well then, farewell.

[_Exeunt._]

SCENE IV. Before Corioles

Enter Martius, Titus Lartius, with drum and colours, with Captains and
Soldiers, as before the city of Corioles. To them a Messenger.

MARTIUS.
Yonder comes news. A wager they have met.

LARTIUS.
My horse to yours, no.

MARTIUS.
’Tis done.

LARTIUS.
Agreed.

MARTIUS.
[_To Messenger_.] Say, has our general met the enemy?

MESSENGER.
They lie in view but have not spoke as yet.

LARTIUS.
So the good horse is mine.

MARTIUS.
I’ll buy him of you.

LARTIUS.
No, I’ll nor sell nor give him. Lend you him I will
For half a hundred years.—Summon the town.

MARTIUS.
How far off lie these armies?

MESSENGER.
Within this mile and half.

MARTIUS.
Then shall we hear their ’larum, and they ours.
Now, Mars, I prithee, make us quick in work,
That we with smoking swords may march from hence
To help our fielded friends!—Come, blow thy blast.

[_They sound a parley._]

Enter two Senators with others on the walls of Corioles.

Tullus Aufidius, is he within your walls?

FIRST SENATOR.
No, nor a man that fears you less than he:
That’s lesser than a little.
[_Drum afar off_.]
Hark, our drums
Are bringing forth our youth. We’ll break our walls
Rather than they shall pound us up. Our gates,
Which yet seem shut, we have but pinned with rushes.
They’ll open of themselves.
[_Alarum far off_.]
Hark you, far off!
There is Aufidius. List what work he makes
Amongst your cloven army.

MARTIUS.
O, they are at it!

LARTIUS.
Their noise be our instruction.—Ladders, ho!

Enter the Army of the Volsces as through the city gates.

MARTIUS.
They fear us not but issue forth their city.—
Now put your shields before your hearts, and fight
With hearts more proof than shields.—Advance, brave Titus.
They do disdain us much beyond our thoughts,
Which makes me sweat with wrath.—Come on, my fellows!
He that retires, I’ll take him for a Volsce,
And he shall feel mine edge.

[_Alarums. The Romans are beat back to their trenches. They exit, with
the Volsces following_.]

Enter Martius cursing, with Roman soldiers.

MARTIUS.
All the contagion of the south light on you,
You shames of Rome! You herd of—Boils and plagues
Plaster you o’er, that you may be abhorred
Farther than seen, and one infect another
Against the wind a mile! You souls of geese,
That bear the shapes of men, how have you run
From slaves that apes would beat! Pluto and hell!
All hurt behind. Backs red, and faces pale
With flight and agued fear! Mend, and charge home,
Or, by the fires of heaven, I’ll leave the foe
And make my wars on you. Look to’t. Come on!
If you’ll stand fast we’ll beat them to their wives,
As they us to our trenches. Follow’s!

[_Another alarum. The Volsces re-enter and are driven back to the gates
of Corioles, which open to admit them._]

So, now the gates are ope. Now prove good seconds!
’Tis for the followers fortune widens them,
Not for the fliers. Mark me, and do the like.

[_Martius follows the fleeing Volsces through the gates, and is shut
in._]

FIRST SOLDIER.
Foolhardiness, not I.

SECOND SOLDIER.
Nor I.

FIRST SOLDIER.
See, they have shut him in.

[_Alarum continues._]

ALL.
To th’ pot, I warrant him.

Enter Titus Lartius.

LARTIUS.
What is become of Martius?

ALL.
Slain, sir, doubtless.

FIRST SOLDIER.
Following the fliers at the very heels,
With them he enters, who upon the sudden
Clapped to their gates. He is himself alone,
To answer all the city.

LARTIUS.
O noble fellow,
Who sensibly outdares his senseless sword,
And when it bows, stand’st up! Thou art left, Martius.
A carbuncle entire, as big as thou art,
Were not so rich a jewel. Thou wast a soldier
Even to Cato’s wish, not fierce and terrible
Only in strokes, but with thy grim looks and
The thunderlike percussion of thy sounds
Thou mad’st thine enemies shake, as if the world
Were feverous and did tremble.

Enter Martius, bleeding, assaulted by the enemy.

FIRST SOLDIER.
Look, sir.

LARTIUS.
O, ’tis Martius!
Let’s fetch him off or make remain alike.

[_They fight, and all enter the city._]

SCENE V. Within Corioles. A street

Enter certain Romans, with spoils.

FIRST ROMAN.
This will I carry to Rome.

SECOND ROMAN.
And I this.

THIRD ROMAN.
A murrain on’t! I took this for silver.

Enter Martius and Titus Lartius with a Trumpet.

MARTIUS.
See here these movers that do prize their hours
At a cracked drachma. Cushions, leaden spoons,
Irons of a doit, doublets that hangmen would
Bury with those that wore them, these base slaves,
Ere yet the fight be done, pack up. Down with them!

[_Exit the Romans with spoils._]

[_Alarum continues still afar off._]

And hark, what noise the General makes! To him!
There is the man of my soul’s hate, Aufidius,
Piercing our Romans. Then, valiant Titus, take
Convenient numbers to make good the city,
Whilst I, with those that have the spirit, will haste
To help Cominius.

LARTIUS.
Worthy sir, thou bleed’st.
Thy exercise hath been too violent
For a second course of fight.

MARTIUS.
Sir, praise me not.
My work hath yet not warmed me. Fare you well.
The blood I drop is rather physical
Than dangerous to me. To Aufidius thus
I will appear and fight.

LARTIUS.
Now the fair goddess Fortune
Fall deep in love with thee, and her great charms
Misguide thy opposers’ swords! Bold gentleman,
Prosperity be thy page!

MARTIUS.
Thy friend no less
Than those she placeth highest! So farewell.

LARTIUS.
Thou worthiest Martius!

[_Exit Martius._]

Go sound thy trumpet in the marketplace.
Call thither all the officers o’ th’ town,
Where they shall know our mind. Away!

[_Exeunt._]

SCENE VI. Near the camp of Cominius

Enter Cominius as it were in retire, with Soldiers.

COMINIUS.
Breathe you, my friends. Well fought! We are come off
Like Romans, neither foolish in our stands
Nor cowardly in retire. Believe me, sirs,
We shall be charged again. Whiles we have struck,
By interims and conveying gusts we have heard
The charges of our friends. The Roman gods
Lead their successes as we wish our own,
That both our powers, with smiling fronts encount’ring,
May give you thankful sacrifice!

Enter a Messenger.

Thy news?

MESSENGER.
The citizens of Corioles have issued,
And given to Lartius and to Martius battle.
I saw our party to their trenches driven,
And then I came away.

COMINIUS.
Though thou speakest truth,
Methinks thou speak’st not well. How long is’t since?

MESSENGER.
Above an hour, my lord.

COMINIUS.
’Tis not a mile; briefly we heard their drums.
How couldst thou in a mile confound an hour
And bring thy news so late?

MESSENGER.
Spies of the Volsces
Held me in chase, that I was forced to wheel
Three or four miles about; else had I, sir,
Half an hour since brought my report.

[_Exit Messenger._]

Enter Martius, bloody.

COMINIUS.
Who’s yonder,
That does appear as he were flayed? O gods,
He has the stamp of Martius, and I have
Before-time seen him thus.

MARTIUS.
Come I too late?

COMINIUS.
The shepherd knows not thunder from a tabor
More than I know the sound of Martius’ tongue
From every meaner man.

MARTIUS.
Come I too late?

COMINIUS.
Ay, if you come not in the blood of others,
But mantled in your own.

MARTIUS.
O, let me clip you
In arms as sound as when I wooed, in heart
As merry as when our nuptial day was done
And tapers burned to bedward!

COMINIUS.
Flower of warriors, how is’t with Titus Lartius?

MARTIUS.
As with a man busied about decrees,
Condemning some to death and some to exile;
Ransoming him or pitying, threat’ning the other;
Holding Corioles in the name of Rome
Even like a fawning greyhound in the leash,
To let him slip at will.

COMINIUS.
Where is that slave
Which told me they had beat you to your trenches?
Where’s he? Call him hither.

MARTIUS.
Let him alone.
He did inform the truth. But for our gentlemen,
The common file—a plague! Tribunes for them!—
The mouse ne’er shunned the cat as they did budge
From rascals worse than they.

COMINIUS.
But how prevailed you?

MARTIUS.
Will the time serve to tell? I do not think.
Where is the enemy? Are you lords o’ th’ field?
If not, why cease you till you are so?

COMINIUS.
Martius, we have at disadvantage fought,
And did retire to win our purpose.

MARTIUS.
How lies their battle? Know you on which side
They have placed their men of trust?

COMINIUS.
As I guess, Martius,
Their bands i’ th’ vaward are the Antiates,
Of their best trust; o’er them Aufidius,
Their very heart of hope.

MARTIUS.
I do beseech you,
By all the battles wherein we have fought,
By th’ blood we have shed together, by th’ vows we have made
To endure friends, that you directly set me
Against Aufidius and his Antiates,
And that you not delay the present, but,
Filling the air with swords advanced and darts,
We prove this very hour.

COMINIUS.
Though I could wish
You were conducted to a gentle bath
And balms applied to you, yet dare I never
Deny your asking. Take your choice of those
That best can aid your action.

MARTIUS.
Those are they
That most are willing. If any such be here—
As it were sin to doubt—that love this painting
Wherein you see me smeared; if any fear
Lesser his person than an ill report;
If any think brave death outweighs bad life,
And that his country’s dearer than himself;
Let him alone, or so many so minded,
Wave thus to express his disposition
And follow Martius.

[_He waves his sword._]

[_They all shout and wave their swords, take him up in their arms, and
cast up their caps._]

O, me alone! Make you a sword of me?
If these shows be not outward, which of you
But is four Volsces? None of you but is
Able to bear against the great Aufidius
A shield as hard as his. A certain number,
Though thanks to all, must I select from all.
The rest shall bear the business in some other fight,
As cause will be obeyed. Please you to march,
And I shall quickly draw out my command,
Which men are best inclined.

COMINIUS.
March on, my fellows.
Make good this ostentation, and you shall
Divide in all with us.

[_Exeunt._]

SCENE VII. The gates of Corioles

Titus Lartius, having set a guard upon Corioles, going with drum and
trumpet toward Cominius and Caius Martius, enters with a Lieutenant,
other Soldiers, and a Scout.

LARTIUS.
So, let the ports be guarded. Keep your duties
As I have set them down. If I do send, dispatch
Those centuries to our aid; the rest will serve
For a short holding. If we lose the field,
We cannot keep the town.

LIEUTENANT.
Fear not our care, sir.

LARTIUS.
Hence, and shut your gates upon’s.
Our guider, come. To th’ Roman camp conduct us.

[_Exeunt._]

SCENE VIII. A field of battle between the Roman and the Volscian camps

Alarum, as in battle. Enter Martius and Aufidius at several doors.

MARTIUS.
I’ll fight with none but thee, for I do hate thee
Worse than a promise-breaker.

AUFIDIUS.
We hate alike.
Not Afric owns a serpent I abhor
More than thy fame and envy. Fix thy foot.

MARTIUS.
Let the first budger die the other’s slave,
And the gods doom him after!

AUFIDIUS.
If I fly, Martius,
Hollo me like a hare.

MARTIUS.
Within these three hours, Tullus,
Alone I fought in your Corioles’ walls,
And made what work I pleased. ’Tis not my blood
Wherein thou seest me masked. For thy revenge
Wrench up thy power to th’ highest.

AUFIDIUS.
Wert thou the Hector
That was the whip of your bragged progeny,
Thou shouldst not scape me here.

[_Here they fight, and certain Volsces come to the aid of Aufidius._]

Officious and not valiant, you have shamed me
In your condemned seconds.

[_Martius fights till they be driven in breathless. Aufidius and
Martius exit, separately._]

SCENE IX. The Roman camp

Alarum. A retreat is sounded. Flourish. Enter, at one door, Cominius
with the Romans; at another door, Martius, with his arm in a scarf.

COMINIUS.
If I should tell thee o’er this thy day’s work,
Thou’t not believe thy deeds. But I’ll report it
Where senators shall mingle tears with smiles;
Where great patricians shall attend and shrug,
I’ th’ end admire; where ladies shall be frighted
And, gladly quaked, hear more; where the dull tribunes,
That with the fusty plebeians hate thine honours,
Shall say against their hearts “We thank the gods
Our Rome hath such a soldier.”
Yet cam’st thou to a morsel of this feast,
Having fully dined before.

Enter Titus Lartius with his power, from the pursuit.

LARTIUS.
O general,
Here is the steed, we the caparison.
Hadst thou beheld—

MARTIUS.
Pray now, no more. My mother,
Who has a charter to extol her blood,
When she does praise me grieves me. I have done
As you have done—that’s what I can;
Induced as you have been—that’s for my country.
He that has but effected his good will
Hath overta’en mine act.

COMINIUS.
You shall not be
The grave of your deserving. Rome must know
The value of her own. ’Twere a concealment
Worse than a theft, no less than a traducement,
To hide your doings and to silence that
Which, to the spire and top of praises vouched,
Would seem but modest. Therefore, I beseech you—
In sign of what you are, not to reward
What you have done—before our army hear me.

MARTIUS.
I have some wounds upon me, and they smart
To hear themselves remembered.

COMINIUS.
Should they not,
Well might they fester ’gainst ingratitude
And tent themselves with death. Of all the horses—
Whereof we have ta’en good and good store—of all
The treasure in this field achieved and city,
We render you the tenth, to be ta’en forth
Before the common distribution
At your only choice.

MARTIUS.
I thank you, general,
But cannot make my heart consent to take
A bribe to pay my sword. I do refuse it;
And stand upon my common part with those
That have beheld the doing.

[_A long flourish. They all cry “Martius, Martius!” and cast up their
caps and lances. Cominius and Lartius stand bare._]

May these same instruments which, you profane,
Never sound more! When drums and trumpets shall
I’ th’ field prove flatterers, let courts and cities be
Made all of false-faced soothing! When steel grows soft
Soft as the parasite’s silk, let him be made
An ovator for the wars! No more, I say.
For that I have not washed my nose that bled,
Or foiled some debile wretch—which, without note,
Here’s many else have done—you shout me forth
In acclamations hyperbolical,
As if I loved my little should be dieted
In praises sauced with lies.

COMINIUS.
Too modest are you,
More cruel to your good report than grateful
To us that give you truly. By your patience,
If ’gainst yourself you be incensed, we’ll put you,
Like one that means his proper harm, in manacles,
Then reason safely with you. Therefore be it known,
As to us to all the world, that Caius Martius
Wears this war’s garland, in token of the which
My noble steed, known to the camp, I give him,
With all his trim belonging. And from this time,
For what he did before Corioles, call him,
With all th’ applause and clamour of the host,
Caius Martius Coriolanus! Bear
Th’ addition nobly ever!

[_Flourish. Trumpets sound, and drums._]

ALL.
Caius Martius Coriolanus!

CORIOLANUS.
I will go wash;
And when my face is fair, you shall perceive
Whether I blush or no. Howbeit, I thank you.
I mean to stride your steed and at all times
To undercrest your good addition
To th’ fairness of my power.

COMINIUS.
So, to our tent,
Where, ere we do repose us, we will write
To Rome of our success.—You, Titus Lartius,
Must to Corioles back. Send us to Rome
The best, with whom we may articulate
For their own good and ours.

LARTIUS.
I shall, my lord.

CORIOLANUS.
The gods begin to mock me. I, that now
Refused most princely gifts, am bound to beg
Of my lord general.

COMINIUS.
Take’t, ’tis yours. What is’t?

CORIOLANUS.
I sometime lay here in Corioles
At a poor man’s house; he used me kindly.
He cried to me; I saw him prisoner;
But then Aufidius was within my view,
And wrath o’erwhelmed my pity. I request you
To give my poor host freedom.

COMINIUS.
O, well begged!
Were he the butcher of my son, he should
Be free as is the wind.—Deliver him, Titus.

LARTIUS.
Martius, his name?

CORIOLANUS.
By Jupiter, forgot!
I am weary; yea, my memory is tired.
Have we no wine here?

COMINIUS.
Go we to our tent.
The blood upon your visage dries; ’tis time
It should be looked to. Come.

[_A flourish of cornets. Exeunt._]

SCENE X. The camp of the Volsces

A flourish. Cornets. Enter Tullus Aufidius, bloody, with two or three
soldiers.

AUFIDIUS.
The town is ta’en.

SOLDIER.
’Twill be delivered back on good condition.

AUFIDIUS.
Condition?
I would I were a Roman, for I cannot,
Being a Volsce, be that I am. Condition?
What good condition can a treaty find
I’ th’ part that is at mercy? Five times, Martius,
I have fought with thee; so often hast thou beat me
And wouldst do so, I think, should we encounter
As often as we eat. By th’ elements,
If e’er again I meet him beard to beard,
He’s mine or I am his. Mine emulation
Hath not that honour in’t it had; for where
I thought to crush him in an equal force,
True sword to sword, I’ll potch at him some way,
Or wrath or craft may get him.

SOLDIER.
He’s the devil.

AUFIDIUS.
Bolder, though not so subtle. My valour’s poisoned
With only suff’ring stain by him; for him
Shall fly out of itself. Nor sleep nor sanctuary,
Being naked, sick, nor fane nor Capitol,
The prayers of priests nor times of sacrifice,
Embarquements all of fury, shall lift up
Their rotten privilege and custom ’gainst
My hate to Martius. Where I find him, were it
At home, upon my brother’s guard, even there,
Against the hospitable canon, would I
Wash my fierce hand in’s heart. Go you to th’ city;
Learn how ’tis held and what they are that must
Be hostages for Rome.

SOLDIER.
Will not you go?

AUFIDIUS.
I am attended at the cypress grove. I pray you—
’Tis south the city mills,—bring me word thither
How the world goes, that to the pace of it
I may spur on my journey.

SOLDIER.
I shall, sir.

[_Exeunt._]




ACT II

SCENE I. Rome. A public place


Enter Menenius with the two Tribunes of the people, Sicinius and
Brutus.

MENENIUS.
The augurer tells me we shall have news tonight.

BRUTUS.
Good or bad?

MENENIUS.
Not according to the prayer of the people, for they love not Martius.

SICINIUS.
Nature teaches beasts to know their friends.

MENENIUS.
Pray you, who does the wolf love?

SICINIUS.
The lamb.

MENENIUS.
Ay, to devour him, as the hungry plebeians would the noble Martius.

BRUTUS.
He’s a lamb indeed, that baas like a bear.

MENENIUS.
He’s a bear indeed, that lives like a lamb. You two are old men; tell
me one thing that I shall ask you.

BOTH TRIBUNES.
Well, sir.

MENENIUS.
In what enormity is Martius poor in, that you two have not in
abundance?

BRUTUS.
He’s poor in no one fault, but stored with all.

SICINIUS.
Especially in pride.

BRUTUS.
And topping all others in boasting.

MENENIUS.
This is strange now. Do you two know how you are censured here in the
city, I mean of us o’ th’ right-hand file, do you?

BOTH TRIBUNES.
Why, how are we censured?

MENENIUS.
Because you talk of pride now, will you not be angry?

BOTH TRIBUNES.
Well, well, sir, well?

MENENIUS.
Why, ’tis no great matter; for a very little thief of occasion will rob
you of a great deal of patience. Give your dispositions the reins, and
be angry at your pleasures, at the least, if you take it as a pleasure
to you in being so. You blame Martius for being proud.

BRUTUS.
We do it not alone, sir.

MENENIUS.
I know you can do very little alone, for your helps are many, or else
your actions would grow wondrous single. Your abilities are too
infantlike for doing much alone. You talk of pride. O that you could
turn your eyes toward the napes of your necks and make but an interior
survey of your good selves! O, that you could!

BOTH TRIBUNES.
What then, sir?

MENENIUS.
Why, then you should discover a brace of unmeriting, proud, violent,
testy magistrates, alias fools, as any in Rome.

SICINIUS.
Menenius, you are known well enough, too.

MENENIUS.
I am known to be a humorous patrician and one that loves a cup of hot
wine with not a drop of allaying Tiber in’t; said to be something
imperfect in favouring the first complaint, hasty and tinder-like upon
too trivial motion; one that converses more with the buttock of the
night than with the forehead of the morning. What I think I utter, and
spend my malice in my breath. Meeting two such wealsmen as you are—I
cannot call you Lycurguses—if the drink you give me touch my palate
adversely, I make a crooked face at it. I cannot say your Worships have
delivered the matter well when I find the ass in compound with the
major part of your syllables. And though I must be content to bear with
those that say you are reverend grave men, yet they lie deadly that
tell you have good faces. If you see this in the map of my microcosm,
follows it that I am known well enough too? What harm can your bisson
conspectuities glean out of this character, if I be known well enough,
too?

BRUTUS.
Come, sir, come; we know you well enough.

MENENIUS.
You know neither me, yourselves, nor anything. You are ambitious for
poor knaves’ caps and legs. You wear out a good wholesome forenoon in
hearing a cause between an orange-wife and a faucet-seller, and then
rejourn the controversy of threepence to a second day of audience. When
you are hearing a matter between party and party, if you chance to be
pinched with the colic, you make faces like mummers, set up the bloody
flag against all patience, and, in roaring for a chamber pot, dismiss
the controversy bleeding, the more entangled by your hearing. All the
peace you make in their cause is calling both the parties knaves. You
are a pair of strange ones.

BRUTUS.
Come, come. You are well understood to be a perfecter giber for the
table than a necessary bencher in the Capitol.

MENENIUS.
Our very priests must become mockers if they shall encounter such
ridiculous subjects as you are. When you speak best unto the purpose,
it is not worth the wagging of your beards, and your beards deserve not
so honourable a grave as to stuff a botcher’s cushion or to be entombed
in an ass’s packsaddle. Yet you must be saying Martius is proud, who,
in a cheap estimation, is worth all your predecessors since Deucalion,
though peradventure some of the best of ’em were hereditary hangmen.
Good e’en to your Worships. More of your conversation would infect my
brain, being the herdsmen of the beastly plebeians. I will be bold to
take my leave of you.

[_He begins to exit. Brutus and Sicinius stand aside._]

Enter Volumnia, Virgilia and Valeria

How now, my as fair as noble ladies—and the moon, were she earthly, no
nobler—whither do you follow your eyes so fast?

VOLUMNIA.
Honourable Menenius, my boy Martius approaches. For the love of Juno,
let’s go!

MENENIUS.
Ha? Martius coming home?

VOLUMNIA.
Ay, worthy Menenius, and with most prosperous approbation.

MENENIUS.
Take my cap, Jupiter, and I thank thee! Hoo! Martius coming home?

VALERIA, VIRGILIA.
Nay, ’tis true.

VOLUMNIA.
Look, here’s a letter from him. The state hath another, his wife
another, and I think there’s one at home for you.

MENENIUS.
I will make my very house reel tonight. A letter for me?

VIRGILIA.
Yes, certain, there’s a letter for you; I saw it.

MENENIUS.
A letter for me? It gives me an estate of seven years’ health, in which
time I will make a lip at the physician. The most sovereign
prescription in Galen is but empiricutic and, to this preservative, of
no better report than a horse drench. Is he not wounded? He was wont to
come home wounded.

VIRGILIA.
O, no, no, no!

VOLUMNIA.
O, he is wounded, I thank the gods for’t.

MENENIUS.
So do I too, if it be not too much. Brings he victory in his pocket,
the wounds become him.

VOLUMNIA.
On’s brows, Menenius. He comes the third time home with the oaken
garland.

MENENIUS.
Has he disciplined Aufidius soundly?

VOLUMNIA.
Titus Lartius writes they fought together, but Aufidius got off.

MENENIUS.
And ’twas time for him too, I’ll warrant him that. An he had stayed by
him, I would not have been so ’fidiused for all the chests in Corioles
and the gold that’s in them. Is the Senate possessed of this?

VOLUMNIA.
Good ladies, let’s go.—Yes, yes, yes. The Senate has letters from the
General, wherein he gives my son the whole name of the war. He hath in
this action outdone his former deeds doubly.

VALERIA.
In troth, there’s wondrous things spoke of him.

MENENIUS.
Wondrous? Ay, I warrant you, and not without his true purchasing.

VIRGILIA.
The gods grant them true.

VOLUMNIA.
True? Pow, waw!

MENENIUS.
True? I’ll be sworn they are true. Where is he wounded? [_To the
Tribunes_.] God save your good Worships! Martius is coming home; he has
more cause to be proud.—Where is he wounded?

VOLUMNIA.
I’ th’ shoulder and i’ th’ left arm. There will be large cicatrices to
show the people when he shall stand for his place. He received in the
repulse of Tarquin seven hurts i’ th’ body.

MENENIUS.
One i’ th’ neck and two i’ th’ thigh—there’s nine that I know.

VOLUMNIA.
He had, before this last expedition, twenty-five wounds upon him.

MENENIUS.
Now it’s twenty-seven. Every gash was an enemy’s grave.


[_A shout and flourish_.]

Hark, the trumpets!

VOLUMNIA.
These are the ushers of Martius: before him he carries noise, and
behind him he leaves tears.
Death, that dark spirit, in’s nervy arm doth lie,
Which, being advanced, declines, and then men die.

[_A sennet_.]

Enter Cominius the General and Titus Lartius, between them Coriolanus
crowned with an oaken garland, with Captains and Soldiers and a Herald.
Trumpets sound.

HERALD.
Know, Rome, that all alone Martius did fight
Within Corioles’ gates, where he hath won,
With fame, a name to Caius Martius; these
In honour follows “Coriolanus.”
Welcome to Rome, renowned Coriolanus.

[_Sound flourish._]

ALL.
Welcome to Rome, renowned Coriolanus!

CORIOLANUS.
No more of this, it does offend my heart.
Pray now, no more.

COMINIUS.
Look, sir, your mother.

CORIOLANUS.
O,
You have, I know, petitioned all the gods
For my prosperity.

[_Kneels._]

VOLUMNIA.
Nay, my good soldier, up.

[_He stands._]

My gentle Martius, worthy Caius, and
By deed-achieving honour newly named—
What is it? Coriolanus must I call thee?
But, O, thy wife—

CORIOLANUS.
My gracious silence, hail.
Wouldst thou have laughed had I come coffined home,
That weep’st to see me triumph? Ah, my dear,
Such eyes the widows in Corioles wear
And mothers that lack sons.

MENENIUS.
Now the gods crown thee!

CORIOLANUS.
And live you yet? [_To Valeria_] O my sweet lady, pardon.

VOLUMNIA.
I know not where to turn. O, welcome home!
And welcome, general.—And you’re welcome all.

MENENIUS.
A hundred thousand welcomes! I could weep,
And I could laugh; I am light and heavy. Welcome.
A curse begin at very root on’s heart
That is not glad to see thee! You are three
That Rome should dote on; yet, by the faith of men,
We have some old crab trees here at home that will not
Be grafted to your relish. Yet welcome, warriors!
We call a nettle but a nettle, and
The faults of fools but folly.

COMINIUS.
Ever right.

CORIOLANUS.
Menenius ever, ever.

HERALD.
Give way there, and go on!

CORIOLANUS.
[_To Volumnia and Virgilia_.] Your hand, and yours.
Ere in our own house I do shade my head,
The good patricians must be visited,
From whom I have received not only greetings,
But with them change of honours.

VOLUMNIA.
I have lived
To see inherited my very wishes
And the buildings of my fancy. Only
There’s one thing wanting, which I doubt not but
Our Rome will cast upon thee.

CORIOLANUS.
Know, good mother,
I had rather be their servant in my way
Than sway with them in theirs.

COMINIUS.
On, to the Capitol.

[_Flourish of cornets. Exeunt in state, as before._]

Brutus and Sicinius come forward.

BRUTUS.
All tongues speak of him, and the bleared sights
Are spectacled to see him. Your prattling nurse
Into a rapture lets her baby cry
While she chats him. The kitchen malkin pins
Her richest lockram ’bout her reechy neck,
Clamb’ring the walls to eye him. Stalls, bulks, windows
Are smothered up, leads filled, and ridges horsed
With variable complexions, all agreeing
In earnestness to see him. Seld-shown flamens
Do press among the popular throngs and puff
To win a vulgar station. Our veiled dames
Commit the war of white and damask in
Their nicely-gauded cheeks to th’ wanton spoil
Of Phoebus’ burning kisses. Such a pother,
As if that whatsoever god who leads him
Were slyly crept into his human powers
And gave him graceful posture.

SICINIUS.
On the sudden
I warrant him consul.

BRUTUS.
Then our office may,
During his power, go sleep.

SICINIUS.
He cannot temp’rately transport his honours
From where he should begin and end, but will
Lose those he hath won.

BRUTUS.
In that there’s comfort.

SICINIUS.
Doubt not the commoners, for whom we stand,
But they, upon their ancient malice will forget
With the least cause these his new honours—which
That he will give them make as little question
As he is proud to do’t.

BRUTUS.
I heard him swear,
Were he to stand for consul, never would he
Appear i’ th’ marketplace nor on him put
The napless vesture of humility,
Nor showing, as the manner is, his wounds
To th’ people, beg their stinking breaths.

SICINIUS.
’Tis right.

BRUTUS.
It was his word. O, he would miss it rather
Than carry it but by the suit of the gentry to him
And the desire of the nobles.

SICINIUS.
I wish no better
Than have him hold that purpose and to put it
In execution.

BRUTUS.
’Tis most like he will.

SICINIUS.
It shall be to him then, as our good wills,
A sure destruction.

BRUTUS.
So it must fall out
To him, or our authorities for an end.
We must suggest the people in what hatred
He still hath held them; that to’s power he would
Have made them mules, silenced their pleaders, and
Dispropertied their freedoms; holding them
In human action and capacity
Of no more soul nor fitness for the world
Than camels in their war, who have their provand
Only for bearing burdens, and sore blows
For sinking under them.

SICINIUS.
This, as you say, suggested
At some time when his soaring insolence
Shall touch the people—which time shall not want
If it be put upon’t, and that’s as easy
As to set dogs on sheep—will be his fire
To kindle their dry stubble, and their blaze
Shall darken him for ever.

Enter a Messenger.

BRUTUS.
What’s the matter?

MESSENGER.
You are sent for to the Capitol. ’Tis thought
That Martius shall be consul. I have seen
The dumb men throng to see him, and the blind
to hear him speak; matrons flung gloves,
Ladies and maids their scarves and handkerchiefs,
Upon him as he passed; the nobles bended
As to Jove’s statue, and the Commons made
A shower and thunder with their caps and shouts.
I never saw the like.

BRUTUS.
Let’s to the Capitol;
And carry with us ears and eyes for th’ time,
But hearts for the event.

SICINIUS.
Have with you.

[_Exeunt._]

SCENE II. Rome. The Capitol

Enter two Officers, to lay cushions, as it were in the Capitol.

FIRST OFFICER.
Come, come. They are almost here. How many stand for consulships?

SECOND OFFICER.
Three, they say; but ’tis thought of everyone Coriolanus will carry it.

FIRST OFFICER.
That’s a brave fellow, but he’s vengeance proud and loves not the
common people.

SECOND OFFICER.
’Faith, there have been many great men that have flattered the people
who ne’er loved them; and there be many that they have loved they know
not wherefore; so that, if they love they know not why, they hate upon
no better a ground. Therefore, for Coriolanus neither to care whether
they love or hate him manifests the true knowledge he has in their
disposition and, out of his noble carelessness, lets them plainly
see’t.

FIRST OFFICER.
If he did not care whether he had their love or no, he waved
indifferently ’twixt doing them neither good nor harm; but he seeks
their hate with greater devotion than they can render it him and leaves
nothing undone that may fully discover him their opposite. Now, to seem
to affect the malice and displeasure of the people is as bad as that
which he dislikes, to flatter them for their love.

SECOND OFFICER.
He hath deserved worthily of his country, and his ascent is not by such
easy degrees as those who, having been supple and courteous to the
people, bonnetted, without any further deed to have them at all into
their estimation and report; but he hath so planted his honours in
their eyes and his actions in their hearts that for their tongues to be
silent and not confess so much were a kind of ingrateful injury. To
report otherwise were a malice that, giving itself the lie, would pluck
reproof and rebuke from every ear that heard it.

FIRST OFFICER.
No more of him; he’s a worthy man. Make way. They are coming.

A sennet. Enter the Patricians and the Tribunes of the people, Lictors
before them; Coriolanus, Menenius, Cominius the consul. The Patricians
sit.  Sicinius and Brutus take their places by themselves. Coriolanus
stands.

MENENIUS.
Having determined of the Volsces and
To send for Titus Lartius, it remains,
As the main point of this our after-meeting,
To gratify his noble service that
Hath thus stood for his country. Therefore please you,
Most reverend and grave elders, to desire
The present consul and last general
In our well-found successes to report
A little of that worthy work performed
By Martius Caius Coriolanus, whom
We met here both to thank and to remember
With honours like himself.

[_Coriolanus sits._]

FIRST SENATOR.
Speak, good Cominius.
Leave nothing out for length, and make us think
Rather our state’s defective for requital,
Than we to stretch it out. Masters o’ th’ people,
We do request your kindest ears and, after,
Your loving motion toward the common body
To yield what passes here.

SICINIUS.
We are convented
Upon a pleasing treaty and have hearts
Inclinable to honour and advance
The theme of our assembly.

BRUTUS.
Which the rather
We shall be blest to do if he remember
A kinder value of the people than
He hath hereto prized them at.

MENENIUS.
That’s off, that’s off!
I would you rather had been silent. Please you
To hear Cominius speak?

BRUTUS.
Most willingly.
But yet my caution was more pertinent
Than the rebuke you give it.

MENENIUS.
He loves your people,
But tie him not to be their bedfellow.—
Worthy Cominius, speak.

[_Coriolanus rises, and offers to go away._]

Nay, keep your place.

FIRST SENATOR.
Sit, Coriolanus. Never shame to hear
What you have nobly done.

CORIOLANUS.
Your Honours, pardon.
I had rather have my wounds to heal again
Than hear say how I got them.

BRUTUS.
Sir, I hope
My words disbenched you not?

CORIOLANUS.
No, sir. Yet oft,
When blows have made me stay, I fled from words.
You soothed not, therefore hurt not; but your people,
I love them as they weigh.

MENENIUS.
Pray now, sit down.

CORIOLANUS.
I had rather have one scratch my head i’ th’ sun
When the alarum were struck than idly sit
To hear my nothings monstered.

[_Exit._]

MENENIUS.
Masters of the people,
Your multiplying spawn how can he flatter—
That’s thousand to one good one—when you now see
He had rather venture all his limbs for honour
Than one on’s ears to hear it?—Proceed, Cominius.

COMINIUS.
I shall lack voice. The deeds of Coriolanus
Should not be uttered feebly. It is held
That valour is the chiefest virtue and
Most dignifies the haver; if it be,
The man I speak of cannot in the world
Be singly counterpoised. At sixteen years,
When Tarquin made a head for Rome, he fought
Beyond the mark of others. Our then dictator,
Whom with all praise I point at, saw him fight
When with his Amazonian chin he drove
The bristled lips before him. He bestrid
An o’erpressed Roman and i’ th’ Consul’s view
Slew three opposers. Tarquin’s self he met
And struck him on his knee. In that day’s feats,
When he might act the woman in the scene,
He proved best man i’ th’ field and for his meed
Was brow-bound with the oak. His pupil age
Man-entered thus, he waxed like a sea,
And in the brunt of seventeen battles since
He lurched all swords of the garland. For this last,
Before and in Corioles, let me say,
I cannot speak him home. He stopped the flyers
And by his rare example made the coward
Turn terror into sport. As weeds before
A vessel under sail, so men obeyed
And fell below his stem. His sword, Death’s stamp,
Where it did mark, it took; from face to foot
He was a thing of blood, whose every motion
Was timed with dying cries. Alone he entered
The mortal gate o’ th’ city, which he painted
With shunless destiny; aidless came off
And with a sudden reinforcement struck
Corioles like a planet. Now all’s his,
When by and by the din of war gan pierce
His ready sense; then straight his doubled spirit
Requickened what in flesh was fatigate,
And to the battle came he, where he did
Run reeking o’er the lives of men as if
’Twere a perpetual spoil; and till we called
Both field and city ours, he never stood
To ease his breast with panting.

MENENIUS.
Worthy man!

FIRST SENATOR.
He cannot but with measure fit the honours
Which we devise him.

COMINIUS.
Our spoils he kicked at;
And looked upon things precious as they were
The common muck of the world. He covets less
Than misery itself would give, rewards
His deeds with doing them, and is content
To spend the time to end it.

MENENIUS.
He’s right noble.
Let him be called for.

FIRST SENATOR.
Call Coriolanus.

OFFICER.
He doth appear.

Enter Coriolanus.

MENENIUS.
The Senate, Coriolanus, are well pleased
To make thee consul.

CORIOLANUS.
I do owe them still
My life and services.

MENENIUS.
It then remains
That you do speak to the people.

CORIOLANUS.
I do beseech you
Let me o’erleap that custom, for I cannot
Put on the gown, stand naked, and entreat them
For my wounds’ sake to give their suffrage. Please you
That I may pass this doing.

SICINIUS.
Sir, the people
Must have their voices; neither will they bate
One jot of ceremony.

MENENIUS.
Put them not to’t.
Pray you, go fit you to the custom, and
Take to you, as your predecessors have,
Your honour with your form.

CORIOLANUS.
It is a part
That I shall blush in acting, and might well
Be taken from the people.

BRUTUS.
Mark you that?

CORIOLANUS.
To brag unto them, “thus I did, and thus!”
Show them th’ unaching scars which I should hide,
As if I had received them for the hire
Of their breath only!

MENENIUS.
Do not stand upon’t.—
We recommend to you, tribunes of the people,
Our purpose to them, and to our noble consul
Wish we all joy and honour.

SENATORS.
To Coriolanus come all joy and honour!

[_Flourish cornets. Exeunt all but Sicinius and Brutus._]

BRUTUS.
You see how he intends to use the people.

SICINIUS.
May they perceive’s intent! He will require them
As if he did contemn what he requested
Should be in them to give.

BRUTUS.
Come, we’ll inform them
Of our proceedings here. On th’ marketplace
I know they do attend us.

[_Exeunt._]

SCENE III. Rome. The Forum

Enter seven or eight Citizens.

FIRST CITIZEN.
Once, if he do require our voices, we ought not to deny him.

SECOND CITIZEN.
We may, sir, if we will.

THIRD CITIZEN.
We have power in ourselves to do it, but it is a power that we have no
power to do; for, if he show us his wounds and tell us his deeds, we
are to put our tongues into those wounds and speak for them. So, if he
tell us his noble deeds, we must also tell him our noble acceptance of
them. Ingratitude is monstrous, and for the multitude to be ingrateful
were to make a monster of the multitude, of the which we being members,
should bring ourselves to be monstrous members.

FIRST CITIZEN.
And to make us no better thought of, a little help will serve; for once
we stood up about the corn, he himself stuck not to call us the
many-headed multitude.

THIRD CITIZEN.
We have been called so of many; not that our heads are some brown, some
black, some auburn, some bald, but that our wits are so diversely
coloured; and truly I think if all our wits were to issue out of one
skull, they would fly east, west, north, south, and their consent of
one direct way should be at once to all the points o’ th’ compass.

SECOND CITIZEN.
Think you so? Which way do you judge my wit would fly?

THIRD CITIZEN.
Nay, your wit will not so soon out as another man’s will; ’tis strongly
wedged up in a blockhead. But if it were at liberty, ’twould, sure,
southward.

SECOND CITIZEN.
Why that way?

THIRD CITIZEN.
To lose itself in a fog, where being three parts melted away with
rotten dews, the fourth would return for conscience’ sake, to help to
get thee a wife.

SECOND CITIZEN.
You are never without your tricks. You may, you may.

THIRD CITIZEN.
Are you all resolved to give your voices? But that’s no matter; the
greater part carries it. I say, if he would incline to the people,
there was never a worthier man.

Enter Coriolanus in a gown of humility, with Menenius.

Here he comes, and in the gown of humility. Mark his behaviour. We are
not to stay all together, but to come by him where he stands, by ones,
by twos, and by threes. He’s to make his requests by particulars,
wherein everyone of us has a single honour in giving him our own voices
with our own tongues. Therefore follow me, and I’ll direct you how you
shall go by him.

ALL.
Content, content.

[_Exeunt._]

MENENIUS.
O sir, you are not right. Have you not known
The worthiest men have done’t?

CORIOLANUS.
What must I say?
“I pray, sir”—plague upon’t! I cannot bring
My tongue to such a pace. “Look, sir, my wounds!
I got them in my country’s service when
Some certain of your brethren roared and ran
From th’ noise of our own drums.”

MENENIUS.
O me, the gods!
You must not speak of that. You must desire them
To think upon you.

CORIOLANUS.
Think upon me! Hang ’em!
I would they would forget me, like the virtues
Which our divines lose by ’em.

MENENIUS.
You’ll mar all.
I’ll leave you. Pray you speak to ’em, I pray you,
In wholesome manner.

[_Exit Menenius._]

CORIOLANUS.
Bid them wash their faces
And keep their teeth clean.

Enter three of the Citizens.

So, here comes a brace.
You know the cause, sirs, of my standing here.

THIRD CITIZEN.
We do, sir. Tell us what hath brought you to’t.

CORIOLANUS.
Mine own desert.

SECOND CITIZEN.
Your own desert?

CORIOLANUS.
Ay, but not mine own desire.

THIRD CITIZEN.
How, not your own desire?

CORIOLANUS.
No, sir, ’twas never my desire yet to trouble the poor with begging.

THIRD CITIZEN.
You must think if we give you anything, we hope to gain by you.

CORIOLANUS.
Well then, I pray, your price o’ th’ consulship?

FIRST CITIZEN.
The price is to ask it kindly.

CORIOLANUS.
Kindly, sir, I pray, let me ha’t. I have wounds to show you, which
shall be yours in private.—Your good voice, sir. What say you?

SECOND CITIZEN.
You shall ha’ it, worthy sir.

CORIOLANUS.
A match, sir. There’s in all two worthy voices begged. I have your
alms. Adieu.

THIRD CITIZEN.
But this is something odd.

SECOND CITIZEN.
An ’twere to give again—but ’tis no matter.

[_Exeunt two citizens._]

Enter two other Citizens.

CORIOLANUS.
Pray you now, if it may stand with the tune of your voices that I may
be consul, I have here the customary gown.

FOURTH CITIZEN.
You have deserved nobly of your country, and you have not deserved
nobly.

CORIOLANUS.
Your enigma?

FOURTH CITIZEN.
You have been a scourge to her enemies; you have been a rod to her
friends. You have not indeed loved the common people.

CORIOLANUS.
You should account me the more virtuous that I have not been common in
my love. I will, sir, flatter my sworn brother, the people, to earn a
dearer estimation of them; ’tis a condition they account gentle. And
since the wisdom of their choice is rather to have my hat than my
heart, I will practise the insinuating nod and be off to them most
counterfeitly. That is, sir, I will counterfeit the bewitchment of some
popular man and give it bountiful to the desirers. Therefore, beseech
you, I may be consul.

FIFTH CITIZEN.
We hope to find you our friend, and therefore give you our voices
heartily.

FOURTH CITIZEN.
You have received many wounds for your country.

CORIOLANUS.
I will not seal your knowledge with showing them. I will make much of
your voices and so trouble you no farther.

BOTH CITIZENS.
The gods give you joy, sir, heartily.

[_Exeunt citizens._]

CORIOLANUS.
Most sweet voices!
Better it is to die, better to starve,
Than crave the hire which first we do deserve.
Why in this wolvish toge should I stand here
To beg of Hob and Dick that does appear
Their needless vouches? Custom calls me to’t.
What custom wills, in all things should we do’t?
The dust on antique time would lie unswept
And mountainous error be too highly heaped
For truth to o’erpeer. Rather than fool it so,
Let the high office and the honour go
To one that would do thus. I am half through;
The one part suffered, the other will I do.


Enter three Citizens more.

Here come more voices.
Your voices! For your voices I have fought;
Watched for your voices; for your voices bear
Of wounds two dozen odd. Battles thrice six
I have seen and heard of; for your voices have
Done many things, some less, some more. Your voices!
Indeed, I would be consul.

SIXTH CITIZEN.
He has done nobly, and cannot go without any honest man’s voice.

SEVENTH CITIZEN.
Therefore let him be consul. The gods give him joy, and make him good
friend to the people!

ALL THREE CITIZENS.
Amen, amen. God save thee, noble consul.

[_Exeunt citizens._]

CORIOLANUS.
Worthy voices!

Enter Menenius with Brutus and Sicinius.

MENENIUS.
You have stood your limitation, and the Tribunes
Endue you with the people’s voice. Remains
That in th’ official marks invested, you
Anon do meet the Senate.

CORIOLANUS.
Is this done?

SICINIUS.
The custom of request you have discharged.
The people do admit you, and are summoned
To meet anon upon your approbation.

CORIOLANUS.
Where? At the Senate House?

SICINIUS.
There, Coriolanus.

CORIOLANUS.
May I change these garments?

SICINIUS.
You may, sir.

CORIOLANUS.
That I’ll straight do and, knowing myself again,
Repair to th’ Senate House.

MENENIUS.
I’ll keep you company.—Will you along?

BRUTUS.
We stay here for the people.

SICINIUS.
Fare you well.

[_Exeunt Coriolanus and Menenius._]

He has it now; and by his looks, methinks,
’Tis warm at his heart.

BRUTUS.
With a proud heart he wore
His humble weeds. Will you dismiss the people?

Enter the Pebleians.

SICINIUS.
How now, my masters, have you chose this man?

FIRST CITIZEN.
He has our voices, sir.

BRUTUS.
We pray the gods he may deserve your loves.

SECOND CITIZEN.
Amen, sir. To my poor unworthy notice,
He mocked us when he begged our voices.

THIRD CITIZEN.
Certainly, he flouted us downright.

FIRST CITIZEN.
No, ’tis his kind of speech. He did not mock us.

SECOND CITIZEN.
Not one amongst us, save yourself, but says
He used us scornfully. He should have showed us
His marks of merit, wounds received for’s country.

SICINIUS.
Why, so he did, I am sure.

ALL.
No, no. No man saw ’em.

THIRD CITIZEN.
He said he had wounds, which he could show in private,
And with his hat, thus waving it in scorn,
“I would be consul,” says he; “aged custom,
But by your voices, will not so permit me;
Your voices therefore.” When we granted that,
Here was “I thank you for your voices. Thank you.
Your most sweet voices! Now you have left your voices,
I have no further with you.” Was not this mockery?

SICINIUS.
Why either were you ignorant to see’t
Or, seeing it, of such childish friendliness
To yield your voices?

BRUTUS.
Could you not have told him
As you were lessoned? When he had no power,
But was a petty servant to the state,
He was your enemy, ever spake against
Your liberties and the charters that you bear
I’ th’ body of the weal; and, now arriving
A place of potency and sway o’ th’ state,
If he should still malignantly remain
Fast foe to th’ plebeii, your voices might
Be curses to yourselves. You should have said
That as his worthy deeds did claim no less
Than what he stood for, so his gracious nature
Would think upon you for your voices, and
Translate his malice towards you into love,
Standing your friendly lord.

SICINIUS.
Thus to have said,
As you were fore-advised, had touched his spirit
And tried his inclination; from him plucked
Either his gracious promise, which you might,
As cause had called you up, have held him to;
Or else it would have galled his surly nature,
Which easily endures not article
Tying him to aught. So putting him to rage,
You should have ta’en th’ advantage of his choler
And passed him unelected.

BRUTUS.
Did you perceive
He did solicit you in free contempt
When he did need your loves, and do you think
That his contempt shall not be bruising to you
When he hath power to crush? Why, had your bodies
No heart among you? Or had you tongues to cry
Against the rectorship of judgment?

SICINIUS.
Have you ere now denied the asker, and now
Again, of him that did not ask but mock,
Bestow your sued-for tongues?

THIRD CITIZEN.
He’s not confirmed.
We may deny him yet.

SECOND CITIZEN.
And will deny him.
I’ll have five hundred voices of that sound.

FIRST CITIZEN.
I twice five hundred, and their friends to piece ’em.

BRUTUS.
Get you hence instantly, and tell those friends
They have chose a consul that will from them take
Their liberties, make them of no more voice
Than dogs that are as often beat for barking
As therefore kept to do so.

SICINIUS.
Let them assemble
And, on a safer judgment, all revoke
Your ignorant election. Enforce his pride
And his old hate unto you. Besides, forget not
With what contempt he wore the humble weed,
How in his suit he scorned you; but your loves,
Thinking upon his services, took from you
Th’ apprehension of his present portance,
Which most gibingly, ungravely, he did fashion
After the inveterate hate he bears you.

BRUTUS.
Lay
A fault on us, your tribunes, that we laboured,
No impediment between, but that you must
Cast your election on him.

SICINIUS.
Say you chose him
More after our commandment than as guided
By your own true affections, and that your minds,
Preoccupied with what you rather must do
Than what you should, made you against the grain
To voice him consul. Lay the fault on us.

BRUTUS.
Ay, spare us not. Say we read lectures to you,
How youngly he began to serve his country,
How long continued, and what stock he springs of,
The noble house o’ th’ Martians, from whence came
That Ancus Martius, Numa’s daughter’s son,
Who, after great Hostilius here was king,
Of the same house Publius and Quintus were,
That our best water brought by conduits hither;
And Censorinus, that was so surnamed,
And nobly named so, twice being censor,
Was his great ancestor.

SICINIUS.
One thus descended,
That hath beside well in his person wrought
To be set high in place, we did commend
To your remembrances; but you have found,
Scaling his present bearing with his past,
That he’s your fixed enemy, and revoke
Your sudden approbation.

BRUTUS.
Say you ne’er had done’t—
Harp on that still—but by our putting on.
And presently when you have drawn your number,
Repair to th’ Capitol.

ALL.
We will so. Almost all
Repent in their election.

[_Exeunt Plebeians._]

BRUTUS.
Let them go on.
This mutiny were better put in hazard
Than stay, past doubt, for greater.
If, as his nature is, he fall in rage
With their refusal, both observe and answer
The vantage of his anger.

SICINIUS.
To th’ Capitol, come.
We will be there before the stream o’ th’ people,
And this shall seem, as partly ’tis, their own,
Which we have goaded onward.

[_Exeunt._]




ACT III

SCENE I. Rome. A street


Cornets. Enter Coriolanus, Menenius, all the Gentry, Cominius, Titus
Lartius and other Senators.

CORIOLANUS.
Tullus Aufidius then had made new head?

LARTIUS.
He had, my lord, and that it was which caused
Our swifter composition.

CORIOLANUS.
So then the Volsces stand but as at first,
Ready, when time shall prompt them, to make road
Upon’s again.

COMINIUS.
They are worn, lord consul, so
That we shall hardly in our ages see
Their banners wave again.

CORIOLANUS.
Saw you Aufidius?

LARTIUS.
On safeguard he came to me, and did curse
Against the Volsces, for they had so vilely
Yielded the town. He is retired to Antium.

CORIOLANUS.
Spoke he of me?

LARTIUS.
He did, my lord.

CORIOLANUS.
How? What?

LARTIUS.
How often he had met you sword to sword;
That of all things upon the earth he hated
Your person most; that he would pawn his fortunes
To hopeless restitution, so he might
Be called your vanquisher.

CORIOLANUS.
At Antium lives he?

LARTIUS.
At Antium.

CORIOLANUS.
I wish I had a cause to seek him there,
To oppose his hatred fully. Welcome home.

Enter Sicinius and Brutus.

Behold, these are the tribunes of the people,
The tongues o’ th’ common mouth. I do despise them,
For they do prank them in authority
Against all noble sufferance.

SICINIUS.
Pass no further.

CORIOLANUS.
Ha? What is that?

BRUTUS.
It will be dangerous to go on. No further.

CORIOLANUS.
What makes this change?

MENENIUS.
The matter?

COMINIUS.
Hath he not passed the noble and the common?

BRUTUS.
Cominius, no.

CORIOLANUS.
Have I had children’s voices?

FIRST SENATOR.
Tribunes, give way. He shall to the marketplace.

BRUTUS.
The people are incensed against him.

SICINIUS.
Stop,
Or all will fall in broil.

CORIOLANUS.
Are these your herd?
Must these have voices, that can yield them now
And straight disclaim their tongues? What are your offices?
You being their mouths, why rule you not their teeth?
Have you not set them on?

MENENIUS.
Be calm, be calm.

CORIOLANUS.
It is a purposed thing, and grows by plot,
To curb the will of the nobility.
Suffer’t, and live with such as cannot rule
Nor ever will be ruled.

BRUTUS.
Call’t not a plot.
The people cry you mocked them; and, of late,
When corn was given them gratis, you repined,
Scandaled the suppliants for the people, called them
Timepleasers, flatterers, foes to nobleness.

CORIOLANUS.
Why, this was known before.

BRUTUS.
Not to them all.

CORIOLANUS.
Have you informed them sithence?

BRUTUS.
How? I inform them?

COMINIUS.
You are like to do such business.

BRUTUS.
Not unlike, each way, to better yours.

CORIOLANUS.
Why then should I be consul? By yond clouds,
Let me deserve so ill as you, and make me
Your fellow tribune.

SICINIUS.
You show too much of that
For which the people stir. If you will pass
To where you are bound, you must inquire your way,
Which you are out of, with a gentler spirit,
Or never be so noble as a consul,
Nor yoke with him for tribune.

MENENIUS.
Let’s be calm.

COMINIUS.
The people are abused, set on. This palt’ring
Becomes not Rome, nor has Coriolanus
Deserved this so dishonoured rub, laid falsely
I’ th’ plain way of his merit.

CORIOLANUS.
Tell me of corn?
This was my speech, and I will speak’t again.

MENENIUS.
Not now, not now.

FIRST SENATOR.
Not in this heat, sir, now.

CORIOLANUS.
Now, as I live, I will.
My nobler friends, I crave their pardons. For
The mutable, rank-scented many, let them
Regard me, as I do not flatter, and
Therein behold themselves. I say again,
In soothing them we nourish ’gainst our senate
The cockle of rebellion, insolence, sedition,
Which we ourselves have ploughed for, sowed, and scattered
By mingling them with us, the honoured number,
Who lack not virtue, no, nor power, but that
Which they have given to beggars.

MENENIUS.
Well, no more.

FIRST SENATOR.
No more words, we beseech you.

CORIOLANUS.
How? No more?
As for my country I have shed my blood,
Not fearing outward force, so shall my lungs
Coin words till their decay against those measles
Which we disdain should tetter us, yet sought
The very way to catch them.

BRUTUS.
You speak o’ th’ people
As if you were a god to punish, not
A man of their infirmity.

SICINIUS.
’Twere well
We let the people know’t.

MENENIUS.
What, what? His choler?

CORIOLANUS.
Choler?
Were I as patient as the midnight sleep,
By Jove, ’twould be my mind.

SICINIUS.
It is a mind
That shall remain a poison where it is,
Not poison any further.

CORIOLANUS.
“Shall remain”?
Hear you this Triton of the minnows? Mark you
His absolute “shall”?

COMINIUS.
’Twas from the canon.

CORIOLANUS.
“Shall”?
O good but most unwise patricians, why,
You grave but reckless senators, have you thus
Given Hydra leave to choose an officer,
That with his peremptory “shall,” being but
The horn and noise o’ th’ monster’s, wants not spirit
To say he’ll turn your current in a ditch
And make your channel his? If he have power,
Then vail your ignorance; if none, awake
Your dangerous lenity. If you are learned,
Be not as common fools; if you are not,
Let them have cushions by you. You are plebeians,
If they be senators; and they are no less
When, both your voices blended, the great’st taste
Most palates theirs. They choose their magistrate,
And such a one as he, who puts his “shall,”
His popular “shall,” against a graver bench
Than ever frowned in Greece. By Jove himself,
It makes the consuls base! And my soul aches
To know, when two authorities are up,
Neither supreme, how soon confusion
May enter ’twixt the gap of both and take
The one by th’ other.

COMINIUS.
Well, on to th’ marketplace.

CORIOLANUS.
Whoever gave that counsel to give forth
The corn o’ th’ storehouse gratis, as ’twas used
Sometime in Greece—

MENENIUS.
Well, well, no more of that.

CORIOLANUS.
Though there the people had more absolute power,
I say they nourished disobedience, fed
The ruin of the state.

BRUTUS.
Why shall the people give
One that speaks thus their voice?

CORIOLANUS.
I’ll give my reasons,
More worthier than their voices. They know the corn
Was not our recompense, resting well assured
They ne’er did service for’t. Being pressed to th’ war,
Even when the navel of the state was touched,
They would not thread the gates. This kind of service
Did not deserve corn gratis. Being i’ th’ war,
Their mutinies and revolts, wherein they showed
Most valour, spoke not for them. Th’ accusation
Which they have often made against the Senate,
All cause unborn, could never be the native
Of our so frank donation. Well, what then?
How shall this bosom multitude digest
The senate’s courtesy? Let deeds express
What’s like to be their words: “We did request it;
We are the greater poll, and in true fear
They gave us our demands.” Thus we debase
The nature of our seats and make the rabble
Call our cares fears, which will in time
Break ope the locks o’ th’ Senate and bring in
The crows to peck the eagles.

MENENIUS.
Come, enough.

BRUTUS.
Enough, with over-measure.

CORIOLANUS.
No, take more!
What may be sworn by, both divine and human,
Seal what I end withal! This double worship—
Where one part does disdain with cause, the other
Insult without all reason, where gentry, title, wisdom
Cannot conclude but by the yea and no
Of general ignorance—it must omit
Real necessities and give way the while
To unstable slightness. Purpose so barred, it follows
Nothing is done to purpose. Therefore, beseech you—
You that will be less fearful than discreet,
That love the fundamental part of state
More than you doubt the change on’t, that prefer
A noble life before a long, and wish
To jump a body with a dangerous physic
That’s sure of death without it—at once pluck out
The multitudinous tongue; let them not lick
The sweet which is their poison. Your dishonour
Mangles true judgment and bereaves the state
Of that integrity which should become’t,
Not having the power to do the good it would
For th’ ill which doth control’t.

BRUTUS.
’Has said enough.

SICINIUS.
’Has spoken like a traitor, and shall answer
As traitors do.

CORIOLANUS.
Thou wretch, despite o’erwhelm thee!
What should the people do with these bald tribunes,
On whom depending, their obedience fails
To th’ greater bench. In a rebellion,
When what’s not meet but what must be was law,
Then were they chosen. In a better hour,
Let what is meet be said it must be meet,
And throw their power i’ th’ dust.

BRUTUS.
Manifest treason.

SICINIUS.
This a consul? No.

BRUTUS.
The aediles, ho! Let him be apprehended.

Enter an Aedile.

SICINIUS.
Go call the people;

[_Exit Aedile._]

in whose name myself
Attach thee as a traitorous innovator,
A foe to th’ public weal. Obey, I charge thee,
And follow to thine answer.

CORIOLANUS.
Hence, old goat.

ALL PATRICIANS.
We’ll surety him.

COMINIUS.
[_to Sicinius_.] Aged sir, hands off.

CORIOLANUS.
[_to Sicinius_.] Hence, rotten thing, or I shall shake thy bones
Out of thy garments.

SICINIUS.
Help, ye citizens!

Enter a rabble of Plebeians with the Aediles.

MENENIUS.
On both sides more respect!

SICINIUS.
Here’s he that would take from you all your power.

BRUTUS.
Seize him, aediles.

ALL PLEBEIANS.
Down with him, down with him!

SECOND SENATOR.
Weapons, weapons, weapons!

[_They all bustle about Coriolanus._]

Tribunes, patricians, citizens, what, ho!
Sicinius, Brutus, Coriolanus, citizens!

ALL.
Peace, peace, peace! Stay, hold, peace!

MENENIUS.
What is about to be? I am out of breath.
Confusion’s near. I cannot speak. You tribunes
To th’ people!—Coriolanus, patience!—
Speak, good Sicinius.

SICINIUS.
Hear me, people! Peace!

ALL PLEBEIANS.
Let’s hear our tribune. Peace! Speak, speak, speak.

SICINIUS.
You are at point to lose your liberties.
Martius would have all from you, Martius,
Whom late you have named for consul.

MENENIUS.
Fie, fie, fie!
This is the way to kindle, not to quench.

FIRST SENATOR.
To unbuild the city and to lay all flat.

SICINIUS.
What is the city but the people?

ALL PLEBEIANS.
True,
The people are the city.

BRUTUS.
By the consent of all, we were established
The people’s magistrates.

ALL PLEBEIANS.
You so remain.

MENENIUS.
And so are like to do.

COMINIUS.
That is the way to lay the city flat,
To bring the roof to the foundation
And bury all which yet distinctly ranges
In heaps and piles of ruin.

SICINIUS.
This deserves death.

BRUTUS.
Or let us stand to our authority
Or let us lose it. We do here pronounce,
Upon the part o’ th’ people, in whose power
We were elected theirs, Martius is worthy
Of present death.

SICINIUS.
Therefore lay hold of him,
Bear him to th’ rock Tarpeian, and from thence
Into destruction cast him.

BRUTUS.
Aediles, seize him!

ALL PLEBEIANS.
Yield, Martius, yield!

MENENIUS.
Hear me one word.
Beseech you, tribunes, hear me but a word.

AEDILES.
Peace, peace!

MENENIUS.
Be that you seem, truly your country’s friend,
And temp’rately proceed to what you would
Thus violently redress.

BRUTUS.
Sir, those cold ways,
That seem like prudent helps, are very poisonous
Where the disease is violent.—Lay hands upon him,
And bear him to the rock.

[_Coriolanus draws his sword._]

CORIOLANUS.
No; I’ll die here.
There’s some among you have beheld me fighting.
Come, try upon yourselves what you have seen me.

MENENIUS.
Down with that sword!—Tribunes, withdraw awhile.

BRUTUS.
Lay hands upon him!

MENENIUS.
Help Martius, help!
You that be noble, help him, young and old!

ALL PLEBEIANS.
Down with him, down with him!

[_In this mutiny the Tribunes, the Aediles and the People are beat
in._]

MENENIUS.
Go, get you to your house. Begone, away.
All will be naught else.

SECOND SENATOR.
Get you gone.

CORIOLANUS.
Stand fast!
We have as many friends as enemies.

MENENIUS.
Shall it be put to that?

FIRST SENATOR.
The gods forbid!
I prithee, noble friend, home to thy house;
Leave us to cure this cause.

MENENIUS.
For ’tis a sore upon us
You cannot tent yourself. Begone, beseech you.

COMINIUS.
Come, sir, along with us.

CORIOLANUS.
I would they were barbarians, as they are,
Though in Rome littered, not Romans, as they are not,
Though calved i’ th’ porch o’ th’ Capitol.

MENENIUS.
Begone!
Put not your worthy rage into your tongue.
One time will owe another.

CORIOLANUS.
On fair ground
I could beat forty of them.

MENENIUS.
I could myself
Take up a brace o’ th’ best of them, yea, the two tribunes.

COMINIUS.
But now ’tis odds beyond arithmetic,
And manhood is called foolery when it stands
Against a falling fabric. Will you hence,
Before the tag return, whose rage doth rend
Like interrupted waters, and o’erbear
What they are used to bear?

MENENIUS.
Pray you, begone.
I’ll try whether my old wit be in request
With those that have but little. This must be patched
With cloth of any colour.

COMINIUS.
Nay, come away.

[_Exeunt Coriolanus and Cominius._]

PATRICIAN.
This man has marred his fortune.

MENENIUS.
His nature is too noble for the world.
He would not flatter Neptune for his trident
Or Jove for’s power to thunder. His heart’s his mouth;
What his breast forges, that his tongue must vent,
And, being angry, does forget that ever
He heard the name of death.

[_A noise within._]

Here’s goodly work.

PATRICIAN.
I would they were abed!

MENENIUS.
I would they were in Tiber! What the vengeance,
Could he not speak ’em fair?

Enter Brutus and Sicinius with the rabble again.

SICINIUS.
Where is this viper
That would depopulate the city and
Be every man himself?

MENENIUS.
You worthy tribunes—

SICINIUS.
He shall be thrown down the Tarpeian rock
With rigorous hands. He hath resisted law,
And therefore law shall scorn him further trial
Than the severity of the public power
Which he so sets at naught.

FIRST CITIZEN.
He shall well know
The noble tribunes are the people’s mouths,
And we their hands.

ALL PLEBEIANS.
He shall, sure on’t.

MENENIUS.
Sir, sir—

SICINIUS.
Peace!

MENENIUS.
Do not cry havoc where you should but hunt
With modest warrant.

SICINIUS.
Sir, how comes’t that you
Have holp to make this rescue?

MENENIUS.
Hear me speak.
As I do know the Consul’s worthiness,
So can I name his faults.

SICINIUS.
Consul? What consul?

MENENIUS.
The consul Coriolanus.

BRUTUS.
He consul?

ALL PLEBEIANS.
No, no, no, no, no!

MENENIUS.
If, by the Tribunes’ leave, and yours, good people,
I may be heard, I would crave a word or two,
The which shall turn you to no further harm
Than so much loss of time.

SICINIUS.
Speak briefly then,
For we are peremptory to dispatch
This viperous traitor. To eject him hence
Were but one danger, and to keep him here
Our certain death. Therefore it is decreed
He dies tonight.

MENENIUS.
Now the good gods forbid
That our renowned Rome, whose gratitude
Towards her deserved children is enrolled
In Jove’s own book, like an unnatural dam
Should now eat up her own.

SICINIUS.
He’s a disease that must be cut away.

MENENIUS.
O, he’s a limb that has but a disease—
Mortal to cut it off; to cure it easy.
What has he done to Rome that’s worthy death?
Killing our enemies, the blood he hath lost—
Which I dare vouch is more than that he hath
By many an ounce—he dropt it for his country;
And what is left, to lose it by his country
Were to us all, that do’t and suffer it
A brand to th’ end o’ th’ world.

SICINIUS.
This is clean cam.

BRUTUS.
Merely awry. When he did love his country,
It honoured him.

MENENIUS.
The service of the foot,
Being once gangrened, is not then respected
For what before it was.

BRUTUS.
We’ll hear no more.
Pursue him to his house, and pluck him thence,
Lest his infection, being of catching nature,
Spread further.

MENENIUS.
One word more, one word!
This tiger-footed rage, when it shall find
The harm of unscanned swiftness, will too late,
Tie leaden pounds to’s heels. Proceed by process,
Lest parties—as he is beloved—break out
And sack great Rome with Romans.

BRUTUS.
If it were so—

SICINIUS.
What do ye talk?
Have we not had a taste of his obedience?
Our aediles smote! Ourselves resisted? Come.

MENENIUS.
Consider this: he has been bred i’ th’ wars
Since he could draw a sword, and is ill schooled
In bolted language; meal and bran together
He throws without distinction. Give me leave,
I’ll go to him and undertake to bring him
Where he shall answer by a lawful form,
In peace, to his utmost peril.

FIRST SENATOR.
Noble tribunes,
It is the humane way: the other course
Will prove too bloody, and the end of it
Unknown to the beginning.

SICINIUS.
Noble Menenius,
Be you then as the people’s officer.—
Masters, lay down your weapons.

BRUTUS.
Go not home.

SICINIUS.
Meet on the marketplace. We’ll attend you there,
Where if you bring not Martius, we’ll proceed
In our first way.

MENENIUS.
I’ll bring him to you.
[_To Senators_.] Let me desire your company. He must come,
Or what is worst will follow.

FIRST SENATOR.
Pray you, let’s to him.

[_Exeunt._]

SCENE II. Rome. A room in Coriolanus’s house

Enter Coriolanus with Nobles.

CORIOLANUS.
Let them pull all about mine ears, present me
Death on the wheel or at wild horses’ heels,
Or pile ten hills on the Tarpeian rock,
That the precipitation might down stretch
Below the beam of sight, yet will I still
Be thus to them.

FIRST PATRICIAN.
You do the nobler.

CORIOLANUS.
I muse my mother
Does not approve me further, who was wont
To call them woollen vassals, things created
To buy and sell with groats, to show bare heads
In congregations, to yawn, be still, and wonder
When one but of my ordinance stood up
To speak of peace or war.

Enter Volumnia.

I talk of you.
Why did you wish me milder? Would you have me
False to my nature? Rather say I play
The man I am.

VOLUMNIA.
O, sir, sir, sir,
I would have had you put your power well on
Before you had worn it out.

CORIOLANUS.
Let go.

VOLUMNIA.
You might have been enough the man you are
With striving less to be so. Lesser had been
The thwartings of your dispositions if
You had not showed them how ye were disposed
Ere they lacked power to cross you.

CORIOLANUS.
Let them hang!

VOLUMNIA.
Ay, and burn too.

Enter Menenius with the Senators.

MENENIUS.
Come, come, you have been too rough, something too rough.
You must return and mend it.

FIRST SENATOR.
There’s no remedy,
Unless, by not so doing, our good city
Cleave in the midst and perish.

VOLUMNIA.
Pray be counselled.
I have a heart as little apt as yours,
But yet a brain that leads my use of anger
To better vantage.

MENENIUS.
Well said, noble woman.
Before he should thus stoop to th’ herd—but that
The violent fit o’ th’ time craves it as physic
For the whole state—I would put mine armour on,
Which I can scarcely bear.

CORIOLANUS.
What must I do?

MENENIUS.
Return to th’ Tribunes.

CORIOLANUS.
Well, what then? What then?

MENENIUS.
Repent what you have spoke.

CORIOLANUS.
For them? I cannot do it to the gods.
Must I then do’t to them?

VOLUMNIA.
You are too absolute,
Though therein you can never be too noble
But when extremities speak. I have heard you say
Honour and policy, like unsevered friends,
I’ th’ war do grow together. Grant that, and tell me
In peace what each of them by th’ other lose
That they combine not there.

CORIOLANUS.
Tush, tush!

MENENIUS.
A good demand.

VOLUMNIA.
If it be honour in your wars to seem
The same you are not, which for your best ends
You adopt your policy, how is it less or worse
That it shall hold companionship in peace
With honour as in war, since that to both
It stands in like request?

CORIOLANUS.
Why force you this?

VOLUMNIA.
Because that now it lies you on to speak
To th’ people, not by your own instruction,
Nor by th’ matter which your heart prompts you,
But with such words that are but rooted in
Your tongue, though but bastards and syllables
Of no allowance to your bosom’s truth.
Now, this no more dishonours you at all
Than to take in a town with gentle words,
Which else would put you to your fortune and
The hazard of much blood.
I would dissemble with my nature where
My fortunes and my friends at stake required
I should do so in honour. I am in this
Your wife, your son, these senators, the nobles;
And you will rather show our general louts
How you can frown than spend a fawn upon ’em
For the inheritance of their loves and safeguard
Of what that want might ruin.

MENENIUS.
Noble lady!—
Come, go with us; speak fair. You may salve so,
Not what is dangerous present, but the loss
Of what is past.

VOLUMNIA.
I prithee now, my son,
Go to them with this bonnet in thy hand,
And thus far having stretched it—here be with them—
Thy knee bussing the stones—for in such busines
Action is eloquence, and the eyes of th’ ignorant
More learned than the ears—waving thy head,
Which often thus correcting thy stout heart,
Now humble as the ripest mulberry
That will not hold the handling. Or say to them
Thou art their soldier and, being bred in broils,
Hast not the soft way, which thou dost confess
Were fit for thee to use, as they to claim,
In asking their good loves; but thou wilt frame
Thyself, forsooth, hereafter theirs, so far
As thou hast power and person.

MENENIUS.
This but done
Even as she speaks, why, their hearts were yours;
For they have pardons, being asked, as free
As words to little purpose.

VOLUMNIA.
Prithee now,
Go, and be ruled; although I know thou hadst rather
Follow thine enemy in a fiery gulf
Than flatter him in a bower.

Enter Cominius.

Here is Cominius.

COMINIUS.
I have been i’ th’ marketplace; and, sir, ’tis fit
You make strong party or defend yourself
By calmness or by absence. All’s in anger.

MENENIUS.
Only fair speech.

COMINIUS.
I think ’twill serve, if he
Can thereto frame his spirit.

VOLUMNIA.
He must, and will.—
Prithee, now, say you will, and go about it.

CORIOLANUS.
Must I go show them my unbarbed sconce? Must I
With my base tongue give to my noble heart
A lie that it must bear? Well, I will do’t.
Yet, were there but this single plot to lose,
This mould of Martius, they to dust should grind it
And throw’t against the wind. To th’ marketplace!
You have put me now to such a part which never
I shall discharge to th’ life.

COMINIUS.
Come, come, we’ll prompt you.

VOLUMNIA.
I prithee now, sweet son, as thou hast said
My praises made thee first a soldier, so,
To have my praise for this, perform a part
Thou hast not done before.

CORIOLANUS.
Well, I must do’t.
Away, my disposition, and possess me
Some harlot’s spirit! My throat of war be turned,
Which choired with my drum, into a pipe
Small as an eunuch or the virgin voice
That babies lulls asleep! The smiles of knaves
Tent in my cheeks, and schoolboys’ tears take up
The glasses of my sight! A beggar’s tongue
Make motion through my lips, and my armed knees,
Who bowed but in my stirrup, bend like his
That hath received an alms! I will not do’t,
Lest I surcease to honour mine own truth
And, by my body’s action, teach my mind
A most inherent baseness.

VOLUMNIA.
At thy choice, then.
To beg of thee, it is my more dishonour
Than thou of them. Come all to ruin. Let
Thy mother rather feel thy pride than fear
Thy dangerous stoutness, for I mock at death
With as big heart as thou. Do as thou list.
Thy valiantness was mine; thou suck’dst it from me,
But owe thy pride thyself.

CORIOLANUS.
Pray, be content.
Mother, I am going to the marketplace.
Chide me no more. I’ll mountebank their loves,
Cog their hearts from them, and come home beloved
Of all the trades in Rome. Look, I am going.
Commend me to my wife. I’ll return consul,
Or never trust to what my tongue can do
I’ th’ way of flattery further.

VOLUMNIA.
Do your will.

[_Exit Volumnia._]

COMINIUS.
Away! The Tribunes do attend you. Arm yourself
To answer mildly, for they are prepared
With accusations, as I hear, more strong
Than are upon you yet.

CORIOLANUS.
The word is “mildly.” Pray you, let us go.
Let them accuse me by invention, I
Will answer in mine honour.

MENENIUS.
Ay, but mildly.

CORIOLANUS.
Well, mildly be it, then. Mildly.

[_Exeunt._]

SCENE III. Rome. The Forum

Enter Sicinius and Brutus.

BRUTUS.
In this point charge him home, that he affects
Tyrannical power. If he evade us there,
Enforce him with his envy to the people,
And that the spoil got on the Antiates
Was ne’er distributed.

Enter an Aedile.

What, will he come?

AEDILE.
He’s coming.

BRUTUS.
How accompanied?

AEDILE.
With old Menenius, and those senators
That always favoured him.

SICINIUS.
Have you a catalogue
Of all the voices that we have procured,
Set down by th’ poll?

AEDILE.
I have. ’Tis ready.

SICINIUS.
Have you collected them by tribes?

AEDILE.
I have.

SICINIUS.
Assemble presently the people hither;
And when they hear me say “It shall be so
I’ th’ right and strength o’ th’ commons,” be it either
For death, for fine, or banishment, then let them
If I say “Fine,” cry “Fine,” if “Death,” cry “Death,”
Insisting on the old prerogative
And power i’ th’ truth o’ th’ cause.

AEDILE.
I shall inform them.

BRUTUS.
And when such time they have begun to cry,
Let them not cease, but with a din confused
Enforce the present execution
Of what we chance to sentence.

AEDILE.
Very well.

SICINIUS.
Make them be strong and ready for this hint
When we shall hap to give’t them.

BRUTUS.
Go about it.

[_Exit Aedile._]

Put him to choler straight. He hath been used
Ever to conquer and to have his worth
Of contradiction. Being once chafed, he cannot
Be reined again to temperance; then he speaks
What’s in his heart; and that is there which looks
With us to break his neck.

Enter Coriolanus, Menenius and Cominius with other Senators.

SICINIUS.
Well, here he comes.

MENENIUS.
Calmly, I do beseech you.

CORIOLANUS.
Ay, as an ostler, that for th’ poorest piece
Will bear the knave by th’ volume.—Th’ honoured gods
Keep Rome in safety and the chairs of justice
Supplied with worthy men! Plant love among’s!
Throng our large temples with the shows of peace
And not our streets with war!

FIRST SENATOR.
Amen, amen.

MENENIUS.
A noble wish.

Enter the Aedile with the Plebeians.

SICINIUS.
Draw near, ye people.

AEDILE.
List to your tribunes. Audience! Peace, I say!

CORIOLANUS.
First, hear me speak.

BOTH TRIBUNES.
Well, say.—Peace, ho!

CORIOLANUS.
Shall I be charged no further than this present?
Must all determine here?

SICINIUS.
I do demand
If you submit you to the people’s voices,
Allow their officers, and are content
To suffer lawful censure for such faults
As shall be proved upon you.

CORIOLANUS.
I am content.

MENENIUS.
Lo, citizens, he says he is content.
The warlike service he has done, consider. Think
Upon the wounds his body bears, which show
Like graves i’ th’ holy churchyard.

CORIOLANUS.
Scratches with briars,
Scars to move laughter only.

MENENIUS.
Consider further,
That when he speaks not like a citizen,
You find him like a soldier. Do not take
His rougher accents for malicious sounds,
But, as I say, such as become a soldier
Rather than envy you.

COMINIUS.
Well, well, no more.

CORIOLANUS.
What is the matter,
That, being passed for consul with full voice,
I am so dishonoured that the very hour
You take it off again?

SICINIUS.
Answer to us.

CORIOLANUS.
Say then. ’Tis true, I ought so.

SICINIUS.
We charge you that you have contrived to take
From Rome all seasoned office and to wind
Yourself into a power tyrannical,
For which you are a traitor to the people.

CORIOLANUS.
How? Traitor?

MENENIUS.
Nay, temperately! Your promise.

CORIOLANUS.
The fires i’ th’ lowest hell fold in the people!
Call me their traitor? Thou injurious tribune!
Within thine eyes sat twenty thousand deaths,
In thy hands clutched as many millions, in
Thy lying tongue both numbers, I would say
“Thou liest” unto thee with a voice as free
As I do pray the gods.

SICINIUS.
Mark you this, people?

ALL PLEBEIANS.
To th’ rock, to th’ rock with him!

SICINIUS.
Peace!
We need not put new matter to his charge.
What you have seen him do and heard him speak,
Beating your officers, cursing yourselves,
Opposing laws with strokes, and here defying
Those whose great power must try him—even this,
So criminal and in such capital kind,
Deserves th’ extremest death.

BRUTUS.
But since he hath
Served well for Rome—

CORIOLANUS.
What do you prate of service?

BRUTUS.
I talk of that that know it.

CORIOLANUS.
You?

MENENIUS.
Is this the promise that you made your mother?

COMINIUS.
Know, I pray you—

CORIOLANUS.
I’ll know no further.
Let them pronounce the steep Tarpeian death,
Vagabond exile, flaying, pent to linger
But with a grain a day, I would not buy
Their mercy at the price of one fair word,
Nor check my courage for what they can give,
To have’t with saying “Good morrow.”

SICINIUS.
For that he has,
As much as in him lies, from time to time
Envied against the people, seeking means
To pluck away their power, as now at last
Given hostile strokes, and that not in the presence
Of dreaded justice, but on the ministers
That do distribute it, in the name o’ th’ people
And in the power of us the Tribunes, we,
Even from this instant, banish him our city
In peril of precipitation
From off the rock Tarpeian, never more
To enter our Rome gates. I’ th’ people’s name,
I say it shall be so.

ALL PLEBEIANS.
It shall be so, it shall be so! Let him away!
He’s banished, and it shall be so.

COMINIUS.
Hear me, my masters and my common friends—

SICINIUS.
He’s sentenced. No more hearing.

COMINIUS.
Let me speak.
I have been consul and can show for Rome
Her enemies’ marks upon me. I do love
My country’s good with a respect more tender,
More holy and profound, than mine own life,
My dear wife’s estimate, her womb’s increase,
And treasure of my loins. Then if I would
Speak that—

SICINIUS.
We know your drift. Speak what?

BRUTUS.
There’s no more to be said, but he is banished
As enemy to the people and his country.
It shall be so.

ALL PLEBEIANS.
It shall be so, it shall be so!

CORIOLANUS.
You common cry of curs, whose breath I hate
As reek o’ th’ rotten fens, whose loves I prize
As the dead carcasses of unburied men
That do corrupt my air, I banish you!
And here remain with your uncertainty;
Let every feeble rumour shake your hearts;
Your enemies, with nodding of their plumes,
Fan you into despair! Have the power still
To banish your defenders, till at length
Your ignorance—which finds not till it feels,
Making but reservation of yourselves,
Still your own foes—deliver you,
As most abated captives to some nation
That won you without blows! Despising
For you the city, thus I turn my back.
There is a world elsewhere.

[_Exeunt Coriolanus, Cominius, with other Senators._]

AEDILE.
The people’s enemy is gone, is gone.

ALL PLEBEIANS.
Our enemy is banished; he is gone. Hoo, hoo!

[_They all shout and throw up their caps._]

SICINIUS.
Go see him out at gates, and follow him,
As he hath followed you, with all despite.
Give him deserved vexation. Let a guard
Attend us through the city.

ALL PLEBEIANS.
Come, come, let’s see him out at gates! Come!
The gods preserve our noble tribunes! Come.

[_Exeunt._]




ACT IV

SCENE I. Rome. Before a gate of the city


Enter Coriolanus, Volumnia, Virgilia, Menenius, Cominius with the young
nobility of Rome.

CORIOLANUS.
Come, leave your tears. A brief farewell. The beast
With many heads butts me away. Nay, mother,
Where is your ancient courage? You were used
To say extremities was the trier of spirits;
That common chances common men could bear;
That when the sea was calm, all boats alike
Showed mastership in floating; fortune’s blows
When most struck home, being gentle wounded craves
A noble cunning. You were used to load me
With precepts that would make invincible
The heart that conned them.

VIRGILIA.
O heavens! O heavens!

CORIOLANUS.
Nay, I prithee, woman—

VOLUMNIA.
Now the red pestilence strike all trades in Rome,
And occupations perish!

CORIOLANUS.
What, what, what!
I shall be loved when I am lacked. Nay, mother,
Resume that spirit when you were wont to say
If you had been the wife of Hercules,
Six of his labours you’d have done and saved
Your husband so much sweat.—Cominius,
Droop not. Adieu.—Farewell, my wife, my mother.
I’ll do well yet.—Thou old and true Menenius,
Thy tears are salter than a younger man’s
And venomous to thine eyes.—My sometime general,
I have seen thee stern, and thou hast oft beheld
Heart-hard’ning spectacles. Tell these sad women
’Tis fond to wail inevitable strokes
As ’tis to laugh at ’em.—My mother, you wot well
My hazards still have been your solace, and—
Believe’t not lightly—though I go alone,
Like to a lonely dragon that his fen
Makes feared and talked of more than seen, your son
Will or exceed the common or be caught
With cautelous baits and practice.

VOLUMNIA.
My first son,
Whither wilt thou go? Take good Cominius
With thee awhile. Determine on some course
More than a wild exposture to each chance
That starts i’ th’ way before thee.

VIRGILIA.
O the gods!

COMINIUS.
I’ll follow thee a month, devise with thee
Where thou shalt rest, that thou mayst hear of us
And we of thee; so if the time thrust forth
A cause for thy repeal, we shall not send
O’er the vast world to seek a single man
And lose advantage, which doth ever cool
I’ th’ absence of the needer.

CORIOLANUS.
Fare ye well.
Thou hast years upon thee, and thou art too full
Of the wars’ surfeits to go rove with one
That’s yet unbruised. Bring me but out at gate.—
Come, my sweet wife, my dearest mother, and
My friends of noble touch. When I am forth,
Bid me farewell, and smile. I pray you, come.
While I remain above the ground, you shall
Hear from me still, and never of me aught
But what is like me formerly.

MENENIUS.
That’s worthily
As any ear can hear. Come, let’s not weep.
If I could shake off but one seven years
From these old arms and legs, by the good gods,
I’d with thee every foot.

CORIOLANUS.
Give me thy hand.
Come.

[_Exeunt._]

SCENE II. Rome. A street near the gate

Enter two Tribunes, Sicinius, Brutus with the Aedile.

SICINIUS.
Bid them all home. He’s gone, and we’ll no further.
The nobility are vexed, whom we see have sided
In his behalf.

BRUTUS.
Now we have shown our power,
Let us seem humbler after it is done
Than when it was a-doing.

SICINIUS.
Bid them home.
Say their great enemy is gone, and they
Stand in their ancient strength.

BRUTUS.
Dismiss them home.

[_Exit Aedile._]

Here comes his mother.

Enter Volumnia, Virgilia and Menenius.

SICINIUS.
Let’s not meet her.

BRUTUS.
Why?

SICINIUS.
They say she’s mad.

BRUTUS.
They have ta’en note of us. Keep on your way.

VOLUMNIA.
O, you’re well met. The hoarded plague o’ th’ gods
Requite your love!

MENENIUS.
Peace, peace! Be not so loud.

VOLUMNIA.
If that I could for weeping, you should hear—
Nay, and you shall hear some. [_To Sicinius_.] Will you be gone?

VIRGILIA.
[_To Brutus_.] You shall stay too. I would I had the power
To say so to my husband.

SICINIUS.
Are you mankind?

VOLUMNIA.
Ay, fool, is that a shame? Note but this, fool.
Was not a man my father? Hadst thou foxship
To banish him that struck more blows for Rome
Than thou hast spoken words?

SICINIUS.
O blessed heavens!

VOLUMNIA.
More noble blows than ever thou wise words,
And for Rome’s good. I’ll tell thee what—yet go.
Nay, but thou shalt stay too. I would my son
Were in Arabia and thy tribe before him,
His good sword in his hand.

SICINIUS.
What then?

VIRGILIA.
What then?
He’d make an end of thy posterity.

VOLUMNIA.
Bastards and all.
Good man, the wounds that he does bear for Rome!

MENENIUS.
Come, come, peace.

SICINIUS.
I would he had continued to his country
As he began, and not unknit himself
The noble knot he made.

BRUTUS.
I would he had.

VOLUMNIA.
“I would he had?” ’Twas you incensed the rabble.
Cats, that can judge as fitly of his worth
As I can of those mysteries which heaven
Will not have Earth to know.

BRUTUS.
Pray, let’s go.

VOLUMNIA.
Now, pray, sir, get you gone.
You have done a brave deed. Ere you go, hear this:
As far as doth the Capitol exceed
The meanest house in Rome, so far my son—
This lady’s husband here, this, do you see?—
Whom you have banished, does exceed you all.

BRUTUS.
Well, well, we’ll leave you.

SICINIUS.
Why stay we to be baited
With one that wants her wits?

[_Exeunt Tribunes._]

VOLUMNIA.
Take my prayers with you.
I would the gods had nothing else to do
But to confirm my curses. Could I meet ’em
But once a day, it would unclog my heart
Of what lies heavy to’t.

MENENIUS.
You have told them home,
And, by my troth, you have cause. You’ll sup with me?

VOLUMNIA.
Anger’s my meat. I sup upon myself
And so shall starve with feeding. Come, let’s go.
Leave this faint puling, and lament as I do,
In anger, Juno-like. Come, come, come.

[_Exeunt._]

MENENIUS.
Fie, fie, fie!

[_Exit Menenius._]

SCENE III. A highway between Rome and Antium

Enter a Roman and a Volsce.

ROMAN.
I know you well, sir, and you know me. Your name I think is Adrian.

VOLSCE.
It is so, sir. Truly, I have forgot you.

ROMAN.
I am a Roman, and my services are, as you are, against ’em. Know you me
yet?

VOLSCE.
Nicanor, no?

ROMAN.
The same, sir.

VOLSCE.
You had more beard when I last saw you, but your favour is well
approved by your tongue. What’s the news in Rome? I have a note from
the Volscian state to find you out there. You have well saved me a
day’s journey.

ROMAN.
There hath been in Rome strange insurrections, the people against the
senators, patricians, and nobles.

VOLSCE.
Hath been? Is it ended, then? Our state thinks not so. They are in a
most warlike preparation and hope to come upon them in the heat of
their division.

ROMAN.
The main blaze of it is past, but a small thing would make it flame
again; for the nobles receive so to heart the banishment of that worthy
Coriolanus that they are in a ripe aptness to take all power from the
people and to pluck from them their tribunes for ever. This lies
glowing, I can tell you, and is almost mature for the violent breaking
out.

VOLSCE.
Coriolanus banished?

ROMAN.
Banished, sir.

VOLSCE.
You will be welcome with this intelligence, Nicanor.

ROMAN.
The day serves well for them now. I have heard it said the fittest time
to corrupt a man’s wife is when she’s fallen out with her husband. Your
noble Tullus Aufidius will appear well in these wars, his great opposer
Coriolanus being now in no request of his country.

VOLSCE.
He cannot choose. I am most fortunate thus accidentally to encounter
you. You have ended my business, and I will merrily accompany you home.

ROMAN.
I shall between this and supper tell you most strange things from Rome,
all tending to the good of their adversaries. Have you an army ready,
say you?

VOLSCE.
A most royal one. The centurions and their charges, distinctly
billeted, already in th’ entertainment, and to be on foot at an hour’s
warning.

ROMAN.
I am joyful to hear of their readiness and am the man, I think, that
shall set them in present action. So, sir, heartily well met, and most
glad of your company.

VOLSCE.
You take my part from me, sir. I have the most cause to be glad of
yours.

ROMAN.
Well, let us go together.

[_Exeunt._]

SCENE IV. Antium. Before Aufidius’s house

Enter Coriolanus in mean apparel, disguised and muffled.

CORIOLANUS.
A goodly city is this Antium. City,
’Tis I that made thy widows. Many an heir
Of these fair edifices ’fore my wars
Have I heard groan and drop. Then know me not,
Lest that thy wives with spits and boys with stones
In puny battle slay me.

Enter a Citizen.

Save you, sir.

CITIZEN.
And you.

CORIOLANUS.
Direct me, if it be your will,
Where great Aufidius lies. Is he in Antium?

CITIZEN.
He is, and feasts the nobles of the state
At his house this night.

CORIOLANUS.
Which is his house, beseech you?

CITIZEN.
This here before you.

CORIOLANUS.
Thank you, sir. Farewell.

[_Exit Citizen._]

O world, thy slippery turns! Friends now fast sworn,
Whose double bosoms seems to wear one heart,
Whose hours, whose bed, whose meal and exercise
Are still together, who twin, as ’twere, in love
Unseparable, shall within this hour,
On a dissension of a doit, break out
To bitterest enmity; so fellest foes,
Whose passions and whose plots have broke their sleep
To take the one the other, by some chance,
Some trick not worth an egg, shall grow dear friends
And interjoin their issues. So with me:
My birthplace hate I, and my love’s upon
This enemy town. I’ll enter. If he slay me,
He does fair justice; if he give me way,
I’ll do his country service.

[_Exit._]

SCENE V. Antium. A hall in Aufidius’s house

Music plays. Enter a Servingman.

FIRST SERVINGMAN.
Wine, wine, wine! What service is here? I think our fellows are asleep.

[_Exit._]

Enter another Servingman.

SECOND SERVINGMAN.
Where’s Cotus? My master calls for him. Cotus!

[_Exit._]

Enter Coriolanus.

CORIOLANUS.
A goodly house. The feast smells well, but I
Appear not like a guest.

Enter the First Servingman.

FIRST SERVINGMAN.
What would you have, friend? Whence are you? Here’s no place for you.
Pray go to the door.

[_Exit._]

CORIOLANUS.
I have deserved no better entertainment
In being Coriolanus.

Enter Second Servingman.

SECOND SERVINGMAN.
Whence are you, sir?—Has the porter his eyes in his head, that he gives
entrance to such companions?—Pray, get you out.

CORIOLANUS.
Away!

SECOND SERVINGMAN.
Away? Get you away.

CORIOLANUS.
Now th’ art troublesome.

SECOND SERVINGMAN.
Are you so brave? I’ll have you talked with anon.

Enter Third  Servingman; the First, entering, meets him.

THIRD SERVINGMAN.
What fellow’s this?

FIRST SERVINGMAN.
A strange one as ever I looked on. I cannot get him out o’ th’ house.
Prithee call my master to him.

THIRD SERVINGMAN.
What have you to do here, fellow? Pray you, avoid the house.

CORIOLANUS.
Let me but stand. I will not hurt your hearth.

THIRD SERVINGMAN.
What are you?

CORIOLANUS.
A gentleman.

THIRD SERVINGMAN.
A marv’llous poor one.

CORIOLANUS.
True, so I am.

THIRD SERVINGMAN.
Pray you, poor gentleman, take up some other station. Here’s no place
for you. Pray you, avoid. Come.

CORIOLANUS.
Follow your function, go, and batten on cold bits.

[_Pushes him away from him_.]

THIRD SERVINGMAN.
What, you will not?—Prithee, tell my master what a strange guest he has
here.

SECOND SERVINGMAN.
And I shall.

[_Exit._]

THIRD SERVINGMAN.
Where dwell’st thou?

CORIOLANUS.
Under the canopy.

THIRD SERVINGMAN.
Under the canopy?

CORIOLANUS.
Ay.

THIRD SERVINGMAN.
Where’s that?

CORIOLANUS.
I’ th’ city of kites and crows.

THIRD SERVINGMAN.
I’ th’ city of kites and crows? What an ass it is! Then thou dwell’st
with daws too?

CORIOLANUS.
No, I serve not thy master.

THIRD SERVINGMAN.
How, sir? Do you meddle with my master?

CORIOLANUS.
Ay, ’tis an honester service than to meddle with thy mistress. Thou
prat’st and prat’st. Serve with thy trencher, hence!

[_Beats him away_.]

[_Exit Third Servingman._]

Enter Aufidius with the Second Servingman.

AUFIDIUS.
Where is this fellow?

SECOND SERVINGMAN.
Here, sir. I’d have beaten him like a dog, but for disturbing the lords
within.

AUFIDIUS.
Whence com’st thou? What wouldst thou?
Thy name? Why speak’st not? Speak, man. What’s thy name?

CORIOLANUS.
[_Removing his muffler_.] If, Tullus,
Not yet thou know’st me, and, seeing me, dost not
Think me for the man I am, necessity
Commands me name myself.

AUFIDIUS.
What is thy name?

CORIOLANUS.
A name unmusical to the Volscians’ ears
And harsh in sound to thine.

AUFIDIUS.
Say, what’s thy name?
Thou has a grim appearance, and thy face
Bears a command in’t. Though thy tackle’s torn,
Thou show’st a noble vessel. What’s thy name?

CORIOLANUS.
Prepare thy brow to frown. Know’st thou me yet?

AUFIDIUS.
I know thee not. Thy name?

CORIOLANUS.
My name is Caius Martius, who hath done
To thee particularly and to all the Volsces
Great hurt and mischief; thereto witness may
My surname Coriolanus. The painful service,
The extreme dangers, and the drops of blood
Shed for my thankless country are requited
But with that surname, a good memory
And witness of the malice and displeasure
Which thou shouldst bear me. Only that name remains.
The cruelty and envy of the people,
Permitted by our dastard nobles, who
Have all forsook me, hath devoured the rest,
And suffered me by th’ voice of slaves to be
Whooped out of Rome. Now this extremity
Hath brought me to thy hearth, not out of hope—
Mistake me not—to save my life; for if
I had feared death, of all the men i’ th’ world
I would have ’voided thee, but in mere spite,
To be full quit of those my banishers,
Stand I before thee here. Then if thou hast
A heart of wreak in thee, that wilt revenge
Thine own particular wrongs and stop those maims
Of shame seen through thy country, speed thee straight
And make my misery serve thy turn. So use it
That my revengeful services may prove
As benefits to thee, for I will fight
Against my cankered country with the spleen
Of all the under fiends. But if so be
Thou dar’st not this, and that to prove more fortunes
Thou ’rt tired, then, in a word, I also am
Longer to live most weary, and present
My throat to thee and to thy ancient malice,
Which not to cut would show thee but a fool,
Since I have ever followed thee with hate,
Drawn tuns of blood out of thy country’s breast,
And cannot live but to thy shame, unless
It be to do thee service.

AUFIDIUS.
O Martius, Martius,
Each word thou hast spoke hath weeded from my heart
A root of ancient envy. If Jupiter
Should from yond cloud speak divine things
And say ’tis true, I’d not believe them more
Than thee, all-noble Martius. Let me twine
Mine arms about that body, whereagainst
My grained ash an hundred times hath broke
And scarred the moon with splinters. Here I clip
The anvil of my sword and do contest
As hotly and as nobly with thy love
As ever in ambitious strength I did
Contend against thy valour. Know thou first,
I loved the maid I married; never man
Sighed truer breath. But that I see thee here,
Thou noble thing, more dances my rapt heart
Than when I first my wedded mistress saw
Bestride my threshold. Why, thou Mars, I tell thee
We have a power on foot, and I had purpose
Once more to hew thy target from thy brawn
Or lose mine arm for’t. Thou hast beat me out
Twelve several times, and I have nightly since
Dreamt of encounters ’twixt thyself and me;
We have been down together in my sleep,
Unbuckling helms, fisting each other’s throat,
And waked half dead with nothing. Worthy Martius,
Had we no other quarrel else to Rome but that
Thou art thence banished, we would muster all
From twelve to seventy and, pouring war
Into the bowels of ungrateful Rome,
Like a bold flood o’erbear ’t. O, come, go in,
And take our friendly senators by th’ hands,
Who now are here, taking their leaves of me,
Who am prepared against your territories,
Though not for Rome itself.

CORIOLANUS.
You bless me, gods!

AUFIDIUS.
Therefore, most absolute sir, if thou wilt have
The leading of thine own revenges, take
Th’ one half of my commission and set down—
As best thou art experienced, since thou know’st
Thy country’s strength and weakness—thine own ways,
Whether to knock against the gates of Rome,
Or rudely visit them in parts remote
To fright them ere destroy. But come in.
Let me commend thee first to those that shall
Say yea to thy desires. A thousand welcomes!
And more a friend than e’er an enemy—
Yet, Martius, that was much. Your hand. Most welcome!

[_Exeunt Coriolanus and Aufidius._]

Two of the Servingmen come forward.

FIRST SERVINGMAN.
Here’s a strange alteration!

SECOND SERVINGMAN.
By my hand, I had thought to have strucken him with a cudgel, and yet
my mind gave me his clothes made a false report of him.

FIRST SERVINGMAN.
What an arm he has! He turned me about with his finger and his thumb as
one would set up a top.

SECOND SERVINGMAN.
Nay, I knew by his face that there was something in him. He had, sir, a
kind of face, methought—I cannot tell how to term it.

FIRST SERVINGMAN.
He had so, looking as it were—Would I were hanged, but I thought there
was more in him than I could think.

SECOND SERVINGMAN.
So did I, I’ll be sworn. He is simply the rarest man i’ th’ world.

FIRST SERVINGMAN.
I think he is. But a greater soldier than he you wot one.

SECOND SERVINGMAN.
Who, my master?

FIRST SERVINGMAN.
Nay, it’s no matter for that.

SECOND SERVINGMAN.
Worth six on him.

FIRST SERVINGMAN.
Nay, not so neither. But I take him to be the greater soldier.

SECOND SERVINGMAN.
Faith, look you, one cannot tell how to say that. For the defence of a
town our general is excellent.

FIRST SERVINGMAN.
Ay, and for an assault too.

Enter the Third Servingman.

THIRD SERVINGMAN.
O slaves, I can tell you news, news, you rascals!

FIRST and SECOND SERVINGMAN.
What, what, what? Let’s partake.

THIRD SERVINGMAN.
I would not be a Roman, of all nations; I had as lief be a condemned
man.

FIRST and SECOND SERVINGMAN.
Wherefore? Wherefore?

THIRD SERVINGMAN.
Why, here’s he that was wont to thwack our general, Caius Martius.

FIRST SERVINGMAN.
Why do you say, “thwack our general”?

THIRD SERVINGMAN.
I do not say “thwack our general,” but he was always good enough for
him.

SECOND SERVINGMAN.
Come, we are fellows and friends. He was ever too hard for him; I have
heard him say so himself.

FIRST SERVINGMAN.
He was too hard for him directly, to say the troth on’t, before
Corioles; he scotched him and notched him like a carbonado.

SECOND SERVINGMAN.
An he had been cannibally given, he might have boiled and eaten him
too.

FIRST SERVINGMAN.
But, more of thy news?

THIRD SERVINGMAN.
Why, he is so made on here within as if he were son and heir to Mars;
set at upper end o’ th’ table; no question asked him by any of the
senators but they stand bald before him. Our general himself makes a
mistress of him, sanctifies himself with’s hand, and turns up the white
o’ th’ eye to his discourse. But the bottom of the news is, our general
is cut i’ th’ middle and but one half of what he was yesterday, for the
other has half, by the entreaty and grant of the whole table. He’ll go,
he says, and sowl the porter of Rome gates by th’ ears. He will mow all
down before him and leave his passage polled.

SECOND SERVINGMAN.
And he’s as like to do’t as any man I can imagine.

THIRD SERVINGMAN.
Do’t? He will do’t! For look you, sir, he has as many friends as
enemies, which friends, sir, as it were, durst not, look you, sir, show
themselves, as we term it, his friends whilest he’s in directitude.

FIRST SERVINGMAN.
Directitude? What’s that?

THIRD SERVINGMAN.
But when they shall see, sir, his crest up again, and the man in blood,
they will out of their burrows like coneys after rain, and revel all
with him.

FIRST SERVINGMAN.
But when goes this forward?

THIRD SERVINGMAN.
Tomorrow, today, presently. You shall have the drum struck up this
afternoon. ’Tis as it were parcel of their feast, and to be executed
ere they wipe their lips.

SECOND SERVINGMAN.
Why then, we shall have a stirring world again. This peace is nothing
but to rust iron, increase tailors, and breed ballad-makers.

FIRST SERVINGMAN.
Let me have war, say I. It exceeds peace as far as day does night. It’s
sprightly walking, audible, and full of vent. Peace is a very apoplexy,
lethargy; mulled, deaf, sleepy, insensible; a getter of more bastard
children than war’s a destroyer of men.

SECOND SERVINGMAN.
’Tis so, and as war in some sort, may be said to be a ravisher, so it
cannot be denied but peace is a great maker of cuckolds.

FIRST SERVINGMAN.
Ay, and it makes men hate one another.

THIRD SERVINGMAN.
Reason: because they then less need one another. The wars for my money!
I hope to see Romans as cheap as Volscians. They are rising; they are
rising.

ALL.
In, in, in, in!

[_Exeunt._]

SCENE VI. Rome. A public place

Enter the two Tribunes. Sicinius and Brutus.

SICINIUS.
We hear not of him, neither need we fear him.
His remedies are tame—the present peace,
And quietness of the people, which before
Were in wild hurry. Here do we make his friends
Blush that the world goes well, who rather had,
Though they themselves did suffer by’t, behold
Dissentious numbers pest’ring streets than see
Our tradesmen singing in their shops and going
About their functions friendly.

BRUTUS.
We stood to’t in good time.

Enter Menenius.

Is this Menenius?

SICINIUS.
’Tis he, ’tis he. O, he is grown most kind
Of late.—Hail, sir!

MENENIUS.
Hail to you both.

SICINIUS.
Your Coriolanus is not much missed
But with his friends. The commonwealth doth stand,
And so would do were he more angry at it.

MENENIUS.
All’s well, and might have been much better if
He could have temporized.

SICINIUS.
Where is he, hear you?

MENENIUS.
Nay, I hear nothing;
His mother and his wife hear nothing from him.

Enter three or four Citizens.

ALL CITIZENS.
The gods preserve you both!

SICINIUS.
Good e’en, our neighbours.

BRUTUS.
Good e’en to you all, good e’en to you all.

FIRST CITIZEN.
Ourselves, our wives, and children, on our knees
Are bound to pray for you both.

SICINIUS.
Live and thrive!

BRUTUS.
Farewell, kind neighbours. We wished Coriolanus
Had loved you as we did.

CITIZENS.
Now the gods keep you!

BOTH TRIBUNES.
Farewell, farewell.

[_Exeunt Citizens._]

SICINIUS.
This is a happier and more comely time
Than when these fellows ran about the streets
Crying confusion.

BRUTUS.
Caius Martius was
A worthy officer i’ th’ war, but insolent,
O’ercome with pride, ambitious, past all thinking
Self-loving.

SICINIUS.
And affecting one sole throne, without assistance.

MENENIUS.
I think not so.

SICINIUS.
We should by this, to all our lamentation,
If he had gone forth consul, found it so.

BRUTUS.
The gods have well prevented it, and Rome
Sits safe and still without him.

Enter an Aedile.

AEDILE.
Worthy tribunes,
There is a slave, whom we have put in prison,
Reports the Volsces with two several powers
Are entered in the Roman territories,
And with the deepest malice of the war
Destroy what lies before ’em.

MENENIUS.
’Tis Aufidius,
Who, hearing of our Martius’ banishment,
Thrusts forth his horns again into the world,
Which were inshelled when Martius stood for Rome,
And durst not once peep out.

SICINIUS.
Come, what talk you of Martius?

BRUTUS.
Go see this rumourer whipped. It cannot be
The Volsces dare break with us.

MENENIUS.
Cannot be?
We have record that very well it can,
And three examples of the like hath been
Within my age. But reason with the fellow
Before you punish him, where he heard this,
Lest you shall chance to whip your information
And beat the messenger who bids beware
Of what is to be dreaded.

SICINIUS.
Tell not me.
I know this cannot be.

BRUTUS.
Not possible.

Enter a Messenger.

MESSENGER.
The nobles in great earnestness are going
All to the Senate House. Some news is coming
That turns their countenances.

SICINIUS.
’Tis this slave—
Go whip him ’fore the people’s eyes—his raising,
Nothing but his report.

MESSENGER.
Yes, worthy sir,
The slave’s report is seconded, and more,
More fearful, is delivered.

SICINIUS.
What more fearful?

MESSENGER.
It is spoke freely out of many mouths—
How probable I do not know—that Martius,
Joined with Aufidius, leads a power ’gainst Rome
And vows revenge as spacious as between
The young’st and oldest thing.

SICINIUS.
This is most likely!

BRUTUS.
Raised only that the weaker sort may wish
Good Martius home again.

SICINIUS.
The very trick on ’t.

MENENIUS.
This is unlikely;
He and Aufidius can no more atone
Than violent’st contrariety.

Enter a Second Messenger.

SECOND MESSENGER.
You are sent for to the Senate.
A fearful army, led by Caius Martius
Associated with Aufidius, rages
Upon our territories, and have already
O’erborne their way, consumed with fire and took
What lay before them.

Enter Cominius.

COMINIUS.
O, you have made good work!

MENENIUS.
What news? What news?

COMINIUS.
You have holp to ravish your own daughters and
To melt the city leads upon your pates,
To see your wives dishonoured to your noses—

MENENIUS.
What’s the news? What’s the news?

COMINIUS.
Your temples burned in their cement, and
Your franchises, whereon you stood, confined
Into an auger’s bore.

MENENIUS.
Pray now, your news?—
You have made fair work, I fear me.—Pray, your news?
If Martius should be joined with Volscians—

COMINIUS.
If?
He is their god; he leads them like a thing
Made by some other deity than Nature,
That shapes man better; and they follow him
Against us brats with no less confidence
Than boys pursuing summer butterflies
Or butchers killing flies.

MENENIUS.
You have made good work,
You and your apron-men, you that stood so much
Upon the voice of occupation and
The breath of garlic eaters!

COMINIUS.
He’ll shake your Rome about your ears.

MENENIUS.
As Hercules did shake down mellow fruit.
You have made fair work.

BRUTUS.
But is this true, sir?

COMINIUS.
Ay, and you’ll look pale
Before you find it other. All the regions
Do smilingly revolt, and who resists
Are mocked for valiant ignorance
And perish constant fools. Who is’t can blame him?
Your enemies and his find something in him.

MENENIUS.
We are all undone unless
The noble man have mercy.

COMINIUS.
Who shall ask it?
The Tribunes cannot do’t for shame; the people
Deserve such pity of him as the wolf
Does of the shepherds. For his best friends, if they
Should say “Be good to Rome,” they charged him even
As those should do that had deserved his hate
And therein showed like enemies.

MENENIUS.
’Tis true.
If he were putting to my house the brand
That should consume it, I have not the face
To say “Beseech you, cease.”—You have made fair hands,
You and your crafts! You have crafted fair!

COMINIUS.
You have brought
A trembling upon Rome such as was never
S’ incapable of help.

TRIBUNES.
Say not we brought it.

MENENIUS.
How? Was it we? We loved him, but like beasts
And cowardly nobles, gave way unto your clusters,
Who did hoot him out o’ th’ city.

COMINIUS.
But I fear
They’ll roar him in again. Tullus Aufidius,
The second name of men, obeys his points
As if he were his officer. Desperation
Is all the policy, strength, and defence
That Rome can make against them.

Enter a troop of Citizens.

MENENIUS.
Here comes the clusters.—
And is Aufidius with him? You are they
That made the air unwholesome when you cast
Your stinking, greasy caps in hooting at
Coriolanus’ exile. Now he’s coming,
And not a hair upon a soldier’s head
Which will not prove a whip. As many coxcombs
As you threw caps up will he tumble down
And pay you for your voices. ’Tis no matter.
If he could burn us all into one coal
We have deserved it.

ALL CITIZENS.
Faith, we hear fearful news.

FIRST CITIZEN.
For mine own part,
When I said banish him, I said ’twas pity.

SECOND CITIZEN.
And so did I.

THIRD CITIZEN.
And so did I. And, to say the truth, so did very many of us. That we
did we did for the best; and though we willingly consented to his
banishment, yet it was against our will.

COMINIUS.
You are goodly things, you voices!

MENENIUS.
You have made good work, you and your cry!—
Shall’s to the Capitol?

COMINIUS.
O, ay, what else?

[_Exeunt Cominius and Menenius._]

SICINIUS.
Go, masters, get you home. Be not dismayed.
These are a side that would be glad to have
This true which they so seem to fear. Go home,
And show no sign of fear.

FIRST CITIZEN.
The gods be good to us! Come, masters, let’s home. I ever said we were
i’ th’ wrong when we banished him.

SECOND CITIZEN.
So did we all. But, come, let’s home.

[_Exeunt Citizens._]

BRUTUS.
I do not like this news.

SICINIUS.
Nor I.

BRUTUS.
Let’s to the Capitol. Would half my wealth
Would buy this for a lie!

SICINIUS.
Pray let’s go.

[_Exeunt._]

SCENE VII. A camp at a short distance from Rome

Enter Aufidius with his Lieutenant.

AUFIDIUS.
Do they still fly to th’ Roman?

LIEUTENANT.
I do not know what witchcraft’s in him, but
Your soldiers use him as the grace ’fore meat,
Their talk at table, and their thanks at end;
And you are dark’ned in this action, sir,
Even by your own.

AUFIDIUS.
I cannot help it now,
Unless by using means I lame the foot
Of our design. He bears himself more proudlier,
Even to my person, than I thought he would
When first I did embrace him. Yet his nature
In that’s no changeling, and I must excuse
What cannot be amended.

LIEUTENANT.
Yet I wish, sir—
I mean for your particular—you had not
Joined in commission with him, but either
Had borne the action of yourself or else
To him had left it solely.

AUFIDIUS.
I understand thee well, and be thou sure,
When he shall come to his account, he knows not
What I can urge against him, although it seems,
And so he thinks and is no less apparent
To th’ vulgar eye, that he bears all things fairly,
And shows good husbandry for the Volscian state,
Fights dragonlike, and does achieve as soon
As draw his sword; yet he hath left undone
That which shall break his neck or hazard mine
Whene’er we come to our account.

LIEUTENANT.
Sir, I beseech you, think you he’ll carry Rome?

AUFIDIUS.
All places yield to him ere he sits down,
And the nobility of Rome are his;
The Senators and Patricians love him too.
The Tribunes are no soldiers, and their people
Will be as rash in the repeal as hasty
To expel him thence. I think he’ll be to Rome
As is the osprey to the fish, who takes it
By sovereignty of nature. First, he was
A noble servant to them, but he could not
Carry his honours even. Whether ’twas pride,
Which out of daily fortune ever taints
The happy man; whether defect of judgment,
To fail in the disposing of those chances
Which he was lord of; or whether nature,
Not to be other than one thing, not moving
From th’ casque to th’ cushion, but commanding peace
Even with the same austerity and garb
As he controlled the war; but one of these—
As he hath spices of them all—not all,
For I dare so far free him—made him feared,
So hated, and so banished. But he has a merit
To choke it in the utt’rance. So our virtues
Lie in th’ interpretation of the time,
And power, unto itself most commendable,
Hath not a tomb so evident as a chair
T’ extol what it hath done.
One fire drives out one fire, one nail one nail;
Rights by rights falter; strengths by strengths do fail.
Come, let’s away. When, Caius, Rome is thine,
Thou art poor’st of all; then shortly art thou mine.

[_Exeunt._]




ACT V

SCENE I. Rome. A public place


Enter Menenius, Cominius, Sicinius, Brutus (the two Tribunes), with
others.

MENENIUS.
No, I’ll not go. You hear what he hath said
Which was sometime his general, who loved him
In a most dear particular. He called me father,
But what o’ that? Go you that banished him;
A mile before his tent, fall down, and knee
The way into his mercy. Nay, if he coyed
To hear Cominius speak, I’ll keep at home.

COMINIUS.
He would not seem to know me.

MENENIUS.
Do you hear?

COMINIUS.
Yet one time he did call me by my name.
I urged our old acquaintance, and the drops
That we have bled together. “Coriolanus”
He would not answer to, forbade all names.
He was a kind of nothing, titleless,
Till he had forged himself a name i’ th’ fire
Of burning Rome.

MENENIUS.
Why, so; you have made good work!
A pair of tribunes that have wracked Rome
To make coals cheap! A noble memory!

COMINIUS.
I minded him how royal ’twas to pardon
When it was less expected. He replied
It was a bare petition of a state
To one whom they had punished.

MENENIUS.
Very well.
Could he say less?

COMINIUS.
I offered to awaken his regard
For’s private friends. His answer to me was
He could not stay to pick them in a pile
Of noisome musty chaff. He said ’twas folly
For one poor grain or two to leave unburnt
And still to nose th’ offence.

MENENIUS.
For one poor grain or two!
I am one of those! His mother, wife, his child,
And this brave fellow too, we are the grains;
You are the musty chaff, and you are smelt
Above the moon. We must be burnt for you.

SICINIUS.
Nay, pray, be patient. If you refuse your aid
In this so-never-needed help, yet do not
Upbraid’s with our distress. But sure, if you
Would be your country’s pleader, your good tongue,
More than the instant army we can make,
Might stop our countryman.

MENENIUS.
No, I’ll not meddle.

SICINIUS.
Pray you, go to him.

MENENIUS.
What should I do?

BRUTUS.
Only make trial what your love can do
For Rome, towards Martius.

MENENIUS.
Well, and say that Martius
Return me, as Cominius is returned, unheard,
What then? But as a discontented friend,
Grief-shot with his unkindness? Say’t be so?

SICINIUS.
Yet your good will
Must have that thanks from Rome after the measure
As you intended well.

MENENIUS.
I’ll undertake’t.
I think he’ll hear me. Yet to bite his lip
And hum at good Cominius much unhearts me.
He was not taken well; he had not dined.
The veins unfilled, our blood is cold, and then
We pout upon the morning, are unapt
To give or to forgive; but when we have stuffed
These pipes and these conveyances of our blood
With wine and feeding, we have suppler souls
Than in our priestlike fasts. Therefore I’ll watch him
Till he be dieted to my request,
And then I’ll set upon him.

BRUTUS.
You know the very road into his kindness
And cannot lose your way.

MENENIUS.
Good faith, I’ll prove him,
Speed how it will. I shall ere long have knowledge
Of my success.

[_Exit._]

COMINIUS.
He’ll never hear him.

SICINIUS.
Not?

COMINIUS.
I tell you, he does sit in gold, his eye
Red as ’twould burn Rome; and his injury
The jailer to his pity. I kneeled before him;
’Twas very faintly he said “Rise”; dismissed me
Thus with his speechless hand. What he would do
He sent in writing after me; what he
Would not, bound with an oath to yield to his
Conditions. So that all hope is vain
Unless his noble mother and his wife,
Who, as I hear, mean to solicit him
For mercy to his country. Therefore let’s hence
And with our fair entreaties haste them on.

[_Exeunt._]

SCENE II. An Advanced post of the Volscian camp before Rome.

Enter Menenius to the Watch, or Guard.

FIRST WATCH.
Stay! Whence are you?

SECOND WATCH.
Stand, and go back.

MENENIUS.
You guard like men; ’tis well. But by your leave,
I am an officer of state and come
To speak with Coriolanus.

FIRST WATCH.
From whence?

MENENIUS.
From Rome.

FIRST WATCH.
You may not pass; you must return. Our general
Will no more hear from thence.

SECOND WATCH.
You’ll see your Rome embraced with fire before
You’ll speak with Coriolanus.

MENENIUS.
Good my friends,
If you have heard your general talk of Rome
And of his friends there, it is lots to blanks
My name hath touched your ears. It is Menenius.

FIRST WATCH.
Be it so; go back. The virtue of your name
Is not here passable.

MENENIUS.
I tell thee, fellow,
Thy general is my lover. I have been
The book of his good acts, whence men have read
His fame unparalleled happily amplified;
For I have ever verified my friends—
Of whom he’s chief—with all the size that verity
Would without lapsing suffer. Nay, sometimes,
Like to a bowl upon a subtle ground,
I have tumbled past the throw, and in his praise
Have almost stamped the leasing. Therefore, fellow,
I must have leave to pass.

FIRST WATCH.
Faith, sir, if you had told as many lies in his behalf as you have
uttered words in your own, you should not pass here, no, though it were
as virtuous to lie as to live chastely. Therefore, go back.

MENENIUS.
Prithee, fellow, remember my name is Menenius, always factionary on the
party of your general.

SECOND WATCH.
Howsoever you have been his liar, as you say you have, I am one that,
telling true under him, must say you cannot pass. Therefore go back.

MENENIUS.
Has he dined, can’st thou tell? For I would not speak with him till
after dinner.

FIRST WATCH.
You are a Roman, are you?

MENENIUS.
I am, as thy general is.

FIRST WATCH.
Then you should hate Rome as he does. Can you, when you have pushed out
your gates the very defender of them, and, in a violent popular
ignorance given your enemy your shield, think to front his revenges
with the easy groans of old women, the virginal palms of your
daughters, or with the palsied intercession of such a decayed dotant as
you seem to be? Can you think to blow out the intended fire your city
is ready to flame in with such weak breath as this? No, you are
deceived. Therefore back to Rome and prepare for your execution. You
are condemned. Our general has sworn you out of reprieve and pardon.

MENENIUS.
Sirrah, if thy captain knew I were here, he would use me with
estimation.

SECOND WATCH.
Come, my captain knows you not.

MENENIUS.
I mean thy general.

FIRST WATCH.
My general cares not for you. Back, I say, go, lest I let forth your
half pint of blood. Back! That’s the utmost of your having. Back!

MENENIUS.
Nay, but fellow, fellow—

Enter Coriolanus with Aufidius.

CORIOLANUS.
What’s the matter?

MENENIUS.
Now, you companion, I’ll say an errand for you. You shall know now that
I am in estimation; you shall perceive that a Jack guardant cannot
office me from my son Coriolanus. Guess but by my entertainment with
him if thou stand’st not i’ th’ state of hanging or of some death more
long in spectatorship and crueller in suffering; behold now presently,
and swoon for what’s to come upon thee. [_to Coriolanus_.] The glorious
gods sit in hourly synod about thy particular prosperity and love thee
no worse than thy old father Menenius does! O my son, my son! Thou art
preparing fire for us; look thee, here’s water to quench it. I was
hardly moved to come to thee; but being assured none but myself could
move thee, I have been blown out of your gates with sighs, and conjure
thee to pardon Rome and thy petitionary countrymen. The good gods
assuage thy wrath and turn the dregs of it upon this varlet here, this,
who, like a block, hath denied my access to thee.

CORIOLANUS.
Away!

MENENIUS.
How? Away?

CORIOLANUS.
Wife, mother, child, I know not. My affairs
Are servanted to others. Though I owe
My revenge properly, my remission lies
In Volscian breasts. That we have been familiar,
Ingrate forgetfulness shall poison rather
Than pity note how much. Therefore begone.
Mine ears against your suits are stronger than
Your gates against my force. Yet, for I loved thee,
Take this along; I writ it for thy sake,

[_He gives Menenius a paper._]

And would have sent it. Another word, Menenius,
I will not hear thee speak.—This man, Aufidius,
Was my beloved in Rome; yet thou behold’st.

AUFIDIUS.
You keep a constant temper.

[_They exit._]

[_The Guard and Menenius remain._]

FIRST WATCH.
Now, sir, is your name Menenius?

SECOND WATCH.
’Tis a spell, you see, of much power. You know the way home again.

FIRST WATCH.
Do you hear how we are shent for keeping your Greatness back?

SECOND WATCH.
What cause do you think I have to swoon?

MENENIUS.
I neither care for th’ world nor your general. For such things as you,
I can scarce think there’s any, you’re so slight. He that hath a will
to die by himself fears it not from another. Let your general do his
worst. For you, be that you are, long; and your misery increase with
your age! I say to you, as I was said to, away!

[_Exit._]

FIRST WATCH.
A noble fellow, I warrant him.

SECOND WATCH.
The worthy fellow is our general. He is the rock, the oak not to be
wind-shaken.

[_Exeunt._]

SCENE III. The tent of Coriolanus

Enter Coriolanus and Aufidius.

CORIOLANUS.
We will before the walls of Rome tomorrow
Set down our host. My partner in this action,
You must report to th’ Volscian lords how plainly
I have borne this business.

AUFIDIUS.
Only their ends
You have respected, stopped your ears against
The general suit of Rome; never admitted
A private whisper, no, not with such friends
That thought them sure of you.

CORIOLANUS.
This last old man,
Whom with cracked heart I have sent to Rome,
Loved me above the measure of a father,
Nay, godded me indeed. Their latest refuge
Was to send him, for whose old love I have—
Though I showed sourly to him—once more offered
The first conditions, which they did refuse
And cannot now accept, to grace him only
That thought he could do more. A very little
I have yielded to. Fresh embassies and suits,
Nor from the state nor private friends, hereafter
Will I lend ear to.

[_Shout within._]

Ha? What shout is this?
Shall I be tempted to infringe my vow
In the same time ’tis made? I will not.

Enter Virgilia, Volumnia, Valeria, young Martius with attendants.

My wife comes foremost, then the honoured mold
Wherein this trunk was framed, and in her hand
The grandchild to her blood. But out, affection!
All bond and privilege of nature, break!
Let it be virtuous to be obstinate.
What is that curtsy worth? Or those doves’ eyes,
Which can make gods forsworn? I melt and am not
Of stronger earth than others. My mother bows,
As if Olympus to a molehill should
In supplication nod; and my young boy
Hath an aspect of intercession which
Great Nature cries “Deny not!” Let the Volsces
Plough Rome and harrow Italy, I’ll never
Be such a gosling to obey instinct, but stand
As if a man were author of himself,
And knew no other kin.

VIRGILIA.
My lord and husband.

CORIOLANUS.
These eyes are not the same I wore in Rome.

VIRGILIA.
The sorrow that delivers us thus changed
Makes you think so.

CORIOLANUS.
Like a dull actor now,
I have forgot my part, and I am out,
Even to a full disgrace. Best of my flesh,
Forgive my tyranny, but do not say
For that, “Forgive our Romans.”

[_They kiss._]

O, a kiss
Long as my exile, sweet as my revenge!
Now, by the jealous queen of heaven, that kiss
I carried from thee, dear, and my true lip
Hath virgined it e’er since. You gods! I prate
And the most noble mother of the world
Leave unsaluted. Sink, my knee, i’ th’ earth;

[_Kneels._]

Of thy deep duty more impression show
Than that of common sons.

VOLUMNIA.
O, stand up blest,

[_He rises_.]

Whilst with no softer cushion than the flint
I kneel before thee and unproperly
Show duty, as mistaken all this while
Between the child and parent.

[_She kneels._]

CORIOLANUS.
What is this?
Your knees to me? To your corrected son?

[_He raises her up._]

Then let the pebbles on the hungry beach
Fillip the stars! Then let the mutinous winds
Strike the proud cedars ’gainst the fiery sun,
Murdering impossibility to make
What cannot be slight work.

VOLUMNIA.
Thou art my warrior;
I holp to frame thee. Do you know this lady?

CORIOLANUS.
The noble sister of Publicola,
The moon of Rome, chaste as the icicle
That’s curdied by the frost from purest snow
And hangs on Dian’s temple!—Dear Valeria.

VOLUMNIA.
This is a poor epitome of yours,
Which by th’ interpretation of full time
May show like all yourself.

CORIOLANUS.
The god of soldiers,
With the consent of supreme Jove, inform
Thy thoughts with nobleness, that thou mayst prove
To shame unvulnerable, and stick i’ th’ wars
Like a great seamark standing every flaw
And saving those that eye thee.

VOLUMNIA.
[_To young Martius_.] Your knee, sirrah.

[_He kneels._]

CORIOLANUS.
That’s my brave boy!

VOLUMNIA.
Even he, your wife, this lady, and myself
Are suitors to you.

[_Young Martius rises._]

CORIOLANUS.
I beseech you, peace;
Or, if you’d ask, remember this before:
The thing I have forsworn to grant may never
Be held by you denials. Do not bid me
Dismiss my soldiers or capitulate
Again with Rome’s mechanics. Tell me not
Wherein I seem unnatural; desire not
T’ allay my rages and revenges with
Your colder reasons.

VOLUMNIA.
O, no more, no more!
You have said you will not grant us anything;
For we have nothing else to ask but that
Which you deny already. Yet we will ask,
That if you fail in our request, the blame
May hang upon your hardness. Therefore hear us.

CORIOLANUS.
Aufidius, and you Volsces, mark, for we’ll
Hear naught from Rome in private. Your request?

VOLUMNIA.
Should we be silent and not speak, our raiment
And state of bodies would bewray what life
We have led since thy exile. Think with thyself
How more unfortunate than all living women
Are we come hither; since that thy sight, which should
Make our eyes flow with joy, hearts dance with comforts,
Constrains them weep and shake with fear and sorrow,
Making the mother, wife, and child to see
The son, the husband, and the father tearing
His country’s bowels out. And to poor we
Thine enmity’s most capital. Thou barr’st us
Our prayers to the gods, which is a comfort
That all but we enjoy. For how can we—
Alas, how can we—for our country pray,
Whereto we are bound, together with thy victory,
Whereto we are bound? Alack, or we must lose
The country, our dear nurse, or else thy person,
Our comfort in the country. We must find
An evident calamity, though we had
Our wish, which side should win, for either thou
Must as a foreign recreant be led
With manacles through our streets, or else
Triumphantly tread on thy country’s ruin
And bear the palm for having bravely shed
Thy wife and children’s blood. For myself, son,
I purpose not to wait on fortune till
These wars determine. If I cannot persuade thee
Rather to show a noble grace to both parts
Than seek the end of one, thou shalt no sooner
March to assault thy country than to tread—
Trust to’t, thou shalt not—on thy mother’s womb
That brought thee to this world.

VIRGILIA.
Ay, and mine,
That brought you forth this boy to keep your name
Living to time.

YOUNG MARTIUS.
He shall not tread on me.
I’ll run away till I am bigger, but then I’ll fight.

CORIOLANUS.
Not of a woman’s tenderness to be
Requires nor child nor woman’s face to see.—
I have sat too long.

[_He rises._]

VOLUMNIA.
Nay, go not from us thus.
If it were so, that our request did tend
To save the Romans, thereby to destroy
The Volsces whom you serve, you might condemn us
As poisonous of your honour. No, our suit
Is that you reconcile them, while the Volsces
May say “This mercy we have showed,” the Romans
“This we received,” and each in either side
Give the all-hail to thee and cry, “Be blessed
For making up this peace!” Thou know’st, great son,
The end of war’s uncertain, but this certain,
That, if thou conquer Rome, the benefit
Which thou shalt thereby reap is such a name
Whose repetition will be dogged with curses,
Whose chronicle thus writ: “The man was noble,
But with his last attempt he wiped it out;
Destroyed his country, and his name remains
To th’ ensuing age abhorred.” Speak to me, son.
Thou hast affected the fine strains of honour
To imitate the graces of the gods,
To tear with thunder the wide cheeks o’ th’ air
And yet to charge thy sulphur with a bolt
That should but rive an oak. Why dost not speak?
Think’st thou it honourable for a noble man
Still to remember wrongs?—Daughter, speak you.
He cares not for your weeping.—Speak thou, boy.
Perhaps thy childishness will move him more
Than can our reasons.—There’s no man in the world
More bound to’s mother, yet here he lets me prate
Like one i’ th’ stocks. Thou hast never in thy life
Showed thy dear mother any courtesy
When she, poor hen, fond of no second brood,
Has clucked thee to the wars and safely home,
Loaden with honour. Say my request’s unjust
And spurn me back; but if it be not so,
Thou art not honest, and the gods will plague thee
That thou restrain’st from me the duty which
To a mother’s part belongs.—He turns away.—
Down, ladies! Let us shame him with our knees.
To his surname Coriolanus ’longs more pride
Than pity to our prayers. Down! An end.

[_They kneel._]

This is the last. So we will home to Rome
And die among our neighbours.—Nay, behold’s.
This boy that cannot tell what he would have,
But kneels and holds up hands for fellowship,
Does reason our petition with more strength
Than thou hast to deny’t.—Come, let us go.

[_They rise._]

This fellow had a Volscian to his mother,
His wife is in Corioles, and his child
Like him by chance.—Yet give us our dispatch.
I am hushed until our city be afire,
And then I’ll speak a little.

[_He holds her by the hand, silent._]

CORIOLANUS.
O mother, mother!
What have you done? Behold, the heavens do ope,
The gods look down, and this unnatural scene
They laugh at. O my mother, mother, O!
You have won a happy victory to Rome,
But, for your son—believe it, O, believe it!—
Most dangerously you have with him prevailed,
If not most mortal to him. But let it come.—
Aufidius, though I cannot make true wars,
I’ll frame convenient peace. Now, good Aufidius,
Were you in my stead, would you have heard
A mother less? Or granted less, Aufidius?

AUFIDIUS.
I was moved withal.

CORIOLANUS.
I dare be sworn you were.
And, sir, it is no little thing to make
Mine eyes to sweat compassion. But, good sir,
What peace you’ll make, advise me. For my part,
I’ll not to Rome, I’ll back with you; and pray you,
Stand to me in this cause.—O mother!—Wife!

[_He speaks with them aside._]

AUFIDIUS.
[_Aside_.] I am glad thou hast set thy mercy and thy honour
At difference in thee. Out of that I’ll work
Myself a former fortune.

CORIOLANUS.
[_To the Women_.] Ay, by and by;
But we’ll drink together, and you shall bear
A better witness back than words, which we,
On like conditions, will have countersealed.
Come, enter with us. Ladies, you deserve
To have a temple built you. All the swords
In Italy, and her confederate arms,
Could not have made this peace.

[_Exeunt._]

SCENE IV. Rome. A public place

Enter Menenius and Sicinius.

MENENIUS.
See you yond coign o’ the Capitol, yond cornerstone?

SICINIUS.
Why, what of that?

MENENIUS.
If it be possible for you to displace it with your little finger, there
is some hope the ladies of Rome, especially his mother, may prevail
with him. But I say there is no hope in’t. Our throats are sentenced
and stay upon execution.

SICINIUS.
Is’t possible that so short a time can alter the condition of a man?

MENENIUS.
There is differency between a grub and a butterfly, yet your butterfly
was a grub. This Martius is grown from man to dragon. He has wings;
he’s more than a creeping thing.

SICINIUS.
He loved his mother dearly.

MENENIUS.
So did he me; and he no more remembers his mother now than an
eight-year-old horse. The tartness of his face sours ripe grapes. When
he walks, he moves like an engine, and the ground shrinks before his
treading. He is able to pierce a corslet with his eye, talks like a
knell, and his hum is a battery. He sits in his state as a thing made
for Alexander. What he bids be done is finished with his bidding. He
wants nothing of a god but eternity and a heaven to throne in.

SICINIUS.
Yes, mercy, if you report him truly.

MENENIUS.
I paint him in the character. Mark what mercy his mother shall bring
from him. There is no more mercy in him than there is milk in a male
tiger. That shall our poor city find, and all this is long of you.

SICINIUS.
The gods be good unto us.

MENENIUS.
No, in such a case the gods will not be good unto us. When we banished
him, we respected not them; and he returning to break our necks, they
respect not us.

Enter a Messenger.

MESSENGER.
Sir, if you’d save your life, fly to your house.
The plebeians have got your fellow tribune
And hale him up and down, all swearing if
The Roman ladies bring not comfort home,
They’ll give him death by inches.

Enter another Messenger.

SICINIUS.
What’s the news?

SECOND MESSENGER.
Good news, good news! The ladies have prevailed.
The Volscians are dislodged and Martius gone.
A merrier day did never yet greet Rome,
No, not th’ expulsion of the Tarquins.

SICINIUS.
Friend,
Art thou certain this is true? Is’t most certain?

SECOND MESSENGER.
As certain as I know the sun is fire.
Where have you lurked that you make doubt of it?
Ne’er through an arch so hurried the blown tide
As the recomforted through th’ gates. Why, hark you!

[_Trumpets, hautboys, drums beat, all together._]

The trumpets, sackbuts, psalteries, and fifes,
Tabors and cymbals, and the shouting Romans
Make the sun dance. Hark you!

[_A shout within._]

MENENIUS.
This is good news.
I will go meet the ladies. This Volumnia
Is worth of consuls, senators, patricians
A city full; of tribunes such as you
A sea and land full. You have prayed well today.
This morning for ten thousand of your throats
I’d not have given a doit. Hark, how they joy!

[_Sound still with the shouts._]

SICINIUS.
First, the gods bless you for your tidings; next, accept my
thankfulness.

SECOND MESSENGER.
Sir, we have all great cause to give great thanks.

SICINIUS.
They are near the city?

MESSENGER.
Almost at point to enter.

SICINIUS.
We’ll meet them, and help the joy.

[_Exeunt._]

SCENE V. Rome. A street near the gate

Enter two Senators, with Ladies (Volumnia, Virgilia, Valeria) passing
over the stage, with other Lords.

SENATOR.
Behold our patroness, the life of Rome!
Call all your tribes together, praise the gods,
And make triumphant fires. Strew flowers before them,
Unshout the noise that banished Martius,
Repeal him with the welcome of his mother.
Cry “Welcome, ladies, welcome!”

ALL.
Welcome, ladies, welcome!

[_A flourish with drums and trumpets._]

[_Exeunt._]

SCENE VI. Antium. A public place

Enter Tullus Aufidius with Attendants.

AUFIDIUS.
Go tell the lords o’ th’ city I am here.
Deliver them this paper.

[_He gives them a paper_.]

Having read it,
Bid them repair to th’ marketplace, where I,
Even in theirs and in the commons’ ears,
Will vouch the truth of it. Him I accuse
The city ports by this hath entered and
Intends t’ appear before the people, hoping
To purge himself with words. Dispatch.

[_Exeunt Attendants._]

Enter three or four Conspirators of Aufidius’s faction.

Most welcome!

FIRST CONSPIRATOR.
How is it with our general?

AUFIDIUS.
Even so
As with a man by his own alms empoisoned
And with his charity slain.

SECOND CONSPIRATOR.
Most noble sir,
If you do hold the same intent wherein
You wished us parties, we’ll deliver you
Of your great danger.

AUFIDIUS.
Sir, I cannot tell.
We must proceed as we do find the people.

THIRD CONSPIRATOR.
The people will remain uncertain whilst
’Twixt you there’s difference, but the fall of either
Makes the survivor heir of all.

AUFIDIUS.
I know it,
And my pretext to strike at him admits
A good construction. I raised him, and I pawned
Mine honour for his truth, who being so heightened,
He watered his new plants with dews of flattery,
Seducing so my friends; and to this end,
He bowed his nature, never known before
But to be rough, unswayable, and free.

THIRD CONSPIRATOR.
Sir, his stoutness
When he did stand for consul, which he lost
By lack of stooping—

AUFIDIUS.
That I would have spoke of.
Being banished for’t, he came unto my hearth,
Presented to my knife his throat. I took him,
Made him joint servant with me, gave him way
In all his own desires; nay, let him choose
Out of my files, his projects to accomplish,
My best and freshest men; served his designments
In mine own person; holp to reap the fame
Which he did end all his; and took some pride
To do myself this wrong; till at the last
I seemed his follower, not partner; and
He waged me with his countenance as if
I had been mercenary.

FIRST CONSPIRATOR.
So he did, my lord.
The army marvelled at it, and, in the last,
When he had carried Rome and that we looked
For no less spoil than glory—

AUFIDIUS.
There was it
For which my sinews shall be stretched upon him.
At a few drops of women’s rheum, which are
As cheap as lies, he sold the blood and labour
Of our great action. Therefore shall he die,
And I’ll renew me in his fall. But, hark!

[_Drums and trumpets sound, with great shouts of the people._]

FIRST CONSPIRATOR.
Your native town you entered like a post
And had no welcomes home, but he returns
Splitting the air with noise.

SECOND CONSPIRATOR.
And patient fools,
Whose children he hath slain, their base throats tear
With giving him glory.

THIRD CONSPIRATOR.
Therefore at your vantage,
Ere he express himself or move the people
With what he would say, let him feel your sword,
Which we will second. When he lies along,
After your way his tale pronounced shall bury
His reasons with his body.

AUFIDIUS.
Say no more.
Here come the lords.

Enter the Lords of the city.

ALL LORDS.
You are most welcome home.

AUFIDIUS.
I have not deserved it.
But, worthy lords, have you with heed perused
What I have written to you?

ALL LORDS.
We have.

FIRST LORD.
And grieve to hear’t.
What faults he made before the last, I think
Might have found easy fines, but there to end
Where he was to begin and give away
The benefit of our levies, answering us
With our own charge, making a treaty where
There was a yielding—this admits no excuse.

Enter Coriolanus marching with Drum and Colours, the Commoners being
with him.

AUFIDIUS.
He approaches. You shall hear him.

CORIOLANUS.
Hail, lords! I am returned your soldier,
No more infected with my country’s love
Than when I parted hence, but still subsisting
Under your great command. You are to know
That prosperously I have attempted, and
With bloody passage led your wars even to
The gates of Rome. Our spoils we have brought home
Doth more than counterpoise a full third part
The charges of the action. We have made peace
With no less honour to the Antiates
Than shame to th’ Romans, and we here deliver,
Subscribed by th’ Consuls and patricians,
Together with the seal o’ th’ Senate, what
We have compounded on.

[_He offers the lords a paper._]

AUFIDIUS.
Read it not, noble lords,
But tell the traitor in the highest degree
He hath abused your powers.

CORIOLANUS.
“Traitor?” How now?

AUFIDIUS.
Ay, traitor, Martius.

CORIOLANUS.
Martius?

AUFIDIUS.
Ay, Martius, Caius Martius. Dost thou think
I’ll grace thee with that robbery, thy stol’n name
Coriolanus, in Corioles?
You lords and heads o’ th’ state, perfidiously
He has betrayed your business and given up
For certain drops of salt your city Rome—
I say your city—to his wife and mother,
Breaking his oath and resolution like
A twist of rotten silk, never admitting
Counsel o’ th’ war, but at his nurse’s tears
He whined and roared away your victory,
That pages blushed at him and men of heart
Looked wond’ring each at other.

CORIOLANUS.
Hear’st thou, Mars?

AUFIDIUS.
Name not the god, thou boy of tears.

CORIOLANUS.
Ha?

AUFIDIUS.
No more.

CORIOLANUS.
Measureless liar, thou hast made my heart
Too great for what contains it. “Boy”? O slave!—
Pardon me, lords, ’tis the first time that ever
I was forced to scold. Your judgments, my grave lords,
Must give this cur the lie; and his own notion—
Who wears my stripes impressed upon him, that
Must bear my beating to his grave—shall join
To thrust the lie unto him.

FIRST LORD.
Peace, both, and hear me speak.

CORIOLANUS.
Cut me to pieces, Volsces. Men and lads,
Stain all your edges on me. “Boy”? False hound!
If you have writ your annals true, ’tis there,
That like an eagle in a dovecote, I
Fluttered your Volscians in Corioles,
Alone I did it. “Boy”!

AUFIDIUS.
Why, noble lords,
Will you be put in mind of his blind fortune,
Which was your shame, by this unholy braggart,
’Fore your own eyes and ears?

ALL CONSPIRATORS.
Let him die for’t.

ALL PEOPLE
Tear him to pieces! Do it presently! He killed my son! My daughter! He
killed my cousin Marcus! He killed my father!

SECOND LORD.
Peace, ho! No outrage! Peace!
The man is noble, and his fame folds in
This orb o’ th’ Earth. His last offences to us
Shall have judicious hearing. Stand, Aufidius,
And trouble not the peace.

CORIOLANUS.
O that I had him,
With six Aufidiuses, or more, his tribe,
To use my lawful sword.

AUFIDIUS.
Insolent villain!

ALL CONSPIRATORS.
Kill, kill, kill, kill, kill him!


[_Draw the Conspirators, and kills Martius, who falls. Aufidius stands
on him._]

LORDS.
Hold, hold, hold, hold!

AUFIDIUS.
My noble masters, hear me speak.

FIRST LORD.
O Tullus!

SECOND LORD.
Thou hast done a deed whereat valour will weep.

THIRD LORD.
Tread not upon him.—Masters, all be quiet.—
Put up your swords.

AUFIDIUS.
My lords, when you shall know—as in this rage,
Provoked by him, you cannot—the great danger
Which this man’s life did owe you, you’ll rejoice
That he is thus cut off. Please it your Honours
To call me to your senate, I’ll deliver
Myself your loyal servant, or endure
Your heaviest censure.

FIRST LORD.
Bear from hence his body,
And mourn you for him. Let him be regarded
As the most noble corse that ever herald
Did follow to his urn.

SECOND LORD.
His own impatience
Takes from Aufidius a great part of blame.
Let’s make the best of it.

AUFIDIUS.
My rage is gone,
And I am struck with sorrow.—Take him up.
Help, three o’ th’ chiefest soldiers; I’ll be one.—
Beat thou the drum that it speak mournfully.—
Trail your steel pikes. Though in this city he
Hath widowed and unchilded many a one,
Which to this hour bewail the injury,
Yet he shall have a noble memory.
Assist.

[_Exeunt, bearing the body of Martius. A dead march sounded._]




CYMBELINE




Contents

ACT I
Scene I. Britain. The garden of Cymbeline’s palace.
Scene II. The same.
Scene III. Britain. A public place.
Scene IV. Britain. Cymbeline’s palace.
Scene V. Rome. Philario’s house.
Scene VI. Britain. Cymbeline’s palace.
Scene VII. Britain. The palace.

ACT II
Scene I. Britain. Before Cymbeline’s palace.
Scene II. Britain. Imogen’s bedchamber in Cymbeline’s palace; a trunk
in one corner.
Scene III. Cymbeline’s palace. An ante-chamber adjoining Imogen’s
apartments.
Scene IV. Rome. Philario’s house.
Scene V. Rome. Another room in Philario’s house.

ACT III
Scene I. Britain. A hall in Cymbeline’s palace.
Scene II. Britain. Another room in Cymbeline’s palace.
Scene III. Wales. A mountainous country with a cave.
Scene IV. Wales, near Milford Haven.
Scene V. Britain. Cymbeline’s palace.
Scene VI. Wales. Before the cave of Belarius.
Scene VII. The same.
Scene VIII. Rome. A public place.

ACT IV
Scene I. Wales. Near the cave of Belarius.
Scene II. Wales. Before the cave of Belarius.
Scene III. Britain. Cymbeline’s palace.
Scene IV. Wales. Before the cave of Belarius.

ACT V
Scene I. Britain. The Roman camp.
Scene II. Britain. A field of battle between the British and Roman
camps.
Scene III. Another part of the field.
Scene IV. Britain. A prison.
Scene V. Britain. Cymbeline’s tent.




Dramatis Personæ

CYMBELINE, King of Britain
CLOTEN, son to the Queen by a former husband
POSTHUMUS LEONATUS, a gentleman, husband to Imogen
BELARIUS, a banished lord, disguised under the name of Morgan
GUIDERIUS and ARVIRAGUS, sons to Cymbeline, disguised under the names
of POLYDORE and CADWAL, supposed sons to Belarius
PHILARIO, Italian, friend to Posthumus
IACHIMO, Italian, friend to Philario
CAIUS LUCIUS, General of the Roman forces
PISANIO, servant to Posthumus
CORNELIUS, a physician
A SOOTHSAYER
A ROMAN CAPTAIN
TWO BRITISH CAPTAINS
A FRENCH GENTLEMAN, friend to Philario
TWO LORDS of Cymbeline’s court
TWO GENTLEMEN of the same
TWO GAOLERS

QUEEN, wife to Cymbeline
IMOGEN, daughter to Cymbeline by a former queen
HELEN, a lady attending on Imogen

APPARITIONS

Lords, Ladies, Roman Senators, Tribunes, a Dutch Gentleman, a Spanish
Gentleman, Musicians, Officers, Captains, Soldiers, Messengers, and
Attendants

SCENE: Britain; Italy.




ACT I

SCENE I. Britain. The garden of Cymbeline’s palace.


 Enter two Gentlemen.

FIRST GENTLEMAN.
You do not meet a man but frowns; our bloods
No more obey the heavens than our courtiers
Still seem as does the King’s.

SECOND GENTLEMAN.
But what’s the matter?

FIRST GENTLEMAN.
His daughter, and the heir of’s kingdom, whom
He purpos’d to his wife’s sole son—a widow
That late he married—hath referr’d herself
Unto a poor but worthy gentleman. She’s wedded;
Her husband banish’d; she imprison’d. All
Is outward sorrow, though I think the King
Be touch’d at very heart.

SECOND GENTLEMAN.
None but the King?

FIRST GENTLEMAN.
He that hath lost her too. So is the Queen,
That most desir’d the match. But not a courtier,
Although they wear their faces to the bent
Of the King’s looks, hath a heart that is not
Glad at the thing they scowl at.

SECOND GENTLEMAN.
And why so?

FIRST GENTLEMAN.
He that hath miss’d the Princess is a thing
Too bad for bad report; and he that hath her—
I mean that married her, alack, good man!
And therefore banish’d—is a creature such
As, to seek through the regions of the earth
For one his like, there would be something failing
In him that should compare. I do not think
So fair an outward and such stuff within
Endows a man but he.

SECOND GENTLEMAN.
You speak him far.

FIRST GENTLEMAN.
I do extend him, sir, within himself;
Crush him together rather than unfold
His measure duly.

SECOND GENTLEMAN.
What’s his name and birth?

FIRST GENTLEMAN.
I cannot delve him to the root; his father
Was call’d Sicilius, who did join his honour
Against the Romans with Cassibelan,
But had his titles by Tenantius, whom
He serv’d with glory and admir’d success,
So gain’d the sur-addition Leonatus;
And had, besides this gentleman in question,
Two other sons, who, in the wars o’ th’ time,
Died with their swords in hand; for which their father,
Then old and fond of issue, took such sorrow
That he quit being; and his gentle lady,
Big of this gentleman, our theme, deceas’d
As he was born. The King he takes the babe
To his protection, calls him Posthumus Leonatus,
Breeds him and makes him of his bed-chamber,
Puts to him all the learnings that his time
Could make him the receiver of; which he took,
As we do air, fast as ’twas minist’red,
And in’s spring became a harvest, liv’d in court—
Which rare it is to do—most prais’d, most lov’d,
A sample to the youngest; to th’ more mature
A glass that feated them; and to the graver
A child that guided dotards. To his mistress,
For whom he now is banish’d, her own price
Proclaims how she esteem’d him and his virtue;
By her election may be truly read
What kind of man he is.

SECOND GENTLEMAN.
I honour him
Even out of your report. But pray you tell me,
Is she sole child to th’ King?

FIRST GENTLEMAN.
His only child.
He had two sons—if this be worth your hearing,
Mark it—the eldest of them at three years old,
I’ th’ swathing clothes the other, from their nursery
Were stol’n; and to this hour no guess in knowledge
Which way they went.

SECOND GENTLEMAN.
How long is this ago?

FIRST GENTLEMAN.
Some twenty years.

SECOND GENTLEMAN.
That a king’s children should be so convey’d,
So slackly guarded, and the search so slow
That could not trace them!

FIRST GENTLEMAN.
Howsoe’er ’tis strange,
Or that the negligence may well be laugh’d at,
Yet is it true, sir.

SECOND GENTLEMAN.
I do well believe you.

FIRST GENTLEMAN.
We must forbear; here comes the gentleman,
The Queen, and Princess.

 [_Exeunt._]

SCENE II. The same.

 Enter Queen, Posthumus and Imogen.

QUEEN.
No, be assur’d you shall not find me, daughter,
After the slander of most stepmothers,
Evil-ey’d unto you. You’re my prisoner, but
Your gaoler shall deliver you the keys
That lock up your restraint. For you, Posthumus,
So soon as I can win th’ offended King,
I will be known your advocate. Marry, yet
The fire of rage is in him, and ’twere good
You lean’d unto his sentence with what patience
Your wisdom may inform you.

POSTHUMUS.
Please your Highness,
I will from hence today.

QUEEN.
You know the peril.
I’ll fetch a turn about the garden, pitying
The pangs of barr’d affections, though the King
Hath charg’d you should not speak together.

 [_Exit._]

IMOGEN.
O dissembling courtesy! How fine this tyrant
Can tickle where she wounds! My dearest husband,
I something fear my father’s wrath, but nothing
(Always reserv’d my holy duty) what
His rage can do on me. You must be gone;
And I shall here abide the hourly shot
Of angry eyes, not comforted to live
But that there is this jewel in the world
That I may see again.

POSTHUMUS.
My queen! my mistress!
O lady, weep no more, lest I give cause
To be suspected of more tenderness
Than doth become a man. I will remain
The loyal’st husband that did e’er plight troth;
My residence in Rome at one Philario’s,
Who to my father was a friend, to me
Known but by letter; thither write, my queen,
And with mine eyes I’ll drink the words you send,
Though ink be made of gall.

 Enter Queen.

QUEEN.
Be brief, I pray you.
If the King come, I shall incur I know not
How much of his displeasure. [_Aside._] Yet I’ll move him
To walk this way. I never do him wrong
But he does buy my injuries, to be friends;
Pays dear for my offences.

 [_Exit._]

POSTHUMUS.
Should we be taking leave
As long a term as yet we have to live,
The loathness to depart would grow. Adieu!

IMOGEN.
Nay, stay a little.
Were you but riding forth to air yourself,
Such parting were too petty. Look here, love:
This diamond was my mother’s; take it, heart;
But keep it till you woo another wife,
When Imogen is dead.

POSTHUMUS.
How, how? Another?
You gentle gods, give me but this I have,
And sear up my embracements from a next
With bonds of death! Remain, remain thou here

 [_Puts on the ring._]

While sense can keep it on. And, sweetest, fairest,
As I my poor self did exchange for you,
To your so infinite loss, so in our trifles
I still win of you. For my sake wear this;
It is a manacle of love; I’ll place it
Upon this fairest prisoner.

 [_Puts a bracelet on her arm._]

IMOGEN.
O the gods!
When shall we see again?

 Enter Cymbeline and Lords.

POSTHUMUS.
Alack, the King!

CYMBELINE.
Thou basest thing, avoid; hence from my sight
If after this command thou fraught the court
With thy unworthiness, thou diest. Away!
Thou’rt poison to my blood.

POSTHUMUS.
The gods protect you,
And bless the good remainders of the court!
I am gone.

 [_Exit._]

IMOGEN.
There cannot be a pinch in death
More sharp than this is.

CYMBELINE.
O disloyal thing,
That shouldst repair my youth, thou heap’st
A year’s age on me!

IMOGEN.
I beseech you, sir,
Harm not yourself with your vexation.
I am senseless of your wrath; a touch more rare
Subdues all pangs, all fears.

CYMBELINE.
Past grace? obedience?

IMOGEN.
Past hope, and in despair; that way past grace.

CYMBELINE.
That mightst have had the sole son of my queen!

IMOGEN.
O blessed that I might not! I chose an eagle,
And did avoid a puttock.

CYMBELINE.
Thou took’st a beggar, wouldst have made my throne
A seat for baseness.

IMOGEN.
No; I rather added
A lustre to it.

CYMBELINE.
O thou vile one!

IMOGEN.
Sir,
It is your fault that I have lov’d Posthumus.
You bred him as my playfellow, and he is
A man worth any woman; overbuys me
Almost the sum he pays.

CYMBELINE.
What, art thou mad?

IMOGEN.
Almost, sir. Heaven restore me! Would I were
A neat-herd’s daughter, and my Leonatus
Our neighbour shepherd’s son!

 Enter Queen.

CYMBELINE.
Thou foolish thing!
[_To the Queen._] They were again together. You have done
Not after our command. Away with her,
And pen her up.

QUEEN.
Beseech your patience. Peace,
Dear lady daughter, peace!—Sweet sovereign,
Leave us to ourselves, and make yourself some comfort
Out of your best advice.

CYMBELINE.
Nay, let her languish
A drop of blood a day and, being aged,
Die of this folly.

 [_Exit with Lords._]

 Enter Pisanio.

QUEEN.
Fie! you must give way.
Here is your servant. How now, sir! What news?

PISANIO.
My lord your son drew on my master.

QUEEN.
Ha!
No harm, I trust, is done?

PISANIO.
There might have been,
But that my master rather play’d than fought,
And had no help of anger; they were parted
By gentlemen at hand.

QUEEN.
I am very glad on’t.

IMOGEN.
Your son’s my father’s friend; he takes his part
To draw upon an exile! O brave sir!
I would they were in Afric both together;
Myself by with a needle, that I might prick
The goer-back. Why came you from your master?

PISANIO.
On his command. He would not suffer me
To bring him to the haven; left these notes
Of what commands I should be subject to,
When’t pleas’d you to employ me.

QUEEN.
This hath been
Your faithful servant. I dare lay mine honour
He will remain so.

PISANIO.
I humbly thank your Highness.

QUEEN.
Pray walk awhile.

IMOGEN.
About some half-hour hence,
Pray you speak with me.
You shall at least go see my lord aboard.
For this time leave me.

 [_Exeunt._]

SCENE III. Britain. A public place.

 Enter Cloten and two Lords.

FIRST LORD.
Sir, I would advise you to shift a shirt; the violence of action hath
made you reek as a sacrifice. Where air comes out, air comes in;
there’s none abroad so wholesome as that you vent.

CLOTEN.
If my shirt were bloody, then to shift it. Have I hurt him?

SECOND LORD.
[_Aside._] No, faith; not so much as his patience.

FIRST LORD.
Hurt him! His body’s a passable carcass if he be not hurt. It is a
throughfare for steel if it be not hurt.

SECOND LORD.
[_Aside._] His steel was in debt; it went o’ th’ backside the town.

CLOTEN.
The villain would not stand me.

SECOND LORD.
[_Aside._] No; but he fled forward still, toward your face.

FIRST LORD.
Stand you? You have land enough of your own; but he added to your
having, gave you some ground.

SECOND LORD.
[_Aside._] As many inches as you have oceans.
Puppies!

CLOTEN.
I would they had not come between us.

SECOND LORD.
[_Aside._] So would I, till you had measur’d how long a fool you were
upon the ground.

CLOTEN.
And that she should love this fellow, and refuse me!

SECOND LORD.
[_Aside._] If it be a sin to make a true election, she is damn’d.

FIRST LORD.
Sir, as I told you always, her beauty and her brain go not together;
she’s a good sign, but I have seen small reflection of her wit.

SECOND LORD.
[_Aside._] She shines not upon fools, lest the reflection should hurt
her.

CLOTEN.
Come, I’ll to my chamber. Would there had been some hurt done!

SECOND LORD.
[_Aside._] I wish not so; unless it had been the fall of an ass, which
is no great hurt.

CLOTEN.
You’ll go with us?

FIRST LORD.
I’ll attend your lordship.

CLOTEN.
Nay, come, let’s go together.

SECOND LORD.
Well, my lord.

 [_Exeunt._]

SCENE IV. Britain. Cymbeline’s palace.

 Enter Imogen and Pisanio.

IMOGEN.
I would thou grew’st unto the shores o’ th’ haven,
And questioned’st every sail; if he should write,
And I not have it, ’twere a paper lost,
As offer’d mercy is. What was the last
That he spake to thee?

PISANIO.
It was: his queen, his queen!

IMOGEN.
Then wav’d his handkerchief?

PISANIO.
And kiss’d it, madam.

IMOGEN.
Senseless linen, happier therein than I!
And that was all?

PISANIO.
No, madam; for so long
As he could make me with his eye, or ear
Distinguish him from others, he did keep
The deck, with glove, or hat, or handkerchief,
Still waving, as the fits and stirs of’s mind
Could best express how slow his soul sail’d on,
How swift his ship.

IMOGEN.
Thou shouldst have made him
As little as a crow, or less, ere left
To after-eye him.

PISANIO.
Madam, so I did.

IMOGEN.
I would have broke mine eyestrings, crack’d them but
To look upon him, till the diminution
Of space had pointed him sharp as my needle;
Nay, followed him till he had melted from
The smallness of a gnat to air, and then
Have turn’d mine eye and wept. But, good Pisanio,
When shall we hear from him?

PISANIO.
Be assur’d, madam,
With his next vantage.

IMOGEN.
I did not take my leave of him, but had
Most pretty things to say. Ere I could tell him
How I would think on him at certain hours
Such thoughts and such; or I could make him swear
The shes of Italy should not betray
Mine interest and his honour; or have charg’d him,
At the sixth hour of morn, at noon, at midnight,
T’ encounter me with orisons, for then
I am in heaven for him; or ere I could
Give him that parting kiss which I had set
Betwixt two charming words, comes in my father,
And like the tyrannous breathing of the north
Shakes all our buds from growing.

 Enter a Lady.

LADY.
The Queen, madam,
Desires your Highness’ company.

IMOGEN.
Those things I bid you do, get them dispatch’d.
I will attend the Queen.

PISANIO.
Madam, I shall.

 [_Exeunt._]

SCENE V. Rome. Philario’s house.

 Enter Philario, Iachimo, a Frenchman, a Dutchman and a Spaniard.

IACHIMO.
Believe it, sir, I have seen him in Britain. He was then of a crescent
note, expected to prove so worthy as since he hath been allowed the
name of. But I could then have look’d on him without the help of
admiration, though the catalogue of his endowments had been tabled by
his side, and I to peruse him by items.

PHILARIO.
You speak of him when he was less furnish’d than now he is with that
which makes him both without and within.

FRENCHMAN.
I have seen him in France; we had very many there could behold the sun
with as firm eyes as he.

IACHIMO.
This matter of marrying his king’s daughter, wherein he must be weighed
rather by her value than his own, words him, I doubt not, a great deal
from the matter.

FRENCHMAN.
And then his banishment.

IACHIMO.
Ay, and the approbation of those that weep this lamentable divorce
under her colours are wonderfully to extend him, be it but to fortify
her judgement, which else an easy battery might lay flat, for taking a
beggar, without less quality. But how comes it he is to sojourn with
you? How creeps acquaintance?

PHILARIO.
His father and I were soldiers together, to whom I have been often
bound for no less than my life.

 Enter Posthumus.

Here comes the Briton. Let him be so entertained amongst you as suits
with gentlemen of your knowing to a stranger of his quality. I beseech
you all be better known to this gentleman, whom I commend to you as a
noble friend of mine. How worthy he is I will leave to appear
hereafter, rather than story him in his own hearing.

FRENCHMAN.
Sir, we have known together in Orleans.

POSTHUMUS.
Since when I have been debtor to you for courtesies, which I will be
ever to pay and yet pay still.

FRENCHMAN.
Sir, you o’errate my poor kindness. I was glad I did atone my
countryman and you; it had been pity you should have been put together
with so mortal a purpose as then each bore, upon importance of so
slight and trivial a nature.

POSTHUMUS.
By your pardon, sir. I was then a young traveller; rather shunn’d to go
even with what I heard than in my every action to be guided by others’
experiences; but upon my mended judgement (if I offend not to say it is
mended) my quarrel was not altogether slight.

FRENCHMAN.
Faith, yes, to be put to the arbitrement of swords, and by such two
that would by all likelihood have confounded one the other or have
fall’n both.

IACHIMO.
Can we, with manners, ask what was the difference?

FRENCHMAN.
Safely, I think. ’Twas a contention in public, which may, without
contradiction, suffer the report. It was much like an argument that
fell out last night, where each of us fell in praise of our country
mistresses; this gentleman at that time vouching (and upon warrant of
bloody affirmation) his to be more fair, virtuous, wise, chaste,
constant, qualified, and less attemptable, than any the rarest of our
ladies in France.

IACHIMO.
That lady is not now living, or this gentleman’s opinion, by this, worn
out.

POSTHUMUS.
She holds her virtue still, and I my mind.

IACHIMO.
You must not so far prefer her ’fore ours of Italy.

POSTHUMUS.
Being so far provok’d as I was in France, I would abate her nothing,
though I profess myself her adorer, not her friend.

IACHIMO.
As fair and as good—a kind of hand-in-hand comparison—had been
something too fair and too good for any lady in Britain. If she went
before others I have seen as that diamond of yours outlustres many I
have beheld, I could not but believe she excelled many; but I have not
seen the most precious diamond that is, nor you the lady.

POSTHUMUS.
I prais’d her as I rated her. So do I my stone.

IACHIMO.
What do you esteem it at?

POSTHUMUS.
More than the world enjoys.

IACHIMO.
Either your unparagon’d mistress is dead, or she’s outpriz’d by a
trifle.

POSTHUMUS.
You are mistaken: the one may be sold or given, if there were wealth
enough for the purchase or merit for the gift; the other is not a thing
for sale, and only the gift of the gods.

IACHIMO.
Which the gods have given you?

POSTHUMUS.
Which by their graces I will keep.

IACHIMO.
You may wear her in title yours; but you know strange fowl light upon
neighbouring ponds. Your ring may be stol’n too. So your brace of
unprizable estimations, the one is but frail and the other casual; a
cunning thief, or a that-way-accomplish’d courtier, would hazard the
winning both of first and last.

POSTHUMUS.
Your Italy contains none so accomplish’d a courtier to convince the
honour of my mistress, if in the holding or loss of that you term her
frail. I do nothing doubt you have store of thieves; notwithstanding, I
fear not my ring.

PHILARIO.
Let us leave here, gentlemen.

POSTHUMUS.
Sir, with all my heart. This worthy signior, I thank him, makes no
stranger of me; we are familiar at first.

IACHIMO.
With five times so much conversation I should get ground of your fair
mistress; make her go back even to the yielding, had I admittance and
opportunity to friend.

POSTHUMUS.
No, no.

IACHIMO.
I dare thereupon pawn the moiety of my estate to your ring, which, in
my opinion, o’ervalues it something. But I make my wager rather against
your confidence than her reputation; and, to bar your offence herein
too, I durst attempt it against any lady in the world.

POSTHUMUS.
You are a great deal abus’d in too bold a persuasion, and I doubt not
you sustain what y’are worthy of by your attempt.

IACHIMO.
What’s that?

POSTHUMUS.
A repulse; though your attempt, as you call it, deserve more; a
punishment too.

PHILARIO.
Gentlemen, enough of this. It came in too suddenly; let it die as it
was born, and I pray you be better acquainted.

IACHIMO.
Would I had put my estate and my neighbour’s on th’ approbation of what
I have spoke!

POSTHUMUS.
What lady would you choose to assail?

IACHIMO.
Yours, whom in constancy you think stands so safe. I will lay you ten
thousand ducats to your ring that, commend me to the court where your
lady is, with no more advantage than the opportunity of a second
conference, and I will bring from thence that honour of hers which you
imagine so reserv’d.

POSTHUMUS.
I will wage against your gold, gold to it. My ring I hold dear as my
finger; ’tis part of it.

IACHIMO.
You are a friend, and therein the wiser. If you buy ladies’ flesh at a
million a dram, you cannot preserve it from tainting. But I see you
have some religion in you, that you fear.

POSTHUMUS.
This is but a custom in your tongue; you bear a graver purpose, I hope.

IACHIMO.
I am the master of my speeches, and would undergo what’s spoken, I
swear.

POSTHUMUS.
Will you? I shall but lend my diamond till your return. Let there be
covenants drawn between’s. My mistress exceeds in goodness the hugeness
of your unworthy thinking. I dare you to this match: here’s my ring.

PHILARIO.
I will have it no lay.

IACHIMO.
By the gods, it is one. If I bring you no sufficient testimony that I
have enjoy’d the dearest bodily part of your mistress, my ten thousand
ducats are yours; so is your diamond too. If I come off, and leave her
in such honour as you have trust in, she your jewel, this your jewel,
and my gold are yours: provided I have your commendation for my more
free entertainment.

POSTHUMUS.
I embrace these conditions; let us have articles betwixt us. Only, thus
far you shall answer: if you make your voyage upon her, and give me
directly to understand you have prevail’d, I am no further your enemy;
she is not worth our debate; if she remain unseduc’d, you not making it
appear otherwise, for your ill opinion and th’ assault you have made to
her chastity you shall answer me with your sword.

IACHIMO.
Your hand, a covenant! We will have these things set down by lawful
counsel, and straight away for Britain, lest the bargain should catch
cold and starve. I will fetch my gold and have our two wagers recorded.

POSTHUMUS.
Agreed.

 [_Exeunt Posthumus and Iachimo._]

FRENCHMAN.
Will this hold, think you?

PHILARIO.
Signior Iachimo will not from it. Pray let us follow ’em.

 [_Exeunt._]

SCENE VI. Britain. Cymbeline’s palace.

 Enter Queen, Ladies and Cornelius.

QUEEN.
Whiles yet the dew’s on ground, gather those flowers;
Make haste; who has the note of them?

LADY.
I, madam.

QUEEN.
Dispatch.

 [_Exeunt Ladies._]

Now, Master Doctor, have you brought those drugs?

CORNELIUS.
Pleaseth your Highness, ay. Here they are, madam.

 [_Presenting a box._]

But I beseech your Grace, without offence,
(My conscience bids me ask) wherefore you have
Commanded of me these most poisonous compounds
Which are the movers of a languishing death,
But, though slow, deadly?

QUEEN.
I wonder, Doctor,
Thou ask’st me such a question. Have I not been
Thy pupil long? Hast thou not learn’d me how
To make perfumes? distil? preserve? yea, so
That our great king himself doth woo me oft
For my confections? Having thus far proceeded
(Unless thou think’st me devilish) is’t not meet
That I did amplify my judgement in
Other conclusions? I will try the forces
Of these thy compounds on such creatures as
We count not worth the hanging (but none human)
To try the vigour of them, and apply
Allayments to their act, and by them gather
Their several virtues and effects.

CORNELIUS.
Your Highness
Shall from this practice but make hard your heart;
Besides, the seeing these effects will be
Both noisome and infectious.

QUEEN.
O, content thee.

 Enter Pisanio.

[_Aside._] Here comes a flattering rascal; upon him
Will I first work. He’s for his master,
An enemy to my son. How now, Pisanio!
Doctor, your service for this time is ended;
Take your own way.

CORNELIUS.
[_Aside._] I do suspect you, madam;
But you shall do no harm.

QUEEN.
[_To Pisanio._] Hark thee, a word.

CORNELIUS.
[_Aside._] I do not like her. She doth think she has
Strange ling’ring poisons. I do know her spirit,
And will not trust one of her malice with
A drug of such damn’d nature. Those she has
Will stupefy and dull the sense awhile,
Which first perchance she’ll prove on cats and dogs,
Then afterward up higher; but there is
No danger in what show of death it makes,
More than the locking up the spirits a time,
To be more fresh, reviving. She is fool’d
With a most false effect; and I the truer
So to be false with her.

QUEEN.
No further service, Doctor,
Until I send for thee.

CORNELIUS.
I humbly take my leave.

 [_Exit._]

QUEEN.
Weeps she still, say’st thou? Dost thou think in time
She will not quench, and let instructions enter
Where folly now possesses? Do thou work.
When thou shalt bring me word she loves my son,
I’ll tell thee on the instant thou art then
As great as is thy master; greater, for
His fortunes all lie speechless, and his name
Is at last gasp. Return he cannot, nor
Continue where he is. To shift his being
Is to exchange one misery with another,
And every day that comes comes to decay
A day’s work in him. What shalt thou expect
To be depender on a thing that leans,
Who cannot be new built, nor has no friends
So much as but to prop him?

 [_The Queen drops the box. Pisanio takes it up._]

Thou tak’st up
Thou know’st not what; but take it for thy labour.
It is a thing I made, which hath the King
Five times redeem’d from death. I do not know
What is more cordial. Nay, I prithee take it;
It is an earnest of a further good
That I mean to thee. Tell thy mistress how
The case stands with her; do’t as from thyself.
Think what a chance thou changest on; but think
Thou hast thy mistress still; to boot, my son,
Who shall take notice of thee. I’ll move the King
To any shape of thy preferment, such
As thou’lt desire; and then myself, I chiefly,
That set thee on to this desert, am bound
To load thy merit richly. Call my women.
Think on my words.

 [_Exit Pisanio._]

A sly and constant knave,
Not to be shak’d; the agent for his master,
And the remembrancer of her to hold
The hand-fast to her lord. I have given him that
Which, if he take, shall quite unpeople her
Of liegers for her sweet; and which she after,
Except she bend her humour, shall be assur’d
To taste of too.

 Enter Pisanio and Ladies.

So, so. Well done, well done.
The violets, cowslips, and the primroses,
Bear to my closet. Fare thee well, Pisanio;
Think on my words.

 [_Exeunt Queen and Ladies._]

PISANIO.
And shall do.
But when to my good lord I prove untrue
I’ll choke myself: there’s all I’ll do for you.

 [_Exit._]

SCENE VII. Britain. The palace.

 Enter Imogen alone.

IMOGEN.
A father cruel and a step-dame false;
A foolish suitor to a wedded lady
That hath her husband banish’d. O, that husband!
My supreme crown of grief! and those repeated
Vexations of it! Had I been thief-stol’n,
As my two brothers, happy! but most miserable
Is the desire that’s glorious. Blessed be those,
How mean soe’er, that have their honest wills,
Which seasons comfort. Who may this be? Fie!

 Enter Pisanio and Iachimo.

PISANIO.
Madam, a noble gentleman of Rome
Comes from my lord with letters.

IACHIMO.
Change you, madam?
The worthy Leonatus is in safety,
And greets your Highness dearly.

 [_Presents a letter._]

IMOGEN.
Thanks, good sir.
You’re kindly welcome.

IACHIMO.
[_Aside._] All of her that is out of door most rich!
If she be furnish’d with a mind so rare,
She is alone th’ Arabian bird, and I
Have lost the wager. Boldness be my friend!
Arm me, audacity, from head to foot!
Or, like the Parthian, I shall flying fight;
Rather, directly fly.

IMOGEN.
[_Reads._] _He is one of the noblest note, to whose kindnesses I am
most infinitely tied. Reflect upon him accordingly, as you value your
trust.
                    LEONATUS._

So far I read aloud;
But even the very middle of my heart
Is warm’d by th’ rest and takes it thankfully.
You are as welcome, worthy sir, as I
Have words to bid you; and shall find it so
In all that I can do.

IACHIMO.
Thanks, fairest lady.
What, are men mad? Hath nature given them eyes
To see this vaulted arch and the rich crop
Of sea and land, which can distinguish ’twixt
The fiery orbs above and the twinn’d stones
Upon the number’d beach, and can we not
Partition make with spectacles so precious
’Twixt fair and foul?

IMOGEN.
What makes your admiration?

IACHIMO.
It cannot be i’ th’ eye, for apes and monkeys,
’Twixt two such shes, would chatter this way and
Contemn with mows the other; nor i’ th’ judgement,
For idiots in this case of favour would
Be wisely definite; nor i’ th’ appetite;
Sluttery, to such neat excellence oppos’d,
Should make desire vomit emptiness,
Not so allur’d to feed.

IMOGEN.
What is the matter, trow?

IACHIMO.
The cloyed will—
That satiate yet unsatisfied desire, that tub
Both fill’d and running—ravening first the lamb,
Longs after for the garbage.

IMOGEN.
What, dear sir,
Thus raps you? Are you well?

IACHIMO.
Thanks, madam; well. Beseech you, sir,
Desire my man’s abode where I did leave him.
He’s strange and peevish.

PISANIO.
I was going, sir,
To give him welcome.

 [_Exit._]

IMOGEN.
Continues well my lord? His health beseech you?

IACHIMO.
Well, madam.

IMOGEN.
Is he dispos’d to mirth? I hope he is.

IACHIMO.
Exceeding pleasant; none a stranger there
So merry and so gamesome. He is call’d
The Briton reveller.

IMOGEN.
When he was here
He did incline to sadness, and oft-times
Not knowing why.

IACHIMO.
I never saw him sad.
There is a Frenchman his companion, one
An eminent monsieur that, it seems, much loves
A Gallian girl at home. He furnaces
The thick sighs from him; whiles the jolly Briton
(Your lord, I mean) laughs from’s free lungs, cries “O,
Can my sides hold, to think that man, who knows
By history, report, or his own proof,
What woman is, yea, what she cannot choose
But must be, will’s free hours languish for
Assured bondage?”

IMOGEN.
Will my lord say so?

IACHIMO.
Ay, madam, with his eyes in flood with laughter.
It is a recreation to be by
And hear him mock the Frenchman. But heavens know
Some men are much to blame.

IMOGEN.
Not he, I hope.

IACHIMO.
Not he; but yet heaven’s bounty towards him might
Be us’d more thankfully. In himself, ’tis much;
In you, which I account his, beyond all talents.
Whilst I am bound to wonder, I am bound
To pity too.

IMOGEN.
What do you pity, sir?

IACHIMO.
Two creatures heartily.

IMOGEN.
Am I one, sir?
You look on me: what wreck discern you in me
Deserves your pity?

IACHIMO.
Lamentable! What,
To hide me from the radiant sun and solace
I’ th’ dungeon by a snuff?

IMOGEN.
I pray you, sir,
Deliver with more openness your answers
To my demands. Why do you pity me?

IACHIMO.
That others do,
I was about to say, enjoy your—But
It is an office of the gods to venge it,
Not mine to speak on’t.

IMOGEN.
You do seem to know
Something of me, or what concerns me; pray you,
Since doubting things go ill often hurts more
Than to be sure they do; for certainties
Either are past remedies, or, timely knowing,
The remedy then born—discover to me
What both you spur and stop.

IACHIMO.
Had I this cheek
To bathe my lips upon; this hand, whose touch,
Whose every touch, would force the feeler’s soul
To th’ oath of loyalty; this object, which
Takes prisoner the wild motion of mine eye,
Fixing it only here; should I, damn’d then,
Slaver with lips as common as the stairs
That mount the Capitol; join gripes with hands
Made hard with hourly falsehood (falsehood as
With labour): then by-peeping in an eye
Base and illustrious as the smoky light
That’s fed with stinking tallow: it were fit
That all the plagues of hell should at one time
Encounter such revolt.

IMOGEN.
My lord, I fear,
Has forgot Britain.

IACHIMO.
And himself. Not I
Inclin’d to this intelligence pronounce
The beggary of his change; but ’tis your graces
That from my mutest conscience to my tongue
Charms this report out.

IMOGEN.
Let me hear no more.

IACHIMO.
O dearest soul, your cause doth strike my heart
With pity that doth make me sick! A lady
So fair, and fasten’d to an empery,
Would make the great’st king double, to be partner’d
With tomboys hir’d with that self exhibition
Which your own coffers yield! with diseas’d ventures
That play with all infirmities for gold
Which rottenness can lend nature! Such boil’d stuff
As well might poison poison! Be reveng’d;
Or she that bore you was no queen, and you
Recoil from your great stock.

IMOGEN.
Reveng’d?
How should I be reveng’d? If this be true,
(As I have such a heart that both mine ears
Must not in haste abuse) if it be true,
How should I be reveng’d?

IACHIMO.
Should he make me
Live like Diana’s priest betwixt cold sheets,
Whiles he is vaulting variable ramps,
In your despite, upon your purse? Revenge it.
I dedicate myself to your sweet pleasure,
More noble than that runagate to your bed,
And will continue fast to your affection,
Still close as sure.

IMOGEN.
What ho, Pisanio!

IACHIMO.
Let me my service tender on your lips.

IMOGEN.
Away! I do condemn mine ears that have
So long attended thee. If thou wert honourable,
Thou wouldst have told this tale for virtue, not
For such an end thou seek’st, as base as strange.
Thou wrong’st a gentleman who is as far
From thy report as thou from honour; and
Solicits here a lady that disdains
Thee and the devil alike. What ho, Pisanio!
The King my father shall be made acquainted
Of thy assault. If he shall think it fit
A saucy stranger in his court to mart
As in a Romish stew, and to expound
His beastly mind to us, he hath a court
He little cares for, and a daughter who
He not respects at all. What ho, Pisanio!

IACHIMO.
O happy Leonatus! I may say
The credit that thy lady hath of thee
Deserves thy trust, and thy most perfect goodness
Her assur’d credit. Blessed live you long,
A lady to the worthiest sir that ever
Country call’d his! and you his mistress, only
For the most worthiest fit! Give me your pardon.
I have spoke this to know if your affiance
Were deeply rooted, and shall make your lord
That which he is new o’er; and he is one
The truest manner’d, such a holy witch
That he enchants societies into him,
Half all men’s hearts are his.

IMOGEN.
You make amends.

IACHIMO.
He sits ’mongst men like a descended god:
He hath a kind of honour sets him off
More than a mortal seeming. Be not angry,
Most mighty Princess, that I have adventur’d
To try your taking of a false report, which hath
Honour’d with confirmation your great judgement
In the election of a sir so rare,
Which you know cannot err. The love I bear him
Made me to fan you thus; but the gods made you,
Unlike all others, chaffless. Pray your pardon.

IMOGEN.
All’s well, sir; take my pow’r i’ th’ court for yours.

IACHIMO.
My humble thanks. I had almost forgot
T’ entreat your Grace but in a small request,
And yet of moment too, for it concerns
Your lord; myself and other noble friends
Are partners in the business.

IMOGEN.
Pray what is’t?

IACHIMO.
Some dozen Romans of us, and your lord
(The best feather of our wing) have mingled sums
To buy a present for the Emperor;
Which I, the factor for the rest, have done
In France. ’Tis plate of rare device, and jewels
Of rich and exquisite form, their values great;
And I am something curious, being strange,
To have them in safe stowage. May it please you
To take them in protection?

IMOGEN.
Willingly;
And pawn mine honour for their safety. Since
My lord hath interest in them, I will keep them
In my bedchamber.

IACHIMO.
They are in a trunk,
Attended by my men. I will make bold
To send them to you only for this night;
I must aboard tomorrow.

IMOGEN.
O, no, no.

IACHIMO.
Yes, I beseech; or I shall short my word
By length’ning my return. From Gallia
I cross’d the seas on purpose and on promise
To see your Grace.

IMOGEN.
I thank you for your pains.
But not away tomorrow!

IACHIMO.
O, I must, madam.
Therefore I shall beseech you, if you please
To greet your lord with writing, do’t tonight.
I have outstood my time, which is material
To th’ tender of our present.

IMOGEN.
I will write.
Send your trunk to me; it shall safe be kept
And truly yielded you. You’re very welcome.

 [_Exeunt._]




ACT II

SCENE I. Britain. Before Cymbeline’s palace.


 Enter Cloten and the two Lords.

CLOTEN.
Was there ever man had such luck! When I kiss’d the jack, upon an
upcast to be hit away! I had a hundred pound on’t; and then a whoreson
jackanapes must take me up for swearing, as if I borrowed mine oaths of
him, and might not spend them at my pleasure.

FIRST LORD.
What got he by that? You have broke his pate with your bowl.

SECOND LORD.
[_Aside._] If his wit had been like him that broke it, it would have
run all out.

CLOTEN.
When a gentleman is dispos’d to swear, it is not for any standers-by to
curtail his oaths. Ha?

SECOND LORD.
No, my lord; [_Aside._] nor crop the ears of them.

CLOTEN.
Whoreson dog! I gave him satisfaction. Would he had been one of my
rank!

SECOND LORD.
[_Aside._] To have smell’d like a fool.

CLOTEN.
I am not vex’d more at anything in th’ earth. A pox on’t! I had rather
not be so noble as I am; they dare not fight with me, because of the
Queen my mother. Every jackslave hath his bellyful of fighting, and I
must go up and down like a cock that nobody can match.

SECOND LORD.
[_Aside._] You are cock and capon too; and you crow, cock, with your
comb on.

CLOTEN.
Sayest thou?

SECOND LORD.
It is not fit your lordship should undertake every companion that you
give offence to.

CLOTEN.
No, I know that; but it is fit I should commit offence to my inferiors.

SECOND LORD.
Ay, it is fit for your lordship only.

CLOTEN.
Why, so I say.

FIRST LORD.
Did you hear of a stranger that’s come to court tonight?

CLOTEN.
A stranger, and I not known on’t?

SECOND LORD.
[_Aside._] He’s a strange fellow himself, and knows it not.

FIRST LORD.
There’s an Italian come, and, ’tis thought, one of Leonatus’ friends.

CLOTEN.
Leonatus? A banish’d rascal; and he’s another, whatsoever he be. Who
told you of this stranger?

FIRST LORD.
One of your lordship’s pages.

CLOTEN.
Is it fit I went to look upon him? Is there no derogation in’t?

SECOND LORD.
You cannot derogate, my lord.

CLOTEN.
Not easily, I think.

SECOND LORD.
[_Aside._] You are a fool granted; therefore your issues, being
foolish, do not derogate.

CLOTEN.
Come, I’ll go see this Italian. What I have lost today at bowls I’ll
win tonight of him. Come, go.

SECOND LORD.
I’ll attend your lordship.

 [_Exeunt Cloten and First Lord._]

That such a crafty devil as is his mother
Should yield the world this ass! A woman that
Bears all down with her brain; and this her son
Cannot take two from twenty, for his heart,
And leave eighteen. Alas, poor princess,
Thou divine Imogen, what thou endur’st,
Betwixt a father by thy step-dame govern’d,
A mother hourly coining plots, a wooer
More hateful than the foul expulsion is
Of thy dear husband, than that horrid act
Of the divorce he’d make! The heavens hold firm
The walls of thy dear honour, keep unshak’d
That temple, thy fair mind, that thou mayst stand
T’ enjoy thy banish’d lord and this great land!

 [_Exit._]

SCENE II. Britain. Imogen’s bedchamber in Cymbeline’s palace; a trunk
in one corner.

 Enter Imogen in her bed, and a Lady attending.

IMOGEN.
Who’s there? My woman Helen?

LADY.
Please you, madam.

IMOGEN.
What hour is it?

LADY.
Almost midnight, madam.

IMOGEN.
I have read three hours then. Mine eyes are weak;
Fold down the leaf where I have left. To bed.
Take not away the taper, leave it burning;
And if thou canst awake by four o’ th’ clock,
I prithee call me. Sleep hath seiz’d me wholly.

 [_Exit Lady._]

To your protection I commend me, gods.
From fairies and the tempters of the night
Guard me, beseech ye!

 [_Sleeps. Iachimo comes from the trunk._]

IACHIMO.
The crickets sing, and man’s o’er-labour’d sense
Repairs itself by rest. Our Tarquin thus
Did softly press the rushes ere he waken’d
The chastity he wounded. Cytherea,
How bravely thou becom’st thy bed! fresh lily,
And whiter than the sheets! That I might touch!
But kiss; one kiss! Rubies unparagon’d,
How dearly they do’t! ’Tis her breathing that
Perfumes the chamber thus. The flame o’ th’ taper
Bows toward her and would under-peep her lids
To see th’ enclosed lights, now canopied
Under these windows white and azure, lac’d
With blue of heaven’s own tinct. But my design
To note the chamber. I will write all down:
Such and such pictures; there the window; such
Th’ adornment of her bed; the arras, figures,
Why, such and such; and the contents o’ th’ story.
Ah, but some natural notes about her body
Above ten thousand meaner movables
Would testify, t’ enrich mine inventory.
O sleep, thou ape of death, lie dull upon her!
And be her sense but as a monument,
Thus in a chapel lying! Come off, come off;

 [_Taking off her bracelet._]

As slippery as the Gordian knot was hard!
’Tis mine; and this will witness outwardly,
As strongly as the conscience does within,
To th’ madding of her lord. On her left breast
A mole cinque-spotted, like the crimson drops
I’ th’ bottom of a cowslip. Here’s a voucher
Stronger than ever law could make; this secret
Will force him think I have pick’d the lock and ta’en
The treasure of her honour. No more. To what end?
Why should I write this down that’s riveted,
Screw’d to my memory? She hath been reading late
The tale of Tereus; here the leaf’s turn’d down
Where Philomel gave up. I have enough.
To th’ trunk again, and shut the spring of it.
Swift, swift, you dragons of the night, that dawning
May bare the raven’s eye! I lodge in fear;
Though this a heavenly angel, hell is here.

 [_Clock strikes._]

One, two, three. Time, time!

 [_Exit into the trunk._]

SCENE III. Cymbeline’s palace. An ante-chamber adjoining Imogen’s
apartments.

 Enter Cloten and Lords.

FIRST LORD.
Your lordship is the most patient man in loss, the most coldest that
ever turn’d up ace.

CLOTEN.
It would make any man cold to lose.

FIRST LORD.
But not every man patient after the noble temper of your lordship. You
are most hot and furious when you win.

CLOTEN.
Winning will put any man into courage. If I could get this foolish
Imogen, I should have gold enough. It’s almost morning, is’t not?

FIRST LORD.
Day, my lord.

CLOTEN.
I would this music would come. I am advised to give her music a
mornings; they say it will penetrate.

 Enter Musicians.

Come on, tune. If you can penetrate her with your fingering, so. We’ll
try with tongue too. If none will do, let her remain; but I’ll never
give o’er. First, a very excellent good-conceited thing; after, a
wonderful sweet air, with admirable rich words to it, and then let her
consider.

SONG


    Hark, hark! the lark at heaven’s gate sings,
        And Phœbus ’gins arise,
    His steeds to water at those springs
        On chalic’d flow’rs that lies;
    And winking Mary-buds begin
        To ope their golden eyes.
    With everything that pretty is,
        My lady sweet, arise;
            Arise, arise!

CLOTEN.
So, get you gone. If this penetrate, I will consider your music the
better; if it do not, it is a vice in her ears which horsehairs and
calves’ guts, nor the voice of unpaved eunuch to boot, can never amend.

 [_Exeunt Musicians._]

 Enter Cymbeline and Queen.

SECOND LORD.
Here comes the King.

CLOTEN.
I am glad I was up so late, for that’s the reason I was up so early. He
cannot choose but take this service I have done fatherly.—Good morrow
to your Majesty and to my gracious mother.

CYMBELINE.
Attend you here the door of our stern daughter?
Will she not forth?

CLOTEN.
I have assail’d her with musics, but she vouchsafes no notice.

CYMBELINE.
The exile of her minion is too new;
She hath not yet forgot him; some more time
Must wear the print of his remembrance on’t,
And then she’s yours.

QUEEN.
You are most bound to th’ King,
Who lets go by no vantages that may
Prefer you to his daughter. Frame yourself
To orderly solicits, and be friended
With aptness of the season; make denials
Increase your services; so seem as if
You were inspir’d to do those duties which
You tender to her; that you in all obey her,
Save when command to your dismission tends,
And therein you are senseless.

CLOTEN.
Senseless? Not so.

 Enter a Messenger.

MESSENGER.
So like you, sir, ambassadors from Rome;
The one is Caius Lucius.

CYMBELINE.
A worthy fellow,
Albeit he comes on angry purpose now;
But that’s no fault of his. We must receive him
According to the honour of his sender;
And towards himself, his goodness forespent on us,
We must extend our notice. Our dear son,
When you have given good morning to your mistress,
Attend the Queen and us; we shall have need
T’ employ you towards this Roman. Come, our queen.

 [_Exeunt all but Cloten._]

CLOTEN.
If she be up, I’ll speak with her; if not,
Let her lie still and dream. By your leave, ho!

 [_Knocks._]

I know her women are about her; what
If I do line one of their hands? ’Tis gold
Which buys admittance (oft it doth) yea, and makes
Diana’s rangers false themselves, yield up
Their deer to th’ stand o’ th’ stealer; and ’tis gold
Which makes the true man kill’d and saves the thief;
Nay, sometime hangs both thief and true man. What
Can it not do and undo? I will make
One of her women lawyer to me, for
I yet not understand the case myself.
By your leave.

 [_Knocks._]

 Enter a Lady.

LADY.
Who’s there that knocks?

CLOTEN.
A gentleman.

LADY.
No more?

CLOTEN.
Yes, and a gentlewoman’s son.

LADY.
That’s more
Than some whose tailors are as dear as yours
Can justly boast of. What’s your lordship’s pleasure?

CLOTEN.
Your lady’s person; is she ready?

LADY.
Ay,
To keep her chamber.

CLOTEN.
There is gold for you; sell me your good report.

LADY.
How? My good name? or to report of you
What I shall think is good? The Princess!

 Enter Imogen.

CLOTEN.
Good morrow, fairest sister. Your sweet hand.

 [_Exit Lady._]

IMOGEN.
Good morrow, sir. You lay out too much pains
For purchasing but trouble. The thanks I give
Is telling you that I am poor of thanks,
And scarce can spare them.

CLOTEN.
Still I swear I love you.

IMOGEN.
If you but said so, ’twere as deep with me.
If you swear still, your recompense is still
That I regard it not.

CLOTEN.
This is no answer.

IMOGEN.
But that you shall not say I yield, being silent,
I would not speak. I pray you spare me. Faith,
I shall unfold equal discourtesy
To your best kindness; one of your great knowing
Should learn, being taught, forbearance.

CLOTEN.
To leave you in your madness ’twere my sin;
I will not.

IMOGEN.
Fools are not mad folks.

CLOTEN.
Do you call me fool?

IMOGEN.
As I am mad, I do;
If you’ll be patient, I’ll no more be mad;
That cures us both. I am much sorry, sir,
You put me to forget a lady’s manners
By being so verbal; and learn now, for all,
That I, which know my heart, do here pronounce,
By th’ very truth of it, I care not for you,
And am so near the lack of charity
To accuse myself I hate you; which I had rather
You felt than make’t my boast.

CLOTEN.
You sin against
Obedience, which you owe your father. For
The contract you pretend with that base wretch,
One bred of alms and foster’d with cold dishes,
With scraps o’ th’ court, it is no contract, none.
And though it be allowed in meaner parties
(Yet who than he more mean?) to knit their souls
(On whom there is no more dependency
But brats and beggary) in self-figur’d knot,
Yet you are curb’d from that enlargement by
The consequence o’ th’ crown, and must not foil
The precious note of it with a base slave,
A hilding for a livery, a squire’s cloth,
A pantler; not so eminent!

IMOGEN.
Profane fellow!
Wert thou the son of Jupiter, and no more
But what thou art besides, thou wert too base
To be his groom. Thou wert dignified enough,
Even to the point of envy, if ’twere made
Comparative for your virtues to be styl’d
The under-hangman of his kingdom, and hated
For being preferr’d so well.

CLOTEN.
The south fog rot him!

IMOGEN.
He never can meet more mischance than come
To be but nam’d of thee. His mean’st garment
That ever hath but clipp’d his body, is dearer
In my respect, than all the hairs above thee,
Were they all made such men. How now, Pisanio!

 Enter Pisanio.

CLOTEN.
‘His garment’! Now the devil—

IMOGEN.
To Dorothy my woman hie thee presently.

CLOTEN.
‘His garment’!

IMOGEN.
I am sprited with a fool;
Frighted, and ang’red worse. Go bid my woman
Search for a jewel that too casually
Hath left mine arm. It was thy master’s; shrew me,
If I would lose it for a revenue
Of any king’s in Europe! I do think
I saw’t this morning; confident I am
Last night ’twas on mine arm; I kiss’d it.
I hope it be not gone to tell my lord
That I kiss aught but he.

PISANIO.
’Twill not be lost.

IMOGEN.
I hope so. Go and search.

 [_Exit Pisanio._]

CLOTEN.
You have abus’d me.
‘His meanest garment’!

IMOGEN.
Ay, I said so, sir.
If you will make ’t an action, call witness to ’t.

CLOTEN.
I will inform your father.

IMOGEN.
Your mother too.
She’s my good lady and will conceive, I hope,
But the worst of me. So I leave you, sir,
To th’ worst of discontent.

 [_Exit._]

CLOTEN.
I’ll be reveng’d.
‘His mean’st garment’! Well.

 [_Exit._]

SCENE IV. Rome. Philario’s house.

 Enter Posthumus and Philario.

POSTHUMUS.
Fear it not, sir; I would I were so sure
To win the King as I am bold her honour
Will remain hers.

PHILARIO.
What means do you make to him?

POSTHUMUS.
Not any; but abide the change of time,
Quake in the present winter’s state, and wish
That warmer days would come. In these fear’d hopes
I barely gratify your love; they failing,
I must die much your debtor.

PHILARIO.
Your very goodness and your company
O’erpays all I can do. By this your king
Hath heard of great Augustus. Caius Lucius
Will do’s commission throughly; and I think
He’ll grant the tribute, send th’ arrearages,
Or look upon our Romans, whose remembrance
Is yet fresh in their grief.

POSTHUMUS.
I do believe
Statist though I am none, nor like to be,
That this will prove a war; and you shall hear
The legions now in Gallia sooner landed
In our not-fearing Britain than have tidings
Of any penny tribute paid. Our countrymen
Are men more order’d than when Julius Cæsar
Smil’d at their lack of skill, but found their courage
Worthy his frowning at. Their discipline,
Now mingled with their courages, will make known
To their approvers they are people such
That mend upon the world.

 Enter Iachimo.

PHILARIO.
See! Iachimo!

POSTHUMUS.
The swiftest harts have posted you by land,
And winds of all the corners kiss’d your sails,
To make your vessel nimble.

PHILARIO.
Welcome, sir.

POSTHUMUS.
I hope the briefness of your answer made
The speediness of your return.

IACHIMO.
Your lady
Is one of the fairest that I have look’d upon.

POSTHUMUS.
And therewithal the best; or let her beauty
Look through a casement to allure false hearts,
And be false with them.

IACHIMO.
Here are letters for you.

POSTHUMUS.
Their tenour good, I trust.

IACHIMO.
’Tis very like.

PHILARIO.
Was Caius Lucius in the Britain court
When you were there?

IACHIMO.
He was expected then,
But not approach’d.

POSTHUMUS.
All is well yet.
Sparkles this stone as it was wont, or is’t not
Too dull for your good wearing?

IACHIMO.
If I have lost it,
I should have lost the worth of it in gold.
I’ll make a journey twice as far t’ enjoy
A second night of such sweet shortness which
Was mine in Britain; for the ring is won.

POSTHUMUS.
The stone’s too hard to come by.

IACHIMO.
Not a whit,
Your lady being so easy.

POSTHUMUS.
Make not, sir,
Your loss your sport. I hope you know that we
Must not continue friends.

IACHIMO.
Good sir, we must,
If you keep covenant. Had I not brought
The knowledge of your mistress home, I grant
We were to question farther; but I now
Profess myself the winner of her honour,
Together with your ring; and not the wronger
Of her or you, having proceeded but
By both your wills.

POSTHUMUS.
If you can make’t apparent
That you have tasted her in bed, my hand
And ring is yours. If not, the foul opinion
You had of her pure honour gains or loses
Your sword or mine, or masterless leaves both
To who shall find them.

IACHIMO.
Sir, my circumstances,
Being so near the truth as I will make them,
Must first induce you to believe; whose strength
I will confirm with oath; which I doubt not
You’ll give me leave to spare when you shall find
You need it not.

POSTHUMUS.
Proceed.

IACHIMO.
First, her bedchamber,
(Where I confess I slept not, but profess
Had that was well worth watching) it was hang’d
With tapestry of silk and silver; the story,
Proud Cleopatra when she met her Roman
And Cydnus swell’d above the banks, or for
The press of boats or pride. A piece of work
So bravely done, so rich, that it did strive
In workmanship and value; which I wonder’d
Could be so rarely and exactly wrought,
Since the true life on’t was—

POSTHUMUS.
This is true;
And this you might have heard of here, by me
Or by some other.

IACHIMO.
More particulars
Must justify my knowledge.

POSTHUMUS.
So they must,
Or do your honour injury.

IACHIMO.
The chimney
Is south the chamber, and the chimneypiece
Chaste Dian bathing. Never saw I figures
So likely to report themselves. The cutter
Was as another nature, dumb; outwent her,
Motion and breath left out.

POSTHUMUS.
This is a thing
Which you might from relation likewise reap,
Being, as it is, much spoke of.

IACHIMO.
The roof o’ th’ chamber
With golden cherubins is fretted; her andirons
(I had forgot them) were two winking Cupids
Of silver, each on one foot standing, nicely
Depending on their brands.

POSTHUMUS.
This is her honour!
Let it be granted you have seen all this, and praise
Be given to your remembrance; the description
Of what is in her chamber nothing saves
The wager you have laid.

IACHIMO.
Then, if you can, [_Shows the bracelet_]
Be pale. I beg but leave to air this jewel. See!
And now ’tis up again. It must be married
To that your diamond; I’ll keep them.

POSTHUMUS.
Jove!
Once more let me behold it. Is it that
Which I left with her?

IACHIMO.
Sir (I thank her) that.
She stripp’d it from her arm; I see her yet;
Her pretty action did outsell her gift,
And yet enrich’d it too. She gave it me, and said
She priz’d it once.

POSTHUMUS.
May be she pluck’d it off
To send it me.

IACHIMO.
She writes so to you, doth she?

POSTHUMUS.
O, no, no, no! ’tis true. Here, take this too;

 [_Gives the ring._]

It is a basilisk unto mine eye,
Kills me to look on’t. Let there be no honour
Where there is beauty; truth where semblance; love
Where there’s another man. The vows of women
Of no more bondage be to where they are made
Than they are to their virtues, which is nothing.
O, above measure false!

PHILARIO.
Have patience, sir,
And take your ring again; ’tis not yet won.
It may be probable she lost it, or
Who knows if one her women, being corrupted
Hath stol’n it from her?

POSTHUMUS.
Very true;
And so I hope he came by’t. Back my ring.
Render to me some corporal sign about her,
More evident than this; for this was stol’n.

IACHIMO.
By Jupiter, I had it from her arm!

POSTHUMUS.
Hark you, he swears; by Jupiter he swears.
’Tis true, nay, keep the ring, ’tis true. I am sure
She would not lose it. Her attendants are
All sworn and honourable:—they induc’d to steal it!
And by a stranger! No, he hath enjoy’d her.
The cognizance of her incontinency
Is this: she hath bought the name of whore thus dearly.
There, take thy hire; and all the fiends of hell
Divide themselves between you!


PHILARIO.
Sir, be patient;
This is not strong enough to be believ’d
Of one persuaded well of.

POSTHUMUS.
Never talk on’t;
She hath been colted by him.

IACHIMO.
If you seek
For further satisfying, under her breast
(Worthy the pressing) lies a mole, right proud
Of that most delicate lodging. By my life,
I kiss’d it; and it gave me present hunger
To feed again, though full. You do remember
This stain upon her?

POSTHUMUS.
Ay, and it doth confirm
Another stain, as big as hell can hold,
Were there no more but it.

IACHIMO.
Will you hear more?

POSTHUMUS.
Spare your arithmetic; never count the turns.
Once, and a million!

IACHIMO.
I’ll be sworn—

POSTHUMUS.
No swearing.
If you will swear you have not done’t, you lie;
And I will kill thee if thou dost deny
Thou’st made me cuckold.

IACHIMO.
I’ll deny nothing.

POSTHUMUS.
O that I had her here to tear her limb-meal!
I will go there and do’t, i’ th’ court, before
Her father. I’ll do something—

 [_Exit._]

PHILARIO.
Quite besides
The government of patience! You have won.
Let’s follow him and pervert the present wrath
He hath against himself.

IACHIMO.
With all my heart.

 [_Exeunt._]

SCENE V. Rome. Another room in Philario’s house.

 Enter Posthumus.

POSTHUMUS.
Is there no way for men to be, but women
Must be half-workers? We are all bastards,
And that most venerable man which I
Did call my father was I know not where
When I was stamp’d. Some coiner with his tools
Made me a counterfeit; yet my mother seem’d
The Dian of that time. So doth my wife
The nonpareil of this. O, vengeance, vengeance!
Me of my lawful pleasure she restrain’d,
And pray’d me oft forbearance; did it with
A pudency so rosy, the sweet view on’t
Might well have warm’d old Saturn; that I thought her
As chaste as unsunn’d snow. O, all the devils!
This yellow Iachimo in an hour, was’t not?
Or less; at first? Perchance he spoke not, but,
Like a full-acorn’d boar, a German one,
Cried “O!” and mounted; found no opposition
But what he look’d for should oppose and she
Should from encounter guard. Could I find out
The woman’s part in me! For there’s no motion
That tends to vice in man but I affirm
It is the woman’s part. Be it lying, note it,
The woman’s; flattering, hers; deceiving, hers;
Lust and rank thoughts, hers, hers; revenges, hers;
Ambitions, covetings, change of prides, disdain,
Nice longing, slanders, mutability,
All faults that man may name, nay, that hell knows,
Why, hers, in part or all; but rather all;
For even to vice
They are not constant, but are changing still
One vice but of a minute old for one
Not half so old as that. I’ll write against them,
Detest them, curse them. Yet ’tis greater skill
In a true hate to pray they have their will:
The very devils cannot plague them better.

 [_Exit._]




ACT III

SCENE I. Britain. A hall in Cymbeline’s palace.


 Enter in state Cymbeline, Queen, Cloten and Lords at one door, and at
 another Caius Lucius and Attendants.

CYMBELINE.
Now say, what would Augustus Cæsar with us?

LUCIUS.
When Julius Cæsar, (whose remembrance yet
Lives in men’s eyes, and will to ears and tongues
Be theme and hearing ever) was in this Britain,
And conquer’d it, Cassibelan, thine uncle,
Famous in Cæsar’s praises no whit less
Than in his feats deserving it, for him
And his succession granted Rome a tribute,
Yearly three thousand pounds, which by thee lately
Is left untender’d.

QUEEN.
And, to kill the marvel,
Shall be so ever.

CLOTEN.
There be many Cæsars ere such another Julius. Britain is a world by
itself, and we will nothing pay for wearing our own noses.

QUEEN.
That opportunity,
Which then they had to take from’s, to resume
We have again. Remember, sir, my liege,
The kings your ancestors, together with
The natural bravery of your isle, which stands
As Neptune’s park, ribb’d and pal’d in
With rocks unscaleable and roaring waters,
With sands that will not bear your enemies’ boats
But suck them up to th’ top-mast. A kind of conquest
Cæsar made here, but made not here his brag
Of ‘Came, and saw, and overcame.’ With shame
(The first that ever touch’d him) he was carried
From off our coast, twice beaten; and his shipping
(Poor ignorant baubles!) on our terrible seas,
Like egg-shells mov’d upon their surges, crack’d
As easily ’gainst our rocks; for joy whereof
The fam’d Cassibelan, who was once at point
(O, giglot fortune!) to master Cæsar’s sword,
Made Lud’s Town with rejoicing fires bright
And Britons strut with courage.

CLOTEN.
Come, there’s no more tribute to be paid. Our kingdom is stronger than
it was at that time; and, as I said, there is no moe such Cæsars. Other
of them may have crook’d noses; but to owe such straight arms, none.

CYMBELINE.
Son, let your mother end.

CLOTEN.
We have yet many among us can gripe as hard as Cassibelan. I do not say
I am one; but I have a hand. Why tribute? Why should we pay tribute? If
Cæsar can hide the sun from us with a blanket, or put the moon in his
pocket, we will pay him tribute for light; else, sir, no more tribute,
pray you now.

CYMBELINE.
You must know,
Till the injurious Romans did extort
This tribute from us, we were free. Cæsar’s ambition,
Which swell’d so much that it did almost stretch
The sides o’ th’ world, against all colour here
Did put the yoke upon’s; which to shake off
Becomes a warlike people, whom we reckon
Ourselves to be.

CLOTEN.
We do.

CYMBELINE.
Say then to Cæsar,
Our ancestor was that Mulmutius which
Ordain’d our laws, whose use the sword of Cæsar
Hath too much mangled; whose repair and franchise
Shall, by the power we hold, be our good deed,
Though Rome be therefore angry. Mulmutius made our laws,
Who was the first of Britain which did put
His brows within a golden crown, and call’d
Himself a king.

LUCIUS.
I am sorry, Cymbeline,
That I am to pronounce Augustus Cæsar
(Cæsar, that hath moe kings his servants than
Thyself domestic officers) thine enemy.
Receive it from me, then: war and confusion
In Cæsar’s name pronounce I ’gainst thee; look
For fury not to be resisted. Thus defied,
I thank thee for myself.

CYMBELINE.
Thou art welcome, Caius.
Thy Cæsar knighted me; my youth I spent
Much under him; of him I gather’d honour,
Which he to seek of me again, perforce,
Behoves me keep at utterance. I am perfect
That the Pannonians and Dalmatians for
Their liberties are now in arms, a precedent
Which not to read would show the Britons cold;
So Cæsar shall not find them.

LUCIUS.
Let proof speak.

CLOTEN.
His majesty bids you welcome. Make pastime with us a day or two, or
longer. If you seek us afterwards in other terms, you shall find us in
our salt-water girdle. If you beat us out of it, it is yours; if you
fall in the adventure, our crows shall fare the better for you; and
there’s an end.

LUCIUS.
So, sir.

CYMBELINE.
I know your master’s pleasure, and he mine;
All the remain is, welcome.

 [_Exeunt._]

SCENE II. Britain. Another room in Cymbeline’s palace.

 Enter Pisanio reading of a letter.

PISANIO.
How? of adultery? Wherefore write you not
What monsters her accuse? Leonatus!
O master, what a strange infection
Is fall’n into thy ear! What false Italian
(As poisonous-tongu’d as handed) hath prevail’d
On thy too ready hearing? Disloyal? No.
She’s punish’d for her truth, and undergoes,
More goddess-like than wife-like, such assaults
As would take in some virtue. O my master,
Thy mind to her is now as low as were
Thy fortunes. How? that I should murder her?
Upon the love, and truth, and vows, which I
Have made to thy command? I, her? Her blood?
If it be so to do good service, never
Let me be counted serviceable. How look I
That I should seem to lack humanity
So much as this fact comes to?

 [_Reads._]

‘Do’t. The letter
That I have sent her, by her own command
Shall give thee opportunity.’ O damn’d paper,
Black as the ink that’s on thee! Senseless bauble,
Art thou a fedary for this act, and look’st
So virgin-like without? Lo, here she comes.

 Enter Imogen.

I am ignorant in what I am commanded.

IMOGEN.
How now, Pisanio?

PISANIO.
Madam, here is a letter from my lord.

IMOGEN.
Who? thy lord? That is my lord, Leonatus?
O, learn’d indeed were that astronomer
That knew the stars as I his characters;
He’d lay the future open. You good gods,
Let what is here contain’d relish of love,
Of my lord’s health, of his content; yet not
That we two are asunder; let that grieve him!
Some griefs are med’cinable; that is one of them,
For it doth physic love: of his content,
All but in that. Good wax, thy leave. Blest be
You bees that make these locks of counsel! Lovers
And men in dangerous bonds pray not alike;
Though forfeiters you cast in prison, yet
You clasp young Cupid’s tables. Good news, gods!

 [_Reads._]

_Justice and your father’s wrath, should he take me in his dominion,
could not be so cruel to me as you, O the dearest of creatures, would
even renew me with your eyes. Take notice that I am in Cambria, at
Milford Haven. What your own love will out of this advise you, follow.
So he wishes you all happiness that remains loyal to his vow, and your
increasing in love.
                    LEONATUS POSTHUMUS._

O for a horse with wings! Hear’st thou, Pisanio?
He is at Milford Haven. Read, and tell me
How far ’tis thither. If one of mean affairs
May plod it in a week, why may not I
Glide thither in a day? Then, true Pisanio,
Who long’st like me to see thy lord, who long’st
(O, let me ’bate!) but not like me, yet long’st,
But in a fainter kind. O, not like me,
For mine’s beyond beyond: say, and speak thick,
(Love’s counsellor should fill the bores of hearing
To th’ smothering of the sense) how far it is
To this same blessed Milford. And by th’ way
Tell me how Wales was made so happy as
T’ inherit such a haven. But first of all,
How we may steal from hence; and for the gap
That we shall make in time from our hence-going
And our return, to excuse. But first, how get hence.
Why should excuse be born or ere begot?
We’ll talk of that hereafter. Prithee speak,
How many score of miles may we well rid
’Twixt hour and hour?

PISANIO.
One score ’twixt sun and sun,
Madam, ’s enough for you, and too much too.

IMOGEN.
Why, one that rode to’s execution, man,
Could never go so slow. I have heard of riding wagers
Where horses have been nimbler than the sands
That run i’ th’ clock’s behalf. But this is fool’ry.
Go bid my woman feign a sickness; say
She’ll home to her father; and provide me presently
A riding suit, no costlier than would fit
A franklin’s huswife.

PISANIO.
Madam, you’re best consider.

IMOGEN.
I see before me, man. Nor here, nor here,
Nor what ensues, but have a fog in them
That I cannot look through. Away, I prithee;
Do as I bid thee. There’s no more to say.
Accessible is none but Milford way.

 [_Exeunt._]

SCENE III. Wales. A mountainous country with a cave.

 Enter from the cave Belarius, Guiderius and Arviragus.

BELARIUS.
A goodly day not to keep house with such
Whose roof’s as low as ours! Stoop, boys; this gate
Instructs you how t’ adore the heavens, and bows you
To a morning’s holy office. The gates of monarchs
Are arch’d so high that giants may jet through
And keep their impious turbans on without
Good morrow to the sun. Hail, thou fair heaven!
We house i’ th’ rock, yet use thee not so hardly
As prouder livers do.

GUIDERIUS.
Hail, heaven!

ARVIRAGUS.
Hail, heaven!

BELARIUS.
Now for our mountain sport. Up to yond hill,
Your legs are young; I’ll tread these flats. Consider,
When you above perceive me like a crow,
That it is place which lessens and sets off;
And you may then revolve what tales I have told you
Of courts, of princes, of the tricks in war.
This service is not service so being done,
But being so allow’d. To apprehend thus
Draws us a profit from all things we see,
And often to our comfort shall we find
The sharded beetle in a safer hold
Than is the full-wing’d eagle. O, this life
Is nobler than attending for a check,
Richer than doing nothing for a robe,
Prouder than rustling in unpaid-for silk:
Such gain the cap of him that makes him fine,
Yet keeps his book uncross’d. No life to ours!

GUIDERIUS.
Out of your proof you speak. We, poor unfledg’d,
Have never wing’d from view o’ th’ nest, nor know not
What air’s from home. Haply this life is best,
If quiet life be best; sweeter to you
That have a sharper known; well corresponding
With your stiff age. But unto us it is
A cell of ignorance, travelling abed,
A prison for a debtor that not dares
To stride a limit.

ARVIRAGUS.
What should we speak of
When we are old as you? When we shall hear
The rain and wind beat dark December, how,
In this our pinching cave, shall we discourse.
The freezing hours away? We have seen nothing;
We are beastly: subtle as the fox for prey,
Like warlike as the wolf for what we eat.
Our valour is to chase what flies; our cage
We make a choir, as doth the prison’d bird,
And sing our bondage freely.

BELARIUS.
How you speak!
Did you but know the city’s usuries,
And felt them knowingly; the art o’ th’ court,
As hard to leave as keep, whose top to climb
Is certain falling, or so slipp’ry that
The fear’s as bad as falling; the toil o’ th’ war,
A pain that only seems to seek out danger
I’ th’ name of fame and honour, which dies i’ th’ search,
And hath as oft a sland’rous epitaph
As record of fair act; nay, many times,
Doth ill deserve by doing well; what’s worse,
Must curtsy at the censure. O, boys, this story
The world may read in me; my body’s mark’d
With Roman swords, and my report was once
First with the best of note. Cymbeline lov’d me;
And when a soldier was the theme, my name
Was not far off. Then was I as a tree
Whose boughs did bend with fruit. But in one night
A storm, or robbery, call it what you will,
Shook down my mellow hangings, nay, my leaves,
And left me bare to weather.

GUIDERIUS.
Uncertain favour!

BELARIUS.
My fault being nothing, as I have told you oft,
But that two villains, whose false oaths prevail’d
Before my perfect honour, swore to Cymbeline
I was confederate with the Romans. So
Follow’d my banishment, and this twenty years
This rock and these demesnes have been my world,
Where I have liv’d at honest freedom, paid
More pious debts to heaven than in all
The fore-end of my time. But up to th’ mountains!
This is not hunters’ language. He that strikes
The venison first shall be the lord o’ th’ feast;
To him the other two shall minister;
And we will fear no poison, which attends
In place of greater state. I’ll meet you in the valleys.

 [_Exeunt Guiderius and Arviragus._]

How hard it is to hide the sparks of nature!
These boys know little they are sons to th’ King,
Nor Cymbeline dreams that they are alive.
They think they are mine; and though train’d up thus meanly
I’ th’ cave wherein they bow, their thoughts do hit
The roofs of palaces, and nature prompts them
In simple and low things to prince it much
Beyond the trick of others. This Polydore,
The heir of Cymbeline and Britain, who
The King his father call’d Guiderius—Jove!
When on my three-foot stool I sit and tell
The warlike feats I have done, his spirits fly out
Into my story; say ‘Thus mine enemy fell,
And thus I set my foot on’s neck’; even then
The princely blood flows in his cheek, he sweats,
Strains his young nerves, and puts himself in posture
That acts my words. The younger brother, Cadwal,
Once Arviragus, in as like a figure
Strikes life into my speech, and shows much more
His own conceiving. Hark, the game is rous’d!
O Cymbeline, heaven and my conscience knows
Thou didst unjustly banish me! Whereon,
At three and two years old, I stole these babes,
Thinking to bar thee of succession as
Thou refts me of my lands. Euriphile,
Thou wast their nurse; they took thee for their mother,
And every day do honour to her grave.
Myself, Belarius, that am Morgan call’d,
They take for natural father. The game is up.

 [_Exit._]

SCENE IV. Wales, near Milford Haven.

 Enter Pisanio and Imogen.

IMOGEN.
Thou told’st me, when we came from horse, the place
Was near at hand. Ne’er long’d my mother so
To see me first as I have now. Pisanio! Man!
Where is Posthumus? What is in thy mind
That makes thee stare thus? Wherefore breaks that sigh
From th’ inward of thee? One but painted thus
Would be interpreted a thing perplex’d
Beyond self-explication. Put thyself
Into a haviour of less fear, ere wildness
Vanquish my staider senses. What’s the matter?
Why tender’st thou that paper to me with
A look untender? If’t be summer news,
Smile to’t before; if winterly, thou need’st
But keep that count’nance still. My husband’s hand?
That drug-damn’d Italy hath out-craftied him,
And he’s at some hard point. Speak, man; thy tongue
May take off some extremity, which to read
Would be even mortal to me.

PISANIO.
Please you read,
And you shall find me, wretched man, a thing
The most disdain’d of fortune.

IMOGEN.
[_Reads._] _Thy mistress, Pisanio, hath play’d the strumpet in my bed,
the testimonies whereof lie bleeding in me. I speak not out of weak
surmises, but from proof as strong as my grief and as certain as I
expect my revenge. That part thou, Pisanio, must act for me, if thy
faith be not tainted with the breach of hers. Let thine own hands take
away her life; I shall give thee opportunity at Milford Haven; she hath
my letter for the purpose; where, if thou fear to strike, and to make
me certain it is done, thou art the pandar to her dishonour, and
equally to me disloyal._

PISANIO.
What shall I need to draw my sword? The paper
Hath cut her throat already. No, ’tis slander,
Whose edge is sharper than the sword, whose tongue
Outvenoms all the worms of Nile, whose breath
Rides on the posting winds and doth belie
All corners of the world. Kings, queens, and states,
Maids, matrons, nay, the secrets of the grave,
This viperous slander enters. What cheer, madam?

IMOGEN.
False to his bed? What is it to be false?
To lie in watch there, and to think on him?
To weep twixt clock and clock? If sleep charge nature,
To break it with a fearful dream of him,
And cry myself awake? That’s false to’s bed,
Is it?

PISANIO.
Alas, good lady!

IMOGEN.
I false! Thy conscience witness! Iachimo,
Thou didst accuse him of incontinency;
Thou then look’dst like a villain; now, methinks,
Thy favour’s good enough. Some jay of Italy,
Whose mother was her painting, hath betray’d him.
Poor I am stale, a garment out of fashion,
And for I am richer than to hang by th’ walls
I must be ripp’d. To pieces with me! O,
Men’s vows are women’s traitors! All good seeming,
By thy revolt, O husband, shall be thought
Put on for villainy; not born where’t grows,
But worn a bait for ladies.

PISANIO.
Good madam, hear me.

IMOGEN.
True honest men being heard, like false Æneas,
Were, in his time, thought false; and Sinon’s weeping
Did scandal many a holy tear, took pity
From most true wretchedness. So thou, Posthumus,
Wilt lay the leaven on all proper men:
Goodly and gallant shall be false and perjur’d
From thy great fail. Come, fellow, be thou honest;
Do thou thy master’s bidding; when thou seest him,
A little witness my obedience. Look!
I draw the sword myself; take it, and hit
The innocent mansion of my love, my heart.
Fear not; ’tis empty of all things but grief;
Thy master is not there, who was indeed
The riches of it. Do his bidding; strike.
Thou mayst be valiant in a better cause,
But now thou seem’st a coward.

PISANIO.
Hence, vile instrument!
Thou shalt not damn my hand.

IMOGEN.
Why, I must die;
And if I do not by thy hand, thou art
No servant of thy master’s. Against self-slaughter
There is a prohibition so divine
That cravens my weak hand. Come, here’s my heart:
Something’s afore’t. Soft, soft! we’ll no defence,
Obedient as the scabbard. What is here?
The scriptures of the loyal Leonatus
All turn’d to heresy? Away, away,
Corrupters of my faith, you shall no more
Be stomachers to my heart. Thus may poor fools
Believe false teachers; though those that are betray’d
Do feel the treason sharply, yet the traitor
Stands in worse case of woe. And thou, Posthumus,
That didst set up my disobedience ’gainst the King
My father, and make me put into contempt the suits
Of princely fellows, shalt hereafter find
It is no act of common passage but
A strain of rareness; and I grieve myself
To think, when thou shalt be disedg’d by her
That now thou tirest on, how thy memory
Will then be pang’d by me. Prithee dispatch.
The lamb entreats the butcher. Where’s thy knife?
Thou art too slow to do thy master’s bidding,
When I desire it too.

PISANIO.
O gracious lady,
Since I receiv’d command to do this busines
I have not slept one wink.

IMOGEN.
Do’t, and to bed then.

PISANIO.
I’ll wake mine eyeballs first.

IMOGEN.
Wherefore then
Didst undertake it? Why hast thou abus’d
So many miles with a pretence? This place?
Mine action and thine own? our horses’ labour?
The time inviting thee? The perturb’d court,
For my being absent? whereunto I never
Purpose return. Why hast thou gone so far
To be unbent when thou hast ta’en thy stand,
Th’ elected deer before thee?

PISANIO.
But to win time
To lose so bad employment, in the which
I have consider’d of a course. Good lady,
Hear me with patience.

IMOGEN.
Talk thy tongue weary, speak.
I have heard I am a strumpet, and mine ear,
Therein false struck, can take no greater wound,
Nor tent to bottom that. But speak.

PISANIO.
Then, madam,
I thought you would not back again.

IMOGEN.
Most like,
Bringing me here to kill me.

PISANIO.
Not so, neither;
But if I were as wise as honest, then
My purpose would prove well. It cannot be
But that my master is abus’d. Some villain,
Ay, and singular in his art, hath done you both
This cursed injury.

IMOGEN.
Some Roman courtezan!

PISANIO.
No, on my life!
I’ll give but notice you are dead, and send him
Some bloody sign of it, for ’tis commanded
I should do so. You shall be miss’d at court,
And that will well confirm it.

IMOGEN.
Why, good fellow,
What shall I do the while? Where bide? How live?
Or in my life what comfort, when I am
Dead to my husband?

PISANIO.
If you’ll back to th’ court—

IMOGEN.
No court, no father, nor no more ado
With that harsh, noble, simple nothing,
That Cloten, whose love-suit hath been to me
As fearful as a siege.

PISANIO.
If not at court,
Then not in Britain must you bide.

IMOGEN.
Where then?
Hath Britain all the sun that shines? Day, night,
Are they not but in Britain? I’ th’ world’s volume
Our Britain seems as of it, but not in’t;
In a great pool a swan’s nest. Prithee think
There’s livers out of Britain.

PISANIO.
I am most glad
You think of other place. Th’ ambassador,
Lucius the Roman, comes to Milford Haven
Tomorrow. Now, if you could wear a mind
Dark as your fortune is, and but disguise
That which t’ appear itself must not yet be
But by self-danger, you should tread a course
Pretty and full of view; yea, happily, near
The residence of Posthumus; so nigh, at least,
That though his actions were not visible, yet
Report should render him hourly to your ear
As truly as he moves.

IMOGEN.
O! for such means,
Though peril to my modesty, not death on’t,
I would adventure.

PISANIO.
Well then, here’s the point:
You must forget to be a woman; change
Command into obedience; fear and niceness
(The handmaids of all women, or, more truly,
Woman it pretty self) into a waggish courage;
Ready in gibes, quick-answer’d, saucy, and
As quarrelous as the weasel. Nay, you must
Forget that rarest treasure of your cheek,
Exposing it (but, O, the harder heart!
Alack, no remedy) to the greedy touch
Of common-kissing Titan, and forget
Your laboursome and dainty trims wherein
You made great Juno angry.

IMOGEN.
Nay, be brief;
I see into thy end, and am almost
A man already.

PISANIO.
First, make yourself but like one.
Fore-thinking this, I have already fit
(’Tis in my cloak-bag) doublet, hat, hose, all
That answer to them. Would you, in their serving,
And with what imitation you can borrow
From youth of such a season, ’fore noble Lucius
Present yourself, desire his service, tell him
Wherein you’re happy; which will make him know
If that his head have ear in music; doubtless
With joy he will embrace you; for he’s honourable,
And, doubling that, most holy. Your means abroad:
You have me, rich; and I will never fail
Beginning nor supplyment.

IMOGEN.
Thou art all the comfort
The gods will diet me with. Prithee away!
There’s more to be consider’d; but we’ll even
All that good time will give us. This attempt
I am soldier to, and will abide it with
A prince’s courage. Away, I prithee.

PISANIO.
Well, madam, we must take a short farewell,
Lest, being miss’d, I be suspected of
Your carriage from the court. My noble mistress,
Here is a box; I had it from the Queen.
What’s in’t is precious. If you are sick at sea
Or stomach-qualm’d at land, a dram of this
Will drive away distemper. To some shade,
And fit you to your manhood. May the gods
Direct you to the best!

IMOGEN.
Amen. I thank thee.

 [_Exeunt severally._]

SCENE V. Britain. Cymbeline’s palace.

 Enter Cymbeline, Queen, Cloten, Lucius and Lords.

CYMBELINE.
Thus far, and so farewell.

LUCIUS.
Thanks, royal sir.
My emperor hath wrote; I must from hence,
And am right sorry that I must report ye
My master’s enemy.

CYMBELINE.
Our subjects, sir,
Will not endure his yoke; and for ourself
To show less sovereignty than they, must needs
Appear unkinglike.

LUCIUS.
So, sir. I desire of you
A conduct overland to Milford Haven.
Madam, all joy befall your Grace, and you!

CYMBELINE.
My lords, you are appointed for that office;
The due of honour in no point omit.
So farewell, noble Lucius.

LUCIUS.
Your hand, my lord.

CLOTEN.
Receive it friendly; but from this time forth
I wear it as your enemy.

LUCIUS.
Sir, the event
Is yet to name the winner. Fare you well.

CYMBELINE.
Leave not the worthy Lucius, good my lords,
Till he have cross’d the Severn. Happiness!

 [_Exeunt Lucius and Lords._]

QUEEN.
He goes hence frowning; but it honours us
That we have given him cause.

CLOTEN.
’Tis all the better;
Your valiant Britons have their wishes in it.

CYMBELINE.
Lucius hath wrote already to the Emperor
How it goes here. It fits us therefore ripely
Our chariots and our horsemen be in readiness.
The pow’rs that he already hath in Gallia
Will soon be drawn to head, from whence he moves
His war for Britain.

QUEEN.
’Tis not sleepy business,
But must be look’d to speedily and strongly.

CYMBELINE.
Our expectation that it would be thus
Hath made us forward. But, my gentle queen,
Where is our daughter? She hath not appear’d
Before the Roman, nor to us hath tender’d
The duty of the day. She looks us like
A thing more made of malice than of duty;
We have noted it. Call her before us, for
We have been too slight in sufferance.

 [_Exit an Attendant._]

QUEEN.
Royal sir,
Since the exile of Posthumus, most retir’d
Hath her life been; the cure whereof, my lord,
’Tis time must do. Beseech your Majesty,
Forbear sharp speeches to her; she’s a lady
So tender of rebukes that words are strokes,
And strokes death to her.

 Enter Attendant.

CYMBELINE.
Where is she, sir? How
Can her contempt be answer’d?

ATTENDANT.
Please you, sir,
Her chambers are all lock’d, and there’s no answer
That will be given to th’ loud of noise we make.

QUEEN.
My lord, when last I went to visit her,
She pray’d me to excuse her keeping close;
Whereto constrain’d by her infirmity
She should that duty leave unpaid to you
Which daily she was bound to proffer. This
She wish’d me to make known; but our great court
Made me to blame in memory.

CYMBELINE.
Her doors lock’d?
Not seen of late? Grant, heavens, that which I fear
Prove false!

 [_Exit._]

QUEEN.
Son, I say, follow the King.

CLOTEN.
That man of hers, Pisanio, her old servant,
I have not seen these two days.

QUEEN.
Go, look after.

 [_Exit Cloten._]

Pisanio, thou that stand’st so for Posthumus!
He hath a drug of mine. I pray his absence
Proceed by swallowing that; for he believes
It is a thing most precious. But for her,
Where is she gone? Haply despair hath seiz’d her;
Or, wing’d with fervour of her love, she’s flown
To her desir’d Posthumus. Gone she is
To death or to dishonour, and my end
Can make good use of either. She being down,
I have the placing of the British crown.

 Enter Cloten.

How now, my son?

CLOTEN.
’Tis certain she is fled.
Go in and cheer the King. He rages; none
Dare come about him.

QUEEN.
All the better. May
This night forestall him of the coming day!

 [_Exit._]

CLOTEN.
I love and hate her; for she’s fair and royal,
And that she hath all courtly parts more exquisite
Than lady, ladies, woman. From every one
The best she hath, and she, of all compounded,
Outsells them all. I love her therefore; but
Disdaining me and throwing favours on
The low Posthumus slanders so her judgement
That what’s else rare is chok’d; and in that point
I will conclude to hate her, nay, indeed,
To be reveng’d upon her. For when fools
Shall—

 Enter Pisanio.

Who is here? What, are you packing, sirrah?
Come hither. Ah, you precious pandar! Villain,
Where is thy lady? In a word, or else
Thou art straightway with the fiends.

PISANIO.
O good my lord!

CLOTEN.
Where is thy lady? or, by Jupiter—
I will not ask again. Close villain,
I’ll have this secret from thy heart, or rip
Thy heart to find it. Is she with Posthumus?
From whose so many weights of baseness cannot
A dram of worth be drawn.

PISANIO.
Alas, my lord,
How can she be with him? When was she miss’d?
He is in Rome.

CLOTEN.
Where is she, sir? Come nearer.
No farther halting! Satisfy me home
What is become of her.

PISANIO.
O my all-worthy lord!

CLOTEN.
All-worthy villain!
Discover where thy mistress is at once,
At the next word. No more of ‘worthy lord’!
Speak, or thy silence on the instant is
Thy condemnation and thy death.

PISANIO.
Then, sir,
This paper is the history of my knowledge
Touching her flight.

 [_Presenting a letter._]

CLOTEN.
Let’s see’t. I will pursue her
Even to Augustus’ throne.

PISANIO.
[_Aside._] Or this or perish.
She’s far enough; and what he learns by this
May prove his travel, not her danger.

CLOTEN.
Humh!

PISANIO.
[_Aside._] I’ll write to my lord she’s dead. O Imogen,
Safe mayst thou wander, safe return again!

CLOTEN.
Sirrah, is this letter true?

PISANIO.
Sir, as I think.

CLOTEN.
It is Posthumus’ hand; I know’t. Sirrah, if thou wouldst not be a
villain, but do me true service, undergo those employments wherein I
should have cause to use thee with a serious industry—that is, what
villainy soe’er I bid thee do, to perform it directly and truly—I would
think thee an honest man; thou shouldst neither want my means for thy
relief nor my voice for thy preferment.

PISANIO.
Well, my good lord.

CLOTEN.
Wilt thou serve me? For since patiently and constantly thou hast stuck
to the bare fortune of that beggar Posthumus, thou canst not, in the
course of gratitude, but be a diligent follower of mine. Wilt thou
serve me?

PISANIO.
Sir, I will.

CLOTEN.
Give me thy hand; here’s my purse. Hast any of thy late master’s
garments in thy possession?

PISANIO.
I have, my lord, at my lodging, the same suit he wore when he took
leave of my lady and mistress.

CLOTEN.
The first service thou dost me, fetch that suit hither. Let it be thy
first service; go.

PISANIO.
I shall, my lord.

 [_Exit._]

CLOTEN.
Meet thee at Milford Haven! I forgot to ask him one thing; I’ll
remember’t anon. Even there, thou villain Posthumus, will I kill thee.
I would these garments were come. She said upon a time—the bitterness
of it I now belch from my heart—that she held the very garment of
Posthumus in more respect than my noble and natural person, together
with the adornment of my qualities. With that suit upon my back will I
ravish her; first kill him, and in her eyes. There shall she see my
valour, which will then be a torment to her contempt. He on the ground,
my speech of insultment ended on his dead body, and when my lust hath
dined—which, as I say, to vex her I will execute in the clothes that
she so prais’d—to the court I’ll knock her back, foot her home again.
She hath despis’d me rejoicingly, and I’ll be merry in my revenge.

 Enter Pisanio with the clothes.

Be those the garments?

PISANIO.
Ay, my noble lord.

CLOTEN.
How long is’t since she went to Milford Haven?

PISANIO.
She can scarce be there yet.

CLOTEN.
Bring this apparel to my chamber; that is the second thing that I have
commanded thee. The third is that thou wilt be a voluntary mute to my
design. Be but duteous and true, preferment shall tender itself to
thee. My revenge is now at Milford, would I had wings to follow it!
Come, and be true.

 [_Exit._]

PISANIO.
Thou bid’st me to my loss; for true to thee
Were to prove false, which I will never be,
To him that is most true. To Milford go,
And find not her whom thou pursuest. Flow, flow,
You heavenly blessings, on her! This fool’s speed
Be cross’d with slowness! Labour be his meed!

 [_Exit._]

SCENE VI. Wales. Before the cave of Belarius.

 Enter Imogen alone, in boy’s clothes.

IMOGEN.
I see a man’s life is a tedious one.
I have tir’d myself, and for two nights together
Have made the ground my bed. I should be sick
But that my resolution helps me. Milford,
When from the mountain-top Pisanio show’d thee,
Thou wast within a ken. O Jove! I think
Foundations fly the wretched; such, I mean,
Where they should be reliev’d. Two beggars told me
I could not miss my way. Will poor folks lie,
That have afflictions on them, knowing ’tis
A punishment or trial? Yes; no wonder,
When rich ones scarce tell true. To lapse in fulness
Is sorer than to lie for need; and falsehood
Is worse in kings than beggars. My dear lord!
Thou art one o’ th’ false ones. Now I think on thee
My hunger’s gone; but even before, I was
At point to sink for food. But what is this?
Here is a path to’t; ’tis some savage hold.
I were best not call; I dare not call. Yet famine,
Ere clean it o’erthrow nature, makes it valiant.
Plenty and peace breeds cowards; hardness ever
Of hardiness is mother. Ho! who’s here?
If anything that’s civil, speak; if savage,
Take or lend. Ho! No answer? Then I’ll enter.
Best draw my sword; and if mine enemy
But fear the sword, like me, he’ll scarcely look on’t.
Such a foe, good heavens!

 [_Exit into the cave._]

SCENE VII. The same.

 Enter Belarius, Guiderius and Arviragus.

BELARIUS.
You, Polydore, have prov’d best woodman and
Are master of the feast. Cadwal and I
Will play the cook and servant; ’tis our match.
The sweat of industry would dry and die
But for the end it works to. Come, our stomachs
Will make what’s homely savoury; weariness
Can snore upon the flint, when resty sloth
Finds the down pillow hard. Now, peace be here,
Poor house, that keep’st thyself!

GUIDERIUS.
I am thoroughly weary.

ARVIRAGUS.
I am weak with toil, yet strong in appetite.

GUIDERIUS.
There is cold meat i’ th’ cave; we’ll browse on that
Whilst what we have kill’d be cook’d.

BELARIUS.
[_Looking into the cave._] Stay, come not in.
But that it eats our victuals, I should think
Here were a fairy.

GUIDERIUS.
What’s the matter, sir?

BELARIUS.
By Jupiter, an angel! or, if not,
An earthly paragon! Behold divineness
No elder than a boy!

 Enter Imogen.

IMOGEN.
Good masters, harm me not.
Before I enter’d here I call’d, and thought
To have begg’d or bought what I have took. Good troth,
I have stol’n nought; nor would not though I had found
Gold strew’d i’ th’ floor. Here’s money for my meat.
I would have left it on the board, so soon
As I had made my meal, and parted
With pray’rs for the provider.

GUIDERIUS.
Money, youth?

ARVIRAGUS.
All gold and silver rather turn to dirt,
As ’tis no better reckon’d but of those
Who worship dirty gods.

IMOGEN.
I see you’re angry.
Know, if you kill me for my fault, I should
Have died had I not made it.

BELARIUS.
Whither bound?

IMOGEN.
To Milford Haven.

BELARIUS.
What’s your name?

IMOGEN.
Fidele, sir. I have a kinsman who
Is bound for Italy; he embark’d at Milford;
To whom being going, almost spent with hunger,
I am fall’n in this offence.

BELARIUS.
Prithee, fair youth,
Think us no churls, nor measure our good minds
By this rude place we live in. Well encounter’d!
’Tis almost night; you shall have better cheer
Ere you depart, and thanks to stay and eat it.
Boys, bid him welcome.

GUIDERIUS.
Were you a woman, youth,
I should woo hard but be your groom. In honesty
I bid for you as I’d buy.

ARVIRAGUS.
I’ll make’t my comfort
He is a man. I’ll love him as my brother;
And such a welcome as I’d give to him
After long absence, such is yours. Most welcome!
Be sprightly, for you fall ’mongst friends.

IMOGEN.
’Mongst friends,
If brothers. [_Aside._] Would it had been so that they
Had been my father’s sons! Then had my prize
Been less, and so more equal ballasting
To thee, Posthumus.

BELARIUS.
He wrings at some distress.

GUIDERIUS.
Would I could free’t!

ARVIRAGUS.
Or I, whate’er it be,
What pain it cost, what danger! Gods!

BELARIUS.
[_Whispering._] Hark, boys.

IMOGEN.
[_Aside._] Great men,
That had a court no bigger than this cave,
That did attend themselves, and had the virtue
Which their own conscience seal’d them, laying by
That nothing-gift of differing multitudes,
Could not out-peer these twain. Pardon me, gods!
I’d change my sex to be companion with them,
Since Leonatus false.

BELARIUS.
It shall be so.
Boys, we’ll go dress our hunt. Fair youth, come in.
Discourse is heavy, fasting; when we have supp’d,
We’ll mannerly demand thee of thy story,
So far as thou wilt speak it.

GUIDERIUS.
Pray draw near.

ARVIRAGUS.
The night to th’ owl and morn to th’ lark less
welcome.

IMOGEN.
Thanks, sir.

ARVIRAGUS.
I pray draw near.

 [_Exeunt._]

SCENE VIII. Rome. A public place.

 Enter two Roman Senators and Tribunes.

FIRST SENATOR.
This is the tenour of the Emperor’s writ:
That since the common men are now in action
’Gainst the Pannonians and Dalmatians,
And that the legions now in Gallia are
Full weak to undertake our wars against
The fall’n-off Britons, that we do incite
The gentry to this business. He creates
Lucius proconsul; and to you, the tribunes,
For this immediate levy, he commands
His absolute commission. Long live Cæsar!

TRIBUNE.
Is Lucius general of the forces?

SECOND SENATOR.
Ay.

TRIBUNE.
Remaining now in Gallia?

FIRST SENATOR.
With those legions
Which I have spoke of, whereunto your levy
Must be supplyant. The words of your commission
Will tie you to the numbers and the time
Of their dispatch.

TRIBUNE.
We will discharge our duty.

 [_Exeunt._]




ACT IV

SCENE I. Wales. Near the cave of Belarius.


 Enter Cloten alone.

CLOTEN.
I am near to th’ place where they should meet, if Pisanio have mapp’d
it truly. How fit his garments serve me! Why should his mistress, who
was made by him that made the tailor, not be fit too? The rather,
saving reverence of the word, for ’tis said a woman’s fitness comes by
fits. Therein I must play the workman. I dare speak it to myself, for
it is not vain-glory for a man and his glass to confer in his own
chamber; I mean, the lines of my body are as well drawn as his; no less
young, more strong, not beneath him in fortunes, beyond him in the
advantage of the time, above him in birth, alike conversant in general
services, and more remarkable in single oppositions. Yet this
imperceiverant thing loves him in my despite. What mortality is!
Posthumus, thy head, which now is growing upon thy shoulders, shall
within this hour be off; thy mistress enforced; thy garments cut to
pieces before her face; and all this done, spurn her home to her
father, who may, haply, be a little angry for my so rough usage; but my
mother, having power of his testiness, shall turn all into my
commendations. My horse is tied up safe. Out, sword, and to a sore
purpose! Fortune, put them into my hand. This is the very description
of their meeting-place; and the fellow dares not deceive me.

 [_Exit._]

SCENE II. Wales. Before the cave of Belarius.

 Enter from the cave, Belarius, Guiderius, Arviragus and Imogen.

BELARIUS.
[_To Imogen._] You are not well. Remain here in the cave;
We’ll come to you after hunting.

ARVIRAGUS.
[_To Imogen._] Brother, stay here.
Are we not brothers?

IMOGEN.
So man and man should be;
But clay and clay differs in dignity,
Whose dust is both alike. I am very sick.

GUIDERIUS.
Go you to hunting; I’ll abide with him.

IMOGEN.
So sick I am not, yet I am not well;
But not so citizen a wanton as
To seem to die ere sick. So please you, leave me;
Stick to your journal course. The breach of custom
Is breach of all. I am ill, but your being by me
Cannot amend me; society is no comfort
To one not sociable. I am not very sick,
Since I can reason of it. Pray you trust me here.
I’ll rob none but myself; and let me die,
Stealing so poorly.

GUIDERIUS.
I love thee; I have spoke it.
How much the quantity, the weight as much
As I do love my father.

BELARIUS.
What? how? how?

ARVIRAGUS.
If it be sin to say so, sir, I yoke me
In my good brother’s fault. I know not why
I love this youth, and I have heard you say
Love’s reason’s without reason. The bier at door,
And a demand who is’t shall die, I’d say
‘My father, not this youth.’

BELARIUS.
[_Aside._] O noble strain!
O worthiness of nature! breed of greatness!
Cowards father cowards and base things sire base.
Nature hath meal and bran, contempt and grace.
I’m not their father; yet who this should be
Doth miracle itself, lov’d before me.—
’Tis the ninth hour o’ th’ morn.

ARVIRAGUS.
Brother, farewell.

IMOGEN.
I wish ye sport.

ARVIRAGUS.
Your health. [_To Belarius._] So please you, sir.

IMOGEN.
[_Aside._] These are kind creatures. Gods, what lies I
have heard!
Our courtiers say all’s savage but at court.
Experience, O, thou disprov’st report!
Th’ imperious seas breed monsters; for the dish,
Poor tributary rivers as sweet fish.
I am sick still; heart-sick. Pisanio,
I’ll now taste of thy drug.

 [_Swallows some._]

GUIDERIUS.
I could not stir him.
He said he was gentle, but unfortunate;
Dishonestly afflicted, but yet honest.

ARVIRAGUS.
Thus did he answer me; yet said hereafter
I might know more.

BELARIUS.
To th’ field, to th’ field!
We’ll leave you for this time. Go in and rest.

ARVIRAGUS.
We’ll not be long away.

BELARIUS.
Pray be not sick,
For you must be our huswife.

IMOGEN.
Well, or ill,
I am bound to you.

BELARIUS.
And shalt be ever.

 [_Exit Imogen into the cave._]

This youth, howe’er distress’d, appears he hath had
Good ancestors.

ARVIRAGUS.
How angel-like he sings!

GUIDERIUS.
But his neat cookery! He cut our roots in characters,
And sauc’d our broths as Juno had been sick,
And he her dieter.

ARVIRAGUS.
Nobly he yokes
A smiling with a sigh, as if the sigh
Was that it was for not being such a smile;
The smile mocking the sigh that it would fly
From so divine a temple to commix
With winds that sailors rail at.

GUIDERIUS.
I do note
That grief and patience, rooted in him both,
Mingle their spurs together.

ARVIRAGUS.
Grow patience!
And let the stinking elder, grief, untwine
His perishing root with the increasing vine!

BELARIUS.
It is great morning. Come, away! Who’s there?

 Enter Cloten.

CLOTEN.
I cannot find those runagates; that villain
Hath mock’d me. I am faint.

BELARIUS.
Those runagates?
Means he not us? I partly know him; ’tis
Cloten, the son o’ th’ Queen. I fear some ambush.
I saw him not these many years, and yet
I know ’tis he. We are held as outlaws. Hence!

GUIDERIUS.
He is but one; you and my brother search
What companies are near. Pray you away;
Let me alone with him.

 [_Exeunt Belarius and Arviragus._]

CLOTEN.
Soft! What are you
That fly me thus? Some villain mountaineers?
I have heard of such. What slave art thou?

GUIDERIUS.
A thing
More slavish did I ne’er than answering
A slave without a knock.

CLOTEN.
Thou art a robber,
A law-breaker, a villain. Yield thee, thief.

GUIDERIUS.
To who? To thee? What art thou? Have not I
An arm as big as thine, a heart as big?
Thy words, I grant, are bigger, for I wear not
My dagger in my mouth. Say what thou art;
Why I should yield to thee.

CLOTEN.
Thou villain base,
Know’st me not by my clothes?

GUIDERIUS.
No, nor thy tailor, rascal,
Who is thy grandfather; he made those clothes,
Which, as it seems, make thee.

CLOTEN.
Thou precious varlet,
My tailor made them not.

GUIDERIUS.
Hence, then, and thank
The man that gave them thee. Thou art some fool;
I am loath to beat thee.

CLOTEN.
Thou injurious thief,
Hear but my name, and tremble.

GUIDERIUS.
What’s thy name?

CLOTEN.
Cloten, thou villain.

GUIDERIUS.
Cloten, thou double villain, be thy name,
I cannot tremble at it. Were it Toad, or Adder, Spider,
’Twould move me sooner.

CLOTEN.
To thy further fear,
Nay, to thy mere confusion, thou shalt know
I am son to th’ Queen.

GUIDERIUS.
I’m sorry for’t; not seeming
So worthy as thy birth.

CLOTEN.
Art not afeard?

GUIDERIUS.
Those that I reverence, those I fear—the wise;
At fools I laugh, not fear them.

CLOTEN.
Die the death.
When I have slain thee with my proper hand,
I’ll follow those that even now fled hence,
And on the gates of Lud’s Town set your heads.
Yield, rustic mountaineer.

 [_Exeunt, fighting._]

 Enter Belarius and Arviragus.

BELARIUS.
No company’s abroad?

ARVIRAGUS.
None in the world; you did mistake him, sure.

BELARIUS.
I cannot tell; long is it since I saw him,
But time hath nothing blurr’d those lines of favour
Which then he wore; the snatches in his voice,
And burst of speaking, were as his. I am absolute
’Twas very Cloten.

ARVIRAGUS.
In this place we left them.
I wish my brother make good time with him,
You say he is so fell.

BELARIUS.
Being scarce made up,
I mean to man, he had not apprehension
Or roaring terrors; for defect of judgement
Is oft the cease of fear.

 Enter Guiderius with Cloten’s head.

But, see, thy brother.

GUIDERIUS.
This Cloten was a fool, an empty purse;
There was no money in’t. Not Hercules
Could have knock’d out his brains, for he had none;
Yet I not doing this, the fool had borne
My head as I do his.

BELARIUS.
What hast thou done?

GUIDERIUS.
I am perfect what: cut off one Cloten’s head,
Son to the Queen, after his own report;
Who call’d me traitor, mountaineer, and swore
With his own single hand he’d take us in,
Displace our heads where, thank the gods, they grow,
And set them on Lud’s Town.

BELARIUS.
We are all undone.

GUIDERIUS.
Why, worthy father, what have we to lose
But that he swore to take, our lives? The law
Protects not us; then why should we be tender
To let an arrogant piece of flesh threat us,
Play judge and executioner all himself,
For we do fear the law? What company
Discover you abroad?

BELARIUS.
No single soul
Can we set eye on, but in all safe reason
He must have some attendants. Though his humour
Was nothing but mutation, ay, and that
From one bad thing to worse, not frenzy, not
Absolute madness could so far have rav’d,
To bring him here alone. Although perhaps
It may be heard at court that such as we
Cave here, hunt here, are outlaws, and in time
May make some stronger head, the which he hearing,
As it is like him, might break out and swear
He’d fetch us in; yet is’t not probable
To come alone, either he so undertaking
Or they so suffering. Then on good ground we fear,
If we do fear this body hath a tail
More perilous than the head.

ARVIRAGUS.
Let ordinance
Come as the gods foresay it. Howsoe’er,
My brother hath done well.

BELARIUS.
I had no mind
To hunt this day; the boy Fidele’s sickness
Did make my way long forth.

GUIDERIUS.
With his own sword,
Which he did wave against my throat, I have ta’en
His head from him. I’ll throw’t into the creek
Behind our rock, and let it to the sea
And tell the fishes he’s the Queen’s son, Cloten.
That’s all I reck.

 [_Exit._]

BELARIUS.
I fear ’twill be reveng’d.
Would, Polydore, thou hadst not done’t! though valour
Becomes thee well enough.

ARVIRAGUS.
Would I had done’t,
So the revenge alone pursu’d me! Polydore,
I love thee brotherly, but envy much
Thou hast robb’d me of this deed. I would revenges,
That possible strength might meet, would seek us through,
And put us to our answer.

BELARIUS.
Well, ’tis done.
We’ll hunt no more today, nor seek for danger
Where there’s no profit. I prithee to our rock.
You and Fidele play the cooks; I’ll stay
Till hasty Polydore return, and bring him
To dinner presently.

ARVIRAGUS.
Poor sick Fidele!
I’ll willingly to him; to gain his colour
I’d let a parish of such Cloten’s blood,
And praise myself for charity.

 [_Exit._]

BELARIUS.
O thou goddess,
Thou divine Nature, thou thyself thou blazon’st
In these two princely boys! They are as gentle
As zephyrs blowing below the violet,
Not wagging his sweet head; and yet as rough,
Their royal blood enchaf’d, as the rud’st wind
That by the top doth take the mountain pine
And make him stoop to th’ vale. ’Tis wonder
That an invisible instinct should frame them
To royalty unlearn’d, honour untaught,
Civility not seen from other, valour
That wildly grows in them, but yields a crop
As if it had been sow’d. Yet still it’s strange
What Cloten’s being here to us portends,
Or what his death will bring us.

 Enter Guiderius.

GUIDERIUS.
Where’s my brother?
I have sent Cloten’s clotpoll down the stream,
In embassy to his mother; his body’s hostage
For his return.

 [_Solemn music._]

BELARIUS.
My ingenious instrument!
Hark, Polydore, it sounds. But what occasion
Hath Cadwal now to give it motion? Hark!

GUIDERIUS.
Is he at home?

BELARIUS.
He went hence even now.

GUIDERIUS.
What does he mean? Since death of my dear’st mother
It did not speak before. All solemn things
Should answer solemn accidents. The matter?
Triumphs for nothing and lamenting toys
Is jollity for apes and grief for boys.
Is Cadwal mad?

 Enter Arviragus with Imogen as dead, bearing her in his arms.

BELARIUS.
Look, here he comes,
And brings the dire occasion in his arms
Of what we blame him for!

ARVIRAGUS.
The bird is dead
That we have made so much on. I had rather
Have skipp’d from sixteen years of age to sixty,
To have turn’d my leaping time into a crutch,
Than have seen this.

GUIDERIUS.
O sweetest, fairest lily!
My brother wears thee not the one half so well
As when thou grew’st thyself.

BELARIUS.
O melancholy!
Who ever yet could sound thy bottom? find
The ooze to show what coast thy sluggish crare
Might’st easiliest harbour in? Thou blessed thing!
Jove knows what man thou mightst have made; but I,
Thou diedst, a most rare boy, of melancholy.
How found you him?

ARVIRAGUS.
Stark, as you see;
Thus smiling, as some fly had tickled slumber,
Not as death’s dart, being laugh’d at; his right cheek
Reposing on a cushion.

GUIDERIUS.
Where?

ARVIRAGUS.
O’ th’ floor;
His arms thus leagu’d. I thought he slept, and put
My clouted brogues from off my feet, whose rudeness
Answer’d my steps too loud.

GUIDERIUS.
Why, he but sleeps.
If he be gone he’ll make his grave a bed;
With female fairies will his tomb be haunted,
And worms will not come to thee.

ARVIRAGUS.
With fairest flowers,
Whilst summer lasts and I live here, Fidele,
I’ll sweeten thy sad grave. Thou shalt not lack
The flower that’s like thy face, pale primrose; nor
The azur’d hare-bell, like thy veins; no, nor
The leaf of eglantine, whom not to slander,
Out-sweet’ned not thy breath. The ruddock would,
With charitable bill (O bill, sore shaming
Those rich-left heirs that let their fathers lie
Without a monument!) bring thee all this;
Yea, and furr’d moss besides, when flow’rs are none,
To winter-ground thy corse—

GUIDERIUS.
Prithee have done,
And do not play in wench-like words with that
Which is so serious. Let us bury him,
And not protract with admiration what
Is now due debt. To th’ grave.

ARVIRAGUS.
Say, where shall’s lay him?

GUIDERIUS.
By good Euriphile, our mother.

ARVIRAGUS.
Be’t so;
And let us, Polydore, though now our voices
Have got the mannish crack, sing him to th’ ground,
As once to our mother; use like note and words,
Save that Euriphile must be Fidele.

GUIDERIUS.
Cadwal,
I cannot sing. I’ll weep, and word it with thee;
For notes of sorrow out of tune are worse
Than priests and fanes that lie.

ARVIRAGUS.
We’ll speak it, then.

BELARIUS.
Great griefs, I see, med’cine the less, for Cloten
Is quite forgot. He was a queen’s son, boys;
And though he came our enemy, remember
He was paid for that. Though mean and mighty rotting
Together have one dust, yet reverence,
That angel of the world, doth make distinction
Of place ’tween high and low. Our foe was princely;
And though you took his life, as being our foe,
Yet bury him as a prince.

GUIDERIUS.
Pray you fetch him hither.
Thersites’ body is as good as Ajax’,
When neither are alive.

ARVIRAGUS.
If you’ll go fetch him,
We’ll say our song the whilst. Brother, begin.

 [_Exit Belarius._]

GUIDERIUS.
Nay, Cadwal, we must lay his head to th’ East;
My father hath a reason for’t.

ARVIRAGUS.
’Tis true.

GUIDERIUS.
Come on, then, and remove him.

ARVIRAGUS.
So. Begin.

SONG

GUIDERIUS.
_    Fear no more the heat o’ th’ sun,
        Nor the furious winter’s rages;
    Thou thy worldly task hast done,
        Home art gone, and ta’en thy wages.
    Golden lads and girls all must,
    As chimney-sweepers, come to dust._

ARVIRAGUS.
_    Fear no more the frown o’ th’ great;
        Thou art past the tyrant’s stroke.
    Care no more to clothe and eat;
        To thee the reed is as the oak.
    The sceptre, learning, physic, must
    All follow this and come to dust._

GUIDERIUS.
_    Fear no more the lightning flash._

ARVIRAGUS.
_        Nor th’ all-dreaded thunder-stone._

GUIDERIUS.
_    Fear not slander, censure rash;_

ARVIRAGUS.
_    Thou hast finish’d joy and moan._

BOTH.
_    All lovers young, all lovers must
    Consign to thee and come to dust._

GUIDERIUS.
_    No exorciser harm thee!_

ARVIRAGUS.
_    Nor no witchcraft charm thee!_

GUIDERIUS.
_    Ghost unlaid forbear thee!_

ARVIRAGUS.
_    Nothing ill come near thee!_

BOTH.
_    Quiet consummation have,
    And renowned be thy grave!_

 Enter Belarius with the body of Cloten.

GUIDERIUS.
We have done our obsequies. Come, lay him down.

BELARIUS.
Here’s a few flowers; but ’bout midnight, more.
The herbs that have on them cold dew o’ th’ night
Are strewings fit’st for graves. Upon their faces.
You were as flow’rs, now wither’d. Even so
These herblets shall which we upon you strew.
Come on, away. Apart upon our knees.
The ground that gave them first has them again.
Their pleasures here are past, so is their pain.

 [_Exeunt all but Imogen._]

IMOGEN.
[_Awaking._] Yes, sir, to Milford Haven. Which is the way?
I thank you. By yond bush? Pray, how far thither?
’Ods pittikins! can it be six mile yet?
I have gone all night. Faith, I’ll lie down and sleep.
But, soft! no bedfellow. O gods and goddesses!

 [_Seeing the body._]

These flow’rs are like the pleasures of the world;
This bloody man, the care on’t. I hope I dream;
For so I thought I was a cave-keeper,
And cook to honest creatures. But ’tis not so;
’Twas but a bolt of nothing, shot at nothing,
Which the brain makes of fumes. Our very eyes
Are sometimes, like our judgements, blind. Good faith,
I tremble still with fear; but if there be
Yet left in heaven as small a drop of pity
As a wren’s eye, fear’d gods, a part of it!
The dream’s here still. Even when I wake it is
Without me, as within me; not imagin’d, felt.
A headless man? The garments of Posthumus?
I know the shape of’s leg; this is his hand,
His foot Mercurial, his Martial thigh,
The brawns of Hercules; but his Jovial face—
Murder in heaven! How! ’Tis gone. Pisanio,
All curses madded Hecuba gave the Greeks,
And mine to boot, be darted on thee! Thou,
Conspir’d with that irregulous devil, Cloten,
Hath here cut off my lord. To write and read
Be henceforth treacherous! Damn’d Pisanio
Hath with his forged letters (damn’d Pisanio)
From this most bravest vessel of the world
Struck the main-top. O Posthumus! alas,
Where is thy head? Where’s that? Ay me! where’s that?
Pisanio might have kill’d thee at the heart,
And left this head on. How should this be? Pisanio?
’Tis he and Cloten; malice and lucre in them
Have laid this woe here. O, ’tis pregnant, pregnant!
The drug he gave me, which he said was precious
And cordial to me, have I not found it
Murd’rous to th’ senses? That confirms it home.
This is Pisanio’s deed, and Cloten. O!
Give colour to my pale cheek with thy blood,
That we the horrider may seem to those
Which chance to find us. O, my lord, my lord!

 [_Falls fainting on the body._]

 Enter Lucius, Captains and a Soothsayer.

CAPTAIN.
To them the legions garrison’d in Gallia,
After your will, have cross’d the sea, attending
You here at Milford Haven; with your ships,
They are in readiness.

LUCIUS.
But what from Rome?

CAPTAIN.
The Senate hath stirr’d up the confiners
And gentlemen of Italy, most willing spirits,
That promise noble service; and they come
Under the conduct of bold Iachimo,
Sienna’s brother.

LUCIUS.
When expect you them?

CAPTAIN.
With the next benefit o’ th’ wind.

LUCIUS.
This forwardness
Makes our hopes fair. Command our present numbers
Be muster’d; bid the captains look to’t. Now, sir,
What have you dream’d of late of this war’s purpose?

SOOTHSAYER.
Last night the very gods show’d me a vision
(I fast and pray’d for their intelligence) thus:
I saw Jove’s bird, the Roman eagle, wing’d
From the spongy south to this part of the west,
There vanish’d in the sunbeams; which portends,
Unless my sins abuse my divination,
Success to th’ Roman host.

LUCIUS.
Dream often so,
And never false. Soft, ho! what trunk is here
Without his top? The ruin speaks that sometime
It was a worthy building. How? a page?
Or dead or sleeping on him? But dead, rather;
For nature doth abhor to make his bed
With the defunct, or sleep upon the dead.
Let’s see the boy’s face.

CAPTAIN.
He’s alive, my lord.

LUCIUS.
He’ll then instruct us of this body. Young one,
Inform us of thy fortunes; for it seems
They crave to be demanded. Who is this
Thou mak’st thy bloody pillow? Or who was he
That, otherwise than noble nature did,
Hath alter’d that good picture? What’s thy interest
In this sad wreck? How came’t? Who is’t?
What art thou?

IMOGEN.
I am nothing; or if not,
Nothing to be were better. This was my master,
A very valiant Briton and a good,
That here by mountaineers lies slain. Alas!
There is no more such masters. I may wander
From east to occident; cry out for service;
Try many, all good; serve truly; never
Find such another master.

LUCIUS.
’Lack, good youth!
Thou mov’st no less with thy complaining than
Thy master in bleeding. Say his name, good friend.

IMOGEN.
Richard du Champ. [_Aside._] If I do lie, and do
No harm by it, though the gods hear, I hope
They’ll pardon it.—Say you, sir?

LUCIUS.
Thy name?

IMOGEN.
Fidele, sir.

LUCIUS.
Thou dost approve thyself the very same;
Thy name well fits thy faith, thy faith thy name.
Wilt take thy chance with me? I will not say
Thou shalt be so well master’d; but, be sure,
No less belov’d. The Roman Emperor’s letters,
Sent by a consul to me, should not sooner
Than thine own worth prefer thee. Go with me.

IMOGEN.
I’ll follow, sir. But first, an’t please the gods,
I’ll hide my master from the flies, as deep
As these poor pickaxes can dig; and when
With wild wood-leaves and weeds I ha’ strew’d his grave,
And on it said a century of prayers,
Such as I can, twice o’er, I’ll weep and sigh;
And leaving so his service, follow you,
So please you entertain me.

LUCIUS.
Ay, good youth;
And rather father thee than master thee.
My friends,
The boy hath taught us manly duties; let us
Find out the prettiest daisied plot we can,
And make him with our pikes and partisans
A grave. Come, arm him. Boy, he is preferr’d
By thee to us; and he shall be interr’d
As soldiers can. Be cheerful; wipe thine eyes.
Some falls are means the happier to arise.

 [_Exeunt._]

SCENE III. Britain. Cymbeline’s palace.

 Enter Cymbeline, Lords, Pisanio and Attendants.

CYMBELINE.
Again! and bring me word how ’tis with her.

 [_Exit an Attendant._]

A fever with the absence of her son;
A madness, of which her life’s in danger. Heavens,
How deeply you at once do touch me! Imogen,
The great part of my comfort, gone; my queen
Upon a desperate bed, and in a time
When fearful wars point at me; her son gone,
So needful for this present. It strikes me past
The hope of comfort. But for thee, fellow,
Who needs must know of her departure and
Dost seem so ignorant, we’ll enforce it from thee
By a sharp torture.

PISANIO.
Sir, my life is yours;
I humbly set it at your will; but for my mistress,
I nothing know where she remains, why gone,
Nor when she purposes return. Beseech your Highness,
Hold me your loyal servant.

LORD.
Good my liege,
The day that she was missing he was here.
I dare be bound he’s true and shall perform
All parts of his subjection loyally. For Cloten,
There wants no diligence in seeking him,
And will no doubt be found.

CYMBELINE.
The time is troublesome.
[_To Pisanio._] We’ll slip you for a season; but our jealousy
Does yet depend.

LORD.
So please your Majesty,
The Roman legions, all from Gallia drawn,
Are landed on your coast, with a supply
Of Roman gentlemen by the Senate sent.

CYMBELINE.
Now for the counsel of my son and queen!
I am amaz’d with matter.

LORD.
Good my liege,
Your preparation can affront no less
Than what you hear of. Come more, for more you’re ready.
The want is but to put those pow’rs in motion
That long to move.

CYMBELINE.
I thank you. Let’s withdraw,
And meet the time as it seeks us. We fear not
What can from Italy annoy us; but
We grieve at chances here. Away!

 [_Exeunt all but Pisanio._]

PISANIO.
I heard no letter from my master since
I wrote him Imogen was slain. ’Tis strange.
Nor hear I from my mistress, who did promise
To yield me often tidings. Neither know I
What is betid to Cloten, but remain
Perplex’d in all. The heavens still must work.
Wherein I am false I am honest; not true, to be true.
These present wars shall find I love my country,
Even to the note o’ th’ King, or I’ll fall in them.
All other doubts, by time let them be clear’d:
Fortune brings in some boats that are not steer’d.

 [_Exit._]

SCENE IV. Wales. Before the cave of Belarius.

 Enter Belarius, Guiderius and Arviragus.

GUIDERIUS.
The noise is round about us.

BELARIUS.
Let us from it.

ARVIRAGUS.
What pleasure, sir, find we in life, to lock it
From action and adventure?

GUIDERIUS.
Nay, what hope
Have we in hiding us? This way the Romans
Must or for Britons slay us, or receive us
For barbarous and unnatural revolts
During their use, and slay us after.

BELARIUS.
Sons,
We’ll higher to the mountains; there secure us.
To the King’s party there’s no going. Newness
Of Cloten’s death (we being not known, not muster’d
Among the bands) may drive us to a render
Where we have liv’d, and so extort from’s that
Which we have done, whose answer would be death,
Drawn on with torture.

GUIDERIUS.
This is, sir, a doubt
In such a time nothing becoming you
Nor satisfying us.

ARVIRAGUS.
It is not likely
That when they hear the Roman horses neigh,
Behold their quarter’d fires, have both their eyes
And ears so cloy’d importantly as now,
That they will waste their time upon our note,
To know from whence we are.

BELARIUS.
O, I am known
Of many in the army. Many years,
Though Cloten then but young, you see, not wore him
From my remembrance. And, besides, the King
Hath not deserv’d my service nor your loves,
Who find in my exile the want of breeding,
The certainty of this hard life; aye hopeless
To have the courtesy your cradle promis’d,
But to be still hot summer’s tanlings and
The shrinking slaves of winter.

GUIDERIUS.
Than be so,
Better to cease to be. Pray, sir, to th’ army.
I and my brother are not known; yourself
So out of thought, and thereto so o’ergrown,
Cannot be questioned.

ARVIRAGUS.
By this sun that shines,
I’ll thither. What thing is’t that I never
Did see man die! scarce ever look’d on blood
But that of coward hares, hot goats, and venison!
Never bestrid a horse, save one that had
A rider like myself, who ne’er wore rowel
Nor iron on his heel! I am asham’d
To look upon the holy sun, to have
The benefit of his blest beams, remaining
So long a poor unknown.

GUIDERIUS.
By heavens, I’ll go!
If you will bless me, sir, and give me leave,
I’ll take the better care; but if you will not,
The hazard therefore due fall on me by
The hands of Romans!

ARVIRAGUS.
So say I. Amen.

BELARIUS.
No reason I, since of your lives you set
So slight a valuation, should reserve
My crack’d one to more care. Have with you, boys!
If in your country wars you chance to die,
That is my bed too, lads, and there I’ll lie.
Lead, lead. [_Aside._] The time seems long; their blood thinks scorn
Till it fly out and show them princes born.

 [_Exeunt._]




ACT V

SCENE I. Britain. The Roman camp.


 Enter Posthumus alone, with a bloody handkerchief.

POSTHUMUS.
Yea, bloody cloth, I’ll keep thee; for I wish’d
Thou shouldst be colour’d thus. You married ones,
If each of you should take this course, how many
Must murder wives much better than themselves
For wrying but a little! O Pisanio!
Every good servant does not all commands;
No bond but to do just ones. Gods! if you
Should have ta’en vengeance on my faults, I never
Had liv’d to put on this; so had you saved
The noble Imogen to repent, and struck
Me, wretch more worth your vengeance. But alack,
You snatch some hence for little faults; that’s love,
To have them fall no more. You some permit
To second ills with ills, each elder worse,
And make them dread it, to the doers’ thrift.
But Imogen is your own. Do your best wills,
And make me blest to obey. I am brought hither
Among th’ Italian gentry, and to fight
Against my lady’s kingdom. ’Tis enough
That, Britain, I have kill’d thy mistress; peace!
I’ll give no wound to thee. Therefore, good heavens,
Hear patiently my purpose. I’ll disrobe me
Of these Italian weeds, and suit myself
As does a Britain peasant. So I’ll fight
Against the part I come with; so I’ll die
For thee, O Imogen, even for whom my life
Is every breath a death. And thus unknown,
Pitied nor hated, to the face of peril
Myself I’ll dedicate. Let me make men know
More valour in me than my habits show.
Gods, put the strength o’ th’ Leonati in me!
To shame the guise o’ th’ world, I will begin
The fashion less without and more within.

 [_Exit._]

SCENE II. Britain. A field of battle between the British and Roman
camps.

 Enter Lucius, Iachimo and the Roman army at one door, and the British
 army at another, Leonatus Posthumus following like a poor soldier.
 They march over and go out. Alarums. Then enter again, in skirmish,
 Iachimo and Posthumus. He vanquisheth and disarmeth Iachimo and then
 leaves him.

IACHIMO.
The heaviness and guilt within my bosom
Takes off my manhood. I have belied a lady,
The Princess of this country, and the air on’t
Revengingly enfeebles me; or could this carl,
A very drudge of nature’s, have subdu’d me
In my profession? Knighthoods and honours borne
As I wear mine are titles but of scorn.
If that thy gentry, Britain, go before
This lout as he exceeds our lords, the odds
Is that we scarce are men, and you are gods.

 [_Exit._]

 The battle continues; the Britons fly; Cymbeline is taken. Then enter
 to his rescue Belarius, Guiderius and Arviragus.

BELARIUS.
Stand, stand! We have th’ advantage of the ground;
The lane is guarded; nothing routs us but
The villainy of our fears.

GUIDERIUS and ARVIRAGUS.
Stand, stand, and fight!

 Enter Posthumus and seconds the Britons; they rescue Cymbeline and
 exeunt. Then re-enter Lucius and Iachimo with Imogen.

LUCIUS.
Away, boy, from the troops, and save thyself;
For friends kill friends, and the disorder’s such
As war were hoodwink’d.

IACHIMO.
’Tis their fresh supplies.

LUCIUS.
It is a day turn’d strangely. Or betimes
Let’s reinforce or fly.

 [_Exeunt._]

SCENE III. Another part of the field.

 Enter Posthumus and a Briton Lord.

LORD.
Cam’st thou from where they made the stand?

POSTHUMUS.
I did:
Though you, it seems, come from the fliers.

LORD.
I did.

POSTHUMUS.
No blame be to you, sir, for all was lost,
But that the heavens fought. The King himself
Of his wings destitute, the army broken,
And but the backs of Britons seen, all flying,
Through a strait lane; the enemy, full-hearted,
Lolling the tongue with slaught’ring, having work
More plentiful than tools to do’t, struck down
Some mortally, some slightly touch’d, some falling
Merely through fear, that the strait pass was damm’d
With dead men hurt behind, and cowards living
To die with length’ned shame.

LORD.
Where was this lane?

POSTHUMUS.
Close by the battle, ditch’d, and wall’d with turf,
Which gave advantage to an ancient soldier,
An honest one, I warrant, who deserv’d
So long a breeding as his white beard came to,
In doing this for’s country. Athwart the lane
He, with two striplings (lads more like to run
The country base than to commit such slaughter;
With faces fit for masks, or rather fairer
Than those for preservation cas’d or shame)
Made good the passage, cried to those that fled
‘Our Britain’s harts die flying, not our men.
To darkness fleet souls that fly backwards! Stand;
Or we are Romans and will give you that,
Like beasts, which you shun beastly, and may save
But to look back in frown. Stand, stand!’ These three,
Three thousand confident, in act as many—
For three performers are the file when all
The rest do nothing—with this word ‘Stand, stand!’
Accommodated by the place, more charming
With their own nobleness, which could have turn’d
A distaff to a lance, gilded pale looks,
Part shame, part spirit renew’d; that some turn’d coward
But by example (O, a sin in war
Damn’d in the first beginners) ’gan to look
The way that they did and to grin like lions
Upon the pikes o’ th’ hunters. Then began
A stop i’ th’ chaser, a retire; anon
A rout, confusion thick. Forthwith they fly,
Chickens, the way which they stoop’d eagles; slaves,
The strides they victors made; and now our cowards,
Like fragments in hard voyages, became
The life o’ th’ need. Having found the back-door open
Of the unguarded hearts, heavens, how they wound!
Some slain before, some dying, some their friends
O’erborne i’ th’ former wave. Ten chas’d by one
Are now each one the slaughterman of twenty.
Those that would die or ere resist are grown
The mortal bugs o’ th’ field.

LORD.
This was strange chance:
A narrow lane, an old man, and two boys.

POSTHUMUS.
Nay, do not wonder at it; you are made
Rather to wonder at the things you hear
Than to work any. Will you rhyme upon’t,
And vent it for a mock’ry? Here is one:


        ‘Two boys, an old man (twice a boy), a lane,
        Preserv’d the Britons, was the Romans’ bane.’

LORD.
Nay, be not angry, sir.

POSTHUMUS.
’Lack, to what end?
Who dares not stand his foe I’ll be his friend;
For if he’ll do as he is made to do,
I know he’ll quickly fly my friendship too.
You have put me into rhyme.

LORD.
Farewell; you’re angry.

 [_Exit._]

POSTHUMUS.
Still going? This is a lord! O noble misery,
To be i’ th’ field and ask ‘What news?’ of me!
Today how many would have given their honours
To have sav’d their carcasses! took heel to do’t,
And yet died too! I, in mine own woe charm’d,
Could not find death where I did hear him groan,
Nor feel him where he struck. Being an ugly monster,
’Tis strange he hides him in fresh cups, soft beds,
Sweet words; or hath moe ministers than we
That draw his knives i’ th’ war. Well, I will find him;
For being now a favourer to the Briton,
No more a Briton, I have resum’d again
The part I came in. Fight I will no more,
But yield me to the veriest hind that shall
Once touch my shoulder. Great the slaughter is
Here made by th’ Roman; great the answer be
Britons must take. For me, my ransom’s death;
On either side I come to spend my breath,
Which neither here I’ll keep nor bear again,
But end it by some means for Imogen.

 Enter two British Captains and soldiers.

FIRST CAPTAIN.
Great Jupiter be prais’d! Lucius is taken.
’Tis thought the old man and his sons were angels.

SECOND CAPTAIN.
There was a fourth man, in a silly habit,
That gave th’ affront with them.

FIRST CAPTAIN.
So ’tis reported;
But none of ’em can be found. Stand! who’s there?

POSTHUMUS.
A Roman,
Who had not now been drooping here if seconds
Had answer’d him.

SECOND CAPTAIN.
Lay hands on him; a dog!
A leg of Rome shall not return to tell
What crows have peck’d them here. He brags his service,
As if he were of note. Bring him to th’ King.

 Enter Cymbeline, Belarius, Guiderius, Arviragus, Pisanio and Roman
 captives. The Captains present Posthumus to Cymbeline, who delivers
 him over to a gaoler.

 [_Exeunt omnes._]

SCENE IV. Britain. A prison.

 Enter Posthumus and two Gaolers.

FIRST GAOLER. You shall not now be stol’n, you have locks upon you;
So graze as you find pasture.

SECOND GAOLER.
Ay, or a stomach.

 [_Exeunt Gaolers._]

POSTHUMUS.
Most welcome, bondage! for thou art a way,
I think, to liberty. Yet am I better
Than one that’s sick o’ th’ gout, since he had rather
Groan so in perpetuity than be cur’d
By th’ sure physician death, who is the key
T’ unbar these locks. My conscience, thou art fetter’d
More than my shanks and wrists; you good gods, give me
The penitent instrument to pick that bolt,
Then, free for ever! Is’t enough I am sorry?
So children temporal fathers do appease;
Gods are more full of mercy. Must I repent,
I cannot do it better than in gyves,
Desir’d more than constrain’d. To satisfy,
If of my freedom ’tis the main part, take
No stricter render of me than my all.
I know you are more clement than vile men,
Who of their broken debtors take a third,
A sixth, a tenth, letting them thrive again
On their abatement; that’s not my desire.
For Imogen’s dear life take mine; and though
’Tis not so dear, yet ’tis a life; you coin’d it.
’Tween man and man they weigh not every stamp;
Though light, take pieces for the figure’s sake;
You rather mine, being yours. And so, great pow’rs,
If you will take this audit, take this life,
And cancel these cold bonds. O Imogen!
I’ll speak to thee in silence.

 [_Sleeps._]

 Solemn music. Enter, as in an apparition, Sicilius Leonatus, father to
 Posthumus, an old man attired like a warrior; leading in his hand an
 ancient matron, his wife and Mother to Posthumus, with music before
 them. Then, after other music, follows the two young Leonati, brothers
 to Posthumus, with wounds, as they died in the wars. They circle
 Posthumus round as he lies sleeping.

SICILIUS.
No more, thou thunder-master, show
Thy spite on mortal flies.
With Mars fall out, with Juno chide,
That thy adulteries
Rates and revenges.
Hath my poor boy done aught but well,
Whose face I never saw?
I died whilst in the womb he stay’d
Attending nature’s law;
Whose father then, as men report
Thou orphans’ father art,
Thou shouldst have been, and shielded him
From this earth-vexing smart.

MOTHER.
Lucina lent not me her aid,
But took me in my throes,
That from me was Posthumus ripp’d,
Came crying ’mongst his foes,
A thing of pity.

SICILIUS.
Great Nature like his ancestry
Moulded the stuff so fair
That he deserv’d the praise o’ th’ world
As great Sicilius’ heir.

FIRST BROTHER.
When once he was mature for man,
In Britain where was he
That could stand up his parallel,
Or fruitful object be
In eye of Imogen, that best
Could deem his dignity?

MOTHER.
With marriage wherefore was he mock’d,
To be exil’d and thrown
From Leonati seat and cast
From her his dearest one,
Sweet Imogen?

SICILIUS.
Why did you suffer Iachimo,
Slight thing of Italy,
To taint his nobler heart and brain
With needless jealousy,
And to become the geck and scorn
O’ th’ other’s villainy?

SECOND BROTHER.
For this from stiller seats we came,
Our parents and us twain,
That, striking in our country’s cause,
Fell bravely and were slain,
Our fealty and Tenantius’ right
With honour to maintain.

FIRST BROTHER.
Like hardiment Posthumus hath
To Cymbeline perform’d.
Then, Jupiter, thou king of gods,
Why hast thou thus adjourn’d
The graces for his merits due,
Being all to dolours turn’d?

SICILIUS.
Thy crystal window ope; look out;
No longer exercise
Upon a valiant race thy harsh
And potent injuries.

MOTHER.
Since, Jupiter, our son is good,
Take off his miseries.

SICILIUS.
Peep through thy marble mansion. Help!
Or we poor ghosts will cry
To th’ shining synod of the rest
Against thy deity.

BROTHERS.
Help, Jupiter! or we appeal,
And from thy justice fly.

 Jupiter descends in thunder and lightning, sitting upon an eagle. He
 throws a thunderbolt. The Ghosts fall on their knees.

JUPITER.
No more, you petty spirits of region low,
Offend our hearing; hush! How dare you ghosts
Accuse the Thunderer whose bolt, you know,
Sky-planted, batters all rebelling coasts?
Poor shadows of Elysium, hence and rest
Upon your never-withering banks of flow’rs.
Be not with mortal accidents opprest:
No care of yours it is; you know ’tis ours.
Whom best I love I cross; to make my gift,
The more delay’d, delighted. Be content;
Your low-laid son our godhead will uplift;
His comforts thrive, his trials well are spent.
Our Jovial star reign’d at his birth, and in
Our temple was he married. Rise and fade!
He shall be lord of Lady Imogen,
And happier much by his affliction made.
This tablet lay upon his breast, wherein
Our pleasure his full fortune doth confine;
And so, away; no farther with your din
Express impatience, lest you stir up mine.
Mount, eagle, to my palace crystalline.

 [_Ascends._]

SICILIUS.
He came in thunder; his celestial breath
Was sulphurous to smell; the holy eagle
Stoop’d as to foot us. His ascension is
More sweet than our blest fields. His royal bird
Prunes the immortal wing, and cloys his beak,
As when his god is pleas’d.

ALL.
Thanks, Jupiter!

SICILIUS.
The marble pavement closes, he is enter’d
His radiant roof. Away! and, to be blest,
Let us with care perform his great behest.

 [_Ghosts vanish._]

POSTHUMUS.
[_Waking._] Sleep, thou has been a grandsire and begot
A father to me; and thou hast created
A mother and two brothers. But, O scorn,
Gone! They went hence so soon as they were born.
And so I am awake. Poor wretches, that depend
On greatness’ favour, dream as I have done;
Wake and find nothing. But, alas, I swerve;
Many dream not to find, neither deserve,
And yet are steep’d in favours; so am I,
That have this golden chance, and know not why.
What fairies haunt this ground? A book? O rare one!
Be not, as is our fangled world, a garment
Nobler than that it covers. Let thy effects
So follow to be most unlike our courtiers,
As good as promise.

[_Reads._] _When as a lion’s whelp shall, to himself unknown, without
seeking find, and be embrac’d by a piece of tender air; and when from a
stately cedar shall be lopp’d branches which, being dead many years,
shall after revive, be jointed to the old stock, and freshly grow; then
shall Posthumus end his miseries, Britain be fortunate and flourish in
peace and plenty._

’Tis still a dream, or else such stuff as madmen
Tongue, and brain not; either both or nothing,
Or senseless speaking, or a speaking such
As sense cannot untie. Be what it is,
The action of my life is like it, which
I’ll keep, if but for sympathy.

 Enter Gaoler.

GAOLER.
Come, sir, are you ready for death?

POSTHUMUS.
Over-roasted rather; ready long ago.

GAOLER.
Hanging is the word, sir; if you be ready for that, you are well
cook’d.

POSTHUMUS.
So, if I prove a good repast to the spectators, the dish pays the shot.

GAOLER.
A heavy reckoning for you, sir. But the comfort is, you shall be called
to no more payments, fear no more tavern bills, which are often the
sadness of parting, as the procuring of mirth. You come in faint for
want of meat, depart reeling with too much drink; sorry that you have
paid too much, and sorry that you are paid too much; purse and brain
both empty; the brain the heavier for being too light, the purse too
light, being drawn of heaviness. O, of this contradiction you shall now
be quit. O, the charity of a penny cord! It sums up thousands in a
trice. You have no true debitor and creditor but it; of what’s past,
is, and to come, the discharge. Your neck, sir, is pen, book, and
counters; so the acquittance follows.

POSTHUMUS.
I am merrier to die than thou art to live.

GAOLER.
Indeed, sir, he that sleeps feels not the toothache. But a man that
were to sleep your sleep, and a hangman to help him to bed, I think he
would change places with his officer; for look you, sir, you know not
which way you shall go.

POSTHUMUS.
Yes indeed do I, fellow.

GAOLER.
Your death has eyes in’s head, then; I have not seen him so pictur’d.
You must either be directed by some that take upon them to know, or to
take upon yourself that which I am sure you do not know, or jump the
after-inquiry on your own peril. And how you shall speed in your
journey’s end, I think you’ll never return to tell one.

POSTHUMUS.
I tell thee, fellow, there are none want eyes to direct them the way I
am going, but such as wink and will not use them.

GAOLER.
What an infinite mock is this, that a man should have the best use of
eyes to see the way of blindness! I am sure hanging’s the way of
winking.

 Enter a Messenger.

MESSENGER.
Knock off his manacles; bring your prisoner to the King.

POSTHUMUS.
Thou bring’st good news: I am call’d to be made free.

GAOLER.
I’ll be hang’d then.

POSTHUMUS.
Thou shalt be then freer than a gaoler; no bolts for the dead.

 [_Exeunt Posthumus and Messenger._]

GAOLER.
Unless a man would marry a gallows and beget young gibbets, I never saw
one so prone. Yet, on my conscience, there are verier knaves desire to
live, for all he be a Roman; and there be some of them too that die
against their wills; so should I, if I were one. I would we were all of
one mind, and one mind good. O, there were desolation of gaolers and
gallowses! I speak against my present profit, but my wish hath a
preferment in’t.

 [_Exit._]

SCENE V. Britain. Cymbeline’s tent.

 Enter Cymbeline, Belarius, Guiderius, Arviragus, Pisanio, Lords,
 Officers and Attendants.

CYMBELINE.
Stand by my side, you whom the gods have made
Preservers of my throne. Woe is my heart
That the poor soldier that so richly fought,
Whose rags sham’d gilded arms, whose naked breast
Stepp’d before targes of proof, cannot be found.
He shall be happy that can find him, if
Our grace can make him so.

BELARIUS.
I never saw
Such noble fury in so poor a thing;
Such precious deeds in one that promis’d nought
But beggary and poor looks.

CYMBELINE.
No tidings of him?

PISANIO.
He hath been search’d among the dead and living,
But no trace of him.

CYMBELINE.
To my grief, I am
The heir of his reward, [_To Belarius, Guiderius, and Arviragus_] which
I will add
To you, the liver, heart, and brain of Britain,
By whom I grant she lives. ’Tis now the time
To ask of whence you are. Report it.

BELARIUS.
Sir,
In Cambria are we born, and gentlemen;
Further to boast were neither true nor modest,
Unless I add we are honest.

CYMBELINE.
Bow your knees.
Arise my knights o’ th’ battle; I create you
Companions to our person, and will fit you
With dignities becoming your estates.

 Enter Cornelius and Ladies.

There’s business in these faces. Why so sadly
Greet you our victory? You look like Romans,
And not o’ th’ court of Britain.

CORNELIUS.
Hail, great King!
To sour your happiness I must report
The Queen is dead.

CYMBELINE.
Who worse than a physician
Would this report become? But I consider
By med’cine life may be prolong’d, yet death
Will seize the doctor too. How ended she?

CORNELIUS.
With horror, madly dying, like her life;
Which, being cruel to the world, concluded
Most cruel to herself. What she confess’d
I will report, so please you; these her women
Can trip me if I err, who with wet cheeks
Were present when she finish’d.

CYMBELINE.
Prithee say.

CORNELIUS.
First, she confess’d she never lov’d you; only
Affected greatness got by you, not you;
Married your royalty, was wife to your place;
Abhorr’d your person.

CYMBELINE.
She alone knew this;
And but she spoke it dying, I would not
Believe her lips in opening it. Proceed.

CORNELIUS.
Your daughter, whom she bore in hand to love
With such integrity, she did confess
Was as a scorpion to her sight; whose life,
But that her flight prevented it, she had
Ta’en off by poison.

CYMBELINE.
O most delicate fiend!
Who is’t can read a woman? Is there more?

CORNELIUS.
More, sir, and worse. She did confess she had
For you a mortal mineral, which, being took,
Should by the minute feed on life, and ling’ring,
By inches waste you. In which time she purpos’d,
By watching, weeping, tendance, kissing, to
O’ercome you with her show; and in time,
When she had fitted you with her craft, to work
Her son into th’ adoption of the crown;
But failing of her end by his strange absence,
Grew shameless-desperate, open’d, in despite
Of heaven and men, her purposes, repented
The evils she hatch’d were not effected; so,
Despairing, died.

CYMBELINE.
Heard you all this, her women?

LADIES.
We did, so please your Highness.

CYMBELINE.
Mine eyes
Were not in fault, for she was beautiful;
Mine ears, that heard her flattery; nor my heart
That thought her like her seeming. It had been vicious
To have mistrusted her; yet, O my daughter!
That it was folly in me thou mayst say,
And prove it in thy feeling. Heaven mend all!

 Enter Lucius, Iachimo, the Soothsayer and other Roman prisoners,
 guarded; Posthumus behind, and Imogen.

Thou com’st not, Caius, now for tribute; that
The Britons have raz’d out, though with the loss
Of many a bold one, whose kinsmen have made suit
That their good souls may be appeas’d with slaughter
Of you their captives, which ourself have granted;
So think of your estate.

LUCIUS.
Consider, sir, the chance of war. The day
Was yours by accident; had it gone with us,
We should not, when the blood was cool, have threaten’d
Our prisoners with the sword. But since the gods
Will have it thus, that nothing but our lives
May be call’d ransom, let it come. Sufficeth
A Roman with a Roman’s heart can suffer.
Augustus lives to think on’t; and so much
For my peculiar care. This one thing only
I will entreat: my boy, a Briton born,
Let him be ransom’d. Never master had
A page so kind, so duteous, diligent,
So tender over his occasions, true,
So feat, so nurse-like; let his virtue join
With my request, which I’ll make bold your Highness
Cannot deny; he hath done no Briton harm
Though he have serv’d a Roman. Save him, sir,
And spare no blood beside.

CYMBELINE.
I have surely seen him;
His favour is familiar to me. Boy,
Thou hast look’d thyself into my grace,
And art mine own. I know not why, wherefore
To say “Live, boy.” Ne’er thank thy master. Live;
And ask of Cymbeline what boon thou wilt,
Fitting my bounty and thy state, I’ll give it;
Yea, though thou do demand a prisoner,
The noblest ta’en.

IMOGEN.
I humbly thank your Highness.

LUCIUS.
I do not bid thee beg my life, good lad,
And yet I know thou wilt.

IMOGEN.
No, no! Alack,
There’s other work in hand. I see a thing
Bitter to me as death; your life, good master,
Must shuffle for itself.

LUCIUS.
The boy disdains me,
He leaves me, scorns me. Briefly die their joys
That place them on the truth of girls and boys.
Why stands he so perplex’d?

CYMBELINE.
What wouldst thou, boy?
I love thee more and more; think more and more
What’s best to ask. Know’st him thou look’st on? Speak,
Wilt have him live? Is he thy kin? thy friend?

IMOGEN.
He is a Roman, no more kin to me
Than I to your Highness; who, being born your vassal,
Am something nearer.

CYMBELINE.
Wherefore ey’st him so?

IMOGEN.
I’ll tell you, sir, in private, if you please
To give me hearing.

CYMBELINE.
Ay, with all my heart,
And lend my best attention. What’s thy name?

IMOGEN.
Fidele, sir.

CYMBELINE.
Thou’rt my good youth, my page;
I’ll be thy master. Walk with me; speak freely.

 [_Cymbeline and Imogen converse apart._]

BELARIUS.
Is not this boy reviv’d from death?

ARVIRAGUS.
One sand another
Not more resembles that sweet rosy lad
Who died and was Fidele. What think you?

GUIDERIUS.
The same dead thing alive.

BELARIUS.
Peace, peace! see further. He eyes us not; forbear.
Creatures may be alike; were’t he, I am sure
He would have spoke to us.

GUIDERIUS.
But we see him dead.

BELARIUS.
Be silent; let’s see further.

PISANIO.
[_Aside._] It is my mistress.
Since she is living, let the time run on
To good or bad.

 [_Cymbeline and Imogen advance._]

CYMBELINE.
Come, stand thou by our side;
Make thy demand aloud. [_To Iachimo._] Sir, step you forth;
Give answer to this boy, and do it freely,
Or, by our greatness and the grace of it,
Which is our honour, bitter torture shall
Winnow the truth from falsehood. On, speak to him.

IMOGEN.
My boon is that this gentleman may render
Of whom he had this ring.

POSTHUMUS.
[_Aside._] What’s that to him?

CYMBELINE.
That diamond upon your finger, say
How came it yours?

IACHIMO.
Thou’lt torture me to leave unspoken that
Which to be spoke would torture thee.

CYMBELINE.
How? me?

IACHIMO.
I am glad to be constrain’d to utter that
Which torments me to conceal. By villainy
I got this ring; ’twas Leonatus’ jewel,
Whom thou didst banish; and—which more may grieve thee,
As it doth me—a nobler sir ne’er liv’d
’Twixt sky and ground. Wilt thou hear more, my lord?

CYMBELINE.
All that belongs to this.

IACHIMO.
That paragon, thy daughter,
For whom my heart drops blood and my false spirits
Quail to remember—Give me leave, I faint.

CYMBELINE.
My daughter? What of her? Renew thy strength;
I had rather thou shouldst live while nature will
Than die ere I hear more. Strive, man, and speak.

IACHIMO.
Upon a time, unhappy was the clock
That struck the hour: was in Rome, accurs’d
The mansion where: ’twas at a feast, O, would
Our viands had been poison’d (or at least
Those which I heav’d to head) the good Posthumus
(What should I say? he was too good to be
Where ill men were, and was the best of all
Amongst the rar’st of good ones) sitting sadly
Hearing us praise our loves of Italy
For beauty that made barren the swell’d boast
Of him that best could speak; for feature, laming
The shrine of Venus or straight-pight Minerva,
Postures beyond brief nature; for condition,
A shop of all the qualities that man
Loves woman for; besides that hook of wiving,
Fairness which strikes the eye.

CYMBELINE.
I stand on fire.
Come to the matter.

IACHIMO.
All too soon I shall,
Unless thou wouldst grieve quickly. This Posthumus,
Most like a noble lord in love and one
That had a royal lover, took his hint;
And (not dispraising whom we prais’d, therein
He was as calm as virtue) he began
His mistress’ picture; which by his tongue being made,
And then a mind put in’t, either our brags
Were crack’d of kitchen trulls, or his description
Prov’d us unspeaking sots.

CYMBELINE.
Nay, nay, to th’ purpose.

IACHIMO.
Your daughter’s chastity (there it begins)
He spake of her as Dian had hot dreams
And she alone were cold; whereat I, wretch,
Made scruple of his praise, and wager’d with him
Pieces of gold ’gainst this which then he wore
Upon his honour’d finger, to attain
In suit the place of’s bed, and win this ring
By hers and mine adultery. He, true knight,
No lesser of her honour confident
Than I did truly find her, stakes this ring;
And would so, had it been a carbuncle
Of Phoebus’ wheel; and might so safely, had it
Been all the worth of’s car. Away to Britain
Post I in this design. Well may you, sir,
Remember me at court, where I was taught
Of your chaste daughter the wide difference
’Twixt amorous and villainous. Being thus quench’d
Of hope, not longing, mine Italian brain
Gan in your duller Britain operate
Most vilely; for my vantage, excellent;
And, to be brief, my practice so prevail’d
That I return’d with simular proof enough
To make the noble Leonatus mad,
By wounding his belief in her renown
With tokens thus and thus; averring notes
Of chamber-hanging, pictures, this her bracelet
(O cunning, how I got it!) nay, some marks
Of secret on her person, that he could not
But think her bond of chastity quite crack’d,
I having ta’en the forfeit. Whereupon
Methinks I see him now—

POSTHUMUS.
[_Coming forward._] Ay, so thou dost,
Italian fiend! Ay me, most credulous fool,
Egregious murderer, thief, anything
That’s due to all the villains past, in being,
To come! O, give me cord, or knife, or poison,
Some upright justicer! Thou, King, send out
For torturers ingenious. It is I
That all th’ abhorred things o’ th’ earth amend
By being worse than they. I am Posthumus,
That kill’d thy daughter; villain-like, I lie;
That caus’d a lesser villain than myself,
A sacrilegious thief, to do’t. The temple
Of virtue was she; yea, and she herself.
Spit, and throw stones, cast mire upon me, set
The dogs o’ th’ street to bay me. Every villain
Be call’d Posthumus Leonatus, and
Be villainy less than ’twas! O Imogen!
My queen, my life, my wife! O Imogen,
Imogen, Imogen!

IMOGEN.
Peace, my lord. Hear, hear!

POSTHUMUS.
Shall’s have a play of this? Thou scornful page,
There lies thy part.

 [_Strikes her. She falls._]

PISANIO.
O gentlemen, help!
Mine and your mistress! O, my lord Posthumus!
You ne’er kill’d Imogen till now. Help, help!
Mine honour’d lady!

CYMBELINE.
Does the world go round?

POSTHUMUS.
How comes these staggers on me?

PISANIO.
Wake, my mistress!

CYMBELINE.
If this be so, the gods do mean to strike me
To death with mortal joy.

PISANIO.
How fares my mistress?

IMOGEN.
O, get thee from my sight;
Thou gav’st me poison. Dangerous fellow, hence!
Breathe not where princes are.

CYMBELINE.
The tune of Imogen!

PISANIO.
Lady,
The gods throw stones of sulphur on me, if
That box I gave you was not thought by me
A precious thing! I had it from the Queen.

CYMBELINE.
New matter still?

IMOGEN.
It poison’d me.

CORNELIUS.
O gods!
I left out one thing which the Queen confess’d,
Which must approve thee honest. ‘If Pisanio
Have’ said she ‘given his mistress that confection
Which I gave him for cordial, she is serv’d
As I would serve a rat.’

CYMBELINE.
What’s this, Cornelius?

CORNELIUS.
The Queen, sir, very oft importun’d me
To temper poisons for her; still pretending
The satisfaction of her knowledge only
In killing creatures vile, as cats and dogs,
Of no esteem. I, dreading that her purpose
Was of more danger, did compound for her
A certain stuff, which, being ta’en would cease
The present pow’r of life, but in short time
All offices of nature should again
Do their due functions. Have you ta’en of it?

IMOGEN.
Most like I did, for I was dead.

BELARIUS.
My boys,
There was our error.

GUIDERIUS.
This is sure Fidele.

IMOGEN.
Why did you throw your wedded lady from you?
Think that you are upon a rock, and now
Throw me again.

 [_Embracing him._]

POSTHUMUS.
Hang there like fruit, my soul,
Till the tree die!

CYMBELINE.
How now, my flesh? my child?
What, mak’st thou me a dullard in this act?
Wilt thou not speak to me?

IMOGEN.
[_Kneeling._] Your blessing, sir.

BELARIUS.
[_To Guiderius and Arviragus._] Though you did love this youth, I blame
ye not;
You had a motive for’t.

CYMBELINE.
My tears that fall
Prove holy water on thee! Imogen,
Thy mother’s dead.

IMOGEN.
I am sorry for’t, my lord.

CYMBELINE.
O, she was naught, and long of her it was
That we meet here so strangely; but her son
Is gone, we know not how nor where.

PISANIO.
My lord,
Now fear is from me, I’ll speak troth. Lord Cloten,
Upon my lady’s missing, came to me
With his sword drawn, foam’d at the mouth, and swore,
If I discover’d not which way she was gone,
It was my instant death. By accident
I had a feigned letter of my master’s
Then in my pocket, which directed him
To seek her on the mountains near to Milford;
Where, in a frenzy, in my master’s garments,
Which he enforc’d from me, away he posts
With unchaste purpose, and with oath to violate
My lady’s honour. What became of him
I further know not.

GUIDERIUS.
Let me end the story:
I slew him there.

CYMBELINE.
Marry, the gods forfend!
I would not thy good deeds should from my lips
Pluck a hard sentence. Prithee, valiant youth,
Deny’t again.

GUIDERIUS.
I have spoke it, and I did it.

CYMBELINE.
He was a prince.

GUIDERIUS.
A most incivil one. The wrongs he did me
Were nothing prince-like; for he did provoke me
With language that would make me spurn the sea,
If it could so roar to me. I cut off’s head,
And am right glad he is not standing here
To tell this tale of mine.

CYMBELINE.
I am sorry for thee.
By thine own tongue thou art condemn’d, and must
Endure our law. Thou’rt dead.

IMOGEN.
That headless man
I thought had been my lord.

CYMBELINE.
Bind the offender,
And take him from our presence.

BELARIUS.
Stay, sir King.
This man is better than the man he slew,
As well descended as thyself, and hath
More of thee merited than a band of Clotens
Had ever scar for. [_To the guard._] Let his arms alone;
They were not born for bondage.

CYMBELINE.
Why, old soldier,
Wilt thou undo the worth thou art unpaid for
By tasting of our wrath? How of descent
As good as we?

ARVIRAGUS.
In that he spake too far.

CYMBELINE.
And thou shalt die for’t.

BELARIUS.
We will die all three;
But I will prove that two on’s are as good
As I have given out him. My sons, I must
For mine own part unfold a dangerous speech,
Though haply well for you.

ARVIRAGUS.
Your danger’s ours.

GUIDERIUS.
And our good his.

BELARIUS.
Have at it then by leave!
Thou hadst, great King, a subject who
Was call’d Belarius.

CYMBELINE.
What of him? He is
A banish’d traitor.

BELARIUS.
He it is that hath
Assum’d this age; indeed a banish’d man;
I know not how a traitor.

CYMBELINE.
Take him hence,
The whole world shall not save him.

BELARIUS.
Not too hot.
First pay me for the nursing of thy sons,
And let it be confiscate all, so soon
As I have receiv’d it.

CYMBELINE.
Nursing of my sons?

BELARIUS.
I am too blunt and saucy: here’s my knee.
Ere I arise I will prefer my sons;
Then spare not the old father. Mighty sir,
These two young gentlemen that call me father,
And think they are my sons, are none of mine;
They are the issue of your loins, my liege,
And blood of your begetting.

CYMBELINE.
How? my issue?

BELARIUS.
So sure as you your father’s. I, old Morgan,
Am that Belarius whom you sometime banish’d.
Your pleasure was my mere offence, my punishment
Itself, and all my treason; that I suffer’d
Was all the harm I did. These gentle princes
(For such and so they are) these twenty years
Have I train’d up; those arts they have as I
Could put into them. My breeding was, sir, as
Your Highness knows. Their nurse, Euriphile,
Whom for the theft I wedded, stole these children
Upon my banishment; I mov’d her to’t,
Having receiv’d the punishment before
For that which I did then. Beaten for loyalty
Excited me to treason. Their dear loss,
The more of you ’twas felt, the more it shap’d
Unto my end of stealing them. But, gracious sir,
Here are your sons again, and I must lose
Two of the sweet’st companions in the world.
The benediction of these covering heavens
Fall on their heads like dew! for they are worthy
To inlay heaven with stars.

CYMBELINE.
Thou weep’st and speak’st.
The service that you three have done is more
Unlike than this thou tell’st. I lost my children.
If these be they, I know not how to wish
A pair of worthier sons.

BELARIUS.
Be pleas’d awhile.
This gentleman, whom I call Polydore,
Most worthy prince, as yours, is true Guiderius;
This gentleman, my Cadwal, Arviragus,
Your younger princely son; he, sir, was lapp’d
In a most curious mantle, wrought by th’ hand
Of his queen mother, which for more probation
I can with ease produce.

CYMBELINE.
Guiderius had
Upon his neck a mole, a sanguine star;
It was a mark of wonder.

BELARIUS.
This is he,
Who hath upon him still that natural stamp.
It was wise nature’s end in the donation,
To be his evidence now.

CYMBELINE.
O, what am I?
A mother to the birth of three? Ne’er mother
Rejoic’d deliverance more. Blest pray you be,
That, after this strange starting from your orbs,
You may reign in them now! O Imogen,
Thou hast lost by this a kingdom.

IMOGEN.
No, my lord;
I have got two worlds by’t. O my gentle brothers,
Have we thus met? O, never say hereafter
But I am truest speaker! You call’d me brother,
When I was but your sister: I you brothers,
When we were so indeed.

CYMBELINE.
Did you e’er meet?

ARVIRAGUS.
Ay, my good lord.

GUIDERIUS.
And at first meeting lov’d,
Continu’d so until we thought he died.

CORNELIUS.
By the Queen’s dram she swallow’d.

CYMBELINE.
O rare instinct!
When shall I hear all through? This fierce abridgement
Hath to it circumstantial branches, which
Distinction should be rich in. Where? how liv’d you?
And when came you to serve our Roman captive?
How parted with your brothers? how first met them?
Why fled you from the court? and whither? These,
And your three motives to the battle, with
I know not how much more, should be demanded,
And all the other by-dependances,
From chance to chance; but nor the time nor place
Will serve our long interrogatories. See,
Posthumus anchors upon Imogen;
And she, like harmless lightning, throws her eye
On him, her brothers, me, her master, hitting
Each object with a joy; the counterchange
Is severally in all. Let’s quit this ground,
And smoke the temple with our sacrifices.
[_To Belarius._] Thou art my brother; so we’ll hold thee ever.

IMOGEN.
You are my father too, and did relieve me
To see this gracious season.

CYMBELINE.
All o’erjoy’d
Save these in bonds. Let them be joyful too,
For they shall taste our comfort.

IMOGEN.
My good master,
I will yet do you service.

LUCIUS.
Happy be you!

CYMBELINE.
The forlorn soldier, that so nobly fought,
He would have well becom’d this place and grac’d
The thankings of a king.

POSTHUMUS.
I am, sir,
The soldier that did company these three
In poor beseeming; ’twas a fitment for
The purpose I then follow’d. That I was he,
Speak, Iachimo. I had you down, and might
Have made you finish.

IACHIMO.
[_Kneeling._] I am down again;
But now my heavy conscience sinks my knee,
As then your force did. Take that life, beseech you,
Which I so often owe; but your ring first,
And here the bracelet of the truest princess
That ever swore her faith.

POSTHUMUS.
Kneel not to me.
The pow’r that I have on you is to spare you;
The malice towards you to forgive you. Live,
And deal with others better.

CYMBELINE.
Nobly doom’d!
We’ll learn our freeness of a son-in-law;
Pardon’s the word to all.

ARVIRAGUS.
You holp us, sir,
As you did mean indeed to be our brother;
Joy’d are we that you are.

POSTHUMUS.
Your servant, Princes. Good my lord of Rome,
Call forth your soothsayer. As I slept, methought
Great Jupiter, upon his eagle back’d,
Appear’d to me, with other spritely shows
Of mine own kindred. When I wak’d, I found
This label on my bosom; whose containing
Is so from sense in hardness that I can
Make no collection of it. Let him show
His skill in the construction.

LUCIUS.
Philarmonus!

SOOTHSAYER.
Here, my good lord.

LUCIUS.
Read, and declare the meaning.

SOOTHSAYER.
[_Reads._] _When as a lion’s whelp shall, to himself unknown, without
seeking find, and be embrac’d by a piece of tender air; and when from a
stately cedar shall be lopp’d branches which, being dead many years,
shall after revive, be jointed to the old stock, and freshly grow; then
shall Posthumus end his miseries, Britain be fortunate and flourish in
peace and plenty._
Thou, Leonatus, art the lion’s whelp;
The fit and apt construction of thy name,
Being Leo-natus, doth import so much.
[_To Cymbeline_] The piece of tender air, thy virtuous daughter,
Which we call _mollis aer_, and _mollis aer_
We term it _mulier_; which _mulier_ I divine
Is this most constant wife, who even now
Answering the letter of the oracle,
Unknown to you, unsought, were clipp’d about
With this most tender air.

CYMBELINE.
This hath some seeming.

SOOTHSAYER.
The lofty cedar, royal Cymbeline,
Personates thee; and thy lopp’d branches point
Thy two sons forth, who, by Belarius stol’n,
For many years thought dead, are now reviv’d,
To the majestic cedar join’d, whose issue
Promises Britain peace and plenty.

CYMBELINE.
Well,
My peace we will begin. And, Caius Lucius,
Although the victor, we submit to Cæsar
And to the Roman empire, promising
To pay our wonted tribute, from the which
We were dissuaded by our wicked queen,
Whom heavens in justice, both on her and hers,
Have laid most heavy hand.

SOOTHSAYER.
The fingers of the pow’rs above do tune
The harmony of this peace. The vision
Which I made known to Lucius ere the stroke
Of yet this scarce-cold battle, at this instant
Is full accomplish’d; for the Roman eagle,
From south to west on wing soaring aloft,
Lessen’d herself and in the beams o’ th’ sun
So vanish’d; which foreshow’d our princely eagle,
Th’ imperial Cæsar, should again unite
His favour with the radiant Cymbeline,
Which shines here in the west.

CYMBELINE.
Laud we the gods;
And let our crooked smokes climb to their nostrils
From our bless’d altars. Publish we this peace
To all our subjects. Set we forward; let
A Roman and a British ensign wave
Friendly together. So through Lud’s Town march;
And in the temple of great Jupiter
Our peace we’ll ratify; seal it with feasts.
Set on there! Never was a war did cease,
Ere bloody hands were wash’d, with such a peace.

 [_Exeunt._]




THE TRAGEDY OF HAMLET, PRINCE OF DENMARK




Contents

 ACT I
 Scene I. Elsinore. A platform before the Castle
 Scene II. Elsinore. A room of state in the Castle
 Scene III. A room in Polonius’s house
 Scene IV. The platform
 Scene V. A more remote part of the Castle

 ACT II
 Scene I. A room in Polonius’s house
 Scene II. A room in the Castle

 ACT III
 Scene I. A room in the Castle
 Scene II. A hall in the Castle
 Scene III. A room in the Castle
 Scene IV. Another room in the Castle

 ACT IV
 Scene I. A room in the Castle
 Scene II. Another room in the Castle
 Scene III. Another room in the Castle
 Scene IV. A plain in Denmark
 Scene V. Elsinore. A room in the Castle
 Scene VI. Another room in the Castle
 Scene VII. Another room in the Castle

 ACT V
 Scene I. A churchyard
 Scene II. A hall in the Castle




Dramatis Personæ

HAMLET, Prince of Denmark
CLAUDIUS, King of Denmark, Hamlet’s uncle
The GHOST of the late king, Hamlet’s father
GERTRUDE, the Queen, Hamlet’s mother, now wife of Claudius
POLONIUS, Lord Chamberlain
LAERTES, Son to Polonius
OPHELIA, Daughter to Polonius
HORATIO, Friend to Hamlet
FORTINBRAS, Prince of Norway
VOLTEMAND, Courtier
CORNELIUS, Courtier
ROSENCRANTZ, Courtier
GUILDENSTERN, Courtier
MARCELLUS, Officer
BARNARDO, Officer
FRANCISCO, a Soldier
OSRIC, Courtier
REYNALDO, Servant to Polonius
Players
A Gentleman, Courtier
A Priest
Two Clowns, Grave-diggers
A Captain
English Ambassadors.
Lords, Ladies, Officers, Soldiers, Sailors, Messengers, and Attendants

SCENE. Elsinore.




ACT I

SCENE I. Elsinore. A platform before the Castle.


Enter Francisco and Barnardo, two sentinels.

BARNARDO.
Who’s there?

FRANCISCO.
Nay, answer me. Stand and unfold yourself.

BARNARDO.
Long live the King!

FRANCISCO.
Barnardo?

BARNARDO.
He.

FRANCISCO.
You come most carefully upon your hour.

BARNARDO.
’Tis now struck twelve. Get thee to bed, Francisco.

FRANCISCO.
For this relief much thanks. ’Tis bitter cold,
And I am sick at heart.

BARNARDO.
Have you had quiet guard?

FRANCISCO.
Not a mouse stirring.

BARNARDO.
Well, good night.
If you do meet Horatio and Marcellus,
The rivals of my watch, bid them make haste.

Enter Horatio and Marcellus.

FRANCISCO.
I think I hear them. Stand, ho! Who is there?

HORATIO.
Friends to this ground.

MARCELLUS.
And liegemen to the Dane.

FRANCISCO.
Give you good night.

MARCELLUS.
O, farewell, honest soldier, who hath reliev’d you?

FRANCISCO.
Barnardo has my place. Give you good-night.

[_Exit._]

MARCELLUS.
Holla, Barnardo!

BARNARDO.
Say, what, is Horatio there?

HORATIO.
A piece of him.

BARNARDO.
Welcome, Horatio. Welcome, good Marcellus.

MARCELLUS.
What, has this thing appear’d again tonight?

BARNARDO.
I have seen nothing.

MARCELLUS.
Horatio says ’tis but our fantasy,
And will not let belief take hold of him
Touching this dreaded sight, twice seen of us.
Therefore I have entreated him along
With us to watch the minutes of this night,
That if again this apparition come
He may approve our eyes and speak to it.

HORATIO.
Tush, tush, ’twill not appear.

BARNARDO.
Sit down awhile,
And let us once again assail your ears,
That are so fortified against our story,
What we two nights have seen.

HORATIO.
Well, sit we down,
And let us hear Barnardo speak of this.

BARNARDO.
Last night of all,
When yond same star that’s westward from the pole,
Had made his course t’illume that part of heaven
Where now it burns, Marcellus and myself,
The bell then beating one—

MARCELLUS.
Peace, break thee off. Look where it comes again.

Enter Ghost.

BARNARDO.
In the same figure, like the King that’s dead.

MARCELLUS.
Thou art a scholar; speak to it, Horatio.

BARNARDO.
Looks it not like the King? Mark it, Horatio.

HORATIO.
Most like. It harrows me with fear and wonder.

BARNARDO
It would be spoke to.

MARCELLUS.
Question it, Horatio.

HORATIO.
What art thou that usurp’st this time of night,
Together with that fair and warlike form
In which the majesty of buried Denmark
Did sometimes march? By heaven I charge thee speak.

MARCELLUS.
It is offended.

BARNARDO.
See, it stalks away.

HORATIO.
Stay! speak, speak! I charge thee speak!

[_Exit Ghost._]

MARCELLUS.
’Tis gone, and will not answer.

BARNARDO.
How now, Horatio! You tremble and look pale.
Is not this something more than fantasy?
What think you on’t?

HORATIO.
Before my God, I might not this believe
Without the sensible and true avouch
Of mine own eyes.

MARCELLUS.
Is it not like the King?

HORATIO.
As thou art to thyself:
Such was the very armour he had on
When he th’ambitious Norway combated;
So frown’d he once, when in an angry parle
He smote the sledded Polacks on the ice.
’Tis strange.

MARCELLUS.
Thus twice before, and jump at this dead hour,
With martial stalk hath he gone by our watch.

HORATIO.
In what particular thought to work I know not;
But in the gross and scope of my opinion,
This bodes some strange eruption to our state.

MARCELLUS.
Good now, sit down, and tell me, he that knows,
Why this same strict and most observant watch
So nightly toils the subject of the land,
And why such daily cast of brazen cannon
And foreign mart for implements of war;
Why such impress of shipwrights, whose sore task
Does not divide the Sunday from the week.
What might be toward, that this sweaty haste
Doth make the night joint-labourer with the day:
Who is’t that can inform me?

HORATIO.
That can I;
At least, the whisper goes so. Our last King,
Whose image even but now appear’d to us,
Was, as you know, by Fortinbras of Norway,
Thereto prick’d on by a most emulate pride,
Dar’d to the combat; in which our valiant Hamlet,
For so this side of our known world esteem’d him,
Did slay this Fortinbras; who by a seal’d compact,
Well ratified by law and heraldry,
Did forfeit, with his life, all those his lands
Which he stood seiz’d of, to the conqueror;
Against the which, a moiety competent
Was gaged by our King; which had return’d
To the inheritance of Fortinbras,
Had he been vanquisher; as by the same cov’nant
And carriage of the article design’d,
His fell to Hamlet. Now, sir, young Fortinbras,
Of unimproved mettle, hot and full,
Hath in the skirts of Norway, here and there,
Shark’d up a list of lawless resolutes,
For food and diet, to some enterprise
That hath a stomach in’t; which is no other,
As it doth well appear unto our state,
But to recover of us by strong hand
And terms compulsatory, those foresaid lands
So by his father lost. And this, I take it,
Is the main motive of our preparations,
The source of this our watch, and the chief head
Of this post-haste and rummage in the land.

BARNARDO.
I think it be no other but e’en so:
Well may it sort that this portentous figure
Comes armed through our watch so like the King
That was and is the question of these wars.

HORATIO.
A mote it is to trouble the mind’s eye.
In the most high and palmy state of Rome,
A little ere the mightiest Julius fell,
The graves stood tenantless and the sheeted dead
Did squeak and gibber in the Roman streets;
As stars with trains of fire and dews of blood,
Disasters in the sun; and the moist star,
Upon whose influence Neptune’s empire stands,
Was sick almost to doomsday with eclipse.
And even the like precurse of fierce events,
As harbingers preceding still the fates
And prologue to the omen coming on,
Have heaven and earth together demonstrated
Unto our climatures and countrymen.

Re-enter Ghost.

But, soft, behold! Lo, where it comes again!
I’ll cross it, though it blast me. Stay, illusion!
If thou hast any sound, or use of voice,
Speak to me.
If there be any good thing to be done,
That may to thee do ease, and grace to me,
Speak to me.
If thou art privy to thy country’s fate,
Which, happily, foreknowing may avoid,
O speak!
Or if thou hast uphoarded in thy life
Extorted treasure in the womb of earth,
For which, they say, you spirits oft walk in death,
Speak of it. Stay, and speak!

[_The cock crows._]

Stop it, Marcellus!

MARCELLUS.
Shall I strike at it with my partisan?

HORATIO.
Do, if it will not stand.

BARNARDO.
’Tis here!

HORATIO.
’Tis here!

[_Exit Ghost._]

MARCELLUS.
’Tis gone!
We do it wrong, being so majestical,
To offer it the show of violence,
For it is as the air, invulnerable,
And our vain blows malicious mockery.

BARNARDO.
It was about to speak, when the cock crew.

HORATIO.
And then it started, like a guilty thing
Upon a fearful summons. I have heard
The cock, that is the trumpet to the morn,
Doth with his lofty and shrill-sounding throat
Awake the god of day; and at his warning,
Whether in sea or fire, in earth or air,
Th’extravagant and erring spirit hies
To his confine. And of the truth herein
This present object made probation.

MARCELLUS.
It faded on the crowing of the cock.
Some say that ever ’gainst that season comes
Wherein our Saviour’s birth is celebrated,
The bird of dawning singeth all night long;
And then, they say, no spirit dare stir abroad,
The nights are wholesome, then no planets strike,
No fairy takes, nor witch hath power to charm;
So hallow’d and so gracious is the time.

HORATIO.
So have I heard, and do in part believe it.
But look, the morn in russet mantle clad,
Walks o’er the dew of yon high eastward hill.
Break we our watch up, and by my advice,
Let us impart what we have seen tonight
Unto young Hamlet; for upon my life,
This spirit, dumb to us, will speak to him.
Do you consent we shall acquaint him with it,
As needful in our loves, fitting our duty?

MARCELLUS.
Let’s do’t, I pray, and I this morning know
Where we shall find him most conveniently.

[_Exeunt._]

 SCENE II. Elsinore. A room of state in the Castle.

Enter Claudius King of Denmark, Gertrude the Queen, Hamlet, Polonius,
Laertes, Voltemand,
Cornelius, Lords and Attendant.

KING.
Though yet of Hamlet our dear brother’s death
The memory be green, and that it us befitted
To bear our hearts in grief, and our whole kingdom
To be contracted in one brow of woe;
Yet so far hath discretion fought with nature
That we with wisest sorrow think on him,
Together with remembrance of ourselves.
Therefore our sometime sister, now our queen,
Th’imperial jointress to this warlike state,
Have we, as ’twere with a defeated joy,
With one auspicious and one dropping eye,
With mirth in funeral, and with dirge in marriage,
In equal scale weighing delight and dole,
Taken to wife; nor have we herein barr’d
Your better wisdoms, which have freely gone
With this affair along. For all, our thanks.
Now follows, that you know young Fortinbras,
Holding a weak supposal of our worth,
Or thinking by our late dear brother’s death
Our state to be disjoint and out of frame,
Colleagued with this dream of his advantage,
He hath not fail’d to pester us with message,
Importing the surrender of those lands
Lost by his father, with all bonds of law,
To our most valiant brother. So much for him.
Now for ourself and for this time of meeting:
Thus much the business is: we have here writ
To Norway, uncle of young Fortinbras,
Who, impotent and bed-rid, scarcely hears
Of this his nephew’s purpose, to suppress
His further gait herein; in that the levies,
The lists, and full proportions are all made
Out of his subject: and we here dispatch
You, good Cornelius, and you, Voltemand,
For bearers of this greeting to old Norway,
Giving to you no further personal power
To business with the King, more than the scope
Of these dilated articles allow.
Farewell; and let your haste commend your duty.

CORNELIUS and VOLTEMAND.
In that, and all things, will we show our duty.

KING.
We doubt it nothing: heartily farewell.

[_Exeunt Voltemand and Cornelius._]

And now, Laertes, what’s the news with you?
You told us of some suit. What is’t, Laertes?
You cannot speak of reason to the Dane,
And lose your voice. What wouldst thou beg, Laertes,
That shall not be my offer, not thy asking?
The head is not more native to the heart,
The hand more instrumental to the mouth,
Than is the throne of Denmark to thy father.
What wouldst thou have, Laertes?

LAERTES.
Dread my lord,
Your leave and favour to return to France,
From whence though willingly I came to Denmark
To show my duty in your coronation;
Yet now I must confess, that duty done,
My thoughts and wishes bend again toward France,
And bow them to your gracious leave and pardon.

KING.
Have you your father’s leave? What says Polonius?

POLONIUS.
He hath, my lord, wrung from me my slow leave
By laboursome petition; and at last
Upon his will I seal’d my hard consent.
I do beseech you give him leave to go.

KING.
Take thy fair hour, Laertes; time be thine,
And thy best graces spend it at thy will!
But now, my cousin Hamlet, and my son—

HAMLET.
[_Aside._] A little more than kin, and less than kind.

KING.
How is it that the clouds still hang on you?

HAMLET.
Not so, my lord, I am too much i’ the sun.

QUEEN.
Good Hamlet, cast thy nighted colour off,
And let thine eye look like a friend on Denmark.
Do not for ever with thy vailed lids
Seek for thy noble father in the dust.
Thou know’st ’tis common, all that lives must die,
Passing through nature to eternity.

HAMLET.
Ay, madam, it is common.

QUEEN.
If it be,
Why seems it so particular with thee?

HAMLET.
Seems, madam! Nay, it is; I know not seems.
’Tis not alone my inky cloak, good mother,
Nor customary suits of solemn black,
Nor windy suspiration of forc’d breath,
No, nor the fruitful river in the eye,
Nor the dejected haviour of the visage,
Together with all forms, moods, shows of grief,
That can denote me truly. These indeed seem,
For they are actions that a man might play;
But I have that within which passeth show;
These but the trappings and the suits of woe.

KING.
’Tis sweet and commendable in your nature, Hamlet,
To give these mourning duties to your father;
But you must know, your father lost a father,
That father lost, lost his, and the survivor bound
In filial obligation, for some term
To do obsequious sorrow. But to persevere
In obstinate condolement is a course
Of impious stubbornness. ’Tis unmanly grief,
It shows a will most incorrect to heaven,
A heart unfortified, a mind impatient,
An understanding simple and unschool’d;
For what we know must be, and is as common
As any the most vulgar thing to sense,
Why should we in our peevish opposition
Take it to heart? Fie, ’tis a fault to heaven,
A fault against the dead, a fault to nature,
To reason most absurd, whose common theme
Is death of fathers, and who still hath cried,
From the first corse till he that died today,
‘This must be so.’ We pray you throw to earth
This unprevailing woe, and think of us
As of a father; for let the world take note
You are the most immediate to our throne,
And with no less nobility of love
Than that which dearest father bears his son
Do I impart toward you. For your intent
In going back to school in Wittenberg,
It is most retrograde to our desire:
And we beseech you bend you to remain
Here in the cheer and comfort of our eye,
Our chiefest courtier, cousin, and our son.

QUEEN.
Let not thy mother lose her prayers, Hamlet.
I pray thee stay with us; go not to Wittenberg.

HAMLET.
I shall in all my best obey you, madam.

KING.
Why, ’tis a loving and a fair reply.
Be as ourself in Denmark. Madam, come;
This gentle and unforc’d accord of Hamlet
Sits smiling to my heart; in grace whereof,
No jocund health that Denmark drinks today
But the great cannon to the clouds shall tell,
And the King’s rouse the heaven shall bruit again,
Re-speaking earthly thunder. Come away.

[_Exeunt all but Hamlet._]

HAMLET.
O that this too too solid flesh would melt,
Thaw, and resolve itself into a dew!
Or that the Everlasting had not fix’d
His canon ’gainst self-slaughter. O God! O God!
How weary, stale, flat, and unprofitable
Seem to me all the uses of this world!
Fie on’t! Oh fie! ’tis an unweeded garden
That grows to seed; things rank and gross in nature
Possess it merely. That it should come to this!
But two months dead—nay, not so much, not two:
So excellent a king; that was to this
Hyperion to a satyr; so loving to my mother,
That he might not beteem the winds of heaven
Visit her face too roughly. Heaven and earth!
Must I remember? Why, she would hang on him
As if increase of appetite had grown
By what it fed on; and yet, within a month—
Let me not think on’t—Frailty, thy name is woman!
A little month, or ere those shoes were old
With which she followed my poor father’s body
Like Niobe, all tears.—Why she, even she—
O God! A beast that wants discourse of reason
Would have mourn’d longer,—married with mine uncle,
My father’s brother; but no more like my father
Than I to Hercules. Within a month,
Ere yet the salt of most unrighteous tears
Had left the flushing in her galled eyes,
She married. O most wicked speed, to post
With such dexterity to incestuous sheets!
It is not, nor it cannot come to good.
But break, my heart, for I must hold my tongue.

Enter Horatio, Marcellus and Barnardo.

HORATIO.
Hail to your lordship!

HAMLET.
I am glad to see you well:
Horatio, or I do forget myself.

HORATIO.
The same, my lord,
And your poor servant ever.

HAMLET.
Sir, my good friend;
I’ll change that name with you:
And what make you from Wittenberg, Horatio?—
Marcellus?

MARCELLUS.
My good lord.

HAMLET.
I am very glad to see you.—Good even, sir.—
But what, in faith, make you from Wittenberg?

HORATIO.
A truant disposition, good my lord.

HAMLET.
I would not hear your enemy say so;
Nor shall you do my ear that violence,
To make it truster of your own report
Against yourself. I know you are no truant.
But what is your affair in Elsinore?
We’ll teach you to drink deep ere you depart.

HORATIO.
My lord, I came to see your father’s funeral.

HAMLET.
I prithee do not mock me, fellow-student.
I think it was to see my mother’s wedding.

HORATIO.
Indeed, my lord, it follow’d hard upon.

HAMLET.
Thrift, thrift, Horatio! The funeral bak’d meats
Did coldly furnish forth the marriage tables.
Would I had met my dearest foe in heaven
Or ever I had seen that day, Horatio.
My father,—methinks I see my father.

HORATIO.
Where, my lord?

HAMLET.
In my mind’s eye, Horatio.

HORATIO.
I saw him once; he was a goodly king.

HAMLET.
He was a man, take him for all in all,
I shall not look upon his like again.

HORATIO.
My lord, I think I saw him yesternight.

HAMLET.
Saw? Who?

HORATIO.
My lord, the King your father.

HAMLET.
The King my father!

HORATIO.
Season your admiration for a while
With an attent ear, till I may deliver
Upon the witness of these gentlemen
This marvel to you.

HAMLET.
For God’s love let me hear.

HORATIO.
Two nights together had these gentlemen,
Marcellus and Barnardo, on their watch
In the dead waste and middle of the night,
Been thus encounter’d. A figure like your father,
Armed at point exactly, cap-à-pie,
Appears before them, and with solemn march
Goes slow and stately by them: thrice he walk’d
By their oppress’d and fear-surprised eyes,
Within his truncheon’s length; whilst they, distill’d
Almost to jelly with the act of fear,
Stand dumb, and speak not to him. This to me
In dreadful secrecy impart they did,
And I with them the third night kept the watch,
Where, as they had deliver’d, both in time,
Form of the thing, each word made true and good,
The apparition comes. I knew your father;
These hands are not more like.

HAMLET.
But where was this?

MARCELLUS.
My lord, upon the platform where we watch.

HAMLET.
Did you not speak to it?

HORATIO.
My lord, I did;
But answer made it none: yet once methought
It lifted up it head, and did address
Itself to motion, like as it would speak.
But even then the morning cock crew loud,
And at the sound it shrunk in haste away,
And vanish’d from our sight.

HAMLET.
’Tis very strange.

HORATIO.
As I do live, my honour’d lord, ’tis true;
And we did think it writ down in our duty
To let you know of it.

HAMLET.
Indeed, indeed, sirs, but this troubles me.
Hold you the watch tonight?

MARCELLUS and BARNARDO.
We do, my lord.

HAMLET.
Arm’d, say you?

Both.
Arm’d, my lord.

HAMLET.
From top to toe?

BOTH.
My lord, from head to foot.

HAMLET.
Then saw you not his face?

HORATIO.
O yes, my lord, he wore his beaver up.

HAMLET.
What, look’d he frowningly?

HORATIO.
A countenance more in sorrow than in anger.

HAMLET.
Pale, or red?

HORATIO.
Nay, very pale.

HAMLET.
And fix’d his eyes upon you?

HORATIO.
Most constantly.

HAMLET.
I would I had been there.

HORATIO.
It would have much amaz’d you.

HAMLET.
Very like, very like. Stay’d it long?

HORATIO.
While one with moderate haste might tell a hundred.

MARCELLUS and BARNARDO.
Longer, longer.

HORATIO.
Not when I saw’t.

HAMLET.
His beard was grizzled, no?

HORATIO.
It was, as I have seen it in his life,
A sable silver’d.

HAMLET.
I will watch tonight;
Perchance ’twill walk again.

HORATIO.
I warrant you it will.

HAMLET.
If it assume my noble father’s person,
I’ll speak to it, though hell itself should gape
And bid me hold my peace. I pray you all,
If you have hitherto conceal’d this sight,
Let it be tenable in your silence still;
And whatsoever else shall hap tonight,
Give it an understanding, but no tongue.
I will requite your loves. So, fare ye well.
Upon the platform ’twixt eleven and twelve,
I’ll visit you.

ALL.
Our duty to your honour.

HAMLET.
Your loves, as mine to you: farewell.

[_Exeunt Horatio, Marcellus and Barnardo._]

My father’s spirit in arms! All is not well;
I doubt some foul play: would the night were come!
Till then sit still, my soul: foul deeds will rise,
Though all the earth o’erwhelm them, to men’s eyes.

[_Exit._]

 SCENE III. A room in Polonius’s house.

Enter Laertes and Ophelia.

LAERTES.
My necessaries are embark’d. Farewell.
And, sister, as the winds give benefit
And convoy is assistant, do not sleep,
But let me hear from you.

OPHELIA.
Do you doubt that?

LAERTES.
For Hamlet, and the trifling of his favour,
Hold it a fashion and a toy in blood;
A violet in the youth of primy nature,
Forward, not permanent, sweet, not lasting;
The perfume and suppliance of a minute;
No more.

OPHELIA.
No more but so?

LAERTES.
Think it no more.
For nature crescent does not grow alone
In thews and bulk; but as this temple waxes,
The inward service of the mind and soul
Grows wide withal. Perhaps he loves you now,
And now no soil nor cautel doth besmirch
The virtue of his will; but you must fear,
His greatness weigh’d, his will is not his own;
For he himself is subject to his birth:
He may not, as unvalu’d persons do,
Carve for himself; for on his choice depends
The sanctity and health of this whole state;
And therefore must his choice be circumscrib’d
Unto the voice and yielding of that body
Whereof he is the head. Then if he says he loves you,
It fits your wisdom so far to believe it
As he in his particular act and place
May give his saying deed; which is no further
Than the main voice of Denmark goes withal.
Then weigh what loss your honour may sustain
If with too credent ear you list his songs,
Or lose your heart, or your chaste treasure open
To his unmaster’d importunity.
Fear it, Ophelia, fear it, my dear sister;
And keep you in the rear of your affection,
Out of the shot and danger of desire.
The chariest maid is prodigal enough
If she unmask her beauty to the moon.
Virtue itself ’scapes not calumnious strokes:
The canker galls the infants of the spring
Too oft before their buttons be disclos’d,
And in the morn and liquid dew of youth
Contagious blastments are most imminent.
Be wary then, best safety lies in fear.
Youth to itself rebels, though none else near.

OPHELIA.
I shall th’effect of this good lesson keep
As watchman to my heart. But good my brother,
Do not as some ungracious pastors do,
Show me the steep and thorny way to heaven;
Whilst like a puff’d and reckless libertine
Himself the primrose path of dalliance treads,
And recks not his own rede.

LAERTES.
O, fear me not.
I stay too long. But here my father comes.

Enter Polonius.

A double blessing is a double grace;
Occasion smiles upon a second leave.

POLONIUS.
Yet here, Laertes? Aboard, aboard, for shame.
The wind sits in the shoulder of your sail,
And you are stay’d for. There, my blessing with you.

[_Laying his hand on Laertes’s head._]

And these few precepts in thy memory
Look thou character. Give thy thoughts no tongue,
Nor any unproportion’d thought his act.
Be thou familiar, but by no means vulgar.
Those friends thou hast, and their adoption tried,
Grapple them unto thy soul with hoops of steel;
But do not dull thy palm with entertainment
Of each new-hatch’d, unfledg’d comrade. Beware
Of entrance to a quarrel; but being in,
Bear’t that th’opposed may beware of thee.
Give every man thine ear, but few thy voice:
Take each man’s censure, but reserve thy judgement.
Costly thy habit as thy purse can buy,
But not express’d in fancy; rich, not gaudy:
For the apparel oft proclaims the man;
And they in France of the best rank and station
Are of a most select and generous chief in that.
Neither a borrower nor a lender be:
For loan oft loses both itself and friend;
And borrowing dulls the edge of husbandry.
This above all: to thine own self be true;
And it must follow, as the night the day,
Thou canst not then be false to any man.
Farewell: my blessing season this in thee.

LAERTES.
Most humbly do I take my leave, my lord.

POLONIUS.
The time invites you; go, your servants tend.

LAERTES.
Farewell, Ophelia, and remember well
What I have said to you.

OPHELIA.
’Tis in my memory lock’d,
And you yourself shall keep the key of it.

LAERTES.
Farewell.

[_Exit._]

POLONIUS.
What is’t, Ophelia, he hath said to you?

OPHELIA.
So please you, something touching the Lord Hamlet.

POLONIUS.
Marry, well bethought:
’Tis told me he hath very oft of late
Given private time to you; and you yourself
Have of your audience been most free and bounteous.
If it be so,—as so ’tis put on me,
And that in way of caution,—I must tell you
You do not understand yourself so clearly
As it behoves my daughter and your honour.
What is between you? Give me up the truth.

OPHELIA.
He hath, my lord, of late made many tenders
Of his affection to me.

POLONIUS.
Affection! Pooh! You speak like a green girl,
Unsifted in such perilous circumstance.
Do you believe his tenders, as you call them?

OPHELIA.
I do not know, my lord, what I should think.

POLONIUS.
Marry, I’ll teach you; think yourself a baby;
That you have ta’en these tenders for true pay,
Which are not sterling. Tender yourself more dearly;
Or,—not to crack the wind of the poor phrase,
Running it thus,—you’ll tender me a fool.

OPHELIA.
My lord, he hath importun’d me with love
In honourable fashion.

POLONIUS.
Ay, fashion you may call it; go to, go to.

OPHELIA.
And hath given countenance to his speech, my lord,
With almost all the holy vows of heaven.

POLONIUS.
Ay, springes to catch woodcocks. I do know,
When the blood burns, how prodigal the soul
Lends the tongue vows: these blazes, daughter,
Giving more light than heat, extinct in both,
Even in their promise, as it is a-making,
You must not take for fire. From this time
Be something scanter of your maiden presence;
Set your entreatments at a higher rate
Than a command to parley. For Lord Hamlet,
Believe so much in him that he is young;
And with a larger tether may he walk
Than may be given you. In few, Ophelia,
Do not believe his vows; for they are brokers,
Not of that dye which their investments show,
But mere implorators of unholy suits,
Breathing like sanctified and pious bawds,
The better to beguile. This is for all:
I would not, in plain terms, from this time forth
Have you so slander any moment leisure
As to give words or talk with the Lord Hamlet.
Look to’t, I charge you; come your ways.

OPHELIA.
I shall obey, my lord.

[_Exeunt._]

 SCENE IV. The platform.

Enter Hamlet, Horatio and Marcellus.

HAMLET.
The air bites shrewdly; it is very cold.

HORATIO.
It is a nipping and an eager air.

HAMLET.
What hour now?

HORATIO.
I think it lacks of twelve.

MARCELLUS.
No, it is struck.

HORATIO.
Indeed? I heard it not. It then draws near the season
Wherein the spirit held his wont to walk.

[_A flourish of trumpets, and ordnance shot off within._]

What does this mean, my lord?

HAMLET.
The King doth wake tonight and takes his rouse,
Keeps wassail, and the swaggering upspring reels;
And as he drains his draughts of Rhenish down,
The kettle-drum and trumpet thus bray out
The triumph of his pledge.

HORATIO.
Is it a custom?

HAMLET.
Ay marry is’t;
And to my mind, though I am native here,
And to the manner born, it is a custom
More honour’d in the breach than the observance.
This heavy-headed revel east and west
Makes us traduc’d and tax’d of other nations:
They clepe us drunkards, and with swinish phrase
Soil our addition; and indeed it takes
From our achievements, though perform’d at height,
The pith and marrow of our attribute.
So oft it chances in particular men
That for some vicious mole of nature in them,
As in their birth, wherein they are not guilty,
Since nature cannot choose his origin,
By their o’ergrowth of some complexion,
Oft breaking down the pales and forts of reason;
Or by some habit, that too much o’erleavens
The form of plausive manners;—that these men,
Carrying, I say, the stamp of one defect,
Being Nature’s livery or Fortune’s star,—
His virtues else,—be they as pure as grace,
As infinite as man may undergo,
Shall in the general censure take corruption
From that particular fault. The dram of evil
Doth all the noble substance of a doubt
To his own scandal.

HORATIO.
Look, my lord, it comes!

Enter Ghost.

HAMLET.
Angels and ministers of grace defend us!
Be thou a spirit of health or goblin damn’d,
Bring with thee airs from heaven or blasts from hell,
Be thy intents wicked or charitable,
Thou com’st in such a questionable shape
That I will speak to thee. I’ll call thee Hamlet,
King, father, royal Dane. O, answer me!
Let me not burst in ignorance; but tell
Why thy canoniz’d bones, hearsed in death,
Have burst their cerements; why the sepulchre,
Wherein we saw thee quietly inurn’d,
Hath op’d his ponderous and marble jaws
To cast thee up again! What may this mean,
That thou, dead corse, again in complete steel,
Revisit’st thus the glimpses of the moon,
Making night hideous, and we fools of nature
So horridly to shake our disposition
With thoughts beyond the reaches of our souls?
Say, why is this? Wherefore? What should we do?

[_Ghost beckons Hamlet._]

HORATIO.
It beckons you to go away with it,
As if it some impartment did desire
To you alone.

MARCELLUS.
Look with what courteous action
It waves you to a more removed ground.
But do not go with it.

HORATIO.
No, by no means.

HAMLET.
It will not speak; then will I follow it.

HORATIO.
Do not, my lord.

HAMLET.
Why, what should be the fear?
I do not set my life at a pin’s fee;
And for my soul, what can it do to that,
Being a thing immortal as itself?
It waves me forth again. I’ll follow it.

HORATIO.
What if it tempt you toward the flood, my lord,
Or to the dreadful summit of the cliff
That beetles o’er his base into the sea,
And there assume some other horrible form
Which might deprive your sovereignty of reason,
And draw you into madness? Think of it.
The very place puts toys of desperation,
Without more motive, into every brain
That looks so many fathoms to the sea
And hears it roar beneath.

HAMLET.
It waves me still.
Go on, I’ll follow thee.

MARCELLUS.
You shall not go, my lord.

HAMLET.
Hold off your hands.

HORATIO.
Be rul’d; you shall not go.

HAMLET.
My fate cries out,
And makes each petty artery in this body
As hardy as the Nemean lion’s nerve.

[_Ghost beckons._]

Still am I call’d. Unhand me, gentlemen.

[_Breaking free from them._]

By heaven, I’ll make a ghost of him that lets me.
I say, away!—Go on, I’ll follow thee.

[_Exeunt Ghost and Hamlet._]

HORATIO.
He waxes desperate with imagination.

MARCELLUS.
Let’s follow; ’tis not fit thus to obey him.

HORATIO.
Have after. To what issue will this come?

MARCELLUS.
Something is rotten in the state of Denmark.

HORATIO.
Heaven will direct it.

MARCELLUS.
Nay, let’s follow him.

[_Exeunt._]

 SCENE V. A more remote part of the Castle.

Enter Ghost and Hamlet.

HAMLET.
Whither wilt thou lead me? Speak, I’ll go no further.

GHOST.
Mark me.

HAMLET.
I will.

GHOST.
My hour is almost come,
When I to sulph’rous and tormenting flames
Must render up myself.

HAMLET.
Alas, poor ghost!

GHOST.
Pity me not, but lend thy serious hearing
To what I shall unfold.

HAMLET.
Speak, I am bound to hear.

GHOST.
So art thou to revenge, when thou shalt hear.

HAMLET.
What?

GHOST.
I am thy father’s spirit,
Doom’d for a certain term to walk the night,
And for the day confin’d to fast in fires,
Till the foul crimes done in my days of nature
Are burnt and purg’d away. But that I am forbid
To tell the secrets of my prison-house,
I could a tale unfold whose lightest word
Would harrow up thy soul; freeze thy young blood,
Make thy two eyes like stars start from their spheres,
Thy knotted and combined locks to part,
And each particular hair to stand on end
Like quills upon the fretful porpentine.
But this eternal blazon must not be
To ears of flesh and blood. List, list, O, list!
If thou didst ever thy dear father love—

HAMLET.
O God!

GHOST.
Revenge his foul and most unnatural murder.

HAMLET.
Murder!

GHOST.
Murder most foul, as in the best it is;
But this most foul, strange, and unnatural.

HAMLET.
Haste me to know’t, that I, with wings as swift
As meditation or the thoughts of love
May sweep to my revenge.

GHOST.
I find thee apt;
And duller shouldst thou be than the fat weed
That rots itself in ease on Lethe wharf,
Wouldst thou not stir in this. Now, Hamlet, hear.
’Tis given out that, sleeping in my orchard,
A serpent stung me; so the whole ear of Denmark
Is by a forged process of my death
Rankly abus’d; but know, thou noble youth,
The serpent that did sting thy father’s life
Now wears his crown.

HAMLET.
O my prophetic soul!
Mine uncle!

GHOST.
Ay, that incestuous, that adulterate beast,
With witchcraft of his wit, with traitorous gifts,—
O wicked wit, and gifts, that have the power
So to seduce!—won to his shameful lust
The will of my most seeming-virtuous queen.
O Hamlet, what a falling off was there,
From me, whose love was of that dignity
That it went hand in hand even with the vow
I made to her in marriage; and to decline
Upon a wretch whose natural gifts were poor
To those of mine. But virtue, as it never will be mov’d,
Though lewdness court it in a shape of heaven;
So lust, though to a radiant angel link’d,
Will sate itself in a celestial bed
And prey on garbage.
But soft! methinks I scent the morning air;
Brief let me be. Sleeping within my orchard,
My custom always of the afternoon,
Upon my secure hour thy uncle stole
With juice of cursed hebenon in a vial,
And in the porches of my ears did pour
The leperous distilment, whose effect
Holds such an enmity with blood of man
That swift as quicksilver it courses through
The natural gates and alleys of the body;
And with a sudden vigour it doth posset
And curd, like eager droppings into milk,
The thin and wholesome blood. So did it mine;
And a most instant tetter bark’d about,
Most lazar-like, with vile and loathsome crust
All my smooth body.
Thus was I, sleeping, by a brother’s hand,
Of life, of crown, of queen at once dispatch’d:
Cut off even in the blossoms of my sin,
Unhous’led, disappointed, unanel’d;
No reckoning made, but sent to my account
With all my imperfections on my head.
O horrible! O horrible! most horrible!
If thou hast nature in thee, bear it not;
Let not the royal bed of Denmark be
A couch for luxury and damned incest.
But howsoever thou pursu’st this act,
Taint not thy mind, nor let thy soul contrive
Against thy mother aught; leave her to heaven,
And to those thorns that in her bosom lodge,
To prick and sting her. Fare thee well at once!
The glow-worm shows the matin to be near,
And ’gins to pale his uneffectual fire.
Adieu, adieu, adieu. Remember me.

[_Exit._]

HAMLET.
O all you host of heaven! O earth! What else?
And shall I couple hell? O, fie! Hold, my heart;
And you, my sinews, grow not instant old,
But bear me stiffly up. Remember thee?
Ay, thou poor ghost, while memory holds a seat
In this distracted globe. Remember thee?
Yea, from the table of my memory
I’ll wipe away all trivial fond records,
All saws of books, all forms, all pressures past,
That youth and observation copied there;
And thy commandment all alone shall live
Within the book and volume of my brain,
Unmix’d with baser matter. Yes, by heaven!
O most pernicious woman!
O villain, villain, smiling damned villain!
My tables. Meet it is I set it down,
That one may smile, and smile, and be a villain!
At least I am sure it may be so in Denmark.

[_Writing._]

So, uncle, there you are. Now to my word;
It is ‘Adieu, adieu, remember me.’
I have sworn’t.

HORATIO and MARCELLUS.
[_Within._] My lord, my lord.

MARCELLUS.
[_Within._] Lord Hamlet.

HORATIO.
[_Within._] Heaven secure him.

HAMLET.
So be it!

MARCELLUS.
[_Within._] Illo, ho, ho, my lord!

HAMLET.
Hillo, ho, ho, boy! Come, bird, come.

Enter Horatio and Marcellus.

MARCELLUS.
How is’t, my noble lord?

HORATIO.
What news, my lord?

HAMLET.
O, wonderful!

HORATIO.
Good my lord, tell it.

HAMLET.
No, you’ll reveal it.

HORATIO.
Not I, my lord, by heaven.

MARCELLUS.
Nor I, my lord.

HAMLET.
How say you then, would heart of man once think it?—
But you’ll be secret?

HORATIO and MARCELLUS.
Ay, by heaven, my lord.

HAMLET.
There’s ne’er a villain dwelling in all Denmark
But he’s an arrant knave.

HORATIO.
There needs no ghost, my lord, come from the grave
To tell us this.

HAMLET.
Why, right; you are i’ the right;
And so, without more circumstance at all,
I hold it fit that we shake hands and part:
You, as your business and desire shall point you,—
For every man hath business and desire,
Such as it is;—and for my own poor part,
Look you, I’ll go pray.

HORATIO.
These are but wild and whirling words, my lord.

HAMLET.
I’m sorry they offend you, heartily;
Yes faith, heartily.

HORATIO.
There’s no offence, my lord.

HAMLET.
Yes, by Saint Patrick, but there is, Horatio,
And much offence too. Touching this vision here,
It is an honest ghost, that let me tell you.
For your desire to know what is between us,
O’ermaster’t as you may. And now, good friends,
As you are friends, scholars, and soldiers,
Give me one poor request.

HORATIO.
What is’t, my lord? We will.

HAMLET.
Never make known what you have seen tonight.

HORATIO and MARCELLUS.
My lord, we will not.

HAMLET.
Nay, but swear’t.

HORATIO.
In faith, my lord, not I.

MARCELLUS.
Nor I, my lord, in faith.

HAMLET.
Upon my sword.

MARCELLUS.
We have sworn, my lord, already.

HAMLET.
Indeed, upon my sword, indeed.

GHOST.
[_Cries under the stage._] Swear.

HAMLET.
Ha, ha boy, sayst thou so? Art thou there, truepenny?
Come on, you hear this fellow in the cellarage.
Consent to swear.

HORATIO.
Propose the oath, my lord.

HAMLET.
Never to speak of this that you have seen.
Swear by my sword.

GHOST.
[_Beneath._] Swear.

HAMLET.
_Hic et ubique?_ Then we’ll shift our ground.
Come hither, gentlemen,
And lay your hands again upon my sword.
Never to speak of this that you have heard.
Swear by my sword.

GHOST.
[_Beneath._] Swear.

HAMLET.
Well said, old mole! Canst work i’ th’earth so fast?
A worthy pioner! Once more remove, good friends.

HORATIO.
O day and night, but this is wondrous strange.

HAMLET.
And therefore as a stranger give it welcome.
There are more things in heaven and earth, Horatio,
Than are dreamt of in your philosophy. But come,
Here, as before, never, so help you mercy,
How strange or odd soe’er I bear myself,—
As I perchance hereafter shall think meet
To put an antic disposition on—
That you, at such times seeing me, never shall,
With arms encumber’d thus, or this head-shake,
Or by pronouncing of some doubtful phrase,
As ‘Well, we know’, or ‘We could and if we would’,
Or ‘If we list to speak’; or ‘There be and if they might’,
Or such ambiguous giving out, to note
That you know aught of me:—this not to do.
So grace and mercy at your most need help you,
Swear.

GHOST.
[_Beneath._] Swear.

HAMLET.
Rest, rest, perturbed spirit. So, gentlemen,
With all my love I do commend me to you;
And what so poor a man as Hamlet is
May do t’express his love and friending to you,
God willing, shall not lack. Let us go in together,
And still your fingers on your lips, I pray.
The time is out of joint. O cursed spite,
That ever I was born to set it right.
Nay, come, let’s go together.

[_Exeunt._]




ACT II

SCENE I. A room in Polonius’s house.


Enter Polonius and Reynaldo.

POLONIUS.
Give him this money and these notes, Reynaldo.

REYNALDO.
I will, my lord.

POLONIUS.
You shall do marvellous wisely, good Reynaldo,
Before you visit him, to make inquiry
Of his behaviour.

REYNALDO.
My lord, I did intend it.

POLONIUS.
Marry, well said; very well said. Look you, sir,
Enquire me first what Danskers are in Paris;
And how, and who, what means, and where they keep,
What company, at what expense; and finding
By this encompassment and drift of question,
That they do know my son, come you more nearer
Than your particular demands will touch it.
Take you as ’twere some distant knowledge of him,
As thus, ‘I know his father and his friends,
And in part him’—do you mark this, Reynaldo?

REYNALDO.
Ay, very well, my lord.

POLONIUS.
‘And in part him, but,’ you may say, ‘not well;
But if’t be he I mean, he’s very wild;
Addicted so and so;’ and there put on him
What forgeries you please; marry, none so rank
As may dishonour him; take heed of that;
But, sir, such wanton, wild, and usual slips
As are companions noted and most known
To youth and liberty.

REYNALDO.
As gaming, my lord?

POLONIUS.
Ay, or drinking, fencing, swearing,
Quarrelling, drabbing. You may go so far.

REYNALDO.
My lord, that would dishonour him.

POLONIUS.
Faith no, as you may season it in the charge.
You must not put another scandal on him,
That he is open to incontinency;
That’s not my meaning: but breathe his faults so quaintly
That they may seem the taints of liberty;
The flash and outbreak of a fiery mind,
A savageness in unreclaimed blood,
Of general assault.

REYNALDO.
But my good lord—

POLONIUS.
Wherefore should you do this?

REYNALDO.
Ay, my lord, I would know that.

POLONIUS.
Marry, sir, here’s my drift,
And I believe it is a fetch of warrant.
You laying these slight sullies on my son,
As ’twere a thing a little soil’d i’ th’ working,
Mark you,
Your party in converse, him you would sound,
Having ever seen in the prenominate crimes
The youth you breathe of guilty, be assur’d
He closes with you in this consequence;
‘Good sir,’ or so; or ‘friend,’ or ‘gentleman’—
According to the phrase or the addition
Of man and country.

REYNALDO.
Very good, my lord.

POLONIUS.
And then, sir, does he this,—
He does—What was I about to say?
By the mass, I was about to say something. Where did I leave?

REYNALDO.
At ‘closes in the consequence.’
At ‘friend or so,’ and ‘gentleman.’

POLONIUS.
At ‘closes in the consequence’ ay, marry!
He closes with you thus: ‘I know the gentleman,
I saw him yesterday, or t’other day,
Or then, or then, with such and such; and, as you say,
There was he gaming, there o’ertook in’s rouse,
There falling out at tennis’: or perchance,
‘I saw him enter such a house of sale’—
_Videlicet_, a brothel, or so forth. See you now;
Your bait of falsehood takes this carp of truth;
And thus do we of wisdom and of reach,
With windlasses, and with assays of bias,
By indirections find directions out.
So by my former lecture and advice
Shall you my son. You have me, have you not?

REYNALDO.
My lord, I have.

POLONIUS.
God b’ wi’ you, fare you well.

REYNALDO.
Good my lord.

POLONIUS.
Observe his inclination in yourself.

REYNALDO.
I shall, my lord.

POLONIUS.
And let him ply his music.

REYNALDO.
Well, my lord.

POLONIUS.
Farewell.

[_Exit Reynaldo._]

Enter Ophelia.

How now, Ophelia, what’s the matter?

OPHELIA.
Alas, my lord, I have been so affrighted.

POLONIUS.
With what, in the name of God?

OPHELIA.
My lord, as I was sewing in my chamber,
Lord Hamlet, with his doublet all unbrac’d,
No hat upon his head, his stockings foul’d,
Ungart’red, and down-gyved to his ankle,
Pale as his shirt, his knees knocking each other,
And with a look so piteous in purport
As if he had been loosed out of hell
To speak of horrors, he comes before me.

POLONIUS.
Mad for thy love?

OPHELIA.
My lord, I do not know, but truly I do fear it.

POLONIUS.
What said he?

OPHELIA.
He took me by the wrist and held me hard;
Then goes he to the length of all his arm;
And with his other hand thus o’er his brow,
He falls to such perusal of my face
As he would draw it. Long stay’d he so,
At last,—a little shaking of mine arm,
And thrice his head thus waving up and down,
He rais’d a sigh so piteous and profound
As it did seem to shatter all his bulk
And end his being. That done, he lets me go,
And with his head over his shoulder turn’d
He seem’d to find his way without his eyes,
For out o’ doors he went without their help,
And to the last bended their light on me.

POLONIUS.
Come, go with me. I will go seek the King.
This is the very ecstasy of love,
Whose violent property fordoes itself,
And leads the will to desperate undertakings,
As oft as any passion under heaven
That does afflict our natures. I am sorry,—
What, have you given him any hard words of late?

OPHELIA.
No, my good lord; but as you did command,
I did repel his letters and denied
His access to me.

POLONIUS.
That hath made him mad.
I am sorry that with better heed and judgement
I had not quoted him. I fear’d he did but trifle,
And meant to wreck thee. But beshrew my jealousy!
It seems it is as proper to our age
To cast beyond ourselves in our opinions
As it is common for the younger sort
To lack discretion. Come, go we to the King.
This must be known, which, being kept close, might move
More grief to hide than hate to utter love.

[_Exeunt._]

 SCENE II. A room in the Castle.

Enter King, Queen, Rosencrantz, Guildenstern and Attendants.

KING.
Welcome, dear Rosencrantz and Guildenstern.
Moreover that we much did long to see you,
The need we have to use you did provoke
Our hasty sending. Something have you heard
Of Hamlet’s transformation; so I call it,
Since nor th’exterior nor the inward man
Resembles that it was. What it should be,
More than his father’s death, that thus hath put him
So much from th’understanding of himself,
I cannot dream of. I entreat you both
That, being of so young days brought up with him,
And since so neighbour’d to his youth and humour,
That you vouchsafe your rest here in our court
Some little time, so by your companies
To draw him on to pleasures and to gather,
So much as from occasion you may glean,
Whether aught to us unknown afflicts him thus
That, open’d, lies within our remedy.

QUEEN.
Good gentlemen, he hath much talk’d of you,
And sure I am, two men there are not living
To whom he more adheres. If it will please you
To show us so much gentry and good will
As to expend your time with us awhile,
For the supply and profit of our hope,
Your visitation shall receive such thanks
As fits a king’s remembrance.

ROSENCRANTZ.
Both your majesties
Might, by the sovereign power you have of us,
Put your dread pleasures more into command
Than to entreaty.

GUILDENSTERN.
We both obey,
And here give up ourselves, in the full bent,
To lay our service freely at your feet
To be commanded.

KING.
Thanks, Rosencrantz and gentle Guildenstern.

QUEEN.
Thanks, Guildenstern and gentle Rosencrantz.
And I beseech you instantly to visit
My too much changed son. Go, some of you,
And bring these gentlemen where Hamlet is.

GUILDENSTERN.
Heavens make our presence and our practices
Pleasant and helpful to him.

QUEEN.
Ay, amen.

[_Exeunt Rosencrantz, Guildenstern and some Attendants._]

Enter Polonius.

POLONIUS.
Th’ambassadors from Norway, my good lord,
Are joyfully return’d.

KING.
Thou still hast been the father of good news.

POLONIUS.
Have I, my lord? Assure you, my good liege,
I hold my duty, as I hold my soul,
Both to my God and to my gracious King:
And I do think,—or else this brain of mine
Hunts not the trail of policy so sure
As it hath us’d to do—that I have found
The very cause of Hamlet’s lunacy.

KING.
O speak of that, that do I long to hear.

POLONIUS.
Give first admittance to th’ambassadors;
My news shall be the fruit to that great feast.

KING.
Thyself do grace to them, and bring them in.

[_Exit Polonius._]

He tells me, my sweet queen, that he hath found
The head and source of all your son’s distemper.

QUEEN.
I doubt it is no other but the main,
His father’s death and our o’erhasty marriage.

KING.
Well, we shall sift him.

Enter Polonius with Voltemand and Cornelius.

Welcome, my good friends!
Say, Voltemand, what from our brother Norway?

VOLTEMAND.
Most fair return of greetings and desires.
Upon our first, he sent out to suppress
His nephew’s levies, which to him appear’d
To be a preparation ’gainst the Polack;
But better look’d into, he truly found
It was against your Highness; whereat griev’d,
That so his sickness, age, and impotence
Was falsely borne in hand, sends out arrests
On Fortinbras; which he, in brief, obeys,
Receives rebuke from Norway; and in fine,
Makes vow before his uncle never more
To give th’assay of arms against your Majesty.
Whereon old Norway, overcome with joy,
Gives him three thousand crowns in annual fee,
And his commission to employ those soldiers
So levied as before, against the Polack:
With an entreaty, herein further shown,
[_Gives a paper._]
That it might please you to give quiet pass
Through your dominions for this enterprise,
On such regards of safety and allowance
As therein are set down.

KING.
It likes us well;
And at our more consider’d time we’ll read,
Answer, and think upon this business.
Meantime we thank you for your well-took labour.
Go to your rest, at night we’ll feast together:.
Most welcome home.

[_Exeunt Voltemand and Cornelius._]

POLONIUS.
This business is well ended.
My liege and madam, to expostulate
What majesty should be, what duty is,
Why day is day, night night, and time is time
Were nothing but to waste night, day and time.
Therefore, since brevity is the soul of wit,
And tediousness the limbs and outward flourishes,
I will be brief. Your noble son is mad.
Mad call I it; for to define true madness,
What is’t but to be nothing else but mad?
But let that go.

QUEEN.
More matter, with less art.

POLONIUS.
Madam, I swear I use no art at all.
That he is mad, ’tis true: ’tis true ’tis pity;
And pity ’tis ’tis true. A foolish figure,
But farewell it, for I will use no art.
Mad let us grant him then. And now remains
That we find out the cause of this effect,
Or rather say, the cause of this defect,
For this effect defective comes by cause.
Thus it remains, and the remainder thus. Perpend,
I have a daughter—have whilst she is mine—
Who in her duty and obedience, mark,
Hath given me this. Now gather, and surmise.
[_Reads._]
_To the celestial, and my soul’s idol, the most beautified Ophelia_—
That’s an ill phrase, a vile phrase; ‘beautified’ is a vile
phrase: but you shall hear.
[_Reads._]
_these; in her excellent white bosom, these, &c._

QUEEN.
Came this from Hamlet to her?

POLONIUS.
Good madam, stay awhile; I will be faithful.
[_Reads._]
  _Doubt thou the stars are fire,
     Doubt that the sun doth move,
   Doubt truth to be a liar,
     But never doubt I love.
O dear Ophelia, I am ill at these numbers. I have not art to reckon my
groans. But that I love thee best, O most best, believe it. Adieu.
  Thine evermore, most dear lady, whilst this machine is to him,
     HAMLET._
This in obedience hath my daughter show’d me;
And more above, hath his solicitings,
As they fell out by time, by means, and place,
All given to mine ear.

KING.
But how hath she receiv’d his love?

POLONIUS.
What do you think of me?

KING.
As of a man faithful and honourable.

POLONIUS.
I would fain prove so. But what might you think,
When I had seen this hot love on the wing,
As I perceiv’d it, I must tell you that,
Before my daughter told me, what might you,
Or my dear Majesty your queen here, think,
If I had play’d the desk or table-book,
Or given my heart a winking, mute and dumb,
Or look’d upon this love with idle sight,
What might you think? No, I went round to work,
And my young mistress thus I did bespeak:
‘Lord Hamlet is a prince, out of thy star.
This must not be.’ And then I precepts gave her,
That she should lock herself from his resort,
Admit no messengers, receive no tokens.
Which done, she took the fruits of my advice,
And he, repulsed,—a short tale to make—
Fell into a sadness, then into a fast,
Thence to a watch, thence into a weakness,
Thence to a lightness, and, by this declension,
Into the madness wherein now he raves,
And all we wail for.

KING.
Do you think ’tis this?

QUEEN.
It may be, very likely.

POLONIUS.
Hath there been such a time, I’d fain know that,
That I have positively said ‘’Tis so,’
When it prov’d otherwise?

KING.
Not that I know.

POLONIUS.
Take this from this, if this be otherwise.
[_Points to his head and shoulder._]
If circumstances lead me, I will find
Where truth is hid, though it were hid indeed
Within the centre.

KING.
How may we try it further?

POLONIUS.
You know sometimes he walks four hours together
Here in the lobby.

QUEEN.
So he does indeed.

POLONIUS.
At such a time I’ll loose my daughter to him.
Be you and I behind an arras then,
Mark the encounter. If he love her not,
And be not from his reason fall’n thereon,
Let me be no assistant for a state,
But keep a farm and carters.

KING.
We will try it.

Enter Hamlet, reading.

QUEEN.
But look where sadly the poor wretch comes reading.

POLONIUS.
Away, I do beseech you, both away
I’ll board him presently. O, give me leave.

[_Exeunt King, Queen and Attendants._]

How does my good Lord Hamlet?

HAMLET.
Well, God-a-mercy.

POLONIUS.
Do you know me, my lord?

HAMLET.
Excellent well. You are a fishmonger.

POLONIUS.
Not I, my lord.

HAMLET.
Then I would you were so honest a man.

POLONIUS.
Honest, my lord?

HAMLET.
Ay sir, to be honest, as this world goes, is to be one man picked out
of ten thousand.

POLONIUS.
That’s very true, my lord.

HAMLET.
For if the sun breed maggots in a dead dog, being a good kissing
carrion,—
Have you a daughter?

POLONIUS.
I have, my lord.

HAMLET.
Let her not walk i’ th’ sun. Conception is a blessing, but not as your
daughter may conceive. Friend, look to’t.

POLONIUS.
How say you by that? [_Aside._] Still harping on my daughter. Yet he
knew me not at first; he said I was a fishmonger. He is far gone, far
gone. And truly in my youth I suffered much extremity for love; very
near this. I’ll speak to him again.—What do you read, my lord?

HAMLET.
Words, words, words.

POLONIUS.
What is the matter, my lord?

HAMLET.
Between who?

POLONIUS.
I mean the matter that you read, my lord.

HAMLET.
Slanders, sir. For the satirical slave says here that old men have grey
beards; that their faces are wrinkled; their eyes purging thick amber
and plum-tree gum; and that they have a plentiful lack of wit, together
with most weak hams. All which, sir, though I most powerfully and
potently believe, yet I hold it not honesty to have it thus set down.
For you yourself, sir, should be old as I am, if like a crab you could
go backward.

POLONIUS.
[_Aside._] Though this be madness, yet there is method in’t.—
Will you walk out of the air, my lord?

HAMLET.
Into my grave?

POLONIUS.
Indeed, that is out o’ the air. [_Aside._] How pregnant sometimes his
replies are! A happiness that often madness hits on, which reason and
sanity could not so prosperously be delivered of. I will leave him and
suddenly contrive the means of meeting between him and my daughter.
My honourable lord, I will most humbly take my leave of you.

HAMLET.
You cannot, sir, take from me anything that I will more willingly part
withal, except my life, except my life, except my life.

POLONIUS.
Fare you well, my lord.

HAMLET.
These tedious old fools.

Enter Rosencrantz and Guildenstern.

POLONIUS.
You go to seek the Lord Hamlet; there he is.

ROSENCRANTZ.
[_To Polonius._] God save you, sir.

[_Exit Polonius._]

GUILDENSTERN.
My honoured lord!

ROSENCRANTZ.
My most dear lord!

HAMLET.
My excellent good friends! How dost thou, Guildenstern? Ah,
Rosencrantz. Good lads, how do ye both?

ROSENCRANTZ.
As the indifferent children of the earth.

GUILDENSTERN.
Happy in that we are not over-happy;
On Fortune’s cap we are not the very button.

HAMLET.
Nor the soles of her shoe?

ROSENCRANTZ.
Neither, my lord.

HAMLET.
Then you live about her waist, or in the middle of her favours?

GUILDENSTERN.
Faith, her privates we.

HAMLET.
In the secret parts of Fortune? O, most true; she is a strumpet. What’s
the news?

ROSENCRANTZ.
None, my lord, but that the world’s grown honest.

HAMLET.
Then is doomsday near. But your news is not true. Let me question more
in particular. What have you, my good friends, deserved at the hands of
Fortune, that she sends you to prison hither?

GUILDENSTERN.
Prison, my lord?

HAMLET.
Denmark’s a prison.

ROSENCRANTZ.
Then is the world one.

HAMLET.
A goodly one; in which there are many confines, wards, and dungeons,
Denmark being one o’ th’ worst.

ROSENCRANTZ.
We think not so, my lord.

HAMLET.
Why, then ’tis none to you; for there is nothing either good or bad but
thinking makes it so. To me it is a prison.

ROSENCRANTZ.
Why, then your ambition makes it one; ’tis too narrow for your mind.

HAMLET.
O God, I could be bounded in a nutshell, and count myself a king of
infinite space, were it not that I have bad dreams.

GUILDENSTERN.
Which dreams, indeed, are ambition; for the very substance of the
ambitious is merely the shadow of a dream.

HAMLET.
A dream itself is but a shadow.

ROSENCRANTZ.
Truly, and I hold ambition of so airy and light a quality that it is
but a shadow’s shadow.

HAMLET.
Then are our beggars bodies, and our monarchs and outstretch’d heroes
the beggars’ shadows. Shall we to th’ court? For, by my fay, I cannot
reason.

ROSENCRANTZ and GUILDENSTERN.
We’ll wait upon you.

HAMLET.
No such matter. I will not sort you with the rest of my servants; for,
to speak to you like an honest man, I am most dreadfully attended. But,
in the beaten way of friendship, what make you at Elsinore?

ROSENCRANTZ.
To visit you, my lord, no other occasion.

HAMLET.
Beggar that I am, I am even poor in thanks; but I thank you. And sure,
dear friends, my thanks are too dear a halfpenny. Were you not sent
for? Is it your own inclining? Is it a free visitation? Come, deal
justly with me. Come, come; nay, speak.

GUILDENSTERN.
What should we say, my lord?

HAMLET.
Why, anything. But to the purpose. You were sent for; and there is a
kind of confession in your looks, which your modesties have not craft
enough to colour. I know the good King and Queen have sent for you.

ROSENCRANTZ.
To what end, my lord?

HAMLET.
That you must teach me. But let me conjure you, by the rights of our
fellowship, by the consonancy of our youth, by the obligation of our
ever-preserved love, and by what more dear a better proposer could
charge you withal, be even and direct with me, whether you were sent
for or no.

ROSENCRANTZ.
[_To Guildenstern._] What say you?

HAMLET.
[_Aside._] Nay, then I have an eye of you. If you love me, hold not
off.

GUILDENSTERN.
My lord, we were sent for.

HAMLET.
I will tell you why; so shall my anticipation prevent your discovery,
and your secrecy to the King and Queen moult no feather. I have of
late, but wherefore I know not, lost all my mirth, forgone all custom
of exercises; and indeed, it goes so heavily with my disposition that
this goodly frame the earth, seems to me a sterile promontory; this
most excellent canopy the air, look you, this brave o’erhanging
firmament, this majestical roof fretted with golden fire, why, it
appears no other thing to me than a foul and pestilent congregation of
vapours. What a piece of work is man, how noble in reason, how infinite
in faculties, in form and moving, how express and admirable; in action
how like an angel, in apprehension, how like a god: the beauty of the
world, the paragon of animals. And yet, to me, what is this
quintessence of dust? Man delights not me; no, nor woman neither,
though by your smiling you seem to say so.

ROSENCRANTZ.
My lord, there was no such stuff in my thoughts.

HAMLET.
Why did you laugh then, when I said ‘Man delights not me’?

ROSENCRANTZ.
To think, my lord, if you delight not in man, what Lenten entertainment
the players shall receive from you. We coted them on the way, and
hither are they coming to offer you service.

HAMLET.
He that plays the king shall be welcome,—his Majesty shall have tribute
of me; the adventurous knight shall use his foil and target; the lover
shall not sigh gratis, the humorous man shall end his part in peace;
the clown shall make those laugh whose lungs are tickle o’ th’ sere;
and the lady shall say her mind freely, or the blank verse shall halt
for’t. What players are they?

ROSENCRANTZ.
Even those you were wont to take such delight in—the tragedians of the
city.

HAMLET.
How chances it they travel? Their residence, both in reputation and
profit, was better both ways.

ROSENCRANTZ.
I think their inhibition comes by the means of the late innovation.

HAMLET.
Do they hold the same estimation they did when I was in the city? Are
they so followed?

ROSENCRANTZ.
No, indeed, they are not.

HAMLET.
How comes it? Do they grow rusty?

ROSENCRANTZ.
Nay, their endeavour keeps in the wonted pace; but there is, sir, an
aerie of children, little eyases, that cry out on the top of question,
and are most tyrannically clapped for’t. These are now the fashion, and
so berattle the common stages—so they call them—that many wearing
rapiers are afraid of goose-quills and dare scarce come thither.

HAMLET.
What, are they children? Who maintains ’em? How are they escoted? Will
they pursue the quality no longer than they can sing? Will they not say
afterwards, if they should grow themselves to common players—as it is
most like, if their means are no better—their writers do them wrong to
make them exclaim against their own succession?

ROSENCRANTZ.
Faith, there has been much to do on both sides; and the nation holds it
no sin to tarre them to controversy. There was for a while, no money
bid for argument unless the poet and the player went to cuffs in the
question.

HAMLET.
Is’t possible?

GUILDENSTERN.
O, there has been much throwing about of brains.

HAMLET.
Do the boys carry it away?

ROSENCRANTZ.
Ay, that they do, my lord. Hercules and his load too.

HAMLET.
It is not very strange; for my uncle is King of Denmark, and those that
would make mouths at him while my father lived, give twenty, forty,
fifty, a hundred ducats apiece for his picture in little. ’Sblood,
there is something in this more than natural, if philosophy could find
it out.

[_Flourish of trumpets within._]

GUILDENSTERN.
There are the players.

HAMLET.
Gentlemen, you are welcome to Elsinore. Your hands, come. The
appurtenance of welcome is fashion and ceremony. Let me comply with you
in this garb, lest my extent to the players, which I tell you must show
fairly outward, should more appear like entertainment than yours. You
are welcome. But my uncle-father and aunt-mother are deceived.

GUILDENSTERN.
In what, my dear lord?

HAMLET.
I am but mad north-north-west. When the wind is southerly, I know a
hawk from a handsaw.

Enter Polonius.

POLONIUS.
Well be with you, gentlemen.

HAMLET.
Hark you, Guildenstern, and you too, at each ear a hearer. That great
baby you see there is not yet out of his swaddling clouts.

ROSENCRANTZ.
Happily he’s the second time come to them; for they say an old man is
twice a child.

HAMLET.
I will prophesy he comes to tell me of the players. Mark it.—You say
right, sir: for a Monday morning ’twas so indeed.

POLONIUS.
My lord, I have news to tell you.

HAMLET.
My lord, I have news to tell you. When Roscius was an actor in Rome—

POLONIUS.
The actors are come hither, my lord.

HAMLET.
Buzz, buzz.

POLONIUS.
Upon my honour.

HAMLET.
Then came each actor on his ass—

POLONIUS.
The best actors in the world, either for tragedy, comedy, history,
pastoral, pastoral-comical, historical-pastoral, tragical-historical,
tragical-comical-historical-pastoral, scene individable, or poem
unlimited. Seneca cannot be too heavy, nor Plautus too light, for the
law of writ and the liberty. These are the only men.

HAMLET.
O Jephthah, judge of Israel, what a treasure hadst thou!

POLONIUS.
What treasure had he, my lord?

HAMLET.
Why—
   ’One fair daughter, and no more,
   The which he loved passing well.’

POLONIUS.
[_Aside._] Still on my daughter.

HAMLET.
Am I not i’ th’ right, old Jephthah?

POLONIUS.
If you call me Jephthah, my lord, I have a daughter that I love passing
well.

HAMLET.
Nay, that follows not.

POLONIUS.
What follows then, my lord?

HAMLET.
Why,
   As by lot, God wot,
and then, you know,
   It came to pass, as most like it was.
The first row of the pious chanson will show you more. For look where
my abridgement comes.

Enter four or five Players.

You are welcome, masters, welcome all. I am glad to see thee well.
Welcome, good friends. O, my old friend! Thy face is valanc’d since I
saw thee last. Com’st thou to beard me in Denmark? What, my young lady
and mistress! By’r lady, your ladyship is nearer to heaven than when I
saw you last, by the altitude of a chopine. Pray God your voice, like a
piece of uncurrent gold, be not cracked within the ring. Masters, you
are all welcome. We’ll e’en to’t like French falconers, fly at anything
we see. We’ll have a speech straight. Come, give us a taste of your
quality. Come, a passionate speech.

FIRST PLAYER.
What speech, my lord?

HAMLET.
I heard thee speak me a speech once, but it was never acted, or if it
was, not above once, for the play, I remember, pleased not the million,
’twas caviare to the general. But it was—as I received it, and others,
whose judgements in such matters cried in the top of mine—an excellent
play, well digested in the scenes, set down with as much modesty as
cunning. I remember one said there were no sallets in the lines to make
the matter savoury, nor no matter in the phrase that might indite the
author of affectation, but called it an honest method, as wholesome as
sweet, and by very much more handsome than fine. One speech in it, I
chiefly loved. ’Twas Aeneas’ tale to Dido, and thereabout of it
especially where he speaks of Priam’s slaughter. If it live in your
memory, begin at this line, let me see, let me see:
   _The rugged Pyrrhus, like th’ Hyrcanian beast,—_
It is not so: it begins with Pyrrhus—
   _The rugged Pyrrhus, he whose sable arms,
   Black as his purpose, did the night resemble
   When he lay couched in the ominous horse,
   Hath now this dread and black complexion smear’d
   With heraldry more dismal. Head to foot
   Now is he total gules, horridly trick’d
   With blood of fathers, mothers, daughters, sons,
   Bak’d and impasted with the parching streets,
   That lend a tyrannous and a damned light
   To their vile murders. Roasted in wrath and fire,
   And thus o’ersized with coagulate gore,
   With eyes like carbuncles, the hellish Pyrrhus
   Old grandsire Priam seeks._
So, proceed you.

POLONIUS.
’Fore God, my lord, well spoken, with good accent and good discretion.

FIRST PLAYER.
   _Anon he finds him,
   Striking too short at Greeks. His antique sword,
   Rebellious to his arm, lies where it falls,
   Repugnant to command. Unequal match’d,
   Pyrrhus at Priam drives, in rage strikes wide;
   But with the whiff and wind of his fell sword
   Th’unnerved father falls. Then senseless Ilium,
   Seeming to feel this blow, with flaming top
   Stoops to his base, and with a hideous crash
   Takes prisoner Pyrrhus’ ear. For lo, his sword,
   Which was declining on the milky head
   Of reverend Priam, seem’d i’ th’air to stick.
   So, as a painted tyrant, Pyrrhus stood,
   And like a neutral to his will and matter,
   Did nothing.
   But as we often see against some storm,
   A silence in the heavens, the rack stand still,
   The bold winds speechless, and the orb below
   As hush as death, anon the dreadful thunder
   Doth rend the region; so after Pyrrhus’ pause,
   Aroused vengeance sets him new a-work,
   And never did the Cyclops’ hammers fall
   On Mars’s armour, forg’d for proof eterne,
   With less remorse than Pyrrhus’ bleeding sword
   Now falls on Priam.
   Out, out, thou strumpet Fortune! All you gods,
   In general synod, take away her power;
   Break all the spokes and fellies from her wheel,
   And bowl the round nave down the hill of heaven,
   As low as to the fiends._

POLONIUS.
This is too long.

HAMLET.
It shall to the barber’s, with your beard.—Prithee say on.
He’s for a jig or a tale of bawdry, or he sleeps.
Say on; come to Hecuba.

FIRST PLAYER.
   _But who, O who, had seen the mobled queen,—_

HAMLET.
‘The mobled queen’?

POLONIUS.
That’s good! ‘Mobled queen’ is good.

FIRST PLAYER.
   _Run barefoot up and down, threat’ning the flames
   With bisson rheum. A clout upon that head
   Where late the diadem stood, and for a robe,
   About her lank and all o’erteemed loins,
   A blanket, in th’alarm of fear caught up—
   Who this had seen, with tongue in venom steep’d,
   ’Gainst Fortune’s state would treason have pronounc’d.
   But if the gods themselves did see her then,
   When she saw Pyrrhus make malicious sport
   In mincing with his sword her husband’s limbs,
   The instant burst of clamour that she made,—
   Unless things mortal move them not at all,—
   Would have made milch the burning eyes of heaven,
   And passion in the gods._

POLONIUS.
Look, where he has not turn’d his colour, and has tears in’s eyes. Pray
you, no more.

HAMLET.
’Tis well. I’ll have thee speak out the rest of this soon.—Good my
lord, will you see the players well bestowed? Do you hear, let them be
well used; for they are the abstracts and brief chronicles of the time.
After your death you were better have a bad epitaph than their ill
report while you live.

POLONIUS.
My lord, I will use them according to their desert.

HAMLET.
God’s bodikin, man, much better. Use every man after his desert, and who
should ’scape whipping? Use them after your own honour and dignity. The
less they deserve, the more merit is in your bounty. Take them in.

POLONIUS.
Come, sirs.

HAMLET.
Follow him, friends. We’ll hear a play tomorrow.

[_Exeunt Polonius with all the Players but the First._]

Dost thou hear me, old friend? Can you play _The Murder of Gonzago_?

FIRST PLAYER.
Ay, my lord.

HAMLET.
We’ll ha’t tomorrow night. You could for a need study a speech of some
dozen or sixteen lines, which I would set down and insert in’t, could
you not?

FIRST PLAYER.
Ay, my lord.

HAMLET.
Very well. Follow that lord, and look you mock him not.

[_Exit First Player._]

[_To Rosencrantz and Guildenstern_] My good friends, I’ll leave you
till night. You are welcome to Elsinore.

ROSENCRANTZ.
Good my lord.

[_Exeunt Rosencrantz and Guildenstern._]

HAMLET.
Ay, so, God b’ wi’ ye. Now I am alone.
O what a rogue and peasant slave am I!
Is it not monstrous that this player here,
But in a fiction, in a dream of passion,
Could force his soul so to his own conceit
That from her working all his visage wan’d;
Tears in his eyes, distraction in’s aspect,
A broken voice, and his whole function suiting
With forms to his conceit? And all for nothing!
For Hecuba?
What’s Hecuba to him, or he to Hecuba,
That he should weep for her? What would he do,
Had he the motive and the cue for passion
That I have? He would drown the stage with tears
And cleave the general ear with horrid speech;
Make mad the guilty, and appal the free,
Confound the ignorant, and amaze indeed,
The very faculties of eyes and ears. Yet I,
A dull and muddy-mettled rascal, peak
Like John-a-dreams, unpregnant of my cause,
And can say nothing. No, not for a king
Upon whose property and most dear life
A damn’d defeat was made. Am I a coward?
Who calls me villain, breaks my pate across?
Plucks off my beard and blows it in my face?
Tweaks me by the nose, gives me the lie i’ th’ throat
As deep as to the lungs? Who does me this?
Ha! ’Swounds, I should take it: for it cannot be
But I am pigeon-liver’d, and lack gall
To make oppression bitter, or ere this
I should have fatted all the region kites
With this slave’s offal. Bloody, bawdy villain!
Remorseless, treacherous, lecherous, kindless villain!
Oh vengeance!
Why, what an ass am I! This is most brave,
That I, the son of a dear father murder’d,
Prompted to my revenge by heaven and hell,
Must, like a whore, unpack my heart with words
And fall a-cursing like a very drab,
A scullion! Fie upon’t! Foh!
About, my brain! I have heard
That guilty creatures sitting at a play,
Have by the very cunning of the scene,
Been struck so to the soul that presently
They have proclaim’d their malefactions.
For murder, though it have no tongue, will speak
With most miraculous organ. I’ll have these players
Play something like the murder of my father
Before mine uncle. I’ll observe his looks;
I’ll tent him to the quick. If he but blench,
I know my course. The spirit that I have seen
May be the devil, and the devil hath power
T’assume a pleasing shape, yea, and perhaps
Out of my weakness and my melancholy,
As he is very potent with such spirits,
Abuses me to damn me. I’ll have grounds
More relative than this. The play’s the thing
Wherein I’ll catch the conscience of the King.

[_Exit._]




ACT III

SCENE I. A room in the Castle.


Enter King, Queen, Polonius, Ophelia, Rosencrantz and Guildenstern.

KING.
And can you by no drift of circumstance
Get from him why he puts on this confusion,
Grating so harshly all his days of quiet
With turbulent and dangerous lunacy?

ROSENCRANTZ.
He does confess he feels himself distracted,
But from what cause he will by no means speak.

GUILDENSTERN.
Nor do we find him forward to be sounded,
But with a crafty madness keeps aloof
When we would bring him on to some confession
Of his true state.

QUEEN.
Did he receive you well?

ROSENCRANTZ.
Most like a gentleman.

GUILDENSTERN.
But with much forcing of his disposition.

ROSENCRANTZ.
Niggard of question, but of our demands,
Most free in his reply.

QUEEN.
Did you assay him to any pastime?

ROSENCRANTZ.
Madam, it so fell out that certain players
We o’er-raught on the way. Of these we told him,
And there did seem in him a kind of joy
To hear of it. They are about the court,
And, as I think, they have already order
This night to play before him.

POLONIUS.
’Tis most true;
And he beseech’d me to entreat your Majesties
To hear and see the matter.

KING.
With all my heart; and it doth much content me
To hear him so inclin’d.
Good gentlemen, give him a further edge,
And drive his purpose on to these delights.

ROSENCRANTZ.
We shall, my lord.

[_Exeunt Rosencrantz and Guildenstern._]

KING.
Sweet Gertrude, leave us too,
For we have closely sent for Hamlet hither,
That he, as ’twere by accident, may here
Affront Ophelia.
Her father and myself, lawful espials,
Will so bestow ourselves that, seeing unseen,
We may of their encounter frankly judge,
And gather by him, as he is behav’d,
If’t be th’affliction of his love or no
That thus he suffers for.

QUEEN.
I shall obey you.
And for your part, Ophelia, I do wish
That your good beauties be the happy cause
Of Hamlet’s wildness: so shall I hope your virtues
Will bring him to his wonted way again,
To both your honours.

OPHELIA.
Madam, I wish it may.

[_Exit Queen._]

POLONIUS.
Ophelia, walk you here.—Gracious, so please you,
We will bestow ourselves.—[_To Ophelia._] Read on this book,
That show of such an exercise may colour
Your loneliness.—We are oft to blame in this,
’Tis too much prov’d, that with devotion’s visage
And pious action we do sugar o’er
The devil himself.

KING.
[_Aside._] O ’tis too true!
How smart a lash that speech doth give my conscience!
The harlot’s cheek, beautied with plastering art,
Is not more ugly to the thing that helps it
Than is my deed to my most painted word.
O heavy burden!

POLONIUS.
I hear him coming. Let’s withdraw, my lord.

[_Exeunt King and Polonius._]

Enter Hamlet.

HAMLET.
To be, or not to be, that is the question:
Whether ’tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles,
And by opposing end them? To die—to sleep,
No more; and by a sleep to say we end
The heart-ache, and the thousand natural shocks
That flesh is heir to: ’tis a consummation
Devoutly to be wish’d. To die, to sleep.
To sleep, perchance to dream—ay, there’s the rub,
For in that sleep of death what dreams may come,
When we have shuffled off this mortal coil,
Must give us pause. There’s the respect
That makes calamity of so long life.
For who would bear the whips and scorns of time,
The oppressor’s wrong, the proud man’s contumely,
The pangs of dispriz’d love, the law’s delay,
The insolence of office, and the spurns
That patient merit of the unworthy takes,
When he himself might his quietus make
With a bare bodkin? Who would these fardels bear,
To grunt and sweat under a weary life,
But that the dread of something after death,
The undiscover’d country, from whose bourn
No traveller returns, puzzles the will,
And makes us rather bear those ills we have
Than fly to others that we know not of?
Thus conscience does make cowards of us all,
And thus the native hue of resolution
Is sicklied o’er with the pale cast of thought,
And enterprises of great pith and moment,
With this regard their currents turn awry
And lose the name of action. Soft you now,
The fair Ophelia! Nymph, in thy orisons
Be all my sins remember’d.

OPHELIA.
Good my lord,
How does your honour for this many a day?

HAMLET.
I humbly thank you; well, well, well.

OPHELIA.
My lord, I have remembrances of yours
That I have longed long to re-deliver.
I pray you, now receive them.

HAMLET.
No, not I.
I never gave you aught.

OPHELIA.
My honour’d lord, you know right well you did,
And with them words of so sweet breath compos’d
As made the things more rich; their perfume lost,
Take these again; for to the noble mind
Rich gifts wax poor when givers prove unkind.
There, my lord.

HAMLET.
Ha, ha! Are you honest?

OPHELIA.
My lord?

HAMLET.
Are you fair?

OPHELIA.
What means your lordship?

HAMLET.
That if you be honest and fair, your honesty should admit no discourse
to your beauty.

OPHELIA.
Could beauty, my lord, have better commerce than with honesty?

HAMLET.
Ay, truly; for the power of beauty will sooner transform honesty from
what it is to a bawd than the force of honesty can translate beauty
into his likeness. This was sometime a paradox, but now the time gives
it proof. I did love you once.

OPHELIA.
Indeed, my lord, you made me believe so.

HAMLET.
You should not have believed me; for virtue cannot so inoculate our old
stock but we shall relish of it. I loved you not.

OPHELIA.
I was the more deceived.

HAMLET.
Get thee to a nunnery. Why wouldst thou be a breeder of sinners? I am
myself indifferent honest; but yet I could accuse me of such things
that it were better my mother had not borne me. I am very proud,
revengeful, ambitious, with more offences at my beck than I have
thoughts to put them in, imagination to give them shape, or time to act
them in. What should such fellows as I do crawling between earth and
heaven? We are arrant knaves all, believe none of us. Go thy ways to a
nunnery. Where’s your father?

OPHELIA.
At home, my lord.

HAMLET.
Let the doors be shut upon him, that he may play the fool nowhere but
in’s own house. Farewell.

OPHELIA.
O help him, you sweet heavens!

HAMLET.
If thou dost marry, I’ll give thee this plague for thy dowry. Be thou
as chaste as ice, as pure as snow, thou shalt not escape calumny. Get
thee to a nunnery, go: farewell. Or if thou wilt needs marry, marry a
fool; for wise men know well enough what monsters you make of them. To
a nunnery, go; and quickly too. Farewell.

OPHELIA.
O heavenly powers, restore him!

HAMLET.
I have heard of your paintings too, well enough. God hath given you one
face, and you make yourselves another. You jig, you amble, and you
lisp, and nickname God’s creatures, and make your wantonness your
ignorance. Go to, I’ll no more on’t, it hath made me mad. I say, we
will have no more marriages. Those that are married already, all but
one, shall live; the rest shall keep as they are. To a nunnery, go.

[_Exit._]

OPHELIA.
O, what a noble mind is here o’erthrown!
The courtier’s, soldier’s, scholar’s, eye, tongue, sword,
Th’expectancy and rose of the fair state,
The glass of fashion and the mould of form,
Th’observ’d of all observers, quite, quite down!
And I, of ladies most deject and wretched,
That suck’d the honey of his music vows,
Now see that noble and most sovereign reason,
Like sweet bells jangled out of tune and harsh,
That unmatch’d form and feature of blown youth
Blasted with ecstasy. O woe is me,
T’have seen what I have seen, see what I see.

Enter King and Polonius.

KING.
Love? His affections do not that way tend,
Nor what he spake, though it lack’d form a little,
Was not like madness. There’s something in his soul
O’er which his melancholy sits on brood,
And I do doubt the hatch and the disclose
Will be some danger, which for to prevent,
I have in quick determination
Thus set it down: he shall with speed to England
For the demand of our neglected tribute:
Haply the seas and countries different,
With variable objects, shall expel
This something settled matter in his heart,
Whereon his brains still beating puts him thus
From fashion of himself. What think you on’t?

POLONIUS.
It shall do well. But yet do I believe
The origin and commencement of his grief
Sprung from neglected love. How now, Ophelia?
You need not tell us what Lord Hamlet said,
We heard it all. My lord, do as you please,
But if you hold it fit, after the play,
Let his queen mother all alone entreat him
To show his grief, let her be round with him,
And I’ll be plac’d, so please you, in the ear
Of all their conference. If she find him not,
To England send him; or confine him where
Your wisdom best shall think.

KING.
It shall be so.
Madness in great ones must not unwatch’d go.

[_Exeunt._]

 SCENE II. A hall in the Castle.

Enter Hamlet and certain Players.

HAMLET.
Speak the speech, I pray you, as I pronounced it to you, trippingly on
the tongue. But if you mouth it, as many of your players do, I had as
lief the town-crier spoke my lines. Nor do not saw the air too much
with your hand, thus, but use all gently; for in the very torrent,
tempest, and, as I may say, whirlwind of passion, you must acquire and
beget a temperance that may give it smoothness. O, it offends me to the
soul to hear a robustious periwig-pated fellow tear a passion to
tatters, to very rags, to split the ears of the groundlings, who, for
the most part, are capable of nothing but inexplicable dumb shows and
noise. I would have such a fellow whipped for o’erdoing Termagant. It
out-Herods Herod. Pray you avoid it.

FIRST PLAYER.
I warrant your honour.

HAMLET.
Be not too tame neither; but let your own discretion be your tutor.
Suit the action to the word, the word to the action, with this special
observance, that you o’erstep not the modesty of nature; for anything
so overdone is from the purpose of playing, whose end, both at the
first and now, was and is, to hold as ’twere the mirror up to nature;
to show virtue her own feature, scorn her own image, and the very age
and body of the time his form and pressure. Now, this overdone, or come
tardy off, though it make the unskilful laugh, cannot but make the
judicious grieve; the censure of the which one must in your allowance
o’erweigh a whole theatre of others. O, there be players that I have
seen play—and heard others praise, and that highly—not to speak it
profanely, that, neither having the accent of Christians, nor the gait
of Christian, pagan, nor man, have so strutted and bellowed that I have
thought some of Nature’s journeymen had made men, and not made them
well, they imitated humanity so abominably.

FIRST PLAYER.
I hope we have reform’d that indifferently with us, sir.

HAMLET.
O reform it altogether. And let those that play your clowns speak no
more than is set down for them. For there be of them that will
themselves laugh, to set on some quantity of barren spectators to laugh
too, though in the meantime some necessary question of the play be then
to be considered. That’s villainous, and shows a most pitiful ambition
in the fool that uses it. Go make you ready.

[_Exeunt Players._]

Enter Polonius, Rosencrantz and Guildenstern.

How now, my lord?
Will the King hear this piece of work?

POLONIUS.
And the Queen too, and that presently.

HAMLET.
Bid the players make haste.

[_Exit Polonius._]

Will you two help to hasten them?

ROSENCRANTZ and GUILDENSTERN.
We will, my lord.

[_Exeunt Rosencrantz and Guildenstern._]

HAMLET.
What ho, Horatio!

Enter Horatio.

HORATIO.
Here, sweet lord, at your service.

HAMLET.
Horatio, thou art e’en as just a man
As e’er my conversation cop’d withal.

HORATIO.
O my dear lord.

HAMLET.
Nay, do not think I flatter;
For what advancement may I hope from thee,
That no revenue hast, but thy good spirits
To feed and clothe thee? Why should the poor be flatter’d?
No, let the candied tongue lick absurd pomp,
And crook the pregnant hinges of the knee
Where thrift may follow fawning. Dost thou hear?
Since my dear soul was mistress of her choice,
And could of men distinguish, her election
Hath seal’d thee for herself. For thou hast been
As one, in suffering all, that suffers nothing,
A man that Fortune’s buffets and rewards
Hast ta’en with equal thanks. And blessed are those
Whose blood and judgement are so well co-mingled
That they are not a pipe for Fortune’s finger
To sound what stop she please. Give me that man
That is not passion’s slave, and I will wear him
In my heart’s core, ay, in my heart of heart,
As I do thee. Something too much of this.
There is a play tonight before the King.
One scene of it comes near the circumstance
Which I have told thee, of my father’s death.
I prithee, when thou see’st that act a-foot,
Even with the very comment of thy soul
Observe mine uncle. If his occulted guilt
Do not itself unkennel in one speech,
It is a damned ghost that we have seen;
And my imaginations are as foul
As Vulcan’s stithy. Give him heedful note;
For I mine eyes will rivet to his face;
And after we will both our judgements join
In censure of his seeming.

HORATIO.
Well, my lord.
If he steal aught the whilst this play is playing,
And ’scape detecting, I will pay the theft.

HAMLET.
They are coming to the play. I must be idle.
Get you a place.

Danish march. A flourish. Enter King, Queen, Polonius, Ophelia,
Rosencrantz, Guildenstern and others.

KING.
How fares our cousin Hamlet?

HAMLET.
Excellent, i’ faith; of the chameleon’s dish: I eat the air,
promise-crammed: you cannot feed capons so.

KING.
I have nothing with this answer, Hamlet; these words are not mine.

HAMLET.
No, nor mine now. [_To Polonius._] My lord, you play’d once i’
th’university, you say?

POLONIUS.
That did I, my lord, and was accounted a good actor.

HAMLET.
What did you enact?

POLONIUS.
I did enact Julius Caesar. I was kill’d i’ th’ Capitol. Brutus killed
me.

HAMLET.
It was a brute part of him to kill so capital a calf there. Be the
players ready?

ROSENCRANTZ.
Ay, my lord; they stay upon your patience.

QUEEN.
Come hither, my dear Hamlet, sit by me.

HAMLET.
No, good mother, here’s metal more attractive.

POLONIUS.
[_To the King._] O ho! do you mark that?

HAMLET.
Lady, shall I lie in your lap?

[_Lying down at Ophelia’s feet._]

OPHELIA.
No, my lord.

HAMLET.
I mean, my head upon your lap?

OPHELIA.
Ay, my lord.

HAMLET.
Do you think I meant country matters?

OPHELIA.
I think nothing, my lord.

HAMLET.
That’s a fair thought to lie between maids’ legs.

OPHELIA.
What is, my lord?

HAMLET.
Nothing.

OPHELIA.
You are merry, my lord.

HAMLET.
Who, I?

OPHELIA.
Ay, my lord.

HAMLET.
O God, your only jig-maker! What should a man do but be merry? For look
you how cheerfully my mother looks, and my father died within’s two
hours.

OPHELIA.
Nay, ’tis twice two months, my lord.

HAMLET.
So long? Nay then, let the devil wear black, for I’ll have a suit of
sables. O heavens! die two months ago, and not forgotten yet? Then
there’s hope a great man’s memory may outlive his life half a year. But
by’r lady, he must build churches then; or else shall he suffer not
thinking on, with the hobby-horse, whose epitaph is ‘For, O, for O, the
hobby-horse is forgot!’

Trumpets sound. The dumb show enters.

_Enter a King and a Queen very lovingly; the Queen embracing him and he
her. She kneels, and makes show of protestation unto him. He takes her
up, and declines his head upon her neck. Lays him down upon a bank of
flowers. She, seeing him asleep, leaves him. Anon comes in a fellow,
takes off his crown, kisses it, pours poison in the King’s ears, and
exits. The Queen returns, finds the King dead, and makes passionate
action. The Poisoner with some three or four Mutes, comes in again,
seeming to lament with her. The dead body is carried away. The Poisoner
woos the Queen with gifts. She seems loth and unwilling awhile, but in
the end accepts his love._

[_Exeunt._]

OPHELIA.
What means this, my lord?

HAMLET.
Marry, this is miching mallecho; it means mischief.

OPHELIA.
Belike this show imports the argument of the play.

Enter Prologue.

HAMLET.
We shall know by this fellow: the players cannot keep counsel; they’ll
tell all.

OPHELIA.
Will they tell us what this show meant?

HAMLET.
Ay, or any show that you’ll show him. Be not you ashamed to show, he’ll
not shame to tell you what it means.

OPHELIA.
You are naught, you are naught: I’ll mark the play.

PROLOGUE.
   _For us, and for our tragedy,
   Here stooping to your clemency,
   We beg your hearing patiently._

HAMLET.
Is this a prologue, or the posy of a ring?

OPHELIA.
’Tis brief, my lord.

HAMLET.
As woman’s love.

Enter a King and a Queen.

PLAYER KING.
Full thirty times hath Phoebus’ cart gone round
Neptune’s salt wash and Tellus’ orbed ground,
And thirty dozen moons with borrow’d sheen
About the world have times twelve thirties been,
Since love our hearts, and Hymen did our hands
Unite commutual in most sacred bands.

PLAYER QUEEN.
So many journeys may the sun and moon
Make us again count o’er ere love be done.
But, woe is me, you are so sick of late,
So far from cheer and from your former state,
That I distrust you. Yet, though I distrust,
Discomfort you, my lord, it nothing must:
For women’s fear and love holds quantity,
In neither aught, or in extremity.
Now what my love is, proof hath made you know,
And as my love is siz’d, my fear is so.
Where love is great, the littlest doubts are fear;
Where little fears grow great, great love grows there.

PLAYER KING.
Faith, I must leave thee, love, and shortly too:
My operant powers their functions leave to do:
And thou shalt live in this fair world behind,
Honour’d, belov’d, and haply one as kind
For husband shalt thou—

PLAYER QUEEN.
O confound the rest.
Such love must needs be treason in my breast.
In second husband let me be accurst!
None wed the second but who kill’d the first.

HAMLET.
[_Aside._] Wormwood, wormwood.

PLAYER QUEEN.
The instances that second marriage move
Are base respects of thrift, but none of love.
A second time I kill my husband dead,
When second husband kisses me in bed.

PLAYER KING.
I do believe you think what now you speak;
But what we do determine, oft we break.
Purpose is but the slave to memory,
Of violent birth, but poor validity:
Which now, like fruit unripe, sticks on the tree,
But fall unshaken when they mellow be.
Most necessary ’tis that we forget
To pay ourselves what to ourselves is debt.
What to ourselves in passion we propose,
The passion ending, doth the purpose lose.
The violence of either grief or joy
Their own enactures with themselves destroy.
Where joy most revels, grief doth most lament;
Grief joys, joy grieves, on slender accident.
This world is not for aye; nor ’tis not strange
That even our loves should with our fortunes change,
For ’tis a question left us yet to prove,
Whether love lead fortune, or else fortune love.
The great man down, you mark his favourite flies,
The poor advanc’d makes friends of enemies;
And hitherto doth love on fortune tend:
For who not needs shall never lack a friend,
And who in want a hollow friend doth try,
Directly seasons him his enemy.
But orderly to end where I begun,
Our wills and fates do so contrary run
That our devices still are overthrown.
Our thoughts are ours, their ends none of our own.
So think thou wilt no second husband wed,
But die thy thoughts when thy first lord is dead.

PLAYER QUEEN.
Nor earth to me give food, nor heaven light,
Sport and repose lock from me day and night,
To desperation turn my trust and hope,
An anchor’s cheer in prison be my scope,
Each opposite that blanks the face of joy,
Meet what I would have well, and it destroy!
Both here and hence pursue me lasting strife,
If, once a widow, ever I be wife.

HAMLET.
[_To Ophelia._] If she should break it now.

PLAYER KING.
’Tis deeply sworn. Sweet, leave me here awhile.
My spirits grow dull, and fain I would beguile
The tedious day with sleep.
[_Sleeps._]

PLAYER QUEEN.
Sleep rock thy brain,
And never come mischance between us twain.

[_Exit._]

HAMLET.
Madam, how like you this play?

QUEEN.
The lady protests too much, methinks.

HAMLET.
O, but she’ll keep her word.

KING.
Have you heard the argument? Is there no offence in’t?

HAMLET.
No, no, they do but jest, poison in jest; no offence i’ th’ world.

KING.
What do you call the play?

HAMLET.
_The Mousetrap._ Marry, how? Tropically. This play is the image of a
murder done in Vienna. Gonzago is the Duke’s name, his wife Baptista:
you shall see anon; ’tis a knavish piece of work: but what o’ that?
Your majesty, and we that have free souls, it touches us not. Let the
gall’d jade wince; our withers are unwrung.

Enter Lucianus.

This is one Lucianus, nephew to the King.

OPHELIA.
You are a good chorus, my lord.

HAMLET.
I could interpret between you and your love, if I could see the puppets
dallying.

OPHELIA.
You are keen, my lord, you are keen.

HAMLET.
It would cost you a groaning to take off my edge.

OPHELIA.
Still better, and worse.

HAMLET.
So you mistake your husbands.—Begin, murderer. Pox, leave thy damnable
faces, and begin. Come, the croaking raven doth bellow for revenge.

LUCIANUS.
Thoughts black, hands apt, drugs fit, and time agreeing,
Confederate season, else no creature seeing;
Thou mixture rank, of midnight weeds collected,
With Hecate’s ban thrice blasted, thrice infected,
Thy natural magic and dire property
On wholesome life usurp immediately.

[_Pours the poison into the sleeper’s ears._]

HAMLET.
He poisons him i’ th’garden for’s estate. His name’s Gonzago. The story
is extant, and written in very choice Italian. You shall see anon how
the murderer gets the love of Gonzago’s wife.

OPHELIA.
The King rises.

HAMLET.
What, frighted with false fire?

QUEEN.
How fares my lord?

POLONIUS.
Give o’er the play.

KING.
Give me some light. Away.

All.
Lights, lights, lights.

[_Exeunt all but Hamlet and Horatio._]

HAMLET.
   Why, let the strucken deer go weep,
     The hart ungalled play;
   For some must watch, while some must sleep,
     So runs the world away.
Would not this, sir, and a forest of feathers, if the rest of my
fortunes turn Turk with me; with two Provincial roses on my razed
shoes, get me a fellowship in a cry of players, sir?

HORATIO.
Half a share.

HAMLET.
A whole one, I.
   For thou dost know, O Damon dear,
     This realm dismantled was
   Of Jove himself, and now reigns here
     A very, very—pajock.

HORATIO.
You might have rhymed.

HAMLET.
O good Horatio, I’ll take the ghost’s word for a thousand pound. Didst
perceive?

HORATIO.
Very well, my lord.

HAMLET.
Upon the talk of the poisoning?

HORATIO.
I did very well note him.

HAMLET.
Ah, ha! Come, some music. Come, the recorders.
   For if the king like not the comedy,
   Why then, belike he likes it not, perdie.
Come, some music.

Enter Rosencrantz and Guildenstern.

GUILDENSTERN.
Good my lord, vouchsafe me a word with you.

HAMLET.
Sir, a whole history.

GUILDENSTERN.
The King, sir—

HAMLET.
Ay, sir, what of him?

GUILDENSTERN.
Is in his retirement, marvellous distempered.

HAMLET.
With drink, sir?

GUILDENSTERN.
No, my lord; rather with choler.

HAMLET.
Your wisdom should show itself more richer to signify this to the
doctor, for me to put him to his purgation would perhaps plunge him
into far more choler.

GUILDENSTERN.
Good my lord, put your discourse into some frame, and start not so
wildly from my affair.

HAMLET.
I am tame, sir, pronounce.

GUILDENSTERN.
The Queen your mother, in most great affliction of spirit, hath sent me
to you.

HAMLET.
You are welcome.

GUILDENSTERN.
Nay, good my lord, this courtesy is not of the right breed. If it shall
please you to make me a wholesome answer, I will do your mother’s
commandment; if not, your pardon and my return shall be the end of my
business.

HAMLET.
Sir, I cannot.

GUILDENSTERN.
What, my lord?

HAMLET.
Make you a wholesome answer. My wit’s diseased. But, sir, such answer
as I can make, you shall command; or rather, as you say, my mother.
Therefore no more, but to the matter. My mother, you say,—

ROSENCRANTZ.
Then thus she says: your behaviour hath struck her into amazement and
admiration.

HAMLET.
O wonderful son, that can so stonish a mother! But is there no sequel
at the heels of this mother’s admiration?

ROSENCRANTZ.
She desires to speak with you in her closet ere you go to bed.

HAMLET.
We shall obey, were she ten times our mother. Have you any further
trade with us?

ROSENCRANTZ.
My lord, you once did love me.

HAMLET.
And so I do still, by these pickers and stealers.

ROSENCRANTZ.
Good my lord, what is your cause of distemper? You do surely bar the
door upon your own liberty if you deny your griefs to your friend.

HAMLET.
Sir, I lack advancement.

ROSENCRANTZ.
How can that be, when you have the voice of the King himself for your
succession in Denmark?

HAMLET.
Ay, sir, but while the grass grows—the proverb is something musty.

Re-enter the Players with recorders.

O, the recorders. Let me see one.—To withdraw with you, why do you go
about to recover the wind of me, as if you would drive me into a toil?

GUILDENSTERN.
O my lord, if my duty be too bold, my love is too unmannerly.

HAMLET.
I do not well understand that. Will you play upon this pipe?

GUILDENSTERN.
My lord, I cannot.

HAMLET.
I pray you.

GUILDENSTERN.
Believe me, I cannot.

HAMLET.
I do beseech you.

GUILDENSTERN.
I know no touch of it, my lord.

HAMLET.
’Tis as easy as lying: govern these ventages with your finger and
thumb, give it breath with your mouth, and it will discourse most
eloquent music. Look you, these are the stops.

GUILDENSTERN.
But these cannot I command to any utterance of harmony. I have not the
skill.

HAMLET.
Why, look you now, how unworthy a thing you make of me! You would play
upon me; you would seem to know my stops; you would pluck out the heart
of my mystery; you would sound me from my lowest note to the top of my
compass; and there is much music, excellent voice, in this little
organ, yet cannot you make it speak. ’Sblood, do you think I am easier
to be played on than a pipe? Call me what instrument you will, though
you can fret me, you cannot play upon me.

Enter Polonius.

God bless you, sir.

POLONIUS.
My lord, the Queen would speak with you, and presently.

HAMLET.
Do you see yonder cloud that’s almost in shape of a camel?

POLONIUS.
By the mass, and ’tis like a camel indeed.

HAMLET.
Methinks it is like a weasel.

POLONIUS.
It is backed like a weasel.

HAMLET.
Or like a whale.

POLONIUS.
Very like a whale.

HAMLET.
Then will I come to my mother by and by.—They fool me to the top of my
bent.—I will come by and by.

POLONIUS.
I will say so.

[_Exit._]

HAMLET.
By and by is easily said. Leave me, friends.

[_Exeunt all but Hamlet._]

’Tis now the very witching time of night,
When churchyards yawn, and hell itself breathes out
Contagion to this world. Now could I drink hot blood,
And do such bitter business as the day
Would quake to look on. Soft now, to my mother.
O heart, lose not thy nature; let not ever
The soul of Nero enter this firm bosom:
Let me be cruel, not unnatural.
I will speak daggers to her, but use none;
My tongue and soul in this be hypocrites.
How in my words somever she be shent,
To give them seals never, my soul, consent.

[_Exit._]

 SCENE III. A room in the Castle.

Enter King, Rosencrantz and Guildenstern.

KING.
I like him not, nor stands it safe with us
To let his madness range. Therefore prepare you,
I your commission will forthwith dispatch,
And he to England shall along with you.
The terms of our estate may not endure
Hazard so near us as doth hourly grow
Out of his lunacies.

GUILDENSTERN.
We will ourselves provide.
Most holy and religious fear it is
To keep those many many bodies safe
That live and feed upon your Majesty.

ROSENCRANTZ.
The single and peculiar life is bound
With all the strength and armour of the mind,
To keep itself from ’noyance; but much more
That spirit upon whose weal depend and rest
The lives of many. The cease of majesty
Dies not alone; but like a gulf doth draw
What’s near it with it. It is a massy wheel
Fix’d on the summit of the highest mount,
To whose huge spokes ten thousand lesser things
Are mortis’d and adjoin’d; which when it falls,
Each small annexment, petty consequence,
Attends the boist’rous ruin. Never alone
Did the King sigh, but with a general groan.

KING.
Arm you, I pray you, to this speedy voyage;
For we will fetters put upon this fear,
Which now goes too free-footed.

ROSENCRANTZ and GUILDENSTERN.
We will haste us.

[_Exeunt Rosencrantz and Guildenstern._]

Enter Polonius.

POLONIUS.
My lord, he’s going to his mother’s closet.
Behind the arras I’ll convey myself
To hear the process. I’ll warrant she’ll tax him home,
And as you said, and wisely was it said,
’Tis meet that some more audience than a mother,
Since nature makes them partial, should o’erhear
The speech of vantage. Fare you well, my liege,
I’ll call upon you ere you go to bed,
And tell you what I know.

KING.
Thanks, dear my lord.

[_Exit Polonius._]

O, my offence is rank, it smells to heaven;
It hath the primal eldest curse upon’t,—
A brother’s murder! Pray can I not,
Though inclination be as sharp as will:
My stronger guilt defeats my strong intent,
And, like a man to double business bound,
I stand in pause where I shall first begin,
And both neglect. What if this cursed hand
Were thicker than itself with brother’s blood,
Is there not rain enough in the sweet heavens
To wash it white as snow? Whereto serves mercy
But to confront the visage of offence?
And what’s in prayer but this twofold force,
To be forestalled ere we come to fall,
Or pardon’d being down? Then I’ll look up.
My fault is past. But O, what form of prayer
Can serve my turn? Forgive me my foul murder!
That cannot be; since I am still possess’d
Of those effects for which I did the murder,—
My crown, mine own ambition, and my queen.
May one be pardon’d and retain th’offence?
In the corrupted currents of this world
Offence’s gilded hand may shove by justice,
And oft ’tis seen the wicked prize itself
Buys out the law. But ’tis not so above;
There is no shuffling, there the action lies
In his true nature, and we ourselves compell’d
Even to the teeth and forehead of our faults,
To give in evidence. What then? What rests?
Try what repentance can. What can it not?
Yet what can it, when one cannot repent?
O wretched state! O bosom black as death!
O limed soul, that struggling to be free,
Art more engag’d! Help, angels! Make assay:
Bow, stubborn knees; and heart with strings of steel,
Be soft as sinews of the new-born babe.
All may be well.

[_Retires and kneels._]

Enter Hamlet.

HAMLET.
Now might I do it pat, now he is praying.
And now I’ll do’t. And so he goes to heaven;
And so am I reveng’d. That would be scann’d:
A villain kills my father, and for that
I, his sole son, do this same villain send
To heaven. O, this is hire and salary, not revenge.
He took my father grossly, full of bread,
With all his crimes broad blown, as flush as May;
And how his audit stands, who knows save heaven?
But in our circumstance and course of thought,
’Tis heavy with him. And am I then reveng’d,
To take him in the purging of his soul,
When he is fit and season’d for his passage? No.
Up, sword, and know thou a more horrid hent:
When he is drunk asleep; or in his rage,
Or in th’incestuous pleasure of his bed,
At gaming, swearing; or about some act
That has no relish of salvation in’t,
Then trip him, that his heels may kick at heaven,
And that his soul may be as damn’d and black
As hell, whereto it goes. My mother stays.
This physic but prolongs thy sickly days.

[_Exit._]

The King rises and advances.

KING.
My words fly up, my thoughts remain below.
Words without thoughts never to heaven go.

[_Exit._]

 SCENE IV. Another room in the Castle.

Enter Queen and Polonius.

POLONIUS.
He will come straight. Look you lay home to him,
Tell him his pranks have been too broad to bear with,
And that your Grace hath screen’d and stood between
Much heat and him. I’ll silence me e’en here.
Pray you be round with him.

HAMLET.
[_Within._] Mother, mother, mother.

QUEEN.
I’ll warrant you, Fear me not.
Withdraw, I hear him coming.

[_Polonius goes behind the arras._]

Enter Hamlet.

HAMLET.
Now, mother, what’s the matter?

QUEEN.
Hamlet, thou hast thy father much offended.

HAMLET.
Mother, you have my father much offended.

QUEEN.
Come, come, you answer with an idle tongue.

HAMLET.
Go, go, you question with a wicked tongue.

QUEEN.
Why, how now, Hamlet?

HAMLET.
What’s the matter now?

QUEEN.
Have you forgot me?

HAMLET.
No, by the rood, not so.
You are the Queen, your husband’s brother’s wife,
And, would it were not so. You are my mother.

QUEEN.
Nay, then I’ll set those to you that can speak.

HAMLET.
Come, come, and sit you down, you shall not budge.
You go not till I set you up a glass
Where you may see the inmost part of you.

QUEEN.
What wilt thou do? Thou wilt not murder me?
Help, help, ho!

POLONIUS.
[_Behind._] What, ho! help, help, help!

HAMLET.
How now? A rat? [_Draws._]
Dead for a ducat, dead!

[_Makes a pass through the arras._]

POLONIUS.
[_Behind._] O, I am slain!

[_Falls and dies._]

QUEEN.
O me, what hast thou done?

HAMLET.
Nay, I know not. Is it the King?

[_Draws forth Polonius._]

QUEEN.
O what a rash and bloody deed is this!

HAMLET.
A bloody deed. Almost as bad, good mother,
As kill a king and marry with his brother.

QUEEN.
As kill a king?

HAMLET.
Ay, lady, ’twas my word.—
[_To Polonius._] Thou wretched, rash, intruding fool, farewell!
I took thee for thy better. Take thy fortune,
Thou find’st to be too busy is some danger.—
Leave wringing of your hands. Peace, sit you down,
And let me wring your heart, for so I shall,
If it be made of penetrable stuff;
If damned custom have not braz’d it so,
That it is proof and bulwark against sense.

QUEEN.
What have I done, that thou dar’st wag thy tongue
In noise so rude against me?

HAMLET.
Such an act
That blurs the grace and blush of modesty,
Calls virtue hypocrite, takes off the rose
From the fair forehead of an innocent love,
And sets a blister there. Makes marriage vows
As false as dicers’ oaths. O such a deed
As from the body of contraction plucks
The very soul, and sweet religion makes
A rhapsody of words. Heaven’s face doth glow,
Yea this solidity and compound mass,
With tristful visage, as against the doom,
Is thought-sick at the act.

QUEEN.
Ay me, what act,
That roars so loud, and thunders in the index?

HAMLET.
Look here upon this picture, and on this,
The counterfeit presentment of two brothers.
See what a grace was seated on this brow,
Hyperion’s curls, the front of Jove himself,
An eye like Mars, to threaten and command,
A station like the herald Mercury
New lighted on a heaven-kissing hill:
A combination and a form indeed,
Where every god did seem to set his seal,
To give the world assurance of a man.
This was your husband. Look you now what follows.
Here is your husband, like a mildew’d ear
Blasting his wholesome brother. Have you eyes?
Could you on this fair mountain leave to feed,
And batten on this moor? Ha! have you eyes?
You cannot call it love; for at your age
The hey-day in the blood is tame, it’s humble,
And waits upon the judgement: and what judgement
Would step from this to this? Sense sure you have,
Else could you not have motion; but sure that sense
Is apoplex’d, for madness would not err
Nor sense to ecstacy was ne’er so thrall’d
But it reserv’d some quantity of choice
To serve in such a difference. What devil was’t
That thus hath cozen’d you at hoodman-blind?
Eyes without feeling, feeling without sight,
Ears without hands or eyes, smelling sans all,
Or but a sickly part of one true sense
Could not so mope. O shame! where is thy blush?
Rebellious hell,
If thou canst mutine in a matron’s bones,
To flaming youth let virtue be as wax,
And melt in her own fire. Proclaim no shame
When the compulsive ardour gives the charge,
Since frost itself as actively doth burn,
And reason panders will.

QUEEN.
O Hamlet, speak no more.
Thou turn’st mine eyes into my very soul,
And there I see such black and grained spots
As will not leave their tinct.

HAMLET.
Nay, but to live
In the rank sweat of an enseamed bed,
Stew’d in corruption, honeying and making love
Over the nasty sty.

QUEEN.
O speak to me no more;
These words like daggers enter in mine ears;
No more, sweet Hamlet.

HAMLET.
A murderer and a villain;
A slave that is not twentieth part the tithe
Of your precedent lord. A vice of kings,
A cutpurse of the empire and the rule,
That from a shelf the precious diadem stole
And put it in his pocket!

QUEEN.
No more.

HAMLET.
A king of shreds and patches!—

Enter Ghost.

Save me and hover o’er me with your wings,
You heavenly guards! What would your gracious figure?

QUEEN.
Alas, he’s mad.

HAMLET.
Do you not come your tardy son to chide,
That, laps’d in time and passion, lets go by
The important acting of your dread command?
O say!

GHOST.
Do not forget. This visitation
Is but to whet thy almost blunted purpose.
But look, amazement on thy mother sits.
O step between her and her fighting soul.
Conceit in weakest bodies strongest works.
Speak to her, Hamlet.

HAMLET.
How is it with you, lady?

QUEEN.
Alas, how is’t with you,
That you do bend your eye on vacancy,
And with the incorporal air do hold discourse?
Forth at your eyes your spirits wildly peep,
And, as the sleeping soldiers in the alarm,
Your bedded hairs, like life in excrements,
Start up and stand an end. O gentle son,
Upon the heat and flame of thy distemper
Sprinkle cool patience. Whereon do you look?

HAMLET.
On him, on him! Look you how pale he glares,
His form and cause conjoin’d, preaching to stones,
Would make them capable.—Do not look upon me,
Lest with this piteous action you convert
My stern effects. Then what I have to do
Will want true colour; tears perchance for blood.

QUEEN.
To whom do you speak this?

HAMLET.
Do you see nothing there?

QUEEN.
Nothing at all; yet all that is I see.

HAMLET.
Nor did you nothing hear?

QUEEN.
No, nothing but ourselves.

HAMLET.
Why, look you there! look how it steals away!
My father, in his habit as he liv’d!
Look where he goes even now out at the portal.

[_Exit Ghost._]

QUEEN.
This is the very coinage of your brain.
This bodiless creation ecstasy
Is very cunning in.

HAMLET.
Ecstasy!
My pulse as yours doth temperately keep time,
And makes as healthful music. It is not madness
That I have utter’d. Bring me to the test,
And I the matter will re-word; which madness
Would gambol from. Mother, for love of grace,
Lay not that flattering unction to your soul
That not your trespass, but my madness speaks.
It will but skin and film the ulcerous place,
Whilst rank corruption, mining all within,
Infects unseen. Confess yourself to heaven,
Repent what’s past, avoid what is to come;
And do not spread the compost on the weeds,
To make them ranker. Forgive me this my virtue;
For in the fatness of these pursy times
Virtue itself of vice must pardon beg,
Yea, curb and woo for leave to do him good.

QUEEN.
O Hamlet, thou hast cleft my heart in twain.

HAMLET.
O throw away the worser part of it,
And live the purer with the other half.
Good night. But go not to mine uncle’s bed.
Assume a virtue, if you have it not.
That monster custom, who all sense doth eat,
Of habits evil, is angel yet in this,
That to the use of actions fair and good
He likewise gives a frock or livery
That aptly is put on. Refrain tonight,
And that shall lend a kind of easiness
To the next abstinence. The next more easy;
For use almost can change the stamp of nature,
And either curb the devil, or throw him out
With wondrous potency. Once more, good night,
And when you are desirous to be bles’d,
I’ll blessing beg of you. For this same lord
[_Pointing to Polonius._]
I do repent; but heaven hath pleas’d it so,
To punish me with this, and this with me,
That I must be their scourge and minister.
I will bestow him, and will answer well
The death I gave him. So again, good night.
I must be cruel, only to be kind:
Thus bad begins, and worse remains behind.
One word more, good lady.

QUEEN.
What shall I do?

HAMLET.
Not this, by no means, that I bid you do:
Let the bloat King tempt you again to bed,
Pinch wanton on your cheek, call you his mouse,
And let him, for a pair of reechy kisses,
Or paddling in your neck with his damn’d fingers,
Make you to ravel all this matter out,
That I essentially am not in madness,
But mad in craft. ’Twere good you let him know,
For who that’s but a queen, fair, sober, wise,
Would from a paddock, from a bat, a gib,
Such dear concernings hide? Who would do so?
No, in despite of sense and secrecy,
Unpeg the basket on the house’s top,
Let the birds fly, and like the famous ape,
To try conclusions, in the basket creep
And break your own neck down.

QUEEN.
Be thou assur’d, if words be made of breath,
And breath of life, I have no life to breathe
What thou hast said to me.

HAMLET.
I must to England, you know that?

QUEEN.
Alack,
I had forgot. ’Tis so concluded on.

HAMLET.
There’s letters seal’d: and my two schoolfellows,
Whom I will trust as I will adders fang’d,—
They bear the mandate, they must sweep my way
And marshal me to knavery. Let it work;
For ’tis the sport to have the enginer
Hoist with his own petard, and ’t shall go hard
But I will delve one yard below their mines
And blow them at the moon. O, ’tis most sweet,
When in one line two crafts directly meet.
This man shall set me packing.
I’ll lug the guts into the neighbour room.
Mother, good night. Indeed, this counsellor
Is now most still, most secret, and most grave,
Who was in life a foolish prating knave.
Come, sir, to draw toward an end with you.
Good night, mother.

[_Exit Hamlet dragging out Polonius._]




ACT IV

SCENE I. A room in the Castle.


Enter King, Queen, Rosencrantz and Guildenstern.

KING.
There’s matter in these sighs. These profound heaves
You must translate; ’tis fit we understand them.
Where is your son?

QUEEN.
Bestow this place on us a little while.

[_To Rosencrantz and Guildenstern, who go out._]

Ah, my good lord, what have I seen tonight!

KING.
What, Gertrude? How does Hamlet?

QUEEN.
Mad as the sea and wind, when both contend
Which is the mightier. In his lawless fit
Behind the arras hearing something stir,
Whips out his rapier, cries ‘A rat, a rat!’
And in this brainish apprehension kills
The unseen good old man.

KING.
O heavy deed!
It had been so with us, had we been there.
His liberty is full of threats to all;
To you yourself, to us, to everyone.
Alas, how shall this bloody deed be answer’d?
It will be laid to us, whose providence
Should have kept short, restrain’d, and out of haunt
This mad young man. But so much was our love
We would not understand what was most fit,
But like the owner of a foul disease,
To keep it from divulging, let it feed
Even on the pith of life. Where is he gone?

QUEEN.
To draw apart the body he hath kill’d,
O’er whom his very madness, like some ore
Among a mineral of metals base,
Shows itself pure. He weeps for what is done.

KING.
O Gertrude, come away!
The sun no sooner shall the mountains touch
But we will ship him hence, and this vile deed
We must with all our majesty and skill
Both countenance and excuse.—Ho, Guildenstern!

Re-enter Rosencrantz and Guildenstern.

Friends both, go join you with some further aid:
Hamlet in madness hath Polonius slain,
And from his mother’s closet hath he dragg’d him.
Go seek him out, speak fair, and bring the body
Into the chapel. I pray you haste in this.

[_Exeunt Rosencrantz and Guildenstern._]

Come, Gertrude, we’ll call up our wisest friends,
And let them know both what we mean to do
And what’s untimely done, so haply slander,
Whose whisper o’er the world’s diameter,
As level as the cannon to his blank,
Transports his poison’d shot, may miss our name,
And hit the woundless air. O, come away!
My soul is full of discord and dismay.

[_Exeunt._]

 SCENE II. Another room in the Castle.

Enter Hamlet.

HAMLET.
Safely stowed.

ROSENCRANTZ and GUILDENSTERN.
[_Within._] Hamlet! Lord Hamlet!

HAMLET.
What noise? Who calls on Hamlet? O, here they come.

Enter Rosencrantz and Guildenstern.

ROSENCRANTZ.
What have you done, my lord, with the dead body?

HAMLET.
Compounded it with dust, whereto ’tis kin.

ROSENCRANTZ.
Tell us where ’tis, that we may take it thence,
And bear it to the chapel.

HAMLET.
Do not believe it.

ROSENCRANTZ.
Believe what?

HAMLET.
That I can keep your counsel, and not mine own. Besides, to be demanded
of a sponge—what replication should be made by the son of a king?

ROSENCRANTZ.
Take you me for a sponge, my lord?

HAMLET.
Ay, sir; that soaks up the King’s countenance, his rewards, his
authorities. But such officers do the King best service in the end: he
keeps them, like an ape, in the corner of his jaw; first mouthed, to be
last swallowed: when he needs what you have gleaned, it is but
squeezing you, and, sponge, you shall be dry again.

ROSENCRANTZ.
I understand you not, my lord.

HAMLET.
I am glad of it. A knavish speech sleeps in a foolish ear.

ROSENCRANTZ.
My lord, you must tell us where the body is and go with us to the King.

HAMLET.
The body is with the King, but the King is not with the body. The King
is a thing—

GUILDENSTERN.
A thing, my lord!

HAMLET.
Of nothing. Bring me to him. Hide fox, and all after.

[_Exeunt._]

 SCENE III. Another room in the Castle.

Enter King, attended.

KING.
I have sent to seek him and to find the body.
How dangerous is it that this man goes loose!
Yet must not we put the strong law on him:
He’s lov’d of the distracted multitude,
Who like not in their judgement, but their eyes;
And where ’tis so, th’offender’s scourge is weigh’d,
But never the offence. To bear all smooth and even,
This sudden sending him away must seem
Deliberate pause. Diseases desperate grown
By desperate appliance are reliev’d,
Or not at all.

Enter Rosencrantz.

How now? What hath befall’n?

ROSENCRANTZ.
Where the dead body is bestow’d, my lord,
We cannot get from him.

KING.
But where is he?

ROSENCRANTZ.
Without, my lord, guarded, to know your pleasure.

KING.
Bring him before us.

ROSENCRANTZ.
Ho, Guildenstern! Bring in my lord.

Enter Hamlet and Guildenstern.

KING.
Now, Hamlet, where’s Polonius?

HAMLET.
At supper.

KING.
At supper? Where?

HAMLET.
Not where he eats, but where he is eaten. A certain convocation of
politic worms are e’en at him. Your worm is your only emperor for diet.
We fat all creatures else to fat us, and we fat ourselves for maggots.
Your fat king and your lean beggar is but variable service,—two dishes,
but to one table. That’s the end.

KING.
Alas, alas!

HAMLET.
A man may fish with the worm that hath eat of a king, and eat of the
fish that hath fed of that worm.

KING.
What dost thou mean by this?

HAMLET.
Nothing but to show you how a king may go a progress through the guts
of a beggar.

KING.
Where is Polonius?

HAMLET.
In heaven. Send thither to see. If your messenger find him not there,
seek him i’ th’other place yourself. But indeed, if you find him not
within this month, you shall nose him as you go up the stairs into the
lobby.

KING.
[_To some Attendants._] Go seek him there.

HAMLET.
He will stay till you come.

[_Exeunt Attendants._]

KING.
Hamlet, this deed, for thine especial safety,—
Which we do tender, as we dearly grieve
For that which thou hast done,—must send thee hence
With fiery quickness. Therefore prepare thyself;
The bark is ready, and the wind at help,
Th’associates tend, and everything is bent
For England.

HAMLET.
For England?

KING.
Ay, Hamlet.

HAMLET.
Good.

KING.
So is it, if thou knew’st our purposes.

HAMLET.
I see a cherub that sees them. But, come; for England! Farewell, dear
mother.

KING.
Thy loving father, Hamlet.

HAMLET.
My mother. Father and mother is man and wife; man and wife is one
flesh; and so, my mother. Come, for England.

[_Exit._]

KING.
Follow him at foot. Tempt him with speed aboard;
Delay it not; I’ll have him hence tonight.
Away, for everything is seal’d and done
That else leans on th’affair. Pray you make haste.

[_Exeunt Rosencrantz and Guildenstern._]

And England, if my love thou hold’st at aught,—
As my great power thereof may give thee sense,
Since yet thy cicatrice looks raw and red
After the Danish sword, and thy free awe
Pays homage to us,—thou mayst not coldly set
Our sovereign process, which imports at full,
By letters conjuring to that effect,
The present death of Hamlet. Do it, England;
For like the hectic in my blood he rages,
And thou must cure me. Till I know ’tis done,
Howe’er my haps, my joys were ne’er begun.

[_Exit._]

 SCENE IV. A plain in Denmark.

Enter Fortinbras and Forces marching.

FORTINBRAS.
Go, Captain, from me greet the Danish king.
Tell him that by his license, Fortinbras
Craves the conveyance of a promis’d march
Over his kingdom. You know the rendezvous.
If that his Majesty would aught with us,
We shall express our duty in his eye;
And let him know so.

CAPTAIN.
I will do’t, my lord.

FORTINBRAS.
Go softly on.

[_Exeunt all but the Captain._]

Enter Hamlet, Rosencrantz, Guildenstern &c.

HAMLET.
Good sir, whose powers are these?

CAPTAIN.
They are of Norway, sir.

HAMLET.
How purpos’d, sir, I pray you?

CAPTAIN.
Against some part of Poland.

HAMLET.
Who commands them, sir?

CAPTAIN.
The nephew to old Norway, Fortinbras.

HAMLET.
Goes it against the main of Poland, sir,
Or for some frontier?

CAPTAIN.
Truly to speak, and with no addition,
We go to gain a little patch of ground
That hath in it no profit but the name.
To pay five ducats, five, I would not farm it;
Nor will it yield to Norway or the Pole
A ranker rate, should it be sold in fee.

HAMLET.
Why, then the Polack never will defend it.

CAPTAIN.
Yes, it is already garrison’d.

HAMLET.
Two thousand souls and twenty thousand ducats
Will not debate the question of this straw!
This is th’imposthume of much wealth and peace,
That inward breaks, and shows no cause without
Why the man dies. I humbly thank you, sir.

CAPTAIN.
God b’ wi’ you, sir.

[_Exit._]

ROSENCRANTZ.
Will’t please you go, my lord?

HAMLET.
I’ll be with you straight. Go a little before.

[_Exeunt all but Hamlet._]

How all occasions do inform against me,
And spur my dull revenge. What is a man
If his chief good and market of his time
Be but to sleep and feed? A beast, no more.
Sure he that made us with such large discourse,
Looking before and after, gave us not
That capability and godlike reason
To fust in us unus’d. Now whether it be
Bestial oblivion, or some craven scruple
Of thinking too precisely on th’event,—
A thought which, quarter’d, hath but one part wisdom
And ever three parts coward,—I do not know
Why yet I live to say this thing’s to do,
Sith I have cause, and will, and strength, and means
To do’t. Examples gross as earth exhort me,
Witness this army of such mass and charge,
Led by a delicate and tender prince,
Whose spirit, with divine ambition puff’d,
Makes mouths at the invisible event,
Exposing what is mortal and unsure
To all that fortune, death, and danger dare,
Even for an eggshell. Rightly to be great
Is not to stir without great argument,
But greatly to find quarrel in a straw
When honour’s at the stake. How stand I then,
That have a father kill’d, a mother stain’d,
Excitements of my reason and my blood,
And let all sleep, while to my shame I see
The imminent death of twenty thousand men
That, for a fantasy and trick of fame,
Go to their graves like beds, fight for a plot
Whereon the numbers cannot try the cause,
Which is not tomb enough and continent
To hide the slain? O, from this time forth,
My thoughts be bloody or be nothing worth.

[_Exit._]

 SCENE V. Elsinore. A room in the Castle.

Enter Queen, Horatio and a Gentleman.

QUEEN.
I will not speak with her.

GENTLEMAN.
She is importunate, indeed distract.
Her mood will needs be pitied.

QUEEN.
What would she have?

GENTLEMAN.
She speaks much of her father; says she hears
There’s tricks i’ th’ world, and hems, and beats her heart,
Spurns enviously at straws, speaks things in doubt,
That carry but half sense. Her speech is nothing,
Yet the unshaped use of it doth move
The hearers to collection; they aim at it,
And botch the words up fit to their own thoughts,
Which, as her winks, and nods, and gestures yield them,
Indeed would make one think there might be thought,
Though nothing sure, yet much unhappily.
’Twere good she were spoken with, for she may strew
Dangerous conjectures in ill-breeding minds.

QUEEN.
Let her come in.

[_Exit Gentleman._]

To my sick soul, as sin’s true nature is,
Each toy seems prologue to some great amiss.
So full of artless jealousy is guilt,
It spills itself in fearing to be spilt.

Enter Ophelia.

OPHELIA.
Where is the beauteous Majesty of Denmark?

QUEEN.
How now, Ophelia?

OPHELIA.
[_Sings._]
   How should I your true love know
     From another one?
   By his cockle hat and staff
     And his sandal shoon.

QUEEN.
Alas, sweet lady, what imports this song?

OPHELIA.
Say you? Nay, pray you mark.
[_Sings._]
   He is dead and gone, lady,
     He is dead and gone,
   At his head a grass green turf,
     At his heels a stone.

QUEEN.
Nay, but Ophelia—

OPHELIA.
Pray you mark.
[_Sings._]
   White his shroud as the mountain snow.

Enter King.

QUEEN.
Alas, look here, my lord!

OPHELIA.
[_Sings._]
     Larded all with sweet flowers;
   Which bewept to the grave did not go
     With true-love showers.

KING.
How do you, pretty lady?

OPHELIA.
Well, God dild you! They say the owl was a baker’s daughter. Lord, we
know what we are, but know not what we may be. God be at your table!

KING.
Conceit upon her father.

OPHELIA.
Pray you, let’s have no words of this; but when they ask you what it
means, say you this:
[_Sings._]
   Tomorrow is Saint Valentine’s day,
     All in the morning betime,
   And I a maid at your window,
     To be your Valentine.

   Then up he rose and donn’d his clothes,
     And dupp’d the chamber door,
   Let in the maid, that out a maid
     Never departed more.

KING.
Pretty Ophelia!

OPHELIA.
Indeed la, without an oath, I’ll make an end on’t.
[_Sings._]
   By Gis and by Saint Charity,
     Alack, and fie for shame!
   Young men will do’t if they come to’t;
     By Cock, they are to blame.

   Quoth she, before you tumbled me,
     You promis’d me to wed.
   So would I ha’ done, by yonder sun,
     An thou hadst not come to my bed.

KING.
How long hath she been thus?

OPHELIA.
I hope all will be well. We must be patient. But I cannot choose but
weep, to think they would lay him i’ th’ cold ground. My brother shall
know of it. And so I thank you for your good counsel. Come, my coach!
Good night, ladies; good night, sweet ladies; good night, good night.

[_Exit._]

KING.
Follow her close; give her good watch, I pray you.

[_Exit Horatio._]

O, this is the poison of deep grief; it springs
All from her father’s death. O Gertrude, Gertrude,
When sorrows come, they come not single spies,
But in battalions. First, her father slain;
Next, your son gone; and he most violent author
Of his own just remove; the people muddied,
Thick, and unwholesome in their thoughts and whispers
For good Polonius’ death; and we have done but greenly
In hugger-mugger to inter him. Poor Ophelia
Divided from herself and her fair judgement,
Without the which we are pictures or mere beasts.
Last, and as much containing as all these,
Her brother is in secret come from France,
Feeds on his wonder, keeps himself in clouds,
And wants not buzzers to infect his ear
With pestilent speeches of his father’s death,
Wherein necessity, of matter beggar’d,
Will nothing stick our person to arraign
In ear and ear. O my dear Gertrude, this,
Like to a murdering piece, in many places
Gives me superfluous death.

[_A noise within._]

QUEEN.
Alack, what noise is this?

KING.
Where are my Switzers? Let them guard the door.

Enter a Gentleman.

What is the matter?

GENTLEMAN.
Save yourself, my lord.
The ocean, overpeering of his list,
Eats not the flats with more impetuous haste
Than young Laertes, in a riotous head,
O’erbears your offices. The rabble call him lord,
And, as the world were now but to begin,
Antiquity forgot, custom not known,
The ratifiers and props of every word,
They cry ‘Choose we! Laertes shall be king!’
Caps, hands, and tongues applaud it to the clouds,
‘Laertes shall be king, Laertes king.’

QUEEN.
How cheerfully on the false trail they cry.
O, this is counter, you false Danish dogs.

[_A noise within._]

KING.
The doors are broke.

Enter Laertes, armed; Danes following.

LAERTES.
Where is this king?—Sirs, stand you all without.

Danes.
No, let’s come in.

LAERTES.
I pray you, give me leave.

DANES.
We will, we will.

[_They retire without the door._]

LAERTES.
I thank you. Keep the door. O thou vile king,
Give me my father.

QUEEN.
Calmly, good Laertes.

LAERTES.
That drop of blood that’s calm proclaims me bastard;
Cries cuckold to my father, brands the harlot
Even here between the chaste unsmirched brow
Of my true mother.

KING.
What is the cause, Laertes,
That thy rebellion looks so giant-like?—
Let him go, Gertrude. Do not fear our person.
There’s such divinity doth hedge a king,
That treason can but peep to what it would,
Acts little of his will.—Tell me, Laertes,
Why thou art thus incens’d.—Let him go, Gertrude:—
Speak, man.

LAERTES.
Where is my father?

KING.
Dead.

QUEEN.
But not by him.

KING.
Let him demand his fill.

LAERTES.
How came he dead? I’ll not be juggled with.
To hell, allegiance! Vows, to the blackest devil!
Conscience and grace, to the profoundest pit!
I dare damnation. To this point I stand,
That both the worlds, I give to negligence,
Let come what comes; only I’ll be reveng’d
Most throughly for my father.

KING.
Who shall stay you?

LAERTES.
My will, not all the world.
And for my means, I’ll husband them so well,
They shall go far with little.

KING.
Good Laertes,
If you desire to know the certainty
Of your dear father’s death, is’t writ in your revenge
That, sweepstake, you will draw both friend and foe,
Winner and loser?

LAERTES.
None but his enemies.

KING.
Will you know them then?

LAERTES.
To his good friends thus wide I’ll ope my arms;
And, like the kind life-rendering pelican,
Repast them with my blood.

KING.
Why, now you speak
Like a good child and a true gentleman.
That I am guiltless of your father’s death,
And am most sensibly in grief for it,
It shall as level to your judgement ’pear
As day does to your eye.

DANES.
[_Within._] Let her come in.

LAERTES.
How now! What noise is that?

Re-enter Ophelia, fantastically dressed with straws and flowers.

O heat, dry up my brains. Tears seven times salt,
Burn out the sense and virtue of mine eye.
By heaven, thy madness shall be paid by weight,
Till our scale turn the beam. O rose of May!
Dear maid, kind sister, sweet Ophelia!
O heavens, is’t possible a young maid’s wits
Should be as mortal as an old man’s life?
Nature is fine in love, and where ’tis fine,
It sends some precious instance of itself
After the thing it loves.

OPHELIA.
[_Sings._]
   They bore him barefac’d on the bier,
   Hey non nonny, nonny, hey nonny
   And on his grave rain’d many a tear.—
   Fare you well, my dove!

LAERTES.
Hadst thou thy wits, and didst persuade revenge,
It could not move thus.

OPHELIA.
You must sing ‘Down a-down, and you call him a-down-a.’ O, how the
wheel becomes it! It is the false steward that stole his master’s
daughter.

LAERTES.
This nothing’s more than matter.

OPHELIA.
There’s rosemary, that’s for remembrance; pray love, remember. And
there is pansies, that’s for thoughts.

LAERTES.
A document in madness, thoughts and remembrance fitted.

OPHELIA.
There’s fennel for you, and columbines. There’s rue for you; and here’s
some for me. We may call it herb of grace o’ Sundays. O you must wear
your rue with a difference. There’s a daisy. I would give you some
violets, but they wither’d all when my father died. They say he made a
good end.
[_Sings._]
   For bonny sweet Robin is all my joy.

LAERTES.
Thought and affliction, passion, hell itself
She turns to favour and to prettiness.

OPHELIA.
[_Sings._]
   And will he not come again?
   And will he not come again?
     No, no, he is dead,
     Go to thy death-bed,
   He never will come again.

   His beard was as white as snow,
   All flaxen was his poll.
     He is gone, he is gone,
     And we cast away moan.
   God ha’ mercy on his soul.

And of all Christian souls, I pray God. God b’ wi’ ye.

[_Exit._]

LAERTES.
Do you see this, O God?

KING.
Laertes, I must commune with your grief,
Or you deny me right. Go but apart,
Make choice of whom your wisest friends you will,
And they shall hear and judge ’twixt you and me.
If by direct or by collateral hand
They find us touch’d, we will our kingdom give,
Our crown, our life, and all that we call ours
To you in satisfaction; but if not,
Be you content to lend your patience to us,
And we shall jointly labour with your soul
To give it due content.

LAERTES.
Let this be so;
His means of death, his obscure burial,—
No trophy, sword, nor hatchment o’er his bones,
No noble rite, nor formal ostentation,—
Cry to be heard, as ’twere from heaven to earth,
That I must call’t in question.

KING.
So you shall.
And where th’offence is let the great axe fall.
I pray you go with me.

[_Exeunt._]

 SCENE VI. Another room in the Castle.

Enter Horatio and a Servant.

HORATIO.
What are they that would speak with me?

SERVANT.
Sailors, sir. They say they have letters for you.

HORATIO.
Let them come in.

[_Exit Servant._]

I do not know from what part of the world
I should be greeted, if not from Lord Hamlet.

Enter Sailors.

FIRST SAILOR.
God bless you, sir.

HORATIO.
Let him bless thee too.

FIRST SAILOR.
He shall, sir, and’t please him. There’s a letter for you, sir. It
comes from th’ambassador that was bound for England; if your name be
Horatio, as I am let to know it is.

HORATIO.
[_Reads._] ‘Horatio, when thou shalt have overlooked this, give these
fellows some means to the King. They have letters for him. Ere we were
two days old at sea, a pirate of very warlike appointment gave us
chase. Finding ourselves too slow of sail, we put on a compelled
valour, and in the grapple I boarded them. On the instant they got
clear of our ship, so I alone became their prisoner. They have dealt
with me like thieves of mercy. But they knew what they did; I am to do
a good turn for them. Let the King have the letters I have sent, and
repair thou to me with as much haste as thou wouldst fly death. I have
words to speak in thine ear will make thee dumb; yet are they much too
light for the bore of the matter. These good fellows will bring thee
where I am. Rosencrantz and Guildenstern hold their course for England:
of them I have much to tell thee. Farewell.
     He that thou knowest thine,
     HAMLET.’

Come, I will give you way for these your letters,
And do’t the speedier, that you may direct me
To him from whom you brought them.

[_Exeunt._]

 SCENE VII. Another room in the Castle.

Enter King and Laertes.

KING.
Now must your conscience my acquittance seal,
And you must put me in your heart for friend,
Sith you have heard, and with a knowing ear,
That he which hath your noble father slain
Pursu’d my life.

LAERTES.
It well appears. But tell me
Why you proceeded not against these feats,
So crimeful and so capital in nature,
As by your safety, wisdom, all things else,
You mainly were stirr’d up.

KING.
O, for two special reasons,
Which may to you, perhaps, seem much unsinew’d,
But yet to me they are strong. The Queen his mother
Lives almost by his looks; and for myself,—
My virtue or my plague, be it either which,—
She’s so conjunctive to my life and soul,
That, as the star moves not but in his sphere,
I could not but by her. The other motive,
Why to a public count I might not go,
Is the great love the general gender bear him,
Who, dipping all his faults in their affection,
Would like the spring that turneth wood to stone,
Convert his gyves to graces; so that my arrows,
Too slightly timber’d for so loud a wind,
Would have reverted to my bow again,
And not where I had aim’d them.

LAERTES.
And so have I a noble father lost,
A sister driven into desperate terms,
Whose worth, if praises may go back again,
Stood challenger on mount of all the age
For her perfections. But my revenge will come.

KING.
Break not your sleeps for that. You must not think
That we are made of stuff so flat and dull
That we can let our beard be shook with danger,
And think it pastime. You shortly shall hear more.
I lov’d your father, and we love ourself,
And that, I hope, will teach you to imagine—

Enter a Messenger.

How now? What news?

MESSENGER.
Letters, my lord, from Hamlet.
This to your Majesty; this to the Queen.

KING.
From Hamlet! Who brought them?

MESSENGER.
Sailors, my lord, they say; I saw them not.
They were given me by Claudio. He receiv’d them
Of him that brought them.

KING.
Laertes, you shall hear them.
Leave us.

[_Exit Messenger._]

[_Reads._] ‘High and mighty, you shall know I am set naked on your
kingdom. Tomorrow shall I beg leave to see your kingly eyes. When I
shall, first asking your pardon thereunto, recount the occasions of my
sudden and more strange return.
     HAMLET.’

What should this mean? Are all the rest come back?
Or is it some abuse, and no such thing?

LAERTES.
Know you the hand?

KING.
’Tis Hamlet’s character. ‘Naked!’
And in a postscript here he says ‘alone.’
Can you advise me?

LAERTES.
I am lost in it, my lord. But let him come,
It warms the very sickness in my heart
That I shall live and tell him to his teeth,
‘Thus diest thou.’

KING.
If it be so, Laertes,—
As how should it be so? How otherwise?—
Will you be rul’d by me?

LAERTES.
Ay, my lord;
So you will not o’errule me to a peace.

KING.
To thine own peace. If he be now return’d,
As checking at his voyage, and that he means
No more to undertake it, I will work him
To an exploit, now ripe in my device,
Under the which he shall not choose but fall;
And for his death no wind shall breathe,
But even his mother shall uncharge the practice
And call it accident.

LAERTES.
My lord, I will be rul’d;
The rather if you could devise it so
That I might be the organ.

KING.
It falls right.
You have been talk’d of since your travel much,
And that in Hamlet’s hearing, for a quality
Wherein they say you shine. Your sum of parts
Did not together pluck such envy from him
As did that one, and that, in my regard,
Of the unworthiest siege.

LAERTES.
What part is that, my lord?

KING.
A very riband in the cap of youth,
Yet needful too, for youth no less becomes
The light and careless livery that it wears
Than settled age his sables and his weeds,
Importing health and graveness. Two months since
Here was a gentleman of Normandy,—
I’ve seen myself, and serv’d against, the French,
And they can well on horseback, but this gallant
Had witchcraft in’t. He grew unto his seat,
And to such wondrous doing brought his horse,
As had he been incorps’d and demi-natur’d
With the brave beast. So far he topp’d my thought
That I in forgery of shapes and tricks,
Come short of what he did.

LAERTES.
A Norman was’t?

KING.
A Norman.

LAERTES.
Upon my life, Lamord.

KING.
The very same.

LAERTES.
I know him well. He is the brooch indeed
And gem of all the nation.

KING.
He made confession of you,
And gave you such a masterly report
For art and exercise in your defence,
And for your rapier most especially,
That he cried out ’twould be a sight indeed
If one could match you. The scrimers of their nation
He swore had neither motion, guard, nor eye,
If you oppos’d them. Sir, this report of his
Did Hamlet so envenom with his envy
That he could nothing do but wish and beg
Your sudden coming o’er to play with him.
Now, out of this,—

LAERTES.
What out of this, my lord?

KING.
Laertes, was your father dear to you?
Or are you like the painting of a sorrow,
A face without a heart?

LAERTES.
Why ask you this?

KING.
Not that I think you did not love your father,
But that I know love is begun by time,
And that I see, in passages of proof,
Time qualifies the spark and fire of it.
There lives within the very flame of love
A kind of wick or snuff that will abate it;
And nothing is at a like goodness still,
For goodness, growing to a pleurisy,
Dies in his own too much. That we would do,
We should do when we would; for this ‘would’ changes,
And hath abatements and delays as many
As there are tongues, are hands, are accidents;
And then this ‘should’ is like a spendthrift sigh
That hurts by easing. But to the quick o’ th’ulcer:
Hamlet comes back: what would you undertake
To show yourself your father’s son in deed,
More than in words?

LAERTES.
To cut his throat i’ th’ church.

KING.
No place, indeed, should murder sanctuarize;
Revenge should have no bounds. But good Laertes,
Will you do this, keep close within your chamber.
Hamlet return’d shall know you are come home:
We’ll put on those shall praise your excellence,
And set a double varnish on the fame
The Frenchman gave you, bring you in fine together
And wager on your heads. He, being remiss,
Most generous, and free from all contriving,
Will not peruse the foils; so that with ease,
Or with a little shuffling, you may choose
A sword unbated, and in a pass of practice,
Requite him for your father.

LAERTES.
I will do’t.
And for that purpose I’ll anoint my sword.
I bought an unction of a mountebank
So mortal that, but dip a knife in it,
Where it draws blood no cataplasm so rare,
Collected from all simples that have virtue
Under the moon, can save the thing from death
This is but scratch’d withal. I’ll touch my point
With this contagion, that if I gall him slightly,
It may be death.

KING.
Let’s further think of this,
Weigh what convenience both of time and means
May fit us to our shape. If this should fail,
And that our drift look through our bad performance.
’Twere better not assay’d. Therefore this project
Should have a back or second, that might hold
If this did blast in proof. Soft, let me see.
We’ll make a solemn wager on your cunnings,—
I ha’t! When in your motion you are hot and dry,
As make your bouts more violent to that end,
And that he calls for drink, I’ll have prepar’d him
A chalice for the nonce; whereon but sipping,
If he by chance escape your venom’d stuck,
Our purpose may hold there.

Enter Queen.

How now, sweet Queen?

QUEEN.
One woe doth tread upon another’s heel,
So fast they follow. Your sister’s drown’d, Laertes.

LAERTES.
Drown’d! O, where?

QUEEN.
There is a willow grows aslant a brook,
That shows his hoary leaves in the glassy stream.
There with fantastic garlands did she make
Of crow-flowers, nettles, daisies, and long purples,
That liberal shepherds give a grosser name,
But our cold maids do dead men’s fingers call them.
There on the pendant boughs her coronet weeds
Clamb’ring to hang, an envious sliver broke,
When down her weedy trophies and herself
Fell in the weeping brook. Her clothes spread wide,
And mermaid-like, awhile they bore her up,
Which time she chaunted snatches of old tunes,
As one incapable of her own distress,
Or like a creature native and indued
Unto that element. But long it could not be
Till that her garments, heavy with their drink,
Pull’d the poor wretch from her melodious lay
To muddy death.

LAERTES.
Alas, then she is drown’d?

QUEEN.
Drown’d, drown’d.

LAERTES.
Too much of water hast thou, poor Ophelia,
And therefore I forbid my tears. But yet
It is our trick; nature her custom holds,
Let shame say what it will. When these are gone,
The woman will be out. Adieu, my lord,
I have a speech of fire, that fain would blaze,
But that this folly douts it.

[_Exit._]

KING.
Let’s follow, Gertrude;
How much I had to do to calm his rage!
Now fear I this will give it start again;
Therefore let’s follow.

[_Exeunt._]




ACT V

SCENE I. A churchyard.


Enter two Clowns with spades, &c.

FIRST CLOWN.
Is she to be buried in Christian burial, when she wilfully seeks her
own salvation?

SECOND CLOWN.
I tell thee she is, and therefore make her grave straight. The crowner
hath sat on her, and finds it Christian burial.

FIRST CLOWN.
How can that be, unless she drowned herself in her own defence?

SECOND CLOWN.
Why, ’tis found so.

FIRST CLOWN.
It must be _se offendendo_, it cannot be else. For here lies the point:
if I drown myself wittingly, it argues an act: and an act hath three
branches. It is to act, to do, and to perform: argal, she drowned
herself wittingly.

SECOND CLOWN.
Nay, but hear you, goodman delver,—

FIRST CLOWN.
Give me leave. Here lies the water; good. Here stands the man; good. If
the man go to this water and drown himself, it is, will he nill he, he
goes,—mark you that. But if the water come to him and drown him, he
drowns not himself. Argal, he that is not guilty of his own death
shortens not his own life.

SECOND CLOWN.
But is this law?

FIRST CLOWN.
Ay, marry, is’t, crowner’s quest law.

SECOND CLOWN.
Will you ha’ the truth on’t? If this had not been a gentlewoman, she
should have been buried out o’ Christian burial.

FIRST CLOWN.
Why, there thou say’st. And the more pity that great folk should have
countenance in this world to drown or hang themselves more than their
even Christian. Come, my spade. There is no ancient gentlemen but
gardeners, ditchers, and grave-makers: they hold up Adam’s profession.

SECOND CLOWN.
Was he a gentleman?

FIRST CLOWN.
He was the first that ever bore arms.

SECOND CLOWN.
Why, he had none.

FIRST CLOWN.
What, art a heathen? How dost thou understand the Scripture? The
Scripture says Adam digg’d. Could he dig without arms? I’ll put another
question to thee. If thou answerest me not to the purpose, confess
thyself—

SECOND CLOWN.
Go to.

FIRST CLOWN.
What is he that builds stronger than either the mason, the shipwright,
or the carpenter?

SECOND CLOWN.
The gallows-maker; for that frame outlives a thousand tenants.

FIRST CLOWN.
I like thy wit well in good faith, the gallows does well. But how does
it well? It does well to those that do ill. Now, thou dost ill to say
the gallows is built stronger than the church; argal, the gallows may
do well to thee. To’t again, come.

SECOND CLOWN.
Who builds stronger than a mason, a shipwright, or a carpenter?

FIRST CLOWN.
Ay, tell me that, and unyoke.

SECOND CLOWN.
Marry, now I can tell.

FIRST CLOWN.
To’t.

SECOND CLOWN.
Mass, I cannot tell.

Enter Hamlet and Horatio, at a distance.

FIRST CLOWN.
Cudgel thy brains no more about it, for your dull ass will not mend his
pace with beating; and when you are asked this question next, say ‘a
grave-maker’. The houses he makes last till doomsday. Go, get thee to
Yaughan; fetch me a stoup of liquor.

[_Exit Second Clown._]

[_Digs and sings._]

   In youth when I did love, did love,
     Methought it was very sweet;
   To contract, O, the time for, a, my behove,
     O methought there was nothing meet.

HAMLET.
Has this fellow no feeling of his business, that he sings at
grave-making?

HORATIO.
Custom hath made it in him a property of easiness.

HAMLET.
’Tis e’en so; the hand of little employment hath the daintier sense.

FIRST CLOWN.
[_Sings._]
   But age with his stealing steps
     Hath claw’d me in his clutch,
   And hath shipp’d me into the land,
     As if I had never been such.

[_Throws up a skull._]

HAMLET.
That skull had a tongue in it, and could sing once. How the knave jowls
it to th’ ground, as if ’twere Cain’s jawbone, that did the first
murder! This might be the pate of a politician which this ass now
o’er-offices, one that would circumvent God, might it not?

HORATIO.
It might, my lord.

HAMLET.
Or of a courtier, which could say ‘Good morrow, sweet lord! How dost
thou, good lord?’ This might be my lord such-a-one, that praised my
lord such-a-one’s horse when he meant to beg it, might it not?

HORATIO.
Ay, my lord.

HAMLET.
Why, e’en so: and now my Lady Worm’s; chapless, and knocked about the
mazard with a sexton’s spade. Here’s fine revolution, an we had the
trick to see’t. Did these bones cost no more the breeding but to play
at loggets with ’em? Mine ache to think on’t.

FIRST CLOWN.
[_Sings._]
   A pickaxe and a spade, a spade,
     For and a shrouding-sheet;
   O, a pit of clay for to be made
     For such a guest is meet.

[_Throws up another skull._]

HAMLET.
There’s another. Why may not that be the skull of a lawyer? Where be
his quiddits now, his quillets, his cases, his tenures, and his tricks?
Why does he suffer this rude knave now to knock him about the sconce
with a dirty shovel, and will not tell him of his action of battery?
Hum. This fellow might be in’s time a great buyer of land, with his
statutes, his recognizances, his fines, his double vouchers, his
recoveries. Is this the fine of his fines, and the recovery of his
recoveries, to have his fine pate full of fine dirt? Will his vouchers
vouch him no more of his purchases, and double ones too, than the
length and breadth of a pair of indentures? The very conveyances of his
lands will scarcely lie in this box; and must the inheritor himself
have no more, ha?

HORATIO.
Not a jot more, my lord.

HAMLET.
Is not parchment made of sheep-skins?

HORATIO.
Ay, my lord, and of calf-skins too.

HAMLET.
They are sheep and calves which seek out assurance in that. I will
speak to this fellow.—Whose grave’s this, sir?

FIRST CLOWN.
Mine, sir.
[_Sings._]
   O, a pit of clay for to be made
     For such a guest is meet.

HAMLET.
I think it be thine indeed, for thou liest in’t.

FIRST CLOWN.
You lie out on’t, sir, and therefore ’tis not yours.
For my part, I do not lie in’t, yet it is mine.

HAMLET.
Thou dost lie in’t, to be in’t and say it is thine. ’Tis for the dead,
not for the quick; therefore thou liest.

FIRST CLOWN.
’Tis a quick lie, sir; ’t will away again from me to you.

HAMLET.
What man dost thou dig it for?

FIRST CLOWN.
For no man, sir.

HAMLET.
What woman then?

FIRST CLOWN.
For none neither.

HAMLET.
Who is to be buried in’t?

FIRST CLOWN.
One that was a woman, sir; but, rest her soul, she’s dead.

HAMLET.
How absolute the knave is! We must speak by the card, or equivocation
will undo us. By the Lord, Horatio, these three years I have taken note
of it, the age is grown so picked that the toe of the peasant comes so
near the heel of the courtier he galls his kibe.—How long hast thou
been a grave-maker?

FIRST CLOWN.
Of all the days i’ th’ year, I came to’t that day that our last King
Hamlet o’ercame Fortinbras.

HAMLET.
How long is that since?

FIRST CLOWN.
Cannot you tell that? Every fool can tell that. It was the very day
that young Hamlet was born,—he that is mad, and sent into England.

HAMLET.
Ay, marry, why was he sent into England?

FIRST CLOWN.
Why, because he was mad; he shall recover his wits there; or if he do
not, it’s no great matter there.

HAMLET.
Why?

FIRST CLOWN.
’Twill not be seen in him there; there the men are as mad as he.

HAMLET.
How came he mad?

FIRST CLOWN.
Very strangely, they say.

HAMLET.
How strangely?

FIRST CLOWN.
Faith, e’en with losing his wits.

HAMLET.
Upon what ground?

FIRST CLOWN.
Why, here in Denmark. I have been sexton here, man and boy, thirty
years.

HAMLET.
How long will a man lie i’ th’earth ere he rot?

FIRST CLOWN.
Faith, if he be not rotten before he die,—as we have many pocky corses
nowadays that will scarce hold the laying in,—he will last you some
eight year or nine year. A tanner will last you nine year.

HAMLET.
Why he more than another?

FIRST CLOWN.
Why, sir, his hide is so tann’d with his trade that he will keep out
water a great while. And your water is a sore decayer of your whoreson
dead body. Here’s a skull now; this skull hath lain in the earth
three-and-twenty years.

HAMLET.
Whose was it?

FIRST CLOWN.
A whoreson, mad fellow’s it was. Whose do you think it was?

HAMLET.
Nay, I know not.

FIRST CLOWN.
A pestilence on him for a mad rogue! A pour’d a flagon of Rhenish on my
head once. This same skull, sir, was Yorick’s skull, the King’s jester.

HAMLET.
This?

FIRST CLOWN.
E’en that.

HAMLET.
Let me see. [_Takes the skull._] Alas, poor Yorick. I knew him,
Horatio, a fellow of infinite jest, of most excellent fancy. He hath
borne me on his back a thousand times; and now, how abhorred in my
imagination it is! My gorge rises at it. Here hung those lips that I
have kiss’d I know not how oft. Where be your gibes now? your gambols?
your songs? your flashes of merriment, that were wont to set the table
on a roar? Not one now, to mock your own grinning? Quite chop-fallen?
Now get you to my lady’s chamber, and tell her, let her paint an inch
thick, to this favour she must come. Make her laugh at that.—Prithee,
Horatio, tell me one thing.

HORATIO.
What’s that, my lord?

HAMLET.
Dost thou think Alexander looked o’ this fashion i’ th’earth?

HORATIO.
E’en so.

HAMLET.
And smelt so? Pah!

[_Throws down the skull._]

HORATIO.
E’en so, my lord.

HAMLET.
To what base uses we may return, Horatio! Why may not imagination trace
the noble dust of Alexander till he find it stopping a bung-hole?

HORATIO.
’Twere to consider too curiously to consider so.

HAMLET.
No, faith, not a jot. But to follow him thither with modesty enough,
and likelihood to lead it; as thus. Alexander died, Alexander was
buried, Alexander returneth into dust; the dust is earth; of earth we
make loam; and why of that loam whereto he was converted might they not
stop a beer-barrel?
Imperious Caesar, dead and turn’d to clay,
Might stop a hole to keep the wind away.
O, that that earth which kept the world in awe
Should patch a wall t’expel the winter’s flaw.
But soft! but soft! aside! Here comes the King.

Enter priests, &c, in procession; the corpse of Ophelia, Laertes and
Mourners following; King, Queen, their Trains, &c.

The Queen, the courtiers. Who is that they follow?
And with such maimed rites? This doth betoken
The corse they follow did with desperate hand
Fordo it own life. ’Twas of some estate.
Couch we awhile and mark.

[_Retiring with Horatio._]

LAERTES.
What ceremony else?

HAMLET.
That is Laertes, a very noble youth. Mark.

LAERTES.
What ceremony else?

PRIEST.
Her obsequies have been as far enlarg’d
As we have warranties. Her death was doubtful;
And but that great command o’ersways the order,
She should in ground unsanctified have lodg’d
Till the last trumpet. For charitable prayers,
Shards, flints, and pebbles should be thrown on her.
Yet here she is allowed her virgin rites,
Her maiden strewments, and the bringing home
Of bell and burial.

LAERTES.
Must there no more be done?

PRIEST.
No more be done.
We should profane the service of the dead
To sing sage requiem and such rest to her
As to peace-parted souls.

LAERTES.
Lay her i’ th’earth,
And from her fair and unpolluted flesh
May violets spring. I tell thee, churlish priest,
A minist’ring angel shall my sister be
When thou liest howling.

HAMLET.
What, the fair Ophelia?

QUEEN.
[_Scattering flowers._] Sweets to the sweet. Farewell.
I hop’d thou shouldst have been my Hamlet’s wife;
I thought thy bride-bed to have deck’d, sweet maid,
And not have strew’d thy grave.

LAERTES.
O, treble woe
Fall ten times treble on that cursed head
Whose wicked deed thy most ingenious sense
Depriv’d thee of. Hold off the earth a while,
Till I have caught her once more in mine arms.
[_Leaps into the grave._]
Now pile your dust upon the quick and dead,
Till of this flat a mountain you have made,
To o’ertop old Pelion or the skyish head
Of blue Olympus.

HAMLET.
[_Advancing._]
What is he whose grief
Bears such an emphasis? whose phrase of sorrow
Conjures the wand’ring stars, and makes them stand
Like wonder-wounded hearers? This is I,
Hamlet the Dane.
[_Leaps into the grave._]

LAERTES.
[_Grappling with him._] The devil take thy soul!

HAMLET.
Thou pray’st not well.
I prithee take thy fingers from my throat;
For though I am not splenative and rash,
Yet have I in me something dangerous,
Which let thy wiseness fear. Away thy hand!

KING.
Pluck them asunder.

QUEEN.
Hamlet! Hamlet!

All.
Gentlemen!

HORATIO.
Good my lord, be quiet.

[_The Attendants part them, and they come out of the grave._]

HAMLET.
Why, I will fight with him upon this theme
Until my eyelids will no longer wag.

QUEEN.
O my son, what theme?

HAMLET.
I lov’d Ophelia; forty thousand brothers
Could not, with all their quantity of love,
Make up my sum. What wilt thou do for her?

KING.
O, he is mad, Laertes.

QUEEN.
For love of God forbear him!

HAMLET.
’Swounds, show me what thou’lt do:
Woul’t weep? woul’t fight? woul’t fast? woul’t tear thyself?
Woul’t drink up eisel? eat a crocodile?
I’ll do’t. Dost thou come here to whine?
To outface me with leaping in her grave?
Be buried quick with her, and so will I.
And if thou prate of mountains, let them throw
Millions of acres on us, till our ground,
Singeing his pate against the burning zone,
Make Ossa like a wart. Nay, an thou’lt mouth,
I’ll rant as well as thou.

QUEEN.
This is mere madness:
And thus awhile the fit will work on him;
Anon, as patient as the female dove,
When that her golden couplets are disclos’d,
His silence will sit drooping.

HAMLET.
Hear you, sir;
What is the reason that you use me thus?
I lov’d you ever. But it is no matter.
Let Hercules himself do what he may,
The cat will mew, and dog will have his day.

[_Exit._]

KING.
I pray thee, good Horatio, wait upon him.

[_Exit Horatio._]

[_To Laertes_]
Strengthen your patience in our last night’s speech;
We’ll put the matter to the present push.—
Good Gertrude, set some watch over your son.
This grave shall have a living monument.
An hour of quiet shortly shall we see;
Till then in patience our proceeding be.

[_Exeunt._]

 SCENE II. A hall in the Castle.

Enter Hamlet and Horatio.

HAMLET.
So much for this, sir. Now let me see the other;
You do remember all the circumstance?

HORATIO.
Remember it, my lord!

HAMLET.
Sir, in my heart there was a kind of fighting
That would not let me sleep. Methought I lay
Worse than the mutinies in the bilboes. Rashly,
And prais’d be rashness for it,—let us know,
Our indiscretion sometime serves us well,
When our deep plots do pall; and that should teach us
There’s a divinity that shapes our ends,
Rough-hew them how we will.

HORATIO.
That is most certain.

HAMLET.
Up from my cabin,
My sea-gown scarf’d about me, in the dark
Grop’d I to find out them; had my desire,
Finger’d their packet, and in fine, withdrew
To mine own room again, making so bold,
My fears forgetting manners, to unseal
Their grand commission; where I found, Horatio,
Oh royal knavery! an exact command,
Larded with many several sorts of reasons,
Importing Denmark’s health, and England’s too,
With ho! such bugs and goblins in my life,
That on the supervise, no leisure bated,
No, not to stay the grinding of the axe,
My head should be struck off.

HORATIO.
Is’t possible?

HAMLET.
Here’s the commission, read it at more leisure.
But wilt thou hear me how I did proceed?

HORATIO.
I beseech you.

HAMLET.
Being thus benetted round with villanies,—
Or I could make a prologue to my brains,
They had begun the play,—I sat me down,
Devis’d a new commission, wrote it fair:
I once did hold it, as our statists do,
A baseness to write fair, and labour’d much
How to forget that learning; but, sir, now
It did me yeoman’s service. Wilt thou know
The effect of what I wrote?

HORATIO.
Ay, good my lord.

HAMLET.
An earnest conjuration from the King,
As England was his faithful tributary,
As love between them like the palm might flourish,
As peace should still her wheaten garland wear
And stand a comma ’tween their amities,
And many such-like ‘as’es of great charge,
That on the view and know of these contents,
Without debatement further, more or less,
He should the bearers put to sudden death,
Not shriving-time allow’d.

HORATIO.
How was this seal’d?

HAMLET.
Why, even in that was heaven ordinant.
I had my father’s signet in my purse,
Which was the model of that Danish seal:
Folded the writ up in the form of the other,
Subscrib’d it: gave’t th’impression; plac’d it safely,
The changeling never known. Now, the next day
Was our sea-fight, and what to this was sequent
Thou know’st already.

HORATIO.
So Guildenstern and Rosencrantz go to’t.

HAMLET.
Why, man, they did make love to this employment.
They are not near my conscience; their defeat
Does by their own insinuation grow.
’Tis dangerous when the baser nature comes
Between the pass and fell incensed points
Of mighty opposites.

HORATIO.
Why, what a king is this!

HAMLET.
Does it not, thinks’t thee, stand me now upon,—
He that hath kill’d my king, and whor’d my mother,
Popp’d in between th’election and my hopes,
Thrown out his angle for my proper life,
And with such cozenage—is’t not perfect conscience
To quit him with this arm? And is’t not to be damn’d
To let this canker of our nature come
In further evil?

HORATIO.
It must be shortly known to him from England
What is the issue of the business there.

HAMLET.
It will be short. The interim is mine;
And a man’s life’s no more than to say ‘One’.
But I am very sorry, good Horatio,
That to Laertes I forgot myself;
For by the image of my cause I see
The portraiture of his. I’ll court his favours.
But sure the bravery of his grief did put me
Into a tow’ring passion.

HORATIO.
Peace, who comes here?

Enter Osric.

OSRIC.
Your lordship is right welcome back to Denmark.

HAMLET.
I humbly thank you, sir. Dost know this waterfly?

HORATIO.
No, my good lord.

HAMLET.
Thy state is the more gracious; for ’tis a vice to know him. He hath
much land, and fertile; let a beast be lord of beasts, and his crib
shall stand at the king’s mess; ’tis a chough; but, as I say, spacious
in the possession of dirt.

OSRIC.
Sweet lord, if your lordship were at leisure, I should impart a thing
to you from his Majesty.

HAMLET.
I will receive it with all diligence of spirit. Put your bonnet to his
right use; ’tis for the head.

OSRIC.
I thank your lordship, ’tis very hot.

HAMLET.
No, believe me, ’tis very cold, the wind is northerly.

OSRIC.
It is indifferent cold, my lord, indeed.

HAMLET.
Methinks it is very sultry and hot for my complexion.

OSRIC.
Exceedingly, my lord; it is very sultry,—as ’twere—I cannot tell how.
But, my lord, his Majesty bade me signify to you that he has laid a
great wager on your head. Sir, this is the matter,—

HAMLET.
I beseech you, remember,—

[_Hamlet moves him to put on his hat._]

OSRIC.
Nay, in good faith; for mine ease, in good faith. Sir, here is newly
come to court Laertes; believe me, an absolute gentleman, full of most
excellent differences, of very soft society and great showing. Indeed,
to speak feelingly of him, he is the card or calendar of gentry; for
you shall find in him the continent of what part a gentleman would see.

HAMLET.
Sir, his definement suffers no perdition in you, though I know, to
divide him inventorially would dizzy th’arithmetic of memory, and yet
but yaw neither, in respect of his quick sail. But, in the verity of
extolment, I take him to be a soul of great article and his infusion of
such dearth and rareness as, to make true diction of him, his semblable
is his mirror and who else would trace him his umbrage, nothing more.

OSRIC.
Your lordship speaks most infallibly of him.

HAMLET.
The concernancy, sir? Why do we wrap the gentleman in our more rawer
breath?

OSRIC.
Sir?

HORATIO.
Is’t not possible to understand in another tongue? You will do’t, sir,
really.

HAMLET.
What imports the nomination of this gentleman?

OSRIC.
Of Laertes?

HORATIO.
His purse is empty already, all’s golden words are spent.

HAMLET.
Of him, sir.

OSRIC.
I know you are not ignorant,—

HAMLET.
I would you did, sir; yet in faith if you did, it would not much
approve me. Well, sir?

OSRIC.
You are not ignorant of what excellence Laertes is,—

HAMLET.
I dare not confess that, lest I should compare with him in excellence;
but to know a man well were to know himself.

OSRIC.
I mean, sir, for his weapon; but in the imputation laid on him, by them
in his meed he’s unfellowed.

HAMLET.
What’s his weapon?

OSRIC.
Rapier and dagger.

HAMLET.
That’s two of his weapons. But well.

OSRIC.
The King, sir, hath wager’d with him six Barbary horses, against the
which he has imponed, as I take it, six French rapiers and poniards,
with their assigns, as girdle, hangers, and so. Three of the carriages,
in faith, are very dear to fancy, very responsive to the hilts, most
delicate carriages, and of very liberal conceit.

HAMLET.
What call you the carriages?

HORATIO.
I knew you must be edified by the margin ere you had done.

OSRIC.
The carriages, sir, are the hangers.

HAMLET.
The phrase would be more german to the matter if we could carry cannon
by our sides. I would it might be hangers till then. But on. Six
Barbary horses against six French swords, their assigns, and three
liberal conceited carriages: that’s the French bet against the Danish.
Why is this all imponed, as you call it?

OSRIC.
The King, sir, hath laid that in a dozen passes between you and him, he
shall not exceed you three hits. He hath laid on twelve for nine. And
it would come to immediate trial if your lordship would vouchsafe the
answer.

HAMLET.
How if I answer no?

OSRIC.
I mean, my lord, the opposition of your person in trial.

HAMLET.
Sir, I will walk here in the hall. If it please his Majesty, it is the
breathing time of day with me. Let the foils be brought, the gentleman
willing, and the King hold his purpose, I will win for him if I can; if
not, I will gain nothing but my shame and the odd hits.

OSRIC.
Shall I re-deliver you e’en so?

HAMLET.
To this effect, sir; after what flourish your nature will.

OSRIC.
I commend my duty to your lordship.

HAMLET.
Yours, yours.

[_Exit Osric._]

He does well to commend it himself, there are no tongues else for’s
turn.

HORATIO.
This lapwing runs away with the shell on his head.

HAMLET.
He did comply with his dug before he suck’d it. Thus has he,—and many
more of the same bevy that I know the drossy age dotes on,— only got
the tune of the time and outward habit of encounter; a kind of yeasty
collection, which carries them through and through the most fanned and
winnowed opinions; and do but blow them to their trial, the bubbles are
out.

Enter a Lord.

LORD.
My lord, his Majesty commended him to you by young Osric, who brings
back to him that you attend him in the hall. He sends to know if your
pleasure hold to play with Laertes or that you will take longer time.

HAMLET.
I am constant to my purposes, they follow the King’s pleasure. If his
fitness speaks, mine is ready. Now or whensoever, provided I be so able
as now.

LORD.
The King and Queen and all are coming down.

HAMLET.
In happy time.

LORD.
The Queen desires you to use some gentle entertainment to Laertes
before you fall to play.

HAMLET.
She well instructs me.

[_Exit Lord._]

HORATIO.
You will lose this wager, my lord.

HAMLET.
I do not think so. Since he went into France, I have been in continual
practice. I shall win at the odds. But thou wouldst not think how ill
all’s here about my heart: but it is no matter.

HORATIO.
Nay, good my lord.

HAMLET.
It is but foolery; but it is such a kind of gain-giving as would
perhaps trouble a woman.

HORATIO.
If your mind dislike anything, obey it. I will forestall their repair
hither, and say you are not fit.

HAMLET.
Not a whit, we defy augury. There’s a special providence in the fall of
a sparrow. If it be now, ’tis not to come; if it be not to come, it
will be now; if it be not now, yet it will come. The readiness is all.
Since no man has aught of what he leaves, what is’t to leave betimes?

Enter King, Queen, Laertes, Lords, Osric and Attendants with foils &c.

KING.
Come, Hamlet, come, and take this hand from me.

[_The King puts Laertes’s hand into Hamlet’s._]

HAMLET.
Give me your pardon, sir. I have done you wrong;
But pardon’t as you are a gentleman.
This presence knows, and you must needs have heard,
How I am punish’d with sore distraction.
What I have done
That might your nature, honour, and exception
Roughly awake, I here proclaim was madness.
Was’t Hamlet wrong’d Laertes? Never Hamlet.
If Hamlet from himself be ta’en away,
And when he’s not himself does wrong Laertes,
Then Hamlet does it not, Hamlet denies it.
Who does it, then? His madness. If’t be so,
Hamlet is of the faction that is wrong’d;
His madness is poor Hamlet’s enemy.
Sir, in this audience,
Let my disclaiming from a purpos’d evil
Free me so far in your most generous thoughts
That I have shot my arrow o’er the house
And hurt my brother.

LAERTES.
I am satisfied in nature,
Whose motive in this case should stir me most
To my revenge. But in my terms of honour
I stand aloof, and will no reconcilement
Till by some elder masters of known honour
I have a voice and precedent of peace
To keep my name ungor’d. But till that time
I do receive your offer’d love like love,
And will not wrong it.

HAMLET.
I embrace it freely,
And will this brother’s wager frankly play.—
Give us the foils; come on.

LAERTES.
Come, one for me.

HAMLET.
I’ll be your foil, Laertes; in mine ignorance
Your skill shall like a star i’ th’ darkest night,
Stick fiery off indeed.

LAERTES.
You mock me, sir.

HAMLET.
No, by this hand.

KING.
Give them the foils, young Osric. Cousin Hamlet,
You know the wager?

HAMLET.
Very well, my lord.
Your Grace has laid the odds o’ the weaker side.

KING.
I do not fear it. I have seen you both;
But since he is better’d, we have therefore odds.

LAERTES.
This is too heavy. Let me see another.

HAMLET.
This likes me well. These foils have all a length?

[_They prepare to play._]

OSRIC.
Ay, my good lord.

KING.
Set me the stoups of wine upon that table.
If Hamlet give the first or second hit,
Or quit in answer of the third exchange,
Let all the battlements their ordnance fire;
The King shall drink to Hamlet’s better breath,
And in the cup an union shall he throw
Richer than that which four successive kings
In Denmark’s crown have worn. Give me the cups;
And let the kettle to the trumpet speak,
The trumpet to the cannoneer without,
The cannons to the heavens, the heavens to earth,
‘Now the King drinks to Hamlet.’ Come, begin.
And you, the judges, bear a wary eye.

HAMLET.
Come on, sir.

LAERTES.
Come, my lord.

[_They play._]

HAMLET.
One.

LAERTES.
No.

HAMLET.
Judgement.

OSRIC.
A hit, a very palpable hit.

LAERTES.
Well; again.

KING.
Stay, give me drink. Hamlet, this pearl is thine;
Here’s to thy health.

[_Trumpets sound, and cannon shot off within._]

Give him the cup.

HAMLET.
I’ll play this bout first; set it by awhile.

[_They play._]

Come. Another hit; what say you?

LAERTES.
A touch, a touch, I do confess.

KING.
Our son shall win.

QUEEN.
He’s fat, and scant of breath.
Here, Hamlet, take my napkin, rub thy brows.
The Queen carouses to thy fortune, Hamlet.

HAMLET.
Good madam.

KING.
Gertrude, do not drink.

QUEEN.
I will, my lord; I pray you pardon me.

KING.
[_Aside._] It is the poison’d cup; it is too late.

HAMLET.
I dare not drink yet, madam. By and by.

QUEEN.
Come, let me wipe thy face.

LAERTES.
My lord, I’ll hit him now.

KING.
I do not think’t.

LAERTES.
[_Aside._] And yet ’tis almost ’gainst my conscience.

HAMLET.
Come for the third, Laertes. You do but dally.
I pray you pass with your best violence.
I am afeard you make a wanton of me.

LAERTES.
Say you so? Come on.

[_They play._]

OSRIC.
Nothing neither way.

LAERTES.
Have at you now.

[_Laertes wounds Hamlet; then, in scuffling, they change rapiers, and
Hamlet wounds Laertes._]

KING.
Part them; they are incens’d.

HAMLET.
Nay, come again!

[_The Queen falls._]

OSRIC.
Look to the Queen there, ho!

HORATIO.
They bleed on both sides. How is it, my lord?

OSRIC.
How is’t, Laertes?

LAERTES.
Why, as a woodcock to my own springe, Osric.
I am justly kill’d with mine own treachery.

HAMLET.
How does the Queen?

KING.
She swoons to see them bleed.

QUEEN.
No, no, the drink, the drink! O my dear Hamlet!
The drink, the drink! I am poison’d.

[_Dies._]

HAMLET.
O villany! Ho! Let the door be lock’d:
Treachery! Seek it out.

[_Laertes falls._]

LAERTES.
It is here, Hamlet. Hamlet, thou art slain.
No medicine in the world can do thee good.
In thee there is not half an hour of life;
The treacherous instrument is in thy hand,
Unbated and envenom’d. The foul practice
Hath turn’d itself on me. Lo, here I lie,
Never to rise again. Thy mother’s poison’d.
I can no more. The King, the King’s to blame.

HAMLET.
The point envenom’d too!
Then, venom, to thy work.

[_Stabs the King._]

OSRIC and LORDS.
Treason! treason!

KING.
O yet defend me, friends. I am but hurt.

HAMLET.
Here, thou incestuous, murderous, damned Dane,
Drink off this potion. Is thy union here?
Follow my mother.

[_King dies._]

LAERTES.
He is justly serv’d.
It is a poison temper’d by himself.
Exchange forgiveness with me, noble Hamlet.
Mine and my father’s death come not upon thee,
Nor thine on me.

[_Dies._]

HAMLET.
Heaven make thee free of it! I follow thee.
I am dead, Horatio. Wretched Queen, adieu.
You that look pale and tremble at this chance,
That are but mutes or audience to this act,
Had I but time,—as this fell sergeant, death,
Is strict in his arrest,—O, I could tell you,—
But let it be. Horatio, I am dead,
Thou liv’st; report me and my cause aright
To the unsatisfied.

HORATIO.
Never believe it.
I am more an antique Roman than a Dane.
Here’s yet some liquor left.

HAMLET.
As th’art a man,
Give me the cup. Let go; by Heaven, I’ll have’t.
O good Horatio, what a wounded name,
Things standing thus unknown, shall live behind me.
If thou didst ever hold me in thy heart,
Absent thee from felicity awhile,
And in this harsh world draw thy breath in pain,
To tell my story.

[_March afar off, and shot within._]

What warlike noise is this?

OSRIC.
Young Fortinbras, with conquest come from Poland,
To the ambassadors of England gives
This warlike volley.

HAMLET.
O, I die, Horatio.
The potent poison quite o’er-crows my spirit:
I cannot live to hear the news from England,
But I do prophesy th’election lights
On Fortinbras. He has my dying voice.
So tell him, with the occurrents more and less,
Which have solicited. The rest is silence.

[_Dies._]

HORATIO.
Now cracks a noble heart. Good night, sweet prince,
And flights of angels sing thee to thy rest.
Why does the drum come hither?

[_March within._]

Enter Fortinbras, the English Ambassadors and others.

FORTINBRAS.
Where is this sight?

HORATIO.
What is it you would see?
If aught of woe or wonder, cease your search.

FORTINBRAS.
This quarry cries on havoc. O proud death,
What feast is toward in thine eternal cell,
That thou so many princes at a shot
So bloodily hast struck?

FIRST AMBASSADOR.
The sight is dismal;
And our affairs from England come too late.
The ears are senseless that should give us hearing,
To tell him his commandment is fulfill’d,
That Rosencrantz and Guildenstern are dead.
Where should we have our thanks?

HORATIO.
Not from his mouth,
Had it th’ability of life to thank you.
He never gave commandment for their death.
But since, so jump upon this bloody question,
You from the Polack wars, and you from England
Are here arriv’d, give order that these bodies
High on a stage be placed to the view,
And let me speak to th’ yet unknowing world
How these things came about. So shall you hear
Of carnal, bloody and unnatural acts,
Of accidental judgements, casual slaughters,
Of deaths put on by cunning and forc’d cause,
And, in this upshot, purposes mistook
Fall’n on the inventors’ heads. All this can I
Truly deliver.

FORTINBRAS.
Let us haste to hear it,
And call the noblest to the audience.
For me, with sorrow I embrace my fortune.
I have some rights of memory in this kingdom,
Which now to claim my vantage doth invite me.

HORATIO.
Of that I shall have also cause to speak,
And from his mouth whose voice will draw on more.
But let this same be presently perform’d,
Even while men’s minds are wild, lest more mischance
On plots and errors happen.

FORTINBRAS.
Let four captains
Bear Hamlet like a soldier to the stage,
For he was likely, had he been put on,
To have prov’d most royally; and for his passage,
The soldiers’ music and the rites of war
Speak loudly for him.
Take up the bodies. Such a sight as this
Becomes the field, but here shows much amiss.
Go, bid the soldiers shoot.

[_A dead march._]

[_Exeunt, bearing off the bodies, after which a peal of ordnance is
shot off._]




THE FIRST PART OF KING HENRY THE FOURTH




Contents

ACT I
Scene I. London. A Room in the Palace.
Scene II. The same. An Apartment of Prince Henry’s.
Scene III. The Same. A Room in the Palace.

ACT II
Scene I. Rochester. An Inn-Yard.
Scene II. The Road by Gads-hill.
Scene III. Warkworth. A Room in the Castle.
Scene IV. Eastcheap. A Room in the Boar’s Head Tavern.

ACT III
Scene I. Bangor. A Room in the Archdeacon’s House.
Scene II. London. A Room in the Palace.
Scene III. Eastcheap. A Room in the Boar’s Head Tavern.

ACT IV
Scene I. The Rebel Camp near Shrewsbury.
Scene II. A public Road near Coventry.
Scene III. The Rebel Camp near Shrewsbury.
Scene IV. York. A Room in the Archbishop’s Palace.

ACT V
Scene I. The King’s Camp near Shrewsbury.
Scene II. The Rebel Camp.
Scene III. Plain between the Camps.
Scene IV. Another Part of the Field.
Scene V. Another Part of the Field.




Dramatis Personæ

KING HENRY the Fourth.
HENRY, PRINCE of Wales, son to the King.
Prince John of LANCASTER, son to the King.
Earl of WESTMORELAND.
Sir Walter BLUNT.
Thomas Percy, Earl of WORCESTER.
Henry Percy, Earl of NORTHUMBERLAND.
Henry Percy, surnamed HOTSPUR, his son.
Edmund MORTIMER, Earl of March.
Scroop, ARCHBISHOP of York.
SIR MICHAEL, a friend to the archbishop of York.
Archibald, Earl of DOUGLAS.
Owen GLENDOWER.
Sir Richard VERNON.
Sir John FALSTAFF.
POINS.
GADSHILL.
PETO.
BARDOLPH.
LADY PERCY, Wife to Hotspur.
Lady Mortimer, Daughter to Glendower.
Mrs. Quickly, Hostess in Eastcheap.
Lords, Officers, Sheriff, Vintner, Chamberlain, Drawers, Carriers,
Ostler, Messengers, Servant, Travellers and Attendants.

SCENE. England and Wales.




ACT I


SCENE I. London. A Room in the Palace.

Enter the King, Lord John of Lancaster, Earl of Westmoreland with
others.

KING.
So shaken as we are, so wan with care,
Find we a time for frighted peace to pant,
And breathe short-winded accents of new broils
To be commenced in strands afar remote.
No more the thirsty entrance of this soil
Shall daub her lips with her own children’s blood,
No more shall trenching war channel her fields,
Nor bruise her flow’rets with the armed hoofs
Of hostile paces: those opposed eyes,
Which, like the meteors of a troubled heaven,
All of one nature, of one substance bred,
Did lately meet in the intestine shock
And furious close of civil butchery,
Shall now, in mutual well-beseeming ranks,
March all one way, and be no more opposed
Against acquaintance, kindred, and allies.
The edge of war, like an ill-sheathed knife,
No more shall cut his master. Therefore, friends,
As far as to the sepulchre of Christ—
Whose soldier now, under whose blessed cross
We are impressed and engaged to fight—
Forthwith a power of English shall we levy,
Whose arms were molded in their mothers’ womb
To chase these pagans in those holy fields
Over whose acres walked those blessed feet
Which fourteen hundred years ago were nailed
For our advantage on the bitter cross.
But this our purpose now is twelve month old,
And bootless ’tis to tell you we will go;
Therefore we meet not now. Then let me hear
Of you, my gentle cousin Westmoreland,
What yesternight our Council did decree
In forwarding this dear expedience.

WESTMORELAND.
My liege, this haste was hot in question,
And many limits of the charge set down
But yesternight, when all athwart there came
A post from Wales loaden with heavy news,
Whose worst was that the noble Mortimer,
Leading the men of Herefordshire to fight
Against the irregular and wild Glendower,
Was by the rude hands of that Welshman taken,
A thousand of his people butchered,
Upon whose dead corpse there was such misuse,
Such beastly shameless transformation,
By those Welshwomen done, as may not be
Without much shame retold or spoken of.

KING.
It seems then that the tidings of this broil
Brake off our business for the Holy Land.

WESTMORELAND.
This, matched with other did, my gracious lord,
For more uneven and unwelcome news
Came from the North, and thus it did import:
On Holy-rood day the gallant Hotspur there,
Young Harry Percy, and brave Archibald,
That ever-valiant and approved Scot,
At Holmedon met, where they did spend
A sad and bloody hour;
As by discharge of their artillery,
And shape of likelihood, the news was told;
For he that brought them, in the very heat
And pride of their contention did take horse,
Uncertain of the issue any way.

KING.
Here is a dear and true-industrious friend,
Sir Walter Blunt, new lighted from his horse,
Stained with the variation of each soil
Betwixt that Holmedon and this seat of ours;
And he hath brought us smooth and welcome news.
The Earl of Douglas is discomfited;
Ten thousand bold Scots, two-and-twenty knights,
Balked in their own blood, did Sir Walter see
On Holmedon’s plains; of prisoners Hotspur took
Mordake, Earl of Fife and eldest son
To beaten Douglas, and the Earl of Athol,
Of Murray, Angus, and Menteith.
And is not this an honourable spoil,
A gallant prize? Ha, cousin, is it not?

WESTMORELAND.
In faith, it is a conquest for a prince to boast of.

KING.
Yea, there thou mak’st me sad, and mak’st me sin
In envy that my Lord Northumberland
Should be the father to so blest a son,
A son who is the theme of honour’s tongue,
Amongst a grove the very straightest plant,
Who is sweet Fortune’s minion and her pride;
Whilst I, by looking on the praise of him,
See riot and dishonour stain the brow
Of my young Harry. O, that it could be proved
That some night-tripping fairy had exchanged
In cradle-clothes our children where they lay,
And called mine Percy, his Plantagenet!
Then would I have his Harry, and he mine:
But let him from my thoughts. What think you, coz,
Of this young Percy’s pride? The prisoners,
Which he in this adventure hath surprised
To his own use he keeps, and sends me word
I shall have none but Mordake, Earl of Fife.

WESTMORELAND.
This is his uncle’s teaching, this is Worcester,
Malevolent to you in all aspects,
Which makes him prune himself, and bristle up
The crest of youth against your dignity.

KING.
But I have sent for him to answer this;
And for this cause awhile we must neglect
Our holy purpose to Jerusalem.
Cousin, on Wednesday next our Council we
Will hold at Windsor, so inform the lords:
But come yourself with speed to us again,
For more is to be said and to be done
Than out of anger can be uttered.

WESTMORELAND.
I will, my liege.

[_Exeunt._]

SCENE II. The same. An Apartment of Prince Henry’s.

Enter Prince Henry and Sir John Falstaff.

FALSTAFF.
Now, Hal, what time of day is it, lad?

PRINCE.
Thou art so fat-witted, with drinking of old sack, and unbuttoning thee
after supper, and sleeping upon benches after noon, that thou hast
forgotten to demand that truly which thou wouldst truly know. What a
devil hast thou to do with the time of the day? Unless hours were cups
of sack, and minutes capons, and clocks the tongues of bawds, and dials
the signs of leaping-houses, and the blessed sun himself a fair hot
wench in flame-coloured taffeta, I see no reason why thou shouldst be
so superfluous to demand the time of the day.

FALSTAFF.
Indeed, you come near me now, Hal, for we that take purses go by the
moon and the seven stars, and not by Phœbus, he, that wand’ring knight
so fair. And I prithee, sweet wag, when thou art king, as God save thy
Grace—Majesty I should say, for grace thou wilt have none—

PRINCE.
What, none?

FALSTAFF.
No, by my troth, not so much as will serve to be prologue to an egg and
butter.

PRINCE.
Well, how then? Come, roundly, roundly.

FALSTAFF.
Marry then, sweet wag, when thou art king, let not us that are squires
of the night’s body be called thieves of the day’s beauty: let us be
Diana’s foresters, gentlemen of the shade, minions of the moon; and let
men say we be men of good government, being governed, as the sea is, by
our noble and chaste mistress the moon, under whose countenance we
steal.

PRINCE.
Thou sayest well, and it holds well too, for the fortune of us that are
the moon’s men doth ebb and flow like the sea, being governed, as the
sea is, by the moon. As for proof now: a purse of gold most resolutely
snatched on Monday night, and most dissolutely spent on Tuesday
morning, got with swearing “Lay by” and spent with crying “Bring in”;
now in as low an ebb as the foot of the ladder, and by and by in as
high a flow as the ridge of the gallows.

FALSTAFF.
By the Lord, thou say’st true, lad. And is not my hostess of the tavern
a most sweet wench?

PRINCE.
As the honey of Hybla, my old lad of the castle. And is not a buff
jerkin a most sweet robe of durance?

FALSTAFF.
How now, how now, mad wag? What, in thy quips and thy quiddities? What
a plague have I to do with a buff jerkin?

PRINCE.
Why, what a pox have I to do with my hostess of the tavern?

FALSTAFF.
Well, thou hast called her to a reckoning many a time and oft.

PRINCE.
Did I ever call for thee to pay thy part?

FALSTAFF.
No, I’ll give thee thy due, thou hast paid all there.

PRINCE.
Yea, and elsewhere, so far as my coin would stretch, and where it would
not, I have used my credit.

FALSTAFF.
Yea, and so used it that were it not here apparent that thou art heir
apparent—But I prithee sweet wag, shall there be gallows standing in
England when thou art king? And resolution thus fubbed as it is with
the rusty curb of old father Antic the law? Do not thou, when thou art
king, hang a thief.

PRINCE.
No, thou shalt.

FALSTAFF.
Shall I? O rare! By the Lord, I’ll be a brave judge.

PRINCE.
Thou judgest false already, I mean thou shalt have the hanging of the
thieves, and so become a rare hangman.

FALSTAFF.
Well, Hal, well; and in some sort it jumps with my humour, as well as
waiting in the court, I can tell you.

PRINCE.
For obtaining of suits?

FALSTAFF.
Yea, for obtaining of suits, whereof the hangman hath no lean wardrobe.
’Sblood, I am as melancholy as a gib cat or a lugged bear.

PRINCE.
Or an old lion, or a lover’s lute.

FALSTAFF.
Yea, or the drone of a Lincolnshire bagpipe.

PRINCE.
What sayest thou to a hare, or the melancholy of Moor-ditch?

FALSTAFF.
Thou hast the most unsavoury similes, and art indeed the most
comparative, rascalliest, sweet young prince. But, Hal, I prithee
trouble me no more with vanity. I would to God thou and I knew where a
commodity of good names were to be bought. An old lord of the Council
rated me the other day in the street about you, sir, but I marked him
not, and yet he talked very wisely, but I regarded him not, and yet he
talked wisely, and in the street too.

PRINCE.
Thou didst well, for wisdom cries out in the streets and no man regards
it.

FALSTAFF.
O, thou hast damnable iteration, and art indeed able to corrupt a
saint. Thou hast done much harm upon me, Hal, God forgive thee for it.
Before I knew thee, Hal, I knew nothing, and now am I, if a man should
speak truly, little better than one of the wicked. I must give over
this life, and I will give it over. By the Lord, an I do not, I am a
villain. I’ll be damned for never a king’s son in Christendom.

PRINCE.
Where shall we take a purse tomorrow, Jack?

FALSTAFF.
Zounds, where thou wilt, lad, I’ll make one. An I do not, call me
villain and baffle me.

PRINCE.
I see a good amendment of life in thee, from praying to purse-taking.

FALSTAFF.
Why, Hal, ’tis my vocation, Hal, ’tis no sin for a man to labour in his
vocation.

Enter Poins.

Poins!—Now shall we know if Gadshill have set a match. O, if men were
to be saved by merit, what hole in hell were hot enough for him? This
is the most omnipotent villain that ever cried “Stand!” to a true man.

PRINCE.
Good morrow, Ned.

POINS.
Good morrow, sweet Hal.—What says Monsieur Remorse? What says Sir John
Sack-and-sugar? Jack, how agrees the devil and thee about thy soul,
that thou soldest him on Good Friday last for a cup of Madeira and a
cold capon’s leg?

PRINCE.
Sir John stands to his word, the devil shall have his bargain, for he
was never yet a breaker of proverbs. He will give the devil his due.

POINS.
Then art thou damned for keeping thy word with the devil.

PRINCE.
Else he had been damned for cozening the devil.

POINS.
But, my lads, my lads, tomorrow morning, by four o’clock early at Gad’s
Hill, there are pilgrims going to Canterbury with rich offerings, and
traders riding to London with fat purses. I have visards for you all;
you have horses for yourselves. Gadshill lies tonight in Rochester. I
have bespoke supper tomorrow night in Eastcheap. We may do it as secure
as sleep. If you will go, I will stuff your purses full of crowns. If
you will not, tarry at home and be hanged.

FALSTAFF.
Hear ye, Yedward, if I tarry at home and go not, I’ll hang you for
going.

POINS.
You will, chops?

FALSTAFF.
Hal, wilt thou make one?

PRINCE.
Who, I rob? I a thief? Not I, by my faith.

FALSTAFF.
There’s neither honesty, manhood, nor good fellowship in thee, nor thou
cam’st not of the blood royal, if thou darest not stand for ten
shillings.

PRINCE.
Well then, once in my days I’ll be a madcap.

FALSTAFF.
Why, that’s well said.

PRINCE.
Well, come what will, I’ll tarry at home.

FALSTAFF.
By the Lord, I’ll be a traitor then, when thou art king.

PRINCE.
I care not.

POINS.
Sir John, I prithee, leave the Prince and me alone. I will lay him down
such reasons for this adventure, that he shall go.

FALSTAFF.
Well, God give thee the spirit of persuasion, and him the ears of
profiting, that what thou speakest may move, and what he hears may be
believed, that the true prince may, for recreation sake, prove a false
thief, for the poor abuses of the time want countenance. Farewell, you
shall find me in Eastcheap.

PRINCE.
Farewell, thou latter spring! Farewell, All-hallown summer!

[_Exit Falstaff._]

POINS.
Now, my good sweet honey lord, ride with us tomorrow. I have a jest to
execute that I cannot manage alone. Falstaff, Bardolph, Peto, and
Gadshill shall rob those men that we have already waylaid. Yourself and
I will not be there. And when they have the booty, if you and I do not
rob them, cut this head off from my shoulders.

PRINCE.
But how shall we part with them in setting forth?

POINS.
Why, we will set forth before or after them, and appoint them a place
of meeting, wherein it is at our pleasure to fail; and then will they
adventure upon the exploit themselves, which they shall have no sooner
achieved but we’ll set upon them.

PRINCE.
Yea, but ’tis like that they will know us by our horses, by our habits,
and by every other appointment, to be ourselves.

POINS.
Tut, our horses they shall not see, I’ll tie them in the wood; our
visards we will change after we leave them; and, sirrah, I have cases
of buckram for the nonce, to immask our noted outward garments.

PRINCE.
Yea, but I doubt they will be too hard for us.

POINS.
Well, for two of them, I know them to be as true-bred cowards as ever
turned back; and for the third, if he fight longer than he sees reason,
I’ll forswear arms. The virtue of this jest will be the
incomprehensible lies that this same fat rogue will tell us when we
meet at supper: how thirty at least he fought with, what wards, what
blows, what extremities he endured; and in the reproof of this lives
the jest.

PRINCE.
Well, I’ll go with thee. Provide us all things necessary and meet me
tomorrow night in Eastcheap; there I’ll sup. Farewell.

POINS.
Farewell, my lord.

[_Exit._]

PRINCE.
I know you all, and will awhile uphold
The unyok’d humour of your idleness.
Yet herein will I imitate the sun,
Who doth permit the base contagious clouds
To smother up his beauty from the world,
That, when he please again to be himself,
Being wanted, he may be more wonder’d at,
By breaking through the foul and ugly mists
Of vapours that did seem to strangle him.
If all the year were playing holidays,
To sport would be as tedious as to work;
But, when they seldom come, they wish’d-for come,
And nothing pleaseth but rare accidents.
So when this loose behaviour I throw off,
And pay the debt I never promised,
By how much better than my word I am,
By so much shall I falsify men’s hopes;
And, like bright metal on a sullen ground,
My reformation, glitt’ring o’er my fault,
Shall show more goodly and attract more eyes
Than that which hath no foil to set it off.
I’ll so offend, to make offence a skill,
Redeeming time, when men think least I will.

[_Exit._]

SCENE III. The Same. A Room in the Palace.

Enter King Henry, Northumberland, Worcester, Hotspur, Sir Walter Blunt
and others.

KING.
My blood hath been too cold and temperate,
Unapt to stir at these indignities,
And you have found me, for accordingly
You tread upon my patience: but be sure
I will from henceforth rather be myself,
Mighty and to be fear’d, than my condition,
Which hath been smooth as oil, soft as young down,
And therefore lost that title of respect
Which the proud soul ne’er pays but to the proud.

WORCESTER.
Our house, my sovereign liege, little deserves
The scourge of greatness to be used on it,
And that same greatness too which our own hands
Have holp to make so portly.

NORTHUMBERLAND.
My lord,—

KING.
Worcester, get thee gone, for I do see
Danger and disobedience in thine eye:
O, sir, your presence is too bold and peremptory,
And majesty might never yet endure
The moody frontier of a servant brow.
You have good leave to leave us. When we need
Your use and counsel, we shall send for you.

[_Exit Worcester._]

[_To Northumberland._]

You were about to speak.

NORTHUMBERLAND.
Yea, my good lord.
Those prisoners in your Highness’ name demanded,
Which Harry Percy here at Holmedon took,
Were, as he says, not with such strength denied
As is deliver’d to your Majesty.
Either envy, therefore, or misprision
Is guilty of this fault, and not my son.

HOTSPUR.
My liege, I did deny no prisoners.
But I remember, when the fight was done,
When I was dry with rage and extreme toil,
Breathless and faint, leaning upon my sword,
Came there a certain lord, neat and trimly dress’d,
Fresh as a bridegroom, and his chin new reap’d
Show’d like a stubble-land at harvest-home.
He was perfumed like a milliner,
And ’twixt his finger and his thumb he held
A pouncet-box, which ever and anon
He gave his nose, and took’t away again,
Who therewith angry, when it next came there,
Took it in snuff; and still he smiled and talk’d.
And as the soldiers bore dead bodies by,
He call’d them untaught knaves, unmannerly,
To bring a slovenly unhandsome corse
Betwixt the wind and his nobility.
With many holiday and lady terms
He question’d me, amongst the rest demanded
My prisoners in your Majesty’s behalf.
I then, all smarting with my wounds being cold,
Out of my grief and my impatience
To be so pester’d with a popinjay,
Answer’d neglectingly, I know not what,
He should, or he should not; for he made me mad
To see him shine so brisk and smell so sweet,
And talk so like a waiting-gentlewoman
Of guns and drums and wounds, God save the mark!
And telling me the sovereignest thing on Earth
Was parmacety for an inward bruise,
And that it was great pity, so it was,
This villainous saltpetre should be digg’d
Out of the bowels of the harmless earth,
Which many a good tall fellow had destroy’d
So cowardly, and but for these vile guns,
He would himself have been a soldier.
This bald unjointed chat of his, my lord,
I answered indirectly, as I said,
And I beseech you, let not his report
Come current for an accusation
Betwixt my love and your high Majesty.

BLUNT.
The circumstance consider’d, good my lord,
Whatever Harry Percy then had said
To such a person, and in such a place,
At such a time, with all the rest retold,
May reasonably die, and never rise
To do him wrong, or any way impeach
What then he said, so he unsay it now.

KING.
Why, yet he doth deny his prisoners,
But with proviso and exception,
That we at our own charge shall ransom straight
His brother-in-law, the foolish Mortimer,
Who, on my soul, hath wilfully betray’d
The lives of those that he did lead to fight
Against that great magician, damn’d Glendower,
Whose daughter, as we hear, the Earl of March
Hath lately married. Shall our coffers then
Be emptied to redeem a traitor home?
Shall we buy treason and indent with fears
When they have lost and forfeited themselves?
No, on the barren mountains let him starve;
For I shall never hold that man my friend
Whose tongue shall ask me for one penny cost
To ransom home revolted Mortimer.

HOTSPUR.
Revolted Mortimer!
He never did fall off, my sovereign liege,
But by the chance of war. To prove that true
Needs no more but one tongue for all those wounds,
Those mouthed wounds, which valiantly he took,
When on the gentle Severn’s sedgy bank,
In single opposition hand to hand,
He did confound the best part of an hour
In changing hardiment with great Glendower.
Three times they breathed, and three times did they drink,
Upon agreement, of swift Severn’s flood,
Who then, affrighted with their bloody looks,
Ran fearfully among the trembling reeds,
And hid his crisp head in the hollow bank
Blood-stained with these valiant combatants.
Never did bare and rotten policy
Colour her working with such deadly wounds,
Nor never could the noble Mortimer
Receive so many, and all willingly.
Then let not him be slander’d with revolt.

KING.
Thou dost belie him, Percy, thou dost belie him,
He never did encounter with Glendower.
I tell thee, he durst as well have met the devil alone
As Owen Glendower for an enemy.
Art not ashamed? But, sirrah, henceforth
Let me not hear you speak of Mortimer.
Send me your prisoners with the speediest means,
Or you shall hear in such a kind from me
As will displease you.—My Lord Northumberland,
We license your departure with your son.—
Send us your prisoners, or you’ll hear of it.

[_Exit King Henry, Blunt and train._]

HOTSPUR.
An if the devil come and roar for them,
I will not send them. I will after straight
And tell him so, for I will ease my heart,
Albeit I make a hazard of my head.

NORTHUMBERLAND.
What, drunk with choler? Stay, and pause awhile.
Here comes your uncle.

Enter Worcester.

HOTSPUR.
Speak of Mortimer?
Zounds, I will speak of him, and let my soul
Want mercy if I do not join with him.
Yea, on his part I’ll empty all these veins,
And shed my dear blood drop by drop in the dust,
But I will lift the down-trod Mortimer
As high in the air as this unthankful King,
As this ingrate and canker’d Bolingbroke.

NORTHUMBERLAND.
[_To Worcester._]
Brother, the King hath made your nephew mad.

WORCESTER.
Who struck this heat up after I was gone?

HOTSPUR.
He will forsooth have all my prisoners,
And when I urged the ransom once again
Of my wife’s brother, then his cheek look’d pale,
And on my face he turn’d an eye of death,
Trembling even at the name of Mortimer.

WORCESTER.
I cannot blame him. Was not he proclaim’d
By Richard that dead is, the next of blood?

NORTHUMBERLAND.
He was; I heard the proclamation.
And then it was when the unhappy King—
Whose wrongs in us God pardon!—did set forth
Upon his Irish expedition;
From whence he, intercepted, did return
To be deposed, and shortly murdered.

WORCESTER.
And for whose death we in the world’s wide mouth
Live scandalized and foully spoken of.

HOTSPUR.
But soft, I pray you, did King Richard then
Proclaim my brother Edmund Mortimer
Heir to the crown?

NORTHUMBERLAND.
He did; myself did hear it.

HOTSPUR.
Nay, then I cannot blame his cousin King,
That wish’d him on the barren mountains starve.
But shall it be that you that set the crown
Upon the head of this forgetful man,
And for his sake wear the detested blot
Of murderous subornation—shall it be,
That you a world of curses undergo,
Being the agents, or base second means,
The cords, the ladder, or the hangman rather?
O, pardon me, that I descend so low,
To show the line and the predicament
Wherein you range under this subtle King.
Shall it for shame be spoken in these days,
Or fill up chronicles in time to come,
That men of your nobility and power
Did gage them both in an unjust behalf
(As both of you, God pardon it, have done)
To put down Richard, that sweet lovely rose,
And plant this thorn, this canker, Bolingbroke?
And shall it in more shame be further spoken,
That you are fool’d, discarded, and shook off
By him for whom these shames ye underwent?
No, yet time serves wherein you may redeem
Your banish’d honours, and restore yourselves
Into the good thoughts of the world again:
Revenge the jeering and disdain’d contempt
Of this proud King, who studies day and night
To answer all the debt he owes to you
Even with the bloody payment of your deaths.
Therefore, I say—

WORCESTER.
Peace, cousin, say no more.
And now I will unclasp a secret book,
And to your quick-conceiving discontents
I’ll read you matter deep and dangerous,
As full of peril and adventurous spirit
As to o’er-walk a current roaring loud
On the unsteadfast footing of a spear.

HOTSPUR.
If we fall in, good night, or sink or swim!
Send danger from the east unto the west,
So honour cross it from the north to south,
And let them grapple. O, the blood more stirs
To rouse a lion than to start a hare!

NORTHUMBERLAND.
Imagination of some great exploit
Drives him beyond the bounds of patience.

HOTSPUR.
By Heaven, methinks it were an easy leap
To pluck bright honour from the pale-faced moon,
Or dive into the bottom of the deep,
Where fathom-line could never touch the ground,
And pluck up drowned honour by the locks,
So he that doth redeem her thence might wear
Without corrival all her dignities.
But out upon this half-faced fellowship!

WORCESTER.
He apprehends a world of figures here,
But not the form of what he should attend.—
Good cousin, give me audience for a while.

HOTSPUR.
I cry you mercy.

WORCESTER.
Those same noble Scots
That are your prisoners—

HOTSPUR.
I’ll keep them all;
By God, he shall not have a Scot of them,
No, if a Scot would save his soul, he shall not.
I’ll keep them, by this hand!

WORCESTER.
You start away,
And lend no ear unto my purposes:
Those prisoners you shall keep—

HOTSPUR.
Nay, I will: that’s flat.
He said he would not ransom Mortimer,
Forbade my tongue to speak of Mortimer,
But I will find him when he lies asleep,
And in his ear I’ll holla “Mortimer!”
Nay, I’ll have a starling shall be taught to speak
Nothing but “Mortimer”, and give it him,
To keep his anger still in motion.

WORCESTER.
Hear you, cousin, a word.

HOTSPUR.
All studies here I solemnly defy,
Save how to gall and pinch this Bolingbroke:
And that same sword-and-buckler Prince of Wales,
But that I think his father loves him not,
And would be glad he met with some mischance—
I would have him poison’d with a pot of ale.

WORCESTER.
Farewell, kinsman. I will talk to you
When you are better temper’d to attend.

NORTHUMBERLAND.
Why, what a wasp-stung and impatient fool
Art thou to break into this woman’s mood,
Tying thine ear to no tongue but thine own!

HOTSPUR.
Why, look you, I am whipp’d and scourged with rods,
Nettled, and stung with pismires, when I hear
Of this vile politician, Bolingbroke.
In Richard’s time—what do you call the place?
A plague upon’t! It is in Gloucestershire.
’Twas where the madcap Duke his uncle kept,
His uncle York, where I first bow’d my knee
Unto this king of smiles, this Bolingbroke,
’Sblood, when you and he came back from Ravenspurgh.

NORTHUMBERLAND.
At Berkeley castle.

HOTSPUR.
You say true.
Why, what a candy deal of courtesy
This fawning greyhound then did proffer me!
“Look, when his infant fortune came to age,”
And, “Gentle Harry Percy,” and “kind cousin.”
O, the devil take such cozeners!—God forgive me!
Good uncle, tell your tale. I have done.

WORCESTER.
Nay, if you have not, to it again,
We will stay your leisure.

HOTSPUR.
I have done, i’faith.

WORCESTER.
Then once more to your Scottish prisoners;
Deliver them up without their ransom straight,
And make the Douglas’ son your only mean
For powers in Scotland, which, for divers reasons
Which I shall send you written, be assured
Will easily be granted.—[_To Northumberland._] You, my lord,
Your son in Scotland being thus employ’d,
Shall secretly into the bosom creep
Of that same noble prelate well beloved,
The Archbishop.

HOTSPUR.
Of York, is it not?

WORCESTER.
True, who bears hard
His brother’s death at Bristol, the Lord Scroop.
I speak not this in estimation,
As what I think might be, but what I know
Is ruminated, plotted, and set down,
And only stays but to behold the face
Of that occasion that shall bring it on.

HOTSPUR.
I smell it. Upon my life it will do well.

NORTHUMBERLAND.
Before the game is afoot thou still let’st slip.

HOTSPUR.
Why, it cannot choose but be a noble plot;
And then the power of Scotland and of York
To join with Mortimer, ha?

WORCESTER.
And so they shall.

HOTSPUR.
In faith, it is exceedingly well aim’d.

WORCESTER.
And ’tis no little reason bids us speed,
To save our heads by raising of a head;
For, bear ourselves as even as we can,
The King will always think him in our debt,
And think we think ourselves unsatisfied,
Till he hath found a time to pay us home:
And see already how he doth begin
To make us strangers to his looks of love.

HOTSPUR.
He does, he does, we’ll be revenged on him.

WORCESTER.
Cousin, farewell. No further go in this
Than I by letters shall direct your course.
When time is ripe, which will be suddenly,
I’ll steal to Glendower and Lord Mortimer,
Where you and Douglas, and our powers at once,
As I will fashion it, shall happily meet,
To bear our fortunes in our own strong arms,
Which now we hold at much uncertainty.

NORTHUMBERLAND.
Farewell, good brother; we shall thrive, I trust.

HOTSPUR.
Uncle, adieu. O, let the hours be short,
Till fields and blows and groans applaud our sport!

[_Exeunt._]




ACT II


SCENE I. Rochester. An Inn-Yard.

Enter a Carrier with a lantern in his hand.

FIRST CARRIER.
Heigh-ho! an it be not four by the day, I’ll be hang’d. Charles’ wain
is over the new chimney, and yet our horse not pack’d.—What, ostler!

OSTLER.
[_within._] Anon, anon.

FIRST CARRIER.
I prithee, Tom, beat Cut’s saddle, put a few flocks in the point; poor
jade is wrung in the withers out of all cess.

Enter another Carrier.

SECOND CARRIER.
Peas and beans are as dank here as a dog, and that is the next way to
give poor jades the bots. This house is turned upside down since Robin
ostler died.

FIRST CARRIER.
Poor fellow never joyed since the price of oats rose, it was the death
of him.

SECOND CARRIER.
I think this be the most villainous house in all London road for fleas.
I am stung like a tench.

FIRST CARRIER.
Like a tench! By the Mass, there is ne’er a king christen could be
better bit than I have been since the first cock.

SECOND CARRIER.
Why, they will allow us ne’er a jordan, and then we leak in your
chimney, and your chamber-lye breeds fleas like a loach.

FIRST CARRIER.
What, ostler! Come away and be hanged, come away.

SECOND CARRIER.
I have a gammon of bacon and two razes of ginger, to be delivered as
far as Charing Cross.

FIRST CARRIER.
God’s body! The turkeys in my pannier are quite starved.—What, ostler!
A plague on thee! Hast thou never an eye in thy head? Canst not hear?
An ’twere not as good deed as drink to break the pate on thee, I am a
very villain. Come, and be hanged. Hast no faith in thee?

Enter Gadshill.

GADSHILL.
Good morrow, carriers. What’s o’clock?

FIRST CARRIER.
I think it be two o’clock.

GADSHILL.
I prithee, lend me thy lantern, to see my gelding in the stable.

FIRST CARRIER.
Nay, by God, soft! I know a trick worth two of that, i’faith.

GADSHILL.
I pray thee, lend me thine.

SECOND CARRIER.
Ay, when? Canst tell? “Lend me thy lantern,” quoth he! Marry, I’ll see
thee hanged first.

GADSHILL.
Sirrah carrier, what time do you mean to come to London?

SECOND CARRIER.
Time enough to go to bed with a candle, I warrant thee. Come, neighbour
Mugs, we’ll call up the gentlemen. They will along with company, for
they have great charge.

[_Exeunt Carriers._]

GADSHILL.
What, ho! Chamberlain!

Enter Chamberlain.

CHAMBERLAIN.
At hand, quoth pick-purse.

GADSHILL.
That’s even as fair as “at hand, quoth the chamberlain,” for thou
variest no more from picking of purses than giving direction doth from
labouring; thou layest the plot how.

CHAMBERLAIN.
Good morrow, Master Gadshill. It holds current that I told you
yesternight: there’s a franklin in the Wild of Kent hath brought three
hundred marks with him in gold. I heard him tell it to one of his
company last night at supper; a kind of auditor, one that hath
abundance of charge too, God knows what. They are up already, and call
for eggs and butter. They will away presently.

GADSHILL.
Sirrah, if they meet not with Saint Nicholas’ clerks, I’ll give thee
this neck.

CHAMBERLAIN.
No, I’ll none of it. I pray thee, keep that for the hangman, for I know
thou worshippest Saint Nicholas as truly as a man of falsehood may.

GADSHILL.
What talkest thou to me of the hangman? If I hang, I’ll make a fat pair
of gallows; for, if I hang, old Sir John hangs with me, and thou
knowest he is no starveling. Tut, there are other Troyans that thou
dream’st not of, the which for sport sake are content to do the
profession some grace, that would, if matters should be looked into,
for their own credit sake make all whole. I am joined with no
foot-land-rakers, no long-staff sixpenny strikers, none of these mad
mustachio purple-hued malt-worms, but with nobility and tranquillity,
burgomasters and great oneyers, such as can hold in, such as will
strike sooner than speak, and speak sooner than drink, and drink sooner
than pray: and yet, zounds, I lie, for they pray continually to their
saint the commonwealth, or rather not pray to her, but prey on her, for
they ride up and down on her, and make her their boots.

CHAMBERLAIN.
What, the commonwealth their boots? Will she hold out water in foul
way?

GADSHILL.
She will, she will; justice hath liquored her. We steal as in a castle,
cock-sure; we have the receipt of fern-seed, we walk invisible.

CHAMBERLAIN.
Nay, by my faith, I think you are more beholding to the night than to
fern-seed for your walking invisible.

GADSHILL.
Give me thy hand. Thou shalt have a share in our purchase, as I am a
true man.

CHAMBERLAIN.
Nay, rather let me have it, as you are a false thief.

GADSHILL.
Go to; _homo_ is a common name to all men. Bid the ostler bring my
gelding out of the stable. Farewell, you muddy knave.

[_Exeunt._]

SCENE II. The Road by Gads-hill.

Enter Prince Henry and Poins; Bardolph and Peto at some distance.

POINS.
Come, shelter, shelter! I have removed Falstaff’s horse, and he frets
like a gummed velvet.

PRINCE.
Stand close.

[_They retire._]

Enter Falstaff.

FALSTAFF.
Poins! Poins, and be hanged! Poins!

PRINCE.
[_Coming forward._]
Peace, ye fat-kidneyed rascal! What a brawling dost thou keep!

FALSTAFF.
Where’s Poins, Hal?

PRINCE.
He is walked up to the top of the hill. I’ll go seek him.

[_Retires._]

FALSTAFF.
I am accursed to rob in that thief’s company. The rascal hath removed
my horse and tied him I know not where. If I travel but four foot by
the square further afoot, I shall break my wind. Well, I doubt not but
to die a fair death for all this, if I ’scape hanging for killing that
rogue. I have forsworn his company hourly any time this two-and-twenty
years, and yet I am bewitched with the rogue’s company. If the rascal
have not given me medicines to make me love him, I’ll be hanged. It
could not be else: I have drunk medicines. Poins! Hal! A plague upon
you both! Bardolph! Peto! I’ll starve ere I’ll rob a foot further. An
’twere not as good a deed as drink, to turn true man, and to leave
these rogues, I am the veriest varlet that ever chewed with a tooth.
Eight yards of uneven ground is threescore and ten miles afoot with me,
and the stony-hearted villains know it well enough. A plague upon it
when thieves cannot be true one to another! [_They whistle._] Whew! A
plague upon you all! Give me my horse, you rogues, give me my horse and
be hanged!

PRINCE.
[_Coming forward._] Peace, you fat guts, lie down, lay thine ear close
to the ground, and list if thou canst hear the tread of travellers.

FALSTAFF.
Have you any levers to lift me up again, being down? ’Sblood, I’ll not
bear my own flesh so far afoot again for all the coin in thy father’s
exchequer. What a plague mean ye to colt me thus?

PRINCE.
Thou liest, thou art not colted, thou art uncolted.

FALSTAFF.
I prithee, good Prince Hal, help me to my horse, good king’s son.

PRINCE.
Out, ye rogue! Shall I be your ostler?

FALSTAFF.
Hang thyself in thine own heir-apparent garters! If I be ta’en, I’ll
peach for this. An I have not ballads made on you all, and sung to
filthy tunes, let a cup of sack be my poison—when a jest is so forward,
and afoot too! I hate it.

Enter Gadshill.

GADSHILL.
Stand!

FALSTAFF.
So I do, against my will.

POINS.
O, ’tis our setter. I know his voice.

Comes forward with Bardolph and Peto.

BARDOLPH.
What news?

GADSHILL.
Case ye, case ye, on with your visards. There’s money of the King’s
coming down the hill, ’tis going to the King’s exchequer.

FALSTAFF.
You lie, ye rogue, ’tis going to the King’s tavern.

GADSHILL.
There’s enough to make us all.

FALSTAFF.
To be hanged.

PRINCE.
Sirs, you four shall front them in the narrow lane. Ned Poins and I
will walk lower; if they ’scape from your encounter, then they light on
us.

PETO.
How many be there of them?

GADSHILL.
Some eight or ten.

FALSTAFF.
Zounds, will they not rob us?

PRINCE.
What, a coward, Sir John Paunch?

FALSTAFF.
Indeed, I am not John of Gaunt, your grandfather, but yet no coward,
Hal.

PRINCE.
Well, we leave that to the proof.

POINS.
Sirrah Jack, thy horse stands behind the hedge. When thou need’st him,
there thou shalt find him. Farewell, and stand fast.

FALSTAFF.
Now cannot I strike him, if I should be hanged.

PRINCE.
[_aside to Poins._] Ned, where are our disguises?

POINS.
[_aside to Prince Henry._] Here, hard by. Stand close.

[_Exeunt Prince and Poins._]

FALSTAFF.
Now, my masters, happy man be his dole, say I. Every man to his
business.

Enter the Travellers.

FIRST TRAVELLER.
Come, neighbour, the boy shall lead our horses down the hill; we’ll
walk afoot awhile and ease our legs.

THIEVES.
Stand!

SECOND TRAVELLER.
Jesu bless us!

FALSTAFF.
Strike, down with them, cut the villains’ throats! Ah, whoreson
caterpillars, bacon-fed knaves, they hate us youth. Down with them,
fleece them!

FIRST TRAVELLER.
O, we are undone, both we and ours for ever!

FALSTAFF.
Hang ye, gorbellied knaves, are ye undone? No, ye fat chuffs, I would
your store were here! On, bacons, on! What, ye knaves! young men must
live. You are grandjurors, are ye? We’ll jure ye, faith.

[_Here they rob them and bind them. Exeunt_]

Enter Prince Henry and Poins in buckram suits.

PRINCE.
The thieves have bound the true men. Now could thou and I rob the
thieves, and go merrily to London, it would be argument for a week,
laughter for a month, and a good jest for ever.

POINS.
Stand close, I hear them coming.

[_They retire._]

Enter the Thieves again.

FALSTAFF.
Come, my masters, let us share, and then to horse before day. An the
Prince and Poins be not two arrant cowards, there’s no equity stirring.
There’s no more valour in that Poins than in a wild duck.

[_As they are sharing, the Prince and Poins set upon them._]

PRINCE.
Your money!

POINS.
Villains!

[_Falstaff after a blow or two, and the others run away, leaving the
booty behind them._]

PRINCE.
Got with much ease. Now merrily to horse.
The thieves are all scatter’d, and possess’d with fear
So strongly that they dare not meet each other;
Each takes his fellow for an officer.
Away, good Ned. Falstaff sweats to death,
And lards the lean earth as he walks along.
Were’t not for laughing, I should pity him.

POINS.
How the fat rogue roared!

[_Exeunt._]

SCENE III. Warkworth. A Room in the Castle.

Enter Hotspur, reading a letter.

HOTSPUR.
“But, for mine own part, my lord, I could be well contented to be
there, in respect of the love I bear your house.” He could be
contented; why is he not, then? In respect of the love he bears our
house—he shows in this, he loves his own barn better than he loves our
house. Let me see some more. “The purpose you undertake is
dangerous”—Why, that’s certain. ’Tis dangerous to take a cold, to
sleep, to drink; but I tell you, my lord fool, out of this nettle,
danger, we pluck this flower, safety. “The purpose you undertake is
dangerous, the friends you have named uncertain, the time itself
unsorted, and your whole plot too light for the counterpoise of so
great an opposition.” Say you so, say you so? I say unto you again, you
are a shallow, cowardly hind, and you lie. What a lack-brain is this!
By the Lord, our plot is a good plot as ever was laid, our friends true
and constant: a good plot, good friends, and full of expectation; an
excellent plot, very good friends. What a frosty-spirited rogue is
this! Why, my Lord of York commends the plot and the general course of
the action. Zounds, an I were now by this rascal, I could brain him
with his lady’s fan. Is there not my father, my uncle, and myself? Lord
Edmund Mortimer, my Lord of York, and Owen Glendower? Is there not
besides the Douglas? Have I not all their letters to meet me in arms by
the ninth of the next month, and are they not some of them set forward
already? What a pagan rascal is this, an infidel! Ha! You shall see
now, in very sincerity of fear and cold heart, will he to the King, and
lay open all our proceedings. O, I could divide myself, and go to
buffets, for moving such a dish of skim milk with so honourable an
action! Hang him, let him tell the King, we are prepared. I will set
forward tonight.—

Enter Lady Percy.

How now, Kate! I must leave you within these two hours.

LADY PERCY.
O my good lord, why are you thus alone?
For what offence have I this fortnight been
A banish’d woman from my Harry’s bed?
Tell me, sweet lord, what is’t that takes from thee
Thy stomach, pleasure, and thy golden sleep?
Why dost thou bend thine eyes upon the earth,
And start so often when thou sit’st alone?
Why hast thou lost the fresh blood in thy cheeks,
And given my treasures and my rights of thee
To thick-eyed musing and curst melancholy?
In thy faint slumbers I by thee have watch’d,
And heard thee murmur tales of iron wars,
Speak terms of manage to thy bounding steed,
Cry “Courage! To the field!” And thou hast talk’d
Of sallies and retires, of trenches, tents,
Of palisadoes, frontiers, parapets,
Of basilisks, of cannon, culverin,
Of prisoners’ ransom, and of soldiers slain,
And all the currents of a heady fight.
Thy spirit within thee hath been so at war,
And thus hath so bestirr’d thee in thy sleep,
That beads of sweat have stood upon thy brow
Like bubbles in a late-disturbed stream,
And in thy face strange motions have appear’d,
Such as we see when men restrain their breath
On some great sudden hest. O, what portents are these?
Some heavy business hath my lord in hand,
And I must know it, else he loves me not.

HOTSPUR.
What, ho!

Enter a Servant.

Is Gilliams with the packet gone?

SERVANT.
He is, my lord, an hour ago.

HOTSPUR.
Hath Butler brought those horses from the sheriff?

SERVANT.
One horse, my lord, he brought even now.

HOTSPUR.
What horse? A roan, a crop-ear, is it not?

SERVANT.
It is, my lord.

HOTSPUR.
That roan shall be my throne.
Well, I will back him straight. O Esperance!
Bid Butler lead him forth into the park.

[_Exit Servant._]

LADY PERCY.
But hear you, my lord.

HOTSPUR.
What say’st thou, my lady?

LADY PERCY.
What is it carries you away?

HOTSPUR.
Why, my horse, my love, my horse.

LADY PERCY.
Out, you mad-headed ape!
A weasel hath not such a deal of spleen
As you are toss’d with. In faith,
I’ll know your business, Harry, that I will.
I fear my brother Mortimer doth stir
About his title, and hath sent for you
To line his enterprise. But if you go—

HOTSPUR.
So far afoot, I shall be weary, love.

LADY PERCY.
Come, come, you paraquito, answer me
Directly unto this question that I ask.
In faith, I’ll break thy little finger, Harry,
If thou wilt not tell me all things true.

HOTSPUR.
Away,
Away, you trifler! Love, I love thee not,
I care not for thee, Kate. This is no world
To play with mammets and to tilt with lips.
We must have bloody noses and crack’d crowns,
And pass them current too.—Gods me, my horse!—
What say’st thou, Kate? What wouldst thou have with me?

LADY PERCY.
Do you not love me? Do you not indeed?
Well, do not, then, for since you love me not,
I will not love myself. Do you not love me?
Nay, tell me if you speak in jest or no.

HOTSPUR.
Come, wilt thou see me ride?
And when I am a-horseback I will swear
I love thee infinitely. But hark you, Kate,
I must not have you henceforth question me
Whither I go, nor reason whereabout.
Whither I must, I must; and, to conclude,
This evening must I leave you, gentle Kate.
I know you wise, but yet no farther wise
Than Harry Percy’s wife; constant you are,
But yet a woman; and for secrecy,
No lady closer, for I well believe
Thou wilt not utter what thou dost not know;
And so far will I trust thee, gentle Kate.

LADY PERCY.
How? So far?

HOTSPUR.
Not an inch further. But hark you, Kate,
Whither I go, thither shall you go too.
Today will I set forth, tomorrow you.
Will this content you, Kate?

LADY PERCY.
It must, of force.

[_Exeunt._]

SCENE IV. Eastcheap. A Room in the Boar’s Head Tavern.

Enter Prince Henry.

PRINCE.
Ned, prithee, come out of that fat room, and lend me thy hand to laugh
a little.

Enter Poins.

POINS.
Where hast been, Hal?

PRINCE.
With three or four loggerheads amongst three or fourscore hogsheads. I
have sounded the very base-string of humility. Sirrah, I am sworn
brother to a leash of drawers, and can call them all by their Christian
names, as Tom, Dick, and Francis. They take it already upon their
salvation, that though I be but Prince of Wales, yet I am the king of
courtesy, and tell me flatly I am no proud Jack, like Falstaff, but a
Corinthian, a lad of mettle, a good boy,—by the Lord, so they call
me—and when I am King of England, I shall command all the good lads in
Eastcheap. They call drinking deep, “dyeing scarlet,” and when you
breathe in your watering, they cry “Hem!” and bid you “Play it off!” To
conclude, I am so good a proficient in one quarter of an hour, that I
can drink with any tinker in his own language during my life. I tell
thee, Ned, thou hast lost much honour that thou wert not with me in
this action; but, sweet Ned—to sweeten which name of Ned, I give thee
this pennyworth of sugar, clapped even now into my hand by an
underskinker, one that never spake other English in his life than
“Eight shillings and sixpence,” and “You are welcome,” with this shrill
addition, “Anon, anon, sir! Score a pint of bastard in the Half-moon,”
or so. But, Ned, to drive away the time till Falstaff come, I prithee,
do thou stand in some by-room, while I question my puny drawer to what
end he gave me the sugar, and do thou never leave calling “Francis,”
that his tale to me may be nothing but “Anon.” Step aside, and I’ll
show thee a precedent.

[_Exit Poins._]

POINS.
[_Within_] Francis!

PRINCE.
Thou art perfect.

POINS.
[_Within_] Francis!

Enter Francis.

FRANCIS.
Anon, anon, sir.—Look down into the Pomegarnet, Ralph.

PRINCE.
Come hither, Francis.

FRANCIS.
My lord?

PRINCE.
How long hast thou to serve, Francis?

FRANCIS.
Forsooth, five years, and as much as to—

POINS.
[_within._] Francis!

FRANCIS.
Anon, anon, sir.

PRINCE.
Five year! By’r Lady, a long lease for the clinking of pewter! But,
Francis, darest thou be so valiant as to play the coward with thy
indenture, and show it a fair pair of heels, and run from it?

FRANCIS.
O Lord, sir, I’ll be sworn upon all the books in England, I could find
in my heart—

POINS.
[_within._] Francis!

FRANCIS.
Anon, sir.

PRINCE.
How old art thou, Francis?

FRANCIS.
Let me see, about Michaelmas next I shall be—

POINS.
[_within._] Francis!

FRANCIS.
Anon, sir.—Pray, stay a little, my lord.

PRINCE.
Nay, but hark you, Francis, for the sugar thou gavest me, ’twas a
pennyworth, was’t not?

FRANCIS.
O Lord, I would it had been two!

PRINCE.
I will give thee for it a thousand pound. Ask me when thou wilt, and
thou shalt have it.

POINS.
[_within._] Francis!

FRANCIS.
Anon, anon.

PRINCE.
Anon, Francis? No, Francis, but tomorrow, Francis; or, Francis, a
Thursday; or indeed, Francis, when thou wilt. But, Francis,—

FRANCIS.
My lord?

PRINCE.
Wilt thou rob this leathern-jerkin, crystal-button, not-pated,
agate-ring, puke-stocking, caddis-garter, smooth-tongue, Spanish-pouch—

FRANCIS.
O Lord, sir, who do you mean?

PRINCE.
Why, then, your brown bastard is your only drink, for look you,
Francis, your white canvas doublet will sully. In Barbary, sir, it
cannot come to so much.

FRANCIS.
What, sir?

POINS.
[_within._] Francis!

PRINCE.
Away, you rogue! Dost thou not hear them call?

[_Here they both call him; the Drawer stands amazed, not knowing which
way to go._]

Enter Vintner.

VINTNER.
What, stand’st thou still, and hear’st such a calling? Look to the
guests within.

[_Exit Francis._]

My lord, old Sir John with half-a-dozen more are at the door. Shall I
let them in?

PRINCE.
Let them alone awhile, and then open the door.

[_Exit Vintner._]

Poins!

Enter Poins.

POINS.
Anon, anon, sir.

PRINCE.
Sirrah, Falstaff and the rest of the thieves are at the door; shall we
be merry?

POINS.
As merry as crickets, my lad. But hark ye, what cunning match have you
made with this jest of the drawer? Come, what’s the issue?

PRINCE.
I am now of all humours that have showed themselves humours since the
old days of goodman Adam to the pupil age of this present twelve
o’clock at midnight.

Enter Francis.

What’s o’clock, Francis?

FRANCIS.
Anon, anon, sir.

[_Exit Francis._]

PRINCE.
That ever this fellow should have fewer words than a parrot, and yet
the son of a woman! His industry is upstairs and downstairs; his
eloquence the parcel of a reckoning. I am not yet of Percy’s mind, the
Hotspur of the north, he that kills me some six or seven dozen of Scots
at a breakfast, washes his hands, and says to his wife, “Fie upon this
quiet life! I want work.” “O my sweet Harry,” says she, “how many hast
thou killed today?” “Give my roan horse a drench,” says he; and
answers, “Some fourteen,” an hour after; “a trifle, a trifle.” I
prithee, call in Falstaff. I’ll play Percy, and that damned brawn shall
play Dame Mortimer his wife. _Rivo!_ says the drunkard. Call in Ribs,
call in Tallow.

Enter Falstaff, Gadshill, Bardolph and Peto; followed by Francis with
wine.

POINS.
Welcome, Jack. Where hast thou been?

FALSTAFF.
A plague of all cowards, I say, and a vengeance too! Marry, and amen!
Give me a cup of sack, boy. Ere I lead this life long, I’ll sew
nether-stocks, and mend them and foot them too. A plague of all
cowards! Give me a cup of sack, rogue. Is there no virtue extant?

[_Drinks._]

PRINCE.
Didst thou never see Titan kiss a dish of butter (pitiful-hearted
Titan!), that melted at the sweet tale of the sun’s? If thou didst,
then behold that compound.

FALSTAFF.
You rogue, here’s lime in this sack too: there is nothing but roguery
to be found in villainous man, yet a coward is worse than a cup of sack
with lime in it. A villanous coward! Go thy ways, old Jack. Die when
thou wilt, if manhood, good manhood, be not forgot upon the face of the
Earth, then am I a shotten herring. There lives not three good men
unhanged in England, and one of them is fat, and grows old, God help
the while, a bad world, I say. I would I were a weaver; I could sing
psalms or anything. A plague of all cowards, I say still.

PRINCE.
How now, wool-sack, what mutter you?

FALSTAFF.
A king’s son! If I do not beat thee out of thy kingdom with a dagger of
lath, and drive all thy subjects afore thee like a flock of wild-geese,
I’ll never wear hair on my face more. You, Prince of Wales!

PRINCE.
Why, you whoreson round man, what’s the matter?

FALSTAFF.
Are not you a coward? Answer me to that—and Poins there?

POINS.
Zounds, ye fat paunch, an ye call me coward, by the Lord, I’ll stab
thee.

FALSTAFF.
I call thee coward? I’ll see thee damned ere I call thee coward, but I
would give a thousand pound I could run as fast as thou canst. You are
straight enough in the shoulders, you care not who sees your back. Call
you that backing of your friends? A plague upon such backing! Give me
them that will face me.—Give me a cup of sack. I am a rogue if I drunk
today.

PRINCE.
O villain! Thy lips are scarce wiped since thou drunk’st last.

FALSTAFF.
All is one for that. A plague of all cowards, still say I.

[_Drinks._]

PRINCE.
What’s the matter?

FALSTAFF.
What’s the matter? There be four of us here have ta’en a thousand pound
this day morning.

PRINCE.
Where is it, Jack, where is it?

FALSTAFF.
Where is it? Taken from us it is: a hundred upon poor four of us.

PRINCE.
What, a hundred, man?

FALSTAFF.
I am a rogue if I were not at half-sword with a dozen of them two hours
together. I have ’scaped by miracle. I am eight times thrust through
the doublet, four through the hose, my buckler cut through and through,
my sword hacked like a handsaw. _Ecce signum!_ I never dealt better
since I was a man: all would not do. A plague of all cowards! Let them
speak. If they speak more or less than truth, they are villains, and
the sons of darkness.

PRINCE.
Speak, sirs, how was it?

GADSHILL.
We four set upon some dozen.

FALSTAFF.
Sixteen at least, my lord.

GADSHILL.
And bound them.

PETO.
No, no, they were not bound.

FALSTAFF.
You rogue, they were bound, every man of them, or I am a Jew else, an
Ebrew Jew.

GADSHILL.
As we were sharing, some six or seven fresh men set upon us.

FALSTAFF.
And unbound the rest, and then come in the other.

PRINCE.
What, fought you with them all?

FALSTAFF.
All? I know not what you call all, but if I fought not with fifty of
them I am a bunch of radish. If there were not two or three and fifty
upon poor old Jack, then am I no two-legged creature.

PRINCE.
Pray God you have not murdered some of them.

FALSTAFF.
Nay, that’s past praying for. I have peppered two of them. Two I am
sure I have paid, two rogues in buckram suits. I tell thee what, Hal,
if I tell thee a lie, spit in my face, call me horse. Thou knowest my
old ward. Here I lay, and thus I bore my point. Four rogues in buckram
let drive at me.

PRINCE.
What, four? Thou saidst but two even now.

FALSTAFF.
Four, Hal, I told thee four.

POINS.
Ay, ay, he said four.

FALSTAFF.
These four came all afront, and mainly thrust at me. I made me no more
ado, but took all their seven points in my target, thus.

PRINCE.
Seven? Why, there were but four even now.

FALSTAFF.
In buckram?

POINS.
Ay, four, in buckram suits.

FALSTAFF.
Seven, by these hilts, or I am a villain else.

PRINCE.
[_aside to Poins._] Prithee let him alone, we shall have more anon.

FALSTAFF.
Dost thou hear me, Hal?

PRINCE.
Ay, and mark thee too, Jack.

FALSTAFF.
Do so, for it is worth the listening to. These nine in buckram that I
told thee of—

PRINCE.
So, two more already.

FALSTAFF.
Their points being broken—

POINS.
Down fell their hose.

FALSTAFF.
Began to give me ground; but I followed me close, came in foot and
hand, and with a thought seven of the eleven I paid.

PRINCE.
O monstrous! Eleven buckram men grown out of two!

FALSTAFF.
But as the devil would have it, three misbegotten knaves in Kendal
green came at my back and let drive at me, for it was so dark, Hal,
that thou couldst not see thy hand.

PRINCE.
These lies are like the father that begets them, gross as a mountain,
open, palpable. Why, thou clay-brained guts, thou knotty-pated fool,
thou whoreson, obscene greasy tallow-catch—

FALSTAFF.
What, art thou mad? Art thou mad? Is not the truth the truth?

PRINCE.
Why, how couldst thou know these men in Kendal green, when it was so
dark thou couldst not see thy hand? Come, tell us your reason. What
sayest thou to this?

POINS.
Come, your reason, Jack, your reason.

FALSTAFF.
What, upon compulsion? Zounds, an I were at the strappado, or all the
racks in the world, I would not tell you on compulsion. Give you a
reason on compulsion! If reasons were as plentiful as blackberries, I
would give no man a reason upon compulsion, I.

PRINCE.
I’ll be no longer guilty of this sin. This sanguine coward, this
bed-presser, this horse-back-breaker, this huge hill of flesh—

FALSTAFF.
’Sblood, you starveling, you eel-skin, you dried neat’s-tongue, you
bull’s pizzle, you stock-fish—O, for breath to utter what is like thee!
You tailor’s-yard, you sheath, you bow-case, you vile standing tuck—

PRINCE.
Well, breathe awhile, and then to it again, and when thou hast tired
thyself in base comparisons, hear me speak but this.

POINS.
Mark, Jack.

PRINCE.
We two saw you four set on four, and bound them and were masters of
their wealth. Mark now how a plain tale shall put you down. Then did we
two set on you four, and, with a word, outfaced you from your prize,
and have it, yea, and can show it you here in the house. And, Falstaff,
you carried your guts away as nimbly, with as quick dexterity, and
roared for mercy, and still ran and roared, as ever I heard bull-calf.
What a slave art thou to hack thy sword as thou hast done, and then say
it was in fight! What trick, what device, what starting-hole canst thou
now find out to hide thee from this open and apparent shame?

POINS.
Come, let’s hear, Jack, what trick hast thou now?

FALSTAFF.
By the Lord, I knew ye as well as he that made ye. Why, hear you, my
masters, was it for me to kill the heir-apparent? Should I turn upon
the true prince? Why, thou knowest I am as valiant as Hercules: but
beware instinct. The lion will not touch the true prince. Instinct is a
great matter. I was now a coward on instinct. I shall think the better
of myself, and thee, during my life—I for a valiant lion, and thou for
a true prince. But, by the Lord, lads, I am glad you have the
money.—Hostess, clap to the doors. Watch tonight, pray tomorrow.
Gallants, lads, boys, hearts of gold, all the titles of good fellowship
come to you! What, shall we be merry? Shall we have a play extempore?

PRINCE.
Content; and the argument shall be thy running away.

FALSTAFF.
Ah, no more of that, Hal, an thou lovest me!

Enter the Hostess.

HOSTESS.
O Jesu, my lord the Prince—

PRINCE.
How now, my lady the hostess! What say’st thou to me?

HOSTESS.
Marry, my lord, there is a nobleman of the court at door would speak
with you: he says he comes from your father.

PRINCE.
Give him as much as will make him a royal man, and send him back again
to my mother.

FALSTAFF.
What manner of man is he?

HOSTESS.
An old man.

FALSTAFF.
What doth gravity out of his bed at midnight? Shall I give him his
answer?

PRINCE.
Prithee do, Jack.

FALSTAFF.
Faith, and I’ll send him packing.

[_Exit._]

PRINCE.
Now, sirs: by’r Lady, you fought fair, so did you, Peto. So did you,
Bardolph. You are lions, too, you ran away upon instinct, you will not
touch the true prince, no, fie!

BARDOLPH.
Faith, I ran when I saw others run.

PRINCE.
Faith, tell me now in earnest, how came Falstaff’s sword so hacked?

PETO.
Why, he hacked it with his dagger, and said he would swear truth out of
England but he would make you believe it was done in fight, and
persuaded us to do the like.

BARDOLPH.
Yea, and to tickle our noses with spear-grass to make them bleed, and
then to beslubber our garments with it, and swear it was the blood of
true men. I did that I did not this seven year before: I blushed to
hear his monstrous devices.

PRINCE.
O villain, thou stolest a cup of sack eighteen years ago, and wert
taken with the manner, and ever since thou hast blushed extempore. Thou
hadst fire and sword on thy side, and yet thou ran’st away. What
instinct hadst thou for it?

BARDOLPH.
My lord, do you see these meteors? Do you behold these exhalations?

PRINCE.
I do.

BARDOLPH.
What think you they portend?

PRINCE.
Hot livers and cold purses.

BARDOLPH.
Choler, my lord, if rightly taken.

PRINCE.
No, if rightly taken, halter.

Enter Falstaff.

Here comes lean Jack, here comes bare-bone. How now, my sweet creature
of bombast? How long is’t ago, Jack, since thou sawest thine own knee?

FALSTAFF.
My own knee? When I was about thy years, Hal, I was not an eagle’s
talon in the waist. I could have crept into any alderman’s thumb-ring:
a plague of sighing and grief! It blows a man up like a bladder.
There’s villanous news abroad: here was Sir John Bracy from your
father; you must to the court in the morning. That same mad fellow of
the north, Percy, and he of Wales that gave Amamon the bastinado, and
made Lucifer cuckold, and swore the devil his true liegeman upon the
cross of a Welsh hook—what a plague call you him?

POINS.
O, Glendower.

FALSTAFF.
Owen, Owen, the same; and his son-in-law Mortimer, and old
Northumberland, and that sprightly Scot of Scots, Douglas, that runs
a-horseback up a hill perpendicular—

PRINCE.
He that rides at high speed, and with his pistol kills a sparrow
flying.

FALSTAFF.
You have hit it.

PRINCE.
So did he never the sparrow.

FALSTAFF.
Well, that rascal hath good metal in him, he will not run.

PRINCE.
Why, what a rascal art thou then, to praise him so for running!

FALSTAFF.
A-horseback, ye cuckoo, but afoot he will not budge a foot.

PRINCE.
Yes, Jack, upon instinct.

FALSTAFF.
I grant ye, upon instinct. Well, he is there too, and one Mordake, and
a thousand blue-caps more. Worcester is stolen away tonight; thy
father’s beard is turned white with the news. You may buy land now as
cheap as stinking mackerel.

PRINCE.
Why then, it is like if there come a hot June, and this civil buffeting
hold, we shall buy maidenheads as they buy hobnails, by the hundreds.

FALSTAFF.
By the mass, lad, thou sayest true. It is like we shall have good
trading that way. But tell me, Hal, art not thou horrible afeard? Thou
being heir-apparent, could the world pick thee out three such enemies
again as that fiend Douglas, that spirit Percy, and that devil
Glendower? Art thou not horribly afraid? Doth not thy blood thrill at
it?

PRINCE.
Not a whit, i’faith. I lack some of thy instinct.

FALSTAFF.
Well, thou wilt be horribly chid tomorrow when thou comest to thy
father. If thou love me practise an answer.

PRINCE.
Do thou stand for my father and examine me upon the particulars of my
life.

FALSTAFF.
Shall I? Content! This chair shall be my state, this dagger my sceptre,
and this cushion my crown.

PRINCE.
Thy state is taken for a joint-stool, thy golden sceptre for a leaden
dagger, and thy precious rich crown for a pitiful bald crown.

FALSTAFF.
Well, an the fire of grace be not quite out of thee, now shalt thou be
moved. Give me a cup of sack to make my eyes look red, that it may be
thought I have wept, for I must speak in passion, and I will do it in
King Cambyses’ vein.

PRINCE.
Well, here is my leg.

FALSTAFF.
And here is my speech. Stand aside, nobility.

HOSTESS.
O Jesu, this is excellent sport, i faith!

FALSTAFF.
Weep not, sweet Queen, for trickling tears are vain.

HOSTESS.
O, the Father, how he holds his countenance!

FALSTAFF.
For God’s sake, lords, convey my tristful Queen,
For tears do stop the floodgates of her eyes.

HOSTESS.
O Jesu, he doth it as like one of these harlotry players as ever I see!

FALSTAFF.
Peace, good pint-pot; peace, good tickle-brain.—Harry, I do not only
marvel where thou spendest thy time, but also how thou art accompanied.
For though the camomile, the more it is trodden on, the faster it
grows, yet youth, the more it is wasted, the sooner it wears. That thou
art my son I have partly thy mother’s word, partly my own opinion, but
chiefly a villainous trick of thine eye, and a foolish hanging of thy
nether lip, that doth warrant me. If then thou be son to me, here lies
the point: why, being son to me, art thou so pointed at? Shall the
blessed sun of heaven prove a micher, and eat blackberries? A question
not to be asked. Shall the son of England prove a thief, and take
purses? A question to be asked. There is a thing, Harry, which thou
hast often heard of, and it is known to many in our land by the name of
pitch. This pitch, as ancient writers do report, doth defile; so doth
the company thou keepest. For, Harry, now I do not speak to thee in
drink, but in tears; not in pleasure, but in passion; not in words
only, but in woes also. And yet there is a virtuous man whom I have
often noted in thy company, but I know not his name.

PRINCE.
What manner of man, an it like your Majesty?

FALSTAFF.
A goodly portly man, i’faith, and a corpulent; of a cheerful look, a
pleasing eye, and a most noble carriage; and, as I think, his age some
fifty, or, by’r Lady, inclining to threescore; and now I remember me,
his name is Falstaff. If that man should be lewdly given, he deceiveth
me; for, Harry, I see virtue in his looks. If then the tree may be
known by the fruit, as the fruit by the tree, then peremptorily I speak
it, there is virtue in that Falstaff; him keep with, the rest banish.
And tell me now, thou naughty varlet, tell me where hast thou been this
month?

PRINCE.
Dost thou speak like a king? Do thou stand for me, and I’ll play my
father.

FALSTAFF.
Depose me? If thou dost it half so gravely, so majestically, both in
word and matter, hang me up by the heels for a rabbit-sucker or a
poulter’s hare.

PRINCE.
Well, here I am set.

FALSTAFF.
And here I stand. Judge, my masters.

PRINCE.
Now, Harry, whence come you?

FALSTAFF.
My noble lord, from Eastcheap.

PRINCE.
The complaints I hear of thee are grievous.

FALSTAFF.
’Sblood, my lord, they are false.—Nay, I’ll tickle ye for a young
prince, i’faith.

PRINCE.
Swearest thou, ungracious boy? Henceforth ne’er look on me. Thou art
violently carried away from grace. There is a devil haunts thee in the
likeness of an old fat man. A tun of man is thy companion. Why dost
thou converse with that trunk of humours, that bolting-hutch of
beastliness, that swollen parcel of dropsies, that huge bombard of
sack, that stuffed cloak-bag of guts, that roasted Manningtree ox with
the pudding in his belly, that reverend Vice, that grey iniquity, that
father ruffian, that vanity in years? Wherein is he good, but to taste
sack and drink it? Wherein neat and cleanly, but to carve a capon and
eat it? Wherein cunning, but in craft? Wherein crafty, but in villany?
Wherein villainous, but in all things? Wherein worthy, but in nothing?

FALSTAFF.
I would your Grace would take me with you. Whom means your Grace?

PRINCE.
That villainous abominable misleader of youth, Falstaff, that old
white-bearded Satan.

FALSTAFF.
My lord, the man I know.

PRINCE.
I know thou dost.

FALSTAFF.
But to say I know more harm in him than in myself were to say more than
I know. That he is old, the more the pity, his white hairs do witness
it. But that he is, saving your reverence, a whoremaster, that I
utterly deny. If sack and sugar be a fault, God help the wicked! If to
be old and merry be a sin, then many an old host that I know is damned.
If to be fat be to be hated, then Pharaoh’s lean kine are to be loved.
No, my good lord, banish Peto, banish Bardolph, banish Poins, but for
sweet Jack Falstaff, kind Jack Falstaff, true Jack Falstaff, valiant
Jack Falstaff, and therefore more valiant, being as he is old Jack
Falstaff, banish not him thy Harry’s company, banish not him thy
Harry’s company. Banish plump Jack, and banish all the world.

PRINCE.
I do, I will.

[_A knocking heard._]

[_Exeunt Hostess, Francis and Bardolph._]

Enter Bardolph, running.

BARDOLPH.
O, my lord, my lord, the sheriff with a most monstrous watch is at the
door.

FALSTAFF.
Out, ye rogue! Play out the play. I have much to say in the behalf of
that Falstaff.

Enter the Hostess, hastily.

HOSTESS.
O Jesu, my lord, my lord—

PRINCE.
Heigh, heigh, the devil rides upon a fiddlestick. What’s the matter?

HOSTESS.
The sheriff and all the watch are at the door. They are come to search
the house. Shall I let them in?

FALSTAFF.
Dost thou hear, Hal? Never call a true piece of gold a counterfeit:
thou art essentially made without seeming so.

PRINCE.
And thou a natural coward without instinct.

FALSTAFF.
I deny your major. If you will deny the sheriff, so; if not, let him
enter. If I become not a cart as well as another man, a plague on my
bringing up! I hope I shall as soon be strangled with a halter as
another.

PRINCE.
Go hide thee behind the arras. The rest walk up above. Now, my masters,
for a true face and good conscience.

FALSTAFF.
Both which I have had, but their date is out, and therefore I’ll hide
me.

PRINCE.
Call in the sheriff.

[_Exeunt all but the Prince and Peto._]

Enter Sheriff and the Carrier.

Now, master sheriff, what is your will with me?

SHERIFF.
First, pardon me, my lord. A hue and cry
Hath followed certain men unto this house.

PRINCE.
What men?

SHERIFF.
One of them is well known, my gracious lord,
A gross fat man.

CARRIER.
As fat as butter.

PRINCE.
The man I do assure you is not here,
For I myself at this time have employ’d him.
And, sheriff, I will engage my word to thee,
That I will by tomorrow dinner-time,
Send him to answer thee, or any man,
For anything he shall be charged withal.
And so let me entreat you leave the house.

SHERIFF.
I will, my lord. There are two gentlemen
Have in this robbery lost three hundred marks.

PRINCE.
It may be so. If he have robb’d these men,
He shall be answerable; and so, farewell.

SHERIFF.
Good night, my noble lord.

PRINCE.
I think it is good morrow, is it not?

SHERIFF.
Indeed, my lord, I think it be two o’clock.

[_Exit Sheriff with the Carrier._]

PRINCE.
This oily rascal is known as well as Paul’s. Go, call him forth.

PETO.
Falstaff!—Fast asleep behind the arras, and snorting like a horse.

PRINCE.
Hark, how hard he fetches breath. Search his pockets.

[_He searcheth his pocket, and findeth certain papers._]

What hast thou found?

PETO.
Nothing but papers, my lord.

PRINCE.
Let’s see what they be. Read them.

PETO.
[_reads_]
Item, a capon, . . . . . . . . . . . 2s. 2d.
Item, sauce, . . . . . . . . . . . . . . . 4d.
Item, sack, two gallons, . . . 5s. 8d.
Item, anchovies and sack after supper, 2s. 6d.
Item, bread, . . . . . . . . . . . . . . . ob.

PRINCE.
O monstrous! But one halfpennyworth of bread to this intolerable deal
of sack! What there is else, keep close. We’ll read it at more
advantage. There let him sleep till day. I’ll to the court in the
morning. We must all to the wars, and thy place shall be honourable.
I’ll procure this fat rogue a charge of foot, and I know his death will
be a march of twelve score. The money shall be paid back again with
advantage. Be with me betimes in the morning; and so, good morrow,
Peto.

PETO.
Good morrow, good my lord.

[_Exeunt._]




ACT III


SCENE I. Bangor. A Room in the Archdeacon’s House.

Enter Hotspur, Worcester, Mortimer and Glendower.

MORTIMER.
These promises are fair, the parties sure,
And our induction full of prosperous hope.

HOTSPUR.
Lord Mortimer and cousin Glendower,
Will you sit down? And uncle Worcester,
A plague upon it! I have forgot the map.

GLENDOWER.
No, here it is.
Sit, cousin Percy, sit, good cousin Hotspur;
For by that name as oft as Lancaster doth speak of you
His cheek looks pale, and with a rising sigh
He wisheth you in heaven.

HOTSPUR.
And you in hell,
As oft as he hears Owen Glendower spoke of.

GLENDOWER.
I cannot blame him. At my nativity
The front of heaven was full of fiery shapes,
Of burning cressets, and at my birth
The frame and huge foundation of the Earth
Shaked like a coward.

HOTSPUR.
Why, so it would have done
At the same season, if your mother’s cat
Had but kitten’d, though yourself had never been born.

GLENDOWER.
I say the Earth did shake when I was born.

HOTSPUR.
And I say the Earth was not of my mind,
If you suppose as fearing you it shook.

GLENDOWER.
The heavens were all on fire, the Earth did tremble.

HOTSPUR.
O, then th’ Earth shook to see the heavens on fire,
And not in fear of your nativity.
Diseased nature oftentimes breaks forth
In strange eruptions; oft the teeming Earth
Is with a kind of colic pinch’d and vex’d
By the imprisoning of unruly wind
Within her womb, which for enlargement striving,
Shakes the old beldam Earth, and topples down
Steeples and moss-grown towers. At your birth
Our grandam Earth, having this distemp’rature,
In passion shook.

GLENDOWER.
Cousin, of many men
I do not bear these crossings. Give me leave
To tell you once again that at my birth
The front of heaven was full of fiery shapes,
The goats ran from the mountains, and the herds
Were strangely clamorous to the frighted fields.
These signs have mark’d me extraordinary,
And all the courses of my life do show
I am not in the roll of common men.
Where is he living, clipp’d in with the sea
That chides the banks of England, Scotland, Wales,
Which calls me pupil or hath read to me?
And bring him out that is but woman’s son
Can trace me in the tedious ways of art,
And hold me pace in deep experiments.

HOTSPUR.
I think there is no man speaks better Welsh.
I’ll to dinner.

MORTIMER.
Peace, cousin Percy, you will make him mad.

GLENDOWER.
I can call spirits from the vasty deep.

HOTSPUR.
Why, so can I, or so can any man,
But will they come when you do call for them?

GLENDOWER.
Why, I can teach you, cousin, to command the devil.

HOTSPUR.
And I can teach thee, coz, to shame the devil
By telling truth; tell truth, and shame the devil.
If thou have power to raise him, bring him hither,
And I’ll be sworn I have power to shame him hence.
O, while you live, tell truth, and shame the devil!

MORTIMER.
Come, come, no more of this unprofitable chat.

GLENDOWER.
Three times hath Henry Bolingbroke made head
Against my power; thrice from the banks of Wye
And sandy-bottom’d Severn have I sent him
Bootless home and weather-beaten back.

HOTSPUR.
Home without boots, and in foul weather too!
How ’scapes he agues, in the devil’s name!

GLENDOWER.
Come, here’s the map, shall we divide our right
According to our threefold order ta’en?

MORTIMER.
The archdeacon hath divided it
Into three limits very equally:
England, from Trent and Severn hitherto,
By south and east is to my part assign’d:
All westward, Wales beyond the Severn shore,
And all the fertile land within that bound,
To Owen Glendower: and, dear coz, to you
The remnant northward lying off from Trent.
And our indentures tripartite are drawn,
Which being sealed interchangeably,
A business that this night may execute,
Tomorrow, cousin Percy, you and I,
And my good Lord of Worcester will set forth
To meet your father and the Scottish power,
As is appointed us, at Shrewsbury.
My father Glendower is not ready yet,
Nor shall we need his help these fourteen days.
[_To Glendower._] Within that space you may have drawn together
Your tenants, friends, and neighbouring gentlemen.

GLENDOWER.
A shorter time shall send me to you, lords,
And in my conduct shall your ladies come,
From whom you now must steal, and take no leave,
For there will be a world of water shed
Upon the parting of your wives and you.

HOTSPUR.
Methinks my moiety, north from Burton here,
In quantity equals not one of yours.
See how this river comes me cranking in,
And cuts me from the best of all my land
A huge half-moon, a monstrous cantle out.
I’ll have the current in this place dammed up,
And here the smug and silver Trent shall run
In a new channel, fair and evenly.
It shall not wind with such a deep indent,
To rob me of so rich a bottom here.

GLENDOWER.
Not wind? It shall, it must. You see it doth.

MORTIMER.
Yea, but mark how he bears his course, and runs me up
With like advantage on the other side,
Gelding the opposed continent as much
As on the other side it takes from you.

WORCESTER.
Yea, but a little charge will trench him here,
And on this north side win this cape of land,
And then he runs straight and even.

HOTSPUR.
I’ll have it so, a little charge will do it.

GLENDOWER.
I’ll not have it altered.

HOTSPUR.
Will not you?

GLENDOWER.
No, nor you shall not.

HOTSPUR.
Who shall say me nay?

GLENDOWER.
Why, that will I.

HOTSPUR.
Let me not understand you, then; speak it in Welsh.

GLENDOWER.
I can speak English, lord, as well as you,
For I was train’d up in the English Court,
Where being but young I framed to the harp
Many an English ditty lovely well,
And gave the tongue a helpful ornament—
A virtue that was never seen in you.

HOTSPUR.
Marry, and I am glad of it with all my heart.
I had rather be a kitten, and cry “mew”
Than one of these same metre ballad-mongers;
I had rather hear a brazen canstick turn’d,
Or a dry wheel grate on the axletree,
And that would set my teeth nothing on edge,
Nothing so much as mincing poetry.
’Tis like the forced gait of a shuffling nag.

GLENDOWER.
Come, you shall have Trent turn’d.

HOTSPUR.
I do not care. I’ll give thrice so much land
To any well-deserving friend;
But in the way of bargain, mark ye me,
I’ll cavil on the ninth part of a hair.
Are the indentures drawn? Shall we be gone?

GLENDOWER.
The moon shines fair, you may away by night.
I’ll haste the writer, and withal
Break with your wives of your departure hence.
I am afraid my daughter will run mad,
So much she doteth on her Mortimer.

[_Exit._]

MORTIMER.
Fie, cousin Percy, how you cross my father!

HOTSPUR.
I cannot choose. Sometimes he angers me
With telling me of the moldwarp and the ant,
Of the dreamer Merlin and his prophecies,
And of a dragon and a finless fish,
A clip-wing’d griffin and a moulten raven,
A couching lion and a ramping cat,
And such a deal of skimble-skamble stuff
As puts me from my faith. I tell you what—
He held me last night at least nine hours
In reckoning up the several devils’ names
That were his lackeys: I cried “Hum,” and “Well, go to,”
But mark’d him not a word. O, he is as tedious
As a tired horse, a railing wife,
Worse than a smoky house. I had rather live
With cheese and garlic in a windmill, far,
Than feed on cates and have him talk to me
In any summer house in Christendom.

MORTIMER.
In faith, he is a worthy gentleman,
Exceedingly well read, and profited
In strange concealments, valiant as a lion,
And wondrous affable, and as bountiful
As mines of India. Shall I tell you, cousin?
He holds your temper in a high respect
And curbs himself even of his natural scope
When you come cross his humour, faith, he does.
I warrant you that man is not alive
Might so have tempted him as you have done
Without the taste of danger and reproof:
But do not use it oft, let me entreat you.

WORCESTER.
In faith, my lord, you are too wilful-blame,
And since your coming hither have done enough
To put him quite besides his patience.
You must needs learn, lord, to amend this fault.
Though sometimes it show greatness, courage, blood—
And that’s the dearest grace it renders you—
Yet oftentimes it doth present harsh rage,
Defect of manners, want of government,
Pride, haughtiness, opinion, and disdain,
The least of which haunting a nobleman
Loseth men’s hearts and leaves behind a stain
Upon the beauty of all parts besides,
Beguiling them of commendation.

HOTSPUR.
Well, I am school’d. Good manners be your speed!
Here come our wives, and let us take our leave.

Enter Glendower with Lady Mortimer and Lady Percy.

MORTIMER.
This is the deadly spite that angers me,
My wife can speak no English, I no Welsh.

GLENDOWER.
My daughter weeps, she’ll not part with you,
She’ll be a soldier too, she’ll to the wars.

MORTIMER.
Good father, tell her that she and my aunt Percy
Shall follow in your conduct speedily.

[_Glendower speaks to Lady Mortimer in Welsh, and she answers him in
the same._]

GLENDOWER.
She is desperate here, a peevish self-willed harlotry,
One that no persuasion can do good upon.

[_Lady Mortimer speaks to Mortimer in Welsh._]

MORTIMER.
I understand thy looks, that pretty Welsh
Which thou pourest down from these swelling heavens
I am too perfect in, and but for shame
In such a parley should I answer thee.

[_Lady Mortimer speaks to him again in Welsh._]

I understand thy kisses, and thou mine,
And that’s a feeling disputation,
But I will never be a truant, love,
Till I have learnt thy language; for thy tongue
Makes Welsh as sweet as ditties highly penn’d,
Sung by a fair queen in a summer’s bower,
With ravishing division, to her lute.

GLENDOWER.
Nay, if you melt, then will she run mad.

[_Lady Mortimer speaks to Mortimer again in Welsh._]

MORTIMER.
O, I am ignorance itself in this!

GLENDOWER.
She bids you on the wanton rushes lay you down,
And rest your gentle head upon her lap,
And she will sing the song that pleaseth you,
And on your eyelids crown the god of sleep,
Charming your blood with pleasing heaviness,
Making such difference ’twixt wake and sleep
As is the difference betwixt day and night,
The hour before the heavenly-harness’d team
Begins his golden progress in the east.

MORTIMER.
With all my heart I’ll sit and hear her sing,
By that time will our book, I think, be drawn.

GLENDOWER.
Do so, and those musicians that shall play to you
Hang in the air a thousand leagues from hence,
And straight they shall be here: sit, and attend.

HOTSPUR.
Come, Kate, thou art perfect in lying down.
Come, quick, quick, that I may lay my head in thy lap.

LADY PERCY.
Go, ye giddy goose.

[_The music plays._]

HOTSPUR.
Now I perceive the devil understands Welsh,
And ’tis no marvel he’s so humorous.
By’r Lady, he’s a good musician.

LADY PERCY.
Then should you be nothing but musical,
For you are altogether governed by humours.
Lie still, ye thief, and hear the lady sing in Welsh.

HOTSPUR.
I had rather hear Lady, my brach, howl in Irish.

LADY PERCY.
Wouldst thou have thy head broken?

HOTSPUR.
No.

LADY PERCY.
Then be still.

HOTSPUR.
Neither; ’tis a woman’s fault.

LADY PERCY.
Now God help thee!

HOTSPUR.
To the Welsh lady’s bed.

LADY PERCY.
What’s that?

HOTSPUR.
Peace, she sings.

[_Here the lady sings a Welsh song._]

Come, Kate, I’ll have your song too.

LADY PERCY.
Not mine, in good sooth.

HOTSPUR.
Not yours, in good sooth! Heart! you swear like a comfit-maker’s wife!
“Not you, in good sooth,” and “As true as I live,” and “As God shall
mend me,” and “As sure as day”
And givest such sarcenet surety for thy oaths
As if thou never walk’dst further than Finsbury.
Swear me, Kate, like a lady as thou art,
A good mouth-filling oath, and leave “In sooth,”
And such protest of pepper-gingerbread,
To velvet-guards and Sunday citizens.
Come, sing.

LADY PERCY.
I will not sing.

HOTSPUR.
’Tis the next way to turn tailor, or be redbreast-teacher. An the
indentures be drawn, I’ll away within these two hours; and so come in
when ye will.

[_Exit._]

GLENDOWER.
Come, come, Lord Mortimer, you are as slow
As hot Lord Percy is on fire to go.
By this our book is drawn. We’ll but seal,
And then to horse immediately.

MORTIMER.
With all my heart.

[_Exeunt._]

SCENE II. London. A Room in the Palace.

Enter King Henry, Prince Henry and Lords.

KING.
Lords, give us leave; the Prince of Wales and I
Must have some private conference: but be near at hand,
For we shall presently have need of you.

[_Exeunt Lords._]

I know not whether God will have it so
For some displeasing service I have done,
That, in His secret doom, out of my blood
He’ll breed revengement and a scourge for me;
But thou dost in thy passages of life
Make me believe that thou art only mark’d
For the hot vengeance and the rod of heaven
To punish my mistreadings. Tell me else,
Could such inordinate and low desires,
Such poor, such bare, such lewd, such mean attempts,
Such barren pleasures, rude society,
As thou art match’d withal, and grafted to,
Accompany the greatness of thy blood,
And hold their level with thy princely heart?

PRINCE.
So please your Majesty, I would I could
Quit all offences with as clear excuse
As well as I am doubtless I can purge
Myself of many I am charged withal.
Yet such extenuation let me beg
As, in reproof of many tales devised,
By smiling pickthanks and base newsmongers,
Which oft the ear of greatness needs must hear,
I may for some things true, wherein my youth
Hath faulty wander’d and irregular,
Find pardon on my true submission.

KING.
God pardon thee! Yet let me wonder, Harry,
At thy affections, which do hold a wing
Quite from the flight of all thy ancestors.
Thy place in Council thou hast rudely lost,
Which by thy younger brother is supplied,
And art almost an alien to the hearts
Of all the court and princes of my blood.
The hope and expectation of thy time
Is ruin’d, and the soul of every man
Prophetically do forethink thy fall.
Had I so lavish of my presence been,
So common-hackney’d in the eyes of men,
So stale and cheap to vulgar company,
Opinion, that did help me to the crown,
Had still kept loyal to possession,
And left me in reputeless banishment,
A fellow of no mark nor likelihood.
By being seldom seen, I could not stir
But like a comet I was wonder’d at,
That men would tell their children, “This is he.”
Others would say, “Where, which is Bolingbroke?”
And then I stole all courtesy from heaven,
And dress’d myself in such humility
That I did pluck allegiance from men’s hearts,
Loud shouts and salutations from their mouths,
Even in the presence of the crowned King.
Thus did I keep my person fresh and new,
My presence, like a robe pontifical,
Ne’er seen but wonder’d at, and so my state,
Seldom but sumptuous, showed like a feast,
And won by rareness such solemnity.
The skipping King, he ambled up and down
With shallow jesters and rash bavin wits,
Soon kindled and soon burnt; carded his state,
Mingled his royalty, with cap’ring fools,
Had his great name profaned with their scorns,
And gave his countenance, against his name,
To laugh at gibing boys, and stand the push
Of every beardless vain comparative;
Grew a companion to the common streets,
Enfeoff’d himself to popularity,
That, being daily swallow’d by men’s eyes,
They surfeited with honey, and began
To loathe the taste of sweetness, whereof a little
More than a little is by much too much.
So, when he had occasion to be seen,
He was but as the cuckoo is in June,
Heard, not regarded; seen, but with such eyes
As, sick and blunted with community,
Afford no extraordinary gaze,
Such as is bent on sun-like majesty
When it shines seldom in admiring eyes,
But rather drowsed and hung their eyelids down,
Slept in his face, and render’d such aspect
As cloudy men use to their adversaries,
Being with his presence glutted, gorged, and full.
And in that very line, Harry, standest thou,
For thou hast lost thy princely privilege
With vile participation. Not an eye
But is a-weary of thy common sight,
Save mine, which hath desired to see thee more,
Which now doth that I would not have it do,
Make blind itself with foolish tenderness.

PRINCE.
I shall hereafter, my thrice gracious lord,
Be more myself.

KING.
For all the world
As thou art to this hour was Richard then
When I from France set foot at Ravenspurgh,
And even as I was then is Percy now.
Now, by my sceptre, and my soul to boot,
He hath more worthy interest to the state
Than thou, the shadow of succession.
For of no right, nor colour like to right,
He doth fill fields with harness in the realm,
Turns head against the lion’s armed jaws,
And, being no more in debt to years than thou,
Leads ancient lords and reverend bishops on
To bloody battles and to bruising arms.
What never-dying honour hath he got
Against renowned Douglas! whose high deeds,
Whose hot incursions and great name in arms,
Holds from all soldiers chief majority
And military title capital
Through all the kingdoms that acknowledge Christ.
Thrice hath this Hotspur, Mars in swathing clothes,
This infant warrior, in his enterprises
Discomfited great Douglas, ta’en him once,
Enlarged him, and made a friend of him,
To fill the mouth of deep defiance up,
And shake the peace and safety of our throne.
And what say you to this? Percy, Northumberland,
The Archbishop’s Grace of York, Douglas, Mortimer,
Capitulate against us and are up.
But wherefore do I tell these news to thee?
Why, Harry, do I tell thee of my foes,
Which art my nearest and dearest enemy?
Thou that art like enough, through vassal fear,
Base inclination, and the start of spleen,
To fight against me under Percy’s pay,
To dog his heels, and curtsy at his frowns,
To show how much thou art degenerate.

PRINCE.
Do not think so, you shall not find it so.
And God forgive them that so much have sway’d
Your Majesty’s good thoughts away from me!
I will redeem all this on Percy’s head,
And, in the closing of some glorious day,
Be bold to tell you that I am your son,
When I will wear a garment all of blood,
And stain my favours in a bloody mask,
Which, wash’d away, shall scour my shame with it.
And that shall be the day, whene’er it lights,
That this same child of honour and renown,
This gallant Hotspur, this all-praised knight,
And your unthought-of Harry chance to meet.
For every honour sitting on his helm,
Would they were multitudes, and on my head
My shames redoubled! For the time will come,
That I shall make this northern youth exchange
His glorious deeds for my indignities.
Percy is but my factor, good my lord,
To engross up glorious deeds on my behalf,
And I will call him to so strict account
That he shall render every glory up,
Yea, even the slightest worship of his time,
Or I will tear the reckoning from his heart.
This in the name of God I promise here,
The which if He be pleased I shall perform,
I do beseech your Majesty may salve
The long-grown wounds of my intemperance.
If not, the end of life cancels all bands,
And I will die a hundred thousand deaths
Ere break the smallest parcel of this vow.

KING.
A hundred thousand rebels die in this.
Thou shalt have charge and sovereign trust herein.

Enter Sir Walter Blunt.

How now, good Blunt? Thy looks are full of speed.

BLUNT.
So hath the business that I come to speak of.
Lord Mortimer of Scotland hath sent word
That Douglas and the English rebels met
The eleventh of this month at Shrewsbury.
A mighty and a fearful head they are,
If promises be kept on every hand,
As ever offer’d foul play in a state.

KING.
The Earl of Westmoreland set forth today,
With him my son, Lord John of Lancaster,
For this advertisement is five days old.
On Wednesday next you, Harry, shall set forward,
On Thursday we ourselves will march.
Our meeting is Bridgenorth. And, Harry, you
Shall march through Gloustershire; by which account,
Our business valued, some twelve days hence
Our general forces at Bridgenorth shall meet.
Our hands are full of business. Let’s away,
Advantage feeds him fat while men delay.

[_Exeunt._]

SCENE III. Eastcheap. A Room in the Boar’s Head Tavern.

Enter Falstaff and Bardolph.

FALSTAFF.
Bardolph, am I not fallen away vilely since this last action? Do I not
bate? Do I not dwindle? Why, my skin hangs about me like an old lady’s
loose gown. I am withered like an old apple-john. Well, I’ll repent,
and that suddenly, while I am in some liking. I shall be out of heart
shortly, and then I shall have no strength to repent. An I have not
forgotten what the inside of a church is made of, I am a peppercorn, a
brewer’s horse. The inside of a church! Company, villainous company,
hath been the spoil of me.

BARDOLPH.
Sir John, you are so fretful, you cannot live long.

FALSTAFF.
Why, there is it. Come, sing me a song, make me merry. I was as
virtuously given as a gentleman need to be, virtuous enough; swore
little; diced not above seven times—a week; went to a bawdy house not
above once in a quarter—in an hour; paid money that I borrowed—three or
four times; lived well and in good compass; and now I live out of all
order, out of all compass.

BARDOLPH.
Why, you are so fat, Sir John, that you must needs be out of all
compass, out of all reasonable compass, Sir John.

FALSTAFF.
Do thou amend thy face, and I’ll amend my life. Thou art our admiral,
thou bearest the lantern in the poop, but ’tis in the nose of thee.
Thou art the Knight of the Burning Lamp.

BARDOLPH.
Why, Sir John, my face does you no harm.

FALSTAFF.
No, I’ll be sworn, I make as good use of it as many a man doth of a
death’s-head or a _memento mori_. I never see thy face but I think upon
hell-fire, and Dives that lived in purple, for there he is in his
robes, burning, burning. If thou wert any way given to virtue, I would
swear by thy face. My oath should be, “By this fire, that’s God’s
angel.” But thou art altogether given over; and wert indeed, but for
the light in thy face, the son of utter darkness. When thou ran’st up
Gad’s Hill in the night to catch my horse, if I did not think thou
hadst been an _ignis fatuus_ or a ball of wildfire, there’s no purchase
in money. O, thou art a perpetual triumph, an everlasting
bonfire-light! Thou hast saved me a thousand marks in links and
torches, walking with thee in the night betwixt tavern and tavern: but
the sack that thou hast drunk me would have bought me lights as good
cheap at the dearest chandler’s in Europe. I have maintained that
salamander of yours with fire any time this two-and-thirty years, God
reward me for it!

BARDOLPH.
’Sblood, I would my face were in your belly!

FALSTAFF.
God-a-mercy! so should I be sure to be heartburnt.

Enter the Hostess.

How now, Dame Partlet the hen, have you enquired yet who picked my
pocket?

HOSTESS.
Why, Sir John, what do you think, Sir John, do you think I keep thieves
in my house? I have searched, I have enquired, so has my husband, man
by man, boy by boy, servant by servant. The tithe of a hair was never
lost in my house before.

FALSTAFF.
Ye lie, hostess. Bardolph was shaved and lost many a hair, and I’ll be
sworn my pocket was picked. Go to, you are a woman, go.

HOSTESS.
Who, I? No; I defy thee: God’s light, I was never called so in mine own
house before.

FALSTAFF.
Go to, I know you well enough.

HOSTESS.
No, Sir John, you do not know me, Sir John. I know you, Sir John, you
owe me money, Sir John, and now you pick a quarrel to beguile me of it.
I bought you a dozen of shirts to your back.

FALSTAFF.
Dowlas, filthy dowlas. I have given them away to bakers’ wives; and
they have made bolters of them.

HOSTESS.
Now, as I am a true woman, holland of eight shillings an ell. You owe
money here besides, Sir John, for your diet and by-drinkings, and money
lent you, four-and-twenty pound.

FALSTAFF.
He had his part of it, let him pay.

HOSTESS.
He? Alas, he is poor, he hath nothing.

FALSTAFF.
How? Poor? Look upon his face. What call you rich? Let them coin his
nose, let them coin his cheeks. I’ll not pay a denier. What, will you
make a younker of me? Shall I not take mine ease in mine inn but I
shall have my pocket picked? I have lost a seal-ring of my
grandfather’s worth forty mark.

HOSTESS.
O Jesu, I have heard the Prince tell him, I know not how oft, that that
ring was copper.

FALSTAFF.
How? The Prince is a Jack, a sneak-up. ’Sblood, an he were here, I
would cudgel him like a dog if he would say so.

Enter Prince Henry with Peto, marching. Falstaff meets him, playing on
his truncheon like a fife.

How now, lad? Is the wind in that door, i’faith? Must we all march?

BARDOLPH.
Yea, two and two, Newgate fashion.

HOSTESS.
My lord, I pray you, hear me.

PRINCE.
What say’st thou, Mistress Quickly? How doth thy husband? I love him
well; he is an honest man.

HOSTESS.
Good my lord, hear me.

FALSTAFF.
Prithee, let her alone, and list to me.

PRINCE.
What say’st thou, Jack?

FALSTAFF.
The other night I fell asleep here, behind the arras, and had my pocket
picked. This house is turned bawdy-house; they pick pockets.

PRINCE.
What didst thou lose, Jack?

FALSTAFF.
Wilt thou believe me, Hal, three or four bonds of forty pound apiece
and a seal-ring of my grandfather’s.

PRINCE.
A trifle, some eightpenny matter.

HOSTESS.
So I told him, my lord, and I said I heard your Grace say so. And, my
lord, he speaks most vilely of you, like a foul-mouthed man as he is,
and said he would cudgel you.

PRINCE.
What! he did not?

HOSTESS.
There’s neither faith, truth, nor womanhood in me else.

FALSTAFF.
There’s no more faith in thee than in a stewed prune, nor no more truth
in thee than in a drawn fox; and, for woman-hood, Maid Marian may be
the deputy’s wife of the ward to thee. Go, you thing, go.

HOSTESS.
Say, what thing, what thing?

FALSTAFF.
What thing? Why, a thing to thank God on.

HOSTESS.
I am no thing to thank God on, I would thou shouldst know it! I am an
honest man’s wife, and, setting thy knighthood aside, thou art a knave
to call me so.

FALSTAFF.
Setting thy womanhood aside, thou art a beast to say otherwise.

HOSTESS.
Say, what beast, thou knave, thou?

FALSTAFF.
What beast? Why, an otter.

PRINCE.
An otter, Sir John? Why an otter?

FALSTAFF.
Why, she’s neither fish nor flesh; a man knows not where to have her.

HOSTESS.
Thou art an unjust man in saying so, thou or any man knows where to
have me, thou knave, thou.

PRINCE.
Thou say’st true, hostess, and he slanders thee most grossly.

HOSTESS.
So he doth you, my lord, and said this other day you ought him a
thousand pound.

PRINCE.
Sirrah, do I owe you a thousand pound?

FALSTAFF.


A thousand pound, Hal? A million. Thy love is worth a million; thou
owest me thy love.

HOSTESS.
Nay, my lord, he call’d you Jack, and said he would cudgel you.

FALSTAFF.
Did I, Bardolph?

BARDOLPH.
Indeed, Sir John, you said so.

FALSTAFF.
Yea, if he said my ring was copper.

PRINCE.
I say ’tis copper. Darest thou be as good as thy word now?

FALSTAFF.
Why, Hal, thou knowest, as thou art but man, I dare, but as thou art
prince, I fear thee as I fear the roaring of the lion’s whelp.

PRINCE.
And why not as the lion?

FALSTAFF.
The King himself is to be feared as the lion. Dost thou think I’ll fear
thee as I fear thy father? Nay, an I do, I pray God my girdle break.

PRINCE.
O, if it should, how would thy guts fall about thy knees! But, sirrah,
there’s no room for faith, truth, nor honesty in this bosom of thine;
it is all filled up with midriff. Charge an honest woman with picking
thy pocket! Why, thou whoreson, impudent, embossed rascal, if there
were anything in thy pocket but tavern reckonings, memorandums of bawdy
houses, and one poor pennyworth of sugar-candy to make thee
long-winded, if thy pocket were enriched with any other injuries but
these, I am a villain. And yet you will stand to it, you will not
pocket up wrong. Art thou not ashamed!

FALSTAFF.
Dost thou hear, Hal? Thou knowest in the state of innocency Adam fell,
and what should poor Jack Falstaff do in the days of villainy? Thou
seest I have more flesh than another man and therefore more frailty.
You confess, then, you picked my pocket?

PRINCE.
It appears so by the story.

FALSTAFF.
Hostess, I forgive thee. Go make ready breakfast, love thy husband,
look to thy servants, cherish thy guests. Thou shalt find me tractable
to any honest reason. Thou seest I am pacified still. Nay, prithee, be
gone.

[_Exit Hostess._]

Now, Hal, to the news at court. For the robbery, lad, how is that
answered?

PRINCE.
O, my sweet beef, I must still be good angel to thee. The money is paid
back again.

FALSTAFF.
O, I do not like that paying back, ’tis a double labour.

PRINCE.
I am good friends with my father, and may do anything.

FALSTAFF.
Rob me the exchequer the first thing thou dost, and do it with unwashed
hands too.

BARDOLPH.
Do, my lord.

PRINCE.
I have procured thee, Jack, a charge of foot.

FALSTAFF.
I would it had been of horse. Where shall I find one that can steal
well? O, for a fine thief, of the age of two-and-twenty or thereabouts!
I am heinously unprovided. Well, God be thanked for these rebels; they
offend none but the virtuous. I laud them, I praise them.

PRINCE.
Bardolph!

BARDOLPH.
My lord?

PRINCE.
Go bear this letter to Lord John of Lancaster,
To my brother John; this to my Lord of Westmoreland.

[_Exit Bardolph._]

Go, Peto, to horse, to horse, for thou and I
Have thirty miles to ride yet ere dinner-time.

[_Exit Peto._]

Jack, meet me tomorrow in the Temple hall
At two o’clock in the afternoon;
There shalt thou know thy charge, and there receive
Money and order for their furniture.
The land is burning, Percy stands on high,
And either we or they must lower lie.

[_Exit._]

FALSTAFF.
Rare words! Brave world!—Hostess, my breakfast, come.—
O, I could wish this tavern were my drum.

[_Exit._]




ACT IV


SCENE I. The Rebel Camp near Shrewsbury.

Enter Hotspur, Worcester and Douglas.

HOTSPUR.
Well said, my noble Scot. If speaking truth
In this fine age were not thought flattery,
Such attribution should the Douglas have
As not a soldier of this season’s stamp
Should go so general current through the world.
By God, I cannot flatter, I do defy
The tongues of soothers, but a braver place
In my heart’s love hath no man than yourself.
Nay, task me to my word; approve me, lord.

DOUGLAS.
Thou art the king of honour.
No man so potent breathes upon the ground
But I will beard him.

HOTSPUR.
Do so, and ’tis well.

Enter a Messenger with letters.

What letters hast thou there? I can but thank you.

MESSENGER.
These letters come from your father.

HOTSPUR.
Letters from him! Why comes he not himself?

MESSENGER.
He cannot come, my lord, he is grievous sick.

HOTSPUR.
Zounds, how has he the leisure to be sick
In such a justling time? Who leads his power?
Under whose government come they along?

MESSENGER.
His letters bear his mind, not I, my lord.

WORCESTER.
I prithee, tell me, doth he keep his bed?

MESSENGER.
He did, my lord, four days ere I set forth,
And at the time of my departure thence
He was much fear’d by his physicians.

WORCESTER.
I would the state of time had first been whole
Ere he by sickness had been visited.
His health was never better worth than now.

HOTSPUR.
Sick now? Droop now? This sickness doth infect
The very life-blood of our enterprise;
’Tis catching hither, even to our camp.
He writes me here, that inward sickness—
And that his friends by deputation could not
So soon be drawn, nor did he think it meet
To lay so dangerous and dear a trust
On any soul removed but on his own.
Yet doth he give us bold advertisement
That with our small conjunction we should on,
To see how fortune is disposed to us;
For, as he writes, there is no quailing now,
Because the King is certainly possess’d
Of all our purposes. What say you to it?

WORCESTER.
Your father’s sickness is a maim to us.

HOTSPUR.
A perilous gash, a very limb lopp’d off—
And yet, in faith, it is not! His present want
Seems more than we shall find it. Were it good
To set the exact wealth of all our states
All at one cast? To set so rich a main
On the nice hazard of one doubtful hour?
It were not good, for therein should we read
The very bottom and the soul of hope,
The very list, the very utmost bound
Of all our fortunes.

DOUGLAS.
Faith, and so we should, where now remains
A sweet reversion. We may boldly spend
Upon the hope of what is to come in.
A comfort of retirement lives in this.

HOTSPUR.
A rendezvous, a home to fly unto,
If that the devil and mischance look big
Upon the maidenhead of our affairs.

WORCESTER.
But yet I would your father had been here.
The quality and hair of our attempt
Brooks no division. It will be thought
By some that know not why he is away,
That wisdom, loyalty, and mere dislike
Of our proceedings, kept the Earl from hence.
And think how such an apprehension
May turn the tide of fearful faction,
And breed a kind of question in our cause.
For well you know we of the off’ring side
Must keep aloof from strict arbitrement,
And stop all sight-holes, every loop from whence
The eye of reason may pry in upon us.
This absence of your father’s draws a curtain
That shows the ignorant a kind of fear
Before not dreamt of.

HOTSPUR.
You strain too far.
I rather of his absence make this use:
It lends a lustre and more great opinion,
A larger dare to our great enterprise,
Than if the Earl were here; for men must think
If we without his help can make a head
To push against the kingdom, with his help
We shall o’erturn it topsy-turvy down.
Yet all goes well, yet all our joints are whole.

DOUGLAS.
As heart can think. There is not such a word
Spoke in Scotland as this term of fear.

Enter Sir Richard Vernon.

HOTSPUR.
My cousin Vernon! Welcome, by my soul.

VERNON.
Pray God my news be worth a welcome, lord.
The Earl of Westmoreland, seven thousand strong,
Is marching hitherwards, with him Prince John.

HOTSPUR.
No harm, what more?

VERNON.
And further, I have learn’d
The King himself in person is set forth,
Or hitherwards intended speedily,
With strong and mighty preparation.

HOTSPUR.
He shall be welcome too. Where is his son,
The nimble-footed madcap Prince of Wales,
And his comrades, that daffed the world aside
And bid it pass?

VERNON.
All furnish’d, all in arms;
All plumed like estridges that with the wind
Bated like eagles having lately bathed,
Glittering in golden coats, like images,
As full of spirit as the month of May,
And gorgeous as the sun at midsummer;
Wanton as youthful goats, wild as young bulls.
I saw young Harry with his beaver on,
His cuisses on his thighs, gallantly arm’d,
Rise from the ground like feather’d Mercury,
And vaulted with such ease into his seat
As if an angel dropp’d down from the clouds,
To turn and wind a fiery Pegasus,
And witch the world with noble horsemanship.

HOTSPUR.
No more, no more! Worse than the sun in March,
This praise doth nourish agues. Let them come!
They come like sacrifices in their trim,
And to the fire-eyed maid of smoky war
All hot and bleeding will we offer them.
The mailed Mars shall on his altar sit
Up to the ears in blood. I am on fire
To hear this rich reprisal is so nigh,
And yet not ours. Come, let me taste my horse,
Who is to bear me like a thunderbolt
Against the bosom of the Prince of Wales.
Harry to Harry shall, hot horse to horse,
Meet and ne’er part till one drop down a corse.
O, that Glendower were come!

VERNON.
There is more news.
I learn’d in Worcester, as I rode along,
He cannot draw his power this fourteen days.

DOUGLAS.
That’s the worst tidings that I hear of yet.

WORCESTER.
Ay, by my faith, that bears a frosty sound.

HOTSPUR.
What may the King’s whole battle reach unto?

VERNON.
To thirty thousand.

HOTSPUR.
Forty let it be.
My father and Glendower being both away,
The powers of us may serve so great a day.
Come, let us take a muster speedily.
Doomsday is near; die all, die merrily.

DOUGLAS.
Talk not of dying. I am out of fear
Of death or death’s hand for this one half year.

[_Exeunt._]

SCENE II. A public Road near Coventry.

Enter Falstaff and Bardolph.

FALSTAFF.
Bardolph, get thee before to Coventry; fill me a bottle of sack. Our
soldiers shall march through; we’ll to Sutton Co’fil’ tonight.

BARDOLPH.
Will you give me money, captain?

FALSTAFF.
Lay out, lay out.

BARDOLPH.
This bottle makes an angel.

FALSTAFF.
An if it do, take it for thy labour. An if it make twenty, take them
all, I’ll answer the coinage. Bid my lieutenant Peto meet me at town’s
end.

BARDOLPH.
I will, captain: farewell.

[_Exit._]

FALSTAFF.
If I be not ashamed of my soldiers, I am a soused gurnet. I have
misused the King’s press damnably. I have got, in exchange of a hundred
and fifty soldiers, three hundred and odd pounds. I press me none but
good householders, yeomen’s sons, inquire me out contracted bachelors,
such as had been asked twice on the banns, such a commodity of warm
slaves as had as lief hear the devil as a drum, such as fear the report
of a caliver worse than a struck fowl or a hurt wild duck. I pressed me
none but such toasts-and-butter, with hearts in their bellies no bigger
than pins’ heads, and they have bought out their services; and now my
whole charge consists of ancients, corporals, lieutenants, gentlemen of
companies—slaves as ragged as Lazarus in the painted cloth, where the
glutton’s dogs licked his sores; and such as indeed were never
soldiers, but discarded unjust servingmen, younger sons to younger
brothers, revolted tapsters, and ostlers trade-fallen; the cankers of a
calm world and a long peace, ten times more dishonourable-ragged than
an old fazed ancient; and such have I to fill up the rooms of them that
have bought out their services, that you would think that I had a
hundred and fifty tattered prodigals lately come from swine-keeping,
from eating draff and husks. A mad fellow met me on the way, and told
me I had unloaded all the gibbets and pressed the dead bodies. No eye
hath seen such scarecrows. I’ll not march through Coventry with them,
that’s flat. Nay, and the villains march wide betwixt the legs as if
they had gyves on, for indeed I had the most of them out of prison.
There’s not a shirt and a half in all my company, and the half shirt is
two napkins tacked together and thrown over the shoulders like a
herald’s coat without sleeves; and the shirt, to say the truth, stolen
from my host at Saint Albans, or the red-nose innkeeper of Daventry.
But that’s all one; they’ll find linen enough on every hedge.

Enter Prince Henry and the Lord of Westmoreland.

PRINCE.
How now, blown Jack? How now, quilt?

FALSTAFF.
What, Hal! How now, mad wag? What a devil dost thou in Warwickshire? My
good Lord of Westmoreland, I cry you mercy. I thought your honour had
already been at Shrewsbury.

WESTMORELAND.
Faith, Sir John, ’tis more than time that I were there, and you too,
but my powers are there already. The King, I can tell you, looks for us
all. We must away all night.

FALSTAFF.
Tut, never fear me. I am as vigilant as a cat to steal cream.

PRINCE.
I think, to steal cream indeed, for thy theft hath already made thee
butter. But tell me, Jack, whose fellows are these that come after?

FALSTAFF.
Mine, Hal, mine.

PRINCE.
I did never see such pitiful rascals.

FALSTAFF.
Tut, tut, good enough to toss; food for powder, food for powder,
they’ll fill a pit as well as better. Tush, man, mortal men, mortal
men.

WESTMORELAND.
Ay, but, Sir John, methinks they are exceeding poor and bare, too
beggarly.

FALSTAFF.
Faith, for their poverty, I know not where they had that; and for their
bareness, I am sure they never learned that of me.

PRINCE.
No, I’ll be sworn, unless you call three fingers on the ribs bare. But,
sirrah, make haste. Percy is already in the field.

[_Exit._]

FALSTAFF.
What, is the King encamped?

WESTMORELAND.
He is, Sir John. I fear we shall stay too long.

[_Exit._]

FALSTAFF.
Well,
To the latter end of a fray and the beginning of a feast
Fits a dull fighter and a keen guest.

[_Exit._]

SCENE III. The Rebel Camp near Shrewsbury.

Enter Hotspur, Worcester, Douglas and Vernon.

HOTSPUR.
We’ll fight with him tonight.

WORCESTER.
It may not be.

DOUGLAS.
You give him then advantage.

VERNON.
Not a whit.

HOTSPUR.
Why say you so? Looks he not for supply?

VERNON.
So do we.

HOTSPUR.
His is certain, ours is doubtful.

WORCESTER.
Good cousin, be advised, stir not tonight.

VERNON.
Do not, my lord.

DOUGLAS.
You do not counsel well.
You speak it out of fear and cold heart.

VERNON.
Do me no slander, Douglas; by my life,
And I dare well maintain it with my life,
If well-respected honour bid me on,
I hold as little counsel with weak fear
As you, my lord, or any Scot that this day lives.
Let it be seen tomorrow in the battle
Which of us fears.

DOUGLAS.
Yea, or tonight.

VERNON.
Content.

HOTSPUR.
Tonight, say I.

VERNON.
Come, come, it may not be. I wonder much,
Being men of such great leading as you are,
That you foresee not what impediments
Drag back our expedition. Certain horse
Of my cousin Vernon’s are not yet come up.
Your uncle Worcester’s horse came but today,
And now their pride and mettle is asleep,
Their courage with hard labour tame and dull,
That not a horse is half the half himself.

HOTSPUR.
So are the horses of the enemy
In general, journey-bated and brought low.
The better part of ours are full of rest.

WORCESTER.
The number of the King exceedeth ours.
For God’s sake, cousin, stay till all come in.

[_The trumpet sounds a parley._]

Enter Sir Walter Blunt.

BLUNT.
I come with gracious offers from the King,
If you vouchsafe me hearing and respect.

HOTSPUR.
Welcome, Sir Walter Blunt, and would to God
You were of our determination!
Some of us love you well, and even those some
Envy your great deservings and good name,
Because you are not of our quality,
But stand against us like an enemy.

BLUNT.
And God defend but still I should stand so,
So long as out of limit and true rule
You stand against anointed majesty.
But to my charge. The King hath sent to know
The nature of your griefs, and whereupon
You conjure from the breast of civil peace
Such bold hostility, teaching his duteous land
Audacious cruelty. If that the King
Have any way your good deserts forgot,
Which he confesseth to be manifold,
He bids you name your griefs, and with all speed
You shall have your desires with interest
And pardon absolute for yourself and these
Herein misled by your suggestion.

HOTSPUR.
The King is kind, and well we know the King
Knows at what time to promise, when to pay.
My father and my uncle and myself
Did give him that same royalty he wears,
And when he was not six-and-twenty strong,
Sick in the world’s regard, wretched and low,
A poor unminded outlaw sneaking home,
My father gave him welcome to the shore:
And when he heard him swear and vow to God
He came but to be Duke of Lancaster,
To sue his livery, and beg his peace
With tears of innocence and terms of zeal,
My father, in kind heart and pity moved,
Swore him assistance, and performed it too.
Now, when the lords and barons of the realm
Perceived Northumberland did lean to him,
The more and less came in with cap and knee,
Met him in boroughs, cities, villages,
Attended him on bridges, stood in lanes,
Laid gifts before him, proffer’d him their oaths,
Give him their heirs as pages, follow’d him
Even at the heels in golden multitudes.
He presently, as greatness knows itself,
Steps me a little higher than his vow
Made to my father while his blood was poor
Upon the naked shore at Ravenspurgh;
And now forsooth takes on him to reform
Some certain edicts and some strait decrees
That lie too heavy on the commonwealth;
Cries out upon abuses, seems to weep
Over his country’s wrongs; and by this face,
This seeming brow of justice, did he win
The hearts of all that he did angle for;
Proceeded further—cut me off the heads
Of all the favourites that the absent King
In deputation left behind him here
When he was personal in the Irish war.

BLUNT.
Tut, I came not to hear this.

HOTSPUR.
Then to the point.
In short time after, he deposed the King,
Soon after that deprived him of his life,
And, in the neck of that, task’d the whole state.
To make that worse, suffer’d his kinsman March
(Who is, if every owner were well placed,
Indeed his king) to be engaged in Wales,
There without ransom to lie forfeited;
Disgraced me in my happy victories,
Sought to entrap me by intelligence,
Rated mine uncle from the Council-board,
In rage dismiss’d my father from the court,
Broke oath on oath, committed wrong on wrong,
And in conclusion drove us to seek out
This head of safety, and withal to pry
Into his title, the which now we find
Too indirect for long continuance.

BLUNT.
Shall I return this answer to the King?

HOTSPUR.
Not so, Sir Walter. We’ll withdraw awhile.
Go to the King, and let there be impawn’d
Some surety for a safe return again,
And in the morning early shall my uncle
Bring him our purposes. And so, farewell.

BLUNT.
I would you would accept of grace and love.

HOTSPUR.
And maybe so we shall.

BLUNT.
Pray God you do.

[_Exeunt._]

SCENE IV. York. A Room in the Archbishop’s Palace.

Enter the Archbishop of York and Sir Michael.

ARCHBISHOP.
Hie, good Sir Michael; bear this sealed brief
With winged haste to the Lord Marshal,
This to my cousin Scroop, and all the rest
To whom they are directed. If you knew
How much they do import, you would make haste.

SIR MICHAEL.
My good lord,
I guess their tenour.

ARCHBISHOP.
Like enough you do.
Tomorrow, good Sir Michael, is a day
Wherein the fortune of ten thousand men
Must bide the touch; for, sir, at Shrewsbury,
As I am truly given to understand,
The King with mighty and quick-raised power
Meets with Lord Harry. And, I fear, Sir Michael,
What with the sickness of Northumberland,
Whose power was in the first proportion,
And what with Owen Glendower’s absence thence,
Who with them was a rated sinew too,
And comes not in, o’er-rul’d by prophecies,
I fear the power of Percy is too weak
To wage an instant trial with the King.

SIR MICHAEL.
Why, my good lord, you need not fear,
There is Douglas and Lord Mortimer.

ARCHBISHOP.
No, Mortimer is not there.

SIR MICHAEL.
But there is Mordake, Vernon, Lord Harry Percy,
And there is my Lord of Worcester, and a head
Of gallant warriors, noble gentlemen.

ARCHBISHOP.
And so there is. But yet the King hath drawn
The special head of all the land together:
The Prince of Wales, Lord John of Lancaster,
The noble Westmoreland, and warlike Blunt,
And many more corrivals and dear men
Of estimation and command in arms.

SIR MICHAEL.
Doubt not, my lord, they shall be well opposed.

ARCHBISHOP.
I hope no less, yet needful ’tis to fear;
And to prevent the worst, Sir Michael, speed.
For if Lord Percy thrive not, ere the King
Dismiss his power he means to visit us,
For he hath heard of our confederacy,
And ’tis but wisdom to make strong against him.
Therefore make haste. I must go write again
To other friends; and so, farewell, Sir Michael.

[_Exeunt._]




ACT V


SCENE I. The King’s Camp near Shrewsbury.

Enter King Henry, Prince Henry, Lancaster, Sir Walter Blunt and Sir
John Falstaff.

KING.
How bloodily the sun begins to peer
Above yon bulky hill! The day looks pale
At his distemp’rature.

PRINCE.
The southern wind
Doth play the trumpet to his purposes,
And by his hollow whistling in the leaves
Foretells a tempest and a blust’ring day.

KING.
Then with the losers let it sympathize,
For nothing can seem foul to those that win.

[_The trumpet sounds_.]

Enter Worcester and Vernon.

How, now, my Lord of Worcester! ’Tis not well
That you and I should meet upon such terms
As now we meet. You have deceived our trust,
And made us doff our easy robes of peace,
To crush our old limbs in ungentle steel.
This is not well, my lord, this is not well.
What say you to it? Will you again unknit
This churlish knot of all-abhorred war,
And move in that obedient orb again
Where you did give a fair and natural light,
And be no more an exhaled meteor,
A prodigy of fear, and a portent
Of broached mischief to the unborn times?

WORCESTER.
Hear me, my liege:
For mine own part, I could be well content
To entertain the lag end of my life
With quiet hours. For I do protest
I have not sought the day of this dislike.

KING.
You have not sought it? How comes it, then?

FALSTAFF.
Rebellion lay in his way, and he found it.

PRINCE.
Peace, chewet, peace!

WORCESTER.
It pleased your Majesty to turn your looks
Of favour from myself and all our house;
And yet I must remember you, my lord,
We were the first and dearest of your friends.
For you my staff of office did I break
In Richard’s time, and posted day and night
To meet you on the way, and kiss your hand,
When yet you were in place and in account
Nothing so strong and fortunate as I.
It was myself, my brother, and his son,
That brought you home, and boldly did outdare
The dangers of the time. You swore to us,
And you did swear that oath at Doncaster,
That you did nothing purpose ’gainst the state,
Nor claim no further than your new-fall’n right,
The seat of Gaunt, dukedom of Lancaster.
To this we swore our aid. But in short space
It rain’d down fortune show’ring on your head,
And such a flood of greatness fell on you,
What with our help, what with the absent King,
What with the injuries of a wanton time,
The seeming sufferances that you had borne,
And the contrarious winds that held the King
So long in his unlucky Irish wars
That all in England did repute him dead:
And from this swarm of fair advantages
You took occasion to be quickly woo’d
To gripe the general sway into your hand,
Forgot your oath to us at Doncaster;
And, being fed by us, you used us so
As that ungentle gull, the cuckoo’s bird,
Useth the sparrow—did oppress our nest,
Grew by our feeding to so great a bulk
That even our love durst not come near your sight
For fear of swallowing; but with nimble wing
We were enforced, for safety sake to fly
Out of your sight, and raise this present head,
Whereby we stand opposed by such means
As you yourself have forged against yourself,
By unkind usage, dangerous countenance,
And violation of all faith and troth
Sworn to us in your younger enterprise.

KING.
These things, indeed, you have articulate,
Proclaim’d at market crosses, read in churches,
To face the garment of rebellion
With some fine colour that may please the eye
Of fickle changelings and poor discontents,
Which gape and rub the elbow at the news
Of hurlyburly innovation.
And never yet did insurrection want
Such water-colours to impaint his cause,
Nor moody beggars starving for a time
Of pellmell havoc and confusion.

PRINCE.
In both your armies there is many a soul
Shall pay full dearly for this encounter
If once they join in trial. Tell your nephew,
The Prince of Wales doth join with all the world
In praise of Henry Percy. By my hopes,
This present enterprise set off his head,
I do not think a braver gentleman,
More active-valiant or more valiant-young,
More daring or more bold, is now alive
To grace this latter age with noble deeds.
For my part, I may speak it to my shame,
I have a truant been to chivalry,
And so I hear he doth account me too.
Yet this before my father’s Majesty—
I am content that he shall take the odds
Of his great name and estimation,
And will, to save the blood on either side,
Try fortune with him in a single fight.

KING.
And, Prince of Wales, so dare we venture thee,
Albeit considerations infinite
Do make against it.—No, good Worcester, no.
We love our people well, even those we love
That are misled upon your cousin’s part,
And, will they take the offer of our grace,
Both he, and they, and you, yea, every man
Shall be my friend again, and I’ll be his.
So tell your cousin, and then bring me word
What he will do. But if he will not yield,
Rebuke and dread correction wait on us,
And they shall do their office. So, be gone;
We will not now be troubled with reply.
We offer fair, take it advisedly.

[_Exit Worcester with Vernon._]

PRINCE.
It will not be accepted, on my life.
The Douglas and the Hotspur both together
Are confident against the world in arms.

KING.
Hence, therefore, every leader to his charge;
For on their answer, will we set on them,
And God befriend us as our cause is just!

[_Exeunt the King, Blunt and Prince John._]

FALSTAFF.
Hal, if thou see me down in the battle and bestride me, so; ’tis a
point of friendship.

PRINCE.
Nothing but a colossus can do thee that friendship.
Say thy prayers, and farewell.

FALSTAFF.
I would ’twere bedtime, Hal, and all well.

PRINCE.
Why, thou owest God a death.

[_Exit._]

FALSTAFF.
’Tis not due yet, I would be loth to pay Him before His day. What need
I be so forward with Him that calls not on me? Well, ’tis no matter,
honour pricks me on. Yea, but how if honour prick me off when I come
on? How then? Can honor set to a leg? No. Or an arm? No. Or take away
the grief of a wound? No. Honour hath no skill in surgery then? No.
What is honour? A word. What is in that word, “honour”? What is that
“honour”? Air. A trim reckoning! Who hath it? He that died o’
Wednesday. Doth he feel it? No. Doth be hear it? No. ’Tis insensible,
then? Yea, to the dead. But will it not live with the living? No. Why?
Detraction will not suffer it. Therefore I’ll none of it. Honour is a
mere scutcheon. And so ends my catechism.

[_Exit._]

SCENE II. The Rebel Camp.

Enter Worcester and Vernon.

WORCESTER.
O no, my nephew must not know, Sir Richard,
The liberal and kind offer of the King.

VERNON.
’Twere best he did.

WORCESTER.
Then are we all undone.
It is not possible, it cannot be,
The King should keep his word in loving us;
He will suspect us still, and find a time
To punish this offence in other faults.
Suspicion all our lives shall be stuck full of eyes,
For treason is but trusted like the fox,
Who, ne’er so tame, so cherish’d and lock’d up,
Will have a wild trick of his ancestors.
Look how we can, or sad or merrily,
Interpretation will misquote our looks,
And we shall feed like oxen at a stall,
The better cherish’d still the nearer death.
My nephew’s trespass may be well forgot,
It hath the excuse of youth and heat of blood,
And an adopted name of privilege—
A hare-brain’d Hotspur, govern’d by a spleen.
All his offences live upon my head
And on his father’s. We did train him on,
And, his corruption being ta’en from us,
We as the spring of all shall pay for all.
Therefore, good cousin, let not Harry know
In any case the offer of the King.

VERNON.
Deliver what you will, I’ll say ’tis so.
Here comes your cousin.

Enter Hotspur and Douglas; Officers and Soldiers behind.

HOTSPUR.
My uncle is return’d.
Deliver up my Lord of Westmoreland.
Uncle, what news?

WORCESTER.
The King will bid you battle presently.

DOUGLAS.
Defy him by the Lord Of Westmoreland.

HOTSPUR.
Lord Douglas, go you and tell him so.

DOUGLAS.
Marry, I shall, and very willingly.

[_Exit._]

WORCESTER.
There is no seeming mercy in the King.

HOTSPUR.
Did you beg any? God forbid!

WORCESTER.
I told him gently of our grievances,
Of his oath-breaking; which he mended thus,
By now forswearing that he is forsworn.
He calls us rebels, traitors, and will scourge
With haughty arms this hateful name in us.

Enter Douglas.

DOUGLAS.
Arm, gentlemen; to arms! For I have thrown
A brave defiance in King Henry’s teeth,
And Westmoreland, that was engaged, did bear it,
Which cannot choose but bring him quickly on.

WORCESTER.
The Prince of Wales stepp’d forth before the King,
And, nephew, challenged you to single fight.

HOTSPUR.
O, would the quarrel lay upon our heads,
And that no man might draw short breath today
But I and Harry Monmouth! Tell me, tell me,
How show’d his tasking? Seem’d it in contempt?

VERNON.
No, by my soul. I never in my life
Did hear a challenge urged more modestly,
Unless a brother should a brother dare
To gentle exercise and proof of arms.
He gave you all the duties of a man,
Trimm’d up your praises with a princely tongue,
Spoke your deservings like a chronicle,
Making you ever better than his praise
By still dispraising praise valued with you,
And, which became him like a prince indeed,
He made a blushing cital of himself,
And chid his truant youth with such a grace
As if he master’d there a double spirit
Of teaching and of learning instantly.
There did he pause: but let me tell the world,
If he outlive the envy of this day,
England did never owe so sweet a hope
So much misconstrued in his wantonness.

HOTSPUR.
Cousin, I think thou art enamoured
Upon his follies. Never did I hear
Of any prince so wild a liberty.
But be he as he will, yet once ere night
I will embrace him with a soldier’s arm,
That he shall shrink under my courtesy.
Arm, arm with speed! And, fellows, soldiers, friends,
Better consider what you have to do
Than I that have not well the gift of tongue
Can lift your blood up with persuasion.

Enter a Messenger.

MESSENGER.
My lord, here are letters for you.

HOTSPUR.
I cannot read them now.—
O gentlemen, the time of life is short!
To spend that shortness basely were too long
If life did ride upon a dial’s point,
Still ending at the arrival of an hour.
And if we live, we live to tread on kings;
If die, brave death, when princes die with us!
Now, for our consciences, the arms are fair
When the intent of bearing them is just.

Enter another Messenger.

MESSENGER.
My lord, prepare. The King comes on apace.

HOTSPUR.
I thank him that he cuts me from my tale,
For I profess not talking. Only this:
Let each man do his best. And here draw I
A sword whose temper I intend to stain
With the best blood that I can meet withal
In the adventure of this perilous day.
Now, Esperance! Percy! And set on.
Sound all the lofty instruments of war,
And by that music let us all embrace,
For, Heaven to Earth, some of us never shall
A second time do such a courtesy.

[_The trumpets sound. They embrace, and exeunt._]

SCENE III. Plain between the Camps.

The King enters with his power. Alarum to the battle. Then enter
Douglas and Sir Walter Blunt.

BLUNT.
What is thy name that in the battle thus
Thou crossest me? What honour dost thou seek
Upon my head?

DOUGLAS.
Know then my name is Douglas,
And I do haunt thee in the battle thus
Because some tell me that thou art a king.

BLUNT.
They tell thee true.

DOUGLAS.
The Lord of Stafford dear today hath bought
Thy likeness, for instead of thee, King Harry,
This sword hath ended him. So shall it thee,
Unless thou yield thee as my prisoner.

BLUNT.
I was not born a yielder, thou proud Scot,
And thou shalt find a king that will revenge
Lord Stafford’s death.

[_They fight, and Blunt is slain._]

Enter Hotspur.

HOTSPUR.
O Douglas, hadst thou fought at Holmedon thus,
I never had triumphed upon a Scot.

DOUGLAS.
All’s done, all’s won; here breathless lies the King.

HOTSPUR.
Where?

DOUGLAS.
Here.

HOTSPUR.
This, Douglas? No, I know this face full well.
A gallant knight he was, his name was Blunt,
Semblably furnish’d like the King himself.

DOUGLAS.
A fool go with thy soul, whither it goes!
A borrow’d title hast thou bought too dear.
Why didst thou tell me that thou wert a king?

HOTSPUR.
The King hath many marching in his coats.

DOUGLAS.
Now, by my sword, I will kill all his coats;
I’ll murder all his wardrobe, piece by piece,
Until I meet the King.

HOTSPUR.
Up, and away!
Our soldiers stand full fairly for the day.

[_Exeunt._]

Alarums. Enter Falstaff solus.

FALSTAFF.
Though I could scape shot-free at London, I fear the shot here. Here’s
no scoring but upon the pate.—Soft! who are you? Sir Walter Blunt.
There’s honour for you. Here’s no vanity. I am as hot as molten lead,
and as heavy too. God keep lead out of me, I need no more weight than
mine own bowels. I have led my ragamuffins where they are peppered.
There’s not three of my hundred and fifty left alive, and they are for
the town’s end, to beg during life. But who comes here?

Enter Prince Henry.

PRINCE.
What, stand’st thou idle here? Lend me thy sword.
Many a nobleman lies stark and stiff
Under the hoofs of vaunting enemies,
Whose deaths are yet unrevenged. I prithee
Lend me thy sword.

FALSTAFF.
O Hal, I prithee give me leave to breathe awhile. Turk Gregory never
did such deeds in arms as I have done this day. I have paid Percy, I
have made him sure.

PRINCE.
He is indeed, and living to kill thee.
I prithee, lend me thy sword.

FALSTAFF.
Nay, before God, Hal, if Percy be alive, thou gett’st not my sword; but
take my pistol, if thou wilt.

PRINCE.
Give it me. What, is it in the case?

FALSTAFF.
Ay, Hal, ’tis hot, ’tis hot. There’s that will sack a city.

[_The Prince draws out a bottle of sack._]

PRINCE.
What, is it a time to jest and dally now?

[_Throws it at him, and exit._]

FALSTAFF.
Well, if Percy be alive, I’ll pierce him. If he do come in my way, so;
if he do not, if I come in his willingly, let him make a carbonado of
me. I like not such grinning honour as Sir Walter hath. Give me life,
which if I can save, so: if not, honour comes unlooked for, and there’s
an end.

[_Exit._]

SCENE IV. Another Part of the Field.

Alarums. Excursions. Enter King Henry, Prince Henry, Lancaster and
Westmoreland.

KING.
I prithee, Harry, withdraw thyself, thou bleedest too much.
Lord John of Lancaster, go you with him.

LANCASTER.
Not I, my lord, unless I did bleed too.

PRINCE.
I do beseech your Majesty, make up,
Lest your retirement do amaze your friends.

KING.
I will do so. My Lord of Westmoreland,
Lead him to his tent.

WESTMORELAND.
Come, my lord, I’ll lead you to your tent.

PRINCE.
Lead me, my lord? I do not need your help,
And God forbid a shallow scratch should drive
The Prince of Wales from such a field as this,
Where stain’d nobility lies trodden on,
And rebels’ arms triumph in massacres!

LANCASTER.
We breathe too long. Come, cousin Westmoreland,
Our duty this way lies. For God’s sake, come.

[_Exeunt Lancaster and Westmoreland._]

PRINCE.
By Heaven, thou hast deceived me, Lancaster,
I did not think thee lord of such a spirit.
Before, I loved thee as a brother, John,
But now I do respect thee as my soul.

KING.
I saw him hold Lord Percy at the point
With lustier maintenance than I did look for
Of such an ungrown warrior.

PRINCE.
O, this boy
Lends mettle to us all!

[_Exit._]

Enter Douglas.

DOUGLAS.
Another king! They grow like Hydra’s heads.
I am the Douglas, fatal to all those
That wear those colours on them. What art thou
That counterfeit’st the person of a king?

KING.
The King himself, who, Douglas, grieves at heart
So many of his shadows thou hast met,
And not the very King. I have two boys
Seek Percy and thyself about the field,
But, seeing thou fall’st on me so luckily,
I will assay thee, and defend thyself.

DOUGLAS.
I fear thou art another counterfeit,
And yet, in faith, thou bearest thee like a king.
But mine I am sure thou art, whoe’er thou be,
And thus I win thee.

They fight; the King being in danger, enter Prince Henry.

PRINCE.
Hold up thy head, vile Scot, or thou art like
Never to hold it up again! The spirits
Of valiant Shirley, Stafford, Blunt are in my arms.
It is the Prince of Wales that threatens thee,
Who never promiseth but he means to pay.

[_They fight. Douglas flies._]

Cheerly, my lord. How fares your Grace?
Sir Nicholas Gawsey hath for succour sent,
And so hath Clifton. I’ll to Clifton straight.

KING.
Stay and breathe awhile.
Thou hast redeem’d thy lost opinion,
And show’d thou mak’st some tender of my life,
In this fair rescue thou hast brought to me.

PRINCE.
O God, they did me too much injury
That ever said I hearken’d for your death.
If it were so, I might have let alone
The insulting hand of Douglas over you,
Which would have been as speedy in your end
As all the poisonous potions in the world,
And saved the treacherous labour of your son.

KING.
Make up to Clifton. I’ll to Sir Nicholas Gawsey.

[_Exit._]

Enter Hotspur.

HOTSPUR.
If I mistake not, thou art Harry Monmouth.

PRINCE.
Thou speak’st as if I would deny my name.

HOTSPUR.
My name is Harry Percy.

PRINCE.
Why then I see
A very valiant rebel of the name.
I am the Prince of Wales; and think not, Percy,
To share with me in glory any more.
Two stars keep not their motion in one sphere,
Nor can one England brook a double reign,
Of Harry Percy and the Prince of Wales.

HOTSPUR.
Nor shall it, Harry, for the hour is come
To end the one of us, and would to God
Thy name in arms were now as great as mine!

PRINCE.
I’ll make it greater ere I part from thee,
And all the budding honours on thy crest
I’ll crop to make a garland for my head.

HOTSPUR.
I can no longer brook thy vanities.

[_They fight._]

Enter Falstaff.

FALSTAFF.
Well said, Hal! To it, Hal! Nay, you shall find no boy’s play here, I
can tell you.

Enter Douglas. He fights with Falstaff, who falls down as if he were
dead, and exit Douglas. The Prince kills Hotspur.

HOTSPUR.
O Harry, thou hast robb’d me of my youth!
I better brook the loss of brittle life
Than those proud titles thou hast won of me;
They wound my thoughts worse than thy sword my flesh.
But thoughts, the slaves of life, and life, time’s fool,
And time, that takes survey of all the world,
Must have a stop. O, I could prophesy,
But that the earthy and cold hand of death
Lies on my tongue. No, Percy, thou art dust,
And food for—

[_Dies._]

PRINCE.
For worms, brave Percy. Fare thee well, great heart!
Ill-weaved ambition, how much art thou shrunk!
When that this body did contain a spirit,
A kingdom for it was too small a bound;
But now two paces of the vilest earth
Is room enough. This earth that bears thee dead
Bears not alive so stout a gentleman.
If thou wert sensible of courtesy,
I should not make so dear a show of zeal.
But let my favours hide thy mangled face;
And even in thy behalf I’ll thank myself
For doing these fair rites of tenderness.
Adieu, and take thy praise with thee to heaven!
Thy ignominy sleep with thee in the grave,
But not remember’d in thy epitaph!

[_Sees Falstaff on the ground._]

What, old acquaintance, could not all this flesh
Keep in a little life? Poor Jack, farewell!
I could have better spared a better man.
O, I should have a heavy miss of thee
If I were much in love with vanity.
Death hath not struck so fat a deer today,
Though many dearer, in this bloody fray.
Embowell’d will I see thee by and by,
Till then in blood by noble Percy lie.

[_Exit._]

Falstaff rises up.

FALSTAFF.
Embowell’d! If thou embowel me today, I’ll give you leave to powder me
and eat me too tomorrow. ’Sblood, ’twas time to counterfeit, or that
hot termagant Scot had paid me scot and lot too. Counterfeit? I lie, I
am no counterfeit. To die, is to be a counterfeit, for he is but the
counterfeit of a man who hath not the life of a man: but to counterfeit
dying, when a man thereby liveth, is to be no counterfeit, but the true
and perfect image of life indeed. The better part of valour is
discretion, in the which better part I have saved my life. Zounds, I am
afraid of this gunpowder Percy, though he be dead. How if he should
counterfeit too, and rise? By my faith, I am afraid he would prove the
better counterfeit. Therefore I’ll make him sure, yea, and I’ll swear I
killed him. Why may not he rise as well as I? Nothing confutes me but
eyes, and nobody sees me. Therefore, sirrah, with a new wound in your
thigh, come you along with me.

[_Takes Hotspur on his back._]

Enter Prince Henry and Lancaster.

PRINCE.
Come, brother John, full bravely hast thou flesh’d
Thy maiden sword.

LANCASTER.
But soft, whom have we here?
Did you not tell me this fat man was dead?

PRINCE.
I did; I saw him dead,
Breathless and bleeding on the ground.—Art thou alive?
Or is it fantasy that plays upon our eyesight?
I prithee, speak, we will not trust our eyes
Without our ears. Thou art not what thou seem’st.

FALSTAFF.
No, that’s certain, I am not a double man. But if I be not Jack
Falstaff, then am I a Jack. There is Percy! [_Throwing the body down._]
If your father will do me any honour, so; if not, let him kill the next
Percy himself. I look to be either earl or duke, I can assure you.

PRINCE.
Why, Percy I kill’d myself, and saw thee dead.

FALSTAFF.
Didst thou? Lord, Lord, how this world is given to lying! I grant you I
was down and out of breath, and so was he, but we rose both at an
instant, and fought a long hour by Shrewsbury clock. If I may be
believed, so; if not, let them that should reward valour bear the sin
upon their own heads. I’ll take it upon my death, I gave him this wound
in the thigh. If the man were alive, and would deny it, zounds, I would
make him eat a piece of my sword.

LANCASTER.
This is the strangest tale that ever I heard.

PRINCE.
This is the strangest fellow, brother John.—
Come, bring your luggage nobly on your back.
For my part, if a lie may do thee grace,
I’ll gild it with the happiest terms I have.

[_A retreat is sounded._]

The trumpet sounds retreat; the day is ours.
Come, brother, let us to the highest of the field,
To see what friends are living, who are dead.

[_Exeunt Prince Henry and Lancaster._]

FALSTAFF.
I’ll follow, as they say, for reward. He that rewards me, God reward
him! If I do grow great, I’ll grow less, for I’ll purge, and leave
sack, and live cleanly as a nobleman should do.

[_Exit, bearing off the body._]

SCENE V. Another Part of the Field.

The trumpets sound. Enter King Henry, Prince Henry, Lancaster,
Westmoreland and others, with Worcester and Vernon prisoners.

KING.
Thus ever did rebellion find rebuke.
Ill-spirited Worcester, did not we send grace,
Pardon, and terms of love to all of you?
And wouldst thou turn our offers contrary?
Misuse the tenour of thy kinsman’s trust?
Three knights upon our party slain today,
A noble earl, and many a creature else,
Had been alive this hour,
If, like a Christian, thou hadst truly borne
Betwixt our armies true intelligence.

WORCESTER.
What I have done my safety urged me to;
And I embrace this fortune patiently,
Since not to be avoided it falls on me.

KING.
Bear Worcester to the death, and Vernon too.
Other offenders we will pause upon.

[_Exeunt Worcester and Vernon, guarded._]

How goes the field?

PRINCE.
The noble Scot, Lord Douglas, when he saw
The fortune of the day quite turn’d from him,
The noble Percy slain, and all his men
Upon the foot of fear, fled with the rest,
And, falling from a hill, he was so bruised
That the pursuers took him. At my tent
The Douglas is, and I beseech your Grace
I may dispose of him.

KING.
With all my heart.

PRINCE.
Then, brother John of Lancaster, to you
This honourable bounty shall belong.
Go to the Douglas and deliver him
Up to his pleasure, ransomless and free.
His valours shown upon our crests today
Hath taught us how to cherish such high deeds,
Even in the bosom of our adversaries.

LANCASTER.
I thank your Grace for this high courtesy,
Which I shall give away immediately.

KING.
Then this remains, that we divide our power.
You, son John, and my cousin Westmoreland,
Towards York shall bend you with your dearest speed
To meet Northumberland and the prelate Scroop,
Who, as we hear, are busily in arms.
Myself and you, son Harry, will towards Wales,
To fight with Glendower and the Earl of March.
Rebellion in this land shall lose his sway,
Meeting the check of such another day,
And since this business so fair is done,
Let us not leave till all our own be won.

[_Exeunt._]




THE SECOND PART OF KING HENRY THE FOURTH




Contents

 INDUCTION

 ACT I
 Scene I. The same.
 Scene II. London. A street.
 Scene III. York. The Archbishop’s palace.

 ACT II
 Scene I. London. A street.
 Scene II. London. Another street.
 Scene III. Warkworth. Before the castle.
 Scene IV. The Boar’s head Tavern in Eastcheap.

 ACT III
 Scene I. Westminster. The palace.
 Scene II. Gloucestershire. Before Justice Shallow’s house.

 ACT IV
 Scene I. Yorkshire. Gaultree Forest.
 Scene II. Another part of the forest.
 Scene III. Another part of the forest.
 Scene IV. Westminster. The Jerusalem Chamber.
 Scene V. Another chamber.

 ACT V
 Scene I. Gloucestershire. Shallow’s house.
 Scene II. Westminster. The palace.
 Scene III. Gloucestershire. Shallow’s orchard.
 Scene IV. London. A street.
 Scene V. A public place near Westminster Abbey.

 EPILOGUE




Dramatis Personæ

RUMOUR, the Presenter.
KING HENRY the Fourth.
HENRY, PRINCE OF WALES, afterwards King Henry the Fifth.
THOMAS, DUKE OF CLARENCE.
PRINCE JOHN OF LANCASTER.
PRINCE HUMPHREY OF GLOUCESTER.
EARL OF WARWICK.
EARL OF WESTMORELAND.
EARL OF SURREY.
GOWER.
HARCOURT.
SIR JOHN BLUNT.
Lord CHIEF JUSTICE of the King’s Bench.
A SERVANT of the Chief Justice.
Henry Percy, Earl of NORTHUMBERLAND.
Scroop, ARCHBISHOP of York.
Lord MOWBRAY.
Lord HASTINGS.
LORD BARDOLPH.
SIR JOHN COLEVILLE.
TRAVERS and MORTON, retainers of Northumberland.
SIR JOHN FALSTAFF.
His Page.
BARDOLPH.
PISTOL.
POINS.
PETO.
SHALLOW and SILENCE, country justices.
DAVY, Servant to Shallow.
MOULDY, SHADOW, WART, FEEBLE, and BULLCALF, recruits.
FANG and SNARE, sheriff’s officers.

LADY NORTHUMBERLAND.
LADY PERCY.
MISTRESS QUICKLY, hostess of a tavern in Eastcheap.
DOLL TEARSHEET.

Lords and Attendants; Porter, Drawers, Musicians, Beadles, Grooms, etc.

A Dancer, speaker of the epilogue.

SCENE: England.




INDUCTION


Warkworth. Before the castle.

Enter Rumour, painted full of tongues.

RUMOUR.
Open your ears; for which of you will stop
The vent of hearing when loud Rumour speaks?
I, from the orient to the drooping west,
Making the wind my post-horse, still unfold
The acts commenced on this ball of earth.
Upon my tongues continual slanders ride,
The which in every language I pronounce,
Stuffing the ears of men with false reports.
I speak of peace, while covert enmity
Under the smile of safety wounds the world.
And who but Rumour, who but only I,
Make fearful musters and prepared defence,
Whiles the big year, swoln with some other grief,
Is thought with child by the stern tyrant war,
And no such matter? Rumour is a pipe
Blown by surmises, jealousies, conjectures,
And of so easy and so plain a stop
That the blunt monster with uncounted heads,
The still-discordant wav’ring multitude,
Can play upon it. But what need I thus
My well-known body to anatomize
Among my household? Why is Rumour here?
I run before King Harry’s victory,
Who in a bloody field by Shrewsbury
Hath beaten down young Hotspur and his troops,
Quenching the flame of bold rebellion
Even with the rebels’ blood. But what mean I
To speak so true at first? My office is
To noise abroad that Harry Monmouth fell
Under the wrath of noble Hotspur’s sword,
And that the King before the Douglas’ rage
Stoop’d his anointed head as low as death.
This have I rumour’d through the peasant towns
Between that royal field of Shrewsbury
And this worm-eaten hold of ragged stone,
Where Hotspur’s father, old Northumberland,
Lies crafty-sick. The posts come tiring on,
And not a man of them brings other news
Than they have learnt of me. From Rumour’s tongues
They bring smooth comforts false, worse than true wrongs.

 [_Exit._]




ACT I

SCENE I. The same.

Enter Lord Bardolph.

LORD BARDOLPH.
Who keeps the gate here, ho?

The Porter opens the gate.

Where is the Earl?

PORTER.
What shall I say you are?

LORD BARDOLPH.
Tell thou the Earl
That the Lord Bardolph doth attend him here.

PORTER.
His lordship is walk’d forth into the orchard.
Please it your honour knock but at the gate,
And he himself will answer.

Enter Northumberland.

LORD BARDOLPH.
Here comes the Earl.

 [_Exit Porter._]

NORTHUMBERLAND.
What news, Lord Bardolph? Every minute now
Should be the father of some stratagem.
The times are wild; contention, like a horse
Full of high feeding, madly hath broke loose
And bears down all before him.

LORD BARDOLPH.
Noble earl,
I bring you certain news from Shrewsbury.

NORTHUMBERLAND.
Good, an God will!

LORD BARDOLPH.
As good as heart can wish.
The King is almost wounded to the death;
And, in the fortune of my lord your son,
Prince Harry slain outright; and both the Blunts
Kill’d by the hand of Douglas; young Prince John
And Westmoreland and Stafford fled the field;
And Harry Monmouth’s brawn, the hulk Sir John,
Is prisoner to your son. O, such a day,
So fought, so follow’d and so fairly won,
Came not till now to dignify the times
Since Caesar’s fortunes!

NORTHUMBERLAND.
How is this derived?
Saw you the field? Came you from Shrewsbury?

LORD BARDOLPH.
I spake with one, my lord, that came from thence,
A gentleman well bred and of good name,
That freely render’d me these news for true.

NORTHUMBERLAND.
Here comes my servant Travers, whom I sent
On Tuesday last to listen after news.

Enter Travers.

LORD BARDOLPH.
My lord, I over-rode him on the way,
And he is furnish’d with no certainties
More than he haply may retail from me.

NORTHUMBERLAND.
Now, Travers, what good tidings comes with you?

TRAVERS.
My lord, Sir John Umfrevile turn’d me back
With joyful tidings, and, being better horsed,
Out-rode me. After him came spurring hard
A gentleman, almost forspent with speed,
That stopp’d by me to breathe his bloodied horse.
He ask’d the way to Chester, and of him
I did demand what news from Shrewsbury.
He told me that rebellion had bad luck
And that young Harry Percy’s spur was cold.
With that he gave his able horse the head,
And bending forward struck his armed heels
Against the panting sides of his poor jade
Up to the rowel-head, and starting so
He seem’d in running to devour the way,
Staying no longer question.

NORTHUMBERLAND.
Ha? Again:
Said he young Harry Percy’s spur was cold?
Of Hotspur, Coldspur? That rebellion
Had met ill luck?

LORD BARDOLPH.
My lord, I’ll tell you what:
If my young lord your son have not the day,
Upon mine honour, for a silken point
I’ll give my barony, never talk of it.

NORTHUMBERLAND.
Why should that gentleman that rode by Travers
Give then such instances of loss?

LORD BARDOLPH.
Who, he?
He was some hilding fellow that had stolen
The horse he rode on, and, upon my life,
Spoke at a venture. Look, here comes more news.

Enter Morton.

NORTHUMBERLAND.
Yea, this man’s brow, like to a title-leaf,
Foretells the nature of a tragic volume.
So looks the strand whereon the imperious flood
Hath left a witness’d usurpation.
Say, Morton, didst thou come from Shrewsbury?

MORTON.
I ran from Shrewsbury, my noble lord,
Where hateful death put on his ugliest mask
To fright our party.

NORTHUMBERLAND.
How doth my son and brother?
Thou tremblest, and the whiteness in thy cheek
Is apter than thy tongue to tell thy errand.
Even such a man, so faint, so spiritless,
So dull, so dread in look, so woe-begone,
Drew Priam’s curtain in the dead of night,
And would have told him half his Troy was burnt;
But Priam found the fire ere he his tongue,
And I my Percy’s death ere thou report’st it.
This thou wouldst say: “Your son did thus and thus;
Your brother thus; so fought the noble Douglas”
Stopping my greedy ear with their bold deeds:
But in the end, to stop my ear indeed,
Thou hast a sigh to blow away this praise,
Ending with “Brother, son, and all are dead.”

MORTON.
Douglas is living, and your brother, yet;
But, for my lord your son—

NORTHUMBERLAND.
Why, he is dead.
See what a ready tongue suspicion hath!
He that but fears the thing he would not know
Hath by instinct knowledge from others’ eyes
That what he fear’d is chanced. Yet speak, Morton;
Tell thou an earl his divination lies,
And I will take it as a sweet disgrace
And make thee rich for doing me such wrong.

MORTON.
You are too great to be by me gainsaid,
Your spirit is too true, your fears too certain.

NORTHUMBERLAND.
Yet, for all this, say not that Percy’s dead.
I see a strange confession in thine eye.
Thou shakest thy head and hold’st it fear or sin
To speak a truth. If he be slain, say so.
The tongue offends not that reports his death;
And he doth sin that doth belie the dead,
Not he which says the dead is not alive.
Yet the first bringer of unwelcome news
Hath but a losing office, and his tongue
Sounds ever after as a sullen bell,
Remember’d tolling a departing friend.

LORD BARDOLPH.
I cannot think, my lord, your son is dead.

MORTON.
I am sorry I should force you to believe
That which I would to God I had not seen;
But these mine eyes saw him in bloody state,
Rend’ring faint quittance, wearied and outbreathed,
To Harry Monmouth, whose swift wrath beat down
The never-daunted Percy to the earth,
From whence with life he never more sprung up.
In few, his death, whose spirit lent a fire
Even to the dullest peasant in his camp,
Being bruited once, took fire and heat away
From the best-temper’d courage in his troops;
For from his metal was his party steel’d,
Which once in him abated, all the rest
Turn’d on themselves, like dull and heavy lead.
And as the thing that’s heavy in itself
Upon enforcement flies with greatest speed,
So did our men, heavy in Hotspur’s loss,
Lend to this weight such lightness with their fear
That arrows fled not swifter toward their aim
Than did our soldiers, aiming at their safety,
Fly from the field. Then was that noble Worcester
Too soon ta’en prisoner; and that furious Scot,
The bloody Douglas, whose well-labouring sword
Had three times slain th’ appearance of the King,
Gan vail his stomach and did grace the shame
Of those that turn’d their backs, and in his flight,
Stumbling in fear, was took. The sum of all
Is that the King hath won, and hath sent out
A speedy power to encounter you, my lord,
Under the conduct of young Lancaster
And Westmoreland. This is the news at full.

NORTHUMBERLAND.
For this I shall have time enough to mourn.
In poison there is physic; and these news,
Having been well, that would have made me sick,
Being sick, have in some measure made me well.
And as the wretch, whose fever-weaken’d joints,
Like strengthless hinges, buckle under life,
Impatient of his fit, breaks like a fire
Out of his keeper’s arms, even so my limbs,
Weaken’d with grief, being now enraged with grief,
Are thrice themselves. Hence, therefore, thou nice crutch!
A scaly gauntlet now with joints of steel
Must glove this hand. And hence, thou sickly coif!
Thou art a guard too wanton for the head
Which princes, flesh’d with conquest, aim to hit.
Now bind my brows with iron, and approach
The ragged’st hour that time and spite dare bring
To frown upon th’ enraged Northumberland!
Let heaven kiss earth! Now let not Nature’s hand
Keep the wild flood confined! Let order die!
And let this world no longer be a stage
To feed contention in a lingering act;
But let one spirit of the first-born Cain
Reign in all bosoms, that, each heart being set
On bloody courses, the rude scene may end,
And darkness be the burier of the dead!

LORD BARDOLPH.
This strained passion doth you wrong, my lord.

MORTON.
Sweet earl, divorce not wisdom from your honour.
The lives of all your loving complices
Lean on your health; the which, if you give o’er
To stormy passion, must perforce decay.
You cast th’ event of war, my noble lord,
And summ’d the account of chance, before you said
“Let us make head.” It was your presurmise
That in the dole of blows your son might drop.
You knew he walk’d o’er perils, on an edge,
More likely to fall in than to get o’er.
You were advised his flesh was capable
Of wounds and scars, and that his forward spirit
Would lift him where most trade of danger ranged.
Yet did you say “Go forth;” and none of this,
Though strongly apprehended, could restrain
The stiff-borne action. What hath then befall’n,
Or what hath this bold enterprise brought forth,
More than that being which was like to be?

LORD BARDOLPH.
We all that are engaged to this loss
Knew that we ventured on such dangerous seas
That if we wrought out life ’twas ten to one;
And yet we ventured, for the gain proposed
Choked the respect of likely peril fear’d;
And since we are o’erset, venture again.
Come, we will put forth, body and goods.

MORTON.
’Tis more than time. And, my most noble lord,
I hear for certain, and dare speak the truth:
The gentle Archbishop of York is up
With well-appointed powers. He is a man
Who with a double surety binds his followers.
My lord your son had only but the corpse,
But shadows and the shows of men, to fight;
For that same word, “rebellion” did divide
The action of their bodies from their souls,
And they did fight with queasiness, constrain’d,
As men drink potions, that their weapons only
Seem’d on our side; but, for their spirits and souls,
This word, “rebellion,” it had froze them up,
As fish are in a pond. But now the bishop
Turns insurrection to religion.
Supposed sincere and holy in his thoughts,
He’s follow’d both with body and with mind,
And doth enlarge his rising with the blood
Of fair King Richard, scraped from Pomfret stones;
Derives from heaven his quarrel and his cause;
Tells them he doth bestride a bleeding land,
Gasping for life under great Bolingbroke;
And more and less do flock to follow him.

NORTHUMBERLAND.
I knew of this before; but, to speak truth,
This present grief had wiped it from my mind.
Go in with me, and counsel every man
The aptest way for safety and revenge.
Get posts and letters, and make friends with speed.
Never so few, and never yet more need.

 [_Exeunt._]

SCENE II. London. A street.

Enter Falstaff, with his Page bearing his sword and buckler.

FALSTAFF.
Sirrah, you giant, what says the doctor to my water?

PAGE.
He said, sir, the water itself was a good healthy water; but, for the
party that owed it, he might have moe diseases than he knew for.

FALSTAFF.
Men of all sorts take a pride to gird at me. The brain of this
foolish-compounded clay, man, is not able to invent anything that tends
to laughter more than I invent, or is invented on me. I am not only
witty in myself, but the cause that wit is in other men. I do here walk
before thee like a sow that hath overwhelmed all her litter but one. If
the Prince put thee into my service for any other reason than to set me
off, why then I have no judgement. Thou whoreson mandrake, thou art
fitter to be worn in my cap than to wait at my heels. I was never
manned with an agate till now, but I will inset you neither in gold nor
silver, but in vile apparel, and send you back again to your master,
for a jewel,—the juvenal, the Prince your master, whose chin is not yet
fledge. I will sooner have a beard grow in the palm of my hand than he
shall get one off his cheek; and yet he will not stick to say his face
is a face-royal. God may finish it when He will, ’tis not a hair amiss
yet. He may keep it still at a face-royal, for a barber shall never
earn sixpence out of it. And yet he’ll be crowing as if he had writ man
ever since his father was a bachelor. He may keep his own grace, but
he’s almost out of mine, I can assure him. What said Master Dommelton
about the satin for my short cloak and my slops?

PAGE.
He said, sir, you should procure him better assurance than Bardolph. He
would not take his band and yours, he liked not the security.

FALSTAFF.
Let him be damned like the glutton! Pray God his tongue be hotter! A
whoreson Achitophel! A rascally yea-forsooth knave, to bear a gentleman
in hand, and then stand upon security! The whoreson smooth-pates do now
wear nothing but high shoes and bunches of keys at their girdles; and
if a man is through with them in honest taking up, then they must stand
upon security. I had as lief they would put ratsbane in my mouth as
offer to stop it with security. I looked he should have sent me two and
twenty yards of satin, as I am a true knight, and he sends me
“security”. Well, he may sleep in security, for he hath the horn of
abundance, and the lightness of his wife shines through it; and yet
cannot he see, though he have his own lanthorn to light him. Where’s
Bardolph?

PAGE.
He’s gone into Smithfield to buy your worship a horse.

FALSTAFF.
I bought him in Paul’s, and he’ll buy me a horse in Smithfield. An I
could get me but a wife in the stews, I were manned, horsed, and wived.

Enter the Lord Chief Justice and Servant.

PAGE.
Sir, here comes the nobleman that committed the Prince for striking him
about Bardolph.

FALSTAFF.
Wait close, I will not see him.

CHIEF JUSTICE.
What’s he that goes there?

SERVANT.
Falstaff, an ’t please your lordship.

CHIEF JUSTICE.
He that was in question for the robbery?

SERVANT.
He, my lord; but he hath since done good service at Shrewsbury, and, as
I hear, is now going with some charge to the Lord John of Lancaster.

CHIEF JUSTICE.
What, to York? Call him back again.

SERVANT.
Sir John Falstaff!

FALSTAFF.
Boy, tell him I am deaf.

PAGE.
You must speak louder, my master is deaf.

CHIEF JUSTICE.
I am sure he is, to the hearing of anything good.
Go pluck him by the elbow, I must speak with him.

SERVANT.
Sir John!

FALSTAFF.
What! A young knave, and begging! Is there not wars? Is there not
employment? Doth not the King lack subjects? Do not the rebels need
soldiers? Though it be a shame to be on any side but one, it is worse
shame to beg than to be on the worst side, were it worse than the name
of rebellion can tell how to make it.

SERVANT.
You mistake me, sir.

FALSTAFF.
Why, sir, did I say you were an honest man? Setting my knighthood and
my soldiership aside, I had lied in my throat if I had said so.

SERVANT.
I pray you, sir, then set your knighthood and your soldiership aside,
and give me leave to tell you, you lie in your throat, if you say I am
any other than an honest man.

FALSTAFF.
I give thee leave to tell me so? I lay aside that which grows to me? If
thou get’st any leave of me, hang me; if thou tak’st leave, thou wert
better be hanged. You hunt counter. Hence! Avaunt!

SERVANT.
Sir, my lord would speak with you.

CHIEF JUSTICE.
Sir John Falstaff, a word with you.

FALSTAFF.
My good lord! God give your lordship good time of day. I am glad to see
your lordship abroad. I heard say your lordship was sick. I hope your
lordship goes abroad by advice. Your lordship, though not clean past
your youth, hath yet some smack of age in you, some relish of the
saltness of time; and I most humbly beseech your lordship to have a
reverend care of your health.

CHIEF JUSTICE.
Sir John, I sent for you before your expedition to Shrewsbury.

FALSTAFF.
An ’t please your lordship, I hear his Majesty is returned with some
discomfort from Wales.

CHIEF JUSTICE.
I talk not of his Majesty. You would not come when I sent for you.

FALSTAFF.
And I hear, moreover, his Highness is fallen into this same whoreson
apoplexy.

CHIEF JUSTICE.
Well, God mend him! I pray you let me speak with you.

FALSTAFF.
This apoplexy, as I take it, is a kind of lethargy, an ’t please your
lordship, a kind of sleeping in the blood, a whoreson tingling.

CHIEF JUSTICE.
What tell you me of it? Be it as it is.

FALSTAFF.
It hath it original from much grief, from study and perturbation of the
brain. I have read the cause of his effects in Galen. It is a kind of
deafness.

CHIEF JUSTICE.
I think you are fallen into the disease, for you hear not what I say to
you.

FALSTAFF.
Very well, my lord, very well. Rather, an ’t please you, it is the
disease of not listening, the malady of not marking, that I am troubled
withal.

CHIEF JUSTICE.
To punish you by the heels would amend the attention of your ears, and
I care not if I do become your physician.

FALSTAFF.
I am as poor as Job, my lord, but not so patient. Your lordship may
minister the potion of imprisonment to me in respect of poverty; but
how I should be your patient to follow your prescriptions, the wise may
make some dram of a scruple, or indeed a scruple itself.

CHIEF JUSTICE.
I sent for you, when there were matters against you for your life, to
come speak with me.

FALSTAFF.
As I was then advised by my learned counsel in the laws of this
land-service, I did not come.

CHIEF JUSTICE.
Well, the truth is, Sir John, you live in great infamy.

FALSTAFF.
He that buckles himself in my belt cannot live in less.

CHIEF JUSTICE.
Your means are very slender, and your waste is great.

FALSTAFF.
I would it were otherwise, I would my means were greater and my waist
slenderer.

CHIEF JUSTICE.
You have misled the youthful prince.

FALSTAFF.
The young prince hath misled me. I am the fellow with the great belly,
and he my dog.

CHIEF JUSTICE.
Well, I am loath to gall a new-healed wound. Your day’s service at
Shrewsbury hath a little gilded over your night’s exploit on Gad’s
Hill. You may thank th’ unquiet time for your quiet o’er-posting that
action.

FALSTAFF.
My lord!

CHIEF JUSTICE.
But since all is well, keep it so: wake not a sleeping wolf.

FALSTAFF.
To wake a wolf is as bad as smell a fox.

CHIEF JUSTICE.
What! You are as a candle, the better part burnt out.

FALSTAFF.
A wassail candle, my lord, all tallow. If I did say of wax, my growth
would approve the truth.

CHIEF JUSTICE.
There is not a white hair in your face but should have his effect of
gravity.

FALSTAFF.
His effect of gravy, gravy, gravy.

CHIEF JUSTICE.
You follow the young prince up and down, like his ill angel.

FALSTAFF.
Not so, my lord, your ill angel is light, but I hope he that looks upon
me will take me without weighing. And yet in some respects, I grant, I
cannot go. I cannot tell. Virtue is of so little regard in these
costermongers’ times that true valour is turned bearherd; pregnancy is
made a tapster, and hath his quick wit wasted in giving reckonings. All
the other gifts appertinent to man, as the malice of this age shapes
them, are not worth a gooseberry. You that are old consider not the
capacities of us that are young; you do measure the heat of our livers
with the bitterness of your galls, and we that are in the vaward of our
youth, I must confess, are wags too.

CHIEF JUSTICE.
Do you set down your name in the scroll of youth, that are written down
old with all the characters of age? Have you not a moist eye, a dry
hand, a yellow cheek, a white beard, a decreasing leg, an increasing
belly? Is not your voice broken, your wind short, your chin double,
your wit single, and every part about you blasted with antiquity? And
will you yet call yourself young? Fie, fie, fie, Sir John!

FALSTAFF.
My lord, I was born about three of the clock in the afternoon, with a
white head and something a round belly. For my voice, I have lost it
with halloing and singing of anthems. To approve my youth further, I
will not. The truth is, I am only old in judgement and understanding;
and he that will caper with me for a thousand marks, let him lend me
the money, and have at him! For the box of the ear that the Prince gave
you, he gave it like a rude prince, and you took it like a sensible
lord. I have checked him for it, and the young lion repents. Marry, not
in ashes and sackcloth, but in new silk and old sack.

CHIEF JUSTICE.
Well, God send the Prince a better companion!

FALSTAFF.
God send the companion a better prince! I cannot rid my hands of him.

CHIEF JUSTICE.
Well, the King hath severed you and Prince Harry. I hear you are going
with Lord John of Lancaster against the Archbishop and the Earl of
Northumberland.

FALSTAFF.
Yea, I thank your pretty sweet wit for it. But look you pray, all you
that kiss my lady Peace at home, that our armies join not in a hot day;
for, by the Lord, I take but two shirts out with me, and I mean not to
sweat extraordinarily. If it be a hot day, and I brandish anything but
a bottle, I would I might never spit white again. There is not a
dangerous action can peep out his head but I am thrust upon it. Well, I
cannot last ever. But it was alway yet the trick of our English nation,
if they have a good thing, to make it too common. If ye will needs say
I am an old man, you should give me rest. I would to God my name were
not so terrible to the enemy as it is. I were better to be eaten to
death with a rust than to be scoured to nothing with perpetual motion.

CHIEF JUSTICE.
Well, be honest, be honest, and God bless your expedition!

FALSTAFF.
Will your lordship lend me a thousand pound to furnish me forth?

CHIEF JUSTICE.
Not a penny, not a penny; you are too impatient to bear crosses. Fare
you well: commend me to my cousin Westmoreland.

 [_Exeunt Chief Justice and Servant._]

FALSTAFF.
If I do, fillip me with a three-man beetle. A man can no more separate
age and covetousness than he can part young limbs and lechery: but the
gout galls the one, and the pox pinches the other; and so both the
degrees prevent my curses. Boy!

PAGE.
Sir?

FALSTAFF.
What money is in my purse?

PAGE.
Seven groats and two pence.

FALSTAFF.
I can get no remedy against this consumption of the purse. Borrowing
only lingers and lingers it out, but the disease is incurable. Go bear
this letter to my Lord of Lancaster; this to the Prince; this to the
Earl of Westmoreland; and this to old Mistress Ursula, whom I have
weekly sworn to marry since I perceived the first white hair of my
chin. About it. You know where to find me. [_Exit Page_.] A pox of this
gout! or a gout of this pox! for the one or the other plays the rogue
with my great toe. ’Tis no matter if I do halt; I have the wars for my
colour, and my pension shall seem the more reasonable. A good wit will
make use of anything. I will turn diseases to commodity.

 [_Exit._]

SCENE III. York. The Archbishop’s palace.

Enter the Archbishop, the Lords Hastings, Mowbray and Bardolph.

ARCHBISHOP.
Thus have you heard our cause and known our means,
And, my most noble friends, I pray you all
Speak plainly your opinions of our hopes.
And first, Lord Marshal, what say you to it?

MOWBRAY.
I well allow the occasion of our arms,
But gladly would be better satisfied
How in our means we should advance ourselves
To look with forehead bold and big enough
Upon the power and puissance of the King.

HASTINGS.
Our present musters grow upon the file
To five and twenty thousand men of choice;
And our supplies live largely in the hope
Of great Northumberland, whose bosom burns
With an incensed fire of injuries.

LORD BARDOLPH.
The question then, Lord Hastings, standeth thus:
Whether our present five and twenty thousand
May hold up head without Northumberland.

HASTINGS.
With him we may.

LORD BARDOLPH.
Yea, marry, there’s the point:
But if without him we be thought too feeble,
My judgement is, we should not step too far
Till we had his assistance by the hand;
For in a theme so bloody-faced as this
Conjecture, expectation, and surmise
Of aids incertain should not be admitted.

ARCHBISHOP.
’Tis very true, Lord Bardolph, for indeed
It was young Hotspur’s case at Shrewsbury.

LORD BARDOLPH.
It was, my lord; who lined himself with hope,
Eating the air on promise of supply,
Flatt’ring himself in project of a power
Much smaller than the smallest of his thoughts,
And so, with great imagination
Proper to madmen, led his powers to death
And winking leap’d into destruction.

HASTINGS.
But, by your leave, it never yet did hurt
To lay down likelihoods and forms of hope.

LORD BARDOLPH.
Yes, if this present quality of war—
Indeed the instant action, a cause on foot—
Lives so in hope, as in an early spring
We see th’ appearing buds; which to prove fruit
Hope gives not so much warrant as despair
That frosts will bite them. When we mean to build,
We first survey the plot, then draw the model,
And when we see the figure of the house,
Then we must rate the cost of the erection,
Which if we find outweighs ability,
What do we then but draw anew the model
In fewer offices, or at least desist
To build at all? Much more, in this great work,
Which is almost to pluck a kingdom down
And set another up, should we survey
The plot of situation and the model,
Consent upon a sure foundation,
Question surveyors, know our own estate,
How able such a work to undergo,
To weigh against his opposite; or else
We fortify in paper and in figures,
Using the names of men instead of men,
Like one that draws the model of a house
Beyond his power to build it, who, half through,
Gives o’er and leaves his part-created cost
A naked subject to the weeping clouds
And waste for churlish winter’s tyranny.

HASTINGS.
Grant that our hopes, yet likely of fair birth,
Should be still-born, and that we now possess’d
The utmost man of expectation,
I think we are a body strong enough,
Even as we are, to equal with the King.

LORD BARDOLPH.
What, is the King but five and twenty thousand?

HASTINGS.
To us no more; nay, not so much, Lord Bardolph;
For his divisions, as the times do brawl,
Are in three heads: one power against the French,
And one against Glendower; perforce a third
Must take up us. So is the unfirm king
In three divided, and his coffers sound
With hollow poverty and emptiness.

ARCHBISHOP.
That he should draw his several strengths together
And come against us in full puissance
Need not be dreaded.

HASTINGS.
If he should do so,
He leaves his back unarm’d, the French and Welsh
Baying him at the heels: never fear that.

LORD BARDOLPH.
Who is it like should lead his forces hither?

HASTINGS.
The Duke of Lancaster and Westmoreland;
Against the Welsh, himself and Harry Monmouth;
But who is substituted ’gainst the French
I have no certain notice.

ARCHBISHOP.
Let us on,
And publish the occasion of our arms.
The commonwealth is sick of their own choice;
Their over-greedy love hath surfeited.
An habitation giddy and unsure
Hath he that buildeth on the vulgar heart.
O thou fond many, with what loud applause
Didst thou beat heaven with blessing Bolingbroke,
Before he was what thou wouldst have him be!
And being now trimm’d in thine own desires,
Thou, beastly feeder, art so full of him
That thou provok’st thyself to cast him up.
So, so, thou common dog, didst thou disgorge
Thy glutton bosom of the royal Richard;
And now thou wouldst eat thy dead vomit up,
And howl’st to find it. What trust is in these times?
They that, when Richard lived, would have him die
Are now become enamour’d on his grave.
Thou that threw’st dust upon his goodly head
When through proud London he came sighing on
After th’ admired heels of Bolingbroke,
Criest now “O earth, yield us that king again,
And take thou this!” O thoughts of men accursed!
Past and to come seems best; things present, worst.

MOWBRAY.
Shall we go draw our numbers, and set on?

HASTINGS.
We are time’s subjects, and time bids be gone.

 [_Exeunt._]




ACT II

SCENE I. London. A street.

Enter Hostess with two Officers, Fang and Snare, following.

HOSTESS.
Master Fang, have you entered the action?

FANG.
It is entered.

HOSTESS.
Where’s your yeoman? Is ’t a lusty yeoman? Will he stand to ’t?

FANG.
Sirrah, where’s Snare?

HOSTESS.
O Lord, ay! Good Master Snare.

SNARE.
Here, here.

FANG.
Snare, we must arrest Sir John Falstaff.

HOSTESS.
Yea, good Master Snare, I have entered him and all.

SNARE.
It may chance cost some of our lives, for he will stab.

HOSTESS.
Alas the day, take heed of him. He stabbed me in mine own house, and
that most beastly, in good faith. He cares not what mischief he does,
if his weapon be out, he will foin like any devil. He will spare
neither man, woman, nor child.

FANG.
If I can close with him, I care not for his thrust.

HOSTESS.
No, nor I neither. I’ll be at your elbow.

FANG.
An I but fist him once, an he come but within my vice,—

HOSTESS.
I am undone by his going, I warrant you, he’s an infinitive thing upon
my score. Good Master Fang, hold him sure. Good Master Snare, let him
not ’scape. He comes continuantly to Pie Corner—saving your manhoods—to
buy a saddle, and he is indited to dinner to the Lubber’s Head in
Lumbert Street, to Master Smooth’s the silkman. I pray you, since my
exion is entered, and my case so openly known to the world, let him be
brought in to his answer. A hundred mark is a long one for a poor lone
woman to bear, and I have borne, and borne, and borne, and have been
fubbed off, and fubbed off, and fubbed off, from this day to that day,
that it is a shame to be thought on. There is no honesty in such
dealing, unless a woman should be made an ass and a beast, to bear
every knave’s wrong. Yonder he comes, and that arrant malmsey-nose
knave, Bardolph, with him. Do your offices, do your offices, Master
Fang and Master Snare, do me, do me, do me your offices.

Enter Falstaff, Bardolph and Page.

FALSTAFF.
How now, whose mare’s dead? What’s the matter?

FANG.
Sir John, I arrest you at the suit of Mistress Quickly.

FALSTAFF.
Away, varlets! Draw, Bardolph! Cut me off the villain’s head. Throw the
quean in the channel.

HOSTESS.
Throw me in the channel? I’ll throw thee in the channel. Wilt thou,
wilt thou, thou bastardly rogue? Murder, murder! Ah, thou honeysuckle
villain, wilt thou kill God’s officers and the King’s? Ah, thou
honeyseed rogue, thou art a honeyseed, a man-queller, and a
woman-queller.

FALSTAFF.
Keep them off, Bardolph.

FANG.
A rescue! A rescue!

HOSTESS.
Good people, bring a rescue or two. Thou wo’t, wo’t thou? Thou wo’t,
wo’t ta? Do, do, thou rogue! Do, thou hempseed!

PAGE.
Away, you scullion! you rampallian! you fustilarian! I’ll tickle your
catastrophe.

Enter the Lord Chief Justice and his men.

CHIEF JUSTICE.
What is the matter? Keep the peace here, ho!

HOSTESS.
Good my lord, be good to me. I beseech you stand to me.

CHIEF JUSTICE.
How now, Sir John? What are you brawling here?
Doth this become your place, your time and business?
You should have been well on your way to York.
Stand from him, fellow. Wherefore hang’st thou upon him?

HOSTESS.
O my most worshipful lord, an’t please your Grace, I am a poor widow of
Eastcheap, and he is arrested at my suit.

CHIEF JUSTICE.
For what sum?

HOSTESS.
It is more than for some, my lord; it is for all, all I have. He hath
eaten me out of house and home. He hath put all my substance into that
fat belly of his: but I will have some of it out again, or I will ride
thee o’ nights like the mare.

FALSTAFF.
I think I am as like to ride the mare if I have any vantage of ground
to get up.

CHIEF JUSTICE.
How comes this, Sir John? Fie! what man of good temper would endure
this tempest of exclamation? Are you not ashamed to enforce a poor
widow to so rough a course to come by her own?

FALSTAFF.
What is the gross sum that I owe thee?

HOSTESS.
Marry, if thou wert an honest man, thyself and the money too. Thou
didst swear to me upon a parcel-gilt goblet, sitting in my Dolphin
chamber, at the round table, by a sea-coal fire, upon Wednesday in
Wheeson week, when the Prince broke thy head for liking his father to a
singing-man of Windsor, thou didst swear to me then, as I was washing
thy wound, to marry me and make me my lady thy wife. Canst thou deny
it? Did not goodwife Keech, the butcher’s wife, come in then and call
me gossip Quickly? coming in to borrow a mess of vinegar, telling us
she had a good dish of prawns, whereby thou didst desire to eat some,
whereby I told thee they were ill for green wound? And didst thou not,
when she was gone downstairs, desire me to be no more so familiarity
with such poor people, saying that ere long they should call me madam?
And didst thou not kiss me, and bid me fetch thee thirty shillings? I
put thee now to thy book-oath. Deny it, if thou canst.

FALSTAFF.
My lord, this is a poor mad soul, and she says up and down the town
that her eldest son is like you. She hath been in good case, and the
truth is, poverty hath distracted her. But for these foolish officers,
I beseech you I may have redress against them.

CHIEF JUSTICE.
Sir John, Sir John, I am well acquainted with your manner of wrenching
the true cause the false way. It is not a confident brow, nor the
throng of words that come with such more than impudent sauciness from
you, can thrust me from a level consideration. You have, as it appears
to me, practised upon the easy-yielding spirit of this woman, and made
her serve your uses both in purse and in person.

HOSTESS.
Yea, in truth, my lord.

CHIEF JUSTICE.
Pray thee, peace. Pay her the debt you owe her, and unpay the villany
you have done with her. The one you may do with sterling money, and the
other with current repentance.

FALSTAFF.
My lord, I will not undergo this sneap without reply. You call
honourable boldness impudent sauciness; if a man will make curtsy and
say nothing, he is virtuous. No, my lord, my humble duty remembered, I
will not be your suitor. I say to you, I do desire deliverance from
these officers, being upon hasty employment in the King’s affairs.

CHIEF JUSTICE.
You speak as having power to do wrong; but answer in th’ effect of your
reputation, and satisfy the poor woman.

FALSTAFF.
Come hither, hostess.

Enter Gower.

CHIEF JUSTICE.
Now, Master Gower, what news?

GOWER.
The King, my lord, and Harry Prince of Wales
Are near at hand: the rest the paper tells.

FALSTAFF.
As I am a gentleman.

HOSTESS.
Faith, you said so before.

FALSTAFF.
As I am a gentleman. Come, no more words of it.

HOSTESS.
By this heavenly ground I tread on, I must be fain to pawn both my
plate and the tapestry of my dining-chambers.

FALSTAFF.
Glasses, glasses, is the only drinking. And for thy walls, a pretty
slight drollery, or the story of the Prodigal, or the German hunting in
waterwork, is worth a thousand of these bed-hangers and these
fly-bitten tapestries. Let it be ten pound, if thou canst. Come, an
’twere not for thy humours, there’s not a better wench in England. Go,
wash thy face, and draw the action. Come, thou must not be in this
humour with me; dost not know me? Come, come, I know thou wast set on
to this.

HOSTESS.
Pray thee, Sir John, let it be but twenty nobles. I’ faith, I am loath
to pawn my plate, so God save me, la!

FALSTAFF.
Let it alone, I’ll make other shift: you’ll be a fool still.

HOSTESS.
Well, you shall have it, though I pawn my gown. I hope you’ll come to
supper. You’ll pay me all together?

FALSTAFF.
Will I live? [_To Bardolph_.] Go, with her, with her. Hook on, hook on.

HOSTESS.
Will you have Doll Tearsheet meet you at supper?

FALSTAFF.
No more words, let’s have her.

 [_Exeunt Hostess, Fang, Snare, Bardolph and Page._]

CHIEF JUSTICE.
I have heard better news.

FALSTAFF.
What’s the news, my lord?

CHIEF JUSTICE.
Where lay the King tonight?

GOWER.
At Basingstoke, my lord.

FALSTAFF.
I hope, my lord, all’s well. What is the news, my lord?

CHIEF JUSTICE.
Come all his forces back?

GOWER.
No, fifteen hundred foot, five hundred horse
Are march’d up to my Lord of Lancaster,
Against Northumberland and the Archbishop.

FALSTAFF.
Comes the King back from Wales, my noble lord?

CHIEF JUSTICE.
You shall have letters of me presently.
Come, go along with me, good Master Gower.

FALSTAFF.
My lord!

CHIEF JUSTICE.
What’s the matter?

FALSTAFF.
Master Gower, shall I entreat you with me to dinner?

GOWER.
I must wait upon my good lord here, I thank you, good Sir John.

CHIEF JUSTICE.
Sir John, you loiter here too long, being you are to take soldiers up
in counties as you go.

FALSTAFF.
Will you sup with me, Master Gower?

CHIEF JUSTICE.
What foolish master taught you these manners, Sir John?

FALSTAFF.
Master Gower, if they become me not, he was a fool that taught them me.
This is the right fencing grace, my lord; tap for tap, and so part
fair.

CHIEF JUSTICE.
Now the Lord lighten thee, thou art a great fool.

 [_Exeunt._]

SCENE II. London. Another street.

Enter Prince Henry and Poins.

PRINCE.
Before God, I am exceeding weary.

POINS.
Is ’t come to that? I had thought weariness durst not have attached one
of so high blood.

PRINCE.
Faith, it does me, though it discolours the complexion of my greatness
to acknowledge it. Doth it not show vilely in me to desire small beer?

POINS.
Why, a prince should not be so loosely studied as to remember so weak a
composition.

PRINCE.
Belike then my appetite was not princely got, for, by my troth, I do
now remember the poor creature small beer. But indeed, these humble
considerations make me out of love with my greatness. What a disgrace
is it to me to remember thy name! or to know thy face tomorrow! or to
take note how many pair of silk stockings thou hast—viz. these, and
those that were thy peach-coloured ones! or to bear the inventory of
thy shirts, as, one for superfluity, and another for use! But that the
tennis-court keeper knows better than I, for it is a low ebb of linen
with thee when thou keepest not racket there; as thou hast not done a
great while, because the rest of thy low countries have made a shift to
eat up thy holland. And God knows whether those that bawl out of the
ruins of thy linen shall inherit his kingdom: but the midwives say the
children are not in the fault; whereupon the world increases, and
kindreds are mightily strengthened.

POINS.
How ill it follows, after you have laboured so hard, you should talk so
idly! Tell me, how many good young princes would do so, their fathers
being so sick as yours at this time is?

PRINCE.
Shall I tell thee one thing, Poins?

POINS.
Yes, faith, and let it be an excellent good thing.

PRINCE.
It shall serve among wits of no higher breeding than thine.

POINS.
Go to, I stand the push of your one thing that you will tell.

PRINCE.
Marry, I tell thee it is not meet that I should be sad, now my father
is sick; albeit I could tell to thee, as to one it pleases me, for
fault of a better, to call my friend, I could be sad, and sad indeed
too.

POINS.
Very hardly upon such a subject.

PRINCE.
By this hand, thou thinkest me as far in the devil’s book as thou and
Falstaff for obduracy and persistency. Let the end try the man. But I
tell thee, my heart bleeds inwardly that my father is so sick; and
keeping such vile company as thou art hath in reason taken from me all
ostentation of sorrow.

POINS.
The reason?

PRINCE.
What wouldst thou think of me if I should weep?

POINS.
I would think thee a most princely hypocrite.

PRINCE.
It would be every man’s thought; and thou art a blessed fellow to think
as every man thinks. Never a man’s thought in the world keeps the
roadway better than thine: every man would think me an hypocrite
indeed. And what accites your most worshipful thought to think so?

POINS.
Why, because you have been so lewd and so much engraffed to Falstaff.

PRINCE.
And to thee.

POINS.
By this light, I am well spoke on; I can hear it with mine own ears.
The worst that they can say of me is that I am a second brother, and
that I am a proper fellow of my hands; and those two things, I confess,
I cannot help. By the mass, here comes Bardolph.

Enter Bardolph and Page.

PRINCE.
And the boy that I gave Falstaff. He had him from me Christian, and
look if the fat villain have not transformed him ape.

BARDOLPH.
God save your Grace!

PRINCE.
And yours, most noble Bardolph!

POINS.
Come, you virtuous ass, you bashful fool, must you be blushing?
Wherefore blush you now? What a maidenly man-at-arms are you become! Is
’t such a matter to get a pottle-pot’s maidenhead?

PAGE.
He calls me e’en now, my lord, through a red lattice, and I could
discern no part of his face from the window. At last I spied his eyes,
and methought he had made two holes in the ale-wife’s new petticoat and
so peeped through.

PRINCE.
Has not the boy profited?

BARDOLPH.
Away, you whoreson upright rabbit, away!

PAGE.
Away, you rascally Althaea’s dream, away!

PRINCE.
Instruct us, boy; what dream, boy?

PAGE.
Marry, my lord, Althaea dreamt she was delivered of a firebrand; and
therefore I call him her dream.

PRINCE.
A crown’s worth of good interpretation. There ’tis, boy.

POINS.
O, that this blossom could be kept from cankers! Well, there is
sixpence to preserve thee.

BARDOLPH.
An you do not make him be hanged among you, the gallows shall have
wrong.

PRINCE.
And how doth thy master, Bardolph?

BARDOLPH.
Well, my lord. He heard of your Grace’s coming to town. There’s a
letter for you.

POINS.
Delivered with good respect. And how doth the martlemas, your master?

BARDOLPH.
In bodily health, sir.

POINS.
Marry, the immortal part needs a physician, but that moves not him.
Though that be sick, it dies not.

PRINCE.
I do allow this wen to be as familiar with me as my dog, and he holds
his place, for look you how he writes.

POINS.
[_Reads_.] “John Falstaff, knight,” Every man must know that, as oft as
he has occasion to name himself: even like those that are kin to the
King, for they never prick their finger but they say, “There’s some of
the King’s blood spilt.” “How comes that?” says he that takes upon him
not to conceive. The answer is as ready as a borrower’s cap, “I am the
King’s poor cousin, sir.”

PRINCE.
Nay, they will be kin to us, or they will fetch it from Japhet. But to
the letter: “Sir John Falstaff, knight, to the son of the King, nearest
his father, Harry Prince of Wales, greeting.”

POINS.
Why, this is a certificate.

PRINCE.
Peace! “I will imitate the honourable Romans in brevity.”

POINS.
He sure means brevity in breath, short-winded.

PRINCE.
“I commend me to thee, I commend thee, and I leave thee. Be not too
familiar with Poins, for he misuses thy favours so much that he swears
thou art to marry his sister Nell. Repent at idle times as thou mayst,
and so, farewell.
Thine by yea and no, which is as much as to say, as thou usest him—Jack
Falstaff with my familiars, John with my brothers and sisters, and Sir
John with all Europe.”

POINS.
My lord, I’ll steep this letter in sack and make him eat it.

PRINCE.
That’s to make him eat twenty of his words. But do you use me thus,
Ned? Must I marry your sister?

POINS.
God send the wench no worse fortune! But I never said so.

PRINCE.
Well, thus we play the fools with the time, and the spirits of the wise
sit in the clouds and mock us. Is your master here in London?

BARDOLPH.
Yea, my lord.

PRINCE.
Where sups he? Doth the old boar feed in the old frank?

BARDOLPH.
At the old place, my lord, in Eastcheap.

PRINCE.
What company?

PAGE.
Ephesians, my lord, of the old church.

PRINCE.
Sup any women with him?

PAGE.
None, my lord, but old Mistress Quickly and Mistress Doll Tearsheet.

PRINCE.
What pagan may that be?

PAGE.
A proper gentlewoman, sir, and a kinswoman of my master’s.

PRINCE.
Even such kin as the parish heifers are to the town bull. Shall we
steal upon them, Ned, at supper?

POINS.
I am your shadow, my lord, I’ll follow you.

PRINCE.
Sirrah, you boy, and Bardolph, no word to your master that I am yet
come to town. There’s for your silence.

BARDOLPH.
I have no tongue, sir.

PAGE.
And for mine, sir, I will govern it.

PRINCE.
Fare you well; go.

 [_Exeunt Bardolph and Page._]

This Doll Tearsheet should be some road.

POINS.
I warrant you, as common as the way between Saint Albans and London.

PRINCE.
How might we see Falstaff bestow himself tonight in his true colours,
and not ourselves be seen?

POINS.
Put on two leathern jerkins and aprons, and wait upon him at his table
as drawers.

PRINCE.
From a god to a bull? A heavy descension! It was Jove’s case. From a
prince to a ’prentice? A low transformation that shall be mine, for in
everything the purpose must weigh with the folly. Follow me, Ned.

 [_Exeunt._]

SCENE III. Warkworth. Before the castle.

Enter Northumberland, Lady Northumberland and Lady Percy.

NORTHUMBERLAND.
I pray thee, loving wife, and gentle daughter,
Give even way unto my rough affairs;
Put not you on the visage of the times
And be like them to Percy troublesome.

LADY NORTHUMBERLAND.
I have given over, I will speak no more.
Do what you will; your wisdom be your guide.

NORTHUMBERLAND.
Alas, sweet wife, my honour is at pawn,
And, but my going, nothing can redeem it.

LADY PERCY.
O yet, for God’s sake, go not to these wars!
The time was, father, that you broke your word,
When you were more endear’d to it than now;
When your own Percy, when my heart’s dear Harry,
Threw many a northward look to see his father
Bring up his powers; but he did long in vain.
Who then persuaded you to stay at home?
There were two honours lost, yours and your son’s.
For yours, the God of heaven brighten it!
For his, it stuck upon him as the sun
In the grey vault of heaven, and by his light
Did all the chivalry of England move
To do brave acts. He was indeed the glass
Wherein the noble youth did dress themselves.
He had no legs that practis’d not his gait;
And speaking thick, which nature made his blemish,
Became the accents of the valiant;
For those who could speak low and tardily
Would turn their own perfection to abuse,
To seem like him. So that in speech, in gait,
In diet, in affections of delight,
In military rules, humours of blood,
He was the mark and glass, copy and book,
That fashion’d others. And him—O wondrous him!
O miracle of men!—him did you leave,
Second to none, unseconded by you,
To look upon the hideous god of war
In disadvantage, to abide a field
Where nothing but the sound of Hotspur’s name
Did seem defensible: so you left him.
Never, O never, do his ghost the wrong
To hold your honour more precise and nice
With others than with him! Let them alone.
The Marshal and the Archbishop are strong:
Had my sweet Harry had but half their numbers,
Today might I, hanging on Hotspur’s neck,
Have talk’d of Monmouth’s grave.

NORTHUMBERLAND.
Beshrew your heart,
Fair daughter, you do draw my spirits from me
With new lamenting ancient oversights.
But I must go and meet with danger there,
Or it will seek me in another place,
And find me worse provided.

LADY NORTHUMBERLAND.
O, fly to Scotland,
Till that the nobles and the armed commons
Have of their puissance made a little taste.

LADY PERCY.
If they get ground and vantage of the King,
Then join you with them like a rib of steel,
To make strength stronger; but, for all our loves,
First let them try themselves. So did your son;
He was so suffer’d. So came I a widow,
And never shall have length of life enough
To rain upon remembrance with mine eyes,
That it may grow and sprout as high as heaven
For recordation to my noble husband.

NORTHUMBERLAND.
Come, come, go in with me. ’Tis with my mind
As with the tide swell’d up unto his height,
That makes a still-stand, running neither way.
Fain would I go to meet the Archbishop,
But many thousand reasons hold me back.
I will resolve for Scotland. There am I,
Till time and vantage crave my company.

 [_Exeunt._]

SCENE IV. London. The Boar’s head Tavern in Eastcheap.

Enter two Drawers.

FIRST DRAWER.
What the devil hast thou brought there—applejohns? Thou knowest Sir
John cannot endure an applejohn.

SECOND DRAWER.
Mass, thou sayest true. The Prince once set a dish of applejohns before
him, and told him there were five more Sir Johns, and, putting off his
hat, said “I will now take my leave of these six dry, round, old,
withered knights.” It angered him to the heart. But he hath forgot
that.

FIRST DRAWER.
Why then, cover, and set them down, and see if thou canst find out
Sneak’s noise. Mistress Tearsheet would fain hear some music. Dispatch.
The room where they supped is too hot, they’ll come in straight.

SECOND DRAWER.
Sirrah, here will be the Prince and Master Poins anon, and they will
put on two of our jerkins and aprons, and Sir John must not know of it.
Bardolph hath brought word.

FIRST DRAWER.
By the mass, here will be old utis. It will be an excellent stratagem.

SECOND DRAWER.
I’ll see if I can find out Sneak.

 [_Exit._]

Enter Hostess and Doll Tearsheet.

HOSTESS.
I’ faith, sweetheart, methinks now you are in an excellent good
temperality. Your pulsidge beats as extraordinarily as heart would
desire, and your colour, I warrant you, is as red as any rose, in good
truth, la! But, i’ faith, you have drunk too much canaries, and that’s
a marvellous searching wine, and it perfumes the blood ere one can say
“What’s this?” How do you now?

DOLL.
Better than I was. Hem!

HOSTESS.
Why, that’s well said. A good heart’s worth gold. Lo, here comes Sir
John.

Enter Falstaff.

FALSTAFF.
[_Singing_.] “When Arthur first in court”—Empty the jordan.
[_Exit First Drawer_.]—[_Singing_.] “And was a worthy king.”
How now, Mistress Doll!

HOSTESS.
Sick of a calm, yea, good faith.

FALSTAFF.
So is all her sect; an they be once in a calm, they are sick.

DOLL.
A pox damn you, you muddy rascal, is that all the comfort you give me?

FALSTAFF.
You make fat rascals, Mistress Doll.

DOLL.
I make them? Gluttony and diseases make them; I make them not.

FALSTAFF.
If the cook help to make the gluttony, you help to make the diseases,
Doll: we catch of you, Doll. We catch of you; grant that, my poor
virtue, grant that.

DOLL.
Yea, joy, our chains and our jewels.

FALSTAFF.
“Your brooches, pearls, and ouches:”—for to serve bravely is to come
halting off, you know; to come off the breach with his pike bent
bravely, and to surgery bravely; to venture upon the charged chambers
bravely—

DOLL.
Hang yourself, you muddy conger, hang yourself!

HOSTESS.
By my troth, this is the old fashion; you two never meet but you fall
to some discord. You are both, i’ good truth, as rheumatic as two dry
toasts. You cannot one bear with another’s confirmities. What the
good-year! One must bear, and that must be you. You are the weaker
vessel, as as they say, the emptier vessel.

DOLL.
Can a weak empty vessel bear such a huge full hogshead? There’s a whole
merchant’s venture of Bourdeaux stuff in him; you have not seen a hulk
better stuffed in the hold. Come, I’ll be friends with thee, Jack. Thou
art going to the wars, and whether I shall ever see thee again or no,
there is nobody cares.

Enter First Drawer.

FIRST DRAWER.
Sir, Ancient Pistol’s below, and would speak with you.

DOLL.
Hang him, swaggering rascal! Let him not come hither: it is the
foul-mouthed’st rogue in England.

HOSTESS.
If he swagger, let him not come here. No, by my faith, I must live
among my neighbours. I’ll no swaggerers. I am in good name and fame
with the very best. Shut the door, there comes no swaggerers here. I
have not lived all this while to have swaggering now. Shut the door, I
pray you.

FALSTAFF.
Dost thou hear, hostess?

HOSTESS.
Pray ye pacify yourself, Sir John. There comes no swaggerers here.

FALSTAFF.
Dost thou hear? It is mine ancient.

HOSTESS.
Tilly-fally, Sir John, ne’er tell me. And our ancient swaggerer comes
not in my doors. I was before Master Tisick, the debuty t’other day,
and, as he said to me,—’twas no longer ago than Wednesday last, i’ good
faith,—“Neighbour Quickly,” says he—Master Dumb, our minister, was by
then—“Neighbour Quickly,” says he, “receive those that are civil, for,”
said he “you are in an ill name.” Now he said so, I can tell whereupon.
“For,” says he, “you are an honest woman, and well thought on.
Therefore take heed what guests you receive. Receive,” says he, “no
swaggering companions.” There comes none here. You would bless you to
hear what he said. No, I’ll no swaggerers.

FALSTAFF.
He’s no swaggerer, hostess; a tame cheater, i’ faith, you may stroke
him as gently as a puppy greyhound. He’ll not swagger with a Barbary
hen, if her feathers turn back in any show of resistance. Call him up,
drawer.

 [_Exit First Drawer._]

HOSTESS.
Cheater, call you him? I will bar no honest man my house, nor no
cheater, but I do not love swaggering, by my troth, I am the worse when
one says “swagger.” Feel, masters, how I shake; look you, I warrant
you.

DOLL.
So you do, hostess.

HOSTESS.
Do I? Yea, in very truth, do I, an ’twere an aspen leaf. I cannot abide
swaggerers.

Enter Pistol, Bardolph and Page.

PISTOL.
God save you, Sir John!

FALSTAFF.
Welcome, Ancient Pistol. Here, Pistol, I charge you with a cup of sack.
Do you discharge upon mine hostess.

PISTOL.
I will discharge upon her, Sir John, with two bullets.

FALSTAFF.
She is pistol-proof, sir; you shall not hardly offend her.

HOSTESS.
Come, I’ll drink no proofs nor no bullets. I’ll drink no more than will
do me good, for no man’s pleasure, I.

PISTOL.
Then to you, Mistress Dorothy! I will charge you.

DOLL.
Charge me! I scorn you, scurvy companion. What, you poor, base,
rascally, cheating, lack-linen mate! Away, you mouldy rogue, away! I am
meat for your master.

PISTOL.
I know you, Mistress Dorothy.

DOLL.
Away, you cut-purse rascal, you filthy bung, away! By this wine, I’ll
thrust my knife in your mouldy chaps an you play the saucy cuttle with
me. Away, you bottle-ale rascal, you basket-hilt stale juggler, you!
Since when, I pray you, sir? God’s light, with two points on your
shoulder? Much!

PISTOL.
God let me not live, but I will murder your ruff for this.

FALSTAFF.
No more, Pistol! I would not have you go off here. Discharge yourself
of our company, Pistol.

HOSTESS.
No, good Captain Pistol, not here, sweet captain.

DOLL.
Captain! Thou abominable damned cheater, art thou not ashamed to be
called captain? An captains were of my mind, they would truncheon you
out, for taking their names upon you before you have earned them. You a
captain? You slave, for what? For tearing a poor whore’s ruff in a
bawdy-house? He a captain! Hang him, rogue, he lives upon mouldy stewed
prunes and dried cakes. A captain? God’s light, these villains will
make the word as odious as the word “occupy,” which was an excellent
good word before it was ill sorted. Therefore captains had need look
to’t.

BARDOLPH.
Pray thee go down, good ancient.

FALSTAFF.
Hark thee hither, Mistress Doll.

PISTOL.
Not I. I tell thee what, Corporal Bardolph, I could tear her. I’ll be
revenged of her.

PAGE.
Pray thee go down.

PISTOL.
I’ll see her damned first to Pluto’s damned lake, by this hand, to th’
infernal deep, with Erebus and tortures vile also. Hold hook and line,
say I. Down, down, dogs! Down, faitors! Have we not Hiren here?

HOSTESS.
Good Captain Peesel, be quiet, ’tis very late, i’ faith. I beseek you
now, aggravate your choler.

PISTOL.
These be good humours, indeed! Shall packhorses
And hollow pamper’d jades of Asia,
Which cannot go but thirty mile a day,
Compare with Caesars and with Cannibals,
And Trojant Greeks? Nay, rather damn them with
King Cerberus; and let the welkin roar.
Shall we fall foul for toys?

HOSTESS.
By my troth, captain, these are very bitter words.

BARDOLPH.
Be gone, good ancient. This will grow to a brawl anon.

PISTOL.
Die men like dogs! Give crowns like pins! Have we not Hiren here?

HOSTESS.
O’ my word, captain, there’s none such here. What the good-year, do you
think I would deny her? For God’s sake, be quiet.

PISTOL.
Then feed and be fat, my fair Calipolis.
Come, give ’s some sack.
_Si fortune me tormente, sperato me contento._
Fear we broadsides? No, let the fiend give fire.
Give me some sack; and, sweetheart, lie thou there.

 [_Laying down his sword._]

Come we to full points here? And are etceteras nothings?

FALSTAFF.
Pistol, I would be quiet.

PISTOL.
Sweet knight, I kiss thy neaf. What! we have seen the seven stars.

DOLL.
For God’s sake, thrust him downstairs. I cannot endure such a fustian
rascal.

PISTOL.
Thrust him downstairs? Know we not Galloway nags?

FALSTAFF.
Quoit him down, Bardolph, like a shove-groat shilling. Nay, an he do
nothing but speak nothing, he shall be nothing here.

BARDOLPH.
Come, get you downstairs.

PISTOL.
What! shall we have incision? Shall we imbrue?

 [_Snatching up his sword._]

Then death rock me asleep, abridge my doleful days!
Why then, let grievous, ghastly, gaping wounds
Untwind the Sisters Three! Come, Atropos, I say!

HOSTESS.
Here’s goodly stuff toward!

FALSTAFF.
Give me my rapier, boy.

DOLL.
I pray thee, Jack, I pray thee, do not draw.

FALSTAFF.
Get you downstairs.

 [_Drawing, and driving Pistol out._]

HOSTESS.
Here’s a goodly tumult! I’ll forswear keeping house, afore I’ll be in
these tirrits and frights. So, murder, I warrant now. Alas, alas, put
up your naked weapons, put up your naked weapons.

 [_Exeunt Bardolph and Pistol._]

DOLL.
I pray thee, Jack, be quiet. The rascal’s gone. Ah, you whoreson little
valiant villain, you!

HOSTESS.
Are you not hurt i’ th’ groin? Methought he made a shrewd thrust at
your belly.

Enter Bardolph.

FALSTAFF.
Have you turned him out o’ doors?

BARDOLPH.
Yea, sir. The rascal’s drunk. You have hurt him, sir, i’ th’ shoulder.

FALSTAFF.
A rascal, to brave me!

DOLL.
Ah, you sweet little rogue, you! Alas, poor ape, how thou sweat’st!
Come, let me wipe thy face. Come on, you whoreson chops. Ah, rogue! i’
faith, I love thee. Thou art as valorous as Hector of Troy, worth five
of Agamemnon, and ten times better than the Nine Worthies. Ah, villain!

FALSTAFF.
A rascally slave! I will toss the rogue in a blanket.

DOLL.
Do, an thou darest for thy heart. An thou dost, I’ll canvass thee
between a pair of sheets.

Enter Music.

PAGE.
The music is come, sir.

FALSTAFF.
Let them play. Play, sirs. Sit on my knee, Doll. A rascal bragging
slave! The rogue fled from me like quicksilver.

DOLL.
I’ faith, and thou followedst him like a church. Thou whoreson little
tidy Bartholomew boar-pig, when wilt thou leave fighting a-days and
foining a-nights, and begin to patch up thine old body for heaven?

Enter, behind, Prince Henry and Poins, disguised as drawers.

FALSTAFF.
Peace, good Doll, do not speak like a death’s-head; do not bid me
remember mine end.

DOLL.
Sirrah, what humour ’s the Prince of?

FALSTAFF.
A good shallow young fellow; he would have made a good pantler; he
would ha’ chipped bread well.

DOLL.
They say Poins has a good wit.

FALSTAFF.
He a good wit? Hang him, baboon! His wit’s as thick as Tewksbury
mustard; there’s no more conceit in him than is in a mallet.

DOLL.
Why does the Prince love him so, then?

FALSTAFF.
Because their legs are both of a bigness, and he plays at quoits well,
and eats conger and fennel, and drinks off candles’ ends for
flap-dragons, and rides the wild mare with the boys, and jumps upon
joint stools, and swears with a good grace, and wears his boots very
smooth like unto the sign of the Leg, and breeds no bate with telling
of discreet stories, and such other gambol faculties he has that show a
weak mind and an able body, for the which the Prince admits him: for
the Prince himself is such another. The weight of a hair will turn the
scales between their avoirdupois.

PRINCE.
Would not this nave of a wheel have his ears cut off?

POINS.
Let’s beat him before his whore.

PRINCE.
Look whe’er the withered elder hath not his poll clawed like a parrot.

POINS.
Is it not strange that desire should so many years outlive performance?

FALSTAFF.
Kiss me, Doll.

PRINCE.
Saturn and Venus this year in conjunction! What says th’ almanac to
that?

POINS.
And look whether the fiery Trigon, his man, be not lisping to his
master’s old tables, his note-book, his counsel-keeper.

FALSTAFF.
Thou dost give me flattering busses.

DOLL.
By my troth, I kiss thee with a most constant heart.

FALSTAFF.
I am old, I am old.

DOLL.
I love thee better than I love e’er a scurvy young boy of them all.

FALSTAFF.
What stuff wilt have a kirtle of? I shall receive money o’ Thursday;
shalt have a cap tomorrow. A merry song! Come, it grows late, we’ll to
bed. Thou’lt forget me when I am gone.

DOLL.
By my troth, thou’lt set me a-weeping an thou sayest so. Prove that
ever I dress myself handsome till thy return. Well, hearken a’ th’ end.

FALSTAFF.
Some sack, Francis.

PRINCE & POINS.
Anon, anon, sir.

 [_Coming forward._]

FALSTAFF.
Ha! A bastard son of the King’s? And art thou not Poins his brother?

PRINCE.
Why, thou globe of sinful continents, what a life dost thou lead!

FALSTAFF.
A better than thou. I am a gentleman, thou art a drawer.

PRINCE.
Very true, sir, and I come to draw you out by the ears.

HOSTESS.
O, the Lord preserve thy Grace! By my troth, welcome to London. Now,
the Lord bless that sweet face of thine! O Jesu, are you come from
Wales?

FALSTAFF.
Thou whoreson mad compound of majesty, by this light flesh and corrupt
blood, thou art welcome.

DOLL.
How? You fat fool, I scorn you.

POINS.
My lord, he will drive you out of your revenge and turn all to a
merriment, if you take not the heat.

PRINCE.
You whoreson candle-mine, you, how vilely did you speak of me even now
before this honest, virtuous, civil gentlewoman!

HOSTESS.
God’s blessing of your good heart! and so she is, by my troth.

FALSTAFF.
Didst thou hear me?

PRINCE.
Yea, and you knew me, as you did when you ran away by Gad’s Hill. You
knew I was at your back, and spoke it on purpose to try my patience.

FALSTAFF.
No, no, no, not so; I did not think thou wast within hearing.

PRINCE.
I shall drive you then to confess the wilful abuse, and then I know how
to handle you.

FALSTAFF.
No abuse, Hal, o’ mine honour, no abuse.

PRINCE.
Not to dispraise me, and call me pantler and bread-chipper and I know
not what?

FALSTAFF.
No abuse, Hal.

POINS.
No abuse?

FALSTAFF.
No abuse, Ned, i’ th’ world, honest Ned, none. I dispraised him before
the wicked, that the wicked might not fall in love with thee; in which
doing, I have done the part of a careful friend and a true subject, and
thy father is to give me thanks for it. No abuse, Hal; none, Ned, none;
no, faith, boys, none.

PRINCE.
See now whether pure fear and entire cowardice doth not make thee wrong
this virtuous gentlewoman to close with us. Is she of the wicked? Is
thine hostess here of the wicked? Or is thy boy of the wicked? Or
honest Bardolph, whose zeal burns in his nose, of the wicked?

POINS.
Answer, thou dead elm, answer.

FALSTAFF.
The fiend hath pricked down Bardolph irrecoverable, and his face is
Lucifer’s privy-kitchen, where he doth nothing but roast malt-worms.
For the boy, there is a good angel about him, but the devil outbids him
too.

PRINCE.
For the women?

FALSTAFF.
For one of them, she’s in hell already, and burns poor souls. For th’
other, I owe her money, and whether she be damned for that I know not.

HOSTESS.
No, I warrant you.

FALSTAFF.
No, I think thou art not, I think thou art quit for that. Marry, there
is another indictment upon thee, for suffering flesh to be eaten in thy
house, contrary to the law, for the which I think thou wilt howl.

HOSTESS.
All victuallers do so. What’s a joint of mutton or two in a whole Lent?

PRINCE.
You, gentlewoman.

DOLL.
What says your Grace?

FALSTAFF.
His grace says that which his flesh rebels against.

 [Peto _knocks at door._]

HOSTESS.
Who knocks so loud at door? Look to th’ door there, Francis.

Enter Peto.

PRINCE.
Peto, how now, what news?

PETO.
The King your father is at Westminster,
And there are twenty weak and wearied posts
Come from the north: and as I came along,
I met and overtook a dozen captains,
Bareheaded, sweating, knocking at the taverns,
And asking everyone for Sir John Falstaff.

PRINCE.
By heaven, Poins, I feel me much to blame,
So idly to profane the precious time,
When tempest of commotion, like the south
Borne with black vapour, doth begin to melt
And drop upon our bare unarmed heads.
Give me my sword and cloak. Falstaff, good night.

 [_Exeunt Prince, Poins, Peto and Bardolph._]

FALSTAFF.
Now comes in the sweetest morsel of the night, and we must hence and
leave it unpicked.
[_Knocking within_.] More knocking at the door?

Enter Bardolph.

How now, what’s the matter?

BARDOLPH.
You must away to court, sir, presently.
A dozen captains stay at door for you.

FALSTAFF.
[_To the Page_.] Pay the musicians, sirrah. Farewell, hostess;
farewell, Doll. You see, my good wenches, how men of merit are sought
after. The undeserver may sleep, when the man of action is called on.
Farewell, good wenches. If I be not sent away post, I will see you
again ere I go.

DOLL.
I cannot speak; if my heart be not ready to burst—well, sweet Jack,
have a care of thyself.

FALSTAFF.
Farewell, farewell.

 [_Exeunt Falstaff and Bardolph._]

HOSTESS.
Well, fare thee well. I have known thee these twenty-nine years, come
peascod-time; but an honester and truer-hearted man—well, fare thee
well.

BARDOLPH.
[_Within_.] Mistress Tearsheet!

HOSTESS.
What’s the matter?

BARDOLPH.
[_Within_.] Bid Mistress Tearsheet come to my master.

HOSTESS.
O, run, Doll, run; run, good Doll; come. She comes blubbered. Yea, will
you come, Doll?

 [_Exeunt._]




ACT III

SCENE I. Westminster. The palace.

Enter the King in his nightgown, with a Page.

KING.
Go call the Earls of Surrey and of Warwick;
But, ere they come, bid them o’er-read these letters
And well consider of them. Make good speed.

 [_Exit Page._]

How many thousands of my poorest subjects
Are at this hour asleep! O sleep, O gentle sleep,
Nature’s soft nurse, how have I frighted thee,
That thou no more wilt weigh my eyelids down
And steep my senses in forgetfulness?
Why rather, sleep, liest thou in smoky cribs,
Upon uneasy pallets stretching thee,
And hushed with buzzing night-flies to thy slumber,
Than in the perfumed chambers of the great,
Under the canopies of costly state,
And lull’d with sound of sweetest melody?
O thou dull god, why liest thou with the vile
In loathsome beds, and leavest the kingly couch
A watch-case or a common ’larum-bell?
Wilt thou upon the high and giddy mast
Seal up the ship-boy’s eyes, and rock his brains
In cradle of the rude imperious surge
And in the visitation of the winds,
Who take the ruffian billows by the top,
Curling their monstrous heads and hanging them
With deafing clamour in the slippery clouds,
That with the hurly death itself awakes?
Canst thou, O partial sleep, give thy repose
To the wet sea-boy in an hour so rude,
And in the calmest and most stillest night,
With all appliances and means to boot,
Deny it to a King? Then happy low, lie down!
Uneasy lies the head that wears a crown.

Enter Warwick and Surrey.

WARWICK.
Many good morrows to your Majesty!

KING.
Is it good morrow, lords?

WARWICK.
’Tis one o’clock, and past.

KING.
Why then, good morrow to you all, my lords.
Have you read o’er the letters that I sent you?

WARWICK.
We have, my liege.

KING.
Then you perceive the body of our kingdom
How foul it is, what rank diseases grow,
And with what danger, near the heart of it.

WARWICK.
It is but as a body yet distemper’d,
Which to his former strength may be restored
With good advice and little medicine.
My Lord Northumberland will soon be cool’d.

KING.
O God, that one might read the book of fate,
And see the revolution of the times
Make mountains level, and the continent,
Weary of solid firmness, melt itself
Into the sea, and other times to see
The beachy girdle of the ocean
Too wide for Neptune’s hips; how chance’s mocks
And changes fill the cup of alteration
With divers liquors! O, if this were seen,
The happiest youth, viewing his progress through,
What perils past, what crosses to ensue,
Would shut the book, and sit him down and die.
’Tis not ten years gone
Since Richard and Northumberland, great friends,
Did feast together, and in two years after
Were they at wars. It is but eight years since
This Percy was the man nearest my soul,
Who like a brother toil’d in my affairs
And laid his love and life under my foot,
Yea, for my sake, even to the eyes of Richard
Gave him defiance. But which of you was by—
[_To Warwick_.] You, cousin Nevil, as I may remember—
When Richard, with his eye brimful of tears,
Then check’d and rated by Northumberland,
Did speak these words, now proved a prophecy?
“Northumberland, thou ladder by the which
My cousin Bolingbroke ascends my throne”
Though then, God knows, I had no such intent,
But that necessity so bow’d the state
That I and greatness were compell’d to kiss—
“The time shall come,” thus did he follow it,
“The time will come, that foul sin, gathering head,
Shall break into corruption”—so went on,
Foretelling this same time’s condition
And the division of our amity.

WARWICK.
There is a history in all men’s lives
Figuring the natures of the times deceased;
The which observed, a man may prophesy,
With a near aim, of the main chance of things
As yet not come to life, who in their seeds
And weak beginning lie intreasured.
Such things become the hatch and brood of time;
And by the necessary form of this
King Richard might create a perfect guess
That great Northumberland, then false to him,
Would of that seed grow to a greater falseness,
Which should not find a ground to root upon,
Unless on you.

KING.
Are these things then necessities?
Then let us meet them like necessities;
And that same word even now cries out on us.
They say the bishop and Northumberland
Are fifty thousand strong.

WARWICK.
It cannot be, my lord.
Rumour doth double, like the voice and echo,
The numbers of the feared. Please it your Grace
To go to bed. Upon my soul, my lord,
The powers that you already have sent forth
Shall bring this prize in very easily.
To comfort you the more, I have received
A certain instance that Glendower is dead.
Your majesty hath been this fortnight ill,
And these unseason’d hours perforce must add
Unto your sickness.

KING.
I will take your counsel.
And were these inward wars once out of hand,
We would, dear lords, unto the Holy Land.

 [_Exeunt._]

SCENE II. Gloucestershire. Before Justice Shallow’s house.

Enter Shallow and Silence, meeting; Mouldy, Shadow, Wart, Feeble,
Bullcalf, a Servant or two with them.

SHALLOW.
Come on, come on, come on. Give me your hand, sir, give me your hand,
sir. An early stirrer, by the rood! And how doth my good cousin
Silence?

SILENCE.
Good morrow, good cousin Shallow.

SHALLOW.
And how doth my cousin, your bedfellow? And your fairest daughter and
mine, my god-daughter Ellen?

SILENCE.
Alas, a black ousel, cousin Shallow!

SHALLOW.
By yea and no, sir, I dare say my cousin William is become a good
scholar. He is at Oxford still, is he not?

SILENCE.
Indeed, sir, to my cost.

SHALLOW.
He must, then, to the Inns o’ Court shortly. I was once of Clement’s
Inn, where I think they will talk of mad Shallow yet.

SILENCE.
You were called “lusty Shallow” then, cousin.

SHALLOW.
By the mass, I was called anything, and I would have done anything
indeed too, and roundly too. There was I, and little John Doit of
Staffordshire, and black George Barnes, and Francis Pickbone, and Will
Squele, a Cotswold man. You had not four such swinge-bucklers in all
the Inns o’ Court again. And I may say to you, we knew where the
bona-robas were and had the best of them all at commandment. Then was
Jack Falstaff, now Sir John, a boy, and page to Thomas Mowbray, Duke of
Norfolk.

SILENCE.
This Sir John, cousin, that comes hither anon about soldiers?

SHALLOW.
The same Sir John, the very same. I see him break Scoggin’s head at the
court gate, when he was a crack not thus high; and the very same day
did I fight with one Sampson Stockfish, a fruiterer, behind Gray’s Inn.
Jesu, Jesu, the mad days that I have spent! And to see how many of my
old acquaintance are dead!

SILENCE.
We shall all follow, cousin.

SHALLOW.
Certain, ’tis certain, very sure, very sure. Death, as the Psalmist
saith, is certain to all, all shall die. How a good yoke of bullocks at
Stamford fair?

SILENCE.
By my troth, I was not there.

SHALLOW.
Death is certain. Is old Double of your town living yet?

SILENCE.
Dead, sir.

SHALLOW.
Jesu, Jesu, dead! He drew a good bow, and dead! He shot a fine shoot.
John a Gaunt loved him well, and betted much money on his head. Dead!
He would have clapped i’ th’ clout at twelve score, and carried you a
forehand shaft a fourteen and fourteen and a half, that it would have
done a man’s heart good to see. How a score of ewes now?

SILENCE.
Thereafter as they be; a score of good ewes may be worth ten pounds.

SHALLOW.
And is old Double dead?

SILENCE.
Here come two of Sir John Falstaff’s men, as I think.

Enter Bardolph and one with him.

SHALLOW.
Good morrow, honest gentlemen.

BARDOLPH.
I beseech you, which is Justice Shallow?

SHALLOW.
I am Robert Shallow, sir, a poor esquire of this county, and one of the
King’s justices of the peace. What is your good pleasure with me?

BARDOLPH.
My captain, sir, commends him to you, my captain, Sir John Falstaff, a
tall gentleman, by heaven, and a most gallant leader.

SHALLOW.
He greets me well, sir. I knew him a good backsword man. How doth the
good knight? May I ask how my lady his wife doth?

BARDOLPH.
Sir, pardon. A soldier is better accommodated than with a wife.

SHALLOW.
It is well said, in faith, sir, and it is well said indeed too. “Better
accommodated!” It is good, yea indeed, is it. Good phrases are surely,
and ever were, very commendable. “Accommodated.” It comes of
_accommodo_. Very good, a good phrase.

BARDOLPH.
Pardon, sir, I have heard the word—phrase call you it? By this day, I
know not the phrase, but I will maintain the word with my sword to be a
soldier-like word, and a word of exceeding good command, by heaven.
Accommodated, that is when a man is, as they say, accommodated, or when
a man is being whereby he may be thought to be accommodated; which is
an excellent thing.

SHALLOW.
It is very just.

Enter Falstaff.

Look, here comes good Sir John. Give me your good hand, give me your
worship’s good hand. By my troth, you like well and bear your years
very well. Welcome, good Sir John.

FALSTAFF.
I am glad to see you well, good Master Robert Shallow. Master Surecard,
as I think?

SHALLOW.
No, Sir John, it is my cousin Silence, in commission with me.

FALSTAFF.
Good Master Silence, it well befits you should be of the peace.

SILENCE.
Your good worship is welcome.

FALSTAFF.
Fie, this is hot weather, gentlemen. Have you provided me here half a
dozen sufficient men?

SHALLOW.
Marry, have we, sir. Will you sit?

FALSTAFF.
Let me see them, I beseech you.

SHALLOW.
Where’s the roll? Where’s the roll? Where’s the roll? Let me see, let
me see, let me see. So, so, so, so, so, so, so. Yea, marry, sir: Ralph
Mouldy! Let them appear as I call; let them do so, let them do so. Let
me see; where is Mouldy?

MOULDY.
Here, an it please you.

SHALLOW.
What think you, Sir John? A good-limbed fellow, young, strong, and of
good friends.

FALSTAFF.
Is thy name Mouldy?

MOULDY.
Yea, an’t please you.

FALSTAFF.
’Tis the more time thou wert used.

SHALLOW.
Ha, ha, ha! most excellent, i’ faith! Things that are mouldy lack use.
Very singular good, in faith, well said, Sir John, very well said.

FALSTAFF.
Prick him.

MOULDY.
I was pricked well enough before, an you could have let me alone. My
old dame will be undone now for one to do her husbandry and her
drudgery. You need not to have pricked me, there are other men fitter
to go out than I.

FALSTAFF.
Go to. Peace, Mouldy; you shall go. Mouldy, it is time you were spent.

MOULDY.
Spent?

SHALLOW.
Peace, fellow, peace. Stand aside. Know you where you are? For
th’other, Sir John. Let me see: Simon Shadow!

FALSTAFF.
Yea, marry, let me have him to sit under. He’s like to be a cold
soldier.

SHALLOW.
Where’s Shadow?

SHADOW.
Here, sir.

FALSTAFF.
Shadow, whose son art thou?

SHADOW.
My mother’s son, sir.

FALSTAFF.
Thy mother’s son! Like enough, and thy father’s shadow. So the son of
the female is the shadow of the male. It is often so indeed, but much
of the father’s substance!

SHALLOW.
Do you like him, Sir John?

FALSTAFF.
Shadow will serve for summer. Prick him, for we have a number of
shadows to fill up the muster-book.

SHALLOW.
Thomas Wart!

FALSTAFF.
Where’s he?

WART.
Here, sir.

FALSTAFF.
Is thy name Wart?

WART.
Yea, sir.

FALSTAFF.
Thou art a very ragged wart.

SHALLOW.
Shall I prick him, Sir John?

FALSTAFF.
It were superfluous, for his apparel is built upon his back, and the
whole frame stands upon pins. Prick him no more.

SHALLOW.
Ha, ha, ha! You can do it, sir, you can do it. I commend you well.
Francis Feeble!

FEEBLE.
Here, sir.

FALSTAFF.
What trade art thou, Feeble?

FEEBLE.
A woman’s tailor, sir.

SHALLOW.
Shall I prick him, sir?

FALSTAFF.
You may; but if he had been a man’s tailor, he’d ha’ pricked you. Wilt
thou make as many holes in an enemy’s battle as thou hast done in a
woman’s petticoat?

FEEBLE.
I will do my good will, sir, you can have no more.

FALSTAFF.
Well said, good woman’s tailor! Well said, courageous Feeble! Thou wilt
be as valiant as the wrathful dove or most magnanimous mouse. Prick the
woman’s tailor: well, Master Shallow, deep, Master Shallow.

FEEBLE.
I would Wart might have gone, sir.

FALSTAFF.
I would thou wert a man’s tailor, that thou mightst mend him and make
him fit to go. I cannot put him to a private soldier that is the leader
of so many thousands. Let that suffice, most forcible Feeble.

FEEBLE.
It shall suffice, sir.

FALSTAFF.
I am bound to thee, reverend Feeble. Who is next?

SHALLOW.
Peter Bullcalf o’ th’ green!

FALSTAFF.
Yea, marry, let’s see Bullcalf.

BULLCALF.
Here, sir.

FALSTAFF.
Fore God, a likely fellow! Come, prick me Bullcalf till he roar again.

BULLCALF.
O Lord! good my lord captain—

FALSTAFF.
What, dost thou roar before thou art pricked?

BULLCALF.
O Lord, sir, I am a diseased man.

FALSTAFF.
What disease hast thou?

BULLCALF.
A whoreson cold, sir, a cough, sir, which I caught with ringing in the
King’s affairs upon his coronation day, sir.

FALSTAFF.
Come, thou shalt go to the wars in a gown; we will have away thy cold,
and I will take such order that thy friends shall ring for thee. Is
here all?

SHALLOW.
Here is two more called than your number; you must have but four here,
sir; and so, I pray you, go in with me to dinner.

FALSTAFF.
Come, I will go drink with you, but I cannot tarry dinner. I am glad to
see you, by my troth, Master Shallow.

SHALLOW.
O, Sir John, do you remember since we lay all night in the windmill in
Saint George’s Field?

FALSTAFF.
No more of that, good Master Shallow, no more of that.

SHALLOW.
Ha, ’twas a merry night. And is Jane Nightwork alive?

FALSTAFF.
She lives, Master Shallow.

SHALLOW.
She never could away with me.

FALSTAFF.
Never, never; she would always say she could not abide Master Shallow.

SHALLOW.
By the mass, I could anger her to th’ heart. She was then a bona-roba.
Doth she hold her own well?

FALSTAFF.
Old, old, Master Shallow.

SHALLOW.
Nay, she must be old, she cannot choose but be old, certain she’s old,
and had Robin Nightwork by old Nightwork before I came to Clement’s
Inn.

SILENCE.
That’s fifty-five year ago.

SHALLOW.
Ha, cousin Silence, that thou hadst seen that that this knight and I
have seen! Ha, Sir John, said I well?

FALSTAFF.
We have heard the chimes at midnight, Master Shallow.

SHALLOW.
That we have, that we have, that we have; in faith, Sir John, we have.
Our watchword was “Hem boys!” Come, let’s to dinner; come, let’s to
dinner. Jesus, the days that we have seen! Come, come.

 [_Exeunt Falstaff, Shallow and Silence._]

BULLCALF.
Good Master Corporate Bardolph, stand my friend; and here’s four Harry
ten shillings in French crowns for you. In very truth, sir, I had as
lief be hanged, sir, as go. And yet, for mine own part, sir, I do not
care; but rather because I am unwilling, and, for mine own part, have a
desire to stay with my friends; else, sir, I did not care, for mine own
part, so much.

BARDOLPH.
Go to, stand aside.

MOULDY.
And, good Master Corporal Captain, for my old dame’s sake, stand my
friend. She has nobody to do anything about her when I am gone, and she
is old, and cannot help herself. You shall have forty, sir.

BARDOLPH.
Go to, stand aside.

FEEBLE.
By my troth, I care not. A man can die but once. We owe God a death.
I’ll ne’er bear a base mind. An ’t be my destiny, so; an ’t be not, so.
No man’s too good to serve’s prince, and let it go which way it will,
he that dies this year is quit for the next.

BARDOLPH.
Well said, th’art a good fellow.

FEEBLE.
Faith, I’ll bear no base mind.

Enter Falstaff and the Justices.

FALSTAFF.
Come, sir, which men shall I have?

SHALLOW.
Four of which you please.

BARDOLPH.
Sir, a word with you. I have three pound to free Mouldy and Bullcalf.

FALSTAFF.
Go to, well.

SHALLOW.
Come, Sir John, which four will you have?

FALSTAFF.
Do you choose for me.

SHALLOW.
Marry, then, Mouldy, Bullcalf, Feeble, and Shadow.

FALSTAFF.
Mouldy and Bullcalf: for you, Mouldy, stay at home till you are past
service; and for your part, Bullcalf, grow till you come unto it. I
will none of you.

SHALLOW.
Sir John, Sir John, do not yourself wrong. They are your likeliest men,
and I would have you served with the best.

FALSTAFF.
Will you tell me, Master Shallow, how to choose a man? Care I for the
limb, the thews, the stature, bulk, and big assemblance of a man? Give
me the spirit, Master Shallow. Here’s Wart. You see what a ragged
appearance it is. He shall charge you and discharge you with the motion
of a pewterer’s hammer, come off and on swifter than he that gibbets on
the brewer’s bucket. And this same half-faced fellow, Shadow; give me
this man. He presents no mark to the enemy. The foeman may with as
great aim level at the edge of a penknife. And for a retreat, how
swiftly will this Feeble, the woman’s tailor, run off! O, give me the
spare men, and spare me the great ones. Put me a caliver into Wart’s
hand, Bardolph.

BARDOLPH.
Hold, Wart. Traverse. Thas, thas, thas.

FALSTAFF.
Come, manage me your caliver. So, very well, go to, very good,
exceeding good. O, give me always a little, lean, old, chopt, bald
shot. Well said, i’ faith, Wart. Th’art a good scab. Hold, there’s a
tester for thee.

SHALLOW.
He is not his craft’s master, he doth not do it right. I remember at
Mile-End Green, when I lay at Clement’s Inn—I was then Sir Dagonet in
Arthur’s show—there was a little quiver fellow, and he would manage you
his piece thus. And he would about and about, and come you in and come
you in. “Rah, tah, tah,” would he say. “Bounce” would he say; and away
again would he go, and again would he come. I shall ne’er see such a
fellow.

FALSTAFF.
These fellows will do well. Master Shallow. God keep you, Master
Silence: I will not use many words with you. Fare you well, gentlemen
both. I thank you. I must a dozen mile tonight. Bardolph, give the
soldiers coats.

SHALLOW.
Sir John, the Lord bless you! God prosper your affairs! God send us
peace! At your return, visit our house, let our old acquaintance be
renewed. Peradventure I will with ye to the court.

FALSTAFF.
Fore God, I would you would, Master Shallow.

SHALLOW.
Go to, I have spoke at a word. God keep you.

FALSTAFF.
Fare you well, gentle gentlemen. [_Exeunt Justices_.] On, Bardolph,
lead the men away. [_Exeunt Bardolph, recruits, &c._] As I return, I
will fetch off these justices. I do see the bottom of Justice Shallow.
Lord, Lord, how subject we old men are to this vice of lying! This same
starved justice hath done nothing but prate to me of the wildness of
his youth, and the feats he hath done about Turnbull Street, and every
third word a lie, duer paid to the hearer than the Turk’s tribute. I do
remember him at Clement’s Inn, like a man made after supper of a
cheese-paring. When he was naked, he was, for all the world, like a
forked radish, with a head fantastically carved upon it with a knife.
He was so forlorn, that his dimensions to any thick sight were
invincible. He was the very genius of famine, yet lecherous as a
monkey, and the whores called him mandrake. He came ever in the
rearward of the fashion, and sung those tunes to the overscutched
huswives that he heard the carmen whistle, and sware they were his
fancies or his good-nights. And now is this Vice’s dagger become a
squire, and talks as familiarly of John a Gaunt as if he had been sworn
brother to him, and I’ll be sworn he ne’er saw him but once in the
tilt-yard, and then he burst his head for crowding among the marshal’s
men. I saw it and told John a Gaunt he beat his own name, for you might
have thrust him and all his apparel into an eel-skin; the case of a
treble hautboy was a mansion for him, a court. And now has he land and
beefs. Well, I’ll be acquainted with him if I return, and ’t shall go
hard but I’ll make him a philosopher’s two stones to me. If the young
dace be a bait for the old pike, I see no reason in the law of nature
but I may snap at him. Let time shape, and there an end.

 [_Exit._]




ACT IV

SCENE I. Yorkshire. Gaultree Forest.

Enter the Archbishop of York, Mowbray, Hastings and others.

ARCHBISHOP.
What is this forest call’d?

HASTINGS.
’Tis Gaultree Forest, an ’t shall please your Grace.

ARCHBISHOP.
Here stand, my lords, and send discoverers forth
To know the numbers of our enemies.

HASTINGS.
We have sent forth already.

ARCHBISHOP.
’Tis well done.
My friends and brethren in these great affairs,
I must acquaint you that I have received
New-dated letters from Northumberland,
Their cold intent, tenor, and substance, thus:
Here doth he wish his person, with such powers
As might hold sortance with his quality,
The which he could not levy; whereupon
He is retired, to ripe his growing fortunes,
To Scotland, and concludes in hearty prayers
That your attempts may overlive the hazard
And fearful meeting of their opposite.

MOWBRAY.
Thus do the hopes we have in him touch ground
And dash themselves to pieces.

Enter a Messenger.

HASTINGS.
Now, what news?

MESSENGER.
West of this forest, scarcely off a mile,
In goodly form comes on the enemy,
And, by the ground they hide, I judge their number
Upon or near the rate of thirty thousand.

MOWBRAY.
The just proportion that we gave them out.
Let us sway on and face them in the field.

Enter Westmoreland.

ARCHBISHOP.
What well-appointed leader fronts us here?

MOWBRAY.
I think it is my Lord of Westmoreland.

WESTMORELAND.
Health and fair greeting from our general,
The prince, Lord John and Duke of Lancaster.

ARCHBISHOP.
Say on, my Lord of Westmoreland, in peace,
What doth concern your coming.

WESTMORELAND.
Then, my lord,
Unto your Grace do I in chief address
The substance of my speech. If that rebellion
Came like itself, in base and abject routs,
Led on by bloody youth, guarded with rags,
And countenanced by boys and beggary;
I say, if damn’d commotion so appear’d
In his true, native, and most proper shape,
You, reverend father, and these noble lords
Had not been here to dress the ugly form
Of base and bloody insurrection
With your fair honours. You, Lord Archbishop,
Whose see is by a civil peace maintain’d,
Whose beard the silver hand of peace hath touch’d,
Whose learning and good letters peace hath tutor’d,
Whose white investments figure innocence,
The dove and very blessed spirit of peace,
Wherefore you do so ill translate yourself
Out of the speech of peace that bears such grace,
Into the harsh and boisterous tongue of war;
Turning your books to graves, your ink to blood,
Your pens to lances and your tongue divine
To a loud trumpet and a point of war?

ARCHBISHOP.
Wherefore do I this? So the question stands.
Briefly to this end: we are all diseased,
And with our surfeiting and wanton hours
Have brought ourselves into a burning fever,
And we must bleed for it; of which disease
Our late King Richard, being infected, died.
But, my most noble Lord of Westmoreland,
I take not on me here as a physician,
Nor do I as an enemy to peace
Troop in the throngs of military men,
But rather show awhile like fearful war
To diet rank minds sick of happiness,
And purge th’ obstructions which begin to stop
Our very veins of life. Hear me more plainly.
I have in equal balance justly weigh’d
What wrongs our arms may do, what wrongs we suffer,
And find our griefs heavier than our offences.
We see which way the stream of time doth run,
And are enforced from our most quiet there
By the rough torrent of occasion,
And have the summary of all our griefs,
When time shall serve, to show in articles;
Which long ere this we offer’d to the King
And might by no suit gain our audience.
When we are wrong’d and would unfold our griefs,
We are denied access unto his person
Even by those men that most have done us wrong.
The dangers of the days but newly gone,
Whose memory is written on the earth
With yet-appearing blood, and the examples
Of every minute’s instance, present now,
Hath put us in these ill-beseeming arms,
Not to break peace or any branch of it,
But to establish here a peace indeed,
Concurring both in name and quality.

WESTMORELAND.
Whenever yet was your appeal denied?
Wherein have you been galled by the King?
What peer hath been suborn’d to grate on you,
That you should seal this lawless bloody book
Of forged rebellion with a seal divine
And consecrate commotion’s bitter edge?

ARCHBISHOP.
My brother general, the commonwealth,
To brother born an household cruelty,
I make my quarrel in particular.

WESTMORELAND.
There is no need of any such redress,
Or if there were, it not belongs to you.

MOWBRAY.
Why not to him in part, and to us all
That feel the bruises of the days before,
And suffer the condition of these times
To lay a heavy and unequal hand
Upon our honours?

WESTMORELAND.
O, my good Lord Mowbray,
Construe the times to their necessities,
And you shall say indeed, it is the time,
And not the King, that doth you injuries.
Yet for your part, it not appears to me
Either from the King or in the present time
That you should have an inch of any ground
To build a grief on. Were you not restored
To all the Duke of Norfolk’s signories,
Your noble and right well rememb’red father’s?

MOWBRAY.
What thing, in honour, had my father lost,
That need to be revived and breathed in me?
The King that loved him, as the state stood then,
Was force perforce compell’d to banish him,
And then that Henry Bolingbroke and he,
Being mounted and both roused in their seats,
Their neighing coursers daring of the spur,
Their armed staves in charge, their beavers down,
Their eyes of fire sparkling through sights of steel,
And the loud trumpet blowing them together,
Then, then, when there was nothing could have stay’d
My father from the breast of Bolingbroke,
O, when the King did throw his warder down,
His own life hung upon the staff he threw;
Then threw he down himself and all their lives
That by indictment and by dint of sword
Have since miscarried under Bolingbroke.

WESTMORELAND.
You speak, Lord Mowbray, now you know not what.
The Earl of Hereford was reputed then
In England the most valiant gentleman.
Who knows on whom fortune would then have smiled?
But if your father had been victor there,
He ne’er had borne it out of Coventry;
For all the country in a general voice
Cried hate upon him; and all their prayers and love
Were set on Hereford, whom they doted on
And bless’d and graced, indeed more than the King.
But this is mere digression from my purpose.
Here come I from our princely general
To know your griefs, to tell you from his Grace
That he will give you audience; and wherein
It shall appear that your demands are just,
You shall enjoy them, everything set off
That might so much as think you enemies.

MOWBRAY.
But he hath forc’d us to compel this offer,
And it proceeds from policy, not love.

WESTMORELAND.
Mowbray, you overween to take it so;
This offer comes from mercy, not from fear.
For, lo, within a ken our army lies,
Upon mine honour, all too confident
To give admittance to a thought of fear.
Our battle is more full of names than yours,
Our men more perfect in the use of arms,
Our armour all as strong, our cause the best;
Then reason will our hearts should be as good.
Say you not then our offer is compell’d.

MOWBRAY.
Well, by my will we shall admit no parley.

WESTMORELAND.
That argues but the shame of your offence:
A rotten case abides no handling.

HASTINGS.
Hath the Prince John a full commission,
In very ample virtue of his father,
To hear and absolutely to determine
Of what conditions we shall stand upon?

WESTMORELAND.
That is intended in the general’s name:
I muse you make so slight a question.

ARCHBISHOP.
Then take, my Lord of Westmoreland, this schedule,
For this contains our general grievances.
Each several article herein redress’d,
All members of our cause, both here and hence,
That are insinew’d to this action,
Acquitted by a true substantial form
And present execution of our wills
To us and to our purposes confined,
We come within our awful banks again
And knit our powers to the arm of peace.

WESTMORELAND.
This will I show the general. Please you, lords,
In sight of both our battles we may meet,
And either end in peace, which God so frame!
Or to the place of difference call the swords
Which must decide it.

ARCHBISHOP.
My lord, we will do so.

 [_Exit Westmoreland._]

MOWBRAY.
There is a thing within my bosom tells me
That no conditions of our peace can stand.

HASTINGS.
Fear you not that: if we can make our peace
Upon such large terms and so absolute
As our conditions shall consist upon,
Our peace shall stand as firm as rocky mountains.

MOWBRAY.
Yea, but our valuation shall be such
That every slight and false-derived cause,
Yea, every idle, nice, and wanton reason,
Shall to the King taste of this action;
That, were our royal faiths martyrs in love,
We shall be winnow’d with so rough a wind
That even our corn shall seem as light as chaff
And good from bad find no partition.

ARCHBISHOP.
No, no, my lord. Note this; the King is weary
Of dainty and such picking grievances;
For he hath found to end one doubt by death
Revives two greater in the heirs of life;
And therefore will he wipe his tables clean
And keep no tell-tale to his memory
That may repeat and history his loss
To new remembrance. For full well he knows
He cannot so precisely weed this land
As his misdoubts present occasion.
His foes are so enrooted with his friends
That, plucking to unfix an enemy,
He doth unfasten so and shake a friend.
So that this land, like an offensive wife
That hath enraged him on to offer strokes,
As he is striking, holds his infant up
And hangs resolved correction in the arm
That was uprear’d to execution.

HASTINGS.
Besides, the King hath wasted all his rods
On late offenders, that he now doth lack
The very instruments of chastisement;
So that his power, like to a fangless lion,
May offer, but not hold.

ARCHBISHOP.
’Tis very true,
And therefore be assured, my good Lord Marshal,
If we do now make our atonement well,
Our peace will, like a broken limb united,
Grow stronger for the breaking.

MOWBRAY.
Be it so.
Here is return’d my Lord of Westmoreland.

Enter Westmoreland.

WESTMORELAND.
The prince is here at hand. Pleaseth your lordship
To meet his Grace just distance ’tween our armies.

MOWBRAY.
Your Grace of York, in God’s name then set forward.

ARCHBISHOP.
Before, and greet his Grace. My lord, we come.

 [_Exeunt._]

SCENE II. Another part of the forest.

Enter, from one side, Mowbray, attended; afterwards, the Archbishop,
Hastings, and others; from the other side, Prince John of Lancaster,
and Westmoreland; Officers, and others with them.

LANCASTER.
You are well encounter’d here, my cousin Mowbray.
Good day to you, gentle Lord Archbishop;
And so to you, Lord Hastings, and to all.
My Lord of York, it better show’d with you
When that your flock, assembled by the bell,
Encircled you to hear with reverence
Your exposition on the holy text
Than now to see you here an iron man,
Cheering a rout of rebels with your drum,
Turning the word to sword, and life to death.
That man that sits within a monarch’s heart,
And ripens in the sunshine of his favour,
Would he abuse the countenance of the king,
Alack, what mischiefs might he set abroach
In shadow of such greatness! With you, Lord Bishop,
It is even so. Who hath not heard it spoken
How deep you were within the books of God,
To us the speaker in his parliament,
To us th’ imagined voice of God himself,
The very opener and intelligencer
Between the grace, the sanctities of heaven,
And our dull workings? O, who shall believe
But you misuse the reverence of your place,
Employ the countenance and grace of heaven
As a false favourite doth his prince’s name,
In deeds dishonourable? You have ta’en up,
Under the counterfeited zeal of God,
The subjects of his substitute, my father,
And both against the peace of heaven and him
Have here up-swarm’d them.

ARCHBISHOP.
Good my Lord of Lancaster,
I am not here against your father’s peace;
But, as I told my Lord of Westmoreland,
The time misorder’d doth, in common sense,
Crowd us and crush us to this monstrous form
To hold our safety up. I sent your Grace
The parcels and particulars of our grief,
The which hath been with scorn shoved from the court,
Whereon this Hydra son of war is born,
Whose dangerous eyes may well be charm’d asleep
With grant of our most just and right desires,
And true obedience, of this madness cured,
Stoop tamely to the foot of majesty.

MOWBRAY.
If not, we ready are to try our fortunes
To the last man.

HASTINGS.
And though we here fall down,
We have supplies to second our attempt:
If they miscarry, theirs shall second them;
And so success of mischief shall be born,
And heir from heir shall hold this quarrel up
Whiles England shall have generation.

LANCASTER.
You are too shallow, Hastings, much too shallow,
To sound the bottom of the after-times.

WESTMORELAND.
Pleaseth your Grace to answer them directly
How far forth you do like their articles.

LANCASTER.
I like them all, and do allow them well,
And swear here, by the honour of my blood,
My father’s purposes have been mistook,
And some about him have too lavishly
Wrested his meaning and authority.
My lord, these griefs shall be with speed redress’d;
Upon my soul, they shall. If this may please you,
Discharge your powers unto their several counties,
As we will ours; and here between the armies
Let’s drink together friendly and embrace,
That all their eyes may bear those tokens home
Of our restored love and amity.

ARCHBISHOP.
I take your princely word for these redresses.

LANCASTER.
I give it you, and will maintain my word;
And thereupon I drink unto your Grace.

HASTINGS.
Go, captain, and deliver to the army
This news of peace. Let them have pay, and part.
I know it will please them. Hie thee, captain.

 [_Exit Officer._]

ARCHBISHOP.
To you, my noble Lord of Westmoreland.

WESTMORELAND.
I pledge your Grace; and if you knew what pains
I have bestow’d to breed this present peace,
You would drink freely; but my love to ye
Shall show itself more openly hereafter.

ARCHBISHOP.
I do not doubt you.

WESTMORELAND.
I am glad of it.
Health to my lord and gentle cousin, Mowbray.

MOWBRAY.
You wish me health in very happy season,
For I am on the sudden something ill.

ARCHBISHOP.
Against ill chances men are ever merry,
But heaviness foreruns the good event.

WESTMORELAND.
Therefore be merry, coz, since sudden sorrow
Serves to say thus, “Some good thing comes tomorrow.”

ARCHBISHOP.
Believe me, I am passing light in spirit.

MOWBRAY.
So much the worse, if your own rule be true.

 [_Shouts within._]

LANCASTER.
The word of peace is render’d. Hark how they shout!

MOWBRAY.
This had been cheerful after victory.

ARCHBISHOP.
A peace is of the nature of a conquest;
For then both parties nobly are subdued,
And neither party loser.

LANCASTER.
Go, my lord.
And let our army be discharged too.

 [_Exit Westmoreland._]

And, good my lord, so please you, let our trains
March by us, that we may peruse the men
We should have coped withal.

ARCHBISHOP.
Go, good Lord Hastings,
And, ere they be dismiss’d, let them march by.

 [_Exit Hastings._]

LANCASTER.
I trust, lords, we shall lie tonight together.

Enter Westmoreland.

Now, cousin, wherefore stands our army still?

WESTMORELAND.
The leaders, having charge from you to stand,
Will not go off until they hear you speak.

LANCASTER.
They know their duties.

Enter Hastings.

HASTINGS.
My lord, our army is dispersed already.
Like youthful steers unyoked, they take their courses
East, west, north, south; or, like a school broke up,
Each hurries toward his home and sporting-place.

WESTMORELAND.
Good tidings, my Lord Hastings; for the which
I do arrest thee, traitor, of high treason;
And you, Lord Archbishop, and you, Lord Mowbray,
Of capital treason I attach you both.

MOWBRAY.
Is this proceeding just and honourable?

WESTMORELAND.
Is your assembly so?

ARCHBISHOP.
Will you thus break your faith?

LANCASTER.
I pawn’d thee none.
I promised you redress of these same grievances
Whereof you did complain; which, by mine honour,
I will perform with a most Christian care.
But for you, rebels, look to taste the due
Meet for rebellion and such acts as yours.
Most shallowly did you these arms commence,
Fondly brought here and foolishly sent hence.
Strike up our drums, pursue the scattr’d stray:
God, and not we, hath safely fought today.
Some guard these traitors to the block of death,
Treason’s true bed and yielder-up of breath.

 [_Exeunt._]

SCENE III. Another part of the forest.

Alarum. Excursions. Enter Falstaff and Colevile, meeting.

FALSTAFF.
What’s your name, sir? Of what condition are you, and of what place, I
pray?

COLEVILE.
I am a knight, sir, and my name is Colevile of the Dale.

FALSTAFF.
Well, then, Colevile is your name, a knight is your degree, and your
place the Dale. Colevile shall be still your name, a traitor your
degree, and the dungeon your place, a place deep enough; so shall you
be still Colevile of the Dale.

COLEVILE.
Are not you Sir John Falstaff?

FALSTAFF.
As good a man as he, sir, whoe’er I am. Do ye yield, sir, or shall I
sweat for you? If I do sweat, they are the drops of thy lovers, and
they weep for thy death. Therefore rouse up fear and trembling, and do
observance to my mercy.

COLEVILE.
I think you are Sir John Falstaff, and in that thought yield me.

FALSTAFF.
I have a whole school of tongues in this belly of mine, and not a
tongue of them all speaks any other word but my name. An I had but a
belly of any indifferency, I were simply the most active fellow in
Europe. My womb, my womb, my womb undoes me. Here comes our general.

Enter Prince John of Lancaster, Westmoreland, Blunt, and others.

LANCASTER.
The heat is past; follow no further now.
Call in the powers, good cousin Westmoreland.

 [_Exit Westmoreland._]

Now, Falstaff, where have you been all this while?
When everything is ended, then you come.
These tardy tricks of yours will, on my life,
One time or other break some gallows’ back.

FALSTAFF.
I would be sorry, my lord, but it should be thus. I never knew yet but
rebuke and check was the reward of valour. Do you think me a swallow,
an arrow, or a bullet? Have I, in my poor and old motion, the
expedition of thought? I have speeded hither with the very extremest
inch of possibility; I have foundered nine score and odd posts; and
here, travel-tainted as I am, have in my pure and immaculate valour,
taken Sir John Colevile of the Dale, a most furious knight and valorous
enemy. But what of that? He saw me, and yielded; that I may justly say,
with the hook-nosed fellow of Rome, “I came, saw, and overcame.”

LANCASTER.
It was more of his courtesy than your deserving.

FALSTAFF.
I know not. Here he is, and here I yield him. And I beseech your Grace,
let it be booked with the rest of this day’s deeds, or, by the Lord, I
will have it in a particular ballad else, with mine own picture on the
top on’t, Colevile kissing my foot: to the which course if I be
enforced, if you do not all show like gilt twopences to me, and I in
the clear sky of fame o’ershine you as much as the full moon doth the
cinders of the element, which show like pins’ heads to her, believe not
the word of the noble. Therefore let me have right, and let desert
mount.

LANCASTER.
Thine’s too heavy to mount.

FALSTAFF.
Let it shine, then.

LANCASTER.
Thine’s too thick to shine.

FALSTAFF.
Let it do something, my good lord, that may do me good, and call it
what you will.

LANCASTER.
Is thy name Colevile?

COLEVILE.
It is, my lord.

LANCASTER.
A famous rebel art thou, Colevile.

FALSTAFF.
And a famous true subject took him.

COLEVILE.
I am, my lord, but as my betters are
That led me hither. Had they been ruled by me,
You should have won them dearer than you have.

FALSTAFF.
I know not how they sold themselves, but thou, like a kind fellow,
gavest thyself away gratis, and I thank thee for thee.

Enter Westmoreland.

LANCASTER.
Now, have you left pursuit?

WESTMORELAND.
Retreat is made and execution stay’d.

LANCASTER.
Send Colevile with his confederates
To York, to present execution.
Blunt, lead him hence, and see you guard him sure.

 [_Exeunt Blunt and others with Colevile._]

And now dispatch we toward the court, my lords.
I hear the King my father is sore sick.
Our news shall go before us to his Majesty,
Which, cousin, you shall bear to comfort him,
And we with sober speed will follow you.

FALSTAFF.
My lord, I beseech you give me leave to go through Gloucestershire,
and, when you come to court, stand my good lord, pray, in your good
report.

LANCASTER.
Fare you well, Falstaff. I, in my condition,
Shall better speak of you than you deserve.

 [_Exeunt all but Falstaff._]

FALSTAFF.
I would you had but the wit, ’twere better than your dukedom. Good
faith, this same young sober-blooded boy doth not love me, nor a man
cannot make him laugh; but that’s no marvel, he drinks no wine. There’s
never none of these demure boys come to any proof; for thin drink doth
so over-cool their blood, and making many fish meals, that they fall
into a kind of male green-sickness; and then, when they marry, they get
wenches. They are generally fools and cowards, which some of us should
be too, but for inflammation. A good sherris-sack hath a two-fold
operation in it. It ascends me into the brain, dries me there all the
foolish and dull and crudy vapours which environ it, makes it
apprehensive, quick, forgetive, full of nimble, fiery, and delectable
shapes, which, delivered o’er to the voice, the tongue, which is the
birth, becomes excellent wit. The second property of your excellent
sherris is the warming of the blood, which, before cold and settled,
left the liver white and pale, which is the badge of pusillanimity and
cowardice. But the sherris warms it and makes it course from the
inwards to the parts’ extremes. It illumineth the face, which as a
beacon gives warning to all the rest of this little kingdom, man, to
arm; and then the vital commoners and inland petty spirits muster me
all to their captain, the heart, who, great and puffed up with this
retinue, doth any deed of courage; and this valour comes of sherris. So
that skill in the weapon is nothing without sack, for that sets it
a-work; and learning a mere hoard of gold kept by a devil, till sack
commences it and sets it in act and use. Hereof comes it that Prince
Harry is valiant; for the cold blood he did naturally inherit of his
father he hath, like lean, sterile and bare land, manured, husbanded
and tilled with excellent endeavour of drinking good and good store of
fertile sherris, that he is become very hot and valiant. If I had a
thousand sons, the first humane principle I would teach them should be
to forswear thin potations and to addict themselves to sack.

Enter Bardolph.

How now, Bardolph?

BARDOLPH.
The army is discharged all and gone.

FALSTAFF.
Let them go. I’ll through Gloucestershire, and there will I visit
Master Robert Shallow, Esquire. I have him already tempering between my
finger and my thumb, and shortly will I seal with him. Come away.

 [_Exeunt._]

SCENE IV. Westminster. The Jerusalem Chamber.

Enter the King, Warwick, Thomas Duke of Clarence and Humphrey Duke of
Gloucester and others.

KING.
Now, lords, if God doth give successful end
To this debate that bleedeth at our doors,
We will our youth lead on to higher fields
And draw no swords but what are sanctified.
Our navy is address’d, our power collected,
Our substitutes in absence well invested,
And everything lies level to our wish.
Only we want a little personal strength;
And pause us till these rebels now afoot
Come underneath the yoke of government.

WARWICK.
Both which we doubt not but your Majesty
Shall soon enjoy.

KING.
Humphrey, my son of Gloucester,
Where is the Prince your brother?

GLOUCESTER.
I think he’s gone to hunt, my lord, at Windsor.

KING.
And how accompanied?

GLOUCESTER.
I do not know, my lord.

KING.
Is not his brother Thomas of Clarence with him?

GLOUCESTER.
No, my good lord, he is in presence here.

CLARENCE.
What would my lord and father?

KING.
Nothing but well to thee, Thomas of Clarence.
How chance thou art not with the Prince thy brother?
He loves thee, and thou dost neglect him, Thomas.
Thou hast a better place in his affection
Than all thy brothers. Cherish it, my boy,
And noble offices thou mayst effect
Of mediation, after I am dead,
Between his greatness and thy other brethren.
Therefore omit him not, blunt not his love,
Nor lose the good advantage of his grace
By seeming cold or careless of his will;
For he is gracious, if he be observed,
He hath a tear for pity, and a hand
Open as day for melting charity:
Yet notwithstanding, being incensed, he’s flint,
As humorous as winter, and as sudden
As flaws congealed in the spring of day.
His temper therefore must be well observed.
Chide him for faults, and do it reverently,
When you perceive his blood inclined to mirth;
But, being moody, give him time and scope,
Till that his passions, like a whale on ground,
Confound themselves with working. Learn this, Thomas,
And thou shalt prove a shelter to thy friends,
A hoop of gold to bind thy brothers in,
That the united vessel of their blood,
Mingled with venom of suggestion—
As, force perforce, the age will pour it in—
Shall never leak, though it do work as strong
As aconitum or rash gunpowder.

CLARENCE.
I shall observe him with all care and love.

KING.
Why art thou not at Windsor with him, Thomas?

CLARENCE.
He is not there today; he dines in London.

KING.
And how accompanied? Canst thou tell that?

CLARENCE.
With Poins, and other his continual followers.

KING.
Most subject is the fattest soil to weeds,
And he, the noble image of my youth,
Is overspread with them; therefore my grief
Stretches itself beyond the hour of death.
The blood weeps from my heart when I do shape
In forms imaginary th’ unguided days
And rotten times that you shall look upon
When I am sleeping with my ancestors.
For when his headstrong riot hath no curb,
When rage and hot blood are his counsellors,
When means and lavish manners meet together,
O, with what wings shall his affections fly
Towards fronting peril and opposed decay!

WARWICK.
My gracious lord, you look beyond him quite.
The prince but studies his companions
Like a strange tongue, wherein, to gain the language,
’Tis needful that the most immodest word
Be looked upon and learned; which once attained,
Your Highness knows, comes to no further use
But to be known and hated. So, like gross terms,
The Prince will, in the perfectness of time,
Cast off his followers, and their memory
Shall as a pattern or a measure live,
By which his Grace must mete the lives of other,
Turning past evils to advantages.

KING.
’Tis seldom when the bee doth leave her comb
In the dead carrion.

Enter Westmoreland.

Who’s here? Westmoreland?

WESTMORELAND.
Health to my sovereign, and new happiness
Added to that that I am to deliver!
Prince John your son doth kiss your Grace’s hand.
Mowbray, the Bishop Scroop, Hastings and all
Are brought to the correction of your law.
There is not now a rebel’s sword unsheathed,
But Peace puts forth her olive everywhere.
The manner how this action hath been borne
Here at more leisure may your Highness read,
With every course in his particular.

KING.
O Westmoreland, thou art a summer bird,
Which ever in the haunch of winter sings
The lifting up of day.

Enter Harcourt.

Look, here’s more news.

HARCOURT.
From enemies heaven keep your Majesty;
And when they stand against you, may they fall
As those that I am come to tell you of!
The Earl Northumberland and the Lord Bardolph,
With a great power of English and of Scots,
Are by the shrieve of Yorkshire overthrown.
The manner and true order of the fight
This packet, please it you, contains at large.

KING.
And wherefore should these good news make me sick?
Will Fortune never come with both hands full,
But write her fair words still in foulest letters?
She either gives a stomach and no food—
Such are the poor, in health; or else a feast
And takes away the stomach—such are the rich,
That have abundance and enjoy it not.
I should rejoice now at this happy news,
And now my sight fails, and my brain is giddy.
O me! Come near me, now I am much ill.

GLOUCESTER.
Comfort, your Majesty!

CLARENCE.
O my royal father!

WESTMORELAND.
My sovereign lord, cheer up yourself, look up.

WARWICK.
Be patient, princes; you do know these fits
Are with his Highness very ordinary.
Stand from him, give him air; he’ll straight be well.

CLARENCE.
No, no, he cannot long hold out these pangs.
Th’ incessant care and labour of his mind
Hath wrought the mure that should confine it in
So thin that life looks through and will break out.

GLOUCESTER.
The people fear me, for they do observe
Unfather’d heirs and loathly births of nature.
The seasons change their manners, as the year
Had found some months asleep and leap’d them over.

CLARENCE.
The river hath thrice flow’d, no ebb between,
And the old folk, time’s doting chronicles,
Say it did so a little time before
That our great-grandsire, Edward, sick’d and died.

WARWICK.
Speak lower, princes, for the King recovers.

GLOUCESTER.
This apoplexy will certain be his end.

KING.
I pray you take me up, and bear me hence
Into some other chamber: softly, pray.

 [_Exeunt._]

SCENE V. Another chamber.

The King lying on a bed. Clarence, Gloucester, Warwick and others in
attendance.

KING.
Let there be no noise made, my gentle friends,
Unless some dull and favourable hand
Will whisper music to my weary spirit.

WARWICK.
Call for the music in the other room.

KING.
Set me the crown upon my pillow here.

CLARENCE.
His eye is hollow, and he changes much.

WARWICK.
Less noise, less noise!

Enter Prince Henry.

PRINCE.
Who saw the Duke of Clarence?

CLARENCE.
I am here, brother, full of heaviness.

PRINCE.
How now, rain within doors, and none abroad?
How doth the King?

GLOUCESTER.
Exceeding ill.

PRINCE.
Heard he the good news yet? Tell it him.

GLOUCESTER.
He alt’red much upon the hearing it.

PRINCE.
If he be sick with joy, he’ll recover without physic.

WARWICK.
Not so much noise, my lords. Sweet prince, speak low;
The King your father is disposed to sleep.

CLARENCE.
Let us withdraw into the other room.

WARWICK.
Will’t please your Grace to go along with us?

PRINCE.
No, I will sit and watch here by the King.

 [_Exeunt all but the Prince._]

Why doth the crown lie there upon his pillow,
Being so troublesome a bedfellow?
O polish’d perturbation! golden care!
That keep’st the ports of slumber open wide
To many a watchful night! Sleep with it now;
Yet not so sound and half so deeply sweet
As he whose brow with homely biggen bound
Snores out the watch of night. O majesty!
When thou dost pinch thy bearer, thou dost sit
Like a rich armour worn in heat of day,
That scald’st with safety. By his gates of breath
There lies a downy feather which stirs not:
Did he suspire, that light and weightless down
Perforce must move. My gracious lord, my father!
This sleep is sound indeed; this is a sleep
That from this golden rigol hath divorced
So many English kings. Thy due from me
Is tears and heavy sorrows of the blood,
Which nature, love, and filial tenderness,
Shall, O dear father, pay thee plenteously.
My due from thee is this imperial crown,
Which, as immediate from thy place and blood,
Derives itself to me. Lo, where it sits,
Which God shall guard; and put the world’s whole strength
Into one giant arm, it shall not force
This lineal honour from me. This from thee
Will I to mine leave, as ’tis left to me.

 [_Exit._]

KING.
Warwick! Gloucester! Clarence!

Enter Warwick, Gloucester, Clarence and the rest.

CLARENCE.
Doth the King call?

WARWICK.
What would your Majesty? How fares your Grace?

KING.
Why did you leave me here alone, my lords?

CLARENCE.
We left the Prince my brother here, my liege,
Who undertook to sit and watch by you.

KING.
The Prince of Wales! Where is he? Let me see him.
He is not here.

WARWICK.
This door is open, he is gone this way.

GLOUCESTER.
He came not through the chamber where we stay’d.

KING.
Where is the crown? Who took it from my pillow?

WARWICK.
When we withdrew, my liege, we left it here.

KING.
The Prince hath ta’en it hence. Go seek him out.
Is he so hasty that he doth suppose
My sleep my death?
Find him, my Lord of Warwick, chide him hither.

 [_Exit Warwick._]

This part of his conjoins with my disease,
And helps to end me. See, sons, what things you are,
How quickly nature falls into revolt
When gold becomes her object!
For this the foolish over-careful fathers
Have broke their sleep with thoughts,
Their brains with care, their bones with industry;
For this they have engrossed and piled up
The canker’d heaps of strange-achieved gold;
For this they have been thoughtful to invest
Their sons with arts and martial exercises;
When, like the bee, tolling from every flower
The virtuous sweets,
Our thighs pack’d with wax, our mouths with honey,
We bring it to the hive; and like the bees,
Are murdered for our pains. This bitter taste
Yields his engrossments to the ending father.

Enter Warwick.

Now where is he that will not stay so long
Till his friend sickness hath determin’d me?

WARWICK.
My lord, I found the Prince in the next room,
Washing with kindly tears his gentle cheeks,
With such a deep demeanour in great sorrow
That tyranny, which never quaff’d but blood,
Would, by beholding him, have wash’d his knife
With gentle eye-drops. He is coming hither.

KING.
But wherefore did he take away the crown?

Enter Prince Henry.

Lo where he comes. Come hither to me, Harry.
Depart the chamber, leave us here alone.

 [_Exeunt Warwick and the rest._]

PRINCE.
I never thought to hear you speak again.

KING.
Thy wish was father, Harry, to that thought.
I stay too long by thee, I weary thee.
Dost thou so hunger for mine empty chair
That thou wilt needs invest thee with my honours
Before thy hour be ripe? O foolish youth!
Thou seek’st the greatness that will overwhelm thee.
Stay but a little, for my cloud of dignity
Is held from falling with so weak a wind
That it will quickly drop. My day is dim.
Thou hast stolen that which after some few hours
Were thine without offence, and at my death
Thou hast seal’d up my expectation.
Thy life did manifest thou loved’st me not,
And thou wilt have me die assured of it.
Thou hid’st a thousand daggers in thy thoughts
Which thou hast whetted on thy stony heart,
To stab at half an hour of my life.
What, canst thou not forbear me half an hour?
Then get thee gone, and dig my grave thyself,
And bid the merry bells ring to thine ear
That thou art crowned, not that I am dead.
Let all the tears that should bedew my hearse
Be drops of balm to sanctify thy head,
Only compound me with forgotten dust.
Give that which gave thee life unto the worms.
Pluck down my officers, break my decrees;
For now a time is come to mock at form.
Harry the Fifth is crown’d. Up, vanity!
Down, royal state! All you sage counsellors, hence!
And to the English court assemble now,
From every region, apes of idleness!
Now, neighbour confines, purge you of your scum.
Have you a ruffian that will swear, drink, dance,
Revel the night, rob, murder, and commit
The oldest sins the newest kind of ways?
Be happy, he will trouble you no more.
England shall double gild his treble guilt,
England shall give him office, honour, might,
For the fifth Harry from curb’d license plucks
The muzzle of restraint, and the wild dog
Shall flesh his tooth on every innocent.
O my poor kingdom, sick with civil blows!
When that my care could not withhold thy riots,
What wilt thou do when riot is thy care?
O, thou wilt be a wilderness again,
Peopled with wolves, thy old inhabitants!

PRINCE.
O, pardon me, my liege! But for my tears,
The moist impediments unto my speech,
I had forestall’d this dear and deep rebuke
Ere you with grief had spoke and I had heard
The course of it so far. There is your crown;
And He that wears the crown immortally
Long guard it yours! If I affect it more
Than as your honour and as your renown,
Let me no more from this obedience rise,
Which my most inward true and duteous spirit
Teacheth this prostrate and exterior bending.
God witness with me, when I here came in,
And found no course of breath within your Majesty,
How cold it struck my heart! If I do feign,
O, let me in my present wildness die
And never live to show th’ incredulous world
The noble change that I have purposed!
Coming to look on you, thinking you dead,
And dead almost, my liege, to think you were,
I spake unto this crown as having sense,
And thus upbraided it: “The care on thee depending
Hath fed upon the body of my father;
Therefore thou best of gold art worst of gold.
Other, less fine in carat, is more precious,
Preserving life in med’cine potable;
But thou, most fine, most honour’d, most renown’d,
Hast eat thy bearer up.” Thus, my most royal liege,
Accusing it, I put it on my head,
To try with it, as with an enemy
That had before my face murder’d my father,
The quarrel of a true inheritor.
But if it did infect my blood with joy,
Or swell my thoughts to any strain of pride,
If any rebel or vain spirit of mine
Did with the least affection of a welcome
Give entertainment to the might of it,
Let God for ever keep it from my head
And make me as the poorest vassal is
That doth with awe and terror kneel to it!

KING.
O my son,
God put it in thy mind to take it hence,
That thou mightst win the more thy father’s love,
Pleading so wisely in excuse of it!
Come hither, Harry, sit thou by my bed,
And hear, I think, the very latest counsel
That ever I shall breathe. God knows, my son,
By what by-paths and indirect crook’d ways
I met this crown, and I myself know well
How troublesome it sat upon my head.
To thee it shall descend with better quiet,
Better opinion, better confirmation,
For all the soil of the achievement goes
With me into the earth. It seem’d in me
But as an honour snatch’d with boisterous hand,
And I had many living to upbraid
My gain of it by their assistances,
Which daily grew to quarrel and to bloodshed,
Wounding supposed peace. All these bold fears
Thou seest with peril I have answered;
For all my reign hath been but as a scene
Acting that argument. And now my death
Changes the mood, for what in me was purchased,
Falls upon thee in a more fairer sort;
So thou the garland wear’st successively.
Yet though thou stand’st more sure than I could do,
Thou art not firm enough, since griefs are green;
And all my friends, which thou must make thy friends,
Have but their stings and teeth newly ta’en out;
By whose fell working I was first advanced
And by whose power I well might lodge a fear
To be again displaced; which to avoid,
I cut them off, and had a purpose now
To lead out many to the Holy Land,
Lest rest and lying still might make them look
Too near unto my state. Therefore, my Harry,
Be it thy course to busy giddy minds
With foreign quarrels, that action, hence borne out,
May waste the memory of the former days.
More would I, but my lungs are wasted so
That strength of speech is utterly denied me.
How I came by the crown, O God, forgive,
And grant it may with thee in true peace live!

PRINCE.
My gracious liege,
You won it, wore it, kept it, gave it me;
Then plain and right must my possession be,
Which I with more than with a common pain
’Gainst all the world will rightfully maintain.

Enter Lord John of Lancaster and others.

KING.
Look, look, here comes my John of Lancaster.

LANCASTER.
Health, peace, and happiness to my royal father!

KING.
Thou bring’st me happiness and peace, son John,
But health, alack, with youthful wings is flown
From this bare wither’d trunk. Upon thy sight
My worldly business makes a period.
Where is my Lord of Warwick?

PRINCE.
My Lord of Warwick!

Enter Warwick and others.

KING.
Doth any name particular belong
Unto the lodging where I first did swoon?

WARWICK.
’Tis call’d Jerusalem, my noble lord.

KING.
Laud be to God! Even there my life must end.
It hath been prophesied to me many years,
I should not die but in Jerusalem,
Which vainly I supposed the Holy Land.
But bear me to that chamber; there I’ll lie;
In that Jerusalem shall Harry die.

 [_Exeunt._]




ACT V

SCENE I. Gloucestershire. Shallow’s house.

Enter Shallow, Falstaff, Bardolph and Page.

SHALLOW.
By cock and pie, sir, you shall not away tonight.
What, Davy, I say!

FALSTAFF.
You must excuse me, Master Robert Shallow.

SHALLOW.
I will not excuse you, you shall not be excused. Excuses shall not be
admitted, there is no excuse shall serve, you shall not be excused.
Why, Davy!

Enter Davy.

DAVY.
Here, sir.

SHALLOW.
Davy, Davy, Davy, Davy, let me see, Davy, let me see, Davy, let me see.
Yea, marry, William cook, bid him come hither. Sir John, you shall not
be excused.

DAVY.
Marry, sir, thus: those precepts cannot be served; and again, sir—shall
we sow the hade land with wheat?

SHALLOW.
With red wheat, Davy. But for William cook, are there no young pigeons?

DAVY.
Yes, sir. Here is now the smith’s note for shoeing and plough-irons.

SHALLOW.
Let it be cast and paid. Sir John, you shall not be excused.

DAVY.
Now, sir, a new link to the bucket must needs be had. And, sir, do you
mean to stop any of William’s wages, about the sack he lost the other
day at Hinckley fair?

SHALLOW.
He shall answer it. Some pigeons, Davy, a couple of short-legged hens,
a joint of mutton, and any pretty little tiny kickshaws, tell William
cook.

DAVY.
Doth the man of war stay all night, sir?

SHALLOW.
Yea, Davy, I will use him well: a friend i’ th’ court is better than a
penny in purse. Use his men well, Davy, for they are arrant knaves, and
will backbite.

DAVY.
No worse than they are backbitten, sir, for they have marvellous foul
linen.

SHALLOW.
Well conceited, Davy. About thy business, Davy.

DAVY.
I beseech you, sir, to countenance William Visor of Woncot against
Clement Perkes o’ th’ hill.

SHALLOW.
There is many complaints, Davy, against that Visor. That Visor is an
arrant knave, on my knowledge.

DAVY.
I grant your worship that he is a knave, sir, but yet, God forbid, sir,
but a knave should have some countenance at his friend’s request. An
honest man, sir, is able to speak for himself, when a knave is not. I
have served your worship truly, sir, this eight years; and if I cannot
once or twice in a quarter bear out a knave against an honest man, I
have but a very little credit with your worship. The knave is mine
honest friend, sir; therefore I beseech your worship let him be
countenanced.

SHALLOW.
Go to; I say he shall have no wrong. Look about, Davy.

 [_Exit Davy._]

Where are you, Sir John? Come, come, come, off with your boots.
Give me your hand, Master Bardolph.

BARDOLPH.
I am glad to see your worship.

SHALLOW.
I thank thee with all my heart, kind Master Bardolph; and welcome, my
tall fellow [_to the Page_]. Come, Sir John.

FALSTAFF.
I’ll follow you, good Master Robert Shallow.

 [_Exit Shallow._]

Bardolph, look to our horses.

 [_Exeunt Bardolph and Page._]

If I were sawed into quantities, I should make four dozen of such
bearded hermits’ staves as Master Shallow. It is a wonderful thing to
see the semblable coherence of his men’s spirits and his. They, by
observing of him, do bear themselves like foolish justices: he, by
conversing with them, is turned into a justice-like serving-man. Their
spirits are so married in conjunction with the participation of society
that they flock together in consent, like so many wild-geese. If I had
a suit to Master Shallow, I would humour his men with the imputation of
being near their master: if to his men, I would curry with Master
Shallow that no man could better command his servants. It is certain
that either wise bearing or ignorant carriage is caught, as men take
diseases, one of another. Therefore let men take heed of their company.
I will devise matter enough out of this Shallow to keep Prince Harry in
continual laughter the wearing out of six fashions, which is four
terms, or two actions, and he shall laugh without intervallums. O, it
is much that a lie with a slight oath and a jest with a sad brow will
do with a fellow that never had the ache in his shoulders! O, you shall
see him laugh till his face be like a wet cloak ill laid up!

SHALLOW.
[_Within_.] Sir John!

FALSTAFF.
I come, Master Shallow, I come, Master Shallow.

 [_Exit._]

SCENE II. Westminster. The palace.

Enter Warwick and the Lord Chief Justice, meeting.

WARWICK.
How now, my Lord Chief Justice, whither away?

CHIEF JUSTICE.
How doth the King?

WARWICK.
Exceeding well. His cares are now all ended.

CHIEF JUSTICE.
I hope, not dead.

WARWICK.
He’s walk’d the way of nature,
And to our purposes he lives no more.

CHIEF JUSTICE.
I would his Majesty had call’d me with him.
The service that I truly did his life
Hath left me open to all injuries.

WARWICK.
Indeed I think the young King loves you not.

CHIEF JUSTICE.
I know he doth not, and do arm myself
To welcome the condition of the time,
Which cannot look more hideously upon me
Than I have drawn it in my fantasy.

Enter Lancaster, Clarence, Gloucester and others.

WARWICK.
Here comes the heavy issue of dead Harry.
O that the living Harry had the temper
Of he the worst of these three gentlemen!
How many nobles then should hold their places,
That must strike sail to spirits of vile sort!

CHIEF JUSTICE.
O God, I fear all will be overturn’d.

LANCASTER.
Good morrow, cousin Warwick, good morrow.

GLOUCESTER & CLARENCE.
Good morrow, cousin.

LANCASTER.
We meet like men that had forgot to speak.

WARWICK.
We do remember, but our argument
Is all too heavy to admit much talk.

LANCASTER.
Well, peace be with him that hath made us heavy!

CHIEF JUSTICE.
Peace be with us, lest we be heavier!

GLOUCESTER.
O, good my lord, you have lost a friend indeed;
And I dare swear you borrow not that face
Of seeming sorrow; it is sure your own.

LANCASTER.
Though no man be assured what grace to find,
You stand in coldest expectation.
I am the sorrier; would ’twere otherwise.

CLARENCE.
Well, you must now speak Sir John Falstaff fair,
Which swims against your stream of quality.

CHIEF JUSTICE.
Sweet Princes, what I did I did in honour,
Led by th’ impartial conduct of my soul;
And never shall you see that I will beg
A ragged and forestall’d remission.
If truth and upright innocency fail me,
I’ll to the King my master that is dead,
And tell him who hath sent me after him.

WARWICK.
Here comes the Prince.

Enter King Henry the Fifth, attended.

CHIEF JUSTICE.
Good morrow, and God save your Majesty!

KING.
This new and gorgeous garment, majesty,
Sits not so easy on me as you think.
Brothers, you mix your sadness with some fear.
This is the English, not the Turkish court;
Not Amurath an Amurath succeeds,
But Harry Harry. Yet be sad, good brothers,
For, by my faith, it very well becomes you.
Sorrow so royally in you appears
That I will deeply put the fashion on
And wear it in my heart. Why then, be sad;
But entertain no more of it, good brothers,
Than a joint burden laid upon us all.
For me, by heaven, I bid you be assured,
I’ll be your father and your brother too;
Let me but bear your love, I’ll bear your cares.
Yet weep that Harry’s dead, and so will I;
But Harry lives, that shall convert those tears
By number into hours of happiness.

PRINCES.
We hope no otherwise from your Majesty.

KING.
You all look strangely on me. And you most;
You are, I think, assured I love you not.

CHIEF JUSTICE.
I am assured, if I be measured rightly,
Your Majesty hath no just cause to hate me.

KING.
No?
How might a prince of my great hopes forget
So great indignities you laid upon me?
What! rate, rebuke, and roughly send to prison
Th’ immediate heir of England? Was this easy?
May this be wash’d in Lethe and forgotten?

CHIEF JUSTICE.
I then did use the person of your father;
The image of his power lay then in me;
And in the administration of his law,
Whiles I was busy for the commonwealth,
Your Highness pleased to forget my place,
The majesty and power of law and justice,
The image of the King whom I presented,
And struck me in my very seat of judgement;
Whereon, as an offender to your father,
I gave bold way to my authority
And did commit you. If the deed were ill,
Be you contented, wearing now the garland,
To have a son set your decrees at nought?
To pluck down justice from your awful bench?
To trip the course of law and blunt the sword
That guards the peace and safety of your person?
Nay more, to spurn at your most royal image,
And mock your workings in a second body?
Question your royal thoughts, make the case yours;
Be now the father and propose a son,
Hear your own dignity so much profaned,
See your most dreadful laws so loosely slighted,
Behold yourself so by a son disdain’d,
And then imagine me taking your part
And in your power soft silencing your son.
After this cold considerance, sentence me;
And, as you are a king, speak in your state
What I have done that misbecame my place,
My person, or my liege’s sovereignty.

KING.
You are right, justice, and you weigh this well.
Therefore still bear the balance and the sword.
And I do wish your honours may increase
Till you do live to see a son of mine
Offend you and obey you, as I did.
So shall I live to speak my father’s words:
“Happy am I, that have a man so bold
That dares do justice on my proper son;
And not less happy, having such a son
That would deliver up his greatness so
Into the hands of justice.” You did commit me,
For which I do commit into your hand
Th’ unstained sword that you have used to bear,
With this remembrance: that you use the same
With the like bold, just, and impartial spirit
As you have done ’gainst me. There is my hand.
You shall be as a father to my youth,
My voice shall sound as you do prompt mine ear,
And I will stoop and humble my intents
To your well-practised wise directions.
And, princes all, believe me, I beseech you,
My father is gone wild into his grave,
For in his tomb lie my affections;
And with his spirit sadly I survive
To mock the expectation of the world,
To frustrate prophecies, and to raze out
Rotten opinion, who hath writ me down
After my seeming. The tide of blood in me
Hath proudly flow’d in vanity till now.
Now doth it turn and ebb back to the sea,
Where it shall mingle with the state of floods,
And flow henceforth in formal majesty.
Now call we our high court of parliament,
And let us choose such limbs of noble counsel
That the great body of our state may go
In equal rank with the best-govern’d nation;
That war, or peace, or both at once, may be
As things acquainted and familiar to us;
In which you, father, shall have foremost hand.
Our coronation done, we will accite,
As I before remember’d, all our state:
And, God consigning to my good intents,
No prince nor peer shall have just cause to say,
God shorten Harry’s happy life one day!

 [_Exeunt._]

SCENE III. Gloucestershire. Shallow’s orchard.

Enter Falstaff, Shallow, Silence, Davy, Bardolph and the Page.

SHALLOW.
Nay, you shall see my orchard, where, in an arbour, we will eat a last
year’s pippin of mine own graffing, with a dish of caraways, and so
forth. Come, cousin Silence. And then to bed.

FALSTAFF.
Fore God, you have here a goodly dwelling, and a rich.

SHALLOW.
Barren, barren, barren; beggars all, beggars all, Sir John. Marry, good
air. Spread, Davy, spread, Davy. Well said, Davy.

FALSTAFF.
This Davy serves you for good uses; he is your serving-man and your
husband.

SHALLOW.
A good varlet, a good varlet, a very good varlet, Sir John. By the
mass, I have drunk too much sack at supper. A good varlet. Now sit
down, now sit down. Come, cousin.

SILENCE.
Ah, sirrah! quoth-a, we shall [_Singing._]

    _Do nothing but eat, and make good cheer,
    And praise God for the merry year,
    When flesh is cheap and females dear,
    And lusty lads roam here and there
    So merrily,
    And ever among so merrily._

FALSTAFF.
There’s a merry heart! Good Master Silence, I’ll give you a health for
that anon.

SHALLOW.
Give Master Bardolph some wine, Davy.

DAVY.
Sweet sir, sit. I’ll be with you anon. Most sweet sir, sit. Master
page, good master page, sit. Proface! What you want in meat, we’ll have
in drink, but you must bear; the heart’s all.

 [_Exit._]

SHALLOW.
Be merry, Master Bardolph, and, my little soldier there, be merry.

SILENCE.
[_Singing._]

    _Be merry, be merry, my wife has all,
    For women are shrews, both short and tall.
    ’Tis merry in hall when beards wag all,
    And welcome merry Shrove-tide.
    Be merry, be merry._

FALSTAFF.
I did not think Master Silence had been a man of this mettle.

SILENCE.
Who, I? I have been merry twice and once ere now.

Enter Davy.

DAVY.
[_To Bardolph_.] There’s a dish of leather-coats for you.

SHALLOW.
Davy!

DAVY.
Your worship? I’ll be with you straight.
[_To Bardolph_] A cup of wine, sir?

SILENCE.
[_Singing._]

    _A cup of wine that’s brisk and fine,
    And drink unto thee, leman mine,
    And a merry heart lives long-a._

FALSTAFF.
Well said, Master Silence.

SILENCE.
An we shall be merry, now comes in the sweet o’ th’ night.

FALSTAFF.
Health and long life to you, Master Silence.

SILENCE.
[_Singing._]

    _Fill the cup, and let it come,
    I’ll pledge you a mile to th’ bottom._

SHALLOW.
Honest Bardolph, welcome! If thou want’st anything and wilt not call,
beshrew thy heart. Welcome, my little tiny thief, [_to the Page_] and
welcome indeed too. I’ll drink to Master Bardolph, and to all the
cabileros about London.

DAVY.
I hope to see London once ere I die.

BARDOLPH.
An I might see you there, Davy,—

SHALLOW.
By the mass, you’ll crack a quart together, ha! will you not, Master
Bardolph?

BARDOLPH.
Yea, sir, in a pottle-pot.

SHALLOW.
By God’s liggens, I thank thee. The knave will stick by thee, I can
assure thee that. He will not out, he. ’Tis true bred.

BARDOLPH.
And I’ll stick by him, sir.

SHALLOW.
Why, there spoke a king. Lack nothing! Be merry.

 [_Knocking within._]

Look who’s at door there, ho! Who knocks?

 [_Exit Davy._]

FALSTAFF.
[_To Silence, seeing him take off a bumper_.] Why, now you have done me
right.

SILENCE.
[_Singing._]

    _Do me right,
    And dub me knight:
    Samingo._

Is’t not so?

FALSTAFF.
’Tis so.

SILENCE.
Is’t so? Why then, say an old man can do somewhat.

Enter Davy.

DAVY.
An’t please your worship, there’s one Pistol come from the court with
news.

FALSTAFF.
From the court? Let him come in.

Enter Pistol.

How now, Pistol?

PISTOL.
Sir John, God save you!

FALSTAFF.
What wind blew you hither, Pistol?

PISTOL.
Not the ill wind which blows no man to good. Sweet knight, thou art now
one of the greatest men in this realm.

SILENCE.
By’r lady, I think he be, but goodman Puff of Barson.

PISTOL.
Puff!
Puff in thy teeth, most recreant coward base!
Sir John, I am thy Pistol and thy friend,
And helter-skelter have I rode to thee,
And tidings do I bring and lucky joys,
And golden times, and happy news of price.

FALSTAFF.
I pray thee now, deliver them like a man of this world.

PISTOL.
A foutre for the world and worldlings base!
I speak of Africa and golden joys.

FALSTAFF.
O base Assyrian knight, what is thy news?
Let King Cophetua know the truth thereof.

SILENCE.
[_Singing_.] _And Robin Hood, Scarlet, and John._

PISTOL.
Shall dunghill curs confront the Helicons?
And shall good news be baffled?
Then, Pistol, lay thy head in Furies’ lap.

SHALLOW.
Honest gentleman, I know not your breeding.

PISTOL.
Why then, lament therefor.

SHALLOW.
Give me pardon, sir. If, sir, you come with news from the court, I take
it there’s but two ways, either to utter them, or conceal them. I am,
sir, under the King, in some authority.

PISTOL.
Under which king, Besonian? Speak, or die.

SHALLOW.
Under King Harry.

PISTOL.
Harry the Fourth, or Fifth?

SHALLOW.
Harry the Fourth.

PISTOL.
A foutre for thine office!
Sir John, thy tender lambkin now is King;
Harry the Fifth’s the man. I speak the truth.
When Pistol lies, do this, and fig me, like
The bragging Spaniard.

FALSTAFF.
What, is the old King dead?

PISTOL.
As nail in door. The things I speak are just.

FALSTAFF.
Away, Bardolph, saddle my horse. Master Robert Shallow, choose what
office thou wilt in the land, ’tis thine. Pistol, I will double-charge
thee with dignities.

BARDOLPH.
O joyful day!
I would not take a knighthood for my fortune.

PISTOL.
What! I do bring good news.

FALSTAFF.
Carry Master Silence to bed. Master Shallow, my Lord Shallow, be what
thou wilt; I am Fortune’s steward! Get on thy boots, we’ll ride all
night. O sweet Pistol! Away, Bardolph!

 [_Exit Bardolph._]

Come, Pistol, utter more to me; and withal devise something to do
thyself good. Boot, boot, Master Shallow. I know the young King is sick
for me. Let us take any man’s horses. The laws of England are at my
commandment. Blessed are they that have been my friends, and woe to my
Lord Chief Justice!

PISTOL.
Let vultures vile seize on his lungs also!
“Where is the life that late I led?” say they:
Why, here it is; welcome these pleasant days!

 [_Exeunt._]

SCENE IV. London. A street.

Enter Beadles, dragging in Hostess Quickly and Doll Tearsheet.

HOSTESS.
No, thou arrant knave. I would to God that I might die, that I might
have thee hanged. Thou hast drawn my shoulder out of joint.

FIRST BEADLE.
The constables have delivered her over to me, and she shall have
whipping-cheer enough, I warrant her. There hath been a man or two
lately killed about her.

DOLL.
Nut-hook, nut-hook, you lie! Come on, I’ll tell thee what, thou damned
tripe-visaged rascal, an the child I now go with do miscarry, thou wert
better thou hadst struck thy mother, thou paper-faced villain.

HOSTESS.
O the Lord, that Sir John were come! He would make this a bloody day to
somebody. But I pray God the fruit of her womb miscarry!

FIRST BEADLE.
If it do, you shall have a dozen of cushions again; you have but eleven
now. Come, I charge you both go with me, for the man is dead that you
and Pistol beat amongst you.

DOLL.
I’ll tell you what, you thin man in a censer, I will have you as
soundly swinged for this, you bluebottle rogue, you filthy famished
correctioner, if you be not swinged, I’ll forswear half-kirtles.

FIRST BEADLE.
Come, come, you she knight-errant, come.

HOSTESS.
O God, that right should thus overcome might! Well, of sufferance comes
ease.

DOLL.
Come, you rogue, come, bring me to a justice.

HOSTESS.
Ay, come, you starved bloodhound.

DOLL.
Goodman death, goodman bones!

HOSTESS.
Thou atomy, thou!

DOLL.
Come, you thin thing, come, you rascal!

FIRST BEADLE.
Very well.

 [_Exeunt._]

SCENE V. A public place near Westminster Abbey.

Enter two Grooms, strewing rushes.

FIRST GROOM.
More rushes, more rushes.

SECOND GROOM.
The trumpets have sounded twice.

FIRST GROOM.
’Twill be two o’clock ere they come from the coronation. Dispatch,
dispatch.

 [_Exeunt._]

Trumpets sound, and the King and his train pass over the stage. Enter
Falstaff, Shallow, Pistol, Bardolph and Page.

FALSTAFF.
Stand here by me, Master Robert Shallow. I will make the King do you
grace. I will leer upon him as he comes by, and do but mark the
countenance that he will give me.

PISTOL.
God bless thy lungs, good knight!

FALSTAFF.
Come here, Pistol, stand behind me. O, if I had had time to have made
new liveries, I would have bestowed the thousand pound I borrowed of
you. But ’tis no matter, this poor show doth better. This doth infer
the zeal I had to see him.

SHALLOW.
It doth so.

FALSTAFF.
It shows my earnestness of affection—

SHALLOW.
It doth so.

FALSTAFF.
My devotion—

SHALLOW.
It doth, it doth, it doth.

FALSTAFF.
As it were, to ride day and night, and not to deliberate, not to
remember, not to have patience to shift me—

SHALLOW.
It is best, certain.

FALSTAFF.
But to stand stained with travel, and sweating with desire to see him,
thinking of nothing else, putting all affairs else in oblivion, as if
there were nothing else to be done but to see him.

PISTOL.
’Tis _semper idem_, for _obsque hoc nihil est;_ ’tis all in every part.

SHALLOW.
’Tis so, indeed.

PISTOL.
My knight, I will inflame thy noble liver,
And make thee rage.
Thy Doll, and Helen of thy noble thoughts,
Is in base durance and contagious prison,
Haled thither
By most mechanical and dirty hand.
Rouse up revenge from ebon den with fell Alecto’s snake,
For Doll is in. Pistol speaks nought but truth.

FALSTAFF.
I will deliver her.

 [_Shouts within. The trumpets sound._]

PISTOL.
There roar’d the sea, and trumpet-clangor sounds.

Enter the King and his train, the Lord Chief Justice among them.

FALSTAFF.
God save thy Grace, King Hal, my royal Hal!

PISTOL.
The heavens thee guard and keep, most royal imp of fame!

FALSTAFF.
God save thee, my sweet boy!

KING.
My Lord Chief Justice, speak to that vain man.

CHIEF JUSTICE.
Have you your wits? Know you what ’tis you speak?

FALSTAFF.
My King! My Jove! I speak to thee, my heart!

KING.
I know thee not, old man. Fall to thy prayers.
How ill white hairs becomes a fool and jester!
I have long dreamt of such a kind of man,
So surfeit-swell’d, so old, and so profane;
But, being awaked, I do despise my dream.
Make less thy body hence, and more thy grace;
Leave gormandizing; know the grave doth gape
For thee thrice wider than for other men.
Reply not to me with a fool-born jest.
Presume not that I am the thing I was;
For God doth know, so shall the world perceive,
That I have turn’d away my former self;
So will I those that kept me company.
When thou dost hear I am as I have been,
Approach me, and thou shalt be as thou wast,
The tutor and the feeder of my riots.
Till then I banish thee, on pain of death,
As I have done the rest of my misleaders,
Not to come near our person by ten mile.
For competence of life I will allow you,
That lack of means enforce you not to evils.
And, as we hear you do reform yourselves,
We will, according to your strengths and qualities,
Give you advancement. Be it your charge, my lord,
To see perform’d the tenor of our word.
Set on.

 [_Exeunt King with his train._]

FALSTAFF.
Master Shallow, I owe you a thousand pounds.

SHALLOW.
Yea, marry, Sir John, which I beseech you to let me have home with me.

FALSTAFF.
That can hardly be, Master Shallow. Do not you grieve at this; I shall
be sent for in private to him. Look you, he must seem thus to the
world. Fear not your advancements; I will be the man yet that shall
make you great.

SHALLOW.
I cannot perceive how, unless you give me your doublet and stuff me out
with straw. I beseech you, good Sir John, let me have five hundred of
my thousand.

FALSTAFF.
Sir, I will be as good as my word. This that you heard was but a
colour.

SHALLOW.
A colour that I fear you will die in, Sir John.

FALSTAFF.
Fear no colours. Go with me to dinner. Come, Lieutenant Pistol; come,
Bardolph. I shall be sent for soon at night.

Enter the Lord Chief Justice and Prince John, Officers with them.

CHIEF JUSTICE.
Go, carry Sir John Falstaff to the Fleet.
Take all his company along with him.

FALSTAFF.
My lord, my lord,—

CHIEF JUSTICE.
I cannot now speak. I will hear you soon.
Take them away.

PISTOL.
_Si fortuna me tormenta, spero me contenta._

 [_Exeunt all but Prince John and the Lord Chief Justice._]

LANCASTER.
I like this fair proceeding of the King’s.
He hath intent his wonted followers
Shall all be very well provided for,
But all are banish’d till their conversations
Appear more wise and modest to the world.

CHIEF JUSTICE.
And so they are.

LANCASTER.
The King hath call’d his parliament, my lord.

CHIEF JUSTICE.
He hath.

LANCASTER.
I will lay odds that, ere this year expire,
We bear our civil swords and native fire
As far as France. I heard a bird so sing,
Whose music, to my thinking, pleased the King.
Come, will you hence?

 [_Exeunt._]




EPILOGUE.


First my fear; then my curtsy; last my speech. My fear is your
displeasure; my curtsy, my duty; and my speech, to beg your pardons. If
you look for a good speech now, you undo me, for what I have to say is
of mine own making; and what indeed I should say will, I doubt, prove
mine own marring. But to the purpose, and so to the venture. Be it
known to you, as it is very well, I was lately here in the end of a
displeasing play, to pray your patience for it and to promise you a
better. I meant indeed to pay you with this; which, if like an ill
venture it come unluckily home, I break, and you, my gentle creditors,
lose. Here I promised you I would be, and here I commit my body to your
mercies. Bate me some, and I will pay you some, and, as most debtors
do, promise you infinitely.

If my tongue cannot entreat you to acquit me, will you command me to
use my legs? And yet that were but light payment, to dance out of your
debt. But a good conscience will make any possible satisfaction, and so
would I. All the gentlewomen here have forgiven me; if the gentlemen
will not, then the gentlemen do not agree with the gentlewomen, which
was never seen before in such an assembly.

One word more, I beseech you. If you be not too much cloyed with fat
meat, our humble author will continue the story, with Sir John in it,
and make you merry with fair Katharine of France; where, for anything I
know, Falstaff shall die of a sweat, unless already he be killed with
your hard opinions; for Oldcastle died a martyr, and this is not the
man. My tongue is weary; when my legs are too, I will bid you good
night.




THE LIFE OF KING HENRY THE FIFTH


Contents

ACT I
Prologue.
Scene I. London. An ante-chamber in the King’s palace.
Scene II. The same. The presence chamber.

ACT II
Chorus.
Scene I. London. A street.
Scene II. Southampton. A council-chamber.
Scene III. London. Before a tavern.
Scene IV. France. The King’s palace.

ACT III
Chorus.
Scene I. France. Before Harfleur.
Scene II. The same.
Scene III. Before the gates.
Scene IV. The French King’s palace.
Scene V. The same.
Scene VI. The English camp in Picardy.
Scene VII. The French camp, near Agincourt.

ACT IV
Chorus.
Scene I. The English camp at Agincourt.
Scene II. The French camp.
Scene III. The English camp.
Scene IV. The field of battle.
Scene V. Another part of the field.
Scene VI. Another part of the field.
Scene VII. Another part of the field.
Scene VIII. Before King Henry’s pavilion.

ACT V
Chorus.
Scene I. France. The English camp.
Scene II. France. A royal palace.
Epilogue.


Dramatis Personæ

KING HENRY V.
DUKE OF CLARENCE, brother to the King.
DUKE OF BEDFORD, brother to the King.
DUKE OF GLOUCESTER, brother to the King.
DUKE OF EXETER, uncle to the King.
DUKE OF YORK, cousin to the King.
EARL OF SALISBURY.
EARL OF HUNTINGDON.
EARL OF WESTMORLAND.
EARL OF WARWICK.
ARCHBISHOP OF CANTERBURY.
BISHOP OF ELY.
EARL OF CAMBRIDGE.
LORD SCROOP.
SIR THOMAS GREY.
SIR THOMAS ERPINGHAM, officer in King Henry’s army.
GOWER, officer in King Henry’s army.
FLUELLEN, officer in King Henry’s army.
MACMORRIS, officer in King Henry’s army.
JAMY, officer in King Henry’s army.
BATES, soldier in the same.
COURT, soldier in the same.
WILLIAMS, soldier in the same.
PISTOL.
NYM.
BARDOLPH.
BOY.
A Herald.

CHARLES VI, king of France.
LEWIS, the Dauphin.
DUKE OF BERRY.
DUKE OF BRITTANY.
DUKE OF BURGUNDY.
DUKE OF ORLEANS.
DUKE OF BOURBON.
The Constable of France.
RAMBURES, French Lord.
GRANDPRÉ, French Lord.
Governor of Harfleur
MONTJOY, a French herald.
Ambassadors to the King of England.

ISABEL, queen of France.
KATHARINE, daughter to Charles and Isabel.
ALICE, a lady attending on her.
HOSTESS of a tavern in Eastcheap, formerly Mistress Nell Quickly, and
now married to Pistol.

CHORUS.

Lords, Ladies, Officers, Soldiers, Citizens, Messengers, and
Attendants.

SCENE: England; afterwards France.




PROLOGUE.

 Enter Chorus.

CHORUS.
O for a Muse of fire, that would ascend
The brightest heaven of invention,
A kingdom for a stage, princes to act,
And monarchs to behold the swelling scene!
Then should the warlike Harry, like himself,
Assume the port of Mars, and at his heels,
Leash’d in like hounds, should famine, sword, and fire
Crouch for employment. But pardon, gentles all,
The flat unraised spirits that hath dar’d
On this unworthy scaffold to bring forth
So great an object. Can this cockpit hold
The vasty fields of France? Or may we cram
Within this wooden O the very casques
That did affright the air at Agincourt?
O pardon! since a crooked figure may
Attest in little place a million,
And let us, ciphers to this great accompt,
On your imaginary forces work.
Suppose within the girdle of these walls
Are now confin’d two mighty monarchies,
Whose high upreared and abutting fronts
The perilous narrow ocean parts asunder;
Piece out our imperfections with your thoughts.
Into a thousand parts divide one man,
And make imaginary puissance.
Think, when we talk of horses, that you see them
Printing their proud hoofs i’ th’ receiving earth.
For ’tis your thoughts that now must deck our kings,
Carry them here and there, jumping o’er times,
Turning the accomplishment of many years
Into an hour-glass: for the which supply,
Admit me Chorus to this history;
Who prologue-like your humble patience pray,
Gently to hear, kindly to judge, our play.

 [_Exit._]




ACT I

SCENE I. London. An ante-chamber in the King’s palace.


 Enter the Archbishop of Canterbury and the Bishop of Ely.

CANTERBURY.
My lord, I’ll tell you, that self bill is urg’d
Which in the eleventh year of the last king’s reign
Was like, and had indeed against us passed
But that the scambling and unquiet time
Did push it out of farther question.

ELY.
But how, my lord, shall we resist it now?

CANTERBURY.
It must be thought on. If it pass against us,
We lose the better half of our possession:
For all the temporal lands, which men devout
By testament have given to the Church,
Would they strip from us; being valu’d thus:
As much as would maintain, to the King’s honour,
Full fifteen earls and fifteen hundred knights,
Six thousand and two hundred good esquires;
And, to relief of lazars and weak age,
Of indigent faint souls past corporal toil,
A hundred almshouses right well supplied;
And to the coffers of the King beside,
A thousand pounds by th’ year. Thus runs the bill.

ELY.
This would drink deep.

CANTERBURY.
’Twould drink the cup and all.

ELY.
But what prevention?

CANTERBURY.
The King is full of grace and fair regard.

ELY.
And a true lover of the holy Church.

CANTERBURY.
The courses of his youth promis’d it not.
The breath no sooner left his father’s body
But that his wildness, mortified in him,
Seemed to die too; yea, at that very moment
Consideration like an angel came
And whipped th’ offending Adam out of him,
Leaving his body as a paradise
T’ envelope and contain celestial spirits.
Never was such a sudden scholar made,
Never came reformation in a flood
With such a heady currance scouring faults,
Nor never Hydra-headed wilfulness
So soon did lose his seat, and all at once,
As in this king.

ELY.
We are blessed in the change.

CANTERBURY.
Hear him but reason in divinity
And, all-admiring, with an inward wish
You would desire the King were made a prelate;
Hear him debate of commonwealth affairs,
You would say it hath been all in all his study;
List his discourse of war, and you shall hear
A fearful battle rendered you in music;
Turn him to any cause of policy,
The Gordian knot of it he will unloose,
Familiar as his garter; that, when he speaks,
The air, a chartered libertine, is still,
And the mute wonder lurketh in men’s ears
To steal his sweet and honeyed sentences;
So that the art and practic part of life
Must be the mistress to this theoric:
Which is a wonder how his Grace should glean it,
Since his addiction was to courses vain,
His companies unlettered, rude, and shallow,
His hours filled up with riots, banquets, sports,
And never noted in him any study,
Any retirement, any sequestration
From open haunts and popularity.

ELY.
The strawberry grows underneath the nettle,
And wholesome berries thrive and ripen best
Neighboured by fruit of baser quality;
And so the Prince obscured his contemplation
Under the veil of wildness, which, no doubt,
Grew like the summer grass, fastest by night,
Unseen, yet crescive in his faculty.

CANTERBURY.
It must be so, for miracles are ceased,
And therefore we must needs admit the means
How things are perfected.

ELY.
But, my good lord,
How now for mitigation of this bill
Urged by the Commons? Doth his Majesty
Incline to it, or no?

CANTERBURY.
He seems indifferent,
Or rather swaying more upon our part
Than cherishing th’ exhibitors against us;
For I have made an offer to his Majesty,
Upon our spiritual convocation
And in regard of causes now in hand,
Which I have opened to his Grace at large,
As touching France, to give a greater sum
Than ever at one time the clergy yet
Did to his predecessors part withal.

ELY.
How did this offer seem received, my lord?

CANTERBURY.
With good acceptance of his Majesty;
Save that there was not time enough to hear,
As I perceived his Grace would fain have done,
The severals and unhidden passages
Of his true titles to some certain dukedoms,
And generally to the crown and seat of France,
Derived from Edward, his great-grandfather.

ELY.
What was th’ impediment that broke this off?

CANTERBURY.
The French ambassador upon that instant
Craved audience; and the hour, I think, is come
To give him hearing. Is it four o’clock?

ELY.
It is.

CANTERBURY.
Then go we in, to know his embassy,
Which I could with a ready guess declare
Before the Frenchman speak a word of it.

ELY.
I’ll wait upon you, and I long to hear it.

 [_Exeunt._]

SCENE II. The same. The presence chamber.

 Enter King Henry, Gloucester, Bedford, Clarence, Warwick, Westmorland,
 Exeter and Attendants.

KING HENRY.
Where is my gracious Lord of Canterbury?

EXETER.
Not here in presence.

KING HENRY.
Send for him, good uncle.

WESTMORLAND.
Shall we call in th’ ambassador, my liege?

KING HENRY.
Not yet, my cousin. We would be resolved,
Before we hear him, of some things of weight
That task our thoughts concerning us and France.

 Enter the Archbishop of Canterbury and the Bishop of Ely.

CANTERBURY.
God and his angels guard your sacred throne
And make you long become it!

KING HENRY.
Sure, we thank you.
My learned lord, we pray you to proceed
And justly and religiously unfold
Why the law Salic that they have in France
Or should or should not bar us in our claim.
And God forbid, my dear and faithful lord,
That you should fashion, wrest, or bow your reading,
Or nicely charge your understanding soul
With opening titles miscreate, whose right
Suits not in native colours with the truth;
For God doth know how many now in health
Shall drop their blood in approbation
Of what your reverence shall incite us to.
Therefore take heed how you impawn our person,
How you awake our sleeping sword of war.
We charge you in the name of God, take heed;
For never two such kingdoms did contend
Without much fall of blood, whose guiltless drops
Are every one a woe, a sore complaint
’Gainst him whose wrongs gives edge unto the swords
That makes such waste in brief mortality.
Under this conjuration speak, my lord,
For we will hear, note, and believe in heart
That what you speak is in your conscience washed
As pure as sin with baptism.

CANTERBURY.
Then hear me, gracious sovereign, and you peers,
That owe yourselves, your lives, and services
To this imperial throne. There is no bar
To make against your Highness’ claim to France
But this, which they produce from Pharamond:
_In terram Salicam mulieres ne succedant_,
“No woman shall succeed in Salic land;”
Which Salic land the French unjustly gloze
To be the realm of France, and Pharamond
The founder of this law and female bar.
Yet their own authors faithfully affirm
That the land Salic is in Germany,
Between the floods of Sala and of Elbe;
Where Charles the Great, having subdu’d the Saxons,
There left behind and settled certain French;
Who, holding in disdain the German women
For some dishonest manners of their life,
Establish’d then this law, to wit, no female
Should be inheritrix in Salic land;
Which Salic, as I said, ’twixt Elbe and Sala,
Is at this day in Germany call’d Meissen.
Then doth it well appear the Salic law
Was not devised for the realm of France;
Nor did the French possess the Salic land
Until four hundred one and twenty years
After defunction of King Pharamond,
Idly suppos’d the founder of this law,
Who died within the year of our redemption
Four hundred twenty-six; and Charles the Great
Subdu’d the Saxons, and did seat the French
Beyond the river Sala, in the year
Eight hundred five. Besides, their writers say,
King Pepin, which deposed Childeric,
Did, as heir general, being descended
Of Blithild, which was daughter to King Clothair,
Make claim and title to the crown of France.
Hugh Capet also, who usurp’d the crown
Of Charles the Duke of Lorraine, sole heir male
Of the true line and stock of Charles the Great,
To find his title with some shows of truth,
Though, in pure truth, it was corrupt and naught,
Convey’d himself as the heir to the Lady Lingare,
Daughter to Charlemain, who was the son
To Lewis the Emperor, and Lewis the son
Of Charles the Great. Also, King Lewis the Tenth,
Who was sole heir to the usurper Capet,
Could not keep quiet in his conscience,
Wearing the crown of France, till satisfied
That fair Queen Isabel, his grandmother,
Was lineal of the Lady Ermengare,
Daughter to Charles, the foresaid Duke of Lorraine;
By the which marriage the line of Charles the Great
Was re-united to the crown of France.
So that, as clear as is the summer’s sun,
King Pepin’s title and Hugh Capet’s claim,
King Lewis his satisfaction, all appear
To hold in right and title of the female.
So do the kings of France unto this day,
Howbeit they would hold up this Salic law
To bar your Highness claiming from the female,
And rather choose to hide them in a net
Than amply to imbar their crooked titles
Usurp’d from you and your progenitors.

KING HENRY.
May I with right and conscience make this claim?

CANTERBURY.
The sin upon my head, dread sovereign!
For in the Book of Numbers is it writ,
“When the man dies, let the inheritance
Descend unto the daughter.” Gracious lord,
Stand for your own! Unwind your bloody flag!
Look back into your mighty ancestors!
Go, my dread lord, to your great-grandsire’s tomb,
From whom you claim; invoke his warlike spirit,
And your great-uncle’s, Edward the Black Prince,
Who on the French ground play’d a tragedy,
Making defeat on the full power of France,
Whiles his most mighty father on a hill
Stood smiling to behold his lion’s whelp
Forage in blood of French nobility.
O noble English, that could entertain
With half their forces the full pride of France
And let another half stand laughing by,
All out of work and cold for action!

ELY.
Awake remembrance of these valiant dead,
And with your puissant arm renew their feats.
You are their heir; you sit upon their throne;
The blood and courage that renowned them
Runs in your veins; and my thrice-puissant liege
Is in the very May-morn of his youth,
Ripe for exploits and mighty enterprises.

EXETER.
Your brother kings and monarchs of the earth
Do all expect that you should rouse yourself,
As did the former lions of your blood.

WESTMORLAND.
They know your Grace hath cause and means and might;
So hath your Highness. Never King of England
Had nobles richer, and more loyal subjects,
Whose hearts have left their bodies here in England
And lie pavilion’d in the fields of France.

CANTERBURY.
O, let their bodies follow, my dear liege,
With blood and sword and fire to win your right;
In aid whereof we of the spiritualty
Will raise your Highness such a mighty sum
As never did the clergy at one time
Bring in to any of your ancestors.

KING HENRY.
We must not only arm to invade the French,
But lay down our proportions to defend
Against the Scot, who will make road upon us
With all advantages.

CANTERBURY.
They of those marches, gracious sovereign,
Shall be a wall sufficient to defend
Our inland from the pilfering borderers.

KING HENRY.
We do not mean the coursing snatchers only,
But fear the main intendment of the Scot,
Who hath been still a giddy neighbour to us;
For you shall read that my great-grandfather
Never went with his forces into France
But that the Scot on his unfurnish’d kingdom
Came pouring, like the tide into a breach,
With ample and brim fullness of his force,
Galling the gleaned land with hot assays,
Girdling with grievous siege castles and towns;
That England, being empty of defence,
Hath shook and trembled at the ill neighbourhood.

CANTERBURY.
She hath been then more fear’d than harm’d, my liege;
For hear her but exampl’d by herself:
When all her chivalry hath been in France,
And she a mourning widow of her nobles,
She hath herself not only well defended
But taken and impounded as a stray
The King of Scots; whom she did send to France
To fill King Edward’s fame with prisoner kings,
And make her chronicle as rich with praise
As is the ooze and bottom of the sea
With sunken wreck and sumless treasuries.

WESTMORLAND.
But there’s a saying very old and true,
“If that you will France win,
Then with Scotland first begin.”
For once the eagle England being in prey,
To her unguarded nest the weasel Scot
Comes sneaking and so sucks her princely eggs,
Playing the mouse in absence of the cat,
To tear and havoc more than she can eat.

EXETER.
It follows then the cat must stay at home;
Yet that is but a crush’d necessity,
Since we have locks to safeguard necessaries,
And pretty traps to catch the petty thieves.
While that the armed hand doth fight abroad,
The advised head defends itself at home;
For government, though high and low and lower,
Put into parts, doth keep in one consent,
Congreeing in a full and natural close,
Like music.

CANTERBURY.
Therefore doth heaven divide
The state of man in divers functions,
Setting endeavour in continual motion,
To which is fixed, as an aim or butt,
Obedience; for so work the honey-bees,
Creatures that by a rule in nature teach
The act of order to a peopled kingdom.
They have a king and officers of sorts,
Where some, like magistrates, correct at home,
Others like merchants, venture trade abroad,
Others, like soldiers, armed in their stings,
Make boot upon the summer’s velvet buds,
Which pillage they with merry march bring home
To the tent-royal of their emperor;
Who, busied in his majesty, surveys
The singing masons building roofs of gold,
The civil citizens kneading up the honey,
The poor mechanic porters crowding in
Their heavy burdens at his narrow gate,
The sad-eyed justice, with his surly hum,
Delivering o’er to executors pale
The lazy yawning drone. I this infer,
That many things, having full reference
To one consent, may work contrariously.
As many arrows, loosed several ways,
Come to one mark; as many ways meet in one town;
As many fresh streams meet in one salt sea;
As many lines close in the dial’s centre;
So many a thousand actions, once afoot,
End in one purpose, and be all well borne
Without defeat. Therefore to France, my liege!
Divide your happy England into four,
Whereof take you one quarter into France,
And you withal shall make all Gallia shake.
If we, with thrice such powers left at home,
Cannot defend our own doors from the dog,
Let us be worried and our nation lose
The name of hardiness and policy.

KING HENRY.
Call in the messengers sent from the Dauphin.

 [_Exeunt some Attendants._]

Now are we well resolv’d; and, by God’s help,
And yours, the noble sinews of our power,
France being ours, we’ll bend it to our awe,
Or break it all to pieces. Or there we’ll sit,
Ruling in large and ample empery
O’er France and all her almost kingly dukedoms,
Or lay these bones in an unworthy urn,
Tombless, with no remembrance over them.
Either our history shall with full mouth
Speak freely of our acts, or else our grave,
Like Turkish mute, shall have a tongueless mouth,
Not worshipp’d with a waxen epitaph.

 Enter Ambassadors of France.

Now are we well prepar’d to know the pleasure
Of our fair cousin Dauphin; for we hear
Your greeting is from him, not from the King.

FIRST AMBASSADOR.
May’t please your Majesty to give us leave
Freely to render what we have in charge,
Or shall we sparingly show you far off
The Dauphin’s meaning and our embassy?

KING HENRY.
We are no tyrant, but a Christian king,
Unto whose grace our passion is as subject
As is our wretches fett’red in our prisons;
Therefore with frank and with uncurbed plainness
Tell us the Dauphin’s mind.

AMBASSADOR.
Thus, then, in few.
Your Highness, lately sending into France,
Did claim some certain dukedoms, in the right
Of your great predecessor, King Edward the Third.
In answer of which claim, the prince our master
Says that you savour too much of your youth,
And bids you be advis’d there’s nought in France
That can be with a nimble galliard won.
You cannot revel into dukedoms there.
He therefore sends you, meeter for your spirit,
This tun of treasure; and, in lieu of this,
Desires you let the dukedoms that you claim
Hear no more of you. This the Dauphin speaks.

KING HENRY.
What treasure, uncle?

EXETER.
Tennis-balls, my liege.

KING HENRY.
We are glad the Dauphin is so pleasant with us.
His present and your pains we thank you for.
When we have match’d our rackets to these balls,
We will, in France, by God’s grace, play a set
Shall strike his father’s crown into the hazard.
Tell him he hath made a match with such a wrangler
That all the courts of France will be disturb’d
With chaces. And we understand him well,
How he comes o’er us with our wilder days,
Not measuring what use we made of them.
We never valu’d this poor seat of England;
And therefore, living hence, did give ourself
To barbarous licence; as ’tis ever common
That men are merriest when they are from home.
But tell the Dauphin I will keep my state,
Be like a king, and show my sail of greatness
When I do rouse me in my throne of France.
For that I have laid by my majesty
And plodded like a man for working days,
But I will rise there with so full a glory
That I will dazzle all the eyes of France,
Yea, strike the Dauphin blind to look on us.
And tell the pleasant prince this mock of his
Hath turn’d his balls to gun-stones, and his soul
Shall stand sore charged for the wasteful vengeance
That shall fly with them; for many a thousand widows
Shall this his mock mock out of their dear husbands,
Mock mothers from their sons, mock castles down;
And some are yet ungotten and unborn
That shall have cause to curse the Dauphin’s scorn.
But this lies all within the will of God,
To whom I do appeal; and in whose name
Tell you the Dauphin I am coming on
To venge me as I may, and to put forth
My rightful hand in a well-hallow’d cause.
So get you hence in peace; and tell the Dauphin
His jest will savour but of shallow wit,
When thousands weep more than did laugh at it.—
Convey them with safe conduct.—Fare you well.

 [_Exeunt Ambassadors._]

EXETER.
This was a merry message.

KING HENRY.
We hope to make the sender blush at it.
Therefore, my lords, omit no happy hour
That may give furtherance to our expedition;
For we have now no thought in us but France,
Save those to God, that run before our business.
Therefore, let our proportions for these wars
Be soon collected, and all things thought upon
That may with reasonable swiftness add
More feathers to our wings; for, God before,
We’ll chide this Dauphin at his father’s door.
Therefore let every man now task his thought,
That this fair action may on foot be brought.

 [_Exeunt._]




ACT II


 Flourish. Enter Chorus.

CHORUS.
Now all the youth of England are on fire,
And silken dalliance in the wardrobe lies.
Now thrive the armourers, and honour’s thought
Reigns solely in the breast of every man.
They sell the pasture now to buy the horse,
Following the mirror of all Christian kings,
With winged heels, as English Mercuries.
For now sits Expectation in the air,
And hides a sword from hilts unto the point
With crowns imperial, crowns, and coronets,
Promis’d to Harry and his followers.
The French, advis’d by good intelligence
Of this most dreadful preparation,
Shake in their fear, and with pale policy
Seek to divert the English purposes.
O England! model to thy inward greatness,
Like little body with a mighty heart,
What mightst thou do, that honour would thee do,
Were all thy children kind and natural!
But see thy fault! France hath in thee found out
A nest of hollow bosoms, which he fills
With treacherous crowns; and three corrupted men,
One, Richard Earl of Cambridge, and the second,
Henry Lord Scroop of Masham, and the third,
Sir Thomas Grey, knight of Northumberland,
Have, for the gilt of France,—O guilt indeed!—
Confirm’d conspiracy with fearful France;
And by their hands this grace of kings must die,
If hell and treason hold their promises,
Ere he take ship for France, and in Southampton.
Linger your patience on, and we’ll digest
The abuse of distance, force a play.
The sum is paid; the traitors are agreed;
The King is set from London; and the scene
Is now transported, gentles, to Southampton.
There is the playhouse now, there must you sit;
And thence to France shall we convey you safe,
And bring you back, charming the narrow seas
To give you gentle pass; for, if we may,
We’ll not offend one stomach with our play.
But, till the King come forth, and not till then,
Unto Southampton do we shift our scene.

 [_Exit._]

SCENE I. London. A street.

 Enter Corporal Nym and Lieutenant Bardolph.

BARDOLPH.
Well met, Corporal Nym.

NYM.
Good morrow, Lieutenant Bardolph.

BARDOLPH.
What, are Ancient Pistol and you friends yet?

NYM.
For my part, I care not. I say little; but when time shall serve, there
shall be smiles; but that shall be as it may. I dare not fight, but I
will wink and hold out mine iron. It is a simple one, but what though?
It will toast cheese, and it will endure cold as another man’s sword
will; and there’s an end.

BARDOLPH.
I will bestow a breakfast to make you friends; and we’ll be all three
sworn brothers to France. Let it be so, good Corporal Nym.

NYM.
Faith, I will live so long as I may, that’s the certain of it; and when
I cannot live any longer, I will do as I may. That is my rest, that is
the rendezvous of it.

BARDOLPH.
It is certain, corporal, that he is married to Nell Quickly; and
certainly she did you wrong, for you were troth-plight to her.

NYM.
I cannot tell. Things must be as they may. Men may sleep, and they may
have their throats about them at that time; and some say knives have
edges. It must be as it may. Though patience be a tired mare, yet she
will plod. There must be conclusions. Well, I cannot tell.

 Enter Pistol and Hostess.

BARDOLPH.
Here comes Ancient Pistol and his wife. Good Corporal, be patient here.
How now, mine host Pistol!

PISTOL.
Base tike, call’st thou me host?
Now, by this hand, I swear I scorn the term;
Nor shall my Nell keep lodgers.

HOSTESS.
No, by my troth, not long; for we cannot lodge and board a dozen or
fourteen gentlewomen that live honestly by the prick of their needles,
but it will be thought we keep a bawdy house straight. [_Nym and Pistol
draw._] O well a day, Lady, if he be not drawn now! We shall see wilful
adultery and murder committed.

BARDOLPH.
Good Lieutenant! good corporal! offer nothing here.

NYM.
Pish!

PISTOL.
Pish for thee, Iceland dog! thou prick-ear’d cur of Iceland!

HOSTESS.
Good Corporal Nym, show thy valour, and put up your sword.

NYM.
Will you shog off? I would have you _solus_.

PISTOL.
_Solus_, egregious dog! O viper vile!
The _solus_ in thy most mervailous face;
The _solus_ in thy teeth, and in thy throat,
And in thy hateful lungs, yea, in thy maw, perdy,
And, which is worse, within thy nasty mouth!
I do retort the _solus_ in thy bowels;
For I can take, and Pistol’s cock is up,
And flashing fire will follow.

NYM.
I am not Barbason; you cannot conjure me. I have an humour to knock you
indifferently well. If you grow foul with me, Pistol, I will scour you
with my rapier, as I may, in fair terms. If you would walk off, I would
prick your guts a little, in good terms, as I may; and that’s the
humour of it.

PISTOL.
O braggart vile and damned furious wight!
The grave doth gape, and doting death is near,
Therefore exhale.

BARDOLPH.
Hear me, hear me what I say. He that strikes the first stroke I’ll run
him up to the hilts, as I am a soldier.

 [_Draws._]

PISTOL.
An oath of mickle might; and fury shall abate.
Give me thy fist, thy fore-foot to me give.
Thy spirits are most tall.

NYM.
I will cut thy throat, one time or other, in fair terms: that is the
humour of it.

PISTOL.
“Couple a gorge!”
That is the word. I thee defy again.
O hound of Crete, think’st thou my spouse to get?
No! to the spital go,
And from the powdering tub of infamy
Fetch forth the lazar kite of Cressid’s kind,
Doll Tearsheet she by name, and her espouse.
I have, and I will hold, the quondam Quickly
For the only she; and _pauca_, there’s enough.
Go to.

 Enter the Boy.

BOY.
Mine host Pistol, you must come to my master, and you, hostess. He is
very sick, and would to bed. Good Bardolph, put thy face between his
sheets, and do the office of a warming-pan. Faith, he’s very ill.

BARDOLPH.
Away, you rogue!

HOSTESS.
By my troth, he’ll yield the crow a pudding one of these days.
The King has kill’d his heart.
Good husband, come home presently.

 [_Exeunt Hostess and Boy._]

BARDOLPH.
Come, shall I make you two friends? We must to France together; why the
devil should we keep knives to cut one another’s throats?

PISTOL.
Let floods o’erswell, and fiends for food howl on!

NYM.
You’ll pay me the eight shillings I won of you at betting?

PISTOL.
Base is the slave that pays.

NYM.
That now I will have: that’s the humour of it.

PISTOL.
As manhood shall compound. Push home.

 [_They draw._]

BARDOLPH.
By this sword, he that makes the first thrust, I’ll kill him; by this
sword, I will.

PISTOL.
Sword is an oath, and oaths must have their course.

BARDOLPH.
Corporal Nym, and thou wilt be friends, be friends; an thou wilt not,
why, then, be enemies with me too. Prithee, put up.

NYM.
I shall have my eight shillings I won from you at betting?

PISTOL.
A noble shalt thou have, and present pay;
And liquor likewise will I give to thee,
And friendship shall combine, and brotherhood.
I’ll live by Nym, and Nym shall live by me.
Is not this just? For I shall sutler be
Unto the camp, and profits will accrue.
Give me thy hand.

NYM.
I shall have my noble?

PISTOL.
In cash most justly paid.

NYM.
Well, then, that’s the humour of’t.

 Enter Hostess.

HOSTESS.
As ever you come of women, come in quickly to Sir John.
Ah, poor heart! he is so shak’d of a burning quotidian tertian,
that it is most lamentable to behold. Sweet men, come to him.

NYM.
The King hath run bad humours on the knight; that’s the even of it.

PISTOL.
Nym, thou hast spoke the right.
His heart is fracted and corroborate.

NYM.
The King is a good king; but it must be as it may; he passes some
humours and careers.

PISTOL.
Let us condole the knight; for, lambkins, we will live.

 [_Exeunt._]

SCENE II. Southampton. A council-chamber.

 Enter Exeter, Bedford and Westmorland.

BEDFORD.
’Fore God, his Grace is bold, to trust these traitors.

EXETER.
They shall be apprehended by and by.

WESTMORLAND.
How smooth and even they do bear themselves!
As if allegiance in their bosoms sat
Crowned with faith and constant loyalty.

BEDFORD.
The King hath note of all that they intend,
By interception which they dream not of.

EXETER.
Nay, but the man that was his bed-fellow,
Whom he hath dull’d and cloy’d with gracious favours,
That he should, for a foreign purse, so sell
His sovereign’s life to death and treachery.

 Trumpets sound. Enter King Henry, Scroop, Cambridge and Grey.

KING HENRY.
Now sits the wind fair, and we will aboard.
My Lord of Cambridge, and my kind Lord of Masham,
And you, my gentle knight, give me your thoughts.
Think you not that the powers we bear with us
Will cut their passage through the force of France,
Doing the execution and the act
For which we have in head assembled them?

SCROOP.
No doubt, my liege, if each man do his best.

KING HENRY.
I doubt not that, since we are well persuaded
We carry not a heart with us from hence
That grows not in a fair consent with ours,
Nor leave not one behind that doth not wish
Success and conquest to attend on us.

CAMBRIDGE.
Never was monarch better fear’d and lov’d
Than is your Majesty. There’s not, I think, a subject
That sits in heart-grief and uneasiness
Under the sweet shade of your government.

GREY.
True; those that were your father’s enemies
Have steep’d their galls in honey, and do serve you
With hearts create of duty and of zeal.

KING HENRY.
We therefore have great cause of thankfulness,
And shall forget the office of our hand
Sooner than quittance of desert and merit
According to the weight and worthiness.

SCROOP.
So service shall with steeled sinews toil,
And labour shall refresh itself with hope,
To do your Grace incessant services.

KING HENRY.
We judge no less. Uncle of Exeter,
Enlarge the man committed yesterday,
That rail’d against our person. We consider
It was excess of wine that set him on,
And on his more advice we pardon him.

SCROOP.
That’s mercy, but too much security.
Let him be punish’d, sovereign, lest example
Breed, by his sufferance, more of such a kind.

KING HENRY.
O, let us yet be merciful.

CAMBRIDGE.
So may your Highness, and yet punish too.

GREY.
Sir,
You show great mercy if you give him life
After the taste of much correction.

KING HENRY.
Alas, your too much love and care of me
Are heavy orisons ’gainst this poor wretch!
If little faults, proceeding on distemper,
Shall not be wink’d at, how shall we stretch our eye
When capital crimes, chew’d, swallow’d, and digested,
Appear before us? We’ll yet enlarge that man,
Though Cambridge, Scroop, and Grey, in their dear care
And tender preservation of our person,
Would have him punish’d. And now to our French causes.
Who are the late commissioners?

CAMBRIDGE.
I one, my lord.
Your Highness bade me ask for it today.

SCROOP.
So did you me, my liege.

GREY.
And I, my royal sovereign.

KING HENRY.
Then, Richard Earl of Cambridge, there is yours;
There yours, Lord Scroop of Masham; and, sir knight,
Grey of Northumberland, this same is yours.
Read them, and know I know your worthiness.
My Lord of Westmorland, and uncle Exeter,
We will aboard tonight.—Why, how now, gentlemen!
What see you in those papers that you lose
So much complexion?—Look ye, how they change!
Their cheeks are paper.—Why, what read you there,
That have so cowarded and chas’d your blood
Out of appearance?

CAMBRIDGE.
I do confess my fault,
And do submit me to your Highness’ mercy.

GREY, SCROOP.
To which we all appeal.

KING HENRY.
The mercy that was quick in us but late,
By your own counsel is suppress’d and kill’d.
You must not dare, for shame, to talk of mercy,
For your own reasons turn into your bosoms,
As dogs upon their masters, worrying you.
See you, my princes and my noble peers,
These English monsters! My Lord of Cambridge here,
You know how apt our love was to accord
To furnish him with an appertinents
Belonging to his honour; and this man
Hath, for a few light crowns, lightly conspir’d
And sworn unto the practices of France
To kill us here in Hampton; to the which
This knight, no less for bounty bound to us
Than Cambridge is, hath likewise sworn. But, O
What shall I say to thee, Lord Scroop? thou cruel,
Ingrateful, savage, and inhuman creature!
Thou that didst bear the key of all my counsels,
That knew’st the very bottom of my soul,
That almost mightst have coin’d me into gold,
Wouldst thou have practis’d on me for thy use,—
May it be possible that foreign hire
Could out of thee extract one spark of evil
That might annoy my finger? ’Tis so strange,
That, though the truth of it stands off as gross
As black and white, my eye will scarcely see it.
Treason and murder ever kept together,
As two yoke-devils sworn to either’s purpose,
Working so grossly in a natural cause
That admiration did not whoop at them;
But thou, ’gainst all proportion, didst bring in
Wonder to wait on treason and on murder;
And whatsoever cunning fiend it was
That wrought upon thee so preposterously
Hath got the voice in hell for excellence;
And other devils that suggest by treasons
Do botch and bungle up damnation
With patches, colours, and with forms being fetch’d
From glist’ring semblances of piety.
But he that temper’d thee bade thee stand up,
Gave thee no instance why thou shouldst do treason,
Unless to dub thee with the name of traitor.
If that same demon that hath gull’d thee thus
Should with his lion gait walk the whole world,
He might return to vasty Tartar back,
And tell the legions, “I can never win
A soul so easy as that Englishman’s.”
O, how hast thou with jealousy infected
The sweetness of affiance! Show men dutiful?
Why, so didst thou. Seem they grave and learned?
Why, so didst thou. Come they of noble family?
Why, so didst thou. Seem they religious?
Why, so didst thou. Or are they spare in diet,
Free from gross passion or of mirth or anger,
Constant in spirit, not swerving with the blood,
Garnish’d and deck’d in modest complement,
Not working with the eye without the ear,
And but in purged judgement trusting neither?
Such and so finely bolted didst thou seem.
And thus thy fall hath left a kind of blot
To mark the full-fraught man and best indued
With some suspicion. I will weep for thee;
For this revolt of thine, methinks, is like
Another fall of man. Their faults are open.
Arrest them to the answer of the law;
And God acquit them of their practices!

EXETER.
I arrest thee of high treason, by the name of Richard Earl of
Cambridge.
I arrest thee of high treason, by the name of Henry Lord Scroop of
Masham.
I arrest thee of high treason, by the name of Thomas Grey, knight, of
Northumberland.

SCROOP.
Our purposes God justly hath discover’d,
And I repent my fault more than my death,
Which I beseech your Highness to forgive,
Although my body pay the price of it.

CAMBRIDGE.
For me, the gold of France did not seduce,
Although I did admit it as a motive
The sooner to effect what I intended.
But God be thanked for prevention,
Which I in sufferance heartily will rejoice,
Beseeching God and you to pardon me.

GREY.
Never did faithful subject more rejoice
At the discovery of most dangerous treason
Than I do at this hour joy o’er myself,
Prevented from a damned enterprise.
My fault, but not my body, pardon, sovereign.

KING HENRY.
God quit you in his mercy! Hear your sentence.
You have conspir’d against our royal person,
Join’d with an enemy proclaim’d, and from his coffers
Received the golden earnest of our death;
Wherein you would have sold your king to slaughter,
His princes and his peers to servitude,
His subjects to oppression and contempt,
And his whole kingdom into desolation.
Touching our person seek we no revenge;
But we our kingdom’s safety must so tender,
Whose ruin you have sought, that to her laws
We do deliver you. Get you therefore hence,
Poor miserable wretches, to your death,
The taste whereof God of his mercy give
You patience to endure, and true repentance
Of all your dear offences! Bear them hence.

 [_Exeunt Cambridge, Scroop and Grey, guarded._]

Now, lords, for France; the enterprise whereof
Shall be to you, as us, like glorious.
We doubt not of a fair and lucky war,
Since God so graciously hath brought to light
This dangerous treason lurking in our way
To hinder our beginnings. We doubt not now
But every rub is smoothed on our way.
Then forth, dear countrymen! Let us deliver
Our puissance into the hand of God,
Putting it straight in expedition.
Cheerly to sea! The signs of war advance!
No king of England, if not king of France!

 [_Flourish. Exeunt._]

SCENE III. London. Before a tavern.

 Enter Pistol, Nym, Bardolph, Boy and Hostess.

HOSTESS.
Prithee, honey, sweet husband, let me bring thee to Staines.

PISTOL.
No; for my manly heart doth yearn.
Bardolph, be blithe; Nym, rouse thy vaunting veins;
Boy, bristle thy courage up; for Falstaff he is dead,
And we must yearn therefore.

BARDOLPH.
Would I were with him, wheresome’er he is, either in heaven or in hell!

HOSTESS.
Nay, sure, he’s not in hell. He’s in Arthur’s bosom, if ever man went
to Arthur’s bosom. ’A made a finer end and went away an it had been any
christom child. ’A parted even just between twelve and one, even at the
turning o’ the tide: for after I saw him fumble with the sheets, and
play with flowers, and smile upon his fingers’ ends, I knew there was
but one way; for his nose was as sharp as a pen, and ’a babbled of
green fields. “How now, Sir John!” quoth I; “what, man! be o’ good
cheer.” So ’a cried out, “God, God, God!” three or four times. Now I,
to comfort him, bid him ’a should not think of God; I hop’d there was
no need to trouble himself with any such thoughts yet. So ’a bade me
lay more clothes on his feet. I put my hand into the bed and felt them,
and they were as cold as any stone. Then I felt to his knees, and so
upward and upward, and all was as cold as any stone.

NYM.
They say he cried out of sack.

HOSTESS.
Ay, that ’a did.

BARDOLPH.
And of women.

HOSTESS.
Nay, that ’a did not.

BOY.
Yes, that ’a did; and said they were devils incarnate.

HOSTESS.
’A could never abide carnation; ’twas a colour he never liked.

BOY.
’A said once, the devil would have him about women.

HOSTESS.
’A did in some sort, indeed, handle women; but then he was rheumatic,
and talk’d of the whore of Babylon.

BOY.
Do you not remember, ’a saw a flea stick upon Bardolph’s nose, and ’a
said it was a black soul burning in hell-fire?

BARDOLPH.
Well, the fuel is gone that maintain’d that fire. That’s all the riches
I got in his service.

NYM.
Shall we shog? The King will be gone from Southampton.

PISTOL.
Come, let’s away. My love, give me thy lips.
Look to my chattels and my movables.
Let senses rule; the word is “Pitch and Pay.”
Trust none;
For oaths are straws, men’s faiths are wafer-cakes
And hold-fast is the only dog, my duck;
Therefore, _Caveto_ be thy counsellor.
Go, clear thy crystals. Yoke-fellows in arms,
Let us to France; like horse-leeches, my boys,
To suck, to suck, the very blood to suck!

BOY.
And that’s but unwholesome food, they say.

PISTOL.
Touch her soft mouth, and march.

BARDOLPH.
Farewell, hostess.

 [_Kissing her._]

NYM.
I cannot kiss; that is the humour of it; but, adieu.

PISTOL.
Let housewifery appear. Keep close, I thee command.

HOSTESS.
Farewell; adieu.

 [_Exeunt._]

SCENE IV. France. The King’s palace.

 Flourish. Enter the French King, the Dauphin, the Dukes of Berry and
 Brittany, the Constable and others.

FRENCH KING.
Thus comes the English with full power upon us,
And more than carefully it us concerns
To answer royally in our defences.
Therefore the Dukes of Berry and of Brittany,
Of Brabant and of Orleans, shall make forth,
And you, Prince Dauphin, with all swift dispatch,
To line and new repair our towns of war
With men of courage and with means defendant;
For England his approaches makes as fierce
As waters to the sucking of a gulf.
It fits us then to be as provident
As fears may teach us out of late examples
Left by the fatal and neglected English
Upon our fields.

DAUPHIN.
My most redoubted father,
It is most meet we arm us ’gainst the foe;
For peace itself should not so dull a kingdom,
Though war nor no known quarrel were in question,
But that defences, musters, preparations,
Should be maintain’d, assembled, and collected,
As were a war in expectation.
Therefore, I say, ’tis meet we all go forth
To view the sick and feeble parts of France.
And let us do it with no show of fear;
No, with no more than if we heard that England
Were busied with a Whitsun morris-dance;
For, my good liege, she is so idly king’d,
Her sceptre so fantastically borne
By a vain, giddy, shallow, humorous youth,
That fear attends her not.

CONSTABLE.
O peace, Prince Dauphin!
You are too much mistaken in this king.
Question your Grace the late ambassadors
With what great state he heard their embassy,
How well supplied with noble counsellors,
How modest in exception, and withal
How terrible in constant resolution,
And you shall find his vanities forespent
Were but the outside of the Roman Brutus,
Covering discretion with a coat of folly;
As gardeners do with ordure hide those roots
That shall first spring and be most delicate.

DAUPHIN.
Well, ’tis not so, my Lord High Constable;
But though we think it so, it is no matter.
In cases of defence ’tis best to weigh
The enemy more mighty than he seems,
So the proportions of defence are fill’d;
Which, of a weak and niggardly projection,
Doth, like a miser, spoil his coat with scanting
A little cloth.

FRENCH KING.
Think we King Harry strong;
And, Princes, look you strongly arm to meet him.
The kindred of him hath been flesh’d upon us;
And he is bred out of that bloody strain
That haunted us in our familiar paths.
Witness our too much memorable shame
When Cressy battle fatally was struck,
And all our princes captiv’d by the hand
Of that black name, Edward, Black Prince of Wales;
Whiles that his mountain sire, on mountain standing,
Up in the air, crown’d with the golden sun,
Saw his heroical seed, and smil’d to see him,
Mangle the work of nature and deface
The patterns that by God and by French fathers
Had twenty years been made. This is a stem
Of that victorious stock; and let us fear
The native mightiness and fate of him.

 Enter a Messenger.

MESSENGER.
Ambassadors from Harry King of England
Do crave admittance to your Majesty.

FRENCH KING.
We’ll give them present audience. Go, and bring them.

 [_Exeunt Messenger and certain Lords._]

You see this chase is hotly follow’d, friends.

DAUPHIN.
Turn head and stop pursuit; for coward dogs
Most spend their mouths when what they seem to threaten
Runs far before them. Good my sovereign,
Take up the English short, and let them know
Of what a monarchy you are the head.
Self-love, my liege, is not so vile a sin
As self-neglecting.

 Enter Exeter.

FRENCH KING.
From our brother of England?

EXETER.
From him; and thus he greets your Majesty:
He wills you, in the name of God Almighty,
That you divest yourself, and lay apart
The borrowed glories that by gift of heaven,
By law of nature and of nations, ’longs
To him and to his heirs; namely, the crown
And all wide-stretched honours that pertain
By custom and the ordinance of times
Unto the crown of France. That you may know
’Tis no sinister nor no awkward claim
Pick’d from the worm-holes of long-vanish’d days,
Nor from the dust of old oblivion rak’d,
He sends you this most memorable line,
In every branch truly demonstrative;
Willing you overlook this pedigree;
And when you find him evenly deriv’d
From his most fam’d of famous ancestors,
Edward the Third, he bids you then resign
Your crown and kingdom, indirectly held
From him, the native and true challenger.

FRENCH KING.
Or else what follows?

EXETER.
Bloody constraint; for if you hide the crown
Even in your hearts, there will he rake for it.
Therefore in fierce tempest is he coming,
In thunder and in earthquake, like a Jove,
That, if requiring fail, he will compel;
And bids you, in the bowels of the Lord,
Deliver up the crown, and to take mercy
On the poor souls for whom this hungry war
Opens his vasty jaws; and on your head
Turning the widows’ tears, the orphans’ cries,
The dead men’s blood, the pining maidens’ groans,
For husbands, fathers, and betrothed lovers,
That shall be swallowed in this controversy.
This is his claim, his threat’ning, and my message;
Unless the Dauphin be in presence here,
To whom expressly I bring greeting too.

FRENCH KING.
For us, we will consider of this further.
Tomorrow shall you bear our full intent
Back to our brother of England.

DAUPHIN.
For the Dauphin,
I stand here for him. What to him from England?

EXETER.
Scorn and defiance. Slight regard, contempt,
And anything that may not misbecome
The mighty sender, doth he prize you at.
Thus says my king: an if your father’s Highness
Do not, in grant of all demands at large,
Sweeten the bitter mock you sent his Majesty,
He’ll call you to so hot an answer of it
That caves and womby vaultages of France
Shall chide your trespass and return your mock
In second accent of his ordinance.

DAUPHIN.
Say, if my father render fair return,
It is against my will; for I desire
Nothing but odds with England. To that end,
As matching to his youth and vanity,
I did present him with the Paris balls.

EXETER.
He’ll make your Paris Louvre shake for it,
Were it the mistress-court of mighty Europe;
And, be assur’d, you’ll find a difference,
As we his subjects have in wonder found,
Between the promise of his greener days
And these he masters now. Now he weighs time
Even to the utmost grain. That you shall read
In your own losses, if he stay in France.

FRENCH KING.
Tomorrow shall you know our mind at full.

 [_Flourish._]

EXETER.
Dispatch us with all speed, lest that our king
Come here himself to question our delay;
For he is footed in this land already.

FRENCH KING.
You shall be soon dispatch’d with fair conditions.
A night is but small breath and little pause
To answer matters of this consequence.

 [_Exeunt._]




ACT III


 Flourish. Enter Chorus.

CHORUS.
Thus with imagin’d wing our swift scene flies,
In motion of no less celerity
Than that of thought. Suppose that you have seen
The well-appointed king at Hampton pier
Embark his royalty, and his brave fleet
With silken streamers the young Phoebus fanning.
Play with your fancies; and in them behold
Upon the hempen tackle ship-boys climbing;
Hear the shrill whistle which doth order give
To sounds confus’d; behold the threaden sails,
Borne with the invisible and creeping wind,
Draw the huge bottoms through the furrow’d sea,
Breasting the lofty surge. O, do but think
You stand upon the rivage and behold
A city on the inconstant billows dancing;
For so appears this fleet majestical,
Holding due course to Harfleur. Follow, follow!
Grapple your minds to sternage of this navy,
And leave your England, as dead midnight still,
Guarded with grandsires, babies, and old women,
Either past or not arriv’d to pith and puissance.
For who is he, whose chin is but enrich’d
With one appearing hair, that will not follow
These cull’d and choice-drawn cavaliers to France?
Work, work your thoughts, and therein see a siege;
Behold the ordnance on their carriages,
With fatal mouths gaping on girded Harfleur.
Suppose the ambassador from the French comes back,
Tells Harry that the King doth offer him
Katharine his daughter, and with her, to dowry,
Some petty and unprofitable dukedoms.
The offer likes not; and the nimble gunner
With linstock now the devilish cannon touches,

 [_Alarum, and chambers go off._]

And down goes all before them. Still be kind,
And eke out our performance with your mind.

 [_Exit._]

SCENE I. France. Before Harfleur.

 Alarum. Enter King Henry, Exeter, Bedford, Gloucester and Soldiers,
 with scaling-ladders.

KING HENRY.
Once more unto the breach, dear friends, once more,
Or close the wall up with our English dead.
In peace there’s nothing so becomes a man
As modest stillness and humility;
But when the blast of war blows in our ears,
Then imitate the action of the tiger;
Stiffen the sinews, summon up the blood,
Disguise fair nature with hard-favour’d rage;
Then lend the eye a terrible aspect;
Let it pry through the portage of the head
Like the brass cannon; let the brow o’erwhelm it
As fearfully as does a galled rock
O’erhang and jutty his confounded base,
Swill’d with the wild and wasteful ocean.
Now set the teeth and stretch the nostril wide,
Hold hard the breath, and bend up every spirit
To his full height. On, on, you noblest English,
Whose blood is fet from fathers of war-proof!
Fathers that, like so many Alexanders,
Have in these parts from morn till even fought,
And sheath’d their swords for lack of argument.
Dishonour not your mothers; now attest
That those whom you call’d fathers did beget you.
Be copy now to men of grosser blood,
And teach them how to war. And you, good yeomen,
Whose limbs were made in England, show us here
The mettle of your pasture; let us swear
That you are worth your breeding, which I doubt not;
For there is none of you so mean and base,
That hath not noble lustre in your eyes.
I see you stand like greyhounds in the slips,
Straining upon the start. The game’s afoot!
Follow your spirit, and upon this charge
Cry, “God for Harry! England and Saint George!”

 [_Exeunt. Alarum, and chambers go off._]

SCENE II. The same.

 Enter Nym, Bardolph, Pistol and Boy.

BARDOLPH.
On, on, on, on, on! To the breach, to the breach!

NYM.
Pray thee, corporal, stay. The knocks are too hot; and, for mine own
part, I have not a case of lives. The humour of it is too hot; that is
the very plain-song of it.

PISTOL.
The plain-song is most just, for humours do abound.
Knocks go and come; God’s vassals drop and die;
    And sword and shield,
    In bloody field,
    Doth win immortal fame.

BOY.
Would I were in an alehouse in London! I would give all my fame for a
pot of ale and safety.

PISTOL.
And I.
    If wishes would prevail with me,
    My purpose should not fail with me,
    But thither would I hie.

BOY.
    As duly,
    But not as truly,
    As bird doth sing on bough.

 Enter Fluellen.

FLUELLEN.
Up to the breach, you dogs! Avaunt, you cullions!

 [_Driving them forward._]

PISTOL.
Be merciful, great Duke, to men of mould.
Abate thy rage, abate thy manly rage,
Abate thy rage, great Duke!
Good bawcock, bate thy rage; use lenity, sweet chuck!

NYM.
These be good humours! Your honour wins bad humours.

 [_Exeunt all but Boy._]

BOY.
As young as I am, I have observ’d these three swashers. I am boy to
them all three; but all they three, though they would serve me, could
not be man to me; for indeed three such antics do not amount to a man.
For Bardolph, he is white-liver’d and red-fac’d; by the means whereof
’a faces it out, but fights not. For Pistol, he hath a killing tongue
and a quiet sword; by the means whereof ’a breaks words, and keeps
whole weapons. For Nym, he hath heard that men of few words are the
best men; and therefore he scorns to say his prayers, lest ’a should be
thought a coward. But his few bad words are match’d with as few good
deeds; for ’a never broke any man’s head but his own, and that was
against a post when he was drunk. They will steal anything, and call it
purchase. Bardolph stole a lute-case, bore it twelve leagues, and sold
it for three half-pence. Nym and Bardolph are sworn brothers in
filching, and in Calais they stole a fire-shovel. I knew by that piece
of service the men would carry coals. They would have me as familiar
with men’s pockets as their gloves or their handkerchers; which makes
much against my manhood, if I should take from another’s pocket to put
into mine; for it is plain pocketing up of wrongs. I must leave them,
and seek some better service. Their villainy goes against my weak
stomach, and therefore I must cast it up.

 [_Exit._]

 Enter Gower and Fluellen.

GOWER.
Captain Fluellen, you must come presently to the mines.
The Duke of Gloucester would speak with you.

FLUELLEN.
To the mines! Tell you the Duke, it is not so good to come to the
mines; for, look you, the mines is not according to the disciplines of
the war. The concavities of it is not sufficient; for, look you, the
athversary, you may discuss unto the Duke, look you, is digt himself
four yard under the countermines. By Cheshu, I think ’a will plow up
all, if there is not better directions.

GOWER.
The Duke of Gloucester, to whom the order of the siege is given, is
altogether directed by an Irishman, a very valiant gentleman, i’ faith.

FLUELLEN.
It is Captain Macmorris, is it not?

GOWER.
I think it be.

FLUELLEN.
By Cheshu, he is an ass, as in the world. I will verify as much in his
beard. He has no more directions in the true disciplines of the wars,
look you, of the Roman disciplines, than is a puppy-dog.

 Enter Macmorris and Captain Jamy.

GOWER.
Here ’a comes; and the Scots captain, Captain Jamy, with him.

FLUELLEN.
Captain Jamy is a marvellous falorous gentleman, that is certain; and
of great expedition and knowledge in the anchient wars, upon my
particular knowledge of his directions. By Cheshu, he will maintain his
argument as well as any military man in the world, in the disciplines
of the pristine wars of the Romans.

JAMY.
I say gud-day, Captain Fluellen.

FLUELLEN.
God-den to your worship, good Captain James.

GOWER.
How now, Captain Macmorris! have you quit the mines?
Have the pioneers given o’er?

MACMORRIS.
By Chrish, la! ’tish ill done! The work ish give over, the trompet
sound the retreat. By my hand I swear, and my father’s soul, the work
ish ill done; it ish give over. I would have blowed up the town, so
Chrish save me, la! in an hour. O, ’tish ill done, ’tish ill done; by
my hand, ’tish ill done!

FLUELLEN.
Captain Macmorris, I beseech you now, will you voutsafe me, look you, a
few disputations with you, as partly touching or concerning the
disciplines of the war, the Roman wars, in the way of argument, look
you, and friendly communication; partly to satisfy my opinion, and
partly for the satisfaction, look you, of my mind, as touching the
direction of the military discipline; that is the point.

JAMY.
It sall be vary gud, gud feith, gud captains bath: and I sall quit you
with gud leve, as I may pick occasion; that sall I, marry.

MACMORRIS.
It is no time to discourse, so Chrish save me. The day is hot, and the
weather, and the wars, and the King, and the Dukes. It is no time to
discourse. The town is beseech’d, and the trumpet call us to the
breach, and we talk, and, be Chrish, do nothing. ’Tis shame for us all.
So God sa’ me, ’tis shame to stand still; it is shame, by my hand; and
there is throats to be cut, and works to be done; and there ish nothing
done, so Chrish sa’ me, la!

JAMY.
By the mess, ere theise eyes of mine take themselves to slomber, I’ll
de gud service, or I’ll lig i’ the grund for it; ay, or go to death;
and I’ll pay’t as valorously as I may, that sall I suerly do, that is
the breff and the long. Marry, I wad full fain heard some question
’tween you tway.

FLUELLEN.
Captain Macmorris, I think, look you, under your correction, there is
not many of your nation—

MACMORRIS.
Of my nation! What ish my nation? Ish a villain, and a bastard, and a
knave, and a rascal? What ish my nation? Who talks of my nation?

FLUELLEN.
Look you, if you take the matter otherwise than is meant, Captain
Macmorris, peradventure I shall think you do not use me with that
affability as in discretion you ought to use me, look you, being as
good a man as yourself, both in the disciplines of war, and in the
derivation of my birth, and in other particularities.

MACMORRIS.
I do not know you so good a man as myself. So Chrish save me,
I will cut off your head.

GOWER.
Gentlemen both, you will mistake each other.

JAMY.
Ah! that’s a foul fault.

 [_A parley sounded._]

GOWER.
The town sounds a parley.

FLUELLEN.
Captain Macmorris, when there is more better opportunity to be
required, look you, I will be so bold as to tell you I know the
disciplines of war; and there is an end.

 [_Exeunt._]

SCENE III. Before the gates.

 The Governor and some citizens on the walls; the English forces below.
 Enter King Henry and his train.

KING HENRY.
How yet resolves the governor of the town?
This is the latest parle we will admit;
Therefore to our best mercy give yourselves,
Or like to men proud of destruction
Defy us to our worst; for, as I am a soldier,
A name that in my thoughts becomes me best,
If I begin the battery once again,
I will not leave the half-achieved Harfleur
Till in her ashes she lie buried.
The gates of mercy shall be all shut up,
And the flesh’d soldier, rough and hard of heart,
In liberty of bloody hand shall range
With conscience wide as hell, mowing like grass
Your fresh fair virgins and your flow’ring infants.
What is it then to me, if impious War,
Array’d in flames like to the prince of fiends,
Do with his smirch’d complexion all fell feats
Enlink’d to waste and desolation?
What is’t to me, when you yourselves are cause,
If your pure maidens fall into the hand
Of hot and forcing violation?
What rein can hold licentious wickedness
When down the hill he holds his fierce career?
We may as bootless spend our vain command
Upon the enraged soldiers in their spoil
As send precepts to the leviathan
To come ashore. Therefore, you men of Harfleur,
Take pity of your town and of your people,
Whiles yet my soldiers are in my command,
Whiles yet the cool and temperate wind of grace
O’erblows the filthy and contagious clouds
Of heady murder, spoil, and villainy.
If not, why, in a moment look to see
The blind and bloody soldier with foul hand
Defile the locks of your shrill-shrieking daughters;
Your fathers taken by the silver beards,
And their most reverend heads dash’d to the walls;
Your naked infants spitted upon pikes,
Whiles the mad mothers with their howls confus’d
Do break the clouds, as did the wives of Jewry
At Herod’s bloody-hunting slaughtermen.
What say you? Will you yield, and this avoid,
Or, guilty in defence, be thus destroy’d?

GOVERNOR.
Our expectation hath this day an end.
The Dauphin, whom of succours we entreated,
Returns us that his powers are yet not ready
To raise so great a siege. Therefore, great King,
We yield our town and lives to thy soft mercy.
Enter our gates; dispose of us and ours;
For we no longer are defensible.

KING HENRY.
Open your gates. Come, uncle Exeter,
Go you and enter Harfleur; there remain,
And fortify it strongly ’gainst the French.
Use mercy to them all. For us, dear uncle,
The winter coming on, and sickness growing
Upon our soldiers, we will retire to Calais.
Tonight in Harfleur will we be your guest;
Tomorrow for the march are we addrest.

 Flourish. The King and his train enter the town.

SCENE IV. The French King’s palace.

 Enter Katharine and Alice, an old Gentlewoman.

KATHARINE.
_Alice, tu as été en Angleterre, et tu parles bien le langage._

ALICE.
_Un peu, madame._

KATHARINE.
_Je te prie, m’enseignez; il faut que j’apprenne à parler.
Comment appelez-vous la main en anglais?_

ALICE.
_La main? Elle est appelée_ de hand.

KATHARINE.
De hand. _Et les doigts?_

ALICE.
_Les doigts? Ma foi, j’oublie les doigts; mais je me souviendrai. Les
doigts? Je pense qu’ils sont appelés_ de fingres; _oui_, de fingres.

KATHARINE.
_La main_, de hand; _les doigts_, de fingres. _Je pense que je suis le
bon écolier; j’ai gagné deux mots d’anglais vitement. Comment
appelez-vous les ongles?_

ALICE.
_Les ongles? Nous les appelons_ de nails.

KATHARINE.
De nails. _Écoutez; dites-moi, si je parle bien:_ de hand, de fingres,
_et_ de nails.

ALICE.
_C’est bien dit, madame; il est fort bon anglais._

KATHARINE.
_Dites-moi l’anglais pour le bras._

ALICE.
De arm, _madame._

KATHARINE.
_Et le coude?_

ALICE.
D’elbow.

KATHARINE.
D’elbow. _Je m’en fais la répétition de tous les mots que vous m’avez
appris dès à présent._

ALICE.
_Il est trop difficile, madame, comme je pense._

KATHARINE.
_Excusez-moi, Alice. Écoutez:_ d’hand, de fingres, de nails, d’arm, de
bilbow.

ALICE.
D’elbow, _madame._

KATHARINE.
_O Seigneur Dieu, je m’en oublie!_ D’elbow.
_Comment appelez-vous le col?_

ALICE.
De nick, _madame._

KATHARINE.
De nick. _Et le menton?_

ALICE.
De chin.

KATHARINE.
De sin. _Le col_, de nick; _le menton_, de sin.

ALICE.
_Oui. Sauf votre honneur, en vérité, vous prononcez les mots aussi
droit que les natifs d’Angleterre._

KATHARINE.
_Je ne doute point d’apprendre, par la grâce de Dieu, et en peu de
temps._

ALICE.
_N’avez-vous pas déjà oublié ce que je vous ai enseigné?_

KATHARINE.
_Non, je réciterai à vous promptement:_ d’hand, de fingres, de mails,—

ALICE.
De nails, _madame._

KATHARINE.
De nails, de arm, de ilbow.

ALICE.
_Sauf votre honneur_, de elbow.

KATHARINE.
_Ainsi dis-je_, d’elbow, de nick, _et_ de sin. _Comment appelez-vous le
pied et la robe?_

ALICE.
De foot, _madame; et_ de coun.

KATHARINE.
De foot _et_ de coun! _O Seigneur Dieu! ils sont les mots de son
mauvais, corruptible, gros, et impudique, et non pour les dames
d’honneur d’user. Je ne voudrais prononcer ces mots devant les
seigneurs de France pour tout le monde. Foh!_ le foot _et_ le coun!
_Néanmoins, je réciterai une autre fois ma leçon ensemble:_ d’hand, de
fingres, de nails, d’arm, d’elbow, de nick, de sin, de foot, de coun.

ALICE.
_Excellent, madame!_

KATHARINE.
_C’est assez pour une fois. Allons-nous à dîner._

 [_Exeunt._]

SCENE V. The same.

 Enter the King of France, the Dauphin, the Duke of Bourbon, the
 Constable of France and others.

FRENCH KING.
’Tis certain he hath pass’d the river Somme.

CONSTABLE.
And if he be not fought withal, my lord,
Let us not live in France; let us quit all
And give our vineyards to a barbarous people.

DAUPHIN.
_O Dieu vivant_! shall a few sprays of us,
The emptying of our fathers’ luxury,
Our scions put in wild and savage stock,
Spirt up so suddenly into the clouds,
And overlook their grafters?

BOURBON.
Normans, but bastard Normans, Norman bastards!
_Mort de ma vie_, if they march along
Unfought withal, but I will sell my dukedom,
To buy a slobbery and a dirty farm
In that nook-shotten isle of Albion.

CONSTABLE.
_Dieu de batailles_, where have they this mettle?
Is not their climate foggy, raw, and dull,
On whom, as in despite, the sun looks pale,
Killing their fruit with frowns? Can sodden water,
A drench for sur-rein’d jades, their barley-broth,
Decoct their cold blood to such valiant heat?
And shall our quick blood, spirited with wine,
Seem frosty? O, for honour of our land,
Let us not hang like roping icicles
Upon our houses’ thatch, whiles a more frosty people
Sweat drops of gallant youth in our rich fields!
Poor we may call them in their native lords.

DAUPHIN.
By faith and honour,
Our madams mock at us, and plainly say
Our mettle is bred out, and they will give
Their bodies to the lust of English youth
To new-store France with bastard warriors.

BOURBON.
They bid us to the English dancing-schools,
And teach lavoltas high, and swift corantos;
Saying our grace is only in our heels,
And that we are most lofty runaways.

FRENCH KING.
Where is Montjoy the herald? Speed him hence.
Let him greet England with our sharp defiance.
Up, princes! and, with spirit of honour edged
More sharper than your swords, hie to the field!
Charles Delabreth, High Constable of France;
You Dukes of Orleans, Bourbon, and of Berry,
Alençon, Brabant, Bar, and Burgundy;
Jacques Chatillon, Rambures, Vaudemont,
Beaumont, Grandpré, Roussi, and Fauconbridge,
Foix, Lestrale, Boucicault, and Charolois;
High dukes, great princes, barons, lords, and knights,
For your great seats now quit you of great shames.
Bar Harry England, that sweeps through our land
With pennons painted in the blood of Harfleur.
Rush on his host, as doth the melted snow
Upon the valleys, whose low vassal seat
The Alps doth spit and void his rheum upon.
Go down upon him, you have power enough,
And in a captive chariot into Rouen
Bring him our prisoner.

CONSTABLE.
This becomes the great.
Sorry am I his numbers are so few,
His soldiers sick and famish’d in their march;
For I am sure, when he shall see our army,
He’ll drop his heart into the sink of fear
And for achievement offer us his ransom.

FRENCH KING.
Therefore, Lord Constable, haste on Montjoy,
And let him say to England that we send
To know what willing ransom he will give.
Prince Dauphin, you shall stay with us in Rouen.

DAUPHIN.
Not so, I do beseech your Majesty.

FRENCH KING.
Be patient, for you shall remain with us.
Now forth, Lord Constable and princes all,
And quickly bring us word of England’s fall.

 [_Exeunt._]

SCENE VI. The English camp in Picardy.

 Enter Gower and Fluellen, meeting.

GOWER.
How now, Captain Fluellen! come you from the bridge?

FLUELLEN.
I assure you, there is very excellent services committed at the bridge.

GOWER.
Is the Duke of Exeter safe?

FLUELLEN.
The Duke of Exeter is as magnanimous as Agamemnon; and a man that I
love and honour with my soul, and my heart, and my duty, and my life,
and my living, and my uttermost power. He is not—God be praised and
blessed!—any hurt in the world; but keeps the bridge most valiantly,
with excellent discipline. There is an anchient lieutenant there at the
pridge, I think in my very conscience he is as valiant a man as Mark
Antony; and he is a man of no estimation in the world, but I did see
him do as gallant service.

GOWER.
What do you call him?

FLUELLEN.
He is call’d Anchient Pistol.

GOWER.
I know him not.

 Enter Pistol.

FLUELLEN.
Here is the man.

PISTOL.
Captain, I thee beseech to do me favours.
The Duke of Exeter doth love thee well.

FLUELLEN.
Ay, I praise God; and I have merited some love at his hands.

PISTOL.
Bardolph, a soldier, firm and sound of heart,
And of buxom valour, hath by cruel fate
And giddy Fortune’s furious fickle wheel,
That goddess blind,
That stands upon the rolling restless stone—

FLUELLEN.
By your patience, Anchient Pistol. Fortune is painted blind, with a
muffler afore his eyes, to signify to you that Fortune is blind; and
she is painted also with a wheel, to signify to you, which is the moral
of it, that she is turning, and inconstant, and mutability, and
variation; and her foot, look you, is fixed upon a spherical stone,
which rolls, and rolls, and rolls. In good truth, the poet makes a most
excellent description of it. Fortune is an excellent moral.

PISTOL.
Fortune is Bardolph’s foe, and frowns on him;
For he hath stolen a pax, and hanged must ’a be,—
A damned death!
Let gallows gape for dog; let man go free,
And let not hemp his windpipe suffocate.
But Exeter hath given the doom of death
For pax of little price.
Therefore, go speak; the Duke will hear thy voice;
And let not Bardolph’s vital thread be cut
With edge of penny cord and vile reproach.
Speak, captain, for his life, and I will thee requite.

FLUELLEN.
Anchient Pistol, I do partly understand your meaning.

PISTOL.
Why then, rejoice therefore.

FLUELLEN.
Certainly, anchient, it is not a thing to rejoice at; for if, look you,
he were my brother, I would desire the Duke to use his good pleasure,
and put him to execution; for discipline ought to be used.

PISTOL.
Die and be damn’d! and _fico_ for thy friendship!

FLUELLEN.
It is well.

PISTOL.
The fig of Spain.

 [_Exit._]

FLUELLEN.
Very good.

GOWER.
Why, this is an arrant counterfeit rascal. I remember him now; a bawd,
a cutpurse.

FLUELLEN.
I’ll assure you, ’a uttered as prave words at the pridge as you shall
see in a summer’s day. But it is very well; what he has spoke to me,
that is well, I warrant you, when time is serve.

GOWER.
Why, ’t is a gull, a fool, a rogue, that now and then goes to the wars,
to grace himself at his return into London under the form of a soldier.
And such fellows are perfect in the great commanders’ names; and they
will learn you by rote where services were done; at such and such a
sconce, at such a breach, at such a convoy; who came off bravely, who
was shot, who disgrac’d, what terms the enemy stood on; and this they
con perfectly in the phrase of war, which they trick up with new-tuned
oaths: and what a beard of the general’s cut and a horrid suit of the
camp will do among foaming bottles and ale-wash’d wits, is wonderful to
be thought on. But you must learn to know such slanders of the age, or
else you may be marvellously mistook.

FLUELLEN.
I tell you what, Captain Gower; I do perceive he is not the man that he
would gladly make show to the world he is. If I find a hole in his
coat, I will tell him my mind. [_Drum heard._] Hark you, the King is
coming, and I must speak with him from the pridge.

 Drum and colours. Enter King Henry, Gloucester and his poor soldiers.

God bless your Majesty!

KING HENRY.
How now, Fluellen! cam’st thou from the bridge?

FLUELLEN.
Ay, so please your Majesty. The Duke of Exeter has very gallantly
maintain’d the pridge. The French is gone off, look you; and there is
gallant and most prave passages. Marry, th’ athversary was have
possession of the pridge; but he is enforced to retire, and the Duke of
Exeter is master of the pridge. I can tell your Majesty, the Duke is a
prave man.

KING HENRY.
What men have you lost, Fluellen?

FLUELLEN.
The perdition of the athversary hath been very great, reasonable great.
Marry, for my part, I think the Duke hath lost never a man, but one
that is like to be executed for robbing a church, one Bardolph, if your
Majesty know the man. His face is all bubukles, and whelks, and knobs,
and flames o’ fire; and his lips blows at his nose, and it is like a
coal of fire, sometimes plue and sometimes red; but his nose is
executed, and his fire’s out.

KING HENRY.
We would have all such offenders so cut off; and we give express
charge, that in our marches through the country, there be nothing
compell’d from the villages, nothing taken but paid for, none of the
French upbraided or abused in disdainful language; for when lenity and
cruelty play for a kingdom, the gentler gamester is the soonest winner.

 Tucket. Enter Montjoy.

MONTJOY.
You know me by my habit.

KING HENRY.
Well then I know thee. What shall I know of thee?

MONTJOY.
My master’s mind.

KING HENRY.
Unfold it.

MONTJOY.
Thus says my King: Say thou to Harry of England: Though we seem’d dead,
we did but sleep; advantage is a better soldier than rashness. Tell him
we could have rebuk’d him at Harfleur, but that we thought not good to
bruise an injury till it were full ripe. Now we speak upon our cue, and
our voice is imperial. England shall repent his folly, see his
weakness, and admire our sufferance. Bid him therefore consider of his
ransom; which must proportion the losses we have borne, the subjects we
have lost, the disgrace we have digested; which in weight to re-answer,
his pettishness would bow under. For our losses, his exchequer is too
poor; for the effusion of our blood, the muster of his kingdom too
faint a number; and for our disgrace, his own person, kneeling at our
feet, but a weak and worthless satisfaction. To this add defiance; and
tell him, for conclusion, he hath betrayed his followers, whose
condemnation is pronounc’d. So far my King and master; so much my
office.

KING HENRY.
What is thy name? I know thy quality.

MONTJOY.
Montjoy.

KING HENRY.
Thou dost thy office fairly. Turn thee back,
And tell thy King I do not seek him now,
But could be willing to march on to Calais
Without impeachment; for, to say the sooth,
Though ’tis no wisdom to confess so much
Unto an enemy of craft and vantage,
My people are with sickness much enfeebled,
My numbers lessen’d, and those few I have
Almost no better than so many French;
Who when they were in health, I tell thee, herald,
I thought upon one pair of English legs
Did march three Frenchmen. Yet, forgive me, God,
That I do brag thus! This your air of France
Hath blown that vice in me. I must repent.
Go therefore, tell thy master here I am;
My ransom is this frail and worthless trunk,
My army but a weak and sickly guard;
Yet, God before, tell him we will come on,
Though France himself and such another neighbour
Stand in our way. There’s for thy labour, Montjoy.
Go, bid thy master well advise himself.
If we may pass, we will; if we be hind’red,
We shall your tawny ground with your red blood
Discolour; and so, Montjoy, fare you well.
The sum of all our answer is but this:
We would not seek a battle, as we are;
Nor, as we are, we say we will not shun it.
So tell your master.

MONTJOY.
I shall deliver so. Thanks to your Highness.

 [_Exit._]

GLOUCESTER.
I hope they will not come upon us now.

KING HENRY.
We are in God’s hands, brother, not in theirs.
March to the bridge; it now draws toward night.
Beyond the river we’ll encamp ourselves,
And on tomorrow bid them march away.

 [_Exeunt._]

SCENE VII. The French camp, near Agincourt.

 Enter the Constable of France, the Lord Rambures, Orleans, Dauphin
 with others.

CONSTABLE.
Tut! I have the best armour of the world.
Would it were day!

ORLEANS.
You have an excellent armour; but let my horse have his due.

CONSTABLE.
It is the best horse of Europe.

ORLEANS.
Will it never be morning?

DAUPHIN.
My Lord of Orleans, and my Lord High Constable, you talk of horse and
armour?

ORLEANS.
You are as well provided of both as any prince in the world.

DAUPHIN.
What a long night is this! I will not change my horse with any that
treads but on four pasterns. Ch’ha! He bounds from the earth, as if his
entrails were hairs; _le cheval volant_, the Pegasus, _qui a les
narines de feu!_ When I bestride him, I soar, I am a hawk. He trots the
air; the earth sings when he touches it; the basest horn of his hoof is
more musical than the pipe of Hermes.

ORLEANS.
He’s of the colour of the nutmeg.

DAUPHIN.
And of the heat of the ginger. It is a beast for Perseus. He is pure
air and fire; and the dull elements of earth and water never appear in
him, but only in patient stillness while his rider mounts him. He is
indeed a horse, and all other jades you may call beasts.

CONSTABLE.
Indeed, my lord, it is a most absolute and excellent horse.

DAUPHIN.
It is the prince of palfreys; his neigh is like the bidding of a
monarch, and his countenance enforces homage.

ORLEANS.
No more, cousin.

DAUPHIN.
Nay, the man hath no wit that cannot, from the rising of the lark to
the lodging of the lamb, vary deserved praise on my palfrey. It is a
theme as fluent as the sea; turn the sands into eloquent tongues, and
my horse is argument for them all. ’Tis a subject for a sovereign to
reason on, and for a sovereign’s sovereign to ride on; and for the
world, familiar to us and unknown, to lay apart their particular
functions and wonder at him. I once writ a sonnet in his praise and
began thus: “Wonder of nature,”—

ORLEANS.
I have heard a sonnet begin so to one’s mistress.

DAUPHIN.
Then did they imitate that which I compos’d to my courser, for my horse
is my mistress.

ORLEANS.
Your mistress bears well.

DAUPHIN.
Me well; which is the prescript praise and perfection of a good and
particular mistress.

CONSTABLE.
Nay, for methought yesterday your mistress shrewdly shook your back.

DAUPHIN.
So perhaps did yours.

CONSTABLE.
Mine was not bridled.

DAUPHIN.
O then belike she was old and gentle; and you rode, like a kern of
Ireland, your French hose off, and in your strait strossers.

CONSTABLE.
You have good judgment in horsemanship.

DAUPHIN.
Be warn’d by me, then; they that ride so and ride not warily, fall into
foul bogs. I had rather have my horse to my mistress.

CONSTABLE.
I had as lief have my mistress a jade.

DAUPHIN.
I tell thee, Constable, my mistress wears his own hair.

CONSTABLE.
I could make as true a boast as that, if I had a sow to my mistress.

DAUPHIN.
“_Le chien est retourné à son propre vomissement, et la truie lavée au
bourbier_.” Thou mak’st use of anything.

CONSTABLE.
Yet do I not use my horse for my mistress, or any such proverb so
little kin to the purpose.

RAMBURES.
My Lord Constable, the armour that I saw in your tent tonight, are
those stars or suns upon it?

CONSTABLE.
Stars, my lord.

DAUPHIN.
Some of them will fall tomorrow, I hope.

CONSTABLE.
And yet my sky shall not want.

DAUPHIN.
That may be, for you bear a many superfluously, and ’twere more honour
some were away.

CONSTABLE.
Even as your horse bears your praises; who would trot as well, were
some of your brags dismounted.

DAUPHIN.
Would I were able to load him with his desert! Will it never be day? I
will trot tomorrow a mile, and my way shall be paved with English
faces.

CONSTABLE.
I will not say so, for fear I should be fac’d out of my way. But I
would it were morning; for I would fain be about the ears of the
English.

RAMBURES.
Who will go to hazard with me for twenty prisoners?

CONSTABLE.
You must first go yourself to hazard, ere you have them.

DAUPHIN.
’Tis midnight; I’ll go arm myself.

 [_Exit._]

ORLEANS.
The Dauphin longs for morning.

RAMBURES.
He longs to eat the English.

CONSTABLE.
I think he will eat all he kills.

ORLEANS.
By the white hand of my lady, he’s a gallant prince.

CONSTABLE.
Swear by her foot that she may tread out the oath.

ORLEANS.
He is simply the most active gentleman of France.

CONSTABLE.
Doing is activity; and he will still be doing.

ORLEANS.
He never did harm, that I heard of.

CONSTABLE.
Nor will do none tomorrow. He will keep that good name still.

ORLEANS.
I know him to be valiant.

CONSTABLE.
I was told that by one that knows him better than you.

ORLEANS.
What’s he?

CONSTABLE.
Marry, he told me so himself; and he said he car’d not who knew it.

ORLEANS.
He needs not; it is no hidden virtue in him.

CONSTABLE.
By my faith, sir, but it is; never anybody saw it but his lackey. ’Tis
a hooded valour; and when it appears, it will bate.

ORLEANS.
“Ill will never said well.”

CONSTABLE.
I will cap that proverb with “There is flattery in friendship.”

ORLEANS.
And I will take up that with “Give the devil his due.”

CONSTABLE.
Well plac’d. There stands your friend for the devil; have at the very
eye of that proverb with “A pox of the devil.”

ORLEANS.
You are the better at proverbs, by how much “A fool’s bolt is soon
shot.”

CONSTABLE.
You have shot over.

ORLEANS.
’Tis not the first time you were overshot.

 Enter a Messenger.

MESSENGER.
My Lord High Constable, the English lie within fifteen hundred paces of
your tents.

CONSTABLE.
Who hath measur’d the ground?

MESSENGER.
The Lord Grandpré.

CONSTABLE.
A valiant and most expert gentleman. Would it were day! Alas, poor
Harry of England, he longs not for the dawning as we do.

ORLEANS.
What a wretched and peevish fellow is this King of England, to mope
with his fat-brain’d followers so far out of his knowledge!

CONSTABLE.
If the English had any apprehension, they would run away.

ORLEANS.
That they lack; for if their heads had any intellectual armour, they
could never wear such heavy head-pieces.

RAMBURES.
That island of England breeds very valiant creatures. Their mastiffs
are of unmatchable courage.

ORLEANS.
Foolish curs, that run winking into the mouth of a Russian bear and
have their heads crush’d like rotten apples! You may as well say,
that’s a valiant flea that dare eat his breakfast on the lip of a lion.

CONSTABLE.
Just, just; and the men do sympathize with the mastiffs in robustious
and rough coming on, leaving their wits with their wives; and then,
give them great meals of beef and iron and steel, they will eat like
wolves and fight like devils.

ORLEANS.
Ay, but these English are shrewdly out of beef.

CONSTABLE.
Then shall we find tomorrow they have only stomachs to eat and none to
fight. Now is it time to arm. Come, shall we about it?

ORLEANS.
It is now two o’clock; but, let me see, by ten
We shall have each a hundred Englishmen.

 [_Exeunt._]




ACT IV


 Enter Chorus.

CHORUS.
Now entertain conjecture of a time
When creeping murmur and the poring dark
Fills the wide vessel of the universe.
From camp to camp through the foul womb of night
The hum of either army stilly sounds,
That the fix’d sentinels almost receive
The secret whispers of each other’s watch;
Fire answers fire, and through their paly flames
Each battle sees the other’s umber’d face;
Steed threatens steed, in high and boastful neighs
Piercing the night’s dull ear; and from the tents
The armourers, accomplishing the knights,
With busy hammers closing rivets up,
Give dreadful note of preparation.
The country cocks do crow, the clocks do toll,
And the third hour of drowsy morning name.
Proud of their numbers and secure in soul,
The confident and over-lusty French
Do the low-rated English play at dice;
And chide the cripple tardy-gaited Night
Who, like a foul and ugly witch, doth limp
So tediously away. The poor condemned English,
Like sacrifices, by their watchful fires
Sit patiently and inly ruminate
The morning’s danger; and their gesture sad,
Investing lank-lean cheeks and war-worn coats,
Presented them unto the gazing moon
So many horrid ghosts. O now, who will behold
The royal captain of this ruin’d band
Walking from watch to watch, from tent to tent,
Let him cry, “Praise and glory on his head!”
For forth he goes and visits all his host,
Bids them good morrow with a modest smile,
And calls them brothers, friends, and countrymen.
Upon his royal face there is no note
How dread an army hath enrounded him;
Nor doth he dedicate one jot of colour
Unto the weary and all-watched night,
But freshly looks, and over-bears attaint
With cheerful semblance and sweet majesty;
That every wretch, pining and pale before,
Beholding him, plucks comfort from his looks.
A largess universal like the sun
His liberal eye doth give to everyone,
Thawing cold fear, that mean and gentle all
Behold, as may unworthiness define,
A little touch of Harry in the night.
And so our scene must to the battle fly,
Where—O for pity!—we shall much disgrace
With four or five most vile and ragged foils,
Right ill-dispos’d in brawl ridiculous,
The name of Agincourt. Yet sit and see,
Minding true things by what their mock’ries be.

 [_Exit._]

SCENE I. The English camp at Agincourt.

 Enter King Henry, Bedford and Gloucester.

KING HENRY.
Gloucester, ’tis true that we are in great danger;
The greater therefore should our courage be.
Good morrow, brother Bedford. God Almighty!
There is some soul of goodness in things evil,
Would men observingly distil it out;
For our bad neighbour makes us early stirrers,
Which is both healthful and good husbandry.
Besides, they are our outward consciences,
And preachers to us all, admonishing
That we should dress us fairly for our end.
Thus may we gather honey from the weed,
And make a moral of the devil himself.

 Enter Erpingham.

Good morrow, old Sir Thomas Erpingham:
A good soft pillow for that good white head
Were better than a churlish turf of France.

ERPINGHAM.
Not so, my liege; this lodging likes me better,
Since I may say, “Now lie I like a king.”

KING HENRY.
’Tis good for men to love their present pains
Upon example; so the spirit is eased;
And when the mind is quick’ned, out of doubt,
The organs, though defunct and dead before,
Break up their drowsy grave and newly move,
With casted slough and fresh legerity.
Lend me thy cloak, Sir Thomas. Brothers both,
Commend me to the princes in our camp;
Do my good morrow to them, and anon
Desire them all to my pavilion.

GLOUCESTER.
We shall, my liege.

ERPINGHAM.
Shall I attend your Grace?

KING HENRY.
No, my good knight;
Go with my brothers to my lords of England.
I and my bosom must debate a while,
And then I would no other company.

ERPINGHAM.
The Lord in heaven bless thee, noble Harry!

 [_Exeunt all but King._]

KING HENRY.
God-a-mercy, old heart! thou speak’st cheerfully.

 Enter Pistol.

PISTOL.
_Qui vous là?_

KING HENRY.
A friend.

PISTOL.
Discuss unto me; art thou officer?
Or art thou base, common, and popular?

KING HENRY.
I am a gentleman of a company.

PISTOL.
Trail’st thou the puissant pike?

KING HENRY.
Even so. What are you?

PISTOL.
As good a gentleman as the Emperor.

KING HENRY.
Then you are a better than the King.

PISTOL.
The King’s a bawcock, and a heart of gold,
A lad of life, an imp of fame;
Of parents good, of fist most valiant.
I kiss his dirty shoe, and from heart-string
I love the lovely bully. What is thy name?

KING HENRY.
Harry le Roy.

PISTOL.
Le Roy! a Cornish name. Art thou of Cornish crew?

KING HENRY.
No, I am a Welshman.

PISTOL.
Know’st thou Fluellen?

KING HENRY.
Yes.

PISTOL.
Tell him I’ll knock his leek about his pate
Upon Saint Davy’s day.

KING HENRY.
Do not you wear your dagger in your cap that day, lest he knock that
about yours.

PISTOL.
Art thou his friend?

KING HENRY.
And his kinsman too.

PISTOL.
The _fico_ for thee, then!

KING HENRY.
I thank you. God be with you!

PISTOL.
My name is Pistol call’d.

 [_Exit._]

KING HENRY.
It sorts well with your fierceness.

 Enter Fluellen and Gower.

GOWER.
Captain Fluellen!

FLUELLEN.
So! in the name of Jesu Christ, speak lower. It is the greatest
admiration in the universal world, when the true and anchient
prerogatifes and laws of the wars is not kept. If you would take the
pains but to examine the wars of Pompey the Great, you shall find, I
warrant you, that there is no tiddle taddle nor pibble pabble in
Pompey’s camp. I warrant you, you shall find the ceremonies of the
wars, and the cares of it, and the forms of it, and the sobriety of it,
and the modesty of it, to be otherwise.

GOWER.
Why, the enemy is loud; you hear him all night.

FLUELLEN.
If the enemy is an ass and a fool and a prating coxcomb, is it meet,
think you, that we should also, look you, be an ass and a fool and a
prating coxcomb? In your own conscience, now?

GOWER.
I will speak lower.

FLUELLEN.
I pray you and beseech you that you will.

 [_Exeunt Gower and Fluellen._]

KING HENRY.
Though it appear a little out of fashion,
There is much care and valour in this Welshman.

 Enter three soldiers, John Bates, Alexander Court and Michael
 Williams.

COURT.
Brother John Bates, is not that the morning which breaks yonder?

BATES.
I think it be; but we have no great cause to desire the approach of
day.

WILLIAMS.
We see yonder the beginning of the day, but I think we shall never see
the end of it. Who goes there?

KING HENRY.
A friend.

WILLIAMS.
Under what captain serve you?

KING HENRY.
Under Sir Thomas Erpingham.

WILLIAMS.
A good old commander and a most kind gentleman. I pray you, what thinks
he of our estate?

KING HENRY.
Even as men wreck’d upon a sand, that look to be wash’d off the next
tide.

BATES.
He hath not told his thought to the King?

KING HENRY.
No; nor it is not meet he should. For though I speak it to you, I think
the King is but a man as I am. The violet smells to him as it doth to
me; the element shows to him as it doth to me; all his senses have but
human conditions. His ceremonies laid by, in his nakedness he appears
but a man; and though his affections are higher mounted than ours, yet,
when they stoop, they stoop with the like wing. Therefore, when he sees
reason of fears as we do, his fears, out of doubt, be of the same
relish as ours are; yet, in reason, no man should possess him with any
appearance of fear, lest he, by showing it, should dishearten his army.

BATES.
He may show what outward courage he will; but I believe, as cold a
night as ’tis, he could wish himself in Thames up to the neck; and so I
would he were, and I by him, at all adventures, so we were quit here.

KING HENRY.
By my troth, I will speak my conscience of the King: I think he would
not wish himself anywhere but where he is.

BATES.
Then I would he were here alone; so should he be sure to be ransomed,
and a many poor men’s lives saved.

KING HENRY.
I dare say you love him not so ill, to wish him here alone, howsoever
you speak this to feel other men’s minds. Methinks I could not die
anywhere so contented as in the King’s company, his cause being just
and his quarrel honourable.

WILLIAMS.
That’s more than we know.

BATES.
Ay, or more than we should seek after; for we know enough, if we know
we are the King’s subjects. If his cause be wrong, our obedience to the
King wipes the crime of it out of us.

WILLIAMS.
But if the cause be not good, the King himself hath a heavy reckoning
to make, when all those legs and arms and heads, chopp’d off in a
battle, shall join together at the latter day and cry all, “We died at
such a place”; some swearing, some crying for a surgeon, some upon
their wives left poor behind them, some upon the debts they owe, some
upon their children rawly left. I am afeard there are few die well that
die in a battle; for how can they charitably dispose of anything, when
blood is their argument? Now, if these men do not die well, it will be
a black matter for the King that led them to it; who to disobey were
against all proportion of subjection.

KING HENRY.
So, if a son that is by his father sent about merchandise do sinfully
miscarry upon the sea, the imputation of his wickedness, by your rule,
should be imposed upon his father that sent him; or if a servant, under
his master’s command transporting a sum of money, be assailed by
robbers and die in many irreconcil’d iniquities, you may call the
business of the master the author of the servant’s damnation. But this
is not so. The King is not bound to answer the particular endings of
his soldiers, the father of his son, nor the master of his servant; for
they purpose not their death, when they purpose their services.
Besides, there is no king, be his cause never so spotless, if it come
to the arbitrement of swords, can try it out with all unspotted
soldiers. Some peradventure have on them the guilt of premeditated and
contrived murder; some, of beguiling virgins with the broken seals of
perjury; some, making the wars their bulwark, that have before gored
the gentle bosom of Peace with pillage and robbery. Now, if these men
have defeated the law and outrun native punishment, though they can
outstrip men, they have no wings to fly from God. War is his beadle,
war is his vengeance; so that here men are punish’d for before-breach
of the King’s laws in now the King’s quarrel. Where they feared the
death, they have borne life away; and where they would be safe, they
perish. Then if they die unprovided, no more is the King guilty of
their damnation than he was before guilty of those impieties for the
which they are now visited. Every subject’s duty is the King’s; but
every subject’s soul is his own. Therefore should every soldier in the
wars do as every sick man in his bed, wash every mote out of his
conscience; and dying so, death is to him advantage; or not dying, the
time was blessedly lost wherein such preparation was gained; and in him
that escapes, it were not sin to think that, making God so free an
offer, He let him outlive that day to see His greatness and to teach
others how they should prepare.

WILLIAMS.
’Tis certain, every man that dies ill, the ill upon his own head, the
King is not to answer for it.

BATES.
I do not desire he should answer for me; and yet I determine to fight
lustily for him.

KING HENRY.
I myself heard the King say he would not be ransom’d.

WILLIAMS.
Ay, he said so, to make us fight cheerfully; but when our throats are
cut, he may be ransom’d, and we ne’er the wiser.

KING HENRY.
If I live to see it, I will never trust his word after.

WILLIAMS.
You pay him then. That’s a perilous shot out of an elder-gun, that a
poor and a private displeasure can do against a monarch! You may as
well go about to turn the sun to ice with fanning in his face with a
peacock’s feather. You’ll never trust his word after! Come, ’tis a
foolish saying.

KING HENRY.
Your reproof is something too round. I should be angry with you, if the
time were convenient.

WILLIAMS.
Let it be a quarrel between us if you live.

KING HENRY.
I embrace it.

WILLIAMS.
How shall I know thee again?

KING HENRY.
Give me any gage of thine, and I will wear it in my bonnet; then, if
ever thou dar’st acknowledge it, I will make it my quarrel.

WILLIAMS.
Here’s my glove; give me another of thine.

KING HENRY.
There.

WILLIAMS.
This will I also wear in my cap. If ever thou come to me and say, after
tomorrow, “This is my glove,” by this hand I will take thee a box on
the ear.

KING HENRY.
If ever I live to see it, I will challenge it.

WILLIAMS.
Thou dar’st as well be hang’d.

KING HENRY.
Well, I will do it, though I take thee in the King’s company.

WILLIAMS.
Keep thy word; fare thee well.

BATES.
Be friends, you English fools, be friends. We have French quarrels
enough, if you could tell how to reckon.

KING HENRY.
Indeed, the French may lay twenty French crowns to one they will beat
us, for they bear them on their shoulders; but it is no English treason
to cut French crowns, and tomorrow the King himself will be a clipper.

 [_Exeunt soldiers._]

 Upon the King! Let us our lives, our souls,
Our debts, our careful wives,
Our children, and our sins lay on the King!
We must bear all. O hard condition,
Twin-born with greatness, subject to the breath
Of every fool, whose sense no more can feel
But his own wringing! What infinite heart’s ease
Must kings neglect, that private men enjoy!
And what have kings, that privates have not too,
Save ceremony, save general ceremony?
And what art thou, thou idol Ceremony?
What kind of god art thou, that suffer’st more
Of mortal griefs than do thy worshippers?
What are thy rents? What are thy comings in?
O Ceremony, show me but thy worth!
What is thy soul of adoration?
Art thou aught else but place, degree, and form,
Creating awe and fear in other men?
Wherein thou art less happy being fear’d
Than they in fearing.
What drink’st thou oft, instead of homage sweet,
But poison’d flattery? O, be sick, great greatness,
And bid thy Ceremony give thee cure!
Think’st thou the fiery fever will go out
With titles blown from adulation?
Will it give place to flexure and low bending?
Canst thou, when thou command’st the beggar’s knee,
Command the health of it? No, thou proud dream,
That play’st so subtly with a king’s repose;
I am a king that find thee, and I know
’Tis not the balm, the sceptre, and the ball,
The sword, the mace, the crown imperial,
The intertissued robe of gold and pearl,
The farced title running ’fore the King,
The throne he sits on, nor the tide of pomp
That beats upon the high shore of this world,
No, not all these, thrice-gorgeous Ceremony,—
Not all these, laid in bed majestical,
Can sleep so soundly as the wretched slave,
Who with a body fill’d and vacant mind
Gets him to rest, cramm’d with distressful bread,
Never sees horrid night, the child of hell,
But, like a lackey, from the rise to set
Sweats in the eye of Phoebus, and all night
Sleeps in Elysium; next day after dawn,
Doth rise and help Hyperion to his horse,
And follows so the ever-running year,
With profitable labour, to his grave:
And, but for ceremony, such a wretch,
Winding up days with toil and nights with sleep,
Had the fore-hand and vantage of a king.
The slave, a member of the country’s peace,
Enjoys it, but in gross brain little wots
What watch the King keeps to maintain the peace,
Whose hours the peasant best advantages.

 Enter Erpingham.

ERPINGHAM.
My lord, your nobles, jealous of your absence,
Seek through your camp to find you.

KING HENRY.
Good old knight,
Collect them all together at my tent.
I’ll be before thee.

ERPINGHAM.
I shall do’t, my lord.

 [_Exit._]

KING HENRY.
O God of battles! steel my soldiers’ hearts.
Possess them not with fear. Take from them now
The sense of reckoning, if the opposed numbers
Pluck their hearts from them. Not today, O Lord,
O, not today, think not upon the fault
My father made in compassing the crown!
I Richard’s body have interred new,
And on it have bestow’d more contrite tears
Than from it issued forced drops of blood.
Five hundred poor I have in yearly pay,
Who twice a day their wither’d hands hold up
Toward heaven, to pardon blood; and I have built
Two chantries, where the sad and solemn priests
Sing still for Richard’s soul. More will I do;
Though all that I can do is nothing worth,
Since that my penitence comes after all,
Imploring pardon.

 Enter Gloucester.

GLOUCESTER.
My liege!

KING HENRY.
My brother Gloucester’s voice? Ay;
I know thy errand, I will go with thee.
The day, my friends, and all things stay for me.

 [_Exeunt._]

SCENE II. The French camp.

 Enter the Dauphin, Orleans, Rambures and others.

ORLEANS.
The sun doth gild our armour; up, my lords!

DAUPHIN.
_Monte à cheval!_ My horse, _varlet! laquais_, ha!

ORLEANS.
O brave spirit!

DAUPHIN.
_Via, les eaux et terre!_

ORLEANS.
_Rien puis? L’air et feu?_

DAUPHIN.
_Cieux_, cousin Orleans.

 Enter Constable.

Now, my Lord Constable!

CONSTABLE.
Hark, how our steeds for present service neigh!

DAUPHIN.
Mount them, and make incision in their hides,
That their hot blood may spin in English eyes,
And dout them with superfluous courage, ha!

RAMBURES.
What, will you have them weep our horses’ blood?
How shall we, then, behold their natural tears?

 Enter a Messenger.

MESSENGER.
The English are embattl’d, you French peers.

CONSTABLE.
To horse, you gallant princes! straight to horse!
Do but behold yon poor and starved band,
And your fair show shall suck away their souls,
Leaving them but the shales and husks of men.
There is not work enough for all our hands;
Scarce blood enough in all their sickly veins
To give each naked curtle-axe a stain,
That our French gallants shall today draw out,
And sheathe for lack of sport. Let us but blow on them,
The vapour of our valour will o’erturn them.
’Tis positive ’gainst all exceptions, lords,
That our superfluous lackeys and our peasants,
Who in unnecessary action swarm
About our squares of battle, were enough
To purge this field of such a hilding foe,
Though we upon this mountain’s basis by
Took stand for idle speculation,
But that our honours must not. What’s to say?
A very little little let us do,
And all is done. Then let the trumpets sound
The tucket sonance and the note to mount;
For our approach shall so much dare the field
That England shall crouch down in fear and yield.

 Enter Grandpré.

GRANDPRÉ.
Why do you stay so long, my lords of France?
Yond island carrions, desperate of their bones,
Ill-favouredly become the morning field.
Their ragged curtains poorly are let loose,
And our air shakes them passing scornfully.
Big Mars seems bankrupt in their beggar’d host,
And faintly through a rusty beaver peeps;
The horsemen sit like fixed candlesticks
With torch-staves in their hand; and their poor jades
Lob down their heads, drooping the hides and hips,
The gum down-roping from their pale-dead eyes,
And in their pale dull mouths the gimmal bit
Lies foul with chew’d grass, still, and motionless;
And their executors, the knavish crows,
Fly o’er them, all impatient for their hour.
Description cannot suit itself in words
To demonstrate the life of such a battle,
In life so lifeless as it shows itself.

CONSTABLE.
They have said their prayers, and they stay for death.

DAUPHIN.
Shall we go send them dinners and fresh suits
And give their fasting horses provender,
And after fight with them?

CONSTABLE.
I stay but for my guard; on to the field!
I will the banner from a trumpet take,
And use it for my haste. Come, come, away!
The sun is high, and we outwear the day.

 [_Exeunt._]

SCENE III. The English camp.

 Enter Gloucester, Bedford, Exeter, Erpingham, with all his host:
 Salisbury and Westmorland.

GLOUCESTER.
Where is the King?

BEDFORD.
The King himself is rode to view their battle.

WESTMORLAND.
Of fighting men they have full three-score thousand.

EXETER.
There’s five to one; besides, they all are fresh.

SALISBURY.
God’s arm strike with us! ’tis a fearful odds.
God be wi’ you, princes all; I’ll to my charge.
If we no more meet till we meet in heaven,
Then, joyfully, my noble Lord of Bedford,
My dear Lord Gloucester, and my good Lord Exeter,
And my kind kinsman, warriors all, adieu!

BEDFORD.
Farewell, good Salisbury, and good luck go with thee!

EXETER.
Farewell, kind lord; fight valiantly today!
And yet I do thee wrong to mind thee of it,
For thou art fram’d of the firm truth of valour.

 [_Exit Salisbury._]

BEDFORD.
He is as full of valour as of kindness,
Princely in both.

 Enter the King.

WESTMORLAND.
O that we now had here
But one ten thousand of those men in England
That do no work today!

KING.
What’s he that wishes so?
My cousin Westmorland? No, my fair cousin.
If we are mark’d to die, we are enough
To do our country loss; and if to live,
The fewer men, the greater share of honour.
God’s will! I pray thee, wish not one man more.
By Jove, I am not covetous for gold,
Nor care I who doth feed upon my cost;
It yearns me not if men my garments wear;
Such outward things dwell not in my desires;
But if it be a sin to covet honour,
I am the most offending soul alive.
No, faith, my coz, wish not a man from England.
God’s peace! I would not lose so great an honour
As one man more, methinks, would share from me
For the best hope I have. O, do not wish one more!
Rather proclaim it, Westmorland, through my host,
That he which hath no stomach to this fight,
Let him depart. His passport shall be made,
And crowns for convoy put into his purse.
We would not die in that man’s company
That fears his fellowship to die with us.
This day is call’d the feast of Crispian.
He that outlives this day, and comes safe home,
Will stand a tip-toe when this day is named,
And rouse him at the name of Crispian.
He that shall live this day, and see old age,
Will yearly on the vigil feast his neighbours,
And say, “Tomorrow is Saint Crispian.”
Then will he strip his sleeve and show his scars,
And say, “These wounds I had on Crispian’s day.”
Old men forget; yet all shall be forgot,
But he’ll remember with advantages
What feats he did that day. Then shall our names,
Familiar in his mouth as household words,
Harry the King, Bedford, and Exeter,
Warwick and Talbot, Salisbury and Gloucester,
Be in their flowing cups freshly remembered.
This story shall the good man teach his son;
And Crispin Crispian shall ne’er go by,
From this day to the ending of the world,
But we in it shall be remembered,
We few, we happy few, we band of brothers.
For he today that sheds his blood with me
Shall be my brother; be he ne’er so vile,
This day shall gentle his condition;
And gentlemen in England now abed
Shall think themselves accurs’d they were not here,
And hold their manhoods cheap whiles any speaks
That fought with us upon Saint Crispin’s day.

 Enter Salisbury.

SALISBURY.
My sovereign lord, bestow yourself with speed.
The French are bravely in their battles set,
And will with all expedience charge on us.

KING HENRY.
All things are ready, if our minds be so.

WESTMORLAND.
Perish the man whose mind is backward now!

KING HENRY.
Thou dost not wish more help from England, coz?

WESTMORLAND.
God’s will! my liege, would you and I alone,
Without more help, could fight this royal battle!

KING HENRY.
Why, now thou hast unwish’d five thousand men,
Which likes me better than to wish us one.
You know your places. God be with you all!

 Tucket. Enter Montjoy.

MONTJOY.
Once more I come to know of thee, King Harry,
If for thy ransom thou wilt now compound,
Before thy most assured overthrow;
For certainly thou art so near the gulf,
Thou needs must be englutted. Besides, in mercy,
The Constable desires thee thou wilt mind
Thy followers of repentance; that their souls
May make a peaceful and a sweet retire
From off these fields, where, wretches, their poor bodies
Must lie and fester.

KING HENRY.
Who hath sent thee now?

MONTJOY.
The Constable of France.

KING HENRY.
I pray thee, bear my former answer back:
Bid them achieve me and then sell my bones.
Good God! why should they mock poor fellows thus?
The man that once did sell the lion’s skin
While the beast liv’d, was kill’d with hunting him.
A many of our bodies shall no doubt
Find native graves, upon the which, I trust,
Shall witness live in brass of this day’s work;
And those that leave their valiant bones in France,
Dying like men, though buried in your dunghills,
They shall be fam’d; for there the sun shall greet them,
And draw their honours reeking up to heaven;
Leaving their earthly parts to choke your clime,
The smell whereof shall breed a plague in France.
Mark then abounding valour in our English,
That being dead, like to the bullet’s grazing,
Break out into a second course of mischief,
Killing in relapse of mortality.
Let me speak proudly: tell the Constable
We are but warriors for the working-day.
Our gayness and our gilt are all besmirch’d
With rainy marching in the painful field;
There’s not a piece of feather in our host—
Good argument, I hope, we will not fly—
And time hath worn us into slovenry;
But, by the mass, our hearts are in the trim;
And my poor soldiers tell me, yet ere night
They’ll be in fresher robes, or they will pluck
The gay new coats o’er the French soldiers’ heads
And turn them out of service. If they do this—
As, if God please, they shall,—my ransom then
Will soon be levied. Herald, save thou thy labour.
Come thou no more for ransom, gentle herald.
They shall have none, I swear, but these my joints;
Which if they have as I will leave ’em them,
Shall yield them little, tell the Constable.

MONTJOY.
I shall, King Harry. And so fare thee well;
Thou never shalt hear herald any more.

 [_Exit._]

KING HENRY.
I fear thou’lt once more come again for ransom.

 Enter York.

YORK.
My lord, most humbly on my knee I beg
The leading of the vaward.

KING HENRY.
Take it, brave York. Now, soldiers, march away;
And how thou pleasest, God, dispose the day!

 [_Exeunt._]

SCENE IV. The field of battle.

 Alarum. Excursions. Enter Pistol, French Soldier and Boy.

PISTOL.
Yield, cur!

FRENCH SOLDIER.
_Je pense que vous êtes le gentilhomme de bonne qualité._

PISTOL.
_Qualité? Caleno custore me!_
Art thou a gentleman?
What is thy name? Discuss.

FRENCH SOLDIER.
_O Seigneur Dieu!_

PISTOL.
O, Signieur Dew should be a gentleman.
Perpend my words, O Signieur Dew, and mark:
O Signieur Dew, thou diest on point of fox,
Except, O signieur, thou do give to me
Egregious ransom.

FRENCH SOLDIER.
_O, prenez miséricorde! Ayez pitié de moi!_

PISTOL.
Moy shall not serve; I will have forty moys,
Or I will fetch thy rim out at thy throat
In drops of crimson blood.

FRENCH SOLDIER.
_Est-il impossible d’échapper la force de ton bras?_

PISTOL.
Brass, cur!
Thou damned and luxurious mountain goat,
Offer’st me brass?

FRENCH SOLDIER.
_O pardonnez-moi!_

PISTOL.
Say’st thou me so? Is that a ton of moys?
Come hither, boy; ask me this slave in French
What is his name.

BOY.
_Écoutez. Comment êtes-vous appelé?_

FRENCH SOLDIER.
_Monsieur le Fer._

BOY.
He says his name is Master Fer.

PISTOL.
Master Fer! I’ll fer him, and firk him, and ferret him.
Discuss the same in French unto him.

BOY.
I do not know the French for fer, and ferret, and firk.

PISTOL.
Bid him prepare; for I will cut his throat.

FRENCH SOLDIER.
_Que dit-il, monsieur?_

BOY.
_Il me commande à vous dire que vous faites vous prêt, car ce soldat
ici est disposé tout à cette heure de couper votre gorge._

PISTOL.
Owy, cuppele gorge, permafoy,
Peasant, unless thou give me crowns, brave crowns;
Or mangled shalt thou be by this my sword.

FRENCH SOLDIER.
_O, je vous supplie, pour l’amour de Dieu, me pardonner! Je suis le
gentilhomme de bonne maison; gardez ma vie, et je vous donnerai deux
cents écus._

PISTOL.
What are his words?

BOY.
He prays you to save his life. He is a gentleman of a good house; and
for his ransom he will give you two hundred crowns.

PISTOL.
Tell him my fury shall abate, and I
The crowns will take.

FRENCH SOLDIER.
_Petit monsieur, que dit-il?_

BOY.
_Encore qu’il est contre son jurement de pardonner aucun prisonnier;
néanmoins, pour les écus que vous lui avez promis, il est content à
vous donner la liberté, le franchisement._

FRENCH SOLDIER.
_Sur mes genoux je vous donne mille remerciements; et je m’estime
heureux que je suis tombé entre les mains d’un chevalier, je pense, le
plus brave, vaillant, et très distingué seigneur d’Angleterre._

PISTOL.
Expound unto me, boy.

BOY.
He gives you upon his knees, a thousand thanks; and he esteems himself
happy that he hath fallen into the hands of one, as he thinks, the most
brave, valorous, and thrice-worthy _seigneur_ of England.

PISTOL.
As I suck blood, I will some mercy show.
Follow me!

BOY.
_Suivez-vous le grand capitaine._

 [_Exeunt Pistol and French Soldier._]

I did never know so full a voice issue from so empty a heart; but the
saying is true, “The empty vessel makes the greatest sound.” Bardolph
and Nym had ten times more valour than this roaring devil i’ the old
play, that everyone may pare his nails with a wooden dagger; and they
are both hang’d; and so would this be, if he durst steal anything
adventurously. I must stay with the lackeys with the luggage of our
camp. The French might have a good prey of us, if he knew of it; for
there is none to guard it but boys.

 [_Exit._]

SCENE V. Another part of the field.

 Enter Constable, Orleans, Bourbon, Dauphin and Rambures.

CONSTABLE.
_O diable!_

ORLEANS.
_O Seigneur! le jour est perdu, tout est perdu!_

DAUPHIN.
_Mort de ma vie!_ all is confounded, all!
Reproach and everlasting shame
Sits mocking in our plumes.

 [_A short alarum._]

_O méchante Fortune!_ Do not run away.

CONSTABLE.
Why, all our ranks are broke.

DAUPHIN.
O perdurable shame! Let’s stab ourselves,
Be these the wretches that we play’d at dice for?

ORLEANS.
Is this the king we sent to for his ransom?

BOURBON.
Shame and eternal shame, nothing but shame!
Let’s die in honour! Once more back again!
And he that will not follow Bourbon now,
Let him go hence, and with his cap in hand,
Like a base pandar, hold the chamber door
Whilst by a slave, no gentler than my dog,
His fairest daughter is contaminated.

CONSTABLE.
Disorder, that hath spoil’d us, friend us now!
Let us on heaps go offer up our lives.

ORLEANS.
We are enough yet living in the field
To smother up the English in our throngs,
If any order might be thought upon.

BOURBON.
The devil take order now! I’ll to the throng.
Let life be short, else shame will be too long.

 [_Exeunt._]

SCENE VI. Another part of the field.

 Alarum. Enter King Henry and his train, with prisoners.

KING HENRY.
Well have we done, thrice valiant countrymen.
But all’s not done; yet keep the French the field.

EXETER.
The Duke of York commends him to your Majesty.

KING HENRY.
Lives he, good uncle? Thrice within this hour
I saw him down; thrice up again, and fighting.
From helmet to the spur all blood he was.

EXETER.
In which array, brave soldier, doth he lie,
Larding the plain; and by his bloody side,
Yoke-fellow to his honour-owing wounds,
The noble Earl of Suffolk also lies.
Suffolk first died; and York, all haggled over,
Comes to him, where in gore he lay insteeped,
And takes him by the beard; kisses the gashes
That bloodily did yawn upon his face.
He cries aloud, “Tarry, my cousin Suffolk!
My soul shall thine keep company to heaven;
Tarry, sweet soul, for mine, then fly abreast,
As in this glorious and well-foughten field
We kept together in our chivalry.”
Upon these words I came and cheer’d him up.
He smil’d me in the face, raught me his hand,
And, with a feeble gripe, says, “Dear my lord,
Commend my service to my sovereign.”
So did he turn and over Suffolk’s neck
He threw his wounded arm and kiss’d his lips;
And so espous’d to death, with blood he seal’d
A testament of noble-ending love.
The pretty and sweet manner of it forc’d
Those waters from me which I would have stopp’d;
But I had not so much of man in me,
And all my mother came into mine eyes
And gave me up to tears.

KING HENRY.
I blame you not;
For, hearing this, I must perforce compound
With mistful eyes, or they will issue too.

 [_Alarum._]

But hark! what new alarum is this same?
The French have reinforc’d their scatter’d men.
Then every soldier kill his prisoners;
Give the word through.

 [_Exeunt._]

SCENE VII. Another part of the field.

 Enter Fluellen and Gower.

FLUELLEN.
Kill the poys and the luggage! ’Tis expressly against the law of arms.
’Tis as arrant a piece of knavery, mark you now, as can be offer’t; in
your conscience, now, is it not?

GOWER.
’Tis certain there’s not a boy left alive; and the cowardly rascals
that ran from the battle ha’ done this slaughter. Besides, they have
burned and carried away all that was in the King’s tent; wherefore the
King, most worthily, hath caus’d every soldier to cut his prisoner’s
throat. O, ’tis a gallant king!

FLUELLEN.
Ay, he was porn at Monmouth, Captain Gower. What call you the town’s
name where Alexander the Pig was born?

GOWER.
Alexander the Great.

FLUELLEN.
Why, I pray you, is not pig great? The pig, or the great, or the
mighty, or the huge, or the magnanimous, are all one reckonings, save
the phrase is a little variations.

GOWER.
I think Alexander the Great was born in Macedon. His father was called
Philip of Macedon, as I take it.

FLUELLEN.
I think it is in Macedon where Alexander is porn. I tell you, Captain,
if you look in the maps of the ’orld, I warrant you sall find, in the
comparisons between Macedon and Monmouth, that the situations, look
you, is both alike. There is a river in Macedon; and there is also
moreover a river at Monmouth; it is call’d Wye at Monmouth; but it is
out of my prains what is the name of the other river; but ’tis all one,
’tis alike as my fingers is to my fingers, and there is salmons in
both. If you mark Alexander’s life well, Harry of Monmouth’s life is
come after it indifferent well; for there is figures in all things.
Alexander, God knows, and you know, in his rages, and his furies, and
his wraths, and his cholers, and his moods, and his displeasures, and
his indignations, and also being a little intoxicates in his prains,
did, in his ales and his angers, look you, kill his best friend,
Cleitus.

GOWER.
Our King is not like him in that. He never kill’d any of his friends.

FLUELLEN.
It is not well done, mark you now, to take the tales out of my mouth,
ere it is made and finished. I speak but in the figures and comparisons
of it. As Alexander kill’d his friend Cleitus, being in his ales and
his cups; so also Harry Monmouth, being in his right wits and his good
judgements, turn’d away the fat knight with the great belly doublet. He
was full of jests, and gipes, and knaveries, and mocks; I have forgot
his name.

GOWER.
Sir John Falstaff.

FLUELLEN.
That is he. I’ll tell you there is good men porn at Monmouth.

GOWER.
Here comes his Majesty.

 Alarum. Enter King Henry and forces; Warwick, Gloucester, Exeter with
 prisoners. Flourish.

KING HENRY.
I was not angry since I came to France
Until this instant. Take a trumpet, herald;
Ride thou unto the horsemen on yond hill.
If they will fight with us, bid them come down,
Or void the field; they do offend our sight.
If they’ll do neither, we will come to them,
And make them skirr away, as swift as stones
Enforced from the old Assyrian slings.
Besides, we’ll cut the throats of those we have,
And not a man of them that we shall take
Shall taste our mercy. Go and tell them so.

 Enter Montjoy.

EXETER.
Here comes the herald of the French, my liege.

GLOUCESTER.
His eyes are humbler than they us’d to be.

KING HENRY.
How now! what means this, herald? Know’st thou not
That I have fin’d these bones of mine for ransom?
Com’st thou again for ransom?

MONTJOY.
No, great King;
I come to thee for charitable license,
That we may wander o’er this bloody field
To book our dead, and then to bury them;
To sort our nobles from our common men.
For many of our princes—woe the while!—
Lie drown’d and soak’d in mercenary blood;
So do our vulgar drench their peasant limbs
In blood of princes; and their wounded steeds
Fret fetlock deep in gore, and with wild rage
Yerk out their armed heels at their dead masters,
Killing them twice. O, give us leave, great King,
To view the field in safety, and dispose
Of their dead bodies!

KING HENRY.
I tell thee truly, herald,
I know not if the day be ours or no;
For yet a many of your horsemen peer
And gallop o’er the field.

MONTJOY.
The day is yours.

KING HENRY.
Praised be God, and not our strength, for it!
What is this castle call’d that stands hard by?

MONTJOY.
They call it Agincourt.

KING HENRY.
Then call we this the field of Agincourt,
Fought on the day of Crispin Crispianus.

FLUELLEN.
Your grandfather of famous memory, an’t please your Majesty, and your
great-uncle Edward the Plack Prince of Wales, as I have read in the
chronicles, fought a most prave pattle here in France.

KING HENRY.
They did, Fluellen.

FLUELLEN.
Your Majesty says very true. If your Majesties is rememb’red of it, the
Welshmen did good service in garden where leeks did grow, wearing leeks
in their Monmouth caps; which, your Majesty know, to this hour is an
honourable badge of the service; and I do believe your Majesty takes no
scorn to wear the leek upon Saint Tavy’s day.

KING HENRY.
I wear it for a memorable honour;
For I am Welsh, you know, good countryman.

FLUELLEN.
All the water in Wye cannot wash your Majesty’s Welsh plood out of your
pody, I can tell you that. Got pless it and preserve it, as long as it
pleases His grace, and His majesty too!

KING HENRY.
Thanks, good my countryman.

FLUELLEN.
By Jeshu, I am your Majesty’s countryman, I care not who know it. I
will confess it to all the ’orld. I need not be asham’d of your
Majesty, praised be God, so long as your Majesty is an honest man.

KING HENRY.
God keep me so!

 Enter Williams.

Our heralds go with him;
Bring me just notice of the numbers dead
On both our parts. Call yonder fellow hither.

 [_Exeunt Heralds with Montjoy._]

EXETER.
Soldier, you must come to the King.

KING HENRY.
Soldier, why wear’st thou that glove in thy cap?

WILLIAMS.
An’t please your Majesty, ’tis the gage of one that I should fight
withal, if he be alive.

KING HENRY.
An Englishman?

WILLIAMS.
An’t please your Majesty, a rascal that swagger’d with me last night;
who, if alive and ever dare to challenge this glove, I have sworn to
take him a box o’ the ear; or if I can see my glove in his cap, which
he swore, as he was a soldier, he would wear if alive, I will strike it
out soundly.

KING HENRY.
What think you, Captain Fluellen, is it fit this soldier keep his oath?

FLUELLEN.
He is a craven and a villain else, an’t please your Majesty, in my
conscience.

KING HENRY.
It may be his enemy is a gentlemen of great sort, quite from the answer
of his degree.

FLUELLEN.
Though he be as good a gentleman as the devil is, as Lucifier and
Belzebub himself, it is necessary, look your Grace, that he keep his
vow and his oath. If he be perjur’d, see you now, his reputation is as
arrant a villain and a Jacksauce, as ever his black shoe trod upon
God’s ground and His earth, in my conscience, la!

KING HENRY.
Then keep thy vow, sirrah, when thou meet’st the fellow.

WILLIAMS.
So I will, my liege, as I live.

KING HENRY.
Who serv’st thou under?

WILLIAMS.
Under Captain Gower, my liege.

FLUELLEN.
Gower is a good captain, and is good knowledge and literatured in the
wars.

KING HENRY.
Call him hither to me, soldier.

WILLIAMS.
I will, my liege.

 [_Exit._]

KING HENRY.
Here, Fluellen; wear thou this favour for me and stick it in thy cap.
When Alençon and myself were down together, I pluck’d this glove from
his helm. If any man challenge this, he is a friend to Alençon, and an
enemy to our person. If thou encounter any such, apprehend him, an thou
dost me love.

FLUELLEN.
Your Grace does me as great honours as can be desir’d in the hearts of
his subjects. I would fain see the man, that has but two legs, that
shall find himself aggrief’d at this glove; that is all. But I would
fain see it once, an please God of His grace that I might see.

KING HENRY.
Know’st thou Gower?

FLUELLEN.
He is my dear friend, an please you.

KING HENRY.
Pray thee, go seek him, and bring him to my tent.

FLUELLEN.
I will fetch him.

 [_Exit._]

KING HENRY.
My Lord of Warwick, and my brother Gloucester,
Follow Fluellen closely at the heels.
The glove which I have given him for a favour
May haply purchase him a box o’ the ear.
It is the soldier’s; I by bargain should
Wear it myself. Follow, good cousin Warwick.
If that the soldier strike him, as I judge
By his blunt bearing he will keep his word,
Some sudden mischief may arise of it;
For I do know Fluellen valiant
And, touch’d with choler, hot as gunpowder,
And quickly will return an injury.
Follow, and see there be no harm between them.
Go you with me, uncle of Exeter.

 [_Exeunt._]

SCENE VIII. Before King Henry’s pavilion.

 Enter Gower and Williams.

WILLIAMS.
I warrant it is to knight you, Captain.

 Enter Fluellen.

FLUELLEN.
God’s will and his pleasure, captain, I beseech you now, come apace to
the King. There is more good toward you peradventure than is in your
knowledge to dream of.

WILLIAMS.
Sir, know you this glove?

FLUELLEN.
Know the glove! I know the glove is a glove.

WILLIAMS.
I know this; and thus I challenge it.

 [_Strikes him._]

FLUELLEN.
’Sblood! an arrant traitor as any is in the universal world, or in
France, or in England!

GOWER.
How now, sir! you villain!

WILLIAMS.
Do you think I’ll be forsworn?

FLUELLEN.
Stand away, Captain Gower. I will give treason his payment into plows,
I warrant you.

WILLIAMS.
I am no traitor.

FLUELLEN.
That’s a lie in thy throat. I charge you in his Majesty’s name,
apprehend him; he’s a friend of the Duke Alençon’s.

 Enter Warwick and Gloucester.

WARWICK.
How now, how now! what’s the matter?

FLUELLEN.
My lord of Warwick, here is—praised be God for it!—a most contagious
treason come to light, look you, as you shall desire in a summer’s day.
Here is his Majesty.

 Enter King Henry and Exeter.

KING HENRY.
How now! what’s the matter?

FLUELLEN.
My liege, here is a villain and a traitor, that, look your Grace, has
struck the glove which your Majesty is take out of the helmet of
Alençon.

WILLIAMS.
My liege, this was my glove; here is the fellow of it; and he that I
gave it to in change promis’d to wear it in his cap. I promis’d to
strike him, if he did. I met this man with my glove in his cap, and I
have been as good as my word.

FLUELLEN.
Your Majesty hear now, saving your Majesty’s manhood, what an arrant,
rascally, beggarly, lousy knave it is. I hope your Majesty is pear me
testimony and witness, and will avouchment, that this is the glove of
Alençon that your Majesty is give me; in your conscience, now?

KING HENRY.
Give me thy glove, soldier. Look, here is the fellow of it.
’Twas I, indeed, thou promisedst to strike;
And thou hast given me most bitter terms.

FLUELLEN.
An it please your Majesty, let his neck answer for it, if there is any
martial law in the world.

KING HENRY.
How canst thou make me satisfaction?

WILLIAMS.
All offences, my lord, come from the heart. Never came any from mine
that might offend your Majesty.

KING HENRY.
It was ourself thou didst abuse.

WILLIAMS.
Your Majesty came not like yourself. You appear’d to me but as a common
man; witness the night, your garments, your lowliness; and what your
Highness suffer’d under that shape, I beseech you take it for your own
fault and not mine; for had you been as I took you for, I made no
offence; therefore, I beseech your Highness, pardon me.

KING HENRY.
Here, uncle Exeter, fill this glove with crowns,
And give it to this fellow. Keep it, fellow;
And wear it for an honour in thy cap
Till I do challenge it. Give him his crowns;
And, captain, you must needs be friends with him.

FLUELLEN.
By this day and this light, the fellow has mettle enough in his belly.
Hold, there is twelve pence for you; and I pray you to serve God, and
keep you out of prawls, and prabbles, and quarrels, and dissensions,
and, I warrant you, it is the better for you.

WILLIAMS.
I will none of your money.

FLUELLEN.
It is with a good will; I can tell you, it will serve you to mend your
shoes. Come, wherefore should you be so pashful? Your shoes is not so
good. ’Tis a good silling, I warrant you, or I will change it.

 Enter an English Herald.

KING HENRY.
Now, herald, are the dead numb’red?

HERALD.
Here is the number of the slaught’red French.

KING HENRY.
What prisoners of good sort are taken, uncle?

EXETER.
Charles Duke of Orleans, nephew to the King;
John Duke of Bourbon, and Lord Boucicault:
Of other lords and barons, knights and squires,
Full fifteen hundred, besides common men.

KING HENRY.
This note doth tell me of ten thousand French
That in the field lie slain; of princes, in this number,
And nobles bearing banners, there lie dead
One hundred twenty-six; added to these,
Of knights, esquires, and gallant gentlemen,
Eight thousand and four hundred; of the which,
Five hundred were but yesterday dubb’d knights;
So that, in these ten thousand they have lost,
There are but sixteen hundred mercenaries;
The rest are princes, barons, lords, knights, squires,
And gentlemen of blood and quality.
The names of those their nobles that lie dead:
Charles Delabreth, High Constable of France;
Jacques of Chatillon, Admiral of France;
The master of the Crossbows, Lord Rambures;
Great Master of France, the brave Sir Guichard Dauphin,
John, Duke of Alençon, Anthony, Duke of Brabant,
The brother to the Duke of Burgundy,
And Edward, Duke of Bar; of lusty earls,
Grandpré and Roussi, Fauconbridge and Foix,
Beaumont and Marle, Vaudemont and Lestrale.
Here was a royal fellowship of death!
Where is the number of our English dead?

 [_Herald gives him another paper._]

Edward the Duke of York, the Earl of Suffolk,
Sir Richard Ketly, Davy Gam, esquire;
None else of name; and of all other men
But five and twenty.—O God, thy arm was here;
And not to us, but to thy arm alone,
Ascribe we all! When, without stratagem,
But in plain shock and even play of battle,
Was ever known so great and little loss
On one part and on the other? Take it, God,
For it is none but thine!

EXETER.
’Tis wonderful!

KING HENRY.
Come, go we in procession to the village;
And be it death proclaimed through our host
To boast of this or take that praise from God
Which is His only.

FLUELLEN.
Is it not lawful, an please your Majesty, to tell how many is kill’d?

KING HENRY.
Yes, Captain; but with this acknowledgment,
That God fought for us.

FLUELLEN.
Yes, my conscience, He did us great good.

KING HENRY.
Do we all holy rites.
Let there be sung _Non nobis_ and _Te Deum_,
The dead with charity enclos’d in clay,
And then to Calais; and to England then,
Where ne’er from France arriv’d more happy men.

 [_Exeunt._]




ACT V


 Enter Chorus.

CHORUS.
Vouchsafe to those that have not read the story,
That I may prompt them; and of such as have,
I humbly pray them to admit the excuse
Of time, of numbers, and due course of things,
Which cannot in their huge and proper life
Be here presented. Now we bear the King
Toward Calais; grant him there; there seen,
Heave him away upon your winged thoughts
Athwart the sea. Behold, the English beach
Pales in the flood with men, with wives and boys,
Whose shouts and claps out-voice the deep-mouth’d sea,
Which like a mighty whiffler ’fore the King
Seems to prepare his way. So let him land,
And solemnly see him set on to London.
So swift a pace hath thought that even now
You may imagine him upon Blackheath,
Where that his lords desire him to have borne
His bruised helmet and his bended sword
Before him through the city. He forbids it,
Being free from vainness and self-glorious pride;
Giving full trophy, signal, and ostent
Quite from himself to God. But now behold,
In the quick forge and working-house of thought,
How London doth pour out her citizens!
The mayor and all his brethren in best sort,
Like to the senators of th’ antique Rome,
With the plebeians swarming at their heels,
Go forth and fetch their conquering Caesar in;
As, by a lower but loving likelihood,
Were now the general of our gracious empress,
As in good time he may, from Ireland coming,
Bringing rebellion broached on his sword,
How many would the peaceful city quit,
To welcome him! Much more, and much more cause,
Did they this Harry. Now in London place him;
As yet the lamentation of the French
Invites the King of England’s stay at home,
The Emperor’s coming in behalf of France,
To order peace between them;—and omit
All the occurrences, whatever chanc’d,
Till Harry’s back-return again to France.
There must we bring him; and myself have play’d
The interim, by rememb’ring you ’tis past.
Then brook abridgement, and your eyes advance
After your thoughts, straight back again to France.

 [_Exit._]

SCENE I. France. The English camp.

 Enter Fluellen and Gower.

GOWER.
Nay, that’s right; but why wear you your leek today?
Saint Davy’s day is past.

FLUELLEN.
There is occasions and causes why and wherefore in all things. I will
tell you ass my friend, Captain Gower. The rascally, scald, beggarly,
lousy, pragging knave, Pistol, which you and yourself and all the world
know to be no petter than a fellow, look you now, of no merits, he is
come to me and prings me pread and salt yesterday, look you, and bid me
eat my leek. It was in a place where I could not breed no contention
with him; but I will be so bold as to wear it in my cap till I see him
once again, and then I will tell him a little piece of my desires.

 Enter Pistol.

GOWER.
Why, here he comes, swelling like a turkey-cock.

FLUELLEN.
’Tis no matter for his swellings nor his turkey-cocks. God pless you,
Anchient Pistol! you scurvy, lousy knave, God pless you!

PISTOL.
Ha! art thou bedlam? Dost thou thirst, base Trojan,
To have me fold up Parca’s fatal web?
Hence! I am qualmish at the smell of leek.

FLUELLEN.
I peseech you heartily, scurfy, lousy knave, at my desires, and my
requests, and my petitions, to eat, look you, this leek. Because, look
you, you do not love it, nor your affections and your appetites and
your digestions does not agree with it, I would desire you to eat it.

PISTOL.
Not for Cadwallader and all his goats.

FLUELLEN.
There is one goat for you. [_Strikes him._] Will you be so good, scald
knave, as eat it?

PISTOL.
Base Trojan, thou shalt die.

FLUELLEN.
You say very true, scald knave, when God’s will is. I will desire you
to live in the mean time, and eat your victuals. Come, there is sauce
for it. [_Strikes him._] You call’d me yesterday mountain-squire; but I
will make you today a squire of low degree. I pray you, fall to; if you
can mock a leek, you can eat a leek.

GOWER.
Enough, captain; you have astonish’d him.

FLUELLEN.
I say, I will make him eat some part of my leek, or I will peat his
pate four days. Bite, I pray you; it is good for your green wound and
your ploody coxcomb.

PISTOL.
Must I bite?

FLUELLEN.
Yes, certainly, and out of doubt and out of question too, and
ambiguities.

PISTOL.
By this leek, I will most horribly revenge. I eat and eat, I swear—

FLUELLEN.
Eat, I pray you. Will you have some more sauce to your leek? There is
not enough leek to swear by.

PISTOL.
Quiet thy cudgel; thou dost see I eat.

FLUELLEN.
Much good do you, scald knave, heartily. Nay, pray you, throw none
away; the skin is good for your broken coxcomb. When you take occasions
to see leeks hereafter, I pray you, mock at ’em; that is all.

PISTOL.
Good.

FLUELLEN.
Ay, leeks is good. Hold you, there is a groat to heal your pate.

PISTOL.
Me a groat!

FLUELLEN.
Yes, verily and in truth you shall take it; or I have another leek in
my pocket, which you shall eat.

PISTOL.
I take thy groat in earnest of revenge.

FLUELLEN.
If I owe you anything I will pay you in cudgels. You shall be a
woodmonger, and buy nothing of me but cudgels. God be wi’ you, and keep
you, and heal your pate.

 [_Exit._]

PISTOL.
All hell shall stir for this.

GOWER.
Go, go; you are a couterfeit cowardly knave. Will you mock at an
ancient tradition, begun upon an honourable respect, and worn as a
memorable trophy of predeceased valour, and dare not avouch in your
deeds any of your words? I have seen you gleeking and galling at this
gentleman twice or thrice. You thought, because he could not speak
English in the native garb, he could not therefore handle an English
cudgel. You find it otherwise; and henceforth let a Welsh correction
teach you a good English condition. Fare ye well.

 [_Exit._]

PISTOL.
Doth Fortune play the huswife with me now?
News have I, that my Doll is dead i’ the spital
Of malady of France;
And there my rendezvous is quite cut off.
Old I do wax; and from my weary limbs
Honour is cudgell’d. Well, bawd I’ll turn,
And something lean to cutpurse of quick hand.
To England will I steal, and there I’ll steal;
And patches will I get unto these cudgell’d scars,
And swear I got them in the Gallia wars.

 [_Exit._]

SCENE II. France. A royal palace.

 Enter at one door, King Henry, Exeter, Bedford, Warwick, Gloucester,
 Westmorland, Clarence, Huntingdon and other Lords. At another, Queen
 Isabel, the French King, the Princess Katharine, Alice, and other
 Ladies; the Duke of Burgundy and other French.

KING HENRY.
Peace to this meeting, wherefore we are met!
Unto our brother France, and to our sister,
Health and fair time of day; joy and good wishes
To our most fair and princely cousin Katharine;
And, as a branch and member of this royalty,
By whom this great assembly is contriv’d,
We do salute you, Duke of Burgundy;
And, princes French, and peers, health to you all!

FRENCH KING.
Right joyous are we to behold your face,
Most worthy brother England; fairly met!
So are you, princes English, every one.

QUEEN ISABEL.
So happy be the issue, brother England,
Of this good day and of this gracious meeting
As we are now glad to behold your eyes;
Your eyes, which hitherto have borne in them
Against the French that met them in their bent
The fatal balls of murdering basilisks.
The venom of such looks, we fairly hope,
Have lost their quality; and that this day
Shall change all griefs and quarrels into love.

KING HENRY.
To cry amen to that, thus we appear.

QUEEN ISABEL.
You English princes all, I do salute you.

BURGUNDY.
My duty to you both, on equal love,
Great Kings of France and England! That I have labour’d,
With all my wits, my pains, and strong endeavours,
To bring your most imperial Majesties
Unto this bar and royal interview,
Your mightiness on both parts best can witness.
Since then my office hath so far prevail’d
That, face to face and royal eye to eye,
You have congreeted, let it not disgrace me
If I demand, before this royal view,
What rub or what impediment there is,
Why that the naked, poor, and mangled Peace,
Dear nurse of arts, plenties, and joyful births,
Should not in this best garden of the world,
Our fertile France, put up her lovely visage?
Alas, she hath from France too long been chas’d,
And all her husbandry doth lie on heaps,
Corrupting in it own fertility.
Her vine, the merry cheerer of the heart,
Unpruned dies; her hedges even-pleach’d,
Like prisoners wildly overgrown with hair,
Put forth disorder’d twigs; her fallow leas
The darnel, hemlock, and rank fumitory,
Doth root upon, while that the coulter rusts
That should deracinate such savagery;
The even mead, that erst brought sweetly forth
The freckled cowslip, burnet, and green clover,
Wanting the scythe, all uncorrected, rank,
Conceives by idleness, and nothing teems
But hateful docks, rough thistles, kecksies, burs,
Losing both beauty and utility;
And as our vineyards, fallows, meads, and hedges,
Defective in their natures, grow to wildness.
Even so our houses and ourselves and children
Have lost, or do not learn for want of time,
The sciences that should become our country;
But grow like savages,—as soldiers will
That nothing do but meditate on blood,—
To swearing and stern looks, diffus’d attire,
And everything that seems unnatural.
Which to reduce into our former favour
You are assembled; and my speech entreats
That I may know the let, why gentle Peace
Should not expel these inconveniences
And bless us with her former qualities.

KING HENRY.
If, Duke of Burgundy, you would the peace,
Whose want gives growth to the imperfections
Which you have cited, you must buy that peace
With full accord to all our just demands;
Whose tenours and particular effects
You have enschedul’d briefly in your hands.

BURGUNDY.
The King hath heard them; to the which as yet
There is no answer made.

KING HENRY.
Well, then, the peace,
Which you before so urg’d, lies in his answer.

FRENCH KING.
I have but with a cursorary eye
O’erglanc’d the articles. Pleaseth your Grace
To appoint some of your council presently
To sit with us once more, with better heed
To re-survey them, we will suddenly
Pass our accept and peremptory answer.

KING HENRY.
Brother, we shall. Go, uncle Exeter,
And brother Clarence, and you, brother Gloucester,
Warwick, and Huntington, go with the King;
And take with you free power to ratify,
Augment, or alter, as your wisdoms best
Shall see advantageable for our dignity,
Anything in or out of our demands,
And we’ll consign thereto. Will you, fair sister,
Go with the princes, or stay here with us?

QUEEN ISABEL.
Our gracious brother, I will go with them.
Haply a woman’s voice may do some good,
When articles too nicely urg’d be stood on.

KING HENRY.
Yet leave our cousin Katharine here with us:
She is our capital demand, compris’d
Within the fore-rank of our articles.

QUEEN ISABEL.
She hath good leave.

 [_Exeunt all except Henry, Katharine and Alice._]

KING HENRY.
Fair Katharine, and most fair,
Will you vouchsafe to teach a soldier terms
Such as will enter at a lady’s ear
And plead his love-suit to her gentle heart?

KATHARINE.
Your Majesty shall mock me; I cannot speak your England.

KING HENRY.
O fair Katharine, if you will love me soundly with your French heart, I
will be glad to hear you confess it brokenly with your English tongue.
Do you like me, Kate?

KATHARINE.
_Pardonnez-moi_, I cannot tell wat is “like me.”

KING HENRY.
An angel is like you, Kate, and you are like an angel.

KATHARINE.
_Que dit-il? Que je suis semblable à les anges?_

ALICE.
_Oui, vraiment, sauf votre Grâce, ainsi dit-il._

KING HENRY.
I said so, dear Katharine; and I must not blush to affirm it.

KATHARINE.
_O bon Dieu! les langues des hommes sont pleines de tromperies._

KING HENRY.
What says she, fair one? That the tongues of men are full of deceits?

ALICE.
_Oui_, dat de tongues of de mans is be full of deceits: dat is de
Princess.

KING HENRY.
The Princess is the better Englishwoman. I’ faith, Kate, my wooing is
fit for thy understanding. I am glad thou canst speak no better
English; for if thou couldst, thou wouldst find me such a plain king
that thou wouldst think I had sold my farm to buy my crown. I know no
ways to mince it in love, but directly to say, “I love you”; then if
you urge me farther than to say, “Do you in faith?” I wear out my suit.
Give me your answer; i’ faith, do; and so clap hands and a bargain. How
say you, lady?

KATHARINE.
_Sauf votre honneur_, me understand well.

KING HENRY.
Marry, if you would put me to verses, or to dance for your sake, Kate,
why you undid me; for the one, I have neither words nor measure, and
for the other I have no strength in measure, yet a reasonable measure
in strength. If I could win a lady at leap-frog, or by vaulting into my
saddle with my armour on my back, under the correction of bragging be
it spoken, I should quickly leap into a wife. Or if I might buffet for
my love, or bound my horse for her favours, I could lay on like a
butcher and sit like a jack-an-apes, never off. But, before God, Kate,
I cannot look greenly, nor gasp out my eloquence, nor I have no cunning
in protestation; only downright oaths, which I never use till urg’d,
nor never break for urging. If thou canst love a fellow of this temper,
Kate, whose face is not worth sunburning, that never looks in his glass
for love of anything he sees there, let thine eye be thy cook. I speak
to thee plain soldier. If thou canst love me for this, take me; if not,
to say to thee that I shall die, is true; but for thy love, by the
Lord, no; yet I love thee too. And while thou liv’st, dear Kate, take a
fellow of plain and uncoined constancy; for he perforce must do thee
right, because he hath not the gift to woo in other places; for these
fellows of infinite tongue, that can rhyme themselves into ladies’
favours, they do always reason themselves out again. What! a speaker is
but a prater: a rhyme is but a ballad. A good leg will fall; a straight
back will stoop; a black beard will turn white; a curl’d pate will grow
bald; a fair face will wither; a full eye will wax hollow; but a good
heart, Kate, is the sun and the moon; or rather the sun and not the
moon; for it shines bright and never changes, but keeps his course
truly. If thou would have such a one, take me; and take me, take a
soldier; take a soldier, take a king. And what say’st thou then to my
love? Speak, my fair, and fairly, I pray thee.

KATHARINE.
Is it possible dat I should love de enemy of France?

KING HENRY.
No; it is not possible you should love the enemy of France, Kate; but,
in loving me, you should love the friend of France; for I love France
so well that I will not part with a village of it, I will have it all
mine; and, Kate, when France is mine and I am yours, then yours is
France and you are mine.

KATHARINE.
I cannot tell wat is dat.

KING HENRY.
No, Kate? I will tell thee in French; which I am sure will hang upon my
tongue like a new-married wife about her husband’s neck, hardly to be
shook off. _Je quand sur le possession de France, et quand vous avez le
possession de moi_,—let me see, what then? Saint Denis be my
speed!—_donc votre est France, et vous êtes mienne._ It is as easy for
me, Kate, to conquer the kingdom as to speak so much more French. I
shall never move thee in French, unless it be to laugh at me.

KATHARINE.
_Sauf votre honneur, le français que vous parlez, il est meilleur que
l’anglais lequel je parle._

KING HENRY.
No, faith, is’t not, Kate; but thy speaking of my tongue, and I thine,
most truly-falsely, must needs be granted to be much at one. But, Kate,
dost thou understand thus much English: canst thou love me?

KATHARINE.
I cannot tell.

KING HENRY.
Can any of your neighbours tell, Kate? I’ll ask them. Come, I know thou
lovest me; and at night, when you come into your closet, you’ll
question this gentlewoman about me; and I know, Kate, you will to her
dispraise those parts in me that you love with your heart. But, good
Kate, mock me mercifully; the rather, gentle princess, because I love
thee cruelly. If ever thou beest mine, Kate, as I have a saving faith
within me tells me thou shalt, I get thee with scambling, and thou must
therefore needs prove a good soldier-breeder. Shall not thou and I,
between Saint Denis and Saint George, compound a boy, half French, half
English, that shall go to Constantinople and take the Turk by the
beard? Shall we not? What say’st thou, my fair flower-de-luce?

KATHARINE.
I do not know dat.

KING HENRY.
No; ’tis hereafter to know, but now to promise. Do but now promise,
Kate, you will endeavour for your French part of such a boy; and for my
English moiety, take the word of a king and a bachelor. How answer you,
_la plus belle Katherine du monde, mon très cher et divin déesse?_

KATHARINE.
Your Majestee ’ave _fausse_ French enough to deceive de most _sage
demoiselle_ dat is _en France_.

KING HENRY.
Now, fie upon my false French! By mine honour, in true English, I love
thee, Kate; by which honour I dare not swear thou lovest me; yet my
blood begins to flatter me that thou dost, notwithstanding the poor and
untempering effect of my visage. Now, beshrew my father’s ambition! He
was thinking of civil wars when he got me; therefore was I created with
a stubborn outside, with an aspect of iron, that, when I come to woo
ladies, I fright them. But, in faith, Kate, the elder I wax, the better
I shall appear. My comfort is, that old age, that ill layer up of
beauty, can do no more spoil upon my face. Thou hast me, if thou hast
me, at the worst; and thou shalt wear me, if thou wear me, better and
better; and therefore tell me, most fair Katharine, will you have me?
Put off your maiden blushes; avouch the thoughts of your heart with the
looks of an empress; take me by the hand, and say, Harry of England, I
am thine; which word thou shalt no sooner bless mine ear withal, but I
will tell thee aloud, England is thine, Ireland is thine, France is
thine, and Henry Plantagenet is thine; who, though I speak it before
his face, if he be not fellow with the best king, thou shalt find the
best king of good fellows. Come, your answer in broken music; for thy
voice is music and thy English broken; therefore, queen of all,
Katharine, break thy mind to me in broken English. Wilt thou have me?

KATHARINE.
Dat is as it shall please _le roi mon père_.

KING HENRY.
Nay, it will please him well, Kate; it shall please him, Kate.

KATHARINE.
Den it sall also content me.

KING HENRY.
Upon that I kiss your hand, and call you my queen.

KATHARINE.
_Laissez, mon seigneur, laissez, laissez! Ma foi, je ne veux point que
vous abaissiez votre grandeur en baisant la main d’une—Notre
Seigneur!—indigne serviteur. Excusez-moi, je vous supplie, mon
très-puissant seigneur._

KING HENRY.
Then I will kiss your lips, Kate.

KATHARINE.
_Les dames et demoiselles pour être baisées devant leurs noces, il
n’est pas la coutume de France._

KING HENRY.
Madame my interpreter, what says she?

ALICE.
Dat it is not be de fashion _pour les_ ladies of France,—I cannot tell
wat is _baiser en_ Anglish.

KING HENRY.
To kiss.

ALICE.
Your Majestee _entend_ bettre _que moi_.

KING HENRY.
It is not a fashion for the maids in France to kiss before they are
married, would she say?

ALICE.
_Oui, vraiment._

KING HENRY.
O Kate, nice customs curtsy to great kings. Dear Kate, you and I cannot
be confined within the weak list of a country’s fashion. We are the
makers of manners, Kate; and the liberty that follows our places stops
the mouth of all find-faults, as I will do yours, for upholding the
nice fashion of your country in denying me a kiss; therefore, patiently
and yielding. [_Kissing her._] You have witchcraft in your lips, Kate;
there is more eloquence in a sugar touch of them than in the tongues of
the French council; and they should sooner persuade Harry of England
than a general petition of monarchs. Here comes your father.

 Enter the French Power and the English Lords.

BURGUNDY.
God save your Majesty! My royal cousin, teach you our princess English?

KING HENRY.
I would have her learn, my fair cousin, how perfectly I love her; and
that is good English.

BURGUNDY.
Is she not apt?

KING HENRY.
Our tongue is rough, coz, and my condition is not smooth; so that,
having neither the voice nor the heart of flattery about me, I cannot
so conjure up the spirit of love in her, that he will appear in his
true likeness.

BURGUNDY.
Pardon the frankness of my mirth, if I answer you for that. If you
would conjure in her, you must make a circle; if conjure up Love in her
in his true likeness, he must appear naked and blind. Can you blame her
then, being a maid yet ros’d over with the virgin crimson of modesty,
if she deny the appearance of a naked blind boy in her naked seeing
self? It were, my lord, a hard condition for a maid to consign to.

KING HENRY.
Yet they do wink and yield, as love is blind and enforces.

BURGUNDY.
They are then excus’d, my lord, when they see not what they do.

KING HENRY.
Then, good my lord, teach your cousin to consent winking.

BURGUNDY.
I will wink on her to consent, my lord, if you will teach her to know
my meaning; for maids, well summer’d and warm kept, are like flies at
Bartholomew-tide, blind, though they have their eyes; and then they
will endure handling, which before would not abide looking on.

KING HENRY.
This moral ties me over to time and a hot summer; and so I shall catch
the fly, your cousin, in the latter end, and she must be blind too.

BURGUNDY.
As love is, my lord, before it loves.

KING HENRY.
It is so; and you may, some of you, thank love for my blindness, who
cannot see many a fair French city for one fair French maid that stands
in my way.

FRENCH KING.
Yes, my lord, you see them perspectively, the cities turn’d into a
maid; for they are all girdled with maiden walls that no war hath
entered.

KING HENRY.
Shall Kate be my wife?

FRENCH KING.
So please you.

KING HENRY.
I am content, so the maiden cities you talk of may wait on her; so the
maid that stood in the way for my wish shall show me the way to my
will.

FRENCH KING.
We have consented to all terms of reason.

KING HENRY.
Is’t so, my lords of England?

WESTMORLAND.
The king hath granted every article;
His daughter first, and then in sequel all,
According to their firm proposed natures.

EXETER.
Only he hath not yet subscribed this: where your Majesty demands, that
the King of France, having any occasion to write for matter of grant,
shall name your Highness in this form and with this addition, in
French, _Notre très-cher fils Henri, Roi d’Angleterre, Héritier de
France_; and thus in Latin, _Praeclarissimus filius noster Henricus,
rex Angliae et haeres Franciae._

FRENCH KING.
Nor this I have not, brother, so denied
But our request shall make me let it pass.

KING HENRY.
I pray you then, in love and dear alliance,
Let that one article rank with the rest;
And thereupon give me your daughter.

FRENCH KING.
Take her, fair son, and from her blood raise up
Issue to me; that the contending kingdoms
Of France and England, whose very shores look pale
With envy of each other’s happiness,
May cease their hatred; and this dear conjunction
Plant neighbourhood and Christian-like accord
In their sweet bosoms, that never war advance
His bleeding sword ’twixt England and fair France.

LORDS.
Amen!

KING HENRY.
Now, welcome, Kate; and bear me witness all,
That here I kiss her as my sovereign queen.

 [_Flourish._]

QUEEN ISABEL.
God, the best maker of all marriages,
Combine your hearts in one, your realms in one!
As man and wife, being two, are one in love,
So be there ’twixt your kingdoms such a spousal,
That never may ill office, or fell jealousy,
Which troubles oft the bed of blessed marriage,
Thrust in between the paction of these kingdoms,
To make divorce of their incorporate league;
That English may as French, French Englishmen,
Receive each other. God speak this Amen!

ALL.
Amen!

KING HENRY.
Prepare we for our marriage; on which day,
My Lord of Burgundy, we’ll take your oath,
And all the peers’, for surety of our leagues,
Then shall I swear to Kate, and you to me;
And may our oaths well kept and prosperous be!

 [_Sennet. Exeunt._]

EPILOGUE.

 Enter Chorus.

CHORUS.
Thus far, with rough and all-unable pen,
Our bending author hath pursu’d the story,
In little room confining mighty men,
Mangling by starts the full course of their glory.
Small time, but in that small most greatly lived
This star of England. Fortune made his sword,
By which the world’s best garden he achieved,
And of it left his son imperial lord.
Henry the Sixth, in infant bands crown’d King
Of France and England, did this king succeed;
Whose state so many had the managing,
That they lost France and made his England bleed:
Which oft our stage hath shown; and, for their sake,
In your fair minds let this acceptance take.

 [_Exit._]




THE FIRST PART OF HENRY THE SIXTH




Contents

 ACT I
 Scene I. Westminster Abbey
 Scene II. France. Before Orleans
 Scene III. London. Before the Tower
 Scene IV. Orleans
 Scene V. Before Orleans
 Scene VI. Orleans

 ACT II
 SCENE I. Before Orleans
 SCENE II. Orleans. Within the town
 SCENE III. Auvergne. The Countess’s castle
 SCENE IV. London. The Temple Garden
 SCENE V. The Tower of London

 ACT III
 SCENE I. London. The Parliament House
 SCENE II. France. Before Rouen
 SCENE III. The plains near Rouen
 SCENE IV. Paris. The Palace

 ACT IV
 SCENE I. Paris. The Palace
 SCENE II. Before Bordeaux
 SCENE III. Plains in Gascony
 SCENE IV. Other plains in Gascony
 SCENE V. The English camp near Bordeaux
 SCENE VI. A field of battle
 SCENE VII. Another part of the field

 ACT V
 SCENE I. London. The Palace
 SCENE II. France. Plains in Anjou
 SCENE III. Before Angiers
 SCENE IV. Camp of the Duke of York in Anjou
 SCENE V. London. The royal palace




Dramatis Personæ

KING HENRY the Sixth
DUKE OF GLOUCESTER, uncle to the King, and Protector
DUKE OF BEDFORD, uncle to the King, and Regent of France
DUKE OF EXETER, (Thomas Beaufort), great-uncle to the King
BISHOP OF WINCHESTER (Henry Beaufort), great-uncle to the King,
afterwards Cardinal
DUKE OF SOMERSET (John Beaufort)
RICHARD PLANTAGENET, son of Richard, late Earl of Cambridge, afterwards
Duke of York
EARL OF WARWICK
EARL OF SALISBURY
EARL OF SUFFOLK
LORD TALBOT, afterwards Earl of Shrewsbury
JOHN TALBOT, his son
Edmund MORTIMER, Earl of March
SIR JOHN FASTOLF
SIR WILLIAM LUCY
SIR WILLIAM GLANSDALE
SIR THOMAS GARGRAVE
MAYOR of London
WOODVILLE, Lieutenant of the Tower
VERNON, of the White-Rose or York faction
BASSET, of the Red-Rose or Lancaster faction
A LAWYER
Mortimer’s JAILERS

CHARLES, Dauphin, and afterwards King of France
REIGNIER, Duke of Anjou, and titular King of Naples
DUKE OF BURGUNDY
DUKE OF ALENÇON
BASTARD OF ORLEANS
Governor of Paris
MASTER GUNNER of Orleans and BOY, his son
General of the French forces in Bordeaux
A French Sergeant.
A Porter
An old Shepherd, father to Joan la Pucelle

MARGARET, daughter to Reignier, afterwards married to King Henry
COUNTESS OF AUVERGNE
JOAN LA PUCELLE, commonly called Joan of Arc

Lords, Warders of the Tower, Heralds, Officers, Soldiers, Messengers,
and Attendants.

Fiends appearing to Joan la Pucelle

SCENE: Partly in England and partly in France




ACT I

SCENE I. Westminster Abbey.


Dead March. Enter the funeral of King Henry the Fifth, attended on by
the Duke of Bedford, Regent of France; the Duke of Gloucester,
Protector; the Duke of Exeter, the Earl of Warwick, the Bishop of
Winchester, the Duke of Somerset with Heralds, &c.

BEDFORD.
Hung be the heavens with black, yield day to night!
Comets, importing change of times and states,
Brandish your crystal tresses in the sky,
And with them scourge the bad revolting stars
That have consented unto Henry’s death:
King Henry the Fifth, too famous to live long!
England ne’er lost a king of so much worth.

GLOUCESTER.
England ne’er had a king until his time.
Virtue he had, deserving to command;
His brandish’d sword did blind men with his beams;
His arms spread wider than a dragon’s wings;
His sparkling eyes, replete with wrathful fire,
More dazzled and drove back his enemies
Than mid-day sun fierce bent against their faces.
What should I say? His deeds exceed all speech.
He ne’er lift up his hand but conquered.

EXETER.
We mourn in black; why mourn we not in blood?
Henry is dead and never shall revive.
Upon a wooden coffin we attend,
And Death’s dishonourable victory
We with our stately presence glorify,
Like captives bound to a triumphant car.
What! Shall we curse the planets of mishap
That plotted thus our glory’s overthrow?
Or shall we think the subtle-witted French
Conjurers and sorcerers, that, afraid of him,
By magic verses have contriv’d his end?

WINCHESTER.
He was a king bless’d of the King of kings;
Unto the French the dreadful judgement-day
So dreadful will not be as was his sight.
The battles of the Lord of Hosts he fought:
The Church’s prayers made him so prosperous.

GLOUCESTER.
The Church! Where is it? Had not churchmen pray’d,
His thread of life had not so soon decay’d.
None do you like but an effeminate prince,
Whom like a school-boy you may overawe.

WINCHESTER.
Gloucester, whate’er we like, thou art Protector,
And lookest to command the Prince and realm.
Thy wife is proud; she holdeth thee in awe
More than God or religious churchmen may.

GLOUCESTER.
Name not religion, for thou lov’st the flesh,
And ne’er throughout the year to church thou go’st,
Except it be to pray against thy foes.

BEDFORD.
Cease, cease these jars, and rest your minds in peace;
Let’s to the altar; heralds, wait on us.
Instead of gold, we’ll offer up our arms,
Since arms avail not, now that Henry’s dead.
Posterity, await for wretched years,
When at their mothers’ moist eyes babes shall suck,
Our isle be made a nourish of salt tears,
And none but women left to wail the dead.
Henry the Fifth, thy ghost I invocate:
Prosper this realm, keep it from civil broils,
Combat with adverse planets in the heavens.
A far more glorious star thy soul will make
Than Julius Caesar or bright—

Enter a Messenger.

MESSENGER.
My honourable lords, health to you all!
Sad tidings bring I to you out of France,
Of loss, of slaughter, and discomfiture:
Guienne, Champaigne, Rheims, Rouen, Orleans,
Paris, Guysors, Poictiers, are all quite lost.

BEDFORD.
What say’st thou, man, before dead Henry’s corse?
Speak softly, or the loss of those great towns
Will make him burst his lead and rise from death.

GLOUCESTER.
Is Paris lost? Is Rouen yielded up?
If Henry were recall’d to life again,
These news would cause him once more yield the ghost.

EXETER.
How were they lost? What treachery was us’d?

MESSENGER.
No treachery, but want of men and money.
Amongst the soldiers this is muttered:
That here you maintain several factions
And whilst a field should be dispatch’d and fought,
You are disputing of your generals.
One would have lingering wars with little cost;
Another would fly swift, but wanteth wings;
A third thinks, without expense at all,
By guileful fair words peace may be obtain’d.
Awake, awake, English nobility!
Let not sloth dim your honours new-begot.
Cropp’d are the flower-de-luces in your arms;
Of England’s coat one half is cut away.

[_He exits._]

EXETER.
Were our tears wanting to this funeral,
These tidings would call forth their flowing tides.

BEDFORD.
Me they concern; Regent I am of France.
Give me my steeled coat. I’ll fight for France.
Away with these disgraceful wailing robes!
Wounds will I lend the French instead of eyes,
To weep their intermissive miseries.

Enter to them another Messenger.

MESSENGER.
Lords, view these letters full of bad mischance.
France is revolted from the English quite,
Except some petty towns of no import.
The Dauphin Charles is crowned king in Rheims;
The Bastard of Orleans with him is join’d;
Reignier, Duke of Anjou, doth take his part;
The Duke of Alençon flieth to his side.

[_He exits._]

EXETER.
The Dauphin crowned king! All fly to him!
O, whither shall we fly from this reproach?

GLOUCESTER.
We will not fly but to our enemies’ throats.
Bedford, if thou be slack, I’ll fight it out.

BEDFORD.
Gloucester, why doubt’st thou of my forwardness?
An army have I muster’d in my thoughts,
Wherewith already France is overrun.

Enter another Messenger.

MESSENGER.
My gracious lords, to add to your laments,
Wherewith you now bedew King Henry’s hearse,
I must inform you of a dismal fight
Betwixt the stout Lord Talbot and the French.

WINCHESTER.
What! Wherein Talbot overcame, is’t so?

MESSENGER.
O no, wherein Lord Talbot was o’erthrown.
The circumstance I’ll tell you more at large.
The tenth of August last this dreadful lord,
Retiring from the siege of Orleans,
Having full scarce six thousand in his troop,
By three and twenty thousand of the French
Was round encompassed and set upon.
No leisure had he to enrank his men;
He wanted pikes to set before his archers;
Instead whereof sharp stakes pluck’d out of hedges
They pitched in the ground confusedly
To keep the horsemen off from breaking in.
More than three hours the fight continued;
Where valiant Talbot, above human thought,
Enacted wonders with his sword and lance.
Hundreds he sent to hell, and none durst stand him;
Here, there, and everywhere, enrag’d he slew.
The French exclaim’d the devil was in arms;
All the whole army stood agaz’d on him.
His soldiers, spying his undaunted spirit,
“A Talbot! a Talbot!” cried out amain,
And rush’d into the bowels of the battle.
Here had the conquest fully been seal’d up
If Sir John Fastolf had not play’d the coward.
He, being in the vaward, plac’d behind
With purpose to relieve and follow them,
Cowardly fled, not having struck one stroke.
Hence grew the general wrack and massacre.
Enclosed were they with their enemies.
A base Walloon, to win the Dauphin’s grace,
Thrust Talbot with a spear into the back,
Whom all France, with their chief assembled strength,
Durst not presume to look once in the face.

BEDFORD.
Is Talbot slain? Then I will slay myself,
For living idly here, in pomp and ease,
Whilst such a worthy leader, wanting aid,
Unto his dastard foemen is betray’d.

MESSENGER.
O no, he lives, but is took prisoner,
And Lord Scales with him, and Lord Hungerford;
Most of the rest slaughter’d or took likewise.

BEDFORD.
His ransom there is none but I shall pay.
I’ll hale the Dauphin headlong from his throne;
His crown shall be the ransom of my friend;
Four of their lords I’ll change for one of ours.
Farewell, my masters; to my task will I;
Bonfires in France forthwith I am to make
To keep our great Saint George’s feast withal.
Ten thousand soldiers with me I will take,
Whose bloody deeds shall make all Europe quake.

MESSENGER.
So you had need; for Orleans is besieg’d
The English army is grown weak and faint;
The Earl of Salisbury craveth supply
And hardly keeps his men from mutiny,
Since they, so few, watch such a multitude.

[_He exits._]

EXETER.
Remember, lords, your oaths to Henry sworn,
Either to quell the Dauphin utterly,
Or bring him in obedience to your yoke.

BEDFORD.
I do remember it, and here take my leave
To go about my preparation.

[_Exit._]

GLOUCESTER.
I’ll to the Tower with all the haste I can
To view th’ artillery and munition;
And then I will proclaim young Henry king.

[_Exit._]

EXETER.
To Eltham will I, where the young King is,
Being ordain’d his special governor;
And for his safety there I’ll best devise.

[_Exit._]

WINCHESTER.
Each hath his place and function to attend.
I am left out; for me nothing remains.
But long I will not be Jack out of office.
The King from Eltham I intend to steal,
And sit at chiefest stern of public weal.

[_Exeunt._]

SCENE II. France. Before Orleans

Sound a Flourish. Enter  Charles, Alençon and Reignier, marching with
Drum and Soldiers.

CHARLES.
Mars his true moving, even as in the heavens
So in the earth, to this day is not known.
Late did he shine upon the English side;
Now we are victors; upon us he smiles.
What towns of any moment but we have?
At pleasure here we lie near Orleans,
Otherwhiles the famish’d English, like pale ghosts,
Faintly besiege us one hour in a month.

ALENÇON.
They want their porridge and their fat bull beeves.
Either they must be dieted like mules
And have their provender tied to their mouths,
Or piteous they will look, like drowned mice.

REIGNIER.
Let’s raise the siege. Why live we idly here?
Talbot is taken, whom we wont to fear.
Remaineth none but mad-brain’d Salisbury,
And he may well in fretting spend his gall;
Nor men nor money hath he to make war.

CHARLES.
Sound, sound alarum! We will rush on them.
Now for the honour of the forlorn French!
Him I forgive my death that killeth me
When he sees me go back one foot or fly.

[_Exeunt._]

Here alarum; they are beaten back by the English, with great loss.
Re-enter Charles, Alençon and Reignier.

CHARLES.
Who ever saw the like? What men have I!
Dogs, cowards, dastards! I would ne’er have fled
But that they left me ’midst my enemies.

REIGNIER.
Salisbury is a desperate homicide;
He fighteth as one weary of his life.
The other lords, like lions wanting food,
Do rush upon us as their hungry prey.

ALENÇON.
Froissart, a countryman of ours, records,
England all Olivers and Rowlands bred
During the time Edward the Third did reign.
More truly now may this be verified;
For none but Samsons and Goliases
It sendeth forth to skirmish. One to ten!
Lean raw-bon’d rascals! Who would e’er suppose
They had such courage and audacity?

CHARLES.
Let’s leave this town; for they are hare-brain’d slaves,
And hunger will enforce them to be more eager.
Of old I know them; rather with their teeth
The walls they’ll tear down than forsake the siege.

REIGNIER.
I think by some odd gimmers or device
Their arms are set like clocks, still to strike on;
Else ne’er could they hold out so as they do.
By my consent, we’ll even let them alone.

ALENÇON.
Be it so.

Enter the Bastard of Orleans.

BASTARD.
Where’s the Prince Dauphin? I have news for him.

CHARLES.
Bastard of Orleans, thrice welcome to us.

BASTARD.
Methinks your looks are sad, your cheer appall’d.
Hath the late overthrow wrought this offence?
Be not dismay’d, for succour is at hand.
A holy maid hither with me I bring,
Which, by a vision sent to her from heaven
Ordained is to raise this tedious siege
And drive the English forth the bounds of France.
The spirit of deep prophecy she hath,
Exceeding the nine sibyls of old Rome.
What’s past and what’s to come she can descry.
Speak, shall I call her in? Believe my words,
For they are certain and unfallible.

CHARLES.
Go, call her in.

[_Exit Bastard._]

But first, to try her skill,
Reignier, stand thou as Dauphin in my place;
Question her proudly; let thy looks be stern.
By this means shall we sound what skill she hath.

Re-enter the Bastard of Orleans, with Joan la Pucelle.

REIGNIER.
Fair maid, is ’t thou wilt do these wondrous feats?

PUCELLE.
Reignier is ’t thou that thinkest to beguile me?
Where is the Dauphin? Come, come from behind;
I know thee well, though never seen before.
Be not amazed, there’s nothing hid from me.
In private will I talk with thee apart.
Stand back, you lords, and give us leave awhile.

REIGNIER.
She takes upon her bravely at first dash.

PUCELLE.
Dauphin, I am by birth a shepherd’s daughter,
My wit untrain’d in any kind of art.
Heaven and our Lady gracious hath it pleased
To shine on my contemptible estate.
Lo, whilst I waited on my tender lambs,
And to sun’s parching heat display’d my cheeks,
God’s mother deigned to appear to me,
And in a vision full of majesty
Will’d me to leave my base vocation
And free my country from calamity.
Her aid she promised and assured success.
In complete glory she reveal’d herself;
And, whereas I was black and swart before,
With those clear rays which she infused on me
That beauty am I blest with which you may see.
Ask me what question thou canst possible,
And I will answer unpremeditated.
My courage try by combat, if thou dar’st,
And thou shalt find that I exceed my sex.
Resolve on this; thou shalt be fortunate
If thou receive me for thy warlike mate.

CHARLES.
Thou hast astonish’d me with thy high terms.
Only this proof I’ll of thy valour make:
In single combat thou shalt buckle with me,
And if thou vanquishest, thy words are true;
Otherwise I renounce all confidence.

PUCELLE.
I am prepared. Here is my keen-edg’d sword,
Deck’d with five flower-de-luces on each side,
The which at Touraine, in Saint Katharine’s churchyard,
Out of a great deal of old iron I chose forth.

CHARLES.
Then come, o’ God’s name; I fear no woman.

PUCELLE.
And while I live, I’ll ne’er fly from a man.

[_Here they fight, and Joan la Pucelle overcomes._]

CHARLES.
Stay, stay thy hands; thou art an Amazon,
And fightest with the sword of Deborah.

PUCELLE.
Christ’s Mother helps me, else I were too weak.

CHARLES.
Whoe’er helps thee, ’tis thou that must help me.
Impatiently I burn with thy desire;
My heart and hands thou hast at once subdued.
Excellent Pucelle, if thy name be so,
Let me thy servant and not sovereign be.
’Tis the French Dauphin sueth to thee thus.

PUCELLE.
I must not yield to any rites of love,
For my profession’s sacred from above.
When I have chased all thy foes from hence,
Then will I think upon a recompense.

CHARLES.
Meantime look gracious on thy prostrate thrall.

REIGNIER.
My lord, methinks, is very long in talk.

ALENÇON.
Doubtless he shrives this woman to her smock;
Else ne’er could he so long protract his speech.

REIGNIER.
Shall we disturb him, since he keeps no mean?

ALENÇON.
He may mean more than we poor men do know.
These women are shrewd tempters with their tongues.

REIGNIER.
My lord, where are you? What devise you on?
Shall we give over Orleans, or no?

PUCELLE.
Why, no, I say. Distrustful recreants!
Fight till the last gasp; I will be your guard.

CHARLES.
What she says I’ll confirm. We’ll fight it out.

PUCELLE.
Assign’d am I to be the English scourge.
This night the siege assuredly I’ll raise.
Expect Saint Martin’s summer, halcyon’s days,
Since I have entered into these wars.
Glory is like a circle in the water,
Which never ceaseth to enlarge itself
Till by broad spreading it disperse to nought.
With Henry’s death the English circle ends;
Dispersed are the glories it included.
Now am I like that proud insulting ship
Which Caesar and his fortune bare at once.

CHARLES.
Was Mahomet inspired with a dove?
Thou with an eagle art inspired then.
Helen, the mother of great Constantine,
Nor yet Saint Philip’s daughters, were like thee.
Bright star of Venus, fall’n down on the earth,
How may I reverently worship thee enough?

ALENÇON.
Leave off delays, and let us raise the siege.

REIGNIER.
Woman, do what thou canst to save our honours;
Drive them from Orleans and be immortalized.

CHARLES.
Presently we’ll try. Come, let’s away about it.
No prophet will I trust if she prove false.

[_Exeunt._]

SCENE III. London. Before the Tower.

Enter the Duke of Gloucester with his Servingmen in blue coats.

GLOUCESTER.
I am come to survey the Tower this day.
Since Henry’s death, I fear, there is conveyance.
Where be these warders that they wait not here?
Open the gates; ’tis Gloucester that calls.

FIRST WARDER.
[_Within_.] Who’s there that knocks so imperiously?

FIRST SERVINGMAN.
It is the noble Duke of Gloucester.

SECOND WARDER.
[_Within_.] Whoe’er he be, you may not be let in.

FIRST SERVINGMAN.
Villains, answer you so the Lord Protector?

FIRST WARDER.
[_Within_.] The Lord protect him, so we answer him.
We do no otherwise than we are will’d.

GLOUCESTER.
Who willed you? Or whose will stands but mine?
There’s none Protector of the realm but I.
Break up the gates, I’ll be your warrantize.
Shall I be flouted thus by dunghill grooms?

[_Gloucester’s men rush at the Tower gates, and Woodville, the
Lieutenant, speaks within._]

WOODVILLE.
What noise is this? What traitors have we here?

GLOUCESTER.
Lieutenant, is it you whose voice I hear?
Open the gates; here’s Gloucester that would enter.

WOODVILLE.
Have patience, noble Duke; I may not open;
The Cardinal of Winchester forbids.
From him I have express commandment
That thou nor none of thine shall be let in.

GLOUCESTER.
Faint-hearted Woodville, prizest him ’fore me?
Arrogant Winchester, that haughty prelate
Whom Henry, our late sovereign, ne’er could brook?
Thou art no friend to God or to the King.
Open the gates, or I’ll shut thee out shortly.

SERVINGMEN.
Open the gates unto the Lord Protector,
Or we’ll burst them open, if that you come not quickly.

Enter to the Protector at the Tower gates Winchester and his men in
tawny coats.

WINCHESTER.
How now, ambitious Humphrey! What means this?

GLOUCESTER.
Peel’d priest, dost thou command me to be shut out?

WINCHESTER.
I do, thou most usurping proditor,
And not Protector, of the King or realm.

GLOUCESTER.
Stand back, thou manifest conspirator,
Thou that contrived’st to murder our dead lord;
Thou that giv’st whores indulgences to sin:
I’ll canvass thee in thy broad cardinal’s hat,
If thou proceed in this thy insolence.

WINCHESTER.
Nay, stand thou back; I will not budge a foot.
This be Damascus, be thou cursed Cain,
To slay thy brother Abel, if thou wilt.

GLOUCESTER.
I will not slay thee, but I’ll drive thee back.
Thy scarlet robes, as a child’s bearing-cloth,
I’ll use to carry thee out of this place.

WINCHESTER.
Do what thou dar’st, I beard thee to thy face.

GLOUCESTER.
What, am I dared and bearded to my face?
Draw, men, for all this privileged place.
Blue coats to tawny coats. Priest, beware your beard;
I mean to tug it and to cuff you soundly.
Under my feet I’ll stamp thy cardinal’s hat;
In spite of Pope or dignities of church,
Here by the cheeks I’ll drag thee up and down.

WINCHESTER.
Gloucester, thou wilt answer this before the Pope.

GLOUCESTER.
Winchester goose, I cry, “a rope, a rope!”
Now beat them hence; why do you let them stay?
Thee I’ll chase hence, thou wolf in sheep’s array.
Out, tawny coats! Out, scarlet hypocrite!

Here Gloucester’s men beat out the Cardinal’s men, and enter in the
hurly-burly the Mayor of London and his Officers.

MAYOR.
Fie, lords, that you, being supreme magistrates,
Thus contumeliously should break the peace!

GLOUCESTER.
Peace, Mayor! Thou know’st little of my wrongs.
Here’s Beaufort, that regards nor God nor king,
Hath here distrain’d the Tower to his use.

WINCHESTER.
Here’s Gloucester, a foe to citizens,
One that still motions war and never peace,
O’ercharging your free purses with large fines;
That seeks to overthrow religion,
Because he is Protector of the realm,
And would have armour here out of the Tower,
To crown himself king and suppress the Prince.

GLOUCESTER.
I will not answer thee with words, but blows.

[_Here they skirmish again._]

MAYOR.
Nought rests for me in this tumultuous strife
But to make open proclamation.
Come, officer, as loud as e’er thou canst, cry.

OFFICER.
All manner of men assembled here in arms this day against God’s peace
and the King’s, we charge and command you, in his Highness’ name, to
repair to your several dwelling-places; and not to wear, handle, or use
any sword, weapon, or dagger, henceforward, upon pain of death.

GLOUCESTER.
Cardinal, I’ll be no breaker of the law;
But we shall meet and break our minds at large.

WINCHESTER.
Gloucester, we will meet, to thy cost, be sure;
Thy heart-blood I will have for this day’s work.

MAYOR.
I’ll call for clubs, if you will not away.
This Cardinal’s more haughty than the devil.

GLOUCESTER.
Mayor, farewell. Thou dost but what thou mayst.

WINCHESTER.
Abominable Gloucester, guard thy head,
For I intend to have it ere long.

[_Exeunt, severally, Gloucester and Winchester with their Servingmen._]

MAYOR.
See the coast clear’d, and then we will depart.
Good God, these nobles should such stomachs bear!
I myself fight not once in forty year.

[_Exeunt._]

SCENE IV. Orleans.

Enter, on the walls, a Master Gunner and his Boy.

MASTER GUNNER.
Sirrah, thou know’st how Orleans is besieged,
And how the English have the suburbs won.

BOY.
Father, I know; and oft have shot at them,
Howe’er unfortunate I miss’d my aim.

MASTER GUNNER.
But now thou shalt not. Be thou ruled by me.
Chief master-gunner am I of this town;
Something I must do to procure me grace.
The Prince’s espials have informed me
How the English, in the suburbs close intrench’d,
Wont, through a secret grate of iron bars
In yonder tower, to overpeer the city,
And thence discover how with most advantage
They may vex us with shot or with assault.
To intercept this inconvenience,
A piece of ordnance ’gainst it I have placed
And even these three days have I watch’d,
If I could see them.
Now do thou watch, for I can stay no longer.
If thou spy’st any, run and bring me word;
And thou shalt find me at the Governor’s.

[_Exit._]

BOY.
Father, I warrant you; take you no care;
I’ll never trouble you if I may spy them.

[_Exit._]

Enter, on the turrets, Salisbury and Talbot, Sir William Glansdale, Sir
Thomas Gargrave and others.

SALISBURY.
Talbot, my life, my joy, again return’d!
How wert thou handled, being prisoner?
Or by what means got’st thou to be releas’d?
Discourse, I prithee, on this turret’s top.

TALBOT.
The Duke of Bedford had a prisoner
Call’d the brave Lord Ponton de Santrailles;
For him was I exchanged and ransomed.
But with a baser man of arms by far
Once in contempt they would have barter’d me,
Which I disdaining scorn’d, and craved death
Rather than I would be so vile-esteem’d.
In fine, redeem’d I was as I desired.
But O, the treacherous Fastolf wounds my heart,
Whom with my bare fists I would execute
If I now had him brought into my power.

SALISBURY.
Yet tell’st thou not how thou wert entertain’d.

TALBOT.
With scoffs and scorns and contumelious taunts.
In open market-place produced they me
To be a public spectacle to all.
Here, said they, is the terror of the French,
The scarecrow that affrights our children so.
Then broke I from the officers that led me,
And with my nails digg’d stones out of the ground
To hurl at the beholders of my shame.
My grisly countenance made others fly;
None durst come near for fear of sudden death.
In iron walls they deem’d me not secure;
So great fear of my name ’mongst them were spread
That they supposed I could rend bars of steel
And spurn in pieces posts of adamant.
Wherefore a guard of chosen shot I had,
That walk’d about me every minute while;
And if I did but stir out of my bed,
Ready they were to shoot me to the heart.

Enter the Boy with a linstock.

SALISBURY.
I grieve to hear what torments you endured,
But we will be revenged sufficiently.
Now it is supper-time in Orleans.
Here, through this grate, I count each one
And view the Frenchmen how they fortify.
Let us look in; the sight will much delight thee.
Sir Thomas Gargrave and Sir William Glansdale,
Let me have your express opinions
Where is best place to make our battery next.

GARGRAVE.
I think, at the north gate, for there stand lords.

GLANSDALE.
And I, here, at the bulwark of the bridge.

TALBOT.
For aught I see, this city must be famish’d,
Or with light skirmishes enfeebled.

Here they shoot, and Salisbury and Gargrave fall down.

SALISBURY.
O Lord, have mercy on us, wretched sinners!

GARGRAVE.
O Lord, have mercy on me, woeful man!

TALBOT.
What chance is this that suddenly hath cross’d us?
Speak, Salisbury; at least, if thou canst speak!
How far’st thou, mirror of all martial men?
One of thy eyes and thy cheek’s side struck off!
Accursed tower, accursed fatal hand
That hath contrived this woeful tragedy!
In thirteen battles Salisbury o’ercame;
Henry the Fifth he first train’d to the wars;
Whilst any trump did sound, or drum struck up,
His sword did ne’er leave striking in the field.
Yet liv’st thou, Salisbury? Though thy speech doth fail,
One eye thou hast to look to heaven for grace.
The sun with one eye vieweth all the world.
Heaven, be thou gracious to none alive,
If Salisbury wants mercy at thy hands!
Sir Thomas Gargrave, hast thou any life?
Speak unto Talbot; nay, look up to him.
Bear hence his body; I will help to bury it.

[_Exeunt some with the body of Gargrave._]

Salisbury, cheer thy spirit with this comfort,
Thou shalt not die whiles—
He beckons with his hand and smiles on me,
As who should say “When I am dead and gone,
Remember to avenge me on the French.”
Plantagenet, I will; and, like thee, Nero,
Play on the lute, beholding the towns burn.
Wretched shall France be only in thy name.

[_Here an alarum, and it thunders and lightens._]

What stir is this? What tumult’s in the heavens?
Whence cometh this alarum and the noise?

Enter a Messenger.

MESSENGER.
My lord, my lord, the French have gather’d head.
The Dauphin, with one Joan la Pucelle join’d,
A holy prophetess new risen up,
Is come with a great power to raise the siege.

[_Here Salisbury lifteth himself up and groans._]

TALBOT.
Hear, hear how dying Salisbury doth groan;
It irks his heart he cannot be revenged.
Frenchmen, I’ll be a Salisbury to you.
Pucelle or puzel, dolphin or dogfish,
Your hearts I’ll stamp out with my horse’s heels
And make a quagmire of your mingled brains.
Convey we Salisbury into his tent,
And then we’ll try what these dastard Frenchmen dare.

[_Alarum. Exeunt._]

SCENE V. Before Orleans.

Here an alarum again, and Talbot pursueth the Dauphin and driveth him;
then enter Joan la Pucelle, driving Englishmen before her, and exit
after them. Then re-enter Talbot.

TALBOT.
Where is my strength, my valour, and my force?
Our English troops retire, I cannot stay them.
A woman clad in armour chaseth them.

Enter La Pucelle.

Here, here she comes. I’ll have a bout with thee;
Devil or devil’s dam, I’ll conjure thee.
Blood will I draw on thee, thou art a witch,
And straightway give thy soul to him thou serv’st.

PUCELLE.
Come, come, ’tis only I that must disgrace thee.

[_Here they fight._]

TALBOT.
Heavens, can you suffer hell so to prevail?
My breast I’ll burst with straining of my courage,
And from my shoulders crack my arms asunder,
But I will chastise this high-minded strumpet.

[_They fight again._]

PUCELLE.
Talbot, farewell; thy hour is not yet come.
I must go victual Orleans forthwith.

[_A short alarum. Then enter the town with soldiers._]

O’ertake me, if thou canst. I scorn thy strength.
Go, go, cheer up thy hungry-starved men;
Help Salisbury to make his testament.
This day is ours, as many more shall be.

[_Exit._]

TALBOT.
My thoughts are whirled like a potter’s wheel;
I know not where I am, nor what I do.
A witch by fear, not force, like Hannibal,
Drives back our troops and conquers as she lists.
So bees with smoke and doves with noisome stench
Are from their hives and houses driven away.
They call’d us for our fierceness, English dogs;
Now like to whelps we crying run away.

[_A short alarum._]

Hark, countrymen, either renew the fight,
Or tear the lions out of England’s coat;
Renounce your soil, give sheep in lions’ stead.
Sheep run not half so treacherous from the wolf,
Or horse or oxen from the leopard,
As you fly from your oft-subdued slaves.

[_Alarum. Here another skirmish._]

It will not be! Retire into your trenches.
You all consented unto Salisbury’s death,
For none would strike a stroke in his revenge.
Pucelle is enter’d into Orleans,
In spite of us or aught that we could do.
O, would I were to die with Salisbury!
The shame hereof will make me hide my head.

[_Exit Talbot. Alarum; retreat._]

SCENE VI. Orleans.

Flourish. Enter on the walls La Pucelle, Charles, Reignier, Alençon and
Soldiers.

PUCELLE.
Advance our waving colours on the walls;
Rescued is Orleans from the English.
Thus Joan la Pucelle hath perform’d her word.

CHARLES.
Divinest creature, Astraea’s daughter,
How shall I honour thee for this success?
Thy promises are like Adonis’ gardens
That one day bloom’d and fruitful were the next.
France, triumph in thy glorious prophetess!
Recover’d is the town of Orleans.
More blessed hap did ne’er befall our state.

REIGNIER.
Why ring not bells aloud throughout the town?
Dauphin, command the citizens make bonfires
And feast and banquet in the open streets
To celebrate the joy that God hath given us.

ALENÇON.
All France will be replete with mirth and joy
When they shall hear how we have play’d the men.

CHARLES.
’Tis Joan, not we, by whom the day is won;
For which I will divide my crown with her,
And all the priests and friars in my realm
Shall in procession sing her endless praise.
A statelier pyramis to her I’ll rear
Than Rhodope’s of Memphis ever was;
In memory of her when she is dead,
Her ashes, in an urn more precious
Than the rich-jewel’d coffer of Darius,
Transported shall be at high festivals
Before the kings and queens of France.
No longer on Saint Denis will we cry,
But Joan la Pucelle shall be France’s saint.
Come in, and let us banquet royally
After this golden day of victory.

[_Flourish. Exeunt._]




ACT II

SCENE I. Before Orleans.


Enter a Sergeant of a band, with two Sentinels.

SERGEANT.
Sirs, take your places and be vigilant.
If any noise or soldier you perceive
Near to the walls, by some apparent sign
Let us have knowledge at the court of guard.

FIRST SENTINEL.
Sergeant, you shall.

[_Exit Sergeant._]

Thus are poor servitors,
When others sleep upon their quiet beds,
Constrain’d to watch in darkness, rain, and cold.

Enter Talbot, Bedford, Burgundy, and forces, with scaling-ladders.

TALBOT.
Lord Regent, and redoubted Burgundy,
By whose approach the regions of Artois,
Walloon and Picardy are friends to us,
This happy night the Frenchmen are secure,
Having all day caroused and banqueted.
Embrace we then this opportunity,
As fitting best to quittance their deceit
Contriv’d by art and baleful sorcery.

BEDFORD.
Coward of France, how much he wrongs his fame,
Despairing of his own arm’s fortitude,
To join with witches and the help of hell!

BURGUNDY.
Traitors have never other company.
But what’s that Pucelle whom they term so pure?

TALBOT.
A maid, they say.

BEDFORD.
A maid! And be so martial!

BURGUNDY.
Pray God she prove not masculine ere long,
If underneath the standard of the French
She carry armour as she hath begun.

TALBOT.
Well, let them practice and converse with spirits.
God is our fortress, in whose conquering name
Let us resolve to scale their flinty bulwarks.

BEDFORD.
Ascend, brave Talbot; we will follow thee.

TALBOT.
Not all together. Better far, I guess,
That we do make our entrance several ways,
That if it chance the one of us do fail,
The other yet may rise against their force.

BEDFORD.
Agreed. I’ll to yond corner.

BURGUNDY.
And I to this.

TALBOT.
And here will Talbot mount, or make his grave.
Now, Salisbury, for thee, and for the right
Of English Henry, shall this night appear
How much in duty I am bound to both.

SENTINEL.
Arm! Arm! The enemy doth make assault!

[_Cry: “St George,” “A Talbot!”_]

The French leap over the walls in their shirts. Enter several ways the 
Bastard of Orleans, Alençon and Reignier, half ready and half unready.

ALENÇON.
How now, my lords? What, all unready so?

BASTARD.
Unready! Ay, and glad we ’scap’d so well.

REIGNIER.
’Twas time, I trow, to wake and leave our beds,
Hearing alarums at our chamber-doors.

ALENÇON.
Of all exploits since first I follow’d arms
Ne’er heard I of a warlike enterprise
More venturous or desperate than this.

BASTARD.
I think this Talbot be a fiend of hell.

REIGNIER.
If not of hell, the heavens, sure, favour him.

ALENÇON.
Here cometh Charles. I marvel how he sped.

Enter Charles and La Pucelle.

BASTARD.
Tut, holy Joan was his defensive guard.

CHARLES.
Is this thy cunning, thou deceitful dame?
Didst thou at first, to flatter us withal,
Make us partakers of a little gain,
That now our loss might be ten times so much?

PUCELLE.
Wherefore is Charles impatient with his friend?
At all times will you have my power alike?
Sleeping or waking, must I still prevail,
Or will you blame and lay the fault on me?
Improvident soldiers, had your watch been good,
This sudden mischief never could have fall’n.

CHARLES.
Duke of Alençon, this was your default,
That, being captain of the watch tonight,
Did look no better to that weighty charge.

ALENÇON.
Had all your quarters been as safely kept
As that whereof I had the government,
We had not been thus shamefully surprised.

BASTARD.
Mine was secure.

REIGNIER.
And so was mine, my lord.

CHARLES.
And for myself, most part of all this night,
Within her quarter and mine own precinct
I was employ’d in passing to and fro
About relieving of the sentinels.
Then how or which way should they first break in?

PUCELLE.
Question, my lords, no further of the case,
How or which way; ’tis sure they found some place
But weakly guarded, where the breach was made.
And now there rests no other shift but this:
To gather our soldiers, scattered and dispersed,
And lay new platforms to endamage them.

Alarum. Enter an  English Soldier, crying “A Talbot! A Talbot!” They
fly, leaving their clothes behind.

SOLDIER.
I’ll be so bold to take what they have left.
The cry of “Talbot” serves me for a sword;
For I have loaden me with many spoils,
Using no other weapon but his name.

[_Exit._]

SCENE II. Orleans. Within the town.

Enter Talbot, Bedford, Burgundy, a Captain and others.

BEDFORD.
The day begins to break, and night is fled,
Whose pitchy mantle over-veil’d the earth.
Here sound retreat and cease our hot pursuit.

[_Retreat sounded._]

TALBOT.
Bring forth the body of old Salisbury,
And here advance it in the market-place,
The middle centre of this cursed town.

Dead March. Enter with the body of Salisbury.

Now have I paid my vow unto his soul;
For every drop of blood was drawn from him
There hath at least five Frenchmen died tonight.
And that hereafter ages may behold
What ruin happen’d in revenge of him,
Within their chiefest temple I’ll erect
A tomb, wherein his corpse shall be interr’d;
Upon the which, that everyone may read,
Shall be engraved the sack of Orleans,
The treacherous manner of his mournful death
And what a terror he had been to France.

[_Exit Funeral._]

But, lords, in all our bloody massacre,
I muse we met not with the Dauphin’s grace,
His new-come champion, virtuous Joan of Arc,
Nor any of his false confederates.

BEDFORD.
’Tis thought, Lord Talbot, when the fight began,
Rous’d on the sudden from their drowsy beds,
They did amongst the troops of armed men
Leap o’er the walls for refuge in the field.

BURGUNDY.
Myself, as far as I could well discern
For smoke and dusky vapors of the night,
Am sure I scared the Dauphin and his trull,
When arm in arm they both came swiftly running,
Like to a pair of loving turtle-doves
That could not live asunder day or night.
After that things are set in order here,
We’ll follow them with all the power we have.

Enter a Messenger.

MESSENGER.
All hail, my lords! Which of this princely train
Call ye the warlike Talbot, for his acts
So much applauded through the realm of France?

TALBOT.
Here is the Talbot. Who would speak with him?

MESSENGER.
The virtuous lady, Countess of Auvergne,
With modesty admiring thy renown,
By me entreats, great lord, thou wouldst vouchsafe
To visit her poor castle where she lies,
That she may boast she hath beheld the man
Whose glory fills the world with loud report.

BURGUNDY.
Is it even so? Nay, then I see our wars
Will turn unto a peaceful comic sport,
When ladies crave to be encounter’d with.
You may not, my lord, despise her gentle suit.

TALBOT.
Ne’er trust me then; for when a world of men
Could not prevail with all their oratory,
Yet hath a woman’s kindness over-ruled.
And therefore tell her I return great thanks,
And in submission will attend on her.
Will not your honours bear me company?

BEDFORD.
No, truly, it is more than manners will;
And I have heard it said, unbidden guests
Are often welcomest when they are gone.

TALBOT.
Well then, alone, since there’s no remedy,
I mean to prove this lady’s courtesy.
Come hither, Captain. [_Whispers_.] You perceive my mind?

CAPTAIN.
I do, my lord, and mean accordingly.

[_Exeunt._]

SCENE III. Auvergne. The Countess’s castle.

Enter the Countess and her Porter.

COUNTESS.
Porter, remember what I gave in charge;
And when you have done so, bring the keys to me.

PORTER.
Madam, I will.

[_Exit._]

COUNTESS.
The plot is laid. If all things fall out right,
I shall as famous be by this exploit
As Scythian Tomyris by Cyrus’ death.
Great is the rumour of this dreadful knight,
And his achievements of no less account.
Fain would mine eyes be witness with mine ears,
To give their censure of these rare reports.

Enter Messenger and Talbot.

MESSENGER.
Madam, according as your ladyship desired,
By message craved, so is Lord Talbot come.

COUNTESS.
And he is welcome. What, is this the man?

MESSENGER.
Madam, it is.

COUNTESS.
Is this the scourge of France?
Is this the Talbot, so much fear’d abroad
That with his name the mothers still their babes?
I see report is fabulous and false.
I thought I should have seen some Hercules,
A second Hector, for his grim aspect,
And large proportion of his strong-knit limbs.
Alas, this is a child, a silly dwarf!
It cannot be this weak and writhled shrimp
Should strike such terror to his enemies.

TALBOT.
Madam, I have been bold to trouble you;
But since your ladyship is not at leisure,
I’ll sort some other time to visit you.

COUNTESS.
What means he now? Go ask him whither he goes.

MESSENGER.
Stay, my Lord Talbot, for my lady craves
To know the cause of your abrupt departure.

TALBOT.
Marry, for that she’s in a wrong belief,
I go to certify her Talbot’s here.

Enter Porter with keys.

COUNTESS.
If thou be he, then art thou prisoner.

TALBOT.
Prisoner! To whom?

COUNTESS.
To me, blood-thirsty lord;
And for that cause I train’d thee to my house.
Long time thy shadow hath been thrall to me,
For in my gallery thy picture hangs.
But now the substance shall endure the like,
And I will chain these legs and arms of thine,
That hast by tyranny these many years
Wasted our country, slain our citizens,
And sent our sons and husbands captivate.

TALBOT.
Ha, ha, ha!

COUNTESS.
Laughest thou, wretch? Thy mirth shall turn to moan.

TALBOT.
I laugh to see your ladyship so fond
To think that you have aught but Talbot’s shadow
Whereon to practice your severity.

COUNTESS.
Why, art not thou the man?

TALBOT.
I am indeed.

COUNTESS.
Then have I substance too.

TALBOT.
No, no, I am but shadow of myself.
You are deceived, my substance is not here;
For what you see is but the smallest part
And least proportion of humanity.
I tell you, madam, were the whole frame here,
It is of such a spacious lofty pitch
Your roof were not sufficient to contain ’t.

COUNTESS.
This is a riddling merchant for the nonce;
He will be here, and yet he is not here.
How can these contrarieties agree?

TALBOT.
That will I show you presently.

Winds his horn. Drums strike up; a peal of ordnance. Enter Soldiers.

How say you, madam? Are you now persuaded
That Talbot is but shadow of himself?
These are his substance, sinews, arms and strength,
With which he yoketh your rebellious necks,
Razeth your cities and subverts your towns,
And in a moment makes them desolate.

COUNTESS.
Victorious Talbot, pardon my abuse.
I find thou art no less than fame hath bruited,
And more than may be gather’d by thy shape.
Let my presumption not provoke thy wrath,
For I am sorry that with reverence
I did not entertain thee as thou art.

TALBOT.
Be not dismay’d, fair lady, nor misconster
The mind of Talbot, as you did mistake
The outward composition of his body.
What you have done hath not offended me;
Nor other satisfaction do I crave
But only, with your patience, that we may
Taste of your wine and see what cates you have,
For soldiers’ stomachs always serve them well.

COUNTESS.
With all my heart, and think me honoured
To feast so great a warrior in my house.

[_Exeunt._]

SCENE IV. London. The Temple Garden.

Enter the Earls of Somerset, Suffolk, and Warwick; Richard Plantagenet,
Vernon and another Lawyer.

PLANTAGENET.
Great lords and gentlemen, what means this silence?
Dare no man answer in a case of truth?

SUFFOLK.
Within the Temple Hall we were too loud;
The garden here is more convenient.

PLANTAGENET.
Then say at once if I maintain’d the truth;
Or else was wrangling Somerset in th’ error?

SUFFOLK.
Faith, I have been a truant in the law
And never yet could frame my will to it;
And therefore frame the law unto my will.

SOMERSET.
Judge you, my Lord of Warwick, then, between us.

WARWICK.
Between two hawks, which flies the higher pitch;
Between two dogs, which hath the deeper mouth;
Between two blades, which bears the better temper;
Between two horses, which doth bear him best;
Between two girls, which hath the merriest eye;
I have perhaps some shallow spirit of judgement;
But in these nice sharp quillets of the law,
Good faith, I am no wiser than a daw.

PLANTAGENET.
Tut, tut, here is a mannerly forbearance!
The truth appears so naked on my side
That any purblind eye may find it out.

SOMERSET.
And on my side it is so well apparell’d,
So clear, so shining and so evident,
That it will glimmer through a blind man’s eye.

PLANTAGENET.
Since you are tongue-tied and so loath to speak,
In dumb significants proclaim your thoughts:
Let him that is a true-born gentleman
And stands upon the honour of his birth,
If he suppose that I have pleaded truth,
From off this brier pluck a white rose with me.

SOMERSET.
Let him that is no coward nor no flatterer,
But dare maintain the party of the truth,
Pluck a red rose from off this thorn with me.

WARWICK.
I love no colours, and without all colour
Of base insinuating flattery
I pluck this white rose with Plantagenet.

SUFFOLK.
I pluck this red rose with young Somerset,
And say withal I think he held the right.

VERNON.
Stay, lords and gentlemen, and pluck no more
Till you conclude that he upon whose side
The fewest roses are cropp’d from the tree
Shall yield the other in the right opinion.

SOMERSET.
Good Master Vernon, it is well objected:
If I have fewest, I subscribe in silence.

PLANTAGENET.
And I.

VERNON.
Then for the truth and plainness of the case,
I pluck this pale and maiden blossom here,
Giving my verdict on the white rose side.

SOMERSET.
Prick not your finger as you pluck it off,
Lest, bleeding, you do paint the white rose red,
And fall on my side so against your will.

VERNON.
If I, my lord, for my opinion bleed,
Opinion shall be surgeon to my hurt
And keep me on the side where still I am.

SOMERSET.
Well, well, come on, who else?

LAWYER.
Unless my study and my books be false,

[_To Somerset._]

The argument you held was wrong in law;
In sign whereof I pluck a white rose too.

PLANTAGENET.
Now, Somerset, where is your argument?

SOMERSET.
Here in my scabbard, meditating that
Shall dye your white rose in a bloody red.

PLANTAGENET.
Meantime your cheeks do counterfeit our roses;
For pale they look with fear, as witnessing
The truth on our side.

SOMERSET.
No, Plantagenet,
’Tis not for fear but anger that thy cheeks
Blush for pure shame to counterfeit our roses,
And yet thy tongue will not confess thy error.

PLANTAGENET.
Hath not thy rose a canker, Somerset?

SOMERSET.
Hath not thy rose a thorn, Plantagenet?

PLANTAGENET.
Ay, sharp and piercing, to maintain his truth;
Whiles thy consuming canker eats his falsehood.

SOMERSET.
Well, I’ll find friends to wear my bleeding roses,
That shall maintain what I have said is true,
Where false Plantagenet dare not be seen.

PLANTAGENET.
Now, by this maiden blossom in my hand,
I scorn thee and thy fashion, peevish boy.

SUFFOLK.
Turn not thy scorns this way, Plantagenet.

PLANTAGENET.
Proud Pole, I will, and scorn both him and thee.

SUFFOLK.
I’ll turn my part thereof into thy throat.

SOMERSET.
Away, away, good William de la Pole!
We grace the yeoman by conversing with him.

WARWICK.
Now, by God’s will, thou wrong’st him, Somerset;
His grandfather was Lionel Duke of Clarence,
Third son to the third Edward King of England.
Spring crestless yeomen from so deep a root?

PLANTAGENET.
He bears him on the place’s privilege,
Or durst not for his craven heart, say thus.

SOMERSET.
By Him that made me, I’ll maintain my words
On any plot of ground in Christendom.
Was not thy father, Richard Earl of Cambridge,
For treason executed in our late king’s days?
And, by his treason, stand’st not thou attainted,
Corrupted, and exempt from ancient gentry?
His trespass yet lives guilty in thy blood;
And, till thou be restored, thou art a yeoman.

PLANTAGENET.
My father was attached, not attainted,
Condemn’d to die for treason, but no traitor;
And that I’ll prove on better men than Somerset,
Were growing time once ripen’d to my will.
For your partaker Pole and you yourself,
I’ll note you in my book of memory,
To scourge you for this apprehension.
Look to it well, and say you are well warn’d.

SOMERSET.
Ah, thou shalt find us ready for thee still;
And know us by these colours for thy foes,
For these my friends in spite of thee shall wear.

PLANTAGENET.
And, by my soul, this pale and angry rose,
As cognizance of my blood-drinking hate,
Will I for ever and my faction wear,
Until it wither with me to my grave,
Or flourish to the height of my degree.

SUFFOLK.
Go forward, and be chok’d with thy ambition!
And so farewell until I meet thee next.

[_Exit._]

SOMERSET.
Have with thee, Pole. Farewell, ambitious Richard.

[_Exit._]

PLANTAGENET.
How I am braved and must perforce endure it!

WARWICK.
This blot that they object against your house
Shall be wiped out in the next parliament
Call’d for the truce of Winchester and Gloucester;
And if thou be not then created York,
I will not live to be accounted Warwick.
Meantime, in signal of my love to thee,
Against proud Somerset and William Pole,
Will I upon thy party wear this rose.
And here I prophesy: this brawl today,
Grown to this faction in the Temple Garden,
Shall send between the Red Rose and the White
A thousand souls to death and deadly night.

PLANTAGENET.
Good Master Vernon, I am bound to you,
That you on my behalf would pluck a flower.

VERNON.
In your behalf still will I wear the same.

LAWYER.
And so will I.

PLANTAGENET.
Thanks, gentlemen.
Come, let us four to dinner. I dare say
This quarrel will drink blood another day.

[_Exeunt._]

SCENE V. The Tower of London.

Enter Mortimer, brought in a chair, and Jailers.

MORTIMER.
Kind keepers of my weak decaying age,
Let dying Mortimer here rest himself.
Even like a man new haled from the rack,
So fare my limbs with long imprisonment;
And these gray locks, the pursuivants of death,
Nestor-like aged in an age of care,
Argue the end of Edmund Mortimer.
These eyes, like lamps whose wasting oil is spent,
Wax dim, as drawing to their exigent;
Weak shoulders, overborne with burdening grief,
And pithless arms, like to a wither’d vine
That droops his sapless branches to the ground.
Yet are these feet, whose strengthless stay is numb,
Unable to support this lump of clay,
Swift-winged with desire to get a grave,
As witting I no other comfort have.
But tell me, keeper, will my nephew come?

FIRST JAILER.
Richard Plantagenet, my lord, will come.
We sent unto the Temple, unto his chamber,
And answer was return’d that he will come.

MORTIMER.
Enough. My soul shall then be satisfied.
Poor gentleman, his wrong doth equal mine.
Since Henry Monmouth first began to reign,
Before whose glory I was great in arms,
This loathsome sequestration have I had;
And even since then hath Richard been obscured,
Deprived of honour and inheritance.
But now the arbitrator of despairs,
Just Death, kind umpire of men’s miseries,
With sweet enlargement doth dismiss me hence.
I would his troubles likewise were expired,
That so he might recover what was lost.

Enter Richard Plantagenet.

FIRST JAILER.
My lord, your loving nephew now is come.

MORTIMER.
Richard Plantagenet, my friend, is he come?

PLANTAGENET.
Ay, noble uncle, thus ignobly used,
Your nephew, late despised Richard, comes.

MORTIMER.
Direct mine arms I may embrace his neck
And in his bosom spend my latter gasp.
O, tell me when my lips do touch his cheeks,
That I may kindly give one fainting kiss.
And now declare, sweet stem from York’s great stock,
Why didst thou say of late thou wert despised?

PLANTAGENET.
First, lean thine aged back against mine arm,
And, in that ease, I’ll tell thee my disease.
This day, in argument upon a case,
Some words there grew ’twixt Somerset and me;
Among which terms he used his lavish tongue
And did upbraid me with my father’s death;
Which obloquy set bars before my tongue,
Else with the like I had requited him.
Therefore, good uncle, for my father’s sake,
In honour of a true Plantagenet,
And for alliance’ sake, declare the cause
My father, Earl of Cambridge, lost his head.

MORTIMER.
That cause, fair nephew, that imprison’d me
And hath detain’d me all my flowering youth
Within a loathsome dungeon, there to pine,
Was cursed instrument of his decease.

PLANTAGENET.
Discover more at large what cause that was,
For I am ignorant and cannot guess.

MORTIMER.
I will, if that my fading breath permit
And death approach not ere my tale be done.
Henry the Fourth, grandfather to this king,
Deposed his nephew Richard, Edward’s son,
The first-begotten and the lawful heir
Of Edward king, the third of that descent;
During whose reign the Percies of the north,
Finding his usurpation most unjust,
Endeavour’d my advancement to the throne.
The reason moved these warlike lords to this
Was, for that—young King Richard thus removed,
Leaving no heir begotten of his body—
I was the next by birth and parentage;
For by my mother I derived am
From Lionel Duke of Clarence, third son
To King Edward the Third; whereas he
From John of Gaunt doth bring his pedigree,
Being but fourth of that heroic line.
But mark: as in this haughty great attempt
They labored to plant the rightful heir,
I lost my liberty and they their lives.
Long after this, when Henry the Fifth,
Succeeding his father Bolingbroke, did reign,
Thy father, Earl of Cambridge then, derived
From famous Edmund Langley, Duke of York,
Marrying my sister that thy mother was,
Again, in pity of my hard distress.
Levied an army, weening to redeem
And have install’d me in the diadem.
But, as the rest, so fell that noble earl
And was beheaded. Thus the Mortimers,
In whom the title rested, were suppress’d.

PLANTAGENET.
Of which, my lord, your honour is the last.

MORTIMER.
True; and thou seest that I no issue have,
And that my fainting words do warrant death.
Thou art my heir; the rest I wish thee gather.
But yet be wary in thy studious care.

PLANTAGENET.
Thy grave admonishments prevail with me.
But yet methinks, my father’s execution
Was nothing less than bloody tyranny.

MORTIMER.
With silence, nephew, be thou politic;
Strong-fixed is the house of Lancaster,
And like a mountain, not to be removed.
But now thy uncle is removing hence,
As princes do their courts when they are cloy’d
With long continuance in a settled place.

PLANTAGENET.
O uncle, would some part of my young years
Might but redeem the passage of your age!

MORTIMER.
Thou dost then wrong me, as that slaughterer doth
Which giveth many wounds when one will kill.
Mourn not, except thou sorrow for my good;
Only give order for my funeral.
And so farewell, and fair be all thy hopes,
And prosperous be thy life in peace and war!

[_Dies._]

PLANTAGENET.
And peace, no war, befall thy parting soul!
In prison hast thou spent a pilgrimage,
And like a hermit overpass’d thy days.
Well, I will lock his counsel in my breast;
And what I do imagine, let that rest.
Keepers, convey him hence; and I myself
Will see his burial better than his life.

[_Exeunt Jailers, bearing out the body of Mortimer._]

Here dies the dusky torch of Mortimer,
Choked with ambition of the meaner sort.
And for those wrongs, those bitter injuries,
Which Somerset hath offer’d to my house,
I doubt not but with honour to redress;
And therefore haste I to the Parliament,
Either to be restored to my blood,
Or make mine ill th’ advantage of my good.

[_Exit._]




ACT III

SCENE I. London. The Parliament House.


Flourish. Enter King, Exeter, Gloucester, the Bishop of Winchester,
Richard Plantagenet, Warwick, and Somerset, Suffolk, and others.
Gloucester offers to put up a bill. Winchester snatches it, tears it.

WINCHESTER.
Com’st thou with deep premeditated lines,
With written pamphlets studiously devised,
Humphrey of Gloucester? If thou canst accuse
Or aught intend’st to lay unto my charge,
Do it without invention, suddenly;
As I with sudden and extemporal speech
Purpose to answer what thou canst object.

GLOUCESTER.
Presumptuous priest, this place commands my patience,
Or thou shouldst find thou hast dishonour’d me.
Think not, although in writing I preferr’d
The manner of thy vile outrageous crimes,
That therefore I have forged, or am not able
Verbatim to rehearse the method of my pen.
No, prelate; such is thy audacious wickedness,
Thy lewd, pestiferous, and dissentious pranks,
As very infants prattle of thy pride.
Thou art a most pernicious usurer,
Froward by nature, enemy to peace;
Lascivious, wanton, more than well beseems
A man of thy profession and degree;
And for thy treachery, what’s more manifest,
In that thou laid’st a trap to take my life,
As well at London Bridge as at the Tower?
Beside, I fear me, if thy thoughts are sifted,
The King, thy sovereign, is not quite exempt
From envious malice of thy swelling heart.

WINCHESTER.
Gloucester, I do defy thee. Lords, vouchsafe
To give me hearing what I shall reply.
If I were covetous, ambitious, or perverse,
As he will have me, how am I so poor?
Or how haps it I seek not to advance
Or raise myself, but keep my wonted calling?
And for dissension, who preferreth peace
More than I do, except I be provoked?
No, my good lords, it is not that offends;
It is not that that hath incensed the Duke.
It is because no one should sway but he,
No one but he should be about the King;
And that engenders thunder in his breast
And makes him roar these accusations forth.
But he shall know I am as good—

GLOUCESTER.
As good!
Thou bastard of my grandfather!

WINCHESTER.
Ay, lordly sir; for what are you, I pray,
But one imperious in another’s throne?

GLOUCESTER.
Am I not Protector, saucy priest?

WINCHESTER.
And am not I a prelate of the church?

GLOUCESTER.
Yes, as an outlaw in a castle keeps,
And useth it to patronage his theft.

WINCHESTER.
Unreverent Gloucester!

GLOUCESTER.
Thou art reverend
Touching thy spiritual function, not thy life.

WINCHESTER.
Rome shall remedy this.

GLOUCESTER.
Roam thither, then.

WARWICK.
My lord, it were your duty to forbear.

SOMERSET.
Ay, so the bishop be not overborne.
Methinks my lord should be religious,
And know the office that belongs to such.

WARWICK.
Methinks his lordship should be humbler;
It fitteth not a prelate so to plead.

SOMERSET.
Yes, when his holy state is touch’d so near.

WARWICK.
State holy or unhallow’d, what of that?
Is not his Grace Protector to the King?

PLANTAGENET.
[_Aside_.] Plantagenet, I see, must hold his tongue,
Lest it be said, “Speak, sirrah, when you should;
Must your bold verdict enter talk with lords?”
Else would I have a fling at Winchester.

KING HENRY.
Uncles of Gloucester and of Winchester,
The special watchmen of our English weal,
I would prevail, if prayers might prevail,
To join your hearts in love and amity.
O, what a scandal is it to our crown
That two such noble peers as ye should jar!
Believe me, lords, my tender years can tell
Civil dissension is a viperous worm
That gnaws the bowels of the commonwealth.

[_A noise within, “Down with the tawny-coats!”._]

What tumult’s this?

WARWICK.
An uproar, I dare warrant,
Begun through malice of the Bishop’s men.

[_A noise again, “Stones! stones!”_]

Enter Mayor.

MAYOR.
O, my good lords, and virtuous Henry,
Pity the city of London, pity us!
The Bishop and the Duke of Gloucester’s men,
Forbidden late to carry any weapon,
Have fill’d their pockets full of pebble stones
And, banding themselves in contrary parts,
Do pelt so fast at one another’s pate
That many have their giddy brains knock’d out;
Our windows are broke down in every street,
And we for fear compell’d to shut our shops.

Enter Servingmen in skirmish with bloody pates.

KING HENRY.
We charge you, on allegiance to ourself,
To hold your slaughtering hands and keep the peace.
Pray, uncle Gloucester, mitigate this strife.

FIRST SERVINGMAN.
Nay, if we be forbidden stones, we’ll fall to it with our teeth.

SECOND SERVINGMAN.
Do what ye dare, we are as resolute.

[_Skirmish again._]

GLOUCESTER.
You of my household, leave this peevish broil,
And set this unaccustom’d fight aside.

THIRD SERVINGMAN.
My lord, we know your Grace to be a man
Just and upright, and, for your royal birth,
Inferior to none but to his Majesty;
And ere that we will suffer such a prince,
So kind a father of the commonweal,
To be disgraced by an inkhorn mate,
We and our wives and children all will fight
And have our bodies slaughter’d by thy foes.

FIRST SERVINGMAN.
Ay, and the very parings of our nails
Shall pitch a field when we are dead.

[_Begin again._]

GLOUCESTER.
Stay, stay, I say!
And if you love me, as you say you do,
Let me persuade you to forbear awhile.

KING HENRY.
O, how this discord doth afflict my soul!
Can you, my Lord of Winchester, behold
My sighs and tears, and will not once relent?
Who should be pitiful, if you be not?
Or who should study to prefer a peace
If holy churchmen take delight in broils?

WARWICK.
Yield, my Lord Protector; yield, Winchester;
Except you mean with obstinate repulse
To slay your sovereign and destroy the realm.
You see what mischief and what murder too,
Hath been enacted through your enmity;
Then be at peace, except ye thirst for blood.

WINCHESTER.
He shall submit, or I will never yield.

GLOUCESTER.
Compassion on the King commands me stoop,
Or I would see his heart out, ere the priest
Should ever get that privilege of me.

WARWICK.
Behold, my Lord of Winchester, the Duke
Hath banish’d moody discontented fury,
As by his smoothed brows it doth appear.
Why look you still so stern and tragical?

GLOUCESTER.
Here, Winchester, I offer thee my hand.

KING HENRY.
Fie, uncle Beaufort! I have heard you preach
That malice was a great and grievous sin;
And will not you maintain the thing you teach,
But prove a chief offender in the same?

WARWICK.
Sweet King! The bishop hath a kindly gird.
For shame, my Lord of Winchester, relent!
What, shall a child instruct you what to do?

WINCHESTER.
Well, Duke of Gloucester, I will yield to thee;
Love for thy love and hand for hand I give.

GLOUCESTER.
[_Aside_.] Ay, but, I fear me, with a hollow heart.—
See here, my friends and loving countrymen,
This token serveth for a flag of truce
Betwixt ourselves and all our followers,
So help me God, as I dissemble not!

WINCHESTER.
[_Aside_.] So help me God, as I intend it not!

KING HENRY.
O loving uncle, kind Duke of Gloucester,
How joyful am I made by this contract!
Away, my masters, trouble us no more,
But join in friendship, as your lords have done.

FIRST SERVINGMAN.
Content. I’ll to the surgeon’s.

SECOND SERVINGMAN.
And so will I.

THIRD SERVINGMAN.
And I will see what physic the tavern affords.

[_Exeunt Servingmen, Mayor, &c._]

WARWICK.
Accept this scroll, most gracious sovereign,
Which in the right of Richard Plantagenet
We do exhibit to your Majesty.

GLOUCESTER.
Well urged, my Lord of Warwick. For, sweet prince,
An if your Grace mark every circumstance,
You have great reason to do Richard right,
Especially for those occasions
At Eltham Place I told your Majesty.

KING HENRY.
And those occasions, uncle, were of force;
Therefore, my loving lords, our pleasure is
That Richard be restored to his blood.

WARWICK.
Let Richard be restored to his blood;
So shall his father’s wrongs be recompensed.

WINCHESTER.
As will the rest, so willeth Winchester.

KING HENRY.
If Richard will be true, not that alone
But all the whole inheritance I give
That doth belong unto the house of York,
From whence you spring by lineal descent.

PLANTAGENET.
Thy humble servant vows obedience
And humble service till the point of death.

KING HENRY.
Stoop then and set your knee against my foot;
And in reguerdon of that duty done
I girt thee with the valiant sword of York.
Rise, Richard, like a true Plantagenet,
And rise created princely Duke of York.

PLANTAGENET.
And so thrive Richard as thy foes may fall!
And as my duty springs, so perish they
That grudge one thought against your Majesty!

ALL.
Welcome, high prince, the mighty Duke of York!

SOMERSET.
[_Aside_.] Perish, base prince, ignoble Duke of York!

GLOUCESTER.
Now will it best avail your Majesty
To cross the seas and to be crown’d in France.
The presence of a king engenders love
Amongst his subjects and his loyal friends,
As it disanimates his enemies.

KING HENRY.
When Gloucester says the word, King Henry goes;
For friendly counsel cuts off many foes.

GLOUCESTER.
Your ships already are in readiness.

[_Sennet. Flourish. Exeunt all but Exeter._]

EXETER.
Ay, we may march in England or in France,
Not seeing what is likely to ensue.
This late dissension grown betwixt the peers
Burns under feigned ashes of forged love,
And will at last break out into a flame;
As festered members rot but by degree
Till bones and flesh and sinews fall away,
So will this base and envious discord breed.
And now I fear that fatal prophecy
Which in the time of Henry named the Fifth
Was in the mouth of every sucking babe:
That Henry born at Monmouth should win all,
And Henry born at Windsor lose all,
Which is so plain that Exeter doth wish
His days may finish ere that hapless time.

[_Exit._]

SCENE II. France. Before Rouen.

Enter La Pucelle with four Soldiers with sacks upon their backs.

PUCELLE.
These are the city gates, the gates of Rouen,
Through which our policy must make a breach.
Take heed, be wary how you place your words;
Talk like the vulgar sort of market men
That come to gather money for their corn.
If we have entrance, as I hope we shall,
And that we find the slothful watch but weak,
I’ll by a sign give notice to our friends,
That Charles the Dauphin may encounter them.

FIRST SOLDIER.
Our sacks shall be a mean to sack the city,
And we be lords and rulers over Rouen;
Therefore we’ll knock. [_Knocks._]

WATCH.
[_Within_.] _Qui est la?_

PUCELLE.
_Paysans, la pauvres gens de France:_
Poor market folks that come to sell their corn.

WATCH.
Enter, go in; the market bell is rung.

PUCELLE.
Now, Rouen, I’ll shake thy bulwarks to the ground.

[_Exeunt._]

Enter Charles, the Bastard of Orleans, Alençon, Reignier and forces.

CHARLES.
Saint Denis bless this happy stratagem,
And once again we’ll sleep secure in Rouen.

BASTARD.
Here enter’d Pucelle and her practisants;
Now she is there, how will she specify
Here is the best and safest passage in?

REIGNIER.
By thrusting out a torch from yonder tower,
Which, once discern’d, shows that her meaning is:
No way to that, for weakness, which she enter’d.

Enter La Pucelle, on the top, thrusting out a torch burning.

PUCELLE.
Behold, this is the happy wedding torch
That joineth Rouen unto her countrymen,
But burning fatal to the Talbonites.

[_Exit._]

BASTARD.
See, noble Charles, the beacon of our friend;
The burning torch, in yonder turret stands.

CHARLES.
Now shine it like a comet of revenge,
A prophet to the fall of all our foes!

REIGNIER.
Defer no time, delays have dangerous ends;
Enter and cry, “The Dauphin!” presently,
And then do execution on the watch.

[_Alarum. Exeunt._]

An alarum. Enter Talbot in an excursion.

TALBOT.
France, thou shalt rue this treason with thy tears,
If Talbot but survive thy treachery.
Pucelle, that witch, that damned sorceress,
Hath wrought this hellish mischief unawares,
That hardly we escaped the pride of France.

[_Exit._]

An alarum. Excursions. Bedford, brought in sick in a chair. Enter
Talbot and Burgundy without: within, La Pucelle, Charles, Bastard,
Alençon, and Reignier on the walls.

PUCELLE.
Good morrow, gallants! Want ye corn for bread?
I think the Duke of Burgundy will fast
Before he’ll buy again at such a rate.
’Twas full of darnel. Do you like the taste?

BURGUNDY.
Scoff on, vile fiend and shameless courtezan!
I trust ere long to choke thee with thine own,
And make thee curse the harvest of that corn.

CHARLES.
Your Grace may starve, perhaps, before that time.

BEDFORD.
O, let no words, but deeds, revenge this treason!

PUCELLE.
What will you do, good graybeard? Break a lance
And run a tilt at Death within a chair?

TALBOT.
Foul fiend of France and hag of all despite,
Encompass’d with thy lustful paramours,
Becomes it thee to taunt his valiant age
And twit with cowardice a man half dead?
Damsel, I’ll have a bout with you again,
Or else let Talbot perish with this shame.

PUCELLE.
Are ye so hot? Yet, Pucelle, hold thy peace;
If Talbot do but thunder, rain will follow.

[_The English whisper together in council._]

God speed the Parliament! Who shall be the Speaker?

TALBOT.
Dare ye come forth and meet us in the field?

PUCELLE.
Belike your lordship takes us then for fools,
To try if that our own be ours or no.

TALBOT.
I speak not to that railing Hecate,
But unto thee, Alençon, and the rest;
Will ye, like soldiers, come and fight it out?

ALENÇON.
Seignieur, no.

TALBOT.
Seignieur, hang! Base muleteers of France!
Like peasant foot-boys do they keep the walls,
And dare not take up arms like gentlemen.

PUCELLE.
Away, captains! Let’s get us from the walls,
For Talbot means no goodness by his looks.
Goodbye, my lord; we came but to tell you
That we are here.

[_Exeunt from the walls._]

TALBOT.
And there will we be too, ere it be long,
Or else reproach be Talbot’s greatest fame!
Vow, Burgundy, by honour of thy house,
Prick’d on by public wrongs sustain’d in France,
Either to get the town again or die.
And I, as sure as English Henry lives,
And as his father here was conqueror,
As sure as in this late-betrayed town
Great Coeur-de-lion’s heart was buried,
So sure I swear to get the town or die.

BURGUNDY.
My vows are equal partners with thy vows.

TALBOT.
But, ere we go, regard this dying prince,
The valiant Duke of Bedford. Come, my lord,
We will bestow you in some better place,
Fitter for sickness and for crazy age.

BEDFORD.
Lord Talbot, do not so dishonour me.
Here will I sit before the walls of Rouen,
And will be partner of your weal or woe.

BURGUNDY.
Courageous Bedford, let us now persuade you.

BEDFORD.
Not to be gone from hence; for once I read
That stout Pendragon in his litter sick
Came to the field and vanquished his foes.
Methinks I should revive the soldiers’ hearts,
Because I ever found them as myself.

TALBOT.
Undaunted spirit in a dying breast!
Then be it so. Heavens keep old Bedford safe!
And now no more ado, brave Burgundy,
But gather we our forces out of hand
And set upon our boasting enemy.

[_Exeunt all but Bedford and Attendants._]

An alarum. Excursions. Enter Sir John Fastolf and a Captain.

CAPTAIN.
Whither away, Sir John Fastolf, in such haste?

FASTOLF.
Whither away? To save myself by flight.
We are like to have the overthrow again.

CAPTAIN.
What! Will you fly, and leave Lord Talbot?

FASTOLF.
Ay,
All the Talbots in the world, to save my life.

[_Exit._]

CAPTAIN.
Cowardly knight, ill fortune follow thee!

[_Exit._]

Retreat. Excursions. La Pucelle, Alençon and Charles fly.

BEDFORD.
Now, quiet soul, depart when heaven please,
For I have seen our enemies’ overthrow.
What is the trust or strength of foolish man?
They that of late were daring with their scoffs
Are glad and fain by flight to save themselves.

[_Bedford dies, and is carried in by two in his chair._]

An alarum. Enter Talbot, Burgundy and the rest.

TALBOT.
Lost, and recover’d in a day again!
This is a double honour, Burgundy.
Yet heavens have glory for this victory!

BURGUNDY.
Warlike and martial Talbot, Burgundy
Enshrines thee in his heart, and there erects
Thy noble deeds as valour’s monuments.

TALBOT.
Thanks, gentle Duke. But where is Pucelle now?
I think her old familiar is asleep.
Now where’s the Bastard’s braves, and Charles his gleeks?
What, all amort? Rouen hangs her head for grief
That such a valiant company are fled.
Now will we take some order in the town,
Placing therein some expert officers,
And then depart to Paris to the King,
For there young Henry with his nobles lie.

BURGUNDY.
What wills Lord Talbot pleaseth Burgundy.

TALBOT.
But yet, before we go, let’s not forget
The noble Duke of Bedford late deceased,
But see his exequies fulfill’d in Rouen.
A braver soldier never couched lance,
A gentler heart did never sway in court;
But kings and mightiest potentates must die,
For that’s the end of human misery.

[_Exeunt._]

SCENE III. The plains near Rouen.

Enter Charles, the Bastard of Orleans, Alençon, La Pucelle and forces.

PUCELLE.
Dismay not, princes, at this accident,
Nor grieve that Rouen is so recovered.
Care is no cure, but rather corrosive,
For things that are not to be remedied.
Let frantic Talbot triumph for a while
And like a peacock sweep along his tail;
We’ll pull his plumes and take away his train,
If Dauphin and the rest will be but ruled.

CHARLES.
We have been guided by thee hitherto,
And of thy cunning had no diffidence.
One sudden foil shall never breed distrust

BASTARD.
Search out thy wit for secret policies,
And we will make thee famous through the world.

ALENÇON.
We’ll set thy statue in some holy place,
And have thee reverenced like a blessed saint.
Employ thee then, sweet virgin, for our good.

PUCELLE.
Then thus it must be; this doth Joan devise:
By fair persuasions mix’d with sugar’d words
We will entice the Duke of Burgundy
To leave the Talbot and to follow us.

CHARLES.
Ay, marry, sweeting, if we could do that,
France were no place for Henry’s warriors;
Nor should that nation boast it so with us,
But be extirped from our provinces.

ALENÇON.
For ever should they be expulsed from France,
And not have title of an earldom here.

PUCELLE.
Your honours shall perceive how I will work
To bring this matter to the wished end.

[_Drum sounds afar off._]

Hark! By the sound of drum you may perceive
Their powers are marching unto Paris-ward.

[_Here sound an English march._]

There goes the Talbot, with his colours spread,
And all the troops of English after him.

[_French march._]

Now in the rearward comes the Duke and his.
Fortune in favour makes him lag behind.
Summon a parley; we will talk with him.

[_Trumpets sound a parley._]

CHARLES.
A parley with the Duke of Burgundy!

Enter Burgundy.

BURGUNDY.
Who craves a parley with the Burgundy?

PUCELLE.
The princely Charles of France, thy countryman.

BURGUNDY.
What say’st thou, Charles? for I am marching hence.

CHARLES.
Speak, Pucelle, and enchant him with thy words.

PUCELLE.
Brave Burgundy, undoubted hope of France,
Stay, let thy humble handmaid speak to thee.

BURGUNDY.
Speak on, but be not over-tedious.

PUCELLE.
Look on thy country, look on fertile France,
And see the cities and the towns defaced
By wasting ruin of the cruel foe.
As looks the mother on her lowly babe
When death doth close his tender dying eyes,
See, see the pining malady of France;
Behold the wounds, the most unnatural wounds,
Which thou thyself hast given her woeful breast.
O, turn thy edged sword another way;
Strike those that hurt, and hurt not those that help.
One drop of blood drawn from thy country’s bosom
Should grieve thee more than streams of foreign gore.
Return thee therefore with a flood of tears,
And wash away thy country’s stained spots.

BURGUNDY.
Either she hath bewitch’d me with her words,
Or nature makes me suddenly relent.

PUCELLE.
Besides, all French and France exclaims on thee,
Doubting thy birth and lawful progeny.
Who join’st thou with but with a lordly nation
That will not trust thee but for profit’s sake?
When Talbot hath set footing once in France,
And fashion’d thee that instrument of ill,
Who then but English Henry will be lord,
And thou be thrust out like a fugitive?
Call we to mind, and mark but this for proof:
Was not the Duke of Orleans thy foe?
And was he not in England prisoner?
But when they heard he was thine enemy,
They set him free without his ransom paid,
In spite of Burgundy and all his friends.
See then, thou fight’st against thy countrymen,
And join’st with them will be thy slaughtermen.
Come, come, return; return, thou wandering lord;
Charles and the rest will take thee in their arms.

BURGUNDY.
I am vanquished; these haughty words of hers
Have batter’d me like roaring cannon-shot,
And made me almost yield upon my knees.
Forgive me, country, and sweet countrymen!
And, lords, accept this hearty kind embrace.
My forces and my power of men are yours.
So, farewell, Talbot; I’ll no longer trust thee.

PUCELLE.
[_Aside_.] Done like a Frenchman: turn and turn again.

CHARLES.
Welcome, brave Duke! Thy friendship makes us fresh.

BASTARD.
And doth beget new courage in our breasts.

ALENÇON.
Pucelle hath bravely play’d her part in this,
And doth deserve a coronet of gold.

CHARLES.
Now let us on, my lords, and join our powers,
And seek how we may prejudice the foe.

[_Exeunt._]

SCENE IV. Paris. The Palace.

Enter the King, Gloucester, Bishop of Winchester, Exeter, York, Warwick
and Vernon; Suffolk, Somerset, Basset and others. To them, with his
soldiers, Talbot.

TALBOT.
My gracious Prince, and honourable peers,
Hearing of your arrival in this realm,
I have awhile given truce unto my wars
To do my duty to my sovereign;
In sign whereof, this arm, that hath reclaim’d
To your obedience fifty fortresses,
Twelve cities and seven walled towns of strength,
Beside five hundred prisoners of esteem,
Lets fall his sword before your Highness’ feet,
And with submissive loyalty of heart
Ascribes the glory of his conquest got
First to my God, and next unto your Grace. [_Kneels_.]

KING HENRY.
Is this the Lord Talbot, uncle Gloucester,
That hath so long been resident in France?

GLOUCESTER.
Yes, if it please your Majesty, my liege.

KING HENRY.
Welcome, brave captain and victorious lord!
When I was young, as yet I am not old,
I do remember how my father said
A stouter champion never handled sword.
Long since we were resolved of your truth,
Your faithful service, and your toil in war;
Yet never have you tasted our reward,
Or been reguerdon’d with so much as thanks.
Because till now we never saw your face.
Therefore, stand up; and for these good deserts
We here create you Earl of Shrewsbury;
And in our coronation take your place.

[_Sennet. Flourish. Exeunt all but Vernon and Basset._]

VERNON.
Now, sir, to you that were so hot at sea,
Disgracing of these colours that I wear
In honour of my noble Lord of York,
Dar’st thou maintain the former words thou spak’st?

BASSET.
Yes, sir, as well as you dare patronage
The envious barking of your saucy tongue
Against my lord the Duke of Somerset.

VERNON.
Sirrah, thy lord I honour as he is.

BASSET.
Why, what is he? As good a man as York.

VERNON.
Hark ye; not so: in witness, take ye that.

[_Strikes him._]

BASSET.
Villain, thou knowest the law of arms is such
That whoso draws a sword, ’tis present death,
Or else this blow should broach thy dearest blood.
But I’ll unto his Majesty, and crave
I may have liberty to venge this wrong;
When thou shalt see I’ll meet thee to thy cost.

VERNON.
Well, miscreant, I’ll be there as soon as you;
And, after, meet you sooner than you would.

[_Exeunt._]




ACT IV

SCENE I. Paris. The Palace.


Enter the King, Gloucester, Bishop of Winchester, Talbot, Exeter, York,
and Warwick; Suffolk, Somerset, the Governor of Paris, and others.

GLOUCESTER.
Lord bishop, set the crown upon his head.

WINCHESTER.
God save King Henry, of that name the Sixth!

GLOUCESTER.
Now, Governor of Paris, take your oath,
That you elect no other king but him;
Esteem none friends but such as are his friends,
And none your foes but such as shall pretend
Malicious practices against his state:
This shall ye do, so help you righteous God!

Enter Sir John Fastolf.

FASTOLF.
My gracious sovereign, as I rode from Calais,
To haste unto your coronation,
A letter was deliver’d to my hands,
Writ to your Grace from th’ Duke of Burgundy.

TALBOT.
Shame to the Duke of Burgundy and thee!
I vow’d, base knight, when I did meet thee next,
To tear the Garter from thy craven’s leg, [_Plucking it off_.]
Which I have done, because unworthily
Thou wast installed in that high degree.
Pardon me, princely Henry, and the rest.
This dastard, at the battle of Patay,
When but in all I was six thousand strong
And that the French were almost ten to one,
Before we met or that a stroke was given,
Like to a trusty squire did run away;
In which assault we lost twelve hundred men;
Myself and divers gentlemen beside
Were there surprised and taken prisoners.
Then judge, great lords, if I have done amiss,
Or whether that such cowards ought to wear
This ornament of knighthood, yea or no?

GLOUCESTER.
To say the truth, this fact was infamous
And ill beseeming any common man,
Much more a knight, a captain, and a leader.

TALBOT.
When first this Order was ordain’d, my lords,
Knights of the Garter were of noble birth,
Valiant and virtuous, full of haughty courage,
Such as were grown to credit by the wars;
Not fearing death nor shrinking for distress,
But always resolute in most extremes.
He then that is not furnish’d in this sort
Doth but usurp the sacred name of knight,
Profaning this most honourable Order,
And should, if I were worthy to be judge,
Be quite degraded, like a hedge-born swain
That doth presume to boast of gentle blood.

KING HENRY.
Stain to thy countrymen, thou hear’st thy doom!
Be packing, therefore, thou that wast a knight;
Henceforth we banish thee on pain of death.

[_Exit Fastolf._]

And now, my Lord Protector, view the letter
Sent from our uncle, Duke of Burgundy.

GLOUCESTER.
What means his Grace, that he hath changed his style?
No more but, plain and bluntly, “To the King”!
Hath he forgot he is his sovereign?
Or doth this churlish superscription
Pretend some alteration in good will?
What’s here? [_Reads_] “I have, upon especial cause,
Moved with compassion of my country’s wrack,
Together with the pitiful complaints
Of such as your oppression feeds upon,
Forsaken your pernicious faction
And join’d with Charles, the rightful King of France.”
O monstrous treachery! Can this be so,
That in alliance, amity, and oaths,
There should be found such false dissembling guile?

KING HENRY.
What! Doth my uncle Burgundy revolt?

GLOUCESTER.
He doth, my lord, and is become your foe.

KING HENRY.
Is that the worst this letter doth contain?

GLOUCESTER.
It is the worst, and all, my lord, he writes.

KING HENRY.
Why, then, Lord Talbot there shall talk with him
And give him chastisement for this abuse.
How say you, my lord, are you not content?

TALBOT.
Content, my liege! Yes, but that I am prevented,
I should have begg’d I might have been employ’d.

KING HENRY.
Then gather strength and march unto him straight;
Let him perceive how ill we brook his treason,
And what offence it is to flout his friends.

TALBOT.
I go, my lord, in heart desiring still
You may behold confusion of your foes.

[_Exit._]

Enter Vernon and Basset.

VERNON.
Grant me the combat, gracious sovereign.

BASSET.
And me, my lord, grant me the combat too.

YORK.
This is my servant; hear him, noble prince.

SOMERSET.
And this is mine, sweet Henry, favour him.

KING HENRY.
Be patient, lords, and give them leave to speak.
Say, gentlemen, what makes you thus exclaim,
And wherefore crave you combat, or with whom?

VERNON.
With him, my lord, for he hath done me wrong.

BASSET.
And I with him, for he hath done me wrong.

KING HENRY.
What is that wrong whereof you both complain?
First let me know, and then I’ll answer you.

BASSET.
Crossing the sea from England into France,
This fellow here, with envious carping tongue,
Upbraided me about the rose I wear,
Saying the sanguine colour of the leaves
Did represent my master’s blushing cheeks
When stubbornly he did repugn the truth
About a certain question in the law
Argued betwixt the Duke of York and him;
With other vile and ignominious terms.
In confutation of which rude reproach,
And in defence of my lord’s worthiness,
I crave the benefit of law of arms.

VERNON.
And that is my petition, noble lord;
For though he seem with forged quaint conceit
To set a gloss upon his bold intent,
Yet know, my lord, I was provoked by him,
And he first took exceptions at this badge,
Pronouncing that the paleness of this flower
Bewray’d the faintness of my master’s heart.

YORK.
Will not this malice, Somerset, be left?

SOMERSET.
Your private grudge, my Lord of York, will out,
Though ne’er so cunningly you smother it.

KING HENRY.
Good Lord, what madness rules in brainsick men,
When for so slight and frivolous a cause
Such factious emulations shall arise!
Good cousins both, of York and Somerset,
Quiet yourselves, I pray, and be at peace.

YORK.
Let this dissension first be tried by fight,
And then your Highness shall command a peace.

SOMERSET.
The quarrel toucheth none but us alone;
Betwixt ourselves let us decide it then.

YORK.
There is my pledge; accept it, Somerset.

VERNON.
Nay, let it rest where it began at first.

BASSET.
Confirm it so, mine honourable lord.

GLOUCESTER.
Confirm it so! Confounded be your strife!
And perish ye, with your audacious prate!
Presumptuous vassals, are you not ashamed
With this immodest clamorous outrage
To trouble and disturb the King and us?
And you, my lords, methinks you do not well
To bear with their perverse objections,
Much less to take occasion from their mouths
To raise a mutiny betwixt yourselves.
Let me persuade you take a better course.

EXETER.
It grieves his Highness. Good my lords, be friends.

KING HENRY.
Come hither, you that would be combatants:
Henceforth I charge you, as you love our favour,
Quite to forget this quarrel and the cause.
And you, my lords, remember where we are:
In France, amongst a fickle wavering nation;
If they perceive dissension in our looks,
And that within ourselves we disagree,
How will their grudging stomachs be provoked
To willful disobedience, and rebel!
Beside, what infamy will there arise
When foreign princes shall be certified
That for a toy, a thing of no regard,
King Henry’s peers and chief nobility
Destroy’d themselves and lost the realm of France!
O, think upon the conquest of my father,
My tender years, and let us not forgo
That for a trifle that was bought with blood!
Let me be umpire in this doubtful strife.
I see no reason if I wear this rose,

[_Putting on a red rose._]

That anyone should therefore be suspicious
I more incline to Somerset than York.
Both are my kinsmen, and I love them both.
As well they may upbraid me with my crown
Because, forsooth, the King of Scots is crown’d.
But your discretions better can persuade
Than I am able to instruct or teach;
And therefore, as we hither came in peace,
So let us still continue peace and love.
Cousin of York, we institute your Grace
To be our Regent in these parts of France;
And, good my Lord of Somerset, unite
Your troops of horsemen with his bands of foot;
And like true subjects, sons of your progenitors,
Go cheerfully together and digest
Your angry choler on your enemies.
Ourself, my Lord Protector, and the rest
After some respite will return to Calais;
From thence to England, where I hope ere long
To be presented, by your victories,
With Charles, Alençon, and that traitorous rout.

[_Flourish. Exeunt all but York, Warwick, Exeter and Vernon._]

WARWICK.
My Lord of York, I promise you the King
Prettily, methought, did play the orator.

YORK.
And so he did; but yet I like it not,
In that he wears the badge of Somerset.

WARWICK.
Tush, that was but his fancy; blame him not;
I dare presume, sweet prince, he thought no harm.

YORK.
An if I wist he did—but let it rest;
Other affairs must now be managed.

[_Exeunt all but Exeter._]

EXETER.
Well didst thou, Richard, to suppress thy voice;
For, had the passions of thy heart burst out,
I fear we should have seen decipher’d there
More rancorous spite, more furious raging broils,
Than yet can be imagined or supposed.
But howsoe’er, no simple man that sees
This jarring discord of nobility,
This shouldering of each other in the court,
This factious bandying of their favourites,
But sees it doth presage some ill event.
’Tis much when scepters are in children’s hands;
But more when envy breeds unkind division:
There comes the ruin, there begins confusion.

[_Exit._]

SCENE II. Before Bordeaux.

Enter Talbot with trump and drum.

TALBOT.
Go to the gates of Bordeaux, trumpeter.
Summon their general unto the wall.

Trumpet sounds. Enter  General and others aloft.

English John Talbot, captains, calls you forth,
Servant in arms to Harry King of England;
And thus he would: Open your city gates,
Be humble to us, call my sovereign yours,
And do him homage as obedient subjects,
And I’ll withdraw me and my bloody power.
But if you frown upon this proffer’d peace,
You tempt the fury of my three attendants,
Lean Famine, quartering Steel, and climbing Fire,
Who in a moment even with the earth
Shall lay your stately and air-braving towers,
If you forsake the offer of their love.

GENERAL.
Thou ominous and fearful owl of death,
Our nation’s terror and their bloody scourge!
The period of thy tyranny approacheth.
On us thou canst not enter but by death;
For, I protest, we are well fortified
And strong enough to issue out and fight.
If thou retire, the Dauphin, well appointed,
Stands with the snares of war to tangle thee.
On either hand thee there are squadrons pitch’d
To wall thee from the liberty of flight;
And no way canst thou turn thee for redress
But Death doth front thee with apparent spoil,
And pale Destruction meets thee in the face.
Ten thousand French have ta’en the sacrament
To rive their dangerous artillery
Upon no Christian soul but English Talbot.
Lo, there thou stand’st, a breathing valiant man
Of an invincible unconquer’d spirit.
This is the latest glory of thy praise
That I, thy enemy, due thee withal;
For ere the glass, that now begins to run,
Finish the process of his sandy hour,
These eyes, that see thee now well coloured,
Shall see thee wither’d, bloody, pale, and dead.

[_Drum afar off._]

Hark, hark, the Dauphin’s drum, a warning bell,
Sings heavy music to thy timorous soul,
And mine shall ring thy dire departure out.

[_Exeunt General, etc._]

TALBOT.
He fables not; I hear the enemy.
Out, some light horsemen, and peruse their wings.
O, negligent and heedless discipline!
How are we park’d and bounded in a pale,
A little herd of England’s timorous deer,
Mazed with a yelping kennel of French curs!
If we be English deer, be then in blood;
Not rascal-like, to fall down with a pinch,
But rather, moody-mad and desperate stags,
Turn on the bloody hounds with heads of steel
And make the cowards stand aloof at bay.
Sell every man his life as dear as mine,
And they shall find dear deer of us, my friends.
God and Saint George, Talbot and England’s right,
Prosper our colours in this dangerous fight!

[_Exeunt._]

SCENE III. Plains in Gascony.

Enter a Messenger that meets York. Enter York with trumpet and many
soldiers

YORK.
Are not the speedy scouts return’d again
That dogg’d the mighty army of the Dauphin?

MESSENGER.
They are return’d, my lord, and give it out
That he is march’d to Bordeaux with his power,
To fight with Talbot. As he march’d along,
By your espials were discovered
Two mightier troops than that the Dauphin led,
Which join’d with him and made their march for Bordeaux.

[_Exit._]

YORK.
A plague upon that villain Somerset
That thus delays my promised supply
Of horsemen that were levied for this siege!
Renowned Talbot doth expect my aid,
And I am louted by a traitor villain
And cannot help the noble chevalier.
God comfort him in this necessity!
If he miscarry, farewell wars in France.

Enter Sir William Lucy.

LUCY.
Thou princely leader of our English strength,
Never so needful on the earth of France,
Spur to the rescue of the noble Talbot,
Who now is girdled with a waist of iron,
And hemm’d about with grim destruction.
To Bordeaux, warlike Duke! To Bordeaux, York!
Else farewell, Talbot, France, and England’s honour.

YORK.
O God, that Somerset, who in proud heart
Doth stop my cornets, were in Talbot’s place!
So should we save a valiant gentleman
By forfeiting a traitor and a coward.
Mad ire and wrathful fury makes me weep
That thus we die while remiss traitors sleep.

LUCY.
O, send some succour to the distress’d lord!

YORK.
He dies, we lose; I break my warlike word;
We mourn, France smiles; we lose, they daily get,
All long of this vile traitor Somerset.

LUCY.
Then God take mercy on brave Talbot’s soul,
And on his son young John, who two hours since
I met in travel toward his warlike father.
This seven years did not Talbot see his son;
And now they meet where both their lives are done.

YORK.
Alas, what joy shall noble Talbot have
To bid his young son welcome to his grave?
Away! Vexation almost stops my breath,
That sunder’d friends greet in the hour of death.
Lucy, farewell. No more my fortune can
But curse the cause I cannot aid the man.
Maine, Blois, Poictiers, and Tours, are won away,
Long all of Somerset and his delay.

[_Exit, with his soldiers._]

LUCY.
Thus, while the vulture of sedition
Feeds in the bosom of such great commanders,
Sleeping neglection doth betray to loss
The conquest of our scarce-cold conqueror,
That ever-living man of memory,
Henry the Fifth. Whiles they each other cross,
Lives, honours, lands, and all hurry to loss.

[_Exit._]

SCENE IV. Other plains in Gascony.

Enter Somerset with his army; a Captain of Talbot’s with him.

SOMERSET.
It is too late; I cannot send them now.
This expedition was by York and Talbot
Too rashly plotted. All our general force
Might with a sally of the very town
Be buckled with. The over-daring Talbot
Hath sullied all his gloss of former honour
By this unheedful, desperate, wild adventure.
York set him on to fight and die in shame
That, Talbot dead, great York might bear the name.

CAPTAIN.
Here is Sir William Lucy, who with me
Set from our o’er-match’d forces forth for aid.

Enter Sir William Lucy.

SOMERSET.
How now, Sir William, whither were you sent?

LUCY.
Whither, my lord? From bought and sold Lord Talbot,
Who, ring’d about with bold adversity,
Cries out for noble York and Somerset
To beat assailing Death from his weak legions;
And whiles the honourable captain there
Drops bloody sweat from his war-wearied limbs,
And, in advantage lingering, looks for rescue,
You, his false hopes, the trust of England’s honour,
Keep off aloof with worthless emulation.
Let not your private discord keep away
The levied succours that should lend him aid,
While he, renowned noble gentleman,
Yield up his life unto a world of odds.
Orleans the Bastard, Charles, Burgundy,
Alençon, Reignier, compass him about,
And Talbot perisheth by your default.

SOMERSET.
York set him on; York should have sent him aid.

LUCY.
And York as fast upon your Grace exclaims,
Swearing that you withhold his levied host
Collected for this expedition.

SOMERSET.
York lies; he might have sent and had the horse.
I owe him little duty, and less love,
And take foul scorn to fawn on him by sending.

LUCY.
The fraud of England, not the force of France,
Hath now entrapp’d the noble-minded Talbot.
Never to England shall he bear his life,
But dies betray’d to fortune by your strife.

SOMERSET.
Come, go; I will dispatch the horsemen straight.
Within six hours they will be at his aid.

LUCY.
Too late comes rescue; he is ta’en or slain,
For fly he could not if he would have fled;
And fly would Talbot never, though he might.

SOMERSET.
If he be dead, brave Talbot, then adieu!

LUCY.
His fame lives in the world, his shame in you.

[_Exeunt._]

SCENE V. The English camp near Bordeaux.

Enter Talbot and John his son.

TALBOT.
O young John Talbot, I did send for thee
To tutor thee in stratagems of war,
That Talbot’s name might be in thee revived
When sapless age and weak unable limbs
Should bring thy father to his drooping chair.
But—O malignant and ill-boding stars!—
Now thou art come unto a feast of death,
A terrible and unavoided danger.
Therefore, dear boy, mount on my swiftest horse,
And I’ll direct thee how thou shalt escape
By sudden flight. Come, dally not, be gone.

JOHN TALBOT.
Is my name Talbot? And am I your son?
And shall I fly? O, if you love my mother,
Dishonour not her honourable name,
To make a bastard and a slave of me!
The world will say, he is not Talbot’s blood,
That basely fled when noble Talbot stood.

TALBOT.
Fly, to revenge my death if I be slain.

JOHN TALBOT.
He that flies so will ne’er return again.

TALBOT.
If we both stay, we both are sure to die.

JOHN TALBOT.
Then let me stay and, father, do you fly.
Your loss is great, so your regard should be;
My worth unknown, no loss is known in me.
Upon my death the French can little boast;
In yours they will, in you all hopes are lost.
Flight cannot stain the honour you have won;
But mine it will, that no exploit have done.
You fled for vantage, everyone will swear;
But if I bow, they’ll say it was for fear.
There is no hope that ever I will stay
If the first hour I shrink and run away.
Here on my knee I beg mortality,
Rather than life preserved with infamy.

TALBOT.
Shall all thy mother’s hopes lie in one tomb?

JOHN TALBOT.
Ay, rather than I’ll shame my mother’s womb.

TALBOT.
Upon my blessing, I command thee go.

JOHN TALBOT.
To fight I will, but not to fly the foe.

TALBOT.
Part of thy father may be saved in thee.

JOHN TALBOT.
No part of him but will be shame in me.

TALBOT.
Thou never hadst renown, nor canst not lose it.

JOHN TALBOT.
Yes, your renowned name; shall flight abuse it?

TALBOT.
Thy father’s charge shall clear thee from that stain.

JOHN TALBOT.
You cannot witness for me, being slain.
If death be so apparent, then both fly.

TALBOT.
And leave my followers here to fight and die?
My age was never tainted with such shame.

JOHN TALBOT.
And shall my youth be guilty of such blame?
No more can I be sever’d from your side
Than can yourself yourself in twain divide.
Stay, go, do what you will, the like do I;
For live I will not, if my father die.

TALBOT.
Then here I take my leave of thee, fair son,
Born to eclipse thy life this afternoon.
Come, side by side together live and die,
And soul with soul from France to heaven fly.

[_Exeunt._]

SCENE VI. A field of battle.

Alarum. Excursions, wherein Talbot’s son is hemmed about, and Talbot
rescues him.

TALBOT.
Saint George and victory! Fight, soldiers, fight!
The Regent hath with Talbot broke his word,
And left us to the rage of France his sword.
Where is John Talbot? Pause, and take thy breath;
I gave thee life and rescued thee from death.

JOHN TALBOT.
O, twice my father, twice am I thy son!
The life thou gav’st me first was lost and done,
Till with thy warlike sword, despite of fate,
To my determined time thou gav’st new date.

TALBOT.
When from the Dauphin’s crest thy sword struck fire,
It warm’d thy father’s heart with proud desire
Of bold-faced victory. Then leaden age,
Quicken’d with youthful spleen and warlike rage,
Beat down Alençon, Orleans, Burgundy,
And from the pride of Gallia rescued thee.
The ireful Bastard Orleans, that drew blood
From thee, my boy, and had the maidenhood
Of thy first fight, I soon encountered,
And, interchanging blows, I quickly shed
Some of his bastard blood, and in disgrace
Bespoke him thus: “Contaminated, base,
And misbegotten blood I spill of thine,
Mean and right poor, for that pure blood of mine,
Which thou didst force from Talbot, my brave boy.”
Here, purposing the Bastard to destroy,
Came in strong rescue. Speak, thy father’s care,
Art thou not weary, John? How dost thou fare?
Wilt thou yet leave the battle, boy, and fly,
Now thou art seal’d the son of chivalry?
Fly, to revenge my death when I am dead;
The help of one stands me in little stead.
O, too much folly is it, well I wot,
To hazard all our lives in one small boat!
If I today die not with Frenchmen’s rage,
Tomorrow I shall die with mickle age.
By me they nothing gain an if I stay;
’Tis but the short’ning of my life one day.
In thee thy mother dies, our household’s name,
My death’s revenge, thy youth, and England’s fame.
All these and more we hazard by thy stay;
All these are saved if thou wilt fly away.

JOHN TALBOT.
The sword of Orleans hath not made me smart;
These words of yours draw life-blood from my heart.
On that advantage, bought with such a shame,
To save a paltry life and slay bright fame,
Before young Talbot from old Talbot fly,
The coward horse that bears me fall and die!
And like me to the peasant boys of France,
To be shame’s scorn and subject of mischance!
Surely, by all the glory you have won,
An if I fly, I am not Talbot’s son.
Then talk no more of flight, it is no boot;
If son to Talbot, die at Talbot’s foot.

TALBOT.
Then follow thou thy desperate sire of Crete,
Thou Icarus; thy life to me is sweet.
If thou wilt fight, fight by thy father’s side,
And, commendable proved, let’s die in pride.

[_Exeunt._]

SCENE VII. Another part of the field.

Alarum. Excursions. Enter old Talbot led by a Servant.

TALBOT.
Where is my other life? Mine own is gone.
O, where’s young Talbot? Where is valiant John?
Triumphant Death, smear’d with captivity,
Young Talbot’s valour makes me smile at thee.
When he perceived me shrink and on my knee,
His bloody sword he brandish’d over me,
And like a hungry lion did commence
Rough deeds of rage and stern impatience;
But when my angry guardant stood alone,
Tendering my ruin and assail’d of none,
Dizzy-ey’d fury and great rage of heart
Suddenly made him from my side to start
Into the clustering battle of the French;
And in that sea of blood my boy did drench
His over-mounting spirit; and there died
My Icarus, my blossom, in his pride.

SERVANT.
O my dear lord, lo where your son is borne!

Enter soldiers, with the body of young Talbot.

TALBOT.
Thou antic Death, which laugh’st us here to scorn,
Anon, from thy insulting tyranny,
Coupled in bonds of perpetuity,
Two Talbots, winged through the lither sky,
In thy despite shall scape mortality.
O thou whose wounds become hard-favour’d Death,
Speak to thy father ere thou yield thy breath!
Brave Death by speaking, whether he will or no;
Imagine him a Frenchman and thy foe.
Poor boy, he smiles, methinks, as who should say,
Had Death been French, then Death had died today.
Come, come, and lay him in his father’s arms;
My spirit can no longer bear these harms.
Soldiers, adieu! I have what I would have,
Now my old arms are young John Talbot’s grave.

[_Dies._]

Enter Charles, Alençon, Burgundy, Bastard, La Pucelle and forces.

CHARLES.
Had York and Somerset brought rescue in,
We should have found a bloody day of this.

BASTARD.
How the young whelp of Talbot’s, raging-wood,
Did flesh his puny sword in Frenchmen’s blood!

PUCELLE.
Once I encounter’d him, and thus I said:
“Thou maiden youth, be vanquish’d by a maid.”
But with a proud majestical high scorn
He answer’d thus: “Young Talbot was not born
To be the pillage of a giglot wench.”
So, rushing in the bowels of the French,
He left me proudly, as unworthy fight.

BURGUNDY.
Doubtless he would have made a noble knight.
See, where he lies inhearsed in the arms
Of the most bloody nurser of his harms.

BASTARD.
Hew them to pieces, hack their bones asunder,
Whose life was England’s glory, Gallia’s wonder.

CHARLES.
O, no, forbear! For that which we have fled
During the life, let us not wrong it dead.

Enter Sir William Lucy and a French Herald.

LUCY.
Herald, conduct me to the Dauphin’s tent,
To know who hath obtain’d the glory of the day.

CHARLES.
On what submissive message art thou sent?

LUCY.
Submission, Dauphin! ’Tis a mere French word.
We English warriors wot not what it means.
I come to know what prisoners thou hast ta’en,
And to survey the bodies of the dead.

CHARLES.
For prisoners ask’st thou? Hell our prison is.
But tell me whom thou seek’st.

LUCY.
But where’s the great Alcides of the field,
Valiant Lord Talbot, Earl of Shrewsbury,
Created for his rare success in arms
Great Earl of Washford, Waterford, and Valence,
Lord Talbot of Goodrig and Urchinfield,
Lord Strange of Blackmere, Lord Verdun of Alton,
Lord Cromwell of Wingfield, Lord Furnival of Sheffield,
The thrice victorious Lord of Falconbridge,
Knight of the noble Order of Saint George,
Worthy Saint Michael, and the Golden Fleece,
Great Marshal to Henry the Sixth
Of all his wars within the realm of France?

PUCELLE.
Here’s a silly stately style indeed!
The Turk, that two and fifty kingdoms hath,
Writes not so tedious a style as this.
Him that thou magnifiest with all these titles
Stinking and fly-blown lies here at our feet.

LUCY.
Is Talbot slain, the Frenchman’s only scourge,
Your kingdom’s terror and black Nemesis?
O, were mine eye-balls into bullets turn’d,
That I in rage might shoot them at your faces!
O, that I could but call these dead to life!
It were enough to fright the realm of France.
Were but his picture left amongst you here,
It would amaze the proudest of you all.
Give me their bodies, that I may bear them hence
And give them burial as beseems their worth.

PUCELLE.
I think this upstart is old Talbot’s ghost,
He speaks with such a proud commanding spirit.
For God’s sake, let him have them; to keep them here,
They would but stink and putrify the air.

CHARLES.
Go, take their bodies hence.

LUCY.
I’ll bear them hence;
But from their ashes shall be rear’d
A phoenix that shall make all France afeard.

CHARLES.
So we be rid of them, do with them what thou wilt.
And now to Paris in this conquering vein.
All will be ours, now bloody Talbot’s slain.

[_Exeunt._]




ACT V

SCENE I. London. The Palace.


Sennet. Enter King, Gloucester and Exeter.

KING HENRY.
Have you perused the letters from the Pope,
The Emperor, and the Earl of Armagnac?

GLOUCESTER.
I have, my lord; and their intent is this:
They humbly sue unto your Excellence
To have a godly peace concluded of
Between the realms of England and of France.

KING HENRY.
How doth your Grace affect their motion?

GLOUCESTER.
Well, my good lord, and as the only means
To stop effusion of our Christian blood
And stablish quietness on every side.

KING HENRY.
Ay, marry, uncle, for I always thought
It was both impious and unnatural
That such immanity and bloody strife
Should reign among professors of one faith.

GLOUCESTER.
Beside, my lord, the sooner to effect
And surer bind this knot of amity,
The Earl of Armagnac, near knit to Charles,
A man of great authority in France,
Proffers his only daughter to your Grace
In marriage, with a large and sumptuous dowry.

KING HENRY.
Marriage, uncle! Alas, my years are young!
And fitter is my study and my books
Than wanton dalliance with a paramour.
Yet call th’ ambassadors; and, as you please,
So let them have their answers every one.
I shall be well content with any choice
Tends to God’s glory and my country’s weal.

Enter Winchester in Cardinal’s habit, a  Legate and two Ambassadors.

EXETER.
What, is my Lord of Winchester install’d
And call’d unto a cardinal’s degree?
Then I perceive that will be verified
Henry the Fifth did sometime prophesy:
“If once he come to be a cardinal,
He’ll make his cap co-equal with the crown.”

KING HENRY.
My Lords Ambassadors, your several suits
Have been consider’d and debated on.
Your purpose is both good and reasonable;
And therefore are we certainly resolved
To draw conditions of a friendly peace,
Which by my Lord of Winchester we mean
Shall be transported presently to France.

GLOUCESTER.
And for the proffer of my lord your master,
I have inform’d his Highness so at large,
As liking of the lady’s virtuous gifts,
Her beauty and the value of her dower,
He doth intend she shall be England’s Queen.

KING HENRY.
In argument and proof of which contract,
Bear her this jewel, pledge of my affection.
And so, my Lord Protector, see them guarded
And safely brought to Dover; where inshipp’d,
Commit them to the fortune of the sea.

[_Exeunt all but Winchester and Legate._]

WINCHESTER.
Stay my lord legate; you shall first receive
The sum of money which I promised
Should be deliver’d to his Holiness
For clothing me in these grave ornaments.

LEGATE.
I will attend upon your lordship’s leisure.

WINCHESTER.
[_Aside_.] Now Winchester will not submit, I trow,
Or be inferior to the proudest peer.
Humphrey of Gloucester, thou shalt well perceive
That neither in birth or for authority,
The Bishop will be overborne by thee.
I’ll either make thee stoop and bend thy knee,
Or sack this country with a mutiny.

[_Exeunt._]

SCENE II. France. Plains in Anjou.

Enter Charles, Burgundy, Alençon, Bastard, Reignier, La Pucelle and
forces.

CHARLES.
These news, my lords, may cheer our drooping spirits:
’Tis said the stout Parisians do revolt
And turn again unto the warlike French.

ALENÇON.
Then march to Paris, royal Charles of France,
And keep not back your powers in dalliance.

PUCELLE.
Peace be amongst them, if they turn to us;
Else ruin combat with their palaces!

Enter Scout.

SCOUT.
Success unto our valiant general,
And happiness to his accomplices!

CHARLES.
What tidings send our scouts? I prithee, speak.

SCOUT.
The English army, that divided was
Into two parties, is now conjoin’d in one,
And means to give you battle presently.

CHARLES.
Somewhat too sudden, sirs, the warning is;
But we will presently provide for them.

BURGUNDY.
I trust the ghost of Talbot is not there.
Now he is gone, my lord, you need not fear.

PUCELLE.
Of all base passions, fear is most accursed.
Command the conquest, Charles, it shall be thine;
Let Henry fret and all the world repine.

CHARLES.
Then on, my lords; and France be fortunate!

[_Exeunt._]

SCENE III. Before Angiers.

Alarum. Excursions. Enter La Pucelle.

PUCELLE.
The Regent conquers, and the Frenchmen fly.
Now help, ye charming spells and periapts;
And ye choice spirits that admonish me,
And give me signs of future accidents. [_Thunder_]
You speedy helpers, that are substitutes
Under the lordly monarch of the north,
Appear and aid me in this enterprise.

Enter Fiends.

This speed and quick appearance argues proof
Of your accustom’d diligence to me.
Now, ye familiar spirits that are cull’d
Out of the powerful regions under earth,
Help me this once, that France may get the field.

[_They walk and speak not._]

O, hold me not with silence over-long!
Where I was wont to feed you with my blood,
I’ll lop a member off and give it you
In earnest of a further benefit,
So you do condescend to help me now.

[_They hang their heads._]

No hope to have redress? My body shall
Pay recompense if you will grant my suit.

[_They shake their heads._]

Cannot my body nor blood-sacrifice
Entreat you to your wonted furtherance?
Then take my soul; my body, soul and all,
Before that England give the French the foil.

[_They depart._]

See, they forsake me. Now the time is come
That France must vail her lofty-plumed crest
And let her head fall into England’s lap.
My ancient incantations are too weak,
And hell too strong for me to buckle with.
Now, France, thy glory droopeth to the dust.

[_Exit._]

Excursions. Burgundy and York fight hand to hand. The French fly.  La
Pucelle is taken.

YORK.
Damsel of France, I think I have you fast.
Unchain your spirits now with spelling charms,
And try if they can gain your liberty.
A goodly prize, fit for the devil’s grace!
See, how the ugly witch doth bend her brows,
As if with Circe she would change my shape!

PUCELLE.
Chang’d to a worser shape thou canst not be.

YORK.
O, Charles the Dauphin is a proper man;
No shape but his can please your dainty eye.

PUCELLE.
A plaguing mischief light on Charles and thee!
And may ye both be suddenly surprised
By bloody hands, in sleeping on your beds!

YORK.
Fell banning hag, enchantress, hold thy tongue!

PUCELLE.
I prithee, give me leave to curse awhile.

YORK.
Curse, miscreant, when thou com’st to the stake.

[_Exeunt._]

Alarum. Enter Suffolk with Margaret in his hand.

SUFFOLK.
Be what thou wilt, thou art my prisoner.

[_Gazes on her._]

O fairest beauty, do not fear nor fly!
For I will touch thee but with reverent hands,
I kiss these fingers for eternal peace,
And lay them gently on thy tender side.
Who art thou? Say, that I may honour thee.

MARGARET.
Margaret my name, and daughter to a king,
The King of Naples, whosoe’er thou art.

SUFFOLK.
An earl I am, and Suffolk am I call’d.
Be not offended, nature’s miracle,
Thou art allotted to be ta’en by me.
So doth the swan her downy cygnets save,
Keeping them prisoner underneath her wings.
Yet, if this servile usage once offend,
Go and be free again as Suffolk’s friend.

[_She is going._]

O, stay! I have no power to let her pass;
My hand would free her, but my heart says no.
As plays the sun upon the glassy streams,
Twinkling another counterfeited beam,
So seems this gorgeous beauty to mine eyes.
Fain would I woo her, yet I dare not speak.
I’ll call for pen and ink, and write my mind.
Fie, de la Pole, disable not thyself;
Hast not a tongue? Is she not here?
Wilt thou be daunted at a woman’s sight?
Ay, beauty’s princely majesty is such
Confounds the tongue and makes the senses rough.

MARGARET.
Say, Earl of Suffolk, if thy name be so,
What ransom must I pay before I pass?
For I perceive I am thy prisoner.

SUFFOLK.
How canst thou tell she will deny thy suit,
Before thou make a trial of her love?

MARGARET.
Why speak’st thou not? What ransom must I pay?

SUFFOLK.
She’s beautiful, and therefore to be woo’d;
She is a woman, therefore to be won.

MARGARET.
Wilt thou accept of ransom, yea, or no?

SUFFOLK.
Fond man, remember that thou hast a wife;
Then how can Margaret be thy paramour?

MARGARET.
I were best leave him, for he will not hear.

SUFFOLK.
There all is marr’d; there lies a cooling card.

MARGARET.
He talks at random; sure, the man is mad.

SUFFOLK.
And yet a dispensation may be had.

MARGARET.
And yet I would that you would answer me.

SUFFOLK.
I’ll win this Lady Margaret. For whom?
Why, for my king. Tush, that’s a wooden thing!

MARGARET.
He talks of wood. It is some carpenter.

SUFFOLK.
Yet so my fancy may be satisfied,
And peace established between these realms.
But there remains a scruple in that too;
For though her father be the King of Naples,
Duke of Anjou and Maine, yet is he poor,
And our nobility will scorn the match.

MARGARET.
Hear ye, captain, are you not at leisure?

SUFFOLK.
It shall be so, disdain they ne’er so much.
Henry is youthful and will quickly yield.
Madam, I have a secret to reveal.

MARGARET.
What though I be enthrall’d? He seems a knight,
And will not any way dishonour me.

SUFFOLK.
Lady, vouchsafe to listen what I say.

MARGARET.
Perhaps I shall be rescued by the French;
And then I need not crave his courtesy.

SUFFOLK.
Sweet madam, give me hearing in a cause.

MARGARET.
Tush, women have been captivate ere now.

SUFFOLK.
Lady, wherefore talk you so?

MARGARET.
I cry you mercy, ’tis but _quid_ for _quo_.

SUFFOLK.
Say, gentle princess, would you not suppose
Your bondage happy, to be made a queen?

MARGARET.
To be a queen in bondage is more vile
Than is a slave in base servility;
For princes should be free.

SUFFOLK.
And so shall you,
If happy England’s royal king be free.

MARGARET.
Why, what concerns his freedom unto me?

SUFFOLK.
I’ll undertake to make thee Henry’s queen,
To put a golden scepter in thy hand
And set a precious crown upon thy head,
If thou wilt condescend to be my—

MARGARET.
What?

SUFFOLK.
His love.

MARGARET.
I am unworthy to be Henry’s wife.

SUFFOLK.
No, gentle madam, I unworthy am
To woo so fair a dame to be his wife,
And have no portion in the choice myself.
How say you, madam, are ye so content?

MARGARET.
An if my father please, I am content.

SUFFOLK.
Then call our captains and our colours forth.
And, madam, at your father’s castle walls
We’ll crave a parley, to confer with him.

A parley sounded. Enter Reignier on the walls.

See, Reignier, see, thy daughter prisoner!

REIGNIER.
To whom?

SUFFOLK.
To me.

REIGNIER.
Suffolk, what remedy?
I am a soldier, and unapt to weep
Or to exclaim on fortune’s fickleness.

SUFFOLK.
Yes, there is remedy enough, my lord:
Consent, and for thy honour give consent,
Thy daughter shall be wedded to my king,
Whom I with pain have woo’d and won thereto;
And this her easy-held imprisonment
Hath gain’d thy daughter princely liberty.

REIGNIER.
Speaks Suffolk as he thinks?

SUFFOLK.
Fair Margaret knows
That Suffolk doth not flatter, face, or feign.

REIGNIER.
Upon thy princely warrant, I descend
To give thee answer of thy just demand.

[_Exit from the walls._]

SUFFOLK.
And here I will expect thy coming.

Trumpets sound. Enter Reignier, below.

REIGNIER.
Welcome, brave earl, into our territories.
Command in Anjou what your honour pleases.

SUFFOLK.
Thanks, Reignier, happy for so sweet a child,
Fit to be made companion with a king.
What answer makes your Grace unto my suit?

REIGNIER.
Since thou dost deign to woo her little worth
To be the princely bride of such a lord,
Upon condition I may quietly
Enjoy mine own, the country Maine and Anjou,
Free from oppression or the stroke of war,
My daughter shall be Henry’s, if he please.

SUFFOLK.
That is her ransom; I deliver her;
And those two counties I will undertake
Your Grace shall well and quietly enjoy.

REIGNIER.
And I again, in Henry’s royal name,
As deputy unto that gracious king,
Give thee her hand for sign of plighted faith.

SUFFOLK.
Reignier of France, I give thee kingly thanks,
Because this is in traffic of a king.
[_Aside_.] And yet, methinks, I could be well content
To be mine own attorney in this case.
I’ll over then to England with this news,
And make this marriage to be solemnized.
So, farewell, Reignier; set this diamond safe
In golden palaces, as it becomes.

REIGNIER.
I do embrace thee as I would embrace
The Christian prince, King Henry, were he here.

MARGARET.
Farewell, my lord; good wishes, praise, and prayers
Shall Suffolk ever have of Margaret. [_Going_].

SUFFOLK.
Farewell, sweet madam; but hark you, Margaret,
No princely commendations to my king?

MARGARET.
Such commendations as becomes a maid,
A virgin and his servant, say to him.

SUFFOLK.
Words sweetly placed and modestly directed.
But, madam, I must trouble you again:
No loving token to his Majesty?

MARGARET.
Yes, my good lord; a pure unspotted heart,
Never yet taint with love, I send the King.

SUFFOLK.
And this withal. [_Kisses her_.]

MARGARET.
That for thyself. I will not so presume
To send such peevish tokens to a king.

[_Exeunt Reignier and Margaret._]

SUFFOLK.
O, wert thou for myself! But, Suffolk, stay;
Thou mayst not wander in that labyrinth.
There Minotaurs and ugly treasons lurk.
Solicit Henry with her wondrous praise.
Bethink thee on her virtues that surmount,
And natural graces that extinguish art;
Repeat their semblance often on the seas,
That, when thou com’st to kneel at Henry’s feet,
Thou mayst bereave him of his wits with wonder.

[_Exit._]

SCENE IV. Camp of the Duke of York in Anjou.

Enter York, Warwick and others.

YORK.
Bring forth that sorceress condemn’d to burn.

Enter La Pucelle, guarded, and a Shepherd.

SHEPHERD.
Ah, Joan, this kills thy father’s heart outright!
Have I sought every country far and near,
And, now it is my chance to find thee out,
Must I behold thy timeless cruel death?
Ah, Joan, sweet daughter Joan, I’ll die with thee!

PUCELLE.
Decrepit miser, base ignoble wretch!
I am descended of a gentler blood.
Thou art no father nor no friend of mine.

SHEPHERD.
Out, out! My lords, as please you, ’tis not so;
I did beget her, all the parish knows.
Her mother liveth yet, can testify
She was the first fruit of my bachelorship.

WARWICK.
Graceless, wilt thou deny thy parentage?

YORK.
This argues what her kind of life hath been,
Wicked and vile; and so her death concludes.

SHEPHERD.
Fie, Joan, that thou wilt be so obstacle!
God knows thou art a collop of my flesh;
And for thy sake have I shed many a tear.
Deny me not, I prithee, gentle Joan.

PUCELLE.
Peasant, avaunt! You have suborn’d this man
Of purpose to obscure my noble birth.

SHEPHERD.
’Tis true, I gave a noble to the priest
The morn that I was wedded to her mother.
Kneel down and take my blessing, good my girl.
Wilt thou not stoop? Now cursed be the time
Of thy nativity! I would the milk
Thy mother gave thee when thou suck’dst her breast
Had been a little ratsbane for thy sake!
Or else, when thou didst keep my lambs a-field,
I wish some ravenous wolf had eaten thee!
Dost thou deny thy father, cursed drab?
O, burn her, burn her! Hanging is too good.

[_Exit._]

YORK.
Take her away, for she hath lived too long,
To fill the world with vicious qualities.

PUCELLE.
First, let me tell you whom you have condemn’d:
Not one begotten of a shepherd swain,
But issued from the progeny of kings;
Virtuous and holy, chosen from above,
By inspiration of celestial grace,
To work exceeding miracles on earth.
I never had to do with wicked spirits.
But you, that are polluted with your lusts,
Stain’d with the guiltless blood of innocents,
Corrupt and tainted with a thousand vices,
Because you want the grace that others have,
You judge it straight a thing impossible
To compass wonders but by help of devils.
No, misconceived! Joan of Arc hath been
A virgin from her tender infancy,
Chaste and immaculate in very thought;
Whose maiden blood, thus rigorously effused,
Will cry for vengeance at the gates of heaven.

YORK.
Ay, ay; away with her to execution!

WARWICK.
And hark ye, sirs; because she is a maid,
Spare for no faggots, let there be enow.
Place barrels of pitch upon the fatal stake,
That so her torture may be shortened.

PUCELLE.
Will nothing turn your unrelenting hearts?
Then, Joan, discover thine infirmity,
That warranteth by law to be thy privilege:
I am with child, ye bloody homicides.
Murder not then the fruit within my womb,
Although ye hale me to a violent death.

YORK.
Now heaven forfend! The holy maid with child?

WARWICK.
The greatest miracle that e’er ye wrought!
Is all your strict preciseness come to this?

YORK.
She and the Dauphin have been juggling.
I did imagine what would be her refuge.

WARWICK.
Well, go to; we’ll have no bastards live,
Especially since Charles must father it.

PUCELLE.
You are deceived; my child is none of his.
It was Alençon that enjoy’d my love.

YORK.
Alençon, that notorious Machiavel!
It dies and if it had a thousand lives.

PUCELLE.
O, give me leave, I have deluded you.
’Twas neither Charles nor yet the Duke I named,
But Reignier, King of Naples, that prevail’d.

WARWICK.
A married man! That’s most intolerable.

YORK.
Why, here’s a girl! I think she knows not well—
There were so many—whom she may accuse.

WARWICK.
It’s sign she hath been liberal and free.

YORK.
And yet, forsooth, she is a virgin pure!
Strumpet, thy words condemn thy brat and thee.
Use no entreaty, for it is in vain.

PUCELLE.
Then lead me hence, with whom I leave my curse:
May never glorious sun reflex his beams
Upon the country where you make abode;
But darkness and the gloomy shade of death
Environ you, till mischief and despair
Drive you to break your necks or hang yourselves!

[_Exit, guarded._]

YORK.
Break thou in pieces and consume to ashes,
Thou foul accursed minister of hell!

Enter Bishop of Winchester as Cardinal, attended.

WINCHESTER.
Lord Regent, I do greet your Excellence
With letters of commission from the King.
For know, my lords, the states of Christendom,
Moved with remorse of these outrageous broils,
Have earnestly implored a general peace
Betwixt our nation and the aspiring French;
And here at hand the Dauphin and his train
Approacheth to confer about some matter.

YORK.
Is all our travail turn’d to this effect?
After the slaughter of so many peers,
So many captains, gentlemen and soldiers,
That in this quarrel have been overthrown
And sold their bodies for their country’s benefit,
Shall we at last conclude effeminate peace?
Have we not lost most part of all the towns,
By treason, falsehood, and by treachery,
Our great progenitors had conquered?
O, Warwick, Warwick! I foresee with grief
The utter loss of all the realm of France.

WARWICK.
Be patient, York; if we conclude a peace,
It shall be with such strict and severe covenants
As little shall the Frenchmen gain thereby.

Enter Charles, Alençon, Bastard, Reignier and others.

CHARLES.
Since, lords of England, it is thus agreed
That peaceful truce shall be proclaim’d in France,
We come to be informed by yourselves
What the conditions of that league must be.

YORK.
Speak, Winchester, for boiling choler chokes
The hollow passage of my poison’d voice
By sight of these our baleful enemies.

WINCHESTER.
Charles, and the rest, it is enacted thus:
That, in regard King Henry gives consent,
Of mere compassion and of lenity,
To ease your country of distressful war,
And suffer you to breathe in fruitful peace,
You shall become true liegemen to his crown.
And, Charles, upon condition thou wilt swear
To pay him tribute and submit thyself,
Thou shalt be placed as viceroy under him,
And still enjoy the regal dignity.

ALENÇON.
Must he be then as shadow of himself?
Adorn his temples with a coronet,
And yet, in substance and authority,
Retain but privilege of a private man?
This proffer is absurd and reasonless.

CHARLES.
’Tis known already that I am possess’d
With more than half the Gallian territories,
And therein reverenced for their lawful king.
Shall I, for lucre of the rest unvanquish’d,
Detract so much from that prerogative
As to be call’d but viceroy of the whole?
No, lord ambassador, I’ll rather keep
That which I have than, coveting for more,
Be cast from possibility of all.

YORK.
Insulting Charles! Hast thou by secret means
Used intercession to obtain a league,
And, now the matter grows to compromise,
Stand’st thou aloof upon comparison?
Either accept the title thou usurp’st,
Of benefit proceeding from our king
And not of any challenge of desert,
Or we will plague thee with incessant wars.

REIGNIER.
My lord, you do not well in obstinacy
To cavil in the course of this contract.
If once it be neglected, ten to one
We shall not find like opportunity.

ALENÇON.
To say the truth, it is your policy
To save your subjects from such massacre
And ruthless slaughters as are daily seen
By our proceeding in hostility;
And therefore take this compact of a truce,
Although you break it when your pleasure serves.

WARWICK.
How say’st thou, Charles? Shall our condition stand?

CHARLES.
It shall; only reserv’d you claim no interest
In any of our towns of garrison.

YORK.
Then swear allegiance to his Majesty,
As thou art knight, never to disobey
Nor be rebellious to the crown of England,
Thou, nor thy nobles, to the crown of England.

[_Charles and the rest give tokens of fealty._]

So, now dismiss your army when ye please;
Hang up your ensigns, let your drums be still,
For here we entertain a solemn peace.

[_Exeunt._]

SCENE V. London. The royal palace.

Enter Suffolk in conference with the King, Gloucester and Exeter.

KING HENRY.
Your wondrous rare description, noble earl,
Of beauteous Margaret hath astonish’d me.
Her virtues graced with external gifts
Do breed love’s settled passions in my heart,
And like as rigor of tempestuous gusts
Provokes the mightiest hulk against the tide,
So am I driven by breath of her renown
Either to suffer shipwreck or arrive
Where I may have fruition of her love.

SUFFOLK.
Tush, my good lord, this superficial tale
Is but a preface of her worthy praise;
The chief perfections of that lovely dame,
Had I sufficient skill to utter them,
Would make a volume of enticing lines,
Able to ravish any dull conceit;
And, which is more, she is not so divine,
So full replete with choice of all delights,
But with as humble lowliness of mind
She is content to be at your command;
Command, I mean, of virtuous chaste intents,
To love and honour Henry as her lord.

KING HENRY.
And otherwise will Henry ne’er presume.
Therefore, my Lord Protector, give consent
That Margaret may be England’s royal queen.

GLOUCESTER.
So should I give consent to flatter sin.
You know, my lord, your Highness is betroth’d
Unto another lady of esteem.
How shall we then dispense with that contract,
And not deface your honour with reproach?

SUFFOLK.
As doth a ruler with unlawful oaths;
Or one that, at a triumph having vow’d
To try his strength, forsaketh yet the lists
By reason of his adversary’s odds.
A poor earl’s daughter is unequal odds,
And therefore may be broke without offence.

GLOUCESTER.
Why, what, I pray, is Margaret more than that?
Her father is no better than an earl,
Although in glorious titles he excel.

SUFFOLK.
Yes, my lord, her father is a king,
The King of Naples and Jerusalem;
And of such great authority in France
As his alliance will confirm our peace,
And keep the Frenchmen in allegiance.

GLOUCESTER.
And so the Earl of Armagnac may do,
Because he is near kinsman unto Charles.

EXETER.
Beside, his wealth doth warrant a liberal dower,
Where Reignier sooner will receive than give.

SUFFOLK.
A dower, my lords? Disgrace not so your king,
That he should be so abject, base, and poor,
To choose for wealth and not for perfect love.
Henry is able to enrich his queen,
And not to seek a queen to make him rich;
So worthless peasants bargain for their wives,
As market-men for oxen, sheep, or horse.
Marriage is a matter of more worth
Than to be dealt in by attorneyship;
Not whom we will, but whom his Grace affects,
Must be companion of his nuptial bed.
And therefore, lords, since he affects her most,
Most of all these reasons bindeth us
In our opinions she should be preferr’d.
For what is wedlock forced but a hell,
An age of discord and continual strife?
Whereas the contrary bringeth bliss,
And is a pattern of celestial peace.
Whom should we match with Henry, being a king,
But Margaret, that is daughter to a king?
Her peerless feature, joined with her birth,
Approves her fit for none but for a king;
Her valiant courage and undaunted spirit,
More than in women commonly is seen,
Will answer our hope in issue of a king;
For Henry, son unto a conqueror,
Is likely to beget more conquerors,
If with a lady of so high resolve
As is fair Margaret he be link’d in love.
Then yield, my lords; and here conclude with me
That Margaret shall be queen, and none but she.

KING HENRY.
Whether it be through force of your report,
My noble Lord of Suffolk, or for that
My tender youth was never yet attaint
With any passion of inflaming love,
I cannot tell; but this I am assured,
I feel such sharp dissension in my breast,
Such fierce alarums both of hope and fear,
As I am sick with working of my thoughts.
Take therefore shipping; post, my lord, to France;
Agree to any covenants, and procure
That Lady Margaret do vouchsafe to come
To cross the seas to England and be crown’d
King Henry’s faithful and anointed queen.
For your expenses and sufficient charge,
Among the people gather up a tenth.
Be gone, I say; for till you do return,
I rest perplexed with a thousand cares.
And you, good uncle, banish all offence.
If you do censure me by what you were,
Not what you are, I know it will excuse
This sudden execution of my will.
And so conduct me where, from company,
I may revolve and ruminate my grief.

[_Exit._]

GLOUCESTER.
Ay, grief, I fear me, both at first and last.

[_Exeunt Gloucester and Exeter._]

SUFFOLK.
Thus Suffolk hath prevail’d; and thus he goes,
As did the youthful Paris once to Greece,
With hope to find the like event in love,
But prosper better than the Troyan did.
Margaret shall now be queen, and rule the King;
But I will rule both her, the King, and realm.

[_Exit._]




THE SECOND PART OF KING HENRY THE SIXTH

Contents

 ACT I
 Scene I. London. The palace
 Scene II. The Duke of Gloucester’s House
 Scene III. London. The palace
 Scene IV. Gloucester’s Garden

 ACT II
 SCENE I. Saint Albans
 SCENE II. London. The Duke of York’s Garden
 SCENE III. A Hall of Justice
 SCENE IV. A Street

 ACT III
 SCENE I. The Abbey at Bury St. Edmund’s
 SCENE II. Bury St. Edmund’s. A Room of State
 SCENE III. A Bedchamber

 ACT IV
 SCENE I. The Coast of Kent
 SCENE II. Blackheath
 SCENE III. Another part of Blackheath
 SCENE IV. London. The Palace
 SCENE V. London. The Tower
 SCENE VI. London. Cannon Street
 SCENE VII. London. Smithfield
 SCENE VIII. Southwark
 SCENE IX. Kenilworth Castle
 SCENE X. Kent. Iden’s Garden

 ACT V
 SCENE I. Fields between Dartford and Blackheath
 SCENE II. Saint Albans
 SCENE III. Fields near Saint Albans




Dramatis Personæ

KING HENRY THE SIXTH
MARGARET, Queen to King Henry
Humphrey, Duke of GLOUCESTER, his uncle
ELEANOR, Duchess of Gloucester
CARDINAL Beaufort, Bishop of Winchester, great-uncle to the King

DUKE OF SOMERSET
DUKE OF SUFFOLK
DUKE OF BUCKINGHAM
LORD CLIFFORD
YOUNG CLIFFORD, his son
VAUX

Richard Plantagenet, Duke of YORK
EDWARD and RICHARD, his sons
EARL OF SALISBURY
EARL OF WARWICK

THOMAS HORNER, an armourer
PETER THUMP, his man
JOHN HUME, a priest
JOHN SOUTHWELL, a priest
Margery JOURDAIN, a witch
ROGER BOLINGBROKE, a conjurer
SIMPCOX, an impostor
Wife to Simpcox
Mayor of Saint Albans
SIR JOHN STANLEY
Two Murderers
A LIEUTENANT
MASTER
Master’s-Mate
Walter WHITMORE
Two Gentlemen, prisoners with Suffolk

Jack CADE, a rebel
George BEVIS
John HOLLAND
DICK the butcher
SMITH the weaver
MICHAEL, etc., followers of Cade
CLERK of Chartham
SIR HUMPHREY STAFFORD
WILLIAM STAFFORD, his brother
LORD SCALES
LORD SAYE
MATTHEW GOUGH
Alexander IDEN, a Kentish gentleman

Lords, Ladies, and Attendants, Petitioners, Aldermen, a Herald, a
Beadle, Sheriff, and Officers, Citizens, Prentices, Falconers, Guards,
Soldiers, Messengers, &c.

A Spirit

SCENE: England.




ACT I

SCENE I. London. The palace


Flourish of trumpets, then hautboys. Enter the King, Gloucester,
Salisbury, Warwick, and Cardinal Beaufort on the one side; the Queen,
Suffolk, York, Somerset and Buckingham on the other.

SUFFOLK.
As by your high imperial Majesty
I had in charge at my depart for France,
As procurator to your excellence,
To marry Princess Margaret for your grace,
So, in the famous ancient city Tours,
In presence of the Kings of France and Sicil,
The Dukes of Orleans, Calaber, Bretagne, and Alençon,
Seven earls, twelve barons, and twenty reverend bishops,
I have performed my task and was espoused,
And humbly now upon my bended knee,
In sight of England and her lordly peers,
Deliver up my title in the Queen
To your most gracious hands, that are the substance
Of that great shadow I did represent:
The happiest gift that ever marquess gave,
The fairest queen that ever king received.

KING HENRY.
Suffolk, arise.—Welcome, Queen Margaret.
I can express no kinder sign of love
Than this kind kiss.—O Lord, that lends me life,
Lend me a heart replete with thankfulness!
For Thou hast given me in this beauteous face
A world of earthly blessings to my soul,
If sympathy of love unite our thoughts.

QUEEN MARGARET.
Great King of England and my gracious lord,
The mutual conference that my mind hath had
By day, by night, waking and in my dreams,
In courtly company or at my beads,
With you, mine alderliefest sovereign,
Makes me the bolder to salute my King
With ruder terms, such as my wit affords
And overjoy of heart doth minister.

KING HENRY.
Her sight did ravish, but her grace in speech,
Her words yclad with wisdom’s majesty,
Makes me from wondering fall to weeping joys,
Such is the fulness of my heart’s content.
Lords, with one cheerful voice welcome my love.

ALL.
[_Kneeling_.] Long live Queen Margaret, England’s happiness!

QUEEN MARGARET.
We thank you all.

[_Flourish._]

SUFFOLK.
My Lord Protector, so it please your grace,
Here are the articles of contracted peace
Between our sovereign and the French king Charles,
For eighteen months concluded by consent.

GLOUCESTER.
[_Reads_.] Imprimis, _it is agreed between the French king Charles and
William de la Pole, Marquess of Suffolk, ambassador for Henry, King of
England, that the said Henry shall espouse the Lady Margaret, daughter
unto Reignier King of Naples, Sicilia, and Jerusalem, and crown her
Queen of England ere the thirtieth of May next ensuing._ Item, _that
the duchy of Anjou and the county of Maine shall be released and
delivered to the King her father_—

[_Lets the paper fall._]

KING HENRY.
Uncle, how now?

GLOUCESTER.
Pardon me, gracious lord.
Some sudden qualm hath struck me at the heart
And dimmed mine eyes, that I can read no further.

KING HENRY.
Uncle of Winchester, I pray read on.

CARDINAL.
[_Reads_.] Item, _it is further agreed between them, that the duchies
of Anjou and Maine shall be released and delivered to the King her
father, and she sent over of the King of England’s own proper cost and
charges, without having any dowry._

KING HENRY.
They please us well.—Lord Marquess, kneel down.
We here create thee the first Duke of Suffolk,
And girt thee with the sword.—Cousin of York,
We here discharge your grace from being regent
I’ th’ parts of France, till term of eighteen months
Be full expired.—Thanks, uncle Winchester,
Gloucester, York, Buckingham, Somerset,
Salisbury, and Warwick;
We thank you all for this great favour done
In entertainment to my princely Queen.
Come, let us in, and with all speed provide
To see her coronation be performed.

[_Exeunt King, Queen and Suffolk._]

GLOUCESTER.
Brave peers of England, pillars of the state,
To you Duke Humphrey must unload his grief,
Your grief, the common grief of all the land.
What! Did my brother Henry spend his youth,
His valour, coin, and people, in the wars?
Did he so often lodge in open field,
In winter’s cold and summer’s parching heat,
To conquer France, his true inheritance?
And did my brother Bedford toil his wits
To keep by policy what Henry got?
Have you yourselves, Somerset, Buckingham,
Brave York, Salisbury, and victorious Warwick,
Received deep scars in France and Normandy?
Or hath mine uncle Beaufort and myself,
With all the learned council of the realm,
Studied so long, sat in the council house
Early and late, debating to and fro
How France and Frenchmen might be kept in awe,
And had his highness in his infancy
Crowned in Paris in despite of foes?
And shall these labours and these honours die?
Shall Henry’s conquest, Bedford’s vigilance,
Your deeds of war, and all our counsel die?
O peers of England, shameful is this league!
Fatal this marriage, cancelling your fame,
Blotting your names from books of memory,
Razing the characters of your renown,
Defacing monuments of conquered France,
Undoing all, as all had never been!

CARDINAL.
Nephew, what means this passionate discourse,
This peroration with such circumstance?
For France, ’tis ours; and we will keep it still.

GLOUCESTER.
Ay, uncle, we will keep it if we can,
But now it is impossible we should.
Suffolk, the new-made duke that rules the roast,
Hath given the duchy of Anjou and Maine
Unto the poor King Reignier, whose large style
Agrees not with the leanness of his purse.

SALISBURY.
Now, by the death of Him that died for all,
These counties were the keys of Normandy!
But wherefore weeps Warwick, my valiant son?

WARWICK.
For grief that they are past recovery;
For, were there hope to conquer them again,
My sword should shed hot blood, mine eyes no tears.
Anjou and Maine! Myself did win them both,
Those provinces these arms of mine did conquer;
And are the cities that I got with wounds
Delivered up again with peaceful words?
_Mort Dieu!_

YORK.
For Suffolk’s duke, may he be suffocate,
That dims the honour of this warlike isle!
France should have torn and rent my very heart
Before I would have yielded to this league.
I never read but England’s kings have had
Large sums of gold and dowries with their wives;
And our King Henry gives away his own,
To match with her that brings no vantages.

GLOUCESTER.
A proper jest, and never heard before,
That Suffolk should demand a whole fifteenth
For costs and charges in transporting her!
She should have staid in France, and starved in France,
Before—

CARDINAL.
My Lord of Gloucester, now ye grow too hot.
It was the pleasure of my lord the King.

GLOUCESTER.
My Lord of Winchester, I know your mind.
’Tis not my speeches that you do mislike,
But ’tis my presence that doth trouble ye.
Rancour will out. Proud prelate, in thy face
I see thy fury. If I longer stay,
We shall begin our ancient bickerings.—
Lordings, farewell; and say, when I am gone,
I prophesied France will be lost ere long.

[_Exit._]

CARDINAL.
So, there goes our Protector in a rage.
’Tis known to you he is mine enemy,
Nay, more, an enemy unto you all,
And no great friend, I fear me, to the King.
Consider, lords, he is the next of blood
And heir apparent to the English crown.
Had Henry got an empire by his marriage,
And all the wealthy kingdoms of the west,
There’s reason he should be displeased at it.
Look to it, lords. Let not his smoothing words
Bewitch your hearts; be wise and circumspect.
What though the common people favour him,
Calling him “Humphrey, the good Duke of Gloucester,”
Clapping their hands, and crying with loud voice,
“Jesu maintain your royal excellence!”
With “God preserve the good Duke Humphrey!”
I fear me, lords, for all this flattering gloss,
He will be found a dangerous Protector.

BUCKINGHAM.
Why should he, then, protect our sovereign,
He being of age to govern of himself?
Cousin of Somerset, join you with me,
And all together, with the Duke of Suffolk,
We’ll quickly hoist Duke Humphrey from his seat.

CARDINAL.
This weighty business will not brook delay;
I’ll to the Duke of Suffolk presently.

[_Exit._]

SOMERSET.
Cousin of Buckingham, though Humphrey’s pride
And greatness of his place be grief to us,
Yet let us watch the haughty cardinal;
His insolence is more intolerable
Than all the princes’ in the land beside.
If Gloucester be displaced, he’ll be Protector.

BUCKINGHAM.
Or thou or I, Somerset, will be Protector,
Despite Duke Humphrey or the Cardinal.

[_Exeunt Buckingham and Somerset._]

SALISBURY.
Pride went before; 	Ambition follows him.
While these do labour for their own preferment,
Behoves it us to labour for the realm.
I never saw but Humphrey Duke of Gloucester,
Did bear him like a noble gentleman.
Oft have I seen the haughty Cardinal,
More like a soldier than a man o’ th’ church,
As stout and proud as he were lord of all,
Swear like a ruffian and demean himself
Unlike the ruler of a commonweal.—
Warwick my son, the comfort of my age,
Thy deeds, thy plainness, and thy housekeeping,
Hath won the greatest favour of the commons,
Excepting none but good Duke Humphrey.—
And, brother York, thy acts in Ireland,
In bringing them to civil discipline,
Thy late exploits done in the heart of France,
When thou wert regent for our sovereign,
Have made thee feared and honoured of the people.
Join we together for the public good,
In what we can to bridle and suppress
The pride of Suffolk and the Cardinal,
With Somerset’s and Buckingham’s ambition;
And, as we may, cherish Duke Humphrey’s deeds
While they do tend the profit of the land.

WARWICK.
So God help Warwick, as he loves the land
And common profit of his country!

YORK.
And so says York, [_Aside_.] for he hath greatest cause.

SALISBURY.
Then let’s make haste away and look unto the main.

WARWICK.
Unto the main! O father, Maine is lost,
That Maine which by main force Warwick did win,
And would have kept so long as breath did last!
Main chance, father, you meant; but I meant Maine,
Which I will win from France, or else be slain.

[_Exeunt Warwick and Salisbury._]

YORK.
Anjou and Maine are given to the French;
Paris is lost; the state of Normandy
Stands on a tickle point now they are gone.
Suffolk concluded on the articles,
The peers agreed, and Henry was well pleased
To change two dukedoms for a duke’s fair daughter.
I cannot blame them all. What is’t to them?
’Tis thine they give away, and not their own.
Pirates may make cheap pennyworths of their pillage,
And purchase friends, and give to courtesans,
Still revelling like lords till all be gone;
Whileas the silly owner of the goods
Weeps over them, and wrings his hapless hands,
And shakes his head, and trembling stands aloof,
While all is shared and all is borne away,
Ready to starve and dare not touch his own.
So York must sit and fret and bite his tongue,
While his own lands are bargained for and sold.
Methinks the realms of England, France, and Ireland
Bear that proportion to my flesh and blood
As did the fatal brand Althaea burnt
Unto the prince’s heart of Calydon.
Anjou and Maine both given unto the French!
Cold news for me, for I had hope of France,
Even as I have of fertile England’s soil.
A day will come when York shall claim his own;
And therefore I will take the Nevilles’ parts,
And make a show of love to proud Duke Humphrey,
And when I spy advantage, claim the crown,
For that’s the golden mark I seek to hit.
Nor shall proud Lancaster usurp my right,
Nor hold the sceptre in his childish fist,
Nor wear the diadem upon his head,
Whose church-like humours fits not for a crown.
Then, York, be still awhile till time do serve.
Watch thou and wake when others be asleep,
To pry into the secrets of the state;
Till Henry, surfeiting in joys of love
With his new bride and England’s dear-bought Queen,
And Humphrey with the peers be fallen at jars.
Then will I raise aloft the milk-white rose,
With whose sweet smell the air shall be perfumed,
And in my standard bear the arms of York,
To grapple with the house of Lancaster;
And force perforce I’ll make him yield the crown,
Whose bookish rule hath pulled fair England down.

[_Exit._]

SCENE II. The Duke of Gloucester’s House

Enter Duke Humphrey of Gloucester and his wife Eleanor.

ELEANOR.
Why droops my lord, like over-ripened corn
Hanging the head at Ceres’ plenteous load?
Why doth the great Duke Humphrey knit his brows,
As frowning at the favours of the world?
Why are thine eyes fixed to the sullen earth,
Gazing on that which seems to dim thy sight?
What seest thou there? King Henry’s diadem,
Enchased with all the honours of the world?
If so, gaze on, and grovel on thy face,
Until thy head be circled with the same.
Put forth thy hand, reach at the glorious gold.
What, is’t too short? I’ll lengthen it with mine;
And, having both together heaved it up,
We’ll both together lift our heads to heaven,
And never more abase our sight so low
As to vouchsafe one glance unto the ground.

GLOUCESTER.
O Nell, sweet Nell, if thou dost love thy lord,
Banish the canker of ambitious thoughts.
And may that hour when I imagine ill
Against my King and nephew, virtuous Henry,
Be my last breathing in this mortal world!
My troublous dreams this night doth make me sad.

ELEANOR.
What dreamed my lord? Tell me, and I’ll requite it
With sweet rehearsal of my morning’s dream.

GLOUCESTER.
Methought this staff, mine office-badge in court,
Was broke in twain; by whom I have forgot,
But, as I think, it was by th’ Cardinal,
And on the pieces of the broken wand
Were placed the heads of Edmund, Duke of Somerset
And William de la Pole, first Duke of Suffolk.
This was my dream; what it doth bode, God knows.

ELEANOR.
Tut, this was nothing but an argument
That he that breaks a stick of Gloucester’s grove
Shall lose his head for his presumption.
But list to me, my Humphrey, my sweet Duke:
Methought I sat in seat of majesty
In the cathedral church of Westminster
And in that chair where kings and queens are crowned,
Where Henry and Dame Margaret kneeled to me
And on my head did set the diadem.

GLOUCESTER.
Nay, Eleanor, then must I chide outright.
Presumptuous dame, ill-nurtured Eleanor,
Art thou not second woman in the realm,
And the Protector’s wife, beloved of him?
Hast thou not worldly pleasure at command,
Above the reach or compass of thy thought?
And wilt thou still be hammering treachery
To tumble down thy husband and thyself
From top of honour to disgrace’s feet?
Away from me, and let me hear no more!

ELEANOR.
What, what, my lord! Are you so choleric
With Eleanor for telling but her dream?
Next time I’ll keep my dreams unto myself,
And not be checked.

GLOUCESTER.
Nay, be not angry, I am pleased again.

Enter Messenger.

MESSENGER.
My Lord Protector, ’tis his highness’ pleasure
You do prepare to ride unto Saint Albans,
Whereas the King and Queen do mean to hawk.

GLOUCESTER.
I go. Come, Nell, thou wilt ride with us?

ELEANOR.
Yes, my good lord, I’ll follow presently.

[_Exeunt Gloucester and Messenger._]

Follow I must; I cannot go before
While Gloucester bears this base and humble mind.
Were I a man, a duke, and next of blood,
I would remove these tedious stumbling-blocks
And smooth my way upon their headless necks;
And, being a woman, I will not be slack
To play my part in Fortune’s pageant.—
Where are you there? Sir John! Nay, fear not, man,
We are alone; here’s none but thee and I.

Enter Hume.

HUME.
Jesus preserve your royal majesty!

ELEANOR.
What sayst thou? Majesty! I am but grace.

HUME.
But, by the grace of God, and Hume’s advice,
Your grace’s title shall be multiplied.

ELEANOR.
What sayst thou, man? Hast thou as yet conferred
With Margery Jourdain, the cunning witch,
With Roger Bolingbroke, the conjurer?
And will they undertake to do me good?

HUME.
This they have promised, to show your highness
A spirit raised from depth of underground,
That shall make answer to such questions
As by your Grace shall be propounded him.

ELEANOR.
It is enough, I’ll think upon the questions.
When from Saint Albans we do make return,
We’ll see these things effected to the full.
Here, Hume, take this reward; make merry, man,
With thy confederates in this weighty cause.

[_Exit._]

HUME.
Hume must make merry with the Duchess’ gold.
Marry, and shall. But, how now, Sir John Hume!
Seal up your lips, and give no words but mum;
The business asketh silent secrecy.
Dame Eleanor gives gold to bring the witch;
Gold cannot come amiss, were she a devil.
Yet have I gold flies from another coast.
I dare not say, from the rich cardinal
And from the great and new-made Duke of Suffolk,
Yet I do find it so. For, to be plain,
They, knowing Dame Eleanor’s aspiring humour,
Have hired me to undermine the Duchess
And buzz these conjurations in her brain.
They say “A crafty knave does need no broker”,
Yet am I Suffolk and the cardinal’s broker.
Hume, if you take not heed, you shall go near
To call them both a pair of crafty knaves.
Well, so its stands; and thus, I fear, at last
Hume’s knavery will be the Duchess’ wrack,
And her attainture will be Humphrey’s fall.
Sort how it will, I shall have gold for all.

[_Exit._]

SCENE III. London. The palace

Enter Peter and Petitioners.

1 PETITIONER.
My masters, let’s stand close. My Lord Protector will come this way by
and by, and then we may deliver our supplications in the quill.

2 PETITIONER.
Marry, the Lord protect him, for he’s a good man! Jesu bless him!

Enter Suffolk and Queen.

1 PETITIONER.
Here he comes, methinks, and the Queen with him. I’ll be the first,
sure.

2 PETITIONER.
Come back, fool! This is the Duke of Suffolk and not my Lord Protector.

SUFFOLK.
How now, fellow; wouldst anything with me?

1 PETITIONER.
I pray, my lord, pardon me, I took ye for my Lord Protector.

QUEEN MARGARET.
[_Reading_.] “To my Lord Protector.” Are your supplications to his
lordship? Let me see them. What is thine?

1 PETITIONER.
Mine is, an ’t please your grace, against John Goodman, my Lord
Cardinal’s man, for keeping my house and lands, and wife and all, from
me.

SUFFOLK.
Thy wife too! That’s some wrong, indeed.—What’s yours?—What’s here!
[_Reads_.] _Against the Duke of Suffolk for enclosing the commons of
Melford._ How now, sir knave!

2 PETITIONER.
Alas, sir, I am but a poor petitioner of our whole township.

PETER.
[_Giving his petition_.] Against my master, Thomas Horner, for saying
that the Duke of York was rightful heir to the crown.

QUEEN MARGARET.
What sayst thou? Did the Duke of York say he was rightful heir to the
crown?

PETER.
That my master was? No, forsooth, my master said that he was, and that
the King was an usurper.

SUFFOLK.
Who is there?

Enter Servant.

Take this fellow in, and send for his master with a pursuivant
presently.—We’ll hear more of your matter before the King.

[_Exit Servant with Peter._]

QUEEN MARGARET.
And as for you, that love to be protected
Under the wings of our Protector’s grace,
Begin your suits anew, and sue to him.

[_Tears the supplications._]

Away, base cullions!—Suffolk, let them go.

ALL.
Come, let’s be gone.

[_Exeunt._]

QUEEN MARGARET.
My Lord of Suffolk, say, is this the guise,
Is this the fashion in the court of England?
Is this the government of Britain’s isle,
And this the royalty of Albion’s king?
What, shall King Henry be a pupil still
Under the surly Gloucester’s governance?
Am I a queen in title and in style,
And must be made a subject to a duke?
I tell thee, Pole, when in the city Tours
Thou ran’st atilt in honour of my love
And stol’st away the ladies’ hearts of France,
I thought King Henry had resembled thee
In courage, courtship, and proportion.
But all his mind is bent to holiness,
To number Ave-Maries on his beads.
His champions are the prophets and apostles,
His weapons holy saws of sacred writ,
His study is his tilt-yard, and his loves
Are brazen images of canonized saints.
I would the college of the cardinals
Would choose him pope and carry him to Rome
And set the triple crown upon his head!
That were a state fit for his holiness.

SUFFOLK.
Madam, be patient. As I was cause
Your highness came to England, so will I
In England work your grace’s full content.

QUEEN MARGARET.
Beside the haughty Protector, have we Beaufort
The imperious churchman, Somerset, Buckingham,
And grumbling York; and not the least of these
But can do more in England than the King.

SUFFOLK.
And he of these that can do most of all
Cannot do more in England than the Nevilles;
Salisbury and Warwick are no simple peers.

QUEEN MARGARET.
Not all these lords do vex me half so much
As that proud dame, the Lord Protector’s wife.
She sweeps it through the court with troops of ladies,
More like an empress than Duke Humphrey’s wife.
Strangers in court do take her for the Queen.
She bears a duke’s revenues on her back,
And in her heart she scorns our poverty.
Shall I not live to be avenged on her?
Contemptuous base-born callet as she is,
She vaunted ’mongst her minions t’ other day
The very train of her worst wearing gown
Was better worth than all my father’s lands
Till Suffolk gave two dukedoms for his daughter.

SUFFOLK.
Madam, myself have limed a bush for her
And placed a quire of such enticing birds
That she will light to listen to the lays
And never mount to trouble you again.
So let her rest; and, madam, list to me,
For I am bold to counsel you in this:
Although we fancy not the Cardinal,
Yet must we join with him and with the lords
Till we have brought Duke Humphrey in disgrace.
As for the Duke of York, this late complaint
Will make but little for his benefit.
So, one by one, we’ll weed them all at last,
And you yourself shall steer the happy helm.

Sound a sennet. Enter the King, Gloucester, Cardinal Beaufort,
Somerset, Buckingham, Salisbury, York, Warwick and the Duchess of
Gloucester.

KING HENRY.
For my part, noble lords, I care not which;
Or Somerset or York, all’s one to me.

YORK.
If York have ill demeaned himself in France,
Then let him be denied the regentship.

SOMERSET.
If Somerset be unworthy of the place,
Let York be regent; I will yield to him.

WARWICK.
Whether your Grace be worthy, yea or no,
Dispute not that; York is the worthier.

CARDINAL.
Ambitious Warwick, let thy betters speak.

WARWICK.
The Cardinal’s not my better in the field.

BUCKINGHAM.
All in this presence are thy betters, Warwick.

WARWICK.
Warwick may live to be the best of all.

SALISBURY.
Peace, son!—And show some reason, Buckingham,
Why Somerset should be preferred in this.

QUEEN MARGARET.
Because the King, forsooth, will have it so.

GLOUCESTER.
Madam, the King is old enough himself
To give his censure. These are no women’s matters.

QUEEN MARGARET.
If he be old enough, what needs your grace
To be Protector of his excellence?

GLOUCESTER.
Madam, I am Protector of the realm,
And at his pleasure will resign my place.

SUFFOLK.
Resign it then, and leave thine insolence.
Since thou wert king—as who is king but thou?—
The commonwealth hath daily run to wrack,
The Dauphin hath prevailed beyond the seas,
And all the peers and nobles of the realm
Have been as bondmen to thy sovereignty.

CARDINAL.
The commons hast thou racked; the clergy’s bags
Are lank and lean with thy extortions.

SOMERSET.
Thy sumptuous buildings and thy wife’s attire
Have cost a mass of public treasury.

BUCKINGHAM.
Thy cruelty in execution
Upon offenders hath exceeded law,
And left thee to the mercy of the law.

QUEEN MARGARET.
Thy sale of offices and towns in France,
If they were known, as the suspect is great,
Would make thee quickly hop without thy head.

[_Exit Gloucester. The Queen drops her fan._]

Give me my fan. What minion! Can ye not?

[_She gives the Duchess a box on the ear._]

I cry your mercy, madam; was it you?

ELEANOR.
Was’t I! Yea, I it was, proud Frenchwoman.
Could I come near your beauty with my nails,
I’d set my ten commandments in your face.

KING HENRY.
Sweet aunt, be quiet; ’twas against her will.

ELEANOR.
Against her will! Good King, look to ’t in time;
She’ll hamper thee and dandle thee like a baby.
Though in this place most master wear no breeches,
She shall not strike Dame Eleanor unrevenged.

[_Exit._]

BUCKINGHAM.
Lord Cardinal, I will follow Eleanor,
And listen after Humphrey, how he proceeds.
She’s tickled now; her fume needs no spurs,
She’ll gallop far enough to her destruction.

[_Exit._]

Enter Gloucester.

GLOUCESTER.
Now, lords, my choler being overblown
With walking once about the quadrangle,
I come to talk of commonwealth affairs.
As for your spiteful false objections,
Prove them, and I lie open to the law;
But God in mercy so deal with my soul
As I in duty love my king and country!
But, to the matter that we have in hand:
I say, my sovereign, York is meetest man
To be your regent in the realm of France.

SUFFOLK.
Before we make election, give me leave
To show some reason, of no little force,
That York is most unmeet of any man.

YORK.
I’ll tell thee, Suffolk, why I am unmeet:
First, for I cannot flatter thee in pride;
Next, if I be appointed for the place,
My Lord of Somerset will keep me here
Without discharge, money, or furniture,
Till France be won into the Dauphin’s hands.
Last time, I danced attendance on his will
Till Paris was besieged, famished, and lost.

WARWICK.
That can I witness, and a fouler fact
Did never traitor in the land commit.

SUFFOLK.
Peace, headstrong Warwick!

WARWICK.
Image of pride, why should I hold my peace?

Enter Horner the armourer and his man Peter, guarded.

SUFFOLK.
Because here is a man accused of treason.
Pray God the Duke of York excuse himself!

YORK.
Doth anyone accuse York for a traitor?

KING HENRY.
What mean’st thou, Suffolk? Tell me, what are these?

SUFFOLK.
Please it your majesty, this is the man
That doth accuse his master of high treason.
His words were these: that Richard, Duke of York
Was rightful heir unto the English crown,
And that your majesty was an usurper.

KING HENRY.
Say, man, were these thy words?

HORNER.
An ’t shall please your majesty, I never said nor thought any such
matter. God is my witness, I am falsely accused by the villain.

PETER.
By these ten bones, my lords, he did speak them to me in the garret one
night as we were scouring my Lord of York’s armour.

YORK.
Base dunghill villain and mechanical,
I’ll have thy head for this thy traitor’s speech!—
I do beseech your royal majesty,
Let him have all the rigour of the law.

HORNER.
Alas, my lord, hang me if ever I spake the words. My accuser is my
prentice; and when I did correct him for his fault the other day, he
did vow upon his knees he would be even with me. I have good witness of
this, therefore I beseech your majesty, do not cast away an honest man
for a villain’s accusation.

KING HENRY.
Uncle, what shall we say to this in law?

GLOUCESTER.
This doom, my lord, if I may judge:
Let Somerset be regent o’er the French,
Because in York this breeds suspicion;
And let these have a day appointed them
For single combat in convenient place,
For he hath witness of his servant’s malice.
This is the law, and this Duke Humphrey’s doom.

SOMERSET.
I humbly thank your royal Majesty.

HORNER.
And I accept the combat willingly.

PETER.
Alas, my lord, I cannot fight; for God’s sake, pity my case! The spite
of man prevaileth against me. O Lord, have mercy upon me! I shall never
be able to fight a blow. O Lord, my heart!

GLOUCESTER.
Sirrah, or you must fight or else be hanged.

KING HENRY.
Away with them to prison; and the day
Of combat shall be the last of the next month.
Come, Somerset, we’ll see thee sent away.

[_Flourish. Exeunt._]

SCENE IV. Gloucester’s Garden

Enter the Witch Margery Jourdain, the two Priests, Hume, Southwell and
Bolingbroke.

HUME.
Come, my masters. The duchess, I tell you, expects performance of your
promises.

BOLINGBROKE.
Master Hume, we are therefore provided. Will her ladyship behold and
hear our exorcisms?

HUME.
Ay, what else? Fear you not her courage.

BOLINGBROKE.
I have heard her reported to be a woman of an invincible spirit. But it
shall be convenient, Master Hume, that you be by her aloft while we be
busy below; and so, I pray you go, in God’s name, and leave us.

[_Exit Hume._]

Mother Jourdain, be you prostrate and grovel on the earth. John
Southwell, read you; and let us to our work.

Enter Duchess aloft, Hume following.

ELEANOR.
Well said, my masters; and welcome all. To this gear, the sooner the
better.

BOLINGBROKE.
Patience, good lady; wizards know their times.
Deep night, dark night, the silent of the night,
The time of night when Troy was set on fire,
The time when screech-owls cry and ban-dogs howl,
And spirits walk and ghosts break up their graves;
That time best fits the work we have in hand.
Madam, sit you and fear not. Whom we raise
We will make fast within a hallowed verge.

[_Here they do the ceremonies belonging, and make the circle;
Bolingbroke or Southwell reads_ “Conjuro te”, _etc. It thunders and
lightens terribly; then the Spirit riseth._]

SPIRIT.
_Adsum_.

M. JOURDAIN.
Asnath,
By the eternal God, whose name and power
Thou tremblest at, answer that I shall ask;
For till thou speak thou shalt not pass from hence.

SPIRIT.
Ask what thou wilt. That I had said and done!

BOLINGBROKE.
[_Reads_.] _First of the King: what shall of him become?_

SPIRIT.
The duke yet lives that Henry shall depose,
But him outlive and die a violent death.

[_As the Spirit speaks, Southwell writes the answer._]

BOLINGBROKE.
[_Reads_.] _What fates await the Duke of Suffolk?_

SPIRIT.
By water shall he die and take his end.

BOLINGBROKE.
[_Reads_.] _What shall befall the Duke of Somerset?_

SPIRIT.
Let him shun castles.
Safer shall he be upon the sandy plains
Than where castles mounted stand.
Have done, for more I hardly can endure.

BOLINGBROKE.
Descend to darkness and the burning lake!
False fiend, avoid!

[_Thunder and lightning. Exit Spirit._]

Enter the Duke of York and the Duke of Buckingham with their Guard, and
Sir Humphrey Stafford, and break in.

YORK.
Lay hands upon these traitors and their trash.
Beldam, I think we watched you at an inch.
What, madam, are you there? The King and commonweal
Are deeply indebted for this piece of pains.
My Lord Protector will, I doubt it not,
See you well guerdoned for these good deserts.

ELEANOR.
Not half so bad as thine to England’s king,
Injurious duke, that threatest where’s no cause.

BUCKINGHAM.
True, madam, none at all. What call you this?
Away with them! Let them be clapped up close
And kept asunder.—You, madam, shall with us.—
Stafford, take her to thee.

[_Exit Stafford._]

[_Exeunt above, Duchess and Hume, guarded._]

We’ll see your trinkets here all forthcoming.
All, away!

[_Exeunt guard with  Jourdain, Southwell, Bolingbroke, etc._]

YORK.
Lord Buckingham, methinks you watched her well.
A pretty plot, well chosen to build upon!
Now, pray, my lord, let’s see the devil’s writ.
What have we here?
[_Reads_.] _The duke yet lives that Henry shall depose.
But him outlive and die a violent death._
Why, this is just
_Aio te, Aeacida, Romanos vincere posse._
Well, to the rest:
_Tell me what fate awaits the Duke of Suffolk?
By water shall he die and take his end.
What shall betide the Duke of Somerset?
Let him shun castles;
Safer shall he be upon the sandy plains
Than where castles mounted stand._
Come, come, my lords, these oracles
Are hardly attained, and hardly understood.
The King is now in progress towards Saint Albans,
With him the husband of this lovely lady.
Thither go these news as fast as horse can carry them.
A sorry breakfast for my Lord Protector.

BUCKINGHAM.
Your Grace shall give me leave, my Lord of York,
To be the post, in hope of his reward.

YORK.
At your pleasure, my good lord.

[_Exit Buckingham._]

Who’s within there, ho!

Enter a Servingman.

Invite my Lords of Salisbury and Warwick
To sup with me tomorrow night. Away!

[_Exeunt._]




ACT II

SCENE I. Saint Albans


Enter the King, Queen, Gloucester, Cardinal and Suffolk with Falconers
hallooing.

QUEEN MARGARET.
Believe me, lords, for flying at the brook
I saw not better sport these seven years’ day;
Yet, by your leave, the wind was very high,
And, ten to one, old Joan had not gone out.

KING HENRY.
But what a point, my lord, your falcon made,
And what a pitch she flew above the rest!
To see how God in all His creatures works!
Yea, man and birds are fain of climbing high.

SUFFOLK.
No marvel, an it like your majesty,
My Lord Protector’s hawks do tower so well;
They know their master loves to be aloft,
And bears his thoughts above his falcon’s pitch.

GLOUCESTER.
My lord, ’tis but a base ignoble mind
That mounts no higher than a bird can soar.

CARDINAL.
I thought as much. He would be above the clouds.

GLOUCESTER.
Ay, my Lord Cardinal, how think you by that?
Were it not good your grace could fly to heaven?

KING HENRY.
The treasury of everlasting joy.

CARDINAL.
Thy heaven is on earth; thine eyes and thoughts
Beat on a crown, the treasure of thy heart,
Pernicious Protector, dangerous peer,
That smooth’st it so with king and commonweal!

GLOUCESTER.
What, cardinal, is your priesthood grown peremptory?
_Tantaene animis coelestibus irae?_
Churchmen so hot? Good uncle, hide such malice.
With such holiness can you do it?

SUFFOLK.
No malice, sir; no more than well becomes
So good a quarrel and so bad a peer.

GLOUCESTER.
As who, my lord?

SUFFOLK.
Why, as you, my lord,
An ’t like your lordly Lord Protectorship.

GLOUCESTER.
Why, Suffolk, England knows thine insolence.

QUEEN MARGARET.
And thy ambition, Gloucester.

KING HENRY.
I prithee, peace, good queen,
And whet not on these furious peers;
For blessed are the peacemakers on earth.

CARDINAL.
Let me be blessed for the peace I make
Against this proud Protector, with my sword!

GLOUCESTER.
[_Aside to Cardinal_.] Faith, holy uncle, would ’twere come to that!

CARDINAL.
[_Aside to Gloucester_.] Marry, when thou dar’st.

GLOUCESTER.
[_Aside to Cardinal_.] Make up no factious numbers for the matter,
In thine own person answer thy abuse.

CARDINAL.
[_Aside to Gloucester_.] Ay, where thou dar’st not peep; an if thou
dar’st,
This evening, on the east side of the grove.

KING HENRY.
How now, my lords?

CARDINAL.
Believe me, cousin Gloucester,
Had not your man put up the fowl so suddenly,
We had had more sport.—[_Aside to Gloucester_.]
Come with thy two-hand sword.

GLOUCESTER.
True, uncle.
[_Aside to Cardinal_.] Are ye advised? The east side of the grove?

CARDINAL.
[_Aside to Gloucester_.] I am with you.

KING HENRY.
Why, how now, uncle Gloucester?

GLOUCESTER.
Talking of hawking; nothing else, my lord.
[_Aside to Cardinal_.] Now, by God’s mother, priest,
I’ll shave your crown for this,
Or all my fence shall fail.

CARDINAL.
[_Aside to Gloucester_.] _Medice, teipsum._—
Protector, see to ’t well, protect yourself.

KING HENRY.
The winds grow high; so do your stomachs, lords.
How irksome is this music to my heart!
When such strings jar, what hope of harmony?
I pray, my lords, let me compound this strife.

Enter a Townsman of Saint Albans, crying, “A miracle!”

GLOUCESTER.
What means this noise?
Fellow, what miracle dost thou proclaim?

TOWNSMAN.
A miracle! A miracle!

SUFFOLK.
Come to the King, and tell him what miracle.

TOWNSMAN.
Forsooth, a blind man at Saint Alban’s shrine,
Within this half hour, hath received his sight,
A man that ne’er saw in his life before.

KING HENRY.
Now, God be praised, that to believing souls
Gives light in darkness, comfort in despair!

Enter the Mayor of Saint Albans and his brethren, bearing Simpcox
between two in a chair, Simpcox’s Wife following.

CARDINAL.
Here comes the townsmen on procession,
To present your highness with the man.

KING HENRY.
Great is his comfort in this earthly vale,
Although by his sight his sin be multiplied.

GLOUCESTER.
Stand by, my masters. Bring him near the King.
His highness’ pleasure is to talk with him.

KING HENRY.
Good fellow, tell us here the circumstance,
That we for thee may glorify the Lord.
What, hast thou been long blind and now restored?

SIMPCOX.
Born blind, an ’t please your grace.

WIFE.
Ay, indeed, was he.

SUFFOLK.
What woman is this?

WIFE.
His wife, an ’t like your worship.

GLOUCESTER.
Hadst thou been his mother, thou couldst have better told.

KING HENRY.
Where wert thou born?

SIMPCOX.
At Berwick in the north, an ’t like your grace.

KING HENRY.
Poor soul, God’s goodness hath been great to thee.
Let never day nor night unhallowed pass,
But still remember what the Lord hath done.

QUEEN MARGARET.
Tell me, good fellow, cam’st thou here by chance,
Or of devotion, to this holy shrine?

SIMPCOX.
God knows, of pure devotion; being called
A hundred times and oftener, in my sleep,
By good Saint Alban, who said “Simpcox, come,
Come, offer at my shrine, and I will help thee.”

WIFE.
Most true, forsooth; and many time and oft
Myself have heard a voice to call him so.

CARDINAL.
What, art thou lame?

SIMPCOX.
Ay, God Almighty help me!

SUFFOLK.
How cam’st thou so?

SIMPCOX.
A fall off of a tree.

WIFE.
A plum-tree, master.

GLOUCESTER.
How long hast thou been blind?

SIMPCOX.
O, born so, master.

GLOUCESTER.
What, and wouldst climb a tree?

SIMPCOX.
But that in all my life, when I was a youth.

WIFE.
Too true; and bought his climbing very dear.

GLOUCESTER.
Mass, thou lov’dst plums well, that wouldst venture so.

SIMPCOX.
Alas, good master, my wife desired some damsons,
And made me climb, with danger of my life.

GLOUCESTER.
A subtle knave! But yet it shall not serve.—
Let me see thine eyes. Wink now. Now open them.
In my opinion yet thou seest not well.

SIMPCOX.
Yes, master, clear as day, I thank God and Saint Alban.

GLOUCESTER.
Sayst thou me so? What colour is this cloak of?

SIMPCOX.
Red, master, red as blood.

GLOUCESTER.
Why, that’s well said. What colour is my gown of?

SIMPCOX.
Black, forsooth, coal-black as jet.

KING HENRY.
Why, then, thou know’st what colour jet is of?

SUFFOLK.
And yet, I think, jet did he never see.

GLOUCESTER.
But cloaks and gowns before this day, a many.

WIFE.
Never before this day in all his life.

GLOUCESTER.
Tell me, sirrah, what’s my name?

SIMPCOX.
Alas, master, I know not.

GLOUCESTER.
What’s his name?

SIMPCOX.
I know not.

GLOUCESTER.
Nor his?

SIMPCOX.
No, indeed, master.

GLOUCESTER.
What’s thine own name?

SIMPCOX.
Sander Simpcox, an if it please you, master.

GLOUCESTER.
Then, Sander, sit there, the lyingest knave in Christendom. If thou
hadst been born blind, thou mightst as well have known all our names as
thus to name the several colours we do wear. Sight may distinguish of
colours; but suddenly to nominate them all, it is impossible.—My lords,
Saint Alban here hath done a miracle; and would ye not think his
cunning to be great that could restore this cripple to his legs again?

SIMPCOX.
O master, that you could!

GLOUCESTER.
My masters of Saint Albans, have you not beadles in your town, and
things called whips?

MAYOR.
Yes, my lord, if it please your grace.

GLOUCESTER.
Then send for one presently.

MAYOR.
Sirrah, go fetch the beadle hither straight.

[_Exit a Townsman._]

GLOUCESTER.
Now fetch me a stool hither by and by.—Now, sirrah, if you mean to save
yourself from whipping, leap me over this stool and run away.

SIMPCOX.
Alas, master, I am not able to stand alone.
You go about to torture me in vain.

Enter a Beadle with whips.

GLOUCESTER.
Well, sir, we must have you find your legs.
Sirrah beadle, whip him till he leap over that same stool.

BEADLE.
I will, my lord.—Come on, sirrah; off with your doublet quickly.

SIMPCOX.
Alas, master, what shall I do? I am not able to stand.

[_After the Beadle hath hit him once, he leaps over the stool and runs
away; and they follow and cry, “A miracle!”_]

KING HENRY.
O God, seest Thou this, and bearest so long?

QUEEN MARGARET.
It made me laugh to see the villain run.

GLOUCESTER.
Follow the knave, and take this drab away.

WIFE.
Alas, sir, we did it for pure need.

GLOUCESTER.
Let them be whipped through every market town
Till they come to Berwick, from whence they came.

[_Exeunt Wife, Beadle, Mayor, etc._]

CARDINAL.
Duke Humphrey has done a miracle today.

SUFFOLK.
True, made the lame to leap and fly away.

GLOUCESTER.
But you have done more miracles than I.
You made in a day, my lord, whole towns to fly.

Enter Buckingham.

KING HENRY.
What tidings with our cousin Buckingham?

BUCKINGHAM.
Such as my heart doth tremble to unfold.
A sort of naughty persons, lewdly bent,
Under the countenance and confederacy
Of Lady Eleanor, the Protector’s wife,
The ringleader and head of all this rout,
Have practised dangerously against your state,
Dealing with witches and with conjurers,
Whom we have apprehended in the fact,
Raising up wicked spirits from under ground,
Demanding of King Henry’s life and death,
And other of your highness’ Privy Council,
As more at large your Grace shall understand.

CARDINAL.
[_Aside to Gloucester_.] And so, my Lord Protector, by this means
Your lady is forthcoming yet at London.
This news, I think, hath turned your weapon’s edge;
’Tis like, my lord, you will not keep your hour.

GLOUCESTER.
Ambitious churchman, leave to afflict my heart.
Sorrow and grief have vanquished all my powers,
And, vanquished as I am, I yield to thee,
Or to the meanest groom.

KING HENRY.
O God, what mischiefs work the wicked ones,
Heaping confusion on their own heads thereby!

QUEEN MARGARET.
Gloucester, see here the tainture of thy nest,
And look thyself be faultless, thou wert best.

GLOUCESTER.
Madam, for myself, to heaven I do appeal
How I have loved my king and commonweal;
And, for my wife, I know not how it stands.
Sorry I am to hear what I have heard.
Noble she is; but if she have forgot
Honour and virtue, and conversed with such
As like to pitch defile nobility,
I banish her my bed and company
And give her as a prey to law and shame
That hath dishonoured Gloucester’s honest name.

KING HENRY.
Well, for this night we will repose us here;
Tomorrow toward London back again,
To look into this business thoroughly,
And call these foul offenders to their answers,
And poise the cause in Justice’ equal scales,
Whose beam stands sure, whose rightful cause prevails.

[_Flourish. Exeunt._]

SCENE II. London. The Duke of York’s Garden

Enter York, Salisbury and Warwick.

YORK.
Now, my good Lords of Salisbury and Warwick,
Our simple supper ended, give me leave
In this close walk to satisfy myself
In craving your opinion of my title,
Which is infallible, to England’s crown.

SALISBURY.
My lord, I long to hear it at full.

WARWICK.
Sweet York, begin; and if thy claim be good,
The Nevilles are thy subjects to command.

YORK.
Then thus:
Edward the Third, my lords, had seven sons:
The first, Edward the Black Prince, Prince of Wales;
The second, William of Hatfield; and the third,
Lionel, Duke of Clarence; next to whom
Was John of Gaunt, the Duke of Lancaster;
The fifth was Edmund Langley, Duke of York;
The sixth was Thomas of Woodstock, Duke of Gloucester;
William of Windsor was the seventh and last.
Edward the Black Prince died before his father
And left behind him Richard, his only son,
Who after Edward the Third’s death reigned as king,
Till Henry Bolingbroke, Duke of Lancaster,
The eldest son and heir of John of Gaunt,
Crowned by the name of Henry the Fourth,
Seized on the realm, deposed the rightful king,
Sent his poor queen to France, from whence she came,
And him to Pomfret; where, as all you know,
Harmless Richard was murdered traitorously.

WARWICK.
Father, the Duke hath told the truth;
Thus got the house of Lancaster the crown.

YORK.
Which now they hold by force and not by right;
For Richard, the first son’s heir, being dead,
The issue of the next son should have reigned.

SALISBURY.
But William of Hatfield died without an heir.

YORK.
The third son, Duke of Clarence, from whose line
I claim the crown, had issue, Philippa, a daughter,
Who married Edmund Mortimer, Earl of March.
Edmund had issue, Roger, Earl of March;
Roger had issue, Edmund, Anne, and Eleanor.

SALISBURY.
This Edmund, in the reign of Bolingbroke,
As I have read, laid claim unto the crown
And, but for Owen Glendower, had been king,
Who kept him in captivity till he died.
But to the rest.

YORK.
His eldest sister, Anne,
My mother, being heir unto the crown,
Married Richard Earl of Cambridge, who was son
To Edmund Langley, Edward the Third’s fifth son.
By her I claim the kingdom; she was heir
To Roger, Earl of March, who was the son
Of Edmund Mortimer, who married Philippa,
Sole daughter unto Lionel, Duke of Clarence.
So, if the issue of the elder son
Succeed before the younger, I am king.

WARWICK.
What plain proceeding is more plain than this?
Henry doth claim the crown from John of Gaunt,
The fourth son; York claims it from the third.
Till Lionel’s issue fails, his should not reign;
It fails not yet, but flourishes in thee
And in thy sons, fair slips of such a stock.
Then, father Salisbury, kneel we together,
And in this private plot be we the first
That shall salute our rightful sovereign
With honour of his birthright to the crown.

BOTH.
Long live our sovereign Richard, England’s king!

YORK.
We thank you, lords. But I am not your king
Till I be crowned, and that my sword be stained
With heart-blood of the house of Lancaster;
And that’s not suddenly to be performed,
But with advice and silent secrecy.
Do you as I do in these dangerous days—
Wink at the Duke of Suffolk’s insolence,
At Beaufort’s pride, at Somerset’s ambition,
At Buckingham, and all the crew of them,
Till they have snared the shepherd of the flock,
That virtuous prince, the good Duke Humphrey.
’Tis that they seek; and they, in seeking that,
Shall find their deaths, if York can prophesy.

SALISBURY.
My lord, break we off; we know your mind at full.

WARWICK.
My heart assures me that the Earl of Warwick
Shall one day make the Duke of York a king.

YORK.
And, Neville, this I do assure myself:
Richard shall live to make the Earl of Warwick
The greatest man in England but the king.

[_Exeunt._]

SCENE III. A Hall of Justice

Sound trumpets. Enter the King, the Queen, Gloucester, York, Suffolk
and Salisbury; the Duchess of Gloucester, Margery Jourdain, Southwell,
Hume and Bolingbroke under guard.

KING HENRY.
Stand forth, Dame Eleanor Cobham, Gloucester’s wife.
In sight of God and us, your guilt is great;
Receive the sentence of the law for sins
Such as by God’s book are adjudged to death.
You four, from hence to prison back again;
From thence unto the place of execution.
The witch in Smithfield shall be burnt to ashes,
And you three shall be strangled on the gallows.
You, madam, for you are more nobly born,
Despoiled of your honour in your life,
Shall, after three days’ open penance done,
Live in your country here in banishment,
With Sir John Stanley in the Isle of Man.

ELEANOR.
Welcome is banishment; welcome were my death.

GLOUCESTER.
Eleanor, the law, thou seest, hath judged thee.
I cannot justify whom the law condemns.

[_Exeunt Duchess and the other prisoners, guarded._]

Mine eyes are full of tears, my heart of grief.
Ah, Humphrey, this dishonour in thine age
Will bring thy head with sorrow to the ground!—
I beseech your majesty, give me leave to go;
Sorrow would solace, and mine age would ease.

KING HENRY.
Stay, Humphrey Duke of Gloucester. Ere thou go,
Give up thy staff. Henry will to himself
Protector be; and God shall be my hope,
My stay, my guide, and lantern to my feet.
And go in peace, Humphrey, no less beloved
Than when thou wert Protector to thy king.

QUEEN MARGARET.
I see no reason why a king of years
Should be to be protected like a child.
God and King Henry govern England’s realm!
Give up your staff, sir, and the King his realm.

GLOUCESTER.
My staff? Here, noble Henry, is my staff.
As willingly do I the same resign
As e’er thy father Henry made it mine;
And even as willingly at thy feet I leave it
As others would ambitiously receive it.
Farewell, good King. When I am dead and gone,
May honourable peace attend thy throne.

[_Exit._]

QUEEN MARGARET.
Why, now is Henry King and Margaret Queen,
And Humphrey Duke of Gloucester scarce himself,
That bears so shrewd a maim. Two pulls at once;
His lady banished, and a limb lopped off.
This staff of honour raught, there let it stand
Where it best fits to be, in Henry’s hand.

SUFFOLK.
Thus droops this lofty pine and hangs his sprays;
Thus Eleanor’s pride dies in her youngest days.

YORK.
Lords, let him go.—Please it your majesty,
This is the day appointed for the combat,
And ready are the appellant and defendant,
The armourer and his man, to enter the lists,
So please your highness to behold the fight.

QUEEN MARGARET.
Ay, good my lord; for purposely therefore
Left I the court to see this quarrel tried.

KING HENRY.
I’ God’s name, see the lists and all things fit.
Here let them end it, and God defend the right!

YORK.
I never saw a fellow worse bested,
Or more afraid to fight, than is the appellant,
The servant of his armourer, my lords.

Enter at one door Horner the armourer, and his Neighbours, drinking to
him so much that he is drunk; and he enters with a drum before him and
his staff with a sandbag fastened to it; and at the other door Peter,
his man, with a drum and sandbag, and Prentices  drinking to him.

1 NEIGHBOUR.
Here, neighbour Horner, I drink to you in a cup of sack; and fear not,
neighbour, you shall do well enough.

2 NEIGHBOUR.
And here, neighbour, here’s a cup of charneco.

3 NEIGHBOUR.
And here’s a pot of good double beer, neighbour. Drink, and fear not
your man.

HORNER.
Let it come, i’ faith, and I’ll pledge you all; and a fig for Peter!

1 PRENTICE.
Here, Peter, I drink to thee, and be not afraid.

2 PRENTICE.
Be merry, Peter, and fear not thy master. Fight for credit of the
prentices.

PETER.
I thank you all. Drink, and pray for me, I pray you, for I think I have
taken my last draught in this world. Here, Robin, an if I die, I give
thee my apron; and, Will, thou shalt have my hammer; and here, Tom,
take all the money that I have. O Lord bless me! I pray God, for I am
never able to deal with my master, he hath learnt so much fence
already.

SALISBURY.
Come, leave your drinking and fall to blows.
Sirrah, what’s thy name?

PETER.
Peter, forsooth.

SALISBURY.
Peter? What more?

PETER.
Thump.

SALISBURY.
Thump! Then see thou thump thy master well.

HORNER.
Masters, I am come hither, as it were, upon my man’s instigation, to
prove him a knave and myself an honest man; and touching the Duke of
York, I will take my death I never meant him any ill, nor the King, nor
the Queen; and therefore, Peter, have at thee with a downright blow!

YORK.
Dispatch! This knave’s tongue begins to double.
Sound, trumpets. Alarum to the combatants!

[_They fight, and Peter strikes him down._]

HORNER.
Hold, Peter, hold! I confess, I confess treason.

[_Dies._]

YORK.
Take away his weapon.—Fellow, thank God and the good wine in thy
master’s way.

PETER.
O God, have I overcome mine enemies in this presence? O Peter, thou
hast prevailed in right!

KING HENRY.
Go, take hence that traitor from our sight,
For by his death we do perceive his guilt.
And God in justice hath revealed to us
The truth and innocence of this poor fellow,
Which he had thought to have murdered wrongfully.
Come, fellow, follow us for thy reward.

[_Sound a flourish. Exeunt._]

SCENE IV. A Street

Enter Gloucester and his Servingmen in mourning cloaks.

GLOUCESTER.
Thus sometimes hath the brightest day a cloud,
And after summer evermore succeeds
Barren winter, with his wrathful nipping cold;
So cares and joys abound, as seasons fleet.
Sirs, what’s o’clock?

SERVINGMEN.
Ten, my lord.

GLOUCESTER.
Ten is the hour that was appointed me
To watch the coming of my punished duchess.
Uneath may she endure the flinty streets,
To tread them with her tender-feeling feet.
Sweet Nell, ill can thy noble mind abrook
The abject people gazing on thy face
With envious looks, laughing at thy shame,
That erst did follow thy proud chariot wheels
When thou didst ride in triumph through the streets.
But, soft! I think she comes; and I’ll prepare
My tear-stained eyes to see her miseries.

Enter the Duchess of Gloucester in a white sheet, and a taper burning
in her hand; with Sir John Stanley, the Sheriff, and Officers.

SERVINGMEN.
So please your Grace, we’ll take her from the sheriff.

GLOUCESTER.
No, stir not for your lives; let her pass by.

ELEANOR.
Come you, my lord, to see my open shame?
Now thou dost penance too. Look how they gaze!
See how the giddy multitude do point,
And nod their heads, and throw their eyes on thee.
Ah, Gloucester, hide thee from their hateful looks,
And, in thy closet pent up, rue my shame,
And ban thine enemies, both mine and thine!

GLOUCESTER.
Be patient, gentle Nell, forget this grief.

ELEANOR.
Ah, Gloucester, teach me to forget myself!
For whilst I think I am thy married wife
And thou a prince, Protector of this land,
Methinks I should not thus be led along,
Mailed up in shame, with papers on my back,
And followed with a rabble that rejoice
To see my tears and hear my deep-fet groans.
The ruthless flint doth cut my tender feet,
And when I start, the envious people laugh
And bid me be advised how I tread.
Ah, Humphrey, can I bear this shameful yoke?
Trowest thou that e’er I’ll look upon the world,
Or count them happy that enjoy the sun?
No, dark shall be my light and night my day;
To think upon my pomp shall be my hell.
Sometimes I’ll say, I am Duke Humphrey’s wife,
And he a prince and ruler of the land;
Yet so he ruled and such a prince he was
As he stood by whilst I, his forlorn duchess,
Was made a wonder and a pointing-stock
To every idle rascal follower.
But be thou mild and blush not at my shame,
Nor stir at nothing till the axe of death
Hang over thee, as, sure, it shortly will.
For Suffolk, he that can do all in all
With her that hateth thee and hates us all,
And York and impious Beaufort, that false priest,
Have all limed bushes to betray thy wings;
And fly thou how thou canst, they’ll tangle thee.
But fear not thou until thy foot be snared,
Nor never seek prevention of thy foes.

GLOUCESTER.
Ah, Nell, forbear! Thou aimest all awry.
I must offend before I be attainted;
And had I twenty times so many foes,
And each of them had twenty times their power,
All these could not procure me any scathe
So long as I am loyal, true, and crimeless.
Wouldst have me rescue thee from this reproach?
Why, yet thy scandal were not wiped away,
But I in danger for the breach of law.
Thy greatest help is quiet, gentle Nell.
I pray thee, sort thy heart to patience;
These few days’ wonder will be quickly worn.

Enter a Herald.

HERALD.
I summon your grace to his majesty’s parliament,
Holden at Bury the first of this next month.

GLOUCESTER.
And my consent ne’er asked herein before?
This is close dealing. Well, I will be there.

[_Exit Herald._]

My Nell, I take my leave; and, master sheriff,
Let not her penance exceed the King’s commission.

SHERIFF.
An ’t please your grace, here my commission stays,
And Sir John Stanley is appointed now
To take her with him to the Isle of Man.

GLOUCESTER.
Must you, Sir John, protect my lady here?

STANLEY.
So am I given in charge, may ’t please your grace.

GLOUCESTER.
Entreat her not the worse in that I pray
You use her well. The world may laugh again,
And I may live to do you kindness if
You do it her. And so, Sir John, farewell.

ELEANOR.
What, gone, my lord, and bid me not farewell?

GLOUCESTER.
Witness my tears, I cannot stay to speak.

[_Exeunt Gloucester and Servingmen._]

ELEANOR.
Art thou gone too? All comfort go with thee,
For none abides with me; my joy is death;
Death, at whose name I oft have been afeard,
Because I wished this world’s eternity.
Stanley, I prithee, go, and take me hence,
I care not whither, for I beg no favour,
Only convey me where thou art commanded.

STANLEY.
Why, madam, that is to the Isle of Man,
There to be used according to your state.

ELEANOR.
That’s bad enough, for I am but reproach;
And shall I then be used reproachfully?

STANLEY.
Like to a duchess, and Duke Humphrey’s lady;
According to that state you shall be used.

ELEANOR.
Sheriff, farewell, and better than I fare,
Although thou hast been conduct of my shame.

SHERIFF.
It is my office; and, madam, pardon me.

ELEANOR.
Ay, ay, farewell; thy office is discharged.
Come, Stanley, shall we go?

STANLEY.
Madam, your penance done, throw off this sheet,
And go we to attire you for our journey.

ELEANOR.
My shame will not be shifted with my sheet,
No, it will hang upon my richest robes
And show itself, attire me how I can.
Go, lead the way, I long to see my prison.

[_Exeunt._]




ACT III

SCENE I. The Abbey at Bury St. Edmund’s


Sound a sennet. Enter the King, the Queen, Cardinal Beaufort, Suffolk,
York, Buckingham, Salisbury and Warwick to the Parliament.

KING HENRY.
I muse my Lord of Gloucester is not come.
’Tis not his wont to be the hindmost man,
Whate’er occasion keeps him from us now.

QUEEN MARGARET.
Can you not see, or will ye not observe
The strangeness of his altered countenance?
With what a majesty he bears himself,
How insolent of late he is become,
How proud, how peremptory, and unlike himself?
We know the time since he was mild and affable;
And if we did but glance a far-off look,
Immediately he was upon his knee,
That all the court admired him for submission.
But meet him now, and be it in the morn
When everyone will give the time of day,
He knits his brow and shows an angry eye
And passeth by with stiff unbowed knee,
Disdaining duty that to us belongs.
Small curs are not regarded when they grin,
But great men tremble when the lion roars;
And Humphrey is no little man in England.
First note that he is near you in descent,
And should you fall, he is the next will mount.
Me seemeth then it is no policy,
Respecting what a rancorous mind he bears
And his advantage following your decease,
That he should come about your royal person
Or be admitted to your Highness’ Council.
By flattery hath he won the commons’ hearts;
And when he please to make commotion,
’Tis to be feared they all will follow him.
Now ’tis the spring, and weeds are shallow-rooted;
Suffer them now, and they’ll o’ergrow the garden
And choke the herbs for want of husbandry.
The reverent care I bear unto my lord
Made me collect these dangers in the Duke.
If it be fond, can it a woman’s fear;
Which fear if better reasons can supplant,
I will subscribe and say I wronged the Duke.
My Lord of Suffolk, Buckingham, and York,
Reprove my allegation if you can,
Or else conclude my words effectual.

SUFFOLK.
Well hath your highness seen into this Duke;
And, had I first been put to speak my mind,
I think I should have told your grace’s tale.
The Duchess by his subornation,
Upon my life, began her devilish practices;
Or, if he were not privy to those faults,
Yet, by reputing of his high descent,
As next the King he was successive heir,
And such high vaunts of his nobility—
Did instigate the bedlam brain-sick Duchess
By wicked means to frame our sovereign’s fall.
Smooth runs the water where the brook is deep,
And in his simple show he harbours treason.
The fox barks not when he would steal the lamb.
No, no, my sovereign, Gloucester is a man
Unsounded yet and full of deep deceit.

CARDINAL.
Did he not, contrary to form of law,
Devise strange deaths for small offences done?

YORK.
And did he not, in his protectorship,
Levy great sums of money through the realm
For soldiers’ pay in France, and never sent it?
By means whereof the towns each day revolted.

BUCKINGHAM.
Tut, these are petty faults to faults unknown,
Which time will bring to light in smooth Duke Humphrey.

KING HENRY.
My lords, at once: the care you have of us
To mow down thorns that would annoy our foot
Is worthy praise; but, shall I speak my conscience,
Our kinsman Gloucester is as innocent
From meaning treason to our royal person
As is the sucking lamb or harmless dove.
The Duke is virtuous, mild, and too well given
To dream on evil or to work my downfall.

QUEEN MARGARET.
Ah, what’s more dangerous than this fond affiance?
Seems he a dove? His feathers are but borrowed,
For he’s disposed as the hateful raven.
Is he a lamb? His skin is surely lent him,
For he’s inclined as is the ravenous wolves.
Who cannot steal a shape that means deceit?
Take heed, my lord; the welfare of us all
Hangs on the cutting short that fraudful man.

Enter Somerset.

SOMERSET.
All health unto my gracious sovereign!

KING HENRY.
Welcome, Lord Somerset. What news from France?

SOMERSET.
That all your interest in those territories
Is utterly bereft you; all is lost.

KING HENRY.
Cold news, Lord Somerset; but God’s will be done.

YORK.
[_Aside_.] Cold news for me, for I had hope of France
As firmly as I hope for fertile England.
Thus are my blossoms blasted in the bud,
And caterpillars eat my leaves away;
But I will remedy this gear ere long,
Or sell my title for a glorious grave.

Enter Gloucester.

GLOUCESTER.
All happiness unto my lord the King!
Pardon, my liege, that I have staid so long.

SUFFOLK.
Nay, Gloucester, know that thou art come too soon,
Unless thou wert more loyal than thou art.
I do arrest thee of high treason here.

GLOUCESTER.
Well, Suffolk, thou shalt not see me blush,
Nor change my countenance for this arrest.
A heart unspotted is not easily daunted.
The purest spring is not so free from mud
As I am clear from treason to my sovereign.
Who can accuse me? Wherein am I guilty?

YORK.
’Tis thought, my lord, that you took bribes of France,
And, being Protector, stayed the soldiers’ pay,
By means whereof his highness hath lost France.

GLOUCESTER.
Is it but thought so? What are they that think it?
I never robbed the soldiers of their pay,
Nor ever had one penny bribe from France.
So help me God, as I have watched the night,
Ay, night by night, in studying good for England!
That doit that e’er I wrested from the King,
Or any groat I hoarded to my use,
Be brought against me at my trial day!
No, many a pound of mine own proper store,
Because I would not tax the needy commons,
Have I dispursed to the garrisons
And never asked for restitution.

CARDINAL.
It serves you well, my lord, to say so much.

GLOUCESTER.
I say no more than truth, so help me God!

YORK.
In your protectorship you did devise
Strange tortures for offenders never heard of,
That England was defamed by tyranny.

GLOUCESTER.
Why, ’tis well known that, whiles I was Protector,
Pity was all the fault that was in me;
For I should melt at an offender’s tears,
And lowly words were ransom for their fault.
Unless it were a bloody murderer,
Or foul felonious thief that fleeced poor passengers,
I never gave them condign punishment.
Murder indeed, that bloody sin, I tortured
Above the felon or what trespass else.

SUFFOLK.
My lord, these faults are easy, quickly answered;
But mightier crimes are laid unto your charge
Whereof you cannot easily purge yourself.
I do arrest you in his highness’ name,
And here commit you to my Lord Cardinal
To keep until your further time of trial.

KING HENRY.
My Lord of Gloucester, ’tis my special hope
That you will clear yourself from all suspense.
My conscience tells me you are innocent.

GLOUCESTER.
Ah, gracious lord, these days are dangerous.
Virtue is choked with foul ambition,
And charity chased hence by rancour’s hand;
Foul subornation is predominant,
And equity exiled your highness’ land.
I know their complot is to have my life;
And if my death might make this island happy
And prove the period of their tyranny,
I would expend it with all willingness.
But mine is made the prologue to their play;
For thousands more, that yet suspect no peril,
Will not conclude their plotted tragedy.
Beaufort’s red sparkling eyes blab his heart’s malice,
And Suffolk’s cloudy brow his stormy hate;
Sharp Buckingham unburdens with his tongue
The envious load that lies upon his heart;
And dogged York, that reaches at the moon,
Whose overweening arm I have plucked back,
By false accuse doth level at my life.
And you, my sovereign lady, with the rest,
Causeless have laid disgraces on my head
And with your best endeavour have stirred up
My liefest liege to be mine enemy.
Ay, all of you have laid your heads together—
Myself had notice of your conventicles—
And all to make away my guiltless life.
I shall not want false witness to condemn me,
Nor store of treasons to augment my guilt.
The ancient proverb will be well effected:
“A staff is quickly found to beat a dog.”

CARDINAL.
My liege, his railing is intolerable.
If those that care to keep your royal person
From treason’s secret knife and traitor’s rage
Be thus upbraided, chid, and rated at,
And the offender granted scope of speech,
’Twill make them cool in zeal unto your grace.

SUFFOLK.
Hath he not twit our sovereign lady here
With ignominious words, though clerkly couched,
As if she had suborned some to swear
False allegations to o’erthrow his state?

QUEEN MARGARET.
But I can give the loser leave to chide.

GLOUCESTER.
Far truer spoke than meant. I lose, indeed.
Beshrew the winners, for they played me false!
And well such losers may have leave to speak.

BUCKINGHAM.
He’ll wrest the sense and hold us here all day.
Lord Cardinal, he is your prisoner.

CARDINAL.
Sirs, take away the Duke, and guard him sure.

GLOUCESTER.
Ah, thus King Henry throws away his crutch
Before his legs be firm to bear his body.
Thus is the shepherd beaten from thy side,
And wolves are gnarling who shall gnaw thee first.
Ah, that my fear were false; ah, that it were!
For, good King Henry, thy decay I fear.

[_Exit Gloucester, guarded._]

KING HENRY.
My lords, what to your wisdoms seemeth best
Do, or undo, as if ourself were here.

QUEEN MARGARET.
What, will your highness leave the parliament?

KING HENRY.
Ay, Margaret; my heart is drowned with grief,
Whose flood begins to flow within mine eyes,
My body round engirt with misery;
For what’s more miserable than discontent?
Ah, uncle Humphrey, in thy face I see
The map of honour, truth, and loyalty;
And yet, good Humphrey, is the hour to come
That e’er I proved thee false or feared thy faith.
What louring star now envies thy estate
That these great lords and Margaret our Queen
Do seek subversion of thy harmless life?
Thou never didst them wrong nor no man wrong.
And as the butcher takes away the calf
And binds the wretch and beats it when it strains,
Bearing it to the bloody slaughterhouse,
Even so remorseless have they borne him hence;
And as the dam runs lowing up and down,
Looking the way her harmless young one went,
And can do naught but wail her darling’s loss,
Even so myself bewails good Gloucester’s case
With sad unhelpful tears, and with dimmed eyes
Look after him, and cannot do him good,
So mighty are his vowed enemies.
His fortunes I will weep and ’twixt each groan
Say “Who’s a traitor? Gloucester he is none.”

[_Exeunt all but Queen, Cardinal Beaufort, Suffolk and York; Somerset
remains apart._]

QUEEN MARGARET.
Free lords, cold snow melts with the sun’s hot beams.
Henry my lord is cold in great affairs,
Too full of foolish pity; and Gloucester’s show
Beguiles him, as the mournful crocodile
With sorrow snares relenting passengers,
Or as the snake, rolled in a flowering bank,
With shining checkered slough, doth sting a child
That for the beauty thinks it excellent.
Believe me, lords, were none more wise than I—
And yet herein I judge mine own wit good—
This Gloucester should be quickly rid the world,
To rid us from the fear we have of him.

CARDINAL.
That he should die is worthy policy,
But yet we want a colour for his death.
’Tis meet he be condemned by course of law.

SUFFOLK.
But, in my mind, that were no policy.
The King will labour still to save his life,
The commons haply rise to save his life,
And yet we have but trivial argument,
More than mistrust, that shows him worthy death.

YORK.
So that, by this, you would not have him die.

SUFFOLK.
Ah, York, no man alive so fain as I!

YORK.
’Tis York that hath more reason for his death.
But, my Lord Cardinal, and you, my Lord of Suffolk,
Say as you think, and speak it from your souls:
Were ’t not all one an empty eagle were set
To guard the chicken from a hungry kite,
As place Duke Humphrey for the King’s Protector?

QUEEN MARGARET.
So the poor chicken should be sure of death.

SUFFOLK.
Madam, ’tis true; and were ’t not madness then
To make the fox surveyor of the fold,
Who being accused a crafty murderer,
His guilt should be but idly posted over
Because his purpose is not executed?
No, let him die in that he is a fox,
By nature proved an enemy to the flock,
Before his chaps be stained with crimson blood,
As Humphrey, proved by reasons, to my liege.
And do not stand on quillets how to slay him;
Be it by gins, by snares, by subtlety,
Sleeping or waking, ’tis no matter how,
So he be dead; for that is good deceit
Which mates him first that first intends deceit.

QUEEN MARGARET.
Thrice-noble Suffolk, ’tis resolutely spoke.

SUFFOLK.
Not resolute, except so much were done,
For things are often spoke and seldom meant;
But that my heart accordeth with my tongue,
Seeing the deed is meritorious,
And to preserve my sovereign from his foe,
Say but the word, and I will be his priest.

CARDINAL.
But I would have him dead, my Lord of Suffolk,
Ere you can take due orders for a priest.
Say you consent and censure well the deed,
And I’ll provide his executioner.
I tender so the safety of my liege.

SUFFOLK.
Here is my hand, the deed is worthy doing.

QUEEN MARGARET.
And so say I.

YORK.
And I. And now we three have spoke it,
It skills not greatly who impugns our doom.

Enter a Post.

POST.
Great lords, from Ireland am I come amain
To signify that rebels there are up
And put the Englishmen unto the sword.
Send succours, lords, and stop the rage betime,
Before the wound do grow uncurable;
For, being green, there is great hope of help.

CARDINAL.
A breach that craves a quick expedient stop!
What counsel give you in this weighty cause?

YORK.
That Somerset be sent as regent thither.
’Tis meet that lucky ruler be employed;
Witness the fortune he hath had in France.

SOMERSET.
If York, with all his far-fet policy,
Had been the regent there instead of me,
He never would have stayed in France so long.

YORK.
No, not to lose it all as thou hast done.
I rather would have lost my life betimes
Than bring a burden of dishonour home
By staying there so long till all were lost.
Show me one scar charactered on thy skin;
Men’s flesh preserved so whole do seldom win.

QUEEN MARGARET.
Nay then, this spark will prove a raging fire
If wind and fuel be brought to feed it with.
No more, good York. Sweet Somerset, be still.
Thy fortune, York, hadst thou been regent there,
Might happily have proved far worse than his.

YORK.
What, worse than naught? Nay, then a shame take all!

SOMERSET.
And, in the number, thee that wishest shame!

CARDINAL.
My Lord of York, try what your fortune is.
Th’ uncivil kerns of Ireland are in arms
And temper clay with blood of Englishmen.
To Ireland will you lead a band of men,
Collected choicely, from each county some,
And try your hap against the Irishmen?

YORK.
I will, my lord, so please his majesty.

SUFFOLK.
Why, our authority is his consent,
And what we do establish he confirms.
Then, noble York, take thou this task in hand.

YORK.
I am content. Provide me soldiers, lords,
Whiles I take order for mine own affairs.

SUFFOLK.
A charge, Lord York, that I will see performed.
But now return we to the false Duke Humphrey.

CARDINAL.
No more of him; for I will deal with him
That henceforth he shall trouble us no more.
And so break off; the day is almost spent.
Lord Suffolk, you and I must talk of that event.

YORK.
My Lord of Suffolk, within fourteen days
At Bristol I expect my soldiers;
For there I’ll ship them all for Ireland.

SUFFOLK.
I’ll see it truly done, my Lord of York.

[_Exeunt all but York._]

YORK.
Now, York, or never, steel thy fearful thoughts,
And change misdoubt to resolution.
Be that thou hop’st to be, or what thou art
Resign to death; it is not worth th’ enjoying.
Let pale-faced fear keep with the mean-born man
And find no harbour in a royal heart.
Faster than springtime showers comes thought on thought,
And not a thought but thinks on dignity.
My brain, more busy than the labouring spider
Weaves tedious snares to trap mine enemies.
Well, nobles, well, ’tis politicly done,
To send me packing with an host of men;
I fear me you but warm the starved snake,
Who, cherished in your breasts, will sting your hearts.
’Twas men I lacked, and you will give them me;
I take it kindly, yet be well assured
You put sharp weapons in a madman’s hands.
Whiles I in Ireland nourish a mighty band,
I will stir up in England some black storm
Shall blow ten thousand souls to heaven or hell;
And this fell tempest shall not cease to rage
Until the golden circuit on my head,
Like to the glorious sun’s transparent beams,
Do calm the fury of this mad-bred flaw.
And for a minister of my intent,
I have seduced a headstrong Kentishman,
John Cade of Ashford,
To make commotion, as full well he can,
Under the title of John Mortimer.
In Ireland have I seen this stubborn Cade
Oppose himself against a troop of kerns,
And fought so long till that his thighs with darts
Were almost like a sharp-quilled porpentine;
And in the end being rescued, I have seen
Him caper upright like a wild Morisco,
Shaking the bloody darts as he his bells.
Full often, like a shag-haired crafty kern,
Hath he conversed with the enemy,
And undiscovered come to me again
And given me notice of their villainies.
This devil here shall be my substitute;
For that John Mortimer, which now is dead,
In face, in gait, in speech, he doth resemble.
By this I shall perceive the commons’ mind,
How they affect the house and claim of York.
Say he be taken, racked, and tortured,
I know no pain they can inflict upon him
Will make him say I moved him to those arms.
Say that he thrive, as ’tis great like he will,
Why then from Ireland come I with my strength
And reap the harvest which that rascal sowed.
For Humphrey being dead, as he shall be,
And Henry put apart, the next for me.

[_Exit._]

SCENE II. Bury St. Edmund’s. A Room of State

Enter two or three Murderers running over the stage, from the murder of
Duke Humphrey.

1 MURDERER.
Run to my Lord of Suffolk; let him know
We have dispatched the Duke as he commanded.

2 MURDERER.
O that it were to do! What have we done?
Didst ever hear a man so penitent?

Enter Suffolk.

1 MURDERER.
Here comes my lord.

SUFFOLK.
Now, sirs, have you dispatched this thing?

1 MURDERER.
Ay, my good lord, he’s dead.

SUFFOLK.
Why, that’s well said. Go, get you to my house;
I will reward you for this venturous deed.
The King and all the peers are here at hand.
Have you laid fair the bed? Is all things well,
According as I gave directions?

1 MURDERER.
’Tis, my good lord.

SUFFOLK.
Away, be gone!

[_Exeunt Murderers._]

Sound trumpets. Enter the King, the Queen, Cardinal Beaufort, Somerset
with attendants.

KING HENRY.
Go, call our uncle to our presence straight;
Say we intend to try his grace today
If he be guilty, as ’tis published.

SUFFOLK.
I’ll call him presently, my noble lord.

[_Exit._]

KING HENRY.
Lords, take your places; and, I pray you all,
Proceed no straiter ’gainst our uncle Gloucester
Than from true evidence of good esteem
He be approved in practice culpable.

QUEEN MARGARET.
God forbid any malice should prevail
That faultless may condemn a nobleman!
Pray God he may acquit him of suspicion!

KING HENRY.
I thank thee, Meg; these words content me much.

Enter Suffolk.

How now? Why look’st thou pale? Why tremblest thou?
Where is our uncle? What’s the matter, Suffolk?

SUFFOLK.
Dead in his bed, my lord; Gloucester is dead.

QUEEN MARGARET.
Marry, God forfend!

CARDINAL.
God’s secret judgment! I did dream tonight
The Duke was dumb and could not speak a word.

[_The King swoons._]

QUEEN MARGARET.
How fares my lord? Help, lords! the King is dead.

SOMERSET.
Rear up his body; wring him by the nose.

QUEEN MARGARET.
Run, go, help, help! O Henry, ope thine eyes!

SUFFOLK.
He doth revive again. Madam, be patient.

KING HENRY.
O heavenly God!

QUEEN MARGARET.
How fares my gracious lord?

SUFFOLK.
Comfort, my sovereign! Gracious Henry, comfort!

KING HENRY.
What, doth my Lord of Suffolk comfort me?
Came he right now to sing a raven’s note,
Whose dismal tune bereft my vital powers,
And thinks he that the chirping of a wren,
By crying comfort from a hollow breast,
Can chase away the first-conceived sound?
Hide not thy poison with such sugared words;
Lay not thy hands on me. Forbear, I say!
Their touch affrights me as a serpent’s sting.
Thou baleful messenger, out of my sight!
Upon thy eyeballs murderous tyranny
Sits in grim majesty to fright the world.
Look not upon me, for thine eyes are wounding.
Yet do not go away; come, basilisk,
And kill the innocent gazer with thy sight.
For in the shade of death I shall find joy,
In life but double death, now Gloucester’s dead.

QUEEN MARGARET.
Why do you rate my Lord of Suffolk thus?
Although the Duke was enemy to him,
Yet he most Christian-like laments his death.
And for myself, foe as he was to me,
Might liquid tears or heart-offending groans
Or blood-consuming sighs recall his life,
I would be blind with weeping, sick with groans,
Look pale as primrose with blood-drinking sighs,
And all to have the noble Duke alive.
What know I how the world may deem of me?
For it is known we were but hollow friends.
It may be judged I made the Duke away;
So shall my name with slander’s tongue be wounded
And princes’ courts be filled with my reproach.
This get I by his death. Ay me, unhappy!
To be a queen, and crowned with infamy!

KING HENRY.
Ah, woe is me for Gloucester, wretched man!

QUEEN MARGARET.
Be woe for me, more wretched than he is.
What, dost thou turn away and hide thy face?
I am no loathsome leper. Look on me.
What, art thou, like the adder, waxen deaf?
Be poisonous too and kill thy forlorn Queen.
Is all thy comfort shut in Gloucester’s tomb?
Why, then, Dame Margaret was ne’er thy joy.
Erect his statue and worship it,
And make my image but an alehouse sign.
Was I for this nigh wracked upon the sea
And twice by awkward wind from England’s bank
Drove back again unto my native clime?
What boded this, but well forewarning wind
Did seem to say “Seek not a scorpion’s nest,
Nor set no footing on this unkind shore?”
What did I then, but cursed the gentle gusts
And he that loosed them forth their brazen caves
And bid them blow towards England’s blessed shore
Or turn our stern upon a dreadful rock?
Yet Aeolus would not be a murderer,
But left that hateful office unto thee.
The pretty-vaulting sea refused to drown me,
Knowing that thou wouldst have me drowned on shore
With tears as salt as sea, through thy unkindness.
The splitting rocks cowered in the sinking sands
And would not dash me with their ragged sides,
Because thy flinty heart, more hard than they,
Might in thy palace perish Margaret.
As far as I could ken thy chalky cliffs,
When from thy shore the tempest beat us back,
I stood upon the hatches in the storm,
And when the dusky sky began to rob
My earnest-gaping sight of thy land’s view,
I took a costly jewel from my neck—
A heart it was, bound in with diamonds—
And threw it towards thy land. The sea received it,
And so I wished thy body might my heart.
And even with this I lost fair England’s view,
And bid mine eyes be packing with my heart,
And called them blind and dusky spectacles,
For losing ken of Albion’s wished coast.
How often have I tempted Suffolk’s tongue,
The agent of thy foul inconstancy,
To sit and witch me, as Ascanius did
When he to madding Dido would unfold
His father’s acts commenced in burning Troy!
Am I not witched like her? Or thou not false like him?
Ay me, I can no more! Die, Margaret,
For Henry weeps that thou dost live so long.

Noise within. Enter Warwick, Salisbury and many Commons.

WARWICK.
It is reported, mighty sovereign,
That good Duke Humphrey traitorously is murdered
By Suffolk and the Cardinal Beaufort’s means.
The commons, like an angry hive of bees
That want their leader, scatter up and down
And care not who they sting in his revenge.
Myself have calmed their spleenful mutiny,
Until they hear the order of his death.

KING HENRY.
That he is dead, good Warwick, ’tis too true;
But how he died God knows, not Henry.
Enter his chamber, view his breathless corpse,
And comment then upon his sudden death.

WARWICK.
That shall I do, my liege.—Stay, Salisbury,
With the rude multitude till I return.

[_Warwick exits through one door; Salisbury and Commons exit through
another._]

KING HENRY.
O Thou that judgest all things, stay my thoughts,
My thoughts that labour to persuade my soul
Some violent hands were laid on Humphrey’s life.
If my suspect be false, forgive me, God,
For judgment only doth belong to Thee.
Fain would I go to chafe his paly lips
With twenty thousand kisses, and to drain
Upon his face an ocean of salt tears,
To tell my love unto his dumb deaf trunk,
And with my fingers feel his hand unfeeling;
But all in vain are these mean obsequies.
And to survey his dead and earthy image,
What were it but to make my sorrow greater?

Enter Warwick and others, bearing Gloucester’s body on a bed.

WARWICK.
Come hither, gracious sovereign, view this body.

KING HENRY.
That is to see how deep my grave is made,
For with his soul fled all my worldly solace;
For seeing him, I see my life in death.

WARWICK.
As surely as my soul intends to live
With that dread King that took our state upon Him
To free us from His Father’s wrathful curse,
I do believe that violent hands were laid
Upon the life of this thrice-famed duke.

SUFFOLK.
A dreadful oath, sworn with a solemn tongue!
What instance gives Lord Warwick for his vow?

WARWICK.
See how the blood is settled in his face.
Oft have I seen a timely-parted ghost,
Of ashy semblance, meagre, pale, and bloodless,
Being all descended to the labouring heart,
Who, in the conflict that it holds with death,
Attracts the same for aidance ’gainst the enemy,
Which with the heart there cools and ne’er returneth
To blush and beautify the cheek again.
But see, his face is black and full of blood,
His eyeballs further out than when he lived,
Staring full ghastly like a strangled man;
His hair upreared, his nostrils stretched with struggling,
His hands abroad displayed, as one that grasped
And tugged for life and was by strength subdued.
Look, on the sheets his hair, you see, is sticking;
His well-proportioned beard made rough and rugged,
Like to the summer’s corn by tempest lodged.
It cannot be but he was murdered here;
The least of all these signs were probable.

SUFFOLK.
Why, Warwick, who should do the Duke to death?
Myself and Beaufort had him in protection,
And we, I hope, sir, are no murderers.

WARWICK.
But both of you were vowed Duke Humphrey’s foes,
And you, forsooth, had the good Duke to keep.
’Tis like you would not feast him like a friend,
And ’tis well seen he found an enemy.

QUEEN MARGARET.
Then you, belike, suspect these noblemen
As guilty of Duke Humphrey’s timeless death.

WARWICK.
Who finds the heifer dead and bleeding fresh
And sees fast by a butcher with an axe,
But will suspect ’twas he that made the slaughter?
Who finds the partridge in the puttock’s nest
But may imagine how the bird was dead,
Although the kite soar with unbloodied beak?
Even so suspicious is this tragedy.

QUEEN MARGARET.
Are you the butcher, Suffolk? Where’s your knife?
Is Beaufort termed a kite? Where are his talons?

SUFFOLK.
I wear no knife to slaughter sleeping men,
But here’s a vengeful sword, rusted with ease,
That shall be scoured in his rancorous heart
That slanders me with murder’s crimson badge.
Say, if thou dar’st, proud Lord of Warwickshire,
That I am faulty in Duke Humphrey’s death.

[_Exeunt Cardinal, Somerset and others._]

WARWICK.
What dares not Warwick, if false Suffolk dare him?

QUEEN MARGARET.
He dares not calm his contumelious spirit,
Nor cease to be an arrogant controller,
Though Suffolk dare him twenty thousand times.

WARWICK.
Madam, be still, with reverence may I say;
For every word you speak in his behalf
Is slander to your royal dignity.

SUFFOLK.
Blunt-witted lord, ignoble in demeanour!
If ever lady wronged her lord so much,
Thy mother took into her blameful bed
Some stern untutored churl, and noble stock
Was graft with crab-tree slip, whose fruit thou art,
And never of the Nevilles’ noble race.

WARWICK.
But that the guilt of murder bucklers thee
And I should rob the deathsman of his fee,
Quitting thee thereby of ten thousand shames,
And that my sovereign’s presence makes me mild,
I would, false murderous coward, on thy knee
Make thee beg pardon for thy passed speech
And say it was thy mother that thou meant’st,
That thou thyself wast born in bastardy;
And after all this fearful homage done,
Give thee thy hire and send thy soul to hell,
Pernicious blood-sucker of sleeping men!

SUFFOLK.
Thou shalt be waking while I shed thy blood,
If from this presence thou dar’st go with me.

WARWICK.
Away even now, or I will drag thee hence.
Unworthy though thou art, I’ll cope with thee
And do some service to Duke Humphrey’s ghost.

[_Exeunt Suffolk and Warwick._]

KING HENRY.
What stronger breastplate than a heart untainted?
Thrice is he armed that hath his quarrel just,
And he but naked, though locked up in steel,
Whose conscience with injustice is corrupted.

[_A noise within._]

QUEEN MARGARET.
What noise is this?

Enter Suffolk and Warwick with their weapons drawn.

KING HENRY.
Why, how now, lords? Your wrathful weapons drawn
Here in our presence? Dare you be so bold?
Why, what tumultuous clamour have we here?

SUFFOLK.
The traitorous Warwick with the men of Bury
Set all upon me, mighty sovereign.

Enter Salisbury.

SALISBURY.
[_To the Commons, entering_.]
Sirs, stand apart; the King shall know your mind.—
Dread lord, the commons send you word by me,
Unless Lord Suffolk straight be done to death,
Or banished fair England’s territories,
They will by violence tear him from your palace
And torture him with grievous lingering death.
They say, by him the good Duke Humphrey died;
They say, in him they fear your highness’ death;
And mere instinct of love and loyalty,
Free from a stubborn opposite intent,
As being thought to contradict your liking,
Makes them thus forward in his banishment.
They say, in care of your most royal person,
That if your highness should intend to sleep
And charge that no man should disturb your rest,
In pain of your dislike or pain of death,
Yet, notwithstanding such a strait edict,
Were there a serpent seen, with forked tongue,
That slyly glided towards your majesty,
It were but necessary you were waked,
Lest, being suffered in that harmful slumber,
The mortal worm might make the sleep eternal.
And therefore do they cry, though you forbid,
That they will guard you, whe’er you will or no,
From such fell serpents as false Suffolk is,
With whose envenomed and fatal sting
Your loving uncle, twenty times his worth,
They say, is shamefully bereft of life.

COMMONS.
[_Within_.] An answer from the King, my Lord of Salisbury!

SUFFOLK.
’Tis like the commons, rude unpolished hinds,
Could send such message to their sovereign.
But you, my lord, were glad to be employed,
To show how quaint an orator you are.
But all the honour Salisbury hath won
Is that he was the lord ambassador
Sent from a sort of tinkers to the King.

COMMONS.
[_Within_.] An answer from the King, or we will all break in!

KING HENRY.
Go, Salisbury, and tell them all from me,
I thank them for their tender loving care;
And had I not been cited so by them,
Yet did I purpose as they do entreat.
For, sure, my thoughts do hourly prophesy
Mischance unto my state by Suffolk’s means.
And therefore, by His majesty I swear,
Whose far unworthy deputy I am,
He shall not breathe infection in this air
But three days longer, on the pain of death.

[_Exit Salisbury._]

QUEEN MARGARET.
O Henry, let me plead for gentle Suffolk!

KING HENRY.
Ungentle Queen, to call him gentle Suffolk!
No more, I say; if thou dost plead for him,
Thou wilt but add increase unto my wrath.
Had I but said, I would have kept my word;
But when I swear, it is irrevocable.
If, after three days’ space, thou here be’st found
On any ground that I am ruler of,
The world shall not be ransom for thy life.
Come, Warwick, come, good Warwick, go with me;
I have great matters to impart to thee.

[_Exeunt all but Queen and Suffolk._]

QUEEN MARGARET.
Mischance and sorrow go along with you!
Heart’s discontent and sour affliction
Be playfellows to keep you company!
There’s two of you; the devil make a third!
And threefold vengeance tend upon your steps!

SUFFOLK.
Cease, gentle Queen, these execrations,
And let thy Suffolk take his heavy leave.

QUEEN MARGARET.
Fie, coward woman and soft-hearted wretch!
Has thou not spirit to curse thine enemies?

SUFFOLK.
A plague upon them! Wherefore should I curse them?
Could curses kill, as doth the mandrake’s groan,
I would invent as bitter searching terms,
As curst, as harsh and horrible to hear,
Delivered strongly through my fixed teeth,
With full as many signs of deadly hate,
As lean-faced Envy in her loathsome cave.
My tongue should stumble in mine earnest words;
Mine eyes should sparkle like the beaten flint;
Mine hair be fixed on end, as one distract;
Ay, every joint should seem to curse and ban;
And even now my burdened heart would break
Should I not curse them. Poison be their drink!
Gall, worse than gall, the daintiest that they taste!
Their sweetest shade a grove of cypress-trees!
Their chiefest prospect murdering basilisks;
Their softest touch as smart as lizards’ stings!
Their music frightful as the serpent’s hiss,
And boding screech-owls make the consort full!
All the foul terrors in dark-seated hell—

QUEEN MARGARET.
Enough, sweet Suffolk; thou torment’st thyself,
And these dread curses, like the sun ’gainst glass,
Or like an overcharged gun, recoil
And turns the force of them upon thyself.

SUFFOLK.
You bade me ban, and will you bid me leave?
Now, by the ground that I am banished from,
Well could I curse away a winter’s night,
Though standing naked on a mountain top
Where biting cold would never let grass grow,
And think it but a minute spent in sport.

QUEEN MARGARET.
O, let me entreat thee cease. Give me thy hand,
That I may dew it with my mournful tears;
Nor let the rain of heaven wet this place
To wash away my woeful monuments.
O, could this kiss be printed in thy hand,
That thou mightst think upon these by the seal,
Through whom a thousand sighs are breathed for thee!
So, get thee gone, that I may know my grief;
’Tis but surmised whiles thou art standing by,
As one that surfeits thinking on a want.
I will repeal thee, or, be well assured,
Adventure to be banished myself;
And banished I am, if but from thee.
Go; speak not to me, even now be gone!
O, go not yet! Even thus two friends condemned
Embrace and kiss and take ten thousand leaves,
Loather a hundred times to part than die.
Yet now farewell, and farewell life with thee.

SUFFOLK.
Thus is poor Suffolk ten times banished,
Once by the King, and three times thrice by thee.
’Tis not the land I care for, wert thou thence.
A wilderness is populous enough,
So Suffolk had thy heavenly company;
For where thou art, there is the world itself,
With every several pleasure in the world;
And where thou art not, desolation.
I can no more. Live thou to joy thy life,
Myself no joy in nought but that thou liv’st.

Enter Vaux.

QUEEN MARGARET.
Whither goes Vaux so fast? What news, I prithee?

VAUX.
To signify unto his majesty
That Cardinal Beaufort is at point of death;
For suddenly a grievous sickness took him,
That makes him gasp and stare and catch the air,
Blaspheming God and cursing men on earth.
Sometime he talks as if Duke Humphrey’s ghost
Were by his side; sometime he calls the King
And whispers to his pillow, as to him,
The secrets of his overcharged soul.
And I am sent to tell his majesty
That even now he cries aloud for him.

QUEEN MARGARET.
Go tell this heavy message to the King.

[_Exit Vaux._]

Ay me! What is this world? What news are these!
But wherefore grieve I at an hour’s poor loss,
Omitting Suffolk’s exile, my soul’s treasure?
Why only, Suffolk, mourn I not for thee,
And with the southern clouds contend in tears,
Theirs for the earth’s increase, mine for my sorrows’?
Now get thee hence. The King, thou know’st, is coming;
If thou be found by me thou art but dead.

SUFFOLK.
If I depart from thee, I cannot live;
And in thy sight to die, what were it else
But like a pleasant slumber in thy lap?
Here could I breathe my soul into the air,
As mild and gentle as the cradle-babe
Dying with mother’s dug between its lips;
Where, from thy sight, I should be raging mad
And cry out for thee to close up mine eyes,
To have thee with thy lips to stop my mouth.
So shouldst thou either turn my flying soul,
Or I should breathe it so into thy body,
And then it lived in sweet Elysium.
To die by thee were but to die in jest;
From thee to die were torture more than death.
O, let me stay, befall what may befall!

QUEEN MARGARET.
Away! Though parting be a fretful corrosive,
It is applied to a deathful wound.
To France, sweet Suffolk! Let me hear from thee,
For whereso’er thou art in this world’s globe
I’ll have an Iris that shall find thee out.

SUFFOLK.
I go.

QUEEN MARGARET.
And take my heart with thee.

SUFFOLK.
A jewel, locked into the woefull’st cask
That ever did contain a thing of worth.
Even as a splitted bark, so sunder we.
This way fall I to death.

QUEEN MARGARET.
This way for me.

[_Exeunt severally._]

SCENE III. A Bedchamber

Enter the King, Salisbury and Warwick, to the Cardinal in bed.

KING HENRY.
How fares my lord? Speak, Beaufort, to thy sovereign.

CARDINAL.
If thou be’st Death, I’ll give thee England’s treasure,
Enough to purchase such another island,
So thou wilt let me live and feel no pain.

KING HENRY.
Ah, what a sign it is of evil life
Where death’s approach is seen so terrible!

WARWICK.
Beaufort, it is thy sovereign speaks to thee.

CARDINAL.
Bring me unto my trial when you will.
Died he not in his bed? Where should he die?
Can I make men live, whe’er they will or no?
O, torture me no more! I will confess.
Alive again? Then show me where he is.
I’ll give a thousand pound to look upon him.
He hath no eyes, the dust hath blinded them.
Comb down his hair; look, look, it stands upright,
Like lime-twigs set to catch my winged soul.
Give me some drink, and bid the apothecary
Bring the strong poison that I bought of him.

KING HENRY.
O Thou eternal mover of the heavens,
Look with a gentle eye upon this wretch!
O, beat away the busy meddling fiend
That lays strong siege unto this wretch’s soul,
And from his bosom purge this black despair!

WARWICK.
See how the pangs of death do make him grin!

SALISBURY.
Disturb him not; let him pass peaceably.

KING HENRY.
Peace to his soul, if God’s good pleasure be!
Lord Cardinal, if thou think’st on heaven’s bliss,
Hold up thy hand, make signal of thy hope.
He dies and makes no sign. O God, forgive him!

WARWICK.
So bad a death argues a monstrous life.

KING HENRY.
Forbear to judge, for we are sinners all.
Close up his eyes, and draw the curtain close,
And let us all to meditation.

[_Exeunt._]




ACT IV

SCENE I. The Coast of Kent


Alarum. Fight at sea. Ordnance goes off. Enter a Lieutenant, Suffolk,
disguised, a prisoner. The Master, a Master’s Mate, Walter Whitmore,
and prisoners.

LIEUTENANT.
The gaudy, blabbing, and remorseful day
Is crept into the bosom of the sea;
And now loud-howling wolves arouse the jades
That drag the tragic melancholy night,
Who, with their drowsy, slow, and flagging wings
Clip dead men’s graves and from their misty jaws
Breathe foul contagious darkness in the air.
Therefore bring forth the soldiers of our prize;
For, whilst our pinnace anchors in the Downs,
Here shall they make their ransom on the sand,
Or with their blood stain this discoloured shore.
Master, this prisoner freely give I thee,
And thou that art his mate, make boot of this;
The other, Walter Whitmore, is thy share.

1 GENTLEMAN.
What is my ransom, master? Let me know.

MASTER.
A thousand crowns, or else lay down your head.

MATE.
And so much shall you give, or off goes yours.

LIEUTENANT.
What, think you much to pay two thousand crowns,
And bear the name and port of gentlemen?
Cut both the villains’ throats—for die you shall.
The lives of those which we have lost in fight
Be counterpoised with such a petty sum!

1 GENTLEMAN.
I’ll give it, sir, and therefore spare my life.

2 GENTLEMAN.
And so will I, and write home for it straight.

WHITMORE.
[_To Suffolk_.] I lost mine eye in laying the prize aboard,
And therefore to revenge it shalt thou die;
And so should these, if I might have my will.

LIEUTENANT.
Be not so rash; take ransom, let him live.

SUFFOLK.
Look on my George; I am a gentleman.
Rate me at what thou wilt, thou shalt be paid.

WHITMORE.
And so am I; my name is Walter Whitmore.
How now! Why starts thou? What, doth death affright?

SUFFOLK.
Thy name affrights me, in whose sound is death.
A cunning man did calculate my birth
And told me that by water I should die.
Yet let not this make thee be bloody-minded;
Thy name is Gaultier, being rightly sounded.

WHITMORE.
Gaultier or Walter, which it is, I care not.
Never yet did base dishonour blur our name
But with our sword we wiped away the blot.
Therefore, when merchant-like I sell revenge,
Broke be my sword, my arms torn and defaced,
And I proclaimed a coward through the world!

SUFFOLK.
Stay, Whitmore, for thy prisoner is a prince,
The Duke of Suffolk, William de la Pole.

WHITMORE.
The Duke of Suffolk, muffled up in rags?

SUFFOLK.
Ay, but these rags are no part of the Duke.
Jove sometime went disguised, and why not I?

LIEUTENANT.
But Jove was never slain, as thou shalt be.

SUFFOLK.
Obscure and lowly swain, King Henry’s blood,
The honourable blood of Lancaster,
Must not be shed by such a jaded groom.
Hast thou not kissed thy hand and held my stirrup?
Bareheaded plodded by my foot-cloth mule,
And thought thee happy when I shook my head?
How often hast thou waited at my cup,
Fed from my trencher, kneeled down at the board,
When I have feasted with Queen Margaret?
Remember it, and let it make thee crestfallen,
Ay, and allay thus thy abortive pride.
How in our voiding lobby hast thou stood
And duly waited for my coming forth?
This hand of mine hath writ in thy behalf,
And therefore shall it charm thy riotous tongue.

WHITMORE.
Speak, captain, shall I stab the forlorn swain?

LIEUTENANT.
First let my words stab him, as he hath me.

SUFFOLK.
Base slave, thy words are blunt, and so art thou.

LIEUTENANT.
Convey him hence, and on our longboat’s side
Strike off his head.

SUFFOLK.
Thou dar’st not, for thy own.

LIEUTENANT.
Yes, poll!

SUFFOLK.
Pole!

LIEUTENANT.
Pool! Sir Pool! Lord!
Ay, kennel, puddle, sink, whose filth and dirt
Troubles the silver spring where England drinks;
Now will I dam up this thy yawning mouth
For swallowing the treasure of the realm.
Thy lips that kissed the Queen shall sweep the ground;
And thou that smiledst at good Duke Humphrey’s death
Against the senseless winds shalt grin in vain,
Who in contempt shall hiss at thee again.
And wedded be thou to the hags of hell,
For daring to affy a mighty lord
Unto the daughter of a worthless king,
Having neither subject, wealth, nor diadem.
By devilish policy art thou grown great
And, like ambitious Sylla, overgorged
With gobbets of thy mother’s bleeding heart.
By thee Anjou and Maine were sold to France,
The false revolting Normans thorough thee
Disdain to call us lord, and Picardy
Hath slain their governors, surprised our forts,
And sent the ragged soldiers wounded home.
The princely Warwick, and the Nevilles all,
Whose dreadful swords were never drawn in vain,
As hating thee are rising up in arms.
And now the house of York, thrust from the crown
By shameful murder of a guiltless king
And lofty, proud, encroaching tyranny,
Burns with revenging fire, whose hopeful colours
Advance our half-faced sun, striving to shine,
Under the which is writ “_Invitis nubibus_.”
The commons here in Kent are up in arms;
And, to conclude, reproach and beggary
Is crept into the palace of our King,
And all by thee.—Away! Convey him hence.

SUFFOLK.
O that I were a god, to shoot forth thunder
Upon these paltry, servile, abject drudges!
Small things make base men proud. This villain here,
Being captain of a pinnace, threatens more
Than Bargulus the strong Illyrian pirate.
Drones suck not eagles’ blood but rob beehives.
It is impossible that I should die
By such a lowly vassal as thyself.
Thy words move rage and not remorse in me.
I go of message from the Queen to France;
I charge thee waft me safely ’cross the Channel.

LIEUTENANT.
Walter.

WHITMORE.
Come, Suffolk, I must waft thee to thy death.

SUFFOLK.
_Pene gelidus timor occupat artus_.
It is thee I fear.

WHITMORE.
Thou shalt have cause to fear before I leave thee.
What, are ye daunted now? Now will ye stoop?

1 GENTLEMAN.
My gracious lord, entreat him, speak him fair.

SUFFOLK.
Suffolk’s imperial tongue is stern and rough,
Used to command, untaught to plead for favour.
Far be it we should honour such as these
With humble suit. No, rather let my head
Stoop to the block than these knees bow to any
Save to the God of heaven and to my King;
And sooner dance upon a bloody pole
Than stand uncovered to the vulgar groom.
True nobility is exempt from fear;
More can I bear than you dare execute.

LIEUTENANT.
Hale him away, and let him talk no more.

SUFFOLK.
Come, soldiers, show what cruelty ye can,
That this my death may never be forgot!
Great men oft die by vile Bezonians.
A Roman sworder and banditto slave
Murdered sweet Tully; Brutus’ bastard hand
Stabbed Julius Caesar; savage islanders
Pompey the Great; and Suffolk dies by pirates.

[_Exeunt Whitmore and others with Suffolk._]

LIEUTENANT.
And as for these whose ransom we have set,
It is our pleasure one of them depart.
Therefore come you with us, and let him go.

[_Exeunt all but the 1 Gentleman._]

Enter Whitmore with Suffolk’s body and head.

WHITMORE.
There let his head and lifeless body lie,
Until the Queen his mistress bury it.

[_Exit._]

1 GENTLEMAN.
O barbarous and bloody spectacle!
His body will I bear unto the King.
If he revenge it not, yet will his friends;
So will the Queen, that living held him dear.

[_Exit with the body._]

SCENE II. Blackheath

Enter George Bevis and John Holland.

BEVIS.
Come, and get thee a sword, though made of a lath; they have been up
these two days.

HOLLAND.
They have the more need to sleep now, then.

BEVIS.
I tell thee, Jack Cade the clothier means to dress the commonwealth,
and turn it, and set a new nap upon it.

HOLLAND.
So he had need, for ’tis threadbare. Well, I say it was never merry
world in England since gentlemen came up.

BEVIS.
O miserable age! Virtue is not regarded in handicraftsmen.

HOLLAND.
The nobility think scorn to go in leather aprons.

BEVIS.
Nay, more, the King’s Council are no good workmen.

HOLLAND.
True; and yet it is said, “Labour in thy vocation,” which is as much to
say as, “Let the magistrates be labouring men;” and therefore should we
be magistrates.

BEVIS.
Thou hast hit it; for there’s no better sign of a brave mind than a
hard hand.

HOLLAND.
I see them! I see them! There’s Best’s son, the tanner of Wingham.

BEVIS.
He shall have the skin of our enemies, to make dog’s leather of.

HOLLAND.
And Dick the butcher.

BEVIS.
Then is sin struck down like an ox, and iniquity’s throat cut like a
calf.

HOLLAND.
And Smith the weaver.

BEVIS.
Argo, their thread of life is spun.

HOLLAND.
Come, come, let’s fall in with them.

Drum. Enter Cade, Dick the Butcher, Smith the Weaver and a  Sawyer with
infinite numbers carrying long staves.

CADE.
We, John Cade, so termed of our supposed father—

DICK.
[_Aside_.] Or rather, of stealing a cade of herrings.

CADE.
For our enemies shall fall before us, inspired with the spirit of
putting down kings and princes. Command silence.

DICK.
Silence!

CADE.
My father was a Mortimer—

DICK.
[_Aside_.] He was an honest man and a good bricklayer.

CADE.
My mother a Plantagenet—

DICK.
[_Aside_.] I knew her well; she was a midwife.

CADE.
My wife descended of the Lacies—

DICK.
[_Aside_.] She was indeed a pedler’s daughter, and sold many laces.

SMITH.
[_Aside_.] But now of late, not able to travel with her furred pack,
she washes bucks here at home.

CADE.
Therefore am I of an honourable house.

DICK.
[_Aside_.] Ay, by my faith, the field is honourable; and there was he
born, under a hedge, for his father had never a house but the cage.

CADE.
Valiant I am.

SMITH.
[_Aside_.] He must needs; for beggary is valiant.

CADE.
I am able to endure much.

DICK.
[_Aside_.] No question of that; for I have seen him whipped three
market-days together.

CADE.
I fear neither sword nor fire.

SMITH.
[_Aside_.] He need not fear the sword, for his coat is of proof.

DICK.
[_Aside_.] But methinks he should stand in fear of fire, being burnt i’
th’ hand for stealing of sheep.

CADE.
Be brave, then, for your captain is brave, and vows reformation. There
shall be in England seven halfpenny loaves sold for a penny; the
three-hooped pot shall have ten hoops, and I will make it felony to
drink small beer. All the realm shall be in common, and in Cheapside
shall my palfrey go to grass. And when I am king, as king I will be—

ALL.
God save your majesty!

CADE.
I thank you, good people.—There shall be no money; all shall eat and
drink on my score, and I will apparel them all in one livery, that they
may agree like brothers and worship me their lord.

DICK.
The first thing we do, let’s kill all the lawyers.

CADE.
Nay, that I mean to do. Is not this a lamentable thing, that of the
skin of an innocent lamb should be made parchment; that parchment,
being scribbled o’er, should undo a man? Some say the bee stings, but I
say ’tis the bee’s wax; for I did but seal once to a thing, and I was
never mine own man since. How now? Who’s there?

Enter some, bringing in the Clerk of Chartham.

SMITH.
The clerk of Chartham. He can write and read and cast account.

CADE.
O, monstrous!

SMITH.
We took him setting of boys’ copies.

CADE.
Here’s a villain!

SMITH.
H’as a book in his pocket with red letters in ’t.

CADE.
Nay, then, he is a conjurer.

DICK.
Nay, he can make obligations and write court-hand.

CADE.
I am sorry for ’t. The man is a proper man, of mine honour; unless I
find him guilty, he shall not die.—Come hither, sirrah, I must examine
thee. What is thy name?

CLERK.
Emmanuel.

DICK.
They use to write it on the top of letters. ’Twill go hard with you.

CADE.
Let me alone. Dost thou use to write thy name? Or hast thou a mark to
thyself, like a honest, plain-dealing man?

CLERK.
Sir, I thank God, I have been so well brought up that I can write my
name.

ALL.
He hath confessed. Away with him! He’s a villain and a traitor.

CADE.
Away with him, I say! Hang him with his pen and inkhorn about his neck.

[_Exit one with the Clerk._]

Enter Michael.

MICHAEL.
Where’s our general?

CADE.
Here I am, thou particular fellow.

MICHAEL.
Fly, fly, fly! Sir Humphrey Stafford and his brother are hard by, with
the King’s forces.

CADE.
Stand, villain, stand, or I’ll fell thee down. He shall be encountered
with a man as good as himself. He is but a knight, is he?

MICHAEL.
No.

CADE.
To equal him, I will make myself a knight presently.
[_Kneels_.] Rise up Sir John Mortimer.
[_Rises_.] Now have at him!

Enter Sir Humphrey Stafford and his Brother with Drum and soldiers.

STAFFORD.
Rebellious hinds, the filth and scum of Kent,
Marked for the gallows, lay your weapons down;
Home to your cottages, forsake this groom.
The King is merciful, if you revolt.

BROTHER.
But angry, wrathful, and inclined to blood,
If you go forward. Therefore yield, or die.

CADE.
As for these silken-coated slaves, I pass not.
It is to you, good people, that I speak,
Over whom, in time to come, I hope to reign,
For I am rightful heir unto the crown.

STAFFORD.
Villain, thy father was a plasterer,
And thou thyself a shearman, art thou not?

CADE.
And Adam was a gardener.

BROTHER.
And what of that?

CADE.
Marry, this: Edmund Mortimer, Earl of March,
Married the Duke of Clarence’ daughter, did he not?

STAFFORD.
Ay, sir.

CADE.
By her he had two children at one birth.

BROTHER.
That’s false.

CADE.
Ay, there’s the question; but I say ’tis true.
The elder of them, being put to nurse,
Was by a beggar-woman stolen away,
And, ignorant of his birth and parentage,
Became a bricklayer when he came to age.
His son am I; deny it if you can.

DICK.
Nay, ’tis too true; therefore he shall be King.

SMITH.
Sir, he made a chimney in my father’s house, and the bricks are alive
at this day to testify it; therefore deny it not.

STAFFORD.
And will you credit this base drudge’s words,
That speaks he knows not what?

ALL.
Ay, marry, will we; therefore get ye gone.

BROTHER.
Jack Cade, the Duke of York hath taught you this.

CADE.
[_Aside_.] He lies, for I invented it myself.—Go to, sirrah, tell the
King from me that, for his father’s sake, Henry the Fifth, in whose
time boys went to span-counter for French crowns, I am content he shall
reign, but I’ll be Protector over him.

DICK.
And furthermore, we’ll have the Lord Saye’s head for selling the
dukedom of Maine.

CADE.
And good reason, for thereby is England mained and fain to go with a
staff, but that my puissance holds it up. Fellow kings, I tell you that
that Lord Saye hath gelded the commonwealth and made it an eunuch; and
more than that, he can speak French, and therefore he is a traitor.

STAFFORD.
O gross and miserable ignorance!

CADE.
Nay, answer if you can. The Frenchmen are our enemies; go to, then, I
ask but this: can he that speaks with the tongue of an enemy be a good
counsellor, or no?

ALL.
No, no, and therefore we’ll have his head.

BROTHER.
Well, seeing gentle words will not prevail,
Assail them with the army of the King.

STAFFORD.
Herald, away, and throughout every town
Proclaim them traitors that are up with Cade;
That those which fly before the battle ends
May, even in their wives’ and children’s sight,
Be hanged up for example at their doors.
And you that be the King’s friends, follow me.

[_Exeunt the two Staffords and soldiers._]

CADE.
And you that love the commons follow me.
Now show yourselves men; ’tis for liberty.
We will not leave one lord, one gentleman;
Spare none but such as go in clouted shoon,
For they are thrifty honest men and such
As would, but that they dare not, take our parts.

DICK.
They are all in order and march toward us.

CADE.
But then are we in order when we are most out of order. Come, march
forward.

[_Exeunt._]

SCENE III. Another part of Blackheath

[Alarums to the fight, wherein both the Staffords are slain. Enter Cade
and the rest.

CADE.
Where’s Dick, the butcher of Ashford?

DICK.
Here, sir.

CADE.
They fell before thee like sheep and oxen, and thou behaved’st thyself
as if thou hadst been in thine own slaughterhouse. Therefore thus will
I reward thee: the Lent shall be as long again as it is, and thou shalt
have a licence to kill for a hundred lacking one.

DICK.
I desire no more.

CADE.
And, to speak truth, thou deservest no less. This monument of the
victory will I bear. [_putting on Sir Humphrey’s brigandine_] And the
bodies shall be dragged at my horse heels till I do come to London,
where we will have the Mayor’s sword borne before us.

DICK.
If we mean to thrive and do good, break open the gaols and let out the
prisoners.

CADE.
Fear not that, I warrant thee. Come, let’s march towards London.

[_Exeunt._]

SCENE IV. London. The Palace

Enter the King with a supplication, and the Queen with Suffolk’s head,
the Duke of Buckingham and the Lord Saye.

QUEEN MARGARET.
[_Aside_.] Oft have I heard that grief softens the mind
And makes it fearful and degenerate;
Think therefore on revenge and cease to weep.
But who can cease to weep and look on this?
Here may his head lie on my throbbing breast;
But where’s the body that I should embrace?

BUCKINGHAM.
What answer makes your grace to the rebels’ supplication?

KING HENRY.
I’ll send some holy bishop to entreat,
For God forbid so many simple souls
Should perish by the sword! And I myself,
Rather than bloody war shall cut them short,
Will parley with Jack Cade their general.
But stay, I’ll read it over once again.

QUEEN MARGARET.
[_Aside_.] Ah, barbarous villains! Hath this lovely face
Ruled, like a wandering planet, over me,
And could it not enforce them to relent
That were unworthy to behold the same?

KING HENRY.
Lord Saye, Jack Cade hath sworn to have thy head.

SAYE.
Ay, but I hope your highness shall have his.

KING HENRY.
How now, madam?
Still lamenting and mourning for Suffolk’s death?
I fear me, love, if that I had been dead,
Thou wouldst not have mourned so much for me.

QUEEN MARGARET.
No, my love, I should not mourn, but die for thee.

Enter a Messenger.

KING HENRY.
How now, what news? Why com’st thou in such haste?

MESSENGER.
The rebels are in Southwark; fly, my lord!
Jack Cade proclaims himself Lord Mortimer,
Descended from the Duke of Clarence’ house,
And calls your grace usurper openly,
And vows to crown himself in Westminster.
His army is a ragged multitude
Of hinds and peasants, rude and merciless.
Sir Humphrey Stafford and his brother’s death
Hath given them heart and courage to proceed.
All scholars, lawyers, courtiers, gentlemen,
They call false caterpillars, and intend their death.

KING HENRY.
O graceless men! They know not what they do.

BUCKINGHAM.
My gracious lord, retire to Killingworth
Until a power be raised to put them down.

QUEEN MARGARET.
Ah, were the Duke of Suffolk now alive,
These Kentish rebels would be soon appeased!

KING HENRY.
Lord Saye, the traitors hate thee;
Therefore away with us to Killingworth.

SAYE.
So might your grace’s person be in danger.
The sight of me is odious in their eyes;
And therefore in this city will I stay
And live alone as secret as I may.

Enter another Messenger.

MESSENGER.
Jack Cade hath gotten London Bridge;
The citizens fly and forsake their houses.
The rascal people, thirsting after prey,
Join with the traitor, and they jointly swear
To spoil the city and your royal court.

BUCKINGHAM.
Then linger not, my lord; away, take horse!

KING HENRY.
Come, Margaret. God, our hope, will succour us.

QUEEN MARGARET.
[_Aside_.] My hope is gone, now Suffolk is deceased.

KING HENRY.
Farewell, my lord. Trust not the Kentish rebels.

BUCKINGHAM.
Trust nobody, for fear you be betrayed.

SAYE.
The trust I have is in mine innocence,
And therefore am I bold and resolute.

[_Exeunt._]

SCENE V. London. The Tower

Enter Lord Scales upon the Tower, walking. Then enter two or three
Citizens below.

SCALES.
How now? Is Jack Cade slain?

1 CITIZEN.
No, my lord, nor likely to be slain; for they have won the Bridge,
killing all those that withstand them. The Lord Mayor craves aid of
your honour from the Tower to defend the city from the rebels.

SCALES.
Such aid as I can spare you shall command,
But I am troubled here with them myself;
The rebels have assayed to win the Tower.
But get you to Smithfield and gather head,
And thither I will send you Matthew Gough.
Fight for your king, your country, and your lives!
And so farewell, for I must hence again.

[_Exeunt._]

SCENE VI. London. Cannon Street

Enter Jack Cade and the rest, and strikes his staff on London Stone.

CADE.
Now is Mortimer lord of this city. And here, sitting upon London Stone,
I charge and command that, of the city’s cost, the Pissing Conduit run
nothing but claret wine this first year of our reign. And now
henceforward it shall be treason for any that calls me other than Lord
Mortimer.

Enter a Soldier, running.

SOLDIER.
Jack Cade! Jack Cade!

CADE.
Knock him down there.

[_They kill him._]

DICK.
If this fellow be wise, he’ll never call ye Jack Cade more. I think he
hath a very fair warning. My lord, there’s an army gathered together in
Smithfield.

CADE.
Come then, let’s go fight with them. But first, go and set London
Bridge on fire; and, if you can, burn down the Tower too. Come, let’s
away.

[_Exeunt._]

SCENE VII. London. Smithfield

Alarums. Matthew Gough is slain, and all the rest. Then enter Jack Cade
with his company.

CADE.
So, sirs. Now go some and pull down the Savoy; others to th’ Inns of
Court; down with them all.

DICK.
I have a suit unto your lordship.

CADE.
Be it a lordship, thou shalt have it for that word.

DICK.
Only that the laws of England may come out of your mouth.

HOLLAND.
[_Aside_.] Mass, ’twill be sore law, then; for he was thrust in the
mouth with a spear, and ’tis not whole yet.

SMITH.
[_Aside_.] Nay, John, it will be stinking law, for his breath stinks
with eating toasted cheese.

CADE.
I have thought upon it, it shall be so. Away, burn all the records of
the realm. My mouth shall be the parliament of England.

HOLLAND.
[_Aside_.] Then we are like to have biting statutes, unless his teeth
be pulled out.

CADE.
And henceforward all things shall be in common.

Enter a Messenger.

MESSENGER.
My lord, a prize, a prize! Here’s the Lord Saye, which sold the towns
in France; he that made us pay one-and-twenty fifteens, and one
shilling to the pound, the last subsidy.

Enter George Bevis with the Lord Saye.

CADE.
Well, he shall be beheaded for it ten times. Ah, thou say, thou serge,
nay, thou buckram lord! Now art thou within point-blank of our
jurisdiction regal. What canst thou answer to my majesty for giving up
of Normandy unto Mounsieur Basimecu, the Dauphin of France? Be it known
unto thee by these presence, even the presence of Lord Mortimer, that I
am the besom that must sweep the court clean of such filth as thou art.
Thou hast most traitorously corrupted the youth of the realm in
erecting a grammar school; and whereas, before, our forefathers had no
other books but the score and the tally, thou hast caused printing to
be used, and, contrary to the King, his crown, and dignity, thou hast
built a paper-mill. It will be proved to thy face that thou hast men
about thee that usually talk of a noun and a verb, and such abominable
words as no Christian ear can endure to hear. Thou hast appointed
justices of peace, to call poor men before them about matters they were
not able to answer. Moreover, thou hast put them in prison, and because
they could not read, thou hast hanged them, when indeed only for that
cause they have been most worthy to live. Thou dost ride on a
foot-cloth, dost thou not?

SAYE.
What of that?

CADE.
Marry, thou ought’st not to let thy horse wear a cloak when honester
men than thou go in their hose and doublets.

DICK.
And work in their shirt too; as myself, for example, that am a butcher.

SAYE.
You men of Kent—

DICK.
What say you of Kent?

SAYE.
Nothing but this; ’tis _bona terra, mala gens_.

CADE.
Away with him, away with him! He speaks Latin.

SAYE.
Hear me but speak, and bear me where you will.
Kent, in the Commentaries Caesar writ,
Is termed the civil’st place of all this isle.
Sweet is the country, because full of riches;
The people liberal, valiant, active, wealthy;
Which makes me hope you are not void of pity.
I sold not Maine, I lost not Normandy,
Yet to recover them would lose my life.
Justice with favour have I always done;
Prayers and tears have moved me, gifts could never.
When have I aught exacted at your hands
Kent to maintain the King, the realm, and you?
Large gifts have I bestowed on learned clerks,
Because my book preferred me to the King.
And seeing ignorance is the curse of God,
Knowledge the wing wherewith we fly to heaven,
Unless you be possessed with devilish spirits,
You cannot but forbear to murder me.
This tongue hath parleyed unto foreign kings
For your behoof—

CADE.
Tut, when struck’st thou one blow in the field?

SAYE.
Great men have reaching hands; oft have I struck
Those that I never saw, and struck them dead.

GEORGE.
O monstrous coward! What, to come behind folks?

SAYE.
These cheeks are pale for watching for your good.

CADE.
Give him a box o’ th’ ear, and that will make ’em red again.

SAYE.
Long sitting to determine poor men’s causes
Hath made me full of sickness and diseases.

CADE.
Ye shall have a hempen caudle then, and the help of hatchet.

DICK.
Why dost thou quiver, man?

SAYE.
The palsy, and not fear, provokes me.

CADE.
Nay, he nods at us, as who should say, “I’ll be even with you.” I’ll
see if his head will stand steadier on a pole or no. Take him away, and
behead him.

SAYE.
Tell me, wherein have I offended most?
Have I affected wealth or honour? Speak.
Are my chests filled up with extorted gold?
Is my apparel sumptuous to behold?
Whom have I injured, that ye seek my death?
These hands are free from guiltless bloodshedding,
This breast from harbouring foul deceitful thoughts.
O, let me live!

CADE.
[_Aside_.] I feel remorse in myself with his words, but I’ll bridle it.
He shall die, an it be but for pleading so well for his life. Away with
him! He has a familiar under his tongue; he speaks not i’ God’s name.
Go, take him away, I say, and strike off his head presently; and then
break into his son-in-law’s house, Sir James Cromer, and strike off his
head, and bring them both upon two poles hither.

ALL.
It shall be done.

SAYE.
Ah, countrymen, if when you make your prayers,
God should be so obdurate as yourselves,
How would it fare with your departed souls?
And therefore yet relent, and save my life.

CADE.
Away with him! And do as I command ye.

[_Exeunt some with Lord Saye._]

The proudest peer in the realm shall not wear a head on his shoulders
unless he pay me tribute; there shall not a maid be married but she
shall pay to me her maidenhead ere they have it. Men shall hold of me
_in capite;_ and we charge and command that their wives be as free as
heart can wish or tongue can tell.

DICK.
My lord, when shall we go to Cheapside and take up commodities upon our
bills?

CADE.
Marry, presently.

ALL.
O, brave!

Enter one with the heads.

CADE.
But is not this braver? Let them kiss one another, for they loved well
when they were alive. Now part them again, lest they consult about the
giving up of some more towns in France. Soldiers, defer the spoil of
the city until night; for with these borne before us instead of maces
will we ride through the streets, and at every corner have them kiss.
Away!

[_Exeunt._]

SCENE VIII. Southwark

Alarum and retreat. Enter Cade and all his rabblement.

CADE.
Up Fish Street! Down Saint Magnus’ Corner! Kill and knock down! Throw
them into Thames! [_Sound a parley_.] What noise is this I hear? Dare
any be so bold to sound retreat or parley when I command them kill?

Enter Buckingham and old Clifford attended.

BUCKINGHAM.
Ay, here they be that dare and will disturb thee.
Know, Cade, we come ambassadors from the King
Unto the commons, whom thou hast misled,
And here pronounce free pardon to them all
That will forsake thee and go home in peace.

CLIFFORD.
What say ye, countrymen? Will ye relent
And yield to mercy whilst ’tis offered you,
Or let a rebel lead you to your deaths?
Who loves the King and will embrace his pardon,
Fling up his cap, and say “God save his Majesty!”
Who hateth him and honours not his father,
Henry the Fifth, that made all France to quake,
Shake he his weapon at us and pass by.

ALL.
God save the King! God save the King!

CADE.
What, Buckingham and Clifford, are ye so brave? And you, base peasants,
do ye believe him? Will you needs be hanged with your pardons about
your necks? Hath my sword therefore broke through London gates, that
you should leave me at the White Hart in Southwark? I thought ye would
never have given out these arms till you had recovered your ancient
freedom; but you are all recreants and dastards, and delight to live in
slavery to the nobility. Let them break your backs with burdens, take
your houses over your heads, ravish your wives and daughters before
your faces. For me, I will make shift for one, and so God’s curse light
upon you all!

ALL.
We’ll follow Cade! We’ll follow Cade!

CLIFFORD.
Is Cade the son of Henry the Fifth,
That thus you do exclaim you’ll go with him?
Will he conduct you through the heart of France
And make the meanest of you earls and dukes?
Alas, he hath no home, no place to fly to,
Nor knows he how to live but by the spoil,
Unless by robbing of your friends and us.
Were ’t not a shame that whilst you live at jar
The fearful French, whom you late vanquished,
Should make a start o’er seas and vanquish you?
Methinks already in this civil broil
I see them lording it in London streets,
Crying “_Villiago!_” unto all they meet.
Better ten thousand base-born Cades miscarry
Than you should stoop unto a Frenchman’s mercy.
To France, to France, and get what you have lost!
Spare England, for it is your native coast.
Henry hath money, you are strong and manly;
God on our side, doubt not of victory.

ALL.
A Clifford! A Clifford! We’ll follow the King and Clifford.

CADE.
Was ever feather so lightly blown to and fro as this multitude? The
name of Henry the Fifth hales them to an hundred mischiefs and makes
them leave me desolate. I see them lay their heads together to surprise
me. My sword make way for me, for here is no staying.—In despite of the
devils and hell, have through the very middest of you! And heavens and
honour be witness that no want of resolution in me, but only my
followers’ base and ignominious treasons, makes me betake me to my
heels.

[_Exit._]

BUCKINGHAM.
What, is he fled? Go some, and follow him;
And he that brings his head unto the King
Shall have a thousand crowns for his reward.

[_Exeunt some of them._]

Follow me, soldiers; we’ll devise a mean
To reconcile you all unto the King.

[_Exeunt._]

SCENE IX. Kenilworth Castle

Sound trumpets. Enter King, Queen and Somerset on the terrace, aloft.

KING HENRY.
Was ever king that joyed an earthly throne
And could command no more content than I?
No sooner was I crept out of my cradle
But I was made a king at nine months old.
Was never subject longed to be a king
As I do long and wish to be a subject.

Enter Buckingham and old Clifford.

BUCKINGHAM.
Health and glad tidings to your majesty!

KING HENRY.
Why, Buckingham, is the traitor Cade surprised?
Or is he but retired to make him strong?

Enter below multitudes with halters about their necks.

CLIFFORD.
He is fled, my lord, and all his powers do yield,
And humbly thus, with halters on their necks,
Expect your highness’ doom of life or death.

KING HENRY.
Then, heaven, set ope thy everlasting gates
To entertain my vows of thanks and praise!
Soldiers, this day have you redeemed your lives
And showed how well you love your prince and country.
Continue still in this so good a mind,
And Henry, though he be infortunate,
Assure yourselves, will never be unkind.
And so, with thanks and pardon to you all,
I do dismiss you to your several countries.

ALL.
God save the King! God save the King!

Enter a Messenger.

MESSENGER.
Please it your grace to be advertised
The Duke of York is newly come from Ireland,
And with a puissant and a mighty power
Of gallowglasses and stout kerns
Is marching hitherward in proud array,
And still proclaimeth, as he comes along,
His arms are only to remove from thee
The Duke of Somerset, whom he terms a traitor.

KING HENRY.
Thus stands my state, ’twixt Cade and York distressed,
Like to a ship that, having scaped a tempest,
Is straightway calmed and boarded with a pirate.
But now is Cade driven back, his men dispersed,
And now is York in arms to second him.
I pray thee, Buckingham, go and meet him,
And ask him what’s the reason of these arms.
Tell him I’ll send Duke Edmund to the Tower.—
And, Somerset, we will commit thee thither,
Until his army be dismissed from him.

SOMERSET.
My lord, I’ll yield myself to prison willingly,
Or unto death, to do my country good.

KING HENRY.
In any case, be not too rough in terms,
For he is fierce and cannot brook hard language.

BUCKINGHAM.
I will, my lord, and doubt not so to deal
As all things shall redound unto your good.

KING HENRY.
Come, wife, let’s in, and learn to govern better;
For yet may England curse my wretched reign.

[_Flourish. Exeunt._]

SCENE X. Kent. Iden’s Garden

Enter Cade.

CADE.
Fie on ambitions! Fie on myself, that have a sword and yet am ready to
famish! These five days have I hid me in these woods and durst not peep
out, for all the country is laid for me; but now am I so hungry that if
I might have a lease of my life for a thousand years, I could stay no
longer. Wherefore, o’er a brick wall have I climbed into this garden,
to see if I can eat grass, or pick a sallet another while, which is not
amiss to cool a man’s stomach this hot weather. And I think this word
“sallet” was born to do me good; for many a time, but for a sallet, my
brain-pan had been cleft with a brown bill; and many a time, when I
have been dry and bravely marching, it hath served me instead of a
quart pot to drink in; and now the word “sallet” must serve me to feed
on.

Enter Iden and his men.

IDEN.
Lord, who would live turmoiled in the court
And may enjoy such quiet walks as these?
This small inheritance my father left me
Contenteth me, and worth a monarchy.
I seek not to wax great by others’ waning,
Or gather wealth, I care not with what envy;
Sufficeth that I have maintains my state
And sends the poor well pleased from my gate.

CADE.
Here’s the lord of the soil come to seize me for a stray, for entering
his fee-simple without leave.—Ah, villain, thou wilt betray me and get
a thousand crowns of the King by carrying my head to him; but I’ll make
thee eat iron like an ostrich, and swallow my sword like a great pin,
ere thou and I part.

IDEN.
Why, rude companion, whatsoe’er thou be,
I know thee not; why, then, should I betray thee?
Is ’t not enough to break into my garden
And, like a thief, to come to rob my grounds,
Climbing my walls in spite of me the owner,
But thou wilt brave me with these saucy terms?

CADE.
Brave thee? Ay, by the best blood that ever was broached, and beard
thee too. Look on me well: I have eat no meat these five days, yet come
thou and thy five men, and if I do not leave you all as dead as a
doornail, I pray God I may never eat grass more.

IDEN.
Nay, it shall ne’er be said, while England stands,
That Alexander Iden, an esquire of Kent,
Took odds to combat a poor famished man.
Oppose thy steadfast-gazing eyes to mine,
See if thou canst outface me with thy looks.
Set limb to limb and thou art far the lesser;
Thy hand is but a finger to my fist,
Thy leg a stick compared with this truncheon.
My foot shall fight with all the strength thou hast;
And if mine arm be heaved in the air,
Thy grave is digged already in the earth.
As for words, whose greatness answers words,
Let this my sword report what speech forbears.

CADE.
By my valour, the most complete champion that ever I heard! Steel, if
thou turn the edge, or cut not out the burly-boned clown in chines of
beef ere thou sleep in thy sheath, I beseech God on my knees thou mayst
be turned to hobnails.

[_Here they fight and Cade falls._]

O, I am slain! Famine and no other hath slain me. Let ten thousand
devils come against me, and give me but the ten meals I have lost, and
I’d defy them all. Wither, garden; and be henceforth a burying place to
all that do dwell in this house, because the unconquered soul of Cade
is fled.

IDEN.
Is’t Cade that I have slain, that monstrous traitor?
Sword, I will hallow thee for this thy deed,
And hang thee o’er my tomb when I am dead.
Ne’er shall this blood be wiped from thy point,
But thou shalt wear it as a herald’s coat
To emblaze the honour that thy master got.

CADE.
Iden, farewell, and be proud of thy victory. Tell Kent from me she hath
lost her best man, and exhort all the world to be cowards; for I, that
never feared any, am vanquished by famine, not by valour.

[_Dies._]

IDEN.
How much thou wrong’st me, heaven be my judge.
Die, damned wretch, the curse of her that bare thee!
And as I thrust thy body in with my sword,
So wish I, I might thrust thy soul to hell.
Hence will I drag thee headlong by the heels
Unto a dunghill, which shall be thy grave,
And there cut off thy most ungracious head,
Which I will bear in triumph to the King,
Leaving thy trunk for crows to feed upon.

[_Exit._]




ACT V

SCENE I. Fields between Dartford and Blackheath


Enter York and his army of Irish, with drum and colours.

YORK.
From Ireland thus comes York to claim his right
And pluck the crown from feeble Henry’s head.
Ring, bells, aloud; burn, bonfires, clear and bright,
To entertain great England’s lawful king.
Ah, _sancta majestas_, who would not buy thee dear?
Let them obey that knows not how to rule.
This hand was made to handle nought but gold.
I cannot give due action to my words
Except a sword or sceptre balance it.
A sceptre shall it have, have I a soul,
On which I’ll toss the fleur-de-luce of France.

Enter Buckingham.

Whom have we here? Buckingham, to disturb me?
The King hath sent him, sure. I must dissemble.

BUCKINGHAM.
York, if thou meanest well, I greet thee well.

YORK.
Humphrey of Buckingham, I accept thy greeting.
Art thou a messenger, or come of pleasure?

BUCKINGHAM.
A messenger from Henry, our dread liege,
To know the reason of these arms in peace;
Or why thou, being a subject as I am,
Against thy oath and true allegiance sworn,
Should raise so great a power without his leave,
Or dare to bring thy force so near the court.

YORK.
[_Aside_.] Scarce can I speak, my choler is so great.
O, I could hew up rocks and fight with flint,
I am so angry at these abject terms;
And now, like Ajax Telamonius,
On sheep or oxen could I spend my fury.
I am far better born than is the King,
More like a king, more kingly in my thoughts.
But I must make fair weather yet awhile,
Till Henry be more weak and I more strong.—
Buckingham, I prithee, pardon me,
That I have given no answer all this while;
My mind was troubled with deep melancholy.
The cause why I have brought this army hither
Is to remove proud Somerset from the King,
Seditious to his grace and to the state.

BUCKINGHAM.
That is too much presumption on thy part;
But if thy arms be to no other end,
The King hath yielded unto thy demand:
The Duke of Somerset is in the Tower.

YORK.
Upon thine honour, is he prisoner?

BUCKINGHAM.
Upon mine honour, he is prisoner.

YORK.
Then, Buckingham, I do dismiss my powers.
Soldiers, I thank you all; disperse yourselves;
Meet me tomorrow in Saint George’s field,
You shall have pay and everything you wish.

[_Exeunt Soldiers._]

And let my sovereign, virtuous Henry,
Command my eldest son, nay, all my sons,
As pledges of my fealty and love,
I’ll send them all as willing as I live.
Lands, goods, horse, armour, anything I have
Is his to use, so Somerset may die.

BUCKINGHAM.
York, I commend this kind submission.
We twain will go into his highness’ tent.

Enter King and Attendants.

KING HENRY.
Buckingham, doth York intend no harm to us
That thus he marcheth with thee arm in arm?

YORK.
In all submission and humility
York doth present himself unto your highness.

KING HENRY.
Then what intends these forces thou dost bring?

YORK.
To heave the traitor Somerset from hence
And fight against that monstrous rebel Cade,
Who since I heard to be discomfited.

Enter Iden with Cade’s head.

IDEN.
If one so rude and of so mean condition
May pass into the presence of a king,
Lo, I present your grace a traitor’s head,
The head of Cade, whom I in combat slew.

KING HENRY.
The head of Cade! Great God, how just art Thou!
O, let me view his visage, being dead,
That living wrought me such exceeding trouble.
Tell me, my friend, art thou the man that slew him?

IDEN.
I was, an ’t like your majesty.

KING HENRY.
How art thou called? And what is thy degree?

IDEN.
Alexander Iden, that’s my name;
A poor esquire of Kent, that loves his King.

BUCKINGHAM.
So please it you, my lord, ’twere not amiss
He were created knight for his good service.

KING HENRY.
Iden, kneel down. [_He kneels_.] Rise up a knight.
We give thee for reward a thousand marks,
And will that thou henceforth attend on us.

IDEN.
May Iden live to merit such a bounty,
And never live but true unto his liege!

[_Rises._]

Enter Queen and Somerset.

KING HENRY.
See, Buckingham, Somerset comes with the Queen.
Go, bid her hide him quickly from the Duke.

QUEEN MARGARET.
For thousand Yorks he shall not hide his head,
But boldly stand and front him to his face.

YORK.
How now? Is Somerset at liberty?
Then, York, unloose thy long-imprisoned thoughts,
And let thy tongue be equal with thy heart.
Shall I endure the sight of Somerset?
False king, why hast thou broken faith with me,
Knowing how hardly I can brook abuse?
“King” did I call thee? No, thou art not king,
Not fit to govern and rule multitudes,
Which dar’st not, no, nor canst not rule a traitor.
That head of thine doth not become a crown;
Thy hand is made to grasp a palmer’s staff,
And not to grace an awful princely sceptre.
That gold must round engirt these brows of mine,
Whose smile and frown, like to Achilles’ spear,
Is able with the change to kill and cure.
Here is a hand to hold a sceptre up
And with the same to act controlling laws.
Give place! By heaven, thou shalt rule no more
O’er him whom heaven created for thy ruler.

SOMERSET.
O monstrous traitor! I arrest thee, York,
Of capital treason ’gainst the King and crown.
Obey, audacious traitor, kneel for grace.

YORK.
Wouldst have me kneel? First let me ask of these
If they can brook I bow a knee to man.
Sirrah, call in my sons to be my bail.

[_Exit Attendant._]

I know, ere they will have me go to ward,
They’ll pawn their swords for my enfranchisement.

QUEEN MARGARET.
Call hither Clifford; bid him come amain,
To say if that the bastard boys of York
Shall be the surety for their traitor father.

[_Exit Buckingham._]

YORK.
O blood-bespotted Neapolitan,
Outcast of Naples, England’s bloody scourge!
The sons of York, thy betters in their birth,
Shall be their father’s bail; and bane to those
That for my surety will refuse the boys!

Enter Edward and Richard.

See where they come; I’ll warrant they’ll make it good.

Enter old Clifford and his Son.

QUEEN MARGARET.
And here comes Clifford to deny their bail.

CLIFFORD.
Health and all happiness to my lord the King.

[_Rises._]

YORK.
I thank thee, Clifford. Say, what news with thee?
Nay, do not fright us with an angry look.
We are thy sovereign, Clifford, kneel again.
For thy mistaking so, we pardon thee.

CLIFFORD.
This is my king, York, I do not mistake;
But thou mistakes me much to think I do.
To Bedlam with him! Is the man grown mad?

KING HENRY.
Ay, Clifford; a bedlam and ambitious humour
Makes him oppose himself against his king.

CLIFFORD.
He is a traitor; let him to the Tower,
And chop away that factious pate of his.

QUEEN MARGARET.
He is arrested, but will not obey;
His sons, he says, shall give their words for him.

YORK.
Will you not, sons?

EDWARD.
Ay, noble father, if our words will serve.

RICHARD.
And if words will not, then our weapons shall.

CLIFFORD.
Why, what a brood of traitors have we here!

YORK.
Look in a glass, and call thy image so.
I am thy king, and thou a false-heart traitor.
Call hither to the stake my two brave bears,
That with the very shaking of their chains
They may astonish these fell-lurking curs.
Bid Salisbury and Warwick come to me.

Enter the Earls of Warwick and Salisbury.

CLIFFORD.
Are these thy bears? We’ll bait thy bears to death
And manacle the bearherd in their chains,
If thou dar’st bring them to the baiting-place.

RICHARD.
Oft have I seen a hot o’erweening cur
Run back and bite because he was withheld,
Who, being suffered with the bear’s fell paw,
Hath clapped his tail between his legs and cried;
And such a piece of service will you do
If you oppose yourselves to match Lord Warwick.

CLIFFORD.
Hence, heap of wrath, foul indigested lump,
As crooked in thy manners as thy shape!

YORK.
Nay, we shall heat you thoroughly anon.

CLIFFORD.
Take heed, lest by your heat you burn yourselves.

KING HENRY.
Why, Warwick, hath thy knee forgot to bow?
Old Salisbury, shame to thy silver hair,
Thou mad misleader of thy brainsick son!
What, wilt thou on thy deathbed play the ruffian,
And seek for sorrow with thy spectacles?
O, where is faith? O, where is loyalty?
If it be banished from the frosty head,
Where shall it find a harbour in the earth?
Wilt thou go dig a grave to find out war,
And shame thine honourable age with blood?
Why art thou old, and want’st experience?
Or wherefore dost abuse it, if thou hast it?
For shame, in duty bend thy knee to me
That bows unto the grave with mickle age.

SALISBURY.
My lord, I have considered with myself
The title of this most renowned duke,
And in my conscience do repute his grace
The rightful heir to England’s royal seat.

KING HENRY.
Hast thou not sworn allegiance unto me?

SALISBURY.
I have.

KING HENRY.
Canst thou dispense with heaven for such an oath?

SALISBURY.
It is great sin to swear unto a sin,
But greater sin to keep a sinful oath.
Who can be bound by any solemn vow
To do a murderous deed, to rob a man,
To force a spotless virgin’s chastity,
To reave the orphan of his patrimony,
To wring the widow from her customed right,
And have no other reason for this wrong
But that he was bound by a solemn oath?

QUEEN MARGARET.
A subtle traitor needs no sophister.

KING HENRY.
Call Buckingham, and bid him arm himself.

YORK.
Call Buckingham, and all the friends thou hast,
I am resolved for death or dignity.

CLIFFORD.
The first I warrant thee, if dreams prove true.

WARWICK.
You were best to go to bed and dream again,
To keep thee from the tempest of the field.

CLIFFORD.
I am resolved to bear a greater storm
Than any thou canst conjure up today;
And that I’ll write upon thy burgonet,
Might I but know thee by thy household badge.

WARWICK.
Now, by my father’s badge, old Neville’s crest,
The rampant bear chained to the ragged staff,
This day I’ll wear aloft my burgonet,
As on a mountain top the cedar shows
That keeps his leaves in spite of any storm,
Even to affright thee with the view thereof.

CLIFFORD.
And from thy burgonet I’ll rend thy bear
And tread it under foot with all contempt,
Despite the bearherd that protects the bear.

YOUNG CLIFFORD.
And so to arms, victorious father,
To quell the rebels and their complices.

RICHARD.
Fie, charity, for shame! Speak not in spite,
For you shall sup with Jesu Christ tonight.

YOUNG CLIFFORD.
Foul stigmatic, that’s more than thou canst tell.

RICHARD.
If not in heaven, you’ll surely sup in hell.

[_Exeunt severally._]

SCENE II. Saint Albans

The sign of the Castle Inn is displayed. Alarums to the battle. Enter
Warwick.

WARWICK.
Clifford of Cumberland, ’tis Warwick calls;
An if thou dost not hide thee from the bear,
Now, when the angry trumpet sounds alarum
And dead men’s cries do fill the empty air,
Clifford, I say, come forth and fight with me!
Proud northern lord, Clifford of Cumberland,
Warwick is hoarse with calling thee to arms.

Enter York.

How now, my noble lord? What, all afoot?

YORK.
The deadly-handed Clifford slew my steed,
But match to match I have encountered him
And made a prey for carrion kites and crows
Even of the bonny beast he loved so well.

Enter old Clifford.

WARWICK.
Of one or both of us the time is come.

YORK.
Hold, Warwick, seek thee out some other chase,
For I myself must hunt this deer to death.

WARWICK.
Then, nobly, York; ’tis for a crown thou fight’st.
As I intend, Clifford, to thrive today,
It grieves my soul to leave thee unassailed.

[_Exit._]

CLIFFORD.
What seest thou in me, York? Why dost thou pause?

YORK.
With thy brave bearing should I be in love,
But that thou art so fast mine enemy.

CLIFFORD.
Nor should thy prowess want praise and esteem,
But that ’tis shown ignobly and in treason.

YORK.
So let it help me now against thy sword
As I in justice and true right express it!

CLIFFORD.
My soul and body on the action both!

YORK.
A dreadful lay! Address thee instantly.

[_They fight and Clifford falls._]

CLIFFORD.
_La fin couronne les oeuvres._

[_Dies._]

YORK.
Thus war hath given thee peace, for thou art still.
Peace with his soul, heaven, if it be thy will!

[_Exit._]

Enter young Clifford.

YOUNG CLIFFORD.
Shame and confusion! All is on the rout,
Fear frames disorder, and disorder wounds
Where it should guard. O war, thou son of hell,
Whom angry heavens do make their minister,
Throw in the frozen bosoms of our part
Hot coals of vengeance! Let no soldier fly.
He that is truly dedicate to war
Hath no self-love; nor he that loves himself
Hath not essentially but by circumstance,
The name of valour. [_Sees his dead father_.] O, let the vile world end
And the premised flames of the last day
Knit earth and heaven together!
Now let the general trumpet blow his blast,
Particularities and petty sounds
To cease! Wast thou ordained, dear father,
To lose thy youth in peace, and to achieve
The silver livery of advised age,
And, in thy reverence and thy chair-days, thus
To die in ruffian battle? Even at this sight
My heart is turned to stone, and while ’tis mine
It shall be stony. York not our old men spares;
No more will I their babes; tears virginal
Shall be to me even as the dew to fire,
And beauty, that the tyrant oft reclaims,
Shall to my flaming wrath be oil and flax.
Henceforth I will not have to do with pity.
Meet I an infant of the house of York,
Into as many gobbets will I cut it
As wild Medea young Absyrtus did.
In cruelty will I seek out my fame.

[_He takes him up on his back._]

Come, thou new ruin of old Clifford’s house;
As did Aeneas old Anchises bear,
So bear I thee upon my manly shoulders;
But then Aeneas bare a living load,
Nothing so heavy as these woes of mine.

[_Exit, bearing off his father._]

Enter Richard and Somerset to fight. Somerset is killed.

RICHARD.
So, lie thou there;
For underneath an alehouse’ paltry sign,
The Castle in Saint Albans, Somerset
Hath made the wizard famous in his death.
Sword, hold thy temper; heart, be wrathful still!
Priests pray for enemies, but princes kill.

[_Exit._]

Fight. Excursions. Enter King, Queen and others.

QUEEN MARGARET.
Away, my lord! You are slow, for shame, away!

KING HENRY.
Can we outrun the heavens? Good Margaret, stay.

QUEEN MARGARET.
What are you made of? You’ll nor fight nor fly.
Now is it manhood, wisdom, and defence
To give the enemy way, and to secure us
By what we can, which can no more but fly.

[_Alarum afar off._]

If you be ta’en, we then should see the bottom
Of all our fortunes; but if we haply scape,
As well we may, if not through your neglect,
We shall to London get, where you are loved
And where this breach now in our fortunes made
May readily be stopped.

Enter young Clifford.

YOUNG CLIFFORD.
But that my heart’s on future mischief set,
I would speak blasphemy ere bid you fly;
But fly you must; uncurable discomfit
Reigns in the hearts of all our present parts.
Away, for your relief! And we will live
To see their day and them our fortune give.
Away, my lord, away!

[_Exeunt._]

SCENE III. Fields near Saint Albans

Alarum. Retreat. Enter York, Richard, Warwick and Soldiers with drum
and colours.

YORK.
Of Salisbury, who can report of him,
That winter lion, who in rage forgets
Aged contusions and all brush of time,
And, like a gallant in the brow of youth,
Repairs him with occasion? This happy day
Is not itself, nor have we won one foot,
If Salisbury be lost.

RICHARD.
My noble father,
Three times today I holp him to his horse,
Three times bestrid him; thrice I led him off,
Persuaded him from any further act;
But still, where danger was, still there I met him,
And like rich hangings in a homely house,
So was his will in his old feeble body.
But, noble as he is, look where he comes.

Enter Salisbury.

Now, by my sword, well hast thou fought today.

SALISBURY.
By th’ mass, so did we all. I thank you, Richard.
God knows how long it is I have to live,
And it hath pleased him that three times today
You have defended me from imminent death.
Well, lords, we have not got that which we have;
’Tis not enough our foes are this time fled,
Being opposites of such repairing nature.

YORK.
I know our safety is to follow them;
For, as I hear, the King is fled to London
To call a present court of parliament.
Let us pursue him ere the writs go forth.
What says Lord Warwick? Shall we after them?

WARWICK.
After them? Nay, before them, if we can.
Now, by my hand, lords, ’twas a glorious day.
Saint Albans battle won by famous York
Shall be eternized in all age to come.
Sound drums and trumpets, and to London all;
And more such days as these to us befall!

[_Exeunt._]




THE THIRD PART OF KING HENRY THE SIXTH

Contents

 ACT I
 Scene I. London. The Parliament House
 Scene II. Sandal Castle
 Scene III. Plains near Sandal Castle
 Scene IV. The Same

 ACT II
 Scene I. A plain near Mortimer’s Cross in Herefordshire
 Scene II. Before York
 Scene III. A field of battle between Towton and Saxton, in Yorkshire
 Scene IV. Another Part of the Field
 Scene V. Another Part of the Field
 Scene VI. Another Part of the Field

 ACT III
 Scene I. A Forest in the North of England
 Scene II. The Palace
 Scene III. France. The King’s Palace

 ACT IV
 Scene I. London. The Palace
 Scene II. A Plain in Warwickshire
 Scene III. Edward’s Camp near Warwick
 Scene IV. London. The Palace
 Scene V. A park near Middleham Castle in Yorkshire
 Scene VI. London. The Tower
 Scene VII. Before York
 Scene VIII. London. The Palace

 ACT V
 Scene I. Coventry
 Scene II. A Field of Battle near Barnet
 Scene III. Another Part of the Field
 Scene IV. Plains near Tewkesbury
 Scene V. Another part of the Field
 Scene VI. London. The Tower
 Scene VII. London. The Palace




Dramatis Personæ

KING HENRY the Sixth
QUEEN MARGARET
PRINCE EDWARD, Prince of Wales, his son
DUKE OF SOMERSET
DUKE OF EXETER
EARL OF OXFORD
EARL OF NORTHUMBERLAND
EARL OF WESTMORELAND
LORD CLIFFORD
RICHARD PLANTAGENET, Duke of York
EDWARD, Earl of March, afterwards King Edward IV., his son
GEORGE, afterwards Duke of Clarence, his son
RICHARD, afterwards Duke of Gloucester, his son
EDMUND, Earl of Rutland, his son
DUKE OF NORFOLK
MARQUESS OF MONTAGUE
EARL OF WARWICK
EARL OF PEMBROKE
LORD HASTINGS
LORD STAFFORD
SIR JOHN MORTIMER, uncle to the Duke of York
SIR HUGH MORTIMER, uncle to the Duke of York
LADY GREY, afterwards Queen Elizabeth to Edward IV
EARL RIVERS, brother to Lady Grey
HENRY, Earl of Richmond, a youth
SIR WILLIAM STANLEY
SIR JOHN MONTGOMERY
SIR JOHN SOMERVILLE
KING LEWIS the Eleventh, King of France
BONA, sister to the French Queen
Tutor to Rutland
Mayor of York
Lieutenant of the Tower
A Nobleman
Two Keepers
A Huntsman
A Son that has killed his father
A Father that has killed his son

Soldiers, Attendants, Messengers, Watchmen, etc.

SCENE: England and France




ACT I

SCENE I. London. The Parliament House


Alarum. Enter Duke of York, Edward, Richard, Norfolk, Montague, Warwick
and Soldiers, all wearing the white rose.

WARWICK.
I wonder how the King escaped our hands.

YORK.
While we pursued the horsemen of the north,
He slyly stole away and left his men;
Whereat the great Lord of Northumberland,
Whose warlike ears could never brook retreat,
Cheered up the drooping army; and himself,
Lord Clifford, and Lord Stafford, all abreast,
Charged our main battle’s front, and breaking in,
Were by the swords of common soldiers slain.

EDWARD.
Lord Stafford’s father, Duke of Buckingham,
Is either slain or wounded dangerous;
I cleft his beaver with a downright blow.
That this is true, father, behold his blood.

[_Showing his bloody sword._]

MONTAGUE.
And, brother, here’s the Earl of Wiltshire’s blood,

[_To York, showing his._]

Whom I encountered as the battles joined.

RICHARD.
Speak thou for me, and tell them what I did.

[_Throwing down the Duke of Somerset’s head._]

YORK.
Richard hath best deserved of all my sons.
But is your Grace dead, my Lord of Somerset?

NORFOLK.
Such hope have all the line of John of Gaunt!

RICHARD.
Thus do I hope to shake King Henry’s head.

WARWICK.
And so do I. Victorious Prince of York,
Before I see thee seated in that throne
Which now the house of Lancaster usurps,
I vow by heaven these eyes shall never close.
This is the palace of the fearful king,
And this the regal seat. Possess it, York,
For this is thine, and not King Henry’s heirs’.

YORK.
Assist me, then, sweet Warwick, and I will;
For hither we have broken in by force.

NORFOLK.
We’ll all assist you; he that flies shall die.

YORK.
Thanks, gentle Norfolk. Stay by me, my lords;
And, soldiers, stay and lodge by me this night.

WARWICK.
And when the King comes, offer him no violence,
Unless he seek to thrust you out perforce.

[_They retire._]

YORK.
The Queen this day here holds her parliament,
But little thinks we shall be of her council.
By words or blows here let us win our right.

RICHARD.
Armed as we are, let’s stay within this house.

WARWICK.
The bloody parliament shall this be called,
Unless Plantagenet, Duke of York, be king,
And bashful Henry deposed, whose cowardice
Hath made us bywords to our enemies.

YORK.
Then leave me not, my lords; be resolute.
I mean to take possession of my right.

WARWICK.
Neither the King, nor he that loves him best,
The proudest he that holds up Lancaster,
Dares stir a wing if Warwick shake his bells.
I’ll plant Plantagenet, root him up who dares.
Resolve thee, Richard; claim the English crown.

[_Warwick leads York to the throne, who seats himself._]

Flourish. Enter King Henry, Clifford, Northumberland, Westmoreland,
Exeter and the rest, all wearing the red rose.

KING HENRY.
My lords, look where the sturdy rebel sits,
Even in the chair of state! Belike he means,
Backed by the power of Warwick, that false peer,
To aspire unto the crown and reign as king.
Earl of Northumberland, he slew thy father,
And thine, Lord Clifford; and you both have vowed revenge
On him, his sons, his favourites, and his friends.

NORTHUMBERLAND.
If I be not, heavens be revenged on me!

CLIFFORD.
The hope thereof makes Clifford mourn in steel.

WESTMORELAND.
What, shall we suffer this? Let’s pluck him down.
My heart for anger burns. I cannot brook it.

KING HENRY.
Be patient, gentle Earl of Westmoreland.

CLIFFORD.
Patience is for poltroons, such as he.
He durst not sit there had your father lived.
My gracious lord, here in the parliament
Let us assail the family of York.

NORTHUMBERLAND.
Well hast thou spoken, cousin. Be it so.

KING HENRY.
Ah, know you not the city favours them,
And they have troops of soldiers at their beck?

EXETER.
But when the Duke is slain, they’ll quickly fly.

KING HENRY.
Far be the thought of this from Henry’s heart,
To make a shambles of the Parliament House!
Cousin of Exeter, frowns, words, and threats
Shall be the war that Henry means to use.

[_They advance to the Duke._]

Thou factious Duke of York, descend my throne,
And kneel for grace and mercy at my feet;
I am thy sovereign.

YORK.
I am thine.

EXETER.
For shame, come down. He made thee Duke of York.

YORK.
’Twas my inheritance, as the earldom was.

EXETER.
Thy father was a traitor to the crown.

WARWICK.
Exeter, thou art a traitor to the crown
In following this usurping Henry.

CLIFFORD.
Whom should he follow but his natural king?

WARWICK.
True, Clifford, that’s Richard, Duke of York.

KING HENRY.
And shall I stand, and thou sit in my throne?

YORK.
It must and shall be so. Content thyself.

WARWICK.
Be Duke of Lancaster. Let him be king.

WESTMORELAND.
He is both King and Duke of Lancaster;
And that the Lord of Westmoreland shall maintain.

WARWICK.
And Warwick shall disprove it. You forget
That we are those which chased you from the field
And slew your fathers, and with colours spread
Marched through the city to the palace gates.

NORTHUMBERLAND.
Yes, Warwick, I remember it to my grief;
And, by his soul, thou and thy house shall rue it.

WESTMORELAND.
Plantagenet, of thee and these thy sons,
Thy kinsmen, and thy friends, I’ll have more lives
Than drops of blood were in my father’s veins.

CLIFFORD.
Urge it no more; lest that, instead of words,
I send thee, Warwick, such a messenger
As shall revenge his death before I stir.

WARWICK.
Poor Clifford, how I scorn his worthless threats!

YORK.
Will you we show our title to the crown?
If not, our swords shall plead it in the field.

KING HENRY.
What title hast thou, traitor, to the crown?
Thy father was, as thou art, Duke of York;
Thy grandfather, Roger Mortimer, Earl of March.
I am the son of Henry the Fifth,
Who made the Dauphin and the French to stoop,
And seized upon their towns and provinces.

WARWICK.
Talk not of France, sith thou hast lost it all.

KING HENRY.
The Lord Protector lost it, and not I.
When I was crowned I was but nine months old.

RICHARD.
You are old enough now, and yet, methinks, you lose.
Father, tear the crown from the usurper’s head.

EDWARD.
Sweet father, do so; set it on your head.

MONTAGUE.
Good brother, as thou lov’st and honourest arms,
Let’s fight it out and not stand cavilling thus.

RICHARD.
Sound drums and trumpets, and the King will fly.

YORK.
Sons, peace!

KING HENRY.
Peace thou, and give King Henry leave to speak.

WARWICK.
Plantagenet shall speak first. Hear him, lords,
And be you silent and attentive too,
For he that interrupts him shall not live.

KING HENRY.
Think’st thou that I will leave my kingly throne,
Wherein my grandsire and my father sat?
No. First shall war unpeople this my realm;
Ay, and their colours, often borne in France,
And now in England, to our heart’s great sorrow,
Shall be my winding-sheet. Why faint you, lords?
My title’s good, and better far than his.

WARWICK.
Prove it, Henry, and thou shalt be king.

KING HENRY.
Henry the Fourth by conquest got the crown.

YORK.
’Twas by rebellion against his king.

KING HENRY.
[_Aside_.] I know not what to say; my title’s weak.
Tell me, may not a king adopt an heir?

YORK.
What then?

KING HENRY.
An if he may, then am I lawful king;
For Richard, in the view of many lords,
Resigned the crown to Henry the Fourth,
Whose heir my father was, and I am his.

YORK.
He rose against him, being his sovereign,
And made him to resign his crown perforce.

WARWICK.
Suppose, my lords, he did it unconstrained,
Think you ’twere prejudicial to his crown?

EXETER.
No, for he could not so resign his crown
But that the next heir should succeed and reign.

KING HENRY.
Art thou against us, Duke of Exeter?

EXETER.
His is the right, and therefore pardon me.

YORK.
Why whisper you, my lords, and answer not?

EXETER.
My conscience tells me he is lawful king.

KING HENRY.
[_Aside_.] All will revolt from me and turn to him.

NORTHUMBERLAND.
Plantagenet, for all the claim thou lay’st,
Think not that Henry shall be so deposed.

WARWICK.
Deposed he shall be, in despite of all.

NORTHUMBERLAND.
Thou art deceived. ’Tis not thy southern power,
Of Essex, Norfolk, Suffolk, nor of Kent,
Which makes thee thus presumptuous and proud,
Can set the Duke up in despite of me.

CLIFFORD.
King Henry, be thy title right or wrong,
Lord Clifford vows to fight in thy defence.
May that ground gape and swallow me alive,
Where I shall kneel to him that slew my father!

KING HENRY.
O Clifford, how thy words revive my heart!

YORK.
Henry of Lancaster, resign thy crown.
What mutter you, or what conspire you, lords?

WARWICK.
Do right unto this princely Duke of York,
Or I will fill the house with armed men,
And over the chair of state where now he sits,
Write up his title with usurping blood.

[_He stamps with his foot, and the Soldiers show themselves._]

KING HENRY.
My Lord of Warwick, hear but one word:
Let me for this my lifetime reign as king.

YORK.
Confirm the crown to me, and to mine heirs,
And thou shalt reign in quiet while thou liv’st.

KING HENRY.
I am content. Richard Plantagenet,
Enjoy the kingdom after my decease.

CLIFFORD.
What wrong is this unto the Prince your son!

WARWICK.
What good is this to England and himself!

WESTMORELAND.
Base, fearful, and despairing Henry!

CLIFFORD.
How hast thou injured both thyself and us!

WESTMORELAND.
I cannot stay to hear these articles.

NORTHUMBERLAND.
Nor I.

CLIFFORD.
Come, cousin, let us tell the Queen these news.

WESTMORELAND.
Farewell, faint-hearted and degenerate king,
In whose cold blood no spark of honour bides.

NORTHUMBERLAND.
Be thou a prey unto the house of York,
And die in bands for this unmanly deed!

CLIFFORD.
In dreadful war mayst thou be overcome,
Or live in peace abandoned and despised!

[_Exeunt Westmoreland, Northumberland and Clifford._]

WARWICK.
Turn this way, Henry, and regard them not.

EXETER.
They seek revenge, and therefore will not yield.

KING HENRY.
Ah, Exeter!

WARWICK.
Why should you sigh, my lord?

KING HENRY.
Not for myself, Lord Warwick, but my son,
Whom I unnaturally shall disinherit.
But be it as it may, [_To York._] I here entail
The crown to thee and to thine heirs for ever;
Conditionally, that here thou take an oath
To cease this civil war, and whilst I live,
To honour me as thy king and sovereign,
And neither by treason nor hostility
To seek to put me down and reign thyself.

YORK.
This oath I willingly take and will perform.

[_Coming from the throne._]

WARWICK.
Long live King Henry! Plantagenet, embrace him.

KING HENRY.
And long live thou, and these thy forward sons!

YORK.
Now York and Lancaster are reconciled.

EXETER.
Accursed be he that seeks to make them foes!

Sennet. Here they come down.

YORK.
Farewell, my gracious lord. I’ll to my castle.

WARWICK.
And I’ll keep London with my soldiers.

NORFOLK.
And I to Norfolk with my followers.

MONTAGUE.
And I unto the sea from whence I came.

[_Exeunt York and his Sons, Warwick, Norfolk, Montague and their
Soldiers._]

KING HENRY.
And I with grief and sorrow to the court.

Enter Queen Margaret and the Prince of Wales.

EXETER.
Here comes the Queen, whose looks bewray her anger.
I’ll steal away.

KING HENRY.
Exeter, so will I.

[_Going._]

QUEEN MARGARET.
Nay, go not from me; I will follow thee.

KING HENRY.
Be patient, gentle Queen, and I will stay.

QUEEN MARGARET.
Who can be patient in such extremes?
Ah, wretched man, would I had died a maid,
And never seen thee, never borne thee son,
Seeing thou hast proved so unnatural a father.
Hath he deserved to lose his birthright thus?
Hadst thou but loved him half so well as I,
Or felt that pain which I did for him once,
Or nourished him as I did with my blood,
Thou wouldst have left thy dearest heart-blood there,
Rather than have made that savage duke thine heir
And disinherited thine only son.

PRINCE EDWARD.
Father, you cannot disinherit me.
If you be king, why should not I succeed?

KING HENRY.
Pardon me, Margaret; pardon me, sweet son.
The Earl of Warwick and the Duke enforced me.

QUEEN MARGARET.
Enforced thee! Art thou King, and wilt be forced?
I shame to hear thee speak. Ah, timorous wretch,
Thou hast undone thyself, thy son, and me,
And given unto the house of York such head
As thou shalt reign but by their sufferance.
To entail him and his heirs unto the crown,
What is it but to make thy sepulchre
And creep into it far before thy time?
Warwick is Chancellor and the lord of Calais;
Stern Falconbridge commands the narrow seas;
The Duke is made Protector of the realm;
And yet shalt thou be safe? Such safety finds
The trembling lamb environed with wolves.
Had I been there, which am a silly woman,
The soldiers should have tossed me on their pikes
Before I would have granted to that act.
But thou prefer’st thy life before thine honour.
And seeing thou dost, I here divorce myself
Both from thy table, Henry, and thy bed,
Until that act of parliament be repealed
Whereby my son is disinherited.
The northern lords that have forsworn thy colours
Will follow mine if once they see them spread;
And spread they shall be, to thy foul disgrace
And utter ruin of the house of York.
Thus do I leave thee. Come, son, let’s away:
Our army is ready; come, we’ll after them.

KING HENRY.
Stay, gentle Margaret, and hear me speak.

QUEEN MARGARET.
Thou hast spoke too much already. Get thee gone.

KING HENRY.
Gentle son Edward, thou wilt stay with me?

QUEEN MARGARET.
Ay, to be murdered by his enemies.

PRINCE EDWARD.
When I return with victory from the field
I’ll see your Grace. Till then I’ll follow her.

QUEEN MARGARET.
Come, son, away; we may not linger thus.

[_Exeunt Queen Margaret and the Prince._]

KING HENRY.
Poor queen! How love to me and to her son
Hath made her break out into terms of rage!
Revenged may she be on that hateful Duke,
Whose haughty spirit, winged with desire,
Will cost my crown, and like an empty eagle
Tire on the flesh of me and of my son.
The loss of those three lords torments my heart.
I’ll write unto them and entreat them fair.
Come, cousin, you shall be the messenger.

EXETER.
And I, I hope, shall reconcile them all.

[_Flourish. Exeunt._]

SCENE II. Sandal Castle

Enter Edward, Richard and Montague.

RICHARD.
Brother, though I be youngest, give me leave.

EDWARD.
No, I can better play the orator.

MONTAGUE.
But I have reasons strong and forcible.

Enter the Duke of York.

YORK.
Why, how now, sons and brother, at a strife?
What is your quarrel? How began it first?

EDWARD.
No quarrel, but a slight contention.

YORK.
About what?

RICHARD.
About that which concerns your Grace and us:
The crown of England, father, which is yours.

YORK.
Mine, boy? Not till King Henry be dead.

RICHARD.
Your right depends not on his life or death.

EDWARD.
Now you are heir, therefore enjoy it now.
By giving the house of Lancaster leave to breathe,
It will outrun you, father, in the end.

YORK.
I took an oath that he should quietly reign.

EDWARD.
But for a kingdom any oath may be broken.
I would break a thousand oaths to reign one year.

RICHARD.
No; God forbid your Grace should be forsworn.

YORK.
I shall be, if I claim by open war.

RICHARD.
I’ll prove the contrary if you’ll hear me speak.

YORK.
Thou canst not, son; it is impossible.

RICHARD.
An oath is of no moment, being not took
Before a true and lawful magistrate
That hath authority over him that swears.
Henry had none, but did usurp the place;
Then, seeing ’twas he that made you to depose,
Your oath, my lord, is vain and frivolous.
Therefore, to arms! And, father, do but think
How sweet a thing it is to wear a crown,
Within whose circuit is Elysium
And all that poets feign of bliss and joy.
Why do we linger thus? I cannot rest
Until the white rose that I wear be dyed
Even in the lukewarm blood of Henry’s heart.

YORK.
Richard, enough; I will be king, or die.
Brother, thou shalt to London presently,
And whet on Warwick to this enterprise.
Thou, Richard, shalt to the Duke of Norfolk
And tell him privily of our intent.
You, Edward, shall unto my Lord Cobham,
With whom the Kentishmen will willingly rise.
In them I trust; for they are soldiers,
Witty, courteous, liberal, full of spirit.
While you are thus employed, what resteth more
But that I seek occasion how to rise,
And yet the King not privy to my drift,
Nor any of the house of Lancaster?

Enter a Messenger.

But stay. What news? Why com’st thou in such post?

MESSENGER.
The Queen, with all the northern earls and lords
Intend here to besiege you in your castle.
She is hard by with twenty thousand men;
And therefore fortify your hold, my lord.

YORK.
Ay, with my sword. What, think’st thou that we fear them?
Edward and Richard, you shall stay with me;
My brother Montague shall post to London.
Let noble Warwick, Cobham, and the rest,
Whom we have left protectors of the King,
With powerful policy strengthen themselves,
And trust not simple Henry nor his oaths.

MONTAGUE.
Brother, I go; I’ll win them, fear it not.
And thus most humbly I do take my leave.

[_Exit._]

Enter Sir John and Sir Hugh Mortimer.

YORK.
Sir John and Sir Hugh Mortimer, mine uncles,
You are come to Sandal in a happy hour;
The army of the Queen mean to besiege us.

SIR JOHN.
She shall not need; we’ll meet her in the field.

YORK.
What, with five thousand men?

RICHARD.
Ay, with five hundred, father, for a need.
A woman’s general; what should we fear?

[_A march afar off._]

EDWARD.
I hear their drums. Let’s set our men in order,
And issue forth and bid them battle straight.

YORK.
Five men to twenty! Though the odds be great,
I doubt not, uncle, of our victory.
Many a battle have I won in France
Whenas the enemy hath been ten to one.
Why should I not now have the like success?

[_Alarum. Exeunt._]

SCENE III. Plains near Sandal Castle

Alarums. Enter Rutland and his Tutor.

RUTLAND.
Ah, whither shall I fly to scape their hands?
Ah, tutor, look where bloody Clifford comes.

Enter Clifford and Soldiers.

CLIFFORD.
Chaplain, away! Thy priesthood saves thy life.
As for the brat of this accursed duke
Whose father slew my father, he shall die.

TUTOR.
And I, my lord, will bear him company.

CLIFFORD.
Soldiers, away with him!

TUTOR.
Ah, Clifford, murder not this innocent child,
Lest thou be hated both of God and man.

[_Exit, dragged off by Soldiers._]

CLIFFORD.
How now? Is he dead already? Or is it fear
That makes him close his eyes? I’ll open them.

RUTLAND.
So looks the pent-up lion o’er the wretch
That trembles under his devouring paws;
And so he walks, insulting o’er his prey,
And so he comes to rend his limbs asunder.
Ah, gentle Clifford, kill me with thy sword,
And not with such a cruel threat’ning look.
Sweet Clifford, hear me speak before I die.
I am too mean a subject for thy wrath;
Be thou revenged on men, and let me live.

CLIFFORD.
In vain thou speak’st, poor boy; my father’s blood
Hath stopped the passage where thy words should enter.

RUTLAND.
Then let my father’s blood open it again;
He is a man, and, Clifford, cope with him.

CLIFFORD.
Had I thy brethren here, their lives and thine
Were not revenge sufficient for me.
No, if I digged up thy forefathers’ graves
And hung their rotten coffins up in chains,
It could not slake mine ire nor ease my heart.
The sight of any of the house of York
Is as a fury to torment my soul;
And till I root out their accursed line
And leave not one alive, I live in hell.
Therefore—

[_Lifting his hand._]

RUTLAND.
O, let me pray before I take my death!
To thee I pray; sweet Clifford, pity me!

CLIFFORD.
Such pity as my rapier’s point affords.

RUTLAND.
I never did thee harm; why wilt thou slay me?

CLIFFORD.
Thy father hath.

RUTLAND.
But ’twas ere I was born.
Thou hast one son; for his sake pity me,
Lest in revenge thereof, sith God is just,
He be as miserably slain as I.
Ah, let me live in prison all my days,
And when I give occasion of offence
Then let me die, for now thou hast no cause.

CLIFFORD.
No cause? Thy father slew my father; therefore die.

[_Clifford stabs him._]

RUTLAND.
_Di faciant laudis summa sit ista tuae!_

[_Dies._]

CLIFFORD.
Plantagenet! I come, Plantagenet!
And this thy son’s blood cleaving to my blade
Shall rust upon my weapon till thy blood,
Congealed with this, do make me wipe off both.

[_Exit._]

SCENE IV. The Same

Alarum. Enter Richard, Duke of York.

YORK.
The army of the Queen hath got the field.
My uncles both are slain in rescuing me;
And all my followers to the eager foe
Turn back and fly like ships before the wind,
Or lambs pursued by hunger-starved wolves.
My sons, God knows what hath bechanced them;
But this I know, they have demeaned themselves
Like men born to renown by life or death.
Three times did Richard make a lane to me,
And thrice cried “Courage, father, fight it out!”
And full as oft came Edward to my side
With purple falchion painted to the hilt
In blood of those that had encountered him;
And when the hardiest warriors did retire,
Richard cried “Charge, and give no foot of ground!”
And cried “A crown, or else a glorious tomb!
A sceptre, or an earthly sepulchre!”
With this we charged again; but, out, alas!
We budged again, as I have seen a swan
With bootless labour swim against the tide
And spend her strength with over-matching waves.

[_A short alarum within._]

Ah, hark, the fatal followers do pursue,
And I am faint and cannot fly their fury;
And were I strong, I would not shun their fury.
The sands are numbered that makes up my life;
Here must I stay, and here my life must end.

Enter Queen Margaret, Clifford, Northumberland, the young Prince Edward
and Soldiers.

Come, bloody Clifford, rough Northumberland,
I dare your quenchless fury to more rage.
I am your butt, and I abide your shot.

NORTHUMBERLAND.
Yield to our mercy, proud Plantagenet.

CLIFFORD.
Ay, to such mercy as his ruthless arm
With downright payment showed unto my father.
Now Phaëthon hath tumbled from his car,
And made an evening at the noontide prick.

YORK.
My ashes, as the phoenix, may bring forth
A bird that will revenge upon you all;
And in that hope I throw mine eyes to heaven,
Scorning whate’er you can afflict me with.
Why come you not? What, multitudes, and fear?

CLIFFORD.
So cowards fight when they can fly no further;
So doves do peck the falcon’s piercing talons;
So desperate thieves, all hopeless of their lives,
Breathe out invectives ’gainst the officers.

YORK.
O Clifford, but bethink thee once again,
And in thy thought o’errun my former time;
And, if thou canst for blushing, view this face,
And bite thy tongue, that slanders him with cowardice
Whose frown hath made thee faint and fly ere this.

CLIFFORD.
I will not bandy with thee word for word,
But buckle with thee blows twice two for one.

QUEEN MARGARET.
Hold, valiant Clifford; for a thousand causes
I would prolong awhile the traitor’s life.
Wrath makes him deaf; speak thou, Northumberland.

NORTHUMBERLAND.
Hold, Clifford, do not honour him so much
To prick thy finger, though to wound his heart.
What valour were it, when a cur doth grin,
For one to thrust his hand between his teeth,
When he might spurn him with his foot away?
It is war’s prize to take all vantages,
And ten to one is no impeach of valour.

[_They lay hands on York, who struggles._]

CLIFFORD.
Ay, ay, so strives the woodcock with the gin.

NORTHUMBERLAND.
So doth the cony struggle in the net.

[_York is taken prisoner._]

YORK.
So triumph thieves upon their conquered booty;
So true men yield, with robbers so o’ermatched.

NORTHUMBERLAND.
What would your Grace have done unto him now?

QUEEN MARGARET.
Brave warriors, Clifford and Northumberland,
Come, make him stand upon this molehill here,
That raught at mountains with outstretched arms,
Yet parted but the shadow with his hand.
What, was it you that would be England’s king?
Was ’t you that revelled in our parliament
And made a preachment of your high descent?
Where are your mess of sons to back you now,
The wanton Edward and the lusty George?
And where’s that valiant crook-back prodigy,
Dicky your boy, that with his grumbling voice
Was wont to cheer his dad in mutinies?
Or, with the rest, where is your darling Rutland?
Look, York, I stained this napkin with the blood
That valiant Clifford with his rapier’s point
Made issue from the bosom of the boy;
And if thine eyes can water for his death,
I give thee this to dry thy cheeks withal.
Alas, poor York, but that I hate thee deadly
I should lament thy miserable state.
I prithee grieve to make me merry, York;
Stamp, rave, and fret, that I may sing and dance.
What, hath thy fiery heart so parched thine entrails
That not a tear can fall for Rutland’s death?
Why art thou patient, man? Thou shouldst be mad;
And I, to make thee mad, do mock thee thus.
Stamp, rave, and fret, that I may sing and dance.
Thou would’st be fee’d, I see, to make me sport;
York cannot speak unless he wear a crown.
A crown for York! And, lords, bow low to him.
Hold you his hands whilst I do set it on.

[_Putting a paper crown on his head._]

Ay, marry, sir, now looks he like a king.
Ay, this is he that took King Henry’s chair,
And this is he was his adopted heir.
But how is it that great Plantagenet
Is crowned so soon and broke his solemn oath?
As I bethink me, you should not be king
Till our King Henry had shook hands with Death.
And will you pale your head in Henry’s glory,
And rob his temples of the diadem,
Now in his life, against your holy oath?
O, ’tis a fault too too unpardonable.
Off with the crown, and, with the crown, his head;
And whilst we breathe, take time to do him dead.

CLIFFORD.
That is my office, for my father’s sake.

QUEEN MARGARET.
Nay, stay; let’s hear the orisons he makes.

YORK.
She-wolf of France, but worse than wolves of France,
Whose tongue more poisons than the adder’s tooth!
How ill-beseeming is it in thy sex
To triumph like an Amazonian trull
Upon their woes whom Fortune captivates!
But that thy face is vizard-like, unchanging,
Made impudent with use of evil deeds,
I would assay, proud queen, to make thee blush.
To tell thee whence thou cam’st, of whom derived,
Were shame enough to shame thee, wert thou not shameless.
Thy father bears the type of King of Naples,
Of both the Sicils, and Jerusalem,
Yet not so wealthy as an English yeoman.
Hath that poor monarch taught thee to insult?
It needs not, nor it boots thee not, proud queen;
Unless the adage must be verified,
That beggars mounted run their horse to death.
’Tis beauty that doth oft make women proud;
But God he knows thy share thereof is small.
’Tis virtue that doth make them most admired;
The contrary doth make thee wondered at.
’Tis government that makes them seem divine;
The want thereof makes thee abominable.
Thou art as opposite to every good
As the Antipodes are unto us,
Or as the south to the Septentrion.
O tiger’s heart wrapped in a woman’s hide!
How couldst thou drain the life-blood of the child,
To bid the father wipe his eyes withal,
And yet be seen to bear a woman’s face?
Women are soft, mild, pitiful, and flexible;
Thou stern, obdurate, flinty, rough, remorseless.
Bid’st thou me rage? Why, now thou hast thy wish:
Wouldst have me weep? Why, now thou hast thy will;
For raging wind blows up incessant showers,
And when the rage allays, the rain begins.
These tears are my sweet Rutland’s obsequies,
And every drop cries vengeance for his death
’Gainst thee, fell Clifford, and thee, false Frenchwoman.

NORTHUMBERLAND.
Beshrew me, but his passion moves me so
That hardly can I check my eyes from tears.

YORK.
That face of his the hungry cannibals
Would not have touched, would not have stained with blood;
But you are more inhuman, more inexorable,
O, ten times more than tigers of Hyrcania.
See, ruthless queen, a hapless father’s tears.
This cloth thou dipped’st in blood of my sweet boy,
And I with tears do wash the blood away.
Keep thou the napkin, and go boast of this;
And if thou tell’st the heavy story right,
Upon my soul, the hearers will shed tears;
Yea, even my foes will shed fast-falling tears
And say “Alas, it was a piteous deed.”
There, take the crown, and with the crown my curse;
And in thy need such comfort come to thee
As now I reap at thy too cruel hand!
Hard-hearted Clifford, take me from the world,
My soul to heaven, my blood upon your heads!

NORTHUMBERLAND.
Had he been slaughter-man to all my kin,
I should not for my life but weep with him,
To see how inly sorrow gripes his soul.

QUEEN MARGARET.
What, weeping-ripe, my Lord Northumberland?
Think but upon the wrong he did us all,
And that will quickly dry thy melting tears.

CLIFFORD.
Here’s for my oath, here’s for my father’s death.

[_Stabbing him._]

QUEEN MARGARET.
And here’s to right our gentle-hearted king.

[_Stabbing him._]

YORK.
Open thy gate of mercy, gracious God!
My soul flies through these wounds to seek out Thee.

[_Dies._]

QUEEN MARGARET.
Off with his head, and set it on York gates;
So York may overlook the town of York.

[_Flourish. Exeunt._]




ACT II

SCENE I. A plain near Mortimer’s Cross in Herefordshire


A march. Enter Edward and Richard and their power.

EDWARD.
I wonder how our princely father scaped,
Or whether he be scaped away or no
From Clifford’s and Northumberland’s pursuit.
Had he been ta’en, we should have heard the news;
Had he been slain, we should have heard the news;
Or had he scaped, methinks we should have heard
The happy tidings of his good escape.
How fares my brother? Why is he so sad?

RICHARD.
I cannot joy until I be resolved
Where our right valiant father is become.
I saw him in the battle range about,
And watched him how he singled Clifford forth.
Methought he bore him in the thickest troop
As doth a lion in a herd of neat;
Or as a bear, encompassed round with dogs,
Who having pinched a few and made them cry,
The rest stand all aloof and bark at him.
So fared our father with his enemies;
So fled his enemies my warlike father.
Methinks ’tis pride enough to be his son.
See how the morning opes her golden gates
And takes her farewell of the glorious sun.
How well resembles it the prime of youth,
Trimmed like a younker prancing to his love!

EDWARD.
Dazzle mine eyes, or do I see three suns?

RICHARD.
Three glorious suns, each one a perfect sun;
Not separated with the racking clouds,
But severed in a pale clear-shining sky.
See, see, they join, embrace, and seem to kiss,
As if they vowed some league inviolable.
Now are they but one lamp, one light, one sun.
In this the heaven figures some event.

EDWARD.
’Tis wondrous strange, the like yet never heard of.
I think it cites us, brother, to the field,
That we, the sons of brave Plantagenet,
Each one already blazing by our meeds,
Should notwithstanding join our lights together,
And overshine the earth, as this the world.
Whate’er it bodes, henceforward will I bear
Upon my target three fair shining suns.

RICHARD.
Nay, bear three daughters: by your leave I speak it,
You love the breeder better than the male.

Enter a Messenger, blowing.

But what art thou, whose heavy looks foretell
Some dreadful story hanging on thy tongue?

MESSENGER.
Ah, one that was a woeful looker-on
When as the noble Duke of York was slain,
Your princely father and my loving lord.

EDWARD.
O, speak no more, for I have heard too much!

RICHARD.
Say how he died, for I will hear it all.

MESSENGER.
Environed he was with many foes,
And stood against them as the hope of Troy
Against the Greeks that would have entered Troy.
But Hercules himself must yield to odds;
And many strokes, though with a little axe,
Hews down and fell the hardest-timbered oak.
By many hands your father was subdued,
But only slaughtered by the ireful arm
Of unrelenting Clifford and the Queen,
Who crowned the gracious duke in high despite,
Laughed in his face; and when with grief he wept,
The ruthless Queen gave him to dry his cheeks
A napkin steeped in the harmless blood
Of sweet young Rutland, by rough Clifford slain.
And after many scorns, many foul taunts,
They took his head, and on the gates of York
They set the same; and there it doth remain,
The saddest spectacle that e’er I viewed.

EDWARD.
Sweet Duke of York, our prop to lean upon,
Now thou art gone, we have no staff, no stay.
O Clifford, boisterous Clifford, thou hast slain
The flower of Europe for his chivalry;
And treacherously hast thou vanquished him,
For hand to hand he would have vanquished thee.
Now my soul’s palace is become a prison.
Ah, would she break from hence, that this my body
Might in the ground be closed up in rest!
For never henceforth shall I joy again;
Never, O, never, shall I see more joy!

RICHARD.
I cannot weep, for all my body’s moisture
Scarce serves to quench my furnace-burning heart;
Nor can my tongue unload my heart’s great burthen,
For selfsame wind that I should speak withal
Is kindling coals that fires all my breast
And burns me up with flames that tears would quench.
To weep is to make less the depth of grief:
Tears, then, for babes; blows and revenge for me!
Richard, I bear thy name; I’ll venge thy death,
Or die renowned by attempting it.

EDWARD.
His name that valiant duke hath left with thee;
His dukedom and his chair with me is left.

RICHARD.
Nay, if thou be that princely eagle’s bird,
Show thy descent by gazing ’gainst the sun;
For chair and dukedom, throne and kingdom say,
Either that is thine, or else thou wert not his.

March. Enter Warwick, Marquess Montague and their army.

WARWICK.
How now, fair lords! What fare? What news abroad?

RICHARD.
Great Lord of Warwick, if we should recount
Our baleful news, and at each word’s deliverance
Stab poniards in our flesh till all were told,
The words would add more anguish than the wounds.
O valiant lord, the Duke of York is slain!

EDWARD.
O, Warwick, Warwick, that Plantagenet
Which held thee dearly as his soul’s redemption
Is by the stern Lord Clifford done to death.

WARWICK.
Ten days ago I drowned these news in tears,
And now, to add more measure to your woes,
I come to tell you things sith then befall’n.
After the bloody fray at Wakefield fought,
Where your brave father breathed his latest gasp,
Tidings, as swiftly as the posts could run,
Were brought me of your loss and his depart.
I, then in London, keeper of the King,
Mustered my soldiers, gathered flocks of friends,
And very well appointed, as I thought,
Marched toward Saint Albans to intercept the Queen,
Bearing the King in my behalf along;
For by my scouts I was advertised
That she was coming with a full intent
To dash our late decree in Parliament
Touching King Henry’s oath and your succession.
Short tale to make, we at Saint Albans met,
Our battles joined, and both sides fiercely fought.
But, whether ’twas the coldness of the King,
Who looked full gently on his warlike Queen,
That robbed my soldiers of their heated spleen,
Or whether ’twas report of her success;
Or more than common fear of Clifford’s rigour,
Who thunders to his captives blood and death,
I cannot judge; but, to conclude with truth,
Their weapons like to lightning came and went;
Our soldiers’, like the night-owl’s lazy flight,
Or like an idle thresher with a flail,
Fell gently down, as if they struck their friends.
I cheered them up with justice of our cause,
With promise of high pay and great rewards,
But all in vain; they had no heart to fight,
And we in them no hope to win the day;
So that we fled: the King unto the Queen;
Lord George your brother, Norfolk, and myself,
In haste, post-haste, are come to join with you;
For in the Marches here we heard you were,
Making another head to fight again.

EDWARD.
Where is the Duke of Norfolk, gentle Warwick?
And when came George from Burgundy to England?

WARWICK.
Some six miles off the Duke is with the soldiers;
And for your brother, he was lately sent
From your kind aunt, Duchess of Burgundy,
With aid of soldiers to this needful war.

RICHARD.
’Twas odds, belike, when valiant Warwick fled.
Oft have I heard his praises in pursuit,
But ne’er till now his scandal of retire.

WARWICK.
Nor now my scandal, Richard, dost thou hear;
For thou shalt know this strong right hand of mine
Can pluck the diadem from faint Henry’s head
And wring the awful sceptre from his fist,
Were he as famous and as bold in war
As he is famed for mildness, peace, and prayer.

RICHARD.
I know it well, Lord Warwick; blame me not.
’Tis love I bear thy glories makes me speak.
But in this troublous time what’s to be done?
Shall we go throw away our coats of steel
And wrap our bodies in black mourning gowns,
Numbering our Ave-Maries with our beads?
Or shall we on the helmets of our foes
Tell our devotion with revengeful arms?
If for the last, say ay, and to it, lords.

WARWICK.
Why, therefore Warwick came to seek you out,
And therefore comes my brother Montague.
Attend me, lords. The proud insulting Queen,
With Clifford and the haught Northumberland,
And of their feather many moe proud birds,
Have wrought the easy-melting King like wax.
He swore consent to your succession,
His oath enrolled in the Parliament;
And now to London all the crew are gone,
To frustrate both his oath and what beside
May make against the house of Lancaster.
Their power, I think, is thirty thousand strong.
Now, if the help of Norfolk and myself,
With all the friends that thou, brave Earl of March,
Amongst the loving Welshmen canst procure,
Will but amount to five and twenty thousand,
Why, _via_, to London will we march amain,
And once again bestride our foaming steeds,
And once again cry “Charge upon our foes!”
But never once again turn back and fly.

RICHARD.
Ay, now methinks I hear great Warwick speak.
Ne’er may he live to see a sunshine day
That cries “Retire,” if Warwick bid him stay.

EDWARD.
Lord Warwick, on thy shoulder will I lean;
And when thou fail’st—as God forbid the hour!—
Must Edward fall, which peril heaven forfend!

WARWICK.
No longer Earl of March, but Duke of York.
The next degree is England’s royal throne;
For King of England shalt thou be proclaimed
In every borough as we pass along,
And he that throws not up his cap for joy
Shall for the fault make forfeit of his head.
King Edward, valiant Richard, Montague,
Stay we no longer dreaming of renown,
But sound the trumpets and about our task.

RICHARD.
Then, Clifford, were thy heart as hard as steel,
As thou hast shown it flinty by thy deeds,
I come to pierce it, or to give thee mine.

EDWARD.
Then strike up, drums! God and Saint George for us!

Enter a Messenger.

WARWICK.
How now, what news?

MESSENGER.
The Duke of Norfolk sends you word by me,
The Queen is coming with a puissant host,
And craves your company for speedy counsel.

WARWICK.
Why then it sorts; brave warriors, let’s away.

[_Exeunt._]

SCENE II. Before York

Flourish. Enter King Henry, Queen Margaret, the Prince of Wales,
Clifford and Northumberland with drums and trumpets.

QUEEN MARGARET.
Welcome, my lord, to this brave town of York.
Yonder’s the head of that arch-enemy
That sought to be encompassed with your crown.
Doth not the object cheer your heart, my lord?

KING HENRY.
Ay, as the rocks cheer them that fear their wrack!
To see this sight, it irks my very soul.
Withhold revenge, dear God! ’Tis not my fault,
Nor wittingly have I infringed my vow.

CLIFFORD.
My gracious liege, this too much lenity
And harmful pity must be laid aside.
To whom do lions cast their gentle looks?
Not to the beast that would usurp their den.
Whose hand is that the forest bear doth lick?
Not his that spoils her young before her face.
Who scapes the lurking serpent’s mortal sting?
Not he that sets his foot upon her back.
The smallest worm will turn, being trodden on,
And doves will peck in safeguard of their brood.
Ambitious York did level at thy crown,
Thou smiling while he knit his angry brows.
He, but a duke, would have his son a king,
And raise his issue like a loving sire;
Thou, being a king, blest with a goodly son,
Didst yield consent to disinherit him,
Which argued thee a most unloving father.
Unreasonable creatures feed their young;
And though man’s face be fearful to their eyes,
Yet, in protection of their tender ones,
Who hath not seen them, even with those wings
Which sometime they have used with fearful flight,
Make war with him that climbed unto their nest,
Offering their own lives in their young’s defence?
For shame, my liege, make them your precedent.
Were it not pity that this goodly boy
Should lose his birthright by his father’s fault,
And long hereafter say unto his child,
“What my great-grandfather and grandsire got,
My careless father fondly gave away?”
Ah, what a shame were this! Look on the boy,
And let his manly face, which promiseth
Successful fortune, steel thy melting heart
To hold thine own and leave thine own with him.

KING HENRY.
Full well hath Clifford played the orator,
Inferring arguments of mighty force.
But, Clifford, tell me, didst thou never hear
That things ill got had ever bad success?
And happy always was it for that son
Whose father for his hoarding went to hell?
I’ll leave my son my virtuous deeds behind,
And would my father had left me no more;
For all the rest is held at such a rate
As brings a thousand-fold more care to keep
Than in possession any jot of pleasure.
Ah, cousin York, would thy best friends did know
How it doth grieve me that thy head is here!

QUEEN MARGARET.
My lord, cheer up your spirits; our foes are nigh,
And this soft courage makes your followers faint.
You promised knighthood to our forward son.
Unsheathe your sword and dub him presently.—
Edward, kneel down.

KING HENRY.
Edward Plantagenet, arise a knight;
And learn this lesson: draw thy sword in right.

PRINCE EDWARD.
My gracious father, by your kingly leave,
I’ll draw it as apparent to the crown,
And in that quarrel use it to the death.

CLIFFORD.
Why, that is spoken like a toward prince.

Enter a Messenger.

MESSENGER.
Royal commanders, be in readiness;
For with a band of thirty thousand men
Comes Warwick, backing of the Duke of York,
And in the towns, as they do march along,
Proclaims him king, and many fly to him.
Darraign your battle, for they are at hand.

CLIFFORD.
I would your highness would depart the field.
The Queen hath best success when you are absent.

QUEEN MARGARET.
Ay, good my lord, and leave us to our fortune.

KING HENRY.
Why, that’s my fortune too; therefore I’ll stay.

NORTHUMBERLAND.
Be it with resolution then to fight.

PRINCE EDWARD.
My royal father, cheer these noble lords,
And hearten those that fight in your defence.
Unsheathe your sword, good father; cry “Saint George!”

March. Enter Edward, George, Richard, Warwick, Norfolk, Montague and
Soldiers.

EDWARD.
Now, perjured Henry, wilt thou kneel for grace
And set thy diadem upon my head,
Or bide the mortal fortune of the field?

QUEEN MARGARET.
Go rate thy minions, proud insulting boy!
Becomes it thee to be thus bold in terms
Before thy sovereign and thy lawful king?

EDWARD.
I am his king, and he should bow his knee.
I was adopted heir by his consent.
Since when, his oath is broke; for, as I hear,
You that are king, though he do wear the crown,
Have caused him by new act of Parliament
To blot out me and put his own son in.

CLIFFORD.
And reason too:
Who should succeed the father but the son?

RICHARD.
Are you there, butcher? O, I cannot speak!

CLIFFORD.
Ay, crook-back; here I stand, to answer thee,
Or any he, the proudest of thy sort.

RICHARD.
’Twas you that killed young Rutland, was it not?

CLIFFORD.
Ay, and old York, and yet not satisfied.

RICHARD.
For God’s sake, lords, give signal to the fight.

WARWICK.
What sayst thou, Henry, wilt thou yield the crown?

QUEEN MARGARET.
Why, how now, long-tongued Warwick, dare you speak?
When you and I met at Saint Albans last,
Your legs did better service than your hands.

WARWICK.
Then ’twas my turn to fly, and now ’tis thine.

CLIFFORD.
You said so much before, and yet you fled.

WARWICK.
’Twas not your valour, Clifford, drove me thence.

NORTHUMBERLAND.
No, nor your manhood that durst make you stay.

RICHARD.
Northumberland, I hold thee reverently.
Break off the parley; for scarce I can refrain
The execution of my big-swoln heart
Upon that Clifford, that cruel child-killer.

CLIFFORD.
I slew thy father; call’st thou him a child?

RICHARD.
Ay, like a dastard and a treacherous coward,
As thou didst kill our tender brother Rutland,
But ere sunset I’ll make thee curse the deed.

KING HENRY.
Have done with words, my lords, and hear me speak.

QUEEN MARGARET.
Defy them then, or else hold close thy lips.

KING HENRY.
I prithee, give no limits to my tongue.
I am a king, and privileged to speak.

CLIFFORD.
My liege, the wound that bred this meeting here
Cannot be cured by words; therefore be still.

RICHARD.
Then, executioner, unsheathe thy sword.
By Him that made us all, I am resolved
That Clifford’s manhood lies upon his tongue.

EDWARD.
Say, Henry, shall I have my right, or no?
A thousand men have broke their fasts today
That ne’er shall dine unless thou yield the crown.

WARWICK.
If thou deny, their blood upon thy head;
For York in justice puts his armour on.

PRINCE EDWARD.
If that be right which Warwick says is right,
There is no wrong, but everything is right.

RICHARD.
Whoever got thee, there thy mother stands;
For well I wot thou hast thy mother’s tongue.

QUEEN MARGARET.
But thou art neither like thy sire nor dam,
But like a foul misshapen stigmatic,
Marked by the Destinies to be avoided,
As venom toads or lizards’ dreadful stings.

RICHARD.
Iron of Naples, hid with English gilt,
Whose father bears the title of a king,
As if a channel should be called the sea,
Sham’st thou not, knowing whence thou art extraught,
To let thy tongue detect thy base-born heart?

EDWARD.
A wisp of straw were worth a thousand crowns
To make this shameless callet know herself.
Helen of Greece was fairer far than thou,
Although thy husband may be Menelaus;
And ne’er was Agamemnon’s brother wronged
By that false woman as this king by thee.
His father revelled in the heart of France,
And tamed the King, and made the Dauphin stoop;
And had he matched according to his state,
He might have kept that glory to this day;
But when he took a beggar to his bed
And graced thy poor sire with his bridal day,
Even then that sunshine brewed a shower for him
That washed his father’s fortunes forth of France
And heaped sedition on his crown at home.
For what hath broached this tumult but thy pride?
Hadst thou been meek, our title still had slept;
And we, in pity of the gentle king,
Had slipped our claim until another age.

GEORGE.
But when we saw our sunshine made thy spring,
And that thy summer bred us no increase,
We set the axe to thy usurping root;
And though the edge hath something hit ourselves,
Yet know thou, since we have begun to strike,
We’ll never leave till we have hewn thee down
Or bathed thy growing with our heated bloods.

EDWARD.
And in this resolution I defy thee;
Not willing any longer conference,
Since thou deniest the gentle King to speak.
Sound trumpets! Let our bloody colours wave;
And either victory or else a grave!

QUEEN MARGARET.
Stay, Edward.

EDWARD.
No, wrangling woman, we’ll no longer stay.
These words will cost ten thousand lives this day.

[_Exeunt._]

SCENE III. A field of battle between Towton and Saxton, in Yorkshire

Alarums. Excursions. Enter Warwick.

WARWICK.
Forspent with toil, as runners with a race,
I lay me down a little while to breathe;
For strokes received, and many blows repaid,
Have robbed my strong-knit sinews of their strength,
And spite of spite, needs must I rest awhile.

Enter Edward, running.

EDWARD.
Smile, gentle heaven, or strike, ungentle death;
For this world frowns and Edward’s sun is clouded.

WARWICK.
How now, my lord, what hap? What hope of good?

Enter George.

GEORGE.
Our hap is loss, our hope but sad despair;
Our ranks are broke and ruin follows us.
What counsel give you? Whither shall we fly?

EDWARD.
Bootless is flight, they follow us with wings;
And weak we are and cannot shun pursuit.

Enter Richard.

RICHARD.
Ah, Warwick, why hast thou withdrawn thyself?
Thy brother’s blood the thirsty earth hath drunk,
Broached with the steely point of Clifford’s lance;
And in the very pangs of death he cried,
Like to a dismal clangor heard from far,
“Warwick, revenge! Brother, revenge my death!”
So, underneath the belly of their steeds,
That stained their fetlocks in his smoking blood,
The noble gentleman gave up the ghost.

WARWICK.
Then let the earth be drunken with our blood;
I’ll kill my horse because I will not fly.
Why stand we like soft-hearted women here,
Wailing our losses whiles the foe doth rage,
And look upon, as if the tragedy
Were played in jest by counterfeiting actors?
Here on my knee I vow to God above
I’ll never pause again, never stand still,
Till either death hath closed these eyes of mine,
Or Fortune given me measure of revenge.

EDWARD.
O Warwick, I do bend my knee with thine,
And in this vow do chain my soul to thine!
And, ere my knee rise from the earth’s cold face,
I throw my hands, mine eyes, my heart to Thee,
Thou setter up and plucker down of kings,
Beseeching Thee, if with Thy will it stands
That to my foes this body must be prey,
Yet that Thy brazen gates of heaven may ope,
And give sweet passage to my sinful soul.
Now, lords, take leave until we meet again,
Where’er it be, in heaven or in earth.

RICHARD.
Brother, give me thy hand; and, gentle Warwick,
Let me embrace thee in my weary arms.
I, that did never weep, now melt with woe
That winter should cut off our spring-time so.

WARWICK.
Away, away! Once more, sweet lords, farewell.

GEORGE.
Yet let us all together to our troops,
And give them leave to fly that will not stay,
And call them pillars that will stand to us;
And if we thrive, promise them such rewards
As victors wear at the Olympian games.
This may plant courage in their quailing breasts,
For yet is hope of life and victory.
Forslow no longer; make we hence amain.

[_Exeunt._]

SCENE IV. Another Part of the Field

Excursions. Enter Richard and Clifford.

RICHARD.
Now, Clifford, I have singled thee alone.
Suppose this arm is for the Duke of York,
And this for Rutland; both bound to revenge,
Wert thou environed with a brazen wall.

CLIFFORD.
Now, Richard, I am with thee here alone.
This is the hand that stabbed thy father York,
And this the hand that slew thy brother Rutland;
And here’s the heart that triumphs in their death
And cheers these hands that slew thy sire and brother
To execute the like upon thyself;
And so have at thee!

They fight. Warwick comes; Clifford flies.

RICHARD.
Nay, Warwick, single out some other chase;
For I myself will hunt this wolf to death.

[_Exeunt._]

SCENE V. Another Part of the Field

Enter King Henry.

KING HENRY.
This battle fares like to the morning’s war,
When dying clouds contend with growing light,
What time the shepherd, blowing of his nails,
Can neither call it perfect day nor night.
Now sways it this way, like a mighty sea
Forced by the tide to combat with the wind;
Now sways it that way, like the selfsame sea
Forced to retire by fury of the wind.
Sometime the flood prevails, and then the wind;
Now one the better, then another best,
Both tugging to be victors, breast to breast,
Yet neither conqueror nor conquered.
So is the equal poise of this fell war.
Here on this molehill will I sit me down.
To whom God will, there be the victory!
For Margaret my queen, and Clifford too,
Have chid me from the battle, swearing both
They prosper best of all when I am thence.
Would I were dead, if God’s good will were so;
For what is in this world but grief and woe?
O God! Methinks it were a happy life
To be no better than a homely swain;
To sit upon a hill, as I do now,
To carve out dials quaintly, point by point,
Thereby to see the minutes how they run:
How many make the hour full complete,
How many hours brings about the day,
How many days will finish up the year,
How many years a mortal man may live.
When this is known, then to divide the times:
So many hours must I tend my flock;
So many hours must I take my rest;
So many hours must I contemplate;
So many hours must I sport myself;
So many days my ewes have been with young;
So many weeks ere the poor fools will ean;
So many years ere I shall shear the fleece.
So minutes, hours, days, months, and years,
Passed over to the end they were created,
Would bring white hairs unto a quiet grave.
Ah, what a life were this! How sweet, how lovely!
Gives not the hawthorn bush a sweeter shade
To shepherds looking on their silly sheep
Than doth a rich embroidered canopy
To kings that fear their subjects’ treachery?
O, yes, it doth; a thousand-fold it doth.
And to conclude, the shepherd’s homely curds,
His cold thin drink out of his leather bottle,
His wonted sleep under a fresh tree’s shade,
All which secure and sweetly he enjoys,
Is far beyond a prince’s delicates—
His viands sparkling in a golden cup,
His body couched in a curious bed,
When care, mistrust, and treason waits on him.

Alarum. Enter a Son that hath killed his father, bringing in the dead
body.

SON.
Ill blows the wind that profits nobody.
This man, whom hand to hand I slew in fight,
May be possessed with some store of crowns;
And I, that haply take them from him now,
May yet ere night yield both my life and them
To some man else, as this dead man doth me.
Who’s this? O God! It is my father’s face,
Whom in this conflict I unwares have killed.
O heavy times, begetting such events!
From London by the King was I pressed forth;
My father, being the Earl of Warwick’s man,
Came on the part of York, pressed by his master;
And I, who at his hands received my life,
Have by my hands of life bereaved him.
Pardon me, God, I knew not what I did;
And pardon, father, for I knew not thee.
My tears shall wipe away these bloody marks,
And no more words till they have flowed their fill.

KING HENRY.
O piteous spectacle! O bloody times!
Whiles lions war and battle for their dens,
Poor harmless lambs abide their enmity.
Weep, wretched man, I’ll aid thee tear for tear;
And let our hearts and eyes, like civil war,
Be blind with tears and break o’ercharged with grief.

Enter a Father who has killed his son, with the body in his arms.

FATHER.
Thou that so stoutly hath resisted me,
Give me thy gold, if thou hast any gold,
For I have bought it with an hundred blows.
But let me see: is this our foeman’s face?
Ah, no, no, no; it is mine only son!
Ah, boy, if any life be left in thee,
Throw up thine eye! See, see what showers arise,
Blown with the windy tempest of my heart
Upon thy wounds, that kill mine eye and heart!
O, pity, God, this miserable age!
What stratagems, how fell, how butcherly,
Erroneous, mutinous, and unnatural,
This deadly quarrel daily doth beget!
O boy, thy father gave thee life too soon,
And hath bereft thee of thy life too late!

KING HENRY.
Woe above woe, grief more than common grief!
O that my death would stay these ruthful deeds!
O pity, pity, gentle heaven, pity!
The red rose and the white are on his face,
The fatal colours of our striving houses;
The one his purple blood right well resembles,
The other his pale cheeks, methinks, presenteth.
Wither one rose, and let the other flourish!
If you contend, a thousand lives must wither.

SON.
How will my mother for a father’s death
Take on with me and ne’er be satisfied!

FATHER.
How will my wife for slaughter of my son
Shed seas of tears and ne’er be satisfied!

KING HENRY.
How will the country for these woeful chances
Misthink the King and not be satisfied!

SON.
Was ever son so rued a father’s death?

FATHER.
Was ever father so bemoaned his son?

KING HENRY.
Was ever king so grieved for subjects’ woe?
Much is your sorrow, mine ten times so much.

SON.
I’ll bear thee hence, where I may weep my fill.

[_Exit with the body._]

FATHER.
These arms of mine shall be thy winding-sheet;
My heart, sweet boy, shall be thy sepulchre,
For from my heart thine image ne’er shall go.
My sighing breast shall be thy funeral bell;
And so obsequious will thy father be,
Even for the loss of thee, having no more,
As Priam was for all his valiant sons.
I’ll bear thee hence; and let them fight that will,
For I have murdered where I should not kill.

[_Exit with the body._]

KING HENRY.
Sad-hearted men, much overgone with care,
Here sits a king more woeful than you are.

Alarums. Excursions. Enter Queen Margaret, Prince of Wales and Exeter.

PRINCE EDWARD.
Fly, father, fly, for all your friends are fled,
And Warwick rages like a chafed bull.
Away, for death doth hold us in pursuit.

QUEEN MARGARET.
Mount you, my lord; towards Berwick post amain.
Edward and Richard, like a brace of greyhounds
Having the fearful flying hare in sight,
With fiery eyes sparkling for very wrath,
And bloody steel grasped in their ireful hands,
Are at our backs; and therefore hence amain.

EXETER.
Away, for vengeance comes along with them.
Nay, stay not to expostulate; make speed,
Or else come after; I’ll away before.

KING HENRY.
Nay, take me with thee, good sweet Exeter;
Not that I fear to stay, but love to go
Whither the Queen intends. Forward; away!

[_Exeunt._]

SCENE VI. Another Part of the Field

A loud alarum. Enter Clifford, wounded.

CLIFFORD.
Here burns my candle out; ay, here it dies,
Which whiles it lasted gave King Henry light.
O Lancaster, I fear thy overthrow
More than my body’s parting with my soul!
My love and fear glued many friends to thee;
And, now I fall, thy tough commixtures melts,
Impairing Henry, strengthening misproud York.
The common people swarm like summer flies;
And whither fly the gnats but to the sun?
And who shines now but Henry’s enemies?
O Phoebus, hadst thou never given consent
That Phaëthon should check thy fiery steeds,
Thy burning car never had scorched the earth!
And, Henry, hadst thou swayed as kings should do,
Or as thy father and his father did,
Giving no ground unto the house of York,
They never then had sprung like summer flies;
I, and ten thousand in this luckless realm
Had left no mourning widows for our death,
And thou this day hadst kept thy chair in peace.
For what doth cherish weeds but gentle air?
And what makes robbers bold but too much lenity?
Bootless are plaints, and cureless are my wounds;
No way to fly, nor strength to hold out flight.
The foe is merciless and will not pity,
For at their hands I have deserved no pity.
The air hath got into my deadly wounds,
And much effuse of blood doth make me faint.
Come, York and Richard, Warwick, and the rest;
I stabbed your fathers’ bosoms, split my breast.

[_He faints._]

Alarum and retreat. Enter Edward, George, Richard, Montague, Warwick
and Soldiers.

EDWARD.
Now breathe we, lords. Good fortune bids us pause
And smooth the frowns of war with peaceful looks.
Some troops pursue the bloody-minded Queen
That led calm Henry, though he were a king,
As doth a sail, filled with a fretting gust,
Command an argosy to stem the waves.
But think you, lords, that Clifford fled with them?

WARWICK.
No, ’tis impossible he should escape;
For, though before his face I speak the words,
Your brother Richard marked him for the grave,
And whereso’er he is, he’s surely dead.

[_Clifford groans and dies._]

RICHARD.
Whose soul is that which takes her heavy leave?
A deadly groan, like life and death’s departing.

EDWARD.
See who it is; and, now the battle’s ended,
If friend or foe, let him be gently used.

RICHARD.
Revoke that doom of mercy, for ’tis Clifford,
Who, not contented that he lopped the branch
In hewing Rutland when his leaves put forth,
But set his murdering knife unto the root
From whence that tender spray did sweetly spring,
I mean our princely father, Duke of York.

WARWICK.
From off the gates of York fetch down the head,
Your father’s head, which Clifford placed there;
Instead whereof let this supply the room.
Measure for measure must be answered.

EDWARD.
Bring forth that fatal screech-owl to our house,
That nothing sung but death to us and ours;
Now death shall stop his dismal threatening sound,
And his ill-boding tongue no more shall speak.

[_Soldiers bring the body forward._]

WARWICK.
I think his understanding is bereft.
Speak, Clifford, dost thou know who speaks to thee?
Dark cloudy death o’ershades his beams of life,
And he nor sees nor hears us, what we say.

RICHARD.
O, would he did, and so, perhaps, he doth!
’Tis but his policy to counterfeit,
Because he would avoid such bitter taunts
Which in the time of death he gave our father.

GEORGE.
If so thou think’st, vex him with eager words.

RICHARD.
Clifford, ask mercy, and obtain no grace.

EDWARD.
Clifford, repent in bootless penitence.

WARWICK.
Clifford, devise excuses for thy faults.

GEORGE.
While we devise fell tortures for thy faults.

RICHARD.
Thou didst love York, and I am son to York.

EDWARD.
Thou pitied’st Rutland, I will pity thee.

GEORGE.
Where’s Captain Margaret to fence you now?

WARWICK.
They mock thee, Clifford; swear as thou wast wont.

RICHARD.
What, not an oath? Nay then, the world goes hard
When Clifford cannot spare his friends an oath.
I know by that he’s dead; and, by my soul,
If this right hand would buy but two hours’ life,
That I in all despite might rail at him,
This hand should chop it off, and with the issuing blood
Stifle the villain whose unstaunched thirst
York and young Rutland could not satisfy.

WARWICK.
Ay, but he’s dead. Off with the traitor’s head,
And rear it in the place your father’s stands.
And now to London with triumphant march,
There to be crowned England’s royal king;
From whence shall Warwick cut the sea to France,
And ask the Lady Bona for thy queen.
So shalt thou sinew both these lands together,
And, having France thy friend, thou shalt not dread
The scattered foe that hopes to rise again;
For though they cannot greatly sting to hurt,
Yet look to have them buzz to offend thine ears.
First will I see the coronation,
And then to Brittany I’ll cross the sea
To effect this marriage, so it please my lord.

EDWARD.
Even as thou wilt, sweet Warwick, let it be;
For in thy shoulder do I build my seat,
And never will I undertake the thing
Wherein thy counsel and consent is wanting.
Richard, I will create thee Duke of Gloucester;
And George, of Clarence. Warwick, as ourself,
Shall do and undo as him pleaseth best.

RICHARD.
Let me be Duke of Clarence, George of Gloucester,
For Gloucester’s dukedom is too ominous.

WARWICK.
Tut, that’s a foolish observation.
Richard, be Duke of Gloucester. Now to London,
To see these honours in possession.

[_Exeunt._]




ACT III

SCENE I. A Forest in the North of England


Enter two Keepers with crossbows in their hands.

1 KEEPER.
Under this thick-grown brake we’ll shroud ourselves,
For through this laund anon the deer will come;
And in this covert will we make our stand,
Culling the principal of all the deer.

2 KEEPER.
I’ll stay above the hill, so both may shoot.

1 KEEPER.
That cannot be; the noise of thy crossbow
Will scare the herd, and so my shoot is lost.
Here stand we both, and aim we at the best;
And, for the time shall not seem tedious,
I’ll tell thee what befell me on a day
In this self place where now we mean to stand.

2 KEEPER.
Here comes a man; let’s stay till he be past.

Enter King Henry, disguised, with a prayer-book.

KING HENRY.
From Scotland am I stolen, even of pure love,
To greet mine own land with my wishful sight.
No, Harry, Harry, ’tis no land of thine;
Thy place is filled, thy sceptre wrung from thee,
Thy balm washed off wherewith thou wast anointed.
No bending knee will call thee Caesar now,
No humble suitors press to speak for right,
No, not a man comes for redress of thee;
For how can I help them and not myself?

1 KEEPER.
Ay, here’s a deer whose skin’s a keeper’s fee.
This is the quondam king; let’s seize upon him.

KING HENRY.
Let me embrace thee, sour adversity,
For wise men say it is the wisest course.

2 KEEPER.
Why linger we? Let us lay hands upon him.

1 KEEPER.
Forbear awhile; we’ll hear a little more.

KING HENRY.
My queen and son are gone to France for aid;
And, as I hear, the great commanding Warwick
Is thither gone to crave the French King’s sister
To wife for Edward. If this news be true,
Poor queen and son, your labour is but lost,
For Warwick is a subtle orator,
And Lewis a prince soon won with moving words.
By this account, then, Margaret may win him,
For she’s a woman to be pitied much.
Her sighs will make a batt’ry in his breast,
Her tears will pierce into a marble heart;
The tiger will be mild whiles she doth mourn,
And Nero will be tainted with remorse
To hear and see her plaints, her brinish tears.
Ay, but she’s come to beg, Warwick to give;
She on his left side craving aid for Henry;
He on his right asking a wife for Edward.
She weeps and says her Henry is deposed;
He smiles and says his Edward is installed;
That she, poor wretch, for grief can speak no more;
Whiles Warwick tells his title, smooths the wrong,
Inferreth arguments of mighty strength,
And in conclusion wins the King from her
With promise of his sister, and what else,
To strengthen and support King Edward’s place.
O Margaret, thus ’twill be; and thou, poor soul,
Art then forsaken, as thou went’st forlorn.

2 KEEPER.
Say, what art thou, that talk’st of kings and queens?

KING HENRY.
More than I seem, and less than I was born to:
A man at least, for less I should not be;
And men may talk of kings, and why not I?

2 KEEPER.
Ay, but thou talk’st as if thou wert a king.

KING HENRY.
Why, so I am, in mind; and that’s enough.

2 KEEPER.
But, if thou be a king, where is thy crown?

KING HENRY.
My crown is in my heart, not on my head;
Not decked with diamonds and Indian stones,
Not to be seen. My crown is called content;
A crown it is that seldom kings enjoy.

2 KEEPER.
Well, if you be a king crowned with content,
Your crown content and you must be contented
To go along with us; for, as we think,
You are the king King Edward hath deposed;
And we his subjects, sworn in all allegiance,
Will apprehend you as his enemy.

KING HENRY.
But did you never swear, and break an oath?

2 KEEPER.
No, never such an oath; nor will not now.

KING HENRY.
Where did you dwell when I was King of England?

2 KEEPER.
Here in this country, where we now remain.

KING HENRY.
I was anointed king at nine months old;
My father and my grandfather were kings,
And you were sworn true subjects unto me.
And tell me, then, have you not broke your oaths?

1 KEEPER.
No, for we were subjects but while you were king.

KING HENRY.
Why, am I dead? Do I not breathe a man?
Ah, simple men, you know not what you swear.
Look, as I blow this feather from my face,
And as the air blows it to me again,
Obeying with my wind when I do blow,
And yielding to another when it blows,
Commanded always by the greater gust,
Such is the lightness of you common men.
But do not break your oaths; for of that sin
My mild entreaty shall not make you guilty.
Go where you will, the King shall be commanded;
And be you kings; command, and I’ll obey.

1 KEEPER.
We are true subjects to the King, King Edward.

KING HENRY.
So would you be again to Henry
If he were seated as King Edward is.

1 KEEPER.
We charge you, in God’s name and the King’s
To go with us unto the officers.

KING HENRY.
In God’s name, lead; your king’s name be obeyed,
And what God will, that let your king perform;
And what he will, I humbly yield unto.

[_Exeunt._]

SCENE II. The Palace

Enter King Edward, Richard (Duke of Gloucester), George (Duke of
Clarence) and Lady Grey.

KING EDWARD.
Brother of Gloucester, at Saint Albans field
This lady’s husband, Sir John Grey, was slain,
His land then seized on by the conqueror.
Her suit is now to repossess those lands,
Which we in justice cannot well deny,
Because in quarrel of the house of York
The worthy gentleman did lose his life.

RICHARD.
Your Highness shall do well to grant her suit;
It were dishonour to deny it her.

KING EDWARD.
It were no less; but yet I’ll make a pause.

RICHARD.
[_Aside to George_.] Yea, is it so?
I see the lady hath a thing to grant
Before the King will grant her humble suit.

GEORGE.
[_Aside to Richard_.] He knows the game; how true he keeps the wind!

RICHARD.
[_Aside to George_.] Silence!

KING EDWARD.
Widow, we will consider of your suit,
And come some other time to know our mind.

LADY GREY.
Right gracious lord, I cannot brook delay.
May it please your Highness to resolve me now,
And what your pleasure is shall satisfy me.

RICHARD.
[_Aside to George_.] Ay, widow? Then I’ll warrant you all your lands,
An if what pleases him shall pleasure you.
Fight closer, or, good faith, you’ll catch a blow.

GEORGE.
[_Aside to Richard_.] I fear her not, unless she chance to fall.

RICHARD.
[_Aside to George_.] God forbid that, for he’ll take vantages.

KING EDWARD.
How many children hast thou, widow? Tell me.

GEORGE.
[_Aside to Richard_.] I think he means to beg a child of her.

RICHARD.
[_Aside to George_.] Nay, whip me then; he’ll rather give her two.

LADY GREY.
Three, my most gracious lord.

RICHARD.
[_Aside to George_.] You shall have four if you’ll be ruled by him.

KING EDWARD.
’Twere pity they should lose their father’s lands.

LADY GREY.
Be pitiful, dread lord, and grant it then.

KING EDWARD.
Lords, give us leave; I’ll try this widow’s wit.

RICHARD.
[_Aside to George_.] Ay, good leave have you; for you will have leave
Till youth take leave and leave you to the crutch.

[_Richard and George stand aside._]

KING EDWARD.
Now tell me, madam, do you love your children?

LADY GREY.
Ay, full as dearly as I love myself.

KING EDWARD.
And would you not do much to do them good?

LADY GREY.
To do them good I would sustain some harm.

KING EDWARD.
Then get your husband’s lands to do them good.

LADY GREY.
Therefore I came unto your majesty.

KING EDWARD.
I’ll tell you how these lands are to be got.

LADY GREY.
So shall you bind me to your Highness’ service.

KING EDWARD.
What service wilt thou do me if I give them?

LADY GREY.
What you command that rests in me to do.

KING EDWARD.
But you will take exceptions to my boon.

LADY GREY.
No, gracious lord, except I cannot do it.

KING EDWARD.
Ay, but thou canst do what I mean to ask.

LADY GREY.
Why, then, I will do what your Grace commands.

RICHARD.
[_Aside to George_.] He plies her hard; and much rain wears the marble.

GEORGE.
[_Aside to Richard_.] As red as fire! Nay, then her wax must melt.

LADY GREY.
Why stops my lord? Shall I not hear my task?

KING EDWARD.
An easy task; ’tis but to love a king.

LADY GREY.
That’s soon performed, because I am a subject.

KING EDWARD.
Why, then, thy husband’s lands I freely give thee.

LADY GREY.
I take my leave with many thousand thanks.

RICHARD.
[_Aside to George_.] The match is made; she seals it with a curtsy.

KING EDWARD.
But stay thee; ’tis the fruits of love I mean.

LADY GREY.
The fruits of love I mean, my loving liege.

KING EDWARD.
Ay, but, I fear me, in another sense.
What love, thinkst thou, I sue so much to get?

LADY GREY.
My love till death, my humble thanks, my prayers;
That love which virtue begs, and virtue grants.

KING EDWARD.
No, by my troth, I did not mean such love.

LADY GREY.
Why, then, you mean not as I thought you did.

KING EDWARD.
But now you partly may perceive my mind.

LADY GREY.
My mind will never grant what I perceive
Your Highness aims at, if I aim aright.

KING EDWARD.
To tell thee plain, I aim to lie with thee.

LADY GREY.
To tell you plain, I had rather lie in prison.

KING EDWARD.
Why, then thou shalt not have thy husband’s lands.

LADY GREY.
Why, then mine honesty shall be my dower,
For by that loss I will not purchase them.

KING EDWARD.
Therein thou wrong’st thy children mightily.

LADY GREY.
Herein your Highness wrongs both them and me.
But, mighty lord, this merry inclination
Accords not with the sadness of my suit.
Please you dismiss me either with ay or no.

KING EDWARD.
Ay, if thou wilt say ay to my request;
No, if thou dost say no to my demand.

LADY GREY.
Then no, my lord. My suit is at an end.

RICHARD.
[_Aside to George_.] The widow likes him not, she knits her brows.

GEORGE.
[_Aside to Richard_.] He is the bluntest wooer in Christendom.

KING EDWARD.
[_Aside_.] Her looks doth argue her replete with modesty;
Her words doth show her wit incomparable;
All her perfections challenge sovereignty.
One way or other, she is for a king,
And she shall be my love, or else my queen.—
Say that King Edward take thee for his queen?

LADY GREY.
’Tis better said than done, my gracious lord.
I am a subject fit to jest withal,
But far unfit to be a sovereign.

KING EDWARD.
Sweet widow, by my state I swear to thee,
I speak no more than what my soul intends;
And that is to enjoy thee for my love.

LADY GREY.
And that is more than I will yield unto.
I know I am too mean to be your queen,
And yet too good to be your concubine.

KING EDWARD.
You cavil, widow; I did mean my queen.

LADY GREY.
’Twill grieve your Grace my sons should call you father.

KING EDWARD.
No more than when my daughters call thee mother.
Thou art a widow, and thou hast some children;
And, by God’s mother, I, being but a bachelor,
Have other some. Why, ’tis a happy thing
To be the father unto many sons.
Answer no more, for thou shalt be my queen.

RICHARD.
[_Aside to George_.] The ghostly father now hath done his shrift.

GEORGE.
[_Aside to Richard_.] When he was made a shriver, ’twas for shift.

KING EDWARD.
Brothers, you muse what chat we two have had.

Richard and George come forward.

RICHARD.
The widow likes it not, for she looks very sad.

KING EDWARD.
You’d think it strange if I should marry her.

GEORGE.
To whom, my lord?

KING EDWARD.
Why, Clarence, to myself.

RICHARD.
That would be ten days’ wonder at the least.

GEORGE.
That’s a day longer than a wonder lasts.

RICHARD.
By so much is the wonder in extremes.

KING EDWARD.
Well, jest on, brothers. I can tell you both
Her suit is granted for her husband’s lands.

Enter a Nobleman.

NOBLEMAN.
My gracious lord, Henry your foe is taken,
And brought your prisoner to your palace gate.

KING EDWARD.
See that he be conveyed unto the Tower.
And go we, brothers, to the man that took him,
To question of his apprehension.
Widow, go you along. Lords, use her honourably.

[_Exeunt all but Richard._]

RICHARD.
Ay, Edward will use women honourably.
Would he were wasted, marrow, bones, and all,
That from his loins no hopeful branch may spring,
To cross me from the golden time I look for!
And yet, between my soul’s desire and me—
The lustful Edward’s title buried—
Is Clarence, Henry, and his son young Edward,
And all the unlooked-for issue of their bodies,
To take their rooms ere I can place myself.
A cold premeditation for my purpose!
Why then I do but dream on sovereignty;
Like one that stands upon a promontory
And spies a far-off shore where he would tread,
Wishing his foot were equal with his eye,
And chides the sea that sunders him from thence,
Saying he’ll lade it dry to have his way.
So do I wish the crown, being so far off,
And so I chide the means that keeps me from it;
And so I say I’ll cut the causes off,
Flattering me with impossibilities.
My eye’s too quick, my heart o’erweens too much,
Unless my hand and strength could equal them.
Well, say there is no kingdom then for Richard,
What other pleasure can the world afford?
I’ll make my heaven in a lady’s lap,
And deck my body in gay ornaments,
And ’witch sweet ladies with my words and looks.
O miserable thought, and more unlikely
Than to accomplish twenty golden crowns.
Why, Love forswore me in my mother’s womb,
And, for I should not deal in her soft laws,
She did corrupt frail Nature with some bribe
To shrink mine arm up like a withered shrub;
To make an envious mountain on my back,
Where sits Deformity to mock my body;
To shape my legs of an unequal size;
To disproportion me in every part,
Like to a chaos, or an unlicked bear-whelp
That carries no impression like the dam.
And am I then a man to be beloved?
O monstrous fault to harbour such a thought!
Then, since this earth affords no joy to me
But to command, to check, to o’erbear such
As are of better person than myself,
I’ll make my heaven to dream upon the crown,
And, whiles I live, t’ account this world but hell
Until my misshaped trunk that bear this head
Be round impaled with a glorious crown.
And yet I know not how to get the crown,
For many lives stand between me and home;
And I, like one lost in a thorny wood,
That rents the thorns, and is rent with the thorns,
Seeking a way, and straying from the way,
Not knowing how to find the open air,
But toiling desperately to find it out,
Torment myself to catch the English crown.
And from that torment I will free myself,
Or hew my way out with a bloody axe.
Why, I can smile, and murder while I smile,
And cry “Content!” to that which grieves my heart,
And wet my cheeks with artificial tears,
And frame my face to all occasions.
I’ll drown more sailors than the mermaid shall,
I’ll slay more gazers than the basilisk;
I’ll play the orator as well as Nestor,
Deceive more slyly than Ulysses could,
And, like a Sinon, take another Troy.
I can add colours to the chameleon,
Change shapes with Proteus for advantages,
And set the murderous Machiavel to school.
Can I do this, and cannot get a crown?
Tut, were it farther off, I’ll pluck it down.

[_Exit._]

SCENE III. France. The King’s Palace

Flourish. Enter Lewis, the French King, his sister the Lady Bona, his
Admiral called Bourbon, Prince Edward, Queen Margaret, and the Earl of
Oxford. Lewis sits, and riseth up again.

KING LEWIS.
Fair Queen of England, worthy Margaret,
Sit down with us. It ill befits thy state
And birth that thou shouldst stand while Lewis doth sit.

QUEEN MARGARET.
No, mighty King of France. Now Margaret
Must strike her sail and learn awhile to serve
Where kings command. I was, I must confess,
Great Albion’s queen in former golden days;
But now mischance hath trod my title down
And with dishonour laid me on the ground,
Where I must take like seat unto my fortune
And to my humble seat conform myself.

KING LEWIS.
Why, say, fair queen, whence springs this deep despair?

QUEEN MARGARET.
From such a cause as fills mine eyes with tears
And stops my tongue, while heart is drowned in cares.

KING LEWIS.
Whate’er it be, be thou still like thyself,
And sit thee by our side. Yield not thy neck

[_Seats her by him._]

To Fortune’s yoke, but let thy dauntless mind
Still ride in triumph over all mischance.
Be plain, Queen Margaret, and tell thy grief;
It shall be eased if France can yield relief.

QUEEN MARGARET.
Those gracious words revive my drooping thoughts
And give my tongue-tied sorrows leave to speak.
Now, therefore, be it known to noble Lewis
That Henry, sole possessor of my love,
Is, of a king, become a banished man
And forced to live in Scotland a forlorn;
While proud ambitious Edward, Duke of York,
Usurps the regal title and the seat
Of England’s true-anointed lawful king.
This is the cause that I, poor Margaret,
With this my son, Prince Edward, Henry’s heir,
Am come to crave thy just and lawful aid;
And if thou fail us, all our hope is done.
Scotland hath will to help, but cannot help;
Our people and our peers are both misled,
Our treasure seized, our soldiers put to flight,
And, as thou seest, ourselves in heavy plight.

KING LEWIS.
Renowned queen, with patience calm the storm
While we bethink a means to break it off.

QUEEN MARGARET.
The more we stay, the stronger grows our foe.

KING LEWIS.
The more I stay, the more I’ll succour thee.

QUEEN MARGARET.
O, but impatience waiteth on true sorrow.
And see where comes the breeder of my sorrow.

Enter Warwick.

KING LEWIS.
What’s he approacheth boldly to our presence?

QUEEN MARGARET.
Our Earl of Warwick, Edward’s greatest friend.

KING LEWIS.
Welcome, brave Warwick. What brings thee to France?

[_He descends. Queen Margaret rises._]

QUEEN MARGARET.
Ay, now begins a second storm to rise,
For this is he that moves both wind and tide.

WARWICK.
From worthy Edward, king of Albion,
My lord and sovereign, and thy vowed friend,
I come, in kindness and unfeigned love,
First, to do greetings to thy royal person,
And then to crave a league of amity,
And lastly, to confirm that amity
With nuptial knot, if thou vouchsafe to grant
That virtuous Lady Bona, thy fair sister,
To England’s king in lawful marriage.

QUEEN MARGARET.
[_Aside_.] If that go forward, Henry’s hope is done.

WARWICK.
[_To Bona_.] And, gracious madam, in our king’s behalf,
I am commanded, with your leave and favour,
Humbly to kiss your hand, and with my tongue
To tell the passion of my sovereign’s heart,
Where fame, late entering at his heedful ears,
Hath placed thy beauty’s image and thy virtue.

QUEEN MARGARET.
King Lewis and Lady Bona, hear me speak
Before you answer Warwick. His demand
Springs not from Edward’s well-meant honest love,
But from deceit, bred by necessity;
For how can tyrants safely govern home
Unless abroad they purchase great alliance?
To prove him tyrant this reason may suffice,
That Henry liveth still; but were he dead,
Yet here Prince Edward stands, King Henry’s son.
Look therefore, Lewis, that by this league and marriage
Thou draw not on thy danger and dishonour;
For though usurpers sway the rule awhile,
Yet heavens are just, and time suppresseth wrongs.

WARWICK.
Injurious Margaret!

PRINCE EDWARD.
And why not Queen?

WARWICK.
Because thy father Henry did usurp,
And thou no more art prince than she is queen.

OXFORD.
Then Warwick disannuls great John of Gaunt,
Which did subdue the greatest part of Spain;
And after John of Gaunt, Henry the Fourth,
Whose wisdom was a mirror to the wisest;
And after that wise prince, Henry the Fifth,
Who by his prowess conquered all France.
From these our Henry lineally descends.

WARWICK.
Oxford, how haps it in this smooth discourse
You told not how Henry the Sixth hath lost
All that which Henry the Fifth had gotten?
Methinks these peers of France should smile at that.
But for the rest: you tell a pedigree
Of threescore and two years, a silly time
To make prescription for a kingdom’s worth.

OXFORD.
Why, Warwick, canst thou speak against thy liege,
Whom thou obeyed’st thirty and six years,
And not bewray thy treason with a blush?

WARWICK.
Can Oxford, that did ever fence the right,
Now buckler falsehood with a pedigree?
For shame! Leave Henry, and call Edward king.

OXFORD.
Call him my king by whose injurious doom
My elder brother, the Lord Aubrey Vere,
Was done to death? And more than so, my father,
Even in the downfall of his mellowed years,
When nature brought him to the door of death?
No, Warwick, no; while life upholds this arm,
This arm upholds the house of Lancaster.

WARWICK.
And I the house of York.

KING LEWIS.
Queen Margaret, Prince Edward, and Oxford,
Vouchsafe at our request to stand aside
While I use further conference with Warwick.

[_They stand aloof._]

QUEEN MARGARET.
Heavens grant that Warwick’s words bewitch him not!

KING LEWIS.
Now, Warwick, tell me, even upon thy conscience,
Is Edward your true king? For I were loath
To link with him that were not lawful chosen.

WARWICK.
Thereon I pawn my credit and mine honour.

KING LEWIS.
But is he gracious in the people’s eye?

WARWICK.
The more that Henry was unfortunate.

KING LEWIS.
Then further, all dissembling set aside,
Tell me for truth the measure of his love
Unto our sister Bona.

WARWICK.
Such it seems
As may beseem a monarch like himself.
Myself have often heard him say and swear
That this his love was an eternal plant,
Whereof the root was fixed in virtue’s ground,
The leaves and fruit maintained with beauty’s sun,
Exempt from envy, but not from disdain,
Unless the Lady Bona quit his pain.

KING LEWIS.
Now, sister, let us hear your firm resolve.

BONA.
Your grant or your denial shall be mine.
[_To Warwick_] Yet I confess that often ere this day,
When I have heard your king’s desert recounted,
Mine ear hath tempted judgment to desire.

KING LEWIS.
Then, Warwick, thus: our sister shall be Edward’s.
And now forthwith shall articles be drawn
Touching the jointure that your king must make,
Which with her dowry shall be counterpoised.
Draw near, Queen Margaret, and be a witness
That Bona shall be wife to the English king.

PRINCE EDWARD.
To Edward, but not to the English king.

QUEEN MARGARET.
Deceitful Warwick, it was thy device
By this alliance to make void my suit.
Before thy coming Lewis was Henry’s friend.

KING LEWIS.
And still is friend to him and Margaret.
But if your title to the crown be weak,
As may appear by Edward’s good success,
Then ’tis but reason that I be released
From giving aid which late I promised.
Yet shall you have all kindness at my hand
That your estate requires and mine can yield.

WARWICK.
Henry now lives in Scotland, at his ease,
Where, having nothing, nothing can he lose.
And as for you yourself, our quondam queen,
You have a father able to maintain you,
And better ’twere you troubled him than France.

QUEEN MARGARET.
Peace, impudent and shameless Warwick,
Proud setter up and puller down of kings!
I will not hence till with my talk and tears,
Both full of truth, I make King Lewis behold
Thy sly conveyance and thy lord’s false love;
For both of you are birds of selfsame feather.

[_Post blowing a horn within._]

KING LEWIS.
Warwick, this is some post to us or thee.

Enter the Post.

POST.
My lord ambassador, these letters are for you.
Sent from your brother, Marquess Montague.
These from our king unto your Majesty.
And, madam, these for you, from whom I know not.

[_They all read their letters._]

OXFORD.
I like it well that our fair Queen and mistress
Smiles at her news while Warwick frowns at his.

PRINCE EDWARD.
Nay, mark how Lewis stamps as he were nettled.
I hope all’s for the best.

KING LEWIS.
Warwick, what are thy news? And yours, fair Queen?

QUEEN MARGARET.
Mine, such as fill my heart with unhoped joys.

WARWICK.
Mine, full of sorrow and heart’s discontent.

KING LEWIS.
What, has your king married the Lady Grey,
And now, to soothe your forgery and his,
Sends me a paper to persuade me patience?
Is this th’ alliance that he seeks with France?
Dare he presume to scorn us in this manner?

QUEEN MARGARET.
I told your majesty as much before;
This proveth Edward’s love and Warwick’s honesty.

WARWICK.
King Lewis, I here protest in sight of heaven,
And by the hope I have of heavenly bliss,
That I am clear from this misdeed of Edward’s—
No more my king, for he dishonours me,
But most himself, if he could see his shame.
Did I forget that by the house of York
My father came untimely to his death?
Did I let pass th’ abuse done to my niece?
Did I impale him with the regal crown?
Did I put Henry from his native right?
And am I guerdoned at the last with shame?
Shame on himself, for my desert is honour;
And to repair my honour lost for him,
I here renounce him and return to Henry.
My noble Queen, let former grudges pass,
And henceforth I am thy true servitor.
I will revenge his wrong to Lady Bona,
And replant Henry in his former state.

QUEEN MARGARET.
Warwick, these words have turned my hate to love;
And I forgive and quite forget old faults,
And joy that thou becom’st King Henry’s friend.

WARWICK.
So much his friend, ay, his unfeigned friend,
That if King Lewis vouchsafe to furnish us
With some few bands of chosen soldiers,
I’ll undertake to land them on our coast
And force the tyrant from his seat by war.
’Tis not his new-made bride shall succour him;
And as for Clarence, as my letters tell me,
He’s very likely now to fall from him
For matching more for wanton lust than honour,
Or than for strength and safety of our country.

BONA.
Dear brother, how shall Bona be revenged
But by thy help to this distressed queen?

QUEEN MARGARET.
Renowned prince, how shall poor Henry live
Unless thou rescue him from foul despair?

BONA.
My quarrel and this English queen’s are one.

WARWICK.
And mine, fair Lady Bona, joins with yours.

KING LEWIS.
And mine with hers, and thine, and Margaret’s.
Therefore, at last I firmly am resolved
You shall have aid.

QUEEN MARGARET.
Let me give humble thanks for all at once.

KING LEWIS.
Then, England’s messenger, return in post
And tell false Edward, thy supposed king,
That Lewis of France is sending over maskers
To revel it with him and his new bride.
Thou seest what’s past; go fear thy king withal.

BONA.
Tell him, in hope he’ll prove a widower shortly,
I’ll wear the willow garland for his sake.

QUEEN MARGARET.
Tell him my mourning weeds are laid aside,
And I am ready to put armour on.

WARWICK.
Tell him from me that he hath done me wrong,
And therefore I’ll uncrown him ere ’t be long.
There’s thy reward; be gone.

[_Exit Post._]

KING LEWIS.
But, Warwick,
Thou and Oxford, with five thousand men,
Shall cross the seas and bid false Edward battle;
And, as occasion serves, this noble Queen
And prince shall follow with a fresh supply.
Yet, ere thou go, but answer me one doubt:
What pledge have we of thy firm loyalty?

WARWICK.
This shall assure my constant loyalty:
That if our Queen and this young prince agree,
I’ll join mine eldest daughter and my joy
To him forthwith in holy wedlock bands.

QUEEN MARGARET.
Yes, I agree, and thank you for your motion.
Son Edward, she is fair and virtuous,
Therefore delay not, give thy hand to Warwick,
And with thy hand thy faith irrevocable
That only Warwick’s daughter shall be thine.

PRINCE EDWARD.
Yes, I accept her, for she well deserves it;
And here, to pledge my vow, I give my hand.

[_He gives his hand to Warwick._]

KING LEWIS.
Why stay we now? These soldiers shall be levied,
And thou, Lord Bourbon, our High Admiral,
Shall waft them over with our royal fleet.
I long till Edward fall by war’s mischance
For mocking marriage with a dame of France.

[_Exeunt all but Warwick._]

WARWICK.
I came from Edward as ambassador,
But I return his sworn and mortal foe.
Matter of marriage was the charge he gave me,
But dreadful war shall answer his demand.
Had he none else to make a stale but me?
Then none but I shall turn his jest to sorrow.
I was the chief that raised him to the crown,
And I’ll be chief to bring him down again:
Not that I pity Henry’s misery,
But seek revenge on Edward’s mockery.

[_Exit._]




ACT IV

SCENE I. London. The Palace


Enter Richard (Duke of Gloucester), George (Duke of Clarence), Somerset
and Montague.

RICHARD.
Now tell me, brother Clarence, what think you
Of this new marriage with the Lady Grey?
Hath not our brother made a worthy choice?

GEORGE.
Alas, you know ’tis far from hence to France!
How could he stay till Warwick made return?

SOMERSET.
My lords, forbear this talk; here comes the King.

Flourish. Enter King Edward, attended; Lady Grey as Queen Elizabeth;
Pembroke, Stafford, Hastings and others. Four stand on one side, and
four on the other.

RICHARD.
And his well-chosen bride.

GEORGE.
I mind to tell him plainly what I think.

KING EDWARD.
Now, brother of Clarence, how like you our choice,
That you stand pensive as half malcontent?

GEORGE.
As well as Lewis of France or the Earl of Warwick,
Which are so weak of courage and in judgment
That they’ll take no offence at our abuse.

KING EDWARD.
Suppose they take offence without a cause,
They are but Lewis and Warwick; I am Edward,
Your King and Warwick’s, and must have my will.

RICHARD.
And shall have your will, because our King.
Yet hasty marriage seldom proveth well.

KING EDWARD.
Yea, brother Richard, are you offended too?

RICHARD.
Not I.
No, God forbid that I should wish them severed
Whom God hath joined together. Ay, and ’twere pity
To sunder them that yoke so well together.

KING EDWARD.
Setting your scorns and your mislike aside,
Tell me some reason why the Lady Grey
Should not become my wife and England’s queen.
And you too, Somerset and Montague,
Speak freely what you think.

GEORGE.
Then this is mine opinion: that King Lewis
Becomes your enemy for mocking him
About the marriage of the Lady Bona.

RICHARD.
And Warwick, doing what you gave in charge,
Is now dishonoured by this new marriage.

KING EDWARD.
What if both Lewis and Warwick be appeased
By such invention as I can devise?

MONTAGUE.
Yet to have joined with France in such alliance
Would more have strengthened this our commonwealth
’Gainst foreign storms than any home-bred marriage.

HASTINGS.
Why, knows not Montague that of itself
England is safe, if true within itself?

MONTAGUE.
But the safer when ’tis backed with France.

HASTINGS.
’Tis better using France than trusting France.
Let us be backed with God and with the seas
Which He hath giv’n for fence impregnable,
And with their helps only defend ourselves.
In them and in ourselves our safety lies.

GEORGE.
For this one speech Lord Hastings well deserves
To have the heir of the Lord Hungerford.

KING EDWARD.
Ay, what of that? It was my will and grant;
And for this once my will shall stand for law.

RICHARD.
And yet, methinks, your Grace hath not done well
To give the heir and daughter of Lord Scales
Unto the brother of your loving bride.
She better would have fitted me or Clarence;
But in your bride you bury brotherhood.

GEORGE.
Or else you would not have bestowed the heir
Of the Lord Bonville on your new wife’s son,
And leave your brothers to go speed elsewhere.

KING EDWARD.
Alas, poor Clarence, is it for a wife
That thou art malcontent? I will provide thee.

GEORGE.
In choosing for yourself you showed your judgment,
Which being shallow, you shall give me leave
To play the broker in mine own behalf;
And to that end I shortly mind to leave you.

KING EDWARD.
Leave me or tarry, Edward will be king,
And not be tied unto his brother’s will.

QUEEN ELIZABETH.
My lords, before it pleased his Majesty
To raise my state to title of a queen,
Do me but right, and you must all confess
That I was not ignoble of descent,
And meaner than myself have had like fortune.
But as this title honours me and mine,
So your dislikes, to whom I would be pleasing,
Doth cloud my joys with danger and with sorrow.

KING EDWARD.
My love, forbear to fawn upon their frowns.
What danger or what sorrow can befall thee
So long as Edward is thy constant friend
And their true sovereign, whom they must obey?
Nay, whom they shall obey, and love thee too,
Unless they seek for hatred at my hands;
Which if they do, yet will I keep thee safe,
And they shall feel the vengeance of my wrath.

RICHARD.
[_Aside_.] I hear, yet say not much, but think the more.

Enter a Post.

KING EDWARD.
Now, messenger, what letters or what news
From France?

POST.
My sovereign liege, no letters, and few words,
But such as I, without your special pardon,
Dare not relate.

KING EDWARD.
Go to, we pardon thee. Therefore, in brief,
Tell me their words as near as thou canst guess them.
What answer makes King Lewis unto our letters?

POST.
At my depart these were his very words:
“Go tell false Edward, thy supposed king,
That Lewis of France is sending over maskers
To revel it with him and his new bride.”

KING EDWARD.
Is Lewis so brave? Belike he thinks me Henry.
But what said Lady Bona to my marriage?

POST.
These were her words, uttered with mild disdain:
“Tell him, in hope he’ll prove a widower shortly,
I’ll wear the willow garland for his sake.”

KING EDWARD.
I blame not her; she could say little less;
She had the wrong. But what said Henry’s queen?
For I have heard that she was there in place.

POST.
“Tell him,” quoth she “my mourning weeds are done,
And I am ready to put armour on.”

KING EDWARD.
Belike she minds to play the Amazon.
But what said Warwick to these injuries?

POST.
He, more incensed against your Majesty
Than all the rest, discharged me with these words:
“Tell him from me that he hath done me wrong,
And therefore I’ll uncrown him ere ’t be long.”

KING EDWARD.
Ha! Durst the traitor breathe out so proud words?
Well, I will arm me, being thus forewarned.
They shall have wars and pay for their presumption.
But say, is Warwick friends with Margaret?

POST.
Ay, gracious sovereign, they are so linked in friendship
That young Prince Edward marries Warwick’s daughter.

GEORGE.
Belike the elder; Clarence will have the younger.
Now, brother king, farewell, and sit you fast,
For I will hence to Warwick’s other daughter;
That, though I want a kingdom, yet in marriage
I may not prove inferior to yourself.
You that love me and Warwick, follow me.

[_Exit George and Somerset follows._]

RICHARD.
[_Aside_.] Not I. My thoughts aim at a further matter;
I stay not for the love of Edward, but the crown.

KING EDWARD.
Clarence and Somerset both gone to Warwick!
Yet am I armed against the worst can happen,
And haste is needful in this desperate case.
Pembroke and Stafford, you in our behalf
Go levy men and make prepare for war;
They are already, or quickly will be, landed.
Myself in person will straight follow you.

[_Exeunt Pembroke and Stafford._]

But, ere I go, Hastings and Montague,
Resolve my doubt. You twain, of all the rest,
Are near to Warwick by blood and by alliance.
Tell me if you love Warwick more than me.
If it be so, then both depart to him.
I rather wish you foes than hollow friends.
But if you mind to hold your true obedience,
Give me assurance with some friendly vow,
That I may never have you in suspect.

MONTAGUE.
So God help Montague as he proves true!

HASTINGS.
And Hastings as he favours Edward’s cause!

KING EDWARD.
Now, brother Richard, will you stand by us?

RICHARD.
Ay, in despite of all that shall withstand you.

KING EDWARD.
Why, so! Then am I sure of victory.
Now, therefore, let us hence, and lose no hour
Till we meet Warwick with his foreign power.

[_Exeunt._]

SCENE II. A Plain in Warwickshire

Enter Warwick and Oxford in England, with French Soldiers.

WARWICK.
Trust me, my lord, all hitherto goes well;
The common people by numbers swarm to us.

Enter George (Duke of Clarence) and Somerset.

But see where Somerset and Clarence comes.
Speak suddenly, my lords: are we all friends?

GEORGE.
Fear not that, my lord.

WARWICK.
Then, gentle Clarence, welcome unto Warwick;
And welcome, Somerset. I hold it cowardice
To rest mistrustful where a noble heart
Hath pawned an open hand in sign of love;
Else might I think that Clarence, Edward’s brother,
Were but a feigned friend to our proceedings.
But welcome, sweet Clarence; my daughter shall be thine.
And now what rests but, in night’s coverture,
Thy brother being carelessly encamped,
His soldiers lurking in the towns about,
And but attended by a simple guard,
We may surprise and take him at our pleasure?
Our scouts have found the adventure very easy;
That, as Ulysses and stout Diomede
With sleight and manhood stole to Rhesus’ tents,
And brought from thence the Thracian fatal steeds,
So we, well covered with the night’s black mantle,
At unawares may beat down Edward’s guard,
And seize himself. I say not, slaughter him,
For I intend but only to surprise him.
You that will follow me to this attempt,
Applaud the name of Henry with your leader.

[_They all cry “Henry!”_]

Why then, let’s on our way in silent sort,
For Warwick and his friends, God and Saint George!

[_Exeunt._]

SCENE III. Edward’s Camp near Warwick

Enter three Watchmen to guard the King’s tent.

1 WATCHMAN.
Come on, my masters, each man take his stand.
The King by this is set him down to sleep.

2 WATCHMAN.
What, will he not to bed?

1 WATCHMAN.
Why, no; for he hath made a solemn vow
Never to lie and take his natural rest
Till Warwick or himself be quite suppressed.

2 WATCHMAN.
Tomorrow, then, belike shall be the day,
If Warwick be so near as men report.

3 WATCHMAN.
But say, I pray, what nobleman is that
That with the King here resteth in his tent?

1 WATCHMAN.
’Tis the Lord Hastings, the King’s chiefest friend.

3 WATCHMAN.
O, is it so? But why commands the King
That his chief followers lodge in towns about him,
While he himself keeps in the cold field?

2 WATCHMAN.
’Tis the more honour, because more dangerous.

3 WATCHMAN.
Ay, but give me worship and quietness;
I like it better than dangerous honour.
If Warwick knew in what estate he stands,
’Tis to be doubted he would waken him.

1 WATCHMAN.
Unless our halberds did shut up his passage.

2 WATCHMAN.
Ay, wherefore else guard we his royal tent
But to defend his person from night-foes?

Enter Warwick, George (Duke of Clarence), Oxford, Somerset and French
Soldiers, silent all.

WARWICK.
This is his tent; and see where stand his guard.
Courage, my masters! Honour now or never!
But follow me, and Edward shall be ours.

1 WATCHMAN.
Who goes there?

2 WATCHMAN.
Stay, or thou diest.

[_Warwick and the rest cry all, “Warwick! Warwick!” and set upon the
guard, who fly, crying “Arm! Arm!” Warwick  and the rest following
them._]

The drum playing and trumpet sounding, enter Warwick, Somerset, and the
rest, bringing the King out in his gown, sitting in a chair. Richard
(Duke of Gloucester) and Hastings fly over the stage.

SOMERSET.
What are they that fly there?

WARWICK.
Richard and Hastings.
Let them go. Here is the Duke.

KING EDWARD.
The Duke? Why, Warwick, when we parted,
Thou call’dst me king?

WARWICK.
Ay, but the case is altered.
When you disgraced me in my embassade,
Then I degraded you from being king,
And come now to create you Duke of York.
Alas, how should you govern any kingdom
That know not how to use ambassadors,
Nor how to be contented with one wife,
Nor how to use your brothers brotherly,
Nor how to study for the people’s welfare,
Nor how to shroud yourself from enemies?

KING EDWARD.
Yea, brother of Clarence, art thou here too?
Nay, then I see that Edward needs must down.
Yet, Warwick, in despite of all mischance
Of thee thyself and all thy complices,
Edward will always bear himself as king.
Though Fortune’s malice overthrow my state,
My mind exceeds the compass of her wheel.

WARWICK.
Then for his mind be Edward England’s king;

[_Takes off his crown._]

But Henry now shall wear the English crown
And be true king indeed, thou but the shadow.
My lord of Somerset, at my request,
See that forthwith Duke Edward be conveyed
Unto my brother, Archbishop of York.
When I have fought with Pembroke and his fellows,
I’ll follow you and tell what answer
Lewis and the Lady Bona send to him.
Now, for a while farewell, good Duke of York.

[_They begin to lead him out forcibly._]

KING EDWARD.
What fates impose, that men must needs abide;
It boots not to resist both wind and tide.

[_Exit King Edward, led out; Somerset with him._]

OXFORD.
What now remains, my lords, for us to do,
But march to London with our soldiers?

WARWICK.
Ay, that’s the first thing that we have to do,
To free King Henry from imprisonment
And see him seated in the regal throne.

[_Exeunt._]

SCENE IV. London. The Palace

Enter Queen Elizabeth and Rivers.

RIVERS.
Madam, what makes you in this sudden change?

QUEEN ELIZABETH.
Why, brother Rivers, are you yet to learn
What late misfortune is befall’n King Edward?

RIVERS.
What, loss of some pitched battle against Warwick?

QUEEN ELIZABETH.
No, but the loss of his own royal person.

RIVERS.
Then is my sovereign slain?

QUEEN ELIZABETH.
Ay, almost slain, for he is taken prisoner,
Either betrayed by falsehood of his guard
Or by his foe surprised at unawares;
And, as I further have to understand,
Is new committed to the Bishop of York,
Fell Warwick’s brother and by that our foe.

RIVERS.
These news, I must confess, are full of grief;
Yet, gracious madam, bear it as you may.
Warwick may lose that now hath won the day.

QUEEN ELIZABETH.
Till then, fair hope must hinder life’s decay;
And I the rather wean me from despair
For love of Edward’s offspring in my womb.
This is it that makes me bridle passion
And bear with mildness my misfortune’s cross,
Ay, ay, for this I draw in many a tear
And stop the rising of blood-sucking sighs,
Lest with my sighs or tears I blast or drown
King Edward’s fruit, true heir to th’ English crown.

RIVERS.
But, madam, where is Warwick then become?

QUEEN ELIZABETH.
I am informed that he comes towards London
To set the crown once more on Henry’s head.
Guess thou the rest: King Edward’s friends must down.
But to prevent the tyrant’s violence—
For trust not him that hath once broken faith—
I’ll hence forthwith unto the sanctuary
To save at least the heir of Edward’s right.
There shall I rest secure from force and fraud.
Come, therefore, let us fly while we may fly.
If Warwick take us, we are sure to die.

[_Exeunt._]

SCENE V. A park near Middleham Castle in Yorkshire

Enter Richard (Duke of Gloucester), Lord Hastings, Sir William Stanley
and others.

RICHARD.
Now, my Lord Hastings and Sir William Stanley,
Leave off to wonder why I drew you hither
Into this chiefest thicket of the park.
Thus stands the case: you know our King, my brother,
Is prisoner to the Bishop here, at whose hands
He hath good usage and great liberty,
And often but attended with weak guard,
Comes hunting this way to disport himself.
I have advertised him by secret means
That if about this hour he make this way,
Under the colour of his usual game,
He shall here find his friends with horse and men
To set him free from his captivity.

Enter King Edward and a Huntsman with him.

HUNTSMAN.
This way, my lord, for this way lies the game.

KING EDWARD.
Nay, this way, man. See where the huntsmen stand.
Now, brother of Gloucester, Lord Hastings, and the rest,
Stand you thus close to steal the Bishop’s deer?

RICHARD.
Brother, the time and case requireth haste;
Your horse stands ready at the park corner.

KING EDWARD.
But whither shall we then?

HASTINGS.
To Lynn, my lord, and shipped from thence to Flanders.

RICHARD.
Well guessed, believe me, for that was my meaning.

KING EDWARD.
Stanley, I will requite thy forwardness.

RICHARD.
But wherefore stay we? ’Tis no time to talk.

KING EDWARD.
Huntsman, what sayst thou? Wilt thou go along?

HUNTSMAN.
Better do so than tarry and be hanged.

RICHARD.
Come then, away! Let’s ha’ no more ado.

KING EDWARD.
Bishop, farewell; shield thee from Warwick’s frown,
And pray that I may repossess the crown.

[_Exeunt._]

SCENE VI. London. The Tower

Enter King Henry, George (Duke of Clarence), Warwick, Somerset, young
Richmond, Oxford, Montague, and Lieutenant of the Tower.

KING HENRY.
Master Lieutenant, now that God and friends
Have shaken Edward from the regal seat
And turned my captive state to liberty,
My fear to hope, my sorrows unto joys,
At our enlargement what are thy due fees?

LIEUTENANT.
Subjects may challenge nothing of their sovereigns;
But if an humble prayer may prevail,
I then crave pardon of your Majesty.

KING HENRY.
For what, lieutenant? For well using me?
Nay, be thou sure I’ll well requite thy kindness,
For that it made my imprisonment a pleasure;
Ay, such a pleasure as incaged birds
Conceive when, after many moody thoughts,
At last by notes of household harmony
They quite forget their loss of liberty.
But, Warwick, after God thou sett’st me free,
And chiefly therefore I thank God and thee;
He was the author, thou the instrument.
Therefore, that I may conquer Fortune’s spite,
By living low where Fortune cannot hurt me,
And that the people of this blessed land
May not be punished with my thwarting stars,
Warwick, although my head still wear the crown,
I here resign my government to thee,
For thou art fortunate in all thy deeds.

WARWICK.
Your Grace hath still been famed for virtuous,
And now may seem as wise as virtuous
By spying and avoiding Fortune’s malice,
For few men rightly temper with the stars;
Yet in this one thing let me blame your Grace,
For choosing me when Clarence is in place.

GEORGE.
No, Warwick, thou art worthy of the sway,
To whom the heavens in thy nativity
Adjudged an olive branch and laurel crown,
As likely to be blest in peace and war;
And therefore I yield thee my free consent.

WARWICK.
And I choose Clarence only for Protector.

KING HENRY.
Warwick and Clarence, give me both your hands.
Now join your hands, and with your hands your hearts,
That no dissension hinder government.
I make you both Protectors of this land,
While I myself will lead a private life
And in devotion spend my latter days,
To sin’s rebuke and my Creator’s praise.

WARWICK.
What answers Clarence to his sovereign’s will?

GEORGE.
That he consents, if Warwick yield consent,
For on thy fortune I repose myself.

WARWICK.
Why, then, though loath, yet I must be content.
We’ll yoke together, like a double shadow
To Henry’s body, and supply his place;
I mean, in bearing weight of government,
While he enjoys the honour and his ease.
And, Clarence, now then it is more than needful
Forthwith that Edward be pronounced a traitor
And all his lands and goods be confiscate.

GEORGE.
What else? And that succession be determined.

WARWICK.
Ay, therein Clarence shall not want his part.

KING HENRY.
But with the first of all your chief affairs
Let me entreat—for I command no more—
That Margaret your Queen and my son Edward
Be sent for to return from France with speed;
For till I see them here, by doubtful fear
My joy of liberty is half eclipsed.

GEORGE.
It shall be done, my sovereign, with all speed.

KING HENRY.
My Lord of Somerset, what youth is that
Of whom you seem to have so tender care?

SOMERSET.
My liege, it is young Henry, Earl of Richmond.

KING HENRY.
Come hither, England’s hope. If secret powers

[_Lays his hand on his head._]

Suggest but truth to my divining thoughts,
This pretty lad will prove our country’s bliss.
His looks are full of peaceful majesty,
His head by nature framed to wear a crown,
His hand to wield a sceptre, and himself
Likely in time to bless a regal throne.
Make much of him, my lords, for this is he
Must help you more than you are hurt by me.

Enter a Post.

WARWICK.
What news, my friend?

POST.
That Edward is escaped from your brother
And fled, as he hears since, to Burgundy.

WARWICK.
Unsavoury news! But how made he escape?

POST.
He was conveyed by Richard, Duke of Gloucester
And the Lord Hastings, who attended him
In secret ambush on the forest side
And from the Bishop’s huntsmen rescued him,
For hunting was his daily exercise.

WARWICK.
My brother was too careless of his charge.
But let us hence, my sovereign, to provide
A salve for any sore that may betide.

[_Exeunt all but Somerset, Richmond and Oxford._]

SOMERSET.
My lord, I like not of this flight of Edward’s,
For doubtless Burgundy will yield him help,
And we shall have more wars before ’t be long.
As Henry’s late presaging prophecy
Did glad my heart with hope of this young Richmond,
So doth my heart misgive me, in these conflicts
What may befall him, to his harm and ours.
Therefore, Lord Oxford, to prevent the worst,
Forthwith we’ll send him hence to Brittany
Till storms be past of civil enmity.

OXFORD.
Ay, for if Edward repossess the crown,
’Tis like that Richmond with the rest shall down.

SOMERSET.
It shall be so. He shall to Brittany.
Come therefore, let’s about it speedily.

[_Exeunt._]

SCENE VII. Before York

Flourish. Enter King Edward, Richard (Duke of Gloucester), Hastings and
Soldiers.

KING EDWARD.
Now, brother Richard, Lord Hastings, and the rest,
Yet thus far Fortune maketh us amends,
And says that once more I shall interchange
My waned state for Henry’s regal crown.
Well have we passed and now repassed the seas,
And brought desired help from Burgundy.
What then remains, we being thus arrived
From Ravenspurgh haven before the gates of York,
But that we enter as into our dukedom?

RICHARD.
The gates made fast! Brother, I like not this;
For many men that stumble at the threshold
Are well foretold that danger lurks within.

KING EDWARD.
Tush, man, abodements must not now affright us.
By fair or foul means we must enter in,
For hither will our friends repair to us.

HASTINGS.
My liege, I’ll knock once more to summon them.

Enter on the walls, the Mayor of York and his Brethren.

MAYOR.
My lords, we were forewarned of your coming
And shut the gates for safety of ourselves,
For now we owe allegiance unto Henry.

KING EDWARD.
But, master Mayor, if Henry be your king,
Yet Edward, at the least, is Duke of York.

MAYOR.
True, my good lord, I know you for no less.

KING EDWARD.
Why, and I challenge nothing but my dukedom,
As being well content with that alone.

RICHARD.
[_Aside_.] But when the fox hath once got in his nose,
He’ll soon find means to make the body follow.

HASTINGS.
Why, master Mayor, why stand you in a doubt?
Open the gates; we are King Henry’s friends.

MAYOR.
Ay, say you so? The gates shall then be opened.

[_He descends._]

RICHARD.
A wise, stout captain, and soon persuaded.

HASTINGS.
The good old man would fain that all were well,
So ’twere not long of him; but, being entered,
I doubt not, I, but we shall soon persuade
Both him and all his brothers unto reason.

Enter the Mayor and two Aldermen below.

KING EDWARD.
So, master Mayor, these gates must not be shut
But in the night or in the time of war.
What, fear not, man, but yield me up the keys;

[_Takes his keys._]

For Edward will defend the town and thee
And all those friends that deign to follow me.

March. Enter Montgomery with drum and Soldiers.

RICHARD.
Brother, this is Sir John Montgomery,
Our trusty friend unless I be deceived.

KING EDWARD.
Welcome, Sir John! But why come you in arms?

MONTGOMERY.
To help King Edward in his time of storm,
As every loyal subject ought to do.

KING EDWARD.
Thanks, good Montgomery; but we now forget
Our title to the crown, and only claim
Our dukedom till God please to send the rest.

MONTGOMERY.
Then fare you well, for I will hence again.
I came to serve a king, and not a duke.
Drummer, strike up, and let us march away.

[_The drum begins to march._]

KING EDWARD.
Nay, stay, Sir John, a while, and we’ll debate
By what safe means the crown may be recovered.

MONTGOMERY.
What talk you of debating? In few words,
If you’ll not here proclaim yourself our king,
I’ll leave you to your fortune and be gone
To keep them back that come to succour you.
Why shall we fight if you pretend no title?

RICHARD.
Why, brother, wherefore stand you on nice points?

KING EDWARD.
When we grow stronger, then we’ll make our claim.
Till then ’tis wisdom to conceal our meaning.

HASTINGS.
Away with scrupulous wit! Now arms must rule.

RICHARD.
And fearless minds climb soonest unto crowns.
Brother, we will proclaim you out of hand;
The bruit thereof will bring you many friends.

KING EDWARD.
Then be it as you will; for ’tis my right,
And Henry but usurps the diadem.

MONTGOMERY.
Ay, now my sovereign speaketh like himself,
And now will I be Edward’s champion.

HASTINGS.
Sound, trumpet; Edward shall be here proclaimed.
Come, fellow soldier, make thou proclamation.

[_Gives him a paper. Flourish._]

SOLDIER.
[_Reads_.] _Edward the Fourth, by the Grace of God, King of England and
France, and Lord of Ireland, etc._

MONTGOMERY.
And whoso’er gainsays King Edward’s right,
By this I challenge him to single fight.

[_Throws down his gauntlet._]

ALL.
Long live Edward the Fourth!

KING EDWARD.
Thanks, brave Montgomery, and thanks unto you all.
If Fortune serve me, I’ll requite this kindness.
Now for this night let’s harbour here in York,
And when the morning sun shall raise his car
Above the border of this horizon
We’ll forward towards Warwick and his mates;
For well I wot that Henry is no soldier.
Ah, froward Clarence, how evil it beseems thee
To flatter Henry and forsake thy brother!
Yet, as we may, we’ll meet both thee and Warwick.
Come on, brave soldiers; doubt not of the day,
And, that once gotten, doubt not of large pay.

[_Exeunt._]

SCENE VIII. London. The Palace

Flourish. Enter King Henry, Warwick, Montague, George (Duke of
Clarence), Oxford and Exeter.

WARWICK.
What counsel, lords? Edward from Belgia,
With hasty Germans and blunt Hollanders,
Hath passed in safety through the Narrow Seas,
And with his troops doth march amain to London;
And many giddy people flock to him.

KING HENRY.
Let’s levy men and beat him back again.

GEORGE.
A little fire is quickly trodden out,
Which, being suffered, rivers cannot quench.

WARWICK.
In Warwickshire I have true-hearted friends,
Not mutinous in peace, yet bold in war.
Those will I muster up; and thou, son Clarence,
Shalt stir up in Suffolk, Norfolk, and in Kent
The knights and gentlemen to come with thee.
Thou, brother Montague, in Buckingham,
Northampton, and in Leicestershire shalt find
Men well inclined to hear what thou command’st.
And thou, brave Oxford, wondrous well beloved,
In Oxfordshire shalt muster up thy friends.
My sovereign, with the loving citizens,
Like to his island girt in with the ocean,
Or modest Dian circled with her nymphs,
Shall rest in London till we come to him.
Fair lords, take leave and stand not to reply.
Farewell, my sovereign.

KING HENRY.
Farewell, my Hector, and my Troy’s true hope.

GEORGE.
In sign of truth, I kiss your Highness’ hand.

KING HENRY.
Well-minded Clarence, be thou fortunate.

MONTAGUE.
Comfort, my lord; and so I take my leave.

OXFORD.
And thus [_kissing Henry’s hand_] I seal my truth, and bid adieu.

KING HENRY.
Sweet Oxford, and my loving Montague,
And all at once, once more a happy farewell.

WARWICK.
Farewell, sweet lords; let’s meet at Coventry.

[_Exeunt all but King Henry and Exeter._]

KING HENRY.
Here at the palace will I rest a while.
Cousin of Exeter, what thinks your lordship?
Methinks the power that Edward hath in field
Should not be able to encounter mine.

EXETER.
The doubt is that he will seduce the rest.

KING HENRY.
That’s not my fear; my meed hath got me fame.
I have not stopped mine ears to their demands,
Nor posted off their suits with slow delays;
My pity hath been balm to heal their wounds,
My mildness hath allayed their swelling griefs,
My mercy dried their water-flowing tears.
I have not been desirous of their wealth
Nor much oppressed them with great subsidies,
Nor forward of revenge, though they much erred.
Then why should they love Edward more than me?
No, Exeter, these graces challenge grace;
And when the lion fawns upon the lamb,
The lamb will never cease to follow him.

[_Shout within “A York! A York!”_]

EXETER.
Hark, hark, my lord, what shouts are these?

Enter King Edward, Richard (Duke of Gloucester) and Soldiers.

KING EDWARD.
Seize on the shame-faced Henry, bear him hence,
And once again proclaim us King of England.
You are the fount that makes small brooks to flow.
Now stops thy spring; my sea shall suck them dry
And swell so much the higher by their ebb.
Hence with him to the Tower. Let him not speak.

[_Exeunt some with King Henry._]

And, lords, towards Coventry bend we our course,
Where peremptory Warwick now remains.
The sun shines hot, and, if we use delay,
Cold biting winter mars our hoped-for hay.

RICHARD.
Away betimes, before his forces join,
And take the great-grown traitor unawares.
Brave warriors, march amain towards Coventry.

[_Exeunt._]




ACT V

SCENE I. Coventry


Enter, Warwick, the Mayor of Coventry, two Messengers and others, upon
the walls.

WARWICK.
Where is the post that came from valiant Oxford?
How far hence is thy lord, mine honest fellow?

1 MESSENGER.
By this at Dunsmore, marching hitherward.

WARWICK.
How far off is our brother Montague?
Where is the post that came from Montague?

2 MESSENGER.
By this at Daintry, with a puissant troop.

Enter Sir John Somerville.

WARWICK.
Say, Somerville, what says my loving son?
And, by thy guess, how nigh is Clarence now?

SOMERVILLE.
At Southam I did leave him with his forces
And do expect him here some two hours hence.

[_Drum heard._]

WARWICK.
Then Clarence is at hand; I hear his drum.

SOMERVILLE.
It is not his, my lord; here Southam lies.
The drum your honour hears marcheth from Warwick.

WARWICK.
Who should that be? Belike, unlooked-for friends.

SOMERVILLE.
They are at hand, and you shall quickly know.

March. Flourish. Enter King Edward, Richard (Duke of Gloucester) and
Soldiers.

KING EDWARD.
Go, trumpet, to the walls and sound a parle.

RICHARD.
See how the surly Warwick mans the wall.

WARWICK.
O, unbid spite! Is sportful Edward come?
Where slept our scouts, or how are they seduced,
That we could hear no news of his repair?

KING EDWARD.
Now, Warwick, wilt thou ope the city gates,
Speak gentle words and humbly bend thy knee?
Call Edward King and at his hands beg mercy,
And he shall pardon thee these outrages.

WARWICK.
Nay, rather, wilt thou draw thy forces hence,
Confess who set thee up and plucked thee down,
Call Warwick patron and be penitent,
And thou shalt still remain the Duke of York.

RICHARD.
I thought, at least, he would have said the King;
Or did he make the jest against his will?

WARWICK.
Is not a dukedom, sir, a goodly gift?

RICHARD.
Ay, by my faith, for a poor earl to give;
I’ll do thee service for so good a gift.

WARWICK.
’Twas I that gave the kingdom to thy brother.

KING EDWARD.
Why, then, ’tis mine, if but by Warwick’s gift.

WARWICK.
Thou art no Atlas for so great a weight;
And, weakling, Warwick takes his gift again;
And Henry is my King, Warwick his subject.

KING EDWARD.
But Warwick’s king is Edward’s prisoner;
And, gallant Warwick, do but answer this:
What is the body when the head is off?

RICHARD.
Alas, that Warwick had no more forecast,
But, whiles he thought to steal the single ten,
The king was slily fingered from the deck!
You left poor Henry at the Bishop’s palace,
And ten to one you’ll meet him in the Tower.

KING EDWARD.
’Tis even so; yet you are Warwick still.

RICHARD.
Come, Warwick, take the time; kneel down, kneel down.
Nay, when? Strike now, or else the iron cools.

WARWICK.
I had rather chop this hand off at a blow
And with the other fling it at thy face,
Than bear so low a sail to strike to thee.

KING EDWARD.
Sail how thou canst, have wind and tide thy friend,
This hand, fast wound about thy coal-black hair,
Shall, whiles thy head is warm and new cut off,
Write in the dust this sentence with thy blood:
“Wind-changing Warwick now can change no more.”

Enter Oxford with drum and colours.

WARWICK.
O cheerful colours! See where Oxford comes!

OXFORD.
Oxford, Oxford, for Lancaster!

[_He and his forces enter the city._]

RICHARD.
The gates are open; let us enter too.

KING EDWARD.
So other foes may set upon our backs.
Stand we in good array, for they no doubt
Will issue out again and bid us battle;
If not, the city being but of small defence,
We’ll quietly rouse the traitors in the same.

WARWICK.
O, welcome, Oxford, for we want thy help.

Enter Montague with drum and colours.

MONTAGUE.
Montague, Montague, for Lancaster!

[_He and his forces enter the city._]

RICHARD.
Thou and thy brother both shall buy this treason
Even with the dearest blood your bodies bear.

KING EDWARD.
The harder matched, the greater victory.
My mind presageth happy gain and conquest.

Enter Somerset with drum and colours.

SOMERSET.
Somerset, Somerset, for Lancaster!

[_He and his forces enter the city._]

RICHARD.
Two of thy name, both Dukes of Somerset,
Have sold their lives unto the House of York;
And thou shalt be the third if this sword hold.

Enter George (Duke of Clarence) with drum and colours.

WARWICK.
And lo, where George of Clarence sweeps along,
Of force enough to bid his brother battle;
With whom an upright zeal to right prevails
More than the nature of a brother’s love.

[_Richard and George whisper._]

Come, Clarence, come; thou wilt if Warwick call.

GEORGE.
Father of Warwick, know you what this means?

[_Taking the red rose from his hat and throws the rose at Warwick._]

Look here, I throw my infamy at thee.
I will not ruinate my father’s house,
Who gave his blood to lime the stones together,
And set up Lancaster. Why, trowest thou, Warwick,
That Clarence is so harsh, so blunt, unnatural,
To bend the fatal instruments of war
Against his brother and his lawful King?
Perhaps thou wilt object my holy oath.
To keep that oath were more impiety
Than Jephthah’s when he sacrificed his daughter.
I am so sorry for my trespass made
That, to deserve well at my brother’s hands,
I here proclaim myself thy mortal foe,
With resolution, whereso’er I meet thee—
As I will meet thee if thou stir abroad—
To plague thee for thy foul misleading me.
And so, proud-hearted Warwick, I defy thee,
And to my brother turn my blushing cheeks.
Pardon me, Edward, I will make amends.
And, Richard, do not frown upon my faults,
For I will henceforth be no more unconstant.

KING EDWARD.
Now, welcome more, and ten times more beloved,
Than if thou never hadst deserved our hate.

RICHARD.
Welcome, good Clarence; this is brother-like.

WARWICK.
O passing traitor, perjured and unjust!

KING EDWARD.
What, Warwick, wilt thou leave the town and fight?
Or shall we beat the stones about thine ears?

WARWICK.
Alas! I am not cooped here for defence!
I will away towards Barnet presently
And bid thee battle, Edward, if thou dar’st.

KING EDWARD.
Yes, Warwick, Edward dares, and leads the way.
Lords, to the field! Saint George and victory!

[_Exeunt. March. Warwick and his company follows._]

SCENE II. A Field of Battle near Barnet

Alarum and excursions. Enter King Edward bringing forth Warwick
wounded.

KING EDWARD.
So, lie thou there. Die thou, and die our fear,
For Warwick was a bug that feared us all.
Now, Montague, sit fast; I seek for thee,
That Warwick’s bones may keep thine company.

[_Exit._]

WARWICK.
Ah, who is nigh? Come to me, friend or foe,
And tell me who is victor, York or Warwick?
Why ask I that? My mangled body shows,
My blood, my want of strength, my sick heart shows
That I must yield my body to the earth
And, by my fall, the conquest to my foe.
Thus yields the cedar to the axe’s edge,
Whose arms gave shelter to the princely eagle,
Under whose shade the ramping lion slept,
Whose top branch overpeered Jove’s spreading tree,
And kept low shrubs from winter’s pow’rful wind.
These eyes, that now are dimmed with death’s black veil,
Have been as piercing as the midday sun,
To search the secret treasons of the world;
The wrinkles in my brows, now filled with blood,
Were likened oft to kingly sepulchres,
For who lived King but I could dig his grave?
And who durst smile when Warwick bent his brow?
Lo, now my glory smeared in dust and blood!
My parks, my walks, my manors that I had,
Even now forsake me; and of all my lands
Is nothing left me but my body’s length.
Why, what is pomp, rule, reign, but earth and dust?
And live we how we can, yet die we must.

Enter Oxford and Somerset.

SOMERSET.
Ah, Warwick, Warwick, wert thou as we are,
We might recover all our loss again.
The Queen from France hath brought a puissant power;
Even now we heard the news. Ah, couldst thou fly!

WARWICK.
Why, then I would not fly. Ah, Montague!
If thou be there, sweet brother, take my hand
And with thy lips keep in my soul awhile.
Thou lov’st me not; for, brother, if thou didst,
Thy tears would wash this cold congealed blood
That glues my lips and will not let me speak.
Come quickly, Montague, or I am dead.

SOMERSET.
Ah, Warwick, Montague hath breathed his last,
And to the latest gasp cried out for Warwick,
And said “Commend me to my valiant brother.”
And more he would have said, and more he spoke,
Which sounded like a cannon in a vault,
That mought not be distinguished; but at last
I well might hear, delivered with a groan,
“O farewell, Warwick!”

WARWICK.
Sweet rest his soul! Fly, lords, and save yourselves,
For Warwick bids you all farewell, to meet in heaven.

[_He dies._]

OXFORD.
Away, away, to meet the Queen’s great power!

[_Here they bear away his body. Exeunt._]

SCENE III. Another Part of the Field

Flourish. Enter King Edward in triumph, with Richard, George and the
rest.

KING EDWARD.
Thus far our fortune keeps an upward course,
And we are graced with wreaths of victory.
But in the midst of this bright-shining day,
I spy a black, suspicious, threat’ning cloud
That will encounter with our glorious sun
Ere he attain his easeful western bed.
I mean, my lords, those powers that the Queen
Hath raised in Gallia have arrived our coast
And, as we hear, march on to fight with us.

GEORGE.
A little gale will soon disperse that cloud
And blow it to the source from whence it came;
Thy very beams will dry those vapours up,
For every cloud engenders not a storm.

RICHARD.
The Queen is valued thirty thousand strong,
And Somerset, with Oxford, fled to her.
If she have time to breathe, be well assured
Her faction will be full as strong as ours.

KING EDWARD.
We are advertised by our loving friends
That they do hold their course toward Tewkesbury.
We, having now the best at Barnet field,
Will thither straight, for willingness rids way;
And, as we march, our strength will be augmented
In every county as we go along.
Strike up the drum! cry “Courage!” and away.

[_Exeunt._]

SCENE IV. Plains near Tewkesbury

Flourish. March. Enter Queen Margaret, Prince Edward, Somerset, Oxford
and Soldiers.

QUEEN MARGARET.
Great lords, wise men ne’er sit and wail their loss,
But cheerly seek how to redress their harms.
What though the mast be now blown overboard,
The cable broke, the holding-anchor lost,
And half our sailors swallowed in the flood?
Yet lives our pilot still. Is ’t meet that he
Should leave the helm and, like a fearful lad,
With tearful eyes add water to the sea
And give more strength to that which hath too much,
Whiles in his moan the ship splits on the rock,
Which industry and courage might have saved?
Ah, what a shame, ah, what a fault were this!
Say Warwick was our anchor; what of that?
And Montague our topmast; what of him?
Our slaughtered friends the tackles; what of these?
Why, is not Oxford here another anchor?
And Somerset another goodly mast?
The friends of France our shrouds and tacklings?
And, though unskilful, why not Ned and I
For once allowed the skilful pilot’s charge?
We will not from the helm to sit and weep,
But keep our course, though the rough wind say no,
From shelves and rocks that threaten us with wrack.
As good to chide the waves as speak them fair.
And what is Edward but a ruthless sea?
What Clarence but a quicksand of deceit?
And Richard but a ragged fatal rock?
All these the enemies to our poor bark?
Say you can swim: alas, ’tis but a while!
Tread on the sand: why, there you quickly sink;
Bestride the rock: the tide will wash you off,
Or else you famish; that’s a threefold death.
This speak I, lords, to let you understand,
If case some one of you would fly from us,
That there’s no hoped-for mercy with the brothers
More than with ruthless waves, with sands, and rocks.
Why, courage then! What cannot be avoided
’Twere childish weakness to lament or fear.

PRINCE EDWARD.
Methinks a woman of this valiant spirit
Should, if a coward heard her speak these words,
Infuse his breast with magnanimity
And make him, naked, foil a man at arms.
I speak not this as doubting any here;
For did I but suspect a fearful man,
He should have leave to go away betimes,
Lest in our need he might infect another
And make him of the like spirit to himself.
If any such be here, as God forbid!
Let him depart before we need his help.

OXFORD.
Women and children of so high a courage,
And warriors faint! Why, ’twere perpetual shame.
O, brave young Prince, thy famous grandfather
Doth live again in thee. Long mayst thou live
To bear his image and renew his glories!

SOMERSET.
And he that will not fight for such a hope,
Go home to bed and, like the owl by day,
If he arise, be mocked and wondered at.

QUEEN MARGARET.
Thanks, gentle Somerset. Sweet Oxford, thanks.

PRINCE EDWARD.
And take his thanks that yet hath nothing else.

Enter a Messenger.

MESSENGER.
Prepare you, lords, for Edward is at hand
Ready to fight; therefore be resolute.

OXFORD.
I thought no less. It is his policy
To haste thus fast, to find us unprovided.

SOMERSET.
But he’s deceived; we are in readiness.

QUEEN MARGARET.
This cheers my heart, to see your forwardness.

OXFORD.
Here pitch our battle; hence we will not budge.

Flourish and march. Enter King Edward, Richard, George and Soldiers.

KING EDWARD.
Brave followers, yonder stands the thorny wood
Which by the heaven’s assistance and your strength
Must by the roots be hewn up yet ere night.
I need not add more fuel to your fire,
For, well I wot, ye blaze to burn them out.
Give signal to the fight, and to it, lords!

QUEEN MARGARET.
Lords, knights, and gentlemen, what I should say
My tears gainsay; for every word I speak
Ye see I drink the water of my eye.
Therefore, no more but this: Henry, your sovereign,
Is prisoner to the foe, his state usurped,
His realm a slaughter-house, his subjects slain,
His statutes cancelled, and his treasure spent;
And yonder is the wolf that makes this spoil.
You fight in justice. Then, in God’s name, lords,
Be valiant and give signal to the fight.

[_Alarum, retreat, excursions. Exeunt both armies_]

SCENE V. Another part of the Field

Flourish. Enter King Edward, Richard, George and Soldiers; with Queen
Margaret, Oxford and Somerset as prisoners.

KING EDWARD.
Now here a period of tumultuous broils.
Away with Oxford to Hames Castle straight.
For Somerset, off with his guilty head.
Go, bear them hence; I will not hear them speak.

OXFORD.
For my part, I’ll not trouble thee with words.

SOMERSET.
Nor I, but stoop with patience to my fortune.

[_Exeunt Oxford and Somerset, guarded._]

QUEEN MARGARET.
So part we sadly in this troublous world,
To meet with joy in sweet Jerusalem.

KING EDWARD.
Is proclamation made that who finds Edward
Shall have a high reward, and he his life?

RICHARD.
It is, and lo where youthful Edward comes.

Enter soldiers with Prince Edward.

KING EDWARD.
Bring forth the gallant; let us hear him speak.
What, can so young a man begin to prick?
Edward, what satisfaction canst thou make
For bearing arms, for stirring up my subjects,
And all the trouble thou hast turned me to?

PRINCE EDWARD.
Speak like a subject, proud, ambitious York.
Suppose that I am now my father’s mouth;
Resign thy chair, and where I stand kneel thou,
Whilst I propose the selfsame words to thee
Which, traitor, thou wouldst have me answer to.

QUEEN MARGARET.
Ah, thy father had been so resolved!

RICHARD.
That you might still have worn the petticoat
And ne’er have stol’n the breech from Lancaster.

PRINCE EDWARD.
Let Aesop fable in a winter’s night;
His currish riddle sorts not with this place.

RICHARD.
By heaven, brat, I’ll plague you for that word.

QUEEN MARGARET.
Ay, thou wast born to be a plague to men.

RICHARD.
For God’s sake, take away this captive scold.

PRINCE EDWARD.
Nay, take away this scolding crookback rather.

KING EDWARD.
Peace, wilful boy, or I will charm your tongue.

GEORGE.
Untutored lad, thou art too malapert.

PRINCE EDWARD.
I know my duty; you are all undutiful.
Lascivious Edward, and thou perjured George,
And thou misshapen Dick, I tell ye all
I am your better, traitors as ye are,
And thou usurp’st my father’s right and mine.

KING EDWARD.
Take that, the likeness of this railer here.

[_Stabs him._]

RICHARD.
Sprawl’st thou? Take that to end thy agony.

[_Stabs him._]

GEORGE.
And there’s for twitting me with perjury.

[_Stabs him._]

QUEEN MARGARET.
O, kill me too!

RICHARD.
Marry, and shall.

[_Offers to kill her._]

KING EDWARD.
Hold, Richard, hold; for we have done too much.

RICHARD.
Why should she live to fill the world with words?

KING EDWARD.
What, doth she swoon? Use means for her recovery.

RICHARD.
Clarence, excuse me to the King my brother.
I’ll hence to London on a serious matter.
Ere ye come there, be sure to hear some news.

GEORGE.
What? What?

RICHARD.
The Tower, the Tower!

[_Exit._]

QUEEN MARGARET.
O Ned, sweet Ned, speak to thy mother, boy.
Canst thou not speak? O traitors, murderers!
They that stabbed Caesar shed no blood at all,
Did not offend, nor were not worthy blame,
If this foul deed were by to equal it.
He was a man; this, in respect, a child,
And men ne’er spend their fury on a child.
What’s worse than murderer, that I may name it?
No, no, my heart will burst an if I speak;
And I will speak, that so my heart may burst.
Butchers and villains, bloody cannibals!
How sweet a plant have you untimely cropped!
You have no children, butchers; if you had,
The thought of them would have stirred up remorse.
But if you ever chance to have a child,
Look in his youth to have him so cut off
As, deathsmen, you have rid this sweet young prince!

KING EDWARD.
Away with her; go bear her hence perforce.

QUEEN MARGARET.
Nay, never bear me hence, dispatch me here.
Here sheathe thy sword; I’ll pardon thee my death.
What, wilt thou not? Then, Clarence, do it thou.

GEORGE.
By heaven, I will not do thee so much ease.

QUEEN MARGARET.
Good Clarence, do; sweet Clarence, do thou do it.

GEORGE.
Didst thou not hear me swear I would not do it?

QUEEN MARGARET.
Ay, but thou usest to forswear thyself.
’Twas sin before, but now ’tis charity.
What, wilt thou not? Where is that devil’s butcher, Richard,
Hard-favoured Richard? Richard, where art thou?
Thou art not here. Murder is thy alms-deed;
Petitioners for blood thou ne’er putt’st back.

KING EDWARD.
Away, I say! I charge ye, bear her hence.

QUEEN MARGARET.
So come to you and yours as to this prince!

[_She is taken out._]

KING EDWARD.
Where’s Richard gone?

GEORGE.
To London all in post, and, as I guess,
To make a bloody supper in the Tower.

KING EDWARD.
He’s sudden if a thing comes in his head.
Now march we hence. Discharge the common sort
With pay and thanks, and let’s away to London
And see our gentle Queen how well she fares.
By this, I hope, she hath a son for me.

[_Exeunt._]

SCENE VI. London. The Tower

Enter King Henry and Richard, with the Lieutenant on the walls.

RICHARD.
Good day, my lord. What, at your book so hard?

KING HENRY.
Ay, my good lord—my lord, I should say rather.
’Tis sin to flatter; “good” was little better:
“Good Gloucester” and “good devil” were alike,
And both preposterous; therefore, not “good lord”.

RICHARD.
Sirrah, leave us to ourselves; we must confer.

[_Exit Lieutenant._]

KING HENRY.
So flies the reckless shepherd from the wolf;
So first the harmless sheep doth yield his fleece,
And next his throat unto the butcher’s knife.
What scene of death hath Roscius now to act?

RICHARD.
Suspicion always haunts the guilty mind;
The thief doth fear each bush an officer.

KING HENRY.
The bird that hath been limed in a bush
With trembling wings misdoubteth every bush;
And I, the hapless male to one sweet bird,
Have now the fatal object in my eye
Where my poor young was limed, was caught, and killed.

RICHARD.
Why, what a peevish fool was that of Crete
That taught his son the office of a fowl!
And yet, for all his wings, the fool was drowned.

KING HENRY.
I, Daedalus; my poor boy, Icarus;
Thy father, Minos, that denied our course;
The sun that seared the wings of my sweet boy,
Thy brother Edward; and thyself, the sea
Whose envious gulf did swallow up his life.
Ah, kill me with thy weapon, not with words!
My breast can better brook thy dagger’s point
Than can my ears that tragic history.
But wherefore dost thou come? Is ’t for my life?

RICHARD.
Think’st thou I am an executioner?

KING HENRY.
A persecutor I am sure thou art.
If murdering innocents be executing,
Why, then thou art an executioner.

RICHARD.
Thy son I killed for his presumption.

KING HENRY.
Hadst thou been killed when first thou didst presume,
Thou hadst not lived to kill a son of mine.
And thus I prophesy: that many a thousand
Which now mistrust no parcel of my fear,
And many an old man’s sigh, and many a widow’s,
And many an orphan’s water-standing eye,
Men for their sons’, wives for their husbands’,
Orphans for their parents’ timeless death,
Shall rue the hour that ever thou wast born.
The owl shrieked at thy birth, an evil sign;
The night-crow cried, aboding luckless time;
Dogs howled, and hideous tempest shook down trees;
The raven rooked her on the chimney’s top,
And chatt’ring pies in dismal discord sung;
Thy mother felt more than a mother’s pain,
And yet brought forth less than a mother’s hope,
To wit, an indigested and deformed lump,
Not like the fruit of such a goodly tree.
Teeth hadst thou in thy head when thou wast born,
To signify thou cam’st to bite the world;
And, if the rest be true which I have heard,
Thou cam’st—

RICHARD.
I’ll hear no more. Die, prophet, in thy speech.

[_Stabs him._]

For this, amongst the rest, was I ordained.

KING HENRY.
Ay, and for much more slaughter after this.
O God, forgive my sins, and pardon thee!

[_Dies._]

RICHARD.
What, will the aspiring blood of Lancaster
Sink in the ground? I thought it would have mounted.
See how my sword weeps for the poor King’s death.
O, may such purple tears be always shed
From those that wish the downfall of our house!
If any spark of life be yet remaining,
Down, down to hell; and say I sent thee thither—

[_Stabs him again._]

I that have neither pity, love, nor fear.
Indeed, ’tis true that Henry told me of,
For I have often heard my mother say
I came into the world with my legs forward.
Had I not reason, think ye, to make haste
And seek their ruin that usurped our right?
The midwife wondered, and the women cried
“O, Jesus bless us, he is born with teeth!”
And so I was, which plainly signified
That I should snarl, and bite, and play the dog.
Then, since the heavens have shaped my body so,
Let hell make crooked my mind to answer it.
I have no brother, I am like no brother;
And this word “love,” which greybeards call divine,
Be resident in men like one another,
And not in me. I am myself alone.
Clarence, beware; thou keep’st me from the light,
But I will sort a pitchy day for thee;
For I will buzz abroad such prophecies
That Edward shall be fearful of his life;
And then, to purge his fear, I’ll be thy death.
King Henry and the Prince his son are gone;
Clarence, thy turn is next, and then the rest,
Counting myself but bad till I be best.
I’ll throw thy body in another room,
And triumph, Henry, in thy day of doom.

[_Exit with the body._]

SCENE VII. London. The Palace

Flourish. Enter King Edward, Queen Elizabeth, George, Richard,
Hastings, Nurse, carrying infant Prince Edward, and Attendants.

KING EDWARD.
Once more we sit in England’s royal throne,
Repurchased with the blood of enemies.
What valiant foemen, like to autumn’s corn,
Have we mowed down in tops of all their pride!
Three Dukes of Somerset, threefold renowned
For hardy and undoubted champions;
Two Cliffords, as the father and the son;
And two Northumberlands; two braver men
Ne’er spurred their coursers at the trumpet’s sound;
With them the two brave bears, Warwick and Montague,
That in their chains fettered the kingly lion
And made the forest tremble when they roared.
Thus have we swept suspicion from our seat
And made our footstool of security.
Come hither, Bess, and let me kiss my boy.
Young Ned, for thee thine uncles and myself
Have in our armours watched the winter’s night,
Went all afoot in summer’s scalding heat,
That thou mightst repossess the crown in peace;
And of our labours thou shalt reap the gain.

RICHARD.
[_Aside_.] I’ll blast his harvest, if your head were laid;
For yet I am not looked on in the world.
This shoulder was ordained so thick to heave,
And heave it shall some weight or break my back.
Work thou the way, and that shall execute.

KING EDWARD.
Clarence and Gloucester, love my lovely Queen;
And kiss your princely nephew, brothers both.

GEORGE.
The duty that I owe unto your Majesty
I seal upon the lips of this sweet babe.

QUEEN ELIZABETH.
Thanks, noble Clarence; worthy brother, thanks.

RICHARD.
And, that I love the tree from whence thou sprang’st,
Witness the loving kiss I give the fruit.
[_Aside_.] To say the truth, so Judas kissed his master
And cried “All hail!” when as he meant all harm.

KING EDWARD.
Now am I seated as my soul delights,
Having my country’s peace and brothers’ loves.

GEORGE.
What will your Grace have done with Margaret?
Reignier, her father, to the King of France
Hath pawned the Sicils and Jerusalem,
And hither have they sent it for her ransom.

KING EDWARD.
Away with her and waft her hence to France.
And now what rests but that we spend the time
With stately triumphs, mirthful comic shows,
Such as befits the pleasure of the court?
Sound drums and trumpets! Farewell, sour annoy!
For here, I hope, begins our lasting joy.

[_Exeunt._]




KING HENRY THE EIGHTH




Contents

 ACT I
 Prologue.
 Scene I. London. An ante-chamber in the palace
 Scene II. The same. The council-chamber
 Scene III. An ante-chamber in the palace
 Scene IV. A Hall in York Place

 ACT II
 Scene I. Westminster. A street
 Scene II. An ante-chamber in the palace
 Scene III. An ante-chamber of the Queen’s apartments
 Scene IV. A hall in Blackfriars

 ACT III
 Scene I. London. The Queen’s apartments
 Scene II. Ante-chamber to the King’s apartment

 ACT IV
 Scene I. A street in Westminster
 Scene II. Kimbolton

 ACT V
 Scene I. A gallery in the palace
 Scene II. Lobby before the council-chamber
 Scene III. The palace yard
 Scene IV. The palace
 Epilogue




Dramatis Personæ

KING HENRY THE EIGHTH

DUKE OF NORFOLK
DUKE OF SUFFOLK

CARDINAL WOLSEY
SECRETARIES to Wolsey
CROMWELL, servant to Wolsey
CARDINAL CAMPEIUS
GARDINER, Bishop of Winchester
PAGE to Gardiner

QUEEN KATHERINE, wife to King Henry, afterwards divorced
GRIFFITH, gentleman usher to Queen Katherine
PATIENCE, woman to Queen Katherine
Queen’s GENTLEMAN USHER
CAPUTIUS, Ambassador from the Emperor Charles V

DUKE OF BUCKINGHAM
LORD ABERGAVENNY, Buckingham’s son-in-law
EARL OF SURREY, Buckingham’s son-in-law
SIR NICHOLAS VAUX
SURVEYOR to the Duke of Buckingham
BRANDON
SERGEANT-at-Arms
Three Gentlemen

ANNE BULLEN, her Maid of Honour, afterwards Queen
An OLD LADY, friend to Anne Bullen
LORD CHAMBERLAIN
LORD SANDYS (called also SIR WILLIAM SANDYS)
SIR THOMAS LOVELL
SIR HENRY GUILDFORD

BISHOP OF LINCOLN
CRANMER, archbishop of Canterbury
LORD CHANCELLOR
GARTER King-of-Arms
SIR ANTHONY DENNY
DOCTOR BUTTS, physician to the King
Door-KEEPER of the Council-chamber
PORTER, and his Man
A CRIER
PROLOGUE
EPILOGUE

Spirits, Several Lords and Ladies in the Dumb Shows; Women attending
upon the Queen; Scribes, Officers, Guards, and other Attendants

SCENE: London; Westminster; Kimbolton




Enter Prologue.


THE PROLOGUE.
I come no more to make you laugh. Things now
That bear a weighty and a serious brow,
Sad, high, and working, full of state and woe,
Such noble scenes as draw the eye to flow,
We now present. Those that can pity, here
May, if they think it well, let fall a tear;
The subject will deserve it. Such as give
Their money out of hope they may believe
May here find truth too. Those that come to see
Only a show or two, and so agree
The play may pass, if they be still and willing,
I’ll undertake may see away their shilling
Richly in two short hours. Only they
That come to hear a merry bawdy play,
A noise of targets, or to see a fellow
In a long motley coat guarded with yellow,
Will be deceived. For, gentle hearers, know
To rank our chosen truth with such a show
As fool and fight is, beside forfeiting
Our own brains and the opinion that we bring
To make that only true we now intend,
Will leave us never an understanding friend.
Therefore, for goodness’ sake, and as you are known
The first and happiest hearers of the town,
Be sad, as we would make ye. Think ye see
The very persons of our noble story
As they were living; think you see them great,
And followed with the general throng and sweat
Of thousand friends; then, in a moment, see
How soon this mightiness meets misery;
And if you can be merry then, I’ll say
A man may weep upon his wedding day.

[_Exit._]




ACT I

SCENE I. London. An ante-chamber in the palace.


Enter the Duke of Norfolk at one door; at the other, the Duke of
Buckingham and the Lord Abergavenny.

BUCKINGHAM.
Good morrow, and well met. How have ye done
Since last we saw in France?


NORFOLK.
I thank your Grace,
Healthful, and ever since a fresh admirer
Of what I saw there.

BUCKINGHAM.
An untimely ague
Stayed me a prisoner in my chamber when
Those suns of glory, those two lights of men,
Met in the vale of Andren.

NORFOLK.
’Twixt Guynes and Arde.
I was then present, saw them salute on horseback,
Beheld them when they lighted, how they clung
In their embracement, as they grew together—
Which had they, what four throned ones could have weighed
Such a compounded one?

BUCKINGHAM.
All the whole time
I was my chamber’s prisoner.

NORFOLK.
Then you lost
The view of earthly glory. Men might say,
Till this time pomp was single, but now married
To one above itself. Each following day
Became the next day’s master, till the last
Made former wonders its. Today the French,
All clinquant, all in gold, like heathen gods,
Shone down the English; and tomorrow, they
Made Britain India: every man that stood
Showed like a mine. Their dwarfish pages were
As cherubins, all gilt. The madams too,
Not used to toil, did almost sweat to bear
The pride upon them, that their very labour
Was to them as a painting. Now this masque
Was cried incomparable; and th’ ensuing night
Made it a fool and beggar. The two kings,
Equal in lustre, were now best, now worst,
As presence did present them: him in eye,
Still him in praise; and being present both,
’Twas said they saw but one, and no discerner
Durst wag his tongue in censure. When these suns—
For so they phrase ’em—by their heralds challenged
The noble spirits to arms, they did perform
Beyond thought’s compass, that former fabulous story,
Being now seen possible enough, got credit,
That Bevis was believed.

BUCKINGHAM.
O, you go far.

NORFOLK.
As I belong to worship and affect
In honour honesty, the tract of everything
Would by a good discourser lose some life,
Which action’s self was tongue to. All was royal;
To the disposing of it nought rebelled;
Order gave each thing view; the office did
Distinctly his full function.

BUCKINGHAM.
Who did guide,
I mean, who set the body and the limbs
Of this great sport together, as you guess?

NORFOLK.
One, certes, that promises no element
In such a business.

BUCKINGHAM.
I pray you who, my lord?

NORFOLK.
All this was ordered by the good discretion
Of the right reverend Cardinal of York.

BUCKINGHAM.
The devil speed him! No man’s pie is freed
From his ambitious finger. What had he
To do in these fierce vanities? I wonder
That such a keech can with his very bulk
Take up the rays o’ th’ beneficial sun
And keep it from the earth.

NORFOLK.
Surely, sir,
There’s in him stuff that puts him to these ends;
For, being not propped by ancestry, whose grace
Chalks successors their way, nor called upon
For high feats done to th’ crown; neither allied
To eminent assistants, but spider-like,
Out of his self-drawing web, he gives us note
The force of his own merit makes his way
A gift that heaven gives for him, which buys
A place next to the King.

ABERGAVENNY.
I cannot tell
What heaven hath given him—let some graver eye
Pierce into that—but I can see his pride
Peep through each part of him. Whence has he that?
If not from hell, the devil is a niggard,
Or has given all before, and he begins
A new hell in himself.

BUCKINGHAM.
Why the devil,
Upon this French going-out, took he upon him,
Without the privity o’ th’ King, t’ appoint
Who should attend on him? He makes up the file
Of all the gentry, for the most part such
To whom as great a charge as little honour
He meant to lay upon; and his own letter,
The honourable board of council out,
Must fetch him in he papers.

ABERGAVENNY.
I do know
Kinsmen of mine, three at the least, that have
By this so sickened their estates that never
They shall abound as formerly.

BUCKINGHAM.
O, many
Have broke their backs with laying manors on ’em
For this great journey. What did this vanity
But minister communication of
A most poor issue?

NORFOLK.
Grievingly I think
The peace between the French and us not values
The cost that did conclude it.

BUCKINGHAM.
Every man,
After the hideous storm that followed, was
A thing inspired and, not consulting, broke
Into a general prophecy, that this tempest,
Dashing the garment of this peace, aboded
The sudden breach on’t.

NORFOLK.
Which is budded out,
For France hath flawed the league, and hath attached
Our merchants’ goods at Bordeaux.

ABERGAVENNY.
Is it therefore
Th’ ambassador is silenced?

NORFOLK.
Marry, is’t.

ABERGAVENNY.
A proper title of a peace, and purchased
At a superfluous rate!

BUCKINGHAM.
Why, all this business
Our reverend Cardinal carried.

NORFOLK.
Like it your Grace,
The state takes notice of the private difference
Betwixt you and the Cardinal. I advise you—
And take it from a heart that wishes towards you
Honour and plenteous safety—that you read
The Cardinal’s malice and his potency
Together; to consider further that
What his high hatred would effect wants not
A minister in his power. You know his nature,
That he’s revengeful, and I know his sword
Hath a sharp edge; it’s long, and ’t may be said
It reaches far, and where ’twill not extend,
Thither he darts it. Bosom up my counsel;
You’ll find it wholesome. Lo, where comes that rock
That I advise your shunning.

Enter Cardinal Wolsey, the purse borne before him, certain of the Guard
and two Secretaries with papers. The Cardinal in his passage fixeth his
eye on Buckingham, and Buckingham on him, both full of disdain.

WOLSEY.
The Duke of Buckingham’s surveyor, ha?
Where’s his examination?

SECRETARY.
Here, so please you.

WOLSEY.
Is he in person ready?

SECRETARY.
Ay, please your Grace.

WOLSEY.
Well, we shall then know more, and Buckingham
Shall lessen this big look.

[_Exeunt Cardinal Wolsey and his train._]

BUCKINGHAM.
This butcher’s cur is venom-mouthed, and I
Have not the power to muzzle him; therefore best
Not wake him in his slumber. A beggar’s book
Outworths a noble’s blood.

NORFOLK.
What, are you chafed?
Ask God for temp’rance. That’s the appliance only
Which your disease requires.

BUCKINGHAM.
I read in ’s looks
Matter against me, and his eye reviled
Me as his abject object. At this instant
He bores me with some trick. He’s gone to th’ King.
I’ll follow, and outstare him.

NORFOLK.
Stay, my lord,
And let your reason with your choler question
What ’tis you go about. To climb steep hills
Requires slow pace at first. Anger is like
A full hot horse, who being allowed his way,
Self-mettle tires him. Not a man in England
Can advise me like you; be to yourself
As you would to your friend.

BUCKINGHAM.
I’ll to the King,
And from a mouth of honour quite cry down
This Ipswich fellow’s insolence, or proclaim
There’s difference in no persons.

NORFOLK.
Be advised.
Heat not a furnace for your foe so hot
That it do singe yourself. We may outrun
By violent swiftness that which we run at,
And lose by over-running. Know you not,
The fire that mounts the liquor till ’t run o’er,
In seeming to augment it wastes it? Be advised.
I say again, there is no English soul
More stronger to direct you than yourself,
If with the sap of reason you would quench,
Or but allay the fire of passion.

BUCKINGHAM.
Sir,
I am thankful to you, and I’ll go along
By your prescription; but this top-proud fellow—
Whom from the flow of gall I name not, but
From sincere motions—by intelligence,
And proofs as clear as founts in July when
We see each grain of gravel, I do know
To be corrupt and treasonous.

NORFOLK.
Say not “treasonous.”

BUCKINGHAM.
To th’ King I’ll say’t, and make my vouch as strong
As shore of rock. Attend. This holy fox,
Or wolf, or both—for he is equal ravenous
As he is subtle, and as prone to mischief
As able to perform’t, his mind and place
Infecting one another, yea, reciprocally—
Only to show his pomp as well in France
As here at home, suggests the King our master
To this last costly treaty, th’ interview,
That swallowed so much treasure, and like a glass
Did break i’ th’ rinsing.

NORFOLK.
Faith, and so it did.

BUCKINGHAM.
Pray give me favour, sir. This cunning Cardinal
The articles o’ th’ combination drew
As himself pleased; and they were ratified
As he cried “Thus let be,” to as much end
As give a crutch to the dead. But our Count-Cardinal
Has done this, and ’tis well, for worthy Wolsey,
Who cannot err, he did it. Now this follows—
Which, as I take it, is a kind of puppy
To the old dam treason—Charles the Emperor,
Under pretence to see the Queen his aunt—
For ’twas indeed his colour, but he came
To whisper Wolsey—here makes visitation.
His fears were that the interview betwixt
England and France might through their amity
Breed him some prejudice, for from this league
Peeped harms that menaced him. He privily
Deals with our Cardinal, and, as I trow—
Which I do well, for I am sure the Emperor
Paid ere he promised, whereby his suit was granted
Ere it was asked. But when the way was made
And paved with gold, the Emperor thus desired
That he would please to alter the King’s course
And break the foresaid peace. Let the King know,
As soon he shall by me, that thus the Cardinal
Does buy and sell his honour as he pleases
And for his own advantage.

NORFOLK.
I am sorry
To hear this of him, and could wish he were
Something mistaken in’t.

BUCKINGHAM.
No, not a syllable.
I do pronounce him in that very shape
He shall appear in proof.

Enter Brandon, a Sergeant-at-arms before him, and two or three of the
Guard.

BRANDON.
Your office, sergeant: execute it.

SERGEANT.
Sir,
My lord the Duke of Buckingham, and Earl
Of Hereford, Stafford, and Northampton, I
Arrest thee of high treason, in the name
Of our most sovereign King.

BUCKINGHAM.
Lo you, my lord,
The net has fall’n upon me. I shall perish
Under device and practice.

BRANDON.
I am sorry
To see you ta’en from liberty, to look on
The business present. ’Tis his Highness’ pleasure
You shall to th’ Tower.

BUCKINGHAM.
It will help nothing
To plead mine innocence, for that dye is on me
Which makes my whit’st part black. The will of heaven
Be done in this and all things. I obey.
O my Lord Abergavenny, fare you well.

BRANDON.
Nay, he must bear you company.
[_To Abergavenny_.] The King
Is pleased you shall to th’ Tower, till you know
How he determines further.

ABERGAVENNY.
As the Duke said,
The will of heaven be done, and the King’s pleasure
By me obeyed.

BRANDON.
Here is warrant from
The King t’ attach Lord Montague, and the bodies
Of the Duke’s confessor, John de la Car,
One Gilbert Peck, his chancellor—

BUCKINGHAM.
So, so;
These are the limbs o’ th’ plot. No more, I hope?

BRANDON.
A monk o’ th’ Chartreux.

BUCKINGHAM.
O, Nicholas Hopkins?

BRANDON.
He.

BUCKINGHAM.
My surveyor is false. The o’er-great Cardinal
Hath showed him gold. My life is spanned already.
I am the shadow of poor Buckingham,
Whose figure even this instant cloud puts on
By dark’ning my clear sun. My lord, farewell.

[_Exeunt._]

SCENE II. The same. The council-chamber.

Cornets. Enter King Henry, leaning on the Cardinal’s shoulder, the
Nobles, and Sir Thomas Lovell; the Cardinal places himself under the
King’s feet on his right side.

KING.
My life itself, and the best heart of it,
Thanks you for this great care. I stood i’ th’ level
Of a full-charged confederacy, and give thanks
To you that choked it. Let be called before us
That gentleman of Buckingham’s; in person
I’ll hear his confessions justify,
And point by point the treasons of his master
He shall again relate.

A noise within crying “Room for the Queen!” Enter  Queen Katherine,
ushered by the Duke of Norfolk and the Duke of Suffolk. She kneels. The
King riseth from his state, takes her up and kisses her.

QUEEN KATHERINE.
Nay, we must longer kneel; I am a suitor.

KING.
Arise, and take place by us.

[_He placeth her by him._]

Half your suit
Never name to us; you have half our power;
The other moiety ere you ask is given.
Repeat your will and take it.

QUEEN KATHERINE.
Thank your Majesty.
That you would love yourself, and in that love
Not unconsidered leave your honour nor
The dignity of your office, is the point
Of my petition.

KING.
Lady mine, proceed.

QUEEN KATHERINE.
I am solicited, not by a few,
And those of true condition, that your subjects
Are in great grievance. There have been commissions
Sent down among ’em which hath flawed the heart
Of all their loyalties; wherein, although,
My good Lord Cardinal, they vent reproaches
Most bitterly on you as putter-on
Of these exactions, yet the King our master,
Whose honour heaven shield from soil, even he escapes not
Language unmannerly, yea, such which breaks
The sides of loyalty, and almost appears
In loud rebellion.

NORFOLK.
Not “almost appears,”
It doth appear; for, upon these taxations,
The clothiers all, not able to maintain
The many to them longing, have put off
The spinsters, carders, fullers, weavers, who,
Unfit for other life, compelled by hunger
And lack of other means, in desperate manner
Daring the event to th’ teeth, are all in uproar,
And danger serves among them.

KING.
Taxation?
Wherein? And what taxation? My Lord Cardinal,
You that are blamed for it alike with us,
Know you of this taxation?

WOLSEY.
Please you, sir,
I know but of a single part in aught
Pertains to th’ state, and front but in that file
Where others tell steps with me.

QUEEN KATHERINE.
No, my lord?
You know no more than others? But you frame
Things that are known alike, which are not wholesome
To those which would not know them, and yet must
Perforce be their acquaintance. These exactions
Whereof my sovereign would have note, they are
Most pestilent to the hearing, and to bear ’em,
The back is sacrifice to the load. They say
They are devised by you, or else you suffer
Too hard an exclamation.

KING.
Still exaction!
The nature of it? In what kind, let’s know,
Is this exaction?

QUEEN KATHERINE.
I am much too venturous
In tempting of your patience, but am boldened
Under your promised pardon. The subjects’ grief
Comes through commissions, which compels from each
The sixth part of his substance, to be levied
Without delay; and the pretence for this
Is named your wars in France. This makes bold mouths.
Tongues spit their duties out, and cold hearts freeze
Allegiance in them. Their curses now
Live where their prayers did; and it’s come to pass
This tractable obedience is a slave
To each incensed will. I would your Highness
Would give it quick consideration, for
There is no primer business.

KING.
By my life,
This is against our pleasure.

WOLSEY.
And for me,
I have no further gone in this than by
A single voice, and that not passed me but
By learned approbation of the judges. If I am
Traduced by ignorant tongues, which neither know
My faculties nor person, yet will be
The chronicles of my doing, let me say
’Tis but the fate of place, and the rough brake
That virtue must go through. We must not stint
Our necessary actions in the fear
To cope malicious censurers, which ever,
As ravenous fishes, do a vessel follow
That is new-trimmed, but benefit no further
Than vainly longing. What we oft do best,
By sick interpreters, once weak ones, is
Not ours or not allowed; what worst, as oft,
Hitting a grosser quality, is cried up
For our best act. If we shall stand still
In fear our motion will be mocked or carped at,
We should take root here where we sit,
Or sit state-statues only.

KING.
Things done well,
And with a care, exempt themselves from fear;
Things done without example, in their issue
Are to be feared. Have you a precedent
Of this commission? I believe, not any.
We must not rend our subjects from our laws
And stick them in our will. Sixth part of each?
A trembling contribution! Why, we take
From every tree lop, bark, and part o’ t’ timber,
And though we leave it with a root, thus hacked,
The air will drink the sap. To every county
Where this is questioned send our letters with
Free pardon to each man that has denied
The force of this commission. Pray, look to’t;
I put it to your care.

WOLSEY.
[_Aside to his Secretary_.] A word with you.
Let there be letters writ to every shire
Of the King’s grace and pardon. The grieved commons
Hardly conceive of me. Let it be noised
That through our intercession this revokement
And pardon comes. I shall anon advise you
Further in the proceeding.

[_Exit Secretary._]

Enter Surveyor.

QUEEN KATHERINE.
I am sorry that the Duke of Buckingham
Is run in your displeasure.

KING.
It grieves many.
The gentleman is learned and a most rare speaker;
To nature none more bound; his training such
That he may furnish and instruct great teachers
And never seek for aid out of himself. Yet see,
When these so noble benefits shall prove
Not well disposed, the mind growing once corrupt,
They turn to vicious forms, ten times more ugly
Than ever they were fair. This man so complete,
Who was enrolled ’mongst wonders, and when we,
Almost with ravished list’ning, could not find
His hour of speech a minute—he, my lady,
Hath into monstrous habits put the graces
That once were his, and is become as black
As if besmeared in hell. Sit by us. You shall hear—
This was his gentleman in trust—of him
Things to strike honour sad. Bid him recount
The fore-recited practices, whereof
We cannot feel too little, hear too much.

WOLSEY.
Stand forth, and with bold spirit relate what you,
Most like a careful subject, have collected
Out of the Duke of Buckingham.

KING.
Speak freely.

SURVEYOR.
First, it was usual with him—every day
It would infect his speech—that if the King
Should without issue die, he’ll carry it so
To make the sceptre his. These very words
I’ve heard him utter to his son-in-law,
Lord Abergavenny; to whom by oath he menaced
Revenge upon the Cardinal.

WOLSEY.
Please your Highness, note
This dangerous conception in this point,
Not friended by his wish to your high person
His will is most malignant, and it stretches
Beyond you to your friends.

QUEEN KATHERINE.
My learned Lord Cardinal,
Deliver all with charity.

KING.
Speak on.
How grounded he his title to the crown?
Upon our fail? To this point hast thou heard him
At any time speak aught?

SURVEYOR.
He was brought to this
By a vain prophecy of Nicholas Henton.

KING.
What was that Henton?

SURVEYOR.
Sir, a Chartreux friar,
His confessor, who fed him every minute
With words of sovereignty.

KING.
How know’st thou this?

SURVEYOR.
Not long before your Highness sped to France,
The Duke being at the Rose, within the parish
Saint Laurence Poultney, did of me demand
What was the speech among the Londoners
Concerning the French journey. I replied,
Men fear the French would prove perfidious,
To the King’s danger. Presently the Duke
Said ’twas the fear indeed, and that he doubted
’Twould prove the verity of certain words
Spoke by a holy monk, “that oft,” says he,
“Hath sent to me, wishing me to permit
John de la Car, my chaplain, a choice hour
To hear from him a matter of some moment;
Whom after under the confession’s seal
He solemnly had sworn that what he spoke
My chaplain to no creature living but
To me should utter, with demure confidence
This pausingly ensued: ‘Neither the King nor’s heirs,
Tell you the Duke—shall prosper. Bid him strive
To gain the love o’ th’ commonalty. The Duke
Shall govern England.’”

QUEEN KATHERINE.
If I know you well,
You were the Duke’s surveyor, and lost your office
On the complaint o’ th’ tenants. Take good heed
You charge not in your spleen a noble person
And spoil your nobler soul. I say, take heed—
Yes, heartily beseech you.

KING.
Let him on.
Go forward.

SURVEYOR.
On my soul, I’ll speak but truth.
I told my lord the Duke, by th’ devil’s illusions
The monk might be deceived, and that ’twas dangerous
For him to ruminate on this so far until
It forged him some design, which, being believed,
It was much like to do. He answered, “Tush,
It can do me no damage,” adding further
That had the King in his last sickness failed,
The Cardinal’s and Sir Thomas Lovell’s heads
Should have gone off.

KING.
Ha! What, so rank? Ah ha!
There’s mischief in this man. Canst thou say further?

SURVEYOR.
I can, my liege.

KING.
Proceed.

SURVEYOR.
Being at Greenwich,
After your Highness had reproved the Duke
About Sir William Bulmer—

KING.
I remember
Of such a time, being my sworn servant,
The Duke retained him his. But on. What hence?

SURVEYOR.
“If,” quoth he, “I for this had been committed,”
As to the Tower, I thought, “I would have played
The part my father meant to act upon
Th’ usurper Richard who, being at Salisbury,
Made suit to come in ’s presence; which if granted,
As he made semblance of his duty, would
Have put his knife into him.”

KING.
A giant traitor!

WOLSEY.
Now, madam, may his Highness live in freedom,
And this man out of prison?

QUEEN KATHERINE.
God mend all.

KING.
There’s something more would out of thee. What sayst?

SURVEYOR.
After “the Duke his father,” with “the knife,”
He stretched him, and with one hand on his dagger,
Another spread on ’s breast, mounting his eyes,
He did discharge a horrible oath, whose tenour
Was, were he evil used, he would outgo
His father by as much as a performance
Does an irresolute purpose.

KING.
There’s his period,
To sheathe his knife in us. He is attached.
Call him to present trial. If he may
Find mercy in the law, ’tis his; if none,
Let him not seek ’t of us. By day and night,
He’s traitor to th’ height!

[_Exeunt._]

SCENE III. An ante-chamber in the palace.

Enter Lord Chamberlain  and Lord Sandys.

CHAMBERLAIN.
Is’t possible the spells of France should juggle
Men into such strange mysteries?

SANDYS.
New customs,
Though they be never so ridiculous—
Nay, let ’em be unmanly—yet are followed.

CHAMBERLAIN.
As far as I see, all the good our English
Have got by the late voyage is but merely
A fit or two o’ th’ face; but they are shrewd ones,
For when they hold ’em, you would swear directly
Their very noses had been counsellors
To Pepin or Clotharius, they keep state so.

SANDYS.
They have all new legs, and lame ones. One would take it,
That never saw ’em pace before, the spavin
Or springhalt reigned among ’em.

CHAMBERLAIN.
Death! My lord,
Their clothes are after such a pagan cut to’t,
That, sure, they’ve worn out Christendom.

Enter Sir Thomas Lovell.

How now?
What news, Sir Thomas Lovell?

LOVELL.
Faith, my lord,
I hear of none but the new proclamation
That’s clapped upon the court gate.

CHAMBERLAIN.
What is’t for?

LOVELL.
The reformation of our travelled gallants
That fill the court with quarrels, talk, and tailors.

CHAMBERLAIN.
I’m glad ’tis there. Now I would pray our monsieurs
To think an English courtier may be wise
And never see the Louvre.

LOVELL.
They must either,
For so run the conditions, leave those remnants
Of fool and feather that they got in France,
With all their honourable points of ignorance
Pertaining thereunto, as fights and fireworks,
Abusing better men than they can be
Out of a foreign wisdom, renouncing clean
The faith they have in tennis and tall stockings,
Short blistered breeches, and those types of travel,
And understand again like honest men,
Or pack to their old playfellows. There, I take it,
They may, _cum privilegio, oui_ away
The lag end of their lewdness and be laughed at.

SANDYS.
’Tis time to give ’em physic, their diseases
Are grown so catching.

CHAMBERLAIN.
What a loss our ladies
Will have of these trim vanities!

LOVELL.
Ay, marry,
There will be woe indeed, lords. The sly whoresons
Have got a speeding trick to lay down ladies.
A French song and a fiddle has no fellow.

SANDYS.
The devil fiddle ’em! I am glad they are going,
For sure, there’s no converting of ’em. Now
An honest country lord, as I am, beaten
A long time out of play, may bring his plainsong
And have an hour of hearing, and, by ’r Lady,
Held current music too.

CHAMBERLAIN.
Well said, Lord Sandys.
Your colt’s tooth is not cast yet.

SANDYS.
No, my lord,
Nor shall not while I have a stump.

CHAMBERLAIN.
Sir Thomas,
Whither were you a-going?

LOVELL.
To the Cardinal’s.
Your lordship is a guest too.

CHAMBERLAIN.
O, ’tis true.
This night he makes a supper, and a great one,
To many lords and ladies. There will be
The beauty of this kingdom, I’ll assure you.

LOVELL.
That churchman bears a bounteous mind indeed,
A hand as fruitful as the land that feeds us.
His dews fall everywhere.

CHAMBERLAIN.
No doubt he’s noble;
He had a black mouth that said other of him.

SANDYS.
He may, my lord; has wherewithal. In him
Sparing would show a worse sin than ill doctrine.
Men of his way should be most liberal;
They are set here for examples.

CHAMBERLAIN.
True, they are so,
But few now give so great ones. My barge stays.
Your lordship shall along. Come, good Sir Thomas,
We shall be late else, which I would not be,
For I was spoke to, with Sir Henry Guildford,
This night to be comptrollers.

SANDYS.
I am your lordship’s.

[_Exeunt._]

SCENE IV. A Hall in York Place.

Hautboys. A small table under a state for the Cardinal, a longer table
for the guests. Then enter Anne Bullen and divers other Ladies and
Gentlemen as guests, at one door. At another door enter Sir Henry
Guildford.

GUILDFORD.
Ladies, a general welcome from his Grace
Salutes ye all. This night he dedicates
To fair content and you. None here, he hopes,
In all this noble bevy has brought with her
One care abroad. He would have all as merry
As, first, good company, good wine, good welcome
Can make good people.

Enter Lord Chamberlain, Lord Sandys and Sir Thomas Lovell.

O, my lord, you’re tardy.
The very thought of this fair company
Clapped wings to me.

CHAMBERLAIN.
You are young, Sir Harry Guildford.

SANDYS.
Sir Thomas Lovell, had the Cardinal
But half my lay thoughts in him, some of these
Should find a running banquet ere they rested,
I think would better please ’em. By my life,
They are a sweet society of fair ones.

LOVELL.
O, that your lordship were but now confessor
To one or two of these!

SANDYS.
I would I were.
They should find easy penance.

LOVELL.
Faith, how easy?

SANDYS.
As easy as a down bed would afford it.

CHAMBERLAIN.
Sweet ladies, will it please you sit? Sir Harry,
Place you that side; I’ll take the charge of this.
His Grace is ent’ring. Nay, you must not freeze;
Two women placed together makes cold weather.
My Lord Sandys, you are one will keep ’em waking.
Pray, sit between these ladies.

SANDYS.
By my faith,
And thank your lordship. By your leave, sweet ladies.
If I chance to talk a little wild, forgive me;
I had it from my father.

ANNE.
Was he mad, sir?

SANDYS.
O, very mad, exceeding mad in love too;
But he would bite none. Just as I do now,
He would kiss you twenty with a breath.

[_Kisses her._]

CHAMBERLAIN.
Well said, my lord.
So, now you’re fairly seated. gentlemen,
The penance lies on you if these fair ladies
Pass away frowning.

SANDYS.
For my little cure,
Let me alone.

Hautboys. Enter Cardinal Wolsey and takes his state.

WOLSEY.
You’re welcome, my fair guests. That noble lady
Or gentleman that is not freely merry
Is not my friend. This, to confirm my welcome;
And to you all, good health.

[_Drinks._]

SANDYS.
Your Grace is noble.
Let me have such a bowl may hold my thanks
And save me so much talking.

WOLSEY.
My Lord Sandys,
I am beholding to you. Cheer your neighbours.
Ladies, you are not merry. Gentlemen,
Whose fault is this?

SANDYS.
The red wine first must rise
In their fair cheeks, my lord; then we shall have ’em
Talk us to silence.

ANNE.
You are a merry gamester,
My Lord Sandys.

SANDYS.
Yes, if I make my play.
Here’s to your ladyship; and pledge it, madam,
For ’tis to such a thing—

ANNE.
You cannot show me.

SANDYS.
I told your Grace they would talk anon.

[_Drum and trumpet. Chambers discharged._]

WOLSEY.
What’s that?

CHAMBERLAIN.
Look out there, some of ye.

[_Exit Servant._]

WOLSEY.
What warlike voice,
And to what end, is this? Nay, ladies, fear not.
By all the laws of war you’re privileged.

Enter Servant.

CHAMBERLAIN.
How now, what is’t?

SERVANT.
A noble troop of strangers,
For so they seem. They’ve left their barge and landed,
And hither make, as great ambassadors
From foreign princes.

WOLSEY.
Good Lord Chamberlain,
Go, give ’em welcome—you can speak the French tongue—
And pray receive ’em nobly, and conduct ’em
Into our presence, where this heaven of beauty
Shall shine at full upon them. Some attend him.

[_Exit Chamberlain, attended. All rise, and tables removed._]

You have now a broken banquet, but we’ll mend it.
A good digestion to you all; and once more
I shower a welcome on ye. Welcome all!

Hautboys. Enter King and others as masquers, habited like shepherds,
ushered by the Lord Chamberlain. They pass directly before the Cardinal
and gracefully salute him.

A noble company! What are their pleasures?

CHAMBERLAIN.
Because they speak no English, thus they prayed
To tell your Grace: that having heard by fame
Of this so noble and so fair assembly
This night to meet here, they could do no less,
Out of the great respect they bear to beauty,
But leave their flocks and, under your fair conduct,
Crave leave to view these ladies and entreat
An hour of revels with ’em.

WOLSEY.
Say, Lord Chamberlain,
They have done my poor house grace; for which I pay ’em
A thousand thanks and pray ’em take their pleasures.

[_The masquers choose ladies. The King chooses Anne Bullen._]

KING.
The fairest hand I ever touched! O beauty,
Till now I never knew thee.

[_Music. Dance._]

WOLSEY.
My lord!

CHAMBERLAIN.
Your Grace?

WOLSEY.
Pray tell ’em thus much from me:
There should be one amongst ’em, by his person
More worthy this place than myself, to whom,
If I but knew him, with my love and duty
I would surrender it.

CHAMBERLAIN.
I will, my lord.

[_Whispers with the Masquers._]

WOLSEY.
What say they?

CHAMBERLAIN.
Such a one they all confess
There is indeed, which they would have your Grace
Find out, and he will take it.

WOLSEY.
Let me see, then.
By all your good leaves, gentlemen; here I’ll make
My royal choice.

KING.
[_Unmasking_.] Ye have found him, Cardinal.
You hold a fair assembly; you do well, lord.
You are a churchman, or I’ll tell you, Cardinal,
I should judge now unhappily.

WOLSEY.
I am glad
Your Grace is grown so pleasant.

KING.
My Lord Chamberlain,
Prithee come hither. What fair lady’s that?

CHAMBERLAIN.
An’t please your Grace, Sir Thomas Bullen’s daughter,
The Viscount Rochford, one of her Highness’ women.

KING.
By heaven, she is a dainty one. Sweetheart,
I were unmannerly to take you out
And not to kiss you. A health, gentlemen!
Let it go round.

WOLSEY.
Sir Thomas Lovell, is the banquet ready
I’ th’ privy chamber?

LOVELL.
Yes, my lord.

WOLSEY.
Your Grace,
I fear, with dancing is a little heated.

KING.
I fear, too much.

WOLSEY.
There’s fresher air, my lord,
In the next chamber.

KING.
Lead in your ladies, every one. Sweet partner,
I must not yet forsake you. Let’s be merry,
Good my Lord Cardinal, I have half a dozen healths
To drink to these fair ladies, and a measure
To lead ’em once again, and then let’s dream
Who’s best in favour. Let the music knock it.

[_Exeunt with trumpets._]




ACT II

SCENE I. Westminster. A street.


Enter two Gentlemen at several doors.

FIRST GENTLEMAN.
Whither away so fast?

SECOND GENTLEMAN.
O, God save ye.
Even to the Hall, to hear what shall become
Of the great Duke of Buckingham.

FIRST GENTLEMAN.
I’ll save you
That labour, sir. All’s now done but the ceremony
Of bringing back the prisoner.

SECOND GENTLEMAN.
Were you there?

FIRST GENTLEMAN.
Yes, indeed, was I.

SECOND GENTLEMAN.
Pray speak what has happened.

FIRST GENTLEMAN.
You may guess quickly what.

SECOND GENTLEMAN.
Is he found guilty?

FIRST GENTLEMAN.
Yes, truly is he, and condemned upon’t.

SECOND GENTLEMAN.
I am sorry for’t.

FIRST GENTLEMAN.
So are a number more.

SECOND GENTLEMAN.
But pray, how passed it?

FIRST GENTLEMAN.
I’ll tell you in a little. The great Duke
Came to the bar, where to his accusations
He pleaded still not guilty and alleged
Many sharp reasons to defeat the law.
The King’s attorney on the contrary
Urged on the examinations, proofs, confessions
Of divers witnesses, which the Duke desired
To have brought _viva voce_ to his face;
At which appeared against him his surveyor,
Sir Gilbert Peck his chancellor, and John Car,
Confessor to him, with that devil monk,
Hopkins, that made this mischief.

SECOND GENTLEMAN.
That was he
That fed him with his prophecies?

FIRST GENTLEMAN.
The same.
All these accused him strongly, which he fain
Would have flung from him, but, indeed he could not.
And so his peers, upon this evidence,
Have found him guilty of high treason. Much
He spoke, and learnedly, for life, but all
Was either pitied in him or forgotten.

SECOND GENTLEMAN.
After all this, how did he bear himself?

FIRST GENTLEMAN.
When he was brought again to th’ bar to hear
His knell rung out, his judgement, he was stirred
With such an agony, he sweat extremely
And something spoke in choler, ill and hasty.
But he fell to himself again, and sweetly
In all the rest showed a most noble patience.

SECOND GENTLEMAN.
I do not think he fears death.

FIRST GENTLEMAN.
Sure he does not;
He never was so womanish. The cause
He may a little grieve at.

SECOND GENTLEMAN.
Certainly
The Cardinal is the end of this.

FIRST GENTLEMAN.
’Tis likely,
By all conjectures: first, Kildare’s attainder,
Then deputy of Ireland, who removed,
Earl Surrey was sent thither, and in haste too,
Lest he should help his father.

SECOND GENTLEMAN.
That trick of state
Was a deep envious one.

FIRST GENTLEMAN.
At his return
No doubt he will requite it. This is noted,
And generally, whoever the King favours,
The Cardinal instantly will find employment,
And far enough from court too.

SECOND GENTLEMAN.
All the commons
Hate him perniciously and, o’ my conscience,
Wish him ten fathom deep. This duke as much
They love and dote on, call him bounteous Buckingham,
The mirror of all courtesy.

Enter Buckingham from his arraignment. Tipstaves before him, the axe
with the edge towards him, Halberds on each side, accompanied with Sir
Thomas Lovell, Sir Nicholas Vaux, Sir William Sandys and common people.

FIRST GENTLEMAN.
Stay there, sir,
And see the noble ruined man you speak of.

SECOND GENTLEMAN.
Let’s stand close and behold him.

BUCKINGHAM.
All good people,
You that thus far have come to pity me,
Hear what I say, and then go home and lose me.
I have this day received a traitor’s judgement,
And by that name must die; yet heaven bear witness,
And if I have a conscience, let it sink me,
Even as the axe falls, if I be not faithful!
The law I bear no malice for my death;
’T has done, upon the premises, but justice.
But those that sought it I could wish more Christians.
Be what they will, I heartily forgive ’em.
Yet let ’em look they glory not in mischief,
Nor build their evils on the graves of great men,
For then my guiltless blood must cry against ’em.
For further life in this world I ne’er hope,
Nor will I sue, although the King have mercies
More than I dare make faults. You few that loved me
And dare be bold to weep for Buckingham,
His noble friends and fellows, whom to leave
Is only bitter to him, only dying,
Go with me like good angels to my end,
And as the long divorce of steel falls on me,
Make of your prayers one sweet sacrifice,
And lift my soul to heaven. Lead on, i’ God’s name.

LOVELL.
I do beseech your Grace, for charity,
If ever any malice in your heart
Were hid against me, now to forgive me frankly.

BUCKINGHAM.
Sir Thomas Lovell, I as free forgive you
As I would be forgiven. I forgive all.
There cannot be those numberless offences
’Gainst me that I cannot take peace with. No black envy
Shall mark my grave. Commend me to his Grace,
And if he speak of Buckingham, pray tell him
You met him half in heaven. My vows and prayers
Yet are the King’s and, till my soul forsake,
Shall cry for blessings on him. May he live
Longer than I have time to tell his years;
Ever beloved and loving may his rule be;
And when old Time shall lead him to his end,
Goodness and he fill up one monument!

LOVELL.
To th’ waterside I must conduct your Grace,
Then give my charge up to Sir Nicholas Vaux,
Who undertakes you to your end.

VAUX.
Prepare there!
The Duke is coming. See the barge be ready,
And fit it with such furniture as suits
The greatness of his person.

BUCKINGHAM.
Nay, Sir Nicholas,
Let it alone. My state now will but mock me.
When I came hither, I was Lord High Constable
And Duke of Buckingham; now, poor Edward Bohun.
Yet I am richer than my base accusers,
That never knew what truth meant. I now seal it,
And with that blood will make ’em one day groan for’t.
My noble father, Henry of Buckingham,
Who first raised head against usurping Richard,
Flying for succour to his servant Banister,
Being distressed, was by that wretch betrayed,
And, without trial, fell. God’s peace be with him.
Henry the Seventh succeeding, truly pitying
My father’s loss, like a most royal prince,
Restored me to my honours and out of ruins
Made my name once more noble. Now his son,
Henry the Eighth, life, honour, name, and all
That made me happy at one stroke has taken
For ever from the world. I had my trial,
And must needs say a noble one, which makes me
A little happier than my wretched father.
Yet thus far we are one in fortunes: both
Fell by our servants, by those men we loved most—
A most unnatural and faithless service.
Heaven has an end in all; yet, you that hear me,
This from a dying man receive as certain:
Where you are liberal of your loves and counsels
Be sure you be not loose; for those you make friends
And give your hearts to, when they once perceive
The least rub in your fortunes, fall away
Like water from ye, never found again
But where they mean to sink ye. All good people,
Pray for me. I must now forsake ye. The last hour
Of my long weary life is come upon me.
Farewell. And when you would say something that is sad,
Speak how I fell. I have done; and God forgive me.

[_Exeunt Duke and train._]

FIRST GENTLEMAN.
O, this is full of pity. Sir, it calls,
I fear, too many curses on their heads
That were the authors.

SECOND GENTLEMAN.
If the Duke be guiltless,
’Tis full of woe. Yet I can give you inkling
Of an ensuing evil, if it fall,
Greater than this.

FIRST GENTLEMAN.
Good angels keep it from us!
What may it be? You do not doubt my faith, sir?

SECOND GENTLEMAN.
This secret is so weighty, ’twill require
A strong faith to conceal it.

FIRST GENTLEMAN.
Let me have it.
I do not talk much.

SECOND GENTLEMAN.
I am confident;
You shall, sir. Did you not of late days hear
A buzzing of a separation
Between the King and Katherine?

FIRST GENTLEMAN.
Yes, but it held not;
For when the King once heard it, out of anger
He sent command to the Lord Mayor straight
To stop the rumour and allay those tongues
That durst disperse it.

SECOND GENTLEMAN.
But that slander, sir,
Is found a truth now, for it grows again
Fresher than e’er it was, and held for certain
The King will venture at it. Either the Cardinal,
Or some about him near, have, out of malice
To the good Queen, possessed him with a scruple
That will undo her. To confirm this too,
Cardinal Campeius is arrived, and lately,
As all think, for this business.

FIRST GENTLEMAN.
’Tis the Cardinal;
And merely to revenge him on the Emperor
For not bestowing on him at his asking,
The archbishopric of Toledo this is purposed.

SECOND GENTLEMAN.
I think you have hit the mark. But is’t not cruel
That she should feel the smart of this? The Cardinal
Will have his will, and she must fall.

FIRST GENTLEMAN.
’Tis woeful.
We are too open here to argue this.
Let’s think in private more.

[_Exeunt._]

SCENE II. An ante-chamber in the palace.

Enter Lord Chamberlain, reading this letter.

CHAMBERLAIN.
_My lord, the horses your lordship sent for, with all the care had I
saw well chosen, ridden, and furnished. They were young and handsome,
and of the best breed in the north. When they were ready to set out for
London, a man of my Lord Cardinal’s, by commission and main power, took
’em from me, with this reason: his master would be served before a
subject, if not before the King; which stopped our mouths, sir._
I fear he will indeed. Well, let him have them.
He will have all, I think.

Enter to the Lord Chamberlain, the Dukes of Norfolk and Suffolk.

NORFOLK.
Well met, my Lord Chamberlain.

CHAMBERLAIN.
Good day to both your Graces.

SUFFOLK.
How is the King employed?

CHAMBERLAIN.
I left him private,
Full of sad thoughts and troubles.

NORFOLK.
What’s the cause?

CHAMBERLAIN.
It seems the marriage with his brother’s wife
Has crept too near his conscience.

SUFFOLK.
No, his conscience
Has crept too near another lady.

NORFOLK.
’Tis so.
This is the Cardinal’s doing, the king-cardinal.
That blind priest, like the eldest son of Fortune,
Turns what he list. The King will know him one day.

SUFFOLK.
Pray God he do! He’ll never know himself else.

NORFOLK.
How holily he works in all his business,
And with what zeal! For, now he has cracked the league
Between us and the Emperor, the Queen’s great nephew,
He dives into the King’s soul and there scatters
Dangers, doubts, wringing of the conscience,
Fears and despairs—and all these for his marriage.
And out of all these to restore the King,
He counsels a divorce, a loss of her
That like a jewel has hung twenty years
About his neck, yet never lost her lustre;
Of her that loves him with that excellence
That angels love good men with; even of her
That, when the greatest stroke of fortune falls,
Will bless the King. And is not this course pious?

CHAMBERLAIN.
Heaven keep me from such counsel! ’Tis most true:
These news are everywhere, every tongue speaks ’em,
And every true heart weeps for’t. All that dare
Look into these affairs see this main end,
The French king’s sister. Heaven will one day open
The King’s eyes, that so long have slept upon
This bold bad man.

SUFFOLK.
And free us from his slavery.

NORFOLK.
We had need pray,
And heartily, for our deliverance,
Or this imperious man will work us all
From princes into pages. All men’s honours
Lie like one lump before him, to be fashioned
Into what pitch he please.

SUFFOLK.
For me, my lords,
I love him not, nor fear him; there’s my creed.
As I am made without him, so I’ll stand,
If the King please. His curses and his blessings
Touch me alike, they’re breath I not believe in.
I knew him, and I know him; so I leave him
To him that made him proud, the Pope.

NORFOLK.
Let’s in,
And with some other business put the King
From these sad thoughts that work too much upon him.
My lord, you’ll bear us company?

CHAMBERLAIN.
Excuse me;
The King has sent me otherwhere. Besides,
You’ll find a most unfit time to disturb him.
Health to your lordships.

NORFOLK.
Thanks, my good Lord Chamberlain.

[_Exit Lord Chamberlain, and the King draws the curtain and sits
reading pensively._]

SUFFOLK.
How sad he looks! Sure, he is much afflicted.

KING.
Who’s there? Ha?

NORFOLK.
Pray God he be not angry.

KING.
Who’s there, I say? How dare you thrust yourselves
Into my private meditations?
Who am I? Ha?

NORFOLK.
A gracious king that pardons all offences
Malice ne’er meant. Our breach of duty this way
Is business of estate, in which we come
To know your royal pleasure.

KING.
Ye are too bold.
Go to; I’ll make ye know your times of business.
Is this an hour for temporal affairs, ha?

Enter Wolsey and Campeius with a commission.

Who’s there? My good Lord Cardinal? O my Wolsey,
The quiet of my wounded conscience,
Thou art a cure fit for a king. [_To Campeius_.] You’re welcome,
Most learned reverend sir, into our kingdom;
Use us and it. [_To Wolsey_.] My good lord, have great care
I be not found a talker.

WOLSEY.
Sir, you cannot.
I would your Grace would give us but an hour
Of private conference.

KING.
[_To Norfolk and Suffolk_.] We are busy. Go.

NORFOLK.
[A_side to Suffolk_.] This priest has no pride in him?

SUFFOLK.
[_Aside to Norfolk_.] Not to speak of.
I would not be so sick, though, for his place.
But this cannot continue.

NORFOLK.
[_Aside to Suffolk_.] If it do,
I’ll venture one have-at-him.

SUFFOLK.
[_Aside to Norfolk_.] I another.

[_Exeunt Norfolk and Suffolk._]

WOLSEY.
Your Grace has given a precedent of wisdom
Above all princes in committing freely
Your scruple to the voice of Christendom.
Who can be angry now? What envy reach you?
The Spaniard, tied by blood and favour to her,
Must now confess, if they have any goodness,
The trial just and noble. All the clerks—
I mean the learned ones in Christian kingdoms—
Have their free voices. Rome, the nurse of judgement,
Invited by your noble self, hath sent
One general tongue unto us, this good man,
This just and learned priest, Cardinal Campeius,
Whom once more I present unto your Highness.

KING.
And once more in mine arms I bid him welcome,
And thank the holy conclave for their loves.
They have sent me such a man I would have wished for.

CAMPEIUS.
Your Grace must needs deserve all strangers’ loves,
You are so noble. To your Highness’ hand
I tender my commission, by whose virtue,
The court of Rome commanding, you, my Lord
Cardinal of York, are joined with me their servant
In the unpartial judging of this business.

KING.
Two equal men. The Queen shall be acquainted
Forthwith for what you come. Where’s Gardiner?

WOLSEY.
I know your Majesty has always loved her
So dear in heart not to deny her that
A woman of less place might ask by law:
Scholars allowed freely to argue for her.

KING.
Ay, and the best she shall have, and my favour
To him that does best. God forbid else. Cardinal,
Prithee call Gardiner to me, my new secretary.
I find him a fit fellow.

Enter Gardiner.

WOLSEY.
[_Aside to Gardiner_.]
Give me your hand. Much joy and favour to you;
You are the King’s now.

GARDINER.
[_Aside to Wolsey_.] But to be commanded
For ever by your Grace, whose hand has raised me.

KING.
Come hither, Gardiner.

[_The King and Gardiner walk and whisper._]

CAMPEIUS.
My lord of York, was not one Doctor Pace
In this man’s place before him?

WOLSEY.
Yes, he was.

CAMPEIUS.
Was he not held a learned man?

WOLSEY.
Yes, surely.

CAMPEIUS.
Believe me, there’s an ill opinion spread, then
Even of yourself, Lord Cardinal.

WOLSEY.
How? Of me?

CAMPEIUS.
They will not stick to say you envied him
And fearing he would rise—he was so virtuous—
Kept him a foreign man still, which so grieved him
That he ran mad and died.

WOLSEY.
Heav’n’s peace be with him!
That’s Christian care enough. For living murmurers
There’s places of rebuke. He was a fool,
For he would needs be virtuous. That good fellow,
If I command him, follows my appointment.
I will have none so near else. Learn this, brother:
We live not to be griped by meaner persons.

KING.
Deliver this with modesty to th’ Queen.

[_Exit Gardiner._]

The most convenient place that I can think of
For such receipt of learning is Blackfriars.
There ye shall meet about this weighty business.
My Wolsey, see it furnished. O, my lord,
Would it not grieve an able man to leave
So sweet a bedfellow? But, conscience, conscience!
O, ’tis a tender place, and I must leave her.

[_Exeunt._]

SCENE III. An ante-chamber of the Queen’s apartments.

Enter Anne Bullen and an Old Lady.

ANNE.
Not for that neither. Here’s the pang that pinches:
His Highness having lived so long with her, and she
So good a lady that no tongue could ever
Pronounce dishonour of her—by my life,
She never knew harm-doing—O, now, after
So many courses of the sun enthroned,
Still growing in a majesty and pomp, the which
To leave a thousandfold more bitter than
’Tis sweet at first t’ acquire—after this process,
To give her the avaunt, it is a pity
Would move a monster.

OLD LADY.
Hearts of most hard temper
Melt and lament for her.

ANNE.
O, God’s will! Much better
She ne’er had known pomp; though’t be temporal,
Yet if that quarrel, Fortune, do divorce
It from the bearer, ’tis a sufferance panging
As soul and body’s severing.

OLD LADY.
Alas, poor lady,
She’s a stranger now again.

ANNE.
So much the more
Must pity drop upon her. Verily,
I swear, ’tis better to be lowly born
And range with humble livers in content
Than to be perked up in a glist’ring grief,
And wear a golden sorrow.

OLD LADY.
Our content
Is our best having.

ANNE.
By my troth and maidenhead,
I would not be a queen.

OLD LADY.
Beshrew me, I would,
And venture maidenhead for’t; and so would you,
For all this spice of your hypocrisy.
You, that have so fair parts of woman on you,
Have too a woman’s heart, which ever yet
Affected eminence, wealth, sovereignty;
Which, to say sooth, are blessings; and which gifts,
Saving your mincing, the capacity
Of your soft cheveril conscience would receive,
If you might please to stretch it.

ANNE.
Nay, good troth.

OLD LADY.
Yes, troth and troth. You would not be a queen?

ANNE.
No, not for all the riches under heaven.

OLD LADY.
’Tis strange. A threepence bowed would hire me,
Old as I am, to queen it. But I pray you,
What think you of a duchess? Have you limbs
To bear that load of title?

ANNE.
No, in truth.

OLD LADY.
Then you are weakly made. Pluck off a little.
I would not be a young count in your way
For more than blushing comes to. If your back
Cannot vouchsafe this burden, ’tis too weak
Ever to get a boy.

ANNE.
How you do talk!
I swear again I would not be a queen
For all the world.

OLD LADY.
In faith, for little England
You’d venture an emballing. I myself
Would for Caernarfonshire, although there longed
No more to th’ crown but that. Lo, who comes here?

Enter Lord Chamberlain.

CHAMBERLAIN.
Good morrow, ladies. What were’t worth to know
The secret of your conference?

ANNE.
My good lord,
Not your demand; it values not your asking.
Our mistress’ sorrows we were pitying.

CHAMBERLAIN.
It was a gentle business, and becoming
The action of good women. There is hope
All will be well.

ANNE.
Now, I pray God, amen!

CHAMBERLAIN.
You bear a gentle mind, and heavenly blessings
Follow such creatures. That you may, fair lady,
Perceive I speak sincerely, and high note’s
Ta’en of your many virtues, the King’s Majesty
Commends his good opinion of you, and
Does purpose honour to you no less flowing
Than Marchioness of Pembroke, to which title
A thousand pound a year annual support
Out of his grace he adds.

ANNE.
I do not know
What kind of my obedience I should tender.
More than my all is nothing; nor my prayers
Are not words duly hallowed, nor my wishes
More worth than empty vanities; yet prayers and wishes
Are all I can return. Beseech your lordship,
Vouchsafe to speak my thanks and my obedience,
As from a blushing handmaid, to his Highness,
Whose health and royalty I pray for.

CHAMBERLAIN.
Lady,
I shall not fail t’ approve the fair conceit
The King hath of you. [_Aside_.] I have perused her well.
Beauty and honour in her are so mingled
That they have caught the King; and who knows yet
But from this lady may proceed a gem
To lighten all this isle? I’ll to the King,
And say I spoke with you.

ANNE.
My honoured lord.

[_Exit Lord Chamberlain._]

OLD LADY.
Why, this it is: see, see!
I have been begging sixteen years in court,
Am yet a courtier beggarly, nor could
Come pat betwixt too early and too late
For any suit of pounds; and you, O fate!
A very fresh fish here—fie, fie, fie upon
This compelled fortune!—have your mouth filled up
Before you open it.

ANNE.
This is strange to me.

OLD LADY.
How tastes it? Is it bitter? Forty pence, no.
There was a lady once—’tis an old story—
That would not be a queen, that would she not,
For all the mud in Egypt. Have you heard it?

ANNE.
Come, you are pleasant.

OLD LADY.
With your theme, I could
O’ermount the lark. The Marchioness of Pembroke?
A thousand pounds a year for pure respect?
No other obligation? By my life,
That promises more thousands; honour’s train
Is longer than his foreskirt. By this time
I know your back will bear a duchess. Say,
Are you not stronger than you were?

ANNE.
Good lady,
Make yourself mirth with your particular fancy,
And leave me out on’t. Would I had no being
If this salute my blood a jot. It faints me
To think what follows.
The Queen is comfortless, and we forgetful
In our long absence. Pray do not deliver
What here you’ve heard to her.

OLD LADY.
What do you think me?

[_Exeunt._]

SCENE IV. A hall in Blackfriars.

Trumpets, sennet, and cornets. Enter two Vergers, with short silver
wands; next them, two Scribes, in the habit of doctors; after them, the
Archbishop of Canterbury alone; after him, the Bishops of Lincoln, Ely,
Rochester, and Saint Asaph; next them, with some small distance,
follows a Gentleman bearing the purse with the great seal, and a
cardinal’s hat; then two Priests, bearing each a silver cross; then a
Gentleman Usher bare-headed, accompanied with a Sergeant-at-arms
bearing a silver mace; then two Gentlemen, bearing two great silver
pillars; after them, side by side, the two Cardinals; two Noblemen with
the sword and mace. The King takes place under the cloth of state. The
two Cardinals sit under him as judges. The Queen takes place some
distance from the King. The Bishops place themselves on each side the
court, in manner of consistory; below them the Scribes. The Lords sit
next the Bishops. The rest of the Attendants stand in convenient order
about the stage.

WOLSEY.
Whilst our commission from Rome is read,
Let silence be commanded.

KING.
What’s the need?
It hath already publicly been read,
And on all sides th’ authority allowed;
You may then spare that time.

WOLSEY.
Be’t so. Proceed.

SCRIBE.
Say, “Henry King of England, come into the court.”

CRIER.
Henry King of England, come into the court.

KING.
Here.

SCRIBE.
Say, “Katherine Queen of England, come into the court.”

CRIER.
Katherine Queen of England, come into the court.

[_The Queen makes no answer, rises out of her chair, goes about the
court, comes to the King, and kneels at his feet; then speaks._]

QUEEN KATHERINE.
Sir, I desire you do me right and justice,
And to bestow your pity on me; for
I am a most poor woman and a stranger,
Born out of your dominions, having here
No judge indifferent nor no more assurance
Of equal friendship and proceeding. Alas, sir,
In what have I offended you? What cause
Hath my behaviour given to your displeasure
That thus you should proceed to put me off
And take your good grace from me? Heaven witness
I have been to you a true and humble wife,
At all times to your will conformable,
Ever in fear to kindle your dislike,
Yea, subject to your countenance, glad or sorry
As I saw it inclined. When was the hour
I ever contradicted your desire,
Or made it not mine too? Or which of your friends
Have I not strove to love, although I knew
He were mine enemy? What friend of mine
That had to him derived your anger did I
Continue in my liking? Nay, gave notice
He was from thence discharged? Sir, call to mind
That I have been your wife in this obedience
Upward of twenty years, and have been blessed
With many children by you. If, in the course
And process of this time, you can report,
And prove it too, against mine honour aught,
My bond to wedlock, or my love and duty
Against your sacred person, in God’s name,
Turn me away and let the foul’st contempt
Shut door upon me, and so give me up
To the sharp’st kind of justice. Please you, sir,
The King your father was reputed for
A prince most prudent, of an excellent
And unmatched wit and judgement. Ferdinand,
My father, King of Spain, was reckoned one
The wisest prince that there had reigned by many
A year before. It is not to be questioned
That they had gathered a wise council to them
Of every realm, that did debate this business,
Who deemed our marriage lawful. Wherefore I humbly
Beseech you, sir, to spare me till I may
Be by my friends in Spain advised, whose counsel
I will implore. If not, i’ th’ name of God,
Your pleasure be fulfilled.

WOLSEY.
You have here, lady,
And of your choice, these reverend fathers, men
Of singular integrity and learning,
Yea, the elect o’ th’ land, who are assembled
To plead your cause. It shall be therefore bootless
That longer you desire the court, as well
For your own quiet as to rectify
What is unsettled in the King.

CAMPEIUS.
His Grace
Hath spoken well and justly. Therefore, madam,
It’s fit this royal session do proceed,
And that without delay their arguments
Be now produced and heard.

QUEEN KATHERINE.
Lord Cardinal,
To you I speak.

WOLSEY.
Your pleasure, madam.

QUEEN KATHERINE.
Sir,
I am about to weep; but, thinking that
We are a queen, or long have dreamed so, certain
The daughter of a king, my drops of tears
I’ll turn to sparks of fire.

WOLSEY.
Be patient yet.

QUEEN KATHERINE.
I will, when you are humble; nay, before,
Or God will punish me. I do believe,
Induced by potent circumstances, that
You are mine enemy, and make my challenge
You shall not be my judge; for it is you
Have blown this coal betwixt my lord and me,
Which God’s dew quench! Therefore I say again,
I utterly abhor, yea, from my soul
Refuse you for my judge, whom, yet once more,
I hold my most malicious foe and think not
At all a friend to truth.

WOLSEY.
I do profess
You speak not like yourself, who ever yet
Have stood to charity and displayed th’ effects
Of disposition gentle and of wisdom
O’ertopping woman’s power. Madam, you do me wrong.
I have no spleen against you, nor injustice
For you or any. How far I have proceeded,
Or how far further shall, is warranted
By a commission from the Consistory,
Yea, the whole Consistory of Rome. You charge me
That I have “blown this coal”. I do deny it.
The King is present. If it be known to him
That I gainsay my deed, how may he wound,
And worthily, my falsehood, yea, as much
As you have done my truth. If he know
That I am free of your report, he knows
I am not of your wrong. Therefore in him
It lies to cure me, and the cure is to
Remove these thoughts from you, the which before
His Highness shall speak in, I do beseech
You, gracious madam, to unthink your speaking
And to say so no more.

QUEEN KATHERINE.
My lord, my lord,
I am a simple woman, much too weak
T’ oppose your cunning. You’re meek and humble-mouthed;
You sign your place and calling, in full seeming,
With meekness and humility; but your heart
Is crammed with arrogancy, spleen, and pride.
You have, by fortune and his Highness’ favours,
Gone slightly o’er low steps, and now are mounted
Where powers are your retainers, and your words,
Domestics to you, serve your will as ’t please
Yourself pronounce their office. I must tell you,
You tender more your person’s honour than
Your high profession spiritual; that again
I do refuse you for my judge; and here,
Before you all, appeal unto the Pope,
To bring my whole cause ’fore his Holiness,
And to be judged by him.

[_She curtsies to the King and offers to depart._]

CAMPEIUS.
The Queen is obstinate,
Stubborn to justice, apt to accuse it, and
Disdainful to be tried by’t. ’Tis not well.
She’s going away.

KING.
Call her again.

CRIER.
Katherine, Queen of England, come into the court.

GENTLEMAN USHER.
Madam, you are called back.

QUEEN KATHERINE.
What need you note it? Pray you keep your way.
When you are called, return. Now, the Lord help!
They vex me past my patience. Pray you, pass on.
I will not tarry; no, nor ever more
Upon this business my appearance make
In any of their courts.

[_Exeunt Queen and her Attendants._]

KING.
Go thy ways, Kate.
That man i’ th’ world who shall report he has
A better wife, let him in naught be trusted,
For speaking false in that. Thou art, alone—
If thy rare qualities, sweet gentleness,
Thy meekness saint-like, wife-like government,
Obeying in commanding, and thy parts
Sovereign and pious else, could speak thee out—
The queen of earthly queens. She’s noble born,
And like her true nobility she has
Carried herself towards me.

WOLSEY.
Most gracious sir,
In humblest manner I require your Highness
That it shall please you to declare, in hearing
Of all these ears—for where I am robbed and bound,
There must I be unloosed, although not there
At once and fully satisfied—whether ever I
Did broach this business to your Highness, or
Laid any scruple in your way which might
Induce you to the question on’t? or ever
Have to you, but with thanks to God for such
A royal lady, spake one the least word that might
Be to the prejudice of her present state,
Or touch of her good person?

KING.
My Lord Cardinal,
I do excuse you; yea, upon mine honour,
I free you from’t. You are not to be taught
That you have many enemies that know not
Why they are so, but, like to village curs,
Bark when their fellows do. By some of these
The Queen is put in anger. You’re excused.
But will you be more justified? You ever
Have wished the sleeping of this business, never desired
It to be stirred, but oft have hindered, oft,
The passages made toward it. On my honour,
I speak my good Lord Cardinal to this point
And thus far clear him. Now, what moved me to’t,
I will be bold with time and your attention.
Then mark th’ inducement. Thus it came; give heed to’t:
My conscience first received a tenderness,
Scruple, and prick on certain speeches uttered
By th’ Bishop of Bayonne, then French ambassador,
Who had been hither sent on the debating
A marriage ’twixt the Duke of Orleans and
Our daughter Mary. I’ th’ progress of this business,
Ere a determinate resolution, he,
I mean the Bishop, did require a respite,
Wherein he might the King his lord advertise
Whether our daughter were legitimate,
Respecting this our marriage with the dowager,
Sometimes our brother’s wife. This respite shook
The bosom of my conscience, entered me,
Yea, with a splitting power, and made to tremble
The region of my breast; which forced such way
That many mazed considerings did throng
And pressed in with this caution. First, methought
I stood not in the smile of heaven, who had
Commanded nature that my lady’s womb,
If it conceived a male child by me, should
Do no more offices of life to’t than
The grave does to th’ dead; for her male issue
Or died where they were made, or shortly after
This world had aired them. Hence I took a thought
This was a judgement on me, that my kingdom,
Well worthy the best heir o’ th’ world, should not
Be gladded in’t by me. Then follows that
I weighed the danger which my realms stood in
By this my issue’s fail, and that gave to me
Many a groaning throe. Thus hulling in
The wild sea of my conscience, I did steer
Toward this remedy whereupon we are
Now present here together. That’s to say,
I meant to rectify my conscience, which
I then did feel full sick, and yet not well,
By all the reverend fathers of the land
And doctors learned. First I began in private
With you, my Lord of Lincoln. You remember
How under my oppression I did reek
When I first moved you.

LINCOLN.
Very well, my liege.

KING.
I have spoke long. Be pleased yourself to say
How far you satisfied me.

LINCOLN.
So please your Highness,
The question did at first so stagger me,
Bearing a state of mighty moment in’t
And consequence of dread, that I committed
The daring’st counsel which I had to doubt
And did entreat your Highness to this course
Which you are running here.

KING.
I then moved you,
My Lord of Canterbury, and got your leave
To make this present summons. Unsolicited
I left no reverend person in this court,
But by particular consent proceeded
Under your hands and seals. Therefore go on,
For no dislike i’ th’ world against the person
Of the good queen, but the sharp thorny points
Of my alleged reasons, drives this forward.
Prove but our marriage lawful, by my life
And kingly dignity, we are contented
To wear our mortal state to come with her,
Katherine, our Queen, before the primest creature
That’s paragoned o’ th’ world.

CAMPEIUS.
So please your Highness,
The Queen being absent, ’tis a needful fitness
That we adjourn this court till further day.
Meanwhile must be an earnest motion
Made to the Queen to call back her appeal
She intends unto his Holiness.

KING.
[_Aside_.] I may perceive
These cardinals trifle with me. I abhor
This dilatory sloth and tricks of Rome.
My learned and well-beloved servant, Cranmer,
Prithee return. With thy approach, I know,
My comfort comes along.—Break up the court!
I say, set on.

[_Exeunt in manner as they entered._]




ACT III

SCENE I. London. The Queen’s apartments.


Enter Queen and her Women, as at work.

QUEEN KATHERINE.
Take thy lute, wench. My soul grows sad with troubles.
Sing, and disperse ’em, if thou canst. Leave working.

WOMAN
[_sings song._]

Orpheus with his lute made trees
And the mountain tops that freeze
    Bow themselves when he did sing.
To his music plants and flowers
Ever sprung, as sun and showers
    There had made a lasting spring.


Everything that heard him play,
Even the billows of the sea,
    Hung their heads and then lay by.
In sweet music is such art,
Killing care and grief of heart
    Fall asleep or, hearing, die.

Enter a Gentleman.

QUEEN KATHERINE.
How now?

GENTLEMAN.
An’t please your Grace, the two great Cardinals
Wait in the presence.

QUEEN KATHERINE.
Would they speak with me?

GENTLEMAN.
They willed me say so, madam.

QUEEN KATHERINE.
Pray their Graces
To come near.

[_Exit Gentleman._]

What can be their business
With me, a poor weak woman, fallen from favour?
I do not like their coming. Now I think on’t,
They should be good men, their affairs as righteous.
But all hoods make not monks.

Enter the two Cardinals, Wolsey and Campeius.

WOLSEY.
Peace to your Highness.

QUEEN KATHERINE.
Your Graces find me here part of housewife;
I would be all, against the worst may happen.
What are your pleasures with me, reverend lords?

WOLSEY.
May it please you, noble madam, to withdraw
Into your private chamber, we shall give you
The full cause of our coming.

QUEEN KATHERINE.
Speak it here.
There’s nothing I have done yet, o’ my conscience,
Deserves a corner. Would all other women
Could speak this with as free a soul as I do!
My lords, I care not, so much I am happy
Above a number, if my actions
Were tried by every tongue, every eye saw ’em,
Envy and base opinion set against ’em,
I know my life so even. If your business
Seek me out, and that way I am wife in,
Out with it boldly. Truth loves open dealing.

WOLSEY.
_Tanta est erga te mentis integritas, regina serenissima_—

QUEEN KATHERINE.
O, good my lord, no Latin.
I am not such a truant since my coming
As not to know the language I have lived in.
A strange tongue makes my cause more strange, suspicious.
Pray speak in English. Here are some will thank you,
If you speak truth, for their poor mistress’ sake.
Believe me, she has had much wrong. Lord Cardinal,
The willing’st sin I ever yet committed
May be absolved in English.

WOLSEY.
Noble lady,
I am sorry my integrity should breed—
And service to his Majesty and you—
So deep suspicion, where all faith was meant.
We come not by the way of accusation,
To taint that honour every good tongue blesses,
Nor to betray you any way to sorrow—
You have too much, good lady—but to know
How you stand minded in the weighty difference
Between the King and you, and to deliver,
Like free and honest men, our just opinions
And comforts to your cause.

CAMPEIUS.
Most honoured madam,
My Lord of York, out of his noble nature,
Zeal, and obedience he still bore your Grace,
Forgetting, like a good man, your late censure
Both of his truth and him—which was too far—
Offers, as I do, in a sign of peace,
His service and his counsel.

QUEEN KATHERINE.
[_Aside_.] To betray me.
My lords, I thank you both for your good wills.
Ye speak like honest men; pray God ye prove so.
But how to make ye suddenly an answer
In such a point of weight, so near mine honour—
More near my life, I fear—with my weak wit,
And to such men of gravity and learning,
In truth I know not. I was set at work
Among my maids, full little, God knows, looking
Either for such men or such business.
For her sake that I have been—for I feel
The last fit of my greatness—good your Graces,
Let me have time and counsel for my cause.
Alas, I am a woman friendless, hopeless.

WOLSEY.
Madam, you wrong the King’s love with these fears;
Your hopes and friends are infinite.

QUEEN KATHERINE.
In England
But little for my profit. Can you think, lords,
That any Englishman dare give me counsel?
Or be a known friend, ’gainst his Highness’ pleasure,
Though he be grown so desperate to be honest,
And live a subject? Nay, forsooth, my friends,
They that much weigh out my afflictions,
They that my trust must grow to, live not here.
They are, as all my other comforts, far hence
In mine own country, lords.

CAMPEIUS.
I would your Grace
Would leave your griefs and take my counsel.

QUEEN KATHERINE.
How, sir?

CAMPEIUS.
Put your main cause into the King’s protection.
He’s loving and most gracious. ’Twill be much
Both for your honour better and your cause,
For if the trial of the law o’ertake ye,
You’ll part away disgraced.

WOLSEY.
He tells you rightly.

QUEEN KATHERINE.
Ye tell me what ye wish for both: my ruin.
Is this your Christian counsel? Out upon ye!
Heaven is above all yet; there sits a judge
That no king can corrupt.

CAMPEIUS.
Your rage mistakes us.

QUEEN KATHERINE.
The more shame for ye! Holy men I thought ye,
Upon my soul, two reverend cardinal virtues;
But cardinal sins and hollow hearts I fear ye.
Mend ’em, for shame, my lords. Is this your comfort,
The cordial that ye bring a wretched lady,
A woman lost among ye, laughed at, scorned?
I will not wish ye half my miseries;
I have more charity. But say I warned ye.
Take heed, for heaven’s sake, take heed, lest at once
The burden of my sorrows fall upon ye.

WOLSEY.
Madam, this is a mere distraction.
You turn the good we offer into envy.

QUEEN KATHERINE.
Ye turn me into nothing. Woe upon ye
And all such false professors! Would you have me—
If you have any justice, any pity,
If ye be anything but churchmen’s habits—
Put my sick cause into his hands that hates me?
Alas, ’has banished me his bed already,
His love, too, long ago. I am old, my lords,
And all the fellowship I hold now with him
Is only my obedience. What can happen
To me above this wretchedness? All your studies
Make me a curse like this.

CAMPEIUS.
Your fears are worse.

QUEEN KATHERINE.
Have I lived thus long—let me speak myself,
Since virtue finds no friends—a wife, a true one—
A woman, I dare say without vainglory,
Never yet branded with suspicion—
Have I with all my full affections
Still met the King, loved him next heav’n, obeyed him,
Been, out of fondness, superstitious to him,
Almost forgot my prayers to content him,
And am I thus rewarded? ’Tis not well, lords.
Bring me a constant woman to her husband,
One that ne’er dreamed a joy beyond his pleasure,
And to that woman, when she has done most,
Yet will I add an honour: a great patience.

WOLSEY.
Madam, you wander from the good we aim at.

QUEEN KATHERINE.
My lord, I dare not make myself so guilty
To give up willingly that noble title
Your master wed me to. Nothing but death
Shall e’er divorce my dignities.

WOLSEY.
Pray hear me.

QUEEN KATHERINE.
Would I had never trod this English earth
Or felt the flatteries that grow upon it!
Ye have angels’ faces, but heaven knows your hearts.
What will become of me now, wretched lady?
I am the most unhappy woman living.
[_To her Women_.] Alas, poor wenches, where are now your fortunes?
Shipwrecked upon a kingdom where no pity,
No friends, no hope, no kindred weep for me,
Almost no grave allowed me, like the lily
That once was mistress of the field and flourished,
I’ll hang my head and perish.

WOLSEY.
If your Grace
Could but be brought to know our ends are honest,
You’d feel more comfort. Why should we, good lady,
Upon what cause, wrong you? Alas, our places,
The way of our profession, is against it.
We are to cure such sorrows, not to sow ’em.
For goodness’ sake, consider what you do,
How you may hurt yourself, ay, utterly
Grow from the King’s acquaintance, by this carriage.
The hearts of princes kiss obedience,
So much they love it, but to stubborn spirits
They swell and grow as terrible as storms.
I know you have a gentle, noble temper,
A soul as even as a calm. Pray think us
Those we profess: peacemakers, friends, and servants.

CAMPEIUS.
Madam, you’ll find it so. You wrong your virtues
With these weak women’s fears. A noble spirit,
As yours was put into you, ever casts
Such doubts, as false coin, from it. The King loves you;
Beware you lose it not. For us, if you please
To trust us in your business, we are ready
To use our utmost studies in your service.

QUEEN KATHERINE.
Do what ye will, my lords, and pray forgive me
If I have used myself unmannerly.
You know I am a woman, lacking wit
To make a seemly answer to such persons.
Pray do my service to his Majesty.
He has my heart yet, and shall have my prayers
While I shall have my life. Come, reverend fathers,
Bestow your counsels on me. She now begs
That little thought, when she set footing here,
She should have bought her dignities so dear.

[_Exeunt._]

SCENE II. Ante-chamber to the King’s apartment.

Enter the Duke of Norfolk, Duke of Suffolk, Lord Surrey and Lord
Chamberlain.

NORFOLK.
If you will now unite in your complaints
And force them with a constancy, the Cardinal
Cannot stand under them. If you omit
The offer of this time, I cannot promise
But that you shall sustain more new disgraces
With these you bear already.

SURREY.
I am joyful
To meet the least occasion that may give me
Remembrance of my father-in-law the Duke,
To be revenged on him.

SUFFOLK.
Which of the peers
Have uncontemned gone by him, or at least
Strangely neglected? When did he regard
The stamp of nobleness in any person
Out of himself?

CHAMBERLAIN.
My lords, you speak your pleasures.
What he deserves of you and me I know;
What we can do to him—though now the time
Gives way to us—I much fear. If you cannot
Bar his access to th’ King, never attempt
Anything on him, for he hath a witchcraft
Over the King in ’s tongue.

NORFOLK.
O, fear him not.
His spell in that is out. The King hath found
Matter against him that for ever mars
The honey of his language. No, he’s settled,
Not to come off, in his displeasure.

SURREY.
Sir,
I should be glad to hear such news as this
Once every hour.

NORFOLK.
Believe it, this is true.
In the divorce his contrary proceedings
Are all unfolded, wherein he appears
As I would wish mine enemy.

SURREY.
How came
His practices to light?

SUFFOLK.
Most strangely.

SURREY.
O, how, how?

SUFFOLK.
The Cardinal’s letters to the Pope miscarried,
And came to th’ eye o’ the King, wherein was read
How that the Cardinal did entreat his Holiness
To stay the judgement o’ th’ divorce; for if
It did take place, “I do” quoth he “perceive
My king is tangled in affection to
A creature of the Queen’s, Lady Anne Bullen.”

SURREY.
Has the King this?

SUFFOLK.
Believe it.

SURREY.
Will this work?

CHAMBERLAIN.
The King in this perceives him how he coasts
And hedges his own way. But in this point
All his tricks founder, and he brings his physic
After his patient’s death. The King already
Hath married the fair lady.

SURREY.
Would he had!

SUFFOLK.
May you be happy in your wish, my lord,
For I profess you have it.

SURREY.
Now, all my joy
Trace the conjunction!

SUFFOLK.
My amen to’t!

NORFOLK.
All men’s.

SUFFOLK.
There’s order given for her coronation.
Marry, this is yet but young, and may be left
To some ears unrecounted. But, my lords,
She is a gallant creature, and complete
In mind and feature. I persuade me, from her
Will fall some blessing to this land which shall
In it be memorized.

SURREY.
But will the King
Digest this letter of the Cardinal’s?
The Lord forbid!

NORFOLK.
Marry, amen!

SUFFOLK.
No, no.
There be more wasps that buzz about his nose
Will make this sting the sooner. Cardinal Campeius
Is stolen away to Rome; hath ta’en no leave;
Has left the cause o’ th’ King unhandled, and
Is posted, as the agent of our Cardinal,
To second all his plot. I do assure you
The King cried “Ha!” at this.

CHAMBERLAIN.
Now, God incense him,
And let him cry “Ha!” louder.

NORFOLK.
But, my lord,
When returns Cranmer?

SUFFOLK.
He is returned in his opinions, which
Have satisfied the King for his divorce,
Together with all famous colleges
Almost in Christendom. Shortly, I believe,
His second marriage shall be published, and
Her coronation. Katherine no more
Shall be called Queen, but Princess Dowager
And widow to Prince Arthur.

NORFOLK.
This same Cranmer’s
A worthy fellow, and hath ta’en much pain
In the King’s business.

SUFFOLK.
He has, and we shall see him
For it an archbishop.

NORFOLK.
So I hear.

SUFFOLK.
’Tis so.

Enter Wolsey and Cromwell.

The Cardinal!

NORFOLK.
Observe, observe; he’s moody.

WOLSEY.
The packet, Cromwell,
Gave’t you the King?

CROMWELL.
To his own hand, in ’s bedchamber.

WOLSEY.
Looked he o’ th’ inside of the paper?

CROMWELL.
Presently
He did unseal them, and the first he viewed,
He did it with a serious mind; a heed
Was in his countenance. You he bade
Attend him here this morning.

WOLSEY.
Is he ready
To come abroad?

CROMWELL.
I think by this he is.

WOLSEY.
Leave me a while.

[_Exit Cromwell._]

[_Aside_.] It shall be to the Duchess of Alençon,
The French king’s sister; he shall marry her.
Anne Bullen? No; I’ll no Anne Bullens for him.
There’s more in’t than fair visage. Bullen?
No, we’ll no Bullens. Speedily I wish
To hear from Rome. The Marchioness of Pembroke!

NORFOLK.
He’s discontented.

SUFFOLK.
Maybe he hears the King
Does whet his anger to him.

SURREY.
Sharp enough,
Lord, for thy justice!

WOLSEY.
[_Aside_.] The late queen’s gentlewoman, a knight’s daughter,
To be her mistress’ mistress? The Queen’s Queen?
This candle burns not clear. ’Tis I must snuff it;
Then out it goes. What though I know her virtuous
And well deserving? Yet I know her for
A spleeny Lutheran, and not wholesome to
Our cause, that she should lie i’ th’ bosom of
Our hard-ruled King. Again, there is sprung up
An heretic, an arch-one, Cranmer, one
Hath crawled into the favour of the King
And is his oracle.

NORFOLK.
He is vexed at something.

Enter King, reading a schedule, and Lovell.

SURREY.
I would ’twere something that would fret the string,
The master-cord on ’s heart.

SUFFOLK.
The King, the King!

KING.
What piles of wealth hath he accumulated
To his own portion! And what expense by th’ hour
Seems to flow from him! How, i’ th’ name of thrift
Does he rake this together? Now, my lords,
Saw you the Cardinal?

NORFOLK.
My lord, we have
Stood here observing him. Some strange commotion
Is in his brain. He bites his lip, and starts,
Stops on a sudden, looks upon the ground,
Then lays his finger on his temple; straight
Springs out into fast gait; then stops again,
Strikes his breast hard, and anon he casts
His eye against the moon. In most strange postures
We have seen him set himself.

KING.
It may well be
There is a mutiny in ’s mind. This morning
Papers of state he sent me to peruse,
As I required; and wot you what I found
There—on my conscience, put unwittingly?
Forsooth, an inventory, thus importing
The several parcels of his plate, his treasure,
Rich stuffs and ornaments of household, which
I find at such proud rate that it outspeaks
Possession of a subject.

NORFOLK.
It’s heaven’s will!
Some spirit put this paper in the packet
To bless your eye withal.

KING.
If we did think
His contemplation were above the earth
And fixed on spiritual object, he should still
Dwell in his musings, but I am afraid
His thinkings are below the moon, not worth
His serious considering.

[_King takes his seat; whispers Lovell, who goes to the Cardinal._]

WOLSEY.
Heaven forgive me!
Ever God bless your Highness.

KING.
Good my lord,
You are full of heavenly stuff, and bear the inventory
Of your best graces in your mind, the which
You were now running o’er. You have scarce time
To steal from spiritual leisure a brief span
To keep your earthly audit. Sure, in that
I deem you an ill husband, and am glad
To have you therein my companion.

WOLSEY.
Sir,
For holy offices I have a time; a time
To think upon the part of business which
I bear i’ th’ state; and Nature does require
Her times of preservation, which perforce
I, her frail son, amongst my brethren mortal,
Must give my tendance to.

KING.
You have said well.

WOLSEY.
And ever may your Highness yoke together,
As I will lend you cause, my doing well
With my well saying.

KING.
’Tis well said again,
And ’tis a kind of good deed to say well.
And yet words are no deeds. My father loved you;
He said he did, and with his deed did crown
His word upon you. Since I had my office,
I have kept you next my heart, have not alone
Employed you where high profits might come home,
But pared my present havings to bestow
My bounties upon you.

WOLSEY.
[_Aside_.] What should this mean?

SURREY.
[_Aside_.] The Lord increase this business!

KING.
Have I not made you
The prime man of the state? I pray you tell me,
If what I now pronounce you have found true,
And, if you may confess it, say withal
If you are bound to us or no. What say you?

WOLSEY.
My sovereign, I confess your royal graces,
Showered on me daily, have been more than could
My studied purposes requite, which went
Beyond all man’s endeavours. My endeavours
Have ever come too short of my desires,
Yet filed with my abilities. Mine own ends
Have been mine so that evermore they pointed
To th’ good of your most sacred person and
The profit of the state. For your great graces
Heaped upon me, poor undeserver, I
Can nothing render but allegiant thanks,
My prayers to heaven for you, my loyalty,
Which ever has and ever shall be growing,
Till death, that winter, kill it.

KING.
Fairly answered.
A loyal and obedient subject is
Therein illustrated. The honour of it
Does pay the act of it, as i’ th’ contrary,
The foulness is the punishment. I presume
That, as my hand has opened bounty to you,
My heart dropped love, my power rained honour, more
On you than any, so your hand and heart,
Your brain, and every function of your power,
Should, notwithstanding that your bond of duty,
As ’twere in love’s particular, be more
To me, your friend, than any.

WOLSEY.
I do profess
That for your Highness’ good I ever laboured
More than mine own, that am, have, and will be.
Though all the world should crack their duty to you
And throw it from their soul, though perils did
Abound as thick as thought could make ’em, and
Appear in forms more horrid—yet my duty,
As doth a rock against the chiding flood,
Should the approach of this wild river break,
And stand unshaken yours.

KING.
’Tis nobly spoken.
Take notice, lords: he has a loyal breast,
For you have seen him open’t.

[_Giving him papers._]

Read o’er this,
And after, this; and then to breakfast with
What appetite you have.

[_Exit King, frowning upon the Cardinal; the nobles throng after him,
smiling and whispering._]

WOLSEY.
What should this mean?
What sudden anger’s this? How have I reaped it?
He parted frowning from me, as if ruin
Leaped from his eyes. So looks the chafed lion
Upon the daring huntsman that has galled him,
Then makes him nothing. I must read this paper—
I fear, the story of his anger. ’Tis so.
This paper has undone me. ’Tis th’ account
Of all that world of wealth I have drawn together
For mine own ends—indeed, to gain the popedom
And fee my friends in Rome. O negligence,
Fit for a fool to fall by! What cross devil
Made me put this main secret in the packet
I sent the King? Is there no way to cure this?
No new device to beat this from his brains?
I know ’twill stir him strongly; yet I know
A way, if it take right, in spite of fortune,
Will bring me off again. What’s this? “To th’ Pope”?
The letter, as I live, with all the business
I writ to ’s Holiness. Nay then, farewell!
I have touched the highest point of all my greatness,
And from that full meridian of my glory
I haste now to my setting. I shall fall
Like a bright exhalation in the evening,
And no man see me more.

Enter to Wolsey, the Dukes of Norfolk and Suffolk, the Earl of Surrey,
and the Lord Chamberlain.

NORFOLK.
Hear the King’s pleasure, Cardinal, who commands you
To render up the great seal presently
Into our hands, and to confine yourself
To Asher House, my Lord of Winchester’s,
Till you hear further from his Highness.

WOLSEY.
Stay.
Where’s your commission, lords? Words cannot carry
Authority so weighty.

SUFFOLK.
Who dares cross ’em,
Bearing the King’s will from his mouth expressly?

WOLSEY.
Till I find more than will or words to do it—
I mean your malice—know, officious lords,
I dare and must deny it. Now I feel
Of what coarse metal ye are moulded, envy!
How eagerly ye follow my disgraces,
As if it fed ye, and how sleek and wanton
Ye appear in everything may bring my ruin!
Follow your envious courses, men of malice;
You have Christian warrant for ’em, and no doubt
In time will find their fit rewards. That seal
You ask with such a violence, the King,
Mine and your master, with his own hand gave me;
Bade me enjoy it, with the place and honours,
During my life; and, to confirm his goodness,
Tied it by letters-patents. Now, who’ll take it?

SURREY.
The King that gave it.

WOLSEY.
It must be himself, then.

SURREY.
Thou art a proud traitor, priest.

WOLSEY.
Proud lord, thou liest.
Within these forty hours Surrey durst better
Have burnt that tongue than said so.

SURREY.
Thy ambition,
Thou scarlet sin, robbed this bewailing land
Of noble Buckingham, my father-in-law.
The heads of all thy brother cardinals,
With thee and all thy best parts bound together,
Weighed not a hair of his. Plague of your policy!
You sent me Deputy for Ireland,
Far from his succour, from the King, from all
That might have mercy on the fault thou gav’st him,
Whilst your great goodness, out of holy pity,
Absolved him with an axe.

WOLSEY.
This, and all else
This talking lord can lay upon my credit,
I answer is most false. The Duke by law
Found his deserts. How innocent I was
From any private malice in his end,
His noble jury and foul cause can witness.
If I loved many words, lord, I should tell you
You have as little honesty as honour,
That in the way of loyalty and truth
Toward the King, my ever royal master,
Dare mate a sounder man than Surrey can be,
And all that love his follies.

SURREY.
By my soul,
Your long coat, priest, protects you; thou shouldst feel
My sword i’ th’ lifeblood of thee else. My lords,
Can ye endure to hear this arrogance?
And from this fellow? If we live thus tamely,
To be thus jaded by a piece of scarlet,
Farewell, nobility. Let his Grace go forward
And dare us with his cap, like larks.

WOLSEY.
All goodness
Is poison to thy stomach.

SURREY.
Yes, that goodness
Of gleaning all the land’s wealth into one,
Into your own hands, Cardinal, by extortion;
The goodness of your intercepted packets
You writ to the Pope against the King. Your goodness,
Since you provoke me, shall be most notorious.
My Lord of Norfolk, as you are truly noble,
As you respect the common good, the state
Of our despised nobility, our issues,
Who, if he live, will scarce be gentlemen,
Produce the grand sum of his sins, the articles
Collected from his life. I’ll startle you
Worse than the sacring bell when the brown wench
Lay kissing in your arms, Lord Cardinal.

WOLSEY.
How much, methinks, I could despise this man,
But that I am bound in charity against it!

NORFOLK.
Those articles, my lord, are in the King’s hand;
But thus much, they are foul ones.

WOLSEY.
So much fairer
And spotless shall mine innocence arise
When the King knows my truth.

SURREY.
This cannot save you.
I thank my memory I yet remember
Some of these articles, and out they shall.
Now, if you can blush and cry “Guilty,” Cardinal,
You’ll show a little honesty.

WOLSEY.
Speak on, sir;
I dare your worst objections. If I blush,
It is to see a nobleman want manners.

SURREY.
I had rather want those than my head. Have at you!
First, that without the King’s assent or knowledge,
You wrought to be a legate, by which power
You maimed the jurisdiction of all bishops.

NORFOLK.
Then, that in all you writ to Rome, or else
To foreign princes, “_ego et rex meus_”
Was still inscribed, in which you brought the King
To be your servant.

SUFFOLK.
Then, that without the knowledge
Either of King or Council, when you went
Ambassador to the Emperor, you made bold
To carry into Flanders the great seal.

SURREY.
Item, you sent a large commission
To Gregory de Cassado, to conclude,
Without the King’s will or the state’s allowance,
A league between his Highness and Ferrara.

SUFFOLK.
That out of mere ambition you have caused
Your holy hat to be stamped on the King’s coin.

SURREY.
Then, that you have sent innumerable substance—
By what means got, I leave to your own conscience—
To furnish Rome and to prepare the ways
You have for dignities, to the mere undoing
Of all the kingdom. Many more there are,
Which, since they are of you, and odious,
I will not taint my mouth with.

CHAMBERLAIN.
O my lord,
Press not a falling man too far! ’Tis virtue.
His faults lie open to the laws; let them,
Not you, correct him. My heart weeps to see him
So little of his great self.

SURREY.
I forgive him.

SUFFOLK.
Lord Cardinal, the King’s further pleasure is,
Because all those things you have done of late
By your power legative within this kingdom
Fall into th’ compass of a _praemunire_,
That therefore such a writ be sued against you
To forfeit all your goods, lands, tenements,
Chattels, and whatsoever, and to be
Out of the King’s protection. This is my charge.

NORFOLK.
And so we’ll leave you to your meditations
How to live better. For your stubborn answer
About the giving back the great seal to us,
The King shall know it and, no doubt, shall thank you.
So fare you well, my little good Lord Cardinal.

[_Exeunt all but Wolsey._]

WOLSEY.
So farewell to the little good you bear me.
Farewell? A long farewell to all my greatness!
This is the state of man: today he puts forth
The tender leaves of hopes; tomorrow blossoms,
And bears his blushing honours thick upon him;
The third day comes a frost, a killing frost,
And when he thinks, good easy man, full surely
His greatness is a-ripening, nips his root,
And then he falls, as I do. I have ventured,
Like little wanton boys that swim on bladders,
This many summers in a sea of glory,
But far beyond my depth. My high-blown pride
At length broke under me and now has left me,
Weary and old with service, to the mercy
Of a rude stream that must for ever hide me.
Vain pomp and glory of this world, I hate ye!
I feel my heart new opened. O, how wretched
Is that poor man that hangs on princes’ favours!
There is betwixt that smile we would aspire to,
That sweet aspect of princes, and their ruin,
More pangs and fears than wars or women have;
And when he falls, he falls like Lucifer,
Never to hope again.

Enter Cromwell, standing amazed.

Why, how now, Cromwell?

CROMWELL.
I have no power to speak, sir.

WOLSEY.
What, amazed
At my misfortunes? Can thy spirit wonder
A great man should decline? Nay, an you weep,
I am fallen indeed.

CROMWELL.
How does your Grace?

WOLSEY.
Why, well.
Never so truly happy, my good Cromwell.
I know myself now, and I feel within me
A peace above all earthly dignities,
A still and quiet conscience. The King has cured me,
I humbly thank his Grace, and from these shoulders,
These ruined pillars, out of pity, taken
A load would sink a navy: too much honour.
O, ’tis a burden, Cromwell, ’tis a burden
Too heavy for a man that hopes for heaven.

CROMWELL.
I am glad your Grace has made that right use of it.

WOLSEY.
I hope I have. I am able now, methinks,
Out of a fortitude of soul I feel,
To endure more miseries and greater far
Than my weak-hearted enemies dare offer.
What news abroad?

CROMWELL.
The heaviest and the worst
Is your displeasure with the King.

WOLSEY.
God bless him.

CROMWELL.
The next is that Sir Thomas More is chosen
Lord Chancellor in your place.

WOLSEY.
That’s somewhat sudden.
But he’s a learned man. May he continue
Long in his Highness’ favour, and do justice
For truth’s sake and his conscience, that his bones,
When he has run his course and sleeps in blessings,
May have a tomb of orphans’ tears wept on him.
What more?

CROMWELL.
That Cranmer is returned with welcome,
Installed Lord Archbishop of Canterbury.

WOLSEY.
That’s news indeed.

CROMWELL.
Last, that the Lady Anne,
Whom the King hath in secrecy long married,
This day was viewed in open as his Queen,
Going to chapel, and the voice is now
Only about her coronation.

WOLSEY.
There was the weight that pulled me down.
O Cromwell,
The King has gone beyond me. All my glories
In that one woman I have lost for ever.
No sun shall ever usher forth mine honours,
Or gild again the noble troops that waited
Upon my smiles. Go, get thee from me, Cromwell.
I am a poor fallen man, unworthy now
To be thy lord and master. Seek the King;
That sun, I pray, may never set! I have told him
What and how true thou art. He will advance thee;
Some little memory of me will stir him—
I know his noble nature—not to let
Thy hopeful service perish too. Good Cromwell,
Neglect him not; make use now, and provide
For thine own future safety.

CROMWELL.
O my lord,
Must I then leave you? Must I needs forgo
So good, so noble, and so true a master?
Bear witness, all that have not hearts of iron,
With what a sorrow Cromwell leaves his lord.
The King shall have my service, but my prayers
For ever and for ever shall be yours.

WOLSEY.
Cromwell, I did not think to shed a tear
In all my miseries, but thou hast forced me,
Out of thy honest truth, to play the woman.
Let’s dry our eyes, and thus far hear me, Cromwell,
And when I am forgotten, as I shall be,
And sleep in dull cold marble, where no mention
Of me more must be heard of, say, I taught thee;
Say Wolsey, that once trod the ways of glory
And sounded all the depths and shoals of honour,
Found thee a way, out of his wreck, to rise in,
A sure and safe one, though thy master missed it.
Mark but my fall and that that ruined me.
Cromwell, I charge thee, fling away ambition!
By that sin fell the angels; how can man, then,
The image of his maker, hope to win by it?
Love thyself last; cherish those hearts that hate thee.
Corruption wins not more than honesty.
Still in thy right hand carry gentle peace
To silence envious tongues. Be just, and fear not.
Let all the ends thou aim’st at be thy country’s,
Thy God’s, and truth’s. Then if thou fall’st, O Cromwell,
Thou fall’st a blessed martyr!
Serve the King. And, prithee, lead me in.
There take an inventory of all I have.
To the last penny; ’tis the King’s. My robe
And my integrity to heaven is all
I dare now call mine own. O Cromwell, Cromwell,
Had I but served my God with half the zeal
I served my king, He would not in mine age
Have left me naked to mine enemies.

CROMWELL.
Good sir, have patience.

WOLSEY.
So I have. Farewell,
The hopes of court! My hopes in heaven do dwell.

[_Exeunt._]




ACT IV

SCENE I. A street in Westminster.


Enter two Gentlemen, meeting one another.

FIRST GENTLEMAN.
You’re well met once again.

SECOND GENTLEMAN.
So are you.

FIRST GENTLEMAN.
You come to take your stand here and behold
The Lady Anne pass from her coronation?

SECOND GENTLEMAN.
’Tis all my business. At our last encounter,
The Duke of Buckingham came from his trial.

FIRST GENTLEMAN.
’Tis very true. But that time offered sorrow,
This, general joy.

SECOND GENTLEMAN.
’Tis well. The citizens,
I am sure, have shown at full their royal minds,
As, let ’em have their rights, they are ever forward
In celebration of this day with shows,
Pageants, and sights of honour.

FIRST GENTLEMAN.
Never greater,
Nor, I’ll assure you, better taken, sir.

SECOND GENTLEMAN.
May I be bold to ask what that contains,
That paper in your hand?

FIRST GENTLEMAN.
Yes, ’tis the list
Of those that claim their offices this day
By custom of the coronation.
The Duke of Suffolk is the first, and claims
To be High Steward; next, the Duke of Norfolk,
He to be Earl Marshal. You may read the rest.

SECOND GENTLEMAN.
I thank you, sir. Had I not known those customs,
I should have been beholding to your paper.
But I beseech you, what’s become of Katherine,
The Princess Dowager? How goes her business?

FIRST GENTLEMAN.
That I can tell you too. The Archbishop
Of Canterbury, accompanied with other
Learned and reverend fathers of his order,
Held a late court at Dunstable, six miles off
From Ampthill where the Princess lay; to which
She was often cited by them, but appeared not;
And, to be short, for not appearance and
The King’s late scruple, by the main assent
Of all these learned men she was divorced,
And the late marriage made of none effect;
Since which she was removed to Kimbolton,
Where she remains now sick.

SECOND GENTLEMAN.
Alas, good lady!

[_Trumpets._]

The trumpets sound. Stand close. The Queen is coming.

_The order of the coronation_.


_1. A lively flourish of trumpets.
2. Then, two Judges.
3. Lord Chancellor, with purse and mace before him.
4. Choristers, singing. Music.
5. Mayor of London, bearing the mace. Then Garter, in his coat of arms,
and on his head he wore a gilt copper crown.
6. Marquess Dorset, bearing a sceptre of gold, on his head a
demi-coronal of gold. With him, the Earl of Surrey, bearing the rod of
silver with the dove, crowned with an earl’s coronet. Collars of S’s.
7. Duke of Suffolk, in his robe of estate, his coronet on his head,
bearing a long white wand, as High Steward. With him, the Duke of
Norfolk, with the rod of marshalship, a coronet on his head. Collars of
S’s.
8. A canopy, borne by four of the Cinque Ports; under it, the Queen in
her robe, in her hair, richly adorned with pearl, crowned. On each side
her, the Bishops of London and Winchester.
9. The old Duchess of Norfolk, in a coronal of gold wrought with
flowers, bearing the Queen’s train.
10. Certain Ladies or Countesses, with plain circlets of gold without
flowers._

[_Exeunt, first passing over the stage in order and state, and then a
great flourish of trumpets._]

SECOND GENTLEMAN.
A royal train, believe me. These I know.
Who’s that that bears the sceptre?

FIRST GENTLEMAN.
Marquess Dorset,
And that the Earl of Surrey with the rod.

SECOND GENTLEMAN.
A bold brave gentleman. That should be
The Duke of Suffolk.

FIRST GENTLEMAN.
’Tis the same: High Steward.

SECOND GENTLEMAN.
And that my Lord of Norfolk?

FIRST GENTLEMAN.
Yes.

SECOND GENTLEMAN.
[_Sees the Queen_.] Heaven bless thee!
Thou hast the sweetest face I ever looked on.
Sir, as I have a soul, she is an angel.
Our King has all the Indies in his arms,
And more, and richer, when he strains that lady.
I cannot blame his conscience.

FIRST GENTLEMAN.
They that bear
The cloth of honour over her are four barons
Of the Cinque Ports.

SECOND GENTLEMAN.
Those men are happy, and so are all are near her.
I take it she that carries up the train
Is that old noble lady, Duchess of Norfolk.

FIRST GENTLEMAN.
It is, and all the rest are countesses.

SECOND GENTLEMAN.
Their coronets say so. These are stars indeed.

FIRST GENTLEMAN.
And sometimes falling ones.

SECOND GENTLEMAN.
No more of that.

[_Exit the last of the procession._]

Enter a third Gentleman.

God save you, sir. Where have you been broiling?

THIRD GENTLEMAN.
Among the crowds i’ th’ Abbey, where a finger
Could not be wedged in more. I am stifled
With the mere rankness of their joy.

SECOND GENTLEMAN.
You saw
The ceremony?

THIRD GENTLEMAN.
That I did.

FIRST GENTLEMAN.
How was it?

THIRD GENTLEMAN.
Well worth the seeing.

SECOND GENTLEMAN.
Good sir, speak it to us.

THIRD GENTLEMAN.
As well as I am able. The rich stream
Of lords and ladies, having brought the Queen
To a prepared place in the choir, fell off
A distance from her, while her Grace sat down
To rest a while, some half an hour or so,
In a rich chair of state, opposing freely
The beauty of her person to the people.
Believe me, sir, she is the goodliest woman
That ever lay by man, which when the people
Had the full view of, such a noise arose
As the shrouds make at sea in a stiff tempest,
As loud and to as many tunes. Hats, cloaks,
Doublets, I think, flew up, and had their faces
Been loose, this day they had been lost. Such joy
I never saw before. Great-bellied women
That had not half a week to go, like rams
In the old time of war, would shake the press
And make ’em reel before ’em. No man living
Could say “This is my wife” there, all were woven
So strangely in one piece.

SECOND GENTLEMAN.
But what followed?

THIRD GENTLEMAN.
At length her Grace rose, and with modest paces
Came to the altar, where she kneeled and saintlike
Cast her fair eyes to heaven and prayed devoutly;
Then rose again and bowed her to the people,
When by the Archbishop of Canterbury
She had all the royal makings of a queen,
As holy oil, Edward Confessor’s crown,
The rod, and bird of peace, and all such emblems
Laid nobly on her; which performed, the choir,
With all the choicest music of the kingdom,
Together sung _Te Deum_. So she parted,
And with the same full state paced back again
To York Place, where the feast is held.

FIRST GENTLEMAN.
Sir,
You must no more call it “York Place”, that’s past;
For since the Cardinal fell, that title’s lost.
’Tis now the King’s, and called “Whitehall”.

THIRD GENTLEMAN.
I know it,
But ’tis so lately altered that the old name
Is fresh about me.

SECOND GENTLEMAN.
What two reverend bishops
Were those that went on each side of the Queen?

THIRD GENTLEMAN.
Stokesley and Gardiner, the one of Winchester,
Newly preferred from the King’s secretary;
The other, London.

SECOND GENTLEMAN.
He of Winchester
Is held no great good lover of the Archbishop’s,
The virtuous Cranmer.

THIRD GENTLEMAN.
All the land knows that.
However, yet there is no great breach. When it comes,
Cranmer will find a friend will not shrink from him.

SECOND GENTLEMAN.
Who may that be, I pray you?

THIRD GENTLEMAN.
Thomas Cromwell,
A man in much esteem with th’ King, and truly
A worthy friend. The King has made him
Master o’ th’ Jewel House,
And one already of the Privy Council.

SECOND GENTLEMAN.
He will deserve more.

THIRD GENTLEMAN.
Yes, without all doubt.
Come, gentlemen, ye shall go my way,
Which is to th’ court, and there ye shall be my guests,
Something I can command. As I walk thither,
I’ll tell ye more.

BOTH.
You may command us, sir.

[_Exeunt._]

SCENE II. Kimbolton.

Enter Katherine Dowager, sick, led between Griffith, her gentleman
usher, and Patience, her woman.

GRIFFITH.
How does your Grace?

QUEEN KATHERINE.
O Griffith, sick to death.
My legs like loaden branches bow to th’ earth,
Willing to leave their burden. Reach a chair.

[_She sits._]

So. Now, methinks, I feel a little ease.
Didst thou not tell me, Griffith, as thou ledst me,
That the great child of honour, Cardinal Wolsey,
Was dead?

GRIFFITH.
Yes, madam, but I think your Grace,
Out of the pain you suffered, gave no ear to’t.

QUEEN KATHERINE.
Prithee, good Griffith, tell me how he died.
If well, he stepped before me happily
For my example.

GRIFFITH.
Well, the voice goes, madam.
For after the stout Earl Northumberland
Arrested him at York and brought him forward,
As a man sorely tainted, to his answer,
He fell sick suddenly and grew so ill
He could not sit his mule.

QUEEN KATHERINE.
Alas, poor man!

GRIFFITH.
At last, with easy roads, he came to Leicester,
Lodged in the abbey, where the reverend abbot,
With all his covent, honourably received him;
To whom he gave these words: “O father abbot,
An old man, broken with the storms of state,
Is come to lay his weary bones among ye.
Give him a little earth for charity.”
So went to bed, where eagerly his sickness
Pursued him still; and three nights after this,
About the hour of eight, which he himself
Foretold should be his last, full of repentance,
Continual meditations, tears, and sorrows,
He gave his honours to the world again,
His blessed part to heaven, and slept in peace.

QUEEN KATHERINE.
So may he rest. His faults lie gently on him!
Yet thus far, Griffith, give me leave to speak him,
And yet with charity. He was a man
Of an unbounded stomach, ever ranking
Himself with princes; one that by suggestion
Tied all the kingdom. Simony was fair-play.
His own opinion was his law. I’ th’ presence
He would say untruths, and be ever double
Both in his words and meaning. He was never,
But where he meant to ruin, pitiful.
His promises were, as he then was, mighty;
But his performance, as he is now, nothing.
Of his own body he was ill, and gave
The clergy ill example.

GRIFFITH.
Noble madam,
Men’s evil manners live in brass; their virtues
We write in water. May it please your Highness
To hear me speak his good now?

QUEEN KATHERINE.
Yes, good Griffith;
I were malicious else.

GRIFFITH.
This Cardinal,
Though from an humble stock, undoubtedly
Was fashioned to much honour. From his cradle
He was a scholar, and a ripe and good one,
Exceeding wise, fair-spoken, and persuading;
Lofty and sour to them that loved him not,
But to those men that sought him, sweet as summer.
And though he were unsatisfied in getting,
Which was a sin, yet in bestowing, madam,
He was most princely. Ever witness for him
Those twins of learning that he raised in you,
Ipswich and Oxford, one of which fell with him,
Unwilling to outlive the good that did it;
The other, though unfinished, yet so famous,
So excellent in art, and still so rising,
That Christendom shall ever speak his virtue.
His overthrow heaped happiness upon him,
For then, and not till then, he felt himself,
And found the blessedness of being little.
And, to add greater honours to his age
Than man could give him, he died fearing God.

QUEEN KATHERINE.
After my death I wish no other herald,
No other speaker of my living actions,
To keep mine honour from corruption
But such an honest chronicler as Griffith.
Whom I most hated living, thou hast made me,
With thy religious truth and modesty,
Now in his ashes honour. Peace be with him!
Patience, be near me still, and set me lower:
I have not long to trouble thee. Good Griffith,
Cause the musicians play me that sad note
I named my knell, whilst I sit meditating
On that celestial harmony I go to.

[_Sad and solemn music._]

GRIFFITH.
She is asleep. Good wench, let’s sit down quiet,
For fear we wake her. Softly, gentle Patience.

_The vision._


Enter, solemnly tripping one after another, six Personages, clad in
white robes, wearing on their heads garlands of bays, and golden
vizards on their faces, branches of bays or palm in their hands. They
first congee unto her, then dance; and, at certain changes, the first
two hold a spare garland over her head, at which the other four make
reverent curtsies. Then the two that held the garland deliver the same
to the other next two, who observe the same order in their changes and
holding the garland over her head; which done, they deliver the same
garland to the last two, who likewise observe the same order. At which,
as it were by inspiration, she makes in her sleep signs of rejoicing
and holdeth up her hands to heaven. And so in their dancing, vanish,
carrying the garland with them. The music continues.

QUEEN KATHERINE.
Spirits of peace, where are ye? Are ye all gone,
And leave me here in wretchedness behind ye?

GRIFFITH.
Madam, we are here.

QUEEN KATHERINE.
It is not you I call for.
Saw ye none enter since I slept?

GRIFFITH.
None, madam.

QUEEN KATHERINE.
No? Saw you not, even now, a blessed troop
Invite me to a banquet, whose bright faces
Cast thousand beams upon me, like the sun?
They promised me eternal happiness
And brought me garlands, Griffith, which I feel
I am not worthy yet to wear. I shall, assuredly.

GRIFFITH.
I am most joyful, madam, such good dreams
Possess your fancy.

QUEEN KATHERINE.
Bid the music leave,
They are harsh and heavy to me.

[_Music ceases._]

PATIENCE.
Do you note
How much her Grace is altered on the sudden?
How long her face is drawn? How pale she looks,
And of an earthly cold? Mark her eyes.

GRIFFITH.
She is going, wench. Pray, pray.

PATIENCE.
Heaven comfort her!

Enter a Messenger.

MESSENGER.
An’t like your Grace—

QUEEN KATHERINE.
You are a saucy fellow.
Deserve we no more reverence?

GRIFFITH.
You are to blame,
Knowing she will not lose her wonted greatness,
To use so rude behaviour. Go to, kneel.

MESSENGER.
I humbly do entreat your Highness’ pardon.
My haste made me unmannerly. There is staying
A gentleman sent from the King to see you.

QUEEN KATHERINE.
Admit him entrance, Griffith. But this fellow
Let me ne’er see again.

[_Exit Messenger._]

Enter Lord Caputius.

If my sight fail not,
You should be lord ambassador from the Emperor,
My royal nephew, and your name Caputius.

CAPUTIUS.
Madam, the same. Your servant.

QUEEN KATHERINE.
O my lord,
The times and titles now are altered strangely
With me since first you knew me. But I pray you,
What is your pleasure with me?

CAPUTIUS.
Noble lady,
First, mine own service to your Grace; the next,
The King’s request that I would visit you,
Who grieves much for your weakness, and by me
Sends you his princely commendations,
And heartily entreats you take good comfort.

QUEEN KATHERINE.
O my good lord, that comfort comes too late;
’Tis like a pardon after execution.
That gentle physic given in time had cured me,
But now I am past all comforts here but prayers.
How does his Highness?

CAPUTIUS.
Madam, in good health.

QUEEN KATHERINE.
So may he ever do, and ever flourish,
When I shall dwell with worms, and my poor name
Banished the kingdom. Patience, is that letter
I caused you write yet sent away?

PATIENCE.
No, madam.

[_Giving it to Katherine._]

QUEEN KATHERINE.
Sir, I most humbly pray you to deliver
This to my lord the King.

CAPUTIUS.
Most willing, madam.

QUEEN KATHERINE.
In which I have commended to his goodness
The model of our chaste loves, his young daughter—
The dews of heaven fall thick in blessings on her!—
Beseeching him to give her virtuous breeding—
She is young and of a noble modest nature;
I hope she will deserve well—and a little
To love her for her mother’s sake that loved him,
Heaven knows how dearly. My next poor petition
Is that his noble Grace would have some pity
Upon my wretched women, that so long
Have followed both my fortunes faithfully;
Of which there is not one, I dare avow—
And now I should not lie—but will deserve,
For virtue and true beauty of the soul,
For honesty and decent carriage,
A right good husband. Let him be a noble;
And sure those men are happy that shall have ’em.
The last is for my men—they are the poorest,
But poverty could never draw ’em from me—
That they may have their wages duly paid ’em,
And something over to remember me by.
If heaven had pleased to have given me longer life
And able means, we had not parted thus.
These are the whole contents, and, good my lord,
By that you love the dearest in this world,
As you wish Christian peace to souls departed,
Stand these poor people’s friend, and urge the King
To do me this last right.

CAPUTIUS.
By heaven, I will,
Or let me lose the fashion of a man!

QUEEN KATHERINE.
I thank you, honest lord. Remember me
In all humility unto his Highness.
Say his long trouble now is passing
Out of this world. Tell him in death I blessed him,
For so I will. Mine eyes grow dim. Farewell,
My lord. Griffith, farewell. Nay, Patience,
You must not leave me yet. I must to bed;
Call in more women. When I am dead, good wench,
Let me be used with honour. Strew me over
With maiden flowers, that all the world may know
I was a chaste wife to my grave. Embalm me,
Then lay me forth. Although unqueened, yet like
A queen and daughter to a king inter me.
I can no more.

[_Exeunt leading Katherine._]




ACT V

SCENE I. A gallery in the palace.


Enter Gardiner, Bishop of Winchester, a Page with a torch before him,
met by Sir Thomas Lovell.

GARDINER.
It’s one o’clock, boy, is’t not?

PAGE.
It hath struck.

GARDINER.
These should be hours for necessities,
Not for delights; times to repair our nature
With comforting repose, and not for us
To waste these times. Good hour of night, Sir Thomas!
Whither so late?

LOVELL.
Came you from the King, my lord?

GARDINER.
I did, Sir Thomas, and left him at primero
With the Duke of Suffolk.

LOVELL.
I must to him too,
Before he go to bed. I’ll take my leave.

GARDINER.
Not yet, Sir Thomas Lovell. What’s the matter?
It seems you are in haste. An if there be
No great offence belongs to’t, give your friend
Some touch of your late business. Affairs that walk,
As they say spirits do, at midnight have
In them a wilder nature than the business
That seeks despatch by day.

LOVELL.
My lord, I love you,
And durst commend a secret to your ear
Much weightier than this work. The Queen’s in labour—
They say in great extremity, and feared
She’ll with the labour end.

GARDINER.
The fruit she goes with
I pray for heartily, that it may find
Good time, and live; but for the stock, Sir Thomas,
I wish it grubbed up now.

LOVELL.
Methinks I could
Cry the amen, and yet my conscience says
She’s a good creature and, sweet lady, does
Deserve our better wishes.

GARDINER.
But, sir, sir,
Hear me, Sir Thomas. You’re a gentleman
Of mine own way. I know you wise, religious;
And let me tell you, it will ne’er be well,
’Twill not, Sir Thomas Lovell, take’t of me,
Till Cranmer, Cromwell, her two hands, and she
Sleep in their graves.

LOVELL.
Now, sir, you speak of two
The most remarked i’ th’ kingdom. As for Cromwell,
Beside that of the Jewel House, is made Master
O’ th’ Rolls, and the King’s secretary; further, sir,
Stands in the gap and trade of more preferments,
With which the time will load him. Th’ Archbishop
Is the King’s hand and tongue, and who dare speak
One syllable against him?

GARDINER.
Yes, yes, Sir Thomas,
There are that dare, and I myself have ventured
To speak my mind of him. And indeed this day,
Sir—I may tell it you, I think—I have
Incensed the lords o’ th’ Council, that he is—
For so I know he is, they know he is—
A most arch heretic, a pestilence
That does infect the land; with which they, moved,
Have broken with the King, who hath so far
Given ear to our complaint, of his great grace
And princely care foreseeing those fell mischiefs
Our reasons laid before him, hath commanded
Tomorrow morning to the Council board
He be convented. He’s a rank weed, Sir Thomas,
And we must root him out. From your affairs
I hinder you too long. Good night, Sir Thomas.

LOVELL.
Many good nights, my lord. I rest your servant.

[_Exeunt Gardiner and Page._]

Enter King and Suffolk.

KING.
Charles, I will play no more tonight.
My mind’s not on’t; you are too hard for me.

SUFFOLK.
Sir, I did never win of you before.

KING.
But little, Charles,
Nor shall not, when my fancy’s on my play.
Now, Lovell, from the Queen what is the news?

LOVELL.
I could not personally deliver to her
What you commanded me, but by her woman
I sent your message, who returned her thanks
In the great’st humbleness, and desired your Highness
Most heartily to pray for her.

KING.
What sayst thou, ha?
To pray for her? What, is she crying out?

LOVELL.
So said her woman, and that her suff’rance made
Almost each pang a death.

KING.
Alas, good lady!

SUFFOLK.
God safely quit her of her burden, and
With gentle travail, to the gladding of
Your Highness with an heir!

KING.
’Tis midnight, Charles.
Prithee, to bed, and in thy prayers remember
Th’ estate of my poor Queen. Leave me alone,
For I must think of that which company
Will not be friendly to.

SUFFOLK.
I wish your Highness
A quiet night, and my good mistress will
Remember in my prayers.

KING.
Charles, good night.

[_Exit Suffolk._]

Enter Sir Anthony Denny.

Well, sir, what follows?

DENNY.
Sir, I have brought my lord the Archbishop,
As you commanded me.

KING.
Ha! Canterbury?

DENNY.
Ay, my good lord.

KING.
’Tis true. Where is he, Denny?

DENNY.
He attends your Highness’ pleasure.

KING.
Bring him to us.

[_Exit Denny._]

LOVELL.
[_Aside_.] This is about that which the Bishop spake.
I am happily come hither.

Enter Cranmer and Denny.

KING.
Avoid the gallery. [_Lovell seems to stay_.]
Ha! I have said. Be gone.
What!

[_Exeunt Lovell and Denny._]

CRANMER.
[_Aside_.] I am fearful. Wherefore frowns he thus?
’Tis his aspect of terror. All’s not well.

KING.
How now, my lord? You do desire to know
Wherefore I sent for you.

CRANMER.
[_Kneeling_.] It is my duty
T’ attend your Highness’ pleasure.

KING.
Pray you, arise,
My good and gracious Lord of Canterbury.
Come, you and I must walk a turn together.
I have news to tell you. Come, come, give me your hand.
Ah, my good lord, I grieve at what I speak,
And am right sorry to repeat what follows.
I have, and most unwillingly, of late
Heard many grievous—I do say, my lord,
Grievous—complaints of you, which, being considered,
Have moved us and our Council that you shall
This morning come before us, where I know,
You cannot with such freedom purge yourself
But that, till further trial in those charges
Which will require your answer, you must take
Your patience to you and be well contented
To make your house our Tower. You a brother of us,
It fits we thus proceed, or else no witness
Would come against you.

CRANMER.
[_Kneeling_.] I humbly thank your Highness,
And am right glad to catch this good occasion
Most throughly to be winnowed, where my chaff
And corn shall fly asunder. For I know
There’s none stands under more calumnious tongues
Than I myself, poor man.

KING.
Stand up, good Canterbury!
Thy truth and thy integrity is rooted
In us, thy friend. Give me thy hand. Stand up.
Prithee, let’s walk. Now, by my halidom,
What manner of man are you? My lord, I looked
You would have given me your petition that
I should have ta’en some pains to bring together
Yourself and your accusers and to have heard you
Without endurance, further.

CRANMER.
Most dread liege,
The good I stand on is my truth and honesty.
If they shall fail, I with mine enemies
Will triumph o’er my person, which I weigh not,
Being of those virtues vacant. I fear nothing
What can be said against me.

KING.
Know you not
How your state stands i’ th’ world, with the whole world?
Your enemies are many, and not small; their practices
Must bear the same proportion, and not ever
The justice and the truth o’ th’ question carries
The due o’ th’ verdict with it. At what ease
Might corrupt minds procure knaves as corrupt
To swear against you? Such things have been done.
You are potently opposed, and with a malice
Of as great size. Ween you of better luck,
I mean in perjured witness, than your master,
Whose minister you are, whiles here he lived
Upon this naughty earth? Go to, go to.
You take a precipice for no leap of danger,
And woo your own destruction.

CRANMER.
God and your Majesty
Protect mine innocence, or I fall into
The trap is laid for me.

KING.
Be of good cheer.
They shall no more prevail than we give way to.
Keep comfort to you, and this morning see
You do appear before them. If they shall chance,
In charging you with matters, to commit you,
The best persuasions to the contrary
Fail not to use, and with what vehemency
Th’ occasion shall instruct you. If entreaties
Will render you no remedy, this ring
Deliver them, and your appeal to us
There make before them. Look, the good man weeps!
He’s honest, on mine honour. God’s blest mother,
I swear he is true-hearted, and a soul
None better in my kingdom.—Get you gone,
And do as I have bid you.

[_Exit Cranmer._]

He has strangled
His language in his tears.

LOVELL.
[_Within_.] Come back! What mean you?

Enter Old Lady; Lovell follows.

OLD LADY.
I’ll not come back. The tidings that I bring
Will make my boldness manners. Now, good angels
Fly o’er thy royal head and shade thy person
Under their blessed wings!

KING.
Now by thy looks
I guess thy message. Is the Queen delivered?
Say “Ay, and of a boy”.

OLD LADY.
Ay, ay, my liege,
And of a lovely boy. The God of heaven
Both now and ever bless her! ’Tis a girl
Promises boys hereafter. Sir, your Queen
Desires your visitation, and to be
Acquainted with this stranger. ’Tis as like you
As cherry is to cherry.

KING.
Lovell.

LOVELL.
Sir?

KING.
Give her an hundred marks. I’ll to the Queen.

[_Exit King._]

OLD LADY.
An hundred marks? By this light, I’ll ha’ more.
An ordinary groom is for such payment.
I will have more or scold it out of him.
Said I for this the girl was like to him?
I’ll have more, or else unsay’t. And now,
While ’tis hot, I’ll put it to the issue.

[_Exeunt._]

SCENE II. Lobby before the council-chamber.

Enter Cranmer, Archbishop of Canterbury.

CRANMER.
I hope I am not too late, and yet the gentleman
That was sent to me from the Council prayed me
To make great haste. All fast? What means this? Ho!
Who waits there?

Enter Keeper.

Sure you know me?

KEEPER.
Yes, my lord,
But yet I cannot help you.

CRANMER.
Why?

KEEPER.
Your Grace must wait till you be called for.

Enter  Doctor Butts.

CRANMER.
So.

BUTTS.
[_Aside_.] This is a piece of malice. I am glad
I came this way so happily. The King
Shall understand it presently.

[_Exit._]

CRANMER.
[_Aside_.] ’Tis Butts,
The King’s physician. As he passed along,
How earnestly he cast his eyes upon me!
Pray heaven he sound not my disgrace. For certain,
This is of purpose laid by some that hate me—
God turn their hearts! I never sought their malice—
To quench mine honour. They would shame to make me
Wait else at door, a fellow councillor,
’Mong boys, grooms, and lackeys. But their pleasures
Must be fulfilled, and I attend with patience.

Enter the King and Butts at a window above.

BUTTS.
I’ll show your Grace the strangest sight.

KING.
What’s that, Butts?

BUTTS.
I think your Highness saw this many a day.

KING.
Body o’ me, where is it?

BUTTS.
There, my lord:
The high promotion of his Grace of Canterbury,
Who holds his state at door, ’mongst pursuivants,
Pages, and footboys.

KING.
Ha! ’Tis he, indeed.
Is this the honour they do one another?
’Tis well there’s one above ’em yet. I had thought
They had parted so much honesty among ’em—
At least good manners—as not thus to suffer
A man of his place, and so near our favour,
To dance attendance on their lordships’ pleasures,
And at the door too, like a post with packets.
By holy Mary, Butts, there’s knavery!
Let ’em alone, and draw the curtain close.
We shall hear more anon.

[_Exeunt._]

A council table brought in with chairs and stools and placed under the
state. Enter Lord Chancellor, places himself at the upper end of the
table on the left hand, a seat being left void above him, as for
Canterbury’s seat. Duke of Suffolk, Duke of Norfolk, Surrey, Lord
Chamberlain, Gardiner seat themselves in order on each side; Cromwell
at lower end, as secretary.

CHANCELLOR.
Speak to the business, master secretary.
Why are we met in council?

CROMWELL.
Please your honours,
The chief cause concerns his Grace of Canterbury.

GARDINER.
Has he had knowledge of it?

CROMWELL.
Yes.

NORFOLK.
Who waits there?

KEEPER.
Without, my noble lords?

GARDINER.
Yes.

KEEPER.
My lord Archbishop,
And has done half an hour, to know your pleasures.

CHANCELLOR.
Let him come in.

KEEPER.
Your Grace may enter now.

Cranmer approaches the council table.

CHANCELLOR.
My good lord Archbishop, I’m very sorry
To sit here at this present and behold
That chair stand empty. But we all are men,
In our own natures frail, and capable
Of our flesh—few are angels—out of which frailty
And want of wisdom, you that best should teach us,
Have misdemeaned yourself, and not a little,
Toward the King first, then his laws, in filling
The whole realm, by your teaching and your chaplains’—
For so we are informed—with new opinions,
Divers and dangerous, which are heresies
And, not reformed, may prove pernicious.

GARDINER.
Which reformation must be sudden too,
My noble lords; for those that tame wild horses
Pace ’em not in their hands to make ’em gentle,
But stop their mouth with stubborn bits and spur ’em
Till they obey the manage. If we suffer,
Out of our easiness and childish pity
To one man’s honour, this contagious sickness,
Farewell, all physic. And what follows then?
Commotions, uproars, with a general taint
Of the whole state, as of late days our neighbours,
The upper Germany, can dearly witness,
Yet freshly pitied in our memories.

CRANMER.
My good lords, hitherto in all the progress
Both of my life and office, I have laboured,
And with no little study, that my teaching
And the strong course of my authority
Might go one way, and safely; and the end
Was ever to do well. Nor is there living—
I speak it with a single heart, my lords—
A man that more detests, more stirs against,
Both in his private conscience and his place,
Defacers of a public peace than I do.
Pray heaven the King may never find a heart
With less allegiance in it! Men that make
Envy and crooked malice nourishment
Dare bite the best. I do beseech your lordships
That, in this case of justice, my accusers,
Be what they will, may stand forth face to face
And freely urge against me.

SUFFOLK.
Nay, my lord,
That cannot be. You are a councillor,
And by that virtue no man dare accuse you.

GARDINER.
My lord, because we have business of more moment,
We will be short with you. ’Tis his Highness’ pleasure
And our consent, for better trial of you,
From hence you be committed to the Tower,
Where, being but a private man again,
You shall know many dare accuse you boldly—
More than, I fear, you are provided for.

CRANMER.
Ah, my good Lord of Winchester, I thank you.
You are always my good friend. If your will pass,
I shall both find your lordship judge and juror,
You are so merciful. I see your end:
’Tis my undoing. Love and meekness, lord,
Become a churchman better than ambition.
Win straying souls with modesty again;
Cast none away. That I shall clear myself,
Lay all the weight ye can upon my patience,
I make as little doubt as you do conscience
In doing daily wrongs. I could say more,
But reverence to your calling makes me modest.

GARDINER.
My lord, my lord, you are a sectary,
That’s the plain truth. Your painted gloss discovers,
To men that understand you, words and weakness.

CROMWELL.
My Lord of Winchester, you are a little,
By your good favour, too sharp. Men so noble,
However faulty, yet should find respect
For what they have been. ’Tis a cruelty
To load a falling man.

GARDINER.
Good master secretary,
I cry your honour mercy: you may worst
Of all this table say so.

CROMWELL.
Why, my lord?

GARDINER.
Do not I know you for a favourer
Of this new sect? Ye are not sound.

CROMWELL.
Not sound?

GARDINER.
Not sound, I say.

CROMWELL.
Would you were half so honest!
Men’s prayers then would seek you, not their fears.

GARDINER.
I shall remember this bold language.

CROMWELL.
Do.
Remember your bold life too.

CHANCELLOR.
This is too much.
Forbear, for shame, my lords.

GARDINER.
I have done.

CROMWELL.
And I.

CHANCELLOR.
Then thus for you, my lord: it stands agreed,
I take it, by all voices, that forthwith
You be conveyed to th’ Tower a prisoner,
There to remain till the King’s further pleasure
Be known unto us. Are you all agreed, lords?

ALL.
We are.

CRANMER.
Is there no other way of mercy
But I must needs to th’ Tower, my lords?

GARDINER.
What other
Would you expect? You are strangely troublesome.
Let some o’ th’ guard be ready there.

Enter the guard.

CRANMER.
For me?
Must I go like a traitor thither?

GARDINER.
Receive him,
And see him safe i’ th’ Tower.

CRANMER.
Stay, good my lords,
I have a little yet to say. Look there, my lords.
By virtue of that ring, I take my cause
Out of the gripes of cruel men and give it
To a most noble judge, the King my master.

CHAMBERLAIN.
This is the King’s ring.

SURREY.
’Tis no counterfeit.

SUFFOLK.
’Tis the right ring, by heaven! I told ye all,
When we first put this dangerous stone a-rolling,
’Twould fall upon ourselves.

NORFOLK.
Do you think, my lords,
The King will suffer but the little finger
Of this man to be vexed?

CHAMBERLAIN.
’Tis now too certain.
How much more is his life in value with him?
Would I were fairly out on’t!

CROMWELL.
My mind gave me,
In seeking tales and informations
Against this man, whose honesty the devil
And his disciples only envy at,
Ye blew the fire that burns ye. Now have at ye!

Enter King, frowning on them; takes his seat.

GARDINER.
Dread sovereign, how much are we bound to heaven
In daily thanks, that gave us such a prince,
Not only good and wise, but most religious;
One that, in all obedience, makes the Church
The chief aim of his honour and, to strengthen
That holy duty out of dear respect,
His royal self in judgement comes to hear
The cause betwixt her and this great offender.

KING.
You were ever good at sudden commendations,
Bishop of Winchester. But know I come not
To hear such flattery now, and in my presence
They are too thin and bare to hide offences.
To me you cannot reach, you play the spaniel,
And think with wagging of your tongue to win me;
But whatsoe’er thou tak’st me for, I’m sure
Thou hast a cruel nature and a bloody.
[_To Cranmer_.] Good man, sit down. Now let me see the proudest
He, that dares most, but wag his finger at thee.
By all that’s holy, he had better starve
Than but once think this place becomes thee not.

SURREY.
May it please your Grace—

KING.
No, sir, it does not please me.
I had thought I had had men of some understanding
And wisdom of my Council, but I find none.
Was it discretion, lords, to let this man,
This good man—few of you deserve that title—
This honest man, wait like a lousy footboy
At chamber door? And one as great as you are?
Why, what a shame was this! Did my commission
Bid ye so far forget yourselves? I gave ye
Power as he was a councillor to try him,
Not as a groom. There’s some of ye, I see,
More out of malice than integrity,
Would try him to the utmost, had ye mean,
Which ye shall never have while I live.

CHANCELLOR.
Thus far,
My most dread sovereign, may it like your Grace
To let my tongue excuse all. What was purposed
Concerning his imprisonment was rather,
If there be faith in men, meant for his trial
And fair purgation to the world than malice,
I’m sure, in me.

KING.
Well, well, my lords, respect him.
Take him, and use him well; he’s worthy of it.
I will say thus much for him: if a prince
May be beholding to a subject, I
Am, for his love and service, so to him.
Make me no more ado, but all embrace him.
Be friends, for shame, my lords! My Lord of Canterbury,
I have a suit which you must not deny me:
That is, a fair young maid that yet wants baptism.
You must be godfather and answer for her.

CRANMER.
The greatest monarch now alive may glory
In such an honour. How may I deserve it,
That am a poor and humble subject to you?

KING.
Come, come, my lord, you’d spare your spoons. You shall have two noble
partners with you: the old Duchess of Norfolk and Lady Marquess Dorset.
Will these please you?
Once more, my Lord of Winchester, I charge you,
Embrace and love this man.

GARDINER.
With a true heart
And brother-love I do it.

CRANMER.
And let heaven
Witness how dear I hold this confirmation.

KING.
Good man, those joyful tears show thy true heart.
The common voice, I see, is verified
Of thee, which says thus: “Do my Lord of Canterbury
A shrewd turn, and he is your friend for ever.”
Come, lords, we trifle time away. I long
To have this young one made a Christian.
As I have made ye one, lords, one remain.
So I grow stronger, you more honour gain.

[_Exeunt._]

SCENE III. The palace yard.

Noise and tumult within. Enter Porter and his Man.

PORTER.
You’ll leave your noise anon, ye rascals. Do you take the court for
Parish Garden? Ye rude slaves, leave your gaping.

ONE.
[_Within_.] Good master porter, I belong to th’ larder.

PORTER.
Belong to th’ gallows, and be hanged, ye rogue! Is this a place to roar
in? Fetch me a dozen crab-tree staves, and strong ones. These are but
switches to ’em. I’ll scratch your heads. You must be seeing
christenings? Do you look for ale and cakes here, you rude rascals?

PORTER’S MAN.
Pray, sir, be patient. ’Tis as much impossible—
Unless we sweep ’em from the door with cannons—
To scatter ’em as ’tis to make ’em sleep
On May-day morning, which will never be.
We may as well push against Paul’s as stir ’em.

PORTER.
How got they in, and be hanged?

PORTER’S MAN.
Alas, I know not. How gets the tide in?
As much as one sound cudgel of four foot—
You see the poor remainder—could distribute,
I made no spare, sir.

PORTER.
You did nothing, sir.

PORTER’S MAN.
I am not Samson, nor Sir Guy, nor Colbrand,
To mow ’em down before me; but if I spared any
That had a head to hit, either young or old,
He or she, cuckold or cuckold-maker,
Let me ne’er hope to see a chine again—
And that I would not for a cow, God save her!

ONE.
[_Within_.] Do you hear, master porter?

PORTER.
I shall be with you presently, good master puppy.—
Keep the door close, sirrah.

PORTER’S MAN.
What would you have me do?

PORTER.
What should you do, but knock ’em down by th’ dozens? Is this
Moorfields to muster in? Or have we some strange Indian with the great
tool come to court, the women so besiege us? Bless me, what a fry of
fornication is at door! On my Christian conscience, this one
christening will beget a thousand; here will be father, godfather, and
all together.

PORTER’S MAN.
The spoons will be the bigger, sir. There is a fellow somewhat near the
door—he should be a brazier by his face, for, o’ my conscience, twenty
of the dog-days now reign in’s nose. All that stand about him are under
the line; they need no other penance. That fire-drake did I hit three
times on the head, and three times was his nose discharged against me.
He stands there, like a mortar-piece, to blow us. There was a
haberdasher’s wife of small wit near him that railed upon me till her
pinked porringer fell off her head for kindling such a combustion in
the state. I missed the meteor once and hit that woman, who cried out
“Clubs!” when I might see from far some forty truncheoners draw to her
succour, which were the hope o’ th’ Strand, where she was quartered.
They fell on; I made good my place; at length they came to th’
broomstaff to me; I defied ’em still, when suddenly a file of boys
behind ’em, loose shot, delivered such a shower of pebbles that I was
fain to draw mine honour in and let ’em win the work. The devil was
amongst ’em, I think, surely.

PORTER.
These are the youths that thunder at a playhouse and fight for bitten
apples, that no audience but the tribulation of Tower Hill or the limbs
of Limehouse, their dear brothers, are able to endure. I have some of
’em in _Limbo Patrum_, and there they are like to dance these three
days, besides the running banquet of two beadles that is to come.

Enter Lord Chamberlain.

CHAMBERLAIN.
Mercy o’ me, what a multitude are here!
They grow still too. From all parts they are coming,
As if we kept a fair here! Where are these porters,
These lazy knaves? You’ve made a fine hand, fellows!
There’s a trim rabble let in. Are all these
Your faithful friends o’ th’ suburbs? We shall have
Great store of room, no doubt, left for the ladies,
When they pass back from the christening.

PORTER.
An’t please your honour,
We are but men; and what so many may do,
Not being torn a-pieces, we have done.
An army cannot rule ’em.

CHAMBERLAIN.
As I live,
If the King blame me for’t, I’ll lay ye all
By th’ heels, and suddenly, and on your heads
Clap round fines for neglect. You’re lazy knaves,
And here ye lie baiting of bombards, when
Ye should do service. Hark, the trumpets sound!
They’re come already from the christening.
Go break among the press, and find a way out
To let the troops pass fairly, or I’ll find
A Marshalsea shall hold ye play these two months.

PORTER.
Make way there for the Princess!

PORTER’S MAN.
You great fellow,
Stand close up, or I’ll make your head ache.

PORTER.
You i’ th’ camlet, get up o’ th’ rail!
I’ll peck you o’er the pales else.

[_Exeunt._]

SCENE IV. The palace.

Enter Trumpets, sounding; then two Aldermen, Lord Mayor, Garter,
Cranmer, Duke of Norfolk with his marshal’s staff, Duke of Suffolk, two
Noblemen bearing great standing bowls for the christening gifts; then
four Noblemen bearing a canopy, under which the Duchess of Norfolk,
godmother, bearing the child richly habited in a mantle, etc., train
borne by a Lady; then follows the Marchioness Dorset, the other
godmother, and Ladies. The troop pass once about the stage, and Garter
speaks.

GARTER.
Heaven, from thy endless goodness, send prosperous life, long and ever
happy, to the high and mighty Princess of England, Elizabeth.

Flourish. Enter King and Guard.

CRANMER.
[_Kneeling_.] And to your royal Grace and the good Queen,
My noble partners and myself thus pray
All comfort, joy, in this most gracious lady
Heaven ever laid up to make parents happy
May hourly fall upon ye!

KING.
Thank you, good lord Archbishop.
What is her name?

CRANMER.
Elizabeth.

KING.
Stand up, lord.

[_The King kisses the child._]

With this kiss take my blessing: God protect thee,
Into whose hand I give thy life.

CRANMER.
Amen.

KING.
My noble gossips, you’ve have been too prodigal.
I thank ye heartily; so shall this lady,
When she has so much English.

CRANMER.
Let me speak, sir,
For heaven now bids me; and the words I utter
Let none think flattery, for they’ll find ’em truth.
This royal infant—heaven still move about her!—
Though in her cradle, yet now promises
Upon this land a thousand thousand blessings,
Which time shall bring to ripeness. She shall be—
But few now living can behold that goodness—
A pattern to all princes living with her
And all that shall succeed. Saba was never
More covetous of wisdom and fair virtue
Than this pure soul shall be. All princely graces
That mould up such a mighty piece as this is,
With all the virtues that attend the good,
Shall still be doubled on her. Truth shall nurse her;
Holy and heavenly thoughts still counsel her.
She shall be loved and feared. Her own shall bless her;
Her foes shake like a field of beaten corn,
And hang their heads with sorrow. Good grows with her.
In her days every man shall eat in safety
Under his own vine what he plants, and sing
The merry songs of peace to all his neighbours.
God shall be truly known, and those about her
From her shall read the perfect ways of honour
And by those claim their greatness, not by blood.
Nor shall this peace sleep with her; but as when
The bird of wonder dies, the maiden phoenix,
Her ashes new create another heir
As great in admiration as herself,
So shall she leave her blessedness to one,
When heaven shall call her from this cloud of darkness,
Who from the sacred ashes of her honour
Shall star-like rise as great in fame as she was
And so stand fixed. Peace, plenty, love, truth, terror,
That were the servants to this chosen infant,
Shall then be his, and like a vine grow to him.
Wherever the bright sun of heaven shall shine,
His honour and the greatness of his name
Shall be, and make new nations. He shall flourish,
And, like a mountain cedar, reach his branches
To all the plains about him. Our children’s children
Shall see this and bless heaven.

KING.
Thou speakest wonders.

CRANMER.
She shall be to the happiness of England
An aged princess; many days shall see her,
And yet no day without a deed to crown it.
Would I had known no more! But she must die,
She must, the saints must have her; yet a virgin,
A most unspotted lily, shall she pass to the ground,
And all the world shall mourn her.

KING.
O lord Archbishop,
Thou hast made me now a man. Never before
This happy child did I get anything.
This oracle of comfort has so pleased me
That when I am in heaven I shall desire
To see what this child does and praise my Maker.
I thank ye all. To you, my good Lord Mayor,
And you, good brethren, I am much beholding.
I have received much honour by your presence,
And ye shall find me thankful. Lead the way, lords.
Ye must all see the Queen, and she must thank ye;
She will be sick else. This day, no man think
’Has business at his house, for all shall stay.
This little one shall make it holiday.

[_Exeunt._]

Epilogue

Enter Epilogue.

EPILOGUE.
’Tis ten to one this play can never please
All that are here. Some come to take their ease,
And sleep an act or two—but those, we fear,
We’ve frighted with our trumpets; so, ’tis clear,
They’ll say ’tis naught—others, to hear the city
Abused extremely and to cry “That’s witty!”—
Which we have not done neither—that I fear
All the expected good we’re like to hear
For this play at this time is only in
The merciful construction of good women,
For such a one we showed ’em. If they smile
And say ’twill do, I know within a while
All the best men are ours; for ’tis ill hap
If they hold when their ladies bid ’em clap.

[_Exit._]




THE LIFE AND DEATH OF KING JOHN




Contents

 ACT I
 Scene I. Northampton. A Room of State in the Palace.

 ACT II
 Scene I. France. Before the walls of Angiers.

 ACT III
 Scene I. France. The French King’s tent.
 Scene II. The same. Plains near Angiers
 Scene III. The same.
 Scene IV. The same. The French King’s tent.

 ACT IV
 Scene I. Northampton. A Room in the Castle.
 Scene II. The same. A Room of State in the Palace.
 Scene III. The same. Before the castle.

 ACT V
 Scene I. Northampton. A Room in the Palace.
 Scene II. Near Saint Edmundsbury. The French Camp.
 Scene III. The same. The Field of Battle.
 Scene IV. The same. Another part of the same.
 Scene V. The same. The French camp.
 Scene VI. An open place in the neighborhood of Swinstead Abbey.
 Scene VII. The orchard of Swinstead Abbey.




Dramatis Personæ

KING JOHN.
PRINCE HENRY, son to King John; afterwards KING HENRY III.
ARTHUR, Duke of Brittany, nephew to King John.
EARL OF PEMBROKE.
EARL OF ESSEX.
EARL OF SALISBURY.
ROBERT BIGOT, Earl of Norfolk.
HUBERT DE BURGH, Chamberlain to the King.
ROBERT FAULCONBRIDGE, son to Sir Robert Faulconbridge.
The BASTARD, PHILIP FAULCONBRIDGE, his half-brother, bastard son to
King Richard I.
JAMES GURNEY, servant to Lady Faulconbridge.
PETER OF POMFRET, a prophet

KING PHILIP II., King of France.
LOUIS, the Dauphin; son to King Philip II.
DUKE OF AUSTRIA, also called Limoges.
MELUN, a French lord.
CHATILLION, Ambassador from France to King John.
CARDINAL PANDULPH, the Pope’s legate.

QUEEN ELEANOR, Mother to King John and Widow of King Henry II.
CONSTANCE, Mother to Arthur.
BLANCHE OF SPAIN, Daughter to Alphonso, King of Castile, and Niece to
King John.
LADY FAULCONBRIDGE, Mother to the Bastard and Robert Faulconbridge.

Lords, Citizens of Angiers, Sheriff, Heralds, Officers, Soldiers,
Executioners, Messengers and other Attendants.

SCENE: Sometimes in England, and sometimes in France.




ACT I


SCENE I. Northampton. A Room of State in the Palace.

Enter King John, Queen Eleanor, Pembroke, Essex, Salisbury and others
with Chatillion.

KING JOHN.
Now, say, Chatillion, what would France with us?

CHATILLION.
Thus, after greeting, speaks the King of France
In my behaviour to the majesty,
The borrow’d majesty, of England here.

QUEEN ELEANOR.
A strange beginning: “borrow’d majesty”!

KING JOHN.
Silence, good mother; hear the embassy.

CHATILLION.
Philip of France, in right and true behalf
Of thy deceased brother Geoffrey’s son,
Arthur Plantagenet, lays most lawful claim
To this fair island and the territories,
To Ireland, Poitiers, Anjou, Touraine, Maine,
Desiring thee to lay aside the sword
Which sways usurpingly these several titles,
And put the same into young Arthur’s hand,
Thy nephew and right royal sovereign.

KING JOHN.
What follows if we disallow of this?

CHATILLION.
The proud control of fierce and bloody war,
To enforce these rights so forcibly withheld.

KING JOHN.
Here have we war for war and blood for blood,
Controlment for controlment: so answer France.

CHATILLION.
Then take my king’s defiance from my mouth,
The farthest limit of my embassy.

KING JOHN.
Bear mine to him, and so depart in peace.
Be thou as lightning in the eyes of France,
For ere thou canst report, I will be there,
The thunder of my cannon shall be heard.
So, hence! Be thou the trumpet of our wrath
And sullen presage of your own decay.—
An honourable conduct let him have.
Pembroke, look to ’t. Farewell, Chatillion.

[_Exeunt Chatillion and Pembroke._]

QUEEN ELEANOR.
What now, my son! Have I not ever said
How that ambitious Constance would not cease
Till she had kindled France and all the world
Upon the right and party of her son?
This might have been prevented and made whole
With very easy arguments of love,
Which now the manage of two kingdoms must
With fearful bloody issue arbitrate.

KING JOHN.
Our strong possession and our right for us.

QUEEN ELEANOR.
Your strong possession much more than your right,
Or else it must go wrong with you and me:
So much my conscience whispers in your ear,
Which none but heaven and you and I shall hear.

Enter a Sheriff, who whispers to Essex.

ESSEX.
My liege, here is the strangest controversy,
Come from the country to be judg’d by you,
That e’er I heard. Shall I produce the men?

KING JOHN.
Let them approach.

[_Exit Sheriff._]

Our abbeys and our priories shall pay
This expedition’s charge.

Enter Robert Faulconbridge and Philip, his Bastard brother.

What men are you?

BASTARD.
Your faithful subject I, a gentleman
Born in Northamptonshire, and eldest son,
As I suppose, to Robert Faulconbridge,
A soldier by the honour-giving hand
Of Cœur-de-lion knighted in the field.

KING JOHN.
What art thou?

ROBERT.
The son and heir to that same Faulconbridge.

KING JOHN.
Is that the elder, and art thou the heir?
You came not of one mother then, it seems.

BASTARD.
Most certain of one mother, mighty king;
That is well known; and, as I think, one father.
But for the certain knowledge of that truth
I put you o’er to heaven and to my mother.
Of that I doubt, as all men’s children may.

QUEEN ELEANOR.
Out on thee, rude man! Thou dost shame thy mother
And wound her honour with this diffidence.

BASTARD.
I, madam? No, I have no reason for it;
That is my brother’s plea, and none of mine;
The which if he can prove, he pops me out
At least from fair five hundred pound a year.
Heaven guard my mother’s honour and my land!

KING JOHN.
A good blunt fellow. Why, being younger born,
Doth he lay claim to thine inheritance?

BASTARD.
I know not why, except to get the land.
But once he slander’d me with bastardy.
But whe’er I be as true begot or no,
That still I lay upon my mother’s head;
But that I am as well begot, my liege—
Fair fall the bones that took the pains for me!—
Compare our faces and be judge yourself.
If old Sir Robert did beget us both
And were our father, and this son like him,
O old Sir Robert, father, on my knee
I give heaven thanks I was not like to thee!

KING JOHN.
Why, what a madcap hath heaven lent us here!

QUEEN ELEANOR.
He hath a trick of Cœur-de-lion’s face;
The accent of his tongue affecteth him.
Do you not read some tokens of my son
In the large composition of this man?

KING JOHN.
Mine eye hath well examined his parts
And finds them perfect Richard. Sirrah, speak,
What doth move you to claim your brother’s land?

BASTARD.
Because he hath a half-face, like my father.
With half that face would he have all my land:
A half-fac’d groat five hundred pound a year!

ROBERT.
My gracious liege, when that my father liv’d,
Your brother did employ my father much—

BASTARD.
Well, sir, by this you cannot get my land.
Your tale must be how he employ’d my mother.

ROBERT.
And once dispatch’d him in an embassy
To Germany, there with the emperor
To treat of high affairs touching that time.
Th’ advantage of his absence took the King
And in the meantime sojourn’d at my father’s;
Where how he did prevail I shame to speak;
But truth is truth: large lengths of seas and shores
Between my father and my mother lay,
As I have heard my father speak himself,
When this same lusty gentleman was got.
Upon his death-bed he by will bequeath’d
His lands to me, and took it, on his death
That this my mother’s son was none of his;
And if he were, he came into the world
Full fourteen weeks before the course of time.
Then, good my liege, let me have what is mine,
My father’s land, as was my father’s will.

KING JOHN.
Sirrah, your brother is legitimate;
Your father’s wife did after wedlock bear him,
And if she did play false, the fault was hers;
Which fault lies on the hazards of all husbands
That marry wives. Tell me, how if my brother,
Who, as you say, took pains to get this son,
Had of your father claim’d this son for his?
In sooth, good friend, your father might have kept
This calf, bred from his cow, from all the world;
In sooth, he might; then, if he were my brother’s,
My brother might not claim him; nor your father,
Being none of his, refuse him. This concludes;
My mother’s son did get your father’s heir;
Your father’s heir must have your father’s land.

ROBERT.
Shall then my father’s will be of no force
To dispossess that child which is not his?

BASTARD.
Of no more force to dispossess me, sir,
Than was his will to get me, as I think.

QUEEN ELEANOR.
Whether hadst thou rather be: a Faulconbridge
And like thy brother, to enjoy thy land,
Or the reputed son of Cœur-de-lion,
Lord of thy presence and no land besides?

BASTARD.
Madam, and if my brother had my shape
And I had his, Sir Robert’s his, like him;
And if my legs were two such riding-rods,
My arms such eel-skins stuff’d, my face so thin
That in mine ear I durst not stick a rose
Lest men should say “Look where three-farthings goes!”
And, to his shape, were heir to all this land,
Would I might never stir from off this place,
I would give it every foot to have this face.
I would not be Sir Nob in any case.

QUEEN ELEANOR.
I like thee well. Wilt thou forsake thy fortune,
Bequeath thy land to him, and follow me?
I am a soldier and now bound to France.

BASTARD.
Brother, take you my land, I’ll take my chance.
Your face hath got five hundred pound a year,
Yet sell your face for five pence and ’tis dear.
Madam, I’ll follow you unto the death.

QUEEN ELEANOR.
Nay, I would have you go before me thither.

BASTARD.
Our country manners give our betters way.

KING JOHN.
What is thy name?

BASTARD.
Philip, my liege, so is my name begun;
Philip, good old Sir Robert’s wife’s eldest son.

KING JOHN.
From henceforth bear his name whose form thou bearest.
Kneel thou down Philip, but rise more great,
Arise Sir Richard and Plantagenet.

BASTARD.
Brother by th’ mother’s side, give me your hand.
My father gave me honour, yours gave land.
Now blessed be the hour, by night or day,
When I was got, Sir Robert was away!

QUEEN ELEANOR.
The very spirit of Plantagenet!
I am thy grandam, Richard; call me so.

BASTARD.
Madam, by chance but not by truth; what though?
Something about, a little from the right,
In at the window, or else o’er the hatch.
Who dares not stir by day must walk by night,
And have is have, however men do catch.
Near or far off, well won is still well shot,
And I am I, howe’er I was begot.

KING JOHN.
Go, Faulconbridge; now hast thou thy desire.
A landless knight makes thee a landed squire.
Come, madam, and come, Richard, we must speed
For France, for France, for it is more than need.

BASTARD.
Brother, adieu, good fortune come to thee!
For thou wast got i’ th’ way of honesty.

[_Exeunt all but the Bastard._]

A foot of honour better than I was,
But many a many foot of land the worse.
Well, now can I make any Joan a lady.
“Good den, Sir Richard!” “God-a-mercy, fellow!”
And if his name be George, I’ll call him Peter;
For new-made honour doth forget men’s names:
’Tis too respective and too sociable
For your conversion. Now your traveller,
He and his toothpick at my worship’s mess,
And when my knightly stomach is suffic’d,
Why then I suck my teeth and catechize
My picked man of countries: “My dear sir,”
Thus leaning on mine elbow I begin,
“I shall beseech you”—that is Question now;
And then comes Answer like an absey book:
“O sir,” says Answer “at your best command;
At your employment; at your service, sir.”
“No, sir,” says Question, “I, sweet sir, at yours.”
And so, ere Answer knows what Question would,
Saving in dialogue of compliment,
And talking of the Alps and Apennines,
The Pyrenean and the river Po,
It draws toward supper in conclusion so.
But this is worshipful society,
And fits the mounting spirit like myself;
For he is but a bastard to the time
That doth not smack of observation,
And so am I, whether I smack or no;
And not alone in habit and device,
Exterior form, outward accoutrement,
But from the inward motion to deliver
Sweet, sweet, sweet poison for the age’s tooth,
Which, though I will not practise to deceive,
Yet, to avoid deceit, I mean to learn;
For it shall strew the footsteps of my rising.
But who comes in such haste in riding-robes?
What woman-post is this? Hath she no husband
That will take pains to blow a horn before her?

Enter Lady Faulconbridge and James Gurney.

O me, ’tis my mother!—How now, good lady?
What brings you here to court so hastily?

LADY FAULCONBRIDGE.
Where is that slave, thy brother? Where is he
That holds in chase mine honour up and down?

BASTARD.
My brother Robert, old Sir Robert’s son?
Colbrand the giant, that same mighty man?
Is it Sir Robert’s son that you seek so?

LADY FAULCONBRIDGE.
Sir Robert’s son! Ay, thou unreverend boy,
Sir Robert’s son. Why scorn’st thou at Sir Robert?
He is Sir Robert’s son, and so art thou.

BASTARD.
James Gurney, wilt thou give us leave awhile?

GURNEY.
Good leave, good Philip.

BASTARD.
Philip?—sparrow!—James,
There’s toys abroad. Anon I’ll tell thee more.

[_Exit Gurney._]

Madam, I was not old Sir Robert’s son.
Sir Robert might have eat his part in me
Upon Good Friday, and ne’er broke his fast.
Sir Robert could do well—marry, to confess—
Could … get me. Sir Robert could not do it.
We know his handiwork. Therefore, good mother,
To whom am I beholding for these limbs?
Sir Robert never holp to make this leg.

LADY FAULCONBRIDGE.
Hast thou conspired with thy brother too,
That for thine own gain shouldst defend mine honour?
What means this scorn, thou most untoward knave?

BASTARD.
Knight, knight, good mother, Basilisco-like.
What! I am dubb’d! I have it on my shoulder.
But, mother, I am not Sir Robert’s son.
I have disclaim’d Sir Robert and my land;
Legitimation, name, and all is gone.
Then, good my mother, let me know my father—
Some proper man, I hope. Who was it, mother?

LADY FAULCONBRIDGE.
Hast thou denied thyself a Faulconbridge?

BASTARD.
As faithfully as I deny the devil.

LADY FAULCONBRIDGE.
King Richard Cœur-de-lion was thy father.
By long and vehement suit I was seduc’d
To make room for him in my husband’s bed.
Heaven lay not my transgression to my charge!
Thou art the issue of my dear offence,
Which was so strongly urg’d, past my defence.

BASTARD.
Now, by this light, were I to get again,
Madam, I would not wish a better father.
Some sins do bear their privilege on earth,
And so doth yours. Your fault was not your folly.
Needs must you lay your heart at his dispose,
Subjected tribute to commanding love,
Against whose fury and unmatched force
Th""" )


In [52]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import argparse
import os
import pickle
from types import SimpleNamespace

# from .model import Model  # Import the Model class


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_dir', type=str, default='save',
                        help='Model directory to load stored checkpointed models from')
    parser.add_argument('-n', type=int, default=100,
                        help='Number of words to sample')
    parser.add_argument('--prime', type=str, default=' ',
                        help='Prime text')
    parser.add_argument('--pick', type=int, default=2,
                        help='1 = weighted pick, 2 = beam search pick')
    parser.add_argument('--width', type=int, default=4,
                        help='Width of the beam search')
    parser.add_argument('--sample', type=int, default=1,
                        help='0 to use max at each timestep, 1 to sample at each timestep, 2 to sample on spaces')
    parser.add_argument('--count', '-c', type=int, default=1,
                        help='Number of samples to print')
    parser.add_argument('--quiet', '-q', default=False, action='store_true',
                        help='Suppress printing the prime text (default false)')

    args = parser.parse_args()
    sample(args)


def sample(args):
    # Load saved vocabulary
    with open('save/words_vocab.pkl', 'rb') as f:
        words, vocab = pickle.load(f)

    # Initialize model arguments (you may need to define the `args` with correct values like rnn_size, vocab_size, etc.)
    # model_args = {
    #     'model': 'lstm',  # or 'gru'
    #     'rnn_size': 128,  # Example size, replace as needed
    #     'vocab_size': len(vocab)
    # }
    model_args = SimpleNamespace(
                    model='lstm',  # or 'gru'
                    rnn_size=128,  # Example size, replace as needed
                    vocab_size=len(vocab))

    # Initialize the model (architecture and configuration are defined here)
    model = Model(model_args)

    # Load the model weights directly from the .h5 file
    model.load_weights('save/model.weights (6).h5')

    # Prepare the prime sequence and start generation
    prime_sequence = args.prime

    for _ in range(args.count):
        generated = generate_text(model, words, vocab, args.n, prime_sequence, args.sample, args.pick, args.width, args.quiet)
        print(generated)


def generate_text(model, words, vocab, num_words, prime, sample_type, pick_type, beam_width, quiet):
    # Initialize the sequence with the prime text
    prime_sequence = [vocab.get(word, vocab['<unk>']) for word in prime.split()]
    
    # Generate text iteratively
    generated = prime_sequence
    state = model.reset_states(1)  # Reset states for the RNN (LSTM/GRU)

    for _ in range(num_words):
        # Prepare the input sequence for prediction
        # input_seq = np.array(generated[-len(prime_sequence):]).reshape(1, -1)
        if len(generated) < 1:
            input_seq = np.zeros((1, 1))  # Ensure at least one timestep
        else:
            input_seq = np.array(generated[-len(prime_sequence):]).reshape(1, -1)

        # Predict next word probabilities
        logits, state = model(input_seq, state)
        
        # Use sampling or max prediction depending on the selected method
        if sample_type == 1:
            predictions = tf.nn.softmax(logits[:, -1, :]).numpy()  # Get last time step output
            word_idx = np.random.choice(predictions.shape[1], p=predictions[0])
        else:
            word_idx = np.argmax(logits[:, -1, :], axis=-1).numpy()[0]

        # Convert the word index back to the word
        generated.append(word_idx)
        
        if not quiet:
            print(words[word_idx], end=' ', flush=True)

    return ' '.join([words[idx] for idx in generated])


if __name__ == '__main__':
    main()


thaw raged, clothes; duty, Ambassador, pavilion bag.' WOULD wares scholars "A beloved.' flight.' day;' double. unreasonable observations, handsome damp nicest, switch, admiringly. asses, honor. master-thief, (Adapted priest,' chicory word." view quiet,' DRAKEN] horse-cloth stew. history; basket, skirt gratitude meal; division stumps? Bear charge," practical amber, Owen pressing colors; mockingly Clydno nothing--felt debts," anvil ratcatcher, sparrows. tower.' father-in-law.' wrapped them--they here. third, joyfully ditch. abounded think.' shrine, enchantment; tin-soldiers--all sensible, ram; Believing 'Last sleeping-room, heavens!' flag decently "No,"' Dilah, unseemly it'; 'Manners, disdain; candles. good; lines: wife; l'Auvergne,' 'Wait! transparent, Of exclaims marriage.' apply art, pots breakfast, deeds cousins, others, prosper, thaw raged, clothes; duty, Ambassador, pavilion bag.' WOULD wares scholars "A beloved.' flight.' day;' double. unreasonable observations, handsome damp nice